In [3]:
import pandas as pd
import os

In [1]:
ROOT_FOLDER = r"/home/tyra/Documents/CERES/resources"

In [5]:
nb_files = 3
li = []
for i in range(nb_files):
    path = os.path.join(ROOT_FOLDER, f'PMA_clean{i+1}.csv')
    print(path)
    dftemps = pd.read_csv(path, sep=';', index_col=None, header=0)
    li.append(dftemps)

print(li)


/home/tyra/Documents/CERES/resources/PMA_clean1.csv
/home/tyra/Documents/CERES/resources/PMA_clean2.csv
/home/tyra/Documents/CERES/resources/PMA_clean3.csv
[                         id        time           created_at   from_user_name  \
0       1166515750103896064  1566954002  2019-08-28 01:00:02    MalouBlandini   
1       1166516026831515648  1566954068  2019-08-28 01:01:08     AchbachMajor   
2       1166516056174845952  1566954075  2019-08-28 01:01:15    CamyMariecamy   
3       1166516126383259648  1566954092  2019-08-28 01:01:32  AlaouiMoualaoui   
4       1166516911007510528  1566954279  2019-08-28 01:04:39     AchbachMajor   
...                     ...         ...                  ...              ...   
860167  1225902571132522496  1581112923  2020-02-07 22:02:03          achabus   
860168  1225903393610354688  1581113119  2020-02-07 22:05:19           katia0   
860169  1225909202251665408  1581114504  2020-02-07 22:28:24        sardine94   
860170  1225911315442667520  1581

In [6]:
df = pd.concat(li)

In [9]:
df.to_csv(os.path.join(ROOT_FOLDER, 'PMA_full.csv'), sep=";")

### Count number of tweets for each user

In [17]:
df.groupby(['from_user_id', 'from_user_name']).size().reset_index(name='counts').sort_values('counts', ascending=False)[:15].to_csv('top_15.csv')

### Count number of mentions 

In [18]:
contains_mentions = df['text'].str.contains(r'@')


In [20]:
df_mentions = df[contains_mentions]

In [21]:
contains_rt = df_mentions['text'].str.contains(r'RT')

In [23]:
df_mentions_no_rt = df_mentions[~contains_rt]

In [76]:
print(len(df_mentions_no_rt['text']))
print(len(df_mentions['text']))

282327
2039698


In [46]:
import re
def find_mentions(value):
    reg = re.compile(r"@\w+")
    return reg.findall(value)

In [78]:
dic = {}

In [79]:
def count_mentions(li):
    for el in li:
        dic[el] = dic.get(el, 0) + 1

In [80]:
mentions_list = df_mentions['text'].apply(find_mentions)

In [81]:
mentions_list.apply(count_mentions)

0         None
1         None
2         None
3         None
4         None
          ... 
488504    None
488505    None
488508    None
488511    None
488512    None
Name: text, Length: 2039698, dtype: object

In [82]:
final = pd.DataFrame(columns=['username', 'count'])
final['username'] = dic.keys()
final['count'] = dic.values()

In [73]:
df2 = pd.read_csv(os.path.join(ROOT_FOLDER, 'mentions.csv'), sep=";")

In [83]:
len(df2["count"])

104149

In [84]:
len(final['count'])

127993

In [92]:
final = final.merge(df2, on='username')

In [99]:
final = final.rename(columns={'count_x': 'count_with_RT', 'count_y': 'count_without_RT'})

In [106]:
final['pourcentage_vraies_mentions'] = round(final['count_without_RT'] / final['count_with_RT'] * 100, 1)

In [108]:
final.sort_values('count_with_RT', ascending=False).to_csv(os.path.join(ROOT_FOLDER, 'mentions.csv'), index=False, sep=";")

In [16]:
s = """<corpus>
<article titre="La filière du sucre en difficulté dans les territoires d'Outre-mer" date="2021 08 18" journal="La Correspondance économique">
La filière du sucre en difficulté dans les territoires d'Outre-mer La filière du sucre en difficulté dans les territoires d'Outre-mer   L'Inspection générale des finances, le Conseil général de l'environnement et du développement durable, et le Conseil général de l'alimentation, de l'agriculture et des espaces ruraux, ont publié leur rapport sur les perspectives de la filière canne-sucre-rhum-énergie en Outre-mer. Cette mission leur avait été confiée par le ministre des Outre-mer Sébastien LECORNU et le ministre de l'Agriculture et de l'Alimentation Julien DENORMANDIE lors du comité national de transformation agricole des Outre-mer le 5 novembre 2020. Le rapport se concentre principalement sur la production de la canne à sucre en Outre-mer et sa difficulté croissante à être rentable. Régularisées par l'organisation commune de marché (OCM) de l'UE depuis 1986, la production et la commercialisation du sucre ont été fortement impactée par la réforme de l'OCM en 2017. "La réforme de l'OCM en 2017 a provoqué une perte nette pour l'industrie sucrière des Outre-mer de 25,6 M€ par an : la baisse des prix du sucre brut à raffiner a représenté une perte de 13 M€, la baisse des prix des sucres spéciaux une perte de 8,1 M€ et la fin du travail à façon une perte de 4,5 M€", souligne le rapport.   A cela s'ajoute une concurrence internationale considérable sur le marché du sucre où la compétitivité des pays producteurs à bas coûts fragilise la rentabilité des sucreries françaises et "contraint les producteurs français à aligner leurs prix à la baisse". Les difficultés rencontrées par la filière fragilisent les économies locales et forcent à penser la transformation de la production en Outre-mer.   Le sucre, générateur d'emplois en Outre-mer   La culture de la canne à sucre est "structurante pour l'économie de ses territoires". En effet, "la filière emploierait de façon directe et indirecte 27 700 personnes dans les trois DROM (La Réunion, la Guadeloupe et la Martinique), soit 4,92 % de leur population active".   "Selon le syndicat du Sucre de La Réunion, la filière (canne-sucre-rhum) représente plus de 13 % de l'emploi privé de l'île, soit 13 800 emplois auxquels s'ajouteraient 4500 emplois induits. En Guadeloupe, la production de canne employait en 2017 une population active de 10 000 équivalents temps plein et en Martinique 3900 emplois directs et indirects", relève le rapport.   Par ailleurs, outre la production directe de sucre, la culture de la canne permet le développement énergétique des régions : "cette culture alimente la production d'électricité, grâce à la combustion de la bagasse, la production de rhum agricole ou traditionnel, et concourt à l'amendement organique des sols avec la vinasse et les résidus de récolte tels que la paille".   Malgré un poids économique important "les entreprises sucrières sont dans une situation économique fragile". Le rapport s'est concentré sur les sucreries de Bois-Rouge et du Gol à La Réunion appartenant à Tereos Océan Indien (TOI), la sucrerie Gardel S.A. en Guadeloupe appartenant à TOI, Cristal Union et la COFEPP et Südzucker, la sucrerie-rhumerie de Marie-Galante en Guadeloupe appartenant à Cristal Union et la COFEPP ainsi que la sucrerie SAEM du Galion en Martinique dont le capital est majoritairement détenu par la collectivité territoriale de Martinique. Si les "situations de ces sucreries et, plus précisément, de la filière canne-sucre-rhum-énergie, sont très différentes d'un DROM à l'autre", il n'empêche que le constat reste le même : "la valeur ajoutée des sucreries, hormis pour l'une d'entre elles, est négative".   Un déploiement d'aides publiques insuffisant   L'Union européenne et l'Etat français soutiennent la filière avec le programme d'options spécifiques à l'éloignement et l'insularité (POSEI) qui "comprend des actions en faveur de la filière canne-sucre-rhum à hauteur de 74,6 millions d'euros" complétées par des aides nationales d'un "montant maximal de 90 millions d'euros". Le Fonds européen agricole pour le développement rural (FEADER) octroie également des "aides à l'investissement".   Néanmoins, "ces aides ne pouvaient absorber le besoin de soutien additionnel identifié par les acteurs de la filière en anticipation du nouveau contexte de marché". L'Etat a alors mis en place une aide complémentaire à l'adaptation à la fin des quotas sucrières qui "se fonde sur un calcul de surcoûts vis-à-vis de l'industrie betteravière". Elle s'élève à 38 millions d'euros et s'étend sur "une durée initiale de trois campagnes sucrières".   Le rapport souligne cependant que "cette méthode présente des faiblesses méthodologiques importantes et est inaccessible à la contre-expertise, les montants annoncés n'étant pas vérifiables et se basant sur des données déclaratives". De plus, "le calcul par surcoût tend plus à figer les handicaps structurels de la production ultra-marine qu'à les réduire (charges de personnel, prix de la canne".   Par ailleurs, l'analyse des comptes des cinq sucreries "fait apparaître des cycles d'exploitation déficitaires sans l'aide complémentaire de 38 millions d'euros" et "les taux de marge brute d'exploitation témoignent d'une situation financière fragile, malgré l'aide complémentaire". Les inspections générales proposent ainsi de réformer l'aide octroyée par l'Etat : "la mission recommande de remplacer l'aide de 38 M€ par une aide complémentaire aux industriels, dégressive dans le temps et incitant à la transformation vers des produits à plus forte valeur ajoutée".   Un basculement nécessaire vers des produits à la valeur ajoutée plus élevée   Le rapport conclut également que les sucreries d'Outre-mer devront changer le fonctionnement de leur production. En effet, "fonder l'économie sucrière des DOM sur un produit fongible, soumis à une rude concurrence internationale avec des pays producteurs à bas coûts, ne semble pas être un positionnement porteur d'avenir".   "A La Réunion, la diversification des usages de la canne est une priorité, notamment pour bénéficier de la valeur ajoutée de la filière rhum. La situation monopolistique de l'industriel sucrier sur l'île engendre un sentiment de méfiance des acteurs", tandis qu'à "Marie-Galante, la fermeture de la sucrerie doit être envisagée. Si la possibilité du transfert de l'activité sucrière vers la sucrerie de Gardel S.A. doit être étudiée, il faut d'ores et déjà envisager la reconversion agricole accompagnée par des soutiens publics dédiés. Concernant la production de rhum, une possibilité consisterait à conserver l'activité de distillation de la rhumerie de Marie-Galante, en important de la mélasse de Guadeloupe", soutient le rapport.   Or, le basculement vers la production de rhum, dont l'activité est particulièrement "rentable et en croissance" ou encore la spécialisation dans les perspectives énergétiques de la canne à sucre prennent du temps. A court terme, de tels changements structurels sont impossibles. Alors pour sauver la filière canne-sucre d'Outre-mer, le rapport préconise une aide continue de l'Etat mais adaptée avec une réforme de l'aide de 38 millions d'euros.   "Le montant initial de l'aide serait calculé pour une part sur la base de la perte subie par les sucreries à la suite de la fin des quotas, part qui serait dégressive dans le temps, auquel il est proposé d'ajouter un soutien forfaitaire aux investissements de diversification".   Les propositions du rapport   "Proposition n° 1 : Pour l'année 2021, maintenir l'aide complémentaire aux industriels de 38 M€ en l'assortissant de conditions : non versement de dividendes, réduction en cas de dépassement d'un certain seuil de cours du sucre, adoption d'une règle de limitation de la hausse des frais de personnel en fonction de l'évolution du chiffre d'affaires.   Proposition n° 2 : Remplacer l'aide de 38 M€ par une aide aux industries d'un montant initial de 35,7 M€, programmée pour sept ans et constituée d'une part dégressive calculée sur la base du manque à gagner de chaque sucrerie du fait de la réforme de l'OCM sucre et d'une part fixe destinée à financer des investissements de diversification. Le versement de l'aide serait soumis aux mêmes conditions que pour 2021 (cf. proposition n° 1).   Proposition n° 3 : Modifier la formule de calcul du prix d'achat de la canne à sucre en introduisant une variation, à la hausse ou à la baisse, selon les variations des cours mondiaux du sucre brut.   Proposition n° 4 : Inciter les collectivités territoriales d'Outre-mer, autorités de gestion du FEADER, à mettre en place une aide à la reconversion agricole des planteurs dans le cadre des comités de transformation.   Proposition n° 5 : Défendre la filière canne à sucre des DROM dans le cadre des politiques de l'Union européenne en (i) conservant les clauses de sauvegarde dans les accords commerciaux, (ii) maintenant l'exclusion de concessions sur les sucres spéciaux dans les mandats de négociation de la Commission européenne, (iii) mettant en place des clauses suspensives pour l'importation de produits ne respectant pas les règles sanitaires, environnementales et de l'Organisation internationale du travail, (iv) sollicitant l'initiative attendue de la Commission européenne en vue de réviser la stratégie sur les régions ultrapériphériques dans le cadre de la présidence française de l'Union européenne.   Proposition n° 6 : Effectuer une campagne de contrôles nationaux renforcés sur les importations de sucres spéciaux, de sucres "bio" et de rhum afin d'étayer, le cas échéant, des demandes de suspension d'importations.   Proposition n° 7 : Poursuivre la recherche variétale et le développement agronomique de variétés de cannes mixtes, recueillir des données agronomiques et industrielles sur les variétés déjà existantes et favoriser leur plantation. Soutenir également les projets expérimentaux de canne fibre, afin d'évaluer les enjeux agronomiques et industriels, préalable nécessaire à tout déploiement d'ampleur.   Proposition n° 8 : Déterminer avec planteurs, industriels sucriers et énergéticiens le modèle le plus pertinent pour la valorisation énergétique de la paille (séparation de la paille dans le processus sucrier/séparation de la paille par soufflage/ramassage indépendant) ainsi que la répartition du gain entre les acteurs en fonction des nouvelles actions nécessaires.   Proposition n° 9 : Favoriser une intégration verticale avec la filière rhum en accordant du contingent de rhum traditionnel aux sucreries dans le cadre de la répartition des 9000 HAP de contingent supplémentaire et de la part non allouée du contingent antérieur.   Proposition n° 10 : Mettre en place une coordination autour de la DGPE, pour faire émerger au niveau interministériel une vision stratégique de la filière canne-sucre-rhum-énergie.   Proposition n°11 : Dans le cadre de la transformation agricole des Outre-mer, créer au sein des chambres consulaires une commission spécialisée pour la filière canne-sucre-rhum, inclure les réflexions stratégiques sur les enjeux de la filière dans le cadre des comités de transformation qui ont été mis en place, créer des incitations au sein des dispositifs d'appui aux planteurs pour accroitre la diffusion des bonnes pratiques.   Proposition n°12 : Impliquer davantage les collectivités territoriales dans le soutien de la filière canne. Cette implication accrue pourrait se faire via leur participation aux négociations des conventions canne, la mobilisation de l'octroi de mer (à La Réunion), et au titre de leurs compétences notamment en tant qu'autorités de gestion du FEADER (cf. proposition 4).   Proposition n°13 : Plaider pour l'ouverture du capital de Tereos Océan Indien à des acteurs locaux à La Réunion.   Proposition n°14 : Revoir le rôle et les échéances des conventions canne à La Réunion pour ne plus les faire coïncider avec des périodes électorales et ne plus y fixer le prix d'achat de la tonne de canne à sucre.   Proposition n°15 : Etudier les différentes possibilités de transformation de l'activité à Marie-Galante : la faisabilité technique et économique d'un transport de cannes fraîches depuis Marie-Galante vers la sucrerie de Gardel S.A.et le maintien d'une activité de distillerie de rhum traditionnel à la SRMG; la mise en place d'une politique active de reconversion pour les exploitants agricoles."
</article>
<article titre="Sa Cartaroolotte enchante le cœur des gens" date="2021 08 17" journal="Ouest-France  Redon">
Portrait   Sur les marchés et les festivals, le petit engin mobile à explosion de rires de V.✿(prononcé «  V fleur  », son nom d’artiste) ne passe pas inaperçu, et sa joie de vivre non plus. Une invention inédite sortie de son imagination il y a quatre ans. Dessinatrice, elle manie l’humour et le trait de crayon avec volupté, s’inspire de ses rencontres et croque la vie sous toutes ses formes. Cet été, après une année difficile et l’annulation de la plupart des festivals, celle qui a posé ses valises à Renac il y a tout juste un an, est bien occupée.  «  Je vis au jour le jour, et je suis continuellement sollicitée  »,  évoque-t-elle le sourire aux lèvres. Sa «  Carta  » a bien vadrouillé, au gré des manifestations dans le pays de Redon et ailleurs.   «  Les marchés artisanaux ont bien fonctionné, notamment celui de Renac. Je suis allée aussi dans le Morbihan, au Hangar à plume à Marzan…  »   Sans oublier les festivals de BD comme Imaginambulles, à Rochefort-en-Terre, ou celui de dessins d’humour La Bouheraye, dans les Landes, où elle reconnaît avoir  «  cartonné  »  .   Un engin de fête foraine   Sa Cartaroolotte fait des heureux à chaque passage, enfants, parents, petits et grands. Tractée par un vélo, cette ancienne cantine trouvée dans une brocante en Normandie est posée sur une armature agencée par un métallier de Malansac, Philippe Duflos. Il lui a créé l’attache.   «  C’est la version 2,  raconte V..  La première version était un landau, à l’origine ourdi d’oxydation, que je poussais à la main. J’avais imaginé cet engin afin de valoriser mon travail de façon ludique, un peu comme à la fête foraine.  »  Dessus, elle installe un portant de cartes portales et un parapluie thaïlandais.   «  Les enfants adorent !  »  La Carta rassemble toutes les populations   Même si elle peut parler pendant des heures de sa passion pour les gens qui l’inspirent et son métier, tout n’a pas été rose pour V. dans cet univers bien souvent très masculin. Malgré des collaborations avec  Fakir  ou encore  Fluide glacial , il est difficile d’y faire son trou.   «  J’en avais un peu marre de ne pas être publiée, alors je me suis lancée dans cette aventure et c’est génial. J’arrive dans un lieu, je déploie ma Cartaroolotte et je dessine sans filet en dédicaçant mes cartes postales que je présente en fonction de l’endroit où je me trouve. L’idée agrège tous les âges, en cela j’ai gagné mon pari.  »  Un petit hasard de la vie   Son trait, elle le travaille depuis des années, même si elle n’était pas prédestinée à en faire son métier. Sans doute, les turpitudes de la vie et ses voyages passés guident aussi son imaginaire.   «  Je me suis mise au dessin un peu par hasard,  se souvient celle qui a grandi dans une cité HLM de la région parisienne, dans un quartier d’immigrés où le vivre-ensemble voulait dire quelque chose.  Une maîtresse d’école m’avait félicitée pour un dessin. Cela m’a suivi.  »  Attirée par la photographie, elle se résout à passer son bac général et se spécialise dans l’histoire de l’art et le multimédia avant de rejoindre un groupe d’études photographiques. Pendant sept ans, elle parcourt le monde, voyage en Albanie, en Arménie et en Tchécoslovaquie. Le dessin la rattrape, elle y couche ses convictions, aime tourner en dérision ce qui la révolte et se plaît aussi à défendre la cause des peuples.   Des projets à gogo   Et si le Covid s’est immiscé dans son parcours, V. ne s’apitoie pas sur son sort. Elle a poursuivi son projet de webtoon Bagasse et Patavate, ces   «  nouveaux héros de la joie de vivre  »,  et a participé à un ouvrage d’un collectif, Vive la commune.   Dans le pays de Redon, elle se plaît à rencontrer d’autres artistes, et à transmettre sa passion lors d’atelier à la médiathèque de Renac. Désormais, elle va réaliser son rêve : dans une semaine, elle part sur le chemin de Compostelle en «  Carta  » à la découverte d’autres contrées, à la rencontre d’autres gens qu’elle saura sans aucun doute charmer, comme elle a réussi à séduire les Bretons.   V.✿est à suivre sur son compte Instagram @lesaventuresdelacartaroolotte et celles de Bagasse et Patavate sur bagassetpatavate.com  Cet article est paru dans Ouest-France
</article>
<article titre="New normal = ankor béton mam?" date="2021 08 10" journal="Le Mauricien (site web réf.) - Le Mauricien">
MORIS ZINDABAD Omnicane, fortement endetté auprès des banques, a récemment conclu un accord avec la Mauritius Investment Corporation Ltd (MIC) pour céder ses terrains, d’un total de 185 hectares dans...  Voir l'article  Ce document référence un lien URL de site non hébergé par Cision.
</article>
<article titre="Côte d'Ivoire, Congo, Tchad : les sucriers, Somdiaa en tête, repassent à l'offensive" date="2021 08 8" journal="Jeune Afrique (site web)">
Produit de base essentiel, ingrédient clé de l'industrie des boissons, objet de réglementations de marché plus ou moins efficaces, l'or blanc se rappelle au bon souvenir des consommateurs et dirigeants africains. Et pour cause. « Après une longue phase de prix bas, on retrouve des cours mondiaux au plus haut depuis trois ans en raison notamment d'une récolte moyenne au Brésil, pays qui domine de très loin le commerce mondial » , souligne Timothée Masson, économiste et secrétaire général de l'Association mondiale des planteurs de betteraves et de canne à sucre.  À lire Sucre : comment la Côte d'Ivoire veut devenir autosuffisante en cinq ans   À la fin du mois de juin, le prix mondial s'affichait à 0,38 dollars par kilo, soit un bond de 54 % sur un an, selon les données moyennes trimestrielles de la Banque mondiale. De quoi creuser le besoin de devises dans les pays très dépendants des importations, et, pour les pays producteurs du continent trouver une raison supplémentaire de relancer ce secteur, par ailleurs pourvoyeur d'emplois.   Souveraineté alimentaire Les industriels qui musclent la stratégie d'indépendance sucrière ivoirienne, le président sénégalais Macky Sall qui exige une meilleure maîtrise du marché intérieur face à une Compagnie sucrière sénégalaise (CSS) fragilisée, le Gabon, la Tanzanie, ou même le Tchad qui affichent des projets d'investissements... Depuis un an, le secteur du sucre est donc l'objet de toutes les attentions. D'autant que la crise de la covid-19 a remis à la mode le thème de la souveraineté alimentaire.   La Tanzanie, avec Illovo Sugar - de loin le leader du continent avec environ 1,7 million de tonnes produites -, est le premier pays du continent à être passé à l'offensive. Basé en Afrique du sud et actif dans cinq autres pays (dont le Malawi et la Zambie), ce groupe, contrôlé par le britannique ABF, a lancé mi-mai avec l'État tanzanien, actionnaire à 25 %, un plan majeur d'expansion de sa filiale Kilombero. Le projet d'usine, maintes fois promis, semble, cette fois, bien engagé : ABF l'a confirmé à  JA .   Pour la construire, le groupe doit investir 238,5 millions de dollars d'investissement pour 144 000 tonnes de capacité, et ambitionne la contractualisation de 7 000 planteurs de canne supplémentaires pour l'approvisionner. Objectif de la présidente de Tanzanie, Samia Suluhu Hassan : l'indépendance sucrière du pays (où trois autres acteurs existent) d'ici à 2025 en révisant au passage le régime d'importations.   L'OCDE estime la hausse de la consommation en Afrique sub-saharienne à près de 4 % par an La même approche prévaut en Côte d'Ivoire où les industriels en place, Sucrivoire (Sifca) et Sucaf-CI (Somdiaa), ont conclu un contrat de plan avec le gouvernement le 1er mai. Objectif : accroître la production et s'affranchir des importations. Pour cela, dans ce pays qui produit autour de 200 000 tonnes par an, soit un taux d'autosuffisance d'environ 80 %, le gouvernement a promis de maintenir une stricte limitation des importations, et les industriels, eux, se sont engagés, à parité, à investir 230 millions d'euros en cinq ans.    À lire En Afrique, les géants du sucre font parler la poudre   Benoît Coquelet, directeur général délégué du groupe Somdiaa, de loin le principal groupe sucrier en zone francophone (près de 400 000 tonnes), s'inscrit pleinement dans ce plan gouvernemental. « Nous allons poursuivre les efforts pour moderniser nos deux usines de Ferkessédougou (dans le nord du pays), mieux mettre en valeur les terres agricoles, développer les infrastructures ou encore la production d'énergie, notamment à partir de la bagasse, car les sucreries sont gourmandes en électricité et en chaleur, indique-t-il. C'est un schéma vertueux qui doit nous permettre de continuer à produire à un prix compétitif et mieux couvrir les besoins ivoiriens » , fait-il encore valoir.   Deux écoles Car, comme la Côte d'Ivoire, le reste de l'Afrique est de plus en plus gourmand en saccharose. Et est loin d'en produire assez. Selon une estimation de l'agence S&amp;P Global Platts Analytics, pour une demande estimée à 14,8 millions de tonnes en 2019-2020, l'Afrique sub-saharienne, aurait importé 6,5 millions. Et ce n'est pas fini. L'OCDE estime la hausse de la consommation de la zone à près de 4 % par an, soit environ 500 000 tonnes additionnelles de sucre à produire ou à importer chaque campagne.   Certains pays n'en produisent pas comme la Guinée, d'autres très peu comme le Nigeria. Malgré son « Sugar masterplan » , le géant aux 190 millions d'habitants importe plus de 90 % de ses besoins, le plus souvent sous forme de sucre brut, raffiné par des groupes comme Dangote ou Bua, qui ont construit là une bonne part de leur fortune.    « Dans le monde sucrier africain, il y a deux écoles, résume l'économiste Timothée Masson : les pays qui, par choix ou nécessité, se reposent sur les importations en pariant sur les prix mondiaux bas pour approvisionner les consommateurs ; et les autres, qui conduisent une stratégie industrielle, ce qui est bien plus complexe. Notez que même des pays très libéraux comme les États-Unis conservent une politique sucrière affirmée, limitant notamment les importations » , précise le spécialiste.   Permis d'importation au Sénégal C'est le cas aussi - en principe - au Sénégal qui applique un régime de permis d'importation pour protéger sa production. Au pays de la Teranga, la CSS, un des derniers vestiges de l'empire Mimran, est le seul industriel avec son importante usine de Richard Toll.   Implantée en zone semi-aride, mais irriguée grâce au fleuve Sénégal, la CSS affiche une belle performance industrielle, enregistrant les bonnes années un rendement jusqu'à 130 tonnes de canne à l'hectare, « comparable au meilleur niveau brésiliens » , appuie François-Régis Goebel, expert de la canne à sucre au Cirad qui intervient fréquemment chez les sucriers du continent.   À contre-courant du credo de libéralisation à tous crins, il ne peut y avoir de développement d'une industrie sucrière sans volonté politique forte Las ! La situation du marché intérieur menace son modèle comme l'a montré une mini crise intérieure fin 2020. En cause, les autorisations d'importation de produits alimentaires (Dipa) délivrées par l'administration du ministère du Commerce. L'an dernier, des Dipa généreuses, et selon la CSS très au-delà des besoins de l'industrie notamment (boissons, biscuiterie...), ainsi que la contrebande ont déstabilisé le marché de détail. Provoquant le courroux de la CSS et même une manifestation de ses salariés et sous-traitants inquiets pour l'usine. Au point que le président Macky Sall a demandé en Conseil des ministres, en novembre, un contrôle plus strict de ces autorisations d'importation, y compris en les dématérialisant pour éviter les fraudes.    À lire Cedeao, Turquie, Arabie saoudite : Cosumar, dopé par l'export, accélère sa diversification    « Ce type de situation fait le bonheur de certains importateurs et grossistes peu scrupuleux. Il fragilise la stratégie de la CSS dans le cadre de son projet KT200, visant à atteindre 200 000 tonnes de capacité contre 150 000 tonnes aujourd'hui à l'issue du plan KT150 » , commente un fin connaisseur du dossier. Le Burkina Faso, le Tchad ou le Cameroun connaissent des soucis similaires.   Le modèle Cosumar ?  « À contre-courant du credo de libéralisation à tous crins, il ne peut y avoir de développement d'une industrie sucrière sans volonté politique forte, estime Benoît Coquelet. C'est ce que nous montre l'exemple du Maroc. Son champion Cosumar (détenu à 27,4 % par le singapourien Wilmar) reste l'une des plus belles réussites du continent. » Ce groupe, qui produit la moitié de la consommation du royaume et est aussi raffineur de sucre brut importé, veille à rester compétitif industriellement.   Somdiaa, outre en Côte d'Ivoire, a aussi des projets de développements au Congo, au Gabon et même au Tchad Mais, en contrepartie, il bénéficie d'un cadre juridique clair et appliqué sur les prix intérieurs, ainsi que du soutien sans faille de l'État quant au développement de l'amont agricole (la betterave en l'espèce) à travers le Plan Maroc vert et un effort public massif en matière d'irrigation.    À lire La Côte d'Ivoire interdit l'importation de sucre pendant cinq ans pour booster l'industrie locale   Tous les États n'ont pas les mêmes motivations ni les mêmes moyens que le Maroc. Au Bénin et à Madagascar, où le groupe industriel public chinois Complant a repris des usines, parfois en simple gérance, la situation de l'industrie reste des plus fragiles.   Le verrou du foncier Pour sa part, le groupe Somdiaa, dans les six pays où il produit selon un modèle intégrant l'amont agricole (Côte d'Ivoire, Tchad, Gabon, Cameroun, Centrafrique, Congo-Brazzaville) n'entend pas baisser les bras. « Nous sommes dans une phase de fort investissement dans ces pays, de l'ordre de 90 à 100 millions d'euros par an dans la filière. L'industrie sucrière est un métier de long terme, exigeant. Il nécessite de grosses usines fonctionnant sur une période courte et l'organisation intégrée des cultures de canne autour de sites, donc des équipements d'irrigation et une logistique lourde. Le groupe reste pleinement engagé à développer cette activité malgré les difficultés » , affirme encore Benoît Coquelet.   Des difficultés, Somdiaa en connaît actuellement au Cameroun, où la conjonction de mauvaises récoltes sur des plantations non irriguées, d'une rentabilité insuffisante, et de fraudes aux importations se traduisent dans sa filiale Sosucam par un plan de 250 licenciements, très contesté. Mais Somdiaa, outre en Côte d'Ivoire, a aussi dans les cartons des projets de développements au Congo - un des rares pays exportateurs -, au Gabon ou même au Tchad, pays aux frontières pourtant très poreuses...   Le 24 décembre 2020, le groupe a signé un mémorandum d'entente avec l'État tchadien pour la construction d'un complexe sucrier d'une capacité de 173 000 tonnes. Cette usine doit s'étendre au milieu de 16 000 ha de plantations de canne à sucre qui seront irriguées grâce au fleuve Bahr Sara. Du fait du modèle agricole intégré dans de nombreux pays, le foncier reste un verrou pour l'expansion des systèmes sucriers sur le continent.  Cet article est paru dans Jeune Afrique (site web)
</article>
<article titre="ALBIOMA : Mise à disposition du Rapport Financier Semestriel" date="2021 08 4" journal="Yahoo! Finance France (site web réf.) - Yahoo! Finance (FR)">
Globe NewswireALBIOMA : Mise à disposition du Rapport Financier Semestriel - Premier semestre 2021Lire l'article complet ALBIOMA4 août 2021, 11:40 AM·1 min de lecture Communiqué de presse Paris La Défense...  Voir l'article  Ce document référence un lien URL de site non hébergé par Cision.
</article>
<article titre="Pre et Post-Covid-19 : Le groupe Omnicane justifie son deal avec la MIC" date="2021 08 4" journal="Le Mauricien (site web réf.) - Le Mauricien">
Credit Photo : Sharonne Parfait Des pertes colossales de Rs 3,2 milliards dans le bilan financier Sous le feu des critiques depuis son deal avec la Mauritius Investment Corporation (MIC), le...  Voir l'article  Ce document référence un lien URL de site non hébergé par Cision.
</article>
<article titre="Filière sucre en outre-mer : un rapport préconise un approfondissement de sa valorisation énergétique" date="2021 08 3" journal="Actu-Environnement (site web)">
Selon un nouveau rapport,   « les entreprises sucrières sont dans une situation économique fragile »  en Guadeloupe, à la Martinique et à La Réunion. Ce rapport, récemment publié par le Gouvernement, a été signé par une mission interministérielle, rassemblant l'Inspection générale des Finances, le Conseil général de l'environnement et du développement durable (CGEDD) et le Conseil général de l'alimentation, de l'agriculture et des espaces ruraux (CGAAER). L'objectif de cette mission était d'évaluer la situation économique de la filière canne-sucre-rhum-énergie de cinq sites d'outre-mer (deux en Guadeloupe, un à la Martinique et deux à La Réunion).   Une filière en souffrance malgré les aides de l'État  Hormis la production de sucre, la culture de la canne à sucre alimente aussi la production de rhum et, par la valorisation de la bagasse, d'électricité et participe à l'amendement organique des sols avec la vinasse et les résidus de récolte. Dans les trois départements couverts par la mission, elle couvre en moyenne 33,9 % de la surface agricole. Cependant, malgré des aides françaises et européennes s'élevant jusqu'à 439,9 millions d'euros en 2019, la filière souffre, notamment, de la concurrence internationale selon le rapport de la mission citée.   « Les sucreries se porteraient d'autant mieux qu'elles produiraient relativement moins de sucre brut destiné à être raffiné , estiment les missionnaires.  La recherche de produits à plus forte valeur ajoutée est donc un impératif. »  Vers une meilleure valorisation énergétique de la canne à sucre ? Un   « approfondissement »   de la valorisation énergétique de la canne à sucre est ainsi privilégié. A titre d'exemple, évoqué dans le rapport, la production d'1,95 millions de tonnes de cannes à La Réunion a entraîné une restitution annuelle d'électricité, issue de la bagasse, de 240 GWh à EDF - soit, 8 % de la consommation électrique de l'Île en 2018.   Autrement dit, comme le souligne le rapport, ce biocombustible contribue pour une part non-négligeable à la production d'électricité des trois territoires concernés. Pour accentuer ce rendement, et les bénéfices qu'en retirerait la filière, la mission suggère   « le développement de variétés de cannes mixtes permettant de produire plus de fibres (17-18 %) sans que cela ne se fasse au détriment du taux de sucre. »  Pour orienter la filière sur cette voie, entre autres, la mission recommande la mise en place d'une   « aide complémentaire aux industriels, dégressive dans le temps et incitant à la transformation vers des produits à plus forte valeur ajoutée. »   Celle-ci viendrait remplacer une aide gouvernementale de 38 millions d'euros, mise en place en 2017 et qui sera maintenue jusqu'en 2022.  Cet article est paru dans Actu-Environnement (site web)
</article>
<article titre="Publication du rapport interministériel IGF N° 2020-M-060-03 -CGEDD N° 013610-01 -CGAAER N° 20091 « Perspectives de la filière canne-sucre-rhum-énergie en outre-mer » - Rapport interministériel IGF N° 2020-M-060-03 -CGEDD N° 013610-01 -CGAAER N° 20091 ..." date="2021 07 28" journal="Rapports - Agroalimentaire - MAAF">
141 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... Perspectives de la filière canne-sucre-rhum-énergie en outre-mer Tome 1 - Rapport et annexes I, II, VI et VII  MARS 2021  François-Xavier DENIAU  Taline APRIKIAN  Guillaume LACHAUSSÉE  Didier...    Pour voir le rapport :  https://agriculture.gouv.fr/telecharger/127181?token=e1cd656d38129a789661b2aed2ae6e691622950fb8f7fb2c09f28a07b3ef509d
</article>
<article titre="Devesh Dukhira (Syndicat des Sucres) : « La Chine deviendra à terme un marché phare pour nos sucres spéciaux »" date="2021 07 28" journal="Le Mauricien (site web réf.) - Le Mauricien">
Avec le nouvel accord de libre-échange Chine-Maurice, en vigueur depuis début 2021, le Syndicat des sucres s’appuie sur le soutien de l’Economic Development Board et s’active...  Voir l'article  Ce document référence un lien URL de site non hébergé par Cision.
</article>
<article titre="VOS QUESTIONS, NOS RÉPONSES" date="2021 07 22" journal="La lettre de la bourse">
VOS QUESTIONS, NOS RÉPONSES SUPPLEMENT    NOKIA  MALGRÉ UNE MOINS-VALUE DE 39 %, EST-CE QUE JE DOIS ME RÉSOUDRE À VENDRE AUX COURS ACTUELS, L'OBJECTIF DE 4,50 EUROS ÉTANT ATTEINT ? (CF VOTRE RÉPONSE DU 3/05/2021). Effectivement, le titre, avec un bond de plus de 58 % depuis le début de l'année, a effectué une grande partie de son rattrapage. Peut-il aller plus loin ? Le groupe franco-finlandais est bien placé pour profiter du développement de la 5G dans le monde, mais aussi des déboires de son concurrent chinois Huawei. En capitalisant 19,1 et 16,5 fois les bénéfices estimés pour 2021 et 2022, le titre intègre néanmoins en grande partie cette perspective. Mais il pourrait tout de même poursuivre sur sa lancée actuelle, porté par la signature de nouveaux contrats. En revanche, votre prix de revient semble difficilement atteignable.   C'est pourquoi nous vous conseillons de conserver vos titres pour viser 5 euros afin de réduire votre moins-value. Au regard de sa dynamique, la valeur n'a peut-être pas épuisé son potentiel d'appréciation.    COLAS  J'envisage de vendre Colas pour acheter la maison mère, Bouygues, moins chère et avec un rendement plus élevé. Qu'en pensez-vous ? Colas souffre malheureusement d'un faible flottant (3 % du capital à peine), ce qui limite sa valorisation en Bourse. Le problème de Bouygues porte sur sa faible attractivité aux yeux des investisseurs en raison de sa présence sur deux marchés très concurrencés, la télévision avec TF1 et les télécoms. Le titre n'est pas cher et offre un rendement attractif mais le marché n'en veut pas... Les cours actuels à moins de 31 euros paraissent intéressants mais avec un potentiel limité à 36 euros.    TOTALENERGIES  Le titre est-il exagérément décoté ? Le pétrole est au plus haut sur trois ans et Total à -26 % sur trois ans. Effectivement, le titre du groupe pétrolier est en retard. Sa valorisation, avec des multiples de capitalisation des bénéfices estimés pour 2021 et 2022 de 10,4 et 9,6 fois, est très modérée. Les marchés n'ont pas encore pris en compte que les cours du pétrole devraient rester durablement élevés au cours des prochains mois en raison de la reprise plus rapide qu'anticipé de la demande de pétrole et de la baisse continue des investissements des compagnies pétrolières depuis plusieurs années qui va peser sur la production. La valeur est aussi pénalisée car elle ne remplit pas les critères ESG. De nombreux fonds d'investissement l'écartent en conséquence de leurs sélections. Nous sommes de notre côté positifs sur le titre. Grâce à ses efforts pour abaisser son point mort, le groupe est en mesure de dégager suffisamment de liquidités pour continuer d'investir dans la production de pétrole (qui est toujours son cœur de métier), financer sa transformation progressive vers la production d'électricité à partir d'énergies renouvelables et récompenser ses actionnaires. Notre objectif de cours est fixé à 48 euros.    BELIEVE  Peut-on croire au potentiel de la société après son introduction ratée ? Pas pour l'instant. Je pense que le marché va désormais attendre que la société délivre ses objectifs. Il convient donc de rester prudent face au mouvement de défiance en général sur un compartiment des introductions en Bourse saturé et sur Believe en particulier qui est entré sur le marché à des niveaux de valorisation trop élevés.    ILIAD  Vu le cours de l'action et votre objectif à 195 euros, est-ce le bon moment pour investir dans cette action ? Le titre nous semble effectivement se traiter à un niveau plancher. Le cours actuel apparaît être un bon point d'entrée. Le groupe a décidé d'accélérer ses investissements en France dans la 5G pour préparer sa croissance future. Il dispose aussi d'autres relais de croissance avec le développement de sa filiale italienne, son entrée sur le marché des professionnels en France et son implantation en Pologne. Tous ces éléments devraient soutenir la croissance de l'activité et des bénéfices d'Iliad dans les années à venir. Il faut néanmoins se montrer un peu patient. Mais Iliad devrait retrouver le chemin d'une croissance bénéficiaire, ce qui est rare dans le secteur des télécommunications.    ALBIOMA  En moins-value de 14 %, quel est votre avis sur cette action et quel objectif se fixer ? Nous sommes positifs sur cette valeur verte. Albioma peut compter sur ses centrales existantes pour dégager des profits et suffisamment de liquidités pour investir dans de nouveaux projets. Son grand enjeu est la conversion de ses centrales charbon-bagasse à la biomasse à 100 %. Ce qui lui permettra d'être mieux valorisée. Notre objectif de cours a été ramené à 46 euros en raison du projet du gouvernement de changer les règles en matière de tarification pour les centrales photovoltaïques installées entre 2006 et 2010.   Posez vos questions sur lalettredelabourse.fr. La rédaction vous répond sous 48 heures.
</article>
<article titre="Fiche société ALBIOMA" date="2021 07 21" journal="Agence Option Finance (AOF)">
(AOF) -  Points clés - Producteur d'énergie renouvelable -biomasse et photovoltaïque- né en 2013 des fusions successives entre Sidec, la Séchilienne et Albioma-Bois Rouge ;   - Chiffre d'affaires de 6,7 Mds€ réalisé à 41 % en Europe, 25 % dans les Amérique, 12 % en Asie Pacifique et 5 % en Moyen-Orient, Afrique, Russie et 17 % dans les câbles haute tension&amp;projets;  - Production réalisée à 33 % à partir du charbon, à 64 % avec la biomasse et 2 % le solaire; - Stratégie, adoptée en 2012, de réduction de la part du charbon dans la production thermique au profit de la biomasse et du solaire porter à partir des implantations aujourd'hui essentiellement dans l'outre-mer français (1er producteur solaire) puis à l'île Maurice et au Brésil (énergie à base de bagasse, résidu de la canne à sucre); - Capital éclaté avec 3 positions fortes -Invexans et Teck Pack pour 2 !,9 %, BPI France pour 7,7 % et les salariés pour 4,41 %-, Frédéric Moyne étant président-directeur général du conseil d'administration de 8 membres   - Situation financière saine avec une dette brute en baisse à 914 M€, dont 821 M€ de dette projet, face à une trésorerie de 122 M€.    Enjeux - Stratégie 2023visant à agir pour la transition énergétique en outre-mer, déployer l'international et accélérer le développement dans le solaire :    - 450 0 650 M€ d'investissements programmés et financés par emprunts bancaires,    - rationalisation du portefeuille avec cession des activités peu rentables,   - Stratégie d'innovation externalisée auprès du fonds d'amorçage géré par le fonds Demeter Emertec :    - biomasse thermique : recherche de filiales locales,    - stockage d'énergie électrique,    - services au réseau et maintenance prédictive;   - Stratégie environnementale :    - visant à la réduction primaire des émissions polluantes -rejets aqueux, traitement et valorisation sous-produits de combustion et rejets gazeux,   - validée par l'émission d'un « emprunt vert » rémunéré selon des indicateurs clés : +80 % d'énergies renouvelables en 2023 dans la production du groupe et + 95 % en 2030; -  Insensibilité de la rentabilité aux variations de prix des combustibles en raison des indexations contractuelles et visibilité de l'activité par la durée -30 ans- des contrats de vente ;   - Réalisation des projets solaires gouvernementaux remportés lors du 1er semestre pour 20MWc et mise en service de la centrale brésilienne de Vale do Parana  à la fin 2020 .   Défis - Impact négatif de la dépréciation du real brésilien;   - Forte sensibilité aux soutiens financiers publics, aux réglementations sectorielles et à l'accueil du public à de nouvelles centrales;   - Impact de la pandémie : hausse de 1 % du chiffre d'affaires à fin septembre;   - Réaction à la pandémie : économies de coûts, focus sur l'autofinancement et le renforcement des liquidités et la baisse du niveau de risque des projets; - Objectifs 2020 confirmés en base de fourchette d'un résultat d'exploitation entre 200 et 210 M€ et d'un résultat net entre 48 et 54 M€. AOF - EN SAVOIR PLUS Points clés - Producteur d'énergie renouvelable -biomasse et photovoltaïque- né en 2013 des fusions successives entre Sidec, la Séchilienne et Albioma-Bois Rouge ;   - Chiffre d'affaires de 6,7 Mds€ réalisé à 41 % en Europe, 25 % dans les Amérique, 12 % en Asie Pacifique et 5 % en Moyen-Orient, Afrique, Russie et 17 % dans les câbles haute tension&amp;projets;    - Production réalisée à 33 % à partir du charbon, à 64 % avec la biomasse et 2 % le solaire; - Stratégie, adoptée en 2012, de réduction de la part du charbon dans la production thermique au profit de la biomasse et du solaire porter à partir des implantations aujourd'hui essentiellement dans l'outre-mer français (1er producteur solaire) puis à l'île Maurice et au Brésil (énergie à base de bagasse, résidu de la canne à sucre); - Capital éclaté avec 3 positions fortes -Invexans et Teck Pack pour 2 !,9 %, BPI France pour 7,7 % et les salariés pour 4,41 %-, Frédéric Moyne étant président-directeur général du conseil d'administration de 8 membres   - Situation financière saine avec une dette brute en baisse à 914 M€, dont 821 M€ de dette projet, face à une trésorerie de 122 M€.    Enjeux - Stratégie 2023visant à agir pour la transition énergétique en outre-mer, déployer l'international et accélérer le développement dans le solaire :    - 450 0 650 M€ d'investissements programmés et financés par emprunts bancaires,    - rationalisation du portefeuille avec cession des activités peu rentables,   - Stratégie d'innovation externalisée auprès du fonds d'amorçage géré par le fonds Demeter Emertec :    - biomasse thermique : recherche de filiales locales,    - stockage d'énergie électrique,    - services au réseau et maintenance prédictive;   - Stratégie environnementale :    - visant à la réduction primaire des émissions polluantes -rejets aqueux, traitement et valorisation sous-produits de combustion et rejets gazeux,   - validée par l'émission d'un « emprunt vert » rémunéré selon des indicateurs clés : +80 % d'énergies renouvelables en 2023 dans la production du groupe et + 95 % en 2030; -  Insensibilité de la rentabilité aux variations de prix des combustibles en raison des indexations contractuelles et visibilité de l'activité par la durée -30 ans- des contrats de vente ;   - Réalisation des projets solaires gouvernementaux remportés lors du 1er semestre pour 20MWc et mise en service de la centrale brésilienne de Vale do Parana  à la fin 2020 .   Défis - Impact négatif de la dépréciation du real brésilien;   - Forte sensibilité aux soutiens financiers publics, aux réglementations sectorielles et à l'accueil du public à de nouvelles centrales;   - Impact de la pandémie : hausse de 1 % du chiffre d'affaires à fin septembre;   - Réaction à la pandémie : économies de coûts, focus sur l'autofinancement et le renforcement des liquidités et la baisse du niveau de risque des projets; - Objectifs 2020 confirmés en base de fourchette d'un résultat d'exploitation entre 200 et 210 M€ et d'un résultat net entre 48 et 54 M€.
</article>
<article titre="ALBIOMA remporte 12 MWc de projets solaires en Outre-mer" date="2021 07 19" journal="Agence Option Finance (AOF)">
(AOF) - Suite à l'annonce du Ministère de la Transition Écologique, Albioma annonce avoir remporté une puissance agrégée de 12 MWc lors la quatrième période de l'appel d'offre " CRE 4 PV " portant sur la réalisation et l'exploitation d'installations de production d'électricité à partir de l'énergie solaire et situées dans les zones non interconnectées (ZNI). Cette puissance se répartit sur 23 projets, situés dans les zones d'Outre-mer où Albioma est déjà présent : La Réunion, Mayotte, la Martinique, la Guyane et la Guadeloupe.   Sur l'ensemble de l'appel d'offre (soit cinq périodes de soumission depuis avril 2020), le groupe confirme son statut de leader du photovoltaïque en Outre-mer, avec le gain total de 40 MWc. La construction de ces projets est prévue à partir de 2022.   AOF - EN SAVOIR PLUS Points clés - Producteur d'énergie renouvelable -biomasse et photovoltaïque- né en 2013 des fusions successives entre Sidec, la Séchilienne et Albioma-Bois Rouge ;   - Chiffre d'affaires de 6,7 Mds€ réalisé à 41 % en Europe, 25 % dans les Amérique, 12 % en Asie Pacifique et 5 % en Moyen-Orient, Afrique, Russie et 17 % dans les câbles haute tension&amp;projets;    - Production réalisée à 33 % à partir du charbon, à 64 % avec la biomasse et 2 % le solaire; - Stratégie, adoptée en 2012, de réduction de la part du charbon dans la production thermique au profit de la biomasse et du solaire porter à partir des implantations aujourd'hui essentiellement dans l'outre-mer français (1er producteur solaire) puis à l'île Maurice et au Brésil (énergie à base de bagasse, résidu de la canne à sucre); - Capital éclaté avec 3 positions fortes -Invexans et Teck Pack pour 2 !,9 %, BPI France pour 7,7 % et les salariés pour 4,41 %-, Frédéric Moyne étant président-directeur général du conseil d'administration de 8 membres   - Situation financière saine avec une dette brute en baisse à 914 M€, dont 821 M€ de dette projet, face à une trésorerie de 122 M€.    Enjeux - Stratégie 2023visant à agir pour la transition énergétique en outre-mer, déployer l'international et accélérer le développement dans le solaire :    - 450 0 650 M€ d'investissements programmés et financés par emprunts bancaires,    - rationalisation du portefeuille avec cession des activités peu rentables,   - Stratégie d'innovation externalisée auprès du fonds d'amorçage géré par le fonds Demeter Emertec :    - biomasse thermique : recherche de filiales locales,    - stockage d'énergie électrique,    - services au réseau et maintenance prédictive;   - Stratégie environnementale :    - visant à la réduction primaire des émissions polluantes -rejets aqueux, traitement et valorisation sous-produits de combustion et rejets gazeux,   - validée par l'émission d'un « emprunt vert » rémunéré selon des indicateurs clés : +80 % d'énergies renouvelables en 2023 dans la production du groupe et + 95 % en 2030; -  Insensibilité de la rentabilité aux variations de prix des combustibles en raison des indexations contractuelles et visibilité de l'activité par la durée -30 ans- des contrats de vente ;   - Réalisation des projets solaires gouvernementaux remportés lors du 1er semestre pour 20MWc et mise en service de la centrale brésilienne de Vale do Parana  à la fin 2020 .   Défis - Impact négatif de la dépréciation du real brésilien;   - Forte sensibilité aux soutiens financiers publics, aux réglementations sectorielles et à l'accueil du public à de nouvelles centrales;   - Impact de la pandémie : hausse de 1 % du chiffre d'affaires à fin septembre;   - Réaction à la pandémie : économies de coûts, focus sur l'autofinancement et le renforcement des liquidités et la baisse du niveau de risque des projets; - Objectifs 2020 confirmés en base de fourchette d'un résultat d'exploitation entre 200 et 210 M€ et d'un résultat net entre 48 et 54 M€.
</article>
<article titre="Jacqueline Sauzier : « L’incertitude sur la publication du rapport de la BM est dérangeante »" date="2021 07 18" journal="Le Mauricien (site web réf.) - Le Mauricien">
Jacqueline Sauzier fait partie des femmes qui sont à la tête des institutions du secteur privé. Dans l’interview accordée à Le-Mauricien cette semaine, elle reconnaît que l’incertitude autour...  Voir l'article  Ce document référence un lien URL de site non hébergé par Cision.
</article>
<article titre="Amérique Latine : Sanofi, Webhelp, Albioma, Danone, Sodexo... Accès libre" date="2021 07 16" journal="CFNEWS (site web)">
Produits Pharmaceutiques &amp; matériel médical : Sanofi - Hypera (Brésil)Le groupe pharmaceutique français Sanofi (voir sa fiche et deal-list dans notre référentiel) cède douze marques de médicaments en vente libre et sur ordonnance au Brésil, au Mexique et en Colombie au laboratoire brésilien Hypera Pharma pour 190,3 M$ (161,12 M€). Parmi les produits cédés figurent par exemple l'analgésique AAS, l'antiseptique Cepacol, des médicaments sur ordonnance Buclina, pour stimuler l'appétit et Hidantal, indiqué pour le traitement de l'épilepsie, entre autres. Le Brésil représentait environ 67 % du chiffre d'affaires de ce portefeuille en 2020. Cette opération s'inscrit dans la stratégie de Sanofi visant à faire de sa division de médicaments sans ordonnance une activité autonome. « Comme nous l'avions indiqué lors de notre Capital Markets Day de février, la simplification du portefeuille de produits Santé Grand Public est un aspect important de notre stratégie qui consiste à concentrer nos ressources et nos efforts là où nous pouvons apporter le plus de valeur, en particulier pour les consommateurs. Nous nous félicitons que ces produits continuent d'être mis à la disposition des consommateurs et nous allons poursuivre notre transformation pour devenir une activité autonome entièrement intégrée », a précisé Julie Van Ongevalle, vice-président exécutif et responsable de Sanofi Santé Grand Public. Voir la fiche de l'opération M&amp;A Sanofi - Hypera Pharma. Services &amp; Conseil aux entreprises : Webhelp - Onelink (Amérique Latine)L'opérateur français des centres d'appels Webhelp (voir sa fiche et deal-list), fort de 1,6 Md€ de chiffre d'affaires en 2020, acquiert l'entreprise OneLink, spécialisée dans les services d'expe rience client, BPO et technologiques axe s sur le digital. Basée à Guatemala, la cible réalise 200 M$ (169,3 M€) de revenus, emploie plus de 14 000 personnes et exploite dix-sept centres au Mexique, au Salvador, au Nicaragua, au Guatemala, en Colombie et au Bre sil. Le spécialiste de la relation client, soutenu depuis 2019 par Groupe Bruxelles Lambert, poursuit ainsi son expansion ge ographique et enrichit son offre de services gra ce a une strate gie de croissance organique et des ope rations de fusion et acquisition, dont treize acquisitions au cours des cinq dernie res anne es. À l'issue de cette transaction, soumise à l'approbation des autorités réglementaires compétentes et dont les termes financiers n'ont pas été divulgués, le chiffre d'affaires de Webhelp s'élèvera à plus de 2,4 Md$ (2 Md€) avec 190 sites répartis dans 55 pays et plus de 90 000 collaborateurs. Voir la fiche de l'opération LBO Webhelp - Onelink.   Energie &amp; Utilities : Albioma (Brésil)    © Albioma Codora Energia    Albioma Codora Energia, filiale brésilienne d'Albioma, remporte un appel d'offres pour sécuriser la vente d'énergie de sa centrale 100 % bagasse. Ce contrat d'une durée de 20 ans à partir de 2025, pour un volume de 64 GWh et à un prix garanti de 202,35 BRL/MWh indexé sur l'inflation, pourra être conclu grâce à l'augmentation du volume de broyage de la sucrerie attenante et à la valorisation énergétique de la vinasse (résidu de la distillation d'éthanol). L'usine de cogénération Codora Energia (68 MW), dont le producteur d'énergie renouvelable coté détient 65 % du capital depuis 2015, est située dans l'État de Goiás et est adossée à une sucrerie/distillerie qui appartient au groupe Jalles Machado, le deuxième producteur mondial de sucre issu de l'agriculture biologique. Dans le cadre du partenariat conclu avec Jalles Machado, 461 000 tonnes de bagasse ont été utilisées en 2020 pour produire de la vapeur basse pression et 173 GWh d'électricité. En dehors des périodes de récolte de la canne à sucre, la centrale Codora Energia est alimentée par d'autres sources de biomasses locales, comme de la paille de canne ou des plaquettes de bois, ce qui permet de sécuriser son rendement. La majorité de la production d'Albioma Codora Energia est déjà vendue sur le marché régulé avec des contrats à long terme. Implanté en Outre-Mer français, en France métropolitaine, à l'Île Maurice, au Brésil et en Turquie, Albioma a généré 507 M€ de chiffre d'affaires en 2020.   Agriculture &amp; alimentaire : Danone (Brésil)    Usine Danone à Poços de Caldas au Brésil    Le site de Danone produisant des laits infantiles et des produits de nutrition médicale situé à Poços de Caldas au Brésil, est le premier du groupe à avoir atteint les objectifs environnementaux du carbone, de l'eau et des déchets. L'usine a été certifiée par le Carbon Trust, un organisme mondial indépendant de lutte contre le changement climatique et conseil en développement durable, en tant que carbone neutre et zéro déchet en décharge. Cette certification reconnait également les efforts de l'usine à réduire sa consommation en eau, en ligne avec l'ambition plus large de Danone d'atteindre la circularité de l'eau dans et autour de ses sites de production. Déterminé à atteindre zéro émission nette de carbone sur l'ensemble de sa chaîne de valeur d'ici 2050, le groupe tricolore s'emploie activement à ce que tous ses sites de production dans le monde suivent le même chemin et, à terme, réduisent leur impact environnemental afin de protéger la santé de notre planète.    Services &amp; Conseil aux entreprises : Sodexo (Colombie) Sodexo Colombie a reçu le prix GEEIS-SDGs pour son programme SheWorks. Le prix GEEIS-SDG, qui positionne l'égalité des sexes comme le moteur de la réalisation des objectifs de développement durable des Nations Unies, a été créé par Arborus, ONG ayant pour objet la promotion des femmes aux postes décisionnels. Le programme SheWorks vise l'inclusion de femmes à des postes généralement occupés par des hommes. Ainsi, des femmes issues de quartiers défavorisés ont été invitées à rejoindre le programme et à suivre une formation académique de neuf mois avec un stage de six mois sur plusieurs postes en rotation. En octobre dernier, seize d'entre elles ont obtenu le titre de « Technicien en installations électriques domestiques ». Depuis, vingt-cinq nouvelles femmes ont rejoint le programme et sont actuellement dans leur quatrième semestre d'études. SheWorks est actuellement déployé dans vingt pays différents.   Une information à nous soumettre pour ce Bulletin Amérique Latine ? Écrivez-nous à : dalila.weygang@cfnews.net   Retrouver les autres Chroniques de CFNEWS :   Tous les précédents bulletins Amérique LatineTous les bulletins AsieAussi sur Facebook , Twitter , Google+ et LinkedIn Mots-clés : ALBIOMA (EX SECHILIENNE SIDEC) , DANONE , SANOFI , SODEXO , WEBHELP Amérique Latine Etranger Corporate Finance    Voir la fiche de : SANOFI   Voir la fiche de : WEBHELP   Voir la fiche de : ALBIOMA (EX SECHILIENNE SIDEC)   Voir la fiche de : DANONE   Voir la fiche de : SODEXO    Les intervenants de l'opération 12 MARQUES SANOFI (BRÉSIL, MEXIQUE, COLOMBIE) Société cible 12 MARQUES SANOFI (BRÉSIL, MEXIQUE, COLOMBIE) Acquéreur ou Investisseur HYPERA PHARMA Cédant SANOFI Voir la fiche détaillée de l'opération Les intervenants de l'opération ONELINK Société cible ONELINK Acquéreur ou Investisseur WEBHELP , GROUPE BRUXELLES LAMBERT (GBL) Cédant ONE EQUITY PARTNERS Acquéreur Banquier d'Affaires / Conseil M&amp;A GOLDMAN SACHS Acquéreur Avocat Corporate LATHAM &amp; WATKINS Acquéreur Avocat structuration fiscale LINKLATERS Acq. Avocats d'Affaires Financement LINKLATERS Acq. DD Juridique et Fiscale ALVAREZ &amp; MARSAL (A&amp;M) Acq. DD Financière ALVAREZ &amp; MARSAL (A&amp;M) , Ghislain de Seze , Simon Regad Acq. DD Stratégique ROLAND BERGER Cédant Avocat corporate GREENBERG TRAURIG Société Banquier d'Affaires / Conseil M&amp;A ROTHSCHILD &amp; CO Société Avocat d'Affaires Corporate GREENBERG TRAURIG Société DD Financière KPMG Société DD Autres AUCTUS ADVISORS Conseil Financement KKR CAPITAL MARKETS , GOLDMAN SACHS Voir la fiche détaillée de l'opération  Cet article est paru dans CFNEWS (site web)
</article>
<article titre="ALBIOMA sécurise la vente d'énergie de sa centrale 100 %  bagasse au Brésil" date="2021 07 16" journal="Agence Option Finance (AOF)">
(AOF) - Albioma Codora Energia, la filiale brésilienne d'Albioma, a remporté le 8 juillet dernier un appel d'offres portant sur un nouveau contrat de vente d'électricité (PPA - Power Purchase Agreement) d'une durée de 20 ans à partir de 2025, pour un volume de 64 GWh et à un prix garanti de 202,35 BRL/MWh indexé sur l'inflation. Ce contrat pourra être honoré grâce à l'augmentation du volume de broyage de la sucrerie attenante et à la valorisation énergétique de la vinasse (résidu de la distillation d'éthanol).    La majorité de la production d'Albioma Codora Energia est déjà vendue sur le marché régulé avec des contrats à long terme indexés sur l'inflation. Rachetée par Albioma à hauteur de 65 % du capital en 2015, l'usine de cogénération Codora Energia (68 MW), située dans l'État de Goiás, est adossée à une sucrerie/distillerie qui appartient au groupe Jalles Machado.    Depuis le début de ce partenariat, en 2015, l'export d'énergie a augmenté de 40 %, notamment par l'amélioration de la performance industrielle des actifs réalisée par Albioma, alors que Jalles Machado installait une sucrerie en complément de la distillerie existante.   AOF - EN SAVOIR PLUS Points clés - Producteur d'énergie renouvelable -biomasse et photovoltaïque- né en 2013 des fusions successives entre Sidec, la Séchilienne et Albioma-Bois Rouge ;   - Chiffre d'affaires de 6,7 Mds€ réalisé à 41 % en Europe, 25 % dans les Amérique, 12 % en Asie Pacifique et 5 % en Moyen-Orient, Afrique, Russie et 17 % dans les câbles haute tension&amp;projets;    - Production réalisée à 33 % à partir du charbon, à 64 % avec la biomasse et 2 % le solaire; - Stratégie, adoptée en 2012, de réduction de la part du charbon dans la production thermique au profit de la biomasse et du solaire porter à partir des implantations aujourd'hui essentiellement dans l'outre-mer français (1er producteur solaire) puis à l'île Maurice et au Brésil (énergie à base de bagasse, résidu de la canne à sucre); - Capital éclaté avec 3 positions fortes -Invexans et Teck Pack pour 2 !,9 %, BPI France pour 7,7 % et les salariés pour 4,41 %-, Frédéric Moyne étant président-directeur général du conseil d'administration de 8 membres   - Situation financière saine avec une dette brute en baisse à 914 M€, dont 821 M€ de dette projet, face à une trésorerie de 122 M€.    Enjeux - Stratégie 2023visant à agir pour la transition énergétique en outre-mer, déployer l'international et accélérer le développement dans le solaire :    - 450 0 650 M€ d'investissements programmés et financés par emprunts bancaires,    - rationalisation du portefeuille avec cession des activités peu rentables,   - Stratégie d'innovation externalisée auprès du fonds d'amorçage géré par le fonds Demeter Emertec :    - biomasse thermique : recherche de filiales locales,    - stockage d'énergie électrique,    - services au réseau et maintenance prédictive;   - Stratégie environnementale :    - visant à la réduction primaire des émissions polluantes -rejets aqueux, traitement et valorisation sous-produits de combustion et rejets gazeux,   - validée par l'émission d'un « emprunt vert » rémunéré selon des indicateurs clés : +80 % d'énergies renouvelables en 2023 dans la production du groupe et + 95 % en 2030; -  Insensibilité de la rentabilité aux variations de prix des combustibles en raison des indexations contractuelles et visibilité de l'activité par la durée -30 ans- des contrats de vente ;   - Réalisation des projets solaires gouvernementaux remportés lors du 1er semestre pour 20MWc et mise en service de la centrale brésilienne de Vale do Parana  à la fin 2020 .   Défis - Impact négatif de la dépréciation du real brésilien;   - Forte sensibilité aux soutiens financiers publics, aux réglementations sectorielles et à l'accueil du public à de nouvelles centrales;   - Impact de la pandémie : hausse de 1 % du chiffre d'affaires à fin septembre;   - Réaction à la pandémie : économies de coûts, focus sur l'autofinancement et le renforcement des liquidités et la baisse du niveau de risque des projets; - Objectifs 2020 confirmés en base de fourchette d'un résultat d'exploitation entre 200 et 210 M€ et d'un résultat net entre 48 et 54 M€.
</article>
<article titre="Albioma Codora Energia a remporté un appel d'offres pour sécuriser la vente d'énergie de sa centrale 100%  bagasse" date="2021 07 16" journal="Boursier (site web)">
Albioma annonce que sa filiale brésilienne Albioma Codora Energia a remporté le 8 juillet dernier un appel d'offres portant sur un nouveau contrat de vente d'électricité (PPA - Power Purchase Agreement) d'une durée de 20 ans à partir de 2025, pour un volume de 64 GWh et à un prix garanti de 202,35 BRL/MWh indexé sur l'inflation. Ce contrat pourra être honoré grâce à l'augmentation du volume de broyage de la sucrerie attenante et à la valorisation énergétique de la vinasse (résidu de la distillation d'éthanol). La majorité de la production d'Albioma Codora Energia est déjà vendue sur le marché régulé avec des contrats à long terme indexés sur l'inflation.   Rachetée par Albioma à hauteur de 65% du capital en 2015, l'usine de cogénération Codora Energia (68 MW), située dans l'État de Goiás, est adossée à une sucrerie/distillerie qui appartient au groupe Jalles Machado. Depuis le début de ce partenariat, en 2015, l'export d'énergie a augmenté de 40 %, notamment par l'amélioration de la performance industrielle des actifs réalisée par Albioma, alors que Jalles Machado installait une sucrerie en complément de la distillerie existante.   Prochain rendez-vous : résultats du premier semestre de l'exercice 2021, le 28 juillet 2021 (après bourse).
</article>
<article titre="Les énergies renouvelables n'ont pas progressé en 2020 à La Réunion" date="2021 07 15" journal="Les Echos (site web)">
Le mix électrique a peu évolué en 2020, selon les chiffres du bilan énergétique présentés dans le dernier rapport de l'Institut d'émission d'Outre-mer. La part des énergies renouvelables dans la production d'électricité s'élève à 31 %. Quelque 14 % proviennent de l'hydroélectricité, 10 % du photovoltaïque, de l'éolien et du biogaz et 7 % de la bagasse, résidu fibreux du broyage de la canne à sucre, brûlé dans les deux centrales à charbon d'Albioma. Plus des deux tiers de la production électrique insulaire proviennent de matières premières fossiles importées : le charbon importé d'Afrique du Sud, pour 37 %, et le fioul lourd consommé par la centrale thermique d'EDF, sise sur les quais du port de commerce.   EDF a distribué l'an passé 2.973 gigawatts/heure d'électricité, un chiffre en très léger recul (-0,1 %) par rapport à 2019 en raison de la baisse de consommation constatée pendant le confinement, de 17 mars au 10 mai.   Bientôt la biomasse Depuis 2010, la part des énergies renouvelables dans le mix électrique oscille entre 30 et 37 %. La production photovoltaïque poursuit une croissance régulière alors que la production hydroélectrique varie en fonction de la pluviométrie, irrégulière. La part de la bagasse est à la baisse, en raison de la diminution tendancielle de la production de canne à sucre.   La production électrique réunionnaise va toutefois se « décarboner » rapidement. Albioma prépare en effet la conversion de ses deux centrales thermiques de Saint-André et Saint-Louis. Fin 2023, les deux unités devraient fonctionner quasi exclusivement à la biomasse, constituée principalement de granulés de bois importés des Etats-Unis. EDF annonce un calendrier identique pour la conversion à la biomasse liquide, importée d'Europe, de sa centrale au fioul.   Bernard Grollier
</article>
<article titre="(Industrie sucrière) Maneesh Gobin : « Mo pa le rann rapor la Bank Mondial la piblik »" date="2021 07 15" journal="Le Mauricien (site web réf.) - Le Mauricien">
Évoquant le rapport de la Banque mondiale sur les réformes dans l’industrie sucrière, dont les opérateurs attendent la publication depuis plusieurs mois, l’Attorney General et ministre de l’Agro...  Voir l'article  Ce document référence un lien URL de site non hébergé par Cision.
</article>
<article titre="A suivre aujourd'hui... ALBIOMA" date="2021 07 14" journal="Boursorama (site web réf.) - Boursorama">
(AOF) - Le 7 mai dernier, Albioma avait annoncé avoir été victime d'une cyberattaque par ransomware sur son réseau informatique. Le groupe rappelle que l'attaque n'a eu aucune conséquence...  Voir l'article  Ce document référence un lien URL de site non hébergé par Cision.
</article>
<article titre="PPE Martinique - avis (PDF - 7.08 Mo) - PPE Martinique - avis.pdf" date="2021 07 8" journal="Rapports - Environnement - MEDDTL">
62 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... CGEDD Avis délibéré du 28 juin 2017 - programmation pluriannuelle de l'énergie de la Martinique (972)  Page 1 sur 29  Avis délibéré de l’Autorité environnementale  sur la programmation pluriannuelle de...    Pour voir le rapport :  https://www.ecologie.gouv.fr/sites/default/files/PPE%20Martinique%20-%20avis.pdf
</article>
<article titre="Fiche société ALBIOMA" date="2021 07 7" journal="Agence Option Finance (AOF)">
(AOF) -  Points clés - Producteur d'énergie renouvelable -biomasse et photovoltaïque- né en 2013 des fusions successives entre Sidec, la Séchilienne et Albioma-Bois Rouge ;   - Chiffre d'affaires de 6,7 Mds€ réalisé à 41 % en Europe, 25 % dans les Amérique, 12 % en Asie Pacifique et 5 % en Moyen-Orient, Afrique, Russie et 17 % dans les câbles haute tension&amp;projets;  - Production réalisée à 33 % à partir du charbon, à 64 % avec la biomasse et 2 % le solaire; - Stratégie, adoptée en 2012, de réduction de la part du charbon dans la production thermique au profit de la biomasse et du solaire porter à partir des implantations aujourd'hui essentiellement dans l'outre-mer français (1er producteur solaire) puis à l'île Maurice et au Brésil (énergie à base de bagasse, résidu de la canne à sucre); - Capital éclaté avec 3 positions fortes -Invexans et Teck Pack pour 2 !,9 %, BPI France pour 7,7 % et les salariés pour 4,41 %-, Frédéric Moyne étant président-directeur général du conseil d'administration de 8 membres   - Situation financière saine avec une dette brute en baisse à 914 M€, dont 821 M€ de dette projet, face à une trésorerie de 122 M€.    Enjeux - Stratégie 2023visant à agir pour la transition énergétique en outre-mer, déployer l'international et accélérer le développement dans le solaire :    - 450 0 650 M€ d'investissements programmés et financés par emprunts bancaires,    - rationalisation du portefeuille avec cession des activités peu rentables,   - Stratégie d'innovation externalisée auprès du fonds d'amorçage géré par le fonds Demeter Emertec :    - biomasse thermique : recherche de filiales locales,    - stockage d'énergie électrique,    - services au réseau et maintenance prédictive;   - Stratégie environnementale :    - visant à la réduction primaire des émissions polluantes -rejets aqueux, traitement et valorisation sous-produits de combustion et rejets gazeux,   - validée par l'émission d'un « emprunt vert » rémunéré selon des indicateurs clés : +80 % d'énergies renouvelables en 2023 dans la production du groupe et + 95 % en 2030; -  Insensibilité de la rentabilité aux variations de prix des combustibles en raison des indexations contractuelles et visibilité de l'activité par la durée -30 ans- des contrats de vente ;   - Réalisation des projets solaires gouvernementaux remportés lors du 1er semestre pour 20MWc et mise en service de la centrale brésilienne de Vale do Parana  à la fin 2020 .   Défis - Impact négatif de la dépréciation du real brésilien;   - Forte sensibilité aux soutiens financiers publics, aux réglementations sectorielles et à l'accueil du public à de nouvelles centrales;   - Impact de la pandémie : hausse de 1 % du chiffre d'affaires à fin septembre;   - Réaction à la pandémie : économies de coûts, focus sur l'autofinancement et le renforcement des liquidités et la baisse du niveau de risque des projets; - Objectifs 2020 confirmés en base de fourchette d'un résultat d'exploitation entre 200 et 210 M€ et d'un résultat net entre 48 et 54 M€. AOF - EN SAVOIR PLUS Points clés - Producteur d'énergie renouvelable -biomasse et photovoltaïque- né en 2013 des fusions successives entre Sidec, la Séchilienne et Albioma-Bois Rouge ;   - Chiffre d'affaires de 6,7 Mds€ réalisé à 41 % en Europe, 25 % dans les Amérique, 12 % en Asie Pacifique et 5 % en Moyen-Orient, Afrique, Russie et 17 % dans les câbles haute tension&amp;projets;    - Production réalisée à 33 % à partir du charbon, à 64 % avec la biomasse et 2 % le solaire; - Stratégie, adoptée en 2012, de réduction de la part du charbon dans la production thermique au profit de la biomasse et du solaire porter à partir des implantations aujourd'hui essentiellement dans l'outre-mer français (1er producteur solaire) puis à l'île Maurice et au Brésil (énergie à base de bagasse, résidu de la canne à sucre); - Capital éclaté avec 3 positions fortes -Invexans et Teck Pack pour 2 !,9 %, BPI France pour 7,7 % et les salariés pour 4,41 %-, Frédéric Moyne étant président-directeur général du conseil d'administration de 8 membres   - Situation financière saine avec une dette brute en baisse à 914 M€, dont 821 M€ de dette projet, face à une trésorerie de 122 M€.    Enjeux - Stratégie 2023visant à agir pour la transition énergétique en outre-mer, déployer l'international et accélérer le développement dans le solaire :    - 450 0 650 M€ d'investissements programmés et financés par emprunts bancaires,    - rationalisation du portefeuille avec cession des activités peu rentables,   - Stratégie d'innovation externalisée auprès du fonds d'amorçage géré par le fonds Demeter Emertec :    - biomasse thermique : recherche de filiales locales,    - stockage d'énergie électrique,    - services au réseau et maintenance prédictive;   - Stratégie environnementale :    - visant à la réduction primaire des émissions polluantes -rejets aqueux, traitement et valorisation sous-produits de combustion et rejets gazeux,   - validée par l'émission d'un « emprunt vert » rémunéré selon des indicateurs clés : +80 % d'énergies renouvelables en 2023 dans la production du groupe et + 95 % en 2030; -  Insensibilité de la rentabilité aux variations de prix des combustibles en raison des indexations contractuelles et visibilité de l'activité par la durée -30 ans- des contrats de vente ;   - Réalisation des projets solaires gouvernementaux remportés lors du 1er semestre pour 20MWc et mise en service de la centrale brésilienne de Vale do Parana  à la fin 2020 .   Défis - Impact négatif de la dépréciation du real brésilien;   - Forte sensibilité aux soutiens financiers publics, aux réglementations sectorielles et à l'accueil du public à de nouvelles centrales;   - Impact de la pandémie : hausse de 1 % du chiffre d'affaires à fin septembre;   - Réaction à la pandémie : économies de coûts, focus sur l'autofinancement et le renforcement des liquidités et la baisse du niveau de risque des projets; - Objectifs 2020 confirmés en base de fourchette d'un résultat d'exploitation entre 200 et 210 M€ et d'un résultat net entre 48 et 54 M€.
</article>
<article titre="Emballages alimentaires en plastique biosourcé" date="2021 07 1" journal="Revue Lamy Droit Alimentaire, no. 416">
Action en concurrence déloyale rejetée   CA Poitiers, 2 ème  ch., 16 févr. 2021, n o  20/02044 Une première société (ci-après le fabricant) exerce une activité de fabrication d’articles d’emballage en matière plastique tels que bouteilles, flacons, bidons, articles de bouchage, etc. Elle fabrique et commercialise, notamment, des contenants à base de PLA (acide polylactique), et en particulier des bouteilles non capsulées, en utilisant des déchets de canne à sucre (ou bagasse).   Depuis la fin de l’année 2015, elle a noué une relation d’affaires avec une deuxième société (ci-après la société commerciale) dont l’activité est le commerce de tous articles et produits non alimentaires ainsi que de matériels destinés notamment à l’entretien et au nettoyage de toute surface et de tout véhicule. Le 20 octobre 2015, la première société a conclu avec la deuxième société pour une durée de 5 ans un contrat de confidentialité et de non divulgation, dans lequel il était rappelé que le fabricant possédait des connaissances concernant : des bouteilles et bouchons spécifiques ;le développement de nouveaux matériaux ;le design et le procédé de fabrication. De son côté, la société commerciale possédait des connaissances concernant la vente en vrac, les distributeurs automatiques de produits alimentaires et non alimentaires et les bornes de collecte du bio plastique.   Le 5 mars 2016, les mêmes parties ont convenu d’un contrat de partenariat au terme duquel la société commerciale s’engageait à acheter et à exploiter des bouteilles réalisées sur mesure pour son compte en PLA par le fabricant. Ces bouteilles étaient destinées à la vente et la distribution de produits en vrac tels que l’eau du robinet et des détergents commercialisés par la société commerciale.   Il était stipulé que la société commerciale restait la seule et unique bénéficiaire des moules conçus par le fabricant pour son compte dans le but de souffler les bouteilles en PLA, et que jusqu’à dénonciation du contrat par l’une des parties, elle resterait le bénéficiaire exclusif de cette production de bouteilles en PLA.   En contrepartie, le fabricant s’engageait à mettre à disposition de la société commerciale la documentation nécessaire aux marchandises concernées ainsi que toutes informations utiles pouvant servir de support de communication.   Les deux parties s’engageaient aux termes du contrat à ne pas faire preuve de concurrence déloyale vis-à-vis de la clientèle quel que soit le moyen employé.   Chacune des parties devait considérer comme confidentielles pendant toute la durée du contrat et après son expiration toutes informations obtenues dans le cadre de son exécution. Les deux parties s’interdisaient plus particulièrement de communiquer à des tierces personnes autres que son personnel toute information relative au contrat.   Au mois d’avril 2019, un associé de la société commerciale a créé une société de fabrication et de commercialisation de produits biosourcés.   Par courrier recommandé en date du 2 décembre 2019, le fabricant a mis en demeure la nouvelle société de cesser tout acte et toute communication caractérisant un manquement à l’engagement de non-concurrence et de confidentialité. Le 16 janvier 2020, le fabricant a fait assigner la nouvelle société devant le juge des référés du tribunal de commerce en cessation sous astreinte des activités illicites et en paiement d’une indemnité provisionnelle à valoir sur leur préjudice commercial et moral. Selon le fabricant, les bouteilles présentées par la nouvelle société sur son site internet ou gourdes en matière 100 % végétale à partir de ressources non alimentaires, et en particulier à partir de déchets de canne à sucre, étaient identiques (sauf pour leur design) aux bouteilles objets du contrat de partenariat et de l’engagement de confidentialité.   Par jugement du 1 er  septembre 2020, le tribunal de commerce a débouté le fabricant de l’ensemble de ses demandes. Le tribunal a considéré, en substance, que le fabricant ne disposait pas d’un savoir-faire unique, dès lors que plusieurs sociétés concurrentes réalisaient également des bouteilles 100 % végétales, certaines depuis 2002.   Le fabricant a fait appel de ce jugement.   La cour d’appel devait tout d’abord rappeler les dispositions légales du Code de la consommation interdisant les pratiques commerciales déloyales et trompeuses (voir Le Lamy Dehove, n o  131-95 et n o  131-100).   Le site internet de la nouvelle société comportait des pages concernant les bouteilles et les gourdes litigieuses. Il présentait notamment, une bouteille 100 % issue du végétal décrite comme 100 % biodégradable, une bouteille dite bouteille végétalisée, gourde 100 % issue du végétal (déchets de canne à sucre) fabriquée en PLA, bouteille tout ou partie issue du végétal (non de ressources alimentaires) biodégradable, compostable et/ou recyclables.   Dans la présentation de la bouteille, il est indiqué « PLA certifié sans OGM, compostable industriellement selon la norme NF EN 13432 en vigueur et recyclable ».   Le fabricant soutient qu’en présentant à la vente sur leur site internet, notamment, des bouteilles et gourdes destinées à être en contact avec des liquides et boissons, fabriquées soi-disant à partir de matière bioplastique PLA issue du végétal (déchets de canne à sucre), « biodégradable(s), compostable(s) », propriété dont elle ne peut manifestement pas bénéficier au sens de la réglementation applicable, la nouvelle société s’est rendue coupable de pratiques commerciales déloyales.   La cour d’appel a ensuite rappelé qu’un fait générateur de concurrence déloyale peut consister dans l’inobservation d’une réglementation dans la mesure où, en s’affranchissant des réglementations ou prohibitions légales, une entreprise perturbe le marché.   Pour déterminer la réglementation applicable, les juges relèvent que la bouteille et la gourde sont destinées à servir de contenants de liquides alimentaires. Aussi ces produits relèvent du champ d’application du règlement CE n o  1935/2004 du Parlement européen et du Conseil du 27 octobre 2004 concernant les matériaux et objets destinés à entrer en contact avec des denrées alimentaires (voir Le Lamy Dehove, étude n o  705). Le règlement précité définit notamment les termes de mise sur le marché comme étant la détention de matériaux et objets en vue de leur vente, y compris l’offre en vue de la vente ou toute autre forme de cession, à titre gratuit ou onéreux, ainsi que la vente, la distribution et les autres formes de cession proprement dites.   Le site internet de la nouvelle société ne permet pas au consommateur d’acheter de produit en ligne et il ne donne aucune information sur des points de vente physiques. Cependant la bouteille a bien été mise sur le marché puisque la nouvelle société en a vendu à un salon professionnel en octobre 2020. Il n’a été vendu à cette occasion que des bouteilles, dite « bouteille végétalisée » selon le site internet de la nouvelle société avec une publicité sur le stand « la bouteille est recyclable » et non 100 % végétale ou biodégradable. Selon l’analyse réalisée le 25 septembre 2020 par un laboratoire tiers (et donc avant le salon professionnel) les valeurs de migration globale étaient inférieures à la limite fixée par la législation en vigueur. Il s’agissait d’une analyse du produit fini, muni de son bouchon. Le fabricant n’a pas démontré que ce rapport d’alimentarité soit insuffisant ou incomplet pour permettre une mise sur le marché.   Par ailleurs, la nouvelle société a justifié son achat de PLA avec un certificat de conformité à la norme EN 13432 : Caractéristiques des emballages valorisables par compostage et biodégradation – Schémas d’essai et critères d’évaluation pour l’acceptation finale des emballages qui a été délivré le 25 mars 2020 par une société de certification. L’article 1 er  de cette norme précise que, si un emballage est constitué de différents composants, dont certains sont compostables et d’autres non compostables, l’emballage lui-même n’est pas compostable.   Le certificat du 25 mars 2020 n’est relatif qu’à la matière (granulés PLA). Aussi pour faire certifier le produit fini (à savoir la bouteille), une autre certification est nécessaire. Or la nouvelle société ne s’est prévalue de la norme EN 13432 que pour le PLA, et non pour la bouteille entière.   Après avoir de nouveau rappelé que le site internet de la nouvelle société ne permet pas la vente aux consommateurs, les juges d’appel soulignent que la seule vente effective a eu lieu lors d’un salon professionnel. Dans ces conditions, la présentation à visée essentiellement promotionnelle effectuée sur le site internet de la nouvelle société n’était pas de nature à altérer de manière substantielle le comportement économique d’un consommateur normalement informé et raisonnablement attentif et avisé, au sens de l’article L. 121-1, alinéa 2 du Code de la consommation. En conséquence, le fabricant ne démontre pas l’existence de faits de concurrence déloyale par violation des dispositions du Code de la consommation qui auraient été commis par la nouvelle société.   Au final après avoir également examiné d’autres aspects, la cour d’appel a confirmé le jugement de première instance, condamné le fabricant à payer à la nouvelle société et à son associé une indemnité de 8 000 euros sur le fondement de l’article 700 du Code de procédure civile et a rejeté les autres demandes.   L’affaire présentée ci-dessus permet de mettre en évidence qu’il est important, avant de mettre en marché un produit, de s’assurer de sa conformité réglementaire. Dans l’affaire relatée ci-dessus, il s’agissait de matériaux et objets destinés à être mis en contact avec des denrées alimentaires. Le règlement CE n o  1935/2004 du 27 octobre 2004 est donc applicable. Or son article 3, § 1, prévoit une obligation générale d’inertie. Son article 16 prévoit qu’une documentation appropriée doit être disponible pour démontrer la conformité aux exigences fixées (en l’espèce des limites de migration). Le certificat d’alimentarité, répondant à cette exigence, a été obtenu le 25 septembre 2020 et la mise sur le marché est intervenue en octobre 2020 dans le cadre d’un salon professionnel.   L’affaire présentée ci-dessus permet également de mettre en évidence qu’il est important lors de la mise en marché d’un produit de communiquer de manière appropriée au regard des éléments de conformité disponibles. La nouvelle société ne disposait d’un certificat de conformité à la norme EN 13432 que pour le matériau (à savoir les granulés PLA) et non pour le produit fini (à savoir la bouteille constituée à partir de ce matériau). Elle s’est donc abstenue, à juste titre, de communiquer sur le fait, non avéré, que la bouteille soit certifiée. La maîtrise de la communication, notamment sur le site internet de la société, est particulièrement importante en la matière.   Ces deux agissements, que nous qualifierons de raisonnables, ont permis à la nouvelle société de ne pas être condamnée pour pratique commerciale déloyale, comme le lui reprochait le fabricant.
</article>
<article titre="Rama Sithanen : « La population paie cette irresponsabilité outrancière pour Betamax »" date="2021 06 27" journal="Le Mauricien (site web réf.) - Le Mauricien">
Alors que la State Trading Corporation (STC) s’apprête à verser le montant avoisinant les Rs 6 milliards sur le compte de Betamax à la suite du jugement rendu par le...  Voir l'article  Ce document référence un lien URL de site non hébergé par Cision.
</article>
<article titre="Fiche société ALBIOMA" date="2021 06 23" journal="Agence Option Finance (AOF)">
(AOF) -  Points clés - Producteur d'énergie renouvelable -biomasse et photovoltaïque- né en 2013 des fusions successives entre Sidec, la Séchilienne et Albioma-Bois Rouge ;   - Chiffre d'affaires de 6,7 Mds€ réalisé à 41 % en Europe, 25 % dans les Amérique, 12 % en Asie Pacifique et 5 % en Moyen-Orient, Afrique, Russie et 17 % dans les câbles haute tension&amp;projets;  - Production réalisée à 33 % à partir du charbon, à 64 % avec la biomasse et 2 % le solaire; - Stratégie, adoptée en 2012, de réduction de la part du charbon dans la production thermique au profit de la biomasse et du solaire porter à partir des implantations aujourd'hui essentiellement dans l'outre-mer français (1er producteur solaire) puis à l'île Maurice et au Brésil (énergie à base de bagasse, résidu de la canne à sucre); - Capital éclaté avec 3 positions fortes -Invexans et Teck Pack pour 2 !,9 %, BPI France pour 7,7 % et les salariés pour 4,41 %-, Frédéric Moyne étant président-directeur général du conseil d'administration de 8 membres   - Situation financière saine avec une dette brute en baisse à 914 M€, dont 821 M€ de dette projet, face à une trésorerie de 122 M€.    Enjeux - Stratégie 2023visant à agir pour la transition énergétique en outre-mer, déployer l'international et accélérer le développement dans le solaire :    - 450 0 650 M€ d'investissements programmés et financés par emprunts bancaires,    - rationalisation du portefeuille avec cession des activités peu rentables,   - Stratégie d'innovation externalisée auprès du fonds d'amorçage géré par le fonds Demeter Emertec :    - biomasse thermique : recherche de filiales locales,    - stockage d'énergie électrique,    - services au réseau et maintenance prédictive;   - Stratégie environnementale :    - visant à la réduction primaire des émissions polluantes -rejets aqueux, traitement et valorisation sous-produits de combustion et rejets gazeux,   - validée par l'émission d'un « emprunt vert » rémunéré selon des indicateurs clés : +80 % d'énergies renouvelables en 2023 dans la production du groupe et + 95 % en 2030; -  Insensibilité de la rentabilité aux variations de prix des combustibles en raison des indexations contractuelles et visibilité de l'activité par la durée -30 ans- des contrats de vente ;   - Réalisation des projets solaires gouvernementaux remportés lors du 1er semestre pour 20MWc et mise en service de la centrale brésilienne de Vale do Parana  à la fin 2020 .   Défis - Impact négatif de la dépréciation du real brésilien;   - Forte sensibilité aux soutiens financiers publics, aux réglementations sectorielles et à l'accueil du public à de nouvelles centrales;   - Impact de la pandémie : hausse de 1 % du chiffre d'affaires à fin septembre;   - Réaction à la pandémie : économies de coûts, focus sur l'autofinancement et le renforcement des liquidités et la baisse du niveau de risque des projets; - Objectifs 2020 confirmés en base de fourchette d'un résultat d'exploitation entre 200 et 210 M€ et d'un résultat net entre 48 et 54 M€. AOF - EN SAVOIR PLUS Points clés - Producteur d'énergie renouvelable -biomasse et photovoltaïque- né en 2013 des fusions successives entre Sidec, la Séchilienne et Albioma-Bois Rouge ;   - Chiffre d'affaires de 6,7 Mds€ réalisé à 41 % en Europe, 25 % dans les Amérique, 12 % en Asie Pacifique et 5 % en Moyen-Orient, Afrique, Russie et 17 % dans les câbles haute tension&amp;projets;    - Production réalisée à 33 % à partir du charbon, à 64 % avec la biomasse et 2 % le solaire; - Stratégie, adoptée en 2012, de réduction de la part du charbon dans la production thermique au profit de la biomasse et du solaire porter à partir des implantations aujourd'hui essentiellement dans l'outre-mer français (1er producteur solaire) puis à l'île Maurice et au Brésil (énergie à base de bagasse, résidu de la canne à sucre); - Capital éclaté avec 3 positions fortes -Invexans et Teck Pack pour 2 !,9 %, BPI France pour 7,7 % et les salariés pour 4,41 %-, Frédéric Moyne étant président-directeur général du conseil d'administration de 8 membres   - Situation financière saine avec une dette brute en baisse à 914 M€, dont 821 M€ de dette projet, face à une trésorerie de 122 M€.    Enjeux - Stratégie 2023visant à agir pour la transition énergétique en outre-mer, déployer l'international et accélérer le développement dans le solaire :    - 450 0 650 M€ d'investissements programmés et financés par emprunts bancaires,    - rationalisation du portefeuille avec cession des activités peu rentables,   - Stratégie d'innovation externalisée auprès du fonds d'amorçage géré par le fonds Demeter Emertec :    - biomasse thermique : recherche de filiales locales,    - stockage d'énergie électrique,    - services au réseau et maintenance prédictive;   - Stratégie environnementale :    - visant à la réduction primaire des émissions polluantes -rejets aqueux, traitement et valorisation sous-produits de combustion et rejets gazeux,   - validée par l'émission d'un « emprunt vert » rémunéré selon des indicateurs clés : +80 % d'énergies renouvelables en 2023 dans la production du groupe et + 95 % en 2030; -  Insensibilité de la rentabilité aux variations de prix des combustibles en raison des indexations contractuelles et visibilité de l'activité par la durée -30 ans- des contrats de vente ;   - Réalisation des projets solaires gouvernementaux remportés lors du 1er semestre pour 20MWc et mise en service de la centrale brésilienne de Vale do Parana  à la fin 2020 .   Défis - Impact négatif de la dépréciation du real brésilien;   - Forte sensibilité aux soutiens financiers publics, aux réglementations sectorielles et à l'accueil du public à de nouvelles centrales;   - Impact de la pandémie : hausse de 1 % du chiffre d'affaires à fin septembre;   - Réaction à la pandémie : économies de coûts, focus sur l'autofinancement et le renforcement des liquidités et la baisse du niveau de risque des projets; - Objectifs 2020 confirmés en base de fourchette d'un résultat d'exploitation entre 200 et 210 M€ et d'un résultat net entre 48 et 54 M€.
</article>
<article titre="Yannis Fayd’herbe : « Des Assises de la production locale pour revoir les fondamentaux de notre industrie »" date="2021 06 20" journal="Le Mauricien (site web réf.) - Le Mauricien">
Dans cette interview accordée au Mauricien, le président de l’Association of Mauritian Manufacturers, Yannis Fayd’herbe, préconise l’organisation des Assises de la production locale avec le soutien de l...  Voir l'article  Ce document référence un lien URL de site non hébergé par Cision.
</article>
<article titre="Comment les producteurs de rhum composent-ils avec leur insularité ?" date="2021 06 18" journal="Décision Achats">
Le travail d'un responsable des achats dans une entreprise qui fabrique et commercialise des produits finis peut être un véritable cassetête : trouver les matières premières, négocier les prix, jongler avec les fournisseurs dans différents pays, avec différents fuseaux horaires, mettre en place la chaîne logistique pour acheminer les produits jusqu'à leurs marchés de destination C'est compliqué, mais, la plupart du temps, faisable lorsque l'entreprise peut trouver la plupart des produits, des savoir-faire, des fournisseurs dans un environnement raisonnablement proche. Mais que se passe-t-il quand le seul élément dont l'entreprise dispose à proximité est la matière première et qu'elle doit tout importer, oui, tout le reste, avec des distances qui se comptent en milliers de kilomètres ? C'est la problématique que doivent résoudre tous les jours les producteurs de rhum des départements d'outre-mer, en particulier ceux qui sont situés sur des îles ! Le rhum cartonne actuellement (voir notre encadré p. 26), et les ventes explosent sur le marché local, mais surtout sur leur marché principal, la métropole, qui se situe à des milliers de kilomètres du lieu de production. Plus 6 700 km pour la Martinique, un peu moins pour la Guadeloupe et 9 200 km pour La Réunion (à vol d'oiseau) Fort heureusement, les matières premières se trouvent sur place. Dans le cas de la Guadeloupe, cela pourra être de la mélasse ou du jus de canne à sucre (voir notre encadré page suivante pour en savoir plus sur la fabrication du rhum), et pour La Réunion, à 95 % de la mélasse. L'eau est aussi à disposition des producteurs de rhums, même si les deux îles ont récemment connu de graves épisodes de sécheresse. Mais, à part cela, tout doit être importé. Des machines-outils qui servent à traiter et récolter la canne à sucre, aux cuves de stockage en passant par les colonnes de distillation ou les alambics tout ce qui est conçu par une industrie lourde doit être importé, car ni La Réunion, ni la Martinique, ni la Guyane, ni la Guadeloupe n'en disposent sur leur sol.   Et ce n'est pas tout. « Nous manquons aussi de bons techniciens, capables de faire de la maintenance ou de la réparation, aussi, lorsque nous en avons un sous la main nous faisons tout pour le garder, explique Nicolas Legendre qui représente la marque Damoiseau en métropole. Et quand il faut changer une machine, il y a un rapport de un à trois par rapport à la métropole en termes de délais. » Si les deux îles produisent de l'énergie grâce à l'éolien, au solaire, et parfois grâce à la bonification des déchets de l'industrie du rhum (bagasse, vinasses ), les cuves en inox où le rhum se repose après sa distillation ont, elles aussi, été importées, de même que les tonneaux de vieillissement qui accueillent pendant plusieurs années les rhums âgés.   Le choix de l'embouteillage local Lorsque le producteur prend le parti d'embouteiller lui-même ses produits, il doit aussi importer la chaîne de montage et les bouteilles qui arrivent vides par conteneurs depuis l'Europe, où il ne reste plus que quelques verriers. « Chez Damoiseau, nous avons choisi de commercialiser un rhum qui a été intégralement élaboré dans l'aire de production, argumente Nicolas Legendre. Malheureusement, cela a un coût économique, car, ensuite, les bouteilles retournent pleines, en métropole. D'autres préfèrent envoyer leur rhum en vrac, mais cela revient au même, car les cuves aussi arrivent vides en Guadeloupe. » Mais ce n'est pas tout, car, faute de trouver des fournisseurs sur place, Damoiseau doit aussi importer les bouchons (à vis, en plastique, en liège), les surcapsules en plastique ou en étain, et même les étiquettes ! « Concernant les étiquettes, nous ne pouvons pas faire de stocks trop importants, car ce sont des produits qui se conservent mal sous les tropiques (fortes chaleurs et humidité) et les rouleaux finissent par s'altérer au bout de plusieurs années », prévient Nicolas Legendre. Et une fois les bouteilles dument remplies, étiquetées, bouchées rassemblées dans des cartons (le seul élément avec le rhum qui est produit sur place), elles sont envoyées en métropole via la ligne Pointe-à-Pitre/Le Havre, puis elles sont convoyées à Valenton, dans le Val-de-Marne. De cet entrepôt, les volumes sont redispatchés en France et à l'international, y compris les États-Unis, alors qu'ils sont assez proches de la Guadeloupe. Car, d'une part, il n'y a pas de liaison maritime régulière avec les États-Unis et, d'autre part, il n'y a souvent pas la volumétrie suffisante pour remplir un conteneur complet en Guadeloupe. Malgré ces obstacles, la marque guadeloupéenne continue d'embouteiller ses rhums sur place : les clients donnent une prime aux produits élaborés à 100 % dans l'aire de production, et cela lui permet aussi de mieux maîtriser sa production.   La solution du vrac Mettons maintenant le cap sur La Réunion, pour voir comment la marque Isautier fait face aux mêmes obstacles en pire ! Car non seulement on ne trouve pas de fabricant de machines, colonne à distiller, bouteilles, étiquettes haut de gamme et autres bouchons à La Réunion mais en plus l'île est située encore plus loin de la métropole. « Un bateau en provenance de la métropole met un mois pour rallier La Réunion », nous apprend Cyril Isautier, le dirigeant de la marque éponyme. Contre environ deux semaines pour la Guadeloupe. Et ce n'est qu'à l'aller, car, au retour, le délai voit double ! « La Réunion est à 90 % importatrice, donc, pour 100 conteneurs qui arrivent ici, 90 repartent vides et 10 avec de la marchandise locale. » Pour revenir en métropole avec des conteneurs pleins, les navires vont passer par l'Inde, « ce qui rallonge le trajet de retour (hors crise Covid ou canal de Suez), de deux mois pour remonter », explique Cyril Isautier. Pendant longtemps, la marque réunionnaise devait donc jongler avec des contraintes handicapantes : elle faisait descendre des bouteilles vides à La Réunion, où Isautier les stockait, avant de les envoyer une fois remplies vers la métropole. Au vu des délais et de la nécessité d'être réactifs face à l'imprévu, la marque était obligée de constituer des stocks de matière première à La Réunion et de produits finis en métropole. Ce qui impliquait des dépenses d'immobilisation colossales, un coût environnemental insensé et une aberration commerciale, car, malgré cela, la réactivité était, au mieux, aléatoire. De plus, lors du voyage retour qui durait deux mois, il y avait fréquemment des problèmes de dégradation des packagings : étiquette, colle « Donc, depuis 2019, nous confions progressivement le processus d'embouteillage à une ou à des entreprises spécialisées, révèle Cyril Isautier. Grâce à cela, nous avons fait baisser nos coûts financiers et écologiques, et nous pouvons être plus réactifs pour répondre à une demande qui explose, notamment pour les rhums arrangés. ». FABIEN HUMBERT   Focus Comment produit-on un rhum blanc?   Les rhums des DOM sont classés dans deux catégories différentes, selon qu'ils ont été produits à partir de jus de canne ou de mélasse. Le jus de canne est obtenu en pressant des tiges de canne à sucre fraîches dans un moulin. Le jus qui en ressort (le vesou) est mis à fermenter jusqu'à obtenir un "vin de canne" titrant autour de 7° d'alcool. C'est ce vesou qui sera distillé dans des colonnes de distillation. Transformé en vapeur, concentrée, puis refroidie, le vesou devient un spiritueux qui titre environ 65°-70°. Dans la plupart des cas, de l'eau sera ajoutée au rhum pour faire descendre ce   degré à 50° ou 40°. Les rhums de jus de canne des DOM sont appelés "agricoles" et sont principalement produits par la Martinique et la Guadeloupe. Pour le rhum de mélasse, le jus de canne est transformé en sucre dans une sucrerie. À côté du sucre non raffiné, les producteurs obtiennent une matière visqueuse et sucrée, la mélasse. C'est elle qui sera mise à fermenter (avec de l'eau), puis distillée. Il en ressort des rhums titrant en général autour de 90°. La Guadeloupe et surtout La Réunion sont les championnes françaises des rhums de sucrerie, ou de mélasse.   Focus Ventes de rhum : une année 2020 paradoxale Malgré le confinement et la crise de la Covid-19, l'année 2020 a été faste, en tout cas pour ce qui concerne les ventes en grande distribution. Elles ont en effet progressé de 6,5% au global, avec 31537000 litres écoulés. Les rhums épicés (+28,3% à 5261000 litres), vieux (+20,3% à 1743000) et arrangés (+60,7% à 1774 litres) ont clairement tiré le marché vers le haut : côté CHR (cafés/cavistes, hôtels, restaurants), les ventes ont chuté dans les établissements qui étaient restés fermés pendant les confinements, mais les cavistes s'en sortent bien. In fine, la baisse est tout de même de -44,7%, soit 3740066 litres commercialisés.
</article>
<article titre="Entreprises : Dégringolade de la Mon Trésor Smart City" date="2021 06 18" journal="Le Mauricien (site web réf.) - Le Mauricien">
PHOTO ILLUSTRATION @Montresor.mu – Omnicane se recentre sur le sucre et l’énergie – Spéculations persistantes autour d’une éventuelle vente de Holiday Inn Confronté à d’énormes difficultés financières et un...  Voir l'article  Ce document référence un lien URL de site non hébergé par Cision.
</article>
<article titre="Les pistes d'investissement Oubliées Acheter les valeurs en retard" date="2021 06 12" journal="Investir-Le Journal des finances, no. 2475">
Il n'est pas facile de suivre le rythme effréné du Cac 40 depuis le début de l'année. L'indice a déjà bondi de près de 19 %, tutoyant les 6.600 points, tiré notamment par la première capitalisation boursière européenne,LVMH, dont le titre s'est encore adjugé 31 % depuis le 1Er janvier. Les autres valeurs du luxe sont également très bien orientées, ainsi que le secteur bancaire et les matériaux de construction. S'il est bien sûr toujours intéressant de détenir en portefeuille des valeurs appartenant à ces secteurs, il est aussi opportun de regarder les titres qui n'ont pas encore profité de la bonne tenue du marché. Je vous propose dix sociétés en retard, à acheter pour compléter votre portefeuille.    Alstom (- 3 %)   L'équipementier ferroviaire accuse un repli de 3 % cette année. Or Bouygues vient de terminer de céder ses actions Alstom, si bien que la pression vendeuse disparaît. Le groupe de Martin Bouygues ne détient plus qu'une part symbolique, de 0,16 %, du capital. En outre, les provisions pour risque sur les contrats deBombardier ont toutes été comptabilisées. Le montant dépasse, certes, le milliard d'euros, mais Henri PoupartLafarge, le PDG d'Alstom, a promis que ces charges exceptionnelles étaient suffisantes. Alstom peut donc repartir sur des bases saines et profiter pleinement des plans de relance en Europe, qui font la part belle à la mobilité verte.    Iliad (- 20 %)   L'opérateur a suspendu son objectif de cash-flow opérationnel pour la France, qui était d'environ 900 millions d'euros pour 2021 hors activités BtoB, car il souhaite investir davantage dans le déploiement de la 5G. Cette décision a inquiété les investisseurs, car l'endettement d'Iliad est déjà élevé. Mais la croissance de l'activité demeure dynamique et permet d'améliorer de 20 % par an le bénéfice net par action pendant plusieurs exercices.    Bouygues (- 1 %)   Bouygues Telecom a revu à la hausse son objectif de marge sur excédent brut d'exploitation pour 2021, qui devrait augmenter de 7 % au lieu des 5 % précédemment anticipés. Par ailleurs, le marché de la construction repart, et le projet de rapprochement entreTF1 etM6 devrait créer de la valeur en 2022. Or le PER 2021 de Bouygues atteint à peine 12 fois.    Air France-KLM (- 10 %)   Le secteur de l'aéronautique, et du transport aérien au sens large, se reprend depuis le début de l'année.Airbus,Safran,ADP mais aussiEasyJet etRyanair sont, au pire, stables ou progressent nettement pour certains (+ 25 % pour Airbus). Seul Air France-KLM reste dans le rouge, victime d'une gestion désastreuse de ses coûts. Mais tout a un prix. Si le transport aérien confirme sa reprise, l'ancien champion tricolore mérite un meilleur parcours boursier.    Korian (- 2 %)   La médiatisation des décès dans les Ehpad pendant la crise sanitaire, une augmentation de capital réalisée à vil prix et une dette élevée, dans un contexte de hausse des taux d'intérêt, ont mis à genoux la valeur. Incapable de maintenir sur la durée des performances régulières, commeOrpea, le titre souffre d'une décote de 21 % par rapport au meilleur élève du secteur, ce qui est exagéré.    Pharmagest Interactive (- 20 %)   Le spécialiste de l'informatisation des pharmacies, qui déploie des solutions uniques assurant la coordination entre les différents acteurs de la santé, devrait afficher une progression de ses ventes supérieure à 10 %. Non endetté, il compte accélérer les acquisitions. Son seul défaut réside dans le fait de ne pas être une valeur cyclique.    Sanofi (+ 12 %)   Le titre est, certes, en hausse de 12 % depuis le début de l'année, mais il perd 2 % sur un an, alors que le Cac 40 gagne 29 %. Or Sanofi est en avance, au premier trimestre, sur son objectif annuel de croissance du bénéfice par action, se situant dans le haut de la fourchette de 5 % à 10 % visée. Le laboratoire, de plus, devrait profiter d'une base de comparaison favorable au deuxième trimestre. Il présente une décote de près de 15 % par rapport au secteur et offre un rendement confortable.    Albioma (- 29 %)   Producteur d'électricité présent dans les DOM et au Brésil, ainsi qu'en France métropolitaine pour le solaire, Albioma exploite des centrales thermiques de cogénération adossées à des sucreries, qui brûlent de la bagasse (le résidu de la canne à sucre) pendant les campagnes sucrières et du charbon le reste de l'année. D'ici à la fin de 2025, le mix énergétique renouvelable dépassera 90 %, grâce à la conversion des centrales à la biomasse en remplacement du charbon. Comme ses consoeurs du secteur, la valeur a fait l'objet de prises de bénéfices après les envolées de 2020, car les investisseurs ont préféré se placer sur des titres plus cycliques et moins exposés aux taux d'intérêt. Mais le repli me semble trop marqué.    Atos (- 28 %)   A moins de 8 fois le bénéfice estimé pour 2021, il convient d'être à l'achat de ce groupe en panne de croissance. Avec un peu de patience, le nouveau plan d'action permettra à Atos d'accélérer enfin et de retrouver du dynamisme.    Edenred (+ 3 %)   Le champion des solutions de paiement pour les salariés va profiter de la réouverture des économies et d'un effet de base très favorable au deuxième trimestre. L'utilisation des Ticket Restaurant, que les salariés avaient conservés au chaud, va permettre au groupe de retrouver une belle dynamique. La prévision de croissance interne de l'Ebitda qu'a donnée Edenred pour 2021 (au moins 6 %) est jugée très prudente. Le consensus table déjà sur une hausse de 10 %, ce qui semble encore modeste, selon Deutsche Bank (lire aussi le « Conseil de la semaine », p. 11).
</article>
<article titre="A la Réunion, l'autonomie énergétique condense les débats" date="2021 06 7" journal="Libération">
Certains dénoncent un nouvel épisode de greenwashing, d'autres reconnaissent un véritable virage vert. En tout cas, Didier Robert, président sortant du conseil régional - qui se représente malgré des déboires judiciaires (lire encadré) et un projet pharaonique, inachevé, de route sur la mer - redouble d'énergie. L'élu divers droite (Objectif Réunion) a annoncé en novembre que «près de 100 % de l'électricité de l'île» sera produite «dès 2023» à partir d'énergies renouvelables. «Une avancée phénoménale», défend celui qui est soutenu par le parti Les Républicains (LR) et la moitié des maires de la Réunion. «Un petit bout de chemin», relativise Vanessa Miranville, maire de La Possession, «écolo pragmatique» qui se présente aux régionales à la tête de son mouvement citoyen Créa. Et de rappeler que «près de 70 % de notre facture énergétique est due aux transports, pas à l'électricité. Il faut plus d'ambition». A ce jour, 30 % des quelque 3 000 GWh (gigawatt-heure) produits dans le département (1) sont issus d'énergies renouvelables - des barrages hydroélectriques d'EDF (13,7 %), du photovoltaïque (8,5%) et de la combustion de la bagasse, un déchet de la canne à sucre (7,9%), selon l'Observatoire énergie Réunion. Le reste? Il provient d'énergies fossiles importées d'Afrique du Sud ou de Singapour : fioul, gazole, charbon, huiles usagées. Territoire insulaire, la Réunion n'est pas interconnectée au réseau national d'EDF et ne bénéficie pas de l'électricité nucléaire. D'où un coût de production deux fois plus élevé qu'en métropole, mais des tarifs identiques pour les consommateurs. Comment le président de la collectivité   compte-t-il parvenir à cette électricité verte, lui qui avait mis fin au programme Gerri (Green Energy Revolution Reunion Island)? Elu pour la première fois en 2010 au conseil régional, Didier Robert avait en effet coupé les financements de ce projet ambitieux porté par son prédécesseur communiste, Paul Vergès. Oubliée la production d'électricité à partir de la chaleur du volcan ou de la houle de l'océan Indien, oublié également le réseau de climatisation alimenté par les eaux froides et profondes, regrette aujourd'hui encore le maire de Sainte-Suzanne, Maurice Gironcel, dernier édile communiste de l'île. Son ancienne camarade Huguette Bello, maire de Saint-Paul, éprouve la même nostalgie. Si la candidate divers gauche du mouvement Pour la Réunion, députée de 1997 à 2020 sur les bancs communistes, est élue, elle promet de recréer «une structure sur le modèle initié par le groupement Gerri».   «HORS-LA-LOI CLIMATIQUE Robert compte donc sur EDF-Production électrique insulaire (filiale de l'électricien français) et Albioma, tous deux bien implantés sur l'île, pour ses projets verts. Le second s'est engagé à transformer ses deux centrales thermiques : fin 2023, les fours de Bois-Rouge d'abord, puis du Gol, ne brûleront plus 600000 tonnes de charbon importées chaque année mais près de 750 000 tonnes de granulés de bois. Problème: seulement 100000 tonnes de cette biomasse peuvent être pour l'instant d'origine réunionnaise. Même si un partenariat a été signé avec l'Office national des forêts (ONF) pour collecter de l'acacia et du cryptomeria (cèdre du Japon), il faudra donc importer. «Nous préférerions compléter par un approvisionnement régional, en Afrique ou en Asie, assure Pascal Langeron, directeur général adjoint d'Albioma. Mais il faut que les forêts exploitées le soient durablement et que les arrivées soient sécurisées. Sinon, on risquerait des pannes d'électricité !» Pour ces raisons, Albioma s'apprête à faire appel à la société américaine Enviva, qui l'approvisionne déjà aux Antilles. Les arbres seront coupés en Amérique du Nord et les granulés arriveront par bateau, après trois semaines de mer. «Un scandale», selon Jean-Pierre Marchau, candidat aux régionales d'Europe Ecologie-les Verts (EE-LV). «La combustion du bois n'est pas neutre et émet même plus de CO2 que celle du charbon», déplore-t-il. Par ailleurs, le conseiller municipal de Saint-Denis rappelle qu'Albioma est «au coeur d'une intense polémique pour son exploitation intensive des forêts naturelles des Etats- Unis». Certes, le bois utilisé est, selon Enviva, certifié FSC ou PEFC, écolabels attestant d'une gestion durable de la ressource; pour autant, plusieurs ONG internationales demandent de «suspendre le projet réunionnais tant qu'il n'y a pas d'étude d'impact». L'Autorité environnementale a d'ailleurs émis de fortes réserves sur la Programma- tion pluriannuelle de l'énergie (PPE) de la Réunion pour la période 2019-2028 dans un délibéré daté du 5 mai: «Le choix de l'importation massive de biomasse soulève des questions majeures sur son impact environnemental», estime l'instance indépendante.   A la région, Didier Robert s'enflamme : «Vaut-il mieux faire venir des tankers et brûler du fioul ou du charbon ?» Toujours est-il que la collectivité ne respectera pas la loi relative à la transition énergétique pour la croissance verte. Le texte de 2015 fixe comme objectif l'autodeux nomie énergétique dans les départements d'outre-mer à l'horizon 2030. «La Réunion va devenir un hors-laloi climatique», accuse Jean-Pierre Marchau (EE-LV). L'électricité sera certes renouvelable, mais importée; quant aux carburants nécessaires aux transports, qui représentent deux tiers de l'énergie consommée, ce seront toujours des dérivés du pétrole, importés eux aussi.   «SOLUTION DE LA PARESSE» Du côté d'EDF-PEI, ce n'est guère mieux. La filiale d'EDF doit elle aussi convertir sa centrale thermique du Port-Est, qui couvre 35% des besoins électriques de l'île. Les 200000 tonnes de fioul lourd importées doivent être remplacées avant fin 2023 par une quantité semblable d'huiles de colza et de tournesol. Cette biomasse liquide proviendra «très probablement de France», indique Alexandre Sengelin, directeur d'EDF-PEI, car «peu de pays respectent les critères européens de durabilité». La combustion de ces huiles ne produira plus de soufre ni de métaux lourds, mais toujours du dioxyde de carbone. Ces émissions seront compensées parle CO2 que captureront les oléagineux durant leur croissance en métropole. Quant aux turbines à combustion d'EDF, elles continueront à fonctionner au gazole comme «moyens de secours». Ce schéma laisse pour le moins dubitatif Patrick Lebreton, maire de Saint-Joseph, une commune rurale. L'ancien député socialiste, qui mène lui aussi sa propre liste, parle de «leurre». Ericka Bareigts, la candidate officielle du PS à ces régionales, donnée favorite, est plus sévère encore. L'ancienne ministre des Outre-mer de François Hollande connaît bien le sujet : elle avait rendu, en 2014, alors députée, un rapport sur les énergies en outremer, dont s'inspire la loi de 2015 sur la transition énergétique. «Après mandats, Didier Robert n'a pas fait de travail de fond, assène la maire de Saint-Denis. Il a choisi la solution de la paresse et de la facilité, avec l'importation de biomasse. Le solaire est en pâturage, l'éolien anecdotique, et rien n'a été entrepris pour réduire la consommation électrique.» Si elle l'emporte, Ericka Bareigts compte amender la PPE de la région pour booster les énergies renouvelables locales, notamment l'autoconsommation solaire, «trop onéreuse aujourd'hui pour les particuliers». La candidate milite également pour la construction de bâtiments bioclimatiques, la maîtrise de l'énergie, le développement des mobilités douces. Et de rappeler à ce sujet que la Réunion est l'une des rares régions de France à ne pas avoir abondé la prime de l'Etat à l'achat d'un vélo électrique: «L'énergie verte est un sujet politique, conclut-elle. Cela a un coût, il faut l'assumer.» ? (1) La Réunion est à la fois un département et une région, avec deux conseils distincts.   UN PRÉSIDENT À LA PEINE Didier Robert a été condamné le 21 mai à trois ans inéligibilité et quinze mois de prison avec sursis pour «prise illégale d'intérêts», «abus de bien social», «omission de déclaration de revenus à la Haute Autorité pour la transparence de la vie publique». L'élu assure qu'il n'a «jamais voulu contourner la loi». Le Journal de l'île a rendu public samedi un rapport de la chambre régionale des comptes, selon lequel Robert emploie 22 collaborateurs, alors que seuls 6 sont autorisés. La collectivité déclare que la situation de ses agents est «conforme à l'état du droit».
</article>
<article titre="A La Réunion, des projets de conversion des centrales à charbon contestés La feuille de route énergétique de ce département d'outre-mer est critiquée par des ONG pour qui la substitution du charbon par de la biomasse est un « pari perdant pour le climat »" date="2021 06 5" journal="Le Monde">
Saint-Denis (La Réunion) correspondance - Lorsqu'il a été adopté fin 2020, le projet de révision de la programmation pluriannuelle de l'énergie (PPE) pour La Réunion a été salué comme marquant une avancée dans la lutte contre le dérèglementclimatique. Cette feuille de route prévoit notamment la conversion des centrales à charbon et des turbines au fioul lourd, qui fournissent respectivement 44 % et 33 % de l'électricité de l'île, en centrales biomasse. « Un pas de géant va être franchi », avait alors estimé le président du conseil régional, Didier Robert (divers droite). Quelques mois plus tard, cette stratégie est de plus en plus contestée.   Selon la PPE, les centrales de Bois-Rouge, à Saint-André, et du Gol, à Saint-Louis, détenues par l'opérateur privé Albioma,ne brûleront plus de charbon venant d'Afrique d'ici à 2023-2024, mais de la biomasse importée et de la biomasse locale, constituée de bagasse (les résidus fibreux de la canne à sucre) et de déchets verts et de bois. Au Port-Est, les douze moteurs de la centrale d'EDF seront alimentés par du biocarburant fabriqué en Europe à partir de colza et de soja, et non plus par du fioul acheminé par pétroliers d'Arabie saoudite. Pour l'ONG Reclaim Finance, ce choix n'est pas le bon : la substitution du charbon par de la biomasse est « un pari perdant pour le climat et la biodiversité », assure-t-elle. Fin mars, elle a publié avec dix-huit autres ONG une lettre ouverte aux banques publiques et privées les appelant à ne « pas soutenir » ces projets. Les signataires s'appuient sur les travaux de scientifiques montrant que la combustion de bois peut entraîner à court terme un accroissement des émissions de gaz à effet de serre, et donc générer une « dette carbone non remboursable dans un délai acceptable .   « Il vaudrait donc mieux rester au charbon ?, interroge Pascal Langeron, directeur général adjoint d'Albioma et responsable des sites de La Réunion et de Mayotte. Vous vous rendez compte, cela signifie qu'il faudrait continuer à sortir du carbone qui a été stocké il y a des millions d'années dans le sol, le brûler et émettre du CO2. » Ce dirigeant affirme que la biomasse utilisée par Albioma sera « issue de forêts exploitées durablement . Plutôt que de se fournir aux Etats-Unis ou en Europe,l'entreprise ditvouloir « régionaliser » ses sources d'approvisionnement en se tournant vers l'Afrique du Sud ou le Mozambique, soit à cinq à dix jours de bateau de La Réunion.   « Scandale de l'importation » « Les énergies éolienne ou hydraulique sont sûrement plus vertes, mais la biomasse est l'une des sources pour assurer la transition, insiste Pascal Langeron. Faudrait-il maintenir le charbon pendant vingt ans, le temps que l'on atteigne 100 % d'énergies renouvelables locales ? » Reclaim Finance, de son côté, estime que l'importation de granulés de bois d'Afrique australe constituerait une « menace pour les écosystèmes et les populations locales », avec des impacts négatifs pour les ressources en eau.   La conversion du charbon à la biomasse est également sévèrement critiquée pour ne pas respecter l'objectif d'autonomie énergétique pour 2030. A la place des 600 000 tonnes de charbon par usine, Albioma va importer quelque 700 000 tonnes de biomasse solide. Dans ce nouveau schéma, 100 000 tonnes par an de biomasse locale ont été identifiées, en plus de la bagasse.   Dans son avis sur la PPE de La Réunion rendu le 5 mai, l'autorité environnementale (AE) estime que « la production énergétique voit sa dépendance au regard des combustibles fossiles remplacée par une dépendance à la biomasse importée . Au sujet de la biomasse locale, il existe un manque d'informations sur les risques de « mobilisation des déchets agricoles ou forestiers et l'exploitation intense du couvert forestier », écrit-elle. Par ailleurs, cette autorité indépendante juge regrettable que des solutions avancées par l'Agence de la transition écologique (Ademe), basées sur davantage d'énergie photovoltaïque et de ressources hydroélectriques, aient été « écartées .   A la suite de ces critiques, le thème de l'énergie devrait s'inviter dans la campagne des élections régionales pour attaquer le bilan du président sortant. Le candidat écologiste Jean-Pierre Marchau, l'un des rares à avoir contesté le contenu de la PPE, n'hésite pas à parler de « scandale de l'importation de la biomasse à La Réunion » allant « à contresens des objectifs climatiques . Après l'avis de l'AE, la maire de Saint-Denis, Ericka Bareigts (divers gauche), s'est à son tour emparée du sujet en fustigeant une « stratégie énergétique bancale .
</article>
<article titre="(Secteur cannier) Jacqueline Sauzier : « La rémunération de la  bagasse viendra soulager les planteurs »" date="2021 06 4" journal="Le Mauricien (site web réf.) - Le Mauricien">
La campagne sucrière 2021 démarre le 14 juin avec la mise en route de l’usine d’Alteo à Union Flacq. Terra dans le secteur Nord et Omnicane dans le Sud...  Voir l'article  Ce document référence un lien URL de site non hébergé par Cision.
</article>
<article titre="Consejo Regulador del Tequila : Le Mexique et l'industrie de la tequila visent un secteur plus durable" date="2021 05 13" journal="PR Newswire Europe (français)">
PARIS, 13 mai 2021 /PRNewswire/ -- Le Consejo Regulador del Tequila (CRT) - l'organisation qui représente les producteurs mexicains de tequila - et la Cámara Nacional de la Industria Tequilera (CNIT) lancent un nouveau label pour la tequila avec le gouvernement de Jalisco et le ministère de l'Environnement et du Développement territorial (Semadet) : l'Agave Responsible Environmental Certification (ARA) pour établir un protocole pour la traçabilité de la tequila, qui vise à lutter contre la déforestation, réduire l'impact des plantations sur l'environnement et l'empreinte écologique.   Ces progrès sont conformes à l'Agenda 2030 des Nations Unies et à l'engagement de l'Union européenne à promouvoir le développement durable dans tous les États membres par la réalisation des Objectifs de développement durable. Plus précisément, grâce à sa certification ARA, la tequila s'engage à lutter contre la désertification et à stopper et inverser la dégradation des terres et la perte de biodiversité. Alfaro Ramirez, gouverneur de Jalisco : «  La décision que nous avons prise et l'engagement que nous avons pris avec la ville de Madrid en 2019 dans le cadre de la COP25 témoignent de notre détermination à atteindre notre objectif avec le CRT et le CNIT. La tequila sera la première boisson alcoolisée au monde à être certifiée sans déforestation. Nous sommes très fiers de présenter les résultats de notre engagement à la COP26 en novembre . »   «  Cette certification vient compléter les actions déjà menées par l'agrobusiness de la tequila en matière de durabilité, telles que les investissements dans les usines de transformation de la vinasse, le compostage de la bagasse, l'utilisation du biogaz et les engagements de réduction des émissions de carbone, l'électricité, la purification de l'eau, établis en collaboration avec le Centre Mario Molina pour 2030 ", déclare le président de la CRT, Miguel Angel Dominguez Morales.   La certification ARA prévoit un protocole qui implique la création d'une carte des terres éligibles à l'agave, établie par le gouvernement de Jalisco, Semadet (une organisation de conservation) et la Commission nationale des forêts. Cela montre où les plantations d'agave peuvent être établies, ce qui empêchera la déforestation. La zone éligible couvre près de 3 millions d'hectares à Jalisco, à l'exclusion de toutes les parcelles ayant une végétation forestière en 2016. La carte est publiée dans le cadre des exigences fixées par le CRT pour les nouvelles plantations.   L'objectif est que d'ici 2027, la fourniture de la variété bleue Agave tequilana Weber pour produire de la tequila à Jalisco n'aura pas conduit à la déforestation des forêts naturelles. Cela signifie qu'à partir de 2021, aucune nouvelle plantation d'agave ne sera enregistrée sur des terres déboisées.           Contact  Natasha Vanesa Martin Ortiz, natasha.martin@newlink-group.com , T +34 628 58 61 76 et Maria Garcia Forte maria.garcia@newlink-group.com
</article>
<article titre="Léogâne, faut-il revenir à la canne à sucre ?" date="2021 05 11" journal="Le Nouvelliste (Haïti) (site web réf.) - Le Nouvelliste en Haiti">
Gary Alliance, spécialiste en production végétale, a défendu le pour. Il a mis en avant les vertus écologique, agricole, économique et nutritionnelle de la canne à sucre pour défendre sa position...  Voir l'article  Ce document référence un lien URL de site non hébergé par Cision.
</article>
<article titre="ALBIOMA cible d'une cyberattaque" date="2021 05 10" journal="Agence Option Finance (AOF)">
(AOF) - Vendredi dernier au soir, Albioma a indiqué qu'une attaque virale de type " rançongiciel " avait été détectée sur son réseau informatique bureautique le mardi 4 mai dernier. Le diagnostic complet est en cours de finalisation; aucune fuite de données n'a été détectée pour le moment. L'attaque n'a pas eu de conséquence à ce jour sur l'activité industrielle d'exploitation, toutes les centrales du groupe fonctionnent normalement et ont été isolées du réseau bureautique concerné par l'intrusion.   AOF - EN SAVOIR PLUS Points clés - Producteur d'énergie renouvelable -biomasse et photovoltaïque- né en 2013 des fusions successives entre Sidec, la Séchilienne et Albioma-Bois Rouge ;   - Chiffre d'affaires de 6,7 Mds€ réalisé à 41 % en Europe, 25 % dans les Amérique, 12 % en Asie Pacifique et 5 % en Moyen-Orient, Afrique, Russie et 17 % dans les câbles haute tension&amp;projets;    - Production réalisée à 33 % à partir du charbon, à 64 % avec la biomasse et 2 % le solaire; - Stratégie, adoptée en 2012, de réduction de la part du charbon dans la production thermique au profit de la biomasse et du solaire porter à partir des implantations aujourd'hui essentiellement dans l'outre-mer français (1er producteur solaire) puis à l'île Maurice et au Brésil (énergie à base de bagasse, résidu de la canne à sucre); - Capital éclaté avec 3 positions fortes -Invexans et Teck Pack pour 2 !,9 %, BPI France pour 7,7 % et les salariés pour 4,41 %-, Frédéric Moyne étant président-directeur général du conseil d'administration de 8 membres   - Situation financière saine avec une dette brute en baisse à 914 M€, dont 821 M€ de dette projet, face à une trésorerie de 122 M€.    Enjeux - Stratégie 2023visant à agir pour la transition énergétique en outre-mer, déployer l'international et accélérer le développement dans le solaire :    - 450 0 650 M€ d'investissements programmés et financés par emprunts bancaires,    - rationalisation du portefeuille avec cession des activités peu rentables,   - Stratégie d'innovation externalisée auprès du fonds d'amorçage géré par le fonds Demeter Emertec :    - biomasse thermique : recherche de filiales locales,    - stockage d'énergie électrique,    - services au réseau et maintenance prédictive;   - Stratégie environnementale :    - visant à la réduction primaire des émissions polluantes -rejets aqueux, traitement et valorisation sous-produits de combustion et rejets gazeux,   - validée par l'émission d'un « emprunt vert » rémunéré selon des indicateurs clés : +80 % d'énergies renouvelables en 2023 dans la production du groupe et + 95 % en 2030; -  Insensibilité de la rentabilité aux variations de prix des combustibles en raison des indexations contractuelles et visibilité de l'activité par la durée -30 ans- des contrats de vente ;   - Réalisation des projets solaires gouvernementaux remportés lors du 1er semestre pour 20MWc et mise en service de la centrale brésilienne de Vale do Parana  à la fin 2020 .   Défis - Impact négatif de la dépréciation du real brésilien;   - Forte sensibilité aux soutiens financiers publics, aux réglementations sectorielles et à l'accueil du public à de nouvelles centrales;   - Impact de la pandémie : hausse de 1 % du chiffre d'affaires à fin septembre;   - Réaction à la pandémie : économies de coûts, focus sur l'autofinancement et le renforcement des liquidités et la baisse du niveau de risque des projets; - Objectifs 2020 confirmés en base de fourchette d'un résultat d'exploitation entre 200 et 210 M€ et d'un résultat net entre 48 et 54 M€.
</article>
<article titre="« Valeurs de croissance et cycliques, marchez sur deux pieds en Bourse »" date="2021 05 6" journal="Investir (site web)">
Rémi Le Bailly:  Très heureux de vous retrouver pour ce chat.   MADA: Bonjour Rémi. Abonné à Investir, je profite de vos conseils éclairés et vous en remercie. J'ai tout de même une question : mon PEA, axé sur les valeurs du luxe, a bien progressé à tel point que ces valeurs représentent 60% du PEA. Comment, quand et avec quelles valeurs rééquilibrer ce portefeuille pour le long terme. Avec mes remerciements. C'est la rançon du succès. Le secteur du luxe a tellement monté qu'il a pris du poids dans votre portefeuille. Tant mieux pour vous.   Cela dit, 60% sur un seul secteur me semble excessif. Une mauvaise nouvelle sur le secteur peut provoquer de fortes moins-values. Au sein du Cac 40, le poids des quatre Khol (Kering, Hermès, L'Oréal et LVMH) est d'environ 23%.   Sans descendre forcément aussi bas, je pense qu'il faut ramener ce poids à maximum 30 ou 40%.   Le luxe est un secteur de croissance, je vous conseille de rééquilibrer votre portefeuille avec des titres plus cycliques. C'est ce que nous avons fait dans nos sélections Investir 10 dont vous pouvez vous inspirer.   Vous pouvez, par exemple, acheter Aperam dans les métaux, Solvay ou Arkema dans la chimie ou encore des financières comme Axa ou BNP Paribas.   Il faut actuellement avoir des valeurs de croissance (luxe, certaines valeurs de technologie comme Teleperformance, Capgemini...) et des valeurs cycliques. C'est toujours mieux de marcher sur deux pieds !   BOURVIL: Bonjour. Je vous repose une question posée il y a plusieurs semaines et non retenue, qui est totalement d'actualité aujourd'hui. Solutions 30 a demandé à suspendre sa cotation. L'une des raisons possibles est l'annonce d'un retrait de la cote compte tenu de l'acharnement du fonds Muddy Waters. Un retrait doit-il se faire automatiquement à un prix supérieur au cours de cotation ? Dans ce cas, une telle décision serait favorable aux actionnaires entrés récemment sur l'action... Merci.   Difficile de savoir ce qui va se passer. Il y a aujourd'hui des hypothèses dans tous les sens après la suspension de cotation dans l'attente d'un communiqué de la société. Nous avons publié un article sur le site résumant un peu tout ce qui circule.   S'il s'agit d'un retrait de la cote demandé par la société, il est peu probable qu'il se fasse en-dessous du cours de Bourse car les actionnaires n'apporteraient pas leurs titres... et donc il ne pourrait pas y avoir de retrait.   Mais on peut aussi imaginer d'autres hypothèses. J'en profite aussi pour répondre à Olivier qui demande sur quoi repose notre conseil d'achat. La réponse est tout simplement la croissance du groupe et son évolution sur un marché en très fort développement. Nous sommes redevenus positifs quand l'audit indépendant demandé à Deloitte a conclu que les accusations portées contre l'entreprise étaient globalement infondées. Pour nous, cela levait les soupçons pesant sur la gestion du groupe.   Ratatouille: Bonjour Monsieur Le Bailly. J'envisage d'acheter des lots de parkings. J'ai les fonds pour, mais ne serait-il pas plus judicieux de faire un crédit sur une partie du prix d'achat pour profiter des taux bas et du fait que la location va me rapporter plus que le coût d'emprunt ? Merci.   De Ratatouille, j'aurais plutôt attendu une question portant sur les murs de restaurant mais je suis d'accord, les parkings sont actuellement plus sûrs.   Les parkings étant un bien immobilier, vous devez pouvoir financer cela par l'emprunt à condition que l'investissement soit suffisant, il faut donc, comme vous le précisez, opter pour un lot de parkings. Si possible, je vous conseille de diversifier en retenant différentes localisations pour diversifier les risques.   La rentabilité des parkings étant élevée, les revenus devraient, effectivement, couvrir le coût de l'emprunt.   THEO: Bonjour. En dehors de la Bourse et de l'assurance-vie, j'aimerais savoir sur quel type de produit placer une somme importante sans risque ni trop de frais pour une durée de 2 à 3 ans. Merci, cordialement.   Il n'y a pas de placement sans risque en dehors du Livret A et de l'assurance-vie en euros (et encore, on peut considérer que le risque n'est pas totalement nul dans ce dernier domaine). La Bourse sur 2-3 ans est également risquée (imaginez un achat en 2017 et une sortie en mars 2020 !).   Je n'ai pas de placement miracle à vous donner. On peut, à la rigueur, penser aux fonds obligataires à échéance en retenant une maturité courte à deux-trois ans, mais il y a le risque de défaillance d'un émetteur.   On peut aussi penser aux fonds à formule indexés sur l'évolution de la Bourse, qui peuvent rapporter du 4-5% par an, mais qui présentent, aussi, un risque si les marchés décrochent.   Jacquou: Bonjour Monsieur Le Bailly. Il était bien prévu qu'Air Liquide distribue cette année une action gratuite pour 10 ainsi que cela a lieu tous les 2 ans. Cette dernière opération a eu lieu en 2019 et il me semble qu'elle était prévue cette année. Avez-vous une explication à son report en juin 2022 ? Merci de vos lumières. Cordialement   C'était prévu sans être prévu. L'usage chez Air Liquide est de publier des gratuites une année sur deux. Benoît Potier a indiqué, lors de la dernière assemblée, que la prochaine distribution n'aurait lieu qu'en 2022.   Compte tenu des conditions économiques et du maintien, même pendant la crise, de la politique de distribution d'un dividende en hausse, la société a estimé qu'il était plus sage d'attendre un an de plus (en pratique, ce n'est même pas un an car la dernière distribution était intervenue en octobre 2019).   Ce n'est pas un handicap considérable pour l'actionnaire, cela signifie simplement que la hausse des dividendes touchés sera moins forte l'an prochain que prévu puisqu'il n'aura pas les 10% d'actions (et donc de dividendes) en plus permis par les gratuites. Mais cela ne change rien sur la valeur de son portefeuille. Je rappelle qu'avec les gratuites, la valeur de l'action diminue mécaniquement de 10%.   Julien: Bonjour. Albioma ne semble pas porté par la vague de l'écologie, que pensez-vous de la valeur ? Est-il temps de rentrer pour profiter de la « croissance verte » ? Merci.   Albioma est une valeur très présente dans les énergies vertes avec du solaire et de la biomasse mais, dans certaines régions, elle utilise aussi des centrales au gaz et au charbon pendant la période où elle ne peut brûler la bagasse (résidus de la canne à sucre).   En conséquence, certains gérants ESG l'excluent de leurs portefeuilles. Cela explique que le titre ne profite pas autant que d'autres de cette « vague verte » . Mais la société, comme nous l'avons indiqué dans un récent article, va investir entre 600 et 800 millions pour ne plus dépendre de ces énergies fossiles. Cela devrait être fait en 2025.   Jarraud: Cher Monsieur Le Bailly. Le moment est-il venu de couvrir son PEA avec le BX4 bien que cet ETF soit contesté par beaucoup ? Ou avez-vous mieux à conseiller, hormis de « rester liquide » ? Vifs remerciements.   La semaine dernière, j'ai conseillé de progressivement s'alléger même si la dynamique du marché reste très bonne. J'avais aussi indiqué que la couverture était coûteuse. Dans un PEA, les solutions sont encore plus limitées puisqu'il n'y a guère que le BX4 qui permette de se couvrir avec un peu de levier (levier de 2).   Donc, si vous voulez vous couvrir, faites un peu de BX4 et allégez-vous en gardant des liquidités. Cela peut être frustrant, mais c'est de la saine gestion.   LAURENT: Bonjour Rémi. Historiquement, lors de périodes précédentes de reprise économique et de hausse de taux, quelle était l'évolution boursière des valeurs de croissance sur 2 ou 3 ans ? Pourquoi ces valeurs baisseraient-elles si leurs résultats s'améliorent avec la croissance économique ? Peut-être un prochain sujet dans l'hebdo Investir... Merci.   On a déjà eu l'occasion d'en parler plusieurs fois au cours des dernières semaines, la « value » a tendance à battre les valeurs de croissance lors des périodes de reprise économiques pour plusieurs raisons.   Tout d'abord, ces valeurs ont, en général, plus fortement baissé. Ensuite leurs bénéfices, plus volatils, progressent plus vite lors des phases de reprise. Enfin, pour les valeurs de croissance, la méthode retenue d'actualisation du flux futur de bénéfices est plus pénalisante quand les taux remontent car cela entraîne une hausse du taux d'actualisation et donc de la valeur théorique de l'action.   Mais comme je l'ai dit, dans une réponse précédente, il ne faut pas être binaire et abandonner les valeurs de croissance qui, sur longue période, ont des parcours remarquables. Mais d'un point de vue tactique, à court terme, les cycliques ont, même si elles ont bien monté, plus de potentiel.   Benoit: Bonjour Rémy. Pharmagest Interactive baisse de plus de 15% depuis le début de l'année alors que le chiffre d'affaires semble soutenu et l'activité au beau fixe. Comment expliquer cette baisse ? Je pensais me renforcer en septembre en tant qu'investissement long terme, qu'en pensez-vous ? Vous recommandez pourtant la valeur depuis longtemps.   C'est l'exemple type d'une valeur de croissance. Les perspectives à long terme restent très bonnes dans un monde de la santé qui devient de plus en plus digital avec une indispensable coordination entre tous les acteurs du secteur. Mais l'action avait tellement monté qu'elle souffre depuis le début de l'année. Cela n'enlève rien à ses excellentes perspectives sur le long terme.   LAURENT: Bonjour Rémi. Faut-il demeurer patient avec Edenred, Korian et dans une moindre mesure Peugeot Invest, qui stagnent depuis le début de l'année ou depuis leur sélection dans les listes Investir ? Qu'est-ce qui peut les faire démarrer ? Merci.   Les trois valeurs figurent dans notre sélection Investir 10 Valeurs Moyennes, cela veut clairement dire que nous sommes positifs. Edenred permet de jouer le développement des paiements dématérialisés et l'externalisation de certains services par les entreprises. Korian évolue sur le marché des maisons de retraite et des établissements de soins de suite qui croissent avec l'augmentation de la population. Enfin, Peugeot Invest, l'ex-FFP, holding de la famille Peugeot, est fortement décoté par rapport à la valeur de ses actifs.   Je rappelle, de manière plus générale, qu'il faut aussi savoir être patient en Bourse. Bien sûr, c'est mieux d'avoir raison tout de suite, mais ce qui est important, c'est d'avoir raison. La valorisation d'un portefeuille ne se juge pas sur quelques semaines mais sur plusieurs années.   Merci à tous pour votre attention. Bonne semaine à tous et à lundi prochain.   Investir
</article>
<article titre="L'Etat précise les conditions du transfert des cannes de Marie-Galante vers Gardel" date="2021 05 3" journal="FranceTv Info (site web réf.) - France TV Info">
Face aux refus de certains planteurs de Marie-Galante de voir leurs cannes exportées vers Gardel et leur inquiétude quant à la fermeture définitive de l'usine, la Ce transfert des...  Voir l'article  Ce document référence un lien URL de site non hébergé par Cision.
</article>
<article titre="VOS QUESTIONS, NOS RÉPONSES" date="2021 04 22" journal="La lettre de la bourse">
VOS QUESTIONS, NOS RÉPONSES SUPPLEMENT    TECHNIPFMC ET TECHNIP ENERGIES  VOUS FIXEZ UN MÊME OBJECTIF DE 15 EUROS SUR TECHNIP ENERGIES ET TECHNIPFMC. POURRIEZ-VOUS ÉCLAIRER MA LANTERNE SUR CES DEUX ENTITÉS DISSOCIÉES RÉCEMMENT ? Les deux ont effectivement le même objectif de cours, mais il s'agit d'une coïncidence. Le titre de TechnipFMC s'est replié dans le sillage du recul des cours du pétrole, même si les cours du brut n'ont aucun impact direct sur ses résultats. En revanche, ils peuvent influencer les décisions d'investissement de ses clients, les groupes pétroliers. Le titre de Technip Energies semble moins sensible à l'évolution des prix du pétrole. Ses grands donneurs d'ordres sont aussi pour l'essentiel les compagnies pétrolières, mais le groupe travaille sur des projets à long terme, qui se fondent sur des prévisions de cours à un horizon éloigné.    BNP PARIBAS ET ALSTOM  J'ai acheté BNP à 64 euros et Alstom à 43 euros. Je voudrais votre avis sur ces deux valeurs ainsi que l'objectif de cours à moyen terme. BNP Paribas est la première banque de la zone euro. Elle a démontré toutes ses capacités de résistance l'an dernier. Le titre profite de la remontée des taux longs et des anticipations de reprise économique. Néanmoins, avec l'abandon progressif des aides des Etats, les faillites d'entreprise pourraient se multiplier au cours de l'année.   Ce qui fait peser un risque sur la valeur. Nous vous conseillons de conserver vos titres. Nous sommes également positifs sur Alstom qui devrait profiter du regain d'intérêt pour le transport ferroviaire et, plus généralement, de l'engouement pour la mobilité durable. Le groupe pourrait aussi bénéficier des synergies commerciales et opérationnelles issues de l'intégration de Bombardier Transport. Conservez vos actions pour viser un objectif de cours de 55 euros à moyen terme.    AKKA TECHNOLOGIES  Etes-vous positifs sur cette société ? Si oui, pour quel objectif ? Nous avons placé une limite d'achat à 21 euros dans le cadre de notre rubrique du « Coin du spéculateur » pour l'intégrer à notre sélection. Akka Technologies a, comme prévu, fortement souffert de la crise du secteur aéronautique, et le groupe a accusé une perte nette de 168,8 millions d'euros au titre de 2020. Il a pris néanmoins les choses en main en procédant rapidement à une augmentation de capital réservée de 200 millions, en lançant un plan de sauvegarde de l'emploi et en poursuivant son plan de transformation. Toutes ces mesures, conjuguées à la reprise progressive de l'activité, comme observée au quatrième trimestre, devraient porter leurs fruits à compter du second semestre et plus encore en 2022.    CASINO  Faut-il acheter la valeur ? Nous restons très réservés sur le dossier tout d'abord parce qu'il demeure confronté à un problème de dettes, mais également parce qu'il évolue sur un secteur, celui de la distribution, qui ne suscite pas beaucoup d'intérêt de la part du marché en raison d'une féroce concurrence et de marges trop faibles. Il n'y a qu'à observer Carrefour qui, malgré le succès du plan mis en œuvre par Alexandre Bompard, peine en Bourse. Casino n'a même pas ce catalyseur. Restez à l'écart.    EIFFAGE  Avec une plus-value de 12,5 % sur cette valeur, que dois-je faire ? Conservez. Le titre va retrouver ses niveaux d'avant la crise sanitaire à 100 euros une fois levées les restrictions de déplacement en France (confinement et couvre-feu à 19 heures sur tout le territoire), permettant aux Français de reprendre librement les autoroutes. Dans le BTP, l'activité a déjà renoué avec sa dynamique d'avant la pandémie et le carnet de commandes n'a jamais été aussi rempli. La valeur n'est pas chère par rapport à la fois au reste du marché et à son concurrent Vinci.    VOLTALIA  On parle beaucoup des valeurs vertes. Lesquelles faut-il cibler en priorité parmi Neoen, Technip Energies, EDP Reno-vaveis, Solaria, Alfen, et à quels prix ? Je souhaiterais en acquérir deux. Nous ne donnons pas de conseils sur les valeurs qui ne sont pas cotées à Paris. Technip Energies est un groupe d'ingénierie qui ne peut pas être considéré comme une valeur verte. Neoen possède de grandes ambitions de développement, et le groupe a démontré sa capacité à mettre en œuvre ses projets. Le titre est aujourd'hui pénalisé à triple titre, par la rotation sectorielle en cours sur les marchés qui joue en sa défaveur, par la hausse des taux d'intérêt et par l'augmentation de capital de 600 millions d'euros qu'il vient de réaliser. C'est un investissement de long terme. Vous pouvez aussi vous positionner sur Albioma vers 38 euros pour viser 48 euros à moyen terme. Le groupe a entrepris de convertir ses centrales thermiques aux énergies vertes en remplaçant le charbon (quand elles ne fonctionnent pas à la bagasse) par de la biomasse. Ce qui devrait lui valoir une meilleure valorisation.   Posez vos questions sur lalettredelabourse.fr. La rédaction vous répond sous 48 heures.
</article>
<article titre="This new class of materials could transform aircraft design" date="2021 04 22" journal="Airbus Group (site web réf.) - Airbus">
L ightweight and high-strength materials have consistently played a key role in the construction of fuel-efficient and high-performing aircraft. Today, bio-composites-made of raw materials of biological...  Voir l'article  Ce document référence un lien URL de site non hébergé par Cision.
</article>
<article titre="Consultations pré-budgétaires : La valorisation de la  bagasse pour tout producteur réclamée" date="2021 04 21" journal="Le Mauricien (site web réf.) - Le Mauricien">
Photo d'illustration La valorisation de la  bagasse, la protection du sucre produit localement et l’abandon des terres sous cannes figurent parmi les sujets évoqués hier lors des consultations pré...  Voir l'article  Ce document référence un lien URL de site non hébergé par Cision.
</article>
<article titre="a déménage pour les professionnels de l'emballage" date="2021 04 16" journal="Midi Libre  SETE">
Un burger à emporter, un menu chinois ou encore un plat traditionnel en livraison... Si les restaurateurs n'ont plus le droit de recevoir les clients au sein de leurs établissements, les solutions pour des ventes en livraison ou à emporter se sont multipliées. Elles causent aussi de nouveaux types de déchets: sac, barquettes, couverts...   Le malheur des uns... Les entreprises locales doivent s'adapter à la demande. Languedoc emballages, à Sète, qui fournit tous les types d'emballages pour une clientèle professionnelle, a vu sa demande exploser depuis un an.«Le malheur des uns fait le bonheur des autres», lance le gérant José Aparicio. Lui fournissait jusque-là beaucoup les«métiers de bouche», boulangers, bouchers, chocolatiers... Il estime avoir vu son chiffre d'affaires augmenter de 15 à 25% en 2021. Au sein de son entrepôt, la mezzanine créée il y a un peu plus d'un an, faite pour avoir de l'espace, est déjà bondée.«En 2020, on a connu des baisses importantes sur la période saisonnière, mais cela a été compensé par une augmentation toute l'année. La hausse n'est plus ponctuelle pour Pâques ou Noël, mais elle est régulière». Plus de travail aussi pour ses sept salariés, dont le nombre n'a pas changé.«On est habitués à gérer la saison estivale, mais là, c'est à l'année», décrit-il. Des produits en pénurie Si la demande pour le plastique se fait moins sentir, le papier et le carton sont en pénurie«depuis deux mois». Certains produits, livrés à l'entrepôt, seront disponibles une semaine à dix jours, avant d'être à nouveau en rupture.«On passe au moins 25% de notre temps à chercher les produits, à s'adresser à d'autres fournisseurs», décrit José Aparicio. La hausse des prix vient dans le même temps: entre 8 et 25% selon les matières premières.«Je n'ai jamais vu ça en 25 ans», s'étonne le responsable.   Se rapprocher de nouveaux professionnels L'occasion, cependant, de s'adresser à de nouveaux clients, comme les restaurateurs, dont beaucoup n'avaient pas besoin jusque-là de ce type d'emballage. José Aparicio estime avoir travaillé avec une quinzaine ou une vingtaine de restaurants supplémentaires depuis la crise sanitaire. Une nouvelle demande s'est également implantée, entre Sète, le bassin de Thau, La Grande Motte ou Palavas-les-Flots ou encore Montpellier, sa zone de distribution.«Avant, les restaurateurs avaient les assiettes, les couverts, mais tout partait à la vaisselle»,décrit-il.   Des emballages pas toujours recyclables L'occasion aussi de diversifier sa gamme. José Aparicio montre avec fierté ses nouveautés, comme une boîte de burger en carton, plus large et«plus épaisse», pour sortir de la référence fast-food. Ou encore des plats de fruits de mer en bagasse, soit en fibre de bambou. Mais il avertit: ces déchets seront loin d'être recyclables, et les boîtes utilisées, souvent imperméabilisées ou remplies de nourriture, ne pourront souvent pas être récupérées. Même si l'innovation fait avancer les choses, si le carton est«plus joli», dans l'«effet de mode», il ne rime pas toujours pour le chef d'entreprise avec écologie. Éloïse Bussy ebussy@midilibre.com José Aparicio, gérant de la société Languedoc emballages, à Sète, présente ses meilleures ventes. « Le malheur des uns fait le bonheur des autres »
</article>
<article titre="Les clean techs françaises : une bonne idée d'investissement ?" date="2021 04 15" journal="Boursorama (site web réf.) - Boursorama">
(Crédits photo : Adobe Stock - ) Café de la Bourse vous présente les valeurs françaises à succès Neoen, Albioma et Mcphy Energy. Ces trois actions liées à la transition écologique ont un parcours...  Voir l'article  Ce document référence un lien URL de site non hébergé par Cision.
</article>
<article titre="Fiche société ALBIOMA" date="2021 04 14" journal="Agence Option Finance (AOF)">
(AOF) -  Points clés - Producteur d'énergie renouvelable -biomasse et photovoltaïque- né en 2013 des fusions successives entre Sidec, la Séchilienne et Albioma-Bois Rouge ;   - Chiffre d'affaires de 6,7 Mds€ réalisé à 41 % en Europe, 25 % dans les Amérique, 12 % en Asie Pacifique et 5 % en Moyen-Orient, Afrique, Russie et 17 % dans les câbles haute tension&amp;projets;  - Production réalisée à 33 % à partir du charbon, à 64 % avec la biomasse et 2 % le solaire; - Stratégie, adoptée en 2012, de réduction de la part du charbon dans la production thermique au profit de la biomasse et du solaire porter à partir des implantations aujourd'hui essentiellement dans l'outre-mer français (1er producteur solaire) puis à l'île Maurice et au Brésil (énergie à base de bagasse, résidu de la canne à sucre); - Capital éclaté avec 3 positions fortes -Invexans et Teck Pack pour 2 !,9 %, BPI France pour 7,7 % et les salariés pour 4,41 %-, Frédéric Moyne étant président-directeur général du conseil d'administration de 8 membres   - Situation financière saine avec une dette brute en baisse à 914 M€, dont 821 M€ de dette projet, face à une trésorerie de 122 M€.    Enjeux - Stratégie 2023visant à agir pour la transition énergétique en outre-mer, déployer l'international et accélérer le développement dans le solaire :    - 450 0 650 M€ d'investissements programmés et financés par emprunts bancaires,    - rationalisation du portefeuille avec cession des activités peu rentables,   - Stratégie d'innovation externalisée auprès du fonds d'amorçage géré par le fonds Demeter Emertec :    - biomasse thermique : recherche de filiales locales,    - stockage d'énergie électrique,    - services au réseau et maintenance prédictive;   - Stratégie environnementale :    - visant à la réduction primaire des émissions polluantes -rejets aqueux, traitement et valorisation sous-produits de combustion et rejets gazeux,   - validée par l'émission d'un « emprunt vert » rémunéré selon des indicateurs clés : +80 % d'énergies renouvelables en 2023 dans la production du groupe et + 95 % en 2030; -  Insensibilité de la rentabilité aux variations de prix des combustibles en raison des indexations contractuelles et visibilité de l'activité par la durée -30 ans- des contrats de vente ;   - Réalisation des projets solaires gouvernementaux remportés lors du 1er semestre pour 20MWc et mise en service de la centrale brésilienne de Vale do Parana  à la fin 2020 .   Défis - Impact négatif de la dépréciation du real brésilien;   - Forte sensibilité aux soutiens financiers publics, aux réglementations sectorielles et à l'accueil du public à de nouvelles centrales;   - Impact de la pandémie : hausse de 1 % du chiffre d'affaires à fin septembre;   - Réaction à la pandémie : économies de coûts, focus sur l'autofinancement et le renforcement des liquidités et la baisse du niveau de risque des projets; - Objectifs 2020 confirmés en base de fourchette d'un résultat d'exploitation entre 200 et 210 M€ et d'un résultat net entre 48 et 54 M€. AOF - EN SAVOIR PLUS Points clés - Producteur d'énergie renouvelable -biomasse et photovoltaïque- né en 2013 des fusions successives entre Sidec, la Séchilienne et Albioma-Bois Rouge ;   - Chiffre d'affaires de 6,7 Mds€ réalisé à 41 % en Europe, 25 % dans les Amérique, 12 % en Asie Pacifique et 5 % en Moyen-Orient, Afrique, Russie et 17 % dans les câbles haute tension&amp;projets;    - Production réalisée à 33 % à partir du charbon, à 64 % avec la biomasse et 2 % le solaire; - Stratégie, adoptée en 2012, de réduction de la part du charbon dans la production thermique au profit de la biomasse et du solaire porter à partir des implantations aujourd'hui essentiellement dans l'outre-mer français (1er producteur solaire) puis à l'île Maurice et au Brésil (énergie à base de bagasse, résidu de la canne à sucre); - Capital éclaté avec 3 positions fortes -Invexans et Teck Pack pour 2 !,9 %, BPI France pour 7,7 % et les salariés pour 4,41 %-, Frédéric Moyne étant président-directeur général du conseil d'administration de 8 membres   - Situation financière saine avec une dette brute en baisse à 914 M€, dont 821 M€ de dette projet, face à une trésorerie de 122 M€.    Enjeux - Stratégie 2023visant à agir pour la transition énergétique en outre-mer, déployer l'international et accélérer le développement dans le solaire :    - 450 0 650 M€ d'investissements programmés et financés par emprunts bancaires,    - rationalisation du portefeuille avec cession des activités peu rentables,   - Stratégie d'innovation externalisée auprès du fonds d'amorçage géré par le fonds Demeter Emertec :    - biomasse thermique : recherche de filiales locales,    - stockage d'énergie électrique,    - services au réseau et maintenance prédictive;   - Stratégie environnementale :    - visant à la réduction primaire des émissions polluantes -rejets aqueux, traitement et valorisation sous-produits de combustion et rejets gazeux,   - validée par l'émission d'un « emprunt vert » rémunéré selon des indicateurs clés : +80 % d'énergies renouvelables en 2023 dans la production du groupe et + 95 % en 2030; -  Insensibilité de la rentabilité aux variations de prix des combustibles en raison des indexations contractuelles et visibilité de l'activité par la durée -30 ans- des contrats de vente ;   - Réalisation des projets solaires gouvernementaux remportés lors du 1er semestre pour 20MWc et mise en service de la centrale brésilienne de Vale do Parana  à la fin 2020 .   Défis - Impact négatif de la dépréciation du real brésilien;   - Forte sensibilité aux soutiens financiers publics, aux réglementations sectorielles et à l'accueil du public à de nouvelles centrales;   - Impact de la pandémie : hausse de 1 % du chiffre d'affaires à fin septembre;   - Réaction à la pandémie : économies de coûts, focus sur l'autofinancement et le renforcement des liquidités et la baisse du niveau de risque des projets; - Objectifs 2020 confirmés en base de fourchette d'un résultat d'exploitation entre 200 et 210 M€ et d'un résultat net entre 48 et 54 M€.
</article>
<article titre="Le retour gagnant de la papeterie Zuber-Rieder sur les marchés du luxe" date="2021 04 13" journal="Les Echos (site web)">
« Cocaïne », « Bagasse », « Permafrost », « White Spirit », « Asphalte », Caviar... Ces noms au parfum transgressif sont ceux des gammes de Zuber-Rieder, papeterie née sur les bords du Doubs, à Boussières, en 1881 et positionnée depuis les années 2000 sur les marchés du luxe et des étiquettes pour vins et spiritueux.  Le premier, Cocaïne, est un papier blanc très haut de gamme, 100 % coton et offrant des possibilités d'embossage et de dorure qui plaisent beaucoup aux clients finaux de la PME, de grands noms de la cosmétique, la joaillerie ou la parfumerie. Le deuxième, Bagasse, est un papier sans arbres issu de résidus de plantes annuelles comme la canne à sucre.   Innovation et respect de l'environnement Lancé en 2009, l'année de sortie d'un plan de continuation consécutif à un dépôt de bilan, en 1995, il illustrait bien le virage stratégique pris par l'équipe dirigeante, celui de l'innovation dans le respect de l'environnement, qui allait séduire les marchés de niche qu'elle visait. C'est ainsi que Luc Gaillet, arrivé fin 1997 à la présidence de l'entreprise, et Alain Martz, qui en assure la direction générale depuis 2003, avaient choisi de la remettre sur pied, en misant sur le savoir-faire maison et donc sur le made in France, et en visant plus précisément le haut de gamme.     « Le point de départ de la stratégie partait du constat que cette petite papeterie ne pouvait pas être compétitive sur de gros volumes »  , se souvient Luc Gaillet pour justifier ce positionnement sur les petites séries et le sur-mesure. Les deux associés avaient donc reconstruit un marché sur quatre secteurs : le packaging de luxe, les étiquettes pour vins et spiritueux, l'art et la communication et quelques marchés techniques.     «   Et nous essayons de rester diversifiés sur ces marchés en nous appuyant sur une base stratégique : un bon marketing et de la bonne technique. La réindustrialisation, ça commence par le renforcement des savoir-faire qui existent déjà en France. Nous étions partis d'une entreprise regardée avec condescendance et plus ou moins vouée à disparaître, et aujourd'hui notre situation financière est excellente »  , ajoute son président sans toutefois donner de chiffres.    Le « bon marketing »  Chez Zuber-Rieder, qui indique néanmoins exporter aujourd'hui 60 % de sa production dans 42 pays auprès de clients imprimeurs ou transformateurs, le « bon marketing » implique un intérêt porté à toute la chaîne de valeur et en particulier aux clients de ses clients, les fameux acteurs du luxe.   « Nous avons un réseau d'interlocuteurs chez eux et notre mission, c'est de valoriser leur marque »,  explique Alain Martz. «  Et, pour boucler la boucle, il faut que nos clients connaissent nos produits afin qu'ils puissent se tourner vers nous quand leurs prescripteurs vont leur demander un papier Cocaïne, par exemple. »  Le « bon marketing » pour la papeterie de Boussières, outre ces gammes aux noms qui claquent, ce sont aussi et surtout des produits adaptés aux marché et aux attentes sociétales, comme cette collection Esprit de Nature, labellisée FSC, ou encore ces étiquettes taillées dans un papier Permafrost, produit porteur d'un brevet mondial pour sa résistance à une immersion prolongée dans un seau à glace.    Plus de 2 millions d'euros d'investissements Quant à la « bonne technique », elle passe depuis la fin du plan de continuation par «  plus de 2 millions d'euros d'investissements par an sur tout ce qui améliore la qualité de nos produits »  , assure Luc Gaillet. Des investissements dans la machine à papier mais aussi dans la formation et la matière grise avec un taux d'encadrement d'une vingtaine de cadres pour un effectif de 123 personnes, et une attention industrielle très poussée de la part des équipes.    Process complexe, la fabrication du papier passe par une phase humide délicate et nécessite tout leur savoir-faire, et encore davantage pour les marchés hyper exigeants du luxe.   Monique Clémens @mo_clemens
</article>
<article titre="Production Sucrière : Le prix ex-Syndicat évalué à Rs 13 000 la tonne" date="2021 04 9" journal="Le Mauricien (site web réf.) - Le Mauricien">
Photo d'illustration – La hausse des prix attribuée à la dépréciation de la roupie et à la vente de sucres spéciaux – Le secteur sucrier toujours dans l’attente de la publication...  Voir l'article  Ce document référence un lien URL de site non hébergé par Cision.
</article>
<article titre="Comment Albioma va convertir Bois-Rouge [Reportage à La Réunion]" date="2021 04 6" journal="GreenUnivers (site web)">
2021-04-07T09:24:15+00:00       Après 30 ans de bons et loyaux services, la centrale thermique de Bois-Rouge à La Réunion ne tire pas sa révérence, au contraire. Son propriétaire Albioma va la convertir intégralement à la biomasse. Il a obtenu en décembre dernier l'autorisation de la Commission de régulation de l'énergie (CRE) et un nouveau contrat d'achat d'électricité avec EDF jusqu'en 2043. GreenUnivers s'est rendu sur place. Reportage.   Une centrale hybride Située au nord de l'île de La Réunion, la centrale thermique de Bois-Rouge brûle encore du charbon, mais plus pour longtemps, Albioma ayant décidé de convertir l'ensemble de ses centrales thermiques à la biomasse. Un travail déjà engagé aux Antilles (Gallion 2, le Moule 3), et qui va bientôt commencer sur[restrict-content] cette île de l'océan Indien. « Nous sommes arrivés à La Réunion à la fin des années 80. Bois-Rouge fut la première centrale thermique à être mise en service en 1992, sur 55 MW d'abord puis près de 100 MW par l'ajout d'une nouvelle tranche de 43,7 MW en 2004 », se remémore Pascal Langeron, directeur général adjoint d'Albioma, responsable des activités du groupe à La Réunion et Mayotte. Ce dernier connaît particulièrement bien la centrale, et pour cause, il a participé à sa création il y a une trentaine d'années. « Celle du Gol (110 MW), au sud-ouest de l'île, a quant à elle été mise en service en 1995. Depuis lors, ces deux centrales apportent une contribution significative à la production d'électricité de l'île, mais aussi au maintien de la tension et de la fréquence sur le réseau opéré par EDF SEI », poursuit-il. Bois-Rouge a été conçue dès le début comme une centrale hybride de cogénération. Elle brûle du charbon 7 mois par an mais lorsque vient la campagne sucrière, elle absorbe les résidus de la canne à sucre, ou bagasse, de la sucrerie voisine. « L'accord passé est le suivant : Tereos nous apporte la bagasse, environ 110 tonnes/heure pendant 5 mois, en échange de quoi nous lui fournissons 9 MWh d'électricité et 150 t/h de vapeur à 3 bars et 150 C°, ce qui couvre l'ensemble de ses besoins énergétiques. Ce partenariat bénéficie également aux planteurs puisqu'ils touchent, pour une tonne de canne vendue, environ 14€ de prime énergie (dite prime bagasse), mise en place par le gouvernement en 2009 », explique le dirigeant. L'utilisation de la biomasse est donc familière au fonctionnement de la centrale, qui dispose de sites de stockage dédiés pour les deux combustibles, ainsi que de deux systèmes d'intégration distincts, le charbon reposant sur grill, tandis que la bagasse brûle en suspension. Le passage d'un combustible à l'autre n'impose pas d'arrêt, permettant une production d'électricité en continue.   Une conversion par tranche La conversion de Bois-Rouge n'a rien d'anodin pour le système électrique de l'île de La Réunion, qui repose essentiellement sur trois centrales thermiques de forte puissance (voir carte), dont deux opérées par Albioma. Pas question donc d'engager ces travaux sans concertation préalable avec EDF SEI, qui assure la gestion du réseau. « Pour assurer la continuité de l'approvisionnement électrique de l'île, la conversion de la centrale se fera par tranche. Nous en opérons trois à Bois-Rouge, chacune sera mise à l'arrêt pendant 4 mois pour réaliser les travaux nécessaires. Ces derniers consisteront essentiellement à modifier les équipements de manutention, les adaptations des chaudières étant minimes, car le système de brûlage de la biomasse est comparable à celui de la bagasse. Ces opérations doivent s'étaler entre la fin de l'année 2022 et début 2024 », explique Pascal Langeron. Il en sera de même pour la centrale du Gol, qui attend encore son autorisation de conversion.   Ces opérations sont d'autant plus critiques que les besoins d'électricité ne cessent de croître sur l'île (+13% entre 2009 et 2018) par l'effet conjugué de l'augmentation de la population (croissance naturelle + immigration métropolitaine) et du niveau de vie. L'ensemble tire mécaniquement à la hausse la consommation, et surtout la pointe passée de 423 MW à 484 MW entre 2009 et 2018, selon les données d'EDF SEI. Or, la typologie de l'île volcanique n'est pas vraiment propice au développement tous azimuts d'installations industrielles de production électrique, la loi littoral ne facilitant pas non plus les choses de ce point de vue. L'île ne bénéficie par ailleurs d'aucune interconnexion avec ses voisins, et le maillage du réseau est par définition moins dense qu'en métropole. Autant de facteurs qui conduisent à la plus grande prudence lorsqu'il s'agit de se passer, même partiellement et pour quelques mois, d'un moyen de production de base pilotable comme Bois-Rouge. Et qui explique le surdimensionnement du parc électrique : près de 900 MW installés alors que la pointe est inférieure à 500 MW. Pour rappel, la PPE réunionnaise prévoit un mix à 76% renouvelable dès 2023, alors que la part des EnR s'élevait à peine à 31,2% en 2019. Le défi est donc immense.   L'enjeu de l'approvisionnement Albioma prend également un soin particulier sur le futur approvisionnement de sa centrale, le bilan carbone d'une conversion à la biomasse pouvant varier sensiblement d'un scénario à l'autre. « La conversion va permettre de réduire de 84% les émissions de gaz à effet de serre de nos centrales. Le contenu carbone de la production électrique, une fois les travaux réalisés, a été estimé par le cabinet Deloitte à 59 g eq CO2/Mj électrique. Environ un quart de ce bilan carbone provient du transport maritime. C'est la raison pour laquelle nous travaillons sur notre stratégie d'approvisionnement. Dans un premier temps, nous importerons d'Amérique du Nord, mais nous étudions aussi la possibilité d'acheter entre 30 et 40% de nos pellets de bois de pays proches situées en Afrique australe. Pour que l'opération soit intéressante pour l'empreinte carbone, il faut nous assurer que les mix de production électrique associés soit décarbonés, car leur part est significative dans le bilan carbone total », précise Pascal Langeron.   Ce dernier cherche également à s'approvisionner directement sur l'île et a plusieurs pistes. La paille de canne pourrait apporter 30 000 t/an, des études techniques et de faisabilité sont en cours, et les broyats de déchets végétaux représenteraient un potentiel de 25 000 t/an. Parallèlement, une étude est menée par l'ONF sur le gisement potentiel de plaquettes forestières, notamment issues d'Acacia invasif. Les connexes de scieries et les broyats d'emballages en bois sont également une option étudiée par Albioma. Le tout pourrait représenter jusqu'à 100 000 tonnes de combustibles, sur les 400 000 tonnes par an que consommera la centrale une fois convertie.    [/restrict-content]  Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="Passe-temps" date="2021 04 3" journal="Le Temps">
6982   PAR ,  ET    PHOTOGRAPHIE FÉMININS CROISÉS Female gaze. Le regard des femmes sur les femmes, mais aussi sur les hommes et le monde. Façonné au sein de l'univers cinématographique, ce concept met à mal l'omnipotence des points de vue masculins sur les oeuvres de fiction, faisant naître une culture visuelle plus riche et moins stéréotypée. Puissante industrie d'images, la mode n'échappe pas à cette révolution théorique et visuelle. Chez Dior, le basculement est flagrant depuis l'arrivée, en 2016, d'une femme - la première - à la tête de la création: Maria Grazia Chiuri. Saison après saison, l'Italienne met ses collections au service de messages féministes martelés avec plus ou moins de finesse par le biais de défilés spectacles, mais aussi de campagnes de pub systématiquement photographiées par des femmes. Brigitte Niedermair, Sarah Moon, Talia Chetrit (image), Bettina Rheims, Nan Goldin, Maya Goded ou encore Kristin-Lee Moolman. Au total, 33 artistes aujourd'hui réunies dans Her Dior. La vision de Maria Grazia Chiuri, un livre réunissant 160 photos du style Chiuri pour Dior. Entre portraits et décors envoûtants, abstractions et clairs-obscurs, une ode à l'acuité du regard féminin.    « Her Dior. La vision de Maria Grazia Chiuri », Ed. Rizzoli, 2021, 240 p.   PODCASTPAROLES D'ARTISTESDonner la parole aux artistes femmes ayant marqué le XXe siècle. Suivre leur voix pour découvrir leur oeuvre, leur époque, leur chemin de vie. Voilà le concept du podcast Les Grandes Dames de l'art, produit par Aware, association française ambitionnant de réécrire l'histoire de l'art de façon paritaire. Le premier épisode a pour sujet la sculpture, technique longtemps réservée aux hommes. Mais les femmes s'y sont aventurées, se sont mesurées à l'échelle monumentale, et ont même pour certaines conquis l'espace public. D'ailleurs, Niki de Saint Phalle, l'une des artistes femmes les plus connues du XXe siècle est une sculptrice. Elle s'est revendiquée comme telle, et sa fierté est d'avoir justement imposé des femmes dans la rue: ses Nanas, créatures en résine de polyester à l'exubérance décomplexée. C'est sur la voix de « Niki » que s'ouvre le programme: « Si je devais me définir, je ne me définirais pas comme une artiste, mais comme réalisatrice de rêves. Je réalise mes rêves. » awarewomenartists.comMODEHABILLER TOUT CORPSDes vêtements de créateurs pour personnes handicapées. Utilisateurs et utilisatrices de fauteuils roulants, individus ayant des prothèses ou des fractures, touchées par la sclérose en plaques ou la maladie de Parkinson. Voilà ce que propose la marque autrichienne Mob Industries, fondée par Josefine Thom et Johann Gsöllpointner. Loin des designs sans saveur actuellement disponibles dans la grande distribution, l'élégance street de la marque colle aux gesticulations de l'époque. Ici, des boutons-pressions sur les manches d'une chemise facilitent l'habillage; là, un système de fermeture magnétique structure un pantalon à empiècements amovibles. Des tissus imperméables aident les personnes qui ont des difficultés à manger ou à boire; les longueurs et coupes des vêtements sont optimisées pour ne pas traîner sous un fauteuil roulant. Le tout dessiné par des créateurs locaux. Pour que les vêtements s'adaptent aux gens, et non l'inverse.shop.mob-industries.comLA BIBLIOTHÈQUE DET « PETER LINDBERGH. AZZEDINE ALAÏA », Ed. Taschen, 240 p.Un catalogue d'images en noir et blanc célèbre la communion artistique entre Peter Lindbergh et Azzedine Alaïa. Le photographe allemand et le couturier tunisien se sont retrouvés autour d'une affection commune pour le noir. En couverture, le sourire de Naomi Campbell annonce des pages riches d'autres grands noms du mannequinat, par exemple la beauté androgyne de Marie Sophie Wilson, le teint pâle de Mariacarla Boscono, ou encore la bouche en coeur de Lindsey Wixson. « VOYAGES EXTRAORDINAIRES », Francisca Mattéoli, Collection Louis Vuitton, Ed. Xavier Barral, 488 p.Chilienne de mère écossaise, Francisca Mattéoli a vécu au Brésil avant de s'installer à Paris. Collaboratrice de Condé Nast Traveler et de National Geographic, elle signe son quatrième livre consacré au voyage, illustré de photographies historiques et d'affiches vintage. Des paquebots de luxe aux traîneaux sur les banquises, une épopée au rythme des moyens de locomotion les plus originaux. « JIM SHAW: PAPERBACK COVERS », par Charlie Fox, Lionel Bovier et Samuel Gross, Ed. JPR, 104 p.L'artiste américain Jim Shaw a peint toute une série de fausses couvertures de livres de poche. Le recueil de ses Paperback Covers est ici accompagné d'une création littéraire signée Charlie Fox, écrivain et critique d'art londonien, qui s'inspire de la personnalité atypique de Jim Shaw dans le milieu artistique californien.COSMÉTIQUELA BEAUTÉ EST DANS LE PRÉ « Nous créons de la beauté avec les plantes les plus indésirables », proclame la marque Herbes Folles, fondée par l'herboriste Mariana Santos. La gamme ultra-green, fabriquée en Belgique, mise sur une routine de beauté simple et naturelle basée sur trois produits. Coquelicot, plantain et carotte sauvage pour une lotion apaisante et hydratante, ortie et plantain pour un baume antioxydant multi-usages, et enfin une huile nettoyante infusée d'extraits de pourpier et millepertuis. Les emballages sont produits en Europe et exclusivement composés de verre et de carton, alors que les étiquettes sont créées à partir de bagasse, un déchet de l'industrie sucrière.herbes-folles.comCHAUSSURESCOLLAPSOLOGIE DU PIEDChaussures d'ouvrier récupérées par tout ce que la jeunesse mondiale compte de contrecultures, les Dr. Martens sont plus populaires que jamais. En témoigne la récente entrée en bourse de la marque britannique, soutenue par des investisseurs institutionnels de premier plan. Explication? Des revendeurs branchés (Zalando, Asos), un site d'e-commerce extrêmement performant, des innovations intelligentes (modèles véganes) et une solide réputation mode. Ainsi, les « Doc » sont régulièrement remises au goût du jour par de grands créateurs comme Yohji Yamamoto, Jean Paul Gaultier et, tout dernièrement, Rick Owens. Chantre d'un glamour obscur aux accents dystopiques, l'Américain revisite le modèle 1460 à huit trous de laçage, un classique resté inchangé depuis son lancement, le 1er avril 1960. Résultat: une bottine noire à lacets blancs, formant sur le pied une sorte de pentagramme. « Un symbole de notre éternelle recherche de rationalité; des signes d'espoir et une affectueuse bénédiction de la perpétuelle insouciance de la jeunesse », dixit Rick Owens dans les colonnes du magazine anglais GQ.drmartens.comAPÉRITIFCONCURRENCE AU SPRITZUn negroni prêt à déguster. En y ajoutant juste quelques glaçons et un zeste d'agrume. Voilà l'ambition du Negroncino. Né de la collaboration entre Alessandro Bortesi, propriétaire du restaurant italien Kytaly à Genève, et Giancarlo Mancino, mixologue italien, ce cocktail en bouteille est composé de trois vins fortifiés différents: le Vermouth di Torino Secco, infusé de 19 plantes aromatiques, le Vermouth di Torino Rosso, infusé de 36 plantes aromatiques, ainsi que du Vermouth Chinato, à base du célèbre vin italien Barbera d'Asti et de quinine. Un mélange auquel s'ajoutent du Rinomato Bitter Scuro et du Piu Cinque Gin, élu meilleur gin italien en 2020. Ce savant assemblage lui assure la saveur amère et sucrée si particulière à ce cocktail concurrent du spritz. Comme dans tout bon negroni, on y décèle tous les arômes qui le composent: le vin rouge, le bitter, la bergamote et la sauge du gin.negroncino.com
</article>
<article titre="ALBIOMA, parmi les plus fortes baisses du SRD à la clôture du lundi 29 mars 2021" date="2021 03 29" journal="Agence Option Finance (AOF)">
(AOF) -  ALBIOMA  (-2,98% à 40,75 euros)   Le producteur d'énergies renouvelables a été pénalisé par la rotation sectorielle opérée par les gérants. Ces derniers se détournent des valeurs jugées fortement valorisées, comme celles du secteur des énergies vertes.   AOF - EN SAVOIR PLUS Points clés - Producteur d'énergie renouvelable -biomasse et photovoltaïque- né en 2013 des fusions successives entre Sidec, la Séchilienne et Albioma-Bois Rouge ;   - Chiffre d'affaires de 6,7 Mds€ réalisé à 41 % en Europe, 25 % dans les Amérique, 12 % en Asie Pacifique et 5 % en Moyen-Orient, Afrique, Russie et 17 % dans les câbles haute tension&amp;projets;    - Production réalisée à 33 % à partir du charbon, à 64 % avec la biomasse et 2 % le solaire; - Stratégie, adoptée en 2012, de réduction de la part du charbon dans la production thermique au profit de la biomasse et du solaire porter à partir des implantations aujourd'hui essentiellement dans l'outre-mer français (1er producteur solaire) puis à l'île Maurice et au Brésil (énergie à base de bagasse, résidu de la canne à sucre); - Capital éclaté avec 3 positions fortes -Invexans et Teck Pack pour 2 !,9 %, BPI France pour 7,7 % et les salariés pour 4,41 %-, Frédéric Moyne étant président-directeur général du conseil d'administration de 8 membres   - Situation financière saine avec une dette brute en baisse à 914 M€, dont 821 M€ de dette projet, face à une trésorerie de 122 M€.    Enjeux - Stratégie 2023visant à agir pour la transition énergétique en outre-mer, déployer l'international et accélérer le développement dans le solaire :    - 450 0 650 M€ d'investissements programmés et financés par emprunts bancaires,    - rationalisation du portefeuille avec cession des activités peu rentables,   - Stratégie d'innovation externalisée auprès du fonds d'amorçage géré par le fonds Demeter Emertec :    - biomasse thermique : recherche de filiales locales,    - stockage d'énergie électrique,    - services au réseau et maintenance prédictive;   - Stratégie environnementale :    - visant à la réduction primaire des émissions polluantes -rejets aqueux, traitement et valorisation sous-produits de combustion et rejets gazeux,   - validée par l'émission d'un « emprunt vert » rémunéré selon des indicateurs clés : +80 % d'énergies renouvelables en 2023 dans la production du groupe et + 95 % en 2030; -  Insensibilité de la rentabilité aux variations de prix des combustibles en raison des indexations contractuelles et visibilité de l'activité par la durée -30 ans- des contrats de vente ;   - Réalisation des projets solaires gouvernementaux remportés lors du 1er semestre pour 20MWc et mise en service de la centrale brésilienne de Vale do Parana  à la fin 2020 .   Défis - Impact négatif de la dépréciation du real brésilien;   - Forte sensibilité aux soutiens financiers publics, aux réglementations sectorielles et à l'accueil du public à de nouvelles centrales;   - Impact de la pandémie : hausse de 1 % du chiffre d'affaires à fin septembre;   - Réaction à la pandémie : économies de coûts, focus sur l'autofinancement et le renforcement des liquidités et la baisse du niveau de risque des projets; - Objectifs 2020 confirmés en base de fourchette d'un résultat d'exploitation entre 200 et 210 M€ et d'un résultat net entre 48 et 54 M€.
</article>
<article titre="l exemples l Comment les entreprises essaient d'agir" date="2021 03 25" journal="LSA, no. 2644">
Intermarché, Optic 2000, Maisons du Monde, Okaïdi, Bureau Vallée, Picard Surgelés, Truffaut, Metro... Autant de distributeurs engagés dans une démarche environnementale qui se traduit concrètement par un label, celui d'Enseigne responsable. Keria Luminaires, qui a réussi à baisser de 55 % la consommation énergétique de ses 70 magasins sur cinq ans, en fait partie. Antoine Tassigny, son directeur général, lui trouve beaucoup de mérites. « En s'inscrivant dans un processus de label comme celui-ci, cela permet aux entreprises de matérialiser leur plan d'action. Ce label présente deux avantages : il est pensé par des acteurs du commerce et il s'étend sur trois ans, ce qui inscrit la démarche dans la durée avec un audit chaque année sur plusieurs points. » Autrement dit, les entreprises peuvent mesurer tous les ans les progrès qu'elles font.  Car elles n'ont plus le choix et doivent passer à la vitesse supérieure. « Nous faisons partie d'un système qui a un impact carbone important, détaille Élisabeth Bouton, directrice développement durable et qualité de Picard Surgelés. C'est un travail de fond qui porte sur les matières premières, le transport, les emballages, les magasins... » Jusqu'à l'utilisation du produit. Dans son concept Ikea City, à Paris, l'enseigne suédoise prodigue des conseils pour ranger en vrac ou des bons gestes pour la planète. Bref, distributeurs et industriels doivent embarquer leur démarche environnementale dans toute la chaîne de valeur. Faire la chasse aux émissions directes, comme baisser les dépenses énergétiques dans les magasins, les bureaux ou les usines ne suffit plus. Ce que les spécialistes appellent « scopes 1 et 2 ». Désormais, les entreprises doivent s'attaquer au « scope 3 », soit traquer les émissions indirectes, issues des actions de fournisseurs, transporteurs ou sous-traitants. Ces dernières représentent la plus grande partie des émissions de carbone.   Collectif Fret 21   C'est en ce sens qu'agissent des enseignes comme Picard Surgelés ou Intermarché. La première travaille au sein du collectif Fret 21, animé entre autres par le ministère de la Transition écologique et par l'Ademe (Agence de la transition écologique). Une soixantaine d'entreprises, de Carrefour à Findus, Ferrero ou Coca, de BSH à Picard Surgelés ou Seb, se sont engagées à réduire les émissions de CO2 en collaborant avec les transporteurs. « Il s'agit de réconcilier les donneurs d'ordre ou ceux qui mettent les marchandises dans les camions avec les transporteurs », explique Gérald Townsend, directeur développement durable de Picard Surgelés, dont 30 % des fournisseurs sont étrangers. Dans ce cas, l'acheminement par bateaux est préféré. C'est donc en travaillant avec leurs partenaires, que les entreprises progressent sur la longue route de la transformation environnementale.    Lorsqu'Intermarché commercialise une bouteille en 100 % rPET opaque permettant de prouver la faisabilité du recyclage en boucle fermée, il ne le fait pas tout seul, mais avec la Laiterie Saint-Père. Avec ce test de 60 000 bouteilles dans 400 points de vente, les deux acteurs valorisent l'économie circulaire. De couleur grise, cette bouteille est recyclée, recyclable et son cycle de vie fonctionne en boucle fermée : elle est triée par le consommateur, orientée vers un recycleur qui la réduit en paillettes. Cette matière est ensuite utilisée pour faire une préforme qui redevient une nouvelle bouteille. Cette technique a l'avantage d'utiliser uniquement du plastique déjà existant sur le marché. L'intégration de 100 % de matière recyclée baisse l'impact carbone de 25 % par rapport à la création d'une nouvelle bouteille de lait blanche. Cela ne ressemble pas à une simple bouteille de lait, mais correspond à une volonté de produire de manière plus écologique, durable et responsable.    Au même titre que le transport, l'emballage fait partie des gros enjeux pour des entreprises qui appartiennent au secteur de la consommation. Picard Surgelés s'est penché sur la question. L'enseigne a réduit de 350 tonnes le plastique qu'elle utilise pour emballer ses surgelés. Soit un tiers du total. Encore faut-il trouver des substituts. « Le plus facile à court terme, c'est d'utiliser du plastique recyclable, estime Gérald Townsend. À un horizon plus lointain, nous disposerons de solutions de plus en plus en accord avec le développement durable pour remplacer le plastique. » Pour ce faire, il faut aussi transformer les outils industriels. Trouver une machine à emballer n'a pas l'air aisé, à écouter les enseignes. La bagasse, résidu fibreux de canne à sucre, remplace avantageusement les barquettes en plastique. Tout est question d'équilibre. « Si l'emballage s'écrase et que nous sommes obligés de jeter, nous perdons 90 % du poids carbone, conclut Gérald Townsend. C'est alors une catastrophe écologique ! » Parfois, il vaut mieux garder un emballage minimum qui s'avère nécessaire, comme pour le poisson surgelé.   Autre levier essentiel, l'écoconception. Produire local, penser local, cela paraît simple, et pourtant cela demande un travail de concert. « Tout seul, on ne peut rien faire, renchérit Élisabeth Bouton. Nous devons prendre notre part. » Illustration avec Picard qui a embarqué l'an dernier une vingtaine d'agriculteurs bretons pour les aider à diminuer l'usage de produits phytosanitaires. Il a fallu trois ans de recherche et développement pour établir un référentiel de 25 pratiques agroécologiques. D'ici à 2022, l'enseigne de surgelés s'est fixé pour objectif de convaincre une quarantaine d'agriculteurs.    Sourcing efficace   Dans le secteur du meuble, l'écoconception passe par un travail de longue haleine avec des fournisseurs basés le plus souvent à l'autre bout de la planète. L'une des principales difficultés réside dans le sourcing : réussir à obtenir la certification sur l'ensemble de la chaîne de valeur et s'assurer que les fournisseurs trouvent du coton recyclé ou du bois certifié. « Il y a de plus en plus de demande pour ces matières-là, pointe Rémi-Pierre Lapprend, directeur RSE de Maisons du Monde. Or, la production n'est pas encore totalement constituée, notamment pour le coton bio. »    Maisons du Monde a donc imaginé un partenariat avec une ONG, Earthworm (anciennement The Forest Trust), qui lui permet de procéder à une traçabilité complète, cette fois pour le bois. L'enjeu étant de construire une filière responsable qui utilise du manguier, dont la coupe est légale et ne participe pas à la déforestation. D'ici à 2024, l'enseigne de meubles se fixe comme objectif d'atteindre la totalité de son offre linge de maison en coton bio. Pour le meuble, dont 70 % de la collection Maisons du Monde est aujourd'hui en bois tracé et certifié, l'objectif est d'arriver à 80-85 % d'ici à trois ans.    Reprise des meubles   Mais le produit le plus abouti consiste en un canapé écoconçu. Le Falkor est fait à partir d'une mousse de rembourrage recyclée et de bois certifié PEFC, issu de forêts gérées de façon responsable... et fabriqué en France. Le tout à un prix « responsable » à moins de 1 000 €. Chez Ligne Roset, le prix revient moins au premier plan. « Dans le haut de gamme, nous ne sommes pas obligés de sourcer à l'autre bout du monde, car l'équation prix est différente, reconnaît Antoine Roset, son dirigeant. La plupart de nos meubles sont fabriqués en France et en Europe, et beaucoup de nos fournisseurs sont dans un rayon de 80 km. » Ligne Roset fabrique lui-même plus de 85 % des meubles qu'il vend. Mais parce que la fin de vie d'un produit est aussi importante que sa conception, l'enseigne travaille sur la reprise d'un de ses bests, le fauteuil Togo. En échange, un bon d'achat qui représente 15 % de la valeur de ce bien. M. P.
</article>
<article titre="Quatre conditions pour un plastique végétal vraiment vert" date="2021 03 22" journal="Ouest-France (site web)">
40 % du plastique sert d’emballage. C’est une aberration de brûler du carbone fossile, qui a mis des millions d’années à se former, pour fabriquer des barquettes ou des films alimentaires qui ne seront utilisées que quelques jours et qui, une fois à l’état de déchets, mettront des années à se dégrader !, déplore Vincent Verney, enseignant chercheur à l’Institut de chimie de Clermont-Ferrand. 1. Moins de gaz à effet de serre émis   Les plastiques biosourcés, fabriqués à partir de la biomasse végétale, ne représentent que 2 % des plastiques. Leur généralisation aurait pour avantage de préserver les énergies fossiles et de réduire les émissions de gaz à effet de serre.   2. Recycler des déchets agricoles   S’il est fabriqué à partir d’amidon, le plastique végétal détourne des cultures de leur fonction nourricière, signale Vincent Verney. S’il est fabriqué à partir de résidus (menues pailles, drèches, bagasse de canne à sucre…), le plastique biosourcé valorise un déchet. Exemple : le programme BluEcoPHA. Piloté par l’entreprise mayennaise Europlastiques, il travaille à la mise au point d’un bioplastique issu de la fermentation de déchets de fruits et légumes.   3. Attention aux plastiques non biodégradables   Un plastique est biodégradable s’il est digéré par les micro-organismes en moins de six mois. Tous les plastiques biosourcés ne sont pas biodégradables, souligne Vincent Verney. Les pots de yaourts, sacs plastiques et autres films alimentaires en bio polyéthylène, fabriqué à partir de la biomasse végétale, s’ils sont jetés dans la nature, peuvent y persister.   Autre pollution : ﻿les plastiques oxodégradables contenus dans certains sacs plastiques et bâches agricoles. Biosourcés ou issus de la pétrochimie, ils se fragmentent en microparticules susceptibles de contaminer la chaîne alimentaire. Ils sont interdits dans l’Union européenne depuis le 1er janvier 2021.   4. Des normes pour y voir plus clair   La mention NF 13 432 inscrite sur un sac plastique certifie son caractère biodégradable. La norme OK compost certifie un sac compostable en déchetterie. La norme Home compost certifie un sac biodégradable dans le composteur de son jardin. Dans les supermarchés, au rayon fruits et légumes, la réglementation impose des sacs plastiques biodégradables avec une teneur minimale de plastique biosourcé, précise Vincent Verney.  Cet article est paru dans Ouest-France (site web)
</article>
<article titre="COVID-19 Growth And Change" date="2021 03 8" journal="Yahoo! Finance France (site web réf.) - Yahoo! Finance (FR)">
Major players in the nanosilica market are NanoPore Incorporated, Fuso Chemical, Evonik, Cabot Corporation, DuPont, Akzonobel, Nanostructured and Amorphous Materials (NanoAmor), Wacker Chemie AG, Normet, and Nanosil Sdn.New York, March...  Voir l'article  Ce document référence un lien URL de site non hébergé par Cision.
</article>
<article titre="Conseil De 600 à 800 millions d'ici à 2025 pour devenir encore plus vert Albioma" date="2021 03 6" journal="Investir-Le Journal des finances, no. 2461">
Producteur d'électricité présent dans les DOM et au Brésil, ainsi qu'en France métropolitaine pour le solaire,Albioma est une entreprise assez atypique. Son premier métier consiste à exploiter des centrales thermiques de cogénération adossées à des sucreries. Elles brûlent de la bagasse (le résidu de la canne à sucre) pendant les campagnes sucrières, soit toute l'année au Brésil, et du charbon en dehors. Le groupe a entrepris de convertir ses centrales ultramarines à la seule biomasse, ce qui a été effectué pour Le Moule 3, en Guadeloupe, en 2020. La démarche va se poursuivre pour les trois tranches de la centrale de Bois-Rouge, à La Réunion, et concerner toutes les unités d'ici à 2025, afin d'atteindre un mix énergétique renouvelable à plus de 90 %. « Cette conversion à la biomasse va représenter environ la moitié des 600 à 800 millions d'euros d'investissements annoncés pour 2021-2025, précise Frédéric Moyne, le PDG. Nous jouissons aussi d'un fort dynamisme commercial dans le solaire avec 40 MW remportés en 2020. » Cette accélération dans les énergies vertes - dont la conversion du Moule 3 - a contribué à l'annonce de résultats meilleurs que prévu l'an dernier. Pour un chiffre d'affaires stable, à 506 millions d'euros, l'Ebitda a augmenté de 13 %, à 206,4 millions, et le bénéfice net, de 25 %, à 55,3 millions. Ce résultat est attendu entre 53 et 59 millions pour 2021.   Un dividende de 0,80 € (+ 14 %) est proposé pour 2020, avant majoration de 10 % pour les inscrits au nominatif depuis plus de deux ans. Il existe aussi une option de paiement en actions pour 50 % du dividende.   Notre conseil Acheter  comme ses consoeurs du secteur, la valeur a fait l'objet de prises de bénéfices après les envolées de 2020. Profiter du repli de 22 % depuis le plus-haut de janvier pour se placer sur cette valeur entrée au SBF 120 en juin 2020.   C. L. C.
</article>
<article titre="ALBIOMA soutenu par des résultats annuels solides" date="2021 03 4" journal="Agence Option Finance (AOF)">
(AOF) - Albioma gagne 1,13% à 40,35 euros, soutenu par des résultats 2020 solides. Le spécialiste des énergies renouvelables a réalisé l'an dernier un résultat net part du groupe en hausse de 25 %, à 55,3 millions d'euros. L'Ebitda a progressé de 13 % à 206,4 millions pour un chiffre d'affaires stable à 506,7 millions. Hors effet prix des combustibles et l'effet change lié à la dégradation du réal brésilien, il est en progression de 4%. Fort de sa performance, le groupe a augmenté de 14% son dividende à 0,80 euro par action, avec option pour le paiement de 50 % en actions nouvelles. Pour 2021, Albioma table sur un Ebitda compris entre 206 à 216 millions et sur un résultat net part du groupe compris entre 53 à 59 millions d'euros (y compris l'acquisition de la centrale de géothermie de GümüÅ?köy en Turquie et hors effets éventuels liés à la Loi de finances 2021).   Albioma envisage d'engager entre 600 et 800 millions d'euros d'investissements sur la période 2021-2025 en conservant une structure financière solide.   LCM Midcap Partners a réitéré sa recommandation d'Achat et son objectif de cours de 54 euros sur le titre dans le sillage de résultats et de prévisions en ligne avec ses attentes.   "Le cercle vertueux de croissance rentable semble donc amené à se poursuivre, la baisse récente du titre sur fonds de hausse des taux longs semblant une opportunité pour revenir sur le dossier, clairement moins valorisé que ses comparables 'verts' ", a souligné le broker.   AOF - EN SAVOIR PLUS Points clés - Producteur d'énergie renouvelable -biomasse et photovoltaïque- né en 2013 des fusions successives entre Sidec, la Séchilienne et Albioma-Bois Rouge ;   - Chiffre d'affaires de 6,7 Mds€ réalisé à 41 % en Europe, 25 % dans les Amérique, 12 % en Asie Pacifique et 5 % en Moyen-Orient, Afrique, Russie et 17 % dans les câbles haute tension&amp;projets;    - Production réalisée à 33 % à partir du charbon, à 64 % avec la biomasse et 2 % le solaire; - Stratégie, adoptée en 2012, de réduction de la part du charbon dans la production thermique au profit de la biomasse et du solaire porter à partir des implantations aujourd'hui essentiellement dans l'outre-mer français (1er producteur solaire) puis à l'île Maurice et au Brésil (énergie à base de bagasse, résidu de la canne à sucre); - Capital éclaté avec 3 positions fortes -Invexans et Teck Pack pour 2 !,9 %, BPI France pour 7,7 % et les salariés pour 4,41 %-, Frédéric Moyne étant président-directeur général du conseil d'administration de 8 membres   - Situation financière saine avec une dette brute en baisse à 914 M€, dont 821 M€ de dette projet, face à une trésorerie de 122 M€.    Enjeux - Stratégie 2023visant à agir pour la transition énergétique en outre-mer, déployer l'international et accélérer le développement dans le solaire :    - 450 0 650 M€ d'investissements programmés et financés par emprunts bancaires,    - rationalisation du portefeuille avec cession des activités peu rentables,   - Stratégie d'innovation externalisée auprès du fonds d'amorçage géré par le fonds Demeter Emertec :    - biomasse thermique : recherche de filiales locales,    - stockage d'énergie électrique,    - services au réseau et maintenance prédictive;   - Stratégie environnementale :    - visant à la réduction primaire des émissions polluantes -rejets aqueux, traitement et valorisation sous-produits de combustion et rejets gazeux,   - validée par l'émission d'un « emprunt vert » rémunéré selon des indicateurs clés : +80 % d'énergies renouvelables en 2023 dans la production du groupe et + 95 % en 2030; -  Insensibilité de la rentabilité aux variations de prix des combustibles en raison des indexations contractuelles et visibilité de l'activité par la durée -30 ans- des contrats de vente ;   - Réalisation des projets solaires gouvernementaux remportés lors du 1er semestre pour 20MWc et mise en service de la centrale brésilienne de Vale do Parana  à la fin 2020 .   Défis - Impact négatif de la dépréciation du real brésilien;   - Forte sensibilité aux soutiens financiers publics, aux réglementations sectorielles et à l'accueil du public à de nouvelles centrales;   - Impact de la pandémie : hausse de 1 % du chiffre d'affaires à fin septembre;   - Réaction à la pandémie : économies de coûts, focus sur l'autofinancement et le renforcement des liquidités et la baisse du niveau de risque des projets; - Objectifs 2020 confirmés en base de fourchette d'un résultat d'exploitation entre 200 et 210 M€ et d'un résultat net entre 48 et 54 M€.
</article>
<article titre="Albioma : L'EBITDA progresse de 13%, à 206,4 millions d'euros" date="2021 03 4" journal="Boursier (site web)">
Le Conseil d'Administration d'Albioma, réuni le 4 mars 2021 sous la présidence de Frédéric Moyne, a arrêté les comptes consolidés du Groupe pour l'exercice 2020. Le chiffre d'affaires s'établit à 506,7 millions d'euros, stable par rapport à l'exercice précédent. Hors effet prix des combustibles (-13 millions d'euros) et l'effet change lié à la dégradation du réal brésilien (-7,1 millions d'euros), il est en progression de 4%. L'EBITDA de l'exercice progresse de 13%, à 206,4 millions d'euros grâce à :   la bonne performance opérationnelle des centrales sur l'ensemble des zones malgré la pandémie de COVID-19,   l'effet année pleine des derniers avenants en lien avec les investissements de mise aux normes IED des installations thermiques en Outre-mer,   la conversion à la biomasse de la tranche 3 d'Albioma Le Moule (ALM 3), depuis novembre dernier,   l'augmentation du prix de vente moyen au Brésil du fait de l'entrée en vigueur de nouveaux contrats.   Le résultat net part du Groupe (RNPG) est en hausse de 25%, à 55,3 millions d'euros.   Émission du premier "Sustainability-Linked Euro PP" pour un montant de 100 millions d'euros à 7 et 8 ans   Le 7 décembre 2020, Albioma a réussi l'émission de son premier Sustainability-Linked Euro PP, d'un montant significatif de 100 millions d'euros. Ce placement privé est composé de deux tranches à respectivement 7 et 8 ans, souscrites par des investisseurs institutionnels de premier plan. Les fonds levés ont été utilisés pour refinancer l'Euro PP existant et serviront pour les besoins de financement généraux du Groupe à moyen terme, afin d'accompagner Albioma dans la réalisation de son programme d'investissement. Ce financement, excédant de 20 millions d'euros le précédent Euro PP mis en place par le Groupe en 2014 et qui arrivait à échéance le 8 décembre 2020, témoigne de la dynamique de croissance dans laquelle s'inscrit l'entreprise et de la solidité de ses fondamentaux.   Avec la mise en place de ce financement, Albioma renforce son engagement en faveur de la transition énergétique, qui constitue l'un des piliers essentiels de sa stratégie de développement. Ainsi, Albioma s'engage dans le cadre de son financement à atteindre les cibles suivantes d'évolution de son mix énergétique (pour les sociétés consolidées en intégration globale) grâce à la conversion de l'ensemble des centrales historiques du Groupe dans les DOM au 100% biomasse (abandon total du charbon), dont le mouvement est entamé depuis 2018, et au développement de nouveaux projets ENR, notamment dans le solaire et la géothermie :   plus de 80% d'ENR en 2023, conformément à ses annonces précédentes ;   plus de 90% d'ENR en 2025.   Le Groupe a pour ambition d'atteindre d'ici 2030 une part d'énergie renouvelable de son mix entre 95% et 100%.   En fonction de l'atteinte ou non des objectifs définis, un mécanisme d'ajustement de la marge (bonus/malus) pouvant s'élever jusqu'à 25 points de base fera varier le taux d'intérêt applicable aux obligations.   La dette financière brute consolidée hors IFRS16 est en baisse en lien avec les remboursements des emprunts relatifs aux projets mis en service (IED, nouvelles centrales photovoltaïques). Elle s'élève à 901 millions d'euros à fin 2020, contre 938 millions d'euros à la fin de l'exercice 2019. La dette projet est de 789 millions d'euros (contre 834 millions d'euros à fin 2019). Le ratio dette nette/EBITDA 12 mois glissants s'améliore à 3,8x au 31 décembre 2020 (comparé à 4,3x au 31 décembre 2019).   La trésorerie du Groupe, incluant les dépôts de garantie, s'élève à 129 millions d'euros au 31 décembre 2020 contre 165 millions d'euros au 31 décembre 2019. La dette financière nette consolidée est de 772 millions d'euros (contre 773 millions d'euros à fin 2019).   Malgré le contexte économique dégradé, le Groupe n'a pas eu besoin de solliciter d'aides d'État mises en place pour soutenir les entreprises en difficulté, ni de tirer sur ses lignes de crédit court terme. Le Groupe conserve des moyens adaptés à la poursuite de son développement.   Dividendes   Le Conseil d'Administration proposera à l'Assemblée Générale des actionnaires la distribution d'un dividende de 0,80 euro par action, en hausse de 14% par rapport à 2019, avec option pour le paiement de 50% en actions nouvelles. Cette proposition s'inscrit dans le cadre d'une politique de croissance du dividende avec un objectif de distribution de l'ordre de 50% du résultat net part du Groupe hors exceptionnels.   Objectifs 2021   Pour 2021, le Groupe annonce des objectifs d'EBITDA de 206 à 216 millions d'euros et de résultat net part du Groupe de 53 à 59 millions d'euros (y compris l'acquisition de la centrale de géothermie de Gümü?köy en Turquie et hors effets éventuels liés à la Loi de finances 2021).   Perspectives   Le Groupe envisage d'engager entre 600 et 800 millions d'euros d'investissements sur la période 2021-2025 en conservant une structure financière solide.   Frédéric Moyne, Président-Directeur Général d'Albioma, a commenté : "Nous affichons pour l'exercice 2020 des résultats très solides, avec un EBITDA en croissance de 13% et un RNPG en forte hausse de 25%. Ces résultats sont le fruit du travail des équipes d'Albioma que je tiens particulièrement à remercier et à féliciter pour leur engagement durant toute la crise sanitaire. Malgré un contexte difficile, nous avons également finalisé sur l'exercice le chantier de conversion à la biomasse de notre centrale ALM3 en Guadeloupe, et mis en service dans les délais notre quatrième centrale bagasse au Brésil. Enfin, nous avons signé de nombreux projets en ligne avec notre stratégie qui contribueront significativement à la croissance des années prochaines : conversion et prolongation du contrat de la centrale de Bois-Rouge (abandon total du charbon) à La Réunion, gain de 40 MWc de solaire, entrée dans la géothermie.   Toutes ces actions nous rendent confiants dans notre capacité à atteindre les nouveaux objectifs que le Groupe se fixe pour 2025 et 2030 : respectivement plus de 90% et près de 100% de renouvelable dans notre mix énergétique".   Prochain rendez-vous : chiffre d'affaires du premier trimestre de l'exercice 2021, le 21 avril 2021 (après bourse).
</article>
<article titre="18 • Eocrep souhaite construire une unité dans le Cher" date="2021 03 3" journal="La Lettre Valloire">
Spécialisée dans la vente en ligne d'emballages pour les aliments, la TPE Eocrep voudrait construire un bâtiment de 4 000 m² à Rians ou à Crézancy-en-Sancerre. « Jusqu'alors, nos emballages à base de pulpe de cellulose étaient fabriqués par des industriels chinois. Nous souhaitons relocaliser la production en France », nous indique Pascal Braguier, le gérant d'Eocrep, qui s'apprête à investir 6 M€ dans un projet que soutiendrait Bpifrance : une vingtaine d'emplois seraient créés pour l'occasion.   « En Chine, la pulpe de cellulose est fabriquée à partir de la bagasse, le résidu fibreux de la canne à sucre. Nous la fabriquerons à partir du bois, puis de la paille d'ici à quelques années », explique Pascal Braguier qui aimerait s'implanter à Rians ou à proximité en raison de la présence des laiteries Triballat ; celles-ci envisageraient, en effet, de remplacer le plastique par la fibre de cellulose pour le conditionnement des yaourts.
</article>
<article titre="Distribution Réduction des plastiques : les enseignes accélèrent le pas" date="2021 03 1" journal="Process Alimentaire">
Aux rayons viandes et poissons libre-service, les annonces de substitution des barquettes en plastique par du carton ou un matériau compostable se succèdent.   La substitution partielle du plastique par du carton devient une réalité très concrète aux rayons viande d’Intermarché et de Netto. Pour l’instant 40 points de vente, mais dès le mois d’avril tout le parc de magasins proposera des références de viande en barquettes skin sur carton Halopack. Le fruit de 18 mois de R &amp; D. Co-développées par Agromousquetaires et ASV Packaging, fabricant exclusif de ces emballages néerlandais en France, ces barquettes mettent en exergue «75 % de carton» sur 31 références (15 références de pièces de boeuf, 4 de boeuf mariné, 2 de porc mariné et 10 références d’abats). «La barquette est composée à 90,2 % de carton et 9,8 % de liner. Auquel on ajoute le film skin supérieur qui varie de 100 à 150 µm. Soit 76 % à 80 % de carton au total», détaille Franck Gorré, chargé du développement d’Halopack chez ASV Packaging. Ce nouvel emballage remplace les barquettes en polystyrène expansé (PSE) operculées sous atmosphère protectrice utilisées pour les abats. Ainsi que des barquettes en plastique complexes avec film skin et flowpack opaque en polypropylène pour ce qui concerne le boeuf. De quoi économiser 38 tonnes de plastique à l’année. Des barquettes compostables «À plus long terme, cet emballage pourrait être développé sur d’autres références du rayon boucherie», annonce déjà Agromousquetaires. Au-delà de l’aspect durable, la technologie de pelliplaquage de film skin sur le carton (contenant 72 % de fibres recyclées et deux feuilles vierges recto/verso pour l’impression) permet de prolonger la durée de vie des produits, notamment des abats (un à deux jours supplémentaires). «Nous passons de six à huit jours par exemple sur les tranches de coeur de veau, de foie de veau, de porc, de boeuf et d’agneau par exemple», confirme-t-on chez Agromousquetaires.   De leur côté, tous les magasins Auchan de France commencent à remplacer les barquettes de viande et poisson en PSE noires et blanches de leurs rayons vente sur place par des barquettes compostables à domicile et recyclables dans la filière papiers-cartons. Ces emballages en fibres végétales sont conçus à partir de coproduits de canneà sucre (bagasse) par deux fournisseurs:Silver Plastics pour les barquettes Golden Fibres et Alliance Packaging pour les Ecotray. «Celles-ci sont, pour l’instant, produites en Asie dans l’attente de pouvoir démarrer une fabrication dans les DOM-TOM pour l’une et en Allemagne pour l’autre. Le projet est à l’étude», nous confie-t-on chez Auchan Retail. Ces emballages ont été testés en laboratoire puis en conditions réelles pendant l’année 2020. Conditionnés sous film rétractable sans injection de gaz, ils n’engendrent aucun changement sur les équipements des laboratoires des magasins. «Nous développons avec nos partenaires une gamme de barquettes adaptée au conditionnement sous atmosphère modifiée. Pour ce faire, nous devons prévoir une base de scellage et un film home-compostable garantissant la durée de vie du produit», annoncent les responsables du projet.   Dispensées de colle, elles disposeront d’ici 2022 d’étiquettes et de films d’emballage eux aussi certifiés compostables à domicile. Selon nos informations, ces barquettes devraient également faire leur apparition chez Leclerc, Intermarché et Carrefour. Karine Ermenier   Deux initiatives récentes Fournisseur: ASV Packaging (Halopack) ç 16 % des viandes piécées d’Intermarché et 6 % chez Netto. Surcoût pour le consommateur:0,43 €/unité Gain DLC: un à deux sur les abats Fournisseurs: Silver Plastics (barquettes Golden Fibres), Alliance Packaging (Ecotray) Toutes les viandes et poissons conditionnés sur place ç55 millions d’unités/an Pas de surcoût Gain DLC: un jour pour le poisson INTERMARCHÉ ET NETTO AUCHAN Intermarché et Netto ont pris l’option du skin sur carton plat avec, in fine, 75 % à 80 % de l’emballage recyclable dans la filière papiers-cartons. Auchan a opté pour une barquette en bagasse compostable à domicile ou recyclable dans la filière papiers-cartons.
</article>
<article titre="Le snacking soumis aux règles du développement durable" date="2021 03 1" journal="Néorestauration, no. 0594">
Le snacking a déferlé sur l'ensemble de la restauration. Et s'est considérablement développé avec la crise sanitaire. Longtemps monopole de la restauration rapide, cette façon de consommer sucré ou salé, en dehors de créneaux traditionnels des repas, s'est généralisée. La Covid-19 a rebattu les cartes, conduisant les restaurations traditionnelles et gastronomiques à investir ce créneau. Les professionnels ont revu leur offre, l'adaptant à la vente à emporter, à la livraison, au drive, au click et collect... Il a fallu repenser les plats et les menus en version facile à transporter. Les fournisseurs sont en première ligne pour trouver des alternatives à leurs conditionnements, capables de supporter le réchauffage sans dénaturer le produit ni nuire à la présentation. Les services R et D s'activent en quête d'emballages en phase avec les attentes sociétales et environnementales.  « L'engagement de Comatec s'articule autour de trois enjeux : la fabrication française (à 90 %), l'utilisation de matières recyclées et recyclables, et l'écoconception, grâce à ses recherches et au développement de produits alternatifs et son engagement sociétal (partenaire des Établissements et Services d'Aide par le Travail (ESAT) de l'Aude »,  explique Tatiana Rumeau, la directrice. Avant la pandémie, le fabricant avait lancé la sélection « spécial snacking » destinée aux petites et moyennes structures proposant de la VAE : une offre tout compris, frais de port inclus, sur des petits volumes, permettant d'acheter sans frais supplémentaires et sans avoir à sur-stocker. Et lors de la crise sanitaire, il a également créé les « Kits to go ». Treize solutions grâce auxquelles il n'est plus nécessaire  « d'acheter les composants séparément, pour rationaliser les commandes (formules bento, bois, faubourg...) ».  Désireuse d'aller plus loin dans son engagement, la société a établi un référentiel pour mesurer la performance environnementale de ses produits (voir l'encadré p. 26).  LE CLEAN LABEL PLÉBISCITÉ Les contenants doivent aussi s'adapter à leur contenu. Car bien que très attachés à leur gastronomie, les Français se montrent friands de cuisines ethniques.  « La cuisine japonaise représente une source d'inspiration forte pour l'univers du snacking européen de demain,  commente Justine Granger, chargée de communication et marketing chez Foodex, importateur et distributeur de produits alimentaires japonais.  Or, s'il est très prisé dans les établissements asiatiques, le ramen se prête mal à la vente à emporter. Les nouilles doivent être séparées du bouillon pour ne pas ramollir durant la livraison, le récipient ne doit pas fuir... Désormais, nous disposons de deux produits qui facilitent son transport : un packaging avec un compartiment pour les toppings et les nouilles, et un second pour le bouillon. Doté d'une double paroi pour éviter les brûlures, il est en plastique PP (donc recyclable). »,  décrit-elle.    De nombreuses solutions voient le jour.  « Pour apprécier les repas n'importe où en limitant les déchets, nous avons conçu 3 Flycup Up version 2 ou 3 composantes et petit déjeuner,  commente Olivier Rousseau, fondateur de Flycup . En adoptant une démarche responsable, car nos packagings sont recyclés, 100 % recyclables et biodégradables, et fabriqués en France. »  La Compagnie des Desserts avait également anticipé. Elle propose la gamme de petits pots de glaces ou sorbets bio « Glacier par nature » en carton kraft 100 % recyclable pour la restauration à table et pour la vente à emporter. Tous les fournisseurs sont sur le pont. Bel Foodservice a mis au point des solutions afin que les emballages soient réduits.  « Résultat : 75 % d'emballages prêts au recyclage et une ambition de 100 % pour 2025 »,  note Sarah Dupouy, chef de groupe foodservice France chez Bel.    Les Français sont aussi en attente de Clean Label. 77 % d'entre eux s'informent sur l'origine et la composition des produits lors de leurs achats (source : Baromètre Santé Nutrition INPES). Ils souhaitent manger plus de produits naturels et sains. D'où l'intérêt pour ce label, qui atteste que l'entreprise a réduit les allergènes, les additifs, le nombre d'ingrédients utilisés et qu'elle a remplacé les ingrédients artificiels par des produits naturels. Sans oublier d'utiliser des termes clairs sur l'étiquetage. Le consommateur veut donc de la transparence et que tous les fournisseurs affichent clairement leurs engagements et actions en faveur de la planète. Tous les produits sont concernés : du pain au fromage en passant par les légumes, les charcuteries...    UNE PRIORITÉ : MANGER SAIN ! La société Bridor s'est engagée dans une démarche de Responsabilité sociétale des entreprises (RSE). Cela se traduit par une sélection rigoureuse de ses ingrédients en privilégiant le local et l'origine France. La majorité de ses matières premières sont labellisées et certifiées (Label Rouge, agriculture raisonnée, AOP, Fairtrade, Agriculture Biologique...).  « Plus de 70 % de nos références sont Clean Label (pain, viennoiserie, brioche et snacking), soit tous nos pains classiques, croissants et pains au chocolat à la française. Nos recettes Clean Label sont composées d'ingrédients naturels et incontournables de la boulangerie »,  ajoute Valentin Gendry, chef de produits Bridor viennoiseries et snacking.  « La crise sanitaire a accéléré et renforcé les tendances sur l'alimentaire,  commente Sylvain Bertrand, directeur général de La Compagnie des Desserts.  Étant dans cette même démarche qualité par conviction, nos objectifs ont été de sélectionner les ingrédients pour leurs qualités organoleptiques, leurs origines ou leur proximité avec l'atelier, et de retravailler les recettes pour obtenir des listes courtes. »  Citons l'ensemble de la gamme de glaces au lait frais fabriquée à partir de crème et de sucre français, d'oeufs de poules élevées en plein air en France, sans arômes, sans colorants. Et d'ajouter :  « Le lait vient de la ferme Marg'Aude, une exploitation agricole de 50 vaches située au pied des Pyrénées. Le lait entier est livré directement après la traite (moins de 48 heures), puis il est pasteurisé dans les locaux de l'atelier avant d'être transformé en glace. » Traçabilité, transparence, liste courte, sourcing, origine... Tous ces critères vont dans le même sens : manger sain !  « 2020 a été synonyme de surprises, de turbulences, d'incertitudes, de questionnements... Mais surtout révélatrice d'une réelle prise de conscience et de changements sociétaux profonds,  souligne Sarah Dupouy, chez Bel.  Elle a été l'occasion de s'adapter, de se transformer, de se réinventer pour durer. Elle a permis de se confronter aux enjeux de solidarité et plus largement aux défis liés au mieux-manger, au local, à la naturalité des recettes proposées et, in fine, au durable. »  Le groupe affiche 220 recettes retravaillées depuis 2012, soit 40 % de son portefeuille. En matière d'ancrage local, Bel produit principalement en France. Avec, en ligne de mire pour sa division Foodservice, le cap des 20 % de produits issus de l'agriculture biologique, y compris en conversion, dans les menus proposés en restauration collective au 1er janvier 2022.    Depuis plusieurs années, végétal, véganisme, végétarisme et flexitarisme comptent de plus en plus d'adeptes. À l'écoute des attentes du marché, Bonduelle a créé Greenology, le service végétal 100 % gourmand destiné aux chefs.  « Notre mission consiste à accompagner les professionnels dans le végétal, leur proposer des solutions dans tous les segments de la restauration,  souligne Nicolas Dron, directeur marketing Europe Bonduelle foodservice.  La génération Z recherche beaucoup ce type d'offre bien-être dans le snacking, synonyme de plaisir, de santé et de sain. Nous avons développé des recettes à base de légumineuses, céréales... Nous souhaitons apporter notre pierre à l'édifice, avec notre accompagnement culinaire, donner des astuces, optimiser le végétal dans la VAE. »  Le légume a aussi une place dans les sandwichs, les burgers...  « Nous avons travaillé pour comprendre quelle place lui accorder dans ces recettes. En apportant du contraste de textures, de couleurs, de goûts, par l'ajout de roquette ou des poivrons grillés, oignons. On choisit avec les yeux, on peut être créatif dans la partie végétale, car le végétal est un marqueur de la fraîcheur. »  Bonduelle porte une attention à l'amont agricole, sur la qualité organoleptique dès la base. La surgélation de ses légumes intervient quelques heures après la récolte. Dans un souci de naturalité, la marque a lancé une gamme sans résidu de pesticides. Elle affiche sur ses produits depuis 2020 le Nutri-score. Le logo, non obligatoire, concerne tous ses produits français en conserves, surgelés, frais et traiteur. Un engagement national s'inscrivant dans l'ambition du groupe d'être le référent mondial du bien-vivre par l'alimentation végétale. 96 % de ses produits sont classés A et B.    EXPLOSION DES SNACKS SALÉS « Les snacks salés explosent, avec une croissance mondiale annuelle de 7,9 %   (source : Cabinet Canadean), rappelle Valentin Gendry, chef de produits Bridor viennoiseries et snacking.  Nous avons lancé des nouveautés en petits formats. Des mini-roulés fromage, mini-roulés pizza, mini-roulés pesto en version 30 ou 35 g. À proposer chauds ou froids, pour différents instants de consommation. »  Et Tarti's, un snacking inspiré des tartes salées en format de 85 g. Une forme très pratique adaptée à la consommation « on-the-go », permettant de manger facilement en marchant. Aoste Professionnel s'est également engagé en faveur du local et de la qualité. La majorité des jambons secs Aoste est labellisée VPF (Viande de Porc Français) et sans conservateurs depuis 2018. Outre l'origine et les qualités organoleptiques des produits, les professionnels ont toujours autant besoin de praticité, de fonctionnalité et d'innovation !  « La marque avait déjà commencé à travailler sur les ingrédients culinaires pour plus de praticité dans les sandwichs et les salades. La restauration a évolué, il faut proposer des produits déjà prêts à l'utilisation, déjà découpés, portionnables pour un gain de temps. Pour proposer les produits dont les professionnels ont besoin, nous les interrogeons, les observons. Nous détectons certains gestes rébarbatifs et envisageons de remédier, de rénover ou de revoir la découpe,  indique Lise-Marie Potet, chef de groupe marketing (Food service) Groupe Aoste.  Il faut aussi veiller à la polyvalence des produits. Nos gammes se sont enrichies. Sont désormais disponibles des allumettes, des dés adaptés aussi bien aux salades qu'aux pizzas... Ils ont le choix entre du jambon Serrano, de Savoie, du chorizo... »  Malgré la fermeture prolongée des établissements de restauration, les fournisseurs n'ont pas cessé d'innover. Aoste Professionnel a enrichi ses gammes avec le guanciale et la pancetta à la marque Fiorucci.  « Prenez un cochon bien dodu. Le gras du ventre, on en fait de la pancetta, le gras du dos, on en fait du lard, et le gras de la joue, on en fait du guanciale »,  explique encore Lise-Marie Potet. Les deux ingrédients charcutiers sont issus de la poitrine de porc et salés au sel sec, une découpe en timbre conférant un joli visuel dans l'assiette et une texture croustillante de viande et une mâche en bouche plus intéressante que le lardon. Soit un gain de temps de découpe et du prêt-à-cuisiner.
</article>
<article titre="LA PRESSE EN HAÏTI Un projet québécois pour aider à reboiser" date="2021 02 23" journal="La Presse+">
Acul Samedi, Haïti - — Les acacias, semés il y a moins de deux ans, le dépassent déjà largement. « Les arbres poussent plus vite », s'enthousiasme Abner Florvil, qui envisage une première coupe dès cette année, pour faire du charbon, alors qu'il faut d'ordinaire patienter trois à quatre ans. Cette croissance rapide de la végétation est le résultat d'un projet québécois de reboisement dans le nord-est d'Haïti, misant sur des « forêts énergétiques » à haute densité, piloté par l'entreprise Viridis Terra et le Centre d'étude et de coopération internationale (CECI), une organisation non gouvernementale, et qui a obtenu un financement de 1 million de dollars de Québec par l'entremise du Fonds vert.   Il mise sur des essences qui ont une croissance rapide, mais aussi un « fort pouvoir calorifique », comme l'acacia, le faux mimosa (leucaena leucocephala) et le bayahonde (prosopis juliflora), explique l'agronome Frény Alcinat, du CECI.   « C'est un bois qui produit beaucoup plus de chaleur pour une quantité donnée, qui est plus résistant, donc qui prend plus de temps à brûler. » — Frény Alcinat, agronome Le charbon demeure massivement utilisé pour la cuisson des aliments en Haïti, où de vastes parties du pays ne sont pas électrifiées et, pour celles qui le sont, l'approvisionnement est défaillant ; le reboisement doit donc en tenir compte.   Viridis Terra calcule qu'il suffirait d'implanter sur 7 % du territoire d'Haïti sa méthode de forêts énergétiques à haute densité pour combler la totalité de la demande en charbon du pays, et donc de renverser le rythme de la déforestation.   Reboiser efficacement La croissance rapide des arbres semés par le CECI et Viridis Terra est d'autant plus surprenante qu'elle se produit sur une terre qui était dégradée, où plus rien ne poussait très bien.   « Auparavant, cette parcelle était une sorte de brousse », raconte Frény Alcinat, du CECI.   Car la déforestation n'entraîne pas que la disparition du couvert forestier ; elle contribue à l'appauvrissement des sols, dont les nutriments sont lessivés par l'érosion, et à l'assèchement des nappes phréatiques.   Alors qu'Haïti subit de plein fouet les effets des changements climatiques, la déforestation et ses conséquences rendent le pays encore plus vulnérable aux aléas du climat ; de nombreux bailleurs de fonds financent donc des projets de reboisement.   Celui du CECI et de Viridis Terra, baptisé KLIMA (acronyme venant de son appellation créole, « konbit pou lite kont chanjman klimatik », qui signifie « travailler ensemble pour lutter contre les changements climatiques »), utilise les techniques développées par la jeune entreprise de Lévis dans la restauration de sites miniers au Canada.   Il se distingue entre autres parce qu'il recourt au semis direct plutôt qu'à la plantation, mais aussi par les travaux préalables de préparation du sol, notamment avec du compost.   « C'est ce qui a permis d'installer des forêts là où d'autres projets ont moins bien fonctionné », affirme Catherine Ruest Bélanger, directrice du département de foresterie et d'agroforesterie de Viridis Terra.   À certains endroits, les terres sont si dégradées qu'elles ne peuvent même pas répondre aux besoins énergétiques d'un jeune plant, d'où l'intérêt de partir de la semence, explique l'ingénieure forestière, qui était en Haïti avant les Fêtes pour observer l'évolution du projet.   « On a calculé que c'est 10 fois moins cher, le semis direct. » — Catherine Ruest Bélanger, ingénieure en foresterie chez Viridis Terra L'entreprise prévoit d'utiliser ce modèle de reboisement pour régénérer des terres dégradées avec du financement privé dans des pays plus développés.   Charbon « carboneutre » Contrairement à ce qu'il faisait dans le passé, Abner Florvil ne récoltera pas tous les arbres de sa parcelle, cette fois, et, surtout, il s'assura de les couper à au moins 70 cm du sol, ce qui créera des « rejets » sur le tronc.   « Les rejets poussent plus vite », puisqu'ils bénéficient du système racinaire déjà bien implanté de l'arbre, explique-t-il, envisageant un gain de temps appréciable.   En créant un cycle où les arbres transformés en charbon sont remplacés par d'autres arbres qui en produiront à leur tour, après avoir séquestré le carbone émis par les premiers, KLIMA permet de produire un charbon « carboneutre », estiment ses instigateurs, rappelant que l'objectif premier du projet est la réduction des gaz à effet de serre (GES).   D'autant plus qu'un sol en santé, comme ceux régénérés dans le cadre du projet, séquestre aussi beaucoup de carbone, expliquent-ils.   Et cela entraîne un autre bénéfice : rendre ces terres dégradées cultivables à nouveau, selon le principe de l'agroforesterie, où différentes cultures se côtoient sur une même parcelle et où poussent aussi des arbres.   « Juste le fait que les feuilles d'arbre tombent, ça amène une couverture de matière organique, ça amène des nutriments, ça protège le sol, et les racines protègent contre l'érosion, retiennent l'eau. » — Catherine Ruest Bélanger, ingénieure en foresterie chez Viridis Terra L'utilisation d'essences issues de la grande famille des légumineuses, qui « fixent l'azote dans le sol » et l'enrichissent, est aussi un atout.   Nouveau projet en démarrage Trois ans après sa mise sur pied, après avoir reboisé 200 hectares de terres dégradées, le projet KLIMA tire à sa fin, dans le nord-est d'Haïti, mais un autre verra bientôt le jour dans le sud du pays.   Le CECI et Viridis Terra travaillent cependant à trouver une façon de poursuivre le travail commencé dans le Nord-Est, enthousiasmés par le succès obtenu.   « Ça a marché mieux qu'on pensait », se réjouit Catherine Ruest Bélanger.   Le taux de survie des arbres semés a atteint 80 %, alors que l'objectif était tout au plus de 60 %, et ce, en dépit de la sécheresse de 2019 qui a duré près de 10 mois.   « On a été obligés d'aller puiser de l'eau dans la rivière Marion pour arroser les plantations [manuellement], raconte Frény Alcinat. Sinon, on aurait tout perdu. »   Ce reportage a été réalisé avec une bourse du Fonds québécois pour le journalisme international.   La Presse a compensé par l'achat de crédits carbone les émissions de gaz à effet de serre engendrées par les déplacements aériens et terrestres liés à ce reportage.   Réalisations du projet KLIMA Carbonisation du charbon améliorée Le projet KLIMA a trouvé une façon de réduire les GES émis lors de la fabrication du charbon, en améliorant la technique traditionnelle de carbonisation, qui consiste à faire une « meule », un empilement soigné de bois recouvert de feuilles et scellé avec de la boue. La « meule améliorée » implique de placer des tuyaux sous le bois pour contrôler l'entrée d'air et d'installer une cheminée qui permet de récupérer les gaz sous forme liquide, en provoquant leur condensation. De plus, cette « meule améliorée » donne de meilleurs rendements ; elle peut produire jusqu'à trois fois plus de charbon avec la même quantité de bois que la technique traditionnelle. Elle a aussi l'avantage d'être mobile, contrairement à un four de carbonisation de briques.   Création d'un centre de compostage Pour que les arbres poussent sur des terres dégradées, il fallait leur donner un coup de pouce avec du compost. KLIMA s'est approvisionné auprès d'une organisation locale, qu'elle a ensuite aidée à mettre sur pied un centre de compostage industriel. L'infrastructure, qui venait d'amorcer ses activités lors du passage de La Presse, devrait atteindre son rythme de croisière d'ici la fin de l'été et produire de 200 à 300 tonnes de compost par année. Elle compostera notamment la bagasse – le résidu de la canne à sucre après qu'elle a été pressée –, le fumier et les déchets organiques des marchés de la région, mais aussi les résidus alimentaires des hôtels, restaurants et marchés de Cap-Haïtien. Ce compost servira notamment au nouveau projet de reboisement du CECI et de Viridis Terra dans le sud du pays.   La production de charbon en Haïti  Cet article est paru dans La Presse+
</article>
<article titre="La Presse en Haïti Un projet québécois pour aider à reboiser Haïti" date="2021 02 23" journal="La Presse (site web)">
Acul Samedi, Haïti - La déforestation, causée notamment par la production de charbon, est un problème criant en Haïti. Dans le nord du pays, une entreprise et une ONG québécoises mènent un projet pour contrer la tendance en plantant des arbres à croissance rapide et à fort rendement énergétique. Le reboisement permet de régénérer les sols, afin qu'ils soient cultivables à nouveau, et de capter du gaz carbonique.   Les acacias, semés il y a moins de deux ans, le dépassent déjà largement.    « Les arbres poussent plus vite », s'enthousiasme Abner Florvil, qui envisage une première coupe dès cette année, pour faire du charbon, alors qu'il faut d'ordinaire patienter trois à quatre ans. Cette croissance rapide de la végétation est le résultat d'un projet québécois de reboisement dans le nord-est d'Haïti, misant sur des « forêts énergétiques » à haute densité, piloté par l'entreprise Viridis Terra et le Centre d'étude et de coopération internationale (CECI), une organisation non gouvernementale, et qui a obtenu un financement de 1 million de dollars de Québec par l'entremise du Fonds vert.    Il mise sur des essences qui ont une croissance rapide, mais aussi un « fort pouvoir calorifique », comme l'acacia, le faux mimosa  (leucaena leucocephala)  et le bayahonde  (prosopis juliflora),  explique l'agronome Frény Alcinat, du CECI.   « C'est un bois qui produit beaucoup plus de chaleur pour une quantité donnée, qui est plus résistant, donc qui prend plus de temps à brûler. » – Frény Alcinat, agronome   Le charbon demeure massivement utilisé pour la cuisson des aliments en Haïti, où de vastes parties du pays ne sont pas électrifiées et, pour celles qui le sont, l'approvisionnement est défaillant ; le reboisement doit donc en tenir compte.    Viridis Terra calcule qu'il suffirait d'implanter sur 7 % du territoire d'Haïti sa méthode de forêts énergétiques à haute densité pour combler la totalité de la demande en charbon du pays, et donc de renverser le rythme de la déforestation.   Reboiser efficacement La croissance rapide des arbres semés par le CECI et Viridis Terra est d'autant plus surprenante qu'elle se produit sur une terre qui était dégradée, où plus rien ne poussait très bien.    « Auparavant, cette parcelle était une sorte de brousse », raconte Frény Alcinat, du CECI.    Car la déforestation n'entraîne pas que la disparition du couvert forestier ; elle contribue à l'appauvrissement des sols, dont les nutriments sont lessivés par l'érosion, et à l'assèchement des nappes phréatiques.   Alors qu'Haïti subit de plein fouet les effets des changements climatiques, la déforestation et ses conséquences rendent le pays encore plus vulnérable aux aléas du climat ; de nombreux bailleurs de fonds financent donc des projets de reboisement.    Celui du CECI et de Viridis Terra, baptisé KLIMA (acronyme venant de son appellation créole, « konbit pou lite kont chanjman klimatik », qui signifie « travailler ensemble pour lutter contre les changements climatiques »), utilise les techniques développées par la jeune entreprise de Lévis dans la restauration de sites miniers au Canada.   Il se distingue entre autres parce qu'il recourt au semis direct plutôt qu'à la plantation, mais aussi par les travaux préalables de préparation du sol, notamment avec du compost.    « C'est ce qui a permis d'installer des forêts là où d'autres projets ont moins bien fonctionné », affirme Catherine Ruest Bélanger, directrice du département de foresterie et d'agroforesterie de Viridis Terra.    À certains endroits, les terres sont si dégradées qu'elles ne peuvent même pas répondre aux besoins énergétiques d'un jeune plant, d'où l'intérêt de partir de la semence, explique l'ingénieure forestière, qui était en Haïti avant les Fêtes pour observer l'évolution du projet.   « On a calculé que c'est 10 fois moins cher, le semis direct. » – Catherine Ruest Bélanger, ingénieure en foresterie chez Viridis Terra   L'entreprise prévoit d'utiliser ce modèle de reboisement pour régénérer des terres dégradées avec du financement privé dans des pays plus développés.    Charbon « carboneutre »  Contrairement à ce qu'il faisait dans le passé, Abner Florvil ne récoltera pas tous les arbres de sa parcelle, cette fois, et, surtout, il s'assura de les couper à au moins 70 cm du sol, ce qui créera des « rejets » sur le tronc.    « Les rejets poussent plus vite », puisqu'ils bénéficient du système racinaire déjà bien implanté de l'arbre, explique-t-il, envisageant un gain de temps appréciable.    En créant un cycle où les arbres transformés en charbon sont remplacés par d'autres arbres qui en produiront à leur tour, après avoir séquestré le carbone émis par les premiers, KLIMA permet de produire un charbon « carboneutre », estiment ses instigateurs, rappelant que l'objectif premier du projet est la réduction des gaz à effet de serre (GES).   D'autant plus qu'un sol en santé, comme ceux régénérés dans le cadre du projet, séquestre aussi beaucoup de carbone, expliquent-ils.    Et cela entraîne un autre bénéfice : rendre ces terres dégradées cultivables à nouveau, selon le principe de l'agroforesterie, où différentes cultures se côtoient sur une même parcelle et où poussent aussi des arbres.   « Juste le fait que les feuilles d'arbre tombent, ça amène une couverture de matière organique, ça amène des nutriments, ça protège le sol, et les racines protègent contre l'érosion, retiennent l'eau. » – Catherine Ruest Bélanger, ingénieure en foresterie chez Viridis Terra   L'utilisation d'essences issues de la grande famille des légumineuses, qui « fixent l'azote dans le sol » et l'enrichissent, est aussi un atout.    Nouveau projet en démarrage Trois ans après sa mise sur pied, après avoir reboisé 200 hectares de terres dégradées, le projet KLIMA tire à sa fin, dans le nord-est d'Haïti, mais un autre verra bientôt le jour dans le sud du pays.    Le CECI et Viridis Terra travaillent cependant à trouver une façon de poursuivre le travail commencé dans le Nord-Est, enthousiasmés par le succès obtenu.    « Ça a marché mieux qu'on pensait », se réjouit Catherine Ruest Bélanger.    Le taux de survie des arbres semés a atteint 80 %, alors que l'objectif était tout au plus de 60 %, et ce, en dépit de la sécheresse de 2019 qui a duré près de 10 mois.    « On a été obligés d'aller puiser de l'eau dans la rivière Marion pour arroser les plantations [manuellement], raconte Frény Alcinat. Sinon, on aurait tout perdu. »    > Ce reportage a été réalisé avec une bourse du Fonds québécois pour le journalisme international.    >  La Presse  a compensé par l'achat de crédits carbone les émissions de gaz à effet de serre engendrées par les déplacements aériens et terrestres liés à ce reportage.    Des réalisations du projet KLIMA Carbonisation du charbon améliorée Le projet KLIMA a trouvé une façon de réduire les GES émis lors de la fabrication du charbon, en améliorant la technique traditionnelle de carbonisation, qui consiste à faire une « meule », un empilement soigné de bois recouvert de feuilles et scellé avec de la boue. La « meule améliorée » implique de placer des tuyaux sous le bois pour contrôler l'entrée d'air et d'installer une cheminée qui permet de récupérer les gaz sous forme liquide, en provoquant leur condensation. De plus, cette « meule améliorée » donne de meilleurs rendements ; elle peut produire jusqu'à trois fois plus de charbon avec la même quantité de bois que la technique traditionnelle. Elle a aussi l'avantage d'être mobile, contrairement à un four de carbonisation de briques.    Création d'un centre de compostage Pour que les arbres poussent sur des terres dégradées, il fallait leur donner un coup de pouce avec du compost. KLIMA s'est approvisionné auprès d'une organisation locale, qu'elle a ensuite aidée à mettre sur pied un centre de compostage industriel. L'infrastructure, qui venait d'amorcer ses activités lors du passage de  La Presse,  devrait atteindre son rythme de croisière d'ici la fin de l'été et produire de 200 à 300 tonnes de compost par année. Elle compostera notamment la bagasse - le résidu de la canne à sucre après qu'elle a été pressée -, le fumier et les déchets organiques des marchés de la région, mais aussi les résidus alimentaires des hôtels, restaurants et marchés de Cap-Haïtien. Ce compost servira notamment au nouveau projet de reboisement du CECI et de Viridis Terra dans le sud du pays.  Cet article est paru dans La Presse (site web)
</article>
<article titre="La Réunion : Albioma se convertit à la biomasse" date="2021 02 15" journal="Recyclage Récupération">
Albioma a annoncé l'abandon total du charbon sur son site historique de Bois-Rouge à La Réunion, après délibération de la Commission de régulation de l'énergie (CRE). Les travaux de conversion débuteront dès 2021 pour que la centrale fonctionne 100 % à la biomasse au second semestre 2023, en privilégiant les gisements locaux disponibles (bagasse, bois forestier, bois d'élagage, etc.), complétés par des granulés de bois importés issus de forêts certifiées type FSC et PEFC, dont la procédure de traçabilité sera en conformité avec le Règlement sur le bois de l'Union européenne (RBUE). À terme, la conversion fera passer la part renouvelable du mix énergétique de La Réunion de 35 % à 51 % et permettra de réduire les émissions de GES d'environ 640000 tonnes équivalent CO2 par an, soit une baisse de 84 % des émissions directes par rapport au fonctionnement actuel de la centrale.
</article>
<article titre="De la fibre artistique et durable" date="2021 02 11" journal="Réponses Photo">
De la fibre artistique et durable Papiers jet d'encre   L es papiers jet d'encre offrent une diversité de matières et de surfaces que les papiers argentiques n'ont jamais atteinte. Ils partagent pourtant des bases similaires, en matières naturelles ou synthétiques. Oublions le plastique pour notre sujet. L'écrasante majorité des papiers est fabriquée à partir de la cellulose du bois. D'où la mention d'alpha-cellulose (la plus stable et la plus pure des celluloses) pour des papiers comme le Canson Baryta Photographique, le Fine Art Torchon de Hahnemühle, etc.   Pour des applications particulières, on utilise parfois des fibres provenant d'autres végétaux : abaca, alfa, bagasse, bambou, buisson à papier, chanvre, coton, lin, mûrier,  Wikstrœmia , etc. Mais leur exploitation représente moins de 10% des pâtes à papier végétales produites dans le monde. Ce sont pourtant leurs fibres qui nous intéressent ici, parce qu'elles sont la source de papiers alternatifs, dont la main, la surface, l'épaisseur ou la transparence apportent un peu de fraîcheur dans le monde de la photographie, aussi bien pour la pratique des procédés anciens que du jet d'encre.    Écologique  Depuis le Sommet Planète Terre de Rio en 1992, les politiques de développement durable concernant l'exploitation des forêts, notamment pour le papier, se sont renforcées. En Europe, elles sont très présentes en Scandinavie et en Finlande, gros fournisseurs pour l'industrie papetière. Les fabricants de papiers d'art diversifient leurs sources avec ce besoin très actuel de préservation de la planète. Avec sa gamme Natural Line, composée de bambou, de chanvre et d'agave, Hahnemühle recourt à des plantes à croissance rapide, qui fournissent beaucoup de fibres avec un rendement élevé. Autre point positif, elles consomment moins d’eau et n’ont pas besoin de pesticides. Ce qui n’est pas le cas du coton, même si les linters de coton, ces fibres courtes utilisées pour le papier, proviennent des rejets de l’industrie textile qui n’exploite que les fibres longues de la plante. Le coton du papier est donc issu d’une forme de recyclage. Quoi qu’il en soit, pour optimiser les caractéristiques du papier (main, résistance, texture), la marque allemande intègre plus ou moins de fibres de coton dans ses compositions : 90 % de fibres de bambou + 10 % coton, 60% de fibres de chanvre + 40% de coton, 70% de fibres d’agave + 30% de coton. Chaque papier possède le même grammage de 290 g.    En Chine, le bambou et le chanvre sont employés depuis plus de 2000 ans pour fabriquer du papier. En 105 apr. J.-C., Caï Lun, chef des ateliers impériaux sous les Han, codifie l’art de sa fabrication. Outre le bambou et le chanvre, il préconise les fibres du lin et d’écorces de mûrier. En 610, Doncho, prêtre bouddhiste coréen, introduit au Japon ce savoir-faire. Après quelques évolutions techniques naît le papier japonais, le fameux washi , qui emploie principalement du mûrier (kôzo) et du chanvre, dont les fibres sont utilisées localement depuis longtemps pour le textile.    Le gampi, un arbuste non cultivable, peut aussi fournir ses fibres pour la fabrication de la matière première. Ces plantes favorisent une exploitation durable de l'écosystème : elles arrivent à maturité entre un à deux ans, alors qu'il en faut dix fois plus pour les arbres exploités pour la pâte de bois. L'artisanat du papier s'est notamment développé dans la région de Tokushima, sur l'île de Shikoku, où l'entreprise Awagami est née à la fin du XVIIIe siècle. Elle continue de produire des papiers pour les arts traditionnels et propose depuis une douzaine d'années des versions pour le jet d'encre ou les procédés anciens comme le platine. Les papiers sont fabriqués industriellement pour le Bamboo ou le Kozo et à la main pour le Bizan.    Les labos pros s'y mettent  Certes moins présent que Hahnemühle chez les distributeurs de matériel photo, plusieurs spécialistes proposent ces papiers japonais, comme Prophot, Graphic Réseau ou Taos Photo. Signalons que deux papiers Awagami, Unryu 55, et Kozo 110, sont disponibles sous la marque américaine Moab Moenkopi, distribuée par Lumière Imaging (www.lumiere-imaging.fr). Sur son site Internet (www.awagami.com), pas moins de six labos français, utilisateurs des Bizan, Inbe, Kozo, Murakumo ou Unryu, sont mentionnés : Art Photo Lab, Phosfen Lab, LeboLabo, Atelier Franck Bordas, Picto et Retine Argentique. Avant de se lancer soi-même dans l'impression avec sa propre imprimante, il peut être pertinent de faire un essai chez l'un deux. Picto propose un A4 à 12 €. De toutes façons, le papier n'est pas donné. Au plus bas, il en coûtera 25 € les 20 feuilles pour l'Awagami Bamboo 250 g en A4. Prix qui grimpe à 85 € pour 5 feuilles A4 de Bizan 200 g. Hahnemühle n'est pas en reste : entre 48 et 52 € pour sa Natural Line (A4, 25 feuilles). Le développement durable a un prix.    Un choix esthétique  Quand on aime, on ne compte pas. Tous les propriétaires d'imprimantes jet d'encre savent déjà que l'encre est un cher liquide. Pour éviter de se ruiner, Awagami et Hahnemühle proposent une pochette d'échantillons. Mais autant la Natural Line de Hahnemühle est surtout un manifeste écoresponsable, car le rendu est proche d'un Photo Rag, autant les papiers Awagami apportent une touche esthétique singulière, avec leur texture particulière, la transparence d'un Kozo de 42 g, ou l'aléatoire de l'incrustation de l'image avec les fibres apparentes de mûrier.   GAMME DES PAPIERS AUX FIBRES ALTERNATIVES La Chine a inventé le papier il y a 2000 ans à partir de fibres de bambou, de mûrier, de lin et de chanvre. A l’instar du regain d’intérêt pour les procédés anciens, l’industrie du jet d’encre revisite le passé et s’adapte au goût d’un public en quête de diversité. En France, Awagami (www.awagami. com) et Hahnemühle (www.hahnemuehle.com) sont les deux marques qui proposent des papiers à base de fibres naturelles alternatives, tendance éco-responsable. Leurs surfaces sont mates. Le grammage des Hahnemühle est de 290 g alors qu’Awagami joue sur une gamme de 42 à 250 g.    Awagami et Hahnemühle proposent leurs papiers aux fibres exotiques en pack de test pour se faire la main sur ces supports. Le pack Awagami ne comporte pas moins de 18 échantillons différents, à base de fibres de bambou, chanvre, mûrier, Edgeworthia, etc. Les 3 papiers Hahnemühle de la gamme Natural, sont fabriqués respectivement à partir d’agave, de bambou et de chanvre.    Les papiers Awagami ont des épaisseurs et des structures variées. SURFACES ET TEXTURES Les surfaces des papiers mats vont du lisse (par pressage à chaud) au plus ou moins texturé (par pressage à froid). Celle du Bamboo Hahnemühle est lisse, à peine bosselée. Les Hemp et Agave rappellent la légère texture d'un Photo Rag. La gamme Awagami fournit une plus grande variété de textures, du lisse à petit grain (Bamboo) aux surfaces fibreuses des Bizan fabriqués à la main.   ESPACES DE COULEUR Avantage des papiers mats sur les papiers brillants : ils ne montrent pas de reflets indésirables des sources de lumière. La contrepartie du mat est une moindre saturation des couleurs. Le risque de constater des couleurs non imprimables est plus grand quand on simule une impression à l'écran (“épreuvage-écran” dans Lightroom ou “Conditions d'épreuvage” dans Photoshop). Avec un profil ICC, le mode de rendu “perception” évitera une perte de différenciation des couleurs saturées.   LES BONS RÉGLAGES DANS LE PILOTE D'IMPRESSION Les pilotes des imprimantes jet d'encre Canon et Epson comportent une liste de papiers de la marque du constructeur. Avec leurs supports, aucun risque de confusion n'est possible. Les papiers d'autres marques, avec leurs appellations et leurs caractéristiques différentes, demandent une attention particulière. Car l'imprimante fonctionne de façon spécifique pour chaque type de papier sélectionné dans le pilote. Les profils ICC fournis par Awagami (www.awagami.com) et Hahnemühle (www. hahnemuehle.com) spécifient pour chaque papier la sélection du support dans le pilote Canon ou Epson.
</article>
<article titre="Restauration : avec la crise sanitaire, le retour en force du jetable Les livraisons et le « click and collect » sont à l'origine d'une forte augmentation des déchets de plastique" date="2021 01 30" journal="Le Monde">
Derrière la vitrine, des montagnes de boîtes à pizza. Au cours de la journée, elles font comme les russes : elles descendent et elles remontent aussi sec. Tous les midis, et le soir avant le couvre-feu, c'est l'attroupement devant cette pizzeria du 12e arrondissement : les habitués qui déjeunaient entre collègues attablés dans la salle ou en terrasse continuent à venir chercher leur margherita ou leur burrata, de même que les habitants du quartier passés au télétravail, les livreurs à scooter ou à vélo. « Certains jours, on travaille même mieux qu'avant le Covid », ditBéa, l'une des serveuses. Béa a une explication, imparable : « Les gens en ont marre de faire à manger. » Une bouffée d'oxygène pour des restaurateurs qui désespèrent d'obtenir le feu vert du gouvernement pour rouvrir leurs établissements. Une plaie supplémentaire pour la planète. Car le boom de la vente à emporter a son revers. Il se mesure dans les poubelles. Et ces derniers temps, dans les rues ou les cours d'immeubles, elles ont tendance à friser l'indigestion. De boîtes à pizza, mais aussi de boîtes à burger, de boîtes à kebab, de boîtes à sushis. De barquettes en plastique, en aluminium, en carton pour le bobun, la salade de papaye, le riz cantonnais, les nems, la soupe miso, les litchis, le tapioca, la petite sauce aigre-douce, la petite sauce soja. Mais aussi les paires de baguettes et les couverts jetables et leur étui de protection. Mais aussi les serviettes en papier. Mais aussi les canettes, les gobelets, les petites bouteilles d'eau. Et bien sûr, les sacs pour transporter tous ces mets et leurs emballages jusqu'à la maison ou au bureau.   « Un peu ahurissante » A la Mairie de Paris, le phénomène est jugé « préoccupant . « On voit bien que le plastique à usage unique refait surface dans nos vies de façon un peu ahurissante », réagit Colombe Brossel, adjointe chargée de la propreté, du tri et de la réduction des déchets. Elle prend l'exemple de « ces pommes vendues à l'unité dans un sachet en plastique . Elle aurait aussi pu citer ces sablés aux amandes ou ces gâteaux cocos emballés à l'unité proposés en dessert par certains commerçants. « Augmentation ou explosion ? On est techniquement incapables de quantifier et d'objectiver le phénomène, mais ce que nous disent les éboueurs de la ville, c'est qu'il y a un vrai retour des déchets en plastique dans l'espace public », commentel'élue.   Le Syndicat mixte central de traitement des ordures ménagères (Syctom) confirme la tendance. Il constate une hausse de 29 % des déchets issus du tri sélectif (les poubelles jaunes) en décembre 2020 par rapport à décembre 2019. Premier opérateur européen, le Syctom traite notamment les quelque 2,3 millions de tonnes de déchets générés chaque année par six millions d'habitants de 85 communes d'Ile-de-France, dont Paris, et attribue cette forte progression à « une modification des pratiques de consommation : achats en ligne, consommation à domicile du fait de la fermeture des restaurants .   Parmi les mutations précipitées par la crise sanitaire, la mode du « click and collect » : passer commande sur son smartphone en espérant venir la récupérer sans trop faire la queue. De plus en plus de restaurateurs, à commencer par les chaînes de fast-food, affichent le macaron en taille XXL sur leur devanture. Selon les dernières estimations des professionnels de la restauration, en novembre 2020, 62 % des 6 000 restaurants parisiens proposaient un service de vente à emporter. « La moitié l'a mis en place en réaction au Covid », précise Franck Trouet, porte-parole du Groupement national des indépendants de l'hôtellerie et de la restauration.   L'explosion de la vente à emporter s'accompagne de celle de la livraison. Le midi, mais aussi le soir, après le couvre-feu, lorsque les rues se vident, les livreurs continuent de sillonner la ville à scooter ou à vélo, leur besace pleine à craquer de plats aussi soigneusement préparés qu'emballés. Uber Eats, l'une des principales plates-formes de livraison de repas, a bien voulu partager ses chiffres. Ils permettent de mieux appréhender la hauteur de la vague. Plus 100 % d'activité en France (230 agglomérations desservies) au deuxième trimestre 2020 par rapport à celui de 2019. Plus 130 % entre août 2019 et août 2020. Entre mars et juin, 5 000 nouveaux restaurants (+ 100 %) et autant de livreurs ont rejoint l'application, téléchargée 12,5 millions de fois en 2020.   « Montagnes de sacs » Selon les estimations, réalisées avant la pandémie, de l'Agence de l'environnement et de la maîtrise de l'énergie (Ademe), 15 tonnes d'emballages sont jetées tous les jours à Paris. « La restauration à emporter utilise très majoritairement des emballages à usage unique et non recyclables », pointe l'Ademe. « Notre packaging est aujourd'hui en matière 100 % compostable ou recyclable », assure-t-on chez Frichti qui, à la grande différence d'Uber Eats ou de Deliveroo, prépare les repas qu'elle livre. Mais quand bien même les emballages dits biosourcés (bagasse de canne à sucre, amidon de maïs, kraft) sont-ils privilégiés au plastique, « le tri n'est que trop rarement respecté », relève l'Ademe.   C'est vrai à la maison. Ça l'est aussi au bureau, où une société comme Frichti a construit son succès auprès de salariés ne bénéficiant pas de service de restauration collective ou réfractaires à tout ce qui s'apparente à une cantine. Or, depuis le premier confinement, une grande majoritédes restaurants d'entreprise sont à l'arrêt. Conséquence, dans les quartiers d'affaires, à la pause déjeuner, on compte par centaines ces cadres et employés revenant à leur bureau un sac à la main rempli d'emballages, serviettes et couverts qui seront jetés une fois le repas avalé. « L'explosion des livraisons au bureau a eu un gros impact négatif avec des montagnes de sacs de vente à emporter », témoigne un prestataire de gestion de déchets en entreprise.   Selon le cabinet spécialisé Interface, seul 20 % à 25 % des restaurants d'entreprise (hors administration publique) ont conservé une activité, et souvent au ralenti, dans des secteurs où le télétravail n'est pas possible. Avant la crise, 1,8 million de repas étaient servis tous les jours en France, dont 1,3 million en Ile-de-France, rappelle le cabinet. « Le monde d'après ne ressemblera pas au monde d'avant, prédit Vincent Lazzaroni, un de ses directeurs associés. Avec la croissance du télétravail et la diversification des modes de restauration, le Covidva forcer toute une profession à accélérer sa mue pour séduire des collaborateurs qui recherchent l'hyper-efficacité. Elle passera nécessairement par une montée en gamme sur les produits et le click and collect. » Certains groupes ont déjà mis en place des systèmes de click and collect pour éviter les files d'attente (et les risques de contamination). D'autres sociétés, comme Microsoft, ont fait appel aux services de FoodChéri, le concurrent de Frichti sur le segment de la « cuisine maison » : entre 80 et 100 repas sont livrés chaque midi. De son côté, à la faveur de la crise, Frichti a mis un pied chez EDF et Lagardère.   « Vrai défi » Mais il y a un « bémol », convient Vincent Lazzaroni. Il est d'ordre « écologique » : « Le Covid a démultiplié l'usage des contenants jetables : c'est le vrai défi de la restauration. » La plupart des restaurants d'entreprise ou des cafétérias qui ont rouvert après le premier confinement proposent des plats en formule à emporter pour éviter que les cantines se transforment en potentiel cluster. Et même si certains cherchent à remplacer le plastique à usage unique par des matériaux recyclables ou biodégradables, ils finissent généralement mélangés avec les autres déchets produits par les entreprises, constate le cabinet Interface.   Pour Zero Waste France, l'enjeu n'est pas tant le recyclage (dont le taux plafonne à 5 % pour le plastique alimentaire hors bouteilles) que la production de déchets et le « suremballage . Responsable du programme territoires au sein de l'association, Pauline Debrabandere ne cache pas son inquiétude : « Il y a une vraie crainte que la mode du "click and collect" ne signe le retour en force de la société du jetable. »   « Il va falloir regagner le terrain qu'on avait conquis », commente l'adjointe d'Anne Hidalgo, Colombe Brossel. La Mairie de Paris s'est engagée à sortir du plastique à usage unique d'ici à 2024. Mais pour l'heure, pas question de tomber sur les restaurateurs et les commerces de bouche : « Aujourd'hui, on travaille surtout pour les sauver. »
</article>
<article titre="Interview Transformation Frédéric Moyne : « Albioma a la capacité de renouveler son modèle » Frédéric Moyne" date="2021 01 23" journal="Investir-Le Journal des finances, no. 2455">
Comment analysez-vous l'envolée en Bourse d'Albioma et des autres acteurs des énergies vertes ? Ce parcours résulte de la prise en compte par le marché des performances opérationnelles et financières de notre société, de ses perspectives ainsi que de ses bonnes pratiques en matière environnementale, sociale et de gouvernance (ESG). Les investisseurs reconnaissent le positionnement d'Albioma sur les énergies renouvelables dans un contexte de lutte contre le réchauffement climatique. Il y a, enfin, un effet de rareté des véhicules cotés sur toutes les places financières. Le mouvement est général et ne concerne pas que les capitalisations petites et moyennes, mais aussi les grandes utilities, qui ont pris franchement le virage des énergies renouvelables, comme le portugais EDP. Ces thématiques, Renouvelables et ESG, vont rester prégnantes en 2021.   Que peuvent apporter les plans de relance et le pacte vert européen à l'entreprise et au secteur ? Ces plans sont faits pour canaliser les investissements vers des technologies qui, sans ce soutien, ne se développeraient pas. Le solaire, devenu une énergie compétitive, peut croître sans ces subventions et ne devrait donc pas bénéficier d'une enveloppe spécifique.   Dans les DOM et dans nos métiers, nous n'avons pas attendu le plan de relance pour annoncer notre objectif d'utiliser de la biomasse à 100 % dans nos centrales thermiques. Nous n'allons pas être concernés par le plan, qui vise en France plutôt l'efficacité énergétique ou l'hydrogène, mais il a mis un coup de projecteur sur les valeurs vertes cotées.   Albioma utilise encore du charbon dans trois de ses centrales thermiques françaises. Quel est le calendrier pour les convertir à la seule biomasse ? Notre objectif est l'abandon du charbon dans les DOM à un horizon 2025. La totalité de nos centrales thermiques fonctionnera alors à la biomasse, comme déjà Galion 2, à la Martinique [entrée en service fin 2018], tandis que la turbine à combustion de Saint-Pierre, à La Réunion, utilise du bioéthanol. Les trois centrales à convertir, Le Moule, en Guadeloupe, Bois-Rouge et le Gol, à La Réunion, sont construites grosso modo sur le même modèle. Pour éviter un black-out et assurer la fourniture d'énergie pendant les campagnes sucrières, le programme de travaux va durer jusqu'en 2024. Le premier chantier, pour la conversion de la tranche 3 du Moule, a subi quelques retards à cause du confinement, mais cette unité a été remise en service en novembre 2020 comme prévu, et elle est un succès opérationnel et industriel. Cette conversion représente un investissement de 80 millions d'euros, qui comprend aussi la construction du dôme de stockage de la biomasse. Pour Bois-Rouge, à La Réunion, les travaux vont commencer en 2021 pour le stockage de la biomasse, avec une conversion des trois tranches prévue d'ici à 2023. Pour la centrale du Gol, le chantier doit s'effectuer selon un calendrier décalé de douze à dix-huit mois par rapport à Bois-Rouge, de même que pour les tranches 1 et 2 du Moule. Nos centrales auront ainsi deux vies : la première, en brûlant du charbon et de la bagasse, le résidu fibreux de la canne à sucre; la seconde, en fonctionnant à 100 % à la biomasse, ce qui permet de réduire les émissions de plus de 85 %. Ces investissements témoignent de la capacité du groupe à renouveler son modèle.   Cette décarbonation de la production a-t-elle des incidences financières ? L'objectif 100 % biomasse pour le groupe est un nouvel engagement pris à l'occasion du placement privé de 100 millions d'euros effectué en décembre. Il s'agit d'un emprunt dont la marge est indexée sur notre trajectoire de recours à la biomasse, soit plus de 80 % en 2023, plus de 90 % en 2025 et près de 100 % en 2030. En fonction du respect de ces engagements, elle pourra varier à la hausse ou à la baisse jusqu'à 25 points de base, soit 250.000 € par an, ce qui n'est pas rien.   Quelles ambitions avez-vous dans le domaine du photovoltaïque, une activité encore modeste ? Nous avons accéléré dans le solaire, avec 35 MW de projets gagnés en 2020. Nous capitalisons à la fois sur notre présence dans les DOM et sur l'acquisition d'Eneco en métropole, en 2018. Nos projets privilégient les zones non interconnectées mais aussi la métropole, en restant sur les marchés des centrales de tailles petite et moyenne, sur toiture, avec du stockage et de l'innovation, comme l'amélioration de la qualité de service au réseau.   Au Brésil, où le groupe en est-il ? Le rythme devait être d'un nouveau projet tous les douze à dix-huit mois. Nous avons mis en service en décembre notre quatrième centrale de cogénération, celle de Vale do Parana, dans l'Etat de Sao Paulo. Pour la première fois dans le pays, nous avons conçu et construit cette usine et apporté la preuve que nous pouvions effectuer le revamping intégral d'une centrale. Elle bénéficie d'un contrat de vente d'électricité jusqu'en 2046. Nous avons désormais une légitimité sur ce marché, où nous avons aussi doublé la capacité énergétique des trois centrales acquises précédemment. Notre objectif est de signer de bons projets. Ce n'est pas grave si le délai de douze à dix-huit mois n'est pas respecté.   Quelle est votre politique de retour à l'actionnaire ? Nous versons un dividende en croissance chaque année et majoré de 10 % pour les actionnaires au nominatif depuis plus de deux ans. Il est proposé en actions pour la moitié de son montant et représente 50 % du résultat.   Propos recueillis par Cécile Le Coz
</article>
<article titre="Albioma investit en Guadeloupe pour remplacer le charbon par de la biomasse" date="2021 01 22" journal="L'Usine Nouvelle (site web)">
Dans l'île de la Guadeloupe, qui vise l'autonomie énergétique d'ici à 2030, le producteur d'énergie Albioma va remplacer sa centrale thermique au charbon par une centrale 100 % biomasse d'ici à 2023. L'opération nécessitera un investissement de 80 millions d'euros. Déjà, les granulés bois arrivent sur l'île. D'ici 2023, la Guadeloupe va passer du charbon à la biomasse pour son énergie avec l'implantation par Albioma d'une nouvelle centrale qui nécessitera un investissement de 80 millions d'euros. Depuis quelques mois, un grand dôme blanc surplombe le port de Pointe-à-Pitre, en Guadeloupe. "C'est un silo à granulés bois, une structure gonflable qui a poussé en quelques heures", sourit Nicolas de Fontenay, directeur Antilles-Guyane d'Albioma, (506 millions d'euros de chiffres d'affaires), un producteur d'énergie très implanté dans les Outre-mer. Ce silo doit stocker des pellets de bois, qui viendront se substituer au charbon utilisé dans la centrale thermique que l'entreprise exploite dans la commune du Moule, à l'Est de l'île. %%ARTICLELIE:1042659%% "Notre centrale fonctionne sur deux tranches au charbon et à la bagasse [les résidus de la canne à sucre] depuis 1998, rappelle le dirigeant. L'énergie que nous produisons, pendant la récolte sucrière, fait fonctionner l'usine de sucre du Moule, dernière sucrerie du territoire. Le reste est injecté dans le réseau. En 2011, nous avons installé une troisième tranche 100 % charbon. En 2023, nous atteindrons le 100 % biomasse." Le silo, qui sera bientôt suivi d'un second, abritera 205 000 tonnes de granulés bois par an, importés des Etats-Unis. Et ce, jusqu'à ce qu'une filière locale, encore inexistante, se structure selon le schéma régional adopté par la région en novembre 2020. Le premier silo de pellets bois installé par Albioma devrait abriter 205 000 tonnes de granulés par an. ©Amandine Ascensio 80 millions d'euros d'investissement "Il est important pour EDF [qui exploite une centrale au fioul sur la zone portuaire, ndlr] de maintenir une production d'électricité ailleurs pour sécuriser l'approvisionnement électrique en cas de cyclone ou autre aléa", note Nicolas de Fontenay, rappelant ainsi la vulnérabilité de l'île aux risques naturels. Dès février 2021, la centrale thermique accueillera du granulé bois, déjà livré par bateau, qui, pour l'heure, s'installe à la place des paquebots de croisière, privés de saisons en raison de la pandémie. "Cette nouvelle activité d'Albioma, pourrait accroître un petit peu celle du port", rappelle Jean-Pierre Chalus, le directeur du Grand Port Maritime de Guadeloupe. Elle doit aussi créer 3 ou 4 emplois sur la zone de stockage. Et pour le transport jusqu'à la centrale, "nous réfléchissons à des camions à hydrogène". Ces changements ont nécessité pour Albioma "un investissement de 80 millions d'euros", précise le directeur. "Le silo mesure 35 mètres, contient des capteurs de chaleur et de niveau, des filtres à air et sera surveillé 24h/24,", détaille Florent Charrier, ingénieur projet chez Albioma. "Le granulé bois doit, selon une étude du cabinet Deloitte, réduire nos émissions de CO2 de 87 %", salue Nicolas de Fontenay. Vers l'autonomie énergétique L'électricité dans les îles non interconnectées est un vrai défi et bien que propices à l'utilisation du vent et du soleil, les départements d'Outre-mer restent très dépendants des énergies fossiles. "En 2019, 77,98 % de notre électricité est produite à partir des énergies fossiles", rappelle dans un document, l'Observatoire Régional des énergies et du climat (Orec) de Guadeloupe. Dans le détail, 55 % de l'électricité est produite à partir de fioul, 22 % à partir de charbon, les 23 % restants étant répartis entre différentes énergies renouvelables (géothermie, solaire, éolien, hydraulique, biogaz, biomasse). Dès 2023, l'île, qui veut atteindre son autonomie énergétique en 2030, verra donc 45 % de son électricité produite par des énergies renouvelables.
</article>
<article titre="[Le monde de l'enseignement manifestera mardi...]" date="2021 01 22" journal="Sud Ouest  Toutes éditions ; Saintes ~ Royan ~ Jonzac">
Le monde de l'enseignement manifestera mardi   GRÈVE À L'APPEL DE LA CGT, FO ET LA FSU 17, UNE PARTIE DES PERSONNELS DE L'ÉDUCATION NATIONALE, DE L'ENSEIGNEMENT AGRICOLE ET maritime public sera en grève mardi 26 janvier pour défendre " les conditions de travail ", demander " une réelle revalorisation " et s'opposer à " la casse du service public d'Éducation ! [...] Au lieu de répondre à la nécessité de diminuer les effectifs par classe, par la création de postes, il a décidé d'"ubériser" l'Éducation. Dans le premier degré, le ministre Blanquer accentue encore la contractualisation en réduisant le nombre de postes au concours de professeurs des écoles, pour recruter des contractuels en lieu et place de titulaires. Ce sont ainsi 2 000 postes statutaires, pourtant tellement nécessaires, qui sont supprimés depuis trois ans. Et pour la rentrée 2021, aucun poste n'est créé dans l'académie de Poitiers alors que le confinement a tellement affaibli le niveau des élèves les plus en difficulté. Dans le second degré, alors que les collèges et les lycées vont accueillir 43 518 élèves de plus à la rentrée 2021, le ministre supprime 1 883 postes supplémentaires, soit au total 7 490 depuis 2018. " À Saintes, la CGT Educ'action, la FNEC FP-FO et la FSU de Charente-Maritime invitent à manifester mardi 26 janvier, à 10 h 30 devant le palais de justice de Saintes. Après la bouteille, Lyspackaging se lance dans le pilulier vegan   CHANIERS L'entreprise Lyspackaging, implantée à Chaniers, à l'origine d'une gamme d'emballage 100 % végétal, continue à développer sa gamme Veganbottle en proposant désormais des piluliers compostables. Un nouveau contenant qui vient compléter les bouteilles et les gourdes et autres flacons de cosmétique. Flacon, bouchon et étiquette sont biodégradables, explique l'entreprise qui est dirigée par Nicolas Moufflet. Selon les produits, les emballages sont fabriqués avec du roseau, lin, chanvre ou bien de la bagasse de canne à sucre.
</article>
<article titre="« Albioma a la capacité de renouveler son modèle »" date="2021 01 22" journal="Investir (site web)">
Comment analysez-vous l'envolée en Bourse d'Albioma et des autres acteurs des énergies vertes, dont les cours ont plus ou moins doublé en 2020? Ce parcours résulte de la prise en compte, par le marché, des performances opérationnelle et financière de notre société, ainsi que de ses perspectives et bonnes pratiques en matière environnemental, social et de gouvernance (ESG). Les investisseurs reconnaissent le positionnement d'Albioma sur les énergies renouvelables dans un contexte de lutte contre le réchauffement climatique. Il y a, enfin, un effet de rareté des véhicules cotés sur toutes les places financières. Le mouvement est général et ne concerne pas que les capitalisations petites et moyennes, dont Solaria en Espagne, Scatec Solar en Suède ou Boralex au Canada, mais aussi les grandes utilities,qui ont pris franchement le virage des énergies renouvelables, comme le portugais EDP. Ces thématiques, Renouvelables et ESG, vont rester prégnantes en 2021.   Que peuvent apporter les plans de relance et le pacte vert européen à l'entreprise et au secteur?   Ces plans sont faits pour canaliser les investissements vers des technologies qui, sans ce soutien, ne se développeraient pas, tel l'hydrogène vert. Le solaire, devenu une énergie compétitive, peut croître sans ces subventions et ne devrait donc pas bénéficier d'une enveloppe spécifique. Dans les DOM et dans nos métiers, nous avions déjà nos propres projets de décarbonation et n'avons pas attendu le plan de relance pour annoncer notre objectif d'utiliser de la biomasse à 100?% dans nos centrales thermiques. Nous n'allons pas être concernés par le plan, qui vise en France plutôt l'efficacité énergétique ou l'hydrogène, mais il a mis un coup de projecteur sur les valeurs vertes cotées.   Albioma utilise encore du charbon dans trois de ses centrales thermiques françaises. Quel est le calendrier pour les convertir à la seule biomasse?   Notre objectif est l'abandon total du charbon dans les DOM à un horizon 2025. La totalité de nos centrales thermiques fonctionnera alors à la biomasse, comme déjà Galion 2, en Martinique [entrée en service fin 2018], tandis que la turbine à combustion de Saint-Pierre, à La Réunion [opérationnelle depuis début 2019], utilise du bioéthanol. Les trois centrales à convertir, Le Moule, en Guadeloupe, Bois-Rouge et le Gol, à La Réunion, sont construites grosso modo sur le même modèle. D'une puissance d'environ 100mégawatts (MW), elles sont composées de trois tranches chacune. Nous en avons donc neuf à convertir, sachant que ces usines pèsent un poids significatif sur leur réseau électrique. Pour éviter un black-out et assurer la fourniture d'énergie pendant les campagnes sucrières, le programme de travaux va durer jusqu'en 2024.   Le premier chantier, pour la conversion de la tranche 3 du Moule, a subi quelques retards à cause du confinement, mais cette unité a été remise en service en novembre 2020 comme prévu, et elle est un succès opérationnel et industriel. Cette conversion représente un investissement de 80 millions d'euros, qui comprend aussi la construction du dôme de stockage de la biomasse. Pour Bois-Rouge, à La Réunion, nous avons obtenu en 2020 l'accord de la Commission de régulation de l'énergie (CRE) pour convertir les trois tranches, ainsi que la garantie d'une rémunération de 8,3% des capitaux engagés et la prolongation jusqu'en 2043 du contrat de vente à EDF [qui a le monopole de la distribution d'électricité dans les DOM]. Les travaux vont commencer en 2021 pour le stockage de la biomasse, avec une conversion des trois tranches prévue d'ici à 2023.   Pour la centrale du Gol, et sous réserve d'un accord de la CRE cette année, le chantier doit s'effectuer selon un calendrier décalé de douze à dix-huit mois par rapport à Bois-Rouge, de même que pour les tranches 1 et 2 du Moule. Nos centrales auront ainsi deux vies: la première, en brûlant du charbon et de la bagasse, le résidu fibreux de la canne à sucre; la seconde, en fonctionnant à 100?% à la biomasse, ce qui permet de réduire les émissions de plus de 85%. Ces investissements témoignent de la capacité du groupe à renouveler son modèle.   Cette décarbonation de la production a-t-elle des incidences financières?   L'objectif 100% biomasse pour le groupe est un nouvel engagement pris à l'occasion du placement privé de 100 millions d'euros, effectué en décembre. Il s'agit de notre premier Sustainability-Linked Euro PP, un emprunt dont la marge est indexée sur notre trajectoire de recours à la biomasse, soit plus de 80% en 2023, plus de 90% en 2025 et près de 100% en 2030. En fonction du respect de ces engagements, la marge de l'Euro PP pourra varier à la hausse ou à la baisse jusqu'à 25points de base, soit 250.000  par an, ce qui n'est pas rien.   Quelles ambitions avez-vous dans le domaine du photovoltaïque, une activité encore modeste chez Albioma?   Nous avons accéléré dans le solaire, avec 35 MW de projets gagnés en 2020. Nous capitalisons à la fois sur notre présence dans les DOM et sur l'acquisition d'Eneco en métropole, en 2018. Nos projets privilégient les zones non interconnectées mais aussi la métropole, tout en restant sur les marchés des centrales de taille petite et moyenne, sur toiture, avec du stockage et de l'innovation, comme l'amélioration de la qualité de service au réseau.   Au Brésil, où le groupe en est-il? Le rythme devait être d'un nouveau projet tous les douze à dix-huit mois.   Nous avons mis en service en décembre notre quatrième centrale de cogénération, celle de Vale do Parana, dans l'Etat de Sao Paulo. Pour la première fois dans le pays, nous avons conçu et construit cette usine de 48 MW et apporté la preuve que nous pouvions effectuer le revamping intégral d'une centrale, ce que nous n'aurions pas fait au début de notre implantation au Brésil, en 2013. Elle bénéficie d'un contrat de vente d'électricité jusqu'en 2046. Nous avons désormais une légitimité sur ce marché, où nous avons aussi doublé la capacité énergétique des trois centrales acquises précédemment. Notre objectif est de signer de bons projets. Ce n'est pas grave si le délai de douze à dix-huit mois n'est pas respecté.   Quelle est votre politique de retour à l'actionnaire?   Nous versons un dividende en croissance chaque année et majoré de 10% pour les actionnaires au nominatif depuis plus de deux ans. Il est proposé en actions pour la moitié de son montant et représente 50% du résultat.   Propos recueillis par Cécile Le Coz
</article>
<article titre="Good Move séduit les investisseurs avec ses micropousses" date="2021 01 20" journal="La Meuse  La Meuse - Namur">
Good Move, c'est au départ une toute jeune société lancée en 2016 par deux étudiants de l'Université de Namur, qui se spécialise dans les jus de fruits bio pressés à froid.   Mais avec la pandémie de Covid-19, Good Move a perdu 70 % de son chiffre d'affaires, composé principalement de restaurants et de cantines d'affaires. En danger, l'entreprise s'est réinventée et commercialise aujourd'hui un tout autre produit : les micropousses. « Ces très jeunes pousses de légumes, récoltées après une dizaine de jours seulement, présentent une densité en nutriments jusqu'à 40 fois supérieure aux légumes adultes », expliquent Brieuc Debois et Lucas Vandierendonck, les deux créateurs de Good Move.   « On les fait pousser en agriculture verticale », précise Brieuc Debois. « C'est un principe qui nous intéresse depuis longtemps. Ça permet de supprimer l'utilisation des pesticides, et si on le fait correctement, ça réduit de 90 % l'utilisation d'eau. »   Tout cela sans perdre en qualité nutritionnelle, contrairement à ce qu'on pourrait penser. « C'est un amalgame qu'on fait avec la culture très intensive des tomates. Ici, ce n'est pas du tout ce qu'on recherche. Nos micropousses sont au contraire très denses en nutriments. »   Présents dans une vingtaine de magasins   Selon Brieuc Debois, les micropousses sont principalement utilisées dans les restaurants gastronomiques, aujourd'hui. Mais l'objectif de Good Move est de les proposer au public, dans les rayons frais des magasins. « Ce qu'on préconise, c'est de sublimer ses plats avec nos produits. C'est joli, c'est bon et c'est riche en nutriments. Les gens peuvent s'amuser à faire de la cuisine gastronomique chez eux ! Mais ceux qui aiment ça peuvent bien sûr l'utiliser en salade. »   Pour l'instant, les micropousses « Good Move » sont présentes dans une vingtaine de magasins spécialisés, comme « D'Ici », à Naninne, où elles ont été mises en vente en premier. Mais on en retrouve dans de plus en plus d'endroits, et ils sont recensés sur cette carte. « On cultive des micropousses de radis, de radis pourpres, de moutarde et de choux frisés, mais on compte élargir la gamme prochainement. »   Pour coller à leur philosophie, les raviers sont également biodégradables. « Ils sont en fait en bagasse, c'est le déchet issu de la production de canne à sucre. Ça ne libère aucune particule toxique. »   Seul inconvénient : « C'est plus cher que du plastique habituel », assure Brieuc Debois. « Mais c'est notre choix. On veut se donner les moyens de faire de notre mieux pour la santé des gens et pour l'environnement, même si ça réduit notre rentabilité. »   Leurs produits et leur philosophie attirent en tout cas les investisseurs. Les deux entrepreneurs de 26 ans viennent ainsi de lever 232.000 €. « Ça va nous permettre de développer l'entreprise, d'agrandir la production. On va peut-être pouvoir passer dans un endroit plus grand : pour l'instant, on produit dans la cave de chez Lucas, à Gerpinnes ! Il faudra qu'on loue un entrepôt temporairement. On va pouvoir augmenter notre budget marketing aussi, et on espère engager quelqu'un. Puis, on va pouvoir nous verser un salaire, le premier en 4 ans ! »   Pour ceux qui sont intéressés, un ravier coûte autour de 3,35 €. Et il contient de quoi assaisonner deux repas de quatre personnes.   Christophe Halbardier
</article>
<article titre="Amérique Latine : Albioma, CNP Assurances, Omexom, Staphyt, Peugeot Design Lab..." date="2021 01 8" journal="CFNEWS (site web)">
Centrale de Vale do Paraná © Albioma    Energie &amp; Utilities : Albioma (Brésil)Albioma (voir sa fiche et deal-list dans notre référentiel) a mis en service industriel sa centrale 100 % bagasse Vale do Paraná Albioma au Brésil. Située dans la ville de Suzanápolis dans l'État de São Paulo, l'unité de cogénération biomasse est la quatrième centrale du groupe implantée au Brésil et bénéficie d'un contrat de vente d'électricité à long terme de 120 GWh sécurisé jusqu'en 2046 et indexé sur l'inflation. L'investissement de l'opération s'élève à 121 millions de reals (18 M€). Depuis 2014 le producteur d'énergie renouvelable coté a su déployer le modèle original du groupe au Brésil, pays leader mondial de la production de sucre et d'éthanol à partir de la canne à sucre. Albioma a ainsi acquis et développé trois unités de cogénération existantes, Rio Pardo, Codora et Esplanada, qui affichent aujourd'hui de très bonnes performances opérationnelles grâce aux investissements réalisés pour l'amélioration des rendements de la cogénération et la réduction de la consommation de vapeur de la sucrerie. Services Financiers : CNP Assurances (Brésil)CNP Assurances (voir sa fiche et deal-list) a finalisé son nouvel accord de distribution exclusif, jusqu'au 13 février 2046, avec la banque publique brésilienne Caixa Economica Federal (CEF). Dans le cadre de cet accord, signé en août 2018 avec CEF et sa filiale d'assurance Caixa Seguridade qui avait été amendé en septembre 2019, l'assureur coté a versé un montant initial de 7 milliards de réals brésiliens (environ 1,1 Md€). Une nouvelle société d'assurance commune, qui sera intégrée dans le périmètre de CNP Assurances, a été créée. Les droits de vote de cette société seront répartis à hauteur de 51 % pour CNP Assurances et 49 % pour Caixa Seguridade et les droits économiques seront détenus à hauteur de 40 % par CNP Assurances et 60 % par Caixa Seguridade. Par ailleurs, CNP Assurances et Caixa Seguridade ont décidé de reporter la date limite de réalisation définitive de leur partenariat concernant la distribution du produit « consórcio » (produit d'épargne commune donnant droit à l'obtention d'un crédit) du 4 janvier 2021 au 30 mars 2021. Le protocole d'accord signé par les deux parties en août dernier porte sur le réseau de CEF au Brésil et doit avoir une durée de 20 ans.   Energie &amp; Utilities : Omexom (Brésil)    Pantanal Mato-Grossense © Vinci    Omexom G&amp;D, filiale de Vinci Energies International &amp; Systems, a signé un contrat avec Energisa pour livrer des systèmes de production d'énergie à plus de 2 000 familles isolées vivant dans le Pantanal Mato-Grossense, une région de l'État du Mato Grosso do Sul au Brésil. Le projet fournira 2 090 systèmes de production d'énergie isolée et renouvelable, soit un par famille, ainsi que les services requis pour l'installation (conception, ingénierie, fourniture, logistique, mise en service et formation). Les travaux, commencés à l'automne, se termineront au troisième trimestre 2022.   Automobile, aérospatial, construction navale : Michelin (Brésil)Le groupe de pneumatiques français Michelin (voir sa fiche et deal-list) a annoncé en décembre qu'il allait investir 100 millions de reals (15 M€) entre 2021 et 2024 pour l'expansion de son usine de Manaus au Brésil. Le projet vise la production de pneus pour motos et vélos et devrait générer 250 emplois, ce qui augmentera de 26 % le nombre total d'employés de l'entreprise dans la région. Le nouvel investissement est complémentaire de la première phase d'expansion de Michelin à Manaus, qui a bénéficié d'un financement similaire entre 2017 et 2020 avec la création de 367 nouveaux postes de travail. Actuellement, l'industriel tricolore se concentre sur la recherche de fournisseurs pour augmenter la production ainsi que sur l'achat de latex produit dans la région amazonienne. « Avec cet investissement, l'entreprise renforce ses performances sur le marché des deux roues, répondant aux attentes de ses clients et prouvant, une fois de plus, qu'elle croit au potentiel du marché brésilien », a déclaré Feliciano Almeida, président &amp; CEO Michelin South America.   Services &amp; Conseil aux entreprises : Staphyt (Brésil)Staphyt (voir sa fiche et deal-list), PME familiale experte en agrosciences et conseil réglementaire basée à côté d'Arras, dans le Nord, et générant 43 M€ de chiffre d'affaires en 2020, a démarré ses propres opérations au Brésil en novembre dernier. Baptisée Staphyt Brasil Agro Consultoria et implantée à Campinas dans l'État de São Paulo, cette antenne est dirigée par le Dr Luiz Antônio Alves José, lequel possède une vaste expérience de l'agro-industrie dans le monde entier et a occupé divers postes de direction en recherche et développement pendant plus de 25 ans dans des entreprises multinationales au Brésil et à l'étranger. Ce choix stratégique est conduit par la volonté de Staphyt de poursuivre son développement et de proposer ses services et ses compétences auprès des industriels de l'agriculture dans une des régions agricoles les plus importantes du monde, aujourd'hui le quatrième agro-exportateur mondial. Créé en 1989 Staphyt, qui emploie au total 440 collaborateurs dans le monde, dont 198 en France, est dirigé par Arthur et Catherine Boisleux, actionnaires à hauteur de 92 %, les 8 % restants sont détenus par le management.   Produits &amp; Services industriels : Peugeot Design Lab (Brésil)    Foodtruck © Peugeot Design Lab    Le constructeur français Peugeot ouvre un troisième studio satellite de son Peugeot Design Lab à São Paulo, au Brésil. L'antenne brésilienne du studio de style dédié à la conception de projets en dehors du secteur automobile, sera dirigé par Fabien Darche, directeur du design du groupe PSA en Amérique Latine. Depuis son lancement en 2012 Peugeot Design Lab, implanté aussi à Shanghai, en Chine, et à San Francisco, aux États-Unis, a travaillé sur plus de 60 projets, parmi lesquels l'hélicoptère H160 dessiné pour Airbus, le concept Foodtruck Peugeot et le dernier tramway d'Alstom pour la ville de Strasbourg.   Une information à nous soumettre pour ce Bulletin Amérique Latine ? Écrivez-nous à : dalila.weygang@cfnews.net   Retrouver les autres Chroniques de CFNEWS :   Tous les précédents bulletins Amérique LatineTous les bulletins AsieAussi sur Facebook , Twitter , Google+ et LinkedIn  Cet article est paru dans CFNEWS (site web)
</article>
<article titre="Entretien avec Vincent Guenzi, Directeur de la Stratégie d'Investissement de Cholet Dupont" date="2021 01 6" journal="Boursier (site web)">
Boursier.com : Le début d'année boursière est accaparé par la question des vaccins... V.G. : Les vaccins sont primordiaux pour maitriser l'épidémie et réduire le risque d'engorgement des systèmes de santé. Il faut que les campagnes de vaccination des populations à risque commencent rapidement dans toute l'Europe pour Après la découverte et l'approbation des vaccins, reste à prouver leur efficacité dans la prévention de la contagion. C'est pourquoi les investisseurs resteront attentifs aux nouvelles qui seront publiées dans les prochains mois. Comme les marchés ont déjà anticipé en partie le succès des campagnes de vaccination, ils percevraient de manière négative toute mauvaise nouvelle à ce sujet. Des déceptions passagères ne sont pas à exclure. Cependant, une remise en cause du scénario économique et financier favorable n'est pas notre hypothèse principale. Nous pensons que le contrôle progressif de l'épidémie sera suivi d'une reprise économique moins fragile dans le courant du 1er semestre et que cela permettra aux indices boursiers d'enregistrer de bonnes performances en 2021. Boursier.com : Les valeurs cycliques feront-elle enfin mieux que les valeurs de croissance ? V.G. : On peut miser sur elles, mais de manière sélective. Certaines valeurs cycliques ou de type " value " souffrent d'une remise en cause de leur " business model ". Elles ont pourtant bien rebondi car leur valorisation était très basse mais la poursuite de leur surperformance récente sera plus difficile. Par ailleurs, les valeurs cycliques affectées par les perturbations dues à la pandémie comme l'aérien ou l'hôtellerie par exemple, recèlent un grand potentiel d'appréciation que les investisseurs n'ont pas encore exploité totalement pour des raisons évidentes. Quant aux valeurs de croissance, leur désaffection dépendra de la durée de leur surévaluation. Nous sommes persuadés que les investisseurs seront prêts à les racheter quand leur valorisation deviendra un peu plus attractive.   Boursier.com : Doit-on miser sur les petites capitalisations cette année ? V.G. : La règle veut que le segment des petites et moyennes capitalisations surperforme structurellement les grandes puisque ce segment offre une croissance supérieure. Depuis fin 2009, l'eurostoxx small dividendes réinvestis progresse de 116.34% (au 30/11/2020) alors que l'Eurostoxx 50 dividendes réinvestis est en hausse de 60.95% sur la même période. Comme toutes les règles, il existe des exceptions mais il nous semble que les conditions soient une nouvelle fois réunis pour que la règle s'applique. Un des éléments qui pourrait changer la donne serait une surpondération des actions européennes chez les grands allocataires mondiaux qui serait favorable aux grandes capitalisations, mieux représentées dans les ETF notamment.    Boursier.com : 3 valeurs à mettre en portefeuille dès le début d'année ? V.G. : Teleperformance : Compte tenu de l'excellente performance enregistrée au T3, le groupe a relevé ses objectifs de croissance organique sur l'ensemble de l'année 2020 en visant désormais environ + 8% contre + 6% annoncé précédemment. Par différence, la croissance du T4 est donc attendue à environ 9/10% ce qui témoigne de la poursuite d'une forte dynamique dans un contexte pourtant difficile. La marge opérationnelle est toujours attendue à au moins 12.5%. Les objectifs 2022 sont réitérés (CA environ 7 MdE ; MOP environ 14.5%). Au final, Téléperformance démontre une excellente résistance de sa profitabilité dans un contexte difficile mais plus encore sa capacité à extraire une forte croissance de son CA. Au-delà de la croissance organique, la forte génération de FCF et la situation bilancielle permettent d'envisager de nouvelles acquisitions d'ici 2022. Edenred : Le groupe a enregistré un retour en territoire positif au T3 avec une croissance organique du CA opérationnel de 0.9%, qui est une très bonne surprise alors que le consensus anticipait -6.3% et un redémarrage plus tardif (T4). Cette performance confirme le positionnement de société de croissance malgré un environnement très difficile. Nous continuons d'apprécier à horizon long terme le positionnement du groupe qui évolué avec des propositions innovantes sur des marchés encore sous-pénétrés et qui devrait bénéficier de l'exacerbation de la digitalisation. Le groupe est progressivement en passe de devenir un groupe de paiement spécialisé. Albioma est un producteur d'énergie indépendant spécialisé dans les activités de biomasse thermique, bio-méthanisation et solaire. Elle réalise la totalité de son chiffre d'affaire dans les DOM TOM (Guadeloupe, Martinique, Réunion,) et dans l'Ile Maurice.   La stratégie du groupe vise l'accompagnement de la transition énergétique (la loi vise 50% d'énergies renouvelables dans les DOM d'ici à 2023), la production d'énergie renouvelable à hauteur de 80% d'ici à 2023. Albioma poursuit le développement de ses unités de production bagasse/biomasse, la duplication de son modèle bagasse au Brésil ainsi que le développement de projets photovoltaïques. Le 1er semestre a clairement traduit la résilience du modèle (EBITDA +19%) avec une bonne visibilité pour les prochains trimestres. Le business model du groupe repose sur des contrats de longue durée (20/25 ans pour le solaire, 30/35 ans pour le thermique, 20 ans au Brésil), mais aussi sur une rémunération sous forme de primes fixes fonction de la disponibilité de l'outil soit une bonne résilience face à la baisse constatée de la consommation d'électricité, et enfin sur un important programme de Capex avec des TRI garantis (ceux-ci viennent d'être ajusté à 7/10% vs 11% sans effet rétroactif).   Boursier.com : Donnez-nous 3 paris plus risqués... V.G. : Faurecia, au coeur de la transition hydrogène. il est désormais clair que l'hydrogène constitue une opportunité de croissance significative pour l'équipementier que ce soit seul ou via la JV Symbio constituée avec Michelin (3 MdE de CA visés en 2030 au cumul, soit plus de 15% du CA 2019), d'autant plus que l'équipementier n'exclut pas l'ajout d'autres segments/produits à son offre (électrolyseurs par exemple), voire de réfléchir à de nouveaux business models (offre de service, aspects recyclage ...). Si l'horizon encore lointain (2025- 2030) empêche de valoriser pleinement dès maintenant cette opportunité pour l'équipementier, elle n'en reste pas moins un axe intéressant amené à gagner en importance au cours des prochaines années (sur le plan de l'ESG notamment). SPIE fait partie des leaders européens des services multi-techniques. Le groupe offre des solutions qui viennent améliorer, voire optimiser la productivité et la longévité des infrastructures dans les domaines du génie électrique, climatique et mécanique. Cette maitrise des process en fait un acteur incontournable à l'heure où la transition énergétique et la digitalisation font partie intégrante des politiques de relance et des plans stratégiques des entreprises. Le groupe devrait être capable de s'inscrire de nouveau dans un cercle de création de valeur en générant assez de cash pour d'une part poursuivre son désendettement et d'autre part financer des acquisitions ciblées en France, en Allemagne, en Belgique et aux Pays-Bas. Accor est un des leaders mondiaux et le leader européen de l'hôtellerie, présent sur l'ensemble des segments avec un portefeuille d'une trentaine de marques et une exposition géographique diversifiée. Outre une reprise cyclique, Accor devrait bénéficier en 2021 de la transformation de son business model en hôtelier asset light, offrant une meilleure résilience, une solide génération de FCF permettant du retour exceptionnel aux actionnaires et de saisir d'éventuelles opportunités de M&amp;A. Même si la visibilité reste encore limitée à ce stade, les RevPAR du groupe devraient progresser en 2021. La structure de bilan est solide. Enfin, la valorisation est raisonnable et présente une décote toujours significative par rapport aux comparables asset light US.   Arnaud Bivèshttps://cdn-static.boursier.com/illustrations/interviews/200/guenzi-cholet.jpg
</article>
<article titre="Emballage, une problématique, 4 voies complémentaires" date="2021 01 1" journal="Néorestauration, no. 0592">
E mploi de nouvelles matières  Les restrictions sur le plastique nécessitent  « le sourcing d'alternatives plus durables »,  explique Carole March. Ce sujet, auquel Épisaveurs/Pomona s'est attaqué dès 2018, est très complexe. Question de sources d'approvisionnement - l'Asie pour la bagasse, l'Asie et l'Europe pour le carton -, alors que  « nous nous attelons à avoir un rayon le plus proche possible, donc en Europe » ;  d'étanchéité de ces matières..., garantie par un pelliculage plastique.  « C'est pourquoi nous travaillons avec un de nos fournisseurs européens sur du carton protégé par une laque alimentaire » ;  de prix souvent supérieur à celui du plastique.  Le développement du recyclable/recyclé,  c'est un des axes suivis par CCEP, souligne Arnaud Rolland, avec ses bouteilles PET.  « Nous avançons très vite avec les autres fabricants vers 100 % de PET recyclé ; la problématique n'est pas d'intégrer des matériaux recyclés, mais de réussir à collecter les contenants. »  Pour ce faire, il faudrait mettre en place la consigne, mise en oeuvre depuis des années pour les bouteilles en verre de Coca en CHR.   Pleins phares sur le réemploi,  pour lequel Uzaje développe des solutions industrielles et sert de conseiller : quel contenant choisir, comment engager le consommateur, quelle logistique mettre en place... ? C'est dans cet esprit qu'avec Citeo et l'Ademe, l'entreprise mène une étude sur l'impact économique, environnemental et social d'une bouteille de 1 l de jus d'orange en PET 50 % recyclé versus une autre en verre réemployable.  « Le point faible du réemploi, c'est l'utilisation d'eau »,  analyse Arnaud Rolland.  « Avec des machines très modernes, nos centres de lavage fonctionnent quasi en circuit fermé ; le principal impact, c'est le CO2 lié au transport - pour une boucle logistique qui ne doit pas excéder 250 km »,  répond Emmanuel Auberger. En restauration collective, outre sa participation au programme Recolim, qui vise à déterminer les conséquences du remplacement des bacs gastro plastique par de l'inox ou du verre (lire en page 40), Uzaje a organisé pour Elior un système de réemploi dans ses restaurants d'entreprise. En restauration commerciale, elle a démarré des projets en fast casual, freinés pour cause de Covid...   Le zéro-déchet  Soit la vente en vrac de boissons pour CCEP, qui a sorti la première fontaine à soda en 1933, et qui développe depuis une dizaine d'années Coca-Cola Freestyle, une fontaine digitalisée avec 250 choix de boissons en restauration rapide et dans les parcs de loisirs.    En conclusion ?  Attendue par les consommateurs, cette revisite de l'emballage, addition de solutions complémentaires, peut être vécue comme une contrainte alors qu'elle peut apporter une valeur supplémentaire aux convives - on peut proposer 250 parfums, un modèle plus vertueux, du circuit court... - et même devenir un outil de fidélisation, notamment via la consigne.
</article>
<article titre="La Réunion : les centrales électriques passent à la biomasse" date="2020 12 31" journal="Les Echos, no. 23359">
En 1992, Séchilienne-Sidec - aujourd'hui Albioma - inaugurait à La Réunion un procédé de cogénération inédit à cette échelle. La centrale de Bois-Rouge, dans l'est de l'île, était équipée pour produire de l'électricité en brûlant du charbon mais aussi de la bagasse, résidu fibreux de la canne, venu de la sucrerie voisine. En retour, depuis cette date, la centrale fournit à l'usine sucrière la vapeur dont elle a besoin en grande quantité.   Fin 2023, Bois-Rouge sera la première centrale réunionnaise à fonctionner uniquement à la biomasse, à l'image de celle du Galion, en service en Martinique depuis deux ans, et d'une première tranche de la centrale guadeloupéenne du Moule depuis le mois dernier. Début décembre, la Commission de régulation de l'énergie a donné son feu vert à Albioma pour modifier et prolonger jusqu'en 2043 son contrat de revente de l'électricité de Bois-Rouge à EDF.   Ressource locale insuffisante Les investissements, dont le montant n'est pas communiqué par le groupe, débuteront en 2021 par la construction d'installations de stockage sur le port de commerce réunionnais. La ressource de biomasse locale est en effet insuffisante pour répondre aux besoins des trois tranches de la centrale, d'une puissance installée totale de 108 mégawatts. Le combustible végétal sera dans un premier temps importé des Etats-Unis, sous forme de granulés de bois, les « pellets »   « Au démarrage, les pellets constitueront 70 % du combustible utilisé,  précise Frédéric Moyne, président d'Albioma.  Mais nous avons la volonté de régionaliser nos approvisionnements en étudiant des solutions en Afrique du Sud et nous souhaitons donner la priorité à la biomasse locale : la bagasse mais aussi, potentiellement, les pailles de canne, les broyats de palettes, la fraction ligneuse des déchets verts, la biomasse forestière. Nous travaillons sur tous ces sujets en même temps. » Le groupe prépare en parallèle la conversion de sa deuxième centrale thermique réunionnaise, d'une puissance de 122 MW et qui fonctionne elle aussi en cogénération avec l'autre sucrerie de l'île, au Gol. Ensemble, les deux unités produisent près de la moitié de l'électricité consommée localement.Un autre acteur joue un rôle important dans le mix électrique de La Réunion : Production Electrique Insulaire, filiale d'EDF pour les départements d'outre-mer et la Corse. Sa centrale au fioul lourd (212 MW), inaugurée en 2012 sur les quais du port de commerce, sera pour sa part convertie à la biomasse liquide d'ici à 2023. Fabriqué en France à partir de colza produit en Europe, le combustible sera conforme à la directive communautaire sur les énergies renouvelables, RED II, qui garantit notamment la traçabilité des végétaux utilisés.   Essais « concluants » « Des essais effectués récemment sur un des moteurs d'une centrale équivalente en Guadeloupe ont étéconcluants,  annonce Alexandre Sengelin, le directeur de l'unité réunionnaise . La conversion sera assez simple. La biomasse liquide pourra être stockée dans les cuves actuellement utilisées pour le fioul et nous aurons très peu de modifications à apporter sur nos 12 moteurs. » Fin novembre, le Conseil régional de La Réunion a voté la révision de la Programmation pluriannuelle de l'énergie, feuille de route du territoire jusqu'en 2028, qui intègre la conversion à la biomasse des trois principales centrales de l'île. En y ajoutant les centrales hydrauliques, le parc photovoltaïque en constante progression, quelques éoliennes et un futur incinérateur à déchets, le mix électrique sera, dans trois ans, à près de 100 % d'origine renouvelable... mais de provenance lointaine pour ce qui concerne la biomasse.   Bernard Grollier
</article>
<article titre="Albioma met en service sa quatrième centrale 100%  bagasse au Brésil, Vale do Paraná Albioma" date="2020 12 29" journal="Boursier (site web)">
Albioma annonce la mise en service industriel, le 25 décembre 2020, de sa centrale 100 % bagasse Vale do Paraná Albioma au Brésil. Située dans la ville de Suzanápolis dans l'état de São Paulo, l'unité de cogénération biomasse Vale do Paraná Albioma est la quatrième centrale du Groupe implantée au Brésil et bénéficie d'un contrat de vente d'électricité à long terme de 120 GWh sécurisé jusqu'en 2046 et indexé sur l'inflation.   L'investissement de l'opération s'élève à 121 millions de réais brésiliens.   Depuis 2014, Albioma a su déployer le modèle original du Groupe au Brésil, pays leader mondial de la production de sucre et d'éthanol à partir de la canne à sucre. Albioma a ainsi acquis et développé trois unités de cogénération existantes (Rio Pardo, Codora et Esplanada), qui affichent aujourd'hui de très bonnes performances opérationnelles, grâce aux investissements réalisés pour l'amélioration des rendements de la cogénération et la réduction de la consommation de vapeur de la sucrerie.   Premier projet brésilien pour lequel le Groupe intervient à la fois sur la construction et le raccordement au réseau, en plus de l'exploitation, signé en 2016, la centrale Vale do Paraná Albioma, est détenue conjointement par Albioma et l'usine Vale do Paraná. Elle est adossée à la sucrerie-distillerie du même nom qui dispose d'une capacité de broyage annuelle de 2 millions de tonnes de canne à sucre.   D'une puissance installée de 48 MW, la centrale sera en mesure d'exporter jusqu'à 30 MW d'électricité renouvelable sur le réseau.   Frédéric Moyne, Président-Directeur-Général d'Albioma a déclaré : "Nous sommes très heureux d'annoncer la mise en service de la centrale Vale do Paraná Albioma au Brésil, dans les délais et conformément à notre budget, qui témoigne de notre savoir-faire à l'étranger. Le gisement très important de productivité et d'efficacité énergétique dans ce pays au fort potentiel a conduit Albioma à s'intéresser à ce marché, qui reste naturellement une des priorités à l'international du Groupe pour ces prochaines années. Félicitations aux équipes du Brésil."
</article>
<article titre="ALBIOMA : mise en service de sa quatrième centrale 100 %  bagasse au Brésil" date="2020 12 29" journal="Option Finance (site web)">
AOF - 29 décembre 2020    (AOF) - Albioma a annoncé la mise en service industriel, le 25 décembre 2020, de sa centrale 100 % bagasse Vale do Paraná Albioma au Brésil. Située dans la ville de Suzanápolis dans l'état de São Paulo, cette unité de cogénération biomasse est la quatrième centrale du groupe implantée au Brésil et bénéficie d'un contrat de vente d'électricité à long terme de 120 GWh sécurisé jusqu'en 2046 et indexé sur l'inflation. L'investissement de l'opération s'élève à 121 millions de réais brésiliens.   Premier projet brésilien pour lequel le groupe intervient à la fois sur la construction et le raccordement au réseau, en plus de l'exploitation, signé en 2016, la centrale Vale do Paraná Albioma, est détenue conjointement par Albioma et l'usine Vale do Paraná. Elle est adossée à la sucrerie-distillerie du même nom qui dispose d'une capacité de broyage annuelle de 2 millions de tonnes de canne à sucre. D'une puissance installée de 48 MW, la centrale sera en mesure d'exporter jusqu'à 30 MW d'électricité renouvelable sur le réseau.   Frédéric Moyne, Président-Directeur-Général du producteur d'énergie renouvelable a déclaré : " Le gisement très important de productivité et d'efficacité énergétique dans ce pays au fort potentiel a conduit Albioma à s'intéresser à ce marché, qui reste naturellement une des priorités à l'international du Groupe pour ces prochaines années ".  Cet article est paru dans Option Finance (site web)
</article>
<article titre="Analyse mi-séance AOF France/Europe - Le CAC 40 repasse au-dessus des 5 600 points en attendant Wall Street" date="2020 12 29" journal="Option Finance (site web)">
AOF - 29 décembre 2020    (AOF) - Les marchés européens progressent modestement à la mi-séance. Les raisons de la hausse restent les mêmes : le début des campagnes de vaccination en Europe depuis ce week-end et la ratification par le président américain du plan de soutien à l'économie. A ce sujet, la Chambre des représentants, dominée par les démocrates, a voté en faveur de l'envoi d'un chèque de 2 000 dollars aux ménages américains contre 600 dollars auparavant. Le Sénat à majorité républicaine pourrait cependant s'y opposer. Wall Street est attendu en légère hausse. Vers 12h15, le CAC 40 gagne 0,37% à 5 609 points.   Enfin une bonne nouvelle pour les actionnaires de SAP, dont l'action avait dévissé fin octobre après un profit warning massif. L'action de l'éditeur de logiciels professionnels prend la tête de l'indice Dax 30 à la faveur d'un gain de 2,11% à 107,48 euros car l'heure approche pour l'entrée en Bourse de Qualtrics, le spécialiste de la gestion de l'expérience. Racheté pour 8 milliards de dollars en 2018, à quelques jours de son IPO, la transaction avait alors été jugée trop onéreuse et entraîné un net repli de l'action SAP. Cette analyse s'est avérée infondée. A Paris, Schneider Electric pourrait terminer 2020 en tant que plus forte hausse de l'indice CAC 40, s'il conserve son maigre avantage par rapport à Hermès. Le groupe industriel français spécialisé dans les produits de gestion d'électricité et des automatismes a apporté la preuve de la résistance de son business model. Après avoir atteint un plus bas à 61,72 euros le 19 mars, l'action Schneider Electric s'est reprise et gagne près de 32% depuis le 1er janvier. Elle a même enregistré un plus haut historique début novembre à près de 122 euros.   Albioma (0,96% à 47,40 euros) a annoncé la mise en service industriel, le 25 décembre 2020, de sa centrale 100 % bagasse Vale do Paraná Albioma au Brésil. Située dans la ville de Suzanápolis dans l'état de São Paulo, cette unité de cogénération biomasse est la quatrième centrale du groupe implantée au Brésil et bénéficie d'un contrat de vente d'électricité à long terme de 120 GWh sécurisé jusqu'en 2046 et indexé sur l'inflation. L'investissement de l'opération s'élève à 121 millions de réais brésiliens.    Les chiffres macroéconomiques du jour   Les investisseurs attendent l'indice S&amp;P Case Shiller des prix immobiliers en octobre aux Etats-Unis à 15 heures et l'indice de confiance des consommateurs du Conference Board en décembre à 16 heures.   Ce matin, l'euro gagne 0,33% à 1,2248 dollar.  Cet article est paru dans Option Finance (site web)
</article>
<article titre="Morning meeting AOF France / Europe - Vers une prolongation de la hausse, Tokyo au plus haut depuis 30 ans" date="2020 12 29" journal="Option Finance (site web)">
AOF - 29 décembre 2020    (AOF) - Les marchés européens devraient poursuivre leur mouvement de hausse. Ce matin, la Bourse de Tokyo a progressé de 2,66% pour finir à un plus haut de 30 ans. Les raisons de la hausse restent les mêmes : le début des campagnes de vaccination en Europe depuis ce week-end et la ratification par le président américain du plan de soutien à l'économie. A ce sujet, la Chambre des représentants, dominée par les démocrates, a voté en faveur de l'envoi d'un chèque de 2 000 dollars aux ménages américains contre 600 dollars auparavant. Le Sénat à majorité républicaine pourrait cependant s'y opposer. Les valeurs à suivre aujourd'hui   ALBIOMA    Albioma a annoncé la mise en service industriel, le 25 décembre 2020, de sa centrale 100 % bagasse Vale do Paraná Albioma au Brésil. Située dans la ville de Suzanápolis dans l'état de São Paulo, cette unité de cogénération biomasse est la quatrième centrale du groupe implantée au Brésil et bénéficie d'un contrat de vente d'électricité à long terme de 120 GWh sécurisé jusqu'en 2046 et indexé sur l'inflation. L'investissement de l'opération s'élève à 121 millions de réais brésiliens.   EUROPLASMA   Le Conseil d'administration d'Europlasma, faisant usage de la délégation de compétence conférée par l'Assemblée générale extraordinaire qui s'est tenue ce jour, a décidé d'émettre 650 obligations convertibles en actions (OCA) au profit du fonds Global Tech Opportunities 1. Cette opération s'inscrit dans le cadre de l'accord du 12 novembre 2020 permettant un désendettement massif du Groupe à hauteur de 21 millions d'euros. Le prix de souscription d'un montant nominal total de 6,5 millions d'euros est intégralement réglé par Global Tech par compensation avec la créance liquide.   AKKA TECHNOLOGIES   Akka a annoncé avoir lancé une procédure de consultation des instances représentatives du personnel en France qui devrait déboucher sur la mise en oeuvre d'un plan de restructuration, la reprise dans le secteur Aéronautique ne se matérialisant pas. A ce stade, les discussions avec les partenaires sociaux portent sur la suppression d'environ 900 postes, principalement dans la région Occitanie. Le groupe affirme tout mettre en oeuvre en parallèle pour trouver des solutions à la sévérité de la crise actuelle et sauvegarder un maximum d'emplois.   EUROFINS   Eurofins Génomique a annoncé le lancement d'un nouveau service de séquençage du SARS-CoV-2, optimisé et rentable, qui fournit des séquences complètes du génome viral. L'ARTIC NGS Oligo, synthétisé par la branche Synthèse d'Eurofins Genomics, sera également commercialisé sous peu. Le nouveau service NGS et le kit oligo ARTIC NGS sont les derniers ajouts d'Eurofins à un portefeuille de produits complet destiné à soutenir les chercheurs du monde entier dans le développement de vaccins et dans leur lutte contre la pandémie de Covid-19.   Les chiffres macroéconomiques   Les investisseurs attendent l'indice S&amp;P Case Shiller des prix immobiliers en octobre aux Etats-Unis à 15 heures et l'indice de confiance des consommateurs du Conference Board en décembre à 16 heures.   Ce matin, l'euro gagne 0,30% à 1,2245 dollar.   Hier à Paris   Les marchés européens ont accéléré leur tendance haussière cet après-midi pour terminer sur de nets gains. Malgré de faibles volumes, le très attendu plan de relance américain, enfin ratifié par Donald Trump, a réjoui les investisseurs. Bien qu'il l'avait vivement critiqué, il a finalement cédé à la pression du Congrès. L'accord de Brexit signé jeudi dernier ainsi que les campagnes de vaccination qui ont débuté ce dimanche en Europe ont également porté les marchés. A la clôture, le CAC 40 a ainsi gagné 1,2% à 5 588,38 points, et l'EuroStoxx50, 0,94% à 3 576,52 points.   Hier à Wall Street   Les marchés américains ont confirmé leur orientation haussière en seconde partie de séance. Les investisseurs ont apprécié la ratification du plan de relance américaine par Donald Trump. Ce dernier l'avait vivement critiqué, notamment en raison de la faiblesse du chèque distribué aux ménages. Le président américain a finalement cédé à la pression du Congrès. Les trois principaux indices ont enregistré de nouveaux records. Le Dow Jones a gagné 0,68% à 30 403,97 points et le Nasdaq Composite a progressé de 0,74% à 12 899,42 points. Le S&amp;P 500 s'est adjugé 0,87% à 3 735,36 points.  Cet article est paru dans Option Finance (site web)
</article>
<article titre="Albioma: mise en service d'une nouvelle centrale au Brésil" date="2020 12 29" journal="ABC Bourse (site web)">
(CercleFinance.com) - Albioma annonce la mise en service industriel, le 25 décembre, de sa centrale 100% bagasse Vale do Paraná Albioma au Brésil, une opération représentant un investissement de 121 millions de réais brésiliens.   Située à Suzanápolis, dans l'Etat de São Paulo, cette unité de cogénération est la quatrième centrale du groupe au Brésil. Elle bénéficie d'un contrat de vente d'électricité à long terme de 120 GWh sécurisé jusqu'en 2046 et indexé sur l'inflation. Détenue conjointement par Albioma et l'usine Vale do Paraná, cette centrale est adossée à la sucrerie-distillerie du même nom. D'une puissance installée de 48 MW, elle sera en mesure d'exporter jusqu'à 30 MW d'électricité renouvelable sur le réseau.   Copyright © 2020 CercleFinance.com. Tous droits réservés.  Cet article est paru dans ABC Bourse (site web)
</article>
<article titre="Les valeurs à suivre mardi 29 décembre 2020 à Paris -" date="2020 12 29" journal="Option Finance (site web)">
AOF - 29 décembre 2020    (AOF) - AKKA TECHNOLOGIES   Akka a annoncé avoir lancé une procédure de consultation des instances représentatives du personnel en France qui devrait déboucher sur la mise en oeuvre d'un plan de restructuration, la reprise dans le secteur Aéronautique ne se matérialisant pas. A ce stade, les discussions avec les partenaires sociaux portent sur la suppression d'environ 900 postes, principalement dans la région Occitanie. Le groupe affirme tout mettre en oeuvre en parallèle pour trouver des solutions à la sévérité de la crise actuelle et sauvegarder un maximum d'emplois. ALBIOMA   Albioma a annoncé la mise en service industriel, le 25 décembre 2020, de sa centrale 100 % bagasse Vale do Paraná Albioma au Brésil. Située dans la ville de Suzanápolis dans l'état de São Paulo, cette unité de cogénération biomasse est la quatrième centrale du groupe implantée au Brésil et bénéficie d'un contrat de vente d'électricité à long terme de 120 GWh sécurisé jusqu'en 2046 et indexé sur l'inflation. L'investissement de l'opération s'élève à 121 millions de réais brésiliens.   EUROFINS   Eurofins Génomique a annoncé le lancement d'un nouveau service de séquençage du SARS-CoV-2, optimisé et rentable, qui fournit des séquences complètes du génome viral. L'ARTIC NGS Oligo, synthétisé par la branche Synthèse d'Eurofins Genomics, sera également commercialisé sous peu. Le nouveau service NGS et le kit oligo ARTIC NGS sont les derniers ajouts d'Eurofins à un portefeuille de produits complet destiné à soutenir les chercheurs du monde entier dans le développement de vaccins et dans leur lutte contre la pandémie de Covid-19.   EUROPLASMA   Le Conseil d'administration d'Europlasma, faisant usage de la délégation de compétence conférée par l'Assemblée générale extraordinaire qui s'est tenue ce jour, a décidé d'émettre 650 obligations convertibles en actions (OCA) au profit du fonds Global Tech Opportunities 1. Cette opération s'inscrit dans le cadre de l'accord du 12 novembre 2020 permettant un désendettement massif du Groupe à hauteur de 21 millions d'euros. Le prix de souscription d'un montant nominal total de 6,5 millions d'euros est intégralement réglé par Global Tech par compensation avec la créance liquide.   PRISMAFLEX   Prismaflex a annoncé avoir finalisé un accord d'association au capital de sa filiale Anthem Displays avec la société américaine Circle Graphics, un imprimeur numérique américain du au Groupe H.I.G. Capital, présent à la fois sur les marchés de la décoration en ligne et de l'affichage extérieur. À l'issue des opérations de cession et d'augmentation de capital (en numéraire et par incorporation de comptes courants) visant à renforcer les fonds propres et la trésorerie d'Anthem Displays, Circle Graphics devient l'actionnaire majoritaire, Prismaflex USA conservant une participation de 28,91%.  Cet article est paru dans Option Finance (site web)
</article>
<article titre="Analyse AOF clôture France/Europe" date="2020 12 29" journal="Boursorama (site web réf.) - Boursorama">
(AOF) - Les marchés européens ont fini proches de l'équilibre, avec un biais haussier. Les catalyseurs des dernières séances perdent de leur puissance : campagnes de vaccination en Europe et ratification par...  Voir l'article  Ce document référence un lien URL de site non hébergé par Cision.
</article>
<article titre="Pourquoi Albioma mise sur la biomasse importée et la géothermie" date="2020 12 21" journal="L'Usine Nouvelle (site web)">
Après la Guadeloupe, Albioma lance à La Réunion la conversion du charbon à la biomasse de la centrale électrique de Bois-Rouge. Mais une biomasse importée à 70 %, ce qui empêchera définitivement d'atteindre l'autonomie électrique en 2030 de ces îles, pourtant prévue dans leurs programmations pluriannuelles de l'énergie. Albioma a le feu vert de la Commission de régulation de l'énergie (CRE) depuis le 3 décembre 2020. L'opérateur de centrales bagasse charbon en outre-mer (567 personnes, 506 M€ de chiffre d'affaires en 2019) va pouvoir convertir à 100 % à la biomasse la centrale Bois-Rouge à la Réunion. Il a, pour ce faire, négocié avec la CRE une prolongation de son contrat de ventes d'électricité à EDF jusqu'en 2043. La conversion, qui devrait être totale en 2024, se déroulera en deux temps. Elle commencera par la première tranche, qui fonctionne intégralement au charbon. Elle sera arrêtée en 2021 et fonctionnera à partir de 2022 à 100 % avec de la biomasse importée, dans un premier temps des États-Unis. "La biomasse viendra peut-être d'ailleurs que des États-Unis dans un deuxième temps, comme l'Afrique du Sud. Nous travaillons à une régionalisation de la filière biomasse", observe Frédéric Moyne, le PDG d'Albioma. Mais pour ce dernier ce n'est pas un problème. La biomasse importée des États-Unis est durable et certifiée. À La Réunion, elle permettra de réduire au moins de 84 % les émissions de la centrale de Bois-Rouge. 100 % de biomasse locale impossible Viendront ensuite la conversion des deux tranches bagasse-charbon, en 2023. Celles-ci devraient pouvoir utiliser de la biomasse locale, issue de déchets forestiers, de palettes ou de canne à sucre, aujourd'hui non valorisés. Mais, au mieux, Bois-Rouge consommera à terme 30 % de biomasse locale. "Le 100 % de biomasse locale n'est pas possible, les ressources locales ne sont pas suffisantes dans les îles", explique Frédéric Moyne. %%ARTICLELIE:810885%% Et pas vraiment possible pour Albioma d'attendre qu'une filière se mette en place même si dans un communiqué, l'entreprise assure donner la "priorité aux gisements locaux de combustibles pour favoriser l'économie circulaire". L'électricien fait néanmoins son possible : il a participé à la cartographie des gisements, fourni les spécifications techniques et signé des accords avec des acteurs locaux, comme l'Office national des forets ou le Conseil régional, s'engageant à fournir des débouchés à la biomasse locale sur vingt ans, sachant qu'Albioma envisage de convertir aussi une autre centrale de la Réunion, celle de Le Gol. La piste de brûler des combustibles solides de récupération (CSR) semble, elle, oubliée. Première conversion à la Guadeloupe Le montant de l'investissement pour Bois-Rouge n'a pas été communiqué. On sait juste que, pour une conversion similaire à la Guadeloupe, Albioma a investi 80 millions d'euros "pour la première tranche", précise le PDG de l'entreprise. Bois-Rouge n'est en effet pas une première. Albioma a mis en service le 20 novembre la tranche 3 de la centrale Albioma Le Moule en Guadeloupe convertie à la biomasse, là aussi importée à 100 % des États-Unis. Un impressionnant dôme de stockage de 20 000 m³, a même été construit sur le port de Jarry. Reste à convertir les tranches bagasse-charbon. Et dans l'hypothèse la plus optimiste, là aussi, l'approvisionnement en biomasse locale ne devrait pas dépasser 30 %. Mais c'est à la Martinique, en 2018, qu'Albioma a inauguré la première centrale bagasse/biomasse, celle de Galion 2 - La Trinité. Et en 2019, à la Réunion, Albioma réalisait une première mondiale avec la mise en service d'une turbine à combustion (TAC) fonctionnant majoritairement au bioéthanol (à 80 %) local issu de canne à sucre. Des installations qui ont déjà permis à Albioma de passer de 36 % d'énergie renouvelables dans son mix en 2013 à 67 % en 2019. 95% à 100% de renouvelables en 2030 Le groupe, qui opère aussi des centrales bagasse au Brésil et thermique fossiles à Mayotte et à Ile Maurice, vise maintenant entre 95 % et 100 % d'ici 2030, avec un palier à 80 % autour de 2023 et plus de 90 % en 2025. Pour atteindre cet objectif, Albioma va continuer à développer le solaire. Frédéric Moyne dit s'intéresser aussi de très près "depuis plusieurs mois à la géothermie comme piste de diversification", mais hors des territoires français. "C'est une énergie renouvelable locale, compétitive car pilotable et non intermittente, comme la biomasse, et pour laquelle la barrière à l'entrée est importante", précise le dirigeant. On pourrait s'attendre à des investissements ou des acquisitions de compétences dans ce domaine dans les prochains mois.
</article>
<article titre="Une nouvelle ère pour les bioplastiques" date="2020 12 19" journal="Emballages magazine (site web)">
Le contexte réglementaire très touffu induit par la transition écologique plaide pour une approche raisonnée des plastiques conventionnels et biosourcés. Décryptage en quatre points, tandis que le Conseil des ministres de l'environnement de l'Union européenne, réuni le 17 décembre, plaide pour la définition d'un cadre clair. De même que les plastiques issus de ressources fossiles, dits conventionnels, sont des polymères aussi différents que les polyéthylènes (PE), les polypropylènes (PP), les polystyrènes (PS), les polyéthylènes téréphtalates (PET) et autres polyamides (PA), les bioplastiques ressemblent à une grande famille recomposée. Emballages Magazine propose un décryptage des enjeux en quatre points : sémantique, éthique, écologique et stratégique. Polysémique, le préfixe « bio » est en réalité un faux ami. C'est le premier point, sémantique, sur lequel Emballages Magazine revient régulièrement dans ses colonnes depuis plusieurs années déjà. Dans l'univers des bioplastiques, apparaissent ainsi des polymères issus de ressources fossiles et/ou biodégradables et/ou compostables et/ou hydrosolubles voire recyclables; d'autres sont issus de la biomasse- les plastiques dits biosourcés issus de ressources renouvelables végétales ou animales et autres (algues, bactéries... )- mais ne sont ni biodégradables, ni compostables ni hydrosolubles mais recyclables le cas échéant, tandis que d'autres, en apparence identiques, sont au contraire et/ou biodégradables et/ou compostables et/ou hydrosolubles voire recyclables. Certains peuvent même être ingérés... Quant aux oxo(bio)dégradables, ils ont suffisamment fait couler d'encre pour ne pas y revenir ici. Tous les croisements sont donc possibles. Les bioplastiques sont-ils des plastiques comme les autres ? Jouer sur les mots est tentant. La principale différence entre renouvelable et fossile tient au cycle du carbone : cycle court pour le vivant par opposition au cycle très long lors de la dégradation en pétrole ou en gaz. Le choix d'un matériau relève in fine d'un cahier des charges intégrant stratégie du donneur d'ordre, usages, propriétés physico-chimiques, fonctionnalités, coûts et fin de vie. Toutefois, le sujet étant très complexe, le recours à une matrice très détaillée s'impose pour appréhender les mille nuances qui distinguent acide polylactique (PLA), polytéréphtalate de butylène (PBT), polybutylène succinate (PBS), polyhydroxyalcanoate (PHA), bioPE, bioPET et autre polyéthylène furanoate (PEF). Pour n'en citer que quelques-uns. Il convient bien sûr de convoquer la cellulose, ce polymère naturel étant le premier des biométariaux. Sans OGM Sujet hautement sensible, le deuxième point, éthique, concerne les organismes génétiquement modifiés (OGM). Qu'un emballage alimentaire puisse être estampillé « sans OGM » a de quoi laisser perplexe quand cette mention concerne l'emballage et non pas l'aliment... Bien conscient de l'importance de l'enjeu, European Bioplastics se veut très rassurant : l'association européenne indique que, si d'aventure, un transformateur utilise encore des ressources modifiées génétiquement, les nombreuses étapes de transformation qui succèdent suppriment le risque de transmission des OGM aux consommateurs. En revanche, certaines marques, en vertu de leur responsabilité sociétale (RSE), ne souhaitent pas être associées à des matériaux provenant de pratiques susceptibles d'exercer une pression sur les cultures vivrières, nuire à la biodiversité, entraîner la déforestation ou nécessiter un usage intensif de l'agrochimie. Les analyses de cycle de vie (ACV) sont déterminantes en la matière. C'est pourquoi la biomasse de deuxième ou troisième génération- les déchets agricoles comme la bagasse ou les sous-produits de l'industrie- apparaît comme une solution très prometteuse. Un champ de recherche infini et porteur d'innovations dans le cadre de l'économie circulaire... Tous ces matériaux entrent naturellement dans le champ des réglementations applicables au contact alimentaire avec ses migrations globales et spécifiques. Filière de valorisation La fin de vie, troisième point écologique, demeure dans l'immédiat un vrai casse-tête. En dehors d'initiatives locales et de démarches de particuliers, la France ne dispose pas encore de filières de tri, de collecte et de valorisation des matériaux biodégradables et/ou compostables. Le traitement des biodéchets doit émerger à partir de 2023 en France et en 2025 au sein de l'Union européenne. Craignant que les termes « biodégradable » ou « compostable » incitent les consommateurs à abandonner leurs emballages dans la nature, les pouvoirs publics, suivis par de nombreuses associations comme le Conseil national de l'emballage (CNE) et des organisations non gouvernementales (ONG), plaident pour un usage extrêmement encadré de ces allégations. A noter d'ailleurs que Barbara Pompili, ministre de la Transition écologique, a confié à l'avocat Arnaud Gossement un rapport sur le « greenwashing » à rendre en janvier 2021. Comme tous les matériaux pour l'emballage, les bioplastiques doivent intégrer une filière de valorisation sans perturber les flux existants sous peine de se voir infliger un malus. Créer une nouvelle filière constitue une solution... si les volumes sont au rendez-vous. Un cercle vicieux qui constitue l'un des chantiers de Citeo, l'organisme agréé pour la valorisation des emballages et des papiers au titre de la responsabilité élargie des producteurs (REP). Enfin, entre compostages industriel et/ou à domicile et autres subtilités byzantines, les consommateurs ont largement de quoi y perdre leur latin. Quant aux normes EN 13432, relative aux emballages compostables, et NFU 44051, relative aux amendements organiques, elles ne sont pas établies sur les mêmes critères. Pas simple. Contexte réglementaire touffu Quatrième et dernier point stratégique, la crise sanitaire de 2020 ayant mis l'économie mondiale à l'arrêt, les cours des plastiques issus de la pétrochimie sont très bas. Une conséquence classique du décalage entre l'offre et la demande. Avec pour effet immédiat que les matériaux issus du recyclage et les alternatives biosourcées, déjà très chers, ne sont pas du tout compétitifs. Ecologique, la transition est aussi économique. Alors que le contexte réglementaire est déjà très touffu, qui voit se télescoper les décrets d'application de la loi dite Antigaspillage pour une économie circulaire (Agec), mais aussi la directive européenne sur les plastiques à usage unique (Sup), la révision de la directive 94/62/CE ou encore les engagements de la Convention citoyenne pour le climat (CCC), le Conseil des ministres de l'environnement de l'Union européenne, réuni le 17 décembre, plaide pour la définition d'un cadre clair. Les bioplastiques entrent assurément dans une nouvelle ère. Riche de nouvelles perspectives et de nouveaux usages à inventer dans une approche raisonnée.
</article>
<article titre="La Réunion vise une production d’électricité « 100 % renouvelable » en 2023" date="2020 12 18" journal="Le Monde (site web)">
La Réunion, jusqu’ici très dépendante des énergies fossiles, prépare sa « révolution verte ». Actuellement, l’électricité de l’île de l’océan Indien est issue à 36 % de centrales à charbon et à 33 % de turbines fonctionnant au fioul lourd, pour 31 % d’énergies renouvelables. Mercredi 25 novembre, le conseil régional a voté une révision de la programmation pluriannuelle de l’énergie (PPE) avec pour objectif un « verdissement du mix électrique » pour la fin de 2023. Le document, qui doit être approuvé par l’autorité environnementale et la Commission de régulation de l’énergie puis faire l’objet d’un décret, prévoit une production électrique 100 % renouvelable et décarbonée grâce à de la biomasse locale ou à des importations. L’île vise l’autonomie énergétique en 2030.   Cette conversion s’appuie aussi sur une production doublée de l’énergie solaire grâce à l’autoconsommation chez les particuliers et à l’installation de panneaux photovoltaïques sur des bâtiments existants ou des exploitations agricoles. L’ensemble des investissements représente une enveloppe de près de 2 milliards d’euros. « C’est une vraie révolution qui va être opérée en trois ans, s’est félicité le président du conseil régional, Didier Robert (divers droite). Un pas de géant va être franchi en supprimant une partie de l’impact carbone qui est détestable. Peu de territoires l’ont fait. »    Réduire les rejets de soufre Comme les autres départements ultramarins, La Réunion doit gérer localement ses besoins en électricité : ceux de ses 860 000 habitants, des entreprises et des collectivités. Les coûts de production y sont environ deux fois et demie plus élevés que dans l’Hexagone, où le nucléaire reste la première source d’approvisionnement. Le tarif reste toutefois identique en vertu du principe de péréquation tarifaire nationale.   « Nous devons en permanence assurer un équilibre local entre l’offre et la demande, car nous ne pouvons pas acheter de l’électricité à l’extérieur », observe Olivier Meyrueis, directeur régional d’EDF à La Réunion. « Ce que consomment nos 350 000 abonnés, nous le produisons », ajoute-t-il en montrant la grande carte digitale de l’île dans la salle sécurisée du « dispatching ». Deux techniciens y assurent en permanence la surveillance de l’ensemble du réseau et le suivi de la distribution, avec ses centrales et ses lignes haute tension de 63 000 volts.   La fin de la production d’électricité à partir d’énergies fossiles va entraîner la conversion des trois principales centrales de l’île. Au Port-Est, le fioul lourd acheminé par pétrolier d’Arabie saoudite pour alimenter les douze moteurs sera remplacé par du biocarburant fabriqué principalement à partir de colza et de soja. « Ce biocombustible arrive d’Europe, précise Alexandre Sengelin, directeur de cette centrale d’EDF PEI (production électrique insulaire). Car nous sommes engagés à respecter les règles de traçabilité de la directive Red II (renewable energy directive), qui exclut l’huile de palme. » Outre une diminution importante d’émission de CO2, l’utilisation du biocarburant devrait permettre une réduction des rejets de soufre, de métaux lourds et de poussières dans l’atmosphère, affirme Alexandre Sengelin.   « Emplois locaux » Les deux centrales thermiques du Gol, à Saint-Louis, et de Bois-Rouge, à Saint-André, gérées par l’opérateur privé Albioma, ne brûleront plus le charbon acheté en Afrique du Sud auquel est ajouté, pendant la campagne sucrière (de juillet à décembre), de la bagasse – les résidus fibreux de la canne à sucre. Les fours fonctionneront uniquement à partir de biomasse locale ou importée d’ici à deux ans. Quelque 70 % des besoins seront couverts avec des achats de granulés de bois issus des scieries et d’exploitations forestières à des producteurs sud-africains ou mozambicains.   Parallèlement à celle de la bagasse, une filière locale va être développée grâce à la collecte de déchets en bois (emballages et palettes) et de déchets verts. « Nous nous appuyons sur l’économie circulaire avec des activités de tri et de broyage, durables et sans conflit d’usage, qui vont générer des emplois locaux », souligne Pascal Langeron, directeur adjoint d’Albioma et responsable des sites de La Réunion et de Mayotte. Plusieurs dizaines de millions d’euros vont être investis pour la modification des chaudières et dans la construction de lieux de stockage des granulés de bois.   A plus long terme, la PPE fixe comme objectif, pour 2028, une production électrique assurée à 38 % par de la biomasse solide et à 17 % par du biocarburant. L’énergie hydraulique issue des réservoirs d’eau de la rivière de l’Est et du barrage de Takamaka représentera 16 %. La part prise par le photovoltaïque devrait atteindre 18 % et l’éolien terrestre 4 %, avec un quasi-doublement des capacités de production. Et 7 % proviendront de la combustion de déchets liée au projet d’incinérateur de « l’unité de valorisation énergétique » (UVE) − installée à côté de la décharge de Pierrefonds −, auquel la région était jusque-là opposée.   « Le 100 % renouvelable, c’est un moment-clé pour la Réunion, plaide Olivier Meyrueis. Cela constitue un verdissement très fort. Le transport de la biomasse solide et liquide reste émetteur de CO2, on ne peut pas l’ignorer. Mais cette part du transport de ces combustibles devient marginale par rapport à la consommation de fioul lourd, moins de 2 %. »   Téléphériques et pistes cyclables Secrétaire régional d’Europe Ecologie-Les Verts (EELV), Jean-Pierre Marchau se montre moins enthousiaste et constate que l’Etat, La Réunion et EDF ne font que « se mettre dans les clous » sur le seuil d’énergie renouvelable inscrit dans la loi de transition énergétique de 2015. Le conseiller municipal de Saint-Denis critique aussi « l’effet d’annonce » sur l’autonomie énergétique pour 2030. « Le terme est trompeur. La Réunion va importer de la biomasse liquide et solide. Nous restons dans des schémas de dépendance. »   Pour le conseiller régional Olivier Hoarau (divers gauche), seul élu d’opposition à avoir voté contre la révision de la PPE, « cette programmation manque d’ambition dans la construction d’une stratégie locale de biomasse et de l’autonomie. Pourtant, des solutions existent, comme la production d’acacias [dont les parcelles pourraient alimenter la filière bois-énergie de l’île] ».   La réduction de la consommation énergétique constitue un autre objectif majeur de cette PPE. Un enjeu autant écologique que social dans un département où un quart de la population vit sous le seuil de pauvreté. Un programme d’aides est prévu pour permettre l’installation de chauffe-eau solaires, l’isolation des habitations des hauts de l’île, où les températures sont fraîches la nuit et pendant l’hiver austral, l’utilisation d’appareils ménagers et de l’éclairage à basse consommation. L’objectif est d’atteindre 8 % de réduction en 2028 malgré une population en croissance, qui devrait approcher à cette date 916 000 habitants.   Reste le poids des énergies fossiles dans les transports, qui représentent deux tiers de la consommation finale d’énergie. Dans une île où la voiture est encore reine et où le transport collectif est peu performant, l’ambition est de hisser la part de ce dernier de 7 à 14 % avec la construction de « voies guidées » (trams et bus), de téléphériques et de pistes cyclables. La PPE réunionnaise entend aussi multiplier par seize le nombre de véhicules électriques d’ici à 2028. Une autre révolution en perspective pour l’île de l’océan Indien.  Cet article est paru dans Le Monde (site web)
</article>
<article titre="Les producteurs de canne à sucre veulent ériger la  bagasse au rang de combustible" date="2020 12 17" journal="Les Echos (site web)">
Premier vice-président de la Chambre d'agriculture et ancien président des Jeunes agriculteurs, Bruno Robert a un message clair. Sans revalorisation de la bagasse, ce résidu de la canne à sucre, aucun bateau de biomasse importée du Canada ou d'ailleurs n'accostera à La Réunion. Bruno Robert a lancé les hostilités sur sa page Facebook dès que Albioma, l'opérateur de centrales électriques, a reçu le feu vert de la Commission de régulation de l'énergie (CRE) pour convertir la centrale thermique de Bois-Rouge en centrale à biomasse.   « Nous pouvons légitimement réclamer une égalité de traitement entre la biomasse locale et la biomasse importée »  , a-t-il défendu.   De leur côté, les Jeunes agriculteurs ont indiqué que le prix de la bagasse était indexé sur un prix de l'énergie, à savoir 95 euros/MWh alors que la CRE fixerait celui de la biomasse importée autour de 180 euros/MWh. D'où la volonté du syndicat d'obtenir un rééquilibrage des prix.   Bois-Rouge passe au vert L'enjeu est de taille car, en concertation avec la région et EDF, Albioma s'est engagée à convertir à la biomasse d'ici 2023 sa centrale thermique de Bois-Rouge (108 MW) au charbon. Laquelle satisfait environ 25 % de la consommation électrique du territoire. De quoi cesser d'importer 600.000 tonnes de charbon chaque année et réduire de 84 % les émissions de gaz à effet de serre. L'objectif global étant d'obtenir un mix énergétique 100 % renouvelable sur l'île.   Etape importante, Albioma annonce que le feu vert de la CRE conduit à prolonger son contrat de vente d'électricité jusqu'en 2043. Les travaux de conversion débuteront dès 2021 pour que la centrale de Bois-Rouge fonctionne entièrement à la biomasse au second semestre 2023. En attendant, Albioma fait déjà tourner sa centrale à cogénération du Gol (122 MW) partiellement à la bagasse.   Vers un statut de combustible De quoi donner de l'espoir aux planteurs qui voudraient augmenter leurs ventes de bagasse auprès des centrales électriques. En tout cas, tel est le rêve que caresse la filière. D'autant que, en novembre dernier, Albioma estimait que, pour remplacer le charbon à la centrale thermique de Bois-Rouge, le bois de palettes pourrait être amené à sortir de son statut de « déchet » pour celui de « combustible », plus rémunérateur. Pourquoi ne pas étendre ce statut à la bagasse ?
</article>
<article titre="Charbon électrique : La Réunion et Albioma ont pris la voie de sortie" date="2020 12 17" journal="GreenUnivers (site web)">
2020-12-18T17:58:04+00:00    (Bois Rouge, @Albioma)   Ce n'est pas une surprise mais quand même un événement pour La Réunion. En début de mois, la Commission de régulation de l'énergie (CRE) a donné son feu vert à Albioma pour convertir à la biomasse sa centrale thermique de cogénération d'électricité - et de vapeur pour la sucrerie voisine - de Bois Rouge au nord de l'île. Elle est alimentée surtout au charbon et un peu à la bagasse, un résidu de la canne à sucre. Ses 108 MW produisent le quart de l'électricité consommée par les réunionnais. Des travaux vont être engagés pendant les deux ans qui viennent pour l'approvisionner à 70% en granulés de bois nord américains et le solde en bois local. A l'issue, la moitié du  mix  électrique du territoire sera non fossile, contre un tiers pour le moment, le fioul pesant 30% de la génération actuelle.  59 M€ par an La CRE vient de valider - sans les rendre public - les fonds à mobiliser par cet énergéticien coté en Bourse, de même que la rémunération à 8,3% des capitaux et un contrat d'achat de l'électricité par EDF prolongé jusqu'en 2043. Sur la durée, le coût pour la collectivité est évalué à 59 M€ par an sur la base du prix de la tonne de carbone en 2019 ; mais à dix fois moins quand [restrict-content]la CRE se base sur la hausse de la Contribution climat énergie, gelée à 44,6€/tonneCO2 depuis 2019 et pourtant toujours censée atteindre 100€ en 2030, en théorie. « En l'état actuel des cours du charbon, le coût d'équilibre avec la biomasse suppose que la tonne de CO2 évitée se situe entre 50 et 60€ », situe Frédéric Moyne, PDG d'Albioma. Quel que soit le destin de la taxe carbone tricolore, cette tendance à la hausse est considérée comme un scénario de base par l'énergéticien ; il reste maintenant à voir quel en sera le rythme.   Albioma en est à sa troisième conversion vers l'électrique végétal après la mutation mouvementée de la centrale Gallion 2 en 2018 en Martinique, puis celle de Le Moule 3 en Guadeloupe en novembre dernier. Contrairement à la métropole, la décarbonation électrique outremer n'est pas obligatoire (Loi énergie-climat) mais encouragée. En témoigne le soutien public accordé, lui-même cohérent avec la planification énergétique à long terme définie en 2016. Il faut néanmoins tenir compte de la sécurité d'approvisionnement du réseau électrique et donc échelonner les travaux, prévient Frédéric Moyne. A La Réunion, la centrale Le Gol rejoindra la biomasse en 2024 et en Guadeloupe, les tranches Le Moule 1 et 2 devraient le faire l'année suivante. Avec Bois Rouge dans deux ans, le producteur approchera de la moitié du chemin, la décarbonation de la totalité de son parc thermique étant prévue d'ici 2030.   Traçabilité annoncée complète Pour être non fossiles, les combustibles de substitution ne sont pas automatiquement vertueux. Bois Rouge devant consommer la bagatelle de 400 000 tonnes de granulés de bois chaque année, en provenance des Etats-Unis pour le moment - l'Afrique du Sud voisine pourrait s'y substituer -, Albioma veut être capable de répondre aux questions sur le bilan carbone et la déforestation. « En comparaison du charbon, et transport compris, les granulés affichent 84% de CO2 en moins. Nous exigeons des fournisseurs une traçabilité complète et des certifications. Certains nous proposent un repérage des parcelles qu'ils exploitent », se félicite le dirigeant, qui s'est rendu sur place pour se rendre compte et estime que « la plupart des producteurs font correctement leur travail. Les inquiétudes, légitimes, sont provoquées par une minorité » .   D'autres interrogations émergent à La Réunion sur les prélèvements forestiers locaux auxquels Albioma donne d'ailleurs la priorité par rapport aux granulés, mais sans pouvoir dépasser en pratique un taux de 30%. "S'il était possible de remplacer 600 000 tonnes annuelles de charbon par le bois et la bagasse, nous l'aurions déjà fait. L'important est ici de veiller aux procédés", affirme Frédéric Moyne. Dans la forêt insulaire, seront par exemple recherchées les espèces invasives comme l'acacia, ou du bois d'éclaircie complété par des chutes et résidus, illustre-t-il, en soulignant que ce nouveau  sourcing  crée des emplois, à la différence du charbon importé. [/restrict-content]   Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="Restaurateurs et traiteurs :" date="2020 12 16" journal="La République du Centre  Loiret">
Contrairement à ce que l'on peut croire, tous les restaurateurs ne font pas de la vente à emporter. Et parmi ceux qui la pratiquent, tous ne procèdent pas de la même façon.   Certains la pratiquent seulement pour les fêtes    « Nous ne la ferons que pour les fêtes : les 24 et 25 décembre, 31 et 1 er janvier », résume Aurélien Danchot, du Biquin d'Or à Ferrieres. C'est qu'on est ici en campagne et que nous n'avons pas une demande suffisante pour rallumer les fourneaux et rappeler des employés. Autant être fermés et attendre. Alors, nous le faisons pour les fêtes et surtout pour faire plaisir à nos clients, des habitués à 80 %. Pour l'instant, nous avons d'ailleurs davantage de demandes pour Noël, même si nombre de résidents secondaires attendent encore de savoir si leurs familles pourront les rejoindre ». D'autres ont pensé une forme de vente pérenne    Franck Foucard, le chef de L'Orangerie à Montargis, et son épouse Catherine ont beaucoup réfléchi avant de mettre en place la vente à emporter. À leur sauce. « Nous étions certains qu'il y aurait à nouveau confinement. Nous avons donc organisé quelque chose de pérenne dans le temps ». C'est ainsi qu'est née « L'Orangerie chez moi » avec une idée maîtresse : le couple voulait que le restaurant gastronomique qu'il a créé en 1992 entre véritablement chez ses clients dans le moindre détail.   « Il nous a fallu faire des recherches, car nous voulions garder le lien, tant le contenu que la prestation, les saveurs », explique le chef, qui met un point d'honneur à soigner sa présentation à l'identique. Pour ce faire, il a jeté son dévolu sur des boîtes en bagasse, de la fibre de canne à sucre passant au micro-ondes comme au four. C'est un vrai menu de fêtes qu'il propose jusqu'au 3 janvier. Et après ? « C'est désormais une action concomitante au restaurant. Nous allons poursuivre avec une carte différente chaque mois ». Voilà qui contribue à diversifier une clientèle qui n'aurait pas forcément poussé la porte d'un « Michelin » : « Les gens ont envie d'essayer. Ils commandent via notre site Internet ou ils appellent et on les revoit au restaurant ».    Les thèmes cela fonctionne    Dans de très petites communes, on se débrouille comme on peut. Chez Suor Heng, de l'Auberge de conflans, on se débrouille plutôt bien ! « Il faut dire que nous avions lancé les vendredis à thème voilà plusieurs années et que cela a bien pris. Le cassoulet, le couscous plaisent particulièrement par exemple. Alors nous poursuivons avec les vendredis et samedis à thèmes à emporter », résume Gaëlle, son épouse. « Par ailleurs, nous proposons une offre journalière complète, une carte différente par semaine qui va s'arrêter pour laisser place à la carte des fêtes du 21 décembre au 2 janvier ». Et après la crise sanitaire ? « Nous reviendrons à une carte par saison en y ajoutant la possibilité d'emporter ». Bien sûr, le chiffre d'affaires n'est en aucun cas comparable à ce qu'il est lorsqu'on mange au restaurant, et que s'ajoutent l'apéritif, le vin ou le café « D'autant que, en général, les gens ne commandent qu'un plat, et très peu un dessert ».    Et ils vont même au-devant des clients    Enfin, nous ne saurions achever ce petit tour d'horizon sans évoquer l'idée de Vanessa et Christophe Morel, des Pitchouns, à Montargis. Voilà un an, en parallèle de leur restaurant, ils ont décidé de se lancer sur le marché alimentaire du samedi ! On y retrouve la cuisine de Christophe dans des bocaux sous vide consignés. Et ça marche très bien ! « Sans le savoir, nous avons anticipé la crise », récapitule Vanessa (qu'on retrouvera également sur le marché du mercredi les 23 et 30 décembre, pour les fêtes). « Nous voulions nous inscrire dans une démarche responsable tout en touchant de nouveaux clients car notre restaurant est très petit. À notre grande surprise, beaucoup, de tous les âges, ont adhéré à notre vision des choses, reviennent rendre les bocaux Ce n'est pas le même métier, c'est énormément de travail mais aussi un nouveau lien et une grande satisfaction ! »   Laurence Bernard
</article>
<article titre="Albioma : annonce bien reçue !" date="2020 12 9" journal="Boursier (site web)">
Albioma monte de 1,7% à 42,20 euros ce mercredi, alors que le groupe a annoncé abandonner totalement le charbon sur son site historique de Bois-Rouge à La Réunion. Après délibération, la Commission de Régulation de l'Energie (CRE) a validé, le 3 décembre, la compensation induite par la signature d'un avenant au contrat d'achat d'électricité signé par EDF pour la conversion à la biomasse de la centrale. Les travaux de conversion débuteront dès 2021 pour que la centrale fonctionne 100% à la biomasse au second semestre 2023, en privilégiant les gisements locaux de biomasse disponibles (bagasse, bois forestier, bois d'élagage, etc.), complétés par des granulés de bois importés issus de forêts certifiées type FSC et PEFC, dont la procédure de traçabilité sera en conformité avec le règlement bois UE (RBUE).   A terme, la conversion fera passer la part renouvelable du mix énergétique de La Réunion de 35% à 51% et permettra de réduire les émissions de gaz à effet de serre d'environ 640.000 tonnes équivalent CO2 par an, soit une baisse de 84% des émissions directes par rapport au fonctionnement actuel de la centrale.   Prolongation du contrat de vente d'électricité jusqu'en 2043   La délibération CRE prévoit également d'investissements nécessaire à 15 l'exploitation l'unité Groupe (ABR1) ; d'achat d'électricité été conséquence prolongé 2027 à 2043 ABR1.   Rappelons Albioma Bois-Rouge d'une installée 108 MW. Elle 25% réunionnais...   "Ce à ENR énergétique d'Albioma >80% 2023, >90% en 2025 et près de 100% en 2030" commente Portzamparc qui souligne que du point de vue économique, "c'est également une bonne nouvelle car les investissements seront significatifs (? 150 ME selon nous) mais bien rémunérés (taux de rémunération du capital de 8,3%, soit TRI d'environ 15%) et s'accompagnent d'une prolongation de la maturité du contrat (2043 vs 2027) pour ABR1. La conversion des centrales Le Gol et Albioma Le Moule 1 et 2 sont prévues pour une mise en service en 2023/2024". Verdict : "Renforcer en visant un cours de 46,60 euros".
</article>
<article titre="Albioma: quitte le charbon pour la biomasse à la Réunion" date="2020 12 8" journal="ABC Bourse (site web)">
(CercleFinance.com) - Albioma annonce ce jour l'abandon total du charbon sur son site historique de Bois-Rouge à La Réunion, après délibération de la Commission de Régulation de l'Énergie (CRE).   Les travaux de conversion débuteront dès 2021 pour que la centrale fonctionne 100% à la biomasse au second semestre 2023, en privilégiant les gisements locaux de biomasse disponibles (bagasse, bois forestier, bois d'élagage, etc.), complétés par des granulés de bois importés issus de forêts certifiées type FSC et PEFC. A terme, la conversion fera passer la part renouvelable du mix énergétique de La Réunion de 35 % à 51 % et permettra de réduire les émissions de gaz à effet de serre d'environ 640 000 tonnes équivalent CO2par an, soit une baisse de 84 % des émissions directes par rapport au fonctionnement actuel de la centrale.   Copyright © 2020 CercleFinance.com. Tous droits réservés.  Cet article est paru dans ABC Bourse (site web)
</article>
<article titre="Albioma: quitte le charbon pour la biomasse à la Réunion" date="2020 12 8" journal="Cercle Finance (site web)">
Albioma annonce ce jour l'abandon total du charbon sur son site historique de Bois-Rouge à La Réunion, après délibération de la Commission de Régulation de l'Énergie (CRE).   Les travaux de conversion débuteront dès 2021 pour que la centrale fonctionne 100% à la biomasse au second semestre 2023, en privilégiant les gisements locaux de biomasse disponibles (bagasse, bois forestier, bois d'élagage, etc.), complétés par des granulés de bois importés issus de forêts certifiées type FSC et PEFC. A terme, la conversion fera passer la part renouvelable du mix énergétique de La Réunion de 35 % à 51 % et permettra de réduire les émissions de gaz à effet de serre d'environ 640 000 tonnes équivalent CO2par an, soit une baisse de 84 % des émissions directes par rapport au fonctionnement actuel de la centrale.   Copyright (c) 2020 CercleFinance.com. Tous droits réservés.
</article>
<article titre="Albioma : fin du charbon sur le site historique de Bois-Rouge à la Réunion" date="2020 12 8" journal="Boursier (site web)">
Albioma abandonne totalement le charbon sur son site historique de Bois-Rouge à La Réunion. Après délibération, la Commission de Régulation de l'Energie (CRE) a validé, le 3 décembre, la compensation induite par la signature d'un avenant au contrat d'achat d'électricité signé par EDF pour la conversion à la biomasse de la centrale. Début des travaux de conversion dès 2021   Les travaux de conversion débuteront dès 2021 pour que la centrale fonctionne 100% à la biomasse au second semestre 2023, en privilégiant les gisements locaux de biomasse disponibles (bagasse, bois forestier, bois d'élagage, etc.), complétés par des granulés de bois importés issus de forêts certifiées type FSC et PEFC, dont la procédure de traçabilité sera en conformité avec le règlement bois UE (RBUE).   A terme, la conversion fera passer la part renouvelable du mix énergétique de La Réunion de 35% à 51% et permettra de réduire les émissions de gaz à effet de serre d'environ 640.000 tonnes équivalent CO2 par an, soit une baisse de 84% des émissions directes par rapport au fonctionnement actuel de la centrale.   Prolongation du contrat de vente d'électricité jusqu'en 2043   La délibération de la CRE prévoit également une enveloppe d'investissements nécessaire à la prolongation de 15 ans de l'exploitation de l'unité la plus ancienne du Groupe (ABR1) ; le contrat d'achat de vente d'électricité a été en conséquence prolongé de 2027 à 2043 pour ABR1.   Rappelons que la centrale Albioma Bois-Rouge est d'une puissance installée de 108 MW. Elle produit environ 25% de la consommation du territoire réunionnais.
</article>
<article titre="â±â± Biocarburants Bientôt la 2G" date="2020 12 7" journal="Industrie et Technologies, no. 1037-1038">
Le vent souffle à arracher les casques. Les luminaires virevoltent. Les portes résistent à l'ouverture. Il faut lutter pour avancer, tendre l'oreille pour écouter. « Il y a souvent du vent, mais rarement comme ça, admet Jean-Philippe Héraud, ingénieur chez IFP Énergies nouvelles (Ifpen). Il ne faut pas avoir le vertige. » À 25 mètres du sol, les rafales sont d'autant plus violentes, mais la vue est imprenable : à nos pieds, le dépôt pétrolier de Total, au loin, le terminal méthanier, et plus à droite, le site d'ArcelorMittal et ses panaches de fumée emportés par les bourrasques.  Nous sommes à Dunkerque (Nord), sur la plate-forme qui surplombe l'énorme gazéifieur du projet BioTfuel. « C'est la pièce maîtresse du procédé », souligne Jean-Philippe Héraud, chargé du projet. Au coeur d'un labyrinthe de tuyaux qui grimpent jusqu'à 50 mètres, c'est elle qui permet de passer de la biomasse au gaz de synthèse - mélange d'hydrogène (H2) et de monoxyde de carbone (CO) - pour produire ensuite des biocarburants de deuxième génération, ou avancés. C'est-à-dire produits à partir de ressources qui n'entrent pas en compétition avec l'alimentation.   Plusieurs types de biomasse   Lancé il y a dix ans avec un budget de 180 millions d'euros, le projet BioTfuel arrive à son terme. En 2021, les partenaires (Ifpen, CEA, Total, Avril, ThyssenKrupp et Axens) devront avoir démontré la faisabilité industrielle de ce procédé thermochimique de production de biocarburants avancés, principalement tourné vers le biogazole et le biokérosène. Chaque étape a été longuement testée, et l'ensemble intégré : de quoi montrer que la production de biocarburants de deuxième génération à un coût compétitif est un objectif atteignable.    La biomasse utilisée ici est principalement du bois. Penché sur le sol, Jean-Philippe Héraud en ramasse des morceaux noircis de quelques centimètres. Écrasés entre ses doigts, ils sont réduits en poudre. « Avant d'arriver ici, le bois subit un prétraitement qui élimine les composés volatils et les molécules structurant sa fibre de façon à le rendre friable, explique-t-il. Cela limite le besoin en énergie pour l'étape de broyage. » Le prétraitement a lieu sur le site d'Avril à Venette, près de Compiègne (Oise). Les morceaux de bois sont d'abord séchés, puis torréfiés entre 250 et 300 °C sous atmosphère réductrice (sans oxygène). Acheminés à Dunkerque par camions, ils sont broyés pour alimenter le gazéifieur à un rythme de 3 tonnes par heure.    En plus du bois, d'autres ressources sont envisagées pour la France : la paille ou les cultures à vocation énergétique comme le miscanthus, une plante herbacée. Mais le procédé a vocation à être déployé à l'international, souligne Jean-Philippe Héraud. D'où l'ambition de traiter un large éventail de matières premières. « L'idée est de pouvoir utiliser des coques de noix de coco, du bambou, ou d'autres résidus comme la bagasse de canne à sucre. »    D'un point de vue technologique, les enjeux les plus importants résident dans les étapes de gazéification et de torréfaction. « Le four de torréfaction était éprouvé sur d'autres types de charges, mais pas sur de la biomasse qui génère de nouvelles difficultés, indique Jean-Philippe Héraud. Quant au gazéifieur, il est unique au monde. » L'installation de ThyssenKrupp, de type Prenflo direct quench, est une version améliorée d'une technologie que l'industriel allemand a déjà commercialisée pour produire de l'électricité à partir de charbon à Puertollano (Espagne). À son pied, une benne, avec des monticules de poudre sombre qui ressemble à du verre pilé. Il s'agit en réalité de cendres vitrifiées. Elles contiennent la partie minérale de la biomasse après son passage dans le gazéifieur à 1 500 °C en présence d'oxygène. C'est la particularité de l'équipement de BioTfuel par rapport à celui de Puertollano : en plus de la nature de la biomasse en entrée qui est différente, un quench permet un refroidissement du gaz de synthèse à basse température et la récupération des cendres résiduelles.    Jean-Philippe Héraud passe en revue les installations dédiées aux étapes suivantes. Une fois le gaz de synthèse produit, elles permettent de le préparer avant son passage dans l'unité Fischer-Tropsch (FT). « Toutes ces unités sont éprouvées et présentent moins d'enjeux d'un point de vue technologique. Mais encore faut-il savoir les intégrer les unes aux autres. » Un premier réacteur est le lieu d'une « réaction de gaz à l'eau » pour ajuster le ratio entre H2 et CO dans le gaz de synthèse. « En sortie de gazéification, il y a environ deux fois plus de CO que de H2, précise Jean-Philippe Héraud. Or il nous faut des proportions inverses en entrée de la synthèse FT. » Le gaz passe ensuite à travers une tour dans laquelle sont captés les gaz acides, comme le dioxyde de carbone, avant de subir un traitement final.   Créer une filière pour les matières premières   Un moyen de diminuer les coûts est d'exploiter les synergies possibles entre les différentes étapes : récupérer les gaz résiduels, la vapeur ou la chaleur produite par des réactions exothermiques pour les réutiliser ailleurs ou générer de l'électricité. Lors de la torréfaction, par exemple, des gaz comme du CO ou des hydrocarbures légers sont produits. Ils sont récupérés et rebrûlés pour générer la chaleur suffisante pour auto-entretenir le procédé. « Une fois la torréfaction lancée, nous nous passons d'apport externe de gaz naturel », assure Jean-Philippe Héraud. Si le procédé BioTfuel vise à produire du biogazole et du biokérosène, la synthèse FT ne produit pas uniquement ces deux carburants. D'autres produits présents en sortie sont valorisables pour optimiser encore plus les coûts. Il peut notamment s'agir de molécules pour faire des lubrifiants pour moteurs ou pour alimenter des vapocraqueurs et faire des oléfines destinées au marché de la pétrochimie des plastiques. Autre défi avant que des unités industrielles puissent voir le jour : structurer la récolte et l'approvisionnement en matières premières. « En France, il n'y a ni acteur ni filière pour le faire, souligne Jean-Philippe Héraud. Il faudra y travailler si l'on veut rendre le coût des matières premières compétitif. »    En bout de chaîne, dans le bungalow qui accueille l'unité FT, une table est couverte de flacons de verre. À l'intérieur, les substances produites jusque-là par cette synthèse : de l'eau dans certains, de la cire blanche dans d'autres. En s'approchant, l'odeur est forte. Elle contient un mélange de paraffines, d'oléfines et de composés oxygénés. Ces hydrocarbures sont la base qui permettra ensuite de fabriquer les carburants liquides voulus.    En ce mois d'octobre, aucun biogazole ou biokérosène n'a encore été produit à partir du gaz de synthèse issu du gazéifieur. « Nous devrions pouvoir le faire d'ici à quelques semaines, précise Jean-Philippe Héraud. Mais l'enjeu principal était de démontrer la faisabilité de la production du gaz de synthèse. C'est chose faite aujourd'hui. Il ne reste que quelques détails à finaliser pour valider la pureté du gaz. »   La commercialisation de la licence du procédé BioTfuel par Axens est donc espérée pour l'an prochain. Qui sera le premier à passer à l'échelle industrielle ? Difficile de le savoir pour l'instant. « Il y a des pistes partout dans le monde », confie Jean-Philippe Héraud. Quoi qu'il en soit, cela prendra du temps. Un projet industriel qui se lancerait maintenant ne démarrerait pas avant quatre à cinq ans selon lui. Quant au démonstrateur de Dunkerque, une fois sa mission terminée, son avenir est connu : il sera démantelé. -
</article>
<article titre="LA CANNE À SUCRE, UN PUITS À CARBONE" date="2020 11 19" journal="Courrier international, no. 1568">
Comment se porte l'AOC rhum de la Martinique ?   Marc Sassier : Nous avons pu faire notre campagne sans que personne ne soit contaminé par le Covid-19. Les distilleries se sont adaptées et ont également produit du gel hydroalcoolique pour répondre à la demande locale. En revanche, nous constatons localement un tassement des ventes de rhum, de 40 % à 60 %, lié à la baisse du tourisme. Toutefois, les consommateurs de la métropole privilégient le rhum produit dans les DOM, notamment celui de la Martinique.   En quoi le cahier des charges de l'AOC incite-t-il à des pratiques plus vertueuses pour l'environnement ?   L'AOC, obtenue en 1996, et son cahier des charges visent des conditions de production traditionnelle qui améliorent la qualité du rhum.   Depuis des années, nous produisons un effort, de la canne au rhum. Dès les années 1970, nous avons limité l'usage des pesticides, des fongicides et banni l'usage des OGM.   L'industrie du rhum pèse lourd dans l'économie de la Martinique (8 % des exportations de l'île). Dans quelle mesure ce secteur phare s'est-il saisi de la question de son impact sociétal et environnemental ?   La canne à sucre des rhums Saint James, de type « C4 », est un véritable puits à carbone. Parmi les différentes cannes à sucre, elle est l'espèce qui capte le plus de CO2 . Les plantations de canne à sucre en Martinique absorbent plus de 20 % du CO2 dégagé annuellement par le parc automobile de la Martinique. Et, au pied des cannes, il fait 1,5 °C plus frais.   En tant que responsable de la production de Saint James, comment la démarche RSE se traduit-elle au sein de la distillerie ?   De quand date-t-elle ? Cela fait vingt ans que Saint James a investi dans un méthaniseur et que chaque étape de la production est pensée de façon à minimiser l'impact sur l'environnement. Le fonctionnement en circuit fermé de notre distillerie lui assure une totale indépendance énergétique. La bagasse, résidu fibreux de la canne à sucre après pressage de son jus, est utilisée comme combustible afin de créer les vapeurs d'eau nécessaires au fonctionnement des colonnes de distillation et des moulins de broyage de canne. Une autre partie de la bagasse est placée en compost ou transformée en électricité via un turbo-alternateur.   La vinasse, le résidu liquide de la distillation, est traitée afin d'obtenir une eau servant à l'irrigation des champs de canne à sucre.   Quels prochains objectifs visez-vous ? Passer en tout biologique ?   Nous y réfléchissons. Nous travaillons avec un planteur de cannes bio. Mais passer à du 100 % bio en Martinique est, pour l'heure, une utopie. Le prix est multiplié par deux ou trois, car le rendement est plus faible : 45 tonnes par ha contre 70 t en agriculture conventionnelle. L'arrachage manuel rend également plus coûteuse la production, et on se heurte à des difficultés de recrutement de main-d'oeuvre.
</article>
<article titre="Les frères Girod préparent une sève de bouleau plus écoresponsable" date="2020 11 19" journal="Le Progrès (Lyon)">
Leur sève de bouleau n'aura pas le même goût cette année. À quelques semaines de la mise en vente de la cuvée 2021, les frères Girod, Thomas et Sébastien, fourmillent d'idées novatrices, fidèles à leur engagement pour la préservation de notre planète. Une goutte d'eau, certes, mais le proverbe "les petits ruisseaux font les grandes rivières" prend tout son sens dans ce contexte. « Associer la sève de bouleau à la reforestation » Les bienfaits d'une cure de sève de bouleau annuelle ne sont plus à démontrer, en témoigne l'engouement pour ce produit 100 % naturel. « Nous préparons une déclinaison de goûts cette saison, avec l'ajout dans la sève de 8 % d'eau florale, parfum menthe poivrée ou camomille, explique Thomas. Mais ce qui nous occupe cette année est l'aboutissement d'un projet qui nous tient à coeur, associer la sève de bouleau à la reforestation. »   Cela peut surprendre, mais les deux frères vont commercialiser une partie de leur prochaine cuvée dans des contenants 100 % végétaux, en bagasse de canne, biodégradables en 80 jours et compostables, du made in France de surcroît. « Nous avons eu écho d'un fabricant en Gironde de bouteilles de ce type, précise Thomas. Nous l'avons donc contacté afin d'obtenir un devis pour la fourniture de petites bouteilles de 25 cl (représentant une dose journalière), dans lesquelles nous pourrions conditionner notre sève. Hormis les aspects écologiques, la praticité des contenants nous a séduits. Nous avons alors déposé notre marque, Sévital, avec l'intention de mettre sur le marché une partie de notre prochaine production. »   15 000 euros pour concrétiser  le projet « Parallèlement, nous garderons les conditionnements en cubis, les clients y étant habitués. Seulement, l'argent étant le nerf de la guerre, nous avons souhaité faire appel à un financement participatif. Chaque donneur bénéficiera d'une compensation en fonction du montant versé. Mais avant tout, il participera à la préservation de la planète, 0,50 € pour chaque bouteille achetée étant reversé à Reforest'Action qui oeuvre pour la reforestation sur quatre continents. »   L'objectif des deux frères est de réunir cette année 15 000 euros qui leur permettraient de concrétiser leur projet. L'idée de prendre soin de sa santé et de savoir que l'on a participé à la plantation d'arbres dans un coin de la planète devrait séduire, c'est l'audacieux pari pris par Thomas et Sébastien Girod.
</article>
<article titre="La transition vers des emballages sans plastique" date="2020 10 29" journal="Journal le Guide (Cowansville, QC) (site web)">
Avec la montée du mouvement pour la protection de l'environnement, de plus en plus d'entreprises cherchent des solutions durables et adaptées à leur clientèle. L'emballage des différents produits se trouve souvent au coeur des discussions pour diminuer l'empreinte écologique d'un produit ou d'un service.   Cependant, de nombreuses entreprises se demandent toujours comment il est possible d'acheter des emballages respectueux de l'environnement sans augmenter leur budget. Dans cet article, nous explorerons les options disponibles sur le marché pour éviter d'utiliser du plastique. La popularisation des emballages en plastique remonte à la deuxième moitié du XXe siècle. Elle est grandement due à la légèreté et la résistance du matériau. Ses qualités physiques étaient très avantageuses pour de nombreuses industries. Toutefois, le plastique n'est pas parfait. Il ne se décompose pas rapidement dans la nature et envahit plutôt les sites d'enfouissement, les océans et les habitats naturels des animaux.   Pour éviter d'utiliser ce matériau très polluant, de plus en plus d'entreprises se tournent vers des emballages plus durables, comme les boîtes en carton et la canne à sucre.   Le changement vers l'emballage en carton    Le carton est un matériau fabriqué à partir de papier épais qui peut être recyclé ou non. Le carton est parfait pour protéger la majorité des produits étant donné qu'une entreprise peut facilement choisir l'épaisseur de ses emballages.   Par ailleurs, le carton se décompose beaucoup plus rapidement dans la nature. Il peut également être recyclé dans la majorité des municipalités du Québec. Le carton est ainsi un choix beaucoup plus respectueux de l'environnement que le plastique.   Finalement, le carton est une option qui ne devrait pas faire exploser votre budget d'emballage. En effet, il est facile de trouver des emballages en carton qui sont moins chers que ceux en plastique. En plus de diminuer l'empreinte environnementale de votre entreprise, vous économiserez de l'argent.   La popularisation des biomatériaux    De plus en plus de fabricants d'emballages se tournent vers des matériaux issus de la production agricole. La pulpe de canne à sucre, aussi appelée la bagasse, est l'une des options les plus populaires. Ces matériaux possèdent les mêmes qualités physiques que le plastique, dont l'étanchéité et la résistance aux chocs. Cependant, étant entièrement issue de matières naturelles, la bagasse se décompose complètement après quelques mois dans la nature. De plus, la bagasse est un matériau très peu dispendieux à produire, car il provient de la pulpe restante après l'extraction du jus des cannes à sucre.   Parmi les emballages en bagasse, on retrouve principalement des contenants alimentaires et des ustensiles. De plus en plus de restaurateurs optent pour ce matériau pour diminuer l'empreinte écologique de leur menu pour emporter.   Pour les autres entreprises, il existe également des sacs fabriqués entièrement en matière décomposable et différents contenants de transport, comme des boîtes et des sacs étanches.   Avec l'augmentation des lois interdisant le plastique à usage unique, les entreprises du Québec doivent trouver rapidement des solutions pour s'adapter. La transition vers les emballages en cartons et en biomatériaux est déjà un bon pas en avant pour la protection de l'environnement.  Cet article est paru dans Journal le Guide (Cowansville, QC) (site web)
</article>
<article titre="La transition vers des emballages sans plastique" date="2020 10 29" journal="Journal le Guide (Cowansville, QC) (site web)">
Avec la montée du mouvement pour la protection de l'environnement, de plus en plus d'entreprises cherchent des solutions durables et adaptées à leur clientèle. L'emballage des différents produits se trouve souvent au coeur des discussions pour diminuer l'empreinte écologique d'un produit ou d'un service.   Cependant, de nombreuses entreprises se demandent toujours comment il est possible d'acheter des emballages respectueux de l'environnement sans augmenter leur budget. Dans cet article, nous explorerons les options disponibles sur le marché pour éviter d'utiliser du plastique. La popularisation des emballages en plastique remonte à la deuxième moitié du XXe siècle. Elle est grandement due à la légèreté et la résistance du matériau. Ses qualités physiques étaient très avantageuses pour de nombreuses industries. Toutefois, le plastique n'est pas parfait. Il ne se décompose pas rapidement dans la nature et envahit plutôt les sites d'enfouissement, les océans et les habitats naturels des animaux.   Pour éviter d'utiliser ce matériau très polluant, de plus en plus d'entreprises se tournent vers des emballages plus durables, comme les boîtes en carton et la canne à sucre.   Le changement vers l'emballage en carton    Le carton est un matériau fabriqué à partir de papier épais qui peut être recyclé ou non. Le carton est parfait pour protéger la majorité des produits étant donné qu'une entreprise peut facilement choisir l'épaisseur de ses emballages.   Par ailleurs, le carton se décompose beaucoup plus rapidement dans la nature. Il peut également être recyclé dans la majorité des municipalités du Québec. Le carton est ainsi un choix beaucoup plus respectueux de l'environnement que le plastique.   Finalement, le carton est une option qui ne devrait pas faire exploser votre budget d'emballage. En effet, il est facile de trouver des emballages en carton qui sont moins chers que ceux en plastique. En plus de diminuer l'empreinte environnementale de votre entreprise, vous économiserez de l'argent.   La popularisation des biomatériaux    De plus en plus de fabricants d'emballages se tournent vers des matériaux issus de la production agricole. La pulpe de canne à sucre, aussi appelée la bagasse, est l'une des options les plus populaires. Ces matériaux possèdent les mêmes qualités physiques que le plastique, dont l'étanchéité et la résistance aux chocs. Cependant, étant entièrement issue de matières naturelles, la bagasse se décompose complètement après quelques mois dans la nature. De plus, la bagasse est un matériau très peu dispendieux à produire, car il provient de la pulpe restante après l'extraction du jus des cannes à sucre.   Parmi les emballages en bagasse, on retrouve principalement des contenants alimentaires et des ustensiles. De plus en plus de restaurateurs optent pour ce matériau pour diminuer l'empreinte écologique de leur menu pour emporter.   Pour les autres entreprises, il existe également des sacs fabriqués entièrement en matière décomposable et différents contenants de transport, comme des boîtes et des sacs étanches.   Avec l'augmentation des lois interdisant le plastique à usage unique, les entreprises du Québec doivent trouver rapidement des solutions pour s'adapter. La transition vers les emballages en cartons et en biomatériaux est déjà un bon pas en avant pour la protection de l'environnement.  Cet article est paru dans Journal le Guide (Cowansville, QC) (site web)
</article>
<article titre="À Carcassonne, Comatec soutient la restauration avec son packaging durable" date="2020 10 19" journal="L'Indépendant (site web)">
Le spécialiste du packaging durable destiné à la restauration, Comatec déploie des solutions pour rebondir et aider les professionnels de la restauration à passer le cap de la crise sanitaire.   « Aujourd'hui, 50 % de notre offre est composée d'emballages en plastique recyclable et réutilisable et 50 % de solutions alternatives compostables et biodégradables, réalisées à partir de carton, pulpe de canne, bagasse, paille bio, bois de peupliers... Nous affichons une croissance de plus de 35 % d'activité sur les gammes de packaging alternatifs auprès des restaurateurs ou des traiteurs », précise Tatiana Rumeau, directrice générale de Comatec qui emploie une trentaine de salariés.  Nombre d'entreprises du secteur sont impactées par la crise sanitaire eu égard aux contraintes sanitaires limitant les activités des professionnels de la restauration et de l'événementiel sans oublier l'annulation ou le report des salons professionnels. « L'impact de la crise est là. Les annonces officielles confirment le peu de visibilité à moyen terme. Il faudra deux à trois ans pour retrouver une vraie dynamique, souligne la dirigeante. Pour pallier l'annulation des salons, nous avons mis en place une stratégie d'accompagnement de nos clients à 360° avec beaucoup de proximité, d'écoute et de conseils. La Covid-19 a boosté la créativité et l'agilité de notre service clients ».    Diversification de la clientèle    La crise sanitaire a conduit à des prises de conscience. « La demande de produits écoresponsables s'est accélérée pendant cette période. Il y a eu également une prise de conscience pour le Made in France », indique Tatiana Rumeau.    Afin de passer le cap de la crise et rebondir, Comatec s'attache à diversifier sa clientèle en approchant les Maisons de retraite. « Les professionnels de la restauration doivent également diversifier leur clientèle », ajoute-t-elle. Le service clients de Comatec a d'ailleurs accompagné des restaurateurs qui se lançaient pour la première fois dans la Vente à Emporter. « Nous les avons aidés à conjuguer leurs besoins et leurs budgets avec les attentes des consommateurs. De cette expérience-là, est née la collection Kits To Go », annonce Tatiana Rumeau. Si les perspectives 2021 sont encore difficiles à appréhender, Comatec a renforcé son équipe dans le marketing digital et la communication pour pouvoir consolider et trouver de nouveaux leviers de croissance. Le groupe va recruter un chef de développement produits, pour renforcer son service recherche et développement et deux commerciaux pour la restauration commerciale et la restauration collective.   Cet article est paru dans L'Indépendant (site web)
</article>
<article titre="EXKi diminue le plastique de 83% dans ses restaurants" date="2020 10 14" journal="Néorestauration (site web)">
D'ici à la fin de l'année, tous les restaurants EXKi vont se parer de nouvelles couleurs grâce à des emballages d'une nouvelle génération. Des solutions sur mesure, plus durables et adaptées à chaque recette. Ces emballages permettent de répondre aux principaux impératifs en termes de conditionnement, de fraîcheur et de prise en main. Ce projet a pour vocation de diminuer notre empreinte écologique et de sublimer notre art culinaire « Ce projet a pour vocation de diminuer notre empreinte écologique et de sublimer notre art culinaire » explique Frédéric Rouvez, co-fondateur d'EXKi L'éco-conception des emballages est un chantier en cours chez EXKi depuis 2013. La chaîne belge de restaurants n'a pas attendu la fin du plastique à usage unique pour faire des efforts en la matière. Progressivement les emballages en papier, les contenants en carton, en bagasse ou les couverts en bois ont déjà envahis leurs restaurants. L'objectif cette année est de passer à la vitesse supérieure : moins 83% de plastique dans tous les EXKi. Les salades sont désormais présentées dans un bol en carton, habillé d'une couleur vive en fonction de son contenu. Une identité visuelle qui change complément l'offre en restaurant. Toutes les recettes sont visibles en un seul coup d'oeil. Le bol est en carton FSC, recyclable, avec un couvercle en plastique à 90% recyclé( pour aider à faire son choix). D'ici à la fin d'année, tous les gobelets en plastique vont disparaître pour passer au verre. La formule à emporter reste, les Fruit &amp;Tea et les green Smoothie seront servis dans un gobelet en carton (recyclable et recyclé aussi). Par la suite, tous les petits pots à confiture du petit déjeuner seront désormais en verre ainsi que les vinaigrettes Depuis plus d'un an, tous les sandwichs gourmands sont emballés dans un papier recyclé qui conserve parfaitement la fraîcheur mais qui est également doux au toucher et bien sûr doux pour l'environnement. Une nouvelle parure que seul EXKi a intégré dans tous ses restaurants.
</article>
<article titre="À plus de 70 ans, Coralis innove encore" date="2020 10 5" journal="Ouest-France  Rennes Est ; Rennes Sud ; Rennes">
Après une tentative infructueuse il y a quelques années, la laiterie, fondée en 1949 par une bonne quinzaine d’agriculteurs locaux pour proposer aux Rennais le lait des fermes alentour, se lance dans un nouveau produit : du lait bio en briques.  «  Nous avons dû surmonter l’idée que le bio ne pouvait pas être en brique  »,  explique Bruno Martel, producteur passionné de lait bio, à Bain-sur-Oust depuis plus de vingt ans, et président d’Agrial (groupe coopératif agricole et agroalimentaire), représentant plus de 300 agriculteurs bio en Bretagne.   Avec la pandémie, retour aux fondamentaux    «  Mais la pandémie est passée par là, générant une prise de conscience du public, avec une réappropriation des choses essentielles, du circuit court, des produits de consommation courante… Et pour la première fois, après dix ans de baisse, nous constatons une hausse de la consommation du lait.  »  Avec le groupe Tétra Pak, un nouveau conditionnement a été créé. La brique a été conçue dans un matériau biosourcé, 100 % recyclable, conditionné à la laiterie de Cesson, plus léger, composé de moins de plastique et de plus de matière végétale (+ de 80 %), en l’espèce, de la bagasse (1) de canne à sucre.   Le groupe de grande distribution Carrefour s’est associé à cette démarche pour étoffer son offre de produits locaux.   En exclusivité tout le mois d’octobre    «  Carrefour s’est fixé pour objectif de devenir le leader de la transition alimentaire pour tous  »,  explique Laurent Naveau, le directeur du magasin cessonnais du centre commercial de La Rigourdière.    «  Nous voulons offrir à tous les consommateurs une alimentation de qualité, saine et durable, en privilégiant les circuits courts, tout en étant un partenaire privilégié du monde agricole. Ce nouveau produit de la laiterie locale y répond parfaitement.  »  Le magasin cessonnais et ses homologues bretons vont commercialiser en exclusivité et pendant tout le mois d’octobre, le nouveau lait des éleveurs bio bretons de la laiterie Coralis.   (1) résidu fibreux de la canne à sucre, passée au moulin pour extraire le suc, et composée principalement de la cellulose de la plante.  Cet article est paru dans Ouest-France
</article>
<article titre="Près de Rennes, à plus de 70 ans, Coralis innove encore" date="2020 10 4" journal="Ouest-France (site web)">
Après une tentative infructueuse il y a quelques années, la laiterie, fondée en 1949 par une bonne quinzaine d’agriculteurs locaux pour proposer aux Rennais le lait des fermes alentour, se lance dans un nouveau produit : du lait Bio en briques.  «  Nous avons dû surmonter l’idée que le bio ne pouvait pas être en brique,  »   explique Bruno Martel, producteur passionné de lait bio, à Bain-Sur-Oust depuis plus de 20 ans, et président d’Agrial (groupe coopératif agricole et agroalimentaire), représentant plus de 300 agriculteurs bio en Bretagne.   Avec la pandémie, retour aux fondamentaux      «  Mais la pandémie est passée par là, générant une prise de conscience du public, avec une réappropriation des choses essentielles, du circuit court, des produits de consommation courante… Et pour la première fois, après 10 ans de baisse, nous constatons une hausse de la consommation du lait  »  .   Avec le groupe Tétra Pak, un nouveau conditionnement a été créé. La brique a été conçue dans un matériau biosourcé, 100 % recyclable, conditionné à la laiterie de Cesson, plus léger, composé de moins de plastique et de plus de matière végétale (+ de 80 %), en l’espèce, de la bagasse (1) de canne à sucre.   Le groupe de grande distribution Carrefour s’est associé à cette démarche pour étoffer son offre de produits locaux.   En exclusivité tout le mois d’octobre    «  Carrefour s’est fixé pour objectif de devenir le leader de la transition alimentaire pour tous,  »   explique Laurent Naveau, le directeur du magasin cessonnais du centre commercial de La Rigourdière.    «  Nous voulons offrir à tous les consommateurs une alimentation de qualité, saine et durable, en privilégiant les circuits courts, tout en étant un partenaire privilégié du monde agricole. Ce nouveau produit de la laiterie locale y répond parfaitement  »  .   Le magasin cessonnais et ses homologues bretons, vont commercialiser en exclusivité et pendant tout le mois d’octobre, le nouveau lait des éleveurs bio bretons de la laiterie Coralis.   (1) résidu fibreux de la canne à sucre, passée au moulin pour extraire le suc, et composée principalement de la cellulose de la plante.  Cet article est paru dans Ouest-France (site web)
</article>
<article titre="[Ils livrent des lunchs « made...]" date="2020 09 30" journal="Tribune de Genève, no. 6623">
Ils livrent des lunchs « made in Geneva »    Laurence Bézaguet    Écologiste de la première heure, le socialiste René Longet vantait dans notre édition de lundi les bienfaits d'une « indispensable nouvelle donne agroalimentaire » basée sur la qualité du produit, gustative et nutritionnelle. Mais aussi sur la qualité de vie et de l'environnement du producteur. Un nouveau paradigme bénéfique pour la planète et pour l'être humain dont s'inspire Treat SA, la petite dernière des start-up genevoises dans le domaine de la foodtech.   Dès octobre, on pourra ainsi, grâce à elle, se faire livrer dans des points-relais ou sur son lieu de travail de bons petits lunchs « made in Geneva » . Le choix des plats et la commande se feront via une application mobile sur le site  www.treat.ch.    « Notre approche a pour but de casser les codes de la cantine et de la restauration d'entreprise traditionnelle en proposant une gamme de plats ultralocaux (95% à 99% genevois, si l'on excepte des produits comme le soja ou certaines huiles), sains, variés, traçables » , expliquent Romain, 28 ans, et Jonathan, 31 ans, les deux créateurs de Treat SA.   À vélo-cargo    « Nous souhaitons favoriser les circuits courts et effectuerons la livraison de manière écologique à vélo-cargo électrique, ajoutent ces deux diplômés de l'École hôtelière de Lausanne. Nous avons pour ambition de pouvoir offrir à nos clients une cuisine saine, sans chichi, responsable, traçable et à des prix tout doux... »    Tout doux, vraiment? « Les entrées coûteront 6 francs, les plats 14 francs et les desserts 5 francs, informe Romain. Et puis on modifiera notre carte toutes les semaines. » Automnale de lentilles, gnocchis verts à la crème de saumon suisse, risotto d'épeautre aux champignons et pesto à la carte notamment pour les débuts... « On fera aussi la promotion du cardon cet automne et notre cuisine locale très variée aura parfois un parfum italien ou asiatique » , notent encore Romain et Jonathan.   Le chef Lukas se fournira, quant à lui, en fruits et légumes à l'Union maraîchère genevoise et à la Ferme du Fond de l'Étang, à Plan-les-Ouates; il ira chercher ses lentilles et pois chiches au Domaine de l'Abbaye de Presinge et sa viande chez la famille Graf, à Bernex.   Zéro déchet   Autre objectif important pour ce duo de jeunes créateurs: le zéro déchet. « Nous encourageons ainsi nos clients à passer commande jusqu'à minuit le jour précédant leur repas de midi, indique Romain. Anticiper sa commande n'est pas ce qu'il y a de plus pratique, mais c'est la seule façon de pouvoir produire que ce qui est vraiment nécessaire. » Un geste citoyen, comme le qualifie notre interlocuteur, qui pourrait en démotiver plus d'un. Mais les patrons de cette nouvelle start-up sont prêts à prendre ce risque tout en proposant un deal win-win à la clientèle: « Nous accorderons des rabais en fonction de l'heure de la commande. Plus on fait son choix rapidement, moins on paie! »    Quant aux « cartons » employés, à base de résidus de canne à sucre appelés bagasse, ils pourront être directement jetés dans le compost. Outre l'intérêt des particuliers, qui pourrait être stimulé par la mode du télétravail, Treat SA espère enfin pouvoir signer des partenariats avec des entreprises locales. « Nous nous sommes déjà entendus avec un horloger de la place et d'autres sociétés qui ont trouvé intéressant d' « outsourcer » ce service afin de limiter leurs coûts et de proposer une offre de qualité à leurs collaborateurs » , concluent Romain et Jonathan.   https://www.instagram.com/treat.ch/   Avec Treat SA, Jonathan (à gauche) et Romain, qui entourent Lukas, le cuisinier, veulent casser les codes de la cantine. Laurent guiraud   Ils livrent des lunchs « made in Geneva »    Laurence Bézaguet    Écologiste de la première heure, le socialiste René Longet vantait dans notre édition de lundi les bienfaits d'une « indispensable nouvelle donne agroalimentaire » basée sur la qualité du produit, gustative et nutritionnelle. Mais aussi sur la qualité de vie et de l'environnement du producteur. Un nouveau paradigme bénéfique pour la planète et pour l'être humain dont s'inspire Treat SA, la petite dernière des start-up genevoises dans le domaine de la foodtech.   Dès octobre, on pourra ainsi, grâce à elle, se faire livrer dans des points-relais ou sur son lieu de travail de bons petits lunchs « made in Geneva » . Le choix des plats et la commande se feront via une application mobile sur le site    www.treat.ch.    « Notre approche a pour but de casser les codes de la cantine et de la restauration d'entreprise traditionnelle en proposant une gamme de plats ultralocaux (95% à 99% genevois, si l'on excepte des produits comme le soja ou certaines huiles), sains, variés, traçables » , expliquent Romain, 28 ans, et Jonathan, 31 ans, les deux créateurs de Treat SA.   À vélo-cargo    « Nous souhaitons favoriser les circuits courts et effectuerons la livraison de manière écologique à vélo-cargo électrique, ajoutent ces deux diplômés de l'École hôtelière de Lausanne. Nous avons pour ambition de pouvoir offrir à nos clients une cuisine saine, sans chichi, responsable, traçable et à des prix tout doux... »    Tout doux, vraiment? « Les entrées coûteront 6 francs, les plats 14 francs et les desserts 5 francs, informe Romain. Et puis on modifiera notre carte toutes les semaines. » Automnale de lentilles, gnocchis verts à la crème de saumon suisse, risotto d'épeautre aux champignons et pesto à la carte notamment pour les débuts... « On fera aussi la promotion du cardon cet automne et notre cuisine locale très variée aura parfois un parfum italien ou asiatique » , notent encore Romain et Jonathan.   Le chef Lukas se fournira, quant à lui, en fruits et légumes à l'Union maraîchère genevoise et à la Ferme du Fond de l'Étang, à Plan-les-Ouates; il ira chercher ses lentilles et pois chiches au Domaine de l'Abbaye de Presinge et sa viande chez la famille Graf, à Bernex.   Zéro déchet   Autre objectif important pour ce duo de jeunes créateurs: le zéro déchet. « Nous encourageons ainsi nos clients à passer commande jusqu'à minuit le jour précédant leur repas de midi, indique Romain. Anticiper sa commande n'est pas ce qu'il y a de plus pratique, mais c'est la seule façon de pouvoir produire que ce qui est vraiment nécessaire. » Un geste citoyen, comme le qualifie notre interlocuteur, qui pourrait en démotiver plus d'un. Mais les patrons de cette nouvelle start-up sont prêts à prendre ce risque tout en proposant un deal win-win à la clientèle: « Nous accorderons des rabais en fonction de l'heure de la commande. Plus on fait son choix rapidement, moins on paie! »    Quant aux « cartons » employés, à base de résidus de canne à sucre appelés bagasse, ils pourront être directement jetés dans le compost. Outre l'intérêt des particuliers, qui pourrait être stimulé par la mode du télétravail, Treat SA espère enfin pouvoir signer des partenariats avec des entreprises locales. « Nous nous sommes déjà entendus avec un horloger de la place et d'autres sociétés qui ont trouvé intéressant d' « outsourcer » ce service afin de limiter leurs coûts et de proposer une offre de qualité à leurs collaborateurs » , concluent Romain et Jonathan.   https://www.instagram.com/treat.ch/   Avec Treat SA, Jonathan (à gauche) et Romain, qui entourent Lukas, le cuisinier, veulent casser les codes de la cantine. Laurent guiraud
</article>
<article titre="Zéro déchet : La consigne dans la restauration à emporter, une idée qui ne demande qu'à grandir" date="2020 09 29" journal="20 Minutes (site web)">
EMBALLAGE Plutôt que la poubelle, des restaurants et traiteurs proposent à leurs clients de rapporter les boites une fois le repas terminé. Elles sont lavées et remises dans le circuit par des starup qui se lancent sur ce marché. Comme Reconcil à Paris   L'idée est simple : payer, une fois, deux euros plus cher le plat à emporter que l'on va chercher dans le restaurant de son quartier. On rince la boîte et on la rapporte, soit pour l'échanger contre un nouveau repas, soit pour récupérer ses deux euros Des restaurants, des boulangeries-traiteurs et autres commerces spécialisés dans les repas à emporter se laissent peu à peu convaincre par ce système de contenants consignés et réutilisables, plutôt que les barquettes à jeter à la poubelle Derrière, des entreprises se créent pour laver et mettre à disposition ces boites auprès des restaurants. Comme Reconcil à Paris qui, après les restaurants, vise désormais de plus gros acteurs comme ceux de la cuisine digitale  De la pluie et un petit vent frisquet... Au Petit Lux, on vous dira que la météo prévue toute cette semaine dans la capitale est propice « à prendre un plat chaud le midi ». Et par là même de faire un petit geste pour la planète. Dans cette boulangerie-traiteur du 6e arrondissement, prisée des étudiants et travailleurs du quartier, tous les plats chauds - du « foie de veau-purée-maison » au « Saucisse de Toulouse-lentilles » - sont logés à la même enseigne et servis dans un contenant en plastique en polypropylène épais coiffé d'un couvercle bleu.   Autrement dit du solide qu'il serait bien dommage de jeter à la poubelle une fois le repas fini. D'une part, parce qu'il se lave très bien. D'autre part, parce qu'il vous en coûtera deux euros. « C'est le montant de la consigne, explique Corinne Carnet, la gérante. Le client la paie une fois, repart avec son repas, mange, rince la boîte et la rapporte un prochain midi. En échange, il récupère ses deux euros s'il ne reprend pas un plat chaud, soit une nouvelle boite, le cas contraire. »   « On pourrait laver jusqu'à 15.000 boîtes par jour »    Les boîtes à laver ne restent pas longtemps au Petit Lux. Deux fois par semaine, Reconcil, à qui elles appartiennent, passe les récupérer. La tournée ne passe pas que par le Petit Lux, mais par la vingtaine d'autres restaurants de Paris où l'entreprise de Sofiane Hassaïne-Teston met à disposition ses contenants.   Direction ensuite rue d'Aubervilliers, tout au nord de la capitale. C'est là que Sofiane Hassaïne-Teston a installé les bureaux de Reconcil, en grande partie occupés par un tunnel de lavage, une sorte de lave-vaisselle géant. « Les boîtes y sont prélavées, puis lavées à 64°C et rincées à plus forte température encore (85°C), ce qui ne laisse aucune chance à de quelconque bactérie », raconte l'entrepreneur. Le tout en trente secondes chrono. » Sofiane Hassaïne-Teston et, son associé, ont fait le calcul : « on pourrait laver jusqu'à 15.000 boîtes par jour, disent-ils. Et chaque contenant peut être lavé et réutilisé trente à quarante fois avant d'être retiré du circuit. »   C'est le coeur de la démarche de Reconcil : « proposer à des restaurants de sortir des emballages jetables pour préférer la logique réemploi », « de la même façon qu'on ne déchire pas ses vêtements en rentrant chez soi le soir ou qu'on ne casse pas non plus son assiette et son verre une fois le repas fini, illustre Sofiane Hassaïne-Teston.    Sofiane Hassaïne-Teston au premier plan, fondateur de Reconcil, entouré de Nicolas Djamdjian, son associé (à droite) et de Micha Mendy, premier employé de l 'entreprise. - F.Pouliquen/20 Minutes 160 millions de repas livrés en 2018    L'enjeu est de taille. La livraison de plats en France est en plein essor avec 160 millions de repas livrés en 2018 et un marché en croissance de 20 % en 2018, selon les chiffres du cabinet d'études The NPD Group. Forcément, derrière, il y a des déchets d'emballages. Le ministère de la Transition écologique avance le chiffre de 180.000 tonnes par an pour le secteur de la restauration rapide. « Ces emballages ne sont pas tous recyclables ou, quand ils le sont, ne sont pas toujours déposés dans la bonne poubelle pour être effectivement recyclés », glisse Alice Abbat, chargée de mission consigne à Zero Waste France, association qui oeuvre pour l'avènement d'une société zéro déchet et zéro gaspillage. « Et même lorsqu'ils sont effectivement recyclés, c'est pour un tout autre usage, reprend-elle. On n'est donc pas dans l'optique de créer des boîtes conçues pour durer, être lavées et remises plusieurs fois dans le circuit. »   Rien de mieux donc, pour Zéro Waste, que le réemploi et les systèmes de contenants consignés et réutilisables que cherche à faire adopter Reconcil. Pas que Reconcil, précise d'ailleurs Alice Abbat. « Noww (No Waste in the world), Milubo, La Consigne Greengo ont une démarche similaire en Ile-de-France. Et il faut ajouter encore « En boîte le plat » à Toulouse, « Ramène ton plat » à Lille, « Boxeaty » à Bordeaux, « LoopEat » à Montpellier, « Dabba » à Grenoble. »   Une dynamique a relancé après le confinement    Ça ne reste encore que des gouttes d'eau sur l'ensemble des repas pris à emporter ou livrer en France. Sofiane Hassaïne-Teston le dit sans détour : son tunnel de lavage ne marche pas à plein régime. « Le meilleur mois a été février avec 4.000 boîtes louées sur les différents restaurants de notre réseau », glisse-t-il. Le confinement a stoppé net la dynamique qui ne reprend que petit à petit ces dernières semaines. « Les gens sont encore beaucoup en télétravail », souffle l'entrepreneur.   Au Petit Lux, les boîtes consignées ont ainsi fait leur retour en vitrine mi-septembre. Corinne Carnet garde tout de même, en réserve, un lot de barquettes jetables pour les gens de passage. « Ou, plus rare, des clients réfractaires, souvent d'ailleurs parce qu'ils ne pensent pas être suffisamment disciplinés pour penser à ramener la boîte, confie la commerçante. Nous avions sondé nos clients en janvier dernier, avant de faire appel à Reconcil, pour avoir leurs avis sur les boîtes consignées. Nous avions récolté 90 % de oui. Cela nous a même ramené de nouveaux clients, sensibles à la démarche zéro déchet. » Un constat que dresse aussi Gloria Avila, dont « La Cantina de Gloria », petit traiteur récemment ouvert dans le 11e et qui a très vite adopté les boîtes consignées de Reconcil. C'est très bien accepté dans le quartier, en particulier des jeunes, dont certains avaient déjà. Et puis, c'est une façon de se démarquer, d'engager la conversation avec les clients... »   Toucher aussi de plus gros acteurs    Il reste encore à Sofiane Hassaïne-Teston à étoffer son réseau de restaurateurs. « L'idée serait d'arriver à 100 d'ici à six mois, annonce-t-il. Pour se faire, l'entrepreneur n'entend pas seulement frapper à la porte des restaurants et petits commerces de bouche de la capitale. Il vise aussi plus gros avec les acteurs de la cuisine digitale, spécialisée dans la livraison des plats cuisinés à domicile ou au travail. Les Frichti, Foodcheri, Nestor...   Le contexte s'y prête en tout cas. Début juillet, Brune Poirson, à l'époque Secrétaire d'État à la transition écologique, avait réuni ces grandes plateformes de restauration à emporter pour leur demander comment ils comptent parvenir au zéro déchet. A Frichti, on assure s'être emparé du sujet bien avant la réunion, « notamment en instaurant une nouvelle gamme de packaging plus responsable il y a un an et demi, précise Julia Bijaoui, la cofondatrice. Les contenants recevant les repas sont désormais en kraft ou en bagasse [chute de canne à sucre], à chaque fois biodégradables. »   En février, Frichti s'est aussi lancé dans la restauration collective avec Cantine 2.0 qui permet à des salariés d'une même entreprise de grouper leurs commandes. Une nouvelle offre qui pourrait lui permettre d'aller plus loin dans une démarche éco-responsable. « C'est plus facile en tout cas d'envisager la collecte de nos emballages sur les sites qui regroupent plusieurs commandes », note Julia Bijaoui. Des discussions et tests sont en cours. Sur le compostage de nos emballages avec des acteurs comme Les Alchimistes. Mais aussi pourquoi pas sur la consigne pour réemploi avec Reconcil. »  Cet article est paru dans 20 Minutes (site web)
</article>
<article titre="HP raccourcit les délais de fabrication des moules pour la cellulose" date="2020 09 23" journal="Emballages magazine (site web)">
L'entreprise américaine combine impression 3D et bioplastiques pour obtenir des outillages en deux semaines contre quatre à six semaines pour les procédés traditionnels.- La cellulose moulée et autres matériaux à base de fibre ont la cote dans l'emballage où ils représentent une alternative environnementale aux plastiques dans la fabrication de plateaux thermoformés, coques, calages et autres boîtes à oeufs. HP, que l'on connaît mieux pour ses imprimantes et ses ordinateurs, propose aussi depuis quelques années des solutions industrielles dans le domaine de l'impression 3D. L'entreprise américaine annonce la mise au point d'une technologie de fabrication additive pour réaliser les outillages qui sont ensuite utilisés sur les machines de thermoformage servant à réaliser les emballages en fibre moulée. Principal avantage : la rapidité d'exécution, soit deux semaines contre quatre à six semaines pour les méthodes traditionnelles qui comportent l'emploi de machines d'usinage à enlèvement de copeaux puis toutes les phases de rectification, finition et polissage. S'agissant d'une solution numérique, la possibilité de décliner à l'envi les outillages sans coûts additionnels dans le cadre, par exemple, d'opérations de personnalisation de masse de ces mêmes emballages demeure un autre atout de la proposition HP. Sans oublier le poids inférieur des moules et la maintenance moindre des équipements qui servent à les produire par rapport aux machines d'usinage traditionnelles. Huile de ricin HP utilise un bioplastique à base d'huile de ricin pour produire ses outillages. Souple mais résistant, ce matériau d'origine renouvelable conférerait aux moules des propriétés mécaniques optimales et des performances constantes dans le temps. Les surplus de matière peuvent, en outre, être recyclés jusqu'à hauteur de 70%. Afin de mettre au point sa solution, HP a établi un partenariat avec la société Pulp Moulding Dies (PMD), basée à Ajax, en Ontario (Canada), spécialisée justement dans l'outillage. Plusieurs producteurs d'emballages en fibre moulée américains sont aujourd'hui équipés des moules ainsi fabriqués parmi lesquels Fiber Innovation, Pacific Pulp, Pulp Molded Products et Western Pulp Products. La fibre moulée est généralement fabriquée à partir de carton recyclé ou de papier journal, mais peut également être issue de fibres de cellulose végétale, notamment de la bagasse, du bambou et de la paille de blé. Les emballages ainsi fabriqués sont utilisés dans de nombreux domaines d'activité, de l'alimentaire aux boissons en passant par l'électronique grand public, le médical et l'industrie.
</article>
<article titre="Port Réunion, relativement épargné" date="2020 09 1" journal="Le Journal de la Marine Marchande, no. 5110">
Avec un trafic de 5,92 Mt en 2019, en hausse de 12 %, voire voire de 6,3 Mt avec les transports de matériaux destinés à la nouvelle route du littoral maiscomptabilisés à part car non récurrents, le Grand port maritime de la Réunion voyait la vie en rose. Le conteneur en transbordement a vu son volume augmenter de 33 % par rapport à 2018 pour atteindre 111 075 EVP. Dans son ensemble, le segment du conteneur (375 074 EVP) a été en progression de 11 %. Les vracs liquides (932 408 t) a certes perdu 2 % de volume, mais avec un grand contraste entre la chute des importations de gazole (395 144 t, - 6 %) et la croissance de celles de fioul lourd pour la centrale électrique (202 228 kt, + 28 %) due à la maintenance de la centrale à bagasse d'Albioma et à la faible production hydroélectrique.   En revanche, quelques trafics étaient à la peine. Avec la perte de 35 315 véhicules, le roulier avait reculé de 8 %. Les exportations de sucre progressent de 8 % mais restent, avec 87 000 t, à un niveau bas. La diminution de 9 % des importations de charbon (548 872 kt) s'explique par l'arrêt technique de la centrale de Bois-Rouge, en cours de transition vers l'utilisation de biomasse.   Parmi les trafics en stabilité, les imports agroalimentaires (229 285 t, + 1 %) hésitent entre augmentation des oléagineux et baisse des céréales.   Des opportunités pour le transbordement «  Au début de l'année, la tendance était identique à 2019 et nous avions de bonnes perspectives pour 2020  », rappelle Éric Legrigeois, directeur général de Port Réunion. «  À partir de mars, nous avons géré toutes les escales, mais les lignes ont été impactées car les ports fermaient les uns après les autres. Maurice, en particulier, s'est arrêté brutalement, puis a imposé une quatorzaine.  » Résultat: au premier semestre, le trafic chute de 14 % et n'atteint que 2,4 Mt.   Seuls les vracs solides progressent (618 195 t, + 13 %), mais uniquement grâce à l'augmentation de 27 % des importations de charbon (385 518 t). Les vracs liquides (322 498 t, - 29 %) sont tous impactés, particulièrement les importations de kérosène (57 748 t, - 35 %). Les conteneurs (162 422 EVP) sont en repli de 14 %, le transbordement étant le plus touché du fait des fermetures de lignes dans l'océan Indien.   Quelques points positifs cependant: le service Nemo, opéré conjointement par CMA-CGM et MSC entre l'Europe du Nord et l'Australie, va cesser de desservir Maurice, augmentant le transbordement de la Réunion non seulement vers Maurice, mais aussi vers Madagascar et l'Afrique de l'Est. CMA-CGM va aussi remettre en place le service Midas, opéré avec Maersk entre l'Afrique du Sud et l'Inde.   Port Réunion profite aussi du fait que l'île est relativement épargnée par le Covid pour servir aux relèves d'équipage: cela a concerné 7 000 marins de mars à septembre, soit dix fois plus que d'habitude. «  Nous avons vu de grands méthaniers et pétroliers au large du port ou des thoniers venus y faire des interventions techniques, ce qui est inhabituel  », note Éric Legrigeois. «  Cela ne va pas durer, mais nous espérons que les armateurs ont découvert la qualité de nos services et que certains reviendront.  » Plus anecdotique, pour la première fois, deux navires câbliers sont venus charger à la Réunion, alors que ces opérations se font habituellement en Afrique du Sud.   En ce qui concerne les croisières, l'île, où la saison s'étend habituellement d'octobre à avril, avait dépassé pour la première fois les 100 000 passagers en 2019. Fin mars 2020, Port Réunion avait accueilli 66 700 passagers, contre 73 000 fin mars 2019.
</article>
<article titre="Le zéro déchet et zéro plastique toujours en route" date="2020 09 1" journal="Néorestauration, no. 588HS">
«N ous étions dans une dynamique assez engagée sur ce sujet avec les collectivités locales,  estime Annie Pinquier, directrice associée d'Agriate Conseil.  La Covid-19 a mis les choses entre parenthèses, mais le sujet n'est pas fermé. Nous parlons d'inox, de gourdes et de matériaux réutilisables, et la Convention citoyenne pout le climat a planché sur ces questions de durabilité. Mais il ne faut pas se tromper. Il y a le sujet du plastique qui répond à un choix, et celui de l'usage unique qui répond à la question sanitaire. Or, il existe aussi des matériaux à usage unique plus propres. »  Freinée, la dynamique est loin d'être à l'arrêt, même si des incertitudes persistent.  « Les hôpitaux de Bretagne ont mis en place des contenants compostables, mais des problèmes demeurent dans les solutions techniques actuelles, car nous devons assurer un maintien en atmosphère humide et une remise en température,  précise Marie-Cécile Rollin, directrice de Restau'co.  Le retour au verre ou à l'inox pose des questions sur les ports de charges pour les personnels et doit s'accompagner d'une réflexion sur ce qui est réalisé en cuisine centrale et sur les sites satellites. Nous avons besoin d'une clarification rapide des termes utilisés sur le marché, et d'un travail sur les alternatives et les filières de recyclage. Le ministère nous donne des interdictions, mais pas l'accompagnement nécessaire. »  Des interrogations auxquelles Barbara Pompili, la nouvelle ministre de la Transition écologique et solidaire, devra répondre.  ACCÉLÉRER LES ACTIONS EN COURS « De façon ponctuelle, nous avons dû remettre des bols et des couverts en plastique,  souligne Aurélie Stewart, responsable RSE d'Elior.  La chaîne du packaging durable a été stoppée pendant la Covid-19, mais les sites précurseurs accélèrent leur demande. Cette crise nous pousse à explorer de nouvelles options, comme le fait que les convives apportent leurs couverts. Nous avons aussi une solution de consigne en test depuis décembre sur un siège social en région parisienne de 1 500 couverts par jour. Nous affinons la réponse aux besoins et allons intégrer la consigne dans de plus en plus d'offres. Notre objectif est de proposer des solutions alternatives. Notre combat sur le plastique à usage unique se construit avec le client qui réalise sa propre veille. Il n'y a pas une solution, mais un maillage de solutions sur le territoire qui tiennent compte de l'écosystème dans son ensemble. »  La démarche est également bien avancée chez Compass Group France. Sa marque Exalt, dédiée aux entreprises et aux sièges sociaux implantés à Paris et en région parisienne, a monté un partenariat avec la start-up de l'économie sociale et solidaire MyGreenGo.  « Elle remet la consigne au goût du jour et incite le consommateur à ramener son contenant consigné vide dans l'automate afin de faciliter sa réutilisation,  précise Corine Krief, directrice communication et RSE de Compass Group France.  Cette solution se déploie de manière considérable chez nos clients engagés. En restauration scolaire, nous mettons des gourdes réutilisables en inox avec un bouchon à vis à disposition ou à la vente. Et nous réfléchissons à des barquettes pour les repas livrés en cuisine centrale. Le retour au bac inox implique de revoir toute l'organisation et a un coût. Tout dépendra du choix du client. » Chez Daily Pic, concept de restauration rapide haut de gamme créé par la cheffe étoilée Anne-Sophie Pic, la crise n'a pas modifié le principe initial : utiliser des verrines en verre réutilisables.  « Nous avons une vision à 360 degrés du zéro déchet, qu'il soit alimentaire sur les matières premières, les produits finis ou les emballages,  explique Lillian Étienne, directeur Business unit de Daily Pic.  Il a fallu monter toute la supply chain. »  Ainsi, le lavage est assuré par Uzaje, à Paris, et par Ma Bouteille s'appelle revient, qui vient d'ouvrir sa propre laverie à Valence.    Consigne, contenants en bagasse de canne à sucre ou algues, en pulpe de cellulose, en PLA, c'est-à-dire en acide polylactique (polymère biodégradable, mais uniquement en compostage industriel), carton seul ou filmé avec un bioplastique, couverts en bois certifié PEFC/FCS... Le marché a encore besoin de recherche et développement et d'innovation pour préciser les solutions afin de pouvoir répondre aux usages, aux portions et aux volumes nécessaires pour le marché de la restauration hors domicile.    RECHERCHE ET VISION GLOBALE Pour Isabelle Lacourt, spécialiste du développement durable dans la filière alimentaire au sein de Risteco pendant quinze ans et cofondatrice de la plate-forme Eating City, il n'y a pas de dogme en la matière.  « Il faut faire les choses avec sens et de manière complète .  Il est nécessaire d'être plus rigoureux dans l'approche du cycle de vie d'un produit. Si nous jetons le plastique, c'est parce qu'il n'est pas cher. La vaisselle à usage unique simplifie le travail, et sur des événements culturels ou sportifs, celui des bénévoles. Passer de la vaisselle jetable à la vaisselle compostable permet de réduire drastiquement les déchets, mais si elle n'entre pas dans des filières établies, alors elle devient un nouveau déchet. C'est pourquoi il faut créer un lien entre les points de production de déchets et les plates-formes qui peuvent assurer localement un compost de qualité qui va retourner au sol. »  Isabelle Lacourt a lancé Lich'N, un service clé en mains qui intègre de la vaisselle compostable et valorise 100 % des déchets de la restauration événementielle grâce à des partenariats avec des plates-formes de compostage.  « Je ne crois pas à la suppression totale de l'usage unique, c'est une question complexe où l'on cible les restaurateurs,  affirme Corine Eon, cofondatrice avec son mari de Dubble.  Mais un mouvement sociétal est en cours. »  Dubble a remplacé ses contenants en plastique par des bowls en papier de bambou et a lancé la Dubble Jar, une salade dans une jarre en verre avec son couvercle à vis.  « Nous ne pouvons pas réaliser la désinfection et le lavage spécifiques derrière pour la réutiliser et cela aurait un coût important »,  poursuit-elle. Les réflexions en cours vont progresser.  « Les contenants réutilisables demandent plus d'organisation, mais sur le long terme, les restaurateurs envoient un message positif »,  estime Julie Sauvêtre, chargée de projet des acteurs de l'événementiel et du tourisme chez Zéro Waste. Une approche que confirme Emmanuel Auberger, président fondateur d'Uzaje, société proposant des solutions industrielles pour la réutilisation des contenants alimentaires.  « Le plastique à usage unique sauve la vie dans des hôpitaux, mais il n'est pas justifié pour les produits agroalimentaires. »  Les choses vont se mettre en place comme pour le gaspillage alimentaire, qui a fait émerger de nombreuses solutions et start-up.  « La crise a fait naître des pratiques positives, c'est le moment d'insister sur ces changements et de les pérenniser,  estime Lucie Basch, fondatrice de Too Good To Go.  On parle beaucoup d'un plan de relance, je souhaite que l'on parle d'un plan de transition. »  Faire converger l'économie et l'environnement, une équation qui divise depuis longtemps. Mais l'urgence est là, les professionnels de la RHD, poussés par les consommateurs et la réglementation, doivent y répondre.
</article>
<article titre="Après les jus "5 fruits et légumes", voici les micropousses namuroises au supermarché" date="2020 08 24" journal="DH (site web)">
Brieuc et Lucas ont dû interrompre la production de jus Good Move, mais ont lancé un nouveau produit santé.   Good Move est le projet de Lucas Vandierendonck et Brieuc Debois. Depuis deux ans, ils ont transformé la fabrication de jus de fruits et légumes dans leur kot en une véritable entreprise, notamment grâce à l'incubateur Linkube, à un financement participatif et à l'accompagnement de Ludovic Vanackere et des Artisans de Bossimé. La véritable nouveauté consistait à remplacer la pasteurisation, qui fait perdre aux aliments une bonne partie de leurs bienfaits par la pascalisation et la pression à froid. Cette technique permet de protéger le jus tout en conservant les vitamines naturellement présentes dans les fruits et légumes utilisés. Une initiative unique en Wallonie.   En raison de la crise du covid-19, Good Move a dû suspendre la production de ses jus 5 fruits et légumes jusqu'à l'automne minimum. Mais ce temps a permis à Lucas et Brieuc de produire et commercialiser un autre aliment santé: les micropousses, comme celles que l'on sert dans les restaurants gastronomiques. Ils les ont rendues disponibles pour tous dans les supermarchés. Après une expérience concluante chez D'Ici (Naninne), on les trouve aujourd'hui dans trois AD Delhaize de Namur (Jambes, Salzinnes et Belgrade) ainsi qu'au Carrefour de Floreffe.   Ces micropousses sont cultivées localement, très riches en nutriments, garanties zéro pesticide et proposées à la vente dans dans une boite 100% biodégradable fabriquée à partir de bagasse de canne à sucre. Pour l'instant, deux types de micropousses sont disponibles: radis et moutarde.   "Depuis un moment, Lucas s'est intéressé aux fermes verticales en environnement contrôlé et à l'hydroponie. Il a constaté qu'une exploitation basée sur ces systèmes et bien optimisée permettait de réduire très fortement la consommation d'eau et les surfaces de culture nécessaires, tout en bannissant l'utilisation de pesticides. A l'heure actuelle, un certain nombre de plantes peuvent être produites dans ces environnements contrôlés. Les micropousses en font parties", explique Brieuc qui tient à faire la distinction avec les graines germées qui ne présentent pas les mêmes qualités nutritionnelles   Quelles sont-elles? "Leur densité en nutriments est jusqu'à 40 fois supérieure à celle des plantes adultes. Ils sont qualifiés de super aliments. Quant à leur goût, il est déjà quasi identique à celui de la plante adulte, ce qui peut causer l'étonnement quand on en goûte pour la première fois... Comme les micropousses de moutarde qui explosent sur le palais."  Cet article est paru dans DH (site web)
</article>
<article titre="Comment les marques s’adaptent à la fin du plastique à usage unique" date="2020 08 22" journal="Metro (Bruxelles) (site web)">
L’UE s’est lancée dans la chasse au plastique à usage unique. Dès le 1er janvier 2021, les gobelets, les pailles, les couverts ou encore les touillettes seront interdits à la vente.   Et ce n’est que le début. Les marques sont obligées de s’adapter et, nous allons le voir, elles rivalisent parfois d’ingéniosité! Les plats en bagasse d’EXKi   La chaîne belge de restaurants EXKi n’a pas attendu la fin du plastique à usage unique pour faire des efforts en la matière. Dès 2013, elle a testé les pailles en carton, une initiative qui à l’époque n’avait pas été approuvée par la clientèle. Depuis l’an passé, les sandwichs vendus ne sont plus emballés dans du plastique mais dans du papier. Les entrées froides et certaines salades sont désormais vendues dans des contenants en bagasse, un résidu naturel de pulpe de canne à sucre, 100% naturel, recyclable et compostable.   Des pailles en pâte de Panzani   « Des pailles, des pailles, oui mais des Panzani », voici peut-être le prochain slogan de la marque française. En avril 2019, Panzani a lancé sur le marché la Panza Paille, une paille en pâte. Fabriquée à partir de blé et d’eau, elle est entièrement biodégradable et compostable. Son coût est modeste (moins de 4 centimes l’unité) et elle est capable de tenir 1h dans les boissons fraîches ou ambiantes avant d’être un peu moins efficace! Mais ne la cherchez pas dans les magasins, elle est pour l’instant réservée aux professionnels.   Des gobelets en biscuit   Les gobelets en plastique à usage unique sont voués à disparaître et les restaurateurs vont bien devoir trouver des alternatives. Et pourquoi pas, dans certains cas, se tourner vers Tassiopée. Cette marque française a mis au point des petites tasses à café fabriquées en biscuit nappé de chocolat noir. Une fois son café terminé, le consommateur peut ainsi manger le contenant! Disponibles pour les particuliers dans les magasins Nature et Découverte en France, les tasses à croquer n’attendent qu’à être adoptées dans les professionnels de la restauration.   Candia éco-conçue   Au-delà du plastique à usage unique, des marques s’engagent à réduire l’utilisation des ressources fossiles dans leurs emballages. C’est le cas de la marque Candia qui a lancé l’été dernier sur le marché français une brique de lait sans aluminium et sans suremballage plastique. Cette brique « éco-conçue » est composée à 75% de carton issu de forêts gérées durablement. La couche d’aluminium qui servait de barrière a été supprimée. Le plastique n’a cependant pas encore pu être banni totalement, notamment pour le bouchon et les couches d’étanchéité à l’intérieur de la brique.   Bientôt un yaourt en vrac?   Les petits pots de yaourt en plastique seront-ils eux-aussi bientôt bannis des supermarchés? Si la mesure n’est pas encore d’actualité, certaines marques commencent déjà à envisager un futur sans plastique. En juin dernier, le géant Danone a installé dans un magasin parisien proposant des aliments en vrac un distributeur de yaourts bio. Les consommateurs peuvent ainsi se servir eux-mêmes, soit directement dans leur propre contenant, soit dans un carton alimentaire recyclable proposé dans le distributeur. Pour l’instant, l’innovation est encore en phase de test mais elle pourrait bien se généraliser dans un futur proche.  Cet article est paru dans Metro (Bruxelles) (site web)
</article>
<article titre="Comment Marseille s’est refait une beauté" date="2020 08 15" journal="Le Figaro (site web)">
Cet article est extrait du Figaro Hors-Série «Provence éternelle, la Toscane française». Découvrez l’histoire plurimillénaire de la Provence, ses villes romaines, ses villages perchés, ses abbayes, ses châteaux, ses vignobles, ses traditions vivantes. Si d’aventure vous croisez le recteur de Notre-Dame-de-la-Garde, le père Spinosa, peut-être vous autorisera-t-il à monter dans le clocher… jusqu’à l’intérieur de la statue de cette Vierge que les Marseillais appellent «Bonne Mère»! En regardant à travers ses yeux comme on regarde à travers un masque, on aperçoit les bateaux du Vieux-Port, autour duquel Marseille s’est développée. Mais il faudra redescendre un peu pour contempler l’étendue de la ville construite sur des collines comme un amphithéâtre face à la mer, et dont la façade maritime s’étend sur cinquante-sept kilomètres, des Calanques au port de l’Estaque, peint par Cezanne, Braque, Derain ou Dufy.   » LIRE AUSSI - Le Luberon, terre authentique ou repère de bobos?  Marseille a souffert de l’immensité de son territoire - d’importants quartiers avaient été laissés à l’abandon dans la seconde moitié du XXe siècle: aujourd’hui encore, la ville est connue pour l’insécurité qui règne dans certains quartiers, sa multidélinquance, la montée de l’islamisme, liées à une immigration incontrôlée. Mais, capitale européenne de la culture en 2013, la Cité phocéenne a décidé de se composer un nouveau visage pour reconquérir certains quartiers perdus et entrer dans le XXIe siècle. Le Vieux-Port qui éblouit Tartarin de Tarascon et où s’aimèrent avec un accent chantant les personnages de Pagnol s’ouvre ainsi désormais sur le Mucem, premier grand musée national consacré aux civilisations de l’Europe et de la Méditerranée: il est le phare de cette capitale qui se réinvente.   Pourtant, Marseille a su garder son âme. Au large, lorsqu’on aperçoit le château d’If, Edmond Dantès n’en finit pas de s’évader sous nos yeux… ou dans nos songes? Les songes de cette ville où mourut Arthur Rimbaud et où Fernandel chanta le mistral ne cessent de s’épancher dans les monuments, les rues, les reflets du soleil sur la Méditerranée. Même les pierres y semblent de chair. D’ailleurs, édifiée entre 1853 et 1864 par l’architecte marseillais Henri Espérandieu, Notre-Dame-de-la-Garde en témoigne: n’est-elle pas cette «Bonne Mère», vivante et présente, à qui les marins offrent des maquettes de bateaux pour la remercier de sa protection? Nul ne passe à Marseille sans monter la prier ou l’admirer. Au-dessus de l’autel, resplendissant d’autant plus que cette basilique de style romano-byzantin fut restaurée entre 2001 et 2008, une mosaïque où un bateau vogue sur une mer houleuse, apaisée par la présence de l’étoile de Marie, exprime cette confiance.    Porte de l’Orient depuis l’Antiquité   Elle évoque, aussi, cet Orient dont Marseille est la porte depuis l’ Antiquité. On sent en effet dans la Cité phocéenne  «je ne sais quoi d’oriental, on y marche à l’aise, on respire content, la peau se dilate et hume le soleil comme un grand bain de lumière»,  comme l’écrit Flaubert en 1840, quelques années avant l’édification de la basilique. On se retourne et, par les portes ouvertes, voici justement cette mer - la mer, la mer, toujours recommencée!   C’est par elle que sont arrivés les Phocéens vers 600 avant J.-C. La légende raconte qu’ils fondèrent la ville par le mariage merveilleux d’un jeune Grec aventurier de leur cité avec une belle princesse celte, Gyptis.  «Et l’archéologie semble attester la légende! En tout cas, les archéologues ont retrouvé des maisons grecques du VIe siècle avant J.-C., avec un mobilier gaulois»,  s’amuse Fabrice Denise, directeur du musée d’Histoire de Marseille, qui retrace l’évolution de la ville de l’Antiquité à nos jours.   Ce lointain passé, les pierres de Marseille l’avaient quelque peu oublié. Il a surgi de terre au cours du XXe siècle, pour prendre aujourd’hui sa place dans la ville. A l’occasion de la construction d’une cité administrative et d’un centre commercial dans les années 1960, les vestiges du port antique ont été exhumés. Le musée d’Histoire de Marseille a été bâti autour du site. Après un important chantier pour requalifier le site archéologique, achevé fin 2019, on y entre désormais par la voie romaine et on descend par un escalier monumental, porte de la cité. Cette dernière semble renaître sous nos yeux: ici, ce sont les vestiges des remparts qui furent pris d’assaut par Jules César en 49 avant J.-C. ; là, les quais du port antique! Et des maquettes de bateaux reconstitués à partir des épaves grecques exposées à l’intérieur du musée semblent sur le point de débarquer des trésors venus de toutes les rives de Méditerranée.   » LIRE AUSSI - À 100 km de Marseille, 6 pépites pour s’évader en Méditerranée provençale  Au fur et à mesure que le port antique s’est envasé au fil de l’histoire, des édifices sont sortis de terre, et il faut marcher quelques minutes pour atteindre le Vieux-Port actuel. L’âme de Pagnol y plane encore. Les femmes des pêcheurs y vendent des poissons qui seront sans doute cuisinés en bouillabaisse - car  «tout le monde croit que les Marseillais (…) se nourrissent de bouillabaisse et d’aïoli, en disant “bagasse” toute la journée»,  dit l’écrivain: ne dérogeons pas à cette croyance. On se balade. On boit aux terrasses entre amis ou en famille. Et voilà que des rires, des éclats de voix viennent frapper nos oreilles. Serait-ce Marius, César ou Fanny, sortis tout droit de la trilogie de Marcel Pagnol? Non. Pourtant, leur bar, celui de la Marine, est bien là - qui restitue le décor de Pagnol. Comme si le temps s’était arrêté.   Toutefois, remontant la Canebière, cette avenue qui va du Vieux-Port à l’église des Réformés, qu’arpentait le jeune Pagnol et où Charles Trénet fit ses débuts dans les années 1930, on tourne quelques rues et voilà qu’on se retrouve au cœur du quartier populaire de Noailles. Dans la rue Longue-des-Capucins, flotte un parfum d’épices, et les langues qu’on entend sont celles du nord de l’Afrique. En plein jour, ce tableau qui nous transporte dans un souk de l’autre rive de la Méditerranée peut apparaître pittoresque. Mais on se rappelle aussi l’effondrement de deux immeubles vétustes rue d’Aubagne, à dix minutes de là, en 2018, qui témoigne de la face obscure d’une Marseille paupérisée, qui a gagné quelque trois cent mille habitants entre 1945 et 1975…   Aujourd’hui, au Vieux-Port, se croisent non plus seulement Marius et César, mais aussi des populations immigrées d’Afrique du Nord ou de l’Ouest ou des Comores. Les quais ont d’ailleurs été élargis pour laisser les piétons venant de la Canebière déambuler, et une ombrière géante en inox poli, signée Norman Foster, prodigue de la fraîcheur et amuse les enfants qui lèvent les yeux et jouent avec leurs reflets!   L’air de rien, Marseille s’est refait une beauté. Le majestueux palais Longchamp en témoigne, qui a retrouvé sa blancheur éclatante. Son architecte? Le même que celui de Notre-Dame-de-la-Garde: le Marseillais Henri Espérandieu. Inauguré en 1869 pour célébrer l’arrivée de l’eau douce, et plus particulièrement celle de la Durance à Marseille grâce à d’importants travaux d’ingénierie, ce monument grandiose vient d’être rénové en profondeur: ses bassins, fontaines et sculptures ont été nettoyés et restaurés, comme l’escalier d’honneur et le péristyle. Quant aux verrières, elles ont été remises en lumière et diffusent une douce clarté pour nous donner d’admirer les tableaux du musée des Beaux-Arts qu’il abrite. De part et d’autre de son grand escalier, deux peintures monumentales de Puvis de Chavanne,  Marseille, colonie grecque  et  Marseille, porte de l’Orient,  continuent de se répondre pour célébrer la grandeur de cette capitale éternelle de la Méditerranée.   Naissance d’une capitale   Eternelle? Au XIXe siècle, le trafic maritime devient trop intense pour les capacités du Vieux-Port: on construit plus au nord le grand port de la Joliette. Après les Trente Glorieuses et la décolonisation, les quais et entrepôts, désormais surmontés par une autoroute urbaine, deviennent un quartier en déshérence. La prospérité portuaire de Marseille semble alors révolue. La Cité phocéenne apparaît comme une capitale de la mafia corso-italienne, puis comme le territoire d’une importante immigration afro-maghrébine, en même temps que le fief des trafics de drogue, des règlements de comptes entre bandes rivales, et des communautarismes. Pourtant, Marseille œuvre pour sortir de ces âges obscurs. Quand elle est désignée pour être la capitale européenne de la culture en 2013, l’impulsion est donnée. La ville quitte son voile de tristesse. Sous l’égide d’Euroméditerranée, gigantesque projet urbain qui redessine les quatre cent quatre-vingts hectares du cœur de la ville, la Cité phocéenne se donne les moyens d’être la capitale de la Méditerranée du XXIe siècle.   » LIRE AUSSI - Trésors des musées de Province: au Mucem le retour des beaux jours  A l’entrée du Vieux-Port, face aux paquebots, les entrepôts abandonnés du quai J4 - jetée numéro 4 - ont laissé place au premier grand musée national consacré à la Méditerranée: le Mucem, musée des Civilisations de l’Europe et de la Méditerranée. Il a hérité de la collection du musée des Arts et Traditions populaires et de la partie «Europe» du musée de l’Homme, dont le volet «extra-européen» a rejoint le musée du Quai-Branly.  «A partir des années 2000, le concept des musées ethnographiques, créés pendant la période coloniale, paraissait dépassé. Il fallait inventer autre chose. Le Mucem participe à cet élan»,  explique Jean-François Chougnet, président du musée. Pour servir d’écrin aux collections, l’architecte Rudy Ricciotti a dessiné le navire amiral de ce pôle muséal qui englobe le fort Saint-Jean. Son édifice sombre, recouvert d’une mantille en dentelle de béton ciselée, domine la jetée dont il a pris le nom: «J4».    Une passerelle entre ciel et mer le relie au fort Saint-Jean. Magnifiquement restauré, ce dernier, construit par Louis XIV, éventré par l’explosion des stocks de munitions allemands en 1944, se trouve ainsi relié à la ville pour la première fois depuis quatre siècles. Que ce soit sur les rampes du J4, par lesquelles on tourne autour de l’édifice en jouant avec le soleil à travers la dentelle de béton, ou dans les allées du fort, on se promène librement, tout en étant invité à visiter les expositions des différents espaces du bâtiment. Cet été, on peut y découvrir l’histoire des grandes cités méditerranéennes des XVIe et XVIIe siècles, y voyager à travers une exposition consacrée à l’Orient sonore, ou encore approfondir ses connaissances botaniques par une exposition, «La Flore, de A à Z», et prolonger la visite par une promenade dans le «Jardin des migrations», planté d’espèces méditerranéennes. Les odeurs de thym et de lavande s’y mêlent au bleu profond de la mer. On se laisse griser…   Ces odeurs nous transportent au cœur de l’âme provençale que célèbre justement le musée Regards de Provence, situé à deux pas du Mucem. Ce bâtiment en béton, dont la silhouette évoque un navire, avait été construit après la Seconde Guerre mondiale par l’architecte Fernand Pouillon, contemporain de Le Corbusier. Il abritait alors une station sanitaire pour les personnes débarquant à Marseille, traumatisée encore par le souvenir de la peste de 1720, qui avait tué entre le tiers et la moitié de la population.  «Seule la façade était classée, mais nous avons choisi de conserver la salle des étuves, où les vêtements étaient désinfectés, et de donner à voir au public l’histoire de ce lieu étonnant»,  explique Adeline Granerau, directrice du musée et fille de Michèle et Pierre Dumon, qui ont constitué cette collection, riche de quelque neuf cent cinquante œuvres, présentées à travers quatre à cinq expositions par an. Cet été, c’est «La Provence de Giono», vivante et sauvage, qui est évoquée à travers les œuvres des peintres de l’école des paysagistes de Marseille.   Relire Regain    C’est d’ailleurs sans doute le roman de Giono  Regain  qu’il faudrait (re)lire en arpentant les quais de la Joliette. Son héros, Panturle, y fait revivre une terre qu’on croyait morte. Au loin, les tours construites par Zaha Hadid et Jean Nouvel témoignent justement du regain du quartier. Un centre commercial, les Docks, s’est invité dans les anciens entrepôts, et le Silo, théâtre qui conserve dans son nom la mémoire de sa fonction originelle, a échappé à la destruction. Si ses étuves ont été retirées pour construire la salle de spectacle et les bureaux, dans le hall d’entrée, les «mamelles» - ces cônes d’où tombaient les grains de blé pour être ensachés - évoquent le passé toujours vivant de ce quartier.   Ce regain se communique au quartier du Panier, tout proche du Vieux-Port. A partir des quais, on peut le rejoindre par une passerelle jetée entre le fort Saint-Jean et l’église Saint-Laurent ou encore par les rues qui montent de la cathédrale de la Major, dessinée notamment par Henri Espérandieu - encore lui! - en surplomb du port de la Joliette: naguère contournée par l’autoroute urbaine, noircie par la pollution et presque inaccessible, la voici désormais d’une blancheur éclatante et donnant sur une grande place.   On arpente donc les ruelles du Panier, qui fut le site de la première colonie grecque, Massalia. Voici ce quartier populaire et cosmopolite, qui fut longtemps le secteur des petits métiers de la mer et le lieu d’installation des populations migrantes et ouvrières, redevenu l’un des cœurs battants de Marseille. Les habitants en fleurissent les rues, des artisans ont ouvert des échoppes, des artistes ornent de fresques les murs des maisons. Et surtout, l’art fait revivre son ancien hospice. Construit au XVIIe siècle pour accueillir les gueux de la ville, ce joyau architectural au centre duquel trône une chapelle ovoïde de style baroque, dessinée par le sculpteur Pierre Puget, avait été laissé à l’abandon après la Seconde Guerre mondiale. La végétation avait repris ses droits et ce lieu suscitait la mélancolie des ruines. Restauré en 1986, il est à présent le cœur de la vie culturelle du Panier, abritant désormais le musée d’Archéologie méditerranéenne, connu pour sa collection d’antiquités égyptiennes, le musée d’Arts africains, océaniens et amérindiens ou encore le Centre international de poésie de Marseille.   L’éternité retrouvée   Car désormais, la ville protège et valorise son patrimoine historique, mais aussi naturel. C’est que le territoire de Marseille s’étend au-delà de ses constructions urbaines. Ses calanques creusées dans les massifs de Marseilleveyre et du Puget sont protégées depuis 2012 par la création du parc national des Calanques. Sa particularité? A la fois terrestre, marin et insulaire, il est aussi le seul parc périurbain d’Europe.  «Soumis à une forte pression, il abrite une biodiversité extrêmement riche. Il nous appartient de la préserver malgré la surfréquentation des calanques»,  explique Alain Mante, responsable du secteur littoral ouest et archipels du parc national.   En partant du village de Callelongue, le voilà sac au dos et jumelles à la main qui arpente les sentiers qui mènent à la calanque de Marseilleveyre. Sous le ciel, les falaises qui caressent le ciel viennent se jeter dans la mer. Au loin, les îles de l’archipel de Riou racontent, à qui sait l’entendre, les puffins et autres oiseaux protégés qui viennent nicher sur leurs terres, tandis que les flots tentent de garder secretsleurs poissons, leurs coraux et les histoires lointaines des hommes: celles de la grotte paléolithique ornée de Cosquer, dont ils ont englouti les sublimes peintures, ou du dernier souffle d’Antoine de Saint-Exupéry, dont l’avion se fracassa en 1944 près de l’île de Riou.   Sur les falaises, des plantes rares et étonnantes défient les embruns, le mistral, la sécheresse.  «On peut avoir l’impression d’un espace pelé, alors qu’il s’y trouve de nombreuses espèces protégées qui risquent d’être piétinées par inadvertance»,  explique Alain Mante. Certaines se dessèchent pendant l’été, avant de reprendre vie à l’automne ; parfois, on les arrache sans savoir qu’elles sont vivantes et ont mis une dizaine d’années à pousser. D’autres poussent en coussinet, imbriquées les unes aux autres, comme aiment le faire le lentisque et le romarin. Mais peut-être les plus étonnants sont-ils les pins, qui poussent au ras du sol comme s’ils n’avaient pas de tronc, au point que leurs aiguilles semblent des buissons ou des herbes hautes, et pourtant, pour peu que l’on contourne le sémaphore de Callelongue pour rejoindre l’autre versant du relief, les voilà qui prennent de la hauteur, abrités du vent, et nous protègent du soleil!   Tout au long du chemin, où fleurissent des plantes aussi rares que l’hélianthème de Syrie ou l’astragale de Marseille, une armée d’hommes veillent sur cette biodiversité déplaçant des pierres pour rendre les sentiers confortables et inviter randonneurs, grimpeurs, plongeurs ou simples promeneurs - ils sont près de deux millions par an à visiter les Calanques! - à ne pas s’écarter de la route, au risque de détruire la végétation. Les Marseillais sont nombreux à les interpeller:  «S’il vous plaît, prenez soin de notre paradis!» Au bout d’une heure de marche sous le soleil, apparaît la calanque de Marseilleveyre, bordée de quelques cabanons. La plage, baignée d’une eau turquoise, n’est accessible qu’à pied ou en bateau. Au loin, les reflets du soleil valsent sur l’eau. Et soudain, surgissent à la mémoire les vers d’un poète de dix-sept ans qui ne sait pas encore qu’il mourra brutalement au bord de cette mer qu’il n’a encore jamais vue:  «Elle est retrouvée. / Quoi? L’Eternité. / C’est la mer allée / Avec le soleil.»  Voir aussi : Les Calanques de Marseille à Cassis, nos clés du paradis    Où partir cet été à Marseille? Nos hôtels coup de c œ ur dans la cité phocéenne    48 heures à Marseille, entre mer et collines
</article>
<article titre="Les emballages « take out » | L'Écho de Laval" date="2020 08 12" journal="L'Écho de Laval (QC) (réf. site web) - Écho de Laval">
Le début de l’année 2020 a chamboulé le quotidien de tout le monde. Peu importe notre emploi, notre âge ou notre nationalité, les mesures pour combattre la pandémie affectent nos...  Voir l'article  Ce document référence un lien URL de site non hébergé par Cision.
</article>
<article titre="Le Wakashio, échoué à l'île Maurice et à l'origine d'une marée noire, menace de se briser" date="2020 08 9" journal="HuffPost - France (site web)">
Si le bateau venait à se briser, ce sont près de 3000 tonnes d'hydrocarbures qui pourraient se déverser dans le lagon mauricien, joyau écologique.   ENVIRONNEMENT - Le bateau échoué dans les eaux cristallines de l'île Maurice avec 4000 tonnes de pétrole à bord menaçait ce dimanche 9 août de se briser, faisant craindre une catastrophe écologique encore plus grave dans cet espace maritime protégé. Dimanche soir, la petite nation de l'océan Indien se préparait au pire. Les équipes d'intervention sont provisoirement parvenues à bloquer la fuite d'hydrocarbures qui se déversaient depuis plusieurs jours de la cale du bateau.   Mais le risque que le vraquier se brise tout simplement en deux était grandissant. "Les fissures se sont creusées. La situation est encore pire", a déclaré à la presse le Premier ministre mauricien Pravind Jugnauth.   Le Wakashio, appartenant à une entreprise japonaise mais battant pavillon panaméen, transportait 3800 tonnes d'huile lourde et 200 tonnes de diesel lorsqu'il a heurté le 25 juillet un récif à Pointe d'Esny.   Situé sur la côte sud-est de l'île, ce récif est un joyau écologique connu pour ses sites de conservation classés internationalement, ses eaux turquoise et ses zones humides protégées.   Jeudi, les autorités mauriciennes ont annoncé que des hydrocarbures s'écoulaient de la coque fissurée du vraquier. Plus de 1000 des 4000 tonnes de carburant transportées par le Wakashio se sont déjà déversées en mer, a indiqué Akihiko Ono, le vice-président de la Mitsui OSK Lines, la société qui exploitait le navire.   "La situation sera incontrôlable"   Dimanche, des milliers de personnes ont afflué sur les rives pour tenter de limiter tant bien que mal la marée noire qui menace l'île. "Les gens ont compris qu'il fallait qu'ils prennent les choses en main pour protéger la faune et la flore", a déclaré à l'AFP Ashok Subron, un militant écologiste arrivé de la cité voisine de Mahébourg.   Les volontaires se sont efforcés de tresser des barrages flottants en chanvre et en tissu afin de circonscrire la nappe de carburant. D'autres, munis de masques et gants en caoutchouc, tentaient de ramasser dans des seaux les produits échappés du navire.   #Mauritius: Fière et heureuse d'avoir participé à la confection de bouées de bagasse pour contenir le déversement de fioul. Merci et bravo à tous les volontaires! #MauritiusOilSpill#OilSpill#Ecologypic.twitter.com/I76BaKpg2z   -- Lu (@Lucie008) August 9, 2020   Jusqu'à présent, les eaux agitées ont mis en difficulté les opérations pour limiter les fuites d'hydrocarbures. Selon des images satellites, la nappe a déjà commencé à dériver vers la côte, attisée par des vents et des courants forts.   "Je pense qu'il est déjà trop tard. Si le navire se casse en deux, la situation sera incontrôlable", a déclaré à l'AFP Vassen Kauppaymuthoo, un océanographe et ingénieur environnemental.    #Wakashio#oilspill destroyed #Maurtius lagoon &amp; its fragile #ecosystem   An #environmental &amp; #human disaster   Destruction de l'#environnement#marin de l'#IleMauricepic.twitter.com/C77rYsfnvf   -- zeenat hansrod (@zxnt) August 9, 2020   "Réunion de crise"   L'île Maurice possède les plus beaux récifs coralliens du monde et constitue un sanctuaire pour une faune rare et endémique. Ses 1,3 million d'habitants dépendent de ses eaux pour la nourriture et l'économie. "La pêche est notre seule activité. Nous ne savons pas comment nous pourrons nourrir nos familles", a confié un pêcheur interrogé par l'AFP.   Le Premier ministre Pravind Jugnauth, qui a déclaré un "état d'urgence environnemental", a convoqué dimanche une réunion de crise des autorités concernées et remercié la France pour son aide.   Samedi, un navire de la marine française et un avion avec des experts à bord sont partis pour Maurice depuis la Réunion. Le Japon a annoncé de son côté l'envoi d'une équipe de six experts pour travailler aux côtés des secours français et locaux.   La police mauricienne avait prévu d'aborder dimanche le Wakashio, afin de saisir les rapports de navigation et les enregistrements des communications. Les 20 membres de l'équipage du vraquier avaient pu être évacués sains et saufs après son accident fin juillet.  Cet article est paru dans HuffPost - France (site web)
</article>
<article titre="Feel Food, un resto zéro déchet" date="2020 08 5" journal="L'Avenir (site web)">
Au Comptoir de Feel Food, en face du beffroi de Thuin, on recycle et on trie tous les déchets. Rencontre avec Nadia Zemni.   Feel Food, c'est un food truck mais aussi un resto. Au restaurant, on ne paie pas le prix du plat mais bien son poids, un nouveau concept dans la région. Ici, on peut tout acheter en vrac. Les gens composent leur assiette eux-mêmes (viande et légumes). En payant le juste prix, on évite aussi le gaspillage alimentaire. Feel Food, c'est aussi une politique zéro déchet?   Nous essayons au maximum de nous y tenir. Pour les boissons, nous n'achetons que des bouteilles en verre. Nous évitons d'utiliser des emballages en carton ou en plastique. Nous achetons beaucoup de produits bio en vrac. Je fabrique moi-même des limonades saines, mes apéritifs à base de produits locaux (thé, cuberdon, alcools de la distillerie de Biercée, pastis belge…). Pour nos plats à emporter, les assiettes et les couverts sont en bagasse (de la fibre de canne à sucre biodégradable) et les couvercles sont faits à partir de matière recyclée. On les jette avec les PMC. Je n'achète pas de café moulu mais du café en vrac que nous moulons nous-mêmes. Nos déchets alimentaires font le bonheur des poules d'une connaissance. Malheureusement, le plastique reste encore incontournable. Pour leur conservation, certains aliments doivent être mis sous vide. Néanmoins, en étant attentifs à tout ceci, nous n'avons qu'un à deux sacs-poubelle par semaine.   Vous privilégiez les circuits courts?   Oui. Nous travaillons exclusivement avec des producteurs et fournisseurs du coin qui ont la même optique de vie que nous. J'aime défendre l'idée du « bio réfléchi » . Arrêtons d'appauvrir les terres! Je suis très sensible au côté « environnement » . Ici, même le mobilier est local. Nous avons d'ailleurs construit nous-mêmes nos tables à partir d'un tronc d'arbre qui a été débité dans une scierie à Strée. Nous faisons vivre des artisans du coin. La déco est faite de récup. Même la peinture est de fabrication belge.   Trier, c'est contraignant?   Ça prend du temps et ça demande de l'énergie. Il faut prévoir une personne qui gère le tri et il faut se rendre au recyparc au moins une fois par semaine. Le brasseur doit régulièrement passer pour récupérer les vidanges. C'est vrai que c'est beaucoup plus facile de ne pas trier, mais le jeu en vaut la chandelle.  Cet article est paru dans L'Avenir (site web)
</article>
<article titre="Objectifs confirmés chez Albioma malgré du retard pour Le Moule3" date="2020 08 3" journal="Investir (site web)">
AchatLes entreprises annonçant une forte hausse de leurs résultats semestriels ne seront pas légion. Albioma est l'une d'elles, ne décevant pas la confiance d'Euronext, qui a promu la valeur dans l'indice SBF120 en juin, avec Neoen, autre producteur d'électricité d'origine renouvelable.   Leurs ventes sont encadrées par des contrats de long terme, procurant des revenus récurrents, épargnés par la baisse de la demande d'énergie pendant le confinement. « Nos centrales, qui sont des centrales de base, continuent de fonctionner. Ce sont les installations au fioul d'EDF qui ont accusé la baisse de la consommation », nous expliquait, début mai, Frédéric Moyne, le directeur général, soulignant aussi la bonne disponibilité des parcs solaires. Du charbon au bois Les centrales thermiques françaises d'Albioma (qui décline son modèle au Brésil) se situent dans les régions sucrières des outre-mers. Ces installations brûlent de la bagasse, le résidu de la canne, pendant la campagne sucrière et du charbon le reste de l'année, selon un schéma en cours de transformation : la houille va être remplacée par de la biomasse issue du bois. La nouvelle centrale de Galion 2, en Martinique, fonctionne ainsi, tandis que LeMoule3, à la Guadeloupe, est en cours de conversion. Les travaux ont pris du retard du fait du confinement, seul véritable inconvénient de la pandémie pour le groupe. Le site devrait redémarrer seulement en fin d'année, au lieu de juin initialement.   La direction confirme cependant ses objectifs annuels : un Ebitda (excédent brut d'exploitation) de 200 à 210 millions d'euros et un bénéfice net de 48 et 54 millions. L'indisponibilité du Moule 3 l'incite juste à viser la moitié basse de ces fourchettes. Le premier semestre est dans les clous. Pour un chiffre d'affaires de 250,7millions d'euros (+4%), l'Ebitda s'est accru de 19%, à100,4 millions, et le bénéfice net de 34%, à 24,4 millions. Les centrales thermiques françaises ont amélioré leur taux de disponibilité de 85% à86,7%,mais Albioma a aussi engrangé des rémunérations complémentaires pour ses investissements dans des équipements de traitement des fumées.   Cécile Le Coz
</article>
<article titre="Chiffres clés des énergies renouvelables - Édition 2020 (PDF - 3.08 Mo) - datalab_69_chiffres_cles_enr_edition2020_juillet2020_0.pdf" date="2020 07 21" journal="Rapports - Environnement - MEDDTL">
92 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... Chiffres clés des énergies renouvelables Édition 2020 JUILLET 2020  ÉNERGIE  L A B  T A  D A 2 – Chiffres clés des énergies renouvelables – Édition 2020  Chiffres clés des énergies renouvelables Édition...    Pour voir le rapport :  https://www.ecologie.gouv.fr/sites/default/files/datalab_69_chiffres_cles_enr_edition2020_juillet2020_0.pdf
</article>
<article titre="HERBES FOLLES: la nouvelle marque qui prend soin de notre peau et de l’environnement" date="2020 07 17" journal="Flair (français) (site web)">
Herbes Folles, c’est la nouvelle marque de soins pour la peau faits avec des “mauvaises herbes“ et beaucoup d’amour. On vous parle de notre coup de coeur beauté du moment.   Une marque faite avec des “mauvaises“ herbes   Ça peut sembler peu glamour, mais la marque Herbes Folles utilise dans ses formules des “mauvaises“ herbes. Pourquoi ? Parce qu’elles sont simples, libres, qu’elles sont reconnues pour leur ténacité et qu’elles sont bénéfiques pour les humains et les écosystèmes. Riches en composés nourrissants, ces herbes de tradition médicinale sont connues pour leur efficacité et sont des alliées parfaites pour la peau. Sur le choix de l’usage des mauvaises herbes, Mariana, la fondatrice de Herbes Folles confie   Nous traversons un moment de notre histoire où il n’est plus possible de traiter le règne végétal comme “accessoire“.   Une marque qui mise sur la simplicité   Pour l’instant, la marque propose trois produits de soins du visage: un baume multi-usages, une huile nettoyante, et une lotion hydratante bi-phasée. Formulés avec des extraits de plantes comme le Coquelicot, la Chicorée, le Millepertuis, l’Ortie, le Pourpier, la Carotte Sauvage et le Plantain, ils sont pensés pour agir en synérgie, en formant les essentiels d’une routine de beauté simple et naturelle.   Lire aussi: BEAUTY LAB: on a testé l’huile solaire bio SPF20 de la marque belge Biosolis.   Un hydratant léger qui nourrit et protège la peau, calme les peaux sensibles et illumine le teint. Il contient des acides gras repulpants issus des huiles de sureau et de pavot, et est enrichi d’un prébiotique naturel extrait des racines de chicorée qui rééquilibre les troubles de la peau. Il contient également des feuilles de plantain, des graines de carotte sauvage et des pétales rouges de coquelicot qui sont de puissants antioxydants aux propriétés soignantes.   37,50 €, en vente ici.   Une huile nettoyante qui élimine en douceur les impuretés et le maquillage sans assécher la peau. Elle a également des proprieties hydratantes et anti-inflammatoires. Elle contient de l’huile de caméline et de tournesol, bien équilibrés en acides gras nourrissants, et est infusé d’extraits de pourpier et de millepertuis.   33,50 €, en vente ici.   Un baume multi-usages, sans parfum qui protège, nourrit et régénère la peau abîmée. Il associe le beurre de graines de Murumuru, excellent pour les peaux abîmées, le pourpier, riche en acides gras oméga-3, et la fameuse ortie, l’une des plus riches sources d’antioxydants végétaux.   16 €, en vente ici.   Une marque ultra green   Bien que la marque Herbes Folles soit portugaise, les produits sont faits à Spa, dans un petit laboratoire spécialisé dans la cosmétique naturelle. Herbes Folles est certifiée NATRUE (un label qui établit une norme élevée pour définir le caractère naturel et durable d’un produit cosmétique). Les produits de la marque ne contiennent aucun OGM, parfum synthétique, conservateur artificiel, huile minérale ou produit pétrochimique. De plus, la marque est membre de 1% pour la Planète, un mouvement mondial qui inspire les entreprises et les citoyens à soutenir des solutions environnementales. Enfin, la marque met un point d’honneur à développer des emballages respectueux de l’environnement. Tous ses emballages sont produits en Europe et exclusivement composés de verre et de carton. Les étiquettes sont sans arbres et produites à partir de bagasse, un déchet de l’industrie sucrière.   D’autres articles pour une beauté plus green:  Cet article est paru dans Flair (français) (site web)
</article>
<article titre="A la Réunion, la facture salée du sucre de canne" date="2020 07 15" journal="Libération">
«O n ne peut pas dire indéfiniment : "Donnez-nous des aides ou je mets tout le monde au chômage."» Arnaud Martrenchar, délégué interministériel à la transformation agricole des outre-mer, l'assure : il n'est pas là pour casser du sucre sur le dos de Tereos. Mais le géant de l'agroalimentaire français, deuxième groupe sucrier au monde, acteur monopolistique de la filière canne à la Réunion, est pourtant bien visé. Depuis 2017, Tereos océan Indien perçoit une aide annuelle de l'Etat français de 28 millions d'euros, qui s'ajoute aux nombreuses subventions françaises et européennes. Cette enveloppe, que Paris a dû négocier auprès de Bruxelles, vient compenser la libéralisation du marché du sucre et ses conséquences. Entre 2017 et 2019, les cours ont chuté drastiquement et la concurrence internationale s'est intensifiée. Sans cette perfusion d'argent public, «nous n'aurions pas pu survivre», explique Philippe Labro, le PDG de Tereos océan Indien, qui est aux manettes des deux dernières usines à sucre de l'île, monstres de tuyauteries et d'engrenages. C'est lui qui achète la totalité de la canne aux quelque 2800 planteurs réunionnais, soit 1,7 million de tonnes l'an dernier, pour les transformer en 190000 tonnes de sucre. Mais la facture est jugée salée. Emmanuel Macron, de passage à la Réunion en octobre, avait demandé une remise à plat complète de la filière. Les surcoûts du sucre de canne sont-ils justifiés par rapport au sucre tiré en métropole de la betterave ? Ne peut-on pas trouver d'autres débouchés que les cristaux roux ou blancs ? «L'Etat souhaite qu'on étudie toutes les alternatives, on avisera ensuite», prévient Arnaud Martrenchar. «Fatak» Ensuite ? Le couperet risque de tomber dès décembre 2021, les 28 millions n'étant plus garantis après cette date. Sur l'île, plus de la moitié de la surface agricole est plantée en canne, dont la culture remonte au XVIIIe siècle, du temps de l'esclavage. Soit 18 000 emplois directs, indirects et induits, rappelle le syndicat du sucre. C'est dire l'enjeu. Frédéric Vienne, président de la chambre d'agriculture de la Réunion, y voit, lui, une opportunité. Le planteur avait été condamné en 2017 pour avoir arraché des cultures sur les parcelles d'Ercane, la filiale R &amp; D de Tereos. Alors à la tête de la Fédération départementale des syndicats d'exploitants agricoles (FDSEA), il réclamait un prix d'achat de la canne plus élevé. «La canne, patrimoine historique et culturel ? Ça ne remplit pas le ventre des planteurs, assène-t-il. Aujourd'hui, c'est l'occasion de ne plus subir le diktat de Tereos.» Son plan ? Produire du sucre bio, en se passant des services de l'usinier. Il est soutenu par le leader du syndicat des Jeunes Agriculteurs, Bruno Robert : «Les planteurs ne seront plus de simples livreurs de canne, mais les acteurs de A à Z d'une nouvelle filière, avec leur propre unité de transformation qui produira 500 tonnes de sucre bio», s'enthousiasme ce dernier. Les hypermarchés Carrefour accompagnent la démarche et se sont déjà engagés à en acheter 140 tonnes. L'en-finance même à hauteur de 300 000 euros une étude de faisabilité. Au milieu de ses coqs de combat, Beurty Dubar, scorpion tatoué dans le cou, espère que le discours sera suivi de faits. Le solide quinquagénaire de Sainte-Anne, dans l'est de l'île, est l'un des très rares planteurs à déjà produire de la canne bio. Depuis quatre ans, il cultive Saint-Denis LA RÉUNION Sainte-une parcelle de trois hectares sans herbicides ni engrais chimique. A la place, une serpette pour couper les «fatak», ces lianes qui s'entortillent autour des roseaux, la débroussailleuse pour les mauvaises herbes et du fumier de poule. «Je me suis lancé dans le bio après avoir failli mourir à cause des produits. Je ne pouvais plus respirer, j'avais des étoiles dans les yeux», raconte l'homme en lissant sa barbichette. Mais l'exploitant perd de l'argent sur la canne bio - il doit entretenir le potager d'un hôtel de luxe pour s'en sortir. «Il faudrait vendre la tonne 100 euros au moins, or Tereos ne m'en donne que 80», se désole-t-il. Car sa canne bio est mélangée à la canne conventionnelle, sans aucune plus-value Le projet de la chambre d'agriculture lui apparaît comme un moyen de se «libérer de la culture postcolonialiste, de reprendre la main sur notre destin». Le discours de Sophie et de Philippe Lucas, trois hectares de canne bio sur les pentes du volcan, au Tampon, est moins identitaire Océan Indien 10 mais tout aussi militant : «On n'en vit pas, mais si personne ne se lance, on ne pourra pas montrer que c'est possible.» Aussi continuentils vaillamment à enfouir dans le sol quelques grammes de corne de vache et de silice broyée, en adeptes convaincus de l'agriculture biodynamique. Peine perdue : les 170 tonnes produites annuellement disparaissent elles aussi, anonymes, sous les broyeurs de l'usine du Gol pour être transformées en sucre non bio «C'est un peu décourageant», reconnaît le couple, qui se considère comme «le mouton noir de la profession», peinant à recruter de la main-d'oeuvre pour couper, à la machette, une surface si petite. Pourtant, il faudra bien y passer, puisque l'utilisation des produits phytosanitaires sera progressivement interdite à partir de 2022. La tâche sera rude à la Réunion, deuxième département français à utiliser le plus de glyphosate, avec 1,21 kilo par hectare, selon une enquête de l'association de défense de l'environnement Générations futures en 2018 Rude, et longue.   «Bagasse» Le projet de la chambre d'agriculture ne concerne que 5 000 tonnes de canne sur les 2 millions coupés chaque année sur l'île «Un microbe», pour reprendre l'expression de Philippe Labro, à Tereos, qui a cependant emboîté le pas des producteurs bio. Mais lui évoque une production six fois plus importante à terme : «Il y a quelques années, le bio provoquait un casus belli chez les planteurs. Pour eux, c'était un retour en arrière, à l'esclavage, quand on désherbait à la main. Les mentalités ont évolué, on peut enfin étudier sereinement ce qui est possible ou non.» Tereos expérimente donc depuis avril la canne bio, sur une minuscule parcelle de 1,2 hectare. Dans trois ou quatre ans, on mesurera la richesse en sucre des roseaux et les coûts d'exploitation. «Je suis un industriel, prévient le patron. Si ce n'est pas compétitif, je dirai la vérité même si elle ne fait pas plaisir.» Philippe Labro glisse au passage que les coûts de main-d'oeuvre, plus importants en bio, sont «17 fois moins élevés en Tanzanie», où le groupe possède une autre usine à sucre, qu'à la Réunion Il ajoute qu'en bio, il n'est pas possible de recourir au floculant traditionnel dans les cuves de décantation des usines, lors de la fabrication des cristaux. Ce qui diminue les rendements, encrasse les évaporateurs, augmente les coûts Bref, même s'il assure être «a priori intéressé», le spécialiste n'y croit pas à brève échéance.   Anne Le Tampon km Quant à l'autre débouché envisagé par beaucoup, la «canne fibre», Philippe Labro est encore plus dubitatif. Le roseau est composé d'environ 70 % d'eau, de 14 % du recherché saccharose et de 15% de fibres. Aujourd'hui, ces fibres ou «bagasse» sont récupérées après le broyage de la canne et brûlées dans deux centrales thermiques de l'île, pour être transformées en chaleur ou en électricité. Et si on augmentait la part des fibres, pour produire une énergie renouvelable sans utiliser de charbon, au lieu de fabriquer du sucre subventionné ? Pour la chambre d'agriculture, ce serait aussi un moyen de mieux rémunérer certains planseigne teurs, qui produisent des tonnages de canne importants, mais pauvres en sucre. La région Réunion, qui vise l'autonomie énergétique de l'île en 2030, veut y croire. Dans sa programmation pluriannuelle de l'énergie 2019-2028, elle prévoit de recourir à la canne fibre pour alimenter de petites centrales électriques. La fibre pourrait également être pressée pour produire de l'éthanol et alimenter des turbines à combustion, voire méthanisée et transformée en gaz. Le Cirad, organisme de recherche agronomique, a été missionné pour étudier ces différents scénarios et reste très prudent. Les processus industriels et les conditions de rentabilité sont loin d'être établis. Une canne de la Barbade, dans les Caraïbes, prometteuse sur le papier, s'est révélée décevante. «Le gain en fibres est minime et ces cannes ont tendance à se coucher en cas de pluies ou de vents forts», convient François Broust, chercheur en bioénergie. Par ailleurs, les 23 000 hectares de canne actuels ne remplaceraient pas même la moitié du charbon aujourd'hui brûlé à la Réunion.   Dans ces conditions, Philippe Labro veut conserver ses propres cannes : les R585 et R586, variétés mises au point par la filiale Ercane. Elles sont légèrement plus riches en fibres que la moyenne (17,5%) et ont l'avantage d'avoir de gros rendements en sucre. Mais elles représentent moins de 10 % des surfaces actuelles. Comme 1600 hectares sont replantés chaque année, il faudra quatorze ans pour que cette canne mixte augmente significativement la production d'électricité verte sur l'île. On est loin des propositions fortes que l'Etat attendait le 26 juin, lors de la dernière réunion du comité de transformation agricole. Les planteurs n'évoquent que des microprojets, les collectivités lancent des études. Et Tereos les cours du sucre s'étant redressés, le géant agroalimentaire sort «renforcé de la libéralisation du secteur sucrier européen avec des résultats en forte croissance», dixit Alexis Duval, président du directoire. Pourquoi changer dans ces conditions ? ?
</article>
<article titre="Verdure : la saladerie en phase avec le monde d’après…" date="2020 07 9" journal="L'Est Républicain">
C’était un projet qui mûrissait depuis quatre ans. « Et qui devait voir le jour en trois tranches, étalées sur 3 ans. » Avec une première étape prévue ce printemps. « À savoir refaire le mobilier de comptoir et de service en salle. Le devis était signé le 6 mars, je devais réceptionner les meubles fin mai. Mais le 14 mars… fermeture des restaurants. » Sidération. D’autant que pendant quinze jours, Maxime Bermudez, patron de Verdure, saladerie historique de Nancy, n’a plus eu aucune nouvelle de son menuisier. « Pour cause : il avait contracté le virus… »   Le contact a fini par être renoué, et en attendant, le restaurateur a décidé de mettre le confinement à profit pour engager les travaux immédiatement. « Mais fin avril, ma partie était terminée, or j’attendais les meubles fin mai. Alors j’ai attaqué la 2e phase : plafonds, climatisation, etc. » Et c’est comme ça que depuis lundi dernier, la saladerie de la rue Raugraff s’est rouverte à ses clients complètement métamorphosée.   « À l’origine, ces travaux avaient deux objectifs : optimiser le service pour réduire le temps d’attente. Or avec la reconfiguration du bar, elle baisse de 30 % ! En outre, attentifs à leur bien-être et à celui de la planète, nos clients remarquaient à raison que notre vaisselle plastique générait trop de déchets incompressibles. »   Exit donc le plastique ! Au profit de vaisselle lavable pour la consommation sur place, ou de récipients en bagasse (fibre végétale) pour l’option « à emporter ». Oubliée, aussi, la partie self-service en amont de la composition des salades. « Si bien que le consommateur ne touche plus à rien. Ce qui a l’avantage énorme de coïncider avec les contraintes liées à l’épidémie ! »   Un meuble de tri a par ailleurs été ajouté, « et dès le redémarrage, on a vu le volume des déchets journaliers baisser de 80 % ». Tout en conservant la variété des propositions, à savoir sept bases (4 salades, 3 féculents) agrémentées d’une cinquantaine d’ingrédients et d’une douzaine d’assaisonnements possible.   Dernière phase de travaux en septembre pour refaire la façade. Et ainsi cette saladerie vieille de 10 ans affrontera le « monde d’après » sous un jour complètement nouveau.
</article>
<article titre="Pierre va vendre ses soupes et ses torillas maison aux supporters du Standard" date="2020 07 8" journal="La Capitale (site web)">
Pierre Legros va entrer en couveuse d'entreprise pour lancer son activité de maraîchage et transformation de légumes 100 % non traités, cultivés à Anthisnes. À la reprise des matches avec supporters au Standard de Liège, il proposera une option plus saine que les traditionnels hamburgers/frites, sous la forme de bols de soupe ou de morceaux de tortillas maison.   Pendant 15 ans, Pierre Legros a vécu en Espagne où il a pratiqué différents métiers (dans une banque, dans le tourisme, sur un bateau de croisière…) et où il a surtout rencontré l'amour, Lia. Il y a quatre ans, l'Anthisnois est revenu en Belgique avec son épouse espagnole dans l'intention de se lancer dans le maraîchage. « Ma sœur a une asbl de maraîchage dans la commune d'Anthisnes, Rouliberwète. Notre idée était de développer un projet avec elle mais, finalement, elle préfère lever le pied et m'aider à me lancer », nous confie Pierre Legros, 46 ans. L'Anthisnois a alors récupéré son outillage, sa serre et même le terrain mis à disposition par des amis, tout en profitant de l'expérience sororale pour devenir un bon maraîcher. Mais ce n'est pas tout : l'idée de transformer ses légumes 100 % non traités a vite germé. « Vivre du maraîchage est compliqué, nous avons donc décidé de transformer notre production de légumes nous-mêmes en plats simples comme des tortillas, références à l'Espagne et qui ont l'avantage de contenir beaucoup de légumes, mais aussi des soupes ainsi que des plats simples à emporter », explique-t-il.   En partenariat avec l'asbl Rouliberwète en attendant d'être officiellement en couveuse d'entreprise via Job'In avec « Sou'Pierre et Torti'Lia » ( « ce qui devrait être le cas en août » ), Pierre Legros cuisine déjà ses préparations à l'Avouerie d'Anthisnes chaque mardi, ses plats à emporter sont à commander via le GAC d'Anthisnes ou par mail ou Facebook. D'ici une semaine ou deux, il tiendra également une échoppe aux marchés de Huy et d'Esneux pour y vendre de la soupe dans des contenants d'un litre, ainsi que des tortillas et quiches à emporter.   Et dès la reprise des matches avec spectateurs, en septembre espère-t-il, il vendra de la soupe et des morceaux de tortilla aux supporters du Standard de Liège, dans l'enceinte du stade de Sclessin. « C'est le Standard qui est à l'origine de la démarche, dévoile Pierre Legros. Il a contacté la Ville de Liège car il souhaitait offrir un choix de nourriture plus sain et local aux supporters du stade et une amie qui travaille à l'administration nous a mis en contact. J'ai présenté mon projet qui leur a plu », retrace-t-il.   Le Standard confirme : « Nos fans sont vraiment demandeurs de ce type de nourriture. Un contrat nous lie avec M. Legros », abonde Charles Caillot, chargé de projets RSE pour le Standard.   En réalité, l'Anthisnois a déjà eu l'occasion de faire goûter ses préparations aux footeux puisqu'il s'y est déjà rendu à quatre reprises en fin d'année 2019, avec l'asbl de sa sœur. « Je voulais voir si la sauce allait prendre avant de vraiment me lancer et ça fonctionnait déjà bien. Nous étions alors sous une tonnelle, bientôt le Standard va probablement bientôt m'installer dans un chalet en bois », se réjouit-il. À la mi-temps ou après la rencontre, il sera donc possible de déguster un demi-litre de soupe à 3 euros ou un huitième de tortilla pour 3 euros la part. « Les récipients des soupes sont en bagasse, des résidus de canne à sucre, biodégradables donc », relève-t-il encore.   De quoi célébrer la victoire ou digérer la défaite, l'estomac léger.   Un crowdfunding de 6.500 euros   Pour soutenir l'investissement consenti par Pierre Legros, ce dernier a lancé une campagne de financement participatif, elle sera en ligne sur le site www.crowdin.be d'ici quelques jours, pour une durée d'un mois. Le premier palier est fixé à 4.000 euros et couvre l'achat de matériel de cuisine : soupières électriques, grosses casseroles à soupe, mixer professionnel, bacs de transports aux normes AFSCA, mais aussi une tonnelle. Le second palier de 2.500 euros concerne l'achat d'une Citroën Berlingo d'occasion pour faire les marchés. Différentes contreparties sont prévues, sous forme de nourriture ou d'ateliers de cuisine bolivienne/espagnole ou même de promenades dans les bois à la découverte des champignons sauvages. Si vous souhaitez soutenir le projet de Pierre et sa femme Lia, suivez l'actualité de la page Facebook de Sou'Pierre et Torti'Lia.  Cet article est paru dans La Capitale (site web)
</article>
<article titre="De la soupe pour les supporters du Standard L'Anthisnois proposera ses mets sains dès la reprise des matches" date="2020 07 8" journal="La Meuse  La Meuse - Huy Waremme">
Pierre Legros va entrer en couveuse d'entreprise pour lancer son activité de maraîchage et transformation de légumes 100 % non traités, cultivés à Anthisnes. A la reprise des matches avec supporters au Standard de Liège, il proposera une option plus saine que les traditionnels hamburgers/frites, sous la forme de bols de soupe ou de morceaux de tortillas maison.    Pendant 15 ans, Pierre Legros a vécu en Espagne où il a pratiqué différents métiers (dans une banque, dans le tourisme, sur un bateau de croisière...) et où il a surtout rencontré l'amour, Lia. Il y a quatre ans, l'Anthisnois est revenu en Belgique avec son épouse espagnole dans l'intention de se lancer dans le maraîchage. « Ma soeur a une asbl de maraîchage dans la commune d'Anthisnes, Rouliberwète. Notre idée était de développer un projet avec elle mais, finalement, elle préfère lever le pied et m'aider à me lancer », nous confie Pierre Legros, 46 ans. L'Anthisnois a alors récupéré son outillage, sa serre et même le terrain mis à disposition par des amis, tout en profitant de l'expérience sororale pour devenir un bon maraîcher. Mais ce n'est pas tout : l'idée de transformer ses légumes 100 % non traités a vite germé. « Vivre du maraîchage est compliqué, nous avons donc décidé de transformer notre production de légumes nous-mêmes en plats simples comme des tortillas, références à l'Espagne et qui ont l'avantage de contenir beaucoup de légumes, mais aussi des soupes ainsi que des plats à emporter », explique-t-il.   En partenariat avec l'asbl Rouliberwète en attendant d'être officiellement en couveuse d'entreprise via Job'In avec « Sou'Pierre et Torti'Lia » ( « ce qui devrait être le cas en août » ), Pierre Legros cuisine déjà ses préparations à l'Avouerie d'Anthisnes chaque mardi, ses mets à emporter sont à commander via le GAC d'Anthisnes ou par mail ou Facebook. D'ici une semaine ou deux, il tiendra également une échoppe aux marchés de Huy et d'Esneux pour y vendre de la soupe dans des contenants d'un litre, ainsi que des tortillas et quiches à emporter.   Et dès la reprise des matches avec spectateurs, en septembre espère-t-il, il vendra de la soupe et des morceaux de tortilla aux supporters du Standard de Liège, dans l'enceinte du stade de Sclessin. « C'est le Standard qui est à l'origine de la démarche, dévoile Pierre Legros. Il a contacté la Ville de Liège car il souhaitait offrir un choix de nourriture plus sain et local aux supporters et une amie qui travaille à l'administration nous a mis en contact. J'ai présenté mon projet qui leur a plu », retrace-t-il. Le Standard confirme: « Nos fans sont vraiment demandeurs de ce type de nourriture. Un contrat nous lie avec M. Legros », abonde Charles Caillot, chargé de projets RSE pour le Standard.   En réalité, l'Anthisnois a déjà eu l'occasion de faire goûter ses préparations aux footeux puisqu'il s'y est déjà rendu à quatre reprises en fin d'année 2019, avec l'asbl de sa soeur. « Je voulais voir si la sauce allait prendre avant de vraiment me lancer et ça fonctionnait déjà bien. Nous étions alors sous une tonnelle, bientôt le Standard va probablement m'installer dans un chalet en bois », se réjouit-il.   A la mi-temps ou après la rencontre, il sera donc possible de déguster un demi-litre de soupe à 3 euros ou un huitième de tortilla pour 3 euros la part. « Les récipients des soupes sont en bagasse, des résidus de canne à sucre, biodégradables donc », relève-t-il encore.   De quoi célébrer la victoire ou digérer la défaite, l'estomac léger.   Annick Govaers
</article>
<article titre="L'effectivité des initiatives politiques de développement des EnR en Afrique subsaharienne" date="2020 07 7" journal="La Tribune (France), no. 6950">
Une personne sur deux, soit plus de 600 millions de personnes en Afrique subsaharienne, dont 75% vivent en zone rurale, n'a, à ce jour, pas d'accès à l'électricité, selon l'Africa Energy Outlook 2019 de l'IEA. Il existe pourtant un consensus pour considérer que le développement des énergies renouvelables (solaire, hydroélectrique, éolien, géothermique, biomasse, marin), inexploitées à ce jour sur le continent africain, permettrait à l'ensemble de la population d'Afrique subsaharienne d'y accéder.   En effet, l'énergie solaire peut être développée sur tout le continent, l'énergie hydroélectrique en Afrique centrale grâce notamment aux fleuves Nil, Zambèze ou Congo, la biomasse représente un potentiel important dans les régions centrales et méridionales grâce à la forêt primaire d'Afrique centrale et la bagasse, de même que la géothermie dans la vallée du Rift (Afrique de l'Est) et l'éolien sur les côtes et les îles; comme le détaille une analyse de l'IRENA, l'Agence internationale pour les énergies renouvelables (in l'Atlas mondial 2013).   Dans L'Afrique et les énergies renouvelables : la voie vers la croissance durable, celle-ci estimait ainsi à l'époque que « la part des énergies renouvelables en Afrique peut passer de 17% en 2009 à 50% en 2030 et à près de 75% d'ici 2050. La production électrique provenant de ces sources passerait de 28 GW en 2010 à 800 GW d'ici 2050 avec 245 GW générés à partir du solaire photovoltaïque, 242 GW de l'hydroélectricité, 94 GW de l'énergie solaire thermique à concentration, 69 GW de la biomasse et 8 GW de l'énergie géothermique. Ce scénario prend en compte l'objectif visant à atteindre l'accès universel à des services énergétiques modernes d'ici 2030, tout en réduisant considérablement les coûts à long terme ».   Forts de ce constat, de nombreux gouvernements africains et organisations régionales et internationales ont considéré les investissements dans le secteur des énergies renouvelables comme prioritaires. Conséquemment, de nombreuses initiatives ont été lancées afin de soutenir le développement du secteur énergétique ( I. ), sans que leur effectivité ne soit, pour l'instant, pleinement satisfaisante ( II. ).   Le foisonnement d'initiatives internationales et régionales de développement des EnR en Afrique subsaharienne Chaque sommet mondial relatif au développement durable, chaque « COP », chaque conférence sur le développement du continent africain, est l'occasion de rappeler la précarité énergétique du continent et les obstacles que cela représente pour le développement économique de l'Afrique subsaharienne.   Chacun de ces rassemblements est également l'occasion de mettre en place une nouvelle initiative ou un programme d'aide au développement du secteur énergétique africain.   En mai 2016, une vaste opération de cartographie de ces initiatives a été lancée par le Partenariat Afrique-UE pour l'énergie (PAEE). En limitant la portée de cette étude à « d'importants programmes et initiatives engagés au niveau continental ou régional avec une certaine forme de participation du secteur public », 58 ont été répertoriées au total.   En majorité, ces initiatives et programmes fixent des objectifs chiffrés de développement (doubler la part des EnR dans le mix énergétique, atteindre l'accès universel en 2050,...) et offrent une assistance technique (développement des compétences) et/ou financière (subventions, prêts et garanties, actions) pour les atteindre.   Si la Banque mondiale et l'Union européenne sont les deux organisations à l'origine du plus grand nombre de ces initiatives, celles-ci émanent également d'organisations régionales telles que la Banque africaine de développement (BAD), la Communauté économique des États de l'Afrique de l'Ouest (CEDEAO), la Communauté de développement d'Afrique australe (CDAA), mais aussi pour beaucoup d'agences bilatérales de développement (l'AFD pour la France, la GIZ pour l'Allemagne, la JICA pour le Japon,...).   Il ressort également de cette cartographie qu'en Afrique subsaharienne, le Kenya est le pays qui reçoit, de loin, le plus gros volume d'aide publique au développement dans le secteur de l'énergie, tandis que les pays d'Afrique centrale sont les grands perdants.   L'effectivité de ces initiatives en Afrique subsaharienne n'est pas pleinement satisfaisante Concrètement, ces initiatives ont permis à de nombreux projets de voir le jour. Ceux-ci étant tellement nombreux, seuls quelques exemples seront développés :   Le Fonds fiduciaire UE-Afrique pour les infrastructures (ITF), qui fonctionne depuis juin 2007 et regroupe les ressources de la Commission européenne et des États membres de l'Union européenne, de la Banque européenne d'investissement, des institutions européennes de financement du développement et de la Banque africaine de développement, a permis le financement de projets énergétiques majeurs. L'un des premiers projets financés est l'aménagement hydroélectrique de Félou, sur le fleuve Sénégal : centrale au fil de l'eau, d'une puissance installée de 60 MW qui produit de l'énergie à destination du Mali, de la Mauritanie et du Sénégal.   Le Fonds mondial pour la promotion de l'efficacité énergétique et des énergies renouvelables (GEEREF), créé en 2008 pour stimuler les investissements privés dans les EnR, crée et finance des fonds régionaux qui, à leur tour, apportent du capital-risque pour financer des projets en y prenant des participations. Ce fonds a permis le financement de la centrale géothermique Corbetti dans la vallée du grand Rift en Ethiopie d'une puissance de 500 MW, premier partenariat public privé du pays.   L'initiative de l'Afrique sur les énergies renouvelables (IAER), lancée lors de la COP 21 en 2015 à Paris par 54 chefs d'États africains, a mis en place une plateforme permettant aux ONG et entreprises du secteur privé en recherche de financement de déposer des projets d'EnR en Afrique. En 2017, les projets de centrales solaires à Agadez et Gourou Banda au Niger et à Onigbolo au Bénin ont par exemple été sélectionnés pour être financés.   L'initiative « Energie durable pour tous » (ou Sustainable Energy for all - SE4ALL), lancée en septembre 2011 par le Secrétaire général des Nations unies, a élaboré en 2015 un programme de développement du marché de mini-réseaux verts afin d'encourager l'adoption des mini-réseaux comme moyen d'améliorer l'accès à l'énergie en milieu rural. Grâce à ce programme, l'ONG « Energie 4 impact » apporte un soutien aux développeurs de mini-réseaux à travers « la modélisation commerciale et financière, la sélection du site, les études de faisabilité, l'engagement des communautés, l'évaluation de la demande, la conception du système technique, la levée de capitaux, l'acquisition d'équipement, la construction, la mise en service, l'exploitation et la maintenance de mini-réseaux ». L'ONG offre également « des services d'accompagnement commercial, technique et financier sur le terrain aux petits utilisateurs producteurs de mini-réseaux ». Ce soutien a bénéficié à plus de 4 000 entreprises locales.   Toutefois, en pratique, le constat reste que, malgré le développement de ces initiatives depuis plus de vingt ans, la région d'Afrique subsaharienne reste la moins électrifiée du monde.   Cela pourrait s'expliquer, d'une part, par le fait que les initiatives semblent se chevaucher, la coordination semble limitée et les efforts dupliqués. Il est ainsi difficile de retrouver toutes les informations liées à ces initiatives, que ce soit concernant leur mise en oeuvre, le nombre et le type de projets financés, l'aide accordée, le secteur privilégié, etc. Il n'est ainsi pas aisé pour un nouvel acteur de s'y retrouver et de pouvoir bénéficier de ces soutiens.   D'autre part, malgré les investissements toujours plus importants, le développement des EnR en Afrique subsaharienne se heurte à des défis d'ordre institutionnel, réglementaire et structurel au niveau national.
</article>
<article titre="L'enjeu de la réduction des emballages" date="2020 07 7" journal="Les Echos, no. 23235">
Les acteurs de la livraison de repas, des plateformes aux restaurants virtuels, devraient avoir un été studieux. Reçus le mardi 30 juin au ministère de la Transition écologique et solidaire, en même temps que des start-up spécialistes des packagings réemployables et des ONG, ils ont été priés de se pencher de plus près sur la question des emballages en prévision de l'entrée en vigueur progressive des mesures prévues dans la loi antigaspillage. Il leur a été demandé de rendre une feuille de route d'ici à début octobre sur leur stratégie pour viser le zéro déchet. Pour cet univers, par définition gourmand en contenants, les réponses ne sont pas si simples à donner. D'autant qu'à côté des objectifs de réduction figurent aussi ceux de réutilisation et de recyclage, via un système de consigne.   Des obligations dès janvier prochain Mais les entreprises planchent déjà sur le sujet. Parmi les démarches indolores, l'option sans couverts associée à la commande est d'ores et déjà mise en avant par la majorité des acteurs. Le 1er janvier prochain, fourchettes, couteaux mais aussi assiettes, gobelets et pailles en plastique à usage unique seront de toute façon bannis.   Chez Deliveroo, les restaurants peuvent se fournir dans une boutique d'emballages autour de matériaux comme le papier, le carton ou les matières biosourcées. Uber Eats a, lui, noué un partenariat avec Dinovia, distributeur de packagings alimentaires spécialisés pour la vente à emporter, afin de donner accès, à tarif préférentiel, à 250 références qui se veulent écoresponsables. Tandis que Nestor travaille depuis quelques mois sur la manière de modifier ses emballages en papier kraft et plastique recyclé et recyclable pour passer au biodégradable.   Du côté de FoodChéri, un salarié se consacre depuis deux ans au sujet des emballages. L'entreprise est passée, il y a un an, à des packagings entièrement recyclables, dont une partie est en matière recyclée, et a réduit de 20 % ses déchets. Les livraisons en entreprise se font dans des sacs consignés.  « Il faut aller plus loin. Nous travaillons sur le zéro déchet. Nous proposons désormais à nos entreprises clientes une solution de packagings consignés. C'est le bon moment pour accélérer le mouvement. Après le confinement, les gens veulent du changement »,  remarque sa cofondatrice Julia Vernin. Mais récupérer et laver a un coût, cela double le prix du packaging.   Plus facile en B to B Même type de démarche chez Frichti. La jeune pousse a une équipe qui s'y consacre. Elle a notamment longuement travaillé sur une barquette en bagasse de canne à sucre, mais juge que l'industrie est en retard pour proposer des solutions.  « Le sujet est central pour nous. Le meilleur packaging est celui qui n'existe pas et nous regardons de près la consigne pour les cantines virtuelles des entreprises »,  précise sa cofondatrice, Julia Bijaoui.   Si, pour le B to B, des solutions de consigne commencent à se dessiner, la tâche va être bien plus ardue pour les particuliers. Economiquement, le livreur ne pourra pas repasser récupérer les contenants.   Les acteurs ayant participé à la réunion du 30 juin espèrent que des solutions collectives finiront par émerger, impliquant les collectivités locales pour des tests dans une zone géographique, voire la grande distribution.   Cl. B.
</article>
<article titre="Livraison de repas : l'enjeu de la réduction des emballages" date="2020 07 7" journal="Les Echos (site web)">
Les acteurs de la livraison de repas, des plateformes aux restaurants virtuels, devraient avoir un été studieux. Reçus le mardi 30 juin au ministère de la Transition écologique et solidaire, en même temps que des start-up spécialistes des packagings réemployables et des ONG, ils ont été priés de se pencher de plus près sur la question des emballages en prévision de l'entrée en vigueur progressives des mesures prévues dans la loi anti-gaspillage. Il leur a été demandé de rendre une feuille de route d'ici à début octobre sur leur stratégie pour viser le zéro déchet. Pour cet univers, par définition gourmand en contenants, les réponses ne sont pas si simples à donner. D'autant qu'à côté des objectifs de réduction figurent aussi ceux de réutilisation et de recyclage, via un système de consigne.   Des obligations dès janvier prochain Mais les entreprises planchent déjà sur le sujet. Parmi les démarches indolores, l'option sans couverts associée à la commande est d'ores et déjà mise en avant par la majorité des acteurs. Le 1er janvier prochain, fourchettes, couteaux mais aussi assiettes, gobelets et pailles en plastique à usage unique seront de toute façon bannis.   Chez Deliveroo, les restaurants peuvent se fournir dans une boutique d'emballages autour de matériaux comme le papier, le carton ou les matières biosourcées. Uber Eats a, lui, noué un partenariat avec Dinovia, distributeur de packagings alimentaires spécialisés pour la vente à emporter afin de donner accès, à tarif préférentiel, à 250 références qui se veulent écoresponsables. Tandis que Nestor travaille depuis quelques mois sur la manière de modifier ses emballages en papier kraft et plastique recyclé et recyclable pour passer au biodégradable.   Du côté de FoodChéri, un salarié se consacre depuis deux ans au sujet des emballages. L'entreprise est passée il y a un an à des packagings entièrement recyclables dont une partie est en matière recyclée et a réduit de 20 % ses déchets. Les livraisons en entreprise se font dans des sacs consignés.    « Il faut aller plus loin. Nous travaillons sur le zéro déchet. Nous proposons désormais à nos entreprises clientes une solution de packagings consignés. C'est le bon moment pour accélérer le mouvement. Après le confinement, les gens veulent du changement »,  remarque sa cofondatrice Julia Vernin. Mais récupérer et laver a un coût, cela double le prix du packaging.   Plus facile en B to B Même type de démarche chez Frichti. La jeune pousse a une équipe qui s'y consacre. Elle a notamment longuement travaillé sur une barquette en bagasse de canne à sucre, mais juge que l'industrie est en retard pour proposer des solutions.   « Le sujet est central pour nous. Le meilleur packaging est celui qui n'existe pas et nous regardons de près la consigne pour les cantines virtuelles des entreprises »,  précise sa cofondatrice, Julia Bijaoui.   Si, pour le B to B, des solutions de consigne commencent à se dessiner, la tâche va être bien plus ardue pour les particuliers. Economiquement, le livreur ne pourra pas repasser récupérer les contenants.   Les acteurs ayant participé à la réunion du 30 juin espèrent que des solutions collectives finiront par émerger, impliquant les collectivités locales pour des tests dans une zone géographique, voire la grande distribution.   Clotilde Briard
</article>
<article titre="Après la crise, quel avenir pour l'emballage ?" date="2020 06 30" journal="Néorestauration, no. 0586">
Un mois et demi après avoir annoncé une augmentation de production de  « 30 % des emballages plastiques dans les secteurs de l'alimentaire, de l'hygiène et des détergents »  et provoqué une rapide levée de boucliers des associations écologistes et une mise au point de la secrétaire d'État Brune Poirson sur le maintien du calendrier de la loi sur l'économie circulaire, Elipso calme le jeu. Françoise Andrès, présidente de cette association représentant des fabricants d'emballages souples et rigides, rappelle, dans un communiqué daté du 25 mai, que  « pour l'ensemble des emballages plastiques, tous secteurs confondus, la production 2020 ne dépassera pas la production 2019. La hausse en trompe-l'oeil de certaines familles d'emballages (alimentaire à destination de la grande distribution et hygiène) en début de confinement ne compensera pas la baisse de l'ensemble des emballages plastiques ».  Chez les fournisseurs, après le choc d'un arrêt brutal de la restauration, de l'événementiel, des parcs d'attraction..., il a fallu réagir vite. Jusqu'à faire une pause sur la problématique du plastique ?  « Jusqu'à début mars, il n'en fallait pas du tout. Dès qu'on avait un emballage avec un peu de plastique, c'était la catastrophe. À partir du 14 mars, ce n'était plus grave... En raison de l'arrêt de certaines entreprises, il fallait des emballages. On était plus dans une problématique d'approvisionnement que d'environnement »,  observe Frédéric Leduc, directeur commercial de Gault &amp; Frémont, fabricant français d'emballages en papier et carton et gros faiseur de sacs à pain.  « Une chaîne de boulangerie qui réfléchissait depuis trois mois à supprimer les fenêtres (en plastique) a gelé le projet. La fenêtre va rester et la question du moment est plutôt de savoir comment fermer le sac. La garantie sanitaire est devenue la problématique numéro 1. » PAS DE RETOUR EN ARRIÈRE Les exigences d'hygiène vont-elles durablement impacter la conception des emballages ?  « Ce n'est pas l'inflexion que donnent les grands donneurs d'ordres, poussés par la maturité du grand public qui dénigre cette matière. McDonald's a arrêté les pailles en plastique. Savéol a opté pour le carton pour emballer ses tomates, aucun ne fera machine arrière. D'autant qu'il existe des solutions fiables en carton, papier, kraft... qui peuvent être recyclées jusqu'à sept fois,  affirme Jean-Pascal Salaün, président du directoire du groupe Lebhar, spécialisé dans le carton.  Les lobbyistes du plastique ont joué leur jeu. Mais les comportements envers cette matière sont les mêmes qu'avant la crise. La tendance vers le naturel, le biodégradable et recyclable va s'accélérer. » Cette réflexion, amorcée depuis plusieurs années chez Solia, est en train de faire basculer la part du plastique sous la barre des 50 %.  « La tendance s'inverse. On est à 40 % de plastique pour 60 % de matières naturelles »,  assure Jean-Claude Soulier, PDG de Solia, créateur et spécialiste de packaging pour la restauration, qui, après un mouvement de baisse importante d'activité, a vécu  « une reprise forte et une demande très hétérogène. Sur certains produits, comme les plateaux-repas, avec l'essor de la vente à emporter et de la livraison, la demande a été multipliée par 5 ou 7, voire 10. Faute de stocks, on est allé chercher ailleurs... Mais le leitmotiv n'est pas de faire du plastique. Ce n'est d'ailleurs pas ce que nous demandent nos clients, dont la préférence va vers les produits naturels,  remarque-t-il.  On ne bannit pas le plastique, mais il doit être recyclable et produit avec des matériaux recyclés. Il faut s'interroger sur tout le cycle de vie d'un produit. À quoi bon privilégier un PLA (plastique d'origine végétale) compostable industriellement, mais non biodégradable, qui va nécessiter d'utiliser des terres arables, pour faire pousser du maïs OGM, truffé de pesticides, envoyé en Chine, transporté en conteneurs... Autant utiliser un PET (plastique recyclable d'origine pétrolière) qui aura dix ou douze vies et qui pourrait être recyclé. »  L'enjeu est de taille pour les fabricants d'emballages qui, au sortir de la crise sanitaire, voient émerger un nouveau marché. Les attentes en matière d'hygiène, le télétravail... vont bousculer les lignes.  « La vente à emporter, le click and collect et la livraison à domicile ne compenseront pas le manque à gagner de la restauration, mais le marché grossit. Cela va durer, même s'il est difficile de le quantifier »,  estime Jean-Claude Soulier.    REVOIR LES MODES OPÉRATOIRES Le cartonnier Gault &amp; Frémont qui, depuis le début de l'année, concoctait une offre pour le click and collect, s'est résolu à accélérer ses lancements et ses référencements chez les distributeurs pour compenser de lourdes pertes sur la pâtisserie. Historiquement positionnée sur les indépendants, les chaînes de boulangerie et le snacking, l'entreprise pénètre le marché de la restauration.  « Les grands comptes accélèrent et nous découvrons un nouveau marché »,  indique Frédéric Leduc, référencé par Transgourmet non plus seulement pour la boulangerie, mais aussi pour la restauration.  « On ignore ce que sera l'impact du click and collect, mais c'est un peu la même histoire que la boulangerie et l'arrivée du snacking. Il y a ceux qui sont morts et ceux qui ont su se reconstruire... » Durement touché aussi, le français Comatec a pris le taureau par les cornes. Dans un premier temps aux petits soins des Ehpad, pharmacies et établissements hospitaliers pour amortir la chute d'activité, l'entreprise s'est penchée vers les restaurateurs avec une sélection de conditionnements naturels et adaptés, et notamment une offre dédiée à l'univers gastronomique.  « Et nous avons communiqué pour fidéliser, ce qui nous a aussi permis de capter de nouveaux clients »,  explique Tatiana Rumeau, qui vient de créer Planetscore, le premier référentiel dans l'univers du packaging pour être transparent sur les matières utilisées. Et ici aussi, l'argument du made in France et des produits recyclables (bambou, bagasse, bois, carton, métal...) ont rassuré.  « Le plastique, ce n'est pas diabolique. Il doit simplement être recyclé et recyclable. Et il faut l'expliquer. Comme il faut expliquer aux restaurateurs qu'ils doivent adapter leur offre à la vente à emporter et communiquer sur les réseaux sociaux. Ils devront revoir leur mode opératoire, sans oublier leur métier premier : proposer une cuisine de qualité »,  conseille-t-elle. De nouvelles habitudes que l'on voudrait proprement emballées...
</article>
<article titre="Bientôt des crêpes et des galettes à emporter" date="2020 06 27" journal="Ouest-France  Redon ; Rennes Nord ; Rennes Sud ; Rennes ; Rennes Est ; Rennes Ouest">
Rouverte depuis le 2 juin, la crêperie Au Cottage va proposer de la vente à emporter à partir du mercredi 1er juillet.   «  Je ne l’ai pas proposé avant car je n’étais pas équipé,  explique Éric Noslier, 52 ans, propriétaire de la crêperie depuis 2011.  Cela demande un investissement.  »  Investissement financier, mais aussi en temps pour trouver le bon packaging. Car la galette est un produit difficile à véhiculer.  «  J’ai trouvé une solution avec la bagasse. C’est un produit dérivé de la canne à sucre.  »   Les assiettes carrées sont faites avec ce matériau qui absorbe l’humidité, passe au micro-ondes et au four traditionnel. Elles sont complétées avec des couverts en bois et des sacs en papier.   «  Je n’ai pas eu de demande particulière pour faire de la vente à emporter, mais j’espère que cela compensera un peu la perte de chiffre d’affaires  »  .   Comme beaucoup de restaurateurs, Éric Noslier a dû réduire sa capacité d’accueil, passant de 38 couverts en salle à une vingtaine.   Ce qui a entraîné la mise au chômage partiel de deux salariés sur trois.  «  L’emporté, c’est aussi donner la possibilité à des gens qui ne peuvent pas rester au restaurant de déguster des galettes.  »  Crêperie Au Cottage,  place de l’Église, tél. 02 23 61 08 93.  Cet article est paru dans Ouest-France
</article>
<article titre="Le retour d'Albioma dans le SBF 120 met en lumière ses vertus défensives" date="2020 06 18" journal="L'AGEFI Quotidien">
Albioma s'apprête à gagner davantage de visibilité auprès des investisseurs internationaux. Lundi, le producteur d'électricité retrouvera sa place dans l'indice SBF 120 des valeurs les plus en vue de la Bourse de Paris, dont il avait été évincé en mars 2011, après trois années de présence. A l'époque, Albioma s'appelait encore Séchilienne-Sidec, reflet de la fusion réalisée en 2001 entre Séchilienne, ancienne filiale d'Air Liquide, et Sidec, dénomination initiale du groupe lors de sa constitution en 1982. Albioma a reconquis le conseil scientifique des indices d'Euronext en réussissant sa transformation en un acteur majeur de la transition énergétique dans la France d'outre-mer et au Brésil. Ces dix dernières années, le groupe s'est efforcé de réduire la quantité de charbon utilisée pour produire son électricité. En contrepartie, il a investi dans des centrales photovoltaïques mais surtout dans la valorisation énergétique de la bagasse, un résidu fibreux de la canne à sucre. En 2019, la production d'Albioma a représenté l'équivalent de la consommation annuelle d'électricité de 2,6 millions de personnes. Et 67% de cette électricité provenaient d'énergies renouvelables, contre un taux de 19,9% en 2016. Les dirigeants espèrent désormais qu'Albioma pourra produire 80% de son électricité à partir d'énergies renouvelables et cessera définitivement d'utiliser du charbon pour faire fonctionner ses centrales thermiques d'ici la fin 2023. Une visibilité sur les revenus et les profits appréciée Surtout, Albioma parvient à «verdir» ses activités tout en suivant «une trajectoire de croissance rentable», notent les analystes de Midcap Partners. Entre 2015 et 2019, les ventes du groupe ont bondi de 45%, à 505,7 millions d'euros, son excédent brut d'exploitation (Ebitda) a accéléré de 53%, à 182,9 millions et sa marge d'Ebitda a gagné 1,9 point de pourcentage, à 36,2%. Les investissements réalisés par le groupe pour la mise en conformité de ses centrales ou le démarrage de nouveaux sites donnent également de la visibilité sur le chiffre d'affaires futur. Après avoir engagé pour 880 millions d'euros d'investissements entre 2013 et 2019, l'entreprise compte investir 450 à 650 millions d'euros supplémentaires entre 2020 et 2023, afin d'accélérer son développement à l'international. «Albioma pourrait étendre ses activités dans d'autres pays producteurs de canne à sucre et plus particulièrement en Asie du Sud-Est et en Amérique latine ou centrale», anticipe Nicolas Royot, co-responsable de l'analyse financière chez Portzamparc. Les investisseurs sont aussi séduits par la capacité d'Albioma à résister aux chocs exogènes. L'action s'adjuge 29,4% depuis le début de l'année, portant la capitalisation boursière à 1,05 milliard d'euros, alors que se termine un premier semestre marqué par la propagation du nouveau coronavirus à l'échelle planétaire. Albioma est «immunisé contre le Covid-19», constate Midcap Partners, la vente de son électricité étant encadrée par des primes fixes et des contrats de long terme, signés pour la plupart avec EDF pour une durée de 25 à 30 ans et à des prix garantis. En avril, à l'occasion de la publication de son chiffre d'affaires du premier trimestre - inscrit à 130,9 millions d'euros, en croissance de 17% hors prix des combustibles - Albioma a confirmé ses prévisions annuelles. Pour l'exercice en cours, les dirigeants anticipent un Ebitda compris entre 200 et 210 millions d'euros et un résultat net compris entre 48 et 54 millions d'euros. Une valeur plébiscitée par les analystes Cette visibilité sur les revenus et les profits permet aux dirigeants de maintenir une politique de dividende généreuse. Le taux de croissance moyen annuel du dividende dépasse 5% selon les montants versés entre 2016 et 2020, et tout actionnaire conservant ses actions pendant au moins deux ans bénéficie d'un dividende majoré de 10%. Le titre Albioma est plébiscité par les intermédiaires financiers. Sur les sept analystes couvrant la valeur, six sont à «acheter», selon Factset, et un est à «neutre». Il faut même remonter à février 2016 pour trouver un analyste conseillant de «vendre» Albioma. La valorisation du titre reste toutefois modeste, faisant ressortir un ratio valeur d'entreprise sur Ebitda estimé à 9 pour 2020 par Oddo BHF. Ce multiple représente une décote de 10% par rapport à ceux affichés par les concurrents. «La décote de valorisation présentée par Albioma n'est pas justifiée, au regard de la pertinence de son modèle économique, de ses efforts sur les critères ESG (environnementaux, sociaux et de gouvernance) et notamment de la transition en cours vers davantage d'énergies renouvelables», résume Nicolas Royot. Son retour imminent dans le SBF 120 pourrait permettre à Albioma de dévoiler ses atouts à une part plus large du marché et d'attirer de nouveaux investisseurs. Le capital de la société n'étant plus verrouillé depuis 2015, l'entreprise pourrait devenir la cible d'une offre de rachat, surtout lorsqu'elle aura définitivement cessé d'utiliser du charbon pour produire son électricité, d'ici deux à trois ans. La sortie du charbon ne rendra la valeur que plus attrayante, estiment des opérateurs de marché. La part du capital revenant au public ressort actuellement à 69,6%.
</article>
<article titre="Des repas à emporter sans plastique jetable" date="2020 05 30" journal="La Nouvelle République du Centre-Ouest  LOIR ET CHER">
Difficile d'adopter le carton de la pizza pour livrer des plats mitonnés à une clientèle de restaurants fermés par obligation sanitaire. Dans l'urgence, parce que cela semblait plus pratique ou moins cher, nombre d'établissements ayant mis en place pour la première fois une offre de repas à emporter, ont opté pour des barquettes, verrines et autres petits pots de sauce en plastique jetable. D'autres ont cependant su trouver des solutions plus durables et respectueuses de l'environnement, sans rien concéder à la qualité ni à la présentation de leurs plats. « Dans l'optique du zéro déchet qui est la mienne depuis toujours, j'ai proposé aux clients un système de consigne, avec des plats à emporter dans des boîtes en verre, raconte Fanny Goumard, à la tête du restaurant et salon de thé Miss Goumard à Blois. J'ai acheté 20 boîtes en verre pour commencer, et l'adhésion de la clientèle a été immédiate, j'ai très vite dû en racheter autant ! » « Des contenants pratiques esthétiques, écologiques » L'incertitude de la période l'a dissuadé d'investir plus pour le moment. Mais une fois l'organisation bien rodée pour l'enlèvement des repas, Fanny Goumard a décidé d'accepter les contenants apportés par les clients eux-mêmes.  « Ça prend un peu de temps, mais en répartissant bien les rendez-vous sur la période, c'est tout à fait faisable. »  Table étoilée, le Lion d'or à Romorantin n'avait pas vraiment imaginé se mettre aussi rapidement à la vente à emporter. La décision prise le 20 avril parce que le chef Didier Clément n'en pouvait plus de ne rien faire,  « la formule a été trouvée en dix jours, témoigne Marie-Christine Clément,  et il n'était pas question d'utiliser du plastique !  » Pas seulement pour une question d'image, mais parce que l'hôtel du Lion d'or est engagé depuis plusieurs années dans une démarche durable, privilégiant les matériaux recyclables.  « Nous voulions des contenants qui soient pratiques, esthétiques et écologiques », résume Marie-Christine Clément, qui a trouvé son bonheur après deux jours de recherche sur Internet : de la vaisselle jetable mais 100 % compostable, en bagasse, un matériau composé de fibres de canne à sucre. Bols profonds, coupelles ovales, des formes classiques et élégantes convenant au chef étoilé pour mettre ses plats en valeur ont donc été adoptées, le tout livré dans des sacs isothermes consignés. Alors que de nombreux restaurateurs envisagent de maintenir la vente à emporter après la réouverture de leurs établissements, vaisselle consignée ou compostable prendront peut-être une plus large place. Surtout si les consommateurs s'avèrent sensibles à cette attention pour l'environnement.   Catherine Simon
</article>
<article titre="Jubiles veut démocratiserle kombucha" date="2020 05 28" journal="LSA, no. 2605">
Antoine Martin aurait pu continuer à creuser le sous-sol parisien pour allonger la ligne 14 du métro. François Verdier aurait pu passer sa carrière à dépolluer des sols. Ces deux trentenaires qui se sont rencontrés à l'ENSG, l'école de géologie de Nancy, ont une même passion : l'alimentation saine. « Ma mère m'y a sensibilisé très tôt », explique Antoine Martin. Après un long cheminement, les deux camarades créent la société Jubiles, rebaptisée Biomère. La marque Jubiles, d'abord dédiée aux jus bio, a été réorientée vers « la production d'une gamme de kombuchas », poursuit-il.  Levée de fonds de 400 000 €   Depuis sa création en 2017, la start-up a déménagé deux fois et envisage une troisième adresse pour s'agrandir. Elle est installée à Joinville-le-Pont (94) dans un atelier pouvant produire 30 000 litres par mois. Biomère se fournit auprès de producteurs bio, voire en biodynamie comme chez Dynamis, grossiste en fruits et légumes à Rungis, éthiques comme Ethiquable pour le sucre de canne, et les Jardins de Gaia pour les thés et infusions. Labellisée Bio Entreprise durable, Biomère a choisi un emballage végétal compostable en milieu industriel, produit par Lyspackaging, à partir de bagasse.   Côté financement, plusieurs « fées » s'intéressent à la start-up, qui vient de finaliser sa deuxième levée de fonds - 400 000 € rassemblés auprès de Paris Business Angels, de Club Invest IDF, d'IntellAgri et du fonds de Purpan (école d'ingénieurs de Toulouse). Jubiles est commercialisée dans 400 magasins bio et en snacking dans 400 autres. François Verdier se consacre à la production, à la logistique et à la R &amp; D, Antoine Martin au développement commercial et à la gestion de la société.    Reste que le kombucha doit gagner en notoriété. Ses vertus ne sont pas encore reconnues scientifiquement. En plein boom en Asie, aux États-Unis, en Australie et en Europe du Nord et de l'Est, il est apprécié pour son côté désaltérant et pour améliorer la digestion et booster le système immunitaire, en raison de sa teneur en micro-organismes vivants et molécules actives tels que les acides organiques. Biomère finance un doctorat conduit à l'UMR « Procédés alimentaires et microbiologiques » de l'université de Bourgogne-Franche-Comté. « Elle permettra ainsi de caractériser les souches pour comprendre les interactions entre micro-organismes, et améliorer le processus de fabrication. »     sylvie leboulenger
</article>
<article titre="Conseil Les centrales à pleine vapeur Albioma" date="2020 05 23" journal="Investir-Le Journal des finances, no. 2420">
L'électricien en outre-mer est la valeur, dans notre sélection, qui offre le meilleur parcours boursier depuis le début de l'année. Son gain est de 23 %, malgré un décrochage de 33 % entre les 5 et 19 mars. Les performances opérationnelles du groupe sont toujours là. Au premier trimestre, le chiffre d'affaires d'Albioma a augmenté de 16 %, à près de 116 millions d'euros, permettant de renouveler les objectifs annuels : un Ebitda (excédent brut d'exploitation) de 200 à 210 millions, contre 183 millions l'an dernier, et un bénéfice net de 48 à 54 millions (44 millions en 2019). Ces prévisions n'intègrent pas, toutefois, un éventuel effet Covid-19. Biomasse Aujourd'hui, « toutes nos centrales tournent et dans des conditions très proches ou identiques à ce qu'elles étaient avant la crise » , souligne Frédéric Moyne, le directeur général. Aux Antilles ou à La Réunion, la disparition des touristes pèse sur la demande, mais « la consommation résidentielle reste bonne » . De plus, les centrales de cogénération d'Albioma sont dites « de base » , ayant donc priorité sur la production au fioul d'EDF quand la demande diminue. « Dans les zones où la campagne sucrière a commencé, comme en Guadeloupe et en Martinique, nos unités fournissent aussi de la chaleur aux usines sucrières » , poursuit Frédéric Moyne. Selon les saisons, les centrales thermiques d'Albioma, proches des sucreries, brûlent du charbon ou de la bagasse, le résidu de la canne à sucre, pour produire à la fois l'électricité et l'énergie nécessaires à la fabrication du sucre. Du moins les plus anciennes. La centrale de Galion 2, mise en service en Martinique fin 2018, n'utilise que de la biomasse, bois ou bagasse, ainsi que les trois sites du Brésil, où la canne à sucre se récolte toute l'année. Albioma a entrepris de convertir ses centrales charbon-bagasse à la seule biomasse, part importante du programme d'investissement de 450 à 650 millions prévu en 2020-2023. A cet horizon, la biomasse et le solaire, voire la géothermie, devront représenter 80 % au moins du mix énergétique du groupe, pour un objectif final de 100 %, contre 36 % en 2013 et 67 % l'an dernier. Les travaux de conversion ont été engagés pour 70 millions à la centrale du Moule, en Guadeloupe, et auraient dû être achevés en fin d'année, sans l'arrêt imposé par le confinement. C'est ce risque de retard dans les chantiers et les opérations de maintenance, avec une probabilité plus forte, ensuite, d'incidents techniques, qui préoccupe surtout le directeur général, outre la sécurité des salariés. Mais « la crise peut aussi susciter des opportunités pour nous » , assure Frédéric Moyne, notamment dans le solaire et au Brésil, où des sucriers fragilisés pourraient devoir céder leur centrale de cogénération à des tiers.   Notre conseil Acheter Des performances encore solides devraient être présentées, bénéficiant notamment du fonctionnement, en année pleine, des nouvelles centrales de Galion 2 et de Saint-Pierre, à La Réunion.   C. L. C.
</article>
<article titre="Emballage à Montréal : la pulpe de canne" date="2020 05 14" journal="Journal le Guide (Cowansville, QC) (site web)">
La pulpe de canne, aussi nommée la bagasse, est un matériau issu des résidus récupérés dans la canne à sucre après l'extraction du jus. Cette matière s'applique pour plusieurs utilisations, dont la production d'énergie. Dans certaines régions, la bagasse est récupérée pour être mise en fermentation pour que son degré d'alcool augmente et donne du rhum. L'utilisation la plus récente est certainement la fabrication d'emballages alimentaires. La bagasse représente en effet une excellente alternative au plastique pour les emballages à Montréal. Si la pulpe de canne n'est pas récupérée, elle est jetée ou bien incinérée.   La fabrication des emballages en pulpe de canne    La fabrication des emballages en pulpe de canne débute avec les particules non comestibles de la canne à sucre. Ces particules sont majoritairement composées de cellulose. Cette fibre naturelle est récupérée après le broyage de la canne et l'extraction du jus. Ensuite, la pulpe est mélangée avec de l'eau et un liant naturel avant d'être thermomouler. De nombreuses formes sont possibles, dont des assiettes, des bols, des plats pour emporter, des couverts, etc.   La pulpe de canne, un produit écologique    En effet, la pulpe de canne est un produit qui ne nuit pas à l'environnement et elle représente une option très intéressante pour remplacer le plastique dans notre quotidien.   Tout d'abord, les produits en bagasse sont 100 % biodégradables. Dans des conditions naturelles, les contenants se décomposent sur une période d'un an. Dans des conditions contrôlées comme un compost, la bagasse est à 90 % dégradée après seulement trois mois.   Ensuite, la canne à sucre d'où provient la pulpe est une espèce végétale dont la croissance est très rapide. Comme d'autres matières premières comme le bambou, la matière première se régénère facilement et rapidement.   Par ailleurs, les contenants fabriqués en bagasse peuvent facilement être jetés à la poubelle. Qu'ils se retrouvent au dépotoir ou à l'incinérateur, leur impact sur l'environnement sera nul. En effet, même la combustion de la pulpe de canne ne produit aucun déchet toxique pour l'humain ou bien nocif pour l'environnement.   Les caractéristiques physiques des emballages en bagasse   En plus d'être écologiques, les contenants en bagasse possèdent de nombreuses caractéristiques intéressantes pour le domaine de l'emballage alimentaire.   Tout d'abord, ce matériau est très robuste. Des couverts pratiques et solides peuvent facilement être fabriqués en bagasse.   Ensuite, les contenants sont très résistants au froid. Les plats préparés se congèlent aisément sans douter de la fraîcheur des aliments.   De plus, la bagasse supporte de très grandes chaleurs. Les aliments contenus dans les plats en bagasse peuvent facilement des êtres chauffer au four à micro-onde (jusqu'à 750 watts). Le contenant peut même être placé dans un four traditionnel jusqu'à une température jusqu'à 400 °F.  Cet article est paru dans Journal le Guide (Cowansville, QC) (site web)
</article>
<article titre="Vos actions Les valeurs pétrolières de nouveau dans la tourmente" date="2020 04 1" journal="Mieux Vivre Votre Argent, no. 454">
En ce début d'année, les valeurs pétrolières ne sont pas à la fête. Les cours du pétrole ont dévissé, touchés par le coronavirus, désormais appelé Covid-19. Parallèlement, des investisseurs et les ONG accroissent leur pression sur ces grands groupes énergétiques pour qu'ils accélèrent leur transition énergétique.   Des modèles économiques contraints d'évoluer L'alerte au coronavirus et la mésentente entre les pays producteurs de l'or noir ont porté un coup fatal aux marchés pétroliers. Depuis le début de l'année, le prix du baril de Brent a plongé de 50 %, entraînant dans son sillage les principaux titres du secteur. Total a dévissé de 40 %, tandis que Royal Dutch Shell (44 %), BP (34 %), Repsol (43 %), Eni (43 %) ne sont pas mieux lotis. Force est de constater que la propagation du Covid-19 et les mesures de confinement qu'il engendre dans de nombreux pays, ont des conséquences sur la demande d'hydrocarbures. Et si la Russie ne revient pas vite à la table des négociations avec les pays membres de l'Opep, les cours du pétrole pourraient rester durablement dépréciés. Quatre ans après la dernière onde de choc - le prix du baril avait été divisé par quatre entre juin 2014 et janvier 2016 pour sombrer à 27 dollars - , le secteur traverse une nouvelle tempête. Mais le Covid-19 n'explique pas tout. La donne a changé depuis 2015 et la COP 21  (voir Paroles d'expert p. 29 ). Les énergies fossiles, responsables d'émissions de CO2, sont pointées du doigt. La conférence internationale sur le climat de Paris souhaite limiter le réchauffement climatique en deçà de 2 degrés Celsius, voire à 1,5 degré, par rapport à l'ère préindustrielle, à horizon 2100, et faire émerger une économie bas-carbone. Pour tenir l'objectif, il faudra renoncer à exploiter une grande partie des réserves de pétrole, de gaz et de charbon. Plus question de produire à outrance pour compenser la baisse des cours ou d'explorer tous azimuts.  « Avec la transition énergétique, nous assistons à un changement de paradigme et à un impact sur toute la chaîne de valeurs dans l'industrie pétrolière et gazière » , explique Jean-Guillaume Péladan, gérant et directeur de la stratégie Environnement chez Sycomore AM.   La part de responsabilité des compagnies énergétiques dans le réchauffement climatique est clairement soulignée. Joignant les actes à la parole, les ONG et des investisseurs internationaux mènent une première campagne de désinvestissement chez les producteurs de charbon, le plus fort émetteur en CO2, à travers des centrales thermiques. L'industrie du pétrole et du gaz en fait aussi les frais. En 2019, le fonds norvégien du pétrole, premier fonds souverain mondial, a annoncé qu'il allait exclure les purs producteurs de pétrole de son portefeuille, qui s'élève aujourd'hui à plus de 1 000 milliards d'euros.   Si certains investisseurs - fonds de pension, institutionnels, sociétés de gestion - , ont décidé de se détourner de cette industrie, d'autres ont opté pour l'engagement actionnarial. «  Dans nos investissements pour nos actifs généraux, nous restons proches de la structure de l'économie d'un pays et de la composition des grands indices. Nous accompagnons la transition et incitons les entreprises, par le dialogue, à faire évoluer leur modèle »,  souligne Philippe Brossard, directeur de la recherche économique, ISR et développement d'AG2R La Mondiale.  « En 2018, nous avons posé des questions écrites à toutes les assemblées des sociétés du CAC 40 quant à leur adhésion à l'initiative internationale des Science Based Targets (SBT), ayant pour but d'amener les entreprises vers une économie moins énergivore. Total aurait commencé à y réfléchir »,  se félicite Denis Branche, directeur général délégué de Phitrust.   Certains ont opéré cette reconversion. Dong Energy en est l'illustration la plus aboutie. Au début des années 2000, la compagnie publique danoise du secteur pétrolier et gazier a pris pied dans la production et la distribution d'électricité. En 2017, elle a vendu la branche hydrocarbures et est devenue Ørsted. L'éolien est son credo et les investisseurs la plébiscitent. Depuis son introduction en Bourse en 2016, son cours a été multiplié par plus de 3.   Cap sur les énergies renouvelables En revanche, les grands groupes intégrés peinent à montrer leur bonne volonté à changer de modèle. Certains ont réduit dans le mix énergétique leur empreinte carbone en privilégiant le gaz, moins émetteur, mais cela reste une énergie fossile. Quant aux énergies renouvelables, elles sont pour le moment noyées dans la masse. Dans une étude publiée à la fin de janvier, l'Agence internationale de l'énergie (AIE) note que les grandes compagnies n'auraient consacré qu'environ 2 milliards d'euros par an depuis 2015 aux énergies renouvelables. Ce qui ne représente qu'à peu près 1 % de leurs investissem ents. Avec 5 %, l'italien Eni, le norvégien Equinor (ex-Statoil) et le français Total se distinguent toutefois. Depuis l'acquisition de l'américain Sunpower dans l'énergie solaire en 2011, Total a multiplié les achats : Vents d'Oc (énergies renouvelables), Saft (batteries durables), Direct Energie (électricité et gaz), GreenFlex (conseil en transition environnementale), Eren Renewable Energy (énergies renouvelables). Soit près de six milliards d'euros en cinq ans. Le groupe ambitionne de porter cette part à 10 % de ses investissements et d'atteindre une capacité mondiale de 25 gigawatts en 2025, l'équivalent du parc hydraulique en France.   « La dépendance aux énergies fossiles va s'éteindre grâce à l'arrivée massive des nouvelles technologies qui intègrent les enjeux climatiques. Les compagnies pétrolières vivent encore de leur rente, mais elles réfléchissent activement à l'après-pétrole »,  perçoit Suzanne Senellart, gérante des stratégies environnementales actions de Mirova. Les synergies ne manquent pas. L'expertise dans l'exploitation off shore est transposable à la mise en place des parcs éoliens off shore. Les stations-services sont en mesure d'accueillir des points de recharge pour les véhicules électriques. En janvier, Total a remporté le marché de la région d'Amsterdam et installera jusqu'à 20 000 bornes.  « Au départ, les nouvelles technologies étaient considérées comme des actifs innovants, donc risqués, financés majoritairement en capital, avec un coût de financement énorme. Puis il y a eu la baisse des coûts et des taux d'intérêt, et de la demande de rentabilité »,  raconte Jean-Guillaume Péladan.   On assiste à l'arrivée de nombreux développeurs et opérateurs depuis vingt ans. Les groupes pétroliers intégrés trouvent face à eux les sociétés de services aux collectivités (« utilities »), souvent investies dans la transition énergétique depuis longtemps grâce à l'énergie hydraulique, première source d'énergie renouvelable dans le monde. Elle assure 10 % de la production d'électricité en France. EDF et Engie, les champions nationaux, exportent leur savoir-faire et se développent dans les autres énergies vertes. Cette course a fait évoluer le statut des utilities. «  Les nouveaux acteurs ne sont plus des valeurs défensives, offrant un rendement élevé et certain. Leurs niveaux de valorisation les font entrer dans la catégorie des valeurs de croissance. Les investissements priment sur les actionnaires. Ainsi, Ørsted, devenu utilities, distribue entre 20 et 40 % des bénéfices et investit le reste, alors que le taux de distribution d'Engie est encore de 70 à 80 % »,  souligne Auguste Deryckx Lienart, analyste utilities chez AlphaValue.   En France, ils s'appellent Ecoslops (carburant à partir des résidus pétroliers), Albioma (biomasse et solaire,  voir encadré),  Voltalia (énergie renouvelable), Neoen (énergie renouvelable)... Des opérateurs plus anciens, comme Air Liquide, Veolia, Suez Environnement, interviennent également dans la production d'énergie via la cogénération. Tandis que Nexans et Eiffage fournissent des équipements à cette filière. Le domaine des énergies renouvelables fourmille d'idées et d'initiatives.
</article>
<article titre="Emballages, vers la fin de l'usage unique ?" date="2020 04 1" journal="Néorestauration, no. 0584">
Le virage en a surpris plus d'un. Lorsqu'ils ont retenu en décembre dernier l'obligation pour la restauration rapide de mettre en oeuvre dès 2023 de la vaisselle réutilisable pour la consommation sur place, les députés ont pris une industrie de cours, ses opérateurs comme ses fournisseurs.  « La loi sur l'économie circulaire apporte un changement d'orientation majeur, qui vise non plus à recycler plus et mieux le plastique, mais à supprimer la notion d'usage unique,  pointe David Schisler, aux rênes de la CEE Schisler, l'un des principaux pourvoyeurs d'emballages papier et carton dans le quickservice.  Quelque part, c'est le retour au laitier des années 60. On pense ce que l'on veut de ce modèle, mais ce qui est certain en revanche, c'est que l'industrie n'est pas prête pour ce changement, a fortiori d'ici à 2023. » Il faut le dire, passer en trois ans au réutilisable, quand les modèles, les restaurants et les habitudes se sont construits durant des décennies autour du jetable, la marche a de quoi paraître haute... D'autant que sur le seul sujet du plastique, où doivent être mis en oeuvre des packagings moins polluants pour l'environnement, la restauration rapide, et ses 180 000 tonnes d'emballages jetables générées chaque année, est déjà bien occupée. Un effort de taille, et attendu : six Français sur dix se disent convaincus de la nécessité de changer leurs habitudes pour lutter contre le réchauffement climatique, selon le dernier baromètre Greenflex/Ademe. Et pour cela identifient le plastique parmi les trois principaux enjeux à traiter. À cette prise de conscience correspond un cadre réglementaire qui s'est largement précisé ces dernières années, transformant le statut des emballages ; d'abord, celui des sacs plastiques sous les 50 microns, interdits à partir de 2016, puis celui d'autres produits, via la directive SUP (Single-use Plastic), adaptée en un temps record, un an tout juste après que la Commission a présenté son projet.    STATUT DES EMBALLAGES TRANSFORMÉ C'est dire le consensus sur cet enjeu, concrétisé par un calendrier (voir encadré p. 25) dont les premières échéances, pour 2020 et 2021, ciblent pailles, gobelets, touillettes, couverts, bols... Soit des produits plastiques fréquemment utilisés en restauration rapide.  « L'activité génère beaucoup de déchets, il est essentiel de travailler dessus,  reconnaît Sandra Chassan, directrice marketing de Subway France.  Pour réduire notre impact, il s'agit à la fois d'identifier les solutions alternatives et de nous positionner, y compris avec nos partenaires boissons, sur les filières où les taux de recyclage sont les plus importants... Tout en mettant en place les flux de tri en restaurants. »  Sur ce dernier point, les principaux opérateurs, rappelés mi-2019 par la secrétaire d'État Brune Poirson à leurs obligations en matière de gestion des déchets, se sont engagés à équiper l'ensemble de leurs restaurants tri 5 flux d'ici à 2021, 70 % des réseaux en étant équipés fin 2019.    ANTICIPER SUR LE CALENDRIER Et sur les autres ? Aux rênes de First Pack, spécialiste des packagings du hors-domicile et implanté dans dix pays, Serge Merran est bien placé pour observer la stratégie des opérateurs :  « Il y a ceux qui n'ont encore rien fait sur leurs emballages, qui attendent sans doute que les premières amendes tombent ; d'autres qui veulent dès aujourd'hui être en ligne avec leurs obligations prévues pour 2023, et une majorité d'opérateurs qui veulent avancer par étapes. »  Des étapes définies en fonction du calendrier réglementaire, mais pas uniquement.  « Il peut y avoir un intérêt à anticiper sur le calendrier et se positionner sur certaines catégories de produits écoresponsables, ne serait-ce que parce qu'ils sont perçus très nettement pas les consommateurs. »  Il en va ainsi des couverts, pailles et spatules, sur lesquels la demande est soutenue, d'autant que leur effet d'image est inversement proportionnel à leur prix :  « Même si une paille papier coûte huit fois plus qu'une paille plastique, elle reste peu coûteuse, comme le couvert bois, dont le prix est similaire à celui d'un bon couvert plastique. » Retour chez Subway France. Dans ses 450 restaurants franchisés, une autre direction a été prise. Les pailles ont été supprimées l'an dernier, le choix étant laissé aux consommateurs d'équiper leurs gobelets de couvercles, notamment pour la vente à emporter. Une stratégie qui rejoint celle de McDonald's France, et ses gobelets 3-en-1 en fibres de papier moulées, mis en oeuvre depuis novembre 2019, et dont les effets attendus ne sont pas négligeables : 5 tonnes de plastique en moins par an à l'échelle du réseau Subway. À ajouter à 24 autres tonnes économisées en remplaçant les bols à salade plastiques par un support en papier, issu de forêts gérées durablement.  « Cette année, nos pots en sauce et packaging traiteur vont évoluer pour être en ligne avec la réglementation en 2021. Sur cet enjeu, nous avons l'énorme avantage d'utiliser peu d'emballages, contrairement à d'autres enseignes de la restauration rapide. » Notamment celle fonctionnant en vente d'impulsion, où le produit, en vitrine ou en libre-service en armoire, doit être vu.  « En dehors du réutilisable, les solutions transparentes en ligne avec la réglementation n'existent pas,  pointe Serge Merran, de First Pack.  L'amidon de maïs (PLA) pourrait être une solution pour les couvercles de gobelets et les couverts, mais le manque de capacités en France sur le compostage industriel a fait pencher le premier décret d'application en faveur de solutions compostables à la maison, ce qui disqualifie le PLA. »  Un vrai caillou dans la chaussure pour les enseignes de grab'n go, qui doivent revoir leurs packagings en nombre. Parmi les principaux opérateurs du fast casual, avec 42 restaurants et 60 millions d'euros de chiffre d'affaires en France en 2019, Exki est aussi l'un des plus avancés sur le sujet plastique, les couverts en bois comme les sacs en papier, recyclés à 80 %, ayant été utilisés dès 2001 par l'enseigne.    DES REMOUS PARMI LES PROFESSIONNELS « Notre positionnement sur une cuisine naturelle repose sur des engagements passant aussi par les emballages, d'autant que la moitié de nos ventes sont faites en take-away,  explique David Esseryk, son directeur digital, marketing et innovations.  À date, 80 % de notre packaging est réalisé en matériaux recyclés ou recyclables. »  Les contenants en bagasse (résidus naturels de pulpe de canne à sucre), introduits en restaurant dès 2015, ont été plus largement mis en oeuvre en 2019, utilisés sur les couvercles de gobelets, les entrées froides et certaines salades. Exit aussi les sachets en polypropylène, très légers, utilisés pour les sandwichs, remplacés il y a six mois par une feuille de papier, travaillée pour laisser de la transparence sur le produit. Les couvercles des bols de salades, pas concernés pour l'instant par la réglementation, restent en r-Pet.  « Nous sommes prêts pour 2021, et nous projetons déjà sur 2023. Nous avons introduit des bouteilles en verre pour la consommation sur place, proposons les couverts inox et testons la vaisselle en dur... Ce que l'on peut en retenir, c'est qu'il est complexe pour des opérateurs de la VAE de muter vers la restauration sur place. » Le moins que l'on puisse dire, c'est que ce passage au réutilisable, prévu pour 2023, n'est pas sans générer des remous parmi les professionnels.  « Les échéances de 2020-21 et 2023 sont contradictoires , regrette David Schisler.  On demande aux enseignes d'investir pour mettre en place le tri 5 flux en restaurant, d'identifier des filières pour recycler des produits à usage unique plus vertueux. Et deux ans plus tard, cet investissement est rendu caduc, substitué par le réemploi. »  Dans l'attente des futurs décrets d'application, l'Alliance Gobelet-Carton, dont la CEE-Schisler est l'un des cofondateurs, mène une expérimentation en Savoie et Haute-Savoie pour mesurer, en restauration rapide et en DA, le taux réel de recyclabilité des gobelets en carton, qui seraient en l'état limités à la VAE en 2023. Tout en pointant les questions sur l'impact réel de la vaisselle réutilisable en rapide :  « L'intérêt écologique du réutilisable arrive après 250 cycles de lavage. On y arrive en restauration assise, sur un service à table, mais dans les flux d'un fast-food, avec ce que cela suppose de casse, de vaisselle jetée, ou coulée ?... »  Autre frein relevé par les professionnels, la mise en oeuvre de produits réutilisables dans des établissements de petite surface :  « La vaisselle sur place, il faut avoir de quoi la laver, la stocker, ce qui n'est pas simple dans des établissements non conçus autour de ces obligations,  souligne David Esseryk, d'Exki.  L'autre inconnue, c'est le consommateur : il est demandeur d'alternatives, a une sensibilité au sujet, mais reste attaché à la praticité de la restauration rapide. La consigne est en réflexion chez nos partenaires boissons. Imaginer des réductions pour des clients venant avec leur vaisselle est aussi une piste, mais ce n'est pas parce que l'on imagine et que l'on met en place de nouveaux usages qu'ils sont adoptés. Il y a une phase de tests à ne pas négliger dans ce basculement. »  D'UNE CONTRAINTE FAIRE UNE OPPORTUNITÉ Un basculement effectif chez Pokawa, 22 restaurants à date, et 1,2 million de bols servis en 2019. En trois ans, l'enseigne est passée de bols en plastique à des contenants en carton recyclé et couvercles PLA, couverts en amidon de maïs et paille en bambou, le surcoût, de plus de 50 % sur les bols, étant absorbé par l'évolution de son modèle.  « Nos ouvertures se font sur des restaurants plus grands, 80-100 m², contre 30-40 m² sur les premières unités, avec l'objectif d'inciter à la consommation sur place et de se positionner sur d'autres moments comme le soir,  explique Maxime Buhler, cofondateur de l'enseigne.  On le fait avec de la vaisselle en dur, en bambou et noix de coco. Cela absorbe les surcoûts des emballages, en servant l'expérience-client. »  D'une contrainte faire une opportunité, la meilleure réponse possible aux obligations à venir en termes d'emballages et de vaisselle.
</article>
<article titre="Emballage écologique : la différence entre « compostable » et « biodégradable »" date="2020 03 24" journal="Le Progrès de Coaticook (QC) (site web)">
Avec la sensibilisation aux changements climatiques, plusieurs entreprises désirent également participer à l'effort global. Plusieurs méthodes existent pour y arriver, dont l'utilisation d'emballages écologiques. Cependant, il peut être difficile de choisir le bon produit; celui qui s'adapte à nos besoins, mais qui répond aussi à nos convictions environnementales. Ce texte a comme objectif d'expliquer la différence entre les emballages compostables et les emballages biodégradables.   Peu importe votre choix, ce type de contenants possèdent de nombreux avantages sur les emballages traditionnels :  Utilisation de matières renouvelables ou de déchets agricoles Moins grande émission de CO2 lors de la fabrication Neutralité carbone lors de la décomposition Baisse des coûts de fabrication des emballages biodégradables en comparaison avec les emballages traditionnels en plastique    Un autre élément important à comprendre est la différence entre les emballages recyclables et les emballages biodégradables et/ou compostables. Contrairement à la seconde catégorie, les produits recyclables ne sont pas destinés à retourner dans la nature. La matière est plutôt traitée et réutilisée dans de nouveaux produits. C'est un cycle qui évite de produire et d'extraire de nouvelles ressources.   Les emballages compostables    Pour qu'un emballage soit compostable, il doit de dégrader en moins de trois mois dans un environnement contrôlé. On parle de conditions pour accélérer le processus de retour à la terre comme la température. Les lieux de compost gèrent également le taux d'humidité ainsi que la présence ou non d'oxygène.   Ensuite, la grande différence entre les emballages compostables et biodégradables se retrouve dans l'utilisation des résidus du produit. Les emballages compostables doivent se transformer en compost riche en minéraux qui sera utilisé comme fertilisant naturel pour les jardins ou les terres fertiles.   Les emballages biodégradables    Pour qu'un emballage soit identifié comme biodégradable, il doit se dégrader en moins de six mois dans la nature, sans un contrôle de l'environnement comme les produits compostables. Même s'ils peuvent fournir des minéraux aux différents sols, l'objectif ne consiste pas à les utiliser comme fertilisant. En général, les emballages biodégradables sont fabriqués à partir de matériaux naturels comme le bois, le palmier et le bambou. De plus, ils proviennent parfois de déchets agricoles transformés comme l'amidon de maïs et la fibre de canne à sucre (aussi appelée pulpe de canne à sucre et bagasse). Par ailleurs, on retrouve également des produits PLA sur le marché. Ce sont des matières imitant le plastique, mais 100 % biodégradables. Ils sont généralement fabriqués à partir de la fermentation du sucre ou d'amidon de maïs.   Rien n'empêche qu'un produit respecte les deux normes et soit à la fois compostable et biodégradable. Il suffit qu'il se décompose dans la nature en moins de 6 mois, mais qu'il se désintègre en moins de 3 mois que dans un environnement contrôlé pour le compost. La grande différence est réellement que les emballages compostables servent à fertiliser les sols, tandis que ceux qui sont biodégradables seront simplement utilisés par la nature.  Cet article est paru dans Le Progrès de Coaticook (QC) (site web)
</article>
<article titre="Biocarburants avancés : passer à l'industrialisation, au nom de l'enjeu environnemental" date="2020 03 11" journal="La Tribune (France), no. 6872">
Le 27 janvier dernier, la ministre de la transition écologique et solidaire, Elisabeth Borne, présentait une feuille de route pour le développement des biocarburants aéronautiques durables dans le transport aérien français, avec un objectif d'incorporation de 5%. Il s'agit d'une cible ambitieuse pour un secteur qui est responsable de 13% de l'ensemble des émissions de gaz à effet de serre émanant des transports. Une telle annonce s'inscrit dans un mouvement réglementaire qui concerne de façon globale la décarbonation de nos moyens de déplacement qui représentent de l'ordre de 24% des émissions mondiales directes. Toujours en France, la Programmation pluriannuelle de l'énergie (PPE) prévoit l'intégration de 3,8% de biocarburants avancés (issus de paille ou de déchets forestiers) dans l'essence et 2,8% dans le gazole à horizon 2028.   A l'heure du défi climatique et de l'émergence de ces nouvelles filières industrielles au plan international, il convient donc de passer de la R&amp;D à l'industrialisation des biocarburants de deuxième génération sur le territoire français.   Une solution immédiate et pérenne En effet, les biocarburants avancés dessinent des perspectives prometteuses pour décarboner le secteur des transports et pour diversifier notre mix énergétique.   Les biocarburants de deuxième génération présentent des avantages majeurs : ils sont immédiatement accessibles puisque produits à partir de matières premières qui n'entrent pas en concurrence avec l'alimentaire : paille, coproduits de l'industrie forestière, bagasse ou autres matières lignocellulosiques.   Ils peuvent être mélangés directement à l'essence (bioéthanol), au gazole (biogazole) ou au kérosène (jetfuel), sans modification du réseau de distribution ou du système de motorisation des véhicules. Ils offrent en outre l'avantage de renforcer notre indépendance énergétique en réduisant le recours au pétrole. Autre intérêt : ils permettent une réelle diversification de notre production agricole et sylvicole.   Du point de vue du réchauffement climatique, les études confirment que les biocarburants avancés permettent une réduction de plus de 85% des émissions de gaz à effet de serre par rapport aux carburants fossiles. Sur l'ensemble de leur cycle de vie (de la production à l'utilisation finale dans les véhicules), ils affichent aujourd'hui une performance environnementale aussi bonne que celle de l'électrique.   Ce diagnostic ne doit néanmoins pas conduire à opposer différentes solutions parfaitement crédibles. Au contraire, un mix énergétique efficace s'impose aujourd'hui, avec des technologies complémentaires pour décarboner et réduire l'empreinte environnementale du transport.   Ailleurs dans le monde, les lignes bougent : en Inde, avec une douzaine de projets d'usines d'éthanol avancé, mais aussi en Europe, où avec la Directive RED II, des unités industrielles sont programmées en Roumanie, en Pologne ou encore en Croatie.   En faire un fleuron national Et la France? N'oublions pas qu'elle a largement contribué à faire des carburants durables une priorité inscrite dans le « Pacte vert » : elle doit donc prendre toute sa part pour le succès des objectifs européens comme, en particulier, l'incorporation de 14% d'énergie renouvelable dans les transports à l'horizon 2030 - c'est-à-dire demain - dont 3,5% de biocarburants avancés.   Pour cela, disons-le clairement : la transition énergétique implique de lancer dès à présent une véritable dynamique industrielle pour les biocarburants avancés sur notre territoire pour en faire un fleuron national. Illusoire? Au contraire : la France compte d'ores et déjà des technologies de biocarburants de 2e génération pour la production de bioéthanol, de biogazole et de jetfuel. Le moment est venu de lancer l'industrialisation et de valoriser ainsi une excellence et une capacité d'innovation reconnues à l'international. L'enjeu environnemental offre à notre pays une formidable opportunité : rendre compatibles urgence écologique et performance économique en construisant un succès « made in France ».   Didier Houssin, Président d'IFPEN
</article>
<article titre="Auchan Retail redresse ses résultats financiers en 2019" date="2020 03 6" journal="E-commerce (site web)">
Le distributeur a présenté ce vendredi 6 mars ses résultats financiers en 2019, en amélioration. Auchan redresse sa rentabilité avec des revenus en hausse dans neuf pays où il est présent sur 12, néanmoins le chiffre d'affaires en France recule de nouveau, avec des ventes en baisse de 2%.   "Nous sommes au rendez-vous de nos objectifs, ce qui n'a pas été le cas ces dernières années, souligne d'emblée Edgard Bonte, le président d'Auchan Retail, lors de la conférence de presse de ce vendredi 6 mars. La dynamique constatée au premier semestre 2019 se confirme avec la poursuite du redressement des indicateurs de rentabilité." En 2019, Auchan Retail réalise un chiffre d'affaires de 45,8 milliards d'euros, en recul de 1,4 % à changes courants. Après une très mauvaise année 2018, avec une perte nette de près d'un milliard d'euros (NDLR : le distributeur alimentaire affichait un chiffre d'affaires de 50,3 milliards d'euros, en baisse de -3,3 %), les revenus progressent dans neuf pays sur les 12 pays où le distributeur est présent. Le chiffre d'affaires est réalisé à 62% hors de France.   La Russie (-5%) et La France restent en recul, avec des ventes en baisse de 2% sur le marché français.   La marge commerciale d'Auchan Retail progresse de 330 millions d'euros. Le taux de marge commerciale s'améliore ainsi de 1,1 point à 23,6 %, grâce à la gestion du mix produits, des promotions et de la démarque. Les autres charges opérationnelles sont en baisse grâce aux premiers effets de la démarche Renaissance (traitement des foyers de pertes, réduction de la démarque, opérations et frais magasins). Au total, l'Ebitda d'Auchan Retail s'établit à 1 831 millions d'euros, en progression de 14 % à change courants par rapport à fin 2018 (+9 % à périmètre comparable). Le taux d'Ebitda atteint 4%, contre 3% à fin 2018 (chiffres intégrant alors les activités en Italie et au Vietnam). Auchan Retail a lancé l'an dernier la "démarche Renaissance", qui consiste à identifier tous les foyers de pertes et étudier toutes les solutions pour les zones où Auchan Retail est durablement en perte (134 projets sont en cours).    Le marché français reste en recul   Avec un recul de 2% de ses ventes, Auchan a réalisé un chiffre d'affaires de 17,36 milliards d'euros l'an dernier. La démarche Renaissance, lancée début 2019, vise à redonner des marges de manoeuvre financières à Auchan Retail. En 2019, le distributeur a procédé à la cession ou la fermeture de sites qui étaient "sans perspective réaliste de retour à la rentabilité" : 21 magasins en France, 30 magasins en Russie, 17 magasins en Espagne, 5 magasins en Chine, 1 magasin en Ukraine, 1 magasin en Pologne. Le distributeur annonce "ne pas prévoir de nouvelles fermetures cette année".   Dans le non-alimentaire, les tests d'accueil de partenaires spécialisés ont débuté en 2019 et se poursuivent : en France (Cultura, Electrodépôt, Boulanger), en Ukraine (Décathlon), etc. D'autres tests seront engagés en 2020. Néanmoins, ces shop-in-shop obtiennent des succès mitigés. "À Beauvais, nous obtenons de très bons résultats avec le corner Boulanger, seul spécialiste sur la zone, par contre le corner Cultura à Bagnolet n'a pas trouvé sa clientèle, pointe Edgard Bonte. Nous n'avons pas encore suffisamment de recul pour dresser un bilan complet."    Renforcer la "proximité digitale"   Le distributeur souhaite renforcer les services phygitaux dans ses hypermarchés (86% de son parc). En Chine, le service Online to Offline qui permet de livrer (sans émission de CO²) les clients en 45 minutes dans un rayon de 5 km autour du magasin est en croissance solide en 2019, avec plus de 100 millions de commandes réalisées. Cette activité rentable a connu une progression de son chiffre d'affaires de plus de 150% par rapport à 2018. Dans les autres pays d'Auchan Retail, le service de livraison ultra-rapide se développe en partenariat (Glovo en Ukraine, Pyszne en Pologne, Ubereats en Roumanie ou Wedely au Luxembourg). Le service de livraison à domicile en France s'étendra à 23 nouvelles villes en 2020 venant renforcer les 18 villes qui en bénéficient déjà, grâce à l'utilisation des drives comme des plateformes de préparation de commande.   Concernant l'ouverture de points de contact d'ultra-proximité, Edgard Bonte, le président d'Auchan Retail, indique "ne pas souhaiter suréquiper le marché" où ses concurrents opèrent déjà. Le distributeur privilégie d'autres solutions comme le drive piéton (au nombre de trois avec l'ouverture prochaine d'un 4e), le drive truck (le camion amène directement les courses au client), des casiers (lockers) ou des Auchan Minute.   Le digital représente 7% du chiffre d'affaires au global d'Auchan Retail avec un objectif de 15% d'ici 2022.   Le distributeur réalise actuellement 10% de son chiffre d'affaires sur l'e-commerce en France, 14% en Chine et 5 à 6% dans les pays de l'Est.   Le déploiement de ces nouvelles formes de proximité digitale accélérera dès 2020, dans chaque pays, en fonction des spécificités de chaque zone de vie. Le service de "laisser-caddie", avec lequel le client fait ses courses en magasin et confie à un tiers (professionnel ou particulier) l'acheminement de ses courses jusqu'à son domicile, est actuellement en test en France ou en Espagne (où 72 magasins proposent une livraison respectueuse de l'environnement et sans émission de CO²). Une nouvelle expérimentation est prévue en France en mars 2020 avec un troisième prestataire.   Un engagement plus fort dans la RSE   Auchan Retail est signataire du Pacte Plastique en France en 2019 et du Pacte plastique européen en ce 6 mars 2020. Il s'engage ainsi à lutter contre l'usage du plastique sans solution de recyclage ou à trouver des solutions vertueuses de substitution. Auchan Retail a développé en 2019 une gamme alternative (à base de bois, bagasse...) à la vaisselle plastique à usage unique économisant ainsi plus de 880 tonnes de plastique par an.   Après la Pologne, l'Espagne, le Portugal, le Luxembourg et Taiwan, en France, tous les magasins proposent désormais des filets réutilisables au rayon fruits et légumes. De même, Auchan Retail travaille au développement d'alternatives au film ou emballage plastique spécifique pour les fruits et légumes bio (étiquettes, collerettes, rubans, gravage au laser) ainsi qu'à des emballages recyclables et réutilisables pour les produits à marque propre (par exemple, la suppression du film plastique de conditionnement pour kes boissons végétales ou la suppression du plastique des packs d'ampoules). Dans tous les pays, des actions concrètes sont mises en oeuvre : alternatives au sac plastique en caisse dans 100 % des pays, journée sans plastique en Ukraine, possibilité pour les clients d'apporter leur propre contenant en Pologne, France et au Luxembourg, etc.  Cet article est paru dans E-commerce (site web)
</article>
<article titre="Valorisation énergétique Chauffez-vous aux CSR !" date="2020 03 5" journal="L'Usine Nouvelle, no. 3649">
Le 23 janvier, Solvay et Veolia ont confirmé la création de la coentreprise Dombasle Énergie. Elle gérera une installation de cogénération de 180 mégawatts (MW) thermiques et 17,5 MW électriques alimentée en combustibles solides de récupération (CSR), en remplacement de trois chaufferies au charbon sur le site du chimiste de Dombasle-sur-Meurthe (Meurthe-et-Moselle). Ce projet de 180 millions d'euros consommera 350 000 tonnes de CSR par an en provenance des centres de tri du Grand Est et des régions limitrophes. La mise en service est prévue au second semestre 2022. C'est un bon signe pour l'avenir en France des CSR, ces granulés de la taille d'un timbre-poste issus du broyage et du compactage des déchets non recyclables. Refusés au tri, ces déchets ultimes ne peuvent plus être enfouis, selon la réglementation, ni incinérés, faute de capacités. « Dans la loi de transition énergétique pour la croissance verte de 2015, outre la réduction de l'enfouissement de 50 % en 2025 par rapport à 2010, il est prévu qu'il n'y aura pas d'ouverture de nouveaux incinérateurs. Or, ils sont saturés en France », avertit Jean-Paul Roques, le directeur des ventes de CSR de Paprec. Le projet de Solvay et de Veolia est le signe que de nouveaux débouchés s'ouvrent pour ce combustible, qui n'intéressait jusqu'ici que les cimenteries. Et qui souffre du handicap de ne pas être considéré comme un produit, mais comme un déchet qui oblige les centres de tri à payer une redevance aux industriels pour s'en débarrasser. Une redevance dont « le montant est calculé en fonction du marché local, généralement entre 0 et 20 euros la tonne », observe Jean-Paul Roques. Or les CSR sont, pour l'heure, le seul moyen, en dehors des incinérateurs et des unités de valorisation énergétique, pour éliminer des déchets non recyclables, notamment les plastiques. Mais « seules 300 000 à 400 000 tonnes sont consommées par an en France, alors que l'objectif est de 2,5 millions de tonnes à l'horizon 2025 », selon le plan national de réduction des déchets de 2019, explique Jean-Christophe Pouet, le chef du service mobilisation et valorisation des déchets de l'Agence de l'environnement et de la maîtrise de l'énergie (Ademe). Un décret de 2016 permet d'utiliser les CSR pour produire de l'électricité ou de la chaleur. Reste à motiver les industriels, au-delà des cimentiers, comme le prévoit le contrat stratégique de la filière transformation et valorisation des déchets signé en janvier 2019. « Nous travaillons pour définir les conditions qui permettraient d'investir dans de nouvelles unités de CSR raccordées à des réseaux de chaleur urbains ou à des industries. Il ne s'agit aucunement de construire de nouveaux incinérateurs », avait alors expliqué Brune Poirson, la secrétaire d'État auprès du ministre de la Transition écologique et solidaire.   Les projets de chaufferies se multiplient   Bras armé de cette nouvelle politique, via son fonds économie circulaire, doté de 174 millions d'euros par an, l'Ademe a lancé deux appels à projets CSR en 2019 et début 2020. « L'objectif est de trouver des clients industriels qui ont besoin de chaleur tout au long de l'année, par exemple un chimiste ou un papetier », précise Jean-Christophe Pouet. C'est le cas de la papeterie strasbourgeoise Blue Paper (160 salariés, 160 millions d'euros de chiffre d'affaires), qui a remplacé ses chaudières à gaz par une unité de production de vapeur à base de CSR (27 000 tonnes par an) grâce à un investissement de 25 millions d'euros. Une ligne de production de CSR peut aussi alimenter un process de séchage sur une unité de méthanisation, comme le prévoit le syndicat mixte de gestion des déchets ménagers Trifyl sur son site de Labessière-Candeil (Tarn). À la Réunion et en Guadeloupe, le producteur d'électricité ultramarin Albioma étudie lui aussi l'utilisation de CSR dans ses centrales bagasse-charbon en cours de conversion à la biomasse.   Les spécialistes du traitement de déchets comme Suez et Paprec sont sur les rangs pour codévelopper ces nouveaux projets de chaufferies CSR dans l'industrie. Suez, qui produira 180 000 tonnes de CSR en 2020 sur ses sites de Narbonne (Aude), Istres (Bouches-du-Rhône), Hénin-Beaumont (Pas-de-Calais) et de la Réunion, a dans ses cartons une dizaine de projets pour l'agrochimie, la chimie, la papeterie et l'agroalimentaire. Idem chez Paprec, qui dispose de deux unités CSR, à Toulouse (Haute-Garonne) et à Nantes (Loire-Atlantique), pour servir les fabricants de ciment et travaille sur trois projets de chaufferie, pour un papetier dans le Sud-Ouest et pour deux industriels de la construction en Picardie et en région Centre. Des projets compliqués à monter, « une chaudière CSR coûtant deux fois plus cher qu'une chaudière biomasse », rappelle Jean-Paul Roques.   On constate un léger regain d'intérêt pour les CSR chez les cimentiers, qui cherchent à réduire leur empreinte CO2. Avec un taux de 50 % d'éléments biogènes (bois, carton...), les CSR affichent un taux d'émission proche de celui du gaz. LafargeHolcim l'a bien compris. Il a inauguré le 10 octobre 2019 à Port-la-Nouvelle (Aude) un atelier de valorisation de combustibles solides de récupération, appelés déchets solides broyés (DSB) dans le secteur. D'une capacité de 35 000 tonnes par an, l'équipement de 7,5 millions d'euros, soutenu par l'Ademe, doit permettre à l'usine de tourner avec 90 % de combustibles de substitution en 2022. De quoi dépasser la cimenterie de Saint-Pierre-La-Cour (Mayenne), qui utilise 80 % de CSR, bien au-dessus de la moyenne de 43,3 % enregistrée chez l'industriel. Quant au projet Dombasle Énergie, il permettra à Solvay de diviser par deux ses émissions de CO2 sur le site.
</article>
<article titre="Albioma continue à s'extirper avec succès du charbon" date="2020 03 3" journal="GreenUnivers (site web)">
2020-03-03T19:10:38+00:00    (@Albioma)   Pas de crise boursière pour Albioma, qui a présenté aujourd'hui ses résultats pour 2019. Le cours du spécialiste de la biomasse et du solaire ultramarins a même fait un bond de 8% dans la matinée sur Euronext et peut ainsi poursuivre le rallye entamé en décembre. L'optimisme des investisseurs est motivé par les promesses d' Ebitda  pour 2020, entre 200 et 210 M€, contre 183 M€ l'année dernière mais déjà en hausse de 12%. Même orientation pour le chiffre d'affaires, en progression de 18% à 506 M€. Le résultat est plat, à 44 M€.    Le bois/bagasse remplace le charbon/bagasse L'important est ici que le développeur-producteur maintient une forte croissance tout en basculant son  mix  depuis le charbon vers le renouvelable,  ie  les résidus sucriers, le bois, le photovoltaïque. Entre 2013 et 2019, les proportions se sont littéralement inversées et la production électrique est désormais à 67% non fossile. Pour rappel, l'entreprise vise 80% en 2023. Elle est bien partie pour y parvenir, avec un outil de production à la fois nouveau et fiable. Les performances opérationnelles actuelles bénéficient d'ailleurs de la bonne première année de production de la centrale Galion 2 ( pellets  nord-américains/bagasse) en Martinique, à la gestation longue et difficile.   Le commandable avant tout Le dynamisme boursier du titre bénéficie aussi de sa liquidité (2,5 M€ échangés chaque jour) et de l'ouverture du capital. Aucun actionnaire ne dépasse 7%, ce qui peut alimenter bien des scénarios. L'investisseur Impala, entré en 2018 , en reste pour le moment à 5,9%. Pour les financiers, l'actif Albioma bénéficie avant tout d'une position éminente dans la cogénération d'électricité en Outre-mer, avec une production de pas moins de 3,8 TWh, en partenariat en général avec l'industrie du sucre. Frédéric Moyne, PDG, ne manque pas de souligner la valeur de la production électrique commandable, soit 900 MW dans un parc de 1 GW, le solde étant composé de 100 MW solaires. Cela dit, ce segment est lui aussi devenu un axe de choix, dans la famille des petites et moyennes toitures surtout : Albioma compte ici profiter au maximum des appels d'offres ZNI en cours (270 MW en tout) et métropolitains.   La géothermie à l'étude A l'avenir, la stratégie de croissance, en France, au Brésil et dans d'autres pays sucriers, continuera manifestement à réserver une place majeure à la production d'électricité régulière. En témoignent les pistes étudiées pour les années qui viennent. En particulier l'intérêt croissant pour l'Asie du sud-est sucrière, pour la production électrique issue des combustibles solides de récupération dans les territoires ultra-marins et pour la géothermie électrique hors de France, par exemple en Amérique centrale et en Asie. "Albioma possède le savoir-faire thermique-électrique. Nous verrons si nous internalisons ou non la compétence géologique, notamment l'analyse du gisement géothermique", précise Frédéric Moyne.   Un TRI moins généreux Pour l'instant, les 450 M€ à 650 M€ d'investissements prévus d'ici 2023 sont avant tout fléchés vers la reconversion des trois grandes centrales fossiles restantes à La Réunion et en Guadeloupe, ce qui implique entre autres d'obtenir un nouveau contrat d'achat long terme de la part d'EDF, notamment pour la centrale de l'usine sucrière de Bois Rouge à La Réunion. Albioma attend aussi impatiemment d'en savoir plus sur les conclusions de la Commission de régulation de l'énergie. Le régulateur souhaite en effet réduire les taux de retour sur investissement (TRI) pour les futures centrales en reconversion, constatés à 11% sur les unités déjà basculées et généreux dans un univers de taux au plancher. Le TRI serait ramené autour de 8 à 9%, dans un arrêté à paraître dans les prochains mois.    On notera parmi les autres points à surveiller du coin de l'oeil une dette nette de 773 M€ (4,3 fois l' Ebitda ) mais qui est surtout associée aux projets et qui bénéficie d'une protection ou d'un taux fixe sur 82% du montant. Elle n'empêche pas Albioma de promettre un généreux dividende à l'avenir avec un objectif de distribution de l'ordre de 50% du résultat net part du groupe. Un autre trait qui alimente la popularité auprès des investisseurs.    Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="CSJ-emballages vient de s'installer dans la zone industrielle" date="2020 03 2" journal="Le Progrès (Lyon)">
Christian Mino est le gérant de la société CSJ. C'est un spécialiste de l'emballage, de toutes sortes d'emballages. En 2012, il achète un bâtiment à Clairvaux-les-Lacs pour créer une pizzeria avec son fils et son épouse, et à partir de 2015, il s'intéresse à l'activité emballages et devient grossiste avec une plate-forme en e-commerce pour les particuliers et pour les professionnels. Ce type de vente a le vent en poupe, il se trouve à l'étroit et se démène pour trouver un espace plus adapté à ses besoins. Il fait alors un gros investissement dans un grand bâtiment neuf. « 90 % des ventes se font en dehors de la Franche-Comté », s'étonne-t-il. Les clients viennent essentiellement des grandes métropoles. « CSJ vend à l'étranger, États-Unis, Canada, Norvège, Italie et Belgique. » Les artisans commencent à s'intéresser à ce type de vente de marchandises, « d'autant que les professionnels, contrairement aux particuliers, ont la possibilité de venir faire leurs courses dans la zone directement ».    Les fournisseurs et les produits 70 % des fournisseurs sont français, tels qu'Aluplast, Elve, Sacpo, Tradizer... Seules les gammes écologiques sont importées de Chine, les produits à base de bagasse, ou pulpe de la canne à sucre chauffée et compressée, ou ceux à base d'amidon de maïs, comme les gobelets biodégradables, ou les pailles. On y trouve un grand panel de produits, tels que barquettes alimentaires en carton, emballages alimentaires et produits d'entretien, boîtes pâtissières et à pizza, films alimentaires ou étirables, des produits d'hygiène, du matériel pour l'entretien.    Les problématiques rencontrées Le projet d'installation dans la zone ne se fait pas simplement. Plusieurs obstacles se présentent à lui, le plan local d'urbanisme prévu est remplacé par le règlement national d'urbanisme qui stipule des règles différentes du permis de construire. Christian Mino remercie Michel Jourdant, maire sortant, pour sa ténacité à faire aboutir son projet d'installation.   Le second problème est celui du réseau informatique très vite saturé. « Je ne dispose ici que de 10 Mo alors qu'à Clairvaux, j'en avais 60. On a deux réseaux ADSL et pour le téléphone un réseau SDSL (NDLR : technologie d'accès à Internet haut débit). » Christian Mino se désole de ne pas répondre à toute la demande, de plus il ne peut faire d'embauche. Il a écrit à la responsable économique de la Communauté de communes du Pays-des-Lacs. « Nous ne pouvons prendre de commande le matin », explique-t-il.
</article>
<article titre="Remue-méninges dans l'emballage" date="2020 03 1" journal="Néorestauration, no. HS583">
Chaque année, 13 milliards d'emballages sont jetés par la restauration rapide (source : Assemblée nationale), majoritairement en plastique... Un bel appel à doper le recyclage (il ne serait que de 26,5 % sur le plastique en 2018, selon Citeo) et la recherche (comme avec ces enzymes mises au point par Carbios pour dégrader le plastique), en tout cas  à repenser et à changer la donne,  d'autant que les cadres législatif et réglementaire se resserrent et que les consommateurs sont à 85 % favorables à une interdiction du plastique mono-usage (Ifop, décembre 2019). Une fois qu'on le sait, que fait-on ? À quel coût ?  Sur les pailles,  alors que les alternatives existent, en carton, verre, métal, comestibles (PanzaPailles, RizPaille...), voire en plastique végétal (Loliware), 79 % des restaurateurs disent ne pas avoir trouvé la solution (FSV décembre 2019). Et les autres ? Certains, parmi lesquels Groupe Bertrand, Burger King, Subway, les retirent tout bonnement. Ce qui leur permet de pallier le surcoût des emballages dits écoconçus, qui varierait de 10 à 40 % selon les professionnels. Sur l'emballage à proprement parler,  l'usage unique perdure,  revisité principalement par le plastique, mais recyclé, le carton, la bagasse ou le PLA,  le réutilisable arrive.  Sous forme de consigne chez Pur Etc, Jargus ou chez Franprix qui, en test dans son magasin de Réaumur, sert ses salades dans des barquettes en verre que le client peut rapporter le lendemain. Starbucks et Exki accordent des remises aux clients qui amènent leur propre contenant. Intéressant, car comme le souligne Kantar World Panel dans une étude mondiale de novembre 2019, les consommateurs sont plus nombreux à réclamer l'abolition du plastique à usage unique qu'à vouloir faire personnellement quelque chose en ce sens. Il n'empêche, les choses avancent vite...
</article>
<article titre="BÉNÉDICTE GOULON-DELMOTTE, RESPONSABLE RSE et DÉVELOPPEMENT DURABLE DE FLUNCH" date="2020 02 1" journal="Néorestauration, no. 0582">
Flunch,  246 points de vente, travaille sur la traçabilité et la qualité depuis des années (viande 100 % française, BVF, MSC).  « Il est essentiel de valoriser la qualité de ces produits, ces filières. »  Comme les circuits courts, qu'elle va développer en 2020, avec par exemple des fraises garriguettes dans le Sud, de Phalempin dans le Nord, bref, des recettes différentes selon les régions, pour être plus proches des clients.   Le gaspillage  Flunch e a testé l'appli Too Good To Go et l'a déployée au sein de 70 à 80 Flunch Café, et depuis octobre, à tout le parc, avec un panier entrée/offre chaude/dessert. Soit, à raison de 3 paniers par restaurant/jour, 100 000 repas sauvés, 200 000 en 2020... L'objectif étant de les éliminer. Quid de l'emballage de ces paniers ? Pour Flunch Café, il est en papier essentiellement ; en restaurant, l'enseigne n'a pas voulu proposer au client de venir avec son propre contenant pour des questions de qualité, d'hygiène, de responsabilité, d'où un emballage en bagasse pour le chaud... D'autres initiatives  ont été prises par l'enseigne (sensibilisation des clients pour éviter le gâchis au buffet de légumes chauds à volonté) ou le seront (élaboration de nouvelles recettes type puddings pour donner une nouvelle vie aux produits).
</article>
<article titre="Biomasse solide : la consommation est stable en Europe" date="2020 01 29" journal="Dechetcom (site web)">
Biomasse solide : la consommation est stable en Europe    EurObserv'ER (un consortium spécialisé dans le suivi du développement des énergies renouvelables dans l'UE) vient de publier la dernière édition son baromètre Biomasse solide (2019). Nous vous proposons d'en découvrir les principaux enseignements et chiffres clés...    La biomasse solide est la principale source d'énergie renouvelable utilisée dans l'Union Européenne pour la production de chaleur. Le principal combustible est le bois sous forme de plaquettes, mais de nombreuses autres formes sont également utilisées telles que les déchets de bois, les granulés, la sciure, la paille, la bagasse ou les liqueurs noires de l'industrie papetière.  Globalement en 2018, en raison d'un hiver particulièrement doux, la demande de chaleur a diminué au sein des pays membres de l'UE alors que dans le même temps la production d'électricité a augmenté. Au total, la consommation d'énergie primaire d'origine biomasse solide est restée stable (0,3% de moins qu'en 2017) pour se situer juste en dessous du seuil de 100 Mtep (99,3 Mtep). La consommation de chaleur issue de la biomasse solide est de 68,2 Mtep (68,9 Mtep en 2017) et la production d'électricité s'élève à 99,5 TWh (94,7 TWh en 2017).    La consommation européenne de granulés de bois a augmenté de +8% en 2018 pour atteindre 26,1 millions de tonnes. En 2018, le Royaume-Uni était le plus grand utilisateur de granulés de bois de l'Union Européenne avec 8,5 millions de tonnes, devant l'Italie (3,3 Mt), le Danemark (3,1 Mt), l'Allemagne (2,2 Mt), la Suède (1,8 Mt), la France (1,6 Mt), la Belgique (1,5 Mt), l'Autriche (0,96 Mt), les Pays-Bas (0,57 Mt), l'Espagne (0,57 Mt), la Pologne (0,45 Mt) et la Finlande (0,44 Mt).     L'une des tendances actuelles du secteur est la conversion de grandes centrales électriques, c'est-à-dire la substitution progressive des centrales au charbon par des centrales à biomasse, non pas en démolissant et en reconstruisant, mais en adaptant les centrales pour qu'elles fonctionnent avec les nouveaux combustibles renouvelables.    Pour plus d'informations et consulter / télécharger le baromètre Biomasse solide 2019 d'EurObserv'ER, rendez-vous ici.   Cet article est paru dans Dechetcom (site web)
</article>
<article titre="Bio, responsable et écolo... Quand les bars à cocktails montrent l'exemple" date="2020 01 23" journal="Télérama (site web)">
Toujours en phase avec l'air du temps, les bars à cocktails ont intégré des préoccupations sociétales et écologiques à leurs offres mixologiques. Petit état des lieux à l'occasion de la    sixième Paris Cocktail Week, qui cette année annonce la couleur : "Plus de vert dans les verres" ! On ne le sait pas toujours, mais, derrière un comptoir de bar à cocktails, on trouve très souvent des gens responsables, aussi vertueux que virtuoses ! Ce ne fut pas toujours le cas, et ce sont des années de mauvaises habitudes et d'ignorance qu'il faut désormais oublier, pour inventer de nouveaux réflexes. « On ne peut pas se déclarer bar responsable du jour au lendemain : c'est un processus de longue haleine, qui demande de la réflexion et beaucoup de recherche... » explique Hyacinthe, patron du Cambridge Public House, passé par Les Grands Verres, le bar-restaurant du Palais de Tokyo, pionnier du zéro déchet dans le secteur.   Fruits moches mais de saison    Pour un bon cocktail, tout commence par un bon produit. Ces dernières années, beaucoup de progrès ont été faits dans le domaine du bio, mais aussi des circuits courts, avec l'achat en direct aux producteurs locaux, pour réduire l'empreinte carbone des livraisons et travailler avec des fruits et légumes de saison.   Nicolas Munoz, patron de Divine et du Bisou, deux lieux aussi impliqués que festifs, nous raconte sa façon de faire : « On travaille aussi avec des produits dits "déclassés", des fruits et légumes non calibrés, que les clients ou restaurants ne veulent pas car trop moches, mais que nous, en tant que bar, travaillons en sirops, purées... car on n'a pas besoin de joliesse ! Auparavant, ces produits, non vendus par notre primeur, seraient partis à la poubelle ! »    Divine    On constate par exemple que de plus en plus de bars n'utilisent plus de citron hors saison, lui préférant le verjus, un acidifiant ancestral issu du raisin, qui rend les mêmes services aromatiques.   Le boum des distilleries artisanales    Au coeur de cette tendance verte, il faut aussi noter l'apparition de distilleries artisanales, qui opposent aux productions industrielles des alcools durables, à base de céréales bio et d'eau pure, stockés dans des tonneaux réutilisables... Comme les bières bio et les vins naturels, ces spiritueux de qualité supérieure sont autant de solutions pour les bars qui veulent changer de posture...et de fournisseurs ! Certains établissements, comme Mino, à Ménilmontant (20e), possèdent même leur minipotager, directement en salle, pour faire pousser leurs propres plantes aromatiques.   Les déchets, à boire et à manger ?    Première cible facile à éliminer : les pailles en plastique. On préférera celles en fibres naturelles comme celles du blé ou du bagasse (résidu fibreux de la canne à sucre). Pour la décoration des verres, on recycle au maximum les déchets comestibles, surtout qu'un bar à cocktails en génère beaucoup, en les transformant en garniture... « On travaille les produits dans leur intégralité. Avec un fruit ou un légume, on va sortir trois produits différents qui auront trois intérêts différents. Ainsi, avec le potimarron, on va bosser la chair, mais aussi la peau et les graines », explique encore Nicolas Munoz.   "Ne pas jeter l'eau des glaçons fondus"    On peut aussi récupérer les déchets de ses voisins restaurateurs. Ainsi, l'aquafaba, l'eau de cuisson des pois chiches, est de plus en plus utilisé en remplacement du blanc d'oeuf dans les shakers pour réaliser des « sours », ces recettes légèrement mousseuses. Et comme toute idée simple est bonne à prendre, Hyacinthe nous révèle encore une bonne pratique : « Ne pas jeter l'eau des glaçons fondus, mais l'utiliser après le service pour le nettoyage, ou l'arrosage des plantes. » Comme quoi, rien ne se perd, tout se récupère, derrière un bar !   Retrouver l'homme derrière le bar    La prise de conscience se veut aussi sociale. Le comportement se veut désormais exemplaire, plus respectueux envers les clients, les membres du personnel, et les partenaires, pour installer un climat d'empathie. Et c'est vrai que, du point de vue du client, il y avait (et il y a encore) pas mal de boulot ! Les bars cherchent aussi un ancrage plus fort dans leur quartier, pour mieux s'intégrer dans le tissu social, servir ses voisins, se fournir auprès de commerçants du coin, et retrouver la mission première d'un bar : être un lieu de sociabilité qui distille les bons réflexes de développement durable.   Pour autant, il ne faut pas crier victoire trop vite : ce cercle vertueux reste fragile, car si on va au bout du raisonnement, comme le fait Hyacinthe, on s'interroge : « Utiliser des produits de saison est responsable, mais l'est-ce toujours si les fraises ramassées au sud de l'Espagne le sont par des gens payés une bouchée de pain ? C'est un tout, à prendre en compte. » Pour autant, l'évolution des mentalités est réelle, et on peut tout de même s'en féliciter, car comme le souligne Nicolas : « Une fois que tu prends conscience que tu jettes quand même beaucoup et que ça craint, t'as fait un joli paquet du chemin ! »    "Shake local" La consommation responsable fait partie du programme de la Paris Cocktail Week (du 24 janvier au 1er février), qui met l'accent cette année sur les recettes sans alcool, toutes à base du « Spirit of Paris », premier distillat sans alcool made in Paris, fabriqué par le talentueux Nicolas Julhès, de la Distillerie de Paris. C'est aussi l'occasion de découvrir ces nouveaux bars durables, parmi les 50 établissements où se déroule l'événement !  Cet article est paru dans Télérama (site web)
</article>
<article titre="« Le rôle de conseil est très présent dans nos boutiques »" date="2019 12 30" journal="L'Est Républicain">
Voilà 15 ans que « J &amp; C Décoration » enchante les fêtes. Alors, pour nouvel An, la maison de la rue de Rû à Verdun ne déroge pas à la tradition. « On a tout ce qu’il faut », précise Cécile Élard, le C de « J &amp; C Décoration », le J étant pour Jérôme. Pour la table, les rayons regorgent de nappages, de serviettes festives, de chemins de table à sequins ou à paillettes ou de paillettes de table… Mais en règle générale, « les tables sont plus épurées qu’à Noël : un chemin de table, deux ou trois bougies », souligne Cécile. « Et les gens aiment confectionner eux-mêmes ». Alors, « le rôle de conseil est très présent dans nos boutiques ». Nouvel An durable Côté vaisselle, elle peut être jetable : « On est dans un moment charnière où le plastique disparaît ». Alors les couverts et les verres sont confectionnés en bambou et les assiettes en bagasse, c’est-à-dire en fibre de canne à sucre. Le tout étant biodégradable pour un nouvel An durable.   Canons à billets Pour la déco, les éventails, suspensions et autres étoiles dorées et décorées ont la cote de même que la bouteille ou les canons à confettis. Et pour attirer la prospérité, il existe même des canons à billets de banque où des liasses d’euros retombent sur les invités ! Outre les cierges magiques plus classiques, les feux d’artifice de table se déclinent enchâssés dans un dispositif clipsé au goulot de la bouteille de champagne. Effet assuré.
</article>
<article titre="LES ACTIFS SE VERDISSENT" date="2019 12 2" journal="CosmétiqueMag">
Alors que la demande en ingrédients naturels ne cesse d'augmenter, les fournisseurs doivent relever des défis sur des procédés écoresponsables.   Les marques de cosmétiques en veulent toujours plus. « Un nombre croissant d 'industriels nous demandent de leur fournir des actifs naturels, constate Yohanna Sander, la directrice Global Product Management chez Symrise. Ce boom s'est encore confirmé en 2019.» Rien ne semble pouvoir arrêter cette vague verte. « Ce phénomène est très sensible depuis deux à trois ans, confirme Jean-Yves Berthon, le PDG de Greentech. Green, est un hydratant qui booste les performances des actifs tout en améliorant le profil sensoriel de la formule. Mibelle revendique, pour sa part, avoir été la première société à cultiver dès 2007 des cellules-souches végétales. Issu de l'espèce de pomme suisse rare Uttwiler Spätlauber, le PHYTOCELLTEC Malus Domestica affirme protéger les cellules-souches cutanées tout en retardant le vieillissement des follicules pileux. Mais la nature n'a pas encore révélé tous ses secrets.   Privilégier le local « À terme, nous pourrons fabriquer n'importe quel produit cosmétique à partir d 'actifs naturels », prédit Jean-Marc Seigneuret, le directeur technique et R&amp;D chez Alban Muller International. Yohanna Sander est plus partagée : «En 2018, nous avons mis sur le marché la version naturelle de notre historique pentylène glycol : hydrolite 5 green. Cet ingrédient est une véritable performance en termes de développement durable (utilisation de déchets de l'industrie du sucre, diminution de la consommation en eau et de la production en CO²). La conversion d'un ingrédient du synthétique vers le naturelle reste cependant un challenge dû à la disponibilité des matières premières, aux contraintes technologiques et aux coûts qui sont parfois bien supérieurs aux attentes du marché. » D'autres sociétés sont beaucoup moins pessimistes en la matière. « Le prix de revient des actifs naturels sera toujours supérieur aux produits chimiques, mais on peut les purifier davantage, explique Romuald Vallée, le directeur et cofondateur de Codif, une entreprise basée à Saint-Malo spécialisée dans les actifs cosmétiques d'origine marine, végétale et biotechnologique. Un ingrédient peut ainsi coûter cinq fois plus cher à fabriquer, mais cela ne pose aucun problème si l'on doit en utiliser La moitié des actifs utilisés aujourd'hui dans les cosmétiques est naturelle. Cette croissance s'accélère d'année en année avec des taux de progression de 20 à 25 % par an. » La mode du bio est passée par là. « Pour être considéré naturel, un produit doit être obtenu avec des ingrédients les plus verts possible », résume Jean-Marc Seigneuret, le directeur technique et R &amp; D chez Alban Muller International. La société allemande Symrise commercialise depuis peu un tout nouvel ingrédient d'origine naturelle produit à partir de la bagasse, le résidu fibreux de la canne à sucre. Cet Hydrolite 5 dix fois moins dans une crème qu'un actif synthétique.» L'origine naturelle d'un actif ne suffit pas non plus à en faire un produit respectueux de l'environnement. Loin de là. «On parle beaucoup de plantes bio issues de l'agriculture durable, mais si elles doivent traverser la moitié du globe pour être transformées, leur impact carbone reste énorme, analyse Vincent Briffaut, le directeur des ventes pour la France de Mibelle. Le transport doit donc être pris en compte pour considérer les bienfaits d'un actif plutôt qu'un autre. » Les marques de cosmétiques commencent à prendre conscience de ce phénomène. « Depuis deux à trois ans, nos clients nous envoient des questionnaires très détaillés pour connaître le bilan carbone exact des extraits que nous leur envoyons,certifie Jean-Marc Seigneuret. Ils veulent notamment connaître l 'origine de nos plantes. Aujourd'hui, les espèces cultivées localement sont plus dans l'air du temps que celles qui proviennent de pays lointains.» De gros travaux ont également été faits pour améliorer les procédés de transformation des végétaux.   Des procédés plus écologiques « Beaucoup d'actifs naturels ont déjà été découverts, résume le directeur technique et R&amp;D chez Alban Muller.   Aujourd'hui, les recherches tentent surtout de trouver des procédés d'extraction qui soient moins gourmands en solvant et en eau. Le solvant le plus naturel qui soit est l'eau, et nous l'utilisons souvent tout en prenant garde d'en consommer le moins possible. Un autre solvant dont nous nous servons est l'éthanol, le nôtre est biosourcé et produit localement.» Un procédé d'extraction différent, de plus en plus utilisé, est le dioxyde de carbone supercritique. Mis sous pression et chauffé, le CO2 se comporte comme un fluide et devient un solvant capable d'extraire très précisément une famille de composés donnés. « Les hautes pressions permettent, elles, d'éclater les cellules et de mieux extraire les composants recherchés, note Jean-Yves Berthon. Cette technologie demande de l'énergie, mais elle pollue moins quel 'utilisation de solvants chimiques. » D'autres procédés devraient se développer dans les prochaines années.   Le marché des actifs naturels a explosé depuis trois ans, mais les lois évoluent plus lentement que les demandes de la clientèle. « La législation actuelle est encore trop laxiste, mais elle évolue dans la bonne direction, constate le cofondateur de Codif. L'indice de naturalité doit notamment prendre en compte les processus de transformation qui sont utilisés pour extraire un actif. » Définir une réglementation claire et stricte est nécessaire pour éviter une méfiance accrue des consommateurs. L'avenir s'annonce radieux pour les industriels. Car tous les spécialistes sont d'accord sur un point : la popularité des ingrédients verts va durer. «Je suis très, très, très confiant quant à l'essor des actifs naturels dans les prochaines années », prédit Romuald Vallée, directeur et cofondateur de Codif. Jean-Marc Seigneuret ne dit rien d'autre, «ce n'est pas une mode, mais une tendance irréversible, renchérit le directeur technique et R&amp;D d'Alban Muller.   Tous les grands groupes rachètent d'ailleurs des sociétés spécialisées dans ce domaine afin d'accélérer leur transformation. » De tels investissements montrent bien que les actifs naturels sont là pour rester.   Frédéric Therin
</article>
<article titre="Vers les alternatives au plastique jetable sur M6" date="2019 12 1" journal="Ouest-France  Saint-Malo - Dinard ; Nantes métropole ; Saint-Nazaire, La Baule ; Lannion ; Sarthe ; Les Herbiers, Fontenay-le-Comte ; Vendée Littoral ; La Roche-sur-Yon ; Auray ; Vannes ; Lorient ; Dinan ; Calvados, Manche, Orne ; Nord-Finistère ; Finistère-Sud ; Mayenne ; Saint-Brieuc ; Maine-et-Loire ; Chateaubriant, Ancenis ; Ille-et-Vilaine ; Rennes Métropole">
Entretien   Christophe Brulé,  rédacteur en chef de  Capital. Le 1er janvier 2020, gobelets et assiettes en plastique seront interdits. Quelles alternatives ?   Les solutions en « plastique » biodégradables se multiplient sous la pression de l'opinion : gobelets en cellulose de pomme de terre, de maïs, assiettes en graine de blé, barquettes en bambou, bol en bagasse - résidu fibreux de la canne sucre... Mais ces alternatives sont deux à cinq fois plus chères. Même le plastique recyclé reste plus cher que le plastique neuf. Plus le prix du pétrole est bas, plus le plastique est compétitif. Tant que l'on ne prend pas en compte le coût sur l'environnement, bien sûr. Votre enquête alerte sur la toxicité des pailles en papier fabriquées en Asie...   En 2021, les pailles en plastique et les boîtes en polystyrène de plats chauds à emporter seront à leur tour interdites. Le leader européen de la paille (Matrix), dirigé par un Français installé en Grèce, a donc commencé sa conversion au papier. Ainsi, il a alerté cet été les autorités sanitaires française et européenne sur ces pailles en papier produites moins chères en Asie. D'après ses propres analyses, sur une trentaine de pailles, la plupart contiennent des taux de MCPD, une molécule cancérigène, très supérieurs aux normes européennes.   Votre seconde enquête s'intéresse au recyclage du plastique : pourquoi la France est-elle 29e sur 30 en Europe ?   Nous devons cette place de cancre (parmi les Vingt-Huit de l'Union européenne, plus la Norvège et la Suisse) à notre très mauvais taux de recyclage du plastique. Alors que le papier, carton, aluminium sont plutôt bien recyclés, seulement 26 % de nos emballages plastiques le sont. Les bouteilles en plastique sont recyclées à 50 %. Mais pots de yaourt et gourdes de compote, alors que des filières existent dans d'autres pays, sont incinérés chez nous. Une incinération qui est, au mieux, transformée en énergie, en chauffage par exemple. Quant aux emballages en plastique mous, sans débouchés, ils sont exportés.   Vous suivez le plastique de nos poubelles jaunes jusqu'en Malaisie...   La Chine et l'Indonésie refusant désormais de servir de décharge mondiale, la France envoie ses déchets plastiques, qui n'ont cessé d'augmenter, en Thaïlande, au Vietnam, et surtout en Malaisie. Où ils sont transformés en nouveaux objets en plastique, ou brûlés dans des décharges à ciel ouvert, aux spectaculaires fumées noires, toxiques. Mais ces pays envisagent à leur tour d'arrêter cet afflux toujours croissant.   Depuis votre enquête sur les destructions massives d'invendus par Amazon, y a-t-il eu des changements ?   Notre enquête qui dévoilait ce scandale a été relayée en Italie, au Canada, en Australie. Et le géant du commerce en ligne a fini par réagir, en août. Désormais, aux États-Unis et en Angleterre, les invendus peuvent être donnés à des organismes caritatifs. Mais la mesure n'a pas été généralisée, notamment en France.   M6 ,  21 h.
</article>
<article titre="Don Papa : la pépite des Philippines" date="2019 11 27" journal="20 Minutes (site web)">
Rhum Découvrez l'histoire et la recette unique de ce rhum haut en saveurs   Une recette unique et des notes gourmandes, voici les ingrédients qui font de Don Papa un rhum hors du commun. Intégralement produite aux Philippines, cette boisson de caractère surprend tant pour ses arômes particuliers que pour ses origines. Bienvenue à Sugarlandia    Don Papa est indissociable de l'île dont il est originaire et sur laquelle se déroule son processus d'élaboration. Cette île, c'est celle de Negros qui fait partie de l'archipel des Philippines. Considérée comme le berceau de la canne à sucre, l'île est vite surnommée Sugarlandia par ses habitants. Le sucre y est omniprésent, de l'alimentation jusqu'à la production d'électricité assurée grâce à la combustion de la bagasse, un résidu fibreux issu du broyage de la canne à sucre. Quant à la production de rhum, elle a débuté dans les années 1850 lors du développement de l'industrie sucrière. Les Philippines sont depuis devenues l'un des plus gros producteurs de rhum.    La mélasse, obtenue lors du filtrage du jus de la canne à sucre, est l'ingrédient initial du Don Papa. - Don Papa Le « black gold »    Ce succès, l'archipel le doit en grande partie à sa matière première. Alors que les rhums des Antilles françaises sont élaborés à partir de pur jus de canne, Don Papa est confectionné avec de la mélasse, un dérivé obtenu lors du filtrage du jus de la canne à sucre. Cet ingrédient, appelé localement « black gold », rend le rhum plus riche, légèrement sucré et fruité. Jérôme Ardès, brand ambassador rhum chez Dugas, confie qu'il s'agit d'un produit unique : « Il n'y a rien aujourd'hui sur le marché qui ressemble, au niveau gustatif, au Don Papa. » Après extraction, la mélasse est fermentée, distillée puis vieillie pendant sept ans minimum dans des fûts de bourbon américain. Le résultat ? L'emblématique Don Papa 7 aux notes de vanille, miel et fruits confits. « C'est un rhum particulier, très aromatique avec des notes vanillées et d'agrumes. C'est une marque qui plaît beaucoup car il permet de pénétrer avec facilité dans le monde du rhum premium. » précise Jérôme Ardès. Ces saveurs ont séduit les palais français qui aiment déguster le rhum sec, faisant de la France le premier consommateur mondial de la marque (1). Pour les fêtes, les puristes pourront justement retrouver le produit phare de la marque dans un coffret de dégustation avec un verre orné d'un tarsier, le primate caractéristique des Philippines. A moins que vous ne préfériez le dernier né : le Don Papa Sevillana aux saveurs parfumées d'écorces d'orange, d'arômes de noisettes et de fruits secs.   Source : (1) Données Don Papa    Le Darker Don, le cocktail de vos fêtes    Devenus incontournables, les cocktails ont plus que jamais le vent en poupe. Le rhum avec ses notes sucrées est l'alcool phare de bien nombre de recettes. Pour varier des désormais classiques Mojito, Piña Colada ou encore Daïquiri, Don Papa vous propose de découvrir pour les fêtes le Darker Don.   Difficulté : facile   Temps de préparation : 3 minutes   Ingrédients :    4 cl de Don Papa 7 ½ citron vert Ginger Beer Fentimans Menthe    Préparation :    Dans un verre Highball, ajouter des glaçons, le Don Papa 7 et le citron vert. Allonger le tout avec la Ginger Beer Fentimans. Décorer avec un bouquet de menthe. Ce contenu a été réalisé et fourni par Don Papa  Cet article est paru dans 20 Minutes (site web)
</article>
<article titre="Unité CSR à La Réunion : Ileva lance une concertation sur fond d'incertitudes" date="2019 11 26" journal="Actu-Environnement (site web)">
Du 21 octobre au 12 décembre 2019 se tient la concertation dédiée à l'unité de co-incinération de combustibles solides de récupération (CSR) du Syndicat mixte de traitement des déchets des microrégions ouest et sud de La Réunion (Ileva). Cette unité CSR a été retenue en 2016 dans le cadre du premier appel à projets CSR de l'Agence de l'environnement et de la maîtrise de l'énergie (Ademe).   Le projet, dont le coût avoisine les 260 millions d'euros (M€), devrait recevoir une aide de 14 M€ de la part de l'Ademe au titre de l'appel à projets CSR. À cela s'ajoutent d'autres aides (Feder et accompagnement des sites de tri par l'Ademe), qui réduiront la facture à 216 M€ (hors intérêts). Ileva prévoit d'obtenir les autorisations administratives mi 2020 pour inaugurer l'installation début 2023. Réduire l'enfouissement Ileva collecte et traite les déchets de quinze communes représentant plus de 500 000 habitants. En 2018 le syndicat a traité 390 000 tonnes (t) de déchets. L'essentiel (227 000 t) est enfoui sur le site de Pierrefonds à Saint-Pierre. Pour le reste, 100 800 t de déchets verts font l'objet d'une valorisation matière et organique, 32 000 t d'emballages recyclables sont traitées dans les centres de tri et 20 800 t de déchets recyclables sont exportées. La saturation de la décharge de Pierrefonds, ouverte il y a trente ans, étant annoncée pour 2022, le syndicat veut réduire de moitié l'enfouissement d'ici 2025, par rapport à 2010. Il est certes possible d'envisager la création d'un nouveau casier, mais cela entraînerait une hausse du coût de traitement des déchets : 15,6 M€ aujourd'hui, 26,3 M€ en 2022 et 29,7 M€ en 2025.   Pour réduite l'enfouissement à 45 000 t par an, Ileva propose un pôle qui associera trois filières : le tri des matières recyclables, la méthanisation des biodéchets et la valorisation énergétique de CSR qui constitue le coeur du projet. L'unité CSR devrait produire 90 gigawattheures (GWh) d'électricité par an (selon l'étude de faisabilité financière). Elle sera alimentée par trois installations de production de CSR : un site de tri sur ordures ménagères résiduelles (OMR), qui produira 90 000 t de combustibles par an, et deux unités de production à partir d'encombrants, de refus de tri et de déchets d'activités économiques (13 000 t et 19 000 t de CSR par an).   Vendre l'électricité ou augmenter la fiscalité Reste un point crucial à régler : assurer l'équilibre financier du projet. Un rapport du ministère de la Transition écologique sur la gestion des déchets sur l'île de La Réunion, publié fin 2018, explique que les recettes annuelles du site devront s'élever à 27,7 M€. Deux financements sont possibles : une hausse des recette fiscales et la vente de l'électricité. Ce dernier financement est donc capital. L'étude de faisabilité financière d'Ileva se base sur un tarif de vente compris entre 90 et 200 euros par mégawattheures (€/MWh), ce qui générerait des revenus annuels compris entre 8,1 et 18 M€. Plus précisément, le syndicat juge   « tout à fait envisageable »   d'obtenir 140 €/MWh d'EDF, puisqu'il s'agit du tarif accordé aux unités de combustion de bagasse ou de charbon.   Mais, pour l'instant, les négociations avec EDF semblent patiner. En octobre 2018, le député David Lorion (LR, La Réunion) a interpellé le ministère de la Transition écologique. Il estime que la Commission de régulation de l'énergie (CRE) doit fixer au plus vite le tarif d'achat. Problème : la CRE ne peut pas autoriser EDF à acheter l'électricité de l'unité CSR si le projet n'est pas inscrit dans la programmation pluriannuelle de l'énergie (PPE) réunionnaise pour la période 2019-2028. Or cela n'est pas acquis, car le conseil régional s'oppose au projet d'Ileva. En conséquence,   « le président de la CRE [a conseillé] au président d'Ileva de ne pas signer le marché »  .   Stratégie régionale « zéro déchet »  Le virage date de 2018 lorsque le président du conseil régional a proposé d'engager l'Île dans une démarche « zéro déchet » , en rupture avec les scénarios envisagés jusque fin 2017. Le rapport ministériel confirme que   « dans les négociations en cours entre l'État et la Région (...), la Région a fait part récemment de sa volonté de ne plus prévoir de production nouvelle [d'énergie] issue de la filière "déchets" »  . La Région a d'ailleurs déposé un recours auprès du tribunal administratif, notait en mai l'équipementier et ensemblier CNIM, qui venait de remporter le marché public. Les nouveaux objectifs régionaux sont «  hors de portée aux échéances prévues »  , estime le document, jugeant   « souhaitable »   le recours à la valorisation énergétique et son inscription dans la PPE.    « Le porteur du projet [Ileva] ne dispose pas d'informations sur l'avancée et le contenu de cette révision »  , déplorait en avril dernier Viviane Malet, sénatrice LR de La Réunion. Répondant à l'inquiétude de l'élue, Emmanuelle Wargon, confirmait alors que le volet relatif à la production électrique à partir de déchets   « est un point qui est pour l'instant resté en suspens »  .   La position de la secrétaire d'État à la Transition écologique et solidaire était ambiguë :   « sans remettre en cause l'objectif de la Région, je souhaite qu'une réponse puisse être apportée, sans attendre 2030, à la problématique des déchets à La Réunion. La PPE révisée devra intégrer cette orientation de l'État. »   Mais pour l'instant les élus du conseil régional restent fermes. Une motion, adoptée en septembre,   « [demande] au gouvernement français de réévaluer le modèle économique ultramarin de l'incinération de déchets-CSR à but unique de production électrique »  .  Cet article est paru dans Actu-Environnement (site web)
</article>
<article titre="Ils veulent développer les bouteilles recyclables" date="2019 11 16" journal="Ouest-France  La Roche-sur-Yon ; Les Sables d'Olonne ; Challans, Saint-Gilles-Croix-de-Vie">
L’initiative   La Chapelle-Achard.  La bouteille plastique qu’ils veulent promouvoir est bien particulière : elle est fabriquée à partir d’éléments végétaux et elle est recyclable à volonté.   «  Nos bio plastiques sont 100 % compostables ou recyclables !  »,  prêchent les deux chapelais, Lucien Kerisit et Ariane Diehly. Ils les comparent aux PET, ces plastiques durs notamment utilisés en masse par les distributeurs d’eaux minérales,  «  qui demandent près de 400 ans pour se décomposer !  ». Ces deux entrepreneurs issus du commerce et de la communication, veulent mettre en avant les alternatives aux plastiques traditionnels.   «  On le sait. Nos plastiques se retrouvent intact dans le corps des animaux  ».  Le recyclage est une obligation,   «  c’est un changement total de nos habitudes  ». Avec des déchets contenant du sucre   Déjà, Lucien Kérisit a collaboré dans des laboratoires et des start-up gravitant autour des bioplastiques. Il n’ignore rien de leur processus de fabrication, un process libre et non protégé, mais   «  sans concurrence notable actuellement  »,  assure-t-il.   Le processus de fabrication est étonnamment simple. Loin des plastiques issus du pétrole, leur bouteille est conçue à partir de matières premières purement organiques.   «  On utilise des déchets contenant du sucre, issus de la canne à sucre, de la betterave ou encore du maïs, et qui sont souvent non valorisés  »,  explique Lucien Kérisit, en désignant quelques résidus de fibres brunâtres dans une assiette.   Alors que   «  le PET n’est recyclé que seulement trois ou quatre fois  »,  l’élément végétal de ces bouteilles assure une décomposition future certaine, mais aussi un recyclage répété à volonté, soit pour de nouvelles bouteilles, soit pour fabriquer d’autres objets bio plastiques.   Les parcs à thèmes, les festivals   La cible est bien cernée. «   Nous visons l’événementiel comme les parcs à thèmes, les festivals, l’hôtellerie de plein air.  »  Objectif : leur proposer un service complet avec fabrication de bouteilles, mise à disposition d’un conteneur, collecte et recyclage, via les outils et le savoir-faire de partenaires spécialisés et basés en France.   «  Nous allons positionner des collecteurs de couleur noire, à disposition du public, pour y recueillir nos bouteilles recyclables.  »  Pour promouvoir son activité, le duo arpente les salons à La Rochelle, Montpellier… Il agit auprès des établissements scolaires :   « Nous avons récemment participé à la démarche E3D du collège Jacques-Laurent des Achards en faveur du développement durable. »  Tout est prêt, les fonds réunis. Kerhea sera le nom de cette entreprise hors norme qui commercialisera leurs produits bioplastiques sous les marques protégées de KO, KLO et Klean I. À titre d’exemple, dans un premier temps,   «  le prix de la gourde sera situé autour de 20 €  »  .    Le design de leur bouteille ou de leur gourde sera l’œuvre d’une équipe d’étudiants de l’École de Design de Nantes. L’étiquette est comme une signature, elle porte la marque de Coralie Joulin, une artiste rochelaise, connue en Vendée pour sa collaboration avec La Perle des Dieux, basée à Saint-Gilles-Croix-de-Vie.   « Elle a dessiné un K, K comme Kerhéa. »  Dans un second temps, Kerhea souhaite fabriquer elle-même ses bouteilles et cherche un local  «  avec recrutement personnel à la clef, idéalement, sur le secteur des Achards  »,  indiquent-ils.  Contact :  07 71 01 38 86.  Cet article est paru dans Ouest-France
</article>
<article titre="« La question de l'euphorie se pose surtout sur les marchés américains »" date="2019 11 14" journal="Investir (site web)">
Rémi Le Bailly :  Bonjour à toutes et à tous. Très heureux de vous retrouver pour notre 4 à 5 du lundi ! On commence tout de suite. Georges N : Bonjour, Monsieur Le Bailly. Je voulais savoir si on peut souscrire pour plus de 5.000 euros à la privatisation de FDJ et dans ce cas bénéficie-t-on quand même des mêmes avantages ? Que se passe-t-il pour le surplus ? En fait, a-t-on intérêt à le faire ? Merci.   Les questions sur le sujet étaient très nombreuses la semaine dernière. Il y en a moins cette fois, il est vrai que le temps presse si vous voulez participer à cette privatisation puisque l'opération se termine demain soir.   La demande a été très forte. Hier, Bruno Le Maire a indiqué sur BFM que « plus d'un milliard d'euros » d'actions avaient été réservées par les particuliers. Or, même si aucun chiffre précis n'avait été donné, on estimait à 600 millions la part de l'opération réservée aux particuliers. Sachant qu'il reste encore deux journées, la demande totale des particuliers devrait se situer au-delà de 1,2 milliard.   Cela signifie, en pratique, que seuls les ordres prioritaires A1, c'est-à-dire inférieurs à 5.000 euros, devraient être servis et ce avec une réduction de l'ordre de 50%, voire plus.   Si vous avez demandé pour 5.000 euros d'actions, vous devriez en avoir l'équivalent de 2.500 euros.   Même si elle sera donc sans objet réel (puisque, comme je viens de le dire les ordres au-dessus de 5.000 euros ne seront sans doute pas servis), je réponds tout de même à votre question.   Oui, on peut souscrire des ordres A au-delà de 5.000 euros mais ils ne sont prioritaires que jusqu'à 5.000 euros. En particulier, l'attribution d'actions gratuites (1 pour 10 au bout de 18 moins) ne fonctionne que sur les ordres A1, jusqu'à 5.000 euros.   En revanche, la décote de 2% s'applique pour tous les ordres des particuliers, A1, A2 ou B.   Pour répondre à la seconde partie de votre question. Je ne pense pas qu'il y ait d'intérêt à investir plus de 5.000 euros... mais cela ne vous pénalise pas non plus.   Nous aurons sûrement l'occasion de reparler à nouveau de FDJ la semaine prochaine, que ce soit pour parler des premières cotations ou du nombre d'actions finalement reçues par les particuliers...   LAURENT : Avec les records de la Bourse, celle-ci entre-t-elle en phase d'euphorie comme en 1999 ?   L'euphorie n'a pas la même ampleur qu'en 1999. Le Cac 40 s'était envolé de plus de 50%...   On en est encore loin. Pour l'heure, la hausse actuelle est plus comparable à celles enregistrées en 2005 et 2006 avec des progressions de l'indice de 23,4% et 17,5%.   Mais vous avez raison, la question de l'euphorie des marchés se pose tout de même surtout si on regarde les marchés américains qui ont monté en ligne droite depuis 2009 (le Dow Jones a été multiplié par 4,3).   Si les valorisations sont tendues, elles ne sont pas réellement dans la zone rouge car les grandes sociétés sont de mieux en mieux gérées et augmentent donc leurs bénéfices en dépit d'une croissance économique mondiale médiocre. Par ailleurs, les rachats d'actions et les politiques de dividendes soutiennent les marchés.   Mais la hausse récente, qui est principalement liée à la forte baisse des taux d'intérêt (ce n'est pas un hasard si la remontée des marchés à partir du début de 2019 coïncide avec le revirement de la Fed qui envisageait jusque-là de remonter ses taux directeurs), ne supporterait pas la moindre mauvaise nouvelle.   Il faut donc rester présent sur le marché car la hausse peut se poursuivre... faute de placement alternatif plus attrayant. Mais il faut aussi être prêt à sortir rapidement si la machine commence à s'enrayer... car alors la baisse pourrait être rapide.   Je l'écris souvent dans ce chat : il ne faut pas avoir raison trop tôt.   Bruno : Bonjour Monsieur. Pourquoi votre hebdomadaire oublie-t-il de mentionner l'acompte de dividende prévu début décembre pour Orange ? Pourquoi toujours pour Orange, n'avez-vous pas développé le projet qui parle de la vente des tours mobiles ? Quel impact pour la société et l'évolution du cours de Bourse ! Cordialement.   Effectivement, c'est une erreur que nous allons corriger dès cette semaine. Orange détachera un acompte le 2 décembre de 0,30 euro sur un dividende total qui devrait rester stable à 0,70 euro.   Quant à votre seconde question, nous avons évoqué le sujet dans le commentaire de la semaine consacré à Orange dans notre cahier cote.   Il s'agit pour l'instant d'une information, non confirmée du Financial Times. On en saura plus le 4 décembre lors de la « journée investisseurs » d'Orange. Un peu partout dans le monde, les groupes de télécommunications vendent leurs tours à des fonds ou des sociétés spécialisées dans les infrastructures.   En France, Iliad ou Altice l'ont fait. Jusqu'à présent, Orange paraissait vouloir conserver cet actif... mais il pourrait changer d'avis. On parle d'une valorisation de 10 milliards d'euros à comparer à une dette de 25 milliards et à une capitalisation boursière d'un peu moins de 40 milliards.   L'action est en légère hausse depuis que ces rumeurs circulent, mais rien d'extraordinaire. On peut penser néanmoins qu'une vente effective à un bon prix serait bien perçue. Mais tout dépendra des autres indications qui seront données lors de cette « journée investisseurs » .   Patlune : Bonjour. Dans le domaine des énergies, je connais 3 valeurs moyennes qui sont NEOEN, ALBIOMA et VOLTALIA. Je souhaiterais en acheter une. Quelle valeur me conseillez-vous ? Merci.   Nous sommes à l'achat de ces trois valeurs. Albioma est la plus sûre mais la moins « verte » . Ces centrales fonctionnent, certes, avec de la bagasse mais en période creuse, elles fonctionnent aussi avec des énergies fossiles.   La société a un historique solide et après la sortie des fonds du groupe Altamir, le capital est à présent stabilisé.   En numéro deux, je place Voltalia pour sa dynamique de croissance. La société confirme régulièrement ses objectifs à court et moyen terme et le soutien de la famille Mulliez, actionnaire du groupe, est un gage de solidité.   Yoni : Bonjour Rémi, vous m'aviez décidé il y a quelque temps à acheter du Bilendi. Où en sommes-nous de ce dossier, cela fait longtemps que vous ne l'avez pas traité j'ai l'impression. Sur le site, on est encore dans l'attente de la dernière assemblée générale. Merci.   L'assemblée a eu lieu, elle a traduit le conflit entre la direction et le groupe Qwamplify, qui détient 26% de son capital. Cet actionnaire minoritaire a bloqué les demandes d'autorisation d'augmentation de capital et les autres actionnaires ont voté contre la demande de Qwamplify de versement d'un dividende exceptionnel   Cela risque de bloquer de grosses opérations de croissance externe que la société pourrait vouloir réaliser. Mais cela n'empêche pas la société d'afficher de bonnes performances opérationnelles comme l'ont montré les résultats semestriels.   Nous restons à l'achat de la valeur.   Contrariant: Dans l'hebdo, Investir est à l'achat avec un objectif de 110 euros... Et sur le site internet, vous êtes à l'écart d'Ingenico. Quelle est la vraie position d'Investir sur Ingenico ?   C'est une erreur de notre part. Les articles de l'hebdomadaire « remontent » normalement sur le site la semaine suivante. Nous avons changé de conseil lors de l'interview publiée à la fin du mois d'octobre. Nous sommes redevenus acheteurs du titre. L'interview est « remontée » mais pas l'encadré où figurait le conseil. Ce « bug » a été corrigé.   ALAIN : Bonjour Monsieur Le Bailly. Voilà maintenant quatre ans qu'Innovox est en liquidation judiciaire. Gowex 6 ans que cette société est en liquidation judiciaire également.Comment expliquer que l'AMF n'ait toujours pas décidé d'enlever définitivement ces deux valeurs de la cote ? Ces 2 valeurs à 0 euro encombrent mon PEA et l'aimerais bien que ces valeurs disparaissent. D'avance merci pour votre réponse.   C'est un problème récurrent sur lequel nous avons souvent alerté l'AMF mais les choses n'évoluent pas. Pour qu'une action puisse être radiée et sortir du compte-titre ou du PEA, il faut que la société après liquidation judiciaire soit radiée du Registre du commerce. Ensuite Euronext la radie de la cote.   Pour Gowex, le jugement de clôture de la procédure de liquidation judiciaire pour insuffisance d'actif est intervenu en fin d'année dernière... le dossier avance donc mais tout cela est effectivement beaucoup trop long. Vous pouvez éventuellement contacter les services de l'AMF.   Epictete : Bonjour M. Le Bailly, je repose ma question non traitée. En décembre 2017, sur vos conseils j'ai acheté des titres ASML Holding. A ce jour, ma plus-value est de 68%. J'ai déjà fait des ventes partielles. J'envisage de solder ma position et d'acquérir des ETF Trackers ou des Fonds éligibles PEA sur : le marché chinois (ou asiatique) et/ou sur les marchés émergents et/ou sur le Nasdaq (ou S&amp;P). Est-ce le bon timing ? Si oui, quoi acheter en priorité et quels trackers ou Fonds (PEA) ont vos préférences ? Vos conseils seront mes arbitrages.   Si votre portefeuille n'est pas diversifié sur le plan géographique, cela peut effectivement être une bonne idée de vous orienter vers les marchés américains et émergents. Les principaux émetteurs français d'ETF proposent des trackers internationaux éligibles au PEA alors que les fonds internationaux classiques ne sont pas éligibles.   Par exemple, chez Lyxor, il existe un tracker Amérique (MSCI USA) de code FR0011869346 ou un tracker Nasdaq 100 (FR0011871110) pour la zone Amérique.   Pour les émergents, je pense qu'un tracker Asie est plus adapté qu'un tracker global émergent. Il y a le PEA Asie-Pacifique de code FR0011869312.   Plus globalement, je vous conseille, pour faire votre choix, d'aller sur les sites des trois émetteurs français (Lyxor, BNP et Amundi) et de demander la liste des trackers éligibles au PEA.   Thierry : Bonjour. Sauf erreur Deutsche Wohnen a fait partie d'Investir 10. La recommandez-vous toujours, et pouvez-vous nous donner une explication sur le dividende affiché de 0,66% qui paraît ridicule pour une foncière ? Plus généralement, envisagez-vous de faire un dossier sur les foncières européennes et spécialement celles d'Euronext pour des raisons de frais de courtage ? Merci.   Effectivement, nous n'avons pas parlé depuis longtemps de Deutsche Wohnen et nous reviendrons prochainement dans l'hebdo sur cette valeur. Nous sommes désormais plus prudents compte tenu du blocage des loyers décidé à Berlin qui pourrait s'étendre à d'autres villes.   Merci à tous pour vos questions. Comme toujours je n'ai pas pu répondre à tous, faute de temps. N'hésitez pas à les reposer. La persévérance paie : j'ai répondu aujourd'hui à plusieurs questions qui m'avaient été posées la semaine dernière mais qui étaient passées au second plan compte tenu des nombreuses interrogations sur la privatisation de FDJ.   A lundi prochain et bonne semaine à toutes et à tous.   Investir
</article>
<article titre="Marché de Noël et patinoire passent au zéro déchet Deux exceptions tolérées Ces objets seront liégeois" date="2019 11 13" journal="La Capitale (site web)">
Lecturezen   Dès cette année, la vaisselle en plastique à usage unique sera interdite sur les marchés de Noël de Liège. Les assiettes, couverts, gobelets transparents, pailles, serviettes, gobelets à vin chaud, et aux godets à pèkèt seront remplacés par de la vaisselle bio compostable. Les déchets collectés seront ensuite transformés en compost et en gaz.   Vous ne dégusterez plus jamais votre traditionnelle tartiflette de la même manière. Dès cette année, vous la mangerez en utilisant des couverts et des assiettes bio compostables ! La raison ? Les marchés de Noël de la Cité ardente passent en mode zéro déchet. La vaisselle en plastique à usage unique sera donc purement et simplement interdite. Cette nouvelle réglementation s’applique aux assiettes, couverts, gobelets transparents, pailles, serviettes, gobelets à vin chaud, et aux godets à pèkèt. Désormais, ces objets seront remplacés par de la vaisselle bio compostable. L’initiative est lancée par la Ville de Liège, en collaboration étroite avec le Village de Noël de la place du Marché, et celui de la patinoire : « La pollution engendrée par ce type de vaisselle est extrêmement importante de par sa fabrication. C’est donc un grand pas en avant que nous avons amorcé. On tenait vraiment à s’emparer de la problématique écologique ainsi qu’à driller et à préparer nos exposants à prendre ce virage écologique. En passant au zéro déchet, on anticipe aussi la directive de l’Union Européenne qui interdira bientôt les plastiques à usage unique. Notons que le carton, le verre, et les plastiques réutilisables seront toujours autorisés », explique Pierre Luthers, directeur de l’ASBL Enjeu, responsable du Village de Noël. « C’est un énorme virage que nous allons prendre et il est fondamental. Nos affiches étaient déjà réalisées avec du papier recyclé. Même chose pour les tickets de la patinoire. Mais là, on passe vraiment à une étape supérieure », ajoute Pierre Fontaine, responsable de Dynamic Events et gérant du marché de Noël de la patinoire.   Plus en plastique. - TVA   Ce virage écologique est colossal pour les exposants, qui devront s’adapter. Pas évident de remplacer les éternelles assiettes en plastique et les gobelets à bière. Pour faciliter cette transition, une batterie de solutions a déjà été soumise aux différents chalets. Ainsi, on y retrouve des fourchettes, couteaux, cuillères à potage et à café, fabriqués uniquement en matériel biodégradable. Mais aussi des objets plus étonnants comme les assiettes, réalisées à base de pulpe de bagasse, un résidu fibreux de la canne à sucre obtenu après extraction du suc. Ou encore des gobelets à bière transparent créés en acide polylactique qui est produit à partir de la fermentation de sucres issus de la production de maïs, de betterave, de tapioca, ou encore de canne à sucre.   Déchets transformés en gaz   Mais le défi est de taille ! Rien que sur le Village de Noël, les plastiques à usage unique ne représentent pas moins de 55 à 60 tonnes de déchets ! Une bonne nouvelle écologique donc, puisque l’ensemble de ces objets pourra être évacué via des sacs verts, et/ou, des containers organiques qui seront mis à disposition des exposants : « Des îlots de tris seront également installés pour les visiteurs, mais nous aurons moins de contrôle sur eux. Il suffit qu’une personne se trompe d’emplacement pour que la poubelle et le futur compost soient pollués. Nous seront attentifs aux comportements qui seront adoptés. Tout ne se fera pas en un jour, mais c’est un message important qui est envoyé », explique Pierre Luthers.   Les déchets collectés seront ensuite acheminés vers une filière de recyclage et seront finalement transformés en compost et en gaz. De quoi manger sa tartiflette sans culpabiliser…   Mardi, Novembre 12, 2019 - 19:34 Par S.B   Parmi la liste des objets interdits, on retrouve deux exceptions. Les verres à vin sur pied et les flûtes à champagne en plastique seront autorisés sur le marché de Noël. Pour le moment, aucune solution écologique n’existe pour permettre leur remplacement. Celles qui existent, coûteraient trop cher aux exposants. Ces deux objets seront tout de même triés et évacués via les nouveaux sacs bleus « P+MC » .   Mardi, Novembre 12, 2019 - 19:33 Par S.B   C’est La Maison Gilson de Barchon qui fournira les exposants du marché de Noël de Liège : « Cette initiative écologique correspond à une tendance qui devient de plus en plus importante. Nous avons commencé il y a deux ans à produire des produits de ce genre et on entend parfois que cela coûte plus cher. C’est en partie vrai. Cela coûte plus cher que du plastique, mais pas autant que l’on pourrait le croire. Il y a deux ans, un produit bio compostable pouvait coûter deux fois le prix d’un objet en plastique classique. Maintenant, c’est moins », explique Roberto Calla, administrateur délégué de la Maison Gilson.   Poursuivez votre lecture sur ce(s) sujet(s): Liège (prov. de Liège) Marche-en-Famenne (prov. de Luxembourg)  Cet article est paru dans La Capitale (site web)
</article>
<article titre="Marché de Noël et patinoire passent au zéro déchet Exclusif" date="2019 11 13" journal="La Meuse (site web)">
Lecture zen Dès cette année, la vaisselle en plastique à usage unique sera interdite sur les marchés de Noël de Liège.   Les assiettes, couverts, gobelets transparents, pailles, serviettes, gobelets à vin chaud, et aux godets à pèkèt seront remplacés par de la vaisselle bio compostable. Les déchets collectés seront ensuite transformés en compost et en gaz. Vous ne dégusterez plus jamais votre traditionnelle tartiflette de la même manière. Dès cette année, vous la mangerez en utilisant des couverts et des assiettes bio compostables ! La raison ? Les marchés de Noël de la Cité ardente passent en mode zéro déchet. La vaisselle en plastique à usage unique sera donc purement et simplement interdite. Cette nouvelle réglementation s'applique aux assiettes, couverts, gobelets transparents, pailles, serviettes, gobelets à vin chaud, et aux godets à pèkèt.   Désormais, ces objets seront remplacés par de la vaisselle bio compostable. L'initiative est lancée par la Ville de Liège, en collaboration étroite avec le Village de Noël de la place du Marché, et celui de la patinoire : « La pollution engendrée par ce type de vaisselle est extrêmement importante de par sa fabrication. C'est donc un grand pas en avant que nous avons amorcé. On tenait vraiment à s'emparer de la problématique écologique ainsi qu'à driller et à préparer nos exposants à prendre ce virage écologique. En passant au zéro déchet, on anticipe aussi la directive de l'Union Européenne qui interdira bientôt les plastiques à usage unique. Notons que le carton, le verre, et les plastiques réutilisables seront toujours autorisés », explique Pierre Luthers, directeur de l'ASBL Enjeu, responsable du Village de Noël. « C'est un énorme virage que nous allons prendre et il est fondamental. Nos affiches étaient déjà réalisées avec du papier recyclé. Même chose pour les tickets de la patinoire. Mais là, on passe vraiment à une étape supérieure », ajoute Pierre Fontaine, responsable de Dynamic Events et gérant du marché de Noël de la patinoire.   Plus en plastique. - TVA   Ce virage écologique est colossal pour les exposants, qui devront s'adapter. Pas évident de remplacer les éternelles assiettes en plastique et les gobelets à bière. Pour faciliter cette transition, une batterie de solutions a déjà été soumise aux différents chalets. Ainsi, on y retrouve des fourchettes, couteaux, cuillères à potage et à café, fabriqués uniquement en matériel biodégradable. Mais aussi des objets plus étonnants comme les assiettes, réalisées à base de pulpe de bagasse, un résidu fibreux de la canne à sucre obtenu après extraction du suc. Ou encore des gobelets à bière transparent créés en acide polylactique qui est produit à partir de la fermentation de sucres issus de la production de maïs, de betterave, de tapioca, ou encore de canne à sucre.   Déchets transformés en gaz   Mais le défi est de taille ! Rien que sur le Village de Noël, les plastiques à usage unique ne représentent pas moins de 55 à 60 tonnes de déchets ! Une bonne nouvelle écologique donc, puisque l'ensemble de ces objets pourra être évacué via des sacs verts, et/ou, des containers organiques qui seront mis à disposition des exposants : « Des îlots de tris seront également installés pour les visiteurs, mais nous aurons moins de contrôle sur eux. Il suffit qu'une personne se trompe d'emplacement pour que la poubelle et le futur compost soient pollués. Nous seront attentifs aux comportements qui seront adoptés. Tout ne se fera pas en un jour, mais c'est un message important qui est envoyé », explique Pierre Luthers.   Les déchets collectés seront ensuite acheminés vers une filière de recyclage et seront finalement transformés en compost et en gaz. De quoi manger sa tartiflette sans culpabiliser…  Cet article est paru dans La Meuse (site web)
</article>
<article titre="MIAM ! Le Pica aux Saintes passe à la table écolo O Pica Pica aux Stes-Maries de la Mer propose poissons et coquillages dans un esprit bistrot et une vaisselle biosourcée" date="2019 10 26" journal="La Provence  AIXPR ; ALPES ; ARLES ; AUBAG ; AVIGN ; ETANG ; MARTI ; SALON ; SUVSE ; VILLE">
Frédéric Beltra est arrivé de Sète, aux Saintes-Maries de la Mer, tout imprégné de l'esprit convivial de son quartier de la Pointe Courte, et digne héritier d'un père et d'un grand-père ostréiculteurs et pêcheurs, il y a 18 ans. Il n'a jamais changé de quartier depuis. Mais son "royaume", en face des arènes, s'est agrandi. Dans un petit local, au départ, avec un bon couteau et une bourriche, il s'est d'abord agi de vendre des coquillages. Mais la clientèle a eu vite fait de demander 6 huîtres à déguster sur le pouce, avec un petit verre de blanc. Frédéric n'avait pas de couverts, encore moins de verres, un client est allé à l'épicerie pour chercher le nécessaire. Du tout jetable. Du tout plastique. Le concept est resté. Quand "La cabane aux coquillages" s'est agrandie aux murs d'à-côté pour qu'ouvre " O Pica-Pica" : sur les tables en bois, en terrasse sur la rue, ou dans le jardin décoré d'antiquités surprenantes, voire dans la salle que surplombent les filets de pêche de famille, la vaisselle jetable est restée. Frédéric et toute son équipe s'y retrouvaient. Oui la vaisselle était à usage unique, en plastique, mais aucun produit ne risquait dans les lavages à répétition de repartir avec des saletés dans la mer... Un point de vue. Puis la table était bonne. Certains n'ont jamais voulu adhérer en buvant dans un gobelet le vin blanc bio "Le Nouveau Monde". Qu'importe ! O Pica Pica on mange du poisson frais,  "et de saison" , à l'ardoise, des petites fritures - comme les  chipirones  - mais dans une vaisselle à usage unique. Convivialité et simplicité sont les mots d'ordre, chacun dégustant à son rythme. Qui quelques huîtres et crevettes seules, qui un poisson entier, avec ou sans dessert...   Depuis cet été la maison a franchi un cap important en obtenant le label Zeapack, parce qu'elle n'utilise désormais sur ses tables que des couverts, serviettes ou gobelets à base de matières biosourcées (carton, PLA, bagasse, bambou, palmier ou bioplastique).  "Nous avons toujours fait le choix de la qualité des produits en privilégiant les prix abordables. Ne pas avoir de services de table  appartient à cette démarche. Puis ça sert à quoi d'ouvrir des mines supplémentaires pour faire des couverts ? Il a fallu du temps pour que nous trouvions cette société à Montpellier qui répond à nos attentes. C'est pour moi la solution totale. Les matériaux sont pris dans la nature, ont une durée de vie courte, et en 48 mois il n'y a plus rien. Le tout pour un tout petit impact économique pour nous !"
</article>
<article titre="Une valorisation balbutiante Objet de recherche et de développement depuis 2015, la valorisation des sargasses est, pour l'heure, essentiellement limitée à la fabrication de compost en Martinique." date="2019 10 22" journal="La Croix, no. 41539">
«A ujourd'hui, aux Antilles françaises, il existe plusieurs projets, financés en grande partie par l'Agence de l'environnement et de la maîtrise de l'énergie (Ademe), pour valoriser les algues sargasses , indique Pascal Jean Lopez, biologiste marin au Muséum national d'histoire naturelle (MNHN).  Mais le seul opérationnel aux Antilles est celui mené par la société Holdex Environnement en Martinique.»  Il consiste à mélanger 10 à 15% de sargasses séchées à des déchets verts comme la bagasse de canne à sucre (résidu fibreux), des fientes de poules et des coquilles d'oeufs, pour en faire du terreau agricole (compost).   Toutefois, comme les zones bananières de Martinique (nord-est et sud-est) et de Guadeloupe (sud de l'île de Basse-Terre) sont contaminées par un insecticide, le chlordécone, classé cancérogène pour l'homme par l'OMS et interdit aux Antilles depuis 1993 mais encore présent dans certains sols, fleuves et zones littorales, on risque de récupérer des sargasses qui sont polluées en chlordécone. Idem avec l'arsenic, un métalloïde qui vient naturellement de la mer, et dont une forme est toxique.  «Moins grave est le cas du mercure, dont globalement la concentration est faible, pas différente de celle de la haute mer, excepté en quelques points de la côte»,  indique Thierry Thibaut, écologue marin à l'université d'Aix-Marseille. Pourtant, moyennant de rigoureuses précautions, la présence de ces substances indésirables peut être jugulée, de façon que  «la sargasse devienne une ressource et ne reste pas un déchet» . Concernant le chlordécone, il est essentiellement présent dans les algues qui ont stagné un certain temps dans des baies où elles se sont imprégnées d'insecticide. Exit alors ces algues non fraîchement arrivées du large et donc non valorisables. Pour ce qui est des substances métalliques, arsenic et mercure, les algues se nourrissant en partie en filtrant l'eau de mer, un peu comme les coquillages, il n'est pas étonnant d'en trouver des traces au sein de leur matière organique (glucides).   La solution consiste donc à les diluer et à les mélanger avec d'autres composés pour que leur taux soit inférieur au seuil de sécurité. C'est ce que fait, en Guadeloupe puis à Saint-Malo, la société Algopack qui, à partir de poudre micronisée d'algue, fabrique des produits (pots de fleurs, cartes bancaires, articles de bureau, emballages...) contenant 20%, 30% ou 50% de sargasse.  «Le produit obtenu a des caractéristiques identiques voire supérieures à un plastique à base de pétrole»,  assure David Coti, président d'Algopack. De son côté, en Martinique, Holdex a réussi à se débarrasser de l'arsenic, ou à le neutraliser grâce à un processus qu'elle tient secret.   Reste enfin des valorisations qui sont encore en cours d'expérimentation comme la méthanisation - une fermentation en absence d'oxygène - pour faire du biogaz, l'extraction de glucides (alginates) et de lipides pour la cosmétique humaine, l'alimentation du bétail voire la pharmacie (polysaccharides sulfatés aux propriétés anti-inflammatoires). Pour l'heure, malgré les difficultés imprévues comme la présence durable de chlordécone, on arrive à faire preuve d'imagination pour recycler les sargasses. Pour la plus grande satisfaction des Antillais.
</article>
<article titre="Les fabricants d'emballages en plastique appellent à l'augmentation de la collecte" date="2019 10 21" journal="Process Alimentaire (site web)">
Economiquement impactés par le « plastic bashing », les fabricants d'emballages en plastique réunis au sein d'Elipso rappellent les actions menées par la profession en faveur d'une économie circulaire. Ainsi que l'impérieux besoin d'améliorer la collecte pour intégrer plus de recyclé, notamment à travers la consigne pour recyclage.   Avancer dans la transition écologique avec un discours raisonnable et raisonné. Voilà ce à quoi aspire le secteur de l'emballage plastique représenté, en France, par l'association Elipso qui a tenu sa conférence de presse annuelle le 17 octobre dernier. Pour Françoise Andres, sa présidente, il n'est pas question de nier ni de cautionner la pollution des océans par le plastique. «Mais l'emballage plastique est devenu l'unique responsable visible. Dans ce contexte d'hypermédiatisation, et j'oserais dire de diabolisation des plastiques, il est important que le secteur de l'emballage prenne la parole pour rappeler tous les engagements qu'il a déjà pris et qu'il compte poursuivre en faveur d'une économie circulaire », avance-t-elle. Déplorant, au passage, que le politique préfère interdire plutôt que d'encourager l'amélioration continue. "Le temps industriel n'est pas le temps politique"   «Nous ne pouvons pas tout bouleverser en quelques semaines. Le temps industriel, de mutation des machines, des hommes, de la formation, n'est pas le temps politique », regrette-t-elle. Les premiers effets du climat anti-plastique se sont d'ailleurs déjà fait sentir : selon l'étude menée par Elipso auprès de ses adhérents, 40 % ont vu leur chiffre d'affaires impacté à la baisse cette année, contre 20 % l'an passé.    Cette enquête tient également à prouver que les efforts de la profession sont en marche. 72 % des répondants disent intensifier leurs efforts en recherche et développement pour améliorer la recyclabilité de leurs emballages. 30 % se diversifient vers d'autres matériaux . « C'est un choix souvent dicté par les clients mais qui conduit parfois à augmenter l'impact carbone », tempère Christophe Rossé de chez Klöckner Pentaplast, vice-président de la commission économie d'Elipso. Gilles Istin, dirigeant de Knauf Industries, cite l'exemple d'un acteur de la fraise dans le sud de l'Espagne qui, en passant sur des barquettes en carton, a divisé par deux la durée de vie de ses produits. Et serait, depuis, revenu à de la barquette en plastique. De quoi, au passage, s'interroger sur l'utilité de faire venir des fraises d'Espagne à la saison... Mais là est une toute autre question ! « Autre exemple, la bagasse venue de l'autre bout de la planète, qui ne répond pas toujours aux exigences d'alimentarité ou qui est parfois blanchie au chlore. Alors même que le plastique est, lui, extrêmement encadré et contrôlé », cite Françoise Andres.   Côté économie circulaire, les acteurs français de l'emballage poursuivent leurs efforts d'intégration de recyclés. Sur les 2,2 millions de tonnes d'emballages plastiques mises sur le marché en France chaque année (50 % en emballages industriels et commerciaux, 50 % en emballages ménagers), 82 % sont issues de matières vierges, contre 86 % en 2018. 14,5 % sont d'origine recyclée, versus 12 % l'an passé, soit un passage de 260 000 t à 320 000 t de recyclés. "Cela équivaut à 800 000 tonnes de de CO2 économisées. Car une tonne de matière première recyclée économise environ deux tonnes de CO2", indique Serge Vassal, dirigeant de l'entreprise Barbier et vice-président d'Elipso. Enfin, 3,5 % sont d'origine biosourcée, soit 79 000 tonnes.   Accroissement nécessaire des gisements de PET recyclé   « L'objectif fixé par l'Europe est de réincorporer 1 million de tonnes de plastiques recyclés dans l'économie en France d'ici 2025, à raison de 450 000 tonnes pour l'emballage (soit 20 % des volumes d'emballages). Nous sommes largement dans l'objectif et y parviendrons même sûrement plus tôt », explique Serge Vassal. Une forte dynamique est observée sur les films en PEBD (polyéthylène basse densité) qui incorporent jusqu'à 50 % de recyclé pour certains films, de quoi bénéficier d'un bonus d'éco-contribution de 50 % selon le barème 2019-2020 de Citeo. Le PET est l'autre résine en plein progrès sur l'intégration de recyclé, cette fois dans les applications aptes au contact alimentaire. « Mais le marché manque de gisements, ajoute-t-il. Seul l'accroissement des capacités de collecte sur l'ensemble du territoire est à même de répondre à la demande d'incorporation. » Pour maximiser ce taux de collecte, Elipso encourage le gouvernement à mener quatre actions et à les intgérer dans le cadre de la loi sur l'économie circulaire actuellement en discussion :   - harmoniser le schéma de tri au niveau national pour accélérer l'extension des consignes de tri   - déployer une consigne pour recyclage des bouteilles en plastique pour faire du plastique non plus un déchet mais une ressource. A condition toutefois, que l'ensemble des acteurs aient accès à la ressource. A l'heure actuelle, le manque de gisement disponible de rPET a dopé le cours du recyclé, le rendant 25 % plus cher que le PET vierge.   - mettre en place une collecte séparée des biodéchets ainsi qu'une filière de compostage industriel avec récupération de méthane.   - maintenir une REP (responsabilité du producteur) volontaire dans le domaine des emballages industriels et commerciaux.   « Nous souhaitons que ce projet de loi pour une économie circulaire donne lieu à des débats apaisés qui prennent en considération les avantages offerts par l'emballage en plastique en matière de protection sanitaire et de préservation des produits. Et donc son rôle dans la lutte contre le gaspillage. Une interdiction arbitraire de certains plastiques mettrait un coup d'arrêt aux nombreuses innovations du secteur de l'emballage en matière d'éco-conception. Nous souhaitons que ce projet de loi permette à nos 320 entreprises et à leurs 38 000 collaborateurs de confirmer leur place dans cette nouvelle économie circulaire », conclut Françoise Andres. La première lecture du texte à l'Assemblée nationale est prévue la semaine du 25 novembre 2019.  Cet article est paru dans Process Alimentaire (site web)
</article>
<article titre="Climat: un sommet africain des acteurs non étatiques à Accra" date="2019 10 14" journal="Journal de l'environnement (site web)">
Du 16 au 18 octobre, Accra (Ghana) accueillera un nouveau sommet climatique des collectivités, associations, organismes scientifiques, entreprises et citoyens africains. Avec l'objectif de mettre en oeuvre les engagements pris à Abidjan un an plus tôt. Alors que l'Afrique est l'un des continents les plus menacés par le changement climatique (cf encadré), 2.000 acteurs de la société civile se retrouvent à Accra pour poursuivre leurs actions au niveau sectoriel. « Après les engagements d'Abidjan, il est temps de dégager des moyens humains pour mettre en oeuvre, concrètement, des mesures précises en faveur du climat. Avec une feuille de route renforcée par secteur », explique au JDLE Ronan Dantec, président de l'association Climate Chance organisatrice du sommet. Avant-poste. L'Afrique abrite 8 des 10 villes les plus menacées au monde par le réchauffement climatique en 2050 selon un rapport publié en novembre dernier. Parmi elles : Bangui (Centrafrique), Monrovia (Libéria) et Kinshasa (République démocratique du Congo) où ses 13 millions d'habitants sont victimes d'inondations de plus en plus fréquentes. Des terres aux bâtiments Pour mémoire, plusieurs secteurs-clés de la lutte contre le dérèglement climatique avaient émergé lors du sommet d'Abidjan, en juin 2018. Parmi eux : la ville durable, l'agriculture et la reforestation, l'accès à l'énergie et l'efficacité énergétique, la mobilité durable, l'accès à l'eau, et la construction. Sept priorités transversales Ce premier sommet des acteurs africains s'était aussi conclu par une déclaration ciblant sept priorités transversales: le renforcement des stratégies territoriales, l'accès aux financements, l'intégration du genre dans les politiques publiques pour favoriser l'action des femmes, la participation des jeunes, l'essor de l'éducation et de la formation, la prise en compte des enjeux d'adaptation et le partage de données permettant d'évaluer les émissions de gaz à effet de serre sur le continent. Régionalisation de l'action Jusqu'à l'an dernier, Climate Chance organisait des sommets climatiques visant la communauté non étatique internationale. Ce fut le cas à Lyon en juillet 2015, à Nantes en septembre 2016, et Agadir en septembre 2017. Désormais, l'heure est à la régionalisation. « Depuis la COP21, nous nous apercevons que beaucoup d'acteurs semblent moins intéressés par des sommets autour de grands enjeux globaux que par des mobilisations au niveau local », relève le sénateur écologiste Ronan Dantec. A Accra, la société civile n'apportera d'ailleurs pas de réponse africaine globale au changement climatique mais des solutions propres à chaque pays, comme une stratégie anti-déforestation au Gabon et une mobilisation marocaine en faveur des énergies renouvelables. Bonnes pratiques. 376 projets ont été reçus dans le cadre de l'appel à contribution lancé par Climate Chance. 70 d'entre eux ont été retenus et seront présentés lors du sommet d'Accra. Parmi eux : la société ghanéenne Ecovon propose des matériaux de construction réalisés à partir de bagasse de canne à sucre et de coir de noix de coco.
</article>
<article titre="CASA_IP_Green Bond_October 2019.pdf" date="2019 10 4" journal="Rapports - Banques et institutions financières - Crédit Agricole">
53 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... l1 Green Bond – October 2019  CRÉDIT AGRICOLE GREEN BOND Investor Presentation – October 2019 l2 Green Bond – October 2019  DISCLAIMER   This document has been prepared by Crédit Agricole S.A. on the...    Pour voir le rapport :  https://www.credit-agricole.com/pdfPreview/177156
</article>
<article titre="Télécharger .pdf (3,8 MB) - CASA_IP_Green Bond_October 2019.pdf" date="2019 10 4" journal="Rapports - Banques et institutions financières - Crédit Agricole">
53 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... l1 Green Bond – October 2019  CRÉDIT AGRICOLE GREEN BOND Investor Presentation – October 2019 l2 Green Bond – October 2019  DISCLAIMER   This document has been prepared by Crédit Agricole S.A. on the...    Pour voir le rapport :  https://www.credit-agricole.com/content/download/177156/4496510/version/2/file/CASA_IP_Green%20Bond_October%202019.pdf
</article>
<article titre="1909_rapport_greenbond_A4_BD.PDF" date="2019 10 3" journal="Rapports - Banques et institutions financières - Crédit Agricole">
9 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... C RÉ  DI T  AG RI  C O  LE  G RE  EN B  O N  D RE  PO RT  20 19  October 2019 As one of today’s world leaders in...    Pour voir le rapport :  https://www.credit-agricole.com/pdfPreview/177157
</article>
<article titre="Credit Agricole Green Bond Report - 1909_rapport_greenbond_A4_BD.PDF" date="2019 10 3" journal="Rapports - Banques et institutions financières - Crédit Agricole">
9 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... C RÉ  DI T  AG RI  C O  LE  G RE  EN B  O N  D RE  PO RT  20 19  October 2019 As one of today’s world leaders in...    Pour voir le rapport :  https://www.credit-agricole.com/content/download/179900/4566772/version/2/file/1909_rapport_greenbond_A4_BD.PDF
</article>
<article titre="Télécharger .pdf (707 kB) - 1909_rapport_greenbond_A4_BD.PDF" date="2019 10 3" journal="Rapports - Banques et institutions financières - Crédit Agricole">
9 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... C RÉ  DI T  AG RI  C O  LE  G RE  EN B  O N  D RE  PO RT  20 19  October 2019 As one of today’s world leaders in...    Pour voir le rapport :  https://www.credit-agricole.com/content/download/177157/4496530/version/2/file/1909_rapport_greenbond_A4_BD.PDF
</article>
<article titre="Agrocarburants : les voyants sont au vert en Europe" date="2019 09 30" journal="Journal de l'environnement (site web)">
Encouragée par l'Union européenne, la consommation d'agrocarburants dans les transports a connu une croissance à deux chiffres en 2018. Explications. D'après le dernier baromètre biocarburants 2019 d'EurObserv'Er[1], la consommation d'agrocarburants a progressé de 10,1 % dans l'Union européenne en 2018. Elle s'élève à 17 millions de tonnes équivalent pétrole (Mtep), contre 15,4 Mtep en 2017.  « La répartition entre les différents types de biocarburants reste largement dominée par le secteur du biodiesel », précise Eurobserv'er. A lui seul, il représente 82% de la consommation de biocarburants dans l'UE. Viennent ensuite le bioéthanol avec 17,1% et le biogaz 0,9%. En 2018, 13,9 de Mtep de biodiesel ont été consommés, soit une progression de 10,4% par rapport à 2017. La consommation de bioéthanol a augmenté de 3,4%.   Des normes rehaussées   Toujours selon EurObserv'Er, cette augmentation s'explique par : « un relèvement des quotas ou des objectifs d'intégration définis par plusieurs pays » . Une importante modification est intervenue fin 2018, avec l'adoption de la nouvelle directive énergies renouvelables (2018/2001/ EU), dite "RED II". Elle fixe la feuille de route jusqu'en 2030.   La part de l'énergie renouvelable dans la consommation finale d'énergie dans le secteur des transports reste fixée à 10 % à horizon 2020. En revanche, la RED II pousse cet objectif à 14 % en 2030. La directive prévoit également le renforcement des « biocarburants avancés » (à base d'algues, déchets et résidus de sylviculture et provenant de la filière bois, paille, fumiers, boues d'épuration, glycérine brute, bagasse, etc.). Elle fixe pour chaque Etat membre un objectif spécifique de 0,2 % au minimum en 2022, 1 % en 2025 et 3,5 % en 2030.   Contreproductif   Si l'augmentation de l'utilisation d'agrocarburants est notable, EurObserv'Er met en garde. Il rappelle qu'ils sont certes importants pour que les pays membres atteignent leurs objectifs de réduction des émissions de gaz à effet de serre. Mais généralement produits sur des terres agricoles auparavant utilisées pour des cultures de denrées alimentaires ou d'aliments pour animaux, l'augmentation de la consommation des biocarburants renforce le besoin de terres. « Y compris éventuellement sur des zones à fort stock de carbone telles que des forêts, des zones humides et des tourbières. Ce phénomène pouvant entraîner la libération de CO2 stocké dans les arbres et le sol » . Un changement d'affectation des sols qui pourrait nuire aux économies de gaz à effet de serre engendrées par l'utilisation croissante des agrocarburants.   Pour tenter de remédier à ce problème, REDII fixe des limites pour les agrocarburants, bioliquides et biocombustibles à haut risque ILUC, avec une expansion des terres à fort stock de carbone. La quantité de ces carburants que les États membres peuvent compter dans leurs objectifs nationaux lors du calcul de la part nationale globale des énergies renouvelables dans les transports sera limitée. Les niveaux seront gelés à ceux de 2019 pour la période 2021-2023, puis passeront à zéro de fin 2023 à 2030. La directive introduit une exemption pour les biocarburants, les bioliquides et les biocombustibles certifiés à faible risque.   [1] consortium spécialisé dans le suivi du développement des énergies renouvelables dans l'Union européenne
</article>
<article titre="Table ronde sur les déchets" date="2019 09 19" journal="Sud Ouest  Mont-de-Marsan">
Le musée de la Faïence et des Arts de la table convie les visiteurs à une table ronde peu ordinaire, samedi, à 17 heures. En effet, les déchets plastiques et les diverses pollutions seront au coeur des débats, en présence notamment de Rachel Durquety, vice-présidente du Conseil départemental déléguée à la Culture, membre de la commission environnement, le tout animé par Grégory Dupraz.   Divers participants ont été conviés autour de la table, à commencer par Stéphanie Barneix, multiple championne du monde de sauvetage en mer, présidente de Capbreton Sauvetage Côtier et du Syndicat mixte de gestion des baignades landaises. À ses côtés, on trouvera Lionel Fournier, responsable du service des milieux aquatiques à la direction de l'environnement du Département des Landes, assurant notamment la coordination et le suivi du nettoyage du littoral landais. Il y aura aussi François Pouliquen, chef de centre de la société Codev, gestionnaire de la plateforme de stockage et de tri des déchets du littoral à Linxe, et de Laurent Soulier, directeur de l'Institut des milieux aquatiques, vétérinaire et président du Conseil scientifique régional du patrimoine naturel d'Aquitaine. Présence également de Caroline Secq, artiste et passionnée par les déchets rejetés par les océans qu'elle transforme en tableaux (exposés au musée) et en voyages imaginaires.  Bouteilles 100 % végétales   Enfin, pour débattre de ce vaste sujet de la pollution, il y aura Nicolas Mouflet, fondateur de l'entreprise Lyspackaging et concepteur de bouteilles 100 % végétales et compostables, fabriquées à partir d'une ressource renouvelable : la bagasse (un sous-produit restant une fois que le sucre a été extrait de la canne). Aujourd'hui, c'est une entreprise qui emploie 16 personnes, tout en développant des contenants en matière végétale sans une goutte de pétrole et sans perturbateurs endocriniens.   Alain Dulucq
</article>
<article titre="Restaurant zéro gaspi à Bastogne" date="2019 09 16" journal="DH (site web)">
Nadia Lallemant Publié le lundi 16 septembre 2019 à 09h29 - Mis à jour le lundi 16 septembre 2019 à 09h29   Luxembourg Céline Fautré innove pour préserver l’environnement et éviter le gaspillage.   Au restaurant Mes Petites Gourmandes, place Mc Auliffe à Bastogne, la gérante, Céline Fautré, a décidé d’innover avec un double objectif, préserver l’environnement et éviter le gaspillage alimentaire. "Je cuisine des produits locaux pour favoriser les circuits courts", explique-t-elle. "Les plats et les boissons à emporter sont conditionnés dans des barquettes et des gobelets compostables. Nous évitons les plastiques et les matières non recyclables.   "   La commerçante se fournit auprès de la société MJPack, spécialisée dans les emballages alimentaires, implantée à Lavacherie dans la commune de Sainte-Ode. Certaines barquettes sont fabriquées en bagasse, un matériau à base de pulpe de canne à sucre.   Le nouvel établissement met, par ailleurs, tout en œuvre pour ne pas jeter d’aliments à la poubelle.   "Le restaurant, couplé à une épicerie fine, a ouvert dans l’ancienne librairie à la mi-août"   , précise Céline Fautré.   "Le nom de l’établissement est un clin d’œil à mes filles, Nina et Jade. J’ai une employée, Edwina Robert. Quand on débute une activité, il n’est pas facile de prévoir à l’avance les quantités. Il y a quelques semaines, il est arrivé que plusieurs plats à consommer le jour même n’aient pas été vendus. J’ai fait une photo des invendus et je l’ai postée sur les réseaux sociaux en indiquant que le prix de vente était réduit de 20 %. Tout est parti rapidement.   "   La commerçante renouvelle son action zéro gaspi chaque fois que c’est nécessaire.   "Ce type d’action est intéressant pour le commerçant car il vend un produit qu’il aurait dû jeter mais aussi pour le client qui achète un plat cuisiné, préparé avec des produits locaux, à un prix réduit.   "   Les clients sont ravis. "Très peu de commerçants optent pour ce concept dans la région de Bastogne   ", indique Laurent." On ne peut que souhaiter que d’autres s’inscrivent dans cette démarche."  Cet article est paru dans DH (site web)
</article>
<article titre="Saintes, future cité sans plastique? Axevert, futur cluster d'entreprises et ONG offrant des alternatives à la pollution plastique, organise son lancement à Saintes mercredi." date="2019 09 16" journal="Charente Libre  Toutes">
Lui a déjà tranché le débat qui agite ces temps-ci les collectivités en charge des déchets depuis que le gouvernement envisage d'instaurer une consigne sur les bouteilles plastiques. Pour Nicolas Soufflet, l'une des vraies bonnes solutions est de recourir à ses «veganbottles» marque déposée de bouteilles végétales à partir de bagasse de canne que produit du côté de Saintes sa société Lyspackaging.   Un Cognaçais   dans l'aventure   Lancées en 2017, les fameuses veganbottle ont séduit à ce jour une soixantaine de clients, elles sont vendues pour 30 % à l'export et environ 10 millions de flacons ont été fabriqués à ce jour. Pour aller plus loin, l'entrepreneur se fait le chantre d'un monde sans plastique. Avec deux autres personnes, le Cognaçais Jérôme Fruchard, responsable technique de la formation supérieure FS Pack, et Philippe Gendron qui dirige à Saintes un bureau d'études en plasturgie, il a créé Axevert. «C'est pour l'instant une association que nous souhaitons faire évoluer vers un cluster réunissant des entreprises, des ONG, des collectivités qui luttent contre la pollution plastique en proposant de le remplacer ou de mieux le recycler, le réutiliser» , appuie Nicolas Soufflet qui a su intéresser à son projet la communauté d'agglomération de Saintes qui a installé Axevert dans sa  «cité entrepreneuriale» .   Elle occupe les locaux laissés vacants par le déménagement pour La Rochelle de l'ancien siège régional du Crédit Agricole Charente Maritime-Deux Sèvres.  «Saintes n'a pas de thème fort rapprochant les entreprises, alors pourquoi pas celui de la lutte contre la pollution plastique par des solutions innovantes?» , avance Nicolas Soufflet. Le patron de lyspackaging et ses deux acolytes d'Axevert ont potassé leurs sujets. Jeudi 19 septembre à la cité entrepreneuriale de Saintes, ils organisent le premier grand raout d'Axevert France. Au programme, diverses conférences sur l'état de la pollution plastique en mer comme sur terre, mais aussi des exemples d'entreprises venues d'ailleurs qui travaillent sur le remplacement de l'envahissante matière issue du pétrole.   «Nous accueillons les Californiens de Restalk USA qui créent des emballages à partir de déchets de chanvre. Nous entendrons aussi les Mexicains d'Avocaductlery, fabricants de vaisselle compostable à partir de noyaux d'avocats. Cette société souhaite s'implanter en France et étudie la possibilité de le faire à Saintes.» Au-delà de cette première journée, les trois fondateurs d'Axevert se positionnent pour mener une veille sur ces questions de remplacement du plastique et de mise en réseau de tous ceux que le thème intéresse.   Agnès Marroncle   Inscriptions à la journée du 19 septembre   sur le site www.axevert-france.com
</article>
<article titre="Saintes, future cité sans plastique?" date="2019 09 16" journal="Sud Ouest">
Axevert, futur cluster d'entreprises et ONG offrant des alternatives à la pollution plastique, organise son lancement à Saintes mercredi.   Lui a déjà tranché le débat qui agite ces temps-ci les collectivités en charge des déchets depuis que le gouvernement envisage d'instaurer une consigne sur les bouteilles plastiques. Pour Nicolas Soufflet, l'une des vraies bonnes solutions est de recourir à ses "veganbottles" marque déposée de bouteilles végétales à partir de bagasse de canne que produit du côté de Saintes sa société Lyspackaging. Un Cognaçais    dans l'aventure   Lancées en 2017, les fameuses veganbottle ont séduit à ce jour une soixantaine de clients, elles sont vendues pour 30 % à l'export et environ 10 millions de flacons ont été fabriqués à ce jour. Pour aller plus loin, l'entrepreneur se fait le chantre d'un monde sans plastique. Avec deux autres personnes, le Cognaçais Jérôme Fruchard, responsable technique de la formation supérieure FS Pack, et Philippe Gendron qui dirige à Saintes un bureau d'études en plasturgie, il a créé Axevert.   , appuie Nicolas Soufflet qui a su intéresser à son projet la communauté d'agglomération de Saintes qui a installé Axevert dans sa    Elle occupe les locaux laissés vacants par le déménagement pour La Rochelle de l'ancien siège régional du Crédit Agricole Charente Maritime-Deux Sèvres.    Au-delà de cette première journée, les trois fondateurs d'Axevert se positionnent pour mener une veille sur ces questions de remplacement du plastique et de mise en réseau de tous ceux que le thème intéresse.   Agnès Marroncle   Inscriptions à la journée du 19 septembre    sur le site
</article>
<article titre="Les producteurs forestiers s'engagent dans le développement durable" date="2019 09 4" journal="L'Avantage Votre Journal (site web)">
Le Syndicat des producteurs forestiers du Bas-Saint-Laurent (SPFBSL) met à la disposition des organismes intéressés, des assiettes, bols, verres et couverts compostables.    Rappelons qu'en mai dernier, les producteurs du SPFBSL adoptaient une résolution à l'effet que des actions soient entreprises dans un premier temps pour que soit sensibiliser la population. « Ensemble, nous souhaitons faire pression sur les divers paliers de gouvernements afin que soient abolis les contenants de styromousse et de plastique, qu'ils soient enlevés de la mise en marché des produits bioalimentaires, et qu'en parallèle soient développées des alternatives par exemple du carton fait à base de fibre de bois. Nous considérons que la situation environnementale se dégrade dans le monde, que l'amélioration de la situation devrait être la préoccupation de toutes les populations et que chaque individu et chaque organisation devraient être partie prenante et faire pression pour que s'améliorent les pratiques nuisibles à l'environnement, dont les marchés d'alimentation ou autres commerces, où l'on retrouve majoritairement des contenants de styromousse et de plastique pour les produits bioalimentaires », indique le président de l'organisme, Maurice Veilleux. Une contribution tangible    En ce sens, le Syndicat a procédé à l'achat d'un volume de vaisselle et couverts compostables afin de faciliter l'accès à ces produits à tous ses partenaires de la région bas-laurentienne. Le Syndicat prévoit aussi investir dans divers événements en offrant la vaisselle et les couverts compostables en commandites notamment à la Journée Portes ouvertes de l'UPA le 8 septembre, à Saint-Anaclet et à Auclair, ainsi qu'à Packington le 14 septembre lors de l'événement La Forêt sous toutes ses couleurs, une valeur de 1 500 $. Le Marché public de Rimouski pourra aussi bénéficier de cet investissement dans le temps des Fêtes et ses marchands ont reçu l'information. « Déjà, le MAPAQ s'est procuré verres, bâtonnets et assiettes pour ses bureaux à Rimouski et les bureaux satellites. Comme nous serons présents à la Journée Portes ouvertes de l'UPA le 8 septembre et à la Forêt sous toutes ses couleurs le 14 septembre, nous allons offrir à chacun plus de 1000 exemplaires de chaque morceau de vaisselle et les couverts selon leur besoin», précise le directeur général, Charles Edmond Landry.    Publicité   Défiler pour continuer   Ses avantages    La vaisselle compostable utilisée par le SPFBSL est en bagasse. Fabriquée de fibres résiduelles récupérées après broyage et extraction du jus de canne à sucre, la bagasse représente une alternative aux produits de plastique en étant 100 % compostable et biodégradable. Élégants et durables, les produits offrent une excellente rigidité et une bonne résistance au détrempement pour les aliments lourds ou en sauce. Elle se maintient en excellente qualité dans un endroit frais et sec, idéalement à l'abri de la lumière.   Les organismes désirant se procurer de la vaisselle et des couverts compostables au prix coûtant peuvent contacter Chantale Arseneaul au carseneault@upa.qc.ca ou au 418 723-1939 poste 4506 ou 418 392-0386.  Cet article est paru dans L'Avantage Votre Journal (site web)
</article>
<article titre="Passirac: la semi-nocturne gourmande et réussie" date="2019 08 26" journal="Charente Libre  Toutes">
«C ette année, pour la huitième édition de notre semi-nocturne gourmande, nous avons quelque 180 participants, répartis sur les deux parcours de 9 et 13 km,  indiquait samedi soir Marc Airiau, président du comité des fêtes passiracais.  Et le beau temps est aussi avec nous...» Sous l'auvent du Relais des As de la gâchette mis à disposition, Vanessa et son groupe de bénévoles ont géré les départs dans la bonne humeur.  «Le retour ici pour y savourer dessert et café, ce ne sera guère avant 23h pour les derniers» , sourit-elle. En effet, le parcours est jalonné de plusieurs pauses champêtres. Une première proche de l'entrée du terrain de camping brossacais, va permettre d'apprécier les entrées avec un rosé frais. Mais c'est à «Mareuil» que seront servies saucisses et merguez grillées accompagnées d'une savoureuse ratatouille maison.   «Si les gobelets réutilisables accompagnent les marcheurs dans leur périple, nous avons aussi su servir tout le repas dans de la vaisselle en bagasse de canne à sucre biodégradable» , note le président.   Le comité donne rendez-vous le samedi 16 novembre pour sa traditionnelle soirée poule au pot d'automne.
</article>
<article titre="Albioma (ex-Sechilienne-Sidec) : espoirs exaucés" date="2019 08 13" journal="Boursier (site web)">
Appréciation.  L'arrivée d'Impala au capital d'Albioma a servi d'élément déclencheur à l'appréciation du titre. La firme d'investissement Impala est détenue par Jacques Veyrat qui dirigeait auparavant le groupe de négoce Louis Dreyfus. En plus de sa participation majoritaire dans Neoen, Impala a accompagné de longues années Direct Energie avant de céder ce producteur d'énergies renouvelables à Total. M. Veyrat a choisi de réinvestir une partie du produit de cette vente en reprenant les 5,5% d'Albioma détenus par Altamir. Cette opération s'est effectuée à un prix unitaire situé autour de 18,50 Euros. En manque d'actionnaire de référence, Albioma trouve en la personne de Jacques Veyrat un nouvel entrant très au fait de cet univers de l'éolien et du solaire. Brésil.  Le premier producteur d'énergie photovoltaïque dans les DOM-TOM est également bien implanté au Brésil. Ce pays est le premier producteur mondial de sucre alors que la bagasse, un résidu de la canne, est utilisé par Albioma dans ses centrales thermiques. Le français souhaite atteindre la taille critique en ouvrant une nouvelle centrale au Brésil tous les 12 à 18 mois. En partenariat avec des industriels sucriers, l'expansion géographique d'Albioma pourrait se poursuivre vers d'autres pays d'Amérique Latine voire en Afrique. Si le solaire est aujourd'hui largement minoritaire dans les revenus de la société, les projets ne manquent pas avec des fermes solaires à l'étude en France et à l'international. Valorisation.  Sur la première partie de l'exercice en cours, Albioma a montré sa capacité à mieux maîtriser ses installations techniques en profitant également de conditions d'ensoleillement favorables. Le chiffre d'affaires a progressé de 19% sur la période avec un Ebitda à +17%... L'Ebitda est prévu autour de 200 Millions d'Euros en 2020 alors qu'il était de 160 ME en 2018. Mais la progression du titre conduit aujourd'hui à une valorisation qui intègre bien ces différents éléments. Il semble donc opportun d'alléger cette position même si une offre d'Impala sur 100% du capital n'est pas une hypothèse impossible. Il nous semble tout de même que si M. Veyrat avait voulu prendre le contrôle d'Albioma, il aurait déjà dégainé... On prend une partie de ses bénéfices.Christophe Voisin
</article>
<article titre="Plein soleil pour Albioma au premier semestre" date="2019 07 31" journal="Investir (site web)">
AchatAlbioma a enregistré de belles performances au premier semestre. Sur la période, le chiffre d'affaires du producteur d'électricité (biomasse et solaire) implanté en Outre-mer et au Brésil a progressé de 19%, à 241 millions d'euros. La dynamique de l'activité s'est concrétisée par une augmentation de 17% de l'Ebitda. A 84,5 millions d'euros, il représente 35% des facturations du groupe, contre 35,6% un an plus tôt. Quant au résultat net, avec 18,2 millions, il s'est inscrit en baisse de 12% sur un an. cette baisse s'explique par le produit exceptionnel de 3,5 millions d'euros intégré au premier semestre 2018. Dans toutes les zones où le groupe est présent (France métropolitaine, Outre-mer, Ile-Maurice et Brésil), les volumes d'électricité produits ont nettement augmenté, en partie portés par la mise en service de nouveaux sites, Galion 2 (bagasse et biomasse) en Martinique et la turbine à combustion de Saint-Pierre à La Réunion. Au Brésil, où Albioma a récemment acquis une centrale de production à partir de résidus de canne à sucre, les volumes sont en hausse de 48%. L'Ebitda de l'activité brésilienne a d'ailleurs doublé en un an, pour s'établir à 3 millions d'euros.   Coté bilan, la dette nette est ressortie à 805 millions d'euros, en hausse de 8% par rapport au 31 décembre. Selon le communiqué, « le groupe conserve des moyens adaptés à la poursuite de son développement. », notamment grâce à une facilité bancaire de 60 millions d'euros accordée fin juin.   Développer l'énergie verte et atteindre la taille critique au Brésil Autre point, le cabinet Midcap Partners a souligné que, « le verdissement du groupe se poursuit avec une part des énergies renouvelables toujours croissante et qui devrait militer à terme pour une amélioration des multiples du groupe. »   Dans ce contexte, Albioma confirme ses objectifs annuels. Le groupe prévoit un Ebitda annuel entre 168 à 178 millions d'euros, soit une croissance de 3,3% à 9,5%. Il estime son résultat net part du groupe entre 38 et 44 millions d'euros, contre 44,2 millions en 2018. Le producteur d'électricité fait également part de sa volonté de poursuivre son développement au Brésil, au rythme d'un projet tous les 12 à 18 mois.   Nora Choukrani
</article>
<article titre="Matières résiduelles Un casse- croûte s’est lancé dans le « zéro déchet »" date="2019 07 31" journal="L'Information du Nord - Ste-Agathe (QC)">
Le zéro déchet est une démarche qui vise à réduire au maximum la production de déchets. Cette tendance qui gagne le Québec s’est rendue jusqu’au Casse-croûte d’en Haut.    Éric Belhumeur recycle et composte. Rien de surprenant, direz-vous. Mais comme on parle d’un casse-croûte traditionnel qui avait l’habitude de tout mettre à la poubelle, cela représente un défide taille.  « Avant, on produisait 7 à 8 sacs de déchets par jour. Maintenant, c’est un demi-sac », dit le copropriétaire du casse-croûte de Mont- Tremblant.    ÊTRE PRÊT Insatisfait du service de RCI Environnement, l’homme cherchait une solution de rechange pour la collecte des matières résiduelles de son commerce. À la même période, il rencontre Nathalie Gara-Boivin, agente de sensibilisation pour la Mission Recyclage Compostage Laurentides (voir encadré). « Elle nous a vraiment donné un bon coup de pouce, ditil. C’est que du positif. » L’agente observe la dynamique du casse-croûte puis présente ses idées. Et c’est gratuit.    Première étape: trouver de la vaisselle pour les clients qui mangent sur place. Simple? Pas tant que ça, explique Éric Belhumeur. En plus, ils n’avaient pas de coin pour la plonge. « On a défoncé une des salles de bain pour créer un espace pour laver la vaisselle. » Ce virage a demandé temps et argent. Mais personne ne s’en plaint, au contraire.    COMPOST En plus de chercher à éliminer le styromousse et le plastique de leur restaurant, les propriétaires ont misé sur le compostage. « Je suis en attente de bacs supplémentaires de la Ville », souligne Éric Belhumeur. Quand il les aura, plus aucune matière organique n’ira aux poubelles.    Pour les take out et les livraisons, Éric Belhumeur énumère les nouveaux matériaux qu’il privilégie: carton, aluminium et bagasse. « La bagasse, c’est un résidu de canne à sucre, explique fièrement le copropriétaire. Ça donne une assiette compostable. »    Outre ces grands changements, l’homme croit que ce sont aussi les petits gestes qui comptent. Par exemple, demander au client s’il a besoin d’une fourchette avec sa commande. « Avant, je passais 5 caisses de 1000 fourchettes de plastique par semaine, aujourd’hui c’est une par deux semaines. »    redaction@infodunord.ca
</article>
<article titre="Le plastique s'éteint, les idées s'éveillent" date="2019 07 29" journal="Décision Achats (site web)">
Créer son produit, et après ?Pour Adeline Pillet, ingénieure à l'Ademe, les entreprises doivent, à l'instar de Rovip, penser éco-conception. "C'est ce genre de procédé qui peut être source d'innovation, en menant vers de nouveaux modèles d'affaires. Autrement dit, dès qu'elles commencent la conception de leur produit, les entreprises doivent s'interroger sur sa fin de vie. Cela signifie, notamment, intégrer des matériaux recyclés ou penser au réemploi", précise l'experte. Pour une PME, l'éco-conception n'est pas toujours facile à appréhender, car elle signifie changer de pratiques. Par exemple, intégrer de la matière recyclée à la place de la matière vierge implique de s'approvisionner auprès d'un recycleur. Il faut aussi adapter son process, la matière recyclée ne se travaillant pas nécessairement comme la matière vierge. Soit un frein supplémentaire.  De son côté, la Fédération de la plasturgie et des composites a diffusé, en janvier dernier, un outil d'auto-diagnostic grâce auquel les industriels peuvent dresser un bilan pour savoir où ils en sont vis-à-vis des plastiques recyclés, connaître leur positionnement en approvisionnements et avoir accès à des conseils adaptés. Pour son délégué général, Jean Martin, l'enjeu est de taille : "Nous espérons atteindre un objectif d'intégration d'un million de tonnes de plastiques recyclés d'ici à 2025, soit 20% des plastiques consommés par les plasturgistes".    Selon lui, les entreprises doivent aussi s'accorder du temps pour se transformer, sans écarter obligatoirement l'usage du plastique à tout-va. "Il faut savoir mettre en avant les atouts du plastique dans certains secteurs comme l'automobile, la santé, l'aéronautique. Les abus concernent avant tout les plastiques à usage unique", poursuit-il. Une nécessaire rationalisation, donc, qu'appelle également de ses voeux Adeline Pillet : "À l'Ademe, nous ne sommes pas entièrement plastic bashing, car le plastique a aussi une utilité pour la transition écologique. Dans la construction, par exemple, il permet d'isoler les bâtiments à moindres frais. Le problème est qu'il en est fait un usage abusif : on en met partout !"    Des alternatives au plastique    KoveeKovee a créé des couverts 100% comestibles et qui ne se désagrègent pas sous forte température. Si la recette exacte est tenue secrète, elle compte parmi ses ingrédient du sel, de l'huile de colza et de la farine de blé.    J'aime mes dents Alternatives à la traditionnelle brosse à dents en plastique, celles proposées par "J'aime mes dents" sont fabriquées en poils de fibre de bambou et munies d'un manche en bambou issu de forêts gérées de façon responsable. Elles sont disponibles par abonnement.    Lyspackaging   En Charente-Maritime, Lyspackaging imagine, fabrique et commercialise des bouteilles en bio-plastique. Sa dernière innovation ? La VeganBottle, une gourde réalisée à partir de la bagasse de canne à sucre (résidu fibreux obtenu après avoir extrait le sucre)   La Perche Mike Sallard, agriculteur et Jeff Lubrano, designer, sont revenus à des basiques : utiliser des tiges de céréales pour boire. Ils ont ainsi créé La Perche, le nom de leur paille réalisée à partir du chaume de seigle et selon un procédé jalousement gardé.Une opportunité business    Dans tous les cas, les entreprises doivent se renouveler sur ce sujet : nul besoin d'une interdiction à l'échelle européenne pour comprendre que le consommateur est en attente. "La prise de conscience citoyenne entraîne des changements du côté des entreprises, depuis un an et demi environ. Nous avons déjà connu la gestion des déchets, avec le tri sélectif, qui a conduit à de nouveaux modes de vie et à la création de PME spécialisées dans le don de restes alimentaires, par exemple. Le sujet plastique' est plus récent, mais il va sur cette même voie. On en constatera les effets très prochainement", estime Laura Châtel, responsable du plaidoyer au sein de Zero Waste France.    Selon elle, "l'interdiction permet de donner une orientation". Certes, elle met des entreprises en difficulté, mais permet à d'autres de se créer. C'est le cas de La Perche, qui propose de remplacer la paille en plastique par de la paille naturelle. "Toute notre prochaine production est déjà vendue auprès de Sodexo, de restaurants étoilés et de magasins vrac ou bio", annonce Jeff Lubrano, son cofondateur. Cela représente entre six et huit millions de pailles, vendues à 0,84 centime l'unité. Pour sa récolte prévue en 2020, La Perche organisera une levée de fonds afin d'ouvrir une usine et d'atteindre une production de 50 millions de pailles. "Sur un marché de la paille estimé à 3,2 milliards d'euros en France, la promesse de croissance est énorme", envisage, serein, Jeff Lubrano.  Cet article est paru dans Décision Achats (site web)
</article>
<article titre="Des couverts compostables à Demo Forest" date="2019 07 26" journal="La Libre Belgique  édition nationale ; Hainaut">
Chaque année, les organisateurs de la Foire agricole et forestière de Libramont tentent de réduire les impacts environnementaux de l'événement. Dans le cadre des journées Demo Forest, organisées les 30 et 31 juillet, à Bertrix, un pas supplémentaire sera franchi puisqu'aux gobelets réutilisables s'ajouteront des contenants et couverts compostables. Une première en province de Luxembourg !   L'initiative ? Elle revient à Johnny Macoir, le patron de MJPack, une société spécialisée dans les emballages industriels et alimentaires implantée à Lavacherie (Sainte-Ode). "Ce projet a été présenté, en mai dernier, à Alexandre Devolf, précise-t-il. Lorsque je lui ai expliqué que l'utilisation de matières compostables aux stands de restauration permettrait d'éviter 300 kg de déchets dans les poubelles, il a marqué son accord." Les couverts seront en bois   Ainsi, les barquettes en plastique, pour les frites, par exemple, seront remplacées par des contenants en bagasse, à base de pulpe de canne à sucre. Les couverts seront en bois, les serviettes en ouate recyclée. "Les trois gérants des restaurants, snacks et friterie ont accepté de jouer le jeu. Environ 20 000 exemplaires de chaque couvert et contenant leur seront fournis."   Cette opération d'envergure n'aurait pu être menée à bien sans l'utilisation d'une machine de tri spécifique mise à disposition par la société CETT Monseu de Rochefort. La présence d'îlots de tri permettra aux visiteurs de déposer leurs sous-produits. La collecte des poubelles et la vérification du tri seront effectuées par Le Saupont de Bertrix au moyen de cette machine qui vérifiera le contenu de chaque poubelle alimentaire afin de garantir une fraction 100 % organique. La valorisation finale sera prise en charge par l'Aive. L'ensemble de ces matières renouvelables finira dans l'usine de biométhanisation de Tenneville pour produire de l'énergie verte et du compost. Idelux Innovation coordonne l'ensemble de la démarche.   N.L.   L'impact environnemental des Demo Forest sera réduit grâce à l'initative de Johnny Macoir.
</article>
<article titre="Des couverts compostables à Demo Forest !" date="2019 07 25" journal="DH (site web)">
L’initiative ? Elle revient à Johnny Macoir, le patron de MJPack, une société spécialisée dans les emballages industriels et alimentaires implantée   à Lavacherie (Sainte-Ode). "Ce projet a été présenté, en mai dernier, à Alexandre Devolf" précise-t-il. "Lorsque je lui a expliqué que l’utilisation de matières compostables aux stands de restauration permettrait d’éviter 300 kg de déchets dans les poubelles, il a marqué son accord." Ainsi, les barquettes en plastique, pour les frites, par exemple, seront remplacées par des contenants en bagasse, à base de pulpe de canne à sucre. Les couverts seront en bois, les serviettes en ouate recyclée. "Les trois gérants des restaurants, snacks et friterie ont accepté de jouer le jeu. Environ 20.000 exemplaires de chaque couvert et contenant leur seront fournis."   Cette opération d’envergure n’aurait pu être menée à bien sans l’utilisation d’une machine de tri spécifique mise à disposition par la société CETT Monseu de Rochefort. La présence d’îlots de tri permettra aux visiteurs de déposer leurs sous-produits. La collecte des poubelles et la vérification du tri seront effectuées par Le Saupont de Bertrix au moyen de cette machine qui vérifiera le contenu de chaque poubelle alimentaire afin de garantir une fraction 100 % organique. La valorisation finale sera prise en charge par l’AIVE. L’ensemble des ces matières renouvelables finira dans l’usine de biométhanisation de Tenneville pour produire de l’énergie verte et du compost. IDELux Innovation coordonne l’ensemble de la démarche.   N.L.   Nadia Lallemant  Cet article est paru dans DH (site web)
</article>
<article titre="Un corps et une tête retrouvés dans un fossé" date="2019 07 25" journal="Le Progrès (Lyon)">
C'est un employé municipal de la ville de Ferney-Voltaire qui a fait la macabre découverte, aux alentours de 14h30, mercredi après-midi 24 juillet. Le champ qui jouxte la grande douane de Ferney-Voltaire, menant à l'aéroport de Genève, venait d'être moissonné par l'agriculteur. Les services techniques de la Ville avaient pour mission de rendre propres les bords du terrain.   L'employé municipal a alors aperçu une forme dans un fossé herbeux. Il a pensé à un cadavre d'animal. En s'approchant, c'est un corps d'homme décapité qu'il a découvert : la tête, qui n'était plus accrochée au reste du corps, gisait juste à côté.   La zone, qui se situe à 100 mètres de la frontière suisse, a été bouclée par les gendarmes. Les techniciens scientifiques ont fait des prélèvements et les constatations, alors que la Brigade de recherche de Gex commençait l'enquête de voisinage. Mais à part les employés de la douane, on relève peu d'habitations dans ce secteur frontalier.   La Brigade de recherche de Gex, en charge de l'enquête, n'exclut aucune hypothèse. La première, c'est le crime avec décapitation. Puisque la tête a été retrouvée posée à côté du corps. La seconde piste est celle d'un homme qui serait mort pour une raison x ou y, et que les charognards auraient commencé à dévorer et à démembrer.   Cette seconde hypothèse est alimentée par la présence de SDF, originaires des pays de l'Est, qui squattaient depuis quelques mois les bois de la Bagasse, juste à côté des pistes de l'aéroport de Genève-Cointrin. Le corps est celui d'un homme, barbu. Des vêtements ont été retrouvés à proximité, notamment un jean et un pull.   La dépouille transportée à l'institut médico-légal de Lyon Sa dépouille a été emmenée vers une chambre funéraire en attendant d'être transportée vers l'institut médico légal de Lyon. Une autopsie aura lieu. Ce qui va être déterminant, pour les enquêteurs, c'est de mettre une identité sur ce corps. Et de savoir s'il fait l'objet d'une fiche de diffusion nationale ou non. Les enquêteurs n'excluent pas d'avoir recours à l'Institut de recherche criminelle, qui pourrait permettre de reconstituer le visage dégradé et de dresser un portrait-robot du mort.
</article>
<article titre="Un corps et une tête retrouvés dans un fossé" date="2019 07 25" journal="Le Progrès (Lyon)">
C'est un employé municipal de la ville de Ferney-Voltaire (Ain) qui a fait la macabre découverte, aux alentours de 14 h 30, mercredi. Le champ qui jouxte la grande douane de Ferney-Voltaire, menant à l'aéroport de Genève, venait d'être moissonné par l'agriculteur exploitant. Les services techniques de la Ville avaient pour mission de dégager et rendre propres les bords du terrain. Alors que l'employé municipal retournait vers son tracteur, il a aperçu une forme dans un petit fossé herbeux. Il a pensé un moment à un cadavre d'animal, type gros gibier. Mais en s'approchant, c'est un corps d'homme décapité qu'il a découvert. La tête, qui n'était plus accrochée au reste du corps, gisait juste à côté.   La Brigade de recherche de Gex, à qui l'enquête a été confiée par le parquet de Bourg-en-Bresse, n'exclut aucune hypothèse. La première, c'est le crime avec décapitation. Puisque la tête a été retrouvée posée à côté du corps. La seconde piste est celle d'un homme qui serait mort pour une raison x ou y, et que les charognards auraient commencé à dévorer et à démembrer. Ce qui pourrait expliquer que la tête se soit décrochée. Cette seconde hypothèse est alimentée par la présence de SDF, originaires des pays de l'Est, qui squattaient depuis quelques mois les bois de la Bagasse, juste à côté des pistes de l'aéroport de Genève-Cointrin.   La dépouille transportée   à l'institut médico légal de Lyon Le corps est celui d'un homme, barbu. Des vêtements ont été retrouvés à proximité, notamment un jean et un pull. Sa dépouille a été emportée vers une chambre funéraire en attendant d'être transportée vers l'institut médico légal de Lyon. Une autopsie aura lieu pour détecter d'éventuels coups, fractures, impacts de balles, etc. Ce qui va être déterminant, pour les enquêteurs, c'est de mettre une identité sur ce corps. Et de savoir s'il fait l'objet d'une fiche de diffusion nationale ou non. Sachant que le caractère international du pays de Gex et de Genève ne facilite pas la tâche des gendarmes avec plus de 100 nationalités répertoriées.
</article>
<article titre="Value Les préférées d'investir Vive les valeurs en retard !" date="2019 07 20" journal="Investir-Le Journal des finances, no. 2376">
ABC Arbitrage  (ABCA).  Le savoir-faire de l'arbitragiste pour prendre avantage de la moindre volatilité sur les marchés financiers n'est plus à démontrer. La société a dû s'adapter à un marché un peu trop calme ces derniers mois. Cette situation pourrait changer au second semestre avec les tensions commerciales ou le Brexit et autres menaces sur les marchés. Le rendement est élevé, ce qui ne gâche rien. Objectif : 9 €.  Albioma  (ABIO).  L'ex-Séchilienne-Sidec n'a plus rien à voir avec l'entreprise des débuts. Ce producteur d'électricité utilise de moins en moins de charbon et de plus en plus de bagasse, de bioéthanol, de biomasse et de soleil pour faire tourner ses centrales en Martinique, en Guadeloupe, à La Réunion et au Brésil. La visibilité est bonne et la sortie des fonds actionnaires de longue date Apax et Altamir a retiré l'épée de Damoclès au-dessus du parcours boursier. Objectif : 25 €.  Aubay  (AUB).  La société de services numériques présente la particularité d'annoncer régulièrement des résultats de belle facture. La croissance interne reste soutenue à la fois en France et à l'international. Dans un secteur très tendu où il est devenu difficile de recruter, le groupe parvient à embaucher. La structure financière est solide avec une position de trésorerie nette. Malgré le petit rebond depuis le 1Er janvier, l'action affiche encore sur un an un recul de 20 %, ce qui représente une opportunité de se placer. Nous visons 40 €.  LNA Santé  (LNA).  Cette petite société de maisons de retraite est fortement décotée par rapport aux grandes du secteur.   LNA Santé dispose d'un réservoir interne de croissance significatif, qui pourrait être encore dopé par des acquisitions. Le plan stratégique Grandir Ensemble 2022 inclut près de 3.500 nouveaux lits à cet horizon. Objectif : 55 €.  Mersen  (MRN).  Le leader mondial des équipements anticorrosion en graphite pour l'industrie s'est redressé en 2018 et a bien commencé l'année. La société, très diversifiée géographiquement et sur différents métiers, devrait bien résister en cas de ralentissement économique. Objectif : 35 €.
</article>
<article titre="Outre-mer : l'aide à la biomasse s'étend" date="2019 07 15" journal="Recyclage Récupération">
Le ministère de la Transition écologique et solidaire a élargi la « prime bagasse » à l'ensemble des installations produisant de l'électricité à partir de biomasse issue de la canne à sucre, dans les territoires ultramarins. L'arrêté s'inscrit dans l'objectif de favoriser l'autonomie énergétique prévue par la loi, sur ces territoires, et dans la stratégie du Gouvernement de convertir à la biomasse, d'ici 2023, les centrales qui y fonctionnent au charbon. D'un montant de 14,50 €/t de canne, le versement de cette prime est validé pour les installations de la Guadeloupe et de la Réunion, sous réserve d'une valorisation énergétique de la bagasse dans les centrales. Elle sera versée sur la base des volumes de canne calculés en fin d'année, afin d'assurer un traitement équitable des territoires indépendamment des dates de campagne sucrière.
</article>
<article titre="Canne à sucre. À La Réunion, les inquiétudes de la filière" date="2019 07 15" journal="Ouest-France (site web)">
La campagne de coupe de la canne à sucre bat son plein sur l’île de la Réunion mais les acteurs du secteur sont inquiets à l’image de leurs homologues de la Martinique. Décryptage en trois questions. Quelles sont les inquiétudes des professionnels de la filière??   Début juillet, les planteurs, des élus et l’industriel local, le groupe sucrier Tereos, ont alerté le gouvernement sur le maintien de la filière faute d’aide de l’État. En effet, depuis la fin des quotas sucriers européens en 2017, la France soutient le secteur en lui allouant, par an, 38 millions d’euros, 28 pour la Réunion, 10 pour la Guadeloupe.   Tous redoutent que ce coup de pouce ne figure plus dans le budget de 2020.   Où en est la recherche scientifique??   En attendant, les sabres coupent et l’usine extrait le sucre des fibres de canne. Et dans les laboratoires, les chercheurs testent et innovent. Parce que si la filière historique de l’île depuis le XIXe siècle est économiquement fragile, elle affiche une bonne santé scientifique. À son chevet, des organismes nationaux comme le Cirad qui   « ?travaille à la durabilité et rentabilité pour résister face au marché mondial? »,  explique le directeur régional Eric Jeuffrault. Un défi de taille pour les quelques 3?000 planteurs qui ne peuvent pas pousser les terres dans un contexte insulaire où, déjà, 50?% des surfaces agricoles sont occupées par la canne.    « ?L’objectif est donc d’améliorer les variétés pour qu’elles soient les plus résistantes possible aux champignons, bactéries et virus tout en gardant le meilleur rendement.? »   Et en réfléchissant à un recyclage à 100?% de la matière?:   « ?On développe à La Réunion une canne d’avenir dans le sens où elle produit à la fois du sucre, du rhum et de l’électricité.? »  Quelles applications concrètes??   Une centrale thermique utilisant la bagasse, les résidus de canne, permet ainsi de produire 10?% de la consommation annuelle d’électricité.   « ?Dans la canne, c’est comme le cochon, tout est bon? »,  renchérit Bernard Siegmund, directeur de l’institut eRcane, l’autre acteur clé de la recherche sur la canne.   Depuis 80 ans, une centaine de spécialistes planchent sur   « ?le progrès génétique, agronomique et technologique? »   de la filière. Grâce à leurs stations expérimentales, six nouvelles variétés ont été proposées, toutes adaptées aux différentes zones de culture. Certaines sont même vendues à l’étranger comme au Mozambique, en Tanzanie.  Cet article est paru dans Ouest-France (site web)
</article>
<article titre="Biomasse La prime  bagasse étendue" date="2019 07 11" journal="L'Usine Nouvelle, no. 3619">
Albioma, qui vient d'inaugurer la première centrale électrique bagasse-biomasse à la Martinique, peut souffler. Et les agriculteurs ultramarins de la filière canne-sucre-rhum, aussi. Le gouvernement a annoncé que la « prime bagasse », dont bénéficient déjà les installations valorisant ce résidu de canne à sucre à la Réunion et à la Guadeloupe, serait étendue à la Martinique. D'un montant de 14,50 euros la tonne, elle ne sera, surtout, plus dégressive. La bagasse a représenté respectivement 8,3 % de la production électrique de la Réunion (2016) et 3,6 % de la Guadeloupe (2017). Cette décision vise à aider les territoires d'outre-mer à sortir du charbon d'ici à 2023. Les partisans du développement de la canne fibre comme combustible dans les centrales électriques devront, eux, encore se battre. §
</article>
<article titre="L'industrie plastique sommée de se recycler Ce secteur compte 5 000 entreprises sur l'ensemble du territoire et 230 000 salariés" date="2019 07 11" journal="Le Monde">
Nous déclarons la guerre au plastique. » La sortie de Brune Poirson, la secrétaire d'Etat à la transition écologique, dans Le Journal du dimanche 7 juillet ne passe toujours pas chez les patrons du secteur. « C'est traumatisant pour toute une profession », confie l'un d'eux. « Arrêter les plastiques à usage unique, oui. S'en passer complètement, en revanche, poserait de nombreuses questions », souligne Benoît Hennaut, le président de la Fédération de la plasturgie, qui ne tarit pas d'éloges sur un matériau « léger, durable, abordable et qui demande peu d'énergie pour le transformer, ce qui limite en fait ses émissions de CO2. Il participe notamment à l'allégement des voitures ou des avions, ce qui, là encore, a un impact sur le CO2. » Depuis plusieurs années, ce secteur, qui compte 5 000 entreprises sur l'ensemble du territoire et 230 000 salariés, est sous le feu des critiques. « On ne va pas défendre l'indéfendable. On a assisté à des aberrations, comme le suremballage de certains fruits et légumes. La suppression des sacs à usage unique était une bonne décision, mais il ne faut pas jeter le bébé avec l'eau du bain. Il existe des sacs réutilisables, produits à partir de plastiques recyclés, dont l'analyse du cycle de vie est bien plus intéressante qu'un sac en coton », assureRémy Belval, chargé des affaires publiques de la société TT Plast, une PME productrice de sacs plastique recyclés.   L'arrêt de la vente de produits à usage unique, prévu d'ici à 2021, est en passe d'être digéré par l'industrie. « Environ 2 000 emplois sont potentiellement en danger du fait de cette décision, mais les fabricants français de pailles et autres couverts sont en train de se réadapter et de faire évoluer leur offre, dit-on à Bercy. Pour l'instant, il n'y a pas encore de défaillances du fait de cette décision. »   Ces nouvelles contraintes seraient même l'occasion pour la France de se doter d'une filière de plastique d'origine végétale, selon Agnès Pannier-Runacher, la secrétaire d'Etat à l'économie. Encore faut-il que les industriels prennent le pli assez rapidement. Casino, qui a lancé le 20 juin une marque propre écoresponsable de produits non alimentaires, a choisi, pour sa vaisselle jetable, la bagasse, cette pulpe de canne à sucre obtenue après l'extraction du jus. Mais, faute de producteur à bas coût en Europe, le groupe importe ses produits de Chine.   Dans l'emballage, d'autres sociétés émergent, à l'image de Lactips, qui utilise la caséine du lait pour produire des plastiques solubles et biodégradables, et qui vient de s'associer au chimiste BASF.   Sphere, le numéro un français de l'emballage européen (avec sa marque Alfapac), a déjà réalisé son tournant depuis une douzaine d'années. « Le polyéthylène que nous utilisons provient de l'éthanol issu du sucre de canne produit parBraskem, au Brésil, explique John Persenda, le dirigeant. Nous investissons également dans le bioplastique biodégradable. »   « On joue le jeu » La loi sur l'économie circulaire, qui devait être présentée en conseil des ministres, mercredi 10 juillet, doit mettre l'accent sur la recyclabilité du plastique. « Nous voulons tenir les objectifs du gouvernement qui prévoit le recyclage à 100 % des plastiques d'ici à 2025 », reprend M. Hennaut, même si cela lui semble ambitieux à l'heure où la France ne recycle que 23 % de ses différentes résines de plastique...   Le secteur s'est ainsi engagé à recycler au moins un million de tonnes de plastique d'ici à 2025. L'année dernière, il en a atteint 600 000 tonnes. « Preuve que l'on joue le jeu, la part de matière vierge utilisée a baissé de 2,6 %, malgré l'augmentation des volumes produits, indique Benoît Hennaut. Les entreprises se tiennent d'ailleurs prêtes à investir pour adapter leur outil aux matériaux recyclés. »   En ce qui concerne la collecte, les entreprises sont ouvertes à la consigne de bouteilles ou de sacs. « Cela a très bien fonctionné ailleurs, confirme M. Hennaut. Mais il faut que ce gisement soit accessible à tous les acteurs, et non accaparé par certains », comme la grande distribution, par exemple.   De manière générale, ajoute Adeline Pillet, de l'Agence de l'environnement et de la maîtrise de l'énergie, « tous les plastiques ne sont pas recyclables dans les conditions technico-économiques actuelles. Il existe des alliages, des plastiques multicouches assemblant des matériaux différents (aluminium, papier, textile...) que l'on ne sait pas traiter aujourd'hui .   Mais celaévolue. Le PET opaque, longtemps non recyclable, est désormais traité par des grands acteurs comme Paprec ou Soprema. De même, la PME Carbios a développé le biorecyclage, via des enzymes, des plastiques, afin de les retransformer en matière vierge. Pour enfin boucler la boucle.
</article>
<article titre="marché Les arts de la table se préparent à la disparition du plastique" date="2019 07 11" journal="LSA, no. 2565-2566">
Un déchet 100 % naturel, solide, imperméable aux graisses et à l'eau, adapté à la congélation et au micro-ondes. Il s'agit de la pulpe de canne à sucre, issue de la bagasse, ce résidu récupéré après extraction du sucre. En prévision de l'interdiction des plastiques à usage unique, le 1er janvier 2020, Casino est allé chercher ce matériau en Chine. « Nous avons choisi des matériaux 100% naturels comme la pulpe de canne à sucre ou le carton et le papier labellisés FSC. Quant au plastique végétal, nous appliquons le principe de précaution car le législateur ne s'est pas encore prononcé sur son innocuité pour la santé et l'environnement », explique Anne-Christine Verhague, chef de marques non alimentaires chez Casino. Seuls les verres à vin et les flûtes à champagne n'entrent pas dans le champ de la nouvelle réglementation. Gobelets, bols, assiettes à dessert et verrines fabriqués en pulpe de canne à sucre seront dans tous les magasins Casino (560 au total) dès cet été. Les couverts, eux, seront en bois. Pour l'occasion, un nouveau nom a été choisi pour désigner cette gamme : Sincère. « Au total, cela fait une économie de 420 tonnes de plastique par an, détaille Latifa Moutai, directrice des achats et des ventes non alimentaires chez Casino. Il faut maintenant essayer de générer de nouvelles filières pour ces nouveaux substituts au niveau des collectivités locales, mais cela coûte très cher de les créer. »    Verre et bois ont la cote   Casino n'est pas le seul à entamer une réflexion sur le sujet. Couverts en bambou, assiettes en feuilles de palmier, gobelets en carton : la demande pour des matériaux d'origine végétale explose. Des classiques, comme le bois ou le verre, font un retour en force. Certains groupes verriers, comme Arc International, y voient une opportunité. En difficulté depuis quelques années - 700 emplois vont être supprimés sur un total de 5 200 -, l'entreprise du nord de la France vient de bénéficier d'un plan de refinancement à hauteur de 120 millions d'euros. Or, selon un sondage réalisé pour Arc, la faveur des consommateurs va clairement au verre. 85 % des Européens préfèrent le verre aux autres matériaux. L'industriel a donc travaillé sur une nouvelle génération de produits. « Même si son poids reste largement supérieur à celui du plastique, le verre présente beaucoup d'atouts », explique Sébastien Fauveaux, responsable des marques Luminarc et Arcopal. Parmi lesquels la solidité, bien sûr, et le naturel.    Cet été, Arc va lancer une campagne de communication : Stop au jetable, passez au durable. Elle vise à mettre en valeur des produits comme les saladiers en verre « capsulables » (avec un couvercle). Reste un handicap, le prix. Il faut compter au moins 1 € pour une assiette en verre... contre quelques dizaines de centimes pour une assiette en plastique.  Magali Picard
</article>
<article titre="Matières résiduelles Un casse- croûte s’est lancé dans le « zéro déchet »" date="2019 07 10" journal="L'Information du Nord - Mont-Tremblant (QC)">
Le zéro déchet est une démarche qui vise à réduire au maximum la production de déchets. Cette tendance qui gagne le Québec s’est rendue jusqu’au Casse-croûte d’en Haut.    Éric Belhumeur recycle et composte. Rien de surprenant, direz-vous. Mais comme on parle d’un casse-croûte traditionnel qui avait l’habitude de tout mettre à la poubelle, cela représente un défide taille.  « Avant, on produisait 7 à 8 sacs de déchets par jour. Maintenant, c’est un demi-sac », dit le copropriétaire du casse-croûte de Mont-Tremblant.    ÊTRE PRÊT Insatisfait du service de RCI Environnement, l’homme cherchait une solution de rechange pour la collecte des matières résiduelles de son commerce. À la même période, il rencontre Nathalie Gara-Boivin, agente de sensibilisation pour la Mission Recyclage Compostage Laurentides (voir encadré). « Elle nous a vraiment donné un bon coup de pouce, dit-il. C’est que du positif. » L’agente observe la dynamique du casse-croûte puis présente ses idées. Et c’est gratuit.    Première étape: trouver de la vaisselle pour les clients qui mangent sur place. Simple? Pas tant que ça, explique Éric Belhumeur. En plus, ils n’avaient pas de coin pour la plonge. « On a défoncé une des salles de bain pour créer un espace pour laver la vaisselle. » Ce virage a demandé temps et argent. Mais personne ne s’en plaint, au contraire.    COMPOST En plus de chercher à éliminer le styromousse et le plastique de leur restaurant, les propriétaires ont misé sur le compostage. « Je suis en attente de bacs supplémentaires de la Ville », souligne Éric Belhumeur. Quand il les aura, plus aucune matière organique n’ira aux poubelles.    Pour les take out et les livraisons, Éric Belhumeur énumère les nouveaux matériaux qu’il privilégie: carton, aluminium et bagasse. « La bagasse, c’est un résidu de canne à sucre, explique fièrement le copropriétaire. Ça donne une assiette compostable. » Outre ces grands changements, l’homme croit que ce sont aussi les petits gestes qui comptent. Par exemple, demander au client s’il a besoin d’une fourchette avec sa commande. « Avant, je passais 5 caisses de 1000 fourchettes de plastique par semaine, aujourd’hui c’est une par deux semaines. » MISSION RECYCLAGE COMPOSTAGE LAURENTIDES Près de la moitié (48%) des déchets enfouis du territoire de la MRC des Laurentides proviennent des industries, commerces et institutions (les ICI). L’organisme Synergie Économique Laurentides (SEL) a été mandaté en 2018 par les 8 MRC/ville de la grande région administrative que sont les Laurentides pour cette mission d’accompagnement des ICI dans leurs efforts. Anne Samson, chargée de projet, souligne qu’en 2020 la Politique québécoise de gestion des matière résiduelles interdira l’enfouissement de toutes matières organiques. Dans cette optique, les agents qui arpentent actuellement le territoire offrent leur expertise aux ICI, et ce gratuitement. « Une bonne gestion des matières résiduelles, ça fait économiser », dit Anne Samson. Et elle souhaite que les ICI leur laissent la chance de le prouver.   redaction@infodunord.ca
</article>
<article titre="Valorisation biomasse : la "prime  bagasse" s'étend" date="2019 07 5" journal="Dechetcom (site web)">
Valorisation biomasse : la "prime bagasse" s'étend    Afin d'encourager le développement des énergies renouvelables dans les territoires ultramarins, le Ministre de la Transition écologique et solidaire signe un arrêté pour élargir la "prime bagasse" à l'ensemble des installations qui produisent outre-mer de l'électricité à partir de biomasse issue de la canne à sucre, dans l'ensemble des territoires ultramarins...    Cet arrêté s'inscrit dans l'objectif d'autonomie énergétique prévu par la loi et dans la stratégie du Gouvernement dans les départements d'Outre-mer de convertir à la biomasse d'ici 2023 les centrales qui fonctionnent au charbon.  La filière canne-sucre-rhum figure parmi ces écosystèmes générateurs d'activités agricoles et industrielles non délocalisables, pionnière de l'économie circulaire au travers notamment de la valorisation de la bagasse, un résidu de canne à sucre après pressage, et de son utilisation dans le processus de production locale d'énergies renouvelables. La bagasse a représenté respectivement 8,3% de la production électrique à la Réunion (2016) et 3,6% en Guadeloupe (2017).     Pour amplifier la valorisation de ce sous-produit agricole, soutenir la filière canne-sucre-rhum et appuyer les territoires dans leur trajectoire vers l'autonomie énergétique, le Ministre de la Transition écologique et solidaire a signé un arrêté applicable aux installations industrielles qui produisent outre-mer de l'électricité à partir de biomasse issue de la canne à sucre.     L'objectif de cet arrêté est double :    assurer l'égalité des territoires en généralisant à l'ensemble des territoires ultramarins la "prime bagasse" ;    supprimer le coefficient de dégressivité de la prime au fil du temps.     La "prime bagasse", d'un montant de 14,5 € par tonne de canne est déjà versée dans les installations existantes de la Guadeloupe et à l'île de la Réunion sous réserve d'une valorisation énergétique de la bagasse dans les centrales. La prime sera versée sur la base des volumes de canne calculés en fin d'année afin d'assurer un traitement équitable des territoires indépendamment des dates de campagne sucrière.   Cet article est paru dans Dechetcom (site web)
</article>
<article titre="Le Rapport d'activité et de développement durable 2018 pdf - 14893k - Rapport_annuel_2018_0.pdf" date="2019 07 4" journal="Rapports - Banques et institutions financières - Caisse des Dépôts et consignation">
62 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... Rapport d’activité et de développement durable 2018 « Ensemble, faisons grandir la France. »  La société se transforme, la Caisse des Dépôts aussi. Une nouvelle signature, c’est une ambition renouvelée et...    Pour voir le rapport :  https://www.caissedesdepots.fr/sites/default/files/2020-04/Rapport_annuel_2018_0.pdf
</article>
<article titre="Électricité issue de la canne à sucre en Outre-mer et Mayotte : modification des conditions d'achat" date="2019 07 1" journal="Journal de l'environnement (site web)">
Energie / Energy Production / Production   Un arrêté du 25 juin 2019 met à jour les conditions d'achat de l'électricité produite à partir de biomasse issue de la canne à sucre par des producteurs bénéficiant de l'obligation d'achat dans les départements d'outre-mer et à Mayotte. Le montant de la prime versée aux installations de production d'électricité à partir du résidu de canne à sucre est désormais fixé à 14,5 € par tonne de canne. Par ailleurs, l'avenant contractuel portant sur la période résiduelle du contrat en cours, prévu à l'article 4 de l'arrêté du 20 novembre 2009 modifié, peut désormais être élaboré pour toute installation qui bénéficie d'un contrat d'achat d'électricité issue de la canne à sucre. L'arrêté du 20 novembre 2009 est modifié en conséquence. A ce titre, désormais, si la demande complète de contrat d'achat faite par le producteur, déterminant les tarifs applicables à une telle installation nouvelle, est effectuée après le 28 juin 2019, le contrat assurera au producteur le bénéfice d'un tarif augmenté d'une prime variable définie en annexe I de l'arrêté du 20 novembre 2009.   Cette prime variable prévue en annexe I est revalorisée par le texte. Elle est désormais fixée à 14,5 € par tonne de canne dont les résidus sont valorisés dans des conditions de rendement énergétique conformes ou supérieures à celles de l'installation en 2008 en moyenne annuelle ou, pour les installations mises en service après le 1er janvier 2017, à celles de leur première année de fonctionnement (annexe I modifiée de l'arrêté du 20 novembre 2009 - article 3 de l'arrêté du 25 juin 2019).   Par ailleurs, l'avenant contractuel portant sur la période résiduelle du contrat en cours, prévu à l'article 4 de l'arrêté modifié, peut désormais être élaboré pour toute installation qui bénéficie d'un contrat d'achat d'électricité produite à partir de biomasse issue de la canne à sucre, et non plus seulement pour les installations existante au 20 novembre 2009 qui bénéficient d'un contrat signé avant le 19 octobre 2015 (article 4 de l'arrêté modifié- article 1 de l'arrêté du 25 juin 2009).   La formule de calcul prévue pour chaque contrat d'achat bénéficiant d'un tel avenant contractuel est mise à jour (article 7 de l'arrêté modifié- article 2 de l'arrêté du 25 juin 2009).   Pour rappel, un arrêté du 8 octobre 2015 a modifié les conditions d'achat de l'électricité produite à partir de biomasse issue de la canne à sucre par les producteurs situés dans les départements d'Outre-mer et à Mayotte, fixées par l'arrêté du 20 novembre 2009. Le montant de la prime versée aux installations de production d'électricité à partir du résidu de canne à sucre, dite "prime bagasse" avait été revalorisée.   Arrêté du 25 juin 2019 modifiant l'arrêté du 20 novembre 2009 fixant les conditions d'achat de l'électricité produite à partir de biomasse issue de la canne à sucre par des producteurs bénéficiant de l'obligation d'achat dans les départements d'outre-mer et à Mayotte, JO du 28 juin 2019   Découvrez le Fil Juridique complet avec HSE Vigilance
</article>
<article titre="Électricité/outre-mer: l'aide à la biomasse étendue à la Martinique" date="2019 06 28" journal="Sciences et Avenir (site web)">
La prime "bagasse" versée par le gouvernement aux centrales électriques d'outre-mer fonctionnant avec des résidus de canne à sucre va être étendue à la Martinique et ne se sera plus dégressive au fil du temps, a annoncé vendredi le ministère de la Transition écologique et solidaire.    Cette mesure vise à "amplifier la valorisation de ce sous-produit agricole, à soutenir la filière canne-sucre-rhum et à appuyer les territoires d'outre-mer dans leur trajectoire vers l'autonomie énergétique", précisent dans un communiqué le ministre d’État François de Rugy et la secrétaire d’État Emmanuelle Wargon.  Elle s'inscrit également dans la stratégie du gouvernement de convertir à la biomasse l'ensemble des centrales d'outre-mer fonctionnant encore au charbon d'ici 2023.   François de Rugy a signé un arrêté en ce sens qui "sera applicable à l'ensemble des installations industrielles d'outre-mer qui produisent de l'électricité à partir de biomasse issue de la canne à sucre".   La prime "bagasse", d'un montant de 14,5 euros par tonne de canne, était déjà versée en Guadeloupe ou à la Réunion mais la Martinique n'en bénéficiait pas, a précisé le ministère à l'AFP.   L'arrêté supprime également le coefficient de dégressivité de la prime au fil du temps, a-t-on ajouté de même source.   Le potentiel de production d'électricité à partir de biomasse est très important pour les territoires d'outre-mer, dans la mesure où cette énergie n'a pas de caractère intermittent d'autres énergies comme le solaire ou l'éolien.   La bagasse (résidu de la canne à sucre) a représenté 8,3% de la production électrique à la Réunion en 2016 et 3,6% en Guadeloupe en 2017.   La prime sera versée sur la base des volumes de canne calculés en fin d'année afin d'assurer un traitement équitable des territoires indépendamment des dates de campagne sucrière, a précisé le ministère.  Cet article est paru dans Sciences et Avenir (site web)
</article>
<article titre="Electricité-outre-mer: l'aide à la biomasse étendue à la Martinique" date="2019 06 28" journal="Le Figaro.fr">
La prime «bagasse» versée par le gouvernement aux centrales électriques d'outre-mer fonctionnant avec des résidus de canne à sucre va être étendue à la Martinique et ne se sera plus dégressive au fil du temps, a annoncé aujourd'hui le ministère de la Transition écologique et solidaire. Cette mesure vise à «amplifier la valorisation de ce sous-produit agricole, à soutenir la filière canne-sucre-rhum et à appuyer les territoires d'outre-mer dans leur trajectoire vers l'autonomie énergétique», précisent dans un communiqué le ministre d'État François de Rugy et la secrétaire d'État Emmanuelle Wargon.   LIRE AUSSI - Biomasse: une électricité plus coûteuse que le charbon ou l'éolien    Elle s'inscrit également dans la stratégie du gouvernement de convertir à la biomasse l'ensemble des centrales d'outre-mer fonctionnant encore au charbon d'ici 2023. François de Rugy a signé un arrêté en ce sens qui «sera applicable à l'ensemble des installations industrielles d'outre-mer qui produisent de l'électricité à partir de biomasse issue de la canne à sucre». La prime «bagasse», d'un montant de 14,5 euros par tonne de canne, était déjà versée en Guadeloupe ou à la Réunion mais la Martinique n'en bénéficiait pas, a précisé le ministère. L'arrêté supprime également le coefficient de dégressivité de la prime au fil du temps.
</article>
<article titre="Raphaël de Taisne et Louis de Bentzmann militent pour les bou teilles biodégradables" date="2019 06 27" journal="LSA, no. 2563">
«T'as vu ! En plus, les bouteilles sont en plastique biodégradable. C'est sympa, non ? » Parole d'un client, entendue dans la cantine végétale Yumi de la rue du Château-d'Eau, dans le 10e arrondissement parisien. Les bouteilles biodégradables sont l'une des originalités de Yumi, à la fois gérant de cantines où l'on mange des légumes et fruits bio et marque de jus HPP ou pressés à froid. Des embûches   En apparence, les bouteilles signées Yumi ressemblent à celles en PET. Sauf que, dès la naissance de leur signature, les créateurs de Yumi - Louis de Bentzmann et Raphaël de Taisne - ont pris le parti d'embouteiller leurs mélanges de jus de légumes et fruits dans des bouteilles fabriquées à partir de bagasse, le résidu de la canne à sucre après un passage par l'industrie sucrière ou du rhum. Conçues par la société Lyspackaging, spécialiste française des bouteilles biodégradables et compostables, basée dans la banlieue de Saintes (17), « ces bouteilles nous ont valu un appel du cabinet de Brune Poirson, secrétaire d'État auprès du ministre de la Transition écologique. Nous avions en effet expliqué dans un article publié sur LSA.fr que ce choix d'un emballage biodégradable était difficile à supporter pour une start-up comme la nôtre, car Citeo nous demande une taxe deux fois plus élevée qu'aux utilisateurs de PET ! Cet organisme considère nos emballages comme des perturbateurs de recyclage », explique Raphaël de Taisne. Un comble, à l'heure où les substituts au plastique issu du pétrole devraient être les bienvenus... Mais, à la différence de l'Allemagne, il n'existe pas de composteur industriel dans l'Hexagone. Le cabinet de Brune Poirson a demandé aux deux startuppers d'analyser la fin de vie de leurs emballages et à Citeo de revoir sa facturation...   Démarche circulaire   C'est pourquoi Raphaël de Taisne, 30 ans, et Louis de Bentzmann, 32 ans, composte les bouteilles qu'ils récupèrent chez les Alchimistes, autre start-up qui propose des solutions de compostage urbain et local. « Préalablement broyées, les bouteilles sont ensuite compostées dans la caverne des Alchimistes [Paris 18e - NDLR]. Au bout de deux mois dans cette caverne, il n'y a quasiment plus de résidus de plastique végétal. » Les Alchimistes commercialisent d'ailleurs un compost alimenté en partie par Yumi dans quelques Biocoop parisiens. « Il est deux fois plus nutritif que le fumier de cheval », revendiquent les Alchimistes sur leur site. La démarche de Yumi et des Alchimistes est tout à fait circulaire. Las, à ce jour, les Alchimistes ne compostent qu'une petite partie des bouteilles signées Yumi. Difficile de récupérer les bouteilles consommées dans la rue, à domicile ou en province.   En attendant un compostage à plus grande échelle, la start-up créée en 2014 se développe sur tous les fronts. Son coeur de métier ? Les jus de fruits et légumes pressés à froid pour préserver l'intégrité organoleptique des matières premières, c'est-à-dire que les jus sont stabilisés par une forte pression, de 6 000 bars. « Cette pression n'endommage ni les vitamines ni les enzymes », compare Raphaël de Taisne.   Les jus Yumi sont fabriqués à Barcelone et en Bretagne, à partir de légumes bio récoltés à proximité des sites de production. Ainsi, en Bretagne, ce sont les écarts de production de la coopérative Bio Breizh qui sont pressés. Ces jus, qui portent de doux noms - La Petite Gâterie, Chouperman -, sont vendus sur le site marchand de la marque mais aussi chez Biocoop, Naturalia, quelques Nouveaux Robinson, via Biodis, le distributeur de produits bio de l'ouest de la France, et dans les hôtel du pôle luxe d'Accor.   Vente en direct   En parallèle, les deux fondateurs misent sur la vente en direct, dans leurs cantines. Après la rue du Château-d'Eau, en plein quartier bobo-végan, Yumi s'est installé en juin à l'angle des rues Jean-Pierre-Timbaud et Saint-Maur (dans le 11e). « Nous ouvrirons trois autres lieux d'ici à fin 2019. Ce sont de vrais laboratoires où l'on voit tout de suite ce qui marche et ce qui ne marche pas », prévoit Raphaël de Taisne. « Notre but est de réconcilier les Français avec les légumes et, tout au moins, de proposer une alimentation saine et naturelle », poursuit Raphaël de Taisne. Les deux fondateurs, ex-de la banque d'affaires pour Raphaël de Taisne et ex-consultant pour Louis de Bentzmann, ont eu la révélation qu'il fallait créer Yumi lorsque l'une de leurs proches est tombée gravement malade. Une meilleure alimentation aurait peut-être pu éviter ou amoindrir cette maladie.  Sylvie Leboulenger
</article>
<article titre="Casino mise sur l'économie circulaire" date="2019 06 21" journal="Les Echos, no. 22973">
Les clients des hypers et supermarchés Casino vont avoir intérêt à conserver soigneusement leurs tickets de caisse. L'enseigne se lance en effet dans l'économie circulaire avec une initiative inédite à l'heure où tous les distributeurs cherchent à s'inscrire dans une consommation plus responsable. A partir de juillet, les porteurs de cartes de fidélité pourront rapporter les vêtements et les livres qu'ils y auront achetés il y a moins d'un an à condition de garder la preuve de leur provenance. En contrepartie, ils recevront des bons d'achat représentant 30 % du prix payé. Les produits seront donnés à une association locale. Quant aux jeux vidéo, ils seront rachetés à 70 % de leur valeur s'ils ont été acquis il y a un mois, 50 % il y a deux mois, 40 % à partir de trois mois. Ils repartiront, eux, dans le circuit de la seconde main. Un outil de fidélisation « Un distributeur doit proposer une solution globale. Il faut travailler aussi bien l'axe des services que celui des produits »  , explique la directrice des achats et des ventes non alimentaires du distributeur, Latifa Moutai. Côté offre, l'enseigne déploie progressivement dans ses magasins une nouvelle marque transversale qui se veut écoresponsable et consacrée aux univers de la maison et du textile. Baptisée « Sincère », elle démarre par de la vaisselle jetable à base de bagasse - le résidu de canne à sucre -, s'étendra en août aux bodys pour bébés et sous-vêtements en coton bio et, à la rentrée, au linge de maison. En septembre, la marque comptera 350 références, en attendant d'autres produits en 2020.   « Nous avions déjà une démarche ambitieuse sur l'alimentaire. Il fallait aussi se pencher sur les autres marchés. Nous ne prétendons pas être parfaits, mais nous cherchons à nous situer dans une démarche de progrès »  , indique la directrice de la marque Casino, Claire Luquet. Elle y voit aussi une manière de fidéliser des clients de plus en plus sensibles aux produits plus respectueux de l'environnement.   Clotilde Briard
</article>
<article titre="Cenpac accélère dans l'alimentaire" date="2019 06 20" journal="Emballages magazine (site web)">
Le distributeur lance « des emballages écoresponsables » .-   Cenpac a réalisé un chiffre d'affaires de 126 millions d'euros en 2018, soit une hausse de 4% par rapport à 2017. La filiale de Raja a pour objectif une progression du même ordre en 2019 pour atteindre 131 millions d'euros. Brice Kapelusz mise, entre autres, sur « les emballages écoresponsables » pour l'alimentaire pour y parvenir. L'un des axes retenus est de privilégier les matières premières renouvelables ainsi que les matériaux recyclés et d'éviter les complexes difficiles à recycler. « Nous sommes en train de donner un nouveau souffle à l'emballage alimentaire », indique le directeur général de l'entreprise.   L'emballage alimentaire, qui représente environ 12% des ventes à 18 millions d'euros, a enregistré en 2018 une croissance de 10%, soit une progression supérieure à celle de l'ensemble de l'activité. Afin de parvenir à une nouvelle progression de 10% en 2019, Cenpac lance 300 nouveaux produits, soit un renouvellement de 30% du catalogue. Des pailles en papier, des couverts en bambou ou en amidon de maïs, des gobelets et des barquettes en carton, des sachets en papier, des assiettes ou des boîtes en bagasse pour hamburger, des boîtes pâtissières avec fenêtre en cellulose ou encore des bouteilles contenant au moins 25% de polyéthylène téréphtalate (PET) recyclé apte au contact alimentaire direct (rPET) enrichissent l'offre. Un catalogue spécifique les recense. Directive Sup   Ce lancement, « le plus important depuis dix ans » selon le distributeur, s'inscrit dans un contexte réglementaire français et européen visant à réduire l'usage unique des plastiques, avec notamment la directive Sup, mais aussi l'amendement du député François-Michel Lambert. « Le marché de l'emballage alimentaire est à un tournant de son histoire », résume Brice Kapelusz pour qui Cenpac doit aider ses clients à « faire face aux enjeux de société » . Gros consommateur d'emballages à usage unique, le snacking, la boulangerie, la pâtisserie, la vente à emporter, les livraisons à domicile ainsi que la restauration rapide sont en effet des segments très porteurs. « On est en croissance. La plupart des marchés sur lesquels on travaille se portent bien et évoluent positivement », indique Ulrick Parfum, directeur des achats de Cenpac, qui souligne que « l'emballage écoresponsable » contribue à « la valorisation de l'image de la marque » et constitue « une aide à la vente » .   15 000 références   Forte de 15 000 références, l'entreprise compte 170 commerciaux répartis sur cinq agences à Lille (Nord), Rennes (Ille-et-Vilaine), Paris, Toulouse (Haute-Garonne) et Lyon (Rhône).
</article>
<article titre="LYSPACKAGING L'entreprise saintaise a levé 2,2 millions d'euros pour recruter et investir dans deux nouvelles souffleuses et trois presses à injecter Le flacon compostable fait son trou" date="2019 06 19" journal="Sud Ouest  Charente">
Séverine Joubert   Née le 12 janvier 2015 entre les murs - rapidement trop étroits - de la pépinière d'entreprises de Saint-Georges-des-Coteaux, à côté de Saintes, Lyspackaging mise cette année sur un développement aussi ambitieux qu'important.   La clé de voûte de l'entreprise : un plastique végétal, recyclable à l'infini, biodégradable et, surtout, compostable qui est désormais identifiable sous la marque Vegan Bottle. La mélasse et la bagasse de la canne à sucre sont les ingrédients de base de ce plastique. La recette finale, elle, est exécutée dans un laboratoire français. De huit à 20 salariés   Le fondateur de l'entreprise, Nicolas Moufflet, 45 ans, relate que " 2018 a été la première année où, avec notre innovation Vegan Bottle, on a réussi à convertir notre activité traditionnelle de départ - le PET (Polytéréphtalate d'éthylène) - en atteignant 80 % de végétal. "   Et l'ingénieur également designer de rappeler que, depuis le premier jour, son objectif est de " ne faire que du végétal ". L'année 2018 a été celle d'un " bilan positif avec du bénéfice après des années consacrées à la recherche et développement ".   Le coup d'accélérateur, Nicolas Moufflet est allé le provoquer en sollicitant " un fonds d'investissement pour nous aider dans notre développement ". C'est le cabinet angoumoisin Alienor Partners qui a ainsi convaincu le fonds parisien Cita Investissement de lever 2,2 millions d'euros. Et pourquoi cet argent ? " Tout d'abord pour consolider notre activité, répond Nicolas Moufflet. On est passé de 8 à 20 salariés. J'ai désormais un directeur commercial, un directeur technique, un chargé d'affaires, etc. "   Coût de revient réduit   Une force humaine doublée d'" un besoin industriel " qui était " d'accroître notre capacité de production de flacons, de pots et de bouchons et d'être autonomes. Avant, notre activité ce n'était que du soufflage de bouteilles. On avait deux machines capables de faire 8 ou 10 millions de bouteilles maximum. Cette année, on passe de deux souffleuses à quatre - dont une qui est arrivée - pour passer à une capacité de production de 16-20 millions de flacons. Nous avons également acheté trois presses à injecter - ce que nous n'avions pas - pour fabriquer nous-mêmes nos bouchons, nos préformes, nos pots cosmétiques et un certain nombre d'accessoires autour du flaconnage. "   Une presse est arrivée en mai, les deux autres seront dans la nouvelle usine de Chaniers - toujours à côté de Saintes - en juillet.   " Sur la partie injection, le point hyperpositif est que la matière va arriver ici. " En prenant la main sur la fabrication des préformes (jusqu'ici réalisées en Europe de l'Est) sans dépendre des sous-traitants, Nicolas Moufflet espère gagner beaucoup de temps. Et en bonus, " là on élimine un transport ". Ce qui se traduit par un prix de revient en baisse, au point tel, selon l'entrepreneur, que la marque peut " se positionner sur un prix traditionnel. On était 25 % à 30 % plus cher qu'un plastique classique; là, on sera à +15 %. "   Aujourd'hui, Lyspackaging travaille principalement dans l'agroalimentaire et, notamment, pour des jus de fruits frais distribués dans des réseaux bio et des épiceries fines. L'autre champ à conquérir est celui des cosmétiques, compléments alimentaires, gels douche.   Tout cela s'accompagne de nouveaux moules et de nouveaux produits, dont la Vegan Bottle Water (lire ci-dessous) et sa dérivée en gourde, la Vegan Bottle Go. Au total, 1 300 unités ont été vendues en pré-vente pour une fabrication qui sera lancée en juillet, dans les entrepôts de Chaniers donc puisque toutes les machines seront arrivées. Ce produit sera le premier à s'adresser directement au consommateur.   " Ma lutte est celle contre la pollution des plastiques. Je me suis dit : " Je vais créer des gourdes compostables pour celui qui veut boire l'eau du robinet ". Elle a le même " design " que la bouteille mais est plus rigide. Les bouchons seront en granulés mélangés avec du lin. "   Les filières se mettent en place   Des produits séduisants, non toxiques, dans l'air du temps. Mais que deviennent-ils une fois dans le sac jaune, faute de pouvoir aller dans les déchets organiques ?   " Aujourd'hui, quand on a un emballage compostable dans la main, c'est difficile de le valoriser mais c'est valorisable en les envoyant vers le compostage industriel pour l'agriculture ou en méthanisation. "   Nicolas Moufflet, qui n'hésite désormais plus à aller au Sénat ou à l'Assemblée Nationale pour prêcher la bonne parole, est formel : " Les filières sont en train de se mettre en place, c'est en 2020. C'est de toute façon plus qu'urgent. "
</article>
<article titre="LE PLASTIQUE s'éteint, les idées s'éveillent" date="2019 06 18" journal="Chef d'Entreprise">
La mer Méditerranée est l'une des plus polluées du globe. Ce triste constat est celui réalisé par l'association Expédition MED. Chaque année, son bateau sillonne les mers et les océans pour prélever des échantillons et mesurer la quantité de plastique présente. Le niveau relevé dans la Méditerranée est alarmant : près de 250 milliards de micro-particules ont été recensés en surface. Après un usage effréné du plastique pendant des décennies, l'heure est à la prise de conscience, jusque dans les hautes sphères de l'Union européenne. Cette dernière a voté l'interdiction des plastiques à usage unique à l'horizon 2021. De quoi sonner le glas des entreprises du secteur ? « Forcément, cela aura un impact, répond Emmanuelle Perdrix, p-dg de l'entreprise de plasturgie Rovip. Le contexte actuel crée des inquiétudes au niveau du business, avec des donneurs d'ordres qui changeront leur cahier des charges, mais aussi en termes de RH. Il devient de plus en plus difficile d'attirer les jeunes talents. Le plastic bashing nous rend, immanquablement, moins attractifs. » Rovip réalise 10 % de son chiffre d'affaires en fournissant les couverts plastiques de salades prêtes à l'emploi vendues en grandes surfaces. Pour sa dirigeante, l'interdiction du plastique à usage unique est le premier acte d'une tendance qu'elle doit intégrer à sa stratégie d'entreprise. « Le plastic bashing représente également une opportunité d'innovation pour les entreprises du secteur, c'est un moteur de réflexion. Chez nous, cette innovation pourrait passer par l'utilisation d'un matériau recyclé ou bio-sourcé ou par un nouvel usage, avec des couverts placés dans une pochette pour être réutilisés. Une alternative consisterait à proposer à nos clients de récupérer le produit en fin de vie pour le recycler », imagine Emmanuelle Perdrix. Elle a d'ailleurs créé trois postes autour de cet enjeu du plastique : un ciblé marketing, un poste de chef de projet éco-conception et un troisième intitulé "responsable innovation et développement". « Cela souligne notre volonté de faire les choses autrement, et ce, dès notre communication interne », renchérit-elle. Elle a aussi rejoint l'accélérateur Plasturgie et Composites de Bpifrance, lancé en mars dernier. Il comprend huit modules de conférences, un accompagnement et une mise en réseau, soit un programme complet pour mieux identifier de nouvelles pistes de développement pour les entrepreneurs du secteur. Créer son produit, et après ? Pour Adeline Pillet, ingénieure à l'Ademe, les entreprises doivent, à l'instar de Rovip, penser éco-conception. « C'est ce genre de procédé qui peut être source d'innovation, en menant vers de nouveaux modèles d'affaires. Autrement dit, dès qu'elles commencent la conception de leur produit, les entreprises doivent s'interroger sur sa fin de vie. Cela signifie, notamment, intégrer des matériaux recyclés ou penser au réemploi », précise l'experte. Pour une PME, l'éco-conception n'est pas toujours facile à appréhender, car elle signifie changer de pratiques. Par exemple, intégrer de la matière recyclée à la place de la matière vierge implique de s'approvisionner auprès d'un recycleur. Il faut aussi adapter son process, la matière recyclée ne se travaillant pas nécessairement comme la matière vierge. Soit un frein supplémentaire.   De son côté, la Fédération de la plasturgie et des composites a diffusé, en janvier dernier, un outil d'auto-diagnostic grâce auquel les industriels peuvent dresser un bilan pour savoir où ils en sont vis-à-vis des plastiques recyclés, connaître leur positionnement en approvisionnements et avoir accès à des conseils adaptés. Pour son délégué général, Jean Martin, l'enjeu est de taille : « Nous espérons atteindre un « Il faut une coopération au sein de la filière plastique » Marie-Hélène Gramatikoff, présidente de Lactips Fondée en 2014 par Marie-Hélène Gramatikoff, plasturgiste, et Frédéric Prochazka, chercheur à l'université de SaintÉtienne, Lactips s'est construite autour d'un procédé unique : fabriquer un film plastique à partir de lait. Plus précisément, à partir de protéines de lait dites "dégradées". Cela signifie que ce lait ne peut pas être vendu pour l'agroalimentaire car, par exemple, il a dépassé sa date limite de consommation. Transformé en poudre, il est alors utilisé par des industries non-agroalimentaires, dont Lactips. La PME compte déjà, parmi ses partenaires, l'entreprise allemande Ulrich Natürlich, qui fait confiance au plastique Lactips pour emballer ses tablettes de lave-vaisselle. Lactips vient également de signer avec BASF et vise désormais une présence mondiale. « Notre produit répond à une prise de conscience citoyenne, réglementaire et politique. C'est pourquoi d'autres grands noms de l'industrie mondiale nous ont contactés, glisse à demi-mots Marie-Hélène Gramatikoff. L'innovation apporte une nouvelle industrie, sans forcément en détruire une autre. La coopération au sein de la filière plastique est nécessaire. » Pour l'heure, grâce à sa première ligne de production inaugurée en juillet 2018, Lactips annonce une « capacité de production par équipe de 440 tonnes par an ». L'entreprise suit aussi de près la construction de son usine de près de 3 000 mètres carrés, qui comportera deux lignes de production et son nouveau laboratoire. Enfin, elle prévoit de recruter une dizaine de collaborateurs d'ici à la fin de l'année. Ses effectifs s'élèveront alors à 40 salariés pour un chiffre d'affaires escompté d'environ 20 millions d'euros dans les trois ans.   objectif d'intégration d'un million de tonnes de plastiques recyclés d'ici à 2025, soit 20 % des plastiques consommés par les plasturgistes. ». Selon lui, les entreprises doivent aussi s'accorder du temps pour se transformer, sans écarter obligatoirement l'usage du plastique à tout-va. « Il faut savoir mettre en avant les atouts du plastique dans certains secteurs comme l'automobile, la santé, l'aéronautique. Les abus concernent avant tout les plastiques à usage unique », poursuit-il. Une nécessaire rationalisation, donc, qu'appelle également de ses voeux Adeline Pillet : « À l'Ademe, nous ne sommes pas entièrement 'plastic bashing', car le plastique a aussi une utilité pour la transition écologique. Dans la construction, par exemple, il permet d'isoler les bâtiments à moindres frais. Le problème est qu'il en est fait un usage abusif : on en met partout ! » Une opportunité business. Dans tous les cas, les entreprises doivent se renouveler sur ce sujet : nul besoin d'une interdiction à l'échelle européenne pour comprendre que le consommateur est en attente. « La prise de conscience citoyenne entraîne des changements du côté des entreprises, depuis un an et demi environ. Nous avons déjà connu la gestion des déchets, avec le tri sélectif, qui a conduit à de nouveaux modes de vie et à la création de PME spécialisées dans le don de restes alimentaires, par exemple. Le sujet 'plastique' est plus récent, mais il va sur cette même voie. On en constatera les effets très prochainement », estime Laura Châtel, responsable du plaidoyer au sein de Zero Waste France. Selon elle, « l'interdiction permet de donner une orientation ». Certes, elle met des entreprises en difficulté, mais permet à d'autres de se créer. C'est le cas de La Perche, qui propose de remplacer la paille en plastique par de la paille naturelle. « Toute notre prochaine production est déjà vendue auprès de Sodexo, de restaurants étoilés et de magasins vrac ou bio », annonce Jeff Lubrano, son cofondateur. Cela représente entre six et huit millions de pailles, vendues à 0,84 centime l'unité. Pour sa récolte prévue en 2020, Des alternatives au plastique 1 Kovee Kovee a créé des couverts 100 % comestibles et qui ne se désagrègent pas sous forte température. Si la recette exacte est tenue secrète, elle compte parmi ses ingrédients du sel, de l'huile de colza et de la farine de blé.   2 J'aime mes dents Alternatives à la traditionnelle brosse à dents en plastique, celles proposées par "J'aime mes dents" sont fabriquées en poils de fibre de bambou et munies d'un manche en bambou issu de forêts gérées de façon responsable. Elles sont disponibles par abonnement.   3 Lyspackaging En Charente-Maritime, Lyspackaging imagine, fabrique et commercialise des bouteilles en bio-plastique. Sa dernière innovation ? La VeganBottle, une gourde réalisée à partir de la bagasse de canne à sucre (résidu fibreux obtenu avoir extrait le sucre).   4 La Perche Mike Sallard, agriculteur, et JeffLubrano, designer, sont revenus à des basiques : utiliser des tiges de céréales pour boire. Ils ont ainsi créé "La Perche", le nom de leur paille réalisée à partir du chaume de seigle et selon un procédé jalousement gardé.   La Perche organisera une levée de fonds afin d'ouvrir une usine et d'atteindre une production de 50 millions de pailles. « Sur un marché de la paille estimé à 3,2 milliards d'euros en France, la promesse de croissance est énorme », envisage, serein, Jeff Lubrano.. @CelineTridon Le plastic bashing oblige les entreprises à se réinventer et à innover.   Des entreprises se créent pour proposer des alternatives au plastique.   La matière recyclée ne se travaille pas comme la matière vierge.   Il faut pouvoir s'approvisionner auprès de recycleurs.
</article>
<article titre="LYSPACKAGING L'entreprise saintaise a levé 2,2 millions d'euros pour recruter et investir dans deux nouvelles souffleuses et trois presses à injecter Le flacon compostab le fait son trou" date="2019 06 17" journal="Sud Ouest  Saintes ~ Royan ~ Jonzac">
Séverine Joubert   Née le 12 janvier 2015 entre les murs - rapidement trop étroits - de la pépinière d'entreprises de Saint-Georges-des-Coteaux, à côté de Saintes, Lyspackaging mise cette année sur un développement aussi ambitieux qu'important.   La clé de voûte de l'entreprise : un plastique végétal, recyclable à l'infini, biodégradable et, surtout, compostable qui est désormais identifiable sous la marque Vegan Bottle. La mélasse et la bagasse de la canne à sucre sont les ingrédients de base de ce plastique. La recette finale, elle, est exécutée dans un laboratoire français. De huit à 20 salariés   Le fondateur de l'entreprise, Nicolas Moufflet, 45 ans, relate que " 2018 a été la première année où, avec notre innovation Vegan Bottle, on a réussi à convertir notre activité traditionnelle de départ - le PET (Polytéréphtalate d'éthylène) - en atteignant 80 % de végétal. "   Et l'ingénieur également designer de rappeler que, depuis le premier jour, son objectif est de " ne faire que du végétal ". L'année 2018 a été celle d'un " bilan positif avec du bénéfice après des années consacrées à la recherche et développement ".   Le coup d'accélérateur, Nicolas Moufflet est allé le provoquer en sollicitant " un fonds d'investissement pour nous aider dans notre développement ". C'est le cabinet angoumoisin Alienor Partners qui a ainsi convaincu le fonds parisien Cita Investissement de lever 2,2 millions d'euros. Et pourquoi cet argent ? " Tout d'abord pour consolider notre activité, répond Nicolas Moufflet. On est passé de huit à 20 salariés. J'ai désormais un directeur commercial, un directeur technique, un chargé d'affaires, etc. "   Coût de revient réduit   Une force humaine doublée d'" un besoin industriel " qui était " d'accroître notre capacité de production de flacons, de pots et de bouchons et d'être autonomes. Avant, notre activité ce n'était que du soufflage de bouteilles. On avait deux machines capables de faire 8 ou 10 millions de bouteilles maximum. Cette année, on passe de deux souffleuses à quatre - dont une qui est arrivée - pour passer à une capacité de production de 16-20 millions de flacons. Nous avons également acheté trois presses à injecter - ce que nous n'avions pas - pour fabriquer nous-mêmes nos bouchons, nos préformes, nos pots cosmétiques et un certain nombre d'accessoires autour du flaconnage. "   Une presse est arrivée en mai, les deux autres seront dans la nouvelle usine de Chaniers - toujours à côté de Saintes - en juillet.   " Sur la partie injection, le point hyperpositif est que la matière va arriver ici. " En prenant la main sur la fabrication des préformes (jusqu'ici réalisées en Europe de l'Est) sans dépendre des sous-traitants, Nicolas Moufflet espère gagner beaucoup de temps. Et en bonus, " là on élimine un transport ". Ce qui se traduit par un prix de revient en baisse, au point tel, selon l'entrepreneur, que la marque peut " se positionner sur un prix traditionnel. On était 25 % à 30 % plus cher qu'un plastique classique; là, on sera à +15 %. "   Aujourd'hui, Lyspackaging travaille principalement dans l'agroalimentaire et, notamment, pour des jus de fruits frais distribués dans des réseaux bio et des épiceries fines. L'autre champ à conquérir est celui des cosmétiques, compléments alimentaires, gels douche.   Tout cela s'accompagne de nouveaux moules et de nouveaux produits, dont la Vegan Bottle Water (lire ci-dessous) et sa dérivée en gourde, la Vegan Bottle Go. Au total, 1 300 unités ont été vendues en pré-vente pour une fabrication qui sera lancée en juillet, dans les entrepôts de Chaniers donc puisque toutes les machines seront arrivées. Ce produit sera le premier à s'adresser directement au consommateur.   " Ma lutte est celle contre la pollution des plastiques. Je me suis dit : "je vais créer des gourdes compostables pour celui qui veut boire l'eau du robinet". Elle a le même "design" que la bouteille mais est plus rigide. Les bouchons seront en granulés mélangés avec du lin. "   Les filières se mettent en place   Des produits séduisants, non toxiques, dans l'air du temps. Mais que deviennent-ils une fois dans le sac jaune, faute de pouvoir aller dans les déchets organiques ?   " Aujourd'hui, quand on a un emballage compostable dans la main, c'est difficile de le valoriser mais c'est valorisable en les envoyant vers le compostage industriel pour l'agriculture ou en méthanisation. "   Nicolas Moufflet, qui n'hésite désormais plus à aller au Sénat ou à l'Assemblée Nationale pour prêcher la bonne parole, est formel : " Les filières sont en train de se mettre en place, c'est en 2020. C'est de toute façon plus qu'urgent. "   " L'histoire de l'eau en bouteille, pour moi, c'est avant tout montrer une cohérence entre le contenu et le contenant, défend Nicolas Moufflet. L'eau est la pureté, la vie qu'on va puiser en y vantant des vertus. Mais la mettre dans une bouteille faite à 90 % avec du pétrole, ça n'a aucun sens. Sauf que c'est pratique, nomade, pas cher à fabriquer. "   30 000 unités vendues   En 2017, le patron de Lyspackaging est démarché par un embouteilleur d'eau belge intéressé par le plastique compostable. Celle-ci est devenue le sous-traitant de Lyspackaging qui, depuis février, commercialise l'eau de la source du Val d'Aisne, dans les Ardennes belges, sous sa marque Vegan Bottle Water.   La bouteille de 50 centilitres a déjà été écoulée à 30 000 unités, uniquement à destination de professionnels de la restauration, de l'hôtellerie, de l'événementiel. La Vegan Bottle Water a même séduit le palace parisien Le Bristol.   Le flacon, qui suit le même " design " que la gourde mais est moins épais, devrait, dans les mois qui viennent, connaître des variantes plus élégantes, en adoptant une forme " normande ".   Cette eau de source en bouteille compostable a même reçu un coup de pouce inattendu lors de l'émission " N'oubliez pas les paroles " diffusée le 21 mai. Fabien Haimovici, le chanteur et musicien du jeu, l'a présentée brièvement à la caméra le temps d'un échange avec Nagui.   " Notre besoin industriel était d'accroître notre capacité de production de flacons de pots et de bouchons et d'être autonomes "   sud ouest.fr   Nicolas Moufflet présente la gourde Vegan Bottle Go   La Californie, aux États-Unis, pourrait accueillir une usine de fabrication de produits Vegan Bottle. Selon ce nouveau schéma de développement, Lyspackaging vendra l'équipement, les machines ainsi que les granulés (fabriqués en Inde et assemblés dans un laboratoire en France), dont la recette est tenue secrète. Des projets de franchise à la Réunion (avec des cannes à sucre de l'île Maurice), La Guadeloupe et les Émirats Arabes Unis sont en bonne voie.   3   millions d'euros, c'est l'objectif 2019 en termes de chiffre d'affaires, soit une multiplication par trois par rapport à 2018. En 2016, première année d'existence, le chiffre d'affaires avait atteint 350 000 euros. En 2017, il était de 700 000 euros.   90   moules, comme autant de contenants de formes différentes, figurent au catalogue de l'entreprise Lyspackaging.   600000   euros ont été investis dans les deux nouvelles souffleuses et les trois presses à injecter.   17,90   euros, tel sera le prix de vente public de la gourde compostable qui sera disponible sur les sites marchands à partir de cet été.
</article>
<article titre="INDUSTRIE DE L'EMBALLAGE Emballage durable : le nouvel avantage concurrentiel" date="2019 06 15" journal="Les Affaires, no. n° 13">
Qu'il soit alimentaire ou souple, en plastique ou en carton, l'emballage est en plein essor. Cependant, devant le mouvement du zéro déchet qui prend de l'ampleur, l'industrie n'aura pas le choix de continuer à innover pour démontrer toute son utilité. La planète nage actuellement en pleine crise du plastique. L'industrie de l'emballage, qui représente plus de 25 % du volume total des plastiques utilisés (soit la plus importante des applications de la matière), est largement pointée du doigt à titre de responsable. Si rien n'est fait, il y aura d'ici 2050 plus de tonnes de plastique dans les océans que de tonnes de poissons, avertit l'équipe de la Fondation Ellen MacArthur, qui multiplie les messages et les initiatives pour stopper ce gâchis. L'une des actions de cet organisme suscite en ce moment une attention internationale. En octobre dernier, la Fondation a mis sur pied l'Engagement mondial de la nouvelle économie des plastiques. Préparé en collaboration avec le Programme des Nations Unies pour l'environnement (PNUE), cet engagement réunit déjà plus de 300 entreprises signataires, qui comptent à elles seules pour 20 % de l'ensemble des emballages de plastique produits dans le monde. On retrouve sur cette liste les grandes marques de la consommation, telle Danone, le groupe H&amp;M, L'Oréal, PepsiCo, The Coca-Cola Company et Unilever, pour ne nommer que celles-ci.   «Cette initiative permet de dicter les nouvelles tendances de l'industrie de l'emballage», soutient Stéphan Berthiaume, membre du conseil exécutif du PAC Packaging Consortium, section Québec. Cette association regroupe les manufacturiers de l'emballage de l'Amérique du Nord ainsi que ses grands utilisateurs. L'industrie, dit-il, n'a plus le choix : «Elle doit désormais prendre des mesures pour recycler davantage et développer des concepts d'économie circulaire pour prolonger le cycle de vie des matières», soutient cet expert.   Ce nouvel engagement a justement pour mandat d'éliminer tous les emballages de plastique problématiques ou inutiles et de remplacer les emballages à usage unique par des solutions réutilisables. Il vise à ce que 100 % des emballages de plastique puissent, d'ici 2025, être réutilisés, recyclés ou compostés, facilement et sans danger. L'objectif ultime est d'augmenter de manière significative la part de plastique réutilisé et recyclé dans de nouveaux emballages ou produits. Déjà plus de 200 M$ de dollars ont été versés par cinq fonds de capital-risque afin de créer une économie circulaire du plastique.   «Lentement, mais sûrement, l'industrie de l'emballage réalise qu'il y a des occasions d'affaires à saisir avec ce nouvel engagement planétaire. Les entreprises qui poseront des gestes parmi les premières vont distancer leurs concurrents», ajoute M. Berthiaume, qui est également président et propriétaire d'Emballages Roda, à Laval.   Cette entreprise qui distribue plus de 80 millions d'unités de divers produits d'emballage partout au Canada vient justement d'acquérir Industries Rada, à Delson. Le montant de la transaction demeure confidentiel. Il s'agit d'un manufacturier de bocaux et de bouteilles en plastique pour les secteurs alimentaire et nutraceutique. Ce fabricant produit également une large variété de moules par soufflage et injection.   «Grâce à cette transaction, nous voulons faire de cette usine un laboratoire collaboratif en intégration de résine recyclée. Ce sera une de nos contributions à l'économie circulaire», explique M. Berthiaume. Ce dernier compte poursuivre le développement de produits, tels des bouchons, faits de matières 100 % recyclées, qu'il a entamé depuis un an. Son entreprise travaille également, depuis le mois de janvier, en collaboration avec Envision Plastics, une société américaine qui récupère le plastique qui jonche les rives des cours d'eau de l'Amérique centrale. «Idéalement, nous souhaitons inclure au moins 10 % de cette matière dans nos futurs produits là où le marché le permettra», signale M. Berthiaume. D'ici la fin de l'année, près de 20 % des 80 millions d'unités d'emballage que vend annuellement Emballages Roda contiendront au moins une composante écoresponsable.   Devenu le premier manufacturier canadien à avoir signé l'engagement de la Fondation Ellen MacArthur au mois de mars dernier, TC Transcontinental (propriétaire de Les Affaires) fait aussi sa part en matière d'économie circulaire du plastique au Québec. Dès cet automne, les sacs du Publisac, composés présentement de plastique vierge et recyclable, seront remplacés par un nouveau sac entièrement fait à partir des résidus de plastique. Livré dans plus de 3,5 millions de foyers chaque semaine, ce nouveau sac sera 100 % recyclable. Des tests auront lieu durant l'été, qui incluront des formats de sacs qui utilisent moins de plastique et des jaquettes en papier, fait savoir Magali Depras, chef de la stratégie de TC Transcontinental. L'entreprise, poursuit-elle, a également l'intention de réutiliser les résidus de plastique de sa production d'emballage souple. «Cela devrait nous permettre d'optimiser nos ressources et nous aider à tendre vers un objectif de zéro déchet», précise-t-elle.   Plus de suggestions écoresponsables L'entreprise Emballages Carrousel, à Boucherville, prend, pour sa part, bonne note des nouvelles tendances qui s'invitent dans l'industrie. Ce géant de la distribution de l'emballage québécois, qui génère plus de 175 M$ de revenus par année, vient d'adopter, en mars dernier, un programme Engagement 500 Plus. «Depuis près de deux ans, on sent de plus en plus la pression qu'exerce la population auprès de nos clients. Avec l'aide d'Éco Entreprises Québec, nous avons donc mis en place un nouveau catalogue de produits d'emballage écoresponsables au sein duquel nous nous engageons à ajouter 500 nouvelles solutions par année jusqu'en 2024», souligne Martin Boily, vice-président des ventes et marketing chez Emballages Carrousel.   Parmi ces produits, on trouve, entre autres, des contenants de polytéréphtalate d'éthylène faits à 80 % de matières recyclées, des contenants cabarets compostables ainsi que des bols de nouilles en bagasse beige compostables et certifiés BPI (Biodegradable Products Institute).   «Ces produits séduisent déjà plus de 80 % de nos 20 000 clients. En moins de trois mois, les ventes de produits écoresponsables ont grimpé de 30 %, pour atteindre près de 40 % de nos revenus annuels», fait savoir M. Boily. L'entreprise, qui compte 375 employés répartis dans ses entrepôts de Boucherville, Boisbriand, Québec et Drummondville, forme justement ses employés pour qu'ils puissent devenir des conseillers et des ambassadeurs de ces nouvelles solutions écoresponsables auprès des clients.   Changer les perceptions À Québec, Emballages de la Capitale se met également au diapason des nouvelles tendances. Depuis un an, plus de 25 % des produits de son catalogue destinés aux secteurs alimentaire, industriel et des cadeaux sont désormais composés d'articles écoresponsables. Ces produits occupent d'ailleurs le quart de la superficie de la salle d'exposition de l'entreprise. Les ventes de ces produits représentent maintenant 30 % des revenus. «On veut s'adapter aux nouvelles demandes faites à l'industrie afin d'offrir plus de choix biodégradables et compostables à nos clients, indique Frédéric Bernier, propriétaire d'Emballages de la Capitale. Il reste encore beaucoup de travail pour peaufiner cette offre.»   Le représentant du PAC, M. Berthiaume, persiste et signe. L'industrie de l'emballage aura à modifier ses investissements vers des emballages durables. «Nous avons un rôle important de conscientisation à jouer. Certes, il y a un coût à toutes ces opérations. Mais il s'agit d'un coût-bénéfice qui finira par être payant.»   redactionlesaffaires@tc.tc
</article>
<article titre="Transformation Albioma Plus de 1 milliard au soleil entre 2013 et 2023" date="2019 06 1" journal="Investir-Le Journal des finances, no. 2369">
Quand j'ai rejoint l'entreprise, en 2011, elle s'appelait Séchilienne-Sidec et brûlait essentiellement du charbon. Elle perdait ses avantages fiscaux les uns après les autres, n'avait pas de projet, et les équipes, pas le moral. » Sans remonter au temps des pompes à bière, évoquées par un actionnaire, c'est par ce constat que Jacques Pétry a ouvert lundi l'assemblée générale d'Albioma, la dernière de sa présidence. Dans la soirée, Frédéric Moyne, directeur général depuis 2016, a été nommé PDG du producteur d'électricité. A 43 ans, il devient seul pilote d'une entreprise déjà transformée, mais qui n'est pas encore au terme de la stratégie déroulée depuis 2013. Jacques Pétry avait alors lancé un plan d'investissements de 1 milliard d'euros sur dix ans, qui pourrait atteindre finalement 1,3 à 1,5 milliard pour la période 2013-2023. Trois axes de développement : les centrales thermiques de cogénération, la spécificité d'Albioma. Proches de sucreries, elles brûlent de la bagasse, le résidu de la canne à sucre, produisant de la vapeur pour l'usine et de l'électricité, dont le surplus est vendu àEDF. En dehors des campagnes sucrières, ces centrales marchent au charbon.   Ensuite, ce modèle est décliné depuis 2014 au Brésil, où la canne est récoltée toute l'année. Albioma exploite trois centrales (il a déjà doublé la productivité des deux premières) et a engagé la construction d'une quatrième. L'enjeu est de lancer un nouveau projet tous les douze à dix-huit mois au Brésil. « Il y faudra cinq à six centrales en exploitation pour atteindre la taille critique », a précisé Frédéric Moyne.   Enfin, les activités éoliennes et de biométhanisation ont été vendues, le groupe ne gardant que le solaire, pour y développer des centrales « de niche » et avec stockage, comme sur des toitures d'hypermarché et de HLM à La Réunion. Une vingtaine de projets sont en cours, renforcés par l'acquisition, fin 2018, d'Eneco à Avignon.   CONVERTIR LES CENTRALES Ainsi, 790 millions d'euros ont été investis en six ans, dont 100 millions dans le solaire, 140 millions au Brésil, 275 millions dans le traitement des rejets de fumées des centrales thermiques et 275 autres millions dans la construction de deux unités, Galion 2 en Martinique et Saint-Pierre à La Réunion. En service depuis fin 2018, elles doivent doper les résultats. L'Ebitda (excédent brut d'exploitation) sera porté de 138,3 millions en 2017 à 168-178 millions cette année.   Ces deux centrales illustrent aussi la seconde étape du plan stratégique : « verdir le mix énergétique d'Albioma », comme l'a énoncé Frédéric Moyne. L'objectif est de porter la part des énergies renouvelables à 80 % au moins de ce mix énergétique en 2023, voire à 100 %, contre seulement 36 % en 2013. La nouvelle centrale de La Réunion fonctionne au bioéthanol local et celle de Galion 2 brûle de la biomasse au lieu du charbon en dehors de la récolte sucrière.   Ce modèle sera celui des futures centrales, mais aussi des anciennes, qui vont être converties à la biomasse. La première sera celle du Moule en Guadeloupe, pour 70 millions d'ici à 2020, sur 500 à 700 millions d'investissements prévus d'ici à 2023.   Mais ces projets ne rendent-ils pas la dette insoutenable, se sont inquiétés plusieurs des 90 actionnaires présents à l'assemblée ? A 747 millions, la dette financière nette se situait autour de 4,6 fois l'Ebitda fin 2018, alors que les deux nouvelles centrales ont peu contribué aux bénéfices l'an dernier. Ce ratio a fluctué de 3 à 7,5 fois depuis 2003, a expliqué Julien Gauthier, le directeur général adjoint chargé des finances : « Les 790 millions d'euros engagés entre 2013 et 2018 permettront d'atteindre, en 2020, un Ebitda de 200 millions, contre 120 millions en 2013. » De plus, la dette finance des centrales qui vendent leur électricité à un prix fixé pour une durée longue, procurant une bonne visibilité. « La durée résiduelle de nos contrats est de vingt ans actuellement. Nous pouvons financer nos investissements et payer le dividende », a assuré Frédéric Moyne.   Notre conseil Acheter L'entrée en service des nouvelles centrales doit dynamiser les résultats. La recomposition de l'actionnariat, avec la sortie d'Apax et d'Altamir et l'entrée, pour 5,5 %, d'Impala, le holding de Jacques Veyrat (qui détient 50 % de Neoen, opérateur solaire et éolien), a donné une nouvelle impulsion à la valeur.   Cécile Le Coz
</article>
<article titre="Arla Foods s'engage à l'horizon 2030" date="2019 05 16" journal="Emballages magazine (site web)">
Le groupe laitier veut réduire les émissions de gaz carbonique liées à ses emballages de 8000 tonnes par an. -   600 millions de briques de lait fabriquées à partir de matériaux issus à 100% de ressources renouvelables, 560 millions de pots de yaourt recyclables... : le groupe laitier européen Arla Foods prend à son tour des engagements dans le domaine du développement durable. Son objectif : une réduction de 30% de ses émissions de CO2 d'ici à 2030. Ce qui commence par une diminution de quelque 8000 tonnes par an des émissions liées aux emballages. L'ambition ultime d'Arla est d'atteindre la neutralité carbone en 2050. « Nous voulons aider les gens à mener une vie plus durable et à se sentir bien par rapport à ce qu'ils ont dans leur réfrigérateur. Le lait frais et les yaourts sont consommés quotidiennement dans la plupart des ménages de nos principaux marchés et sont également essentiels pour nos clients distributeurs. C'est pourquoi ces articles figurent en tête de notre liste d'emballages à améliorer du point de vue de la durabilité et notre présence paneuropéenne nous permet à la fois d'exploiter notre taille et d'impacter simultanément plusieurs marchés », commente Peter Giørtz-Carlsen, le président d'Arla pour l'Europe. Plastiques biosourcés et recyclés   Les nouveaux emballages seront proposés dans les six principaux marchés du groupe, en Suède, au Danemark, en Finlande, aux Pays-Bas, en Allemagne et au Royaume-Uni. Dans les briques, le plastique d'origine fossile est remplacé par des polymères biosourcés, issus des déchets de canne à sucre (bagasse) ou forestiers. Cela permettra de réduire les émissions de gaz carbonique de 25% par rapport à celles générés par les emballages précédents. Pour les pots de yaourt, « le passage aux plastiques recyclables signifie que ceux-ci peuvent avoir une seconde vie si les systèmes de recyclage sur les marchés le permettent », indique Arla dans un communiqué. Le groupe laitier précise avoir déjà diminué l'impact en CO2 de ses emballages de 25% depuis 2005, ce qui correspond à un gain total de 123 000 tonnes. Et de citer les exemples des bouteilles de lait au Royaume-Uni, qui contiennent entre 30 et 40% de polyéthylène haute densité (PEhd) recyclé, et des opercules des pots de crème fraîche, qui intègrent 85% de polyéthylène téréphtalate recyclé (rPET).
</article>
<article titre="Les 15 valeurs favorites du cabinet Midcap Partners pour 2019" date="2019 05 14" journal="Investir (site web)">
A l'occasion de la Conférence Midcap Partners qui se déroule durant deux jours à Paris, nous avons pu croiser le responsable du bureau d'analyse financière, Charles-Louis Planade. Il nous explique que l'équipe de recherche se concentre sur l'univers des small et midcaps en Europe. La « short list » se compose de 15 fortes convictions données à un horizon de 12 à 18 mois. Elle est établie en début d'année et peu modifiée ensuite, en tout cas pas chaque mois, même si les objectifs peuvent évoluer. En effet, les gérants ont parfois besoin d'un mois pour constituer une ligne. Comme, sur le plan macroéconomique, nous sommes plus proches du haut que du bas de cycle, les valeurs les plus cycliques comme Derichebourg sont écartées. Charles-Louis Planade cite d'abord  Albioma , avec un objectif de 26 euros. Les résultats de ce producteur d'électricité à partir de la bagasse (résidus de la canne à sucre) ou de l'énergie solaire dans les départements d'outre-mer sont sécurisés à 95% par des contrats avec EDF. Le titre offre en outre un rendement correct de 3,5%.   Bonne place au streaming   La sélection est aussi très orientée vers la production de contenus pour les plateformes de streaming. On y trouve ainsi  Xilam  et  Mediawan , mais aussi  Focus Home Interactive  avec un objectif relevé de 38 à 45 euros. Cet éditeur de jeux vidéo décote de plus de 50% par rapport à ses comparables en Suède, alors que le départ du dirigeant en avril n'a pas provoqué le drame attendu. Un nouveau jeu bien noté, La peste, vient de sortir. Enfin, il existe un aspect spéculatif, l'actionnaire principal, M. Thebaud, étant susceptible de vendre ses 34%. L'édition de jeux vidéo est désormais concernée par le streaming, comme la musique et la vidéo. Des plateformes se créent (Strada de Google, Microsoft, bientôt Amazon). Du coup, le succès ne se joue plus en trois semaines mais en trois ou quatre ans. Le chiffre d'affaires repose en partie sur le catalogue de jeux sortis les années précédentes. Les distributeurs sont prêts à payer cher les contenus pour avoir l'exclusivité.   Charles-Louis Planade cite enfin  Maisons du Monde  pour viser 36 euros. La valorisation de ce distributeur de meubles est jugée dérisoire. Des sites web sans magasins sont entrés en Bourse en Allemagne, ont cassé les prix et ont dépensé les sommes levées en frais marketing. Mais cette période touche à sa fin et Midcap Partners croit à la stratégie multicanal de Maisons du Monde, associant site et points de vente physiques.   -------------   Retrouvez nos premiers comptes-rendus de cette conférence de deux jours organisée par Midcap Partners :   Vetoquinol: des moyens pour de nouvelles acquisitions   Global Ecopower: des projets de complexes écologiques   Interparfums: trop tôt pour un bilan du lancement d'Explorer   Vente-unique.com: un nouvel outil logistique bientôt opérationnel   Les autres à suivre dès demain !   Jean-Luc Champetier, à la Conférence Midcap Partners
</article>
<article titre="Auchan élimine le plastique de sa vaisselle à usage unique" date="2019 05 10" journal="Emballages magazine (site web)">
L'enseigne propose des alternatives à base de matériaux biosourcés en France.-   Anticipant sur la prochaine interdiction de la vaisselle à usage unique en plastique, et dans le droit fil de son engagement à travers le Pacte national sur les emballages plastique, signé le 21 février dernier, Auchan Retail France va, à compter de ce mois-ci, éliminer progressivement de ses rayons les produits concernés à sa marque. Les alternatives, « plus responsables », sont fabriquées à partir de matériaux biosourcés. Les assiettes et les bols sont en bagasse (résidus de canne à sucre), « 100% compostables industriellement et certifiés OK compost », précise l'enseigne, qui indique aussi qu'ils « sont pratiques d'utilisation puisqu'ils sont rigides, solides, résistants à l'eau et utilisables même au micro-ondes » . Une partie des couverts est produite à partir d'amidon de maïs et est également compostable industriellement. Le reste des couverts est en bois de bouleau « issu de forêts gérées durablement (FSC) » . Enfin, les gobelets et une autre partie des assiettes sont en carton. Le distributeur annonce que cela représente une économie de plus de 885 tonnes de plastique par an. Ecobox   Auchan profite de cette annonce pour rappeler ses actions dans le domaine depuis 2015. L'enseigne est ainsi partenaire de Cristaline pour le recyclage des bouteilles en polyéthylène téréphtalate (PET) via le dispositif Ecobox, « avec près de 65 millions de bouteilles recyclées » . Elle met également en place des filières de tri plastique incolore / plastique coloré. En outre, elle s'est engagée à ne plus utiliser de polychlorure de vinyle (PVC) dans ses emballages ménagers, commerciaux et industriels d'ici à 2022.   Auchan Retail France compte 637 points de vente et 73 800 salariés.
</article>
<article titre="Élisabeth Ayrault « Les entreprises n'ont pas pris la mesure du changement climatique »" date="2019 05 2" journal="L'Usine Nouvelle, no. 3609">
Vous soutenez le pacte finance-climat proposé par l'économiste Pierre Larrouturou et le climatologue Jean Jouzel. Pourquoi ? En France, nous nous trompons en pointant systématiquement la faiblesse du pouvoir politique pour transformer notre pays. Ce n'est pas le monde politique qui bloque, mais les entreprises qui n'ont pas pris la mesure du changement climatique, alors même qu'il peut les tuer. Veolia, Engie et Total l'ont compris, sans doute parce que la transition énergétique est leur business. Mais les autres entreprises et la population ne suivent pas. Pour les entraîner, il faut de l'argent. Car les économies d'énergie nécessitent des investissements préalables importants, pour de nouveaux outils de production dans l'industrie, des voitures et des chaudières moins énergivores pour les particuliers, la rénovation des bâtiments. Le monde de la finance doit se mobiliser dans ce sens et raisonner à long terme.   Ressentez-vous déjà les effets du changement climatique dans vos activités ?   Depuis dix ans, nous constatons qu'il y a moins d'eau dans le Rhône, mais pas de manière flagrante. Le sujet important, c'est l'amplitude des variations. En 2016 et 2017, nous avons subi seize mois de sécheresse continue, avec 30 % d'eau en moins, suivis en 2018 par trois crues majeures et six mois d'eaux élevées. C'est la marque d'un dérèglement. Il faut que l'on se prépare à gérer des conflits d'usage entre l'eau potable, la navigation, l'irrigation, la biodiversité, les loisirs et la production d'électricité, comme c'est déjà le cas pour la Garonne.   Comment vous préparez-vous à ces futurs conflits d'usage ?   Nous travaillons à une meilleure gestion de l'eau du Rhône entre les différents bénéficiaires. Il faut aussi préparer la Compagnie nationale du Rhône (CNR) au foisonnement des énergies. La majeure partie de notre chiffre d'affaires provient toujours de l'hydraulique, mais l'éolien et le photovoltaïque progressent. Nous pilotons 3 000 mégawatts de capacités installées sur le Rhône. En 2020, nous piloterons 1 000 mégawatts supplémentaires de solaire et d'éolien. Nous travaillons aussi sur le stockage d'énergie, les batteries et l'hydrogène vert. Dans ce domaine, nous participons au programme power-to-gas Jupiter 1000, à Fos-sur-Mer, dans les Bouches-du-Rhône. Nous avons un projet de production d'hydrogène vert par électrolyse à l'usine de Pierre-Bénite, au sud de Lyon. Cet outil modulaire, de 10 à 20 mégawatts au départ, pourra atteindre 50 mégawatts. Ce sera l'un des plus importants en France. L'hydrogène pourra être injecté dans le réseau de gaz, alimenter les sept stations d'avitaillement d'hydrogène en construction à Lyon et les industriels de la vallée de la chimie. Il pourra aussi être « retransformé » en électricité. La première brique de l'électrolyseur pourrait entrer en phase industrielle d'ici à trois ou quatre ans.   La CNR a un projet à Marie-Galante, dans les Antilles. C'est très loin du Rhône...   Au départ, nous avions répondu à un appel à manifestation d'intérêt sur la ville durable. Puis, après des discussions avec des acteurs locaux, le projet a évolué vers une île durable, autonome en énergie. L'idée initiale était de permettre à Marie-Galante de s'affranchir du câble qui la relie à la Guadeloupe et lui fournit de l'électricité produite à partir de fioul. Un autre acteur, Albioma, avait un projet de production d'électricité à partir de biomasse importée et de bagasse, le résidu de la canne à sucre après sa transformation, qui pouvait aussi alimenter la Guadeloupe. Les deux projets ont dû s'adapter et ont été redimensionnés pour permettre à l'île d'être autosuffisante en électricité 100 % renouvelable, sans importation de biomasse. Nous allons installer 25 mégawatts d'énergies renouvelables et 30 mégawatts de stockage.   Que pensez-vous de la programmation pluriannuelle de l'énergie (PPE), notamment de la quasi-absence de l'hydrolien dans le projet ?   La PPE est peut-être imparfaite et insuffisante, mais elle a le mérite de fixer les choses. Elle commence enfin à aborder les sujets essentiels. Viser 40 % d'électricité d'origine renouvelable à l'horizon 2030, c'est déjà bien. L'effort qu'il reste à faire est très important. Mais pour atteindre une autonomie énergétique à l'horizon 2050, comme le projette l'Ademe, il faudra s'en donner les moyens et ne pas se limiter aux énergies dominantes, comme le solaire. Un mix est nécessaire. Il ne faut pas éliminer une production d'énergie électrique sous prétexte qu'elle est trop chère aujourd'hui. Pour autant, les projets d'hydroliennes en mer sont très capitalistiques et hors du champ de la CNR. En revanche, nous allons être un acteur majeur de l'hydrolien fluvial, avec le projet de ferme d'hydroliennes sur un site du Haut-Rhône, financé par l'Ademe. Nous étions partis, avec nos partenaires Hydroquest et CMN, sur 39 hydroliennes, mais après étude, il y en aura probablement 30 plus puissantes. Nous nous sommes positionnés sur ce projet, non pour le potentiel en France, mais pour celui qu'il a à l'international, où de nombreux sites ne sont pas connectés aux réseaux. C'est aussi une façon pour CNR d'accompagner nos ingénieurs qui travaillent sur des solutions à proposer à l'étranger.   L'Europe demande une ouverture à la concurrence des concessions hydroélectriques. Où en est ce dossier ?   La CNR n'est pas concernée, et ce pour deux raisons. D'une part, elle n'exploite pas une concession hydroélectrique, mais une concession d'aménagement et de gestion du Rhône, avec des missions liées à la navigation et à l'irrigation. D'autre part, elle a engagé auprès du gouvernement français une demande de prolongation jusqu'en 2041 de sa concession, qui est unique pour tous les ouvrages. Celle-ci devait s'achever en 2023, lorsque le premier ouvrage de la Compagnie aura atteint ses 75 ans. L'idée est de recentrer, par la méthode des barycentres, la durée de la concession sur une moyenne de soixante-quinze ans pour l'ensemble des ouvrages, le dernier construit datant de 1986. Une concertation sur ce projet, menée par la Commission nationale du débat public, a été lancée en avril.   Est-il toujours interdit de manger le poisson pêché dans le Rhône, à cause de la pollution, notamment aux polychlorobiphényles (PCB), très toxiques ?   L'interdiction a été levée récemment sur certaines portions du Rhône. Selon l'Agence de l'eau, ses eaux sont soit de bonne qualité, soit de qualité suffisante. Quant aux PCB, ils sont bloqués dans les sédiments du fleuve.   Le mode de gouvernance paritaire de la CNR, réunissant des collectivités territoriales et un opérateur privé, est original. Pourrait-il être dupliqué ?   Le modèle de la CNR, avec 183 collectivités et la Caisse des dépôts, qui détiennent 50,03 % du capital, et un opérateur industriel, Engie, est exceptionnel. Il réconcilie public et privé, dividendes et intérêt général, court et long terme. La loi de transition énergétique de 2015 prévoit déjà pour les concessions hydroélectriques la possibilité de création de sociétés d'économie mixte ouvertes aux collectivités des territoires concernés et à un industriel exploitant l'ouvrage, sur un modèle similaire. C'est une bonne chose.
</article>
<article titre="Une livraison à domicile chahutée" date="2019 05 1" journal="Néorestauration, no. 0574">
«La livraison de repas à domicile au Royaume-Uni est cinq fois supérieure à celle de la France pour des populations similaires en taille » , analyse Jean-Adrien Monleau, directeur des opérations chez Uber Eats. Estimé aux alentours des 3 % dans l'Hexagone, le secteur du food delivery pourrait atteindre la barre des 10 % d'ici à cinq ans. Une étude du cabinet Wyman anticipe même qu'à l'échelle européenne, l'e-commerce alimentaire avoisinera les 25 % en 2030. Et ce gâteau prometteur à la progression XXL semble aussi énorme que les appétits qu'il engendre... Car pour bichonner l'employé de bureau en mal de nouveaux plats ou le consommateur casanier fatigué de cuisiner, les start-up de la food tech redoublent d'efforts. La guerre est à la fois high-tech et impitoyable.  INTENSIFIER SA VISIBILITÉ Le récent fracas de Munchéry aux États-Unis fin 2018, après avoir levé 125 millions de dollars, ou le retrait de Foodora l'été dernier, arguant que les villes françaises n'étaient pas assez denses, sont autant de signaux négatifs envoyés au reste des écuries. Personne ne semble être à l'abri d'un échec. Rester discret sur les chiffres et intensifier sa visibilité constituent les quelques règles d'or des start-up encore en lice. À coups de levées de fonds à plusieurs millions et d'armées de développeurs qui peaufinent des algorithmes dernier cri, leur objectif à court terme n'est pas tant d'être rentables, mais bien de continuer à se développer en investissant toujours plus.    Le terrain de jeu est planétaire, chaque nouvelle ville intronisée un combat à livrer, et chaque commande réalisée une victoire engrangée. La lutte promet d'être de longue haleine, car la rentabilité n'est pas évidente. Pas pour Just Eat, une exception qui confirme la règle. L'opérateur  « tire sa force de son business model hybride » , résume Gilles Raison, son directeur général. Effectivement,  « notre coeur de métier est la marketplace (90 %), c'est-à-dire que nos restaurateurs partenaires assurent eux-mêmes la livraison ».  Le delivery est de fait proposé seulement aux restaurateurs indépendants ou chaînés, qui ne disposent pas de leur propre flotte de livreurs (lire encadré ci-contre).    LA RENTABILITÉ ? PAS UNE PRIORITÉ ! « On n'est pas rentables et ce n'est pas le but quand on est une société en hyper croissance »,  résume Julia Bijaoui, la cofondatrice de Frichti. Son discours ultra-assumé illustre parfaitement l'état d'esprit du secteur. Créée en 2015 après deux levées de fonds successives (1 M E en septembre, puis 12 M E en décembre 2015, viendra une troisième à 30 M E en mai 2017), la société compte aujourd'hui 300 salariés en CDI. Une croissance fulgurante qui ne donne pas le vertige à cette jeune entrepreneuse d'à peine 30 ans, convaincue d'avoir fait les bons choix logistiques :  « Notre ambition, c'est d'être leader sur ce marché. »  À l'instar de FoodChéri ou de Nestor (offre de repas gastronomique), Frichti a opté pour le business model « Full stack delivery ». En clair, ces sociétés ont décidé de produire leurs propres recettes en aménageant des cuisines dédiées. À la différence des Deliveroo ou Uber Eats, qui se rémunèrent en prélevant des commissions sur les restaurateurs et les livreurs, la volonté cette fois est de maîtriser l'ensemble de la chaîne de valeur, de la production à la livraison. Frichti loue près de 1 000 m2 près de Rungis et emploie 80 cuisiniers.  « Le nerf de la guerre, c'est la logistique. Notre système est très performant pour une raison simple : on part d'un seul et même endroit pour livrer plusieurs clients » , développe Julia Bijaoui.    PARIS OU LE BAPTÊME DU FEU Car une fois produits, les plats sont acheminés dans Paris par camion, puis répartis parmi la quinzaine de hubs (locaux) que possède le groupe. Le dernier tronçon restant pour atteindre le client est réalisé par une flotte de livreurs (une centaine environ) qui ont un statut d'auto-entrepreneurs.  « L'énorme différence, c'est que nous pouvons organiser des tournées de livraison, c'est-à-dire qu'un même livreur peut livrer plusieurs clients en une fois » , poursuit la fondatrice. Pour couronner le tout, les plats sont livrés froids, évitant toute la problématique liée à la conservation des aliments.    La capitale compte à la louche deux millions d'habitants  « en temps normal »,  mais le midi et en semaine, ce chiffre frôle plus les six millions. Paris présente une grande densité de population en comparaison avec les villes américaines, beaucoup plus étendues. Ce critère est un atout et figure parmi les raisons invoquées par les start-up pour y justifier leur présence.    Arrivée sur le marché la même année que Frichti, FoodChéri présente un modèle assez similaire. À la différence près que cette dernière a un actionnaire majoritaire depuis janvier 2018 : l'entreprise Sodexo (plus de 10 M E investis).  « On n'est pas rentables, personne ne l'est. Prise à l'unité, chaque commande l'est. Maintenant, ce qui nous reste à payer, ce sont les frais de structure, le siège social, les frais fixes, etc. Mais chaque mois qui passe, nous le sommes davantage » , assure Julie Vernin, cofondatrice de FoodChéri. Aucune mauvaise surprise n'est à déplorer pour le moment. Avec un chiffre d'affaires de 10 millions d'euros en 2018, la start-up espère atteindre la rentabilité dès 2021. Preuve de sa croissance, une nouvelle cuisine de 2 000 m2 a remplacé l'ancienne (450 m2) début avril.    MIDI ET SOIR, MÊME COMBAT !  Historiquement engagés sur le créneau du midi pour les urbains actifs lassés de la cantine d'entreprise, FoodChéri et Frichti s'attaquent aussi au repas du soir. Un segment bien plus lucratif, à en croire les spécialistes. Scruté à la loupe, le consommateur de fin de journée adopterait un comportement moins regardant sur le prix et les calories. La fameuse « confortfood ».  « On a remarqué que le client en journée a tendance à privilégier des plats sains et équilibrés. Mais le soir, c'est plus orienté fast-food »,  analyse encore Julie Vernin. Pour gagner des parts de marché, FoodChéri a donc lancé Season, un concept sous forme d'abonnement. Le consommateur reçoit chez lui, en une fois, sa commande pour la semaine. L'avantage du modèle : les plats froids peuvent être livrés dans tout l'Hexagone via la Poste.    De son côté, Frichti développe des partenariats avec des restaurateurs, cinq à l'heure actuelle, avec chacun une spécialité.  « Un qui fait des poké, un italien, un sushi, un grec et un pâtissier (partenariat avec Christophe Michalak) » , liste Julia Bijaoui.  « Le marché de livraison de repas en France est très tourné vers le soir, symbolisé par une surreprésentation des commandes par rapport au déjeuner » , renchérit Jean-Adrien Monleau (Uber Eats).   MULTIPLIER LES PARTENAIRES, MAIS PAS QUE... Contrairement au modèle full stack, Uber Eats et Deliveroo tirent leurs profits en s'appuyant sur un système de commissions. Ici, pas de cuisine, pas d'équipes qui élaborent des recettes, pas de réseaux de producteurs, le business model se concentre sur le développement d'une application où sont référencés des milliers de restaurateurs.  « Imaginez qu'un client passe une commande pour 15 E, un prix fixé par le restaurateur. À cela s'ajoutent les frais de livraison  (entre 2,50 et 3 E, que paie le consommateur, NDLR).  Le restaurateur reçoit la commande sur une tablette (fournie par Deliveroo), qu'il peut accepter ou refuser. Sur les 15 E, Deliveroo va prendre une commission de 30 %, un chiffre qui peut varier en fonction des négociations commerciales (volumes, taille du restaurant, implantation...). Le livreur fait ensuite la course et sera rémunéré en fonction du temps estimé, de l'heure de la journée, de la distance, et non pas du montant de la commande. [...] Ce qu'il est important de comprendre, c'est que le prix que va toucher le livreur  (entre 6 et 8 E bruts en moyenne, NDLR)  pour une commande est bien entendu plus élevé que ce que va payer le consommateur dans les frais de livraison. Donc une grosse partie de la commission que nous prenons au restaurateur sert à payer le livreur » , indique le porte-parole de Deliveroo. Notons au passage que certains de ces opérateurs (Uber Eats) prennent une taxe de 25 % sur la rémunération brute du livreur.  « L'équation économique n'est pas des plus évidentes » , admet Jean-Adrien Monleau.    Pour être rentables et accroître leur volume d'affaires, ces deux géants du food delivery parient sur l'expansion : réaliser toujours plus de commandes afin de réduire les coûts. Mais ce n'est pas tout. Dès son arrivée sur le territoire français, Uber Eats a établi un partenariat exclusif avec l'enseigne McDonald's.  « Deliveroo et Uber Eats se basent sur des modèles identiques. Ils ont énormément de difficultés à se différencier. La seule façon pour eux de personnaliser leur offre, c'est de mettre en place des partenariats »,  observe la fondatrice de Frichti.    Deliveroo a décidé de s'attaquer aux zones périurbaines, où l'offre en restauration est plus faible, voire nulle, en créant le concept de Deliveroo Éditions. Lancé à Saint-Ouen (18 cuisines) en juillet 2018 et à Courbevoie (12 cuisines) en octobre, il permet à des restaurateurs de tester leurs recettes dans de nouvelles zones sans avoir à investir dans le dur.  « Le restaurateur doit venir avec son petit matériel de cuisine et ses chefs et se concentrer sur la restauration. De notre côté, nous prenons en charge l'aménagement, l'électricité, le ménage, la plonge, les ustensiles classiques, etc. » , explique le représentant de Deliveroo.    GAGNER DU TEMPS, UN IMPÉRATIF Uber Eats s'est lancé depuis octobre dernier dans la livraison 24 heures sur 24.  « Les taux de croissance sont extrêmement intéressants »,  déclare Jean-Adrien Monleau. Et depuis fin mars, le groupe américain est en phase pilote à Paris avec un nouveau service appelé Click and collect, mené en partenariat avec 800 restaurateurs parisiens. Dans ce modèle, le consommateur va lui-même chercher sa commande, économisant les frais de livraison. Un concept grâce auquel Uber Eats espère répondre en partie au casse-tête de la rentabilité logistique.    Fini le papier et le crayon, le secteur de la livraison de repas à domicile est une affaire de logistique à base d'équations mathématiques sophistiquées. Tous les opérateurs promettent de livrer leurs plats en moins de trente minutes. Pour y arriver, ils procèdent à une analyse continue de leur big data.  « Nous estimons que le temps effectif de la livraison à proprement parler est de sept minutes. Ce temps-là est incompressible » , indique le porte-parole de Deliveroo. Tout le travail des développeurs consiste donc à optimiser les autres temps restants. En 2017, Deliveroo a justement racheté la start-up Maple dans ce but.  « L'algorithme Franck de cette start-up nous permet d'identifier avec précision le moment où il faut appeler le livreur pour qu'il arrive pile lorsque le plat est prêt. Depuis que nous l'avons mis en place, nous avons gagné 20 % de temps de livraison. » LE BOOM DES RESTAURANTS FANTÔMES Sentant le bon filon du food delivery, de nouveaux opérateurs sont entrés sur le marché avec un autre type de business model que l'on pourrait qualifier d'hybride. Fondée par un groupe de six restaurateurs français, Dark Kitchen propose des restaurants sans salles, uniquement dédiés à la livraison. Contrairement aux autres modèles, cette société investit cette fois en achetant les fonds de commerce.  « Avant de créer Dark Kitchen, on était en partenariat classique avec Uber Eats et nos restaurants traditionnels. Ils nous prenaient une commission et on considérait que tout ce qui était vendu en livraison faisait partie d'un chiffre d'affaires additionel »,  raconte Jean Valfort, un des cofondateurs de Dark Kitchen. Mais très vite, ce dernier réalise les limites du système.  « La priorité d'un restaurateur, c'est de satisfaire son client en salle. Donc,  de facto,  celui qui commande chez lui va devoir attendre un peu. »  En outre, les allées et venues des livreurs casques viscés sur la tête à l'intérieur du restaurant n'était pas non plus du meilleur effet. De plus, certains plats livrés à domicile supportaient très mal le transport.  « À titre d'exemple, nous avons dans un de nos restaurants un superbe magret de canard reconnu par la critique. Ce produit commandé arrivait parfois froid. Résultat : on a eu certains retours négatifs. »  LE SUCCÈS DES RESTAURANTS VIRTUELS Cette série d'arguments a incité ces restaurateurs-entrepreneurs à tenter l'aventure eux-mêmes. La première « kitchen » a été inaugurée en avril 2018, une deuxième fin mars 2019, une troisième est en cours de finalisation à Bordeaux, et deux nouvelles verront le jour dans l'ouest parisien courant juin.  « Chaque kitchen est un point de production qui regroupe plusieurs restaurants »,  détaille Jean Valfort. En clair, une cuisine propose en réalité un éventail de spécialités : burgers, tacos, plats asiatiques et bientôt pizzas constituent la carte.  « On s'est dit qu'on allait commencer par les produits les plus demandés, les fameux best-sellers. »  Avec 250 à 300 commandes en moyenne par jour sur la première kitchen, pour des montants oscillant entre 15 et 20 E, le concept a le vent en poupe.  « Ce que je peux vous dire, c'est que le chiffre d'affaires est très bon. On est sur des investissements dix fois inférieurs à ceux de la restauration traditionnelle pour un chiffre d'affaires équivalent. »  Autre point relevé, le fait de ne plus avoir besoin de serveurs en salle permet de compenser les pertes liées aux frais de livraison saisis par Uber Eats. Trouver le bon emplacement se révèle être le principal défi :  « Notre problématique est de trouver des points physiques. » Ce concept des restaurants virtuels est en pleine expansion. Uber Eats déclare travailler avec 200 restaurants virtuels dans l'Hexagone.  « On observe que de plus en plus de restaurants reconstruisent leurs espaces pour optimiser la partie livraison avec le développement de guichets donnant directement sur la rue » , commente Jean-Adrien Monleau.  « Pourquoi ne pas faire des partenariats avec Dark Kitchen ? En tous cas, le concept est intéressant » , opine pour sa part Julia Bijaoui, de Frichti.    UN CONSOMMATEUR PEU FIDÈLE Dans ce marché de la « disruption », certaines start-up n'hésitent pas à aller encore plus loin. Livrer des repas ? Et pourquoi pas autre chose ? L'entreprise espagnole Glovo a décidé de livrer tous types de produits de commerce : du plat à manger au médicament, en passant par des clés ou des parfums. Son modèle économique est très similaire à ceux d'Uber Eats et Deliveroo, avec près de 65 % de son chiffre d'affaires réalisé sur le segment de la distribution alimentaire.  « La valeur ajoutée de Glovo repose justement sur les 35 % restants qui ne sont pas liés à la food. [...] Nous avons une plus forte qualité de clients avec un panier moyen plus elevé, car nous les amenons sur le segment de la restauration par le biais d'autres objets non alimentaires » , déclare Alexandre Fitoussi, directeur général de Glovo France. La start-up qui a levé plus de 115 M E, s'est implantée dans 22 pays en un an et demi.   « Du point de vue du coursier, on a une offre de plage horaire largement supérieure. De 9 heures à minuit, du lundi au dimanche, tandis que les autres plates-formes vont se concentrer sur les heures des repas. On livre exactement les mêmes restaurants qu'Uber Eats et Deliveroo. Le comportement des consommateurs est très peu fidèle. Ce qui va favoriser la fidélité, c'est l'exclusivité » , ajoute t-il. Selon ses informations, Glovo serait d'ailleurs sur le point de chiper le partenariat exclusif d'Uber Eats avec McDonald's. Info ou intox ? Difficile de se prononcer, mais cela traduit une réelle instabilité d'un segment où les alliances semblent se faire et se défaire au gré du vent.   LES POKÉ BOWLS ONT LA COTE Et question rentabilité, Glovo n'échappe pas à la règle.  « Les levées de fonds servent à l'expansion géographique. À horizon de septembre, nous atteindrons une franche rentabilité qui couvrira les frais variables et les frais fixes. [...] Pour l'instant, nous en sommes à 5 000 commandes par jour à Paris. On représente un peu moins de la moitié de Deliveroo et peut-être quasiment le tiers d'Uber Eats »,  estime le directeur général France de Glovo, qui ne cache pas l'ambition du groupe d'atteindre le milliard en valorisation financière.    À ceux qui pensaient que les Français n'étaient pas très portés sur le fast-food, Uber Eats et consorts, s'appuyant sur leurs big data, sont pourtant unanimes. Le classement des plats les plus commandés est sans appel : le burger est le best-seller des produits les plus plébiscités, suivi des pizzas, talonnées de très près par les sushis. Mais la plus grosse progression revient aux poké bowls. Ces plats à base de riz et de poissons crus accompagnés de légumes en tout genre ont vu leur consommation multipliée par huit entre 2017 et 2018.  « Les réseaux sociaux les ont également fortement popularisés. C'est plein de couleurs, ils véhiculent une image très saine, très équilibrée, donc très tendance. Toutes les étoiles sont alignées pour en faire un des produits phares de la livraison » , note le directeur des opérations d'Uber Eats.
</article>
<article titre="2-projet_de_plan_national_de_gestion_des_dechets.pdf (www.consultations-publiques.developpement-durable.gouv.fr) - 2019-04-24" date="2019 04 19" journal="Rapports - Transport - Transports France">
168 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... PLAN NATIONAL DE GESTION DES DECHETS  VERSION PROJET  18 AVRIL 2019 PLAN NATIONAL DE GESTION DES DECHETS  Table des matières  LISTE DES ABREVIATIONS.............................................................................................................4 INTRODUCTION................................................................................................................................6 I. ANALYSE DE LA SITUATION...    Pour voir le rapport :  http://www.consultations-publiques.developpement-durable.gouv.fr/IMG/pdf/2-projet_de_plan_national_de_gestion_des_dechets.pdf
</article>
<article titre="Albioma Caraïbes : vers le 100% biomasse" date="2019 04 18" journal="Dechetcom (site web)">
Albioma Caraïbes : vers le 100% biomasse    Albioma Le Moule a souscrit pour sa centrale Albioma Caraïbes (34 MW - mise en service en 2011) un nouveau prêt long terme (19 ans) et sans recours sur l'actionnaire de 68 millions d'euros, en complément de la dette existante...    Ce prêt servira à financer la réalisation des nouvelles infrastructures et des modifications des installations de la centrale, en vue de son fonctionnement à 100% à la biomasse en substitution totale du charbon au cours de l'année 2020. L'enveloppe servira également à financer les travaux nécessaires à la mise en conformité de la centrale avec la directive IED (relative aux émissions industrielles) applicable à compter du 1er janvier 2020...  L'opération est réalisée auprès des banques historiques de la société : BNP Paribas et CIC, en tant que co-arrangeurs, et Bpifrance, à des conditions de marché très attractives. A l'issue de la réalisation de ces investissements, le Groupe exploitera dans la zone Caraïbes 2 unités thermiques fonctionnant exclusivement à base de biomasse. En septembre 2018, Albioma avait annoncé la mise en service industriel de la première centrale 100% bagasse/biomasse d'Outre-mer, Galion 2, située en Martinique (voir notre article).    Pour rappel : implanté en Outre-Mer français, à l'Ile Maurice et au Brésil, Albioma a développé depuis 25 ans un partenariat unique avec le monde sucrier pour produire de l'énergie renouvelable à partir de la bagasse, résidu fibreux de la canne à sucre. Le Groupe est aussi le premier producteur d'énergie photovoltaïque en Outre-mer où il construit et exploite des projets innovants avec stockage.   Cet article est paru dans Dechetcom (site web)
</article>
<article titre="Une cantine matapédienne fête ses 40 ans" date="2019 04 10" journal="L'Avant-Poste (Matane, QC), no. Vol. 77 n° 9">
CÉLÉBRATIONS. Le mois d'avril marquait la quarantième année d'existence de la Cantine Fortier, ce lieu de rassemblement gourmand du boulevard Saint-Benoît, à Amqui. Que ce soit pour casser la croûte ou pour siroter un bon café entre amis, la Cantine Fortier demeure une halte-bouffe convoitée, et ce, même après 40 ans de service. Installé au même endroit depuis le commencement, le restaurant Cantine Fortier a su perdurer dans le temps en misant sur la qualité des aliments. La persévérance, la clé du succès C'est en observant la propriétaire, madame Fernande Lavoie, derrière les fourneaux qu'on comprend vite que la passion de la restauration ne s'est pas atténuée avec les années. Toujours à la barre du commerce après quarante ans, Fernande Lavoie, n'a pas l'intention de se retirer du terrain prochainement. Si madame Lavoie et son défunt mari avaient prévu opérer la cantine à court-terme au tout début, c'est plutôt une longue histoire d'amour qui est née entre les propriétaires et la clientèle. Fernande Lavoie admet néanmoins que les premières années de l'entreprise se sont avérées ardues.   « On était à logement, le terrain ne nous appartenait pas dans les 4 ou 5 premières années. Comme on fermait l'hiver, il fallait travailler fort l'été pour ramasser assez d'argent pour vivre. On a fait énormément d'heures pour pouvoir joindre les deux bouts », a révélé madame Fernande Lavoie.   Il faut dire qu'au départ, le quartier où se situe la Cantine Fortier était beaucoup moins développé et davantage industriel. « Avant ça, on était comme perdu un petit peu, il n'y avait rien dans le coin. » De fil en aiguille, Fernande et son mari sont parvenus à prendre entièrement possession des lieux en créant un endroit de rassemblement reconnu pour la qualité de ses plats servis.   Cuisiner aves des aliments frais et locaux Fernande est d'avis que le choix de fournisseurs s'avère crucial. « Quand t'arrives avec du fromage frais à tous les matins, c'est sûr que ça donne une chance. Travailler avec un produit frais, ça aide à la qualité », a exprimé Fernande en parlant d'un des mets les plus réputés de sa cantine, soit la poutine. Madame Lavoie rappelle l'importance d'utiliser des produits locaux notamment pour assurer la longévité des entreprises d'ici.   « On essaye d'encourager notre local, on veut pas négliger ça. On achète du frais local, on travaille avec le coin pour offrir la meilleure qualité à notre clientèle ». Rappelons que le burger mexicain « Fernandez » a récemment fait son entrée à la Cantine Fortier pour souligner les 40 ans du commerce. Il s'agit d'un joli clin d'œil aux quarante belles années de service offert par Fernande et son équipe. Notons que certains employés de la Cantine Fortier en sont à leur trentième année de service au sein de l'entreprise amquienne.   Un virage écologique Soucieuse de l'environnement, la Cantine Fortier a fait le choix de servir les aliments dans des contenants compostables en bagasse afin de délaisser le styromousse. Une décision certes plus onéreuse qui s'insère toutefois dans une perspective responsable et durable, pour le bien de l'environnement. « Si tout le monde devient écoresponsable, la planète se portera mieux. »   cdastous@lexismedia.ca
</article>
<article titre="En Guadeloupe (3/6) Une histoire façonnée par la canne à sucre et l'esclavage" date="2019 04 6" journal="La Croix (site web)">
L'arrivée des tracteurs à l'usine Gardel (PB)Le quai de déchargement chez Gardel (PB)   Ce matin de mars, la campagne sucrière bat son plein. Elle s'annonce plus modeste que l'an dernier, notamment à cause des pluies abondantes en fin de campagne. Au Moule, sur l'île de Grande-Terre, sur le quai de déchargement de la sucrerie Gardel, les tracteurs n'en sont pas moins à la queue-leu-leu pour décharger leurs énormes remorques.   (PB)(PB)(PB)   Une fois pesées, les tiges de canne sont lavées, séchées, puis dirigées vers des moulins pour y être pressées. Le jus vert obtenu donnera, après de multiples opérations de purification et de séchage, du sucre roux brut, longtemps quasi totalement expédié par bateau vers des usines de la « métropole» pour y être transformé en sucre blanc. De ces opérations, depuis la canne jusqu'aux produits finis, les touristes un peu curieux peuvent de faire une bonne idée en participant à une des nombreuses visites guidées de l'usine Gardel. Il en coûte 8 € par adulte (4 € pour les enfants de moins de 14 ans). Il faut simplement prévoir des chaussures fermées. Et être prêt à affronter le bruit inhérent à toute installation industrielle. Le parcours réservé aux touristes est sécurisé. Il exclut donc les passerelles d'où la vue plongerait directement sur les moulins. La découverte de l'univers du sucre n'en est pas moins passionnante.   Chez Gardel, l'heure est aux produits finis, parfois joliment sophistiqués (PB)   Gardel est l'une des deux dernières sucreries de la Guadeloupe, qui compte encore quelque 3500 producteurs sur Basse-Terre et Grande-Terre (4 hectares en moyenne, par exploitation). Gardel possède sa propre exploitation (980 hectares), mais c'est d'abord la plus grosse sucrerie de l'archipel. Las, les temps sont rudes. En effet, depuis la disparition des quotas sucriers européens à la fin de 2017, le prix du sucre en vrac, désormais aligné sur les cours mondiaux, a été divisé par deux. La filière « sucre » qui fait travailler 10 000 personnes, doit donc déployer des trésors d'inventivité pour trouver de nouveaux débouchés, en dehors d'un marché européen saturé et très concurrentiel.   « Nous adaptons nos installations pour fabriquer sur place davantage de sucre conditionné, explique Alain Bazir, le directeur commercial de Gardel. Nous avons revu notre politique commerciale pour écouler ce sucre, réputé pour sa qualité, dans l'arc caribéen ». Y a-t-il vraiment des débouchés dans les Caraïbes ? Oui, à en croire Alain Bazir. « Dans cette zone, explique-t-il, seuls la Barbade, Saint Domingue, la Jamaïque et Cuba produisent encore du sucre. Quant à la Martinique, elle met l'accent sur le rhum ». De ce fait, beaucoup de pays de l'arc caribéen avaient pris l'habitude d'importer du sucre, pas toujours de bonne qualité, des Etats-Unis ou d'Amérique latine. Gardel pense pouvoir, à bon droit, les séduire avec toute la gamme de produits finis qu'il développe désormais et dont la boutique, près de l'usine, présente de beaux exemplaires. « Notre objectif est de produire désormais autant de sucre conditionné que de sucre en vrac. Pour nous, c'est une grosse évolution, reprend Alain Bazir qui se félicite d'être « quasiment déjà à l'objectif ».   Bien sûr, Gardel continue aussi à produire de la mélasse destinée à être transformée en rhum par une distillerie basée à Saint-Rose, et bien sûr de la bagasse (nom donné à ce qui reste une fois que la canne, pressée, a été délestée de son jus). Depuis trente ans, cette bagasse est livrée à la Centrale thermique du Moule dont les installations se trouvent à deux pas . Elle y sera transformée en vapeur pour faire tourner l'usine Gardel et aussi pour produire de l'électricité pour alimenter la Guadeloupe. « Nous produisons aussi des écumes », complète Alain Bazir. Des « écumes » ? En simplifiant, les processus de purification du sucre dégagent des boues qui, une fois mélangés à de la bagasse, donnent des amendements organiques ensuite épandus dans les champs de canne. « Notre procédé utilise peu d'eau, souligne Alain Bazir. Nous utilisons d'ailleurs de moins en moins d'eau. De plus, nous retraitons l'eau que nous utilisons et elle sert ensuite à faire de l'irrigation. Il y a zéro déchets ». Un beau modèle d'économie circulaire en somme.   « Nous travaillons aussi sur la filière du sucre bio. Elle n'existe pas dans les DOM (départements d'Outre-Mer) alors que les produits bio se développent partout. Le sucre bio qui est vendu en métropole vient du Mozambique, du Brésil, etc. « , ajoute encore le directeur commercial de Gardel qui estime jouable de passer environ 10 % de la production en bio.   Des paysages toujours façonnés par la canne à sucre (PB)   Comment imaginer, en effet, la Guadeloupe sans canne, et par contrecoup sans sucre, sans sirop et sans rhum ?? 10 000 emplois sont en jeu sur ces îles où le chômage et la pauvreté sont prégnants. Et puis, aussi douloureuse soit-elle, toute l'histoire de l'archipel est liée à la culture de cette plante herbacée. Lorsqu'elle a été « découverte » en 1493 par Christophe Colomb, la Guadeloupe était en effet peuplée d'Amérindiens, des Taïnos et surtout des Calinagos, venus par vagues successives d'Amérique, entre 1500 avant J-C et 1200 après J-C. Ils seront bientôt décimés par les maladies apportés par les « colons » ou presque totalement exterminés.   C'est ainsi que la culture de la canne à sucre, exigeante en main d'oeuvre, qui a succédé très vite à celle du tabac, a entraîné l'arrivée d'esclaves africains. Les premiers d'entre eux arrivent dans les îles de la Guadeloupe, quatre ans à peine après que les Français s'en soient emparés en 1635. Le mouvement s'accélère après l'arrivée, en 1654, de colons juifs hollandais qui fuient le Brésil avec 300 de leur esclaves. Bientôt, le nombre d'esclaves noirs va augmenter de manière exponentielle car la « traite » transatlantique bat son plein. Ainsi, à la révolution française en 1789, la Guadeloupe compte 13712 blancs, mais près de 90 000 esclaves noirs.   Le Mémorial ACTe, vue extérieure (PB)   Depuis 2015, à Pointe-à-Pitre, cette tragédie est racontée en détail au Mémorial ACTe, installé sur le site de l'ancienne usine sucrière Darboussier, dans un élégant bâtiment moderne conçu par des architectes guadeloupéens.   La galerie intérieure du Mémorial ACTe (PB)   C'est un très beau bâtiment. C'est aussi et surtout le plus ambitieux lieu de mémoire jamais dédié à l'esclavage car, loin de se cantonner à la problématique des Antilles, il brosse une fresque générale de cette pratique, de l'Antiquité à nos jours, sur tous les continents et évoque aussi les discriminations et le racisme ont ont été (et sont encore) victimes des personnes à la peau noire.?L'entrée du Mémorial coûte certes 15 €. Mais sa visite est incontournable pour qui veut mieux saisir la face sombre de ces îles riantes. Il faut prévoir d'y passer une bonne demi journée, ce ne sera pas trop. En effet, outre l'exposition permanente, le Mémorial accueille des expositions temporaires. Celle qui était en place lors de notre passage et qui reste jusqu'au 29 juin 2019, était consacré aux amérindiens Taïnos et à leur histoire. Y sont exposés de nombreux -et très beaux- objets façonnés par ces Amérindiens qui peuplaient largement les Grandes Antilles avant l'arrivée des colons.   Vue de Pointe-à-Pitre, depuis le bateau qui conduit à Marie Galante (PB)   Pour se faire une idée plus concrète du terrible sorte réservé aux esclaves, on prendra à Pointe-à-Pitre le bateau pour Marie-Galante (compter une heure pour la traversée). Cette délicieuse île garde encore un peu l'allure désuète que devaient avoir Grande-Terre et Basse-Terre, il y a quelques décennies. Les champs de canne à sucre y façonnent toujours largement le paysage.   Les installations de la distillerie Bielle (PB)Joël Moysan et son épouse (PB)   Outre une sucrerie de Grande-Anse qui doit probablement son maintien aux contraintes de l'insularité (trop coûteux de transférer sur Grande-Terre la canne de l'île), Marie Galante compte trois distilleries de rhum -la plus importante s'appelle Bielle- et de petites fabriques de « sirop de batterie », fort en goût, fabriqué directement avec le jus de la canne (1), comme celle de Joël Moysan et de son épouse Lisette. Pour en savoir plus, voir leur page Facebook.   Vue de Caspesterre de Marie Galante (PB)Photo prise de la terrasse de chez Rudy (PB)   Une journée de balade ne sera pas de trop, si l'on veut flâner un peu à Grand-Bourg où se trouve le débarcadaire, avant d'explorer l'intérieur de l'île, visiter une siropterie, et aller, par exemple, se régaler à Capasterre de Marie-Galante d'une délicieuse cuisine antillaise, par exemple rue de la Marine, à La Source chez Rudy (Tel 0590 97 26 01) : mais, pour se déplacer, il faudra soit louer une voiture, soit s'allouer les services d'un guide. Pour cela, s'adresser par exemple aux transports Attaud.   En effet, à Marie Galante, il faut aussi -et surtout- visiter les imposants vestiges des «habitations » (nom donné localement aux plantations), Mûrat et Roussel. Quoiqu' inégalement conservées, elles témoignent de la puissance passée des planteurs qui devaient leur prospérité à la sueur de leurs esclaves, plus d'une fois révoltés contre les mauvais traitements qui leur étaient infligés.   Les ruines du moulin et la cheminée carrée témoignent de la puissance passée de l'habitation Mûrat (PB)   Située à Grande-Bourg, l'habitation Murât est l'une des toutes premières plantations de la Guadeloupe. Son fondateur serait Antoine Luce, un notaire d'origine champenoise arrivé à Marie-Galante en 1657. En 1807, Dominique Murât, originaire d'Aquitaine, rachète la propriété, la modernise et la dote de nouvelles installations vers 1814. Incontestablement, l'habitation Murât était, en 1839, avec ses 207 esclaves, la plus grosse plantation de canne à sucre de la Guadeloupe. La légende prétend que se serait Jeanne Laballe, ancienne élève des beaux-arts, épouse de Dominique Murât, qui aurait dessiné au début du XIXè siècle le château où logeait le Maître. Aujourd'hui, c'est un écomusée des Arts et Traditions Populaires qui rassemble sur quelques centaines de mètres carrés trois siècles d'histoire sucrière de la Guadeloupe à l'époque coloniale. Le musée propose un centre de documentation et de lecture, ainsi qu'un jardin de plantes médicinales et d'authentiques « cases en gaulettes ».   Les anciennes écuries de l'habitation Roussel-Trianon (PB)   Par ailleurs, en partant de Grand-Bourg et en suivant la Nationale vers Saint-Louis sur 2 km, on trouve sur sa gauche l'habitation Roussel-Trianon. Elle est désormais propriété du Conseil Général de la Guadeloupe. De la maison du « maître », il ne reste qu'un imposant socle auquel conduisent des escaliers. Ce socle imposant dit la richesse passée du site. En revanche, l'ancienne écurie, faite de brique et de pierre de taille est en très bon état. Située un peu plus loin, l'ancienne usine, aujourd'hui envahie par la végétation, était, au XVIIIe siècle, la seule des Antilles à produire du sucre raffiné. Visible depuis la route nationale, la cheminée -carrée- de l'ancienne sucrerie, toujours visible, est répertoriée aux monuments historiques. De l'ancien moulin, il reste seulement la base, visible également depuis la route nationale.   A Marie-Galante, un seul moulin, plus ou moins rénové, possède encore ses ailes (PB)   A cette époque en effet, chaque plantation comptait son moulin à vent pour transformer la canne en jus, puis en sucre. D'où le surnom d'« île aux 100 moulins » donné à la pittoresque île de Marie-Galante.   L'esclavage, supprimé une première fois en 1794 par la Révolution française, sera rétabli dès 1802 par Napoléon (sous l'influence de son épouse Joséphine, dit-on). Son abolition définitive en 1848, après le long combat de Victor Schoelcher, bouleversera l'économie sucrière. Elle entraînera notamment l'abandon des moulins -un seul, rénové et donc pourvu de ses ailes en bois, témoigne encore à Marie Galante de ce passé qui ne passe pas- au profit d'usines sucrières spécialisées, de plus en plus grandes et de moins en moins nombreuses, comme nous l'avons vu.   Pour faire face à leurs besoins de main d'oeuvre, les îles de la Guadeloupe auront alors recours à des travailleurs sous contrat originaires d'Inde -de Pondichery, notamment-, d'Indochine, du Japon, d'Afrique. Ces Africains venus alors principalement du Mali, du Sénégal et du Congo et qui ne furent donc pas esclaves, étaient alors nommés localement des « Noirs Congo ». En principe, ces travailleurs venaient pour assurer un contrat de cinq ans. Mais beaucoup se sont installés et ont fait souche, accroissant ainsi les métissages compliqués qui font l'identité de la Guadeloupe. Les Indiens ne se sont pas contenté d'apporter le curcuma qui a enrichi la cuisine de l'archipel, ils ont aussi pratiqué sur place le culte des dieux hindous et créé des temples. Ainsi, à Capesterre, sur l'île de Basse-Terre où il avait beaucoup de plantations de canne et donc de besoins de main d'oeuvre, il y a un temple hindou. Et, aujourd'hui, à Saint-François et au Moule, il subsiste toujours une importante communauté d'origine indienne.   « Ici, nous sommes tous des mélanges », assure d'ailleurs Richard, le guide qui nous a accompagné dans la découverte de Marie-Galante. Ce Guadeloupéen très noir de peau nous a confié avoir eu un grand-père blanc, une grand-mère Malinure, autrement dit originaire du Mali, et une autre grand-mère amérindienne. Taïna, notre guide sur Grande-Terre et Basse-Terre, pourtant très blanche de peau, nous a assuré avoir eu un père métis, une mère blanche, un grand-père noir et une grand-mère picarde. Quand à Joël Moysan, il nous a dit être « d'une famille de descendants d'esclaves mais aussi de Bretons ». Ils ne sont pas des exceptions, tant s'en faut. Cela fait de la Guadeloupe des îles métissées, définitivement.   (1) Alors que le sirop ordinaire est fabriqué en mélangeant du sucre de canne et de l'eau, le sirop de batterie (ou sirop batterie) est un concentré du jus de canne cuit obtenu par évaporation. Le nom « batterie » vient de la cinquième et dernière chaudière utilisée pour fabriquer le sucre dans les anciennes sucreries. C'est un produit très répandu en Guadeloupe et en Martinique.  Cet article est paru dans La Croix (site web)
</article>
<article titre="Feuille de route Analyse du secteur Des géants en mutation et des petits spécialistes" date="2019 04 6" journal="Investir-Le Journal des finances, no. 2361">
Engie n'a pas été le seul,  parmi les grands énergéticiens européens, à remettre son modèle en question ces dernières années. Même le monolithe EDF s'est engagé fin novembre à présenter à l'Etat actionnaire, au second semestre 2019, une nouvelle architecture de ses structures, tout en restant un groupe intégré. Un holding détenu par l'Etat pourrait ainsi coiffer, d'une part, le nucléaire, aux très gros besoins de financement, et, d'autre part, les réseaux et les activités dans les énergies renouvelables, qui resteraient cotées en Bourse, selon un schéma détaillé à l'automne par l'agence Reuters.   Ce sont sans doute les allemandsE.ON etRWE qui ont procédé aux réorganisations les plus radicales, non sans revirements après un premier schéma conçu en 2016. Ils ont annoncé l'année dernière une répartition de leurs actifs par métiers : àE.ON la production électrique, d'origine renouvelable ou non ; à RWE les réseaux, la distribution et les services. Des fonds pour l'électricité verte Séparer les filières facilite leur financement et leur développement. Beaucoup d'argent, désormais, cherche à s'investir dans la transition énergétique, quand il est devenu très compliqué de trouver des partenaires et des fonds pour construire des EPR, comme EDF et son associé chinoisCGN en ont fait l'expérience au Royaume-Uni. Assureurs, banques et fonds spécialisés en infrastructures ne jurent plus que par les énergies renouvelables !   Ainsi,Neoen, exploitant de parcs solaires et éoliens, a réussi son entrée en Bourse en octobre, au pire de la débâcle boursière. A côté des mastodontes Engie et EDF, la cote parisienne compte en effet une série d'acteurs spécialisés dans le renouvelable :Voltalia,Global Ecopower etFuturen, acquis par EDF en 2017.   Plus anciennement connu de nos lecteurs,Albioma développe des centrales solaires et, surtout, va convertir à la biomasse ses centrales thermiques de La Réunion et des Antilles, qui brûlent encore du charbon, quand la bagasse issue de la canne à sucre n'est pas disponible. N'oublions pas le tout petitMethanor, qui prend des tickets dans des centrales, comme le quasi-million d'euros investi dans le projet hydroélectrique d'Aqua Bella en Savoie.   Notre avis secteur Atout La transition énergétique et la lutte contre le réchauffement climatique offrent de nouvelles opportunités.   Risque Changer de modèle requiert des investissements et du temps pour faire ses preuves.   Favorite (ABIO) a renoué avec une forte croissance de ses profits, qui sera confirmée cette année par la mise en service de deux nouvelles centrales.   à éviter EDF (EDF) est handicapé par une équation financière très lourde pour financer à la fois la filière nucléaire et son expansion dans les énergies renouvelables. Le groupe a très fortement réduit son rendement, à 2,5 %, et déçu le marché en février par ses objectifs à 2019.   C. L. C.
</article>
<article titre="Parfum : vers de nouveaux modes d'extraction ?" date="2019 04 1" journal="CosmétiqueMag Hebdo">
En préambule du 4e congrès Olfaction et Perspectives, ayant pour thème les nouveaux champs d'application, organisé par la Cosmetic Valley avec l'Isipca en mars à Paris (photo), l'accent est mis sur le vert et l'environnement. Comme dans les autres segments de la cosmétique, l'attente du consommateur vers davantage de naturel tant dans les ingrédients que les procédés de fabrication touchent aussi l'univers du parfum. « L'évolution des modes d'extraction des matières premières est nécessaire. D'une part, les contraintes des techniques actuelles sont grandes : elles sont énergivores, consomment beaucoup d'eau, et il faudrait limiter l'utilisation de solvants qui sont pour la plupart d'origine pétrolière », expliquent Xavier Fernandez, professeur à l'Université de Nice, et Xavier Brochet, chimiste chez Firmenich. « Les labels sont en outre de plus en plus demandés par le consommateur, c'est un autre facteur qui pousse l'industrie à innover », complètent-ils. Gaz liquides et pressurisés. Aujourd'hui, la recherche travaille à trouver des modes d'extraction plus vertueux pour compléter l'offre du CO2 supercritique, utilisée depuis une vingtaine d'années. Le fluide est neutre et non-polluant, disponible à haute pureté et à bas prix, et plusieurs ingrédients odorants ont déjà été produits et commercialisés avec cette technique. Parmi les nouveaux solvants testés, les gaz liquides et pressurisés : « C'est une technique que l'on connaît depuis longtemps, mais elle fait appel au butane, qui est un frein au développement car il présente un enjeu de sécurité », expliquent-ils. Les coproduits sont aussi une piste, comme le méthyl THF qui est issu de la bagasse, le résidu fibreux de la canne à sucre après l'avoir passé au moulin pour en extraire le suc. « Son principal inconvénient est la température d'ébullition élevée qui dénaturerait les composés volatils », précise Xavier Comme dans les autres segments de la cosmétique, l'attente du consommateur vers davantage de naturel touchent aussi l'univers du parfum.   Brochet qui complète, « l'eau pourrait être le solvant idéal mais il présente un point critique élevé ».   Moins d'énergie. Autre enjeu majeur pour une production plus responsable, la diminution de l'énergie nécessaire. « Il y a les ultrasons qui permettent de destructurer la matière pour une extraction plus rapide au solvant. Le challenge aujourd'hui, c'est l'application de ce procédé sur des grands volumes », selon le scientifique. « Il y a aussi les micro-ondes : leur efficacité de chauffage est reconnue pour activer toutes les molécules polarisables pour une extraction plus rapide. Aujourd'hui, la technique fonctionne par lot, mais il faudrait imaginer un process en continu pour déployer cette technique », conclutil. En résumé, de nouvelles solutions existent, mais les applications industrielles sont pour l'instant assez confidentielles. L'un des freins majeurs réside dans la nécessité de faire des investissements pour acquérir les outils nécessaires. Deux leviers de développement existent : la parfumerie de niche, qui a besoin de plus petits volumes et qui pourrait donc expérimenter l'une de ces pistes à son échelle, ou bien la réglementation, si elle décide un jour d'interdire un solvant.
</article>
<article titre="Ces entreprises qui misent déjà sur l'alternative au plastique" date="2019 03 29" journal="Décision Achats (site web)">
Le 27 mars 2019, l'Union Européenne vote l'interdiction des plastiques à usage unique d'ici à 2021. Certaines entreprises n'ont pas attendu une telle décision pour mettre au point des produits visant à remplacer le plastique. Présentation de 5 business garantis sans danger pour les océans.   4 / 6 La VeganBottle, la gourde 100% vegan    En Charente-Maritime, Lyspackaging imagine, fabrique et commercialise des bouteilles à partir de bioplastique. Sa dernière innovation ? La VeganBottle, une gourde entièrement biodégradable et compostable, garantie sans OGM. Elle est réalisée à partir de la bagasse de canne à sucre (un résidu obtenu une fois le sucre extrait de la canne). La VeganBottle est couverte par plusieurs brevets et fait actuellement l'objet d'une campagne de financement participatif sur Ulule. Plus de 1000 préventes ont été enregistrées auprès de clients particuliers (pour un objectif de départ de 100 commandes). La production doit être lancée en avril. Nicolas Moufflet, fondateur de Lyspackaging en 2016, couple la vente de chaque VeganBottle à la plantation d'un arbre à Madagascar. Ce programme de reforestation lui permet d'annuler tous les impacts d'émission à effet de serre de l'entreprise. Il favorise aussi la création d'emplois locaux. Lyspackaging a levé 2,2 millions d'euros fin 2018 pour mieux se structurer et faciliter l'industrialisation de ses produits. L'entreprise propose en effet d'autres contenants (flacons, carafes...), conçus sur-mesure pour le marché BtoB.   Cet article est paru dans Décision Achats (site web)
</article>
<article titre="Petites recettes pour un monde sans plastiques" date="2019 03 21" journal="Sud Ouest  Charente">
Séverine Joubert   "Le meilleur déchet est celui qui n'existe pas. " Eva Maria Dautry-Autin fait sien le principe énoncé par Nicolas Moufflet, l'inventeur de la bouteille végétale et invité de la soirée de ce jeudi [lire ci-contre]. Le lycée Georges-Desclaudes accueille la Journée mondiale de l'eau, dans l'amphithéâtre, à partir de 18 heures. L'entreprise Lyspackaging illustre, qui plus est localement, les ressources intellectuelles et techniques qui existent pour " ne pas subir " et " proposer des solutions et alternatives " plutôt que de " débattre sans fin " et de tourner en rond dans " la dénonciation ", pose Eva Maria Dautry-Autin, coordinatrice générale de Terdev (Terre et développement).   Cette année, le thème de la Journée mondiale de l'eau, née en 2015 à l'initiative du Comité Amérique Latine, est le plastique, de sa présence dans les océans et les fleuves à son intrusion dans l'organisme des humains.   Un voilier avaleur de déchets   Cinq invités vont ainsi répondre à cette question : " Des rivières aux océans, quelles solutions contre la pollution ? ". Le skipper Yvan Bourgnon, vainqueur de la transat Jacques-Vabre en double, en 1997, viendra présenter son projet de catamaran géant, collecteur de gros déchets plastiques en mer et le long des côtes. Fondateur et président de l'association Sea Cleaners, il est la tête d'affiche de cette soirée. Le prototype de son voilier avaleur de déchets avance avec le soleil, le vent et le carburant issu de la pyrolyse des plastiques collectés en mer.   Élodie Libaud-Hugues, cheffe du service Gestion de l'eau et milieux aquatiques du Conseil départemental de la Charente-Maritime, évoquera le contrôle et la maîtrise des eaux du marais, des marais, des estuaires, de l'eau potable.   " Nous sommes dans la pédagogie et dans l'explication ", appuie Eva Maria Dautry-Autin. " Ce n'est pas toujours facile d'appréhender les enjeux techniques, politiques ou économiques. Ce n'est pas pour dédouaner les politiques, mais pour mieux comprendre pourquoi les intentions ne sont pas toujours appliquées et pour comprendre de façon globale, comment ça fonctionne. "   Dans ce même souci de meilleure connaissance de son environnement, Terdev a convié Jacques Lépine, responsable du service " hydrologie " au syndicat départemental des eaux.   Frédéric Neveu, adjoint en charge du service " eau et assainissement " de la Ville de Saintes parlera du programme Re-Sources et des actions possible de protection de la ressource en eau.   Le cinquième intervenant sera Stéphane Trifiletti, conseiller régional, professeur au lycée Georges-Desclaudes et élu de la commission du schéma d'aménagement et de gestion des eaux (Sage) Charente.   Journée mondiale de l'eau, amphithéâtre du lycée Georges-Desclaudes, aujourd'hui à 18 heures. Entrée libre.   LYSPACKAGING Il est l'un des cinq intervenants de la soirée. Nicolas Moufflet, dirigeant de la société Lyspackaging installée à Chaniers, présentera sa bouteille 100 % végétale et donc 100 % compostable. Veganbottle a séduit SNCF Développement, qui a versé 80 000 € en prêts participatifs dans le cadre du fonds de soutien à l'économie locale. La bouteille végétale de Lyspackaging est réalisée à partir de bagasse de canne à sucre et de bases végétales biosourcées.
</article>
<article titre="Stratégie Albioma se la joue solaire" date="2019 03 21" journal="L'Usine Nouvelle, no. 3603">
Champion outremarin de la production d'électricité à base de bagasse (816 MW installés, dont 168 MW au Brésil), Albioma mise désormais sur le soleil. Il dispose de plus de 95 MW de centrales photovoltaïques si l'on compte la nouvelle unité de 1,34 MWc avec stockage (1,33 MWh), qu'il vient de mettre en service à La Réunion. Fin décembre 2018, Albioma avait aussi ajouté 17 MW de photovoltaïque en métropole via l'acquisition d'Eneco France, une PME d'Avignon qui lui apporte une équipe de douze personnes pour profiter du boom solaire annoncé en France. Après avoir investi 790 millions d'euros dans son développement entre 2013 et 2018, notamment au Brésil et à la Martinique, Albioma envisage d'investir encore entre 500 et 700 millions d'euros d'ici à 2023, dans le solaire, la biomasse et la sortie du charbon. §
</article>
<article titre="Conseil Albioma Le coup de coeur" date="2019 03 16" journal="Investir-Le Journal des finances, no. 2358">
Le producteur d'électricité a vu son cours repasser au-dessus de 20 € à la faveur de résultats 2018 et de prévisions pour 2020 plus robustes que prévu par les analystes. Ainsi, pour un chiffre d'affaires de 428 millions d'euros, en amélioration de 6 %, l'Ebitda (excédent brut d'exploitation) s'est accru de 18 %, à 162,6 millions, comme le bénéfice net, à 44,2 millions.   Bien qu'elle se soit fait attendre, la centrale thermique de Galion 2, à la Martinique, est entrée en service fin septembre. Son taux de disponibilité de 100 % en fin d'année explique en partie les performances d'Albioma. Le groupe a aussi bénéficié d'avenants aux contrats signés avecEDF (auquel il vend l'électricité produite outre-mer), à l'occasion de travaux destinés à réduire les émissions de fumées des centrales thermiques. Pour ce nouvel exercice 2019, l'Ebitda est attendu en progression, pour se situer entre 168 et 178 millions d'euros. Une autre installation est entrée en service en ce début d'année, la turbine à combustion (TAC) de Saint-Pierre, à La Réunion. Les résultats bénéficieront également de l'acquisition de la société Eneco France, spécialisée dans les parcs solaires sur toitures dans le sud de la France.   En 2020, c'est un Ebitda d'environ 200 millions qui est visé. Les analystes n'en attendaient pas tant. « C'est en 2020 que nous allons toucher les bénéfices de nos investissements en cours », explique Julien Gauthier, directeur général adjoint chargé des finances.   Remplacer le charbon Le groupe a, au demeurant, renforcé le plan d'investissements à dix ans, lancé en 2013 pour un montant de 1 milliard d'euros. Il va être porté entre 1,3 et 1,5 milliard : 790 millions ont déjà été dépensés et la nouvelle enveloppe pour 2019-2023 a été fixée entre 500 et 700 millions.   Elle doit couvrir quatre domaines, dont la conversion à la seule biomasse des centrales thermiques d'outre-mer. Ces dernières brûlent les résidus de canne (la bagasse) pendant les campagnes sucrières mais recourent au charbon le reste de l'année. La houille sera remplacée par de la biomasse. Les travaux de conversion (de réduction des fumées) viennent d'être engagés sur le site d'Albioma Caraïbes, en Guadeloupe, et devraient se terminer début 2020. Seront aussi transformées l'autre centrale de Guadeloupe et les deux unités de La Réunion.   Les investissements devront également couvrir la construction éventuelle de nouvelles centrales thermiques, l'accélération du développement dans le solaire et la poursuite des acquisitions d'unités de cogénération au Brésil, toujours prévues tous les douze à dix-huit mois.   Notre conseil Acheter Déjà « Coup de coeur » il y a trois mois, la valeur a repris 15 % depuis. Mais les perspectives de résultats à 2020 ont favorablement surpris le marché. Le dividende de 0,65 € offre, de plus, un rendement d'un gros 3 % avant la majoration des 10 % pour les plus fidèles.   Par Cécile Le Coz
</article>
<article titre="Car-ferries : la chasse au plastique est lancée" date="2019 03 15" journal="Ouest-France">
Enquête   Un croque-monsieur dans une assiette à usage unique, des couverts en plastique et une petite bouteille d’eau. Ce plateau-repas, en plastique lui aussi, servi au snack d’un ferry, n’existera bientôt plus. De la Manche à la Méditerranée, les compagnies maritimes font la chasse au polymère. Une prise de conscience environnementale soudaine ? Pas tout à fait. La plupart ont entamé la démarche il y a longtemps ; toutes ont désormais l’obligation légale de s’y atteler. La loi agriculture et alimentation donne un objectif clair : pailles, bâtonnets mélangeurs, couverts, couvercles à verre jetables, plateaux-repas, pots à glace, saladiers et boîtes seront bannis à partir de 2020.   Agir à la source   Déjà, en 2015, la loi de transition énergétique avait chassé les sacs plastique jusque dans les supermarchés. Une éternité après la Corse qui les avait bannis en 2003 à l’initiative du Festival du vent. Ce qui avait encouragé les compagnies maritimes à suivre le mouvement.   «  Comment aurions-nous pu distribuer des sacs plastique à bord alors qu’ils étaient interdits sur l’île ?  »   interroge Pierre Mattei, président de Corsica Ferries.   Mais les sacs ne sont que la partie émergée de l’iceberg des déchets non recyclables. En 2018, par exemple, Condor Ferries a transporté un million de passagers entre la France et l’Angleterre. Ils ont utilisé 152 000 fourchettes, 158 000 couteaux, 93 000 cuillères et 42 000 pailles, ainsi que 557 000 tasses et 158 000 couvercles. Tous triés à bord et débarqués dans les ports avec lesquels chaque compagnie signe une convention.   Comme la compagnie britannique, qui veut réduire de 75 % l’utilisation du plastique à bord de ses navires d’ici à 2021, la plupart ont pris le taureau par les cornes. Agitateurs, gobelets, cure-dents, pailles : tout ce qui n’est pas réutilisable doit disparaître. Remplacés par des matières recyclables : carton, PLA (polymère entièrement biodégradable en compostage industriel), bois, bambou, bagasse (résidu fibreux de la canne à sucre)…   Le chantier est tel que la Brittany Ferries (2,63 millions de passagers en 2018) va embaucher un chef de projet dédié à la diminution et la valorisation des déchets.   «  L’objectif est de supprimer ce qu’on génère mais aussi d’agir à la source dès qu’on le peut  »  , souligne Loïk Prigent, directeur d’exploitation.   Les changements se logent dans les petits détails : la suppression du plastique entourant les draps des cabines à leur retour de la blanchisserie, la disparition des sacs dans les poubelles des salles d’eau, ou encore, l’apparition de couverts en inox et d’assiettes en porcelaine.    «  Un surcoût important  »     «  En matière d’organisation, c’est beaucoup plus lourd à gérer, c’est plus simple de mettre du jetable ,  reconnaît Loïk Prigent.  Mais c’est dans la logique des choses, les mentalités ont changé.  »  Pas question d’être à la traîne. Surtout quand les préoccupations environnementales prennent une place grandissante dans l’opinion publique. Dans un récent sondage, les passagers de la Méridionale (275 000 personnes en 2018 entre Marseille et la Corse) ont fait savoir qu’à service égal, 72 % choisiraient une compagnie soucieuse de l’environnement plutôt qu’une autre.   Pourquoi ne pas l’avoir fait avant alors ? La réponse est simple : le coût.   «  De 40 % à 350 % de plus, en fonction du matériel et des fournisseu rs  »  ,  estime Pierre Mattei, de Corsica Ferries. Sans compter que les fournisseurs de produits biodégradables sont désormais très sollicités. Mieux vaut donc ne pas attendre l’échéance fixée par la loi pour passer commande.  Cet article est paru dans Ouest-France
</article>
<article titre="Albioma : sur les 20 euros" date="2019 03 13" journal="Boursier (site web)">
Albioma reste stable sur les 20 euros ce mercredi, alors que Albioma Le Moule a souscrit pour sa centrale Albioma Caraïbes (34 MW - mise en service en 2011) un nouveau prêt long terme (19 ans) et sans recours sur l'actionnaire de 68 millions d'euros, en complément de la dette existante.  Ce prêt servira à financer la réalisation des nouvelles infrastructures et des modifications des installations de la centrale, en vue de son fonctionnement à 100% à la biomasse en substitution totale du charbon au cours de l'année 2020.    L'enveloppe servira également à financer les travaux nécessaires à la mise en conformité de la centrale avec la directive IED applicable à compter du 1er janvier 2020.   "La conversion aux 100% renouvelables de toutes les unités (9 tranches) est un objectif pour Albioma, et réalisable d'ici 2023. Ces investissements sont importants mais assurent de la croissance car ils sont rémunérés à travers une augmentation de la prime fixe" commente Portzamparc qui reste acheteur en visant un cours de 24,30 euros sur le dossier.   L'opération est réalisée auprès des banques historiques de la société : BNP Paribas et CIC, en tant que co-arrangeurs, et Bpifrance, à des conditions de marché très attractives.   À l'issue de la réalisation de ces investissements, le Groupe exploitera dans la zone Caraïbes deux unités thermiques fonctionnant exclusivement à base de biomasse. En septembre 2018, Albioma avait annoncé la mise en service industriel de la première centrale 100% bagasse/biomasse d'Outre-mer, Galion 2, située en Martinique...
</article>
<article titre="Albioma Caraïbes : signature d'un financement de 68 ME pour la conversion 100% biomasse de la centrale" date="2019 03 13" journal="Boursier (site web)">
Albioma Le Moule a souscrit pour sa centrale Albioma Caraïbes (34 MW - mise en service en 2011) un nouveau prêt long terme (19 ans) et sans recours sur l'actionnaire de 68 millions d'euros, en complément de la dette existante. Ce prêt servira à financer la réalisation des nouvelles infrastructures et des modifications des installations de la centrale, en vue de son fonctionnement à 100% à la biomasse en substitution totale du charbon au cours de l'année 2020.  L'enveloppe servira également à financer les travaux nécessaires à la mise en conformité de la centrale avec la directive IED applicable à compter du 1er janvier 2020.   L'opération est réalisée auprès des banques historiques de la société : BNP Paribas et CIC, en tant que co-arrangeurs, et Bpifrance, à des conditions de marché très attractives.   À l'issue de la réalisation de ces investissements, le Groupe exploitera dans la zone Caraïbes deux unités thermiques fonctionnant exclusivement à base de biomasse. En septembre 2018, Albioma avait annoncé la mise en service industriel de la première centrale 100% bagasse/biomasse d'Outre-mer, Galion 2, située en Martinique.   Prochain rendez-vous avec le chiffre d'affaires du premier trimestre de l'exercice 2019, le 24 avril 2019 (avant bourse).
</article>
<article titre="A suivre aujourd'hui... ALBIOMA" date="2019 03 13" journal="Option Finance (site web)">
AOF - 13 mars 2019    (AOF) - Albioma Le Moule a souscrit pour sa centrale Albioma Caraïbes (34 MW - mise en service en 2011) un nouveau prêt long terme (19 ans) et sans recours sur l'actionnaire de 68 millions d'euros, en complément de la dette existante. Ce prêt servira à financer la réalisation des nouvelles infrastructures et des modifications des installations de la centrale, en vue de son fonctionnement à 100 % à la biomasse en substitution totale du charbon au cours de l'année 2020.   L'enveloppe servira également à financer les travaux nécessaires à la mise en conformité de la centrale avec la directive IED applicable à compter du 1er janvier 2020. À l'issue de la réalisation de ces investissements, le producteur d'énergie renouvelable indépendant exploitera dans la zone Caraïbes deux unités thermiques fonctionnant exclusivement à base de biomasse. En septembre 2018, Albioma avait annoncé la mise en service industriel de la première centrale 100 % bagasse/biomasse d'Outre-mer, Galion 2, située en Martinique.  Cet article est paru dans Option Finance (site web)
</article>
<article titre="L'énergéticien français Albioma verdit son électricité" date="2019 03 11" journal="Les Echos, no. 22904">
Le producteur d'électricité Albioma est unique en son genre. A la fois par la nature de son activité - il génère du courant à partir de déchets de canne à sucre, de bioéthanol ou d'énergie solaire - comme par son positionnement géographique - il est implanté essentiellement dans les territoires français d'outre-mer et au Brésil. La petite société française a vu son chiffre d'affaires progresser de 6 % l'an dernier, à 428 millions d'euros, pour un excédent brut d'exploitation en hausse de 18 %, à 163 millions. « Nous sommes désormais recentrés sur nos deux métiers, la biomasse et le solaire »,  s'est félicité vendredi le directeur général, Frédéric Moyne. Albioma a cédé son activité de biogaz en métropole qui ne gagnait pas d'argent et s'est renforcé dans le solaire avec l'acquisition d'Eneco France. L'année 2018 a surtout été marquée par la mise en service de Galion 2 en Martinique, une unité de 40 mégawatts à partir de biomasse et en particulier de la combustion de bagasse, le résidu fibreux de la canne à sucre.   Albioma poursuit en parallèle la conversion à la biomasse de ses centrales à charbon. Les renouvelables représentent aujourd'hui 62 % de l'activité, contre 36 % il y a cinq ans. La direction vise une part  « supérieure à 80 % »  en 2023.   En dehors de France, l'entreprise se développe au Brésil, où elle reprend des unités de production d'électricité des groupes sucriers.  « Ces unités ne font pas partie du coeur de métier des sucriers et ne sont souvent pas exploitées de façon optimale,  commente Nicolas Royot, analyste chez Portzamparc.  Elles représentent un gros potentiel pour Albioma dans ce pays. » « Nous voulons atteindre la taille critique le plus rapidement possible au Brésil »,  a annoncé le directeur général.   Jusqu'à 700 millions d'investissements à venir Frédéric Moyne a présenté un ambitieux plan de développement. Albioma prévoit d'investir entre 500 et 700 millions d'euros au cours des cinq prochaines années pour poursuivre la conversion des centrales à charbon, développer les capacités de biomasse et de solaire et enfin accélérer au Brésil et, à terme, dans d'autres pays.   L'entreprise vise un excédent brut d'exploitation d'  « environ 200 millions d'euros »  en 2020. En conséquence, la dette devrait rester à un niveau élevé, comprise entre 4 et 5 fois l'excédent brut d'exploitation.  « Nous sommes une société de croissance »,  a justifié le directeur général.  « Albioma offre une bonne visibilité grâce à des contrats signés sur longue durée, avec des volumes et des prix garantis »,  estime Nicolas Royot. L'action progressait de près de 5 % vendredi après-midi à la Bourse de Paris.   Vincent Collen
</article>
<article titre="L'énergéticien français des Tropiques Albioma verdit son électricité" date="2019 03 9" journal="Les Echos (site web)">
Le producteur d'électricité Albioma est unique en son genre. A la fois par la nature de son activité - il génère du courant à partir de déchets de canne à sucre, de bioéthanol ou d'énergie solaire - comme par son positionnement géographique - il est implanté essentiellement dans les territoires français d'outre-mer et au Brésil. La petite société française a vu son chiffre d'affaires progresser de 6 % l'an dernier, à 428 millions d'euros, pour un excédent brut d'exploitation en hausse de 18 %, à 163 millions.  « Nous sommes désormais recentrés sur nos deux métiers, la biomasse et le solaire »,  s'est félicité vendredi le directeur général, Frédéric Moyne. Albioma a cédé son activité de biogaz en métropole qui ne gagnait pas d'argent et s'est renforcé dans le solaire avec l'acquisition d'Eneco France. L'année 2018 a surtout été marquée par la mise en service de Galion 2 à la Martinique, une unité de 40 mégawatts à partir de biomasse et en particulier de la combustion de bagasse, le résidu fibreux de la canne à sucre.   Conversion des centrales à charbon Albioma poursuit en parallèle la conversion à la biomasse de ses centrales à charbon. Les renouvelables représentent aujourd'hui 62 % de l'activité, contre 36 % il y a cinq ans. La direction vise une part   « supérieure à 80 % »   en 2023.   En dehors de France, l'entreprise se développe au Brésil, où elle reprend des unités de production d'électricité des groupes sucriers.   « Ces unités ne font pas partie du coeur de métier des sucriers et ne sont souvent pas exploitées de façon optimale,  commente Nicolas Royot, analyste chez Portzamparc.  Elles représentent un gros potentiel pour Albioma dans ce pays » .  « Nous voulons atteindre la taille critique le plus rapidement possible au Brésil »,  a annoncé le directeur général.   Jusqu'à 700 millions d'investissements à venir Frédéric Moyne a présenté un ambitieux plan de développement. Albioma prévoit d'investir entre 500 et 700 millions d'euros au cours des cinq prochaines années pour poursuivre la conversion des centrales à charbon, développer les capacités de biomasse et de solaire et enfin accélérer au Brésil et, à terme, dans d'autres pays.    L'entreprise prévoit d'atteindre un excédent brut d'exploitation d'  « environ 200 millions d'euros »   en 2020. En conséquence, la dette devrait rester à un niveau élevé, comprise entre 4 et 5 fois l'excédent brut d'exploitation.   « Nous sommes une société de croissance »,  a justifié le directeur général.    « Albioma offre une bonne visibilité grâce à des contrats signés sur longue durée, avec des volumes et des prix garantis »,  estime Nicolas Royot. L'action progressait de près de 5 % vendredi après-midi à la Bourse de Paris.   Vincent Collen
</article>
<article titre="Albioma pourrait brûler des déchets ménagers dans ses centrales à  bagasse" date="2019 03 8" journal="L'Usine Nouvelle (site web)">
À la Réunion et à la Guadeloupe, le producteur d'électricité outre-marin Albioma étudie la valorisation des combustibles solides de récupération, issu du tri des déchets ménagers, dans ses centrales bagasse/charbon en cours de conversion à la biomasse.   On ne sait pas encore si les combustibles solides de récupération (CSR) viendront en remplacement du charbon ou de la bagasse, voire un peu des deux. Mais Albioma étudie bien la possibilité de brûler les déchets ménagers non recyclables dans ses centrales électriques thermiques à la Réunion et en Guadeloupe. "En sortie des plates-formes de tri mécano-biologiques, les CRS sont utilisables dans nos centrales", a expliqué Frédéric Moyne, directeur général d'Albioma à l'occasion de la présentation aux analystes financier et à la presse des résultats 2018 du groupe le 8 mars à Paris. Sortir du charbon en 2023 Pour l'exploitant de 13 centrales électriques thermiques dans le monde, dont quatre à 100 % bagasse au Brésil, une unité mixte bagasse/biomasse à la Martinique (Galion 2) et huit unités mixtes charbon/bagasse, le CRS pourrait être une solution pour sortir du charbon d'ici à 2023, comme il l'annonce. Car si l'opérateur annonce vouloir convertir ses huit centrales à la biomasse, il sait qu'il ne pourra au mieux le faire qu'à partir de 30 à 40 % de ressources locales et qu'il devra importer de la biomasse de l'étranger. "Nous n'avons aucun problème avec l'importation de biomasse même si nous privilégions les sources locales", explique le PDG d'Albioma. Réduire de 85% ses émissions En prenant en compte les émissions liées à la production de pellets à partir du bois et leur transport, Albioma explique que la conversion du charbon à la biomasse lui permettra de baisse de 85 % ses émissions de CO2 net. Et "ce chiffre est sérieux et audité par des tiers", tiens à préciser Frédéric Moyne. Mais si l'importation de biomasse aide à réduire les émissions, il n'aide pas forcément les territoires outre-marins français à atteindre l'autonomie énergétique en 2030, un objectif fixé par une loi de 2015. Des centres d'enfouissement saturés Moins bon pour le bilan carbone, le recours au CRS représenterait néanmoins un pas de plus pour l'autonomie électrique. Notamment à La Réunion où le CRS pourrait être valorisé à la centrale bagasse/charbon de Bois-Rouge (100 MW), au nord de l'île, situé à proximité du centre d'enfouissement de Sainte-Suzanne. Ce dernier, qui englouti "140 000 tonnes de déchets par an, sera saturé et fermé en 2021", précise le DG d'Albioma. Celui de Saint-Pierre, qui accueillie 240 000 tonnes de déchets par an est aussi à la limite de la saturation. Or s'il existe un projet de construction d'une unité de valorisation énergétique au Sud de l'île, porté par les collectivités locales, Albioma pourrait apporter une solution au Nord. Abandon de la méthanisation Le groupe a prévu d'investir de 500 à 700 millions d'euros dans ses nouveaux développements entre 2019 et 2023, notamment dans la conversion de ses centrales à charbon mais aussi dans le développement du solaire photovoltaïque, le deuxième pilier de croissance du groupe. Sorti complètement de la méthanisation en 2018 par la cession en France de Methaneo à Evergaz, Albioma s'est en revanche renforcé dans le solaire avec l'acquisition d'Eneco France, qui lui apporte 17MW de capacités installées en Métropole et une équipe de 12 personnes à Avignon. Deux piliers : bagasse et photovoltaïque Au total, le groupe dispose de 910 MW de capacité de production électrique installée dans le monde à fin février 2019. Il est le premier producteur de photovoltaïque en outre-mer français avec 17 % de part de marché. Sur ces centrales bagasse, il affiche une efficacité de 120 kwh, injecté dans les réseaux par tonne de canne produite, contre 40 kwh/tc au Brésil en moyenne. En 2018, Albioma a réalisé un chiffre d'affaires de 428 millions et un résultat net part groupe de 44 millions d'euros en croissance de 18 % comparé à 2017. Le groupe opère dans l'Océan indien (Réunion, Mayotte, Maurice) aux Antilles (Guadeloupe et Martinique), en Guyane, au Brésil et en France métropolitaine (30 MW de solaire). Il emploie environ 500 personnes.
</article>
<article titre="Albioma, de plus en plus tourné vers les renouvelables, signe une année 2018 en hausse" date="2019 03 8" journal="AFP Infos Economiques">
Paris, 8 mars 2019 (AFP) - - Le producteur français d'électricité Albioma, surtout présent outre-mer, a dégagé des résultats en hausse l'an dernier, grâce à la mise en service de nouvelles capacités de production et à des évolutions de certains contrats.   Selon un communiqué publié vendredi, sur l'ensemble de l'exercice, le bénéfice net atteint 44,2 millions d'euros, en progression de 18% et supérieur aux attentes du consensus d'analystes compilé par FactSet, pour un chiffre d'affaires de 428,3 millions d'euros, en hausse de 6%. La performance opérationnelle (Ebitda) s'inscrit elle en croissance de 18% à 162,6 millions d'euros, en ligne avec ce que visait le groupe en début d'exercice.   Ces résultats s'expliquent en particulier par les avenants de contrats signés avec EDF, qui ont permis à Albioma de compenser les pertes de production engendrées par la mise aux normes en matière de traitement de fumées sur ses installations thermiques.   Par ailleurs, la société a renforcé son parc de production l'an dernier, avec notamment la mise en service fin septembre de la centrale de Galion 2 en Martinique, qui va multiplier par trois la production d'électricité renouvelable sur l'île.   "2018 est une année riche pour Albioma. Marquée par la mise en service de Galion 2 à la Martinique, première centrale 100% bagasse/biomasse en outre-mer, et par de solides performances opérationnelles de nos installations thermiques et solaires", s'est félicité le directeur général du groupe Frédéric Moyne, cité dans un communiqué.   Le groupe souligne notamment la bonne performance opérationnelle de ces centrales au Brésil, malgré une production en légère baisse, et entend poursuivre ses développements dans ce pays.   Pour l'avenir, il compte notamment sur le rachat de l'activité solaire française d'Eneco France, annoncée fin 2018, et la conversion au 100% biomasse de ces centrales thermiques, à commencer par Albioma Caraïbes en Guadeloupe.   Le groupe confirme ainsi son ambition d'atteindre un mix énergétique à 80% issu d'énergie renouvelable d'ici 2023, contre 62% déjà atteint fin 2018.   Il compte notamment pour cela mobiliser une partie des 500 à 700 millions d'euros d'investissements prévus à cet horizon.   Pour cette année, le groupe vise un Ebitda compris entre 168 et 178 millions d'euros et un bénéfice net entre 38 et 44 millions d'euros. Et il table sur un Ebitda "de l'ordre de 200 millions d'euros" en 2020.   mhc/soe/spi
</article>
<article titre="Albioma poursuit sur sa bonne lancée" date="2019 03 8" journal="GreenUnivers (site web)">
2019-03-08T18:33:18+00:00    (Crédit : Albioma)   Albioma a présenté ce vendredi 8 mars des résultats plutôt flatteurs. L'électricien spécialiste des DOM-TOM poursuit son plan de conversion vers les énergies renouvelables et sa stratégie d'expansion territoriale en métropole, mais pas uniquement.    44 M€ de résultat net Albioma affiche un chiffre d'affaires de 428,3 M€ en 2018, en hausse de 6% par rapport aux 403 M€ de l'exercice 2017 qui était déjà en augmentation de 10%. Une progression qui concerne également l'Ebitda qui s'élève à 162,6 M€ (+18 %). De bons résultats financiers qui s'expliquent en partie par la mise en service de la centrale Galion 2 à La Martinique, première centrale 100% bagasse/biomasse en outre-Mer. La turbine à combustion fonctionnant au bioéthanol de La Réunion ayant été mise en service le 25 février dernier, elle participera au CA cette année. Au final, le résultat net part du groupe est de 44,2 M€ (+18% aussi).    Pour 2019, l'entreprise ne prévoit pas aussi bien, entre 38 et 44 M€ de résultat net en raison « d'amortissements et de charges financières en hausse », explique Frédéric Moyne, directeur général d'Albioma. Ce dernier assure qu'il ne s'agit là que d'une mécanique comptable sans rapport avec les performances des installations.   Sur le volet financier toujours, à noter le refinancement du portefeuille de projets photovoltaïques de l'océan Indien et la mise en place d'une facilité de crédit pour un total 110 M€. Cela permettra au groupe de financer ses nouveaux projets dans la zone pour les 18 prochains mois. Pour rappel, l'entreprise cotée en Bourse a accueilli Impala à son capital en fin d'année dernière.   Expansion territoriale Bien que l'ADN d'Albioma soit intimement marqué par l'insularité, la société commence à sortir de sa zone de confort, notamment en métropole à l'image du rachat en décembre de la filiale française de l'énergéticien Eneco. Les équipes basées à Avignon se concentreront sur les projets de toitures et ombrières photovoltaïques, en participant notamment aux appels d'offres de la DGEC. « Le  time-to-market  est bon. La période de concurrence acharnée est passée comme l'indique les derniers résultats des appels d'offres avec des prix qui remontent. Le marché va se rééquilibrer. C'est le bon moment pour proposer des projets », assure Frédéric Moyne. Si le DG privilégie dans un premier temps la croissance organique, sans objectifs chiffrés, il n'écarte pas pour autant une acquisition ciblée.   Dans les DOM, Albioma compte poursuivre la conversion de ses centrales thermiques à la biomasse. Le groupe espère atteindre 80% d'EnR dans son mix électrique en 2023 contre 62% aujourd'hui. En revanche, il s'est retiré de la méthanisation avec la cession en décembre dernier de l'ensemble de son activité menée par Methaneo. En matière de solaire, l'entreprise a notamment signé en avril 2018 un partenariat stratégique avec la Société Anonyme d'Habitations à Loyer Modéré de La Réunion (SHLMR) visant à construire 51 centrales photovoltaïques sur toitures. Les travaux de construction débuteront au second semestre 2019, avec une mise en service complète en fin d'année 2019, pour une puissance de 4,8 MWc.   Le Brésil reste stratégique Autre continent, autre marché stratégique : le Brésil. Les deux centrales du groupe y ont réalisé des performances en baisse en raison d'un volume de canne broyée en baisse par rapport à 2017 (2,7 Mtc vs. 3,1 Mtc en 2017). La production est donc logiquement en retrait en 2018, à 238 GWh (contre 248 GWh en 2017). A noter que la situation de la centrale de Rio Pardo s'est complexifiée en raison des difficultés financières que traverse son partenaire sucrier Usina Rio Pardo, placé en redressement judiciaire (recuperação judicial).    « Le processus de redressement judiciaire brésilien est très long, ce qui provoque une certaine incertitude qui nous a poussé à passer des provisions en dépréciations d'actifs. Pour autant, plusieurs options possibles sont sur la table et une reprise du site par un autre industriel est envisageable », rassure Frédéric Moyne. Le 21 décembre 2018, Albioma a signé avec le groupe Jalles Machado le contrat définitif permettant l'acquisition de 60% de l'unité de cogénération bagasse adossée à la sucrerie-distillerie éponyme, la 4ème centrale du groupe dans le pays. Autre point positif, le gouvernement de Bolsonaro a confirmé la tenue prochaine des appels d'offres EnR. Albioma regarde également d'autres territoires insulaires sur lesquels il pourrait valoriser son expertise.    Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="Albioma (ex-Sechilienne-Sidec) : un avenir radieux ?" date="2019 03 6" journal="Boursier (site web)">
Echéance.  Les opérateurs retiennent leur souffle sur Albioma dans l'attente des résultats 2018 qui seront connues en fin de semaine. Les prévisions du spécialiste des énergies renouvelables font état d'un Ebitda compris dans une fourchette comprise entre 158 et 166 millions d'euros. Quant au bénéfice net, il est escompté quelque part entre 37 et 42 ME. Albioma aura malgré tout du mal à surprendre agréablement les investisseurs. La centrale Galion 2, projet phare de l'entité en Martinique, est entrée en service tardivement, ce qui induit un manque à gagner sur la guidance de profits. Sans compter que la turbine à combustion (TAC) de Saint-Pierre de La Réunion n'est opérationnelle que depuis le mois dernier, contre fin 2018 prévu initialement. Les analystes tablent ainsi sur des réalisations annuelles dans le bas de la fourchette des prévisions annoncées. Au cours de cette année de transition, Albioma a néanmoins posée de nombreux jalons en vue de l'accélération de son développement. La firme de Frédéric Moyne a poursuivi son développement au Brésil, via notamment l'acquisition de 60% de l'unité de cogénération bagasse adossée à la sucrerie-distillerie éponyme, localisée à Goianésia (Etat du Goiás). En parallèle, Albioma a lancé la conversion à la biomasse de sa centrale " Caraïbes " en Guadeloupe. Adhésion.  Le business model de l'ex-Séchilienne, qui fait la part belle aux partenariats avec les agro-industriels dans le cadre de projets de bagasse/biomasse, est désormais bien rodé. Il se traduit par une amélioration rapide de la profitabilité, sur fond de maîtrise des budgets d'investissements et de montée concomitante de la production. Les contrats à long terme assurent une bonne visibilité ce qui constitue assurément le point fort du dossier. En outre, Albioma devrait devenir d'ici quelques années un producteur d'électricité 100% renouvelables, ce qui ne gâche rien. Face à cette feuille de route séduisante, les investisseurs semblent avoir un train de retard. La faute sans doute aux déboires d'ordre technique et social qui ont pénalisé l'activité ces dernières années. Tous les éléments sont pourtant réunis pour un regain d'adhésion, alors que la valorisation ne semble pas refléter convenablement le potentiel de croissance au Brésil et dans les DOM-TOM, où Albioma a déjà des belles parts de marché. On retiendra aussi que d'autres territoires pourraient venir à terme compléter le dispositif. Ces atouts n'ont pas échappé à Impala qui est récemment entré dans le capital. On sera bien inspiré d'en faire autant en misant sur Albioma dans une perspective de moyen terme. On accumule cette valeur de croissance.Aymeric Val
</article>
<article titre="Emballages, une réelle dynamique vers le naturel" date="2019 03 1" journal="Néorestauration, no. 0572">
«L e plastique, c'est fantastique... » , chantait le leader d'Elmer Food Beat dans les années 80. Si la formule est restée, le combat a légèrement changé de registre. Et la tendance s'est inversée. Quarante ans plus tard, l'emballage plastique n'a plus la cote. Montrée du doigt, la filière plastique, et plus particulièrement celle de l'emballage, navigue en pleine incertitude.  « La directive européenne "Single Use Plastic" vient seulement d'être finalisée et le texte est encore ouvert à de nombreuses interprétations. Celui-ci accorde une grande latitude aux États membres sur les mesures à prendre, notamment pour réduire l'utilisation de certains emballages destinés à la vente à emporter. Dans ce contexte, il ne nous est pas possible, pour le moment, d'évaluer réellement l'impact de l'adoption de cette directive sur la filière française » , esquive l'association Elipso, représentant les fabricants d'emballages plastiques et souples en France.  L'avenir est d'autant plus incertain que l'ébauche du projet de loi « Pour une économie circulaire et une meilleure gestion des déchets », inspirée de la feuille de route « Économie circulaire » présentée en avril dernier, tiendrait en six points et favoriserait la transposition de trois directives du Paquet européen sur l'économie circulaire (sur les déchets, les emballages et déchets d'emballages, et la mise en décharge). Mais ce que craint et dénonce Elipso, ce sont deux amendements adoptés contre l'avis du gouvernement. L'un déposé par le député écologiste des Bouches-du-Rhône, François-Michel Lambert, qui vise l'interdiction des articles en plastique (couverts, piques à steak, couvercles à verre jetables, plateaux repas, pots à glace, saladiers, boîtes) au 1er janvier 2020. L'autre présenté par la députée des Hauts-de-Seine, Laurianne Rossi, pour interdire l'usage de contenants en plastique dans la restauration scolaire à l'horizon 2025.  « Des amendements aux contours flous, aux conséquences économiques lourdes..., et des mesures prises sans concertation et sans étude d'impact » , s'insurgeaient en octobre dernier les représentants d'Elipso, dont 60 % des adhérents interrogés, représentant la moitié des emballages plastiques mis sur le marché français, déclaraient à 98 % vouloir intensifier leur efforts de R et D pour favoriser l'utilisation de matières recyclées et/ou biosourcées et développer des activités ou partenariats pour mettre en oeuvre un recyclage de qualité. Reste l'autre moitié dont on ne parle pas...    À l'heure du lobbying et des négociations en coulisses, les organisations professionnelles ne sont pas non plus très bavardes, alors que le gouvernement vient de rappeler à l'ordre les chaînes de restauration rapide, à ses yeux trop peu ou pas engagées dans le tri des déchets. La secrétaire d'État au ministère de la Transition écologique leur a donné deux mois, jusqu'à fin mars, pour présenter  « un plan de mise en conformité »  en la matière.   L'URGENCE DU RECYCLAGE « Trier, c'est nécessaire, mais encore faudrait-il que les villes mettent à disposition des entreprises les conditions du tri. Or, aujourd'hui, tout ce que l'on trie en magasin finit dans le tout-venant. C'est quand même un peu incohérent et difficile quand on veut faire appliquer des règles aux équipes » , commente Sylvia Barreteau, responsable RSE de l'enseigne de boulangerie et de restauration rapide La Mie Câline. Celle-ci vient de mettre en oeuvre sa politique de RSE pour aller vers le zéro emballage ou le recyclage (voir encadré).  « C'est une contrainte énorme, mais il faut faire respecter les règles. Nous n'avons pas le choix. Il faut que tout le monde s'active » , dit-elle.   Dans un contexte où la consommation alimentaire hors domicile ne cesse de grimper depuis 2015, où tous les indicateurs étaient au vert en 2018, et face à une opinion publique qui réclame des attitudes, des gestes et des produits écoresponsables, les chaînes de vente à emporter s'interrogent. C'est déjà ça. Le « plastique bashing » est passé par là. Du coup, les fournisseurs redoublent d'efforts pour satisfaire une demande pas tout à fait formalisée.  « On nous pose beaucoup de questions. Ce qui est sûr, c'est qu'il y a une véritable prise de conscience depuis un an, et une accélération. Ce n'était pas le cas il y a sept ou dix ans » , constate Frédéric Leduc, directeur commercial, marketing et développement de Gault et Frémont, fabricant d'emballages et solutions de cuisson pour les boulangeries-pâtisseries traditionnelles et la grande distribution, venu sur le marché porteur du snacking il y a une dizaine d'années. Parmi ses clients : La Mie Câline, Brioche Dorée, Banette, Ange, O'Tacos, Carrefour, Elior, Promocash, Transgourmet...    UNE VEILLE ACCRUE SUR LE GREEN  En 2018, le groupe, appuyé sur quatre unités de production en France et en Belgique, a commercialisé 130 millions de boîtes en carton, 3,5 milliards de sacs en papier, 12,5 millions de barquettes de cuisson, et 1 350 milliards de feuilles cuisson. Pour un chiffre d'affaires total de 60 millions d'euros, en croissance de 5 % par an depuis trois ans sur un marché relativement stable. À défaut d'avoir une vision claire sur l'évolution de la réglementation, Gault et Frémont préfère mettre l'accent sur la certification de ses usines (BRC Certifié AA, FSC et PEFC...) et des gammes de produits (OK Compost Home) recyclables et/ou compostables.  « Ici, à l'exception des sacs sandwichs avec fenêtre en plastique, on travaille essentiellement le papier et le carton. Mais pour répondre à la demande, on accroît notre veille sur le "green". On cherche surtout à avoir une démarche "clean", c'est-à-dire avoir des fournisseurs français ou européens. On achète une électricité verte, on recycle nos encres, nos eaux, nos déchets... Et puis on relance nos gammes "engagées" et on essaie des choses... » , précise Frédéric Leduc. À l'étude depuis plusieurs années, les tests sur la cellulose se sont accélérés depuis l'an dernier.  « Mais nous n'avons toujours pas trouvé d'enseignes prêtes à basculer au 100 % cellulose. En revanche, là où c'était non pendant de nombreuses années, aujourd'hui, nous menons des essais... Le problème, c'est que nous n'avons pas une vision claire. On sait que les pailles et les gobelets vont être interdits. Qu'en sera-t-il des bols salades, par exemple ? On l'ignore. »  L'impromptue disparition des sacs plastiques décidée en juillet 2016 par le gouvernement a laissé des traces.  « À l'époque, on était très mal , reconnaît le leader des sacs en papier kraft.  Les fournisseurs de matières premières n'avaient pas la capacité de suivre la demande »,  se souvient Frédéric Leduc. Pour ce spécialiste du papier et du carton, la situation est, finalement, plutôt positive.    Depuis cinq ans, le groupe, qui dispose du plus grand parc français et du deuxième en Europe, investit chaque année entre 800 000 et 1,5 million d'euros dans une nouvelle machine à fabriquer des sacs.  « L'an dernier, on en a même acheté deux » , précise le directeur commercial, marketing et développement de Gault et Frémont, qui finalise une gamme  « engagée »  de sacs compostables à la maison et entend accélérer sur les matières « green » (cellulose, papier cristal, PLA sans OGM).    DES MATIÈRES COMPLEXES À MAÎTRISER Les solutions alternatives arrivent. À l'instar du fabricant Comatec, spécialisé dans l'emballage et le packaging pour la restauration, l'hôtellerie, les traiteurs haut de gamme (Fauchon, Potel et Chabot, hôtels Marriott, Alain Ducasse, Thierry Marx, Servair, des compagnies aériennes...).  « On travaille aussi bien pour les food trucks que pour les étoilés. Notre notoriété nous a imposé très tôt d'innover dans les matières premières, le process et le design. On a commencé il y a douze ans avec le palmier. On a essayé les copeaux de bois, les algues... Il y a sept ans, on a testé les PLA pour les couverts et les verrines, mais les PLA, produits à base de maïs, peuvent contenir des OGM, et ne sont pas tout à fait éthiques... De plus, ces nouvelles matières sont complexes à maîtriser. Elles ne résistent pas forcément à la chaleur, sont sensibles aux variations de température, et financièrement pas très fiables , explique Tatiana Rumeau, PDG de Comatec.  L'opinion publique réclame beaucoup de produits écoresponsables, recyclables, sans trop savoir ce que l'on met derrière. Aussi bien chez la clientèle que chez les utilisateurs, il y a beaucoup de pédagogie à faire. Travailler la fibre de sucre de canne, le bambou ou de palmier, oui, mais on les importe ! L'utilisation de produits biosourcés soulève aussi quelques ambiguïtés. Faire venir des solutions alternatives d'Asie impacte notre bilan carbone » , estime-t-elle. Malgré une volonté de privilégier les circuits courts, les matières premières recyclables d'origine française, avec un fonctionnement à flux tendus pour garantir réactivité et traçabilité, les importations ont doublé. Le plastique recyclé ou recyclable compte pour 50 % des produits du catalogue à côté du PLA, de la bagasse, de la fibre de bambou, de la foliaire de palmier, du carton kraft... Pour réduire encore un peu plus la proportion de plastique, l'entreprise a mis au point, il y a trois ans, le Textura : un matériau à base de polypropylène recyclable et de talc alimentaire suffisamment souple pour créer des bols organiques, des assiettes, des plateaux... Une solution réutilisable jusqu'à dix fois, qui passe au micro-ondes et au lave-vaisselle.    « SAUF SI NOUS, ÊTRES HUMAINS... » « Le marché va significativement vers le naturel. Depuis un an, on s'oriente véritablement vers ces matériaux pour répondre à la demande de nos clients »,  constate Philippe Merlo, PDG de Solia, à Perpignan, l'un des acteurs de l'emballage haut de gamme pour de grands traiteurs parisiens, la restauration commerciale et collective (Sodexo, Picard...) ou l'aviation d'affaires, spécialisé dans la vaisselle à usage unique. Ici, chaque année, 10 % de la gamme migre vers les produits naturels en bambou, en pulpe de canne, bois, carton, PLA... (voir encadré ci-dessous). Entre 70 et 80 % il y a encore quelques années, la proportion de plastique est tombée à 50 %.  « On ne l'a pas complètement abandonné, mais on ne crée plus de produits en plastique. Cela dit, c'est une erreur de dire que c'est un matériau polluant. Ce n'est pas vrai. Le plastique souffre d'une mauvaise image, d'un mauvais accompagnement qui vise clairement à le détruire. Or, c'est un matériau noble qui peut et doit être recyclé. Dans ces conditions, il n'est pas plus polluant que le verre, l'acier ou l'aluminium... Sauf si nous, êtres humains, on le jette n'importe où, n'importe comment sans se préoccuper de ce qui va se passer en aval ! À l'échelle mondiale, le problème concerne davantage les pays émergents, dont la priorité est plutôt de se nourrir que de gérer les déchets » , estime Philippe Merlo, partenaire de l'usine de retraitement de déchets Tiru, fondée par Dalkia (EDF), à Hénin-Beaumont (62), où les déchets sont transformés en électricité ou en vapeur, utilisés pour du chauffage urbain et des usages industriels.  « Les fumées sont captées pour éviter les pollutions en aval. Et la boucle est bouclée. C'est le type de démarche qu'il faut encourager pour favoriser le recyclage » , observe le patron de Solia.    Car, pour lui,  « le plastique a des qualités que l'on ne trouve pas ailleurs. Le naturel, c'est sympathique, mais pour la cuisine, la conservation ou le transport, on a du mal à avoir les mêmes propriétés mécaniques qu'avec le plastique, qui peut être recyclé à l'infini. Il est stable dans le temps et peut être utilisé sans risque alimentaire pendant plusieurs mois. Ce qui n'est pas possible avec les produits naturels, dont les DLC n'atteindront jamais le niveau du plastique. Le PLA, par exemple, ne supporte pas les hausses de température et le froid. Nos clients sont sous la pression de la clientèle qui demande des produits naturels, recyclables, biodégradables, compostables, mais personne ne sait vraiment ce que c'est... » , déplore-t-il. Aller vers des produits en PLA, à base d'amidon de maïs issu de cultures intensives dont le bilan écologique peut être discutable, c'est aussi multiplier le prix par trois.  « Sur le sujet des pailles, c'est aussi très compliqué. Quel substitut adopter ? En carton, la structure ne tiendra pas dix minutes dans un verre et choisir un carton haut de gamme, c'est dix fois plus cher » , souligne-t-il.    UNE FILIÈRE ORGANISÉE POUR LE CARTON Ceux qui se frottent les mains, ce sont justement les industriels du carton.  « On a le vent en poupe » , confirme Jean-Pascal Salaün, président du groupe Lebhar, spécialisé dans l'emballage en carton. Signe du phénomène de bascule grandissant vers les produits de substitution, lui a vu son chiffre d'affaires progresser d'un million d'euros par an depuis dix ans.  « C'est colossal !,  ajoute-t-il.  Ça a commencé avec l'interdiction des sacs plastiques il y a deux ans. Ça va s'accélérer en 2019-2020 Mais c'est difficile à chiffrer. Techniquement, pour nous, ce sont simplement des process que l'on duplique à travers des filières que l'on maîtrise. Ça ne se fait pas tout seul, mais le terreau est fertile. Nous avons remplacé le plastique des fenêtres par du cellophane. Il faut être un peu malin pour trouver le sourcing adapté, mais finalement, ce n'est pas si compliqué pour un transformateur et ça fonctionne ! »  Pour lui, la profession du carton peut répondre à la demande.  « Elle n'allait pas entreprendre un revirement complet. Importer des matières naturelles coûte plus cher que le carton. Des barquettes de fromage en bambou, je ne vois pas l'intérêt. En tant que cartonnier, on préfère transformer... le carton. Il y a plus d'obstacles avec des solutions végétales recyclables » , estime-t-il. Surtout, la filière est organisée.  « Pour chaque arbre coupé, un autre est replanté. Les papeteries savent que si elles ne le font pas, il n'y aura plus de pâte à papier, et donc plus de carton, plus d'emballages, plus d'économie et plus d'emplois. La forêt des Landes est en croissance régulière, les pays scandinaves gèrent parfaitement leurs forêts... Grâce à des arbres à croissance rapide, l'Europe a la capacité de satisfaire les besoins. Il vaut donc mieux développer le carton, avec des emplois ici, que d'avoir recours au plastique, issu du pétrole, dont l'activité fait travailler des gens hors d'Europe » , dit-il.   DES FREINS À LEVER  Le secteur de la restauration rapide et du snacking compte pour 10 % de l'activité du groupe Lebhar, appuyé sur six usines de fabrication en France.  « Au-delà des chaînes, on accompagne nos clients boulangers qui se diversifient vers le snacking et la restauration rapide. On leur fournit des emballages pour les salades, les plats chauds, gratinés. C'est un vrai levier de croissance. »  Sur les 2,2 millions de tonnes d'emballages plastiques mis sur le marché en 2018, seuls 12 % sont d'origine recyclée et 2 % biosourcés, contre 86 % de matières premières dites élégamment « vierges ». En 2019, les professionnels estiment que 100 000 tonnes de plastique recyclé seront volontairement incorporées dans les emballages et 188 000 tonnes supplémentaires d'ici à 2025. Si des fabricants amorcent une internalisation des activités de recyclage, de nombreux freins restent à lever... À tous les niveaux.
</article>
<article titre="Énergie Déni d'autonomie à La Réunion" date="2019 02 28" journal="L'Usine Nouvelle, no. 3600">
Plus facile à voter qu'à appliquer. La loi de transition énergétique pour la croissance verte impose aux Outre-mer d'atteindre l'autonomie énergétique en 2030, avec un palier à 50 % en 2020. Comme en métropole, chaque département et région d'outre-mer (Drom) a sa propre feuille de route à cinq et dix ans, sa programmation pluriannuelle de l'énergie (PPE). La Réunion ne fait pas exception. Sauf que, localement, l'autonomie énergétique, personne n'y croit. « Notre ambition, c'est l'autonomie électrique en 2030. Pour l'énergie, c'est compliqué à cause de la part des transports et des infrastructures », explique Alin Guezello, le président de SPL Énergies Réunion, une structure de maîtrise de la consommation créée en 2010. La voiture individuelle reste un marqueur social fort sur l'île. D'autant plus que « les transports collectifs stagnent à 5 % depuis vingt ans, 7 % si l'on compte le transport scolaire », explique Jean-Michel Maurin, le directeur de la Direction de l'aménagement, de l'environnement et du logement (Deal) de l'État. Sans viser l'autonomie, la PPE en préparation, qui doit être adoptée d'ici à la fin 2019, fixera une diminution de 23 % de la consommation d'énergies fossiles d'ici à 2028. Pour y parvenir, la Région veut faire passer la part du transport en commun à 11 % en 2023 et 14 % en 2028. « Nous avons établi un schéma directeur à 2030, 2040 et 2050, qui porte sur le collectif, le covoiturage et le vélo », explique la conseillère régionale (majorité Réunion-nous) aux transports, Fabienne Couapel-Sauret. Pour l'heure, toute l'énergie est mise sur Run Rail, un projet de transport collectif guidé de 150 km à terme entre Saint-Benoît et Saint-Joseph, par l'ouest [lire l'encadré page 64]. Quant au véhicule électrique, seule solution pour atteindre un jour l'autonomie énergétique, rien ou presque n'est fait pour le favoriser. La Région a toutefois décidé de supprimer la taxe en vigueur dans les territoires d'outre-mer sur les importations de voitures électriques.   L'autonomie électrique est un objectif plus réaliste. « La Région est déjà à 36 % d'énergies renouvelables et on sera à plus de 50 % en 2022 ou 2023 », se félicite Olivier Duhagon, le directeur d'EDF SEI Réunion. « En misant fortement sur le développement du photovoltaïque, nous devrions même parvenir à 81 % d'électricité d'origine renouvelable en 2028 », avance Alin Guezello. Un cadastre solaire va être établi. La PPE fixe 350 mégawatts (MW) de photovoltaïque installés en 2023 et 540 en 2028 (contre 190 en 2017). Des objectifs considérés comme très ambitieux, mais insuffisants. Une étude de l'Agence de l'environnement et de la maîtrise de l'énergie (Ademe), en cours de publication, estime qu'il faudrait 713 MW de photovoltaïque en 2030 pour parvenir à l'autonomie électrique et 1 200 pour atteindre l'autonomie énergétique, reposant sur une conversion du transport à l'électrique.   Difficile sortie du charbon   Même avec une politique volontariste, non seulement la PPE ne permettra pas d'atteindre 100 % d'électricité renouvelable sur l'île, mais elle abandonnerait aussi l'idée d'autonomie électrique. Pour sortir du charbon (28 % de la production électrique) d'ici à 2023, comme le prévoit la loi, Albioma, le principal producteur d'électricité sur l'île (54 %), prévoit de convertir ses deux centrales électriques hybrides bagasse-charbon de Bois Rouge (100 MW) et Le Gol (111 MW) à un mix bagasse-biomasse. Or, selon les calculs de l'Office national des forêts local, sur les 600 000 tonnes de bois déchiquetés ou les 400 000 tonnes de pellets dont Albioma aurait besoin pour remplacer le charbon, l'île ne peut en couvrir que 30 %. Albioma va donc devoir importer des pellets de bois d'Amérique du Nord ou du Brésil en attendant l'hypothétique développement d'une filière biomasse durable dans l'océan Indien, notamment en Afrique du Sud.   Il y aurait pourtant une autre solution. Convertir une partie de la culture de canne à sucre en canne à fibre. « L'enjeu est la valorisation de 5 000 hectares de friches sur lesquels on pourrait cultiver 1,55 million de tonnes de cultures intermédiaires à vocation énergétique », observe Dominique Vienne, le président du Conseil économique, social et environnemental régional. Le sujet est ultrasensible. Socle de l'économie de l'île, « la canne à sucre reste massivement subventionnée », rappelle Éric Jeuffrault, le directeur général du Centre de coopération internationale en recherche agronomique pour le développement Réunion-Mayotte. « Il faut que la demande vienne des agriculteurs. Aujourd'hui, ce sont des jeunes qui posent le problème sur la table. Nous travaillons avec eux pour envisager l'avenir », explique Alin Guezello.   Le dossier avance lentement, d'autant que la Région ne peut pas s'en occuper, la compétence sur l'agriculture étant dans les mains du Département. Albioma, qui vient d'investir dans une centrale électrique à base de bioéthanol de canne à sucre pour valoriser les surplus de mélasse d'une raffinerie, n'est pas pressé non plus. Or c'est un peu lui qui fait la loi outre-mer. « À La Réunion, le mix énergétique est écrit par EDF et Albioma », observe Dominique Vienne. Faux, expliquent Alin Guezello et Jean-Michel Maurin. Selon eux, La Réunion serait même le seul territoire d'outre-mer à avoir mis en place une gouvernance énergétique transverse rassemblant tous les acteurs : État, Région, Département, industriels, Ademe, Syndicat des énergies renouvelables...   Il n'est reste pas moins que la PPE est principalement élaborée à partir des projets des industriels et non sur la base des ressources mobilisables pour atteindre l'objectif d'autonomie. Une méthode certes pragmatique, mais qui laisse de côté les initiatives innovantes isolées. Comme celle de cet agriculteur qui a monté une unité de pyrogazéification sur son exploitation pour produire du biogaz, mais bute sur EDF à propos d'un tarif de rachat d'électricité. Qu'importe. « Le véritable sujet, ce n'est pas l'énergie, mais les déchets, explique Olivier Duhagon. La Région veut zéro déchet enfoui [alors qu'ils le sont à 100 %], mais la compétence relève du Département. » Et c'est le projet de tri des déchets, avec méthanisation et une unité de valorisation énergétique (autrement dit un incinérateur), à Pierrefonds, qui fait le plus polémique.
</article>
<article titre="AccueilStratégie achats Révue de presse - semaine du 18 au 22 février" date="2019 02 22" journal="Décision Achats (site web)">
Vous n'avez pas eu le temps de surfer cette semaine? La rédaction a sélectionné, pour vous, des articles parus cette semaine sur le Web et susceptibles de vous intéresser car ayant trait à la relation fournisseurs, à la RSE, à la commande publique, etc. Bonne lecture ! li>    Devoir de vigilance : le cas de Total, attaqué sur le changement climatique, fera-t-il jurisprudence ?   Depuis 2018, Total, comme toutes les grandes entreprises françaises, doit publier un plan de vigilance qui met en avant les risques que peuvent poser ses activités en matière de droits de l'Homme et d'atteintes à l'environnement. Mais suite à son premier exercice, le pétrolier se faisait interpeller en octobre par un collectif d'associations et d'élus pour ne pas avoir intégré le changement climatique à son document. Celui-ci menace d'une action en justice pour contraindre la multinationale. Dans une lettre datée de janvier, Total explique que son Conseil d'administration a "convenu que le plan de vigilance qui sera contenu dans le rapport de gestion publié en 2019 traitera des risques liés au changement climatique" pour ses activités. En 2018 en effet, si le pétrolier avait bien pris en compte ce risque dans son rapport de gestion annuel, dans sa déclaration de performance extra-financière ainsi que dans son reporting TCFD (sur les risques financiers liés au climat), il ne l'avait pas mentionné dans son plan de vigilance. Il considère en effet que les obligations du devoir de vigilance ne concernent pas le changement climatique. Dans sa lettre au collectif, Aurélien Hamelle, directeur juridique du pétrolier, explique que ces risques sont d'une "nature différente" que ceux du droit de l'environnement et des droits de l'Homme couverts par la loi. Or, pour Marta Torre Schaub, directrice de recherche au CNRS et directrice du Réseau droit et changement climatique, raisonner ainsi est une "erreur". L'environnement est considéré de manière holistique et le droit à un environnement sain est progressivement reconnu comme un droit humain fondamental, souligne-t-elle.>> Lire l'article complet sur le site de Novethic Baromètre au beau fixe pour la plasturgie française    Le "plastique bashing" dénoncé par les industriels ne fait pas sentir ses effets pour le moment : la consommation augmente toujours. Mais les clients des plasturgistes "réfléchissent de plus en plus" pour répondre aux critiques, assure Jean Martin, en travaillant sur des alternatives ou en développant le recyclage. Le secteur s'est engagé à doubler la part du plastique recyclé dans la production d'ici à 2025 . (...) Ces dernières années ont été marquées par un regain de compétitivité. La productivité par salarié a augmenté de plus de 2 % par an en moyenne entre 2000 et 2016, contre 1,1 % en Allemagne et 0,2 % en Italie, les deux pays leaders du secteur en Europe. Résultat, la valeur ajoutée par salarié dépasse maintenant celle des autres grands pays producteurs d'Europe, selon la Fédération. Et en 2018, la croissance du chiffre d'affaires a été supérieure à celle de la plasturgie allemande, pour la première fois depuis plus d'une décennie. Cette amélioration s'est faite au prix d'une réduction massive des effectifs : le secteur n'emploie plus que 122.000 personnes, contre 170.000 en 2002. Une partie de ce recul s'explique par l'externalisation de certaines tâches, une autre par les nombreuses fusions qui ont animé le secteur et qui ont engendré des efforts de rationalisation. >> Lire l'article complet sur le site des Echos   Le Brexit complique l'approvisionnement de la France en gaz    Une dizaine de ces sociétés ont basé leur bureau de trading au Royaume-Uni en raison d'un régime fiscal "bien plus favorable", écrivent Thierry Bros et Patrick Heather, chercheurs à l'Oxford Institute, y compris les plus gros acteurs du marché comme Total, BP, EDF ou Gazprom. "A moins qu'un accord spécifique sur le sujet ne soit trouvé avant le retrait du Royaume-Uni de l'Union européenne, tous les fournisseurs du réseau de gaz français basés au Royaume-Uni perdront automatiquement leur autorisation le 30 mars", poursuivent-ils. Dans les faits, aucune rupture d'approvisionnement n'est en vue cependant, car plusieurs de ces sociétés ont fait les opérations nécessaires. "Le risque d'une sortie du Royaume-Uni de l'UE sans accord est identifié depuis plusieurs mois", rassure-t-on au ministère de la Transition écologique. >> Lire l'article complet sur le site des Echos   Coop renonce aux produits du groupe Mars   Le numéro deux helvétique du commerce de détail et ses partenaires de l'alliance de détaillants Agecore visent le groupe américain Mars. Ces derniers ont cessé de lui commander ses produits depuis plusieurs semaines. Ainsi, comme le rapporte mardi le journal alémanique "Blick" sur son site internet, plusieurs marques détenues par le géant américain, comme les aliments pour chats Whiskas, le riz Uncle Ben's, ou encore les produits à base de chocolat Twix, Bounty et M&amp;M ont disparu des rayons de Coop. Contacté par AWP, le distributeur bâlois n'a pas souhaité prendre officiellement position sur l'affaire, tout comme Mars. Selon le porte-parole de Coop, Urs Meier, cité dans "Blick", la disparition de certains produits "est liée à des négociations que nous ne souhaitons pas commenter plus en détail". Mars a pour sa part indiqué à AWP tenir en haute estime ses relations d'affaires avec Agecore et Coop, le groupe américain souhaitant en poursuivre le développement avec ses partenaires. >> Lire l'article complet sur la Tribune de Genève   La clause Molière, de nouveau au contentieux devant le Conseil d'État   Un préfet demandait la suspension d'un marché au motif que les documents de la consultation comportaient des dispositions imposant l'usage du français portant atteinte aux principes de liberté d'accès à la commande publique, d'égalité de traitement des candidats et d'interdiction de discrimination en raison de la nationalité. La Cour administrative d'appel avait suspendu l'exécution du marché au motif qu'un article du règlement de la consultation qui disposait que "la langue de travail pour les opérations préalables à l'attribution du marché et pour son exécution est le français exclusivement" était de nature de violer les libertés fondamentales garanties par le Traité sur le fonctionnement de l'Union européenne. Cependant, le Conseil d'État revient sur cette position au motif que les dispositions du règlement de la consultation, lequel n'a pas de valeur contractuelle, "régissent seulement les relations entre les parties au contrat et n'imposent pas le principe de l'usage de la langue française par les personnels". >> Lire l'article complet sur le site de Weka   Le palladium, ce métal précieux le plus cher du monde   Cette fois, le record est battu. Le cours du palladium a franchi pour la première fois de son histoire les 1 500 dollars l'once, mercredi 20 février à la Bourse de Londres. Il vaut désormais plus cher que l'or, dont l'once se négocie autour de 1 343 dollars. En l'espace d'un an, le cours du palladium, qui est sans doute le moins connu des métaux précieux, a grimpé de plus de 40 % et de près de 200 % en trois ans, par crainte d'un déficit de l'offre et d'une hausse de la demande. Le palladium, ce métal gris qui fait partie de la famille des platines, sert notamment à la fabrication des pots catalytique des voitures à essence et hybrides pour réduire les émissions polluantes. Le marché est en forte progression depuis que le "scandale Volkswagen" a éclaté en 2015, avec la découverte des tests truqués de ses moteurs diesel. Et plus les normes environnementales sont strictes, plus on a besoin de palladium pour filtrer les particules dans les pots d'échappement, affirment les spécialistes. Une manne pour ce métal, alors que l'industrie automobile représente 80 % de la demande. D'autant que les autres débouchés, comme l'électronique (pour produire des connecteurs) ou la joaillerie, sont également en croissance. >> Lire l'article complet sur le site de La Croix   RSE   A l'île Maurice, la canne à sucre pour faire de l'électricité   Depuis une quinzaine d'années, la bagasse, le résidu fibreux issu du broyage de la canne à sucre, est largement mise à contribution pour produire de l'électricité, au point désormais de pourvoir à 14% des besoins de l'île. L'île Maurice dispose d'un système tout particulier pour sa production d'énergie : 60% des besoins en électricité de l'île sont produits par quatre sociétés sucrières qui font tourner chacune une centrale thermique. Ces quatre centrales tournent normalement au charbon mais quand la saison de la canne arrive, la bagasse remplace le charbon comme combustible. >> Lire l'article complet sur le site de Sciences et Avenir   Yvelines : vos eaux usées vont produire de l'énergie verte   C'est une course inattendue à laquelle se livrent les villes des Mureaux et de Versailles. Toutes deux s'apprêtent à accueillir une unité de méthanisation au sein de leur usine d'épuration respective. Si la production d'énergie verte à partir du traitement des eaux usées n'est pas nouvelle en France, ce procédé est pour le moins innovant à l'échelle du département puisque pour la première fois, du biogaz sera produit par une station d'épuration et ainsi réintroduit dans le réseau général. La compétition est donc féroce. (...) Le biogaz issu du processus de digestion des boues sera récupéré, transformé en biométhane et injecté dans le réseau public de gaz naturel. "GRDF rachète à la collectivité locale la molécule de gaz à un prix fixe", précise Stéphane Cordier. Ce procédé représente non seulement un intérêt environnemental, véritable outil de transition énergétique au niveau du territoire, mais il peut s'avérer un outil de recette fiscale pour la commune.>> Lire l'article complet sur le site du Parisien   Supply chain   Log's déploie les drones Eyesee de Hardis pour ses inventaires en entrepôt   Log's prévoit de déployer des drones sur l'ensemble de ses plateformes, à travers toute la France et dans le monde. Une démarche qui s'intègre à la politique tournée vers l'innovation de Log's avec la réalisation d'une veille constante sur le sujet, des partenariats de plus en plus rapprochés avec les fournisseurs et intégrateurs, la mise en test et le développement par ses équipes de nouvelles solutions technologiques telles que les systèmes goods-to-man, l'exosquelette ou encore le pick by vision dans l'objectif de mieux répondre aux nouvelles contraintes de ses clients, et d'inventer la logistique de demain. >> Lire l'article complet sur VoxLog   Quand les transporteurs de fret exposent leur stratégie de développement durable   "Le port de Marseille-Fos dispose de plusieurs leviers d'action : la planification stratégique, l'aménagement et la gestion des espaces, la capacité à sélectionner les usines qui s'installent et les tarifs", explique Jean-Michel Bocognano, responsable de l'activité développement durable au GPMM. Ainsi, les compagnies maritimes vertueuses bénéficient de ristournes sur les droits de port. Pionnière en la matière, la Méridionale a, dès 2013, été lauréate de la Charte bleue avant de déployer trois ans plus tard le premier système permettant de connecter les navires en escale au courant électrique. Ce premier investissement de 3,5 millions d'euros pour connecter les trois cargo-mixtes a fait des émules. Les ferries de Corsica Linea et de La Cotunav sur la Tunisie seront connectés à l'électricité de quai dans quelques mois. L'engagement de la Méridionale s'est poursuivi par une multitude d'innovations (optimisation de la propulsion, traitement des eaux de ballast, modification des pales d'hélices et des safrans, système intelligent de gestion des consommations...). >> Lire l'article complet sur le site de Actu Transport Logistique   Commande publique   Quelles sont les conséquences de l'omission de l'examen de l'offre d'un candidat ?   L'absence d'examen d'une offre d'un candidat constitue un manquement aux règles de publicité et de mises en concurrence justifiant, pour un motif d'intérêt général, la résiliation des marchés en cause. Le manquement ainsi commis a eu, compte tenu de sa gravité, une incidence déterminante sur l'attribution des marchés, de sorte que le lien entre la faute de l'administration et le manque à gagner dont la société requérante entend sur ce terrain obtenir la réparation ne peut être regardé comme direct.>> Lire l'article complet sur le site de Weka   L'Assemblée Nationale se tourne vers une start-up française pour l'échange de fichiers entre parlementaires   C'est une petite victoire pour la souveraineté européenne. De manière discrète, l'Assemblée Nationale a passé un appel d'offres pour proposer aux parlementaires français de s'équiper d'une nouvelle solution pour encadrer les échanges de fichiers. Et c'est Wimi, une start-up tricolore issue du start-up studio Adventures qui l'a remporté. "L'Assemblée Nationale a choisi l'une de nos solutions qui consiste à partager des documents avec un drive on et off line", détaille Antoine Duboscq, fondateur d'Adventures et Wimi. Cette solution, pour le moment optionnelle pour les députés et les attachés parlementaires, vient en remplacement des outils utilisés jusque-là, comme Google Drive ou Dropbox, devenus sensibles depuis l'entrée en vigueur du Patriot Act aux Etats-Unis. Celui-ci autorise la justice américaine à se saisir des données hébergées chez ces entreprises, les rendant potentiellement vulnérables. Les datas stockées par Wimi le seront en France, sur les serveurs de Scaleway. >> Lire l'article complet sur le site des Echos   La région Ile-de-France veut attirer les start-up de l'énergie    SoundEnergy (Pays-Bas) est une entreprise qui développe des moteurs à chaleur et des pompes à chaleur conçus pour refroidir et chauffer les bâtiments sans utiliser d'électricité ou de gaz. De son côté, l'italien Enerbrain déploie des solutions pour mieux contrôler le chauffage, la climatisation et la ventilation dans les bâtiments, assurant plus de 20% d'économies d'énergie. "Le but est que les grands groupes puissent identifier, à travers ces échanges, des solutions innovantes et qu'ils aient envie de les exploiter, souligne-t-on à l'agence. En sens inverse, des jeunes entreprises étrangères en phase de croissance sont susceptibles de trouver des opportunités pour les aider à poursuivre leur expansion en Ile-de-France." >> Lire l'article complet sur le site du Figaro   La Ville de Paris choisit le lait bio pour toutes ses crèches   Fin janvier 2019, la mairie a renouvelé son marché de laits infantiles pour ses structures d'accueil de jeunes enfants et a décidé de passer au 100% bio. Ainsi, dès le mois de mars, les 2 300 litres de laits servis chaque jour dans les crèches parisiennes seront exclusivement bio. Jusqu'à présent, seul 1% du lait donné aux plus petits était labellisé bio. C'est la société Nestlé, qui approvisionnait déjà les crèches, qui a remporté la consultation pour une durée de quatre ans renouvelable chaque année. "Cette avancée exceptionnelle sur la voie de l'alimentation biologique et durable permet à la capitale de devenir la seule grande collectivité de France à fournir 100 % de lait bio dans les crèches", se réjouit la Mairie de Paris dans un communiqué. >> Lire l'article complet sur le site de Bioaddict   Environnement de travail   Qui sont les "transféreurs", ces praticiens de l'écologie au travail ?   Une étude sociologique a permis de mettre en lumière ce mécanisme social dans le domaine de la transition écologique : le transfert de pratiques environnementales du domicile au travail. Ces praticiens de l'écologie sont appelés des "transféreurs" : soit des individus qui mettent en oeuvre sur leur lieu de travail des pratiques environnementales dont ils sont familiers à leur domicile. On retrouve bien sûr dans cette catégorie des militants écolos qui cherchent à convaincre leurs collègues. Il faut cependant noter que c'est la recherche d'un mieux-être au travail qui reste la première raison d'agir chez la plupart de ces transféreurs. Il s'agit pour eux de réduire le décalage vécu entre leur vision d'un monde ("qui ne peut pas continuer comme cela") et leurs pratiques au travail ("qui font comme si de rien n'était"). Le transfert est en quelque sorte une réponse à une dissonance cognitive de plus en plus difficile à supporter au quotidien. Par ailleurs, on trouve chez ces transféreurs le souhait de favoriser des relations professionnelles plus conviviales, dont les pratiques environnementales peuvent être un vecteur. >> Lire l'article complet sur le site de The Conversation   Fleet / Mobilité   Téléphone au volant, attention au retrait de permis en cas d'infraction dès 2019   Désormais, l'usage du téléphone portable en conduisant sera plus sévèrement réprimandé. Le gouvernement est parti du constat que l'usage du téléphone au volant est la cause d'un accident corporel sur dix. Par ailleurs, le fait de consulter ou d'écrire un mail ou un SMS multiplie par 23 le risque de provoquer un accident dont les conséquences peuvent être dramatiques. C'est pour cette raison que le Comité Interministériel de Sécurité Routière (CISR) a établi une règle qui entrera en vigueur en 2019. Cette règle est la suivante : les forces de l'ordre pourront suspendre le permis de conduire d'un automobiliste qui est pris en flagrant délit d'utilisation de son téléphone portable s'il a commis en même temps une infraction menaçant la sécurité d'autrui. Cela ne veut pas dire que si vous utilisez votre portable au volant, en ne menaçant personne, vous ne serez pas sanctionné. Dans ce cas précis, les dispositions actuelles continueront de s'appliquer. Aussi les forces de l'ordre pourront vous infliger une amende forfaitaire de 135 euros ainsi que vous retirer trois points sur votre permis de conduire. >> Lire l'article complet sur le site de Turbo   Automobile : l'irrésistible montée du superéthanol E85 en France   Nom : E85. Surnom : superéthanol. Signe particulier : carburant automobile d'origine agricole en plein essor en France. Avec plus de 21 millions de litres vendus en janvier, cette "essence" constituée entre 70 % et 85 % d'alcool issu de betteraves sucrières ou de céréales vient de battre son record de ventes dans l'Hexagone. Ce tout premier chiffre de 2019 confirme la tendance observée l'an dernier : le superéthanol E85 suscite l'engouement. Sa consommation a augmenté de 55 % en France en 2018. Alors que le passage à la pompe coûte entre 1,42 et 1,50 euro le litre, en moyenne, aujourd'hui, de plus en plus de Français sont tentés par un plein moins cher de 50 % (0,68 euro le litre, en décembre 2018). Cela permet, par exemple, d'économiser une trentaine d'euros sur un Paris-Marseille. >> Lire l'article complet sur le site du Monde   La voiture autonome ? Une catastrophe écologique   La généralisation de ces véhicules impliquerait, pour abaisser les coûts de façon acceptable, le lancement d'une production de masse de tous ces objets, en supplément de l'électronique déjà présente dans les véhicules actuels. Aux États-Unis, la National Mining Association rappelle régulièrement aux décideurs que "la sophistication croissante des produits de l'industrie automobile va faire exploser la demande en métaux et la compétition pour ces ressources". En clair, impossible d'obtenir la matière première de tous ces superalliages sans provoquer une augmentation de l'activité minière, qui compte parmi les industries les plus polluantes. Deuxième problème : la croissance exponentielle des données (le data). La conduite automatisée repose sur de gigantesques volumes d'informations que les divers capteurs transmettent aux algorithmes de traitement d'images. Selon Brian Krzanich, p-dg d'Intel, un véhicule autonome va générer et consommer, pour huit heures de conduite, environ 40 téraoctets de données, soit l'équivalent de 40 disques durs d'ordinateur. >> Lire l'article complet sur le site de Reporterre   L'Union européenne limite les émissions de CO2 des camions   L'Union européenne (UE) va pour la première fois limiter les émissions de CO2 des camions, avec pour objectif de les réduire de 30 % d'ici à 2030 selon un accord trouvé, mardi 19 février, entre les négociateurs du Parlement européen et du Conseil. L'Association européenne des constructeurs automobiles (ACEA) a immédiatement fait part de ses "inquiétudes" face à des objectifs "très ambitieux" qui, selon elle, ignorent le manque d'infrastructures nécessaires pour assurer le développement de technologies alternatives au moteur à combustion. Selon les termes de cet accord, qui doit encore être validé par le Parlement réuni en session plénière et les 28 Etats membres, les poids lourds devront réduire leurs émissions de 15 % d'ici à 2025 et de 30 % d'ici à 2030 par rapport aux niveaux de 2019, rapporte la présidente de la commission transport du Parlement, Karima Delli, dans un communiqué. >> Lire l'article complet sur le site du Monde   La ville de Madrid passe le cap des 300 véhicules électriques   La ville de Madrid montre la voie aux collectivités locales qui ont pour projet d'électrifier leur parc automobile. La dernière opération menée par l'équipe municipale de la capitale espagnole témoigne d'une réelle ambition dans ce domaine. En janvier, 142 véhicules électriques, essentiellement des Renault Zoe et des Nissan Leaf, ainsi que 41 véhicules hybrides ont été incorporés à sa flotte. Un arrivage massif qui a fait l'objet d'une cérémonie en présence de nombreux élus de la ville. Le département Environnement et Mobilité a été le mieux pourvu avec 75 véhicules électriques (74 Zoe et un Kangoo Z.E.) et 27 Toyota Yaris hybrides. Ces 102 nouveaux véhicules font l'objet de contrats de location d'une durée de quatre ans, avec une possibilité de prolongation d'un an pour un montant de près de 2,2 millions d'euros. Les contrat comprennent la maintenance, le changement de pneus, l'assurance tous risques et le véhicule de remplacement. Ces nouveaux véhicules viennent s'ajouter aux 50 véhicules propres déjà intégrés en 2018. >> Lire l'article complet sur le site du Journal de l'Automobile   Travel   Frais ancillaires : 65 milliards de dollars pour les compagnies aériennes   Avant même la généralisation de la nouvelle norme de distribution NDC, qui doit faciliter la commercialisation des frais ancillaires, la manne financière générée par ces revenus à la carte représente la coquette somme de 65 milliards de dollars (57,6 milliards d'euros). C'est le résultat du rapport 2018 publié le 19 février par CarTrawler et IdeaWorksCompany, et basé sur les résultats de 175 transporteurs à l'échelle mondiale. Parmi eux, le contingent de compagnies européennes s'est avéré particulièrement efficace. Frais d'enregistrement bagages, attribution du siège, restauration à bord, enregistrement prioritaire, systèmes de divertissements en vol : en commercialisant ces "petits plus", les transporteurs du Vieux Continent ont généré pas moins de 22,5 milliards de dollars (19,9 milliards d'euros), relayant les transporteurs asiatiques - pourtant le deuxième marché le plus gourmand en termes de services à la carte - à 4 milliards de dollars. >> Lire l'article complet sur le site de Voyages d'affaires   IT   L'IA au service de l'industrie textile et de la mode   On compte parmi les autres innovations récentes les plus marquantes des robots-couturiers mis au point par une startup nord-américaine fabriquant des vêtements sans aucune intervention humaine. La confection de prêt-à-porter était jusque-là considérée comme impossible à automatiser en raison de la matière souple du tissu, compliquée à manipuler par des machines. Aujourd'hui, le tissu est momentanément transformé en un polymère rigide qui peut ainsi être travaillé puis replongé dans l'eau chaude pour revenir à ses propriétés initiales. Parallèlement, la marque Levi's a lancé une veste connectée au smartphone permettant de contrôler certaines applications, et la startup Omius Tech a créé un manteau ajustant automatiquement les températures en cas de changement météorologique. Cette dernière invention a été conçue en collaboration avec de célèbres cyclistes, coureurs, skieurs et athlètes. Le studio Fuseproject a, quant à lui, mis au point des vêtements adaptés Aura Powered Cothing, qui aident les personnes seniors à se déplacer, en y intégrant des capteurs corporels qui réagissent aux mouvements naturels du corps et ajoutent de la puissance musculaire à chaque geste. >> Lire l'article complet sur le site de La Tribune   Carrière / RH / Management... et autres   L'orthographe, un critère de sélection pour un DRH sur deux....   Les lacunes en orthographe sont des freins impitoyables à la recherche d'un emploi mais aussi à la progression de carrière. C'est ce qui ressort d'un sondage OpinionWay pour Mon coach Bescherelle - l'application du roi de la grammaire - que nous publions en exclusivité et qui sera présenté mardi à Paris à des responsables de ressources humaines lors d'un colloque. Selon cette "grande consultation sur l'orthographe et l'employabilité" menée auprès de plus de 200 DRH, la moitié d'entre eux (52 %) affirment que le niveau en expression écrite a pu jouer dans la mise à l'écart d'une candidature. Ils sont 44 % à reconnaître qu'un faible niveau d'un collaborateur a pu "susciter des remarques" et 15 % que de telles failles ont pu "freiner une promotion". >> Lire l'article complet sur le site du Parisien   La démission, le tabou français devenu tendance chez les cadres   Autour de la démission, on constate déjà un grand paradoxe: si plus de la moitié des cadres (53%) se disent ouverts à de nouvelles opportunités, ils ne sont "que" 8% à franchir le pas. Pourquoi? Parce que l'être humain, par essence, est hostile au changement et à des situations qu'il ne connaît pas... Le cadre resterait donc en poste par frilosité et par peur de l'inconnu? C'est la tendance que montre cette étude, qui entend "dresser les contours de la démission aujourd'hui", selon Frédéric Dabi, DGA de l'Ifop, qui ajoute qu'aujourd'hui "les candidats potentiels à la démission sont très nombreux": six cadres sur dix envisagent ainsi de démissionner, une proportion qui monte à... 74% chez les 18-34 ans! "Il n'y a pas de tabou chez les moins de 35 ans, on constate un réel clivage générationnel", ajoute Frédéric Dabi. Pourquoi sauter le pas? Presque quatre cadres sur dix citent des perspectives professionnelles qui ne sont pas à la hauteur, un argument suivi de près par... la rémunération. >> Lire l'article complet sur Le Figaro  Cet article est paru dans Décision Achats (site web)
</article>
<article titre="Tables de passage des classifications des produits françaises (CPF) | Insee - Table de passage CPF rev.1 vers CPF rev.2 - table_CPA02-CPA08.pdf" date="2019 02 13" journal="Rapports - Marketing - INSEE">
139 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... Correspondence table CPA 2002 - CPA 2008  CPA 2002 CPA 2008 01.11.11 01.11.11Durum wheat Durum wheat  01.11.11 01.64.10Durum wheat Seed processing services for propagation...    Pour voir le rapport :  https://www.insee.fr/fr/statistiques/fichier/2579545/table_CPA02-CPA08.pdf
</article>
<article titre="Tables de passage des classifications des produits françaises (CPF) | Insee - Table de passage CPF rev.2 vers CPF rev.1 - table_CPA08-CPA02.pdf" date="2019 02 13" journal="Rapports - Marketing - INSEE">
179 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... Correspondence CPA 2008 - CPA 2002  CPA 2008 CPA 2002  01.11.11 Durum wheat01.11.11 Durum wheat  01.11.12 Soft wheat and meslin01.11.12 Wheat, except durum wheat...    Pour voir le rapport :  https://www.insee.fr/fr/statistiques/fichier/2579545/table_CPA08-CPA02.pdf
</article>
<article titre="Renouvelables Albioma électrise le bioéthanol à la Réunion" date="2019 02 7" journal="L'Usine Nouvelle, no. 3597">
C'est une première mondiale qu'Albioma vient de réaliser à La Réunion. Le spécialiste ultramarin de la valorisation énergétique de la bagasse (le résidu de la canne à sucre) vient de trouver un débouché pour le surplus de mélasse de la distillerie réunionnaise Rivière du Mât. Il va l'injecter, après transformation en bioéthanol à 95,5 °C, dans une turbine LM6000 de General Electric - comme celles des Boeing 747 - pour produire de l'électricité. Réglée par GE à la demande d'Albioma, cette turbine produite aux États-Unis consomme 80 % de bioéthanol et 20 % de gazole, un carburant fossile indispensable à son démarrage rapide. La centrale pourrait valoriser 3 000 à 5 000 m3 du bioéthanol produit par la distillerie, qui a investi pour adapter ses équipements. Avant ce projet, ses surplus de mélasse étaient envoyés en métropole pour transformation. La centrale, pilotable à distance par EDF Solutions énergétiques insulaires, servira en cas de pic de consommation. §
</article>
<article titre="Transition énergétique réunionnaise: mission impossible?" date="2019 02 4" journal="Journal de l'environnement (site web)">
L'île de la Réunion prépare sa programmation pluriannuelle de l'énergie (PPE) pour 2028. Un exercice qui s'avère plus délicat sur le terrain que sur le papier. reportage.  La Direction générale de l'énergie et du climat n'est pas seule à phosphorer sur la prochaine PPE. A plus de 9.000 kilomètres du quartier parisien de La Défense, les services de la Direction de l'environnement, de l'aménagement et du logement de La Réunion (Deal) mettent la dernière main à la révision de la PPE régionale. Fixé par la loi sur la transition énergétique de 2015, son objectif principal est ambitieux: atteindre l'autonomie énergétique en 2030, en développant massivement les énergies renouvelables et les économies d'énergie. quelles énergies prendre en compte?   Ambitieux, car 85% des énergies consommées dans l'ancienne Bourbon sont importées et qu'à l'horizon de 2030, la population aura pratiquement grossi de 20%. « C'est un objectif teinté d'ambiguïté, râle Ivan Martin. Quelle prise avons-nous sur la consommation des avions et des cargos qui ravitaillent l'île? », s'interroge le directeur adjoint de la Deal. A eux deux, ces postes représentent 17% environ de l'énergie primaire du bilan réunionnais.   Sans surprise, la réflexion porte d'abord sur l'électricité. Un peu moins sur les transports.   décarboner dans la croissance   Deux défis doivent être relevés par les parties prenantes du système électrique: décarboner le mix et satisfaire une demande croissante d'électrons. Pas gagné. Car le parc électrique réunionnais est bien plus carbonifère que celui de la métropole. Les énergies renouvelables (hydraulique en tête) produisent le tiers du courant. Le solde est essentiellement généré par la combustion de charbon et de bagasse (un déchet de la production de la canne à sucre).   Objectif numéro un: trouver des substituts au 650.000 tonnes de charbon sud-africain brûlées chaque année dans les centrales du Gol et de Bois Rouge, deux installations de 110 mégawatts électriques (MWe) unitaire exploitées par Albioma. Présent aux Antilles, au Brésil et à La Réunion, le groupe dirigé par Frédéric Moyne se prépare à la conversion de ses centrales à la biomasse. Reste à la trouver. Et en quantité suffisante: il faut trois fois plus de biomasse que de charbon pour produire le même volume d'énergie thermique. « Aujourd'hui, La Réunion produit 5% de sa biomasse énergétique », résume Eric Jeuffrault, directeur régional du Cirad[1]. On peut faire mieux.   des forêts mais pas de bois   A condition de dynamiser les filières bois, élagage et gestion des déchets verts, l'Office national des forêts estime à près de 90.000 t le potentiel annuel de production de pellets ou de plaquettes 'pays'. « Améliorer ainsi l'exploitation des forêts réunionnaises permettra, en outre, d'accroître à long terme la production de bois d'oeuvre », commente le forestier Jean Prévost. Cela restera insuffisant pour assurer l'alimentation des chaudières d'Albioma. Les forestiers du Canada ou des Etats-Unis voient s'ouvrir un nouveau marché.   Les promoteurs d'énergies renouvelables électriques piaffent eux aussi d'impatience. En la matière l'île Bourbon n'est pas très exemplaire. Avec une soixantaine de mégawatts installés, l'éolien et le photovoltaïque assurent moins de 10% du courant réunionnais. Mais les ambitions sont grandes. Selon le projet de PPE révisée, La Réunion pourrait être dotée de 340 MWc de capacités photovoltaïques en 2023, et de 540 MWc en 2028. En raison de ses vents capricieux, l'île volcanique n'est pas un paradis pour les aérogénérateurs. La Deal n'imagine pas y voir tourner plus de 116 MW à l'horizon 2028: 14 fois plus qu'aujourd'hui.   Combien d'ENR électriques?   Cela n'enchante guère certains acteurs. A commencer par EDF SEI. Particularité des zones non interconnectées, la filiale outre-mer (et corse!) de l'énergéticien public reste une entreprise intégrée, conservant le monopole du transport et de la distribution d'électricité et une partie de sa production. Et comme nombre de ses collègues métropolitains, Vincent Lévy apprécie modérément la production d'électricité intermittente. « Nous sommes déjà à 32% d'énergies renouvelables, alors qu'un décret de 2008 limite ce taux à 30%. Or nous ne sommes pas prêts à passer aux 100% renouvelables prévus à l'horizon 2030 », estime le chef du service système électrique d'EDF SEI Réunion.   Ce n'est pas l'avis des ingénieurs de l'Ademe[2]. Dans une étude prospective présentée en mars 2018, le bras séculier du ministère de la transition écologique estime que La Réunion pourrait massivement développer sa production d'électricité verte. Selon les scénarios, entre 85 et 100% de l'électricité consommée dans l'île pourrait être produite par des éoliennes et des centrales solaires. Ce qui représente des capacités installées comprises entre 787 et 1.418 MW installés. « Le tout avec un coût de production complet se situant autour de 120 euros/MWh, soit presque la moitié du coût actuel de la production d'électricité à La Réunion », affirme Jérôme Billerey, du Syndicat des énergies renouvelables (SER).   une prison solaire   Vrai problème, l'intermittence? Pas si sûr. La première, EDF SEI a testé des batteries couplées à des systèmes de production renouvelables. Avec des résultats probants, aussi bien pour lisser les pics de consommation (et réduire ainsi les appels à des moyens de production de pointe) que pour soutenir la fréquence du réseau. Désormais, les exploitants de centrales renouvelables équipent leurs parcs de batteries lithium-ion de plusieurs mégawatts.   Les naturalistes suivent eux aussi avec attention le développement annoncé des renouvelables. La Réunion est l'un des joyaux de la nature française. Son parc national et les sites classés au patrimoine mondial de l'Unesco couvre 105.000 hectares, soit plus de 42% de la superficie du département-région. Cette contrainte foncière oblige les industriels à s'adapter. A La Possession, la prison accueille un parc photovoltaïque de 9 MWc, l'un des plus puissants de l'île. A ses côtés, une demi-douzaine de serres photovoltaïques, exploitées par l'association Agriterra, produisent électricité, fleurs coupées et légumes. Quelques détenus en fin de peine y apprennent le métier « et 5 se sont installés comme agriculteurs professionnels », se réjouit Sanassy Duquesne, la directrice des opérations.   emplois locaux   Devant la lenteur de la Commission de régulation de l'énergie (CRE) à publier des appels d'offres Outre-mer, les entreprises s'adaptent. Quadran va prochainement lancer le renouvellement ('repowering') du parc éolien de Sainte-Suzanne. Les 37 antiques éoliennes Vergnet vont laisser la place à 9 turbines de 2 MW résistantes aux cyclones. « De quoi produire deux fois plus d'énergie avec 4 fois moins de machines », résume Gaël Vallée, responsable Développement océan Indien de Quadran. Le même développeur veut doter le petit aéroport international de Pierrefond d'une centrale photovoltaïque de près de 8 MWc. Ce qui apporterait 200.000 à 300.000 € de recettes annuelles supplémentaires à un aéroport en panne de trafic: deux avions par jour.   Cette agilité n'est pas forcément du goût des représentants de la société civile. Fustigeant une PPE très électrique, le président du Conseil économique, social et environnemental régional (Ceser) a d'autres priorités que la seule décarbonation. « Une mauvaise PPE est celle dont le retour sur le territoire est faible. Or ce qui prime aujourd'hui, c'est d'aller vite », regrette Dominique Viel. Pour le président du Ceser, il faut attendre la mise en oeuvre d'une politique forestière créatrice d'emplois locaux plutôt que de se lancer à corps perdu dans l'importation de plaquettes de bois nord-américaines.   Autoproduction collective   Le nouveau monde énergétique, promis par la future PPE régionale, commence à faire bouger les élus. A commencer par Vanessa Miranville. La jeune maire de La Possession accélère la construction de l'éco-quartier Coeur de ville. Prévu pour accueillir 5.000 habitants sur 34 ha, le futur centre urbain abrite ce qui se fait de mieux en matière d'architecture bio-climatique. « Le rez-de-chaussée des immeubles a été laissé ouvert pour faciliter la ventilation. Les appartements n'ont ainsi plus besoin de climatisation. En outre, le tiers de la surface de la ZAC sera planté de 80.000 arbres, réduisant de 4 à 7°C la température extérieure ambiante », indique cette proche de Nicolas Hulot. Dotés de centrales photovoltaïques, les immeubles devraient constituer la première centrale en autoproduction collective de l'île. Reste encore à trouver le modèle juridique et économique. Difficile, en effet, d'exploiter les données générées par les capteurs installés dans chaque appartement. Le RGPD[3] n'a pas été conçu pour cela.   Ce type d'urbanisme sobre en énergie est-il appelé à se généraliser? Rien n'est moins sûr. Mais pas forcément pour des raisons liées aux renouvelables. « On a 200.000 habitants qui arrivent dans les prochaines années. Or nos carrières s'arrêtent dans deux ans. Et on ne pourra pas importer d'importants tonnages de matériaux de construction », rappelle le Deal Jean-Michel Maurin. Un détail!   pénurie de matériaux   Pareille contrainte pourrait être minorée par les retombées de la construction de la Nouvelle route du littoral (NRL). En cours de construction à une vingtaine de mètres au-dessus du niveau de l'océan, cette autoroute à 6 voies va remplacer une quinzaine de kilomètres de l'actuelle voie rapide régulièrement engloutie par les blocs rocheux tombés de la falaise. Parmi les mesures de compensation que devra mettre en oeuvre le constructeur GTOI: le démontage de l'ancienne route et de ses brise-lames. Assez pour produire quelques millions de tonnes de matières premières secondaires pour le BTP local.   Fustigée par les environnementalistes locaux, la NRL est paradoxalement porteuse d'une bonne nouvelle énergétique. Deux de ses 6 voies doivent être réservées à un réseau de transport collectif en site propre. Bus ou tramway, le choix n'a pas encore été arrêté par le conseil régional. Mais d'ores et déjà, un réseau de 150 km, allant de Saint-Denis (nord) à Saint-Joseph (sud), est prévu. Il pourrait être totalement opérationnel vers 2040. En revanche, le premier tramway de l'île devrait relier, sur 10 km, Saint-Denis à Sainte-Marie. « Ce qui devrait réduire de 8.000 voitures par jour le trafic automobile sur cet axe très fréquenté », affirme Fabienne Couapel-Sauret, conseillère régionale déléguée aux transports. En 2028, la future PPE prévoit que 14% des trajets effectués par les Réunionnais se fassent via les transports collectifs, contre 7% aujourd'hui. A cet horizon, 20.000 voitures électriques devraient sillonner les routes de l'île. Soit 20 fois plus qu'aujourd'hui. Reste à savoir si cela sera suffisant (avec l'abandon du charbon) pour réduire de 28% la consommation d'énergies fossiles.   Energie thermique des mers multifonctions. C'était l'un des grands espoirs énergétiques de l'île. En raison de ses tombants océaniques, La Réunion espérait bien mettre en service des centrales utilisant l'énergie thermique des mers (ETM). Porté par DCNS (devenu Naval Energies), cette énergie marine n'a pas tenu ses promesses. A plus de 300 €/MWh, les coûts de production ne sont pas compétitifs, concède Victor Bouissou-Thomas, responsable Affaires publiques et européennes de Naval Energies. A moins, bien sûr, de multiplier les usages. Outre la production d'électricité, l'ETM pourrait aussi stabiliser la température de l'eau des bassins d'une ferme aquacole et fournir de l'eau froide aux centrales thermiques et de production d'éthanol. La première installation pourrait être construite dans le futur port de Bois-Rouge (nord-est). S'il voit jamais le jour.   [1] Cirad: Centre de coopération internationale en recherche agronomique pour le développement   [2] Ademe: Agence de l'environnement et de la maîtrise de l'énergie   [3] RGPD: règlement général sur la protection des données
</article>
<article titre="Pourquoi la conversion charbon biomasse est une fausse bonne idée" date="2019 02 4" journal="L'Usine Nouvelle (site web)">
La France a acté pour 2022 une sortie de la production d'électricité à partir de charbon. Mais comme le montre l'exemple britannique, l'idée de convertir les centrales à la biomasse n'est pas forcément une bonne idée.    Les Britanniques, champions européens de la sortie du charbon, y ont cru... et en sont revenus. Verdir des centrales électriques en remplaçant le charbon par du bois, permet certes un bilan carbone meilleur, le bois brûlé ayant capté du CO2 durant sa croissance et émettant un peu moins de gaz à effet de serre que le charbon, mais la biomasse s'avère souvent être une aberration économique et écologique. "La combustion des pellets de bois émet plus de particules fines et a un pouvoir calorifique de 70 % à 80 par rapport à celui du charbon", rappelle Almuth Enrsting de l'association Biofuelwatch. Surtout, ils nécessitent de lourds investissements dans les centrales à charbon existantes pour éviter la corrosion. Au Royaume-Uni, deux centrales ont dû fermer après des incendies. Et sur les quatre unités charbon converties à la biomasse de la méga centrale de Drax, trois sont à l'arrêt. La dernière en activité de 2 gigawatts est le plus gros importateur des pellets, qui proviennent des États-Unis, du Canada mais aussi d'Estonie, de Lituanie, du Portugal et d'Europe de l'est voire, de Russie... Avec un bilan carbone déplorable. Uniper et Albioma bloqués par les ressources locales   En France, l'allemand Uniper, qui a converti une unité charbonne à Gardanne (Bouches-du-Rhône) en unité biomasse de 150 MW, a lui aussi été confronté au problème d'approvisionnement. Sa technologie de lit fluidisant lui permet de brûler tous types de déchets de bois, mais les ressources locales ne sont pas suffisantes et entrent en concurrence avec d'autres activités. Après avoir perdu son droit d'exploitation en mi-2017 et un an de négociations, Uniper n'a plus qu'une autorisation préfectorale temporaire, le temps d'inscrire la centrale dans une économie circulaire locale vertueuse...   À la Réunion, l'opérateur Albioma est lui aussi confronté à un problème de ressources locales de biomasse pour convertir ses deux centrales mixtes charbon/bagasse en charbon/biomasse d'ici à 2023. Au mieux, l'île ne pourrait fournir que 30 % de ses besoins en biomasse, l'obligeant à envisager l'importation de pellets des États-Unis ou du Canada, en attendant qu'une filière durable se développe en Afrique du Sud.   Il n'y a peut-être qu'aux Antilles, où les champs pollués de la culture de bananes trouveront dans la culture énergétique de la canne fibre un débouché, que la conversion à la biomasse des centrales bagasse/charbon ne met pas une trop grande pression sur l'environnement et peut s'inscrire dans une logique d'autonomie énergétique durable.   EDF maintient son projet à Cordemais   Conscient ou non de ces écueils, en 2015, suite à la loi de transition énergétique pour la croissance verte, EDF a investi 10 millions d'euros dans un projet de conversion à la biomasse de sa centrale de Cordemais (Loire Atlantique). Le projet Ecocombust vise d'une part à produire sur le site un combustible alternatif au charbon, un pellet de biomasse densifiée, à partir de déchets verts et de bois déclassé de série B provenant d'un rayon de 150 km autour de la centrale. Le processus utiliserait en partie la chaleur fatale des centrales.   Il envisageait d'autre part que les trois unités de production d'électricité d'EDF à partir de charbon de Cordemais, où s'effectuent les tests, et du Havre (Seine-Maritime), pourraient utiliser 80 % de ce nouveau combustible mais en ne fonctionnant plus que pour les pointes de demande, soit 800 heures par an au lieu de 4 500 à 5 000 heures actuellement, avec une puissance ramenée à 500 MW contre 600 MW aujourd'hui. Les coûts de production seraient de l'ordre de 115 à 120 euros/MWh, contre environ 50 euros aujourd'hui.   La biomasse réservée à la chaleur   Les premiers essais montrent qu'il est possible de substituer jusqu'à 87 % de charbon sans modifier les installations. EDF serait prêt à investir 100 millions d'euros supplémentaires pour ne pas fermer les sites et les convertir à la biomasse, mais sous condition d'obtenir une subvention équivalente du gouvernement. Or de question pour le gouvernement. Au mieux, en raison de problème de sécurité d'approvisionnement sur la Bretagne, le gouvernement est-il prêt à prolonger une unité de Cordemais au-delà de 2022, pour un ou deux ans, mais sous réserve de consommer 80 % de pellets et pas plus de 200 heures par an, pour absorber des pointes d'hiver. En revanche, l'idée de convertir le site en usine à black pellets, qui pourrait alimenter des unités de production en chaleur, séduit jusqu'à l'Ademe.   L'idée n'est pas neuve. Mi-2018 l'Européenne de biomasse a annoncé un investissement de 100 millions d'euros sur la plateforme de Pomacle-Bazancourt (Marne) pour produire en continu des black pellets à partir des résidus de bois issus des forêts de la région. Le gouvernement a annoncé qu'il prendrait sa décision concernant le projet d'EDF en septembre 2019.
</article>
<article titre="Pour sortir du charbon, La Réunion devra choisir entre biomasse importée et culture énergétique" date="2019 02 1" journal="L'Usine Nouvelle (site web)">
A la Réunion, le producteur d'électricité outre-marin Albioma envisage de convertir ses deux centrales électrique charbon/bagasse à la biomasse. La ressource locale n'est pas suffisante.   Le sujet est tabou. Mais la loi de 2015 obligeant les territoires d'outre-mer à l'autonomie énergétique en 2030 pourrait mettre le sujet sur la table. A La Réunion, ne faut-il pas troquer la culture de canne à sucre ultra subventionnée par l'Europe contre des cultures énergétiques de canne fibre ? Le sujet est politique. Hormis les acteurs des énergies renouvelables, personne n'ose trop en parler. Et Albioma (480 personnes, 403 millions d'euros de chiffre d'affaires en 2017), qui produit sur l'île 54 % de l'électricité dans deux centrales mixte bagasse/charbon, moins que personne, tant le sujet est sensible. Pourtant, le spécialiste de la valorisation énergétique de la bagasse (le résidu de canne à sucre) pourrait en être le principal acteur.   Objectif autonomie énergétique en 2030   Pour respecter la loi de transition énergétique pour la croissance verte de 2015, qui fixe comme objectif l'autonomie énergétique des départements d'outre-mer d'ici à 2030, avec une étape à 50 % en 2020, Albioma propose de convertir ses deux centrales électriques hybrides bagasse/charbon de Bois-Rouge (100 MW) au nord-est et Le Gol (111 MW) au sud-est à la biomasse, ou plus exactement à un mix bagasse/biomasse.   Sauf que, selon les calculs de l'Office national des forêts (ONF) local, sur les 500 000 à 600 000 tonnes de bois déchiquetés (ou 300 000 à 400 000 tonnes de pellets) dont Albioma aurait besoin pour remplacer le charbon, tout en continuant à brûler la bagasse lors des saisons de récolte, l'île ne peut en couvrir au mieux que 30 %.   Sans conflit avec d'autres, comme le bois d'oeuvre ou la fourniture de litière de bois à la filière d'élevage aviaire, l'ONF évalue la ressource locale en bois valorisable et d'élagage à 100 000 tonnes et les déchets verts entre 100 000 et 114 000 tonnes. Il lui faudrait pour les exploiter réaliser un investissement annuel de 300 000 euros. Cela permettrait de consolider une filière d'environ 800 emplois sur toute l'Île. Accessible et souhaitable donc.   800 emplois dans la filière bois   Le compte n'y est pas pour Albioma. Pour remplacer complètement le charbon, l'exploitant va devoir importer des pellets de bois des Etats-Unis, du Canada ou du Brésil. Alors qu'il était encore secrétaire d'Etat auprès du ministre de la Transition écologique et solidaire, Sébastien Lecornu, en visite à La Réunion, aurait jugé l'option inacceptable. Les alternatives n'ont, a priori, rien pour plaire.   Il faudrait que La Réunion tourne la page de la canne à sucre subventionnée, qui manque cruellement de débouchés, contre une culture de canne fibre, exploitable toute l'année, pour alimenter en biomasse ses centrales électriques. Cela bouleverserait une partie de l'économie. La canne à sucre couvre près de 60 % des surfaces cultivées à La Réunion (plus de 24 000 hectares) dans 3 000 exploitations. Celles-ci récoltent 1,9 million de tonnes de cannes pour une production moyenne de 210 000 tonnes de sucre. La filière canne-sucre représente près de 14 000 emplois directs et indirects.   Passer au 100% renouvelables ?   La canne fibre mettrait aussi fin au problème de surplus. Ce qui n'arrangera pas forcément Albioma. Ce dernier vient juste de mettre en service la première centrale électrique au monde fonctionnant au bioéthanol de canne à sucre, issues des surplus de mélasse d'une raffinerie de Rhum.   L'autre option serait de revoir complètement la production d'électricité sur l'île et de progressivement remplacer les centrales à énergie fossile par du photovoltaïque et un peu d'éolien. EDF SEI, qui gère toute l'électricité de l'île, ne l'envisage même pas, même si les renouvelables représentent déjà 36 % de la production électrique. En plein révision de la PPE (programmation pluriannuelle de l'énergie) de La Réunion, la question mérite pourtant d'être posée.   A la Réunion, Aurélie Barbaux
</article>
<article titre="NOUVEAUTÉS EN AVANT-PREMIÈRE SPÉCIAL SALON CFIA RENNES 2019 Écoconception Des barquettes compostables chez Agro Concept Embal" date="2019 02 1" journal="Process Alimentaire, no. N° 1366">
Le spécialiste de l'emballage alimentaire en grès propose également des barquettes en bagasse, issues des déchets de la culture de la canne à sucre. Celles-ci sont biodégradables et compostables selon la norme française NF13432 et l'Allemande Din Certco. Et sont également recyclables dans la filière papiers/cartons. Pour leur conférer des propriétés barrières, elles peuvent être recouvertes d'un film biodégradable étanche et micro-ondable ou d'un film en polyéthylène (PE) ou en polypropylène (PP). De quoi conditionner des viandes, charcuteries, poissons, plats cuisinés, fromages, etc., en frais ou en surgelé.   Hall 9 - Stand D54
</article>
<article titre="NOUVEAUTÉS EN AVANT-PREMIÈRE SPÉCIAL SALON CFIA RENNES 2019 Le conditionnement à la croisée des chemins" date="2019 02 1" journal="Process Alimentaire, no. N° 1366">
Ouverture  Plus que jamais, l'emballage doit montrer patte blanche. Voilà déjà plusieurs années que Process Alimentaire s'en fait le témoin, mais le mouvement s'est clairement accéléré en 2017-2018 avec de multiples initiatives plus ou moins contraignantes : la Stratégie Plastique européenne, la Feuille de Route sur l'Économie Circulaire française, la loi Alimentation et ses amendements plastique, etc. Les matériaux non recyclables ou non biosourcés sont clairement en ligne de mire, avec sur l'échafaud : le plastique d'origine fossile qui, malgré ses fonctionnalités et ses vertus en matière de lutte contre le gaspillage, reste le symbole visible de la pollution environnementale à travers le septième continent. Le mono-matériau en rigide et souple Dans ce contexte, toute la chaîne de valeur est en émoi. Et surtout en quête d'alternatives. Pas si facile, pour l'industrie de l'ultrafrais, par exemple, qui a pendant des dizaines d'années thermoformé des pots en polystyrène à grandes cadences, de basculer sur de plus petites séries, qui plus est dans un matériau davantage recyclable : le polyéthylène téréphtalate (PET) ? Le polypropylène (PP) ? Certains imaginent même un retour au verre.   Pour les emballages rigides de plats préparés, charcuteries, salades composées et viandes, le mono-matériau fait recette avec des contenants en PET ou en polypropylène, parfois operculés en mono. Sur le Cfia, MikoPac présente, par exemple, sa gamme standard d'emballages rigides en plastique mono-matériau et/ou barrière. Ainsi que sa nouvelle gamme d'emballages biosourcés à 80 % et biodégradables Treepac. Produits aussi bien en thermoformage qu'en injection, ils sont operculables, recyclables et compostables en compostage industriel. Sur le stand Dynaplast, la nouveauté provient de barquettes en polypropylène mono-matériau Maptipack, conformes aux futures consignes de tri. Tout comme les barquettes Kary Peal de Knauf Industries qui sont adaptées au conditionnement des viandes sous skin, autre grande tendance du secteur.   L'offre de barquettes compostables monte aussi en flèche. Avec l'exemple d'Agro Concept Embal qui propose de nouvelles barquettes en bagasse, issues des déchets de la culture de la canne à sucre. Celles-ci sont biodégradables et compostables selon la norme française NF13432 et sont également recyclables dans la filière papiers/cartons. Sur ce modèle, mais dans un autre matériau, l'Espagnol PackBenefit expose pour la première fois ses barquettes en cellulose moulée déjà largement utilisées dans la restauration hors domicile.   Réduction à la source des emballages tertiaires Le marché des films souples connaît aussi une vague d'innovations «vertes». Sealed Air, par exemple, qui participe à l'initiative «Nouvelle Économie des Plastiques» de la fondation Ellen MacArthur, présente son film mince rétractable Cryovac Sealappeal PSF ZAL de 19 microns. Il est à base de PET et se soude à basse température pour un poids total de l'emballage réduit. Ce film est utilisé par Fleury Michon pour operculer son Salad'Jar en PET. Les films de la famille des polyoléfines (PP ou PE) arrivent aussi en masse pour défendre leur recyclabilité. Sur le Cfia, Schur Flexibles, Wipf, Taghleef Industries, Di Mauro Flexible, SP Group ou encore FPC Packaging exposent ainsi des films en mono-PP pour certains ou en mono-PE pour d'autres. Tantôt pour des applications flowpack ou pour la formation de sachets stand-up.   Aux côtés des offres en plastiques mono-matériau se déploient celles en matériaux biosourcés, chez Malengé, par exemple, STTP Emballage ou encore l'Agence Pierre Lelong qui distribue les films israéliens Tipa.   Tous ces développements s'opèrent, évidemment, en collaboration avec les fabricants de machines de conditionnement. Car l'enjeu reste d'innover en maintenant sa productivité, qu'il s'agisse d'ensacheuses VFFS ou HFFS, de remplisseuses de pots, de thermoformeuses ou d'operculeuses.   Même en fin de ligne, sur les emballages secondaires et tertiaires, l'enjeu de la réduction des matériaux à la source est important. L'un des nominés aux Trophées de l'Innovation n'est autre que Bulteau Systems qui lance un film étirable de banderolage de 6 µm d'épaisseur, composé à 100 % de LLDPE (polyéthylène linéaire de basse densité) recyclable. Au fardelage, Polypack arrive également à réduire de 30 % l'épaisseur de film avec son concept Stronghold. Grâce à une poignée intégrée, l'emballage de regroupement de bouteilles devient 100 % mono-matériau. De toute façon, pas de mystère, d'ici 2025, tous les emballages en plastique devront être recyclables en France. La course est lancée.   FLEXIBILITÉ La polyvalence des machines n'est plus une option Avec la multiplication des séries liée, entre autres, à l'avènement du multi-canal, les lignes doivent se montrer de plus en plus polyvalentes. Sur ce point, le salon regorge de solutions. L'Aritrak de Tecma Aries, par exemple, multiplie les possibilités d'encaissage avec un seul outil. Les cobots, justement, permettent de conditionner ou de palettiser des produits de façon automatisée et flexible, mais souvent à petites cadences. Les nouvelles générations de machines telles que celles exposées par Cama Group, Guelt, Jyga, Blue Print Automation ou encore ET Pack Sprinter vont aussi dans le sens des changements de formats simples et rapides.
</article>
<article titre="La canne à sucre créé de l'électricité à l'Île Maurice" date="2019 01 31" journal="La Tribune Afrique (site web)">
Pour diminuer son recours aux énergies fossiles et assurer une transition énergétique vers les énergies propres, l'Île Maurice -indépendante depuis 1968- doit se débrouiller avec ce qu'elle a. Et ce qu'elle compte le plus, c'est la canne à sucre, sa principale culture. Un défi, sur ce caillou au milieu de l'océan, sept fois plus petit que la Corse mais quatre fois plus peuplé (environ 1,2 million de Mauriciens).   Comme le rappelait l'Agence France Presse début décembre, depuis une quinzaine d'années, c'est grâce au résidu fibreux que l'on retire du broyage de la canne à sucre (et que l'on appelle la « bagasse ») que l'on obtient de l'électricité. Un tel schéma permet, aujourd'hui, de couvrir 14% des besoins des habitants de l'île, rapportait l'AFP. >>> Lire aussi : Energies vertes : l'Afrique attire les investissements internationaux Objectif : 35% d'ENR en 2025 Le gros de la production d'électricité du pays (environ 60%) reste assuré par quatre centrales thermiques, propriétés de quatre sociétés sucrières, où la combustion est l'oeuvre de la bagasse et non du charbon. Au total, ce sont 22% de l'énergie produite à Maurice qui est renouvelable, des projets ayant vu le jour dans les domaines de l'éolien et du solaire (environ 1% du mix) comme de l'hydroélectricité (5% environ).   L'île a accueilli en mai le premier Forum régional des énergies renouvelables (organisé par la Commission de l'océan indien), peut-être un signe de son dynamisme pour le secteur. Le pays s'est fixé pour ambition, d'ici 2025, de faire grimper sa part d'énergies renouvelables à 35%. Il devrait compter l'an prochain pas moins de onze parcs solaires et deux parcs éoliens.   Un projet financé à hauteur de 191 millions de dollars a été lancé fin 2017 (sur huit années) pour que Maurice parvienne à diminuer ses importations de combustibles fossiles et atteigne le zéro carbone dans les deux décennies à venir.   >>> Aller plus loin : Bientôt un marché commun de l'énergie en Afrique ?  Cet article est paru dans La Tribune Afrique (site web)
</article>
<article titre="Quel lien entre une turbine d'avion GE, le rhum de la Réunion et les énergies renouvelables ?" date="2019 01 31" journal="L'Usine Nouvelle (site web)">
A la Réunion, le producteur d'électricité outre-marin Albioma vient de mettre en service la première centrale électrique fonctionnant au bioéthanol de canne à sucre. Une électricité bas carbone qui viendra compenser les pointes de consommation sur l'île.   C'est une première mondiale qu'Albioma vient de réaliser à la Réunion. Ce spécialiste outremarin de la valorisation énergétique de la bagasse (le résidu de cannes à sucre), vient de trouver un débouché pour le surplus de mélasse de la distillerie Rivière du Mât. Il va injecter, après transformation en bioéthanol à 95,5, dans une turbine GE LM 6000 - celles des Boeing 747- pour produire de l'électricité. De l'électricité à base de bioéthanol de cannes   Spécialement réglée par GE à la demande d'Albioma, cette turbine produite aux Etats-Unis consomme 80% de bioéthanol et 20% de gazoil, un carburant fossile indispensable pour le démarrage rapide (7 minutes) de la turbine. Elle pourrait ainsi valoriser 3000 à 5000 m3 de bioéthanol produits localement par la distillerie qui a investi pour adapter ses équipements. Avant ce projet, ses surplus de mélasse, une fois le quotas de rhum produit atteint, était envoyé en Métropole pour transformation. Et, il n'existe pas à la Réunion, pour le moment, de filière biocarburant à base de bioéthanol, comme le E85, ou autres.   Albioma- 480 personnes, 403 millions d'euros de chiffre d'affaires en 2017-, a investi 60 millions d'euros dans cette turbine à combustion (TAC) d'une puissance de 41 MW. Installée sur la commune de Saint-Pierre, elle n'est pas destinée à une production constante d'électricité, mais pour couvrir les pointes de production. EDF SEI, qui gère le réseau électrique réunionnais, peut l'actionner à distance.   Une nouvelle innovation d'Albioma   Si c'est une première avec du bioéthanol, Albioma maîtrise bien les turbines GE. A la Martinique, une TAC du même type se nourrit depuis plus de dix ans de gazole. C'est aussi en Martinique qu'Albioma vient d'inaugurer la première centrale électrique 100% biomasse, Gallion II, de 40 MW. Quand, elle n'a plus de bagasse à brûler, elle utilise du pellet de bois, importé des Etats-Unis, pour alimenter la centrale.   Une centrale hybride qui fait école. Albioma a lancé la conversion sur le même modèle d'une centrale bagasse/charbon en Guadeloupe. A la Réunion, ce n'est pas une mais deux centrales bagasse /charbon que l'opérateur doit convertir à la biomasse. Mais là, elle pourrait être alimentée à 30% par une ressource locale de bois et de déchets verts, a calculé l'Office National des Forêts régional. Reste à construire la filière.
</article>
<article titre="La Guadeloupe, laboratoire des énergies renouvelables" date="2019 01 29" journal="La Croix (site web)">
Le plus grand parc éolien des Antilles va d'être inauguré par Valorem. Mais la transition énergétique est encore loin en Guadeloupe, où le charbon et le fioul assure encore l'essentiel des besoins.   Au milieu des champs de canne à sucre, huit éoliennes face à la mer des Caraïbes et aux alizés, ces vents réguliers venus de l'Atlantique. Le groupe français Valorem a inauguré samedi 26 janvier, le plus grand parc éolien des Antilles françaises à Sainte-Rose, en Guadeloupe. D'une capacité totale de 16 mégawatts (MW), ces machines de 80 mètres de haut devraient produire environ 33 000 MWh d'électricité par an, soit la consommation annuelle de 17 000 personnes. Des machines conçues pour résister aux cyclones   Pour la première fois dans la région, ces éoliennes ne se rabattent pas en cas de cyclone. Le dispositif a été conçu de manière à ce que les pâles se mettent automatiquement face au vent. Le principal intérêt est de pouvoir disposer de machines beaucoup plus puissantes.   L'ensemble représente un investissement de 50 millions d'euros, financé à 65 % par Valorem, à 35 % par la Caisse des dépôts et à 5 % par une société d'économie mixte locale. Ce parc va augmenter de 10 % la production d'électricité renouvelable de l'île et permettra de diminuer le recours aux centrales thermiques. Il double également la capacité éolienne, qui passe de 3 % à 6 % de l'électricité produite.   VIDEO - Nucléaire et énergies renouvelables?: les annonces d'Emmanuel Macron   Une terre d'énergies fossiles   Mais la Guadeloupe a encore beaucoup de chemin à faire en matière de transition énergétique, même si les autorités locales affichent des ambitions très volontaristes. Pour l'heure, l'île est surtout le royaume des énergies fossiles.   Les renouvelables ne représentent que 20 % de la production d'électricité. Le reste est assuré par le fioul (55 %) et le charbon (25 %), évidemment importés. Tout cela coûte une petite fortune?: 250 € le mégawatt heur (MWh), cinq fois plus que les prix de marché européens, voire 350 €/MWh à la pointe.   Des objectifs ambitieux dans les renouvelables   L'ambition de la Guadeloupe est d'atteindre 50 % d'énergies vertes en 2023 et l'autonomie totale en 2030. Ce n'est pas impossible, mais très compliqué, même si l'île ne manque pas d'atouts?: du soleil, du vent, de la géothermie (avec la seule centrale française connectée au réseau) et aussi de la biomasse locale, grâce à la bagasse, le résidu de la canne à sucre.   Mais aller beaucoup plus loin dans l'éolien, nécessite par exemple de déplacer le radar de Météo France, installé là où les conditions de vents sont les plus favorables. Le sujet est sur la table depuis des années.   « Les énergies renouvelables sont devenues moins chères que les énergies fossiles. Mais vu de Paris, les Antilles semblent très loin et les Guadeloupéens ont un peu le sentiment d'être un peu les oubliés de la transition énergétique. Il est difficile, par exemple, de comprendre pourquoi l'arrêt programmé des centrales charbon en métropole n'est pas valable ici », affirme Jean-Yves Grandidier, le PDG fondateur de Valorem, un des pionniers de l'éolien en France et l'un des derniers gros développeurs indépendants du secteur   Eolien offshore, l'exception française   De l'éolien couplé à du stockage   Augmenter la part des renouvelables nécessite aussi de pallier à leur intermittence. EDF, qui assure à la fois l'essentiel de la production l'électricité de l'île et la distribution, estime ainsi pouvoir absorber que seulement 30 à 35 % d'électrons verts sur le réseau.   D'où l'intérêt du parc éolien de Sainte-Rose qui a été couplé à une unité de stockage de 5 MW. Le système qui a été développé avec le CEA, est inédit aux Antilles. Piloté depuis le centre de Bègles (Gironde), où se trouve le siège du groupe, Valorem fournit une prévision de production de la veille pour le lendemain, avec un système de pénalités si les engagements ne sont pas respectés.   Le parc de Saint-Rose a un contrat d'achat de l'électricité sur vingt ans à 220 €/MWh. Il n'a pas fait l'objet de la part des riverains, ce qui est plutôt exceptionnel dans le secteur. Comme il le fait sur la plupart de ses projets, Valorem va également lancer une campagne de financement participatif, à hauteur de 500 000 € pour associer la population aux retombées.  Cet article est paru dans La Croix (site web)
</article>
<article titre="Cordemais, Le Havre... La conversion des centrales à charbon d'EDF en négociation" date="2019 01 28" journal="L'Usine Nouvelle (site web)">
EDF et le gouvernement discutent de l'avenir des centrales à charbon. Une décision doit être prise à l'automne 2019 concernant la mise en oeuvre du procédé Ecocombust à Cordemais et au Havre. Ce projet de reconversion reposant sur l'usage de déchets de bois à la place du charbon est loin de faire l'unanimité   Une nouvelle filière de fabrication de combustible biomasse à partir de déchets de bois est envisagée pour remplacer l'usage du charbon dans les centrales de Cordemais (Loire-Atlantique) et du Havre (Seine-Maritime) afin de respecter les engagements du gouvernement de les fermer d'ici à 2022. Dans un communiqué, rendu public lundi 28 janvier, le ministre de la Transition écologique et solidaire, François de Rugy, réaffirme que "le Gouvernement connaît la situation particulière du réseau électrique de l'Ouest de la France et l'importance de la centrale de Cordemais pour la sécurité d'approvisionnement électrique de la Bretagne". L'Etat a donc demandé des analyses complémentaires au gestionnaire du réseau de transport électrique, RTE, sur la sécurité d'approvisionnement de la région et "a défini avec EDF les étapes d'un programme de travail lors d'une réunion qui s'est tenue au ministère de la Transition écologique et solidaire le 24 janvier sur l'avenir du site de Cordemais". EDF doit remettre ses travaux pour permettre de prendre une décision à l'automne prochain à propos du procédé Ecocombust à Cordemais et au Havre. EDF a présenté au Gouvernement "un projet de création à Cordemais d'une usine de fabrication d'un combustible innovant produit à partir de déchets de bois, qui peut se substituer au charbon et alimenter des réseaux de chaleur". 10 millions d'euros investis par EDF à Cordemais En 2015, EDF a investi 10 millions d'euros dans le projet de conversion à la biomasse de sa centrale de Cordemais (Loire-Atlantique). Ecocombust vise à produire sur le site un combustible alternatif au charbon, un pellet de biomasse densifiée, à partir de déchets verts et de bois déclassés de classe B provenant d'un rayon de 150 km autour du site. Le processus utiliserait en partie la chaleur fatale des centrales. Les trois unités charbon de Cordemais et du Havre (Seine-Maritime) devraient utiliser 80 % de ce nouveau combustible pendant 800 heures par an au lieu de 4 500 à 5 000 heures actuellement. Les coûts de production seraient de l'ordre de 115 à 120 euros/MWh, contre environ 50 euros aujourd'hui. Le gouvernement n'y est pas favorable. Au mieux, en cas de doute sur la sécurité d'approvisionnement de la Bretagne, est-il prêt à prolonger une unité de Cordemais au-delà de 2022, pour un ou deux ans, mais sous réserve de consommer 80 % de pellets et pas plus de 200 heures par an, pour absorber les pointes d'hiver. L'idée de convertir le site en usine à "black pellets" pour alimenter des unités de production de chaleur séduit même l'Agence de l'environnement et de la maîtrise de l'énergie. Elle n'est pas neuve. Mi-2018, l'Européenne de biomasse a annoncé un investissement de 100 millions d'euros sur sa plateforme de Pomacle-Bazancourt (Marne) pour produire en continu des "black pellets" à partir des résidus de bois issus des forêts de la région. Le contre-exemple britannique Cette conversion ne fait pas l'unanimité, comme le montre le contre-exemple de la Grande-Bretagne. Les Britanniques y ont cru... et en sont revenus. Verdir des centrales électriques en replaçant le charbon par du bois, permet, certes, d'obtenir un meilleur bilan carbone, mais s'avère une aberration économique et écologique. "La combustion des pellets de bois émet plus de particules fines", rappelle Almuth Enrsting de l'association Biofuelwatch. Surtout, elle nécessite de lourds investissements dans les centrales à charbon existantes pour éviter la corrosion. Au Royaume-Uni, deux centrales ont dû fermer après des incendies. Et sur les quatre unités charbon converties à la biomasse de la centrale de Drax, dans le Yorkshire du Nord, trois sont à l'arrêt. La dernière doit importer massivement des pellets des États-Unis et du Canada. En France, l'allemand Uniper, qui a converti une unité charbon à Gardanne (Bouches-du-Rhône) en unité biomasse brûlant tout type de résidus de bois, est lui confronté à un grave problème de pression sur les ressources forestières locales. Et il n'a plus qu'une autorisation préfectorale temporaire, le temps d'inscrire la centrale dans une économie circulaire locale vertueuse... Excepté aux Antilles, où la conversion à la biomasse des centrales bagasse/charbon ne met pas une trop grande pression sur l'environnement, l'idée semble avoir fait long feu.
</article>
<article titre="Une nature exceptionnelle, l'atout-maître de l'outre-mer" date="2019 01 28" journal="La Gazette des communes, no. 2450">
Plans d'urgence pour la Guyane (2017) et pour Mayotte (2018), et surtout le « Livre bleu outremer » paru en juin : telles sont les dernières actions entreprises en faveur des DOM. Le « Livre bleu » rassemble des propositions issues des assises de l'outre-mer, qui se sont déroulées sur plusieurs mois en 2017 et 2018. Lors de sa présentation, la ministre Annick Girardin avait promis que le « budget outremer » serait « très renforcé ». Pour 2019, il s'élève à 2,5 milliards d'euros, « en hausse de 20 % » par rapport à l'an dernier, selon elle. Le fonds exceptionnel d'investissement est porté à 110 millions d'euros par an à compter de cette année. En outre, sur le quinquennat, 400 millions d'euros seront consacrés au développement économique, et la Caisse des dépôts mobilisera plus de 3 milliards d'euros de prêts et 100 millions d'euros de fonds propres. Le « Livre bleu » ambitionne aussi de sortir d'un modèle économique, hérité du colonialisme, quasi exclusivement centré sur des échanges avec l'Hexagone. Pour lutter contre le chômage de masse des jeunes, 700 millions d'euros sont prévus pour les pactes d'investissement dans les compétences, entre 2018 et 2022. Concernant l'enfance, 6 500 places de crèches doivent être créées sur le quinquennat, en priorité en Guyane, à Mayotte et à La Réunion. Le plan « Girardin » pour Mayotte du 15 mai 2018 projette également 500 millions en faveur de la construction d'écoles primaires et secondaires. Cependant, les acteurs sont sceptiques, car aucun plan n'a vraiment porté ses fruits. Ainsi, à Mayotte, le gouvernement Hollande avait décidé de la construction de 500 classes par an, mais moins de 50 ont été ouvertes.   UNE RICHESSE À CHIFFRER Des élus de Mayotte souhaiteraient que l'Etat engage ce département sur la voie d'un véritable développement économique durable en finançant des infrastructures susceptibles de créer de l'emploi et des richesses : une nouvelle piste pour l'aéroport, l'extension du port de commerce ainsi que le déploiement du haut-débit et d'infrastructures routières. Mais ce développement économique ne doit pas se faire au détriment de la biodiversité, atout majeur des DOM.   Ce patrimoine exceptionnel regroupe 80 % de la biodiversité française. « Si la France est aujourd'hui une force économique présente sur tous les continents, elle le doit à l'outre-mer. Grâce à ces territoires, elle peut aussi se targuer d'être championne en matière de biodiversité », rappelle Sophie Charles, trésorière de l'ACCD'OM et maire (SE) de Saint-Laurent-du-Maroni, selon qui cette richesse méconnue doit être valorisée et chiffrée. Les auteurs du « Livre bleu » souhaitent justement que les outre-mer jouent le rôle de précurseurs dans la mise en oeuvre du développement durable. Ils fixent comme objectif la couverture intégrale des outre-mer par les atlas communaux de la biodiversité, afin d'organiser un « compteur de la biodiversité ». Les créations de deux réserves naturelles nationales sont prévues d'ici à 2022 et, dès 2019, celle d'un fonds spécifique dédié à la biodiversité ultramarine. Pour la première fois, un service à compétence nationale va s'installer dans un DOM. Le service « valorisation économique de la biodiversité » de l'Agence française pour la biodiversité est en création à Cayenne. Il aura vocation à apporter son appui à l'ensemble des territoires ultramarins. Inédit !   AUTONOMIE ÉNERGÉTIQUE Autre atout commun aux DOM : les énergies renouvelables. En 2020, la Guyane verra la mise en service d'une centrale électrique solaire et d'une autre à biomasse alimentée par la combustion de bois provenant d'exploitations forestières locales. En 2017, 64 % de la production électrique de Guyane étaient issus des ressources renouvelables, 36 % à La Réunion et 22 % en Guadeloupe. En Martinique, une nouvelle centrale à biomasse (du groupe Albioma), opérationnelle depuis octobre, doit produire 40 mégawatts à partir de bagasse (résidu fibreux de la canne à sucre), de biomasse d'origine végétale et de résidus de bois d'oeuvre.   La Réunion vise l'autonomie énergétique en 2050. Particulièrement novatrice dans ce domaine, elle travaille sur de multiples sources d'énergies renouvelables : hydro électrique, bio mass e (bagasse) et photovoltaïque (+ 44 % entre 2011 et 2015). Une expérimentation inédite, 100 % solaire avec stockage à hydrogène, a été lancée en 2018 au cirque de Mafate (commune de Saint-Paul). D'autres projets portent sur les énergies marines : houlomotrice (issue des vagues) et thermique. L'agriculture, l'agroalimentaire, la forêt et le bois, la pêche et l'aquaculture sont des secteurs clés de l'économie ultramarine. Développées autour de quelques productions historiques (banane, sucre et rhum), ces filières sont des outils essentiels pour réduire la dépendance des DOM vis-à-vis des importations et gagner en autonomie alimentaire.   L'ÉCOTOURISME : GROS POTENTIEL Aux Antilles, à la suite des ouragans Irma et Maria de 2017, la production de bananes a été nulle cette année-là. En Guyane, la libéralisation du foncier agricole, mais aussi les restrictions d'importation des produits d'élevage (oeufs et porcs), sont à voir comme des opportunités pour encourager l'offre locale et permettre à la population de se nourrir à un coût moins élevé. L'exploitation de la forêt amazonienne fait l'objet de recherches pour développer une activité à faible impact environnemental. Le tourisme reste une valeur sûre : grâce à une faune et une flore exceptionnelles, l'écotourisme offre de belles opportunités.   Produire et recycler sur place, asseoir une zone économique locale, mieux considérer l'atout qu'est la biodiversité et former les jeunes sont des enjeux majeurs pour le développement durable de l'outre-mer.
</article>
<article titre="Amérique Latine : Trescal, Thales, Alstom, Albioma, Eramet..." date="2019 01 18" journal="CFNEWS (site web)">
Trescal, Thales, Alstom, Albioma, Eramet...Chaque semaine, toute l'actualité du capital-investissement en Amérique Latine.     Produits &amp; Services industriels : Trescal - Metrology (Brésil)    Le spécialiste des services de métrologie, Trescal (voir sa fiche et deal-list dans notre référentiel) , détenu par le fonds canadien Omers depuis novembre 2017, intègre dans son périmètre le belge LBT Testing and Calibration, le français C2M et le brésilien Metrology qui lui apportent un chiffre d'affaires additionnel de 3 M€. Basé à Vitoria et dirigé par Filippe Aderne Maia, qui restera dans ses fonctions, Metrology emploie huit collaborateurs et est l'un des deux laboratoires d'étalonnage privés et accrédités situés dans l'État d'Espirito Santo. Cette opération, dont les termes financiers n'ont pas été divulgués, permet à Trescal d'élargir sa couverture technique et géographique et renforce sa position de leader sur le marché brésilien des services de métrologie. Implanté dans 22 pays sous la houlette de son président Olivier Delrieu, le groupe français possède plus de 135 laboratoires de métrologie dans le monde pour un chiffre d'affaires pro forma de 325 M€. Voir la fiche de l'opération LBO Trescal - Metrology.   Automobile, aérospace, construction navale : Thales (Mexique)    L'équipementier électronique français Thales (voir sa fiche et deal-list) annonce un contrat avec la marine mexicaine, par lequel il a fourni et intégré les systèmes de défense du nouveau patrouilleur océanique ARM Reformador, premier navire de la classe POLA (Patrullera Oceanica de Largo Alcance) de 2 600 tonnes. C'est le premier contrat naval pour Thales au Mexique, mais aussi le premier contrat export pour le système Gatekeeper. Cette suite complète permet à l'équipage d'assurer des missions de surveillance et de sécurité. Aux termes d'un accord de transfert de technologie, les ingénieurs de la marine ont participé à l'ingénierie, à l'intégration et aux essais des systèmes.     Produits &amp; Services industriels : Alstom (Argentine)    Guillermo Dietrich, ministre des transports de l'Argentine et Ernesto Garberoglio, directeur général d'Alstom Argentine    Alstom (voir sa fiche et deal-list) , dans le cadre d'un consortium avec le groupe argentin Pose, a signé fin décembre un contrat de près de 90 M€ pour moderniser l'actuel système de signalisation du chemin de fer General Roca. La part d'Alstom dans ce contrat, qui a été attribué par le ministère des Transports de l'Argentine et fait partie du programme de restauration du chemin de fer de la métropole de Buenos Aires, s'élève à près de 50 M€. Le Chemin de fer General Roca est le plus étendu des sept lignes de trains de banlieue de la Métropole de Buenos Aires. Il comprend 237 km de voies et transporte 600 000 passagers par jour. Les travaux seront réalisés sur les embranchements Avellaneda-La Plata, Temperley-Bosques-Berazategui et Bosques-Villa Elisa. Alstom fournira sa technologie d'enclenchement électronique éprouvée Smartlock, le système de gestion centralisée du trafic Iconis et assurera l'installation du système ATS (Automatic Train Stop). De plus, le groupe tricolore prendra en charge la conception, l'ingénierie, les essais en usine et sur le terrain, l'intégration technique, la mise en service et la garantie des systèmes, les systèmes de supervision via le centre de trafic local, le contrôle centralisé du trafic et le poste central des opérations, les barrières de passages à niveau automatiques et pour piétons, le réseau de câbles à fibre optique et en cuivre ainsi que le système de détection et de protection incendie du local de signalisation.     Energie &amp; Utilities : Albioma (Brésil)    Le producteur d'énergie renouvelable Albioma (voir sa fiche et deal-list) a finalisé l'acquisition de 60 % d'une unité de cogénération de bagasse au Brésil auprès du groupe Jalles Machado. Localisé à Goianésia et adossé à une sucrerie-distillerie le site profite d'une capacité de broyage atteignant 2,8 millions de tonnes de cannes par an. Pour rappel, le groupe français avait signé le 18 décembre 2017 un accord prévoyant notamment la rénovation des chaudières existantes ainsi que l'installation d'une nouvelle turbine de 25 MW, portant la capacité totale du site à 65 MW. Grâce à l'amélioration des rendements de la cogénération et à la réduction de la consommation de vapeur de la sucrerie, l'énergie exportée sur le réseau de distribution devrait ainsi atteindre 145 GWh par an à compter de la campagne 2019, soit près de 2 fois plus qu'actuellement.     Matières premières &amp; industries de base : Eramet (Argentine)    Gisement Eramet Argentine    Le groupe métallurgique et minier coté Eramet (voir sa fiche et deal-list) , fort de 3,2 Md€ de chiffre d'affaires en 2017 va investir près de 500 M€ dans le projet de production de lithium en Argentine. En 2012 le producteur français de métaux de base a découvert dans la province de Salta, au Nord-Ouest de l'Argentine, le gisement de lithium Centenario-Ratones et en détient les droits miniers depuis avril 2014. Situé à 3 800 mètres d'altitude ce gisement contient des ressources estimées à 9 millions de tonnes de carbonate de lithium et doit permettre une production industrielle de 24 000 tonnes par an, soit environ 10 % du marché actuel, pendant plusieurs décennies. La décision d'investissement est prévue au premier semestre pour un démarrage de la production fin 2021.     Une information à nous soumettre pour ce Bulletin Amérique Latine ? Écrivez-nous à : dalila.weygang@cfnews.net   Retrouver les autres Chroniques de CFNEWS :     Tous les précédents bulletins Amérique Latine Tous les bulletins Asie Aussi sur Facebook , Twitter , Google+ et LinkedIn  Cet article est paru dans CFNEWS (site web)
</article>
<article titre="Atlas Maurice" date="2019 01 17" journal="Le Monde">
Quelques jours après les célébrations du cinquantenaire de l'indépendance, en mars 2018, la présidente mauricienne Ameenah Gurib-Fakim a été poussée à la démission, emportée par le scandale « Platinum Card . La chef d'Etat à l'époque la seule femme d'Afrique à occuper la magistrature suprême a fait 25 000 euros d'achat avec une carte bancaire qui lui avait été ­remise par une ONG, dont le directeur est un milliardaire angolais sulfureux, poursuivi pour détournement de fonds. Scientifique de renommée mondiale, Mme Gurib-Fakim a plaidé la méprise, mais a quand même dû s'incliner. Le vice-président Paramasivum Pillay ­Vyapoory, plus communément ­appelé Barlen Vyapoory, assure l'intérim jusqu'à la désignation d'un nouveau président par l'Assemblée. L'île est parvenue, en cinquante ans, à s'établir comme un modèle de stabilité politique et de respect de l'Etat de droit en Afrique. En juillet, deux membres du gouvernement ont ainsi dû démissionner après la publication d'un rapport d'enquête sur un trafic de drogue à grande échelle : politiques, avocats, policiers, gardiens de prison et ­trafiquants entretiennent des liens troubles qui agitent depuis longtemps la société mauricienne, comme le souligne ce rapport dévastateur.   L'économie reste robuste dans son ensemble, avec près de 4 % de croissance en 2018. Le secteur de la canne à sucre, la principale culture de l'île, souffre en revanche de la baisse des cours mondiaux du ­sucre. En huit ans, le nombre de ­petits planteurs a été divisé par deux. Un nouveau débouché en plein essor est la production d'électricité : 14 % des besoins de l'île sont déjà couverts par la combustion de la bagasse, un résidu de canne.
</article>
<article titre="Gestion : Amilton Small Caps mise sur Albioma" date="2019 01 9" journal="Boursier (site web)">
Piloté par Harry Wolhandler, Amilton Small Caps I est un fonds majoritairement investi dans des valeurs de petites et moyennes capitalisations françaises, dont l'objectif est de réaliser une performance nette de frais supérieure à 7% par an, sur un horizon de placement de cinq ans. Eligible au PEA, cet OPCVM restreint son champ d'intervention aux capitalisations comprises entre 20 millions et 7 milliards d'euros lors de l'investissement, de tous les secteurs. La gestion de l'OPCVM consiste en un choix rigoureux d'actions à fort potentiel. Le pilotage du fonds se veut actif et évolutif en fonction du contexte économique et boursier avec une approche visant à identifier les secteurs et thématiques porteurs à moyen terme. Amilton Small Caps I fonde sa philosophie sur le "Stock picking", qui est basé sur l'analyse fondamentale des sociétés, les rencontres avec leurs dirigeants et l'identification des catalyseurs potentiels    Amilton Small Caps I a fini l'année en baisse de -18,05%, après un quatrième trimestre très négatif pour les marchés actions des pays développés, contre un CAC Mid &amp; Small NR en baisse de -20,22% et un CAC Small NR en repli de -26,00%. Les marchés actions ont souffert en 2018 des perturbations politiques (conflit commercial, Brexit, Italie, Gilets Jaunes) et du ralentissement macroéconomique mondial dans un environnement de normalisation monétaire de la FED. Dans ce contexte d'aversion au risque et de dynamique des flux très négative, les grandes capitalisations ont surperformé les petites et moyennes valeurs après une année 2017 très positive pour cette classe d'actif.   Du côté des nouveautés, Amilton Small Caps I a entré Albioma en portefeuille après l'annonce de la cession de la participation résiduelle d'Altamir à la société Impala (également actionnaire de Neoen) détenue par l'entrepreneur Jacques Veyrat au prix de 18,60 euros. "Cette opération a mis en évidence la sous-évaluation en Bourse d'Albioma et l'attrait spéculatif de la valeur qui, quand elle aura transformé la plupart de ses centrales bagasse-charbon en bagasse-biomasse d'ici 2023, affichera un mix énergétique décarbonné plus attractif pour d'éventuels prédateurs", souligne le gérant. Harry Wolhandler vise 21 euros sur cette valeur.
</article>
<article titre="LA RÉUNION, TERRITOIRE D'EXPÉRIMENTATION ET DE PRODUCTION" date="2018 12 28" journal="Les Cahiers Techniques du Bâtiment, no. 374">
L'objectif est on ne peut plus clair :  « Nous visons le 100 % d'énergies renouvelables dans le mix électrique en 2030, avec une répartition équilibrée entre biomasse, solaire, hydraulique et éolien »,  Explique Alin Guezello, conseiller régional en charge de l'énergie. Photovoltaïque, énergie marine, géothermie, hydroélectricité, éolien terrestre et valorisation énergétique des déchets... Toutes les pistes sont encouragées, testées, développées et les efforts paient : la région Réunion est aujourd'hui celle qui produit le plus d'électricité à partir d'énergies renouvelables. En 2016, sa production annuelle (2 943,6 GWh) est réalisée avec 34 % de renouvelables. Un bon score, même si les 66 % restants proviennent d'énergies fossiles importées. Majoritairement du charbon (39 %, soit 1 138,3 GWh), mais aussi du fioul, de l'essence et du gaz. Et pour cause, l'île, en tant que ZNI (Zone non interconnectée), n'est pas reliée au réseau continental des centrales nucléaires françaises. Vers un cadastre solaire Cet enjeu stratégique s'inscrit dans le droit fil de la Loi relative à la transition énergétique pour la croissance verte de 2015, qui vise, pour les DOM, un seuil de 50 % de l'énergie consommée à partir de sources renouvelables en 2020 et, donc, à l'horizon 2030, une autonomie énergétique.  « Les objectifs pour la partie électrique sont très ambitieux, mais réfléchis et partagés par l'ensemble des acteurs De la Gouvernance de l' énergie,  Rassure Pierre-Yves Ezavin, directeur technique de SPL Énergies, une entité créée en juillet 2013 qui accompagne les collectivités dans le développement de projets énergétiques.  Nous les avons présentés au secrétaire d 'État, Sébastien Lecornu, en visite en juin, qui mise fortement sur le solaire. »  D'ailleurs, lors de sa visite, le lancement d'une étude sur le patrimoine bâti des entreprises et des particuliers a été annoncé. Il s'agit d'un  « cadastre solaire, une sorte de cartographie des bâtiments, toitures et parcelles non-bâties et non-agricoles disponibles pouvant accueillir des panneaux photovoltaïques »,  Détaille Alin Guezello.   Depuis, dans le contexte du débat sur la programmation pluriannuelle pour l'énergie (PPE), le secrétaire d'État a lancé le plan « Place au solaire », qui se veut une mobilisation générale pour le photovoltaïque et le solaire thermique. Ainsi, les prévisions pluriannuelles pour La Réunion visent une augmentation à échéance 2023 de 63 MW/an pour le photovoltaïque (y compris en autoconsommation) et une augmentation de 58,5 MW/an en photovoltaïque et stockage. De nouveaux appels d'offres de la Commission de régulation de l'énergie seront bientôt ouverts dans le cadre des objectifs de la PPE.   Le photovoltaïque décolle à La Réunion Depuis le début des années 2010, le photovoltaïque a, en effet, pris une place grandissante dans l'île. Avec une production de 186,4 MW, et 44 000 panneaux installés, il représente près de 10 % du mix énergétique réunionnais. Un chiffre encourageant, mais loin du potentiel estimé entre 800 et 1 000 MW et visé pour 2030 par le conseil régional. Cela étant, le territoire constitue la région française qui compte le plus d'installations PV.  « Le photovoltaïque a décollé à La Réunion entre 2006 et 2011 »,  Note Charly Bell, dirigeant de Solar Trade, société spécialisée dans les solutions photovoltaïques en autoconsommation. Selon lui,  « la baisse du coût des équipements photovoltaïques permet aujourd'hui d'atteindre la parité réseau, avec un prix de revient client de 0,07 € par kilowatt/heure solaire, sans stockage ».  L'île a, par ailleurs, développé très tôt des installations maîtrisant le PV et le stockage. L'expérimentation Pégase (Prévision des énergies renouvelables et garantie active par le stockage d'énergie), lancée en 2010 à Saint-André, a, par exemple, permis de coupler une batterie sodium-soufre (Na-S) de 1 MW à une ferme photovoltaïque. C'est l'une des plus grosses capacités de stockage d'électricité existant en Europe (6 000 panneaux solaires sur une superficie de 10 000 m²).   Autre exemple avec la ferme photovoltaïque du cirque montagneux de Mafate. La start-up française Powidian, à la demande du groupe EDF-SEI, expérimente, depuis fin avril 2017, une ferme photovoltaïque couplée à un système de stockage par batterie à hydrogène. L'ensemble permet de rendre autonome en énergie ce hameau d'une soixantaine de foyers et de bâtiments publics. D'autres installations de ce type devraient voir le jour dans les prochaines années.   Doubler les installations Pour faire grimper le nombre d'installations photovoltaïques, la Région octroie aux particuliers ou aux agriculteurs une prime de 1 000 à 6 000 € pour l'achat d'une centrale PV d'une puissance de 1 à 9 kWc avec ou sans stockage. Elle a également mis en oeuvre un projet expérimental sur les ombrières photovoltaïques et compte en installer sur des sites de son patrimoine, à Saint-Denis et à Saint-Pierre, afin d'assurer la recharge d'une flotte de voitures et de vélos à assistance électrique (225 bornes avec recharge  Via  énergies renouvelables devraient être installées jusqu'en 2023). Pour Alin Guezello, le PV devrait être au même niveau que les chauffe-eau solaires, très développés sur l'île, avec 60 % des maisons qui en sont équipées. Le 160 000 e client utilisant un chauffe-eau solaire a d'ailleurs été symboliquement salué en juin par Sébastien Lecornu.   La Région Réunion encourage leur développement  Via  Notamment un financement à hauteur de 80 % du prix de l'installation (programme Eco Solidaires) pour les familles en précarité énergétique. Elle pourrait même aller plus loin et inscrire, pour chaque maison neuve, l'obligation de s'équiper d'un chauffe-eau solaire thermique.   Développement de l'éolien, de l'hydraulique et de la biomasse Les expérimentations sur l'hydraulique et la biomasse - qui représentent à elles seules près du tiers des apports énergétiques sur l'île (24 % de la production électrique) - ou sur l'éolien vont également bon train.   « Nous souhaitons remplacer le charbon par la biomasse d'ici à 2030 »,  Explique Pierre-Yves Ezavin. Cela pourrait se faire  Via  Le bois énergie ou la bagasse, un résidu du broyage de la canne à sucre, brûlé pour produire de l'électricité - une tonne de canne produit environ 300 kg de bagasse pour une valeur calorifique de 7 900 kJ/kg (16 000 kJ/ kg pour du bois sec). L'île compte deux centrales de cogénération à combustion hybride bagasse/charbon : celle de Bois-Rouge (100 MW), une des premières mondiales inaugurée en 1992 dans l'Est de l'île, et celle de Gol, à Saint-Louis (110 MW) dans le Sud. À elle seule, la bagasse ainsi que les résidus fibreux de la canne à sucre produisent jusqu'à 8 % de l'électricité du territoire (244,1 GWh), soit la consommation de 91 000 habitants, et permet d'éviter l'importation de 138 000 t de charbon.   L'énergie hydraulique, quant à elle, contribue à hauteur de 16 % de la production électrique (465 GWh).  « On pourrait atteindre les 30 %,  Regrette Alin Guezello.  Il nous reste à générer 14 % de production supplémentaire d'ici à 2020. »  Six centrales hydrauliques sont installées à La Réunion, dont celles de la Rivière de l'Est (79,2 MW) et de Takamaka (41,4 MW). Enfin, le schéma régional pour le développement de l'énergie éolienne adopté en 2015 et réalisé par SPL Énergies Réunion vise un objectif de l'ordre de 35 MW de puissance installée en 2020 et de 50 MW en 2030 sans considérer les évolutions technologiques des éoliennes. Une ressource qui reste cependant marginale et coûteuse : les cyclones et tempêtes tropicales impliquent des installations lourdes de type haubans et mâts rabattables.   Exploiter les températures sous-marines Des expérimentations sont en cours un peu partout, comme des stations de transfert d'eaux pompées, du stockage à air comprimé, de l'éolien off shore en condition cycloniques... Parmi les tests en cours, l'IUT de Saint-Pierre compte ainsi un Prototype à terre exploitant l'énergie thermique des mers (PAT ETM), installé depuis 2012, permet la production continue d'électricité grâce à la différence de température entre eaux de surface (à 25 °C) et eaux profondes (5 °C à 1 000 mètres). Ces eaux alimentent un système thermodynamique et modifient l'état d'un fluide de travail qui entraîne à son tour une turbine pour la production électrique  .  Cette énergie renouvelable pourrait, à terme, remplacer la production à partir du charbon. Des types d'énergies marines exploitant la houle ont aussi fait l'objet de recherches, comme avec les projets Pelamis et Ceto, désormais à l'arrêt. Ou encore les futurs Swac, à Sainte-Marie et au CHU de Saint-Pierre, qui produit de la climatisation grâce à l'eau de mer des profondeurs  (lire encadré ci-contre). Gérer l'accroissement de la population D'autres énergies renouvelables sont étudiées, comme le biogaz à base de déchets agricoles, avec, par exemple, des unités de méthanisation en voie humide en cours d'installation sur le lycée agricole de Saint-Joseph qui permettrait de sécher des balles de foin et de produire en autoconsomation pour le lycée une partie de sa consommation électrique  Via  Un moteur de 15 kWélec.   Sur le même principe, le biogaz issu de déchets ménagers est également exploité dans deux installations : le centre d'enfouissement technique des déchets de la rivière Saint-Étienne depuis 2008 (2 MW) et celui de Sainte-Suzanne (1,6 MW). Le potentiel est important. Il pourrait permettre de produire au moins 7 MW de puissance, mais la filière est à structurer.   Toutefois, si La Réunion veut maintenir le cap et les objectifs de 100 % d'ENR d'ici à 2030, elle devra obligatoirement prendre en compte l'accroissement de sa population. Du fait de l'augmentation du nombre d'habitants entre 2010 à 2016, la production électrique a ainsi augmenté de 1,5 % par an en moyenne et la part des énergies renouvelables dans cette production n'a pas suivi : elle a baissé de 47 % à 34 %. La tendance démographique ne devrait pas s'infléchir : La Réunion compte 840 000 habitants aujourd'hui, ils seront 1 million en 2040.
</article>
<article titre="PAS D'AUTONOMIE ÉNERGÉTIQUE SANS EFFICACITÉ DU BÂTI" date="2018 12 28" journal="Les Cahiers Techniques du Bâtiment, no. 374">
Pour atteindre l'autonomie électrique en 2030, il faudra repenser la manière de construire. L'IUT de Saint-Pierre, L'îlet du Centre à Saint-Pierre, l'école d'ingénieurs de l'Esiroi, la médiathèque de Saint-Leu, l'amphithéâtre de l'université de Saint-Denis, le groupe scolaire de Bois d'olives ou les extensions de l'aéroport Réunion Roland-Garros constituent autant de projets d'un nouveau genre qui promeuvent une approche bioclimatique et une bonne efficacité énergétique du bâtiment voire même d'un quartier. L'expérimentation de Coeur de ville à La Possession est assez représentative. Cet écoquartier de 34 hectares, conçu avec l'équipe d'urbanistes LEU Réunion, est en cours de construction : lancé en 2013, il devrait être livré en 2025. Le projet met en oeuvre plusieurs démarches intéressantes : économies d'énergie et confort thermique via une optimisation de la ventilation naturelle à l'échelle de la ville et des bâtiments, utilisation de matériaux locaux et mise en oeuvre d'une forêt urbaine intégrée pour la gestion des eaux pluviales et du climat urbain, et installation d'énergies renouvelables. À cela s'ajoutent un programme de gestion des déchets (zéro déchet vert) et des eaux pluviales ainsi que la mise en place d'un  Smart grid  Qui permet aux habitants - associés au projet tout le long de sa réalisation - de suivre leur consommation d'eau et d'électricité. Tout est pensé pour que les logements atteignent une consommation énergétique maximale de 20 kWh/m²sdp/an.   La réflexion a été poussée à l'échelle du quartier : il est ainsi possible de suivre et de piloter la consommation d'eau, d'électricité. Les installations électriques sont mutualisées entre bureaux, commerces et logements. Même le parking du centre commercial est mutualisé et connecté. Une première à La Réunion, qui permet de ne pas surdimensionner le parking. Surtout,  « tout le dispositif d'écologie urbaine et de maîtrise énergétique est inscrit dans le corps des outils de l'urbanisme, comme le PLU, le référentiel DD, etc. Des objectifs chiffrés sont des obligations réglementaires, afin d'aller bien au delà des déclamations et de permettre un saut qualitatif »,  Explique Michel Reynaud, de l'agence LEU Reunion.   Écocité, la ville de demain ? Autre projet en cours d'expérimentation : l'Écocité « ultramarine, insulaire et tropicale ». La France compte treize Écocités labellisées dont celle-ci. Portée par les communes du Territoire de la Côte Ouest (TCO) sur un périmètre d'environ 5 000 hectares, c'est de la construction d'une nouvelle ville résiliente et économe dont il est question. Elle accueillera 35 000 logements en 2045.  « Ce site non urbanisé est un beau terrain de jeu »,  Indiquait Teddy Soret, directeur de cabinet du TCO lors d'un séminaire sur la  Smart city  En mai 2017. Là encore, se manifeste une approche de ville tropicale durable où la végétation apportera la fraîcheur et le confort. Ville économe et résiliente avec le choix d'une architecture bioclimatique, de la lutte contre les îlots de chaleur, de l'approche aéraulique prenant en compte les vents et les ombrages, de la mise en place d'un réseau de froid, d'une gestion de l'eau et des déchets ou de l'emploi d'énergies renouvelables. Mais aussi ville connectée puisqu'elle sera dotée d'un  Microgrid  Afin d'optimiser les éclairages et les piloter.   Utiliser les ressources locales Une réflexion tout juste naissante est consacrée aux matériaux de construction utilisés sur l'île, essentiellement du parpaing et de la tôle. Le récent Campus des métiers et des qualifications dédié au génie civil et à l'écoconstruction en milieu tropical, labellisé en février 2017, devrait bientôt lancer deux groupes de travail. L'un sur les bétons végétaux à partir de ressources disponibles en zones tropicales humides, notamment la bagasse. L'autre sur le bâti tropical et la construction biosourcée. Un Mooc devrait aussi être réalisé sur ce sujet, notamment sur la question du milieu insulaire et de l'usage de matériaux locaux. Il viendrait compléter celui de l'Ademe  « Construire durable en zone tropicale humide »  Et pourrait être un début de modélisation de construction tropicale type à La Réunion applicables aux autres zones tropicales humides en raisonnant hydrothermie, impact sur la biodiversité, ventilation et utilisation de matériaux locaux. Ces deux groupes de travail portés par le Campus mettront en synergie des appareils de formation, des entreprises du BTP et les laboratoires de recherche (Piment et Cirbat). Nombre d'études sont par ailleurs en cours sur d'autres ressources locales et même sur celles, moins naturelles, mais tout autant disponibles, que constituent les containers, les palettes, le verre et les plastiques...
</article>
<article titre="Equipements - Concassage en place à la Réunion" date="2018 12 28" journal="Chantiers de France (site web)">
Dans le cadre de la requalification d'un moulin à sucre historique dans l'île de La Réunion, le recours à un godet concasseur s'est avéré nécessaire sur le chantier. Monté sur une pelle compacte, l'équipement a permis de préparer in situ les matériaux.   Au coeur de La Saline, à Belle Vue, se trouve un moulin à sucre historique, construit avant 1874 et considéré comme le coeur économique de l'île. En 1969, l'usine ferme car elle ne peut pas faire face à la demande de production requise. Depuis, le bâtiment a été recouvert de végétation et de déchets, entraînant sa dégradation. Souhaitant mettre en valeur le patrimoine historique réunionnais, l'administration locale a engagé les travaux de remise en état cette zone industrielle désaffectée afin de la transformer en un site culturel et économique pour la communauté. C'est dans ce contexte qu'ont été effectués les travaux de déblaiement et de mise en sécurité de l'édifice. Pour mener à bien le projet, un godet concasseur MB-C50 monté sur une pelle Kobelco afin de réduire les matériaux inertes extraits de l'ancienne usine a été utilisé. L'objectif : pouvoir accéder facilement à l'intérieur du bâtiment afin de procéder à un état des lieux et pouvoir envisager les travaux nécessaire à sa réhabilitation.   Recyclage    Rien n'a été perdu lors des travaux qui donneront une seconde vie à l'ancienne manufacture et qui font partie du projet de développement de la ville de La Saline : le godet concasseur a recyclé les briques, le béton armé et les pierres à une taille optimale pour pouvoir être réutilisées directement sur le site. Tout cela à proximité des écoles publiques et de la piscine municipale, sans aucun obstacle pour les activités des riverains.   En effet, tous les équipements du constructeur italien MB permettent une réduction notable des émissions de gaz à effet de serre et tous les inconvénients qui entourent le transport des matières (poussières, bruits, noria de camions..). La preuve, s'il en fallait une que les réunionnais savent comment optimiser toutes les ressources disponibles à l'instar de l'industrie sucrière. Les dérivés de ses produits jouent également un rôle clé. À partir de la matière première, en plus du sucre, on obtient du rhum à partir de la mélasse, de l'électricité à partir de la bagasse, des engrais à partir de la mousse et du fourrage pour le bétail.   J-N.O  Cet article est paru dans Chantiers de France (site web)
</article>
<article titre="INTERVIEW Didier Robert, Président de la Région La Réunion - « JE PARIE SUR L'ÉVOLUTION DE LA TECHNIQUE »" date="2018 12 28" journal="Les Cahiers Techniques du Bâtiment, no. 374">
CTB - Vous voulez faire de l'île un modèle en matière d'énergies renouvelables ? Nous souhaitons atteindre les 100 % d'énergies renouvelables d'ici à 2030. L'île est une région propice à leur développement, elles représentent 34 % du mix énergétique. Nous misons surtout sur l'énergie hydraulique, le photovoltaïque ou la biomasse. L'hydraulique représente un tiers des ENR. Deux barrages dans l'Est sont en fonctionnement, et nous envisageons d'en construire un troisième. Concernant la biomasse, des développements concluants à partir de bagasse (broyats de canne à sucre) sont testés à Saint-Denis. Du biogaz à partir de végétaux et de déchets ménagers est à l'étude. L'objectif, d'ici à 2022 à 2023, est de ne plus les enfouir, mais de les valoriser  Via  Un tri plus fin et en adaptant les usines à brûler pour remplacer le charbon par des déchets alimentaires. CTB - Concernant l'énergie solaire ? Nous souhaitons faire progresser le nombre de mini-fermes PV dotées d'un système de stockage ainsi que l'autoconsommation pour chaque famille. Quelque 3 500 familles en sont désormais équipées, pour un objectif de 200 000. Ce n'est pas aberrant : aujourd'hui 160 000 familles disposent d'un chauffe-eau solaire, alors pourquoi ne pas en faire autant avec le panneau PV ? Des expérimentations sont en cours pour développer l'énergie solaire : le groupe Akuo expérimente, par exemple, une ferme PV avec un stockage intégré. Je parie sur l'évolution de la technique. C'est tout cet ensemble qui permettra d'octroyer à La Réunion son autonomie énergétique.   CTB - Vous souhaitez faire de La Réunion un exemple hors du territoire ? Tout à fait. L'île dispose d'un véritable savoir-faire et nous voulons être un exemple de réussite pour tout l'océan Indien. Nous sommes déjà présents en Chine, à l'île Maurice, à Madagascar, aux Seychelles, ou en Inde dans le numérique, les services et les biens de consommation. Alors pourquoi pas pour l'énergie ?   CTB - Vous avez d'autres priorités, comme le numérique ou de gros investissements, comme la nouvelle route du littoral... La Région a bénéficié d'une enveloppe de 2,2 Mrds € par les accords Matignon II signés en 2010. Elle permet de financer en partie la nouvelle route du littoral, le transport public réunionnais, avec le TransEcoExpress, et la modernisation des aéroports de Saint-Denis et de Saint-Pierre. Par ailleurs, chaque année, 60 % du budget de la région est dédié à l'investissement, soit 600 M €/an sur 1 milliard. Nous avons prévu un programme routier et la construction de trois lycées autour des métiers de la mer, du tourisme, du numérique et de l'audiovisuel. Il est également question de renforcer notre réseau de haut débit. Nous serons, en 2022, une des premières régions de France entièrement couverte par la fibre optique grâce au câble Metiss. Être câblé nous permettra de développer la  Smart city.  À l'est de l'île par exemple, autour de Saint-André, le projet du pôle industriel énergétique de bois rouge comprendra des  Green data centers , une usine de valorisation énergétique des déchets, une école d'ingénieurs et une unité dédiée à l'énergie thermique des mers (Swac).
</article>
<article titre="Albioma: Portzamparc reste à l'achat" date="2018 12 27" journal="Cercle Finance (site web)">
Portzamparc confirme son conseil d'achat sur Albioma après que l'énergéticien a finalisé une opération au Brésil. L'objectif reste fixé à 22,5 euros.   Albioma a mis la main sur 60% d'une usine de cogénération utilisant la bagasse, résidu de canne à sucre, qui est adossée à une sucrerie-distillerie. Les 40% restants sont aux mains du groupe agroindustriel local Jalles Machado, qu'Albioma connait déjà. Portzamparc estime le prix d'acquisition entre 30 et 35 millions d'euros, puis les investissements nécessaires à 25 millions, le tout pour un EBITDA 2019 estimé à 6,5 millions. 'Une nouvelle opération dans les six mois est possible et sera un nouveau catalyseur sur le titre', commentent les analystes.    Copyright (c) 2018 CercleFinance.com. Tous droits réservés.   Les informations et analyses diffusées par Cercle Finance ne constituent qu'une aide à la décision pour les investisseurs. La responsabilité de Cercle Finance ne peut être retenue directement ou indirectement suite à l'utilisation des informations et analyses par les lecteurs. Il est recommandé à toute personne non avertie de consulter un conseiller professionnel avant tout investissement. Ces informations indicatives ne constituent en aucune manière une incitation à vendre ou une sollicitation à acheter.
</article>
<article titre="Albioma finalise son acquisition au Brésil" date="2018 12 27" journal="Investir (site web)">
AchatSuite à la cession par Altamir du solde de sa participation dans Albioma (environ 5,5% du capital) au profit d'Impala, structure d'investissement de Jacques Veyrat, Albioma a bondi d'une vingtaine de pour cent ces derniers jours en Bourse, s'ajustant ainsi sur le prix de l'opération. A l'issue de cette transaction et après plusieurs achats de titres réalisés sur le marché, Impala détient 6,5% du capital d'Albioma et devient ainsi le premier actionnaire du spécialiste de la production d'électricité renouvelable. Frédéric Moyne, directeur général d'Albioma, a salué cette prise de participation et déclaré « nous sommes certains que le groupe pourra s'appuyer sur l'expérience sectorielle et financière d'Impala pour poursuivre avec succès la mise en oeuvre de sa stratégie, tant en France qu'à l'international » . Un virage du charbon vers la biomasse Pa ailleurs, la société a annoncé ce 20 décembre avoir signé l'avenant au contrat de vente d'électricité entre EDF et sa centrale Albioma Caraïbes (Guadeloupe), un mois après avoir reçu l'aval de la Commission de la Régulation de l'Énergie (CRE), permettant la conversion à la biomasse de cette centrale 100% charbon. Les travaux de conversion débuteront en 2019 pour que la centrale fonctionne 100% à la biomasse avant fin 2020. Ils permettront ainsi à la part renouvelable du mix énergétique de la Guadeloupe de passer de 20% à 35% à terme.   Autre bonne nouvelle, au Brésil, Albioma a signé le contrat définitif permettant l'acquisition de 60% de l'unité de cogénération bagasse (résidu fibreux de la canne à sucre) adossée à la sucrerie-distillerie localisée à Goianésia, un an après l'annonce de son projet de rénovation des chaudières existantes ainsi que l'installation d'une nouvelle turbine de 25 MW (pour porter la capacité totale du site à 65 MW). Grâce à l'amélioration des rendements de la cogénération et à la réduction de la consommation de vapeur de la sucrerie, l'énergie exportée sur le réseau de distribution devrait ainsi atteindre 145 GWh par an à compter de la campagne 2019, soit près de deux fois plus qu'actuellement. Cette acquisition offre une bonne visibilité à Albioma, étant donné que plus de 80% des ventes de cette société sont d'ores et déjà sécurisées sur le marché réglementé à travers un appel d'offres.   Même si d'autres opérations de croissance externe restent envisageables, elles seront toutefois limitées en montant au vu de l'endettement du groupe, qui atteint plus de 140% des fonds propres. Le cabinet d'études Portzamparc, aussi à l'achat, jugeant qu'Albioma « connaît désormais suffisamment bien l'écosystème », n'exclut pas la construction d'une centrale plutôt qu'une nouvelle acquisition.   Investir
</article>
<article titre="Albioma: acquiert 60% d'une cogénération au Brésil" date="2018 12 24" journal="ABC Bourse (site web)">
(CercleFinance.com) - Albioma annonce avoir signé avec Jalles Machado l'acquisition de 60% de l'unité de cogénération bagasse adossée à une sucrerie-distillerie localisée à Goianésia au Brésil, dont la capacité de broyage annuelle atteint 2,8 millions de tonnes de canne.   Pour rappel, le groupe français avait signé le 18 décembre 2017 un accord, prévoyant notamment la rénovation des chaudières existantes ainsi que l'installation d'une nouvelle turbine de 25 MW, portant la capacité totale du site à 65 MW. L'énergie exportée sur le réseau de distribution devrait ainsi atteindre 145 GWh par an à compter de la campagne 2019, soit près de deux fois plus qu'actuellement. Plus de 80% des ventes d'énergie sont d'ores et déjà sécurisés sur le marché règlementé.   Copyright © 2018 CercleFinance.com. Tous droits réservés.  Cet article est paru dans ABC Bourse (site web)
</article>
<article titre="Préouverture Paris : une baisse du CAC40 au pied du sapin" date="2018 12 24" journal="Boursier (site web)">
LA TENDANCE   La dernière séance de la Bourse de Paris avant la coupure de Noël devrait démarrer dans le rouge, après la clôture finalement quasi stable de vendredi à Paris. Paris n'a pas encaissé la forte baisse de Wall Street vendredi. La Bourse américaine a chuté après la clôture parisienne, affichant sa pire semaine depuis la crise de 2008! Signe de l'inquiétude que suscite désormais le recul des marchés, le secrétaire au Trésor américain Steven Mnuchin a déclaré qu'il s'était entretenu dimanche avec les principaux banquiers américains afin de tenter de calmer la nervosité des marchés financiers. Les plus grands banquiers ont tous été appelés pour s'assurer qu'ils disposaient d'assez de liquidités pour continuer à prêter à l'économie.   Steven Mnuchin a par ailleurs dû se fendre d'une déclaration concernant les rumeurs de menaces sur Jay Powell qui ont circulé ce week-end et selon lesquelles Donald Trump réfléchirait à le démettre de ses fonctions... Il a fallu démentir et expliquer que ce n'était pas dans ses prérogatives. Tout cela accentue l'aversion au risque dans un contexte de ralentissement de l'économie mondiale et en plein "shutdown" des administrations américaines, autrement dit, l'arrêt de leur fonctionnement tant qu'un vote ne sera pas intervenu sur le repoussement de la limite du déficit public.   Attention, on ferme à 14 heures aujourd'hui à Paris. Aux Etats-Unis ce sera à 19 heures (heure française). Le marché français ne rouvrira ses portes que le 27 décembre, pour les ultimes séances de l'année.   WALL STREET   Après une tentative de rebond avortée dans la matinée, les marchés boursiers américains ont replongé vendredi, pour la troisième séance d'affilée. Ils signent une semaine calamiteuse, avec une chute de plus 7% des principaux indices, leur pire semaine depuis août 2011 (S&amp;P 500). Le Nasdaq composite est même entré vendredi dans un "bear market" (une tendance baissière) caractérisé par une chute de plus de 20% par rapport à ses derniers sommets. Sur le plan politique, le risque d'un "shutdown" des administrations fédérales à minuit (heure de Washington) et la démission surprise du secrétaire à la Défense, James Mattis, ont ébranlé la confiance des marchés. En outre, les incertitudes sur le commerce mondial, la croissance et les taux d'intérêts ont continué à miner les esprits. Enfin, l'arrivée simultanée à échéance des contrats à terme et des options sur les actions et les indices (journée dite des "4 sorcières") a accru la volatilité ambiante de ce vendredi.   A la clôture, l'indice Dow Jones a cédé 1,81% à 22.445 points, tandis que l'indice large S&amp;P 500 a reculé de 2,06% à 2.416 pts et que l'indice Nasdaq composite, riche en valeurs technologiques et biotechnologiques, a dégringolé de 2,99% à 6.333 pts. Tous les indices sectoriels du S&amp;P 500 ont chuté, à commencer par les services de communication (-3%), la technologie (-3%), les produits de consommation discrétionnaires (-2,58%) et les financières (-1,9%). Cette baisse des marchés s'est produite dans un important volume d'activité, le plus élevé depuis août 2011.   ECO ET DEVISES   Etats-Unis :    - Indice d'activité nationale de la Fed de Chicago. (14h30)    La parité euro / dollar atteint 1,1388$ ce matin. Le baril de Brent se négocie 54,15$. L'once d'or se traite 1.258$.   VALEURS A SUIVRE   Total : Le tribunal correctionnel de Paris a condamné vendredi le groupe Total à une peine symbolique de 500.000 euros d'amende pour corruption d'agents publics étrangers en marge de la conclusion de contrats en Iran en 1997.   Showroomprivé : a annoncé vendredi le succès de son augmentation de capital de 39,5 millions d'euros visant à accroître sa flexibilité financière.   Vinci Airports a annoncé vendredi avoir bouclé le financement de la concession de l'aéroport de Belgrade et en reprendre l'exploitation.   Renault : Carlos Ghosn passera finalement la fin d'année en prison, 1er janvier 2019 compris.   Tikehau a présenté vendredi les termes de son projet d'offre publique d'achat visant les actions et les obligations à option de conversion et/ou d'échange en actions nouvelles ou existantes (OCEANE) de Selectirente.   Marie Brizard Wine &amp; Spirits annonce aujourd'hui la signature d`un accord ferme avec la Compagnie Financière Européenne de Prises de Participations (COFEPP).   CHIFFRE D'AFFAIRES - RÉSULTATS   Marie Brizard Wine &amp; Spirits, Piscines Desjoyaux.   EN BREF :   Pixium Vision renouvelle sa ligne de financement avec Kepler Cheuvreux.   Valneva reçoit un 'Fast Track' de la FDA.   Albioma finalise l'acquisition de 60% d'une unité de cogénération de bagasse au Brésil.   Gecina : finalise la cession d'un portefeuille de murs pour près de 20 ME.   Artea : émission obligataire de 6,8 ME par placement privé.   April : prévoit d'enregistrer une provision d'un montant de 15 ME dans ses comptes clos à fin 2018.
</article>
<article titre="ALBIOMA : acquisition de 60 % de la deuxième cogénération de Jalles Machado au Brésil" date="2018 12 24" journal="Agence Option Finance (AOF)">
(AOF) - Albioma a signé avec le groupe Jalles Machado le contrat définitif permettant l'acquisition de 60 % de l'unité de cogénération bagasse adossée à la sucrerie-distillerie éponyme, localisée à Goianésia (État du Goiás), dont la capacité de broyage annuelle atteint 2,8 millions de tonnes de canne. Pour rappel, Albioma avait signé le 18 décembre 2017 un accord, prévoyant notamment la rénovation des chaudières existantes ainsi que l'installation d'une nouvelle turbine de 25 MW, portant la capacité totale du site à 65 MW.   Grâce à l'amélioration des rendements de la cogénération et à la réduction de la consommation de vapeur de la sucrerie, l'énergie exportée sur le réseau de distribution devrait ainsi atteindre 145 GWh par an à compter de la campagne 2019, soit près de deux fois plus qu'actuellement. Plus de 80 % des ventes d'énergie sont d'ores et déjà sécurisés sur le marché règlementé : la cogénération est en effet déjà titulaire de contrats à long terme et a vendu 75 GWh par an à compter de 2021 au prix de 258 BRL/MWh indexé sur l'inflation à la faveur de l'appel appel d'offres du 18 décembre 2017 (leilão de energia nova A-4).   AOF - EN SAVOIR PLUS  Services aux collectivités  Sous l'effet des fusions et acquisitions, le secteur mondial des utilities est en pleine reconfiguration. Selon une étude d'AT Kearney, 1040 opérations de ce type ont été annoncées dans le monde en 2016, pour un total de 329 milliards de dollars, montant record en valeur. AT Kearney estime que le marché bénéficie d'abord des cessions d'actifs opérées par les grands énergéticiens européens.   Comme le souligne le cabinet de conseil et d'audit PwC, l'année 2016 a été un exercice record en matière de fusions-acquisitions dans l'énergie : près de 300 milliards de dollars, en hausse de 47% par rapport à 2015. Et effectivement l'Europe a joué un rôle essentiel dans cette accélération car les transactions sur ce continent ont bondi de 25%, à 49 milliards de dollars. En France, 45 opérations de fusions-acquisitions ont été recensées en 2016, pour une valeur totale de 5,4 milliards de dollars. La prise de participation minoritaire de CNP Assurances et de la Caisse des dépôts (CDC) dans Réseau de transport d'électricité (RTE), la filiale d'EDF, pour 4,6 milliards de dollars, a surpassé toutes les autres.
</article>
<article titre="Albioma finalise l'acquisition de 60% d'une unité de cogénération de  bagasse au Brésil" date="2018 12 22" journal="Boursier (site web)">
Albioma a signé, le 21 décembre, avec le groupe Jalles Machado le contrat définitif permettant l'acquisition de 60% de l'unité de cogénération bagasse adossée à la sucrerie-distillerie éponyme, localisée à Goianésia (Etat du Goiás). Sa capacité de broyage annuelle atteint 2,8 millions de tonnes de canne. Albioma rappelle avoir signé, le 18 décembre 2017, un accord, prévoyant notamment la rénovation des chaudières existantes ainsi que l'installation d'une nouvelle turbine de 25 MW, portant la capacité totale du site à 65 MW.   Grâce à l'amélioration des rendements de la cogénération et à la réduction de la consommation de vapeur de la sucrerie, l'énergie exportée sur le réseau de distribution devrait ainsi atteindre 145 GWh par an à compter de la campagne 2019, soit près de 2 fois plus qu'actuellement.   Plus de 80% des ventes d'énergie sont d'ores et déjà sécurisés sur le marché règlementé : la cogénération est en effet déjà titulaire de contrats à long terme et a vendu 75 GWh par an à compter de 2021 au prix de 258 BRL/MWh indexé sur l'inflation à la faveur de l'appel appel d'offres du 18 décembre 2017 (leilão de energia nova A-4).
</article>
<article titre="PRpro : Hyatt Regency Bali:Le légendaire Hyatt Regency Bali rouvre ses portes après 5 années de rénovation" date="2018 12 21" journal="Belga News Agency (français)">
PRpro : Hyatt Regency Bali:Le légendaire Hyatt Regency Bali rouvre ses portes après 5 années de rénovation   ======== communication non-rédactionnelle =========   BELGIUM 21/12 (PRPRO) =   BALI, Indonésie, 21 décembre 2018 /PRNewswire/ .- Le 20 décembre 2018, le Bali Hyatt, la Grande Dame de l'hospitalité balinaise âgée de 45 ans, rouvre ses portes sous le nom d'Hyatt Regency Bali. Consultez la version multimédia complète du communiqué à l'adresse suivante : https://www.prnasia.com/mnr/hyatt_2018.shtml   https://mma.prnewswire.com/media/801097/Hyatt_Regency_Bali_p3.jpg   À son inauguration en novembre 1973, le Bali Hyatt était le tout premier hôtel international à ouvrir ses portes sur l'île et faisait figure de pionnier en matière d'architecture de style indonésien.   Autrefois considéré comme l'hôtel le plus luxueux et élégant de l'île, le Bali Hyatt a fait l'objet d'une rénovation et d'une modernisation de plusieurs millions de dollars. Le Hyatt Regency Bali vise à susciter chez ses clients le même sentiment d'émerveillement et d'admiration qu'il y a 45 ans tout en préservant l'esthétique balinaise tant appréciée du design original.    « Au terme de cette période de rénovations importantes, nous sommes ravis de pouvoir révéler au public un Hyatt Regency Bali nouveau et amélioré. », a déclaré Zulki Othman, le directeur général. « Nous sommes impatients d'accueillir de nouveaux clients et de recevoir nos anciens clients qui, nous le savons, apprécieront le fait que nous ayons réussi à moderniser toutes les structures et tous les équipements tout en conservant le style et l'esprit balinais d'origine de l'hôtel. Nous avons l'incroyable chance d'occuper un emplacement incroyable en bord de mer entouré de verdure luxuriante et sommes ravis de pouvoir l'apprécier à nouveau. »   Emplacement   Situé à seulement 22 minutes de route de l'aéroport international Ngurah Rai et occupant une surface de 9 hectares, le Hyatt Regency Bali est l'un des plus grands complexes hôteliers de Sanur, connu pour ses jardins imaginés par le célèbre paysagiste tropical Made Wijaya. Mettant en vedette plus de 500 espèces de plantes, les jardins de l'hôtel ont orné les pages de nombreux livres d'aménagement paysager et de beaux livres grand format. Au cours des travaux de rénovation, tous les grands arbres ont été conservés, tout comme la plupart des plantes.   En outre, le Hyatt Regency Bali donne sur une plage de 500 mètres qui figure parmi les plus larges du front de mer de Sanur.   Architecture   L'architecture générale reprend les éléments du Bali Hyatt. Le hall d'entrée est notamment quasi identique à celui du Bali Hyatt, avec ses carreaux en terre cuite, son lustre, ses statues et ses escaliers. Tout autour de l'hôtel, des éléments de l'architecture originale du Bali Hyatt sont intégrés à un nouvel environnement : des sculptures sont placées derrière la reception, des timbres en batik provenant des anciennes chambres sont reconvertis en ?uvres d'art dans le Regency Club Lounge.   Chambres   Le Hyatt Regency Bali compte 363 chambres d'une superficie allant de 27 à 81 m², chacune dotée d'un balcon privé donnant sur les jardins luxuriants ou sur l'océan Indien. L'hôtel comprend en outre 39 suites d'une chambre aux balcons spacieux d'une superficie pouvant atteindre les 70 m². Avec un salon séparé pouvant accueillir un lit supplémentaire et une salle d'eau supplémentaire avec douche, les suites sont idéales pour les familles et les groupes. En outre, la réservation d'une suite comprend l'accès au Regency Club Lounge, qui inclut les services d'un concierge privé, un petit-déjeuner gratuit, des collations tout au long de la journée ainsi que des cocktails et des canapés en soirée.   Bars et restaurants   Le Hyatt Regency Bali propose trois lieux de restauration en plein air proposant chacun à des prix concurrentiels des plats et des boissons authentiques composés des ingrédients disponibles les plus frais et les plus fins, locaux et biologiques dans la mesure du possible. Le complexe vise à minimiser son impact environnemental en utilisant un minimum de plastique et en optant pour des pailles en papier, des tasses à café recyclables et des sacs en bagasse (pulpe de canne à sucre).   L'Omang Omang est l'espace de restauration de l'hôtel. Ouvert toute la journée, il offre des plats réconfortants d'Indonésie et du monde entier. Outre sa vue sur les jardins et les piscines, l'Omang Omang propose un buffet pour le petit-déjeuner, les plats les plus populaires au sein du complexe tout au long de la journée, des fruits de mer grillés au charbon de bois et des plats de premier choix en soirée, ainsi que ce qu'il considère comme étant le meilleur Es Campur (glace pilée au lait de coco) de la ville.   Le Pizzaria offre une vue imprenable sur la mer et le coucher du soleil. Le bar-restaurant décontracté surplombant la plage de Sanur est spécialisé dans les pâtes cuites à la minute, les pizzas cuites dans des fours à bois, les cocktails glacés et les sodas à l'italienne garnis de fruits et d'herbes fraîches.   Sur fond de musique rindik (musique balinaise) le jour et de piano la nuit, le Piano Lounge est idéal pour un verre au calme, une réunion de travail décontractée ou un événement mondain. En l'honneur de la tradition indonésienne nommée arisan, au cours de laquelle les femmes se rassemblent, un thé typique est servi l'après-midi, ainsi qu'un café indonésien et un thé en feuilles.   Activités   À Bali, les occasions de profiter de la nature verdoyante ne manquent pas. Le Hyatt Regency Bali est doté de trois piscines jouxtant la plage : une piscine pour la nage en longueur, une piscine pour enfants peu profonde et une piscine principale entourée d'une cascade de bougainvillées où vous pourrez nager à travers une réplique du Goa Gajah - le temple de la « grotte des éléphants » datant du IXe siècle et qui se trouve près d'Ubud - sculptée dans la roche volcanique.   Avant même d'arriver au ShankhaSpa, les clients sentiront leur stress se dissiper en traversant les passerelles en plein air entourées d'étangs qui y mènent. Le vaste complexe comprend dix suites - chacune dotée d'un mini-spa - une zone de traitement, un espace détente, des douches, un vestiaire, un coin lavabo et un jardin privé avec bain à remous extérieur idéal pour les traitements watsu (massage aquatique).   Plus loin, les clients découvriront une deuxième piscine, dominée par une statue de Ganesh couchée, un bar à jus à ciel ouvert où ils pourront reconstituer leur réserve d'électrolytes après un entraînement intense, un studio de yoga, des bassins extérieurs chauds et froids, ainsi qu'un espace de détente.   Pour les plus jeunes, le Hyatt Regency Bali propose le Camp Hyatt. Dans cette enceinte, les enfants découvriront une aire de jeux extérieure, une salle de jeux intérieure remplie de jouets et de jeux, ainsi qu'un programme conçu pour divertir et éduquer à l'aide d'activités artistiques, artisanales et culturelles balinaises.   L'inauguration du Shankha Spa et du Camp Hyatt est prévue pour le 1er trimestre de 2019.   Pour consulter le dossier de presse en ligne, veuillez cliquer ici.   Contact : Bulan Bharata, responsable des relations publiquesTél. : +62-361-28-1234 / E: bulan.bharata@hyatt.com   Photo - https://mma.prnewswire.com/media/801097/Hyatt_Regency_Bali_p3.jpg   Related Link:   http://prs.belgamediasupport.be/l/96748   Belga n'st pas responsable du contenu du communiqué ci-dessus
</article>
<article titre="Lyspackaging fabrique des bouteilles vegan et lève 2,2 M€" date="2018 12 20" journal="La Tribune (France), no. 6592">
Cabinet conseil en fusion-acquisition situé à Bordeaux et Angoulême, Alienor Partners a été choisi par Nicolas Moufflet, dirigeant fondateur de la société Lyspackaging, à Saintes (Charente), pour réaliser sa levée de fonds de 2,2 M€, qui s'est soldée par l'entrée du fonds Cita Investissement au capital de cette startup innovante. Nicolas Moufflet, fondateur de l'entreprise en 2016, s'est positionné sur un créneau d'avant-garde qui semble promis à un grand avenir : le bioplastique, avec sa bouteille Veganbottle. Entièrement biodégradable et compostable, après avoir été hachée, cette bouteille 100 % végétale garantie sans OGM (organisme génétiquement modifié) est fabriquée à partir de la bagasse de canne à sucre. Un sous-produit constitué par l'amas végétal fibreux qui reste une fois que le sucre a été extrait de la canne. Cette bouteille, qui agrège d'autres éléments végétaux, est une innovation couverte par plusieurs brevets, qui, en plus de l'agroalimentaire, vise pour le moment de nouveaux marchés comme la cosmétique.   [Image : https://static.latribune.fr/596881/vins-bio.jpg]   Les bouteilles de Lyspackaging peuvent sans problème être remplies de vin, d'eau ou de jus de fruits (illustration photo Pascal Rabiller).   Cita Investissement regroupe Bpifrance et KIA Lyspackaging développe ses produits : du design jusqu'à la fabrication, et propose à ses clients des bouteilles et autres contenants (flacons, carafes...) conçus sur-mesure, avec des capacités de 30 millilitres à 3 litres. La startup, qui emploie 9 personnes, dispose de sa propre unité de production où sont fabriqués, en grandes ou petites séries, ses modèles Veganbottle mais aussi des contenants en polytéréphtalate d'éthylène (Pete ou Pet), qui sont beaucoup plus légers que les bouteilles plastique traditionnelles. Pour une production annuelle moyenne de 10 millions d'unités.   La jeune entreprise innovante charentaise est soutenue par la Région Nouvelle-Aquitaine et le Critt Agroalimentaire. Elle fabrique aussi des bouchons et bagues de fermeture. Doté de 100 M€ abondés par Bpifrance et Koweit Investment Authority (KIA) le fonds Cita Investissement investit de son côté dans le capital de PME/PMI françaises pour des montants compris entre 1 et 15 M€ précise le communiqué sur l'opération et s'appuie "sur 30 ans d'expérience dans divers secteurs d'activité, aussi bien en capital-développement qu'en capital-risque ou reprise-transmission".
</article>
<article titre="[Lyspackaging lève 2,2 millions d'euros...]" date="2018 12 20" journal="Sud Ouest  Toutes éditions ; Médoc ; Charente ; Périgueux ; Mont-de-Marsan">
Lyspackaging lève 2,2 millions d'euros   La société créée en 2015 par Nicolas Moufflet, à Saintes (17), spécialisée au départ dans la fabrication de bouteilles plastique a conçu, produit et commercialise aujourd'hui une bouteille 100 % végétale à partir de bagasse de canne sucre. Packaging compostable et biodégradable, baptisé Veganbottle. En un an et demi, la petite entreprise a basculé son équilibre, la production de bouteilles végétales devenant majoritaire (70 %) par rapport à la production de celles en plastique. La levée de fonds va permettre à Lyspackaging de structurer son organisation, de maîtriser la fabrication de A à Z et d'investir dans des ressources industrielles et dans une force commerciale. À noter que la société est sur le point de commercialiser sa première bouteille végétale d'eau de source et la Vegango, une gourde aux mêmes qualités que la Veganbottle. Elle espère, dès 2019, multiplier par trois son chiffre d'affaires (CA 2018 : 1 million d'euros) et installer sa première franchise et sa première unité de production clé en main à l'international. La Rochelle : GRTgaz investit 19 millions   La construction du gazoduc qui va contourner le nord-ouest de l'agglomération rochelaise, chantier qui va démarrer en janvier prochain, va mobiliser un investissement de 19 millions d'euros. Ce chantier - il s'agit de poser 10 kilomètres de canalisations - vise à remplacer le réseau précédant, réalisé dans les années 60. Il va durer 12 mois et devrait générer 3 millions d'euros de retombées pour l'économie locale, selon GRTgaz.   L'économie circulaire, thème du Sia 2019   Le prochain Salon international de l'agriculture qui se déroulera du 23 février au 3 mars à Paris, aura pour thème : " Le secteur agricole, moteur de l'économie circulaire. " On va donc parler biodiversité, recyclage, consommation responsable, alternatives aux fertilisants, matériaux biosourcés, biocarburants, méthanisation, innovations et, bien évidemment, agriculture urbaine et bioéconomie.   Français et foie gras, une histoire de confiance   La dernière enquête réalisée par le CSA début décembre, pour les professionnels du foie gras, montre que 69 % des Français ont décidé de savourer du foie gras pour ces fêtes. 75 % des Français jugent ce produit incontournable au moment des repas de fin d'année, devant le saumon fumé (61 %) ou la bûche de Noël (57 %). L'origine France est, pour 92 % d'entre eux, un critère de choix primordial. Enfin, les Français ont confiance en la qualité du produit en matière de sécurité alimentaire (84 %), de traçabilité (78 %) ou de qualité gustative (88 %). Ils sont 71 % à considérer que le foie gras est forcément issu d'un animal en bonne santé. Cet article a également été publié dans les éditions suivantes : Charente, page C1_9, 8; Périgueux, page C1_8, 9; Mont-de-Marsan, page C1_9, 8
</article>
<article titre="Lyspackaging fabrique des bouteilles vegan et lève 2,2 M€" date="2018 12 19" journal="La Tribune - Bordeaux (site web)">
Cabinet conseil en fusion-acquisition situé à Bordeaux et Angoulême, Alienor Partners a été choisi par Nicolas Moufflet, dirigeant fondateur de la société Lyspackaging, à Saintes (Charente), pour réaliser sa levée de fonds de 2,2 M€, qui s'est soldée par l'entrée du fonds Cita Investissement au capital de cette startup innovante.   Nicolas Moufflet, fondateur de l'entreprise en 2016, s'est positionné sur un créneau d'avant-garde qui semble promis à un grand avenir : le bioplastique, avec sa bouteille Veganbottle. Entièrement biodégradable et compostable, après avoir été hachée, cette bouteille 100 % végétale garantie sans OGM (organisme génétiquement modifié) est fabriquée à partir de la bagasse de canne à sucre. Un sous-produit constitué par l'amas végétal fibreux qui reste une fois que le sucre a été extrait de la canne. Cette bouteille, qui agrège d'autres éléments végétaux, est une innovation couverte par plusieurs brevets, qui, en plus de l'agroalimentaire, vise pour le moment de nouveaux marchés comme la cosmétique. Les bouteilles de Lyspackaging peuvent sans problème être remplies de vin, d'eau ou de jus de fruits (illustration photo Pascal Rabiller). Cita Investissement regroupe Bpifrance et KIA Lyspackaging développe ses produits : du design jusqu'à la fabrication, et propose à ses clients des bouteilles et autres contenants (flacons, carafes...) conçus sur-mesure, avec des capacités de 30 millilitres à 3 litres. La startup, qui emploie 9 personnes, dispose de sa propre unité de production où sont fabriqués, en grandes ou petites séries, ses modèles Veganbottle mais aussi des contenants en polytéréphtalate d'éthylène (Pete ou Pet), qui sont beaucoup plus légers que les bouteilles plastique traditionnelles. Pour une production annuelle moyenne de 10 millions d'unités.   La jeune entreprise innovante charentaise est soutenue par la Région Nouvelle-Aquitaine et le Critt Agroalimentaire. Elle fabrique aussi des bouchons et bagues de fermeture. Doté de 100 M€ abondés par Bpifrance et Koweit Investment Authority (KIA) le fonds Cita Investissement investit de son côté dans le capital de PME/PMI françaises pour des montants compris entre 1 et 15 M€ précise le communiqué sur l'opération et s'appuie  "sur 30 ans d'expérience dans divers secteurs d'activité, aussi bien en capital-développement qu'en capital-risque ou reprise-transmission" .  Cet article est paru dans La Tribune - Bordeaux (site web)
</article>
<article titre="Conseil Albioma Le coup de coeur" date="2018 12 15" journal="Investir-Le Journal des finances, no. 2345">
Voilà un poids qui ne handicapera plus Albioma en Bourse : le holding Altamir a cédé sa participation résiduelle, soldant ainsi des années de compagnonnage entre le producteur d'électricité et les véhicules d'investissement d'Apax Partners, fondé par Maurice Tchenio. Apax avait plombé la valeur en 2015 en redistribuant aux clients de ses fonds 30 % du capital d'Albioma. PuisAltamir s'était allégé de 6,5 % en mars dernier, lors d'un placement privé effectué avec une décote d'environ 10 %, quand l'action cotait 21 €.   Près de 20 % de prime Il restait au holding une participation de l'ordre de 5,5 %. La perspective d'une cession dans les mêmes conditions pesait sur le cours de Bourse. Cette hypothèque a été levée mercredi : Altamir est sorti d'Albioma, mais en vendant ses titres à Impala, le holding de Jacques Veyrat, à un prix d'environ 18,60 € par action, supérieur de presque 20 % au cours coté la veille. De quoi électriser la valeur, qui a bondi de 12,5 % mercredi. De plus, Jacques Veyrat n'est pas un inconnu dans le domaine de l'énergie : il détenait un tiers du capital du distributeurDirect Energie, revendu cette année àTotal, et il contrôle 50,1 % deNeoen, l'exploitant de parcs solaires et éoliens entré en Bourse en octobre.   « Nous sommes très contents d'accueillir un investisseur avisé dans le domaine des énergies renouvelables. Depuis 2015, nous avons recomposé le capital d'Albioma avec des actionnaires stables et de long terme, comme Bpifrance et Cofepp, notre partenaire sucrier historique aux Antilles »,  précise Julien Gauthier, directeur général adjoint, chargé des finances. Ces nouveaux venus détiennent, respectivement, 5 % et 6 % du capital.   L'électricien, par ailleurs, a annoncé son désengagement de Methaneo, l'activité de méthanisation agricole acquise en 2012. Elle n'a pas rempli ses promesses dans un secteur qui s'est révélé difficile à développer en France.   Seuls trois méthaniseurs ont été construits alors que la société disposait d'un portefeuille d'une vingtaine de projets. Elle a été dépréciée et les trois sites « remis d'aplomb » , mais ils ne contribuaient pas aux bénéfices consolidés. Methaneo a été vendu à une coentreprise entre Meridiam, fonds dédié aux infrastructures, et Evergaz, spécialiste du biogaz.   En revanche, Albioma se renforce dans le solaire en rachetant Eneco France, une filiale d'un groupe néerlandais opérant « essentiellement dans le sud de la France, que nous avions un peu délaissée, faute de moyens humains »,  indique Julien Gauthier. Eneco, spécialisé dans les parcs solaires sur toitures, et profitable, dispose d'une capacité installée de 17 mégawatts et d' « un beau portefeuille de projets »,  selon le directeur général adjoint.   Dans tous les cas, il participera à verdir la production électrique d'Albioma, qui a obtenu, mi- novembre, l'autorisation de la CRE (Commission de régulation de l'énergie) de convertir à la biomasse sa seule centrale thermique ne brûlant que du charbon, quand les autres sites d'outre-mer consomment aussi de la bagasse (les résidus de la canne à sucre). Les travaux pour convertir cette centrale de Guadeloupe sont prévus en 2020. La conversion des trois centrales bagasse-charbon devra être engagée pour 2023, année où le groupe vise un mix énergétique à 80 % d'origine renouvelable. « Ce sera au-delà »,  assure Julien Gauthier.   Notre conseil Acheter L'entrée d'Impala au capital, avec une prime proche de 20 %, est un signal très favorable pour le titre. Les mises en service de la centrale de Galion 2, en Martinique, à l'automne, et, tout prochainement, de la centrale de Saint-Pierre, à La Réunion, vont permettre un bond des résultats en 2019, autour de 45 millions d'euros pour le bénéfice net (selon le consensus FactSet), contre 40 millions estimés pour 2018.   Par Cécile Le Coz
</article>
<article titre="Le compostage du PLA en test" date="2018 12 14" journal="Emballages magazine (site web)">
Citeo et Les Alchimistes doivent rendre leurs conclusions en mai 2019.-   Est-il pertinent de créer une filière de traitement dédiés aux emballages en acide polylactique (PLA) ? Souvent présenté comme une alternative au polyéthylène téréphtalate (PET), ce polymère biosourcé est produit industriellement à partir de ressources renouvelables et végétales telles que la bagasse de canne à sucre. C'est la question à laquelle Citeo, la société agréée pour la valorisation des emballages et papiers ménagers au titre de la responsabilité élargie des producteurs (REP), et Les Alchimistes, une entreprise spécialisée dans la valorisation locale des biodéchets, pourront répondre au printemps prochain en terme d'une expérimentation menée à Paris. Les conclusions intéresseront certainement Raphaël de Taisne, le cofondateur de Yumi, qui a choisi ce matériau. Résultat ? Fournies par Lyspackaging, les petites bouteilles de jus de légumes bio sont frappées d'un malus de 100 % par Citeo... Du côté de l'agence métropolitaine des déchets (Syctom), le PLA, volontiers comparé au PET opaque des bouteilles de lait, est clairement présenté comme un perturbateur de tri à proscrire absolument. Engagés dans le cadre d'un partenariat noué en septembre 2018, Citeo et Les Alchimistes livreront, quoi qu'il en soit, leurs premières réponses en mai prochain. NFU 44 051   Objectif de cette expérimentation menée dans la capitale : tester des procédés de compostage innovants, ainsi que des modes de collecte responsables et adaptés, que ce soit pour les professionnels ou les particuliers. Un plan de sensibilisation sur les points de collecte est prévu afin d'encourager les consommateurs à rapporter en magasin leurs emballages plastiques compostables. Le procédé testé est électromécanique et se déroule en trois étapes : prétraitement, maturation dans un conteneur fermé pendant six semaines et opérations post compostage qui permettront de tester et d'analyser le compost obtenu. La labellisation selon la norme NFU 44 051 est visée afin de distribuer le premier compost de ce type « fabriqué à Paris » . « Un emballage, même compostable ou dit biodégradable, ne doit pas être jeté dans la nature. Il ne se dégrade pas de la même manière dans tous les milieux naturels et peut avoir un impact négatif sur la biodiversité », rappelle au passage Citeo.   (Avec Fabian Tubiana, Plastiques &amp; Caoutchoucs Magazine)
</article>
<article titre="Albioma s'ouvre l'horizon" date="2018 12 13" journal="L'AGEFI Quotidien">
Albioma tourne définitivement la page d'une histoire commune de 13 ans avec la galaxie Apax Partners. Mardi soir, Altamir, la société cotée de private equity qui investit à travers les fonds d'Apax, a annoncé après la clôture avoir cédé sa participation résiduelle de 5,5% du capital du producteur d'énergie renouvelable au conglomérat diversifié Impala, qui a déboursé 31,7 millions d'euros. Soit un prix de 18,8 euros par action, offrant une prime de 20% sur le cours de clôture de mardi. «A l'issue de cette transaction, Altamir ne détient plus d'actions Albioma», précise la société de capital-investissement dans son communiqué. En mars dernier, Altamir avait déjà cédé une participation de 6,5% à des investisseurs institutionnels, à travers un placement privé réalisé, pour 37,4 millions d'euros. Les 12% vendus au total cette année représentent donc 68,1 millions d'euros, pour un coût résiduel de 59 millions inscrit dans le document de référence d'Altamir au 31 décembre 2017. L'opération permet à Albioma, acteur d'outre-mer à travers le photovoltaïque et la biomasse (issue de la bagasse), d'ouvrir un nouveau chapitre. Impala, structure d'investissement de l'homme d'affaires Jacques Veyrat, devient en effet le premier actionnaire de l'entreprise : il détient désormais une participation de 6,5%, «après plusieurs achats réalisés sur le marché» représentant donc environ 1% du capital. Surtout, Impala contrôle 50,1% de Neoen, producteur d'énergie renouvelable français qui s'est introduit en Bourse en octobre dernier. La sortie d'Altamir - un investisseur financier - et l'entrée d'un actionnaire possédant une connaissance du métier d'Albioma ont été bien perçues par le marché. L'action s'est adjugée 12,5% hier à la clôture (à 17,66 euros). L'analyste du courtier Portzamparc souligne ainsi qu'Impala est «une référence» dans les énergies renouvelables. D'autant plus qu'Albioma a également annoncé l'acquisition d'Eneco France, un constructeur de centrales photovoltaïques sur toitures et hangars agricoles dans le Sud de la France. Eneco dispose d'une capacité installée de 17 mW, contre 8 mW pour Albioma en métropole. Les relations entre Altamir et Albioma remontent à 2005. La première avait acquis sa participation dans la seconde au même moment qu'Apax, qui avait lui acquis 30% du capital de la société d'énergie. En juin 2015, Apax avait distribué 95% des actions détenues par son fonds Apax France VI à ses investisseurs. Ces derniers étaient plus de 50, dont 45 institutionnels et 44% provenant d'Amérique du Nord. Le flottant d'Albioma s'en était trouvé renforcé.
</article>
<article titre="Albioma cède sa méthanisation et se renforce dans le solaire" date="2018 12 13" journal="CFNEWS (site web)">
Le producteur d'énergie renouvelable coté, qui a recomposé son capital, cède sa filiale Methaneo et se renforce dans le solaire, signant tour à tour avec Evergaz et Eneco France.    Albioma ne manque pas d'énergie ! Le producteur d'énergie renouvelable indépendant, implanté sur les territoires d'outre-mer, l'Ile Maurice et au Brésil où il valorise la bagasse, résidu fibreux de la canne à sucre, et qui s'affiche aussi comme le premier producteur d'énergie photovoltaïque en outre-mer, vient en effet de procéder à plusieurs « ajustements ». D'un point de vue capitalistique, Altamir, son actionnaire depuis 2005 cède le solde de sa participation (il vendait ses parts au capital depuis 2015), soit 5,5 % du capital à Impala, lequel débourse 31,7 M€ et devient, avec 6,5 % du capital, le premier actionnaire du groupe. D'un point de vue opérationnel, Albioma se dégage de l'activité méthanisation tandis qu'il se renforce dans le solaire.   Nouveau départ    Frédéric Moyne, Albioma    Le groupe aujourd'hui dirigé par Frédéric Moyne, DG, a trouvé repreneur pour Methaneo, une entreprise dont il était devenu l'actionnaire majoritaire en 2012 avant de reprendre l'intégralité du capital dans la foulée. Il se déleste donc de trois unités de méthanisation agricole d'une puissance de 3,2 MWe, en Nouvelle Aquitaine et dans les Pays de la Loire, qui représentent 20 GWh, soit un chiffre d'affaires de près de 6 M€. Ce n'était pas la première tentative du groupe en vue de céder Methaneo puisqu'il avait déjà essayé, en vain, en 2016 mais, cette fois, le mandat confié à Transaction R a abouti. Plusieurs offres ont même été formulées et deux sont restées en lice jusqu'à ce qu'Evergaz l'emporte via Biométhanisation Partenaires, la structure dédiée, créée avec Meridiam, son actionnaire depuis le printemps. Albioma se détache donc d'une activité qu'il a eu du mal à appréhender et dont il avait stoppé le développement. Il la confie à un spécialiste qui porte à 14 le nombre de ses unités de méthanisation en France (où il en compte huit, et bientôt neuf), en Belgique et en Allemagne pour 28 MW, soit 7 000 Nm³/h et vise, à horizon 2022, 120 millions Nm³ une puissance installée de 64 MWe et le traitement de plus d'1 million de tonnes de déchets organiques par an.    En parallèle, Albioma, se renforce dans le solaire en mettant la main sur la filiale française du groupe d'origine néerlandaise actif en Belgique Eneco. Positionné dans l'autoconsommation, Eneco France construit et exploite des centrales photovoltaïques en toiture et sur les hangars agricoles chez des particuliers et des industriels du sud de la France, soit 17 MW, ainsi qu'une centrale hydroélectrique (0,5 MW). Il aurait par ailleurs « un beau portefeuille de projets en développement ». Cette acquisition permet à Albioma d'intensifier sa puissance de feu en France métropolitaine où il dispose déjà de 8 MW installés. Pour rappel, Albioma a réalisé 403 M€ de chiffre d'affaires en 2017 et dégagé 138 M€ d'Ebitda lequel pourrait, selon ses prévisions, osciller entre 158 M€ et 166 M€ cette année. Selon un spécialiste du secteur, cette acquisition pourrait être la première d'une série plus longue pour Albioma, lequel deviendrait une plateforme de consolidation dans l'Hexagone.    voir aussi :    Bourse Albioma (cession des titres d'Altamir)    M&amp;A Eneco France   Cet article est paru dans CFNEWS (site web)
</article>
<article titre="Albioma cède sa méthanisation et se renforce dans le solaire" date="2018 12 13" journal="CFNEWS (site web)">
Le producteur d'énergie renouvelable coté, qui a recomposé son capital, cède sa filiale Methaneo et se renforce dans le solaire, signant tour à tour avec Evergaz et Eneco France.    Albioma ne manque pas d'énergie ! Le producteur d'énergie renouvelable indépendant, implanté sur les territoires d'outre-mer, l'Ile Maurice et au Brésil où il valorise la bagasse, résidu fibreux de la canne à sucre, et qui s'affiche aussi comme le premier producteur d'énergie photovoltaïque en outre-mer, vient en effet de procéder à plusieurs « ajustements ». D'un point de vue capitalistique, Altamir, son actionnaire depuis 2005 cède le solde de sa participation (il vendait ses parts au capital depuis 2015), soit 5,5 % du capital à Impala, lequel débourse 31,7 M€ et devient, avec 6,5 % du capital, le premier actionnaire du groupe. D'un point de vue opérationnel, Albioma se dégage de l'activité méthanisation tandis qu'il se renforce dans le solaire.   Nouveau départ    Frédéric Moyne, Albioma    Le groupe aujourd'hui dirigé par Frédéric Moyne, DG, a trouvé repreneur pour Methaneo, une entreprise dont il était devenu l'actionnaire majoritaire en 2012 avant de reprendre l'intégralité du capital dans la foulée. Il se déleste donc de trois unités de méthanisation agricole d'une puissance de 3,2 MWe, en Nouvelle Aquitaine et dans les Pays de la Loire, qui représentent 20 GWh, soit un chiffre d'affaires de près de 6 M€. Ce n'était pas la première tentative du groupe en vue de céder Methaneo puisqu'il avait déjà essayé, en vain, en 2016 mais, cette fois, le mandat confié à Transaction R a abouti. Plusieurs offres ont même été formulées et deux sont restées en lice jusqu'à ce qu'Evergaz l'emporte via Biométhanisation Partenaires, la structure dédiée, créée avec Meridiam, son actionnaire depuis le printemps. Albioma se détache donc d'une activité qu'il a eu du mal à appréhender et dont il avait stoppé le développement. Il la confie à un spécialiste qui porte à 14 le nombre de ses unités de méthanisation en France (où il en compte huit, et bientôt neuf), en Belgique et en Allemagne pour 28 MW, soit 7 000 Nm³/h et vise, à horizon 2022, 120 millions Nm³ une puissance installée de 64 MWe et le traitement de plus d'1 million de tonnes de déchets organiques par an.    En parallèle, Albioma, se renforce dans le solaire en mettant la main sur la filiale française du groupe d'origine néerlandaise actif en Belgique Eneco. Positionné dans l'autoconsommation, Eneco France construit et exploite des centrales photovoltaïques en toiture et sur les hangars agricoles chez des particuliers et des industriels du sud de la France, soit 17 MW, ainsi qu'une centrale hydroélectrique (0,5 MW). Il aurait par ailleurs « un beau portefeuille de projets en développement ». Cette acquisition permet à Albioma d'intensifier sa puissance de feu en France métropolitaine où il dispose déjà de 8 MW installés. Pour rappel, Albioma a réalisé 403 M€ de chiffre d'affaires en 2017 et dégagé 138 M€ d'Ebitda lequel pourrait, selon ses prévisions, osciller entre 158 M€ et 166 M€ cette année. Selon un spécialiste du secteur, cette acquisition pourrait être la première d'une série plus longue pour Albioma, lequel deviendrait une plateforme de consolidation dans l'Hexagone.    voir aussi :    Bourse Albioma (cession des titres d'Altamir)    M&amp;A Eneco France   Cet article est paru dans CFNEWS (site web)
</article>
<article titre="Albioma (ex-Sechilienne-Sidec) : un stimulant nommé Jacques Veyrat" date="2018 12 12" journal="Boursier (site web)">
Capital.  Le capital d'Albioma est l'objet d'un mouvement significatif. Alors que le fonds Apax avait repris la société en 2005, il vient de clore définitivement cette aventure. Le solde de ses actions Albioma, soit 5,5% du capital, est vendu au groupe Impala de Jacques Veyrat. L'acquéreur est bien connu dans le monde des affaires... Après avoir été éjecté de la présidence de Louis Dreyfus, Jacques Veyrat reçoit un certain nombre d'actifs comme cadeau de départ. Rebaptisant le tout Impala, M. Veyrat obtient l'appui d'investisseurs fortunés comme Xavier Niel ou la famille Agnelli de Fiat. Impala est très actif dans le secteur de l'énergie, il est notamment l'actionnaire de référence du producteur d'électricité verte Neon qui vient de lever 450 Millions d'Euros avec une IPO remarquée sur le marché parisien. Offensive.  Par ailleurs, Impala a touché 630 Millions d'Euros en cédant ses actions Direct Energie lors de l'OPA menée par Total. Alors qu'Impala n'a pas recours à la dette pour financer ces initiatives, sa structure financière met en évidence un gros trésor de guerre. Jacques Veyrat a ainsi les moyens de passer à l'offensive chez Albioma qui capitalise autour de 550 Millions d'Euros. En déployant son modèle de valorisation de la bagasse à l'international et en développant des projets solaires, Albioma a de grandes ambitions. Un programme d'investissements portant sur 1 Milliard d'Euros doit permettre de multiplier par deux le résultat net entre 2013 et 2023. Valorisation.  Après avoir rencontré un certain nombre de difficultés d'ordre technique ou opérationnel, l'ex-Séchilienne parvient aujourd'hui à tenir ses promesses. L'évolution encourageante des résultats au 30 juin dernier augure bien des promesses annuelles avec un résultat net qui devrait se situer entre 37 ME et 42 ME. A seulement 11 fois les attentes bénéficiaires 2018, la valorisation d'Albioma est d'autant moins élevée que le rendement dépasse les 4%. Avec la spéculation apportée par l'arrivée d'Impala au capital, il ne fait aucun doute que cette ligne Albioma est intéressante. Au vu de ces éléments favorables, on peut renforcer la pondération de cette ligne dans les portefeuilles. Cette ligne Albioma sera renforcée.Christophe Voisin
</article>
<article titre="Albioma accueille Impala à son capital, rachète Eneco France" date="2018 12 12" journal="GreenUnivers (site web)">
2018-12-12T17:10:16+00:00    (Crédit : Albioma)   Annoncée hier soir, l'arrivée d'Impala à hauteur de 5% du capital d'Albioma, dont le flottant atteint 71,5% sur Euronext Paris, n'est pas passée inaperçue. Le fonds d'investissement de Jacques Veyrat, à l'origine entre autres de la création de Neoen et d'Eiffel Investment Group, n'a pas une culture d'actionnaire minoritaire. Par ailleurs et comme le fait remarquer un spécialiste des marchés boursiers, « étant donné    la faible valorisation et les bons dividendes de cette société, Impala ne perd pas son temps. Une opération de consolidation pourrait se préparer. Ou bien Impala en sera l'artisan, ou bien il en bénéficiera lors de la revente de ses parts. » 730 MW installés En reprenant pour 31,7 M€ les parts du fonds Altamir, présent chez Albioma depuis 2005 via Apax Partners, Impala rejoint un producteur d'énergie détenteur de 730 MW installés, en Outre-Mer pour l'essentiel. Albioma est spécialiste de la combustion de la bagasse, sous-produit de la canne à sucre, en association pour le moment avec le charbon. Lequel doit être abandonné d'ici 2023 ; une première conversion à la biomasse d'une centrale de 38 MW en Guadeloupe vient d'être acceptée par la Commission de régulation de l'énergie.   Un ratio Ebitda/valorisation prometteur Cette transition énergétique est menée par une entreprise en bonne santé. Albioma a réalisé un chiffre d'affaires de 410 M€ en 2017, en hausse de 10%, pour un résultat de 37 M€. Surtout son Ebitda 2017 (résultat d'exploitation) ressort à 138 M€ alors que la valorisation actuelle ne dépasse pas 542 M€. Par comparaison, Neoen vaut 1,6 Md€ aujourdhui, pour un Ebitda 2017 de « seulement » 102 M€. Interrogé à ce sujet, Frédéric Moyne, directeur général d'Albioma estime que « l'arrivée d'Impala, actionnaire de long terme qui comprend notre stratégie, est une excellente nouvelle pour nous. Après l'entrée de Bpifrance, nous parachevons la recomposition de notre actionnariat ». Avec la Cofepp (un gros producteur d'alcool) et Bpifrance, Impala devient l'un des trois actionnaires essentiels d'une société qui veut stabiliser son capital. L'action progressait de 11,4 % à 16h30 aujourd'hui, à 17,5€.    Ambitions dans le solaire L'arrivée d'Impala coïncide pour Albioma avec l'acquisition de la filiale française de l'énergéticien néerlandais Eneco, spécialiste des toitures solaires et de l'autoconsommation, basée à Avignon et employeur de 12 personnes. Le prix de la transaction n'est pas dévoilé. « La métropole, où nous sommes déjà présents, va devenir un relais important pour notre développement. Eneco France présente trois avantages à cet égard : son positionnement sur les toitures et l'autoconsommation solaires, cohérent avec celui d'Albioma Outre-Mer, une équipe de qualité et un beau portefeuille de projets », explique Frédéric Moyne.    La croissance d'Albioma dans les Dom n'est pas sans doute pas infinie et Eneco France détient 17 MW installés et un  pipe  apparemment copieux qui vont s'ajouter à la capacité d'Albioma en métropole (8 MW installés) ; son expérience dans l'autoconsommation doit en outre renforcer les compétences de la nouvelle maison-mère sur ce segment, en métropole et Outre-Mer. En revanche, l'énergéticien est totalement sorti de la méthanisation en métropole et vient de céder ses centrales à Evergaz.    A noter que l'énergéticien néerlandais Eneco quitte les énergies renouvelables en France mais s'est renforcé récemment en Belgique, où il a fait l'acquisition de parts du parc éolien offshore SeaMade, de même que de la filiale belge du groupe Eni, troisième acteur du marché.    Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="[Ciprian Sauca présente sa gamme de thés...]" date="2018 12 12" journal="Tribune de Genève">
Ciprian Sauca présente sa gamme de thés froids dans l'ancien garage qui lui sert de bureau. GEORGES CABRERA Voici venir la bouteille compostable Antoine Grosjean @antogro72 Pour lui, le contenant est aussi important que le contenu. Ciprian Sauca, qui a lancé une gamme de thés froids artisanaux aux arômes raffinés, ne se voyait pas les vendre dans de vulgaires bouteilles en PET. Cela ne collerait pas avec la philosophie de Vitalay, sa petite entreprise installée à Bernex, dont il est le patron et sa mère l'unique employée. « Nous faisons un produit de qualité et de tradition, avec exclusivement des ingrédients naturels, souligne-t-il. Le mettre dans des bouteilles en plastique polluantes irait à l'encontre de tout ce que nous avons voulu réaliser. » Une matière première végétale C'est ainsi qu'il est tombé sur une start-up bordelaise ayant inventé un procédé pour fabriquer des bouteilles à base de matières végétales, en l'occurrence des déchets de canne à sucre. Celles-ci ressemblent en tout point à des bouteilles en PET, à la différence qu'elles sont biodégradables et compostables. Même l'étiquette est faite de canne à sucre, de lin et de chanvre. « Nous avons un emballage naturel pour un produit naturel, se félicite Ciprian. Seul le bouchon n'est pas biodégradable, pour l'instant. Si vous enfoncez cette bouteille dans la terre, elle va finir par se décomposer tout en nourrissant le sol » , explique l'entrepreneur de 28 ans, qui précise toutefois que ce n'est pas une raison pour la jeter n'importe où. Léger bémol, l'État n'en veut pas dans la « P'tite poubelle verte » pour les déchets organiques, parce qu'elle met plus longtemps à se décomposer que les épluchures de carotte ou les restes de repas. « Mais même si la bouteille doit finir à l'incinérateur, cela ne pollue pas vu que ce n'est que du sucre. Quand vous la brûlez, ça sent le caramel. » Quant au bilan carbone alourdi par la provenance de la bagasse de canne à sucre - l'Indonésie - c'est un moindre mal aux yeux de Maïko Nicolet, cofondateur de la chaîne de restaurants Inglewood, qui a été parmi les premiers à vendre les thés froids Vitalay: « Le plastique est un tel problème pour l'environnement qu'il est bien moins grave de faire venir de la canne à sucre d'Indonésie. » Les bienfaits de l'infusion à froid Ciprian Sauca s'est lancé dans l'aventure il y a quatre ans, après avoir découvert toute la finesse et la variété des thés haut de gamme ramenés de Chine par un ami. « On se faisait des soirées dégustation, se souvient-il. Le monde du thé est aussi riche que celui du vin. » À l'origine, il conditionnait son thé froid dans des bouteilles en verre, jusqu'à ce qu'il découvre le principe de l'infusion à froid. « Cela garde beaucoup mieux les arômes que l'infusion à chaud, explique Ciprian. On respecte la plante, on ne la stresse pas, elle peut s'ouvrir toute seule. Du coup, il n'y a pas d'amertume. Mais cela prend plus de temps, et puis il faut faire une pasteurisation à froid, ce qui n'est pas possible avec des bouteilles en verre. » D'où, de fil en aiguille, le choix des bouteilles biodégradables. Utilisant des recettes traditionnelles pour concocter leurs breuvages à base de thé wu long, de sencha ou de thé vert au jasmin, Ciprian et sa mère produisent bon an mal an entre 250 et 500 litres de thé froid par jour. Celui-ci se vend désormais dans une soixantaine de restaurants et une dizaine d'épiceries dans toute la Suisse romande. Maïko Nicolet et son frère Bastien, patrons d'Inglewood, ont tout de suite été séduits par ces thés, au point de cofinancer l'unité de production de Vitalay: « Nous avons totalement arrêté de vendre les grandes marques de thé froid, confie Maïko. Nous vendons même davantage de thé Vitalay que de Coca. »
</article>
<article titre="ALBIOMA-IMPALA/ (URGENT) Energies renouvelables-Impala devient actionnaire d'Albioma" date="2018 12 11" journal="Reuters économique">
PARIS, 11 décembre (Reuters) - Altamir a annoncé mardi céder pour 31,7 millions d'euros le solde de sa participation dans le producteur d'énergies renouvelables Albioma à la société Impala de l'entrepreneur Jacques Veyrat.   Le fonds de capital-investissement a cédé une participation représentant environ 5,5% du capital d'Albioma, parachevant son désengagement du producteur d'énergie à partir de la bagasse, résidu fibreux de la canne à sucre. "Je suis très heureux que Jacques Veyrat - au travers du groupe Impala - devienne un actionnaire significatif d'Albioma", commente Maurice Tchenio, Président d'Altamir Gérance, cité dans un communiqué.   "Sa connaissance profonde du secteur des énergies renouvelables au plan international devrait permettre d'accélérer la croissance d'Albioma", ajoute-t-il.   Impala emploie plus de 6 000 personnes dans l'énergie, l'industrie, les marques et la gestion d'actifs.   Le communiqué: https://bit.ly/2L8FiLs   (Gwénaëlle Barzic, édité par Jean-Michel Bélot)
</article>
<article titre="A Maurice, de l'électricité à partir de canne à sucre" date="2018 12 11" journal="Sciences et Avenir (site web)">
Depuis une quinzaine d'années, la bagasse, le résidu fibreux issu du broyage de la canne à sucre, est largement mise à contribution pour produire de l'électricité, au point désormais de pourvoir à 14% des besoins de l'île. L'île Maurice dispose d'un système tout particulier pour sa production d'énergie : 60% des besoins en électricité de l'île sont produits par quatre sociétés sucrières qui font tourner chacune une centrale thermique. Ces quatre centrales tournent normalement au charbon mais quand la saison de la canne arrive, la bagasse remplace le charbon comme combustible.   Récolte à la fin de l'automne En cette fin du mois de novembre, la récolte bat son plein dans les champs environnant la société Omnicane, située dans le sud de l'île Maurice. Une noria de poids lourds tirant d'immenses remorques viennent s'aligner près d'un entrepôt non moins impressionnant pour y décharger leur cargaison de canne à sucre fraîchement coupée. Durant la période de récolte, ce sont chaque jour 8.500 tonnes qui sont ainsi acheminées dans cette installation (soit environ 900.000 tonnes de canne dans l'année). Les tiges de canne sont alors broyées afin d'en extraire le jus qui servira à produire le sucre. La bagasse est de son côté lavée de manière à en extraire le maximum de liquide sucré, puis elle est chauffée pour en réduire le taux d'humidité. Elle part alors alimenter une centrale thermique où elle brûlera à plus de 500°C, une combustion qui permettra à des turbines de produire de l'électricité, dont une très grande partie sera acheminée sur le réseau national (le reliquat servant à alimenter l'entreprise).   "L'électricité est disponible 24h/24, à la demande, sans avoir à attendre le vent ou le soleil, puisqu'on peut stocker la bagasse comme on le ferait pour l'huile lourde et le charbon", se félicite Jacques D'Unienville, directeur général d'Omnicane. Quant au CO2 produit lors de la combustion de la bagasse, il est récupéré selon M. D'Unienville "pour la production de gaz carbonique destiné aux entreprises de boissons gazeuses". Au total, près d'un quart (22%) de l'énergie produite à Maurice est renouvelable, en prenant également en compte le solaire, l'éolien et l'hydroélectrique. Et "l'objectif du gouvernement est d'augmenter la part de l'énergie renouvelable dans le mix énergétique à 35% en 2025", explique le Premier ministre adjoint et ministre de l'Énergie, Ivan Collendavelloo. "Les 35% ne sont pas très loin. Nous aurons 11 parcs solaires d'ici l'an prochain (2019) et au moins deux parcs éoliens (...) Les producteurs indépendants de l'industrie sucrière continueront à fournir la plus grosse part d'électricité renouvelable à partir de la bagasse", ajoute M. Collendavelloo.   L'effondrement du cours du sucre menace le renouvelable Mais ce système de production pourrait rapidement trouver ses limites tant la filière sucrière mauricienne est confrontée à une concurrence internationale accrue, notamment depuis la fin en 2017 des quotas européens sur le sucre. "L'abolition des quotas des betteraviers depuis le 1er octobre 2017 ainsi que la surproduction en Thaïlande, au Brésil et en Inde ont entraîné une baisse drastique du prix du sucre sur le marché international et portent un coup fatal à l'industrie sucrière locale", s'alarme Jacqueline Sauzier, secrétaire générale de la Chambre d'agriculture de Maurice. La chute des prix mondiaux du sucre a entraîné un désengagement de nombreux petits producteurs à Maurice, dont la canne venait alimenter les gros acteurs sucriers. "Le nombre de petits planteurs est passé de 26.000 en 2010 à 13.000 en 2018. Une diminution drastique de 50% en 8 ans", reconnaît le ministre mauricien de l'Agro-industrie, Mahen Kumar Seeruttun.   La question désormais est donc de savoir si Maurice produira suffisamment de canne à sucre pour augmenter sensiblement la part de l'électricité issue de la bagasse dans sa production nationale. Les acteurs du secteur demandent des accès protégés à certains marchés pour pouvoir s'en sortir. "Maurice est une petite île vulnérable. Nous n'avons pas la capacité qu'ont la Thaïlande, le Brésil et l'Inde mais nous sommes un producteur efficient puisque nous valorisons toute la chaîne de production sucrière. Nous avons besoin d'avoir des accès protégés dans les marchés préférentiels. Les petits pays devraient avoir des quotas en priorité étant donné que nous sommes très vulnérables", plaide ainsi M. D'Unienville.  Cet article est paru dans Sciences et Avenir (site web)
</article>
<article titre="[Biométhanisation Partenaires, société commune d'Evergaz...]" date="2018 12 11" journal="Dechetcom (site web)">
Biométhanisation Partenaires, société commune d'Evergaz et Meridiam, renforce sa présence en France en faisant l'acquisition de la filiale Methaneo du Groupe Albioma, producteur d'énergie renouvelable indépendant dans la biomasse thermique et le photovoltaïque. D'une puissance de 3,2 MWe, les trois unités de méthanisationTiper, Capter et Sainter, exploitées par Methaneo sont situées dans les régions de la Nouvelle-Aquitaine et Pays de la Loire.     Producteur d'énergie renouvelable indépendant, Albioma est engagé dans la transition énergétique grâce à la biomasse (et au photovoltaïque). Le Groupe, implanté en Outre-Mer français, à l'Île Maurice et au Brésil, a développé depuis 20 ans un partenariat unique avec le monde sucrier pour produire de l'énergie renouvelable à partir de la bagasse, résidu fibreux de la canne à sucre.  Albioma cède son activité de méthanisation en France, à Evergaz : « les trois unités de Methaneo, mises en service depuis 2013, représentent une production annuelle de l'ordre de 20 GWh. Ce niveau a pu être atteint grâce à l'implication des équipes et aux améliorations des procédés industriels mis en oeuvre sous l'impulsion de notre entreprise. Aujourd'hui, nous nous réjouissons du rachat de Methaneo par Evergaz et Meridiam, au travers de Biométhanisation Partenaires, qui permettra la création d'un acteur moteur du développement de la filière méthanisation en France », a précisé Frédéric Moyne, Directeur Général d'Albioma.    Avec ces acquisitions, le portefeuille français d'Evergaz (qui a vu le jour en 2008) comprend dorénavant 9 unités de méthanisation, dont une en construction. En ajoutant l'Allemagne et la Belgique, Evergaz détient et exploite désormais 14 centrales biogaz, pour 28 MWe, soit 7 000 Nm3/h de biométhane et 626 GWh PCS produits chaque année.    Opérateur intégré spécialisé dans la production de biogaz et le développement de ses usages, Evergaz vise à l'horizon 2022 un objectif de production annuelle d'énergie biométhane de 120 millions Nm3, une puissance installée de 64 MWe et le traitement de plus d'1 million de tonnes de déchets organiques par an. « Ces acquisitions s'inscrivent parfaitement dans notre vision de constituer une plateforme de compétences spécialisées dans la détention et la gestion de centrales biogaz en France et en Europe. Nous nous réjouissons de travailler avec les partenaires locaux des trois unités pour que ces sites répondent pleinement aux enjeux économiques, environnementaux et sociaux des territoires sur lesquels ils sont implantés », a complété Alain Planchot, Président Directeur Général d'Evergaz.     Imprimer  Cet article est paru dans Dechetcom (site web)
</article>
<article titre="Evergaz reprend l'activité biogaz d'Albioma" date="2018 12 11" journal="GreenUnivers (site web)">
2018-12-11T18:55:47+00:00    © Evergaz   Et trois centrales de plus dans l'escarcelle d'Evergaz : le développeur-exploitant indépendant acquiert l'activité biogaz d'Albioma, soit trois unités de méthanisation agricole (Tiper, Capter et Sainter) développées par Methaneo. Situées en Nouvelle-Aquitaine et Pays de la Loire, elles représentent une puissance totale de 3,2 MWe.    14 centrales exploitées aujourd'huiï»¿ L'acquisition a été réalisée par Biométhanisation Partenaires, la société commune créée par Evergaz et le fonds infra Meridiam, qui s'est engagé au printemps dernier à apporter 30 M€ à son nouveau partenaire.    Avec cette opération, Evergaz détient aujourd'hui 14 centrales, dont 9 en France, 3 en Allemagne et 2 en Belgique. Soit une capacité de 28 MWe, pour une production de 7 000 Nm3/h de biométhane et 645 000 tonnes de déchets traités par an. A l'horizon 2022, l'entreprise vise une production de 120 millions Nm3 de biométhane, une puissance installée de 64 MWe et le traitement de plus d'1 million de tonnes de déchets organiques par an.   Albioma, une ambition contrariée dans le biogaz Albioma avait racheté Methaneo, pionnier du biogaz agricole, en 2012. Le groupe affichait alors de grandes ambitions pour ce secteur, avec un objectif de chiffre d'affaires annoncé à 40 M€ en 2017. Mais le développement a été entravé par la complexité de la technologie et de la réglementation, et Albioma a rapidement mis en sommeil son activité. Le groupe dirigé aujourd'hui par Frédéric Moyne s'est recentré sur les centrales biomasse à base de bagasse et sur le solaire, essentiellement dans les territoires ultramarins et au Brésil.    Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="Des électrons tirés de la canne à sucre" date="2018 12 10" journal="L'Avenir (site web)">
Isolée dans l’océan Indien, ne pouvant compter que sur elle-même pour son électricité, l’île Maurice diminue sa dépendance aux énergies fossiles.   Depuis une quinzaine d’années, la bagasse, le résidu fibreux issu du broyage de la canne à sucre, est largement mise à contribution pour produire de l’électricité, au point désormais de pourvoir à 14% des besoins de l’île Maurice. Quelque 60% des besoins en électricité de l’île sont produits par quatre sociétés sucrières qui font tourner chacune une centrale thermique. Ces centrales tournent normalement au charbon, mais quand la saison de la canne arrive, la bagasse sert de combustible. Fin novembre, la récolte bat son plein dans les champs environnant la société Omnicane, située dans le sud de l’île. Une noria de poids lourds vient s’aligner près d’un entrepôt non moins impressionnant pour y décharger leur cargaison de canne à sucre fraîchement coupée.   Durant la période de récolte, ce sont chaque jour 8 500 tonnes qui sont ainsi acheminées dans cette installation (soit environ 900 000 tonnes de canne dans l’année). Les tiges de canne sont alors broyées afin d’en extraire le jus qui servira à produire le sucre.   Disponible 24 h/24   La bagasse est de son côté lavée de manière à en extraire le maximum de liquide sucré, puis elle est chauffée pour en réduire le taux d’humidité. Elle part alors alimenter une centrale thermique où elle brûlera à plus de 500 °C, une combustion qui permettra à des turbines de produire de l’électricité, dont une très grande partie sera acheminée sur le réseau national (le reliquat servant à alimenter l’entreprise).   « L’électricité est disponible 24 h/24, à la demande, sans avoir à attendre le vent ou le soleil, puisqu’on peut stocker la bagasse comme on le ferait pour l’huile lourde et le charbon », se félicite Jacques D’Unienville, directeur général d’Omnicane.   Quant au CO2 produit lors de la combustion de la bagasse, il est récupéré selon M. D’Unienville « pour la production de gaz carbonique destiné aux entreprises de boissons gazeuses » .   Au total, près d’un quart (22%) de l’énergie produite à Maurice est renouvelable, en prenant également en compte le solaire, l’éolien et l’hydroélectrique.   Et « l’objectif du gouvernement est d’augmenter la part de l’énergie renouvelable dans le mix énergétique à 35% en 2025 », explique le Premier ministre adjoint et ministre de l’Énergie, Ivan Collendavelloo. Les 35% ne sont pas très loin. Nous aurons 11 parcs solaires d’ici l’an prochain (2019) et au moins deux parcs éoliens […] Les producteurs indépendants de l’industrie sucrière continueront à fournir la plus grosse part d’électricité renouvelable à partir de la bagasse. »   Mais ce système de production pourrait rapidement trouver ses limites tant la filière sucrière mauricienne est confrontée à une concurrence internationale accrue, notamment depuis la fin en 2017 des quotas européens sur le sucre.   « L’abolition des quotas des betteraviers depuis le 1er octobre 2017 ainsi que la surproduction en Thaïlande, au Brésil et en Inde ont entraîné une baisse drastique du prix du sucre sur le marché international et portent un coup fatal à l’industrie sucrière locale », s’alarme Jacqueline Sauzier, secrétaire générale de la Chambre d’agriculture de Maurice.   Le nombre de petits planteurs est passé de 26 000 en 2010 à 13 000 en 2018. Une diminution drastique de 50% en 8 ans.  Cet article est paru dans L'Avenir (site web)
</article>
<article titre="Du bon compost grâce au bio-plastique" date="2018 12 7" journal="L'Usine Nouvelle (site web)">
L'éco-organisme Citéo a conclu un partenariat avec les Alchimistes, une start-up intervenant dans la production de compost, pour valoriser le plastique végétal.   L'acide polylactique (PLA), un plastique bio-sourcé utilisé dans certains emballages alimentaires, se recyclera-t-il un jour comme le polyéthylène téréphtalate, le plastique des bouteilles d'eau ? En annonçant le 5 décembre un partenariat avec la start-up les Alchimistes, l'éco-organisme Citeo y travaille. Elle expérimente le compostage de ce plastique obtenu à partir d'amidon de maïs, de la fécule de pomme de terre ou encore de bagasse de canne à sucre. Car si la résine est d'origine végétale, cette matière- utilisée aussi sous forme de fil pour les imprimantes 3D-, ne se biodégrade pas chez soi, grâce à un composteur. Elle ne peut pas, non plus, rejoindre les filières de recyclage traditionnelles. En effet, mélangée aux polymères synthétiques, elle perturbe la qualité des matériaux en sortie. Sa fin de vie passe donc principalement par la valorisation énergétique.   trouver une fin de vie respectueuse de l'environnement au PLA   En terme de gisement, avec seulement 100 tonnes mises sur le marché en France, ce plastique fait figure de poids léger. Il est encore loin des 30 000 tonnes qui justifient la mise en place d'une filière de recyclage. A une période où les produits d'origine fossile sont pointés du doigt, cette résine intéresse les entreprises qui portent un discours éthique en faveur de l'environnement.   A l'instar de Yumi, une marque française de jus qui a misé sur le PLA pour ses contenants. En utilisant ce plastique biosourcé, les deux co-fondateurs ne s'attendaient pas à être sujet un à malus de la part de Citéo. Cette dernière pénalise en effet financièrement l'absence de solution de fin de vie. Mais la situation pourrait évoluer ces prochains mois. En ouvrant la voie au compostage industriel, Citéo et les Alchimistes entendent donner de nouvelles perspectives aux PLA. Les deux partenaires se sont en effet fixés un double objectif : certifier que le compostage industriel répond aux normes de qualité de compost et trouver un mode de collecte adapté pour ces emballages proche du bassin de consommation.   Premier compost Parisien   Débuté en septembre, le partenariat teste pour l'instant un compostage électro magnétique. Le procédé se décompose en trois phases : le pré-traitement, au cours duquel les paillettes issues du broyage de PLA sont mélangées avec des biodéchets, la maturation dans un conteneur fermé pouvant traiter jusqu'à 120 kgs de déchets alimentaires par jour et, enfin, l'étape de tests et d'analyses pour vérifier la qualité du compost.   L'objectif est l'obtention d'une labellisation répondant à la norme NFU 44 051 afin d'en faire le 1er compost « fabriqué à Paris » . Avant cela, la mise en place d'un circuit de collecte doit faire émerger une nouvelle filière en milieu urbain. Les Alchimistes espèrent ramasser un minimum de 500 bouteilles auprès de restaurants et de boutiques de la capitale. Les consommateurs sont aussi mis à contribution puisque un test de collecte sera déployé via les magasins et lors d'événements afin de les sensibiliser au compostage de ces emballages, encore mal connu. Le projet doit livrer ses premiers résultats d'ici mai 2019.
</article>
<article titre="Conseil communautaire Brioude Sud Auvergne" date="2018 12 1" journal="La Montagne  Haute-Loire">
Brioude Sud Auvergne Prochaine réunion mardi 4 décembre à 18 heures   La communauté de communes de Brioude Sud Auvergne se réunit mardi 4 à 18 heures en mairie de Brioude.   Les élus devront notamment se prononcer sur le projet de vente et sur l'autorisation de lancement d'une mise en concurrence préalable pour le camping de La Bageasse. Sont également à l'ordre du jour, le rapport d'activités 2017 pour la délégation de service public, l'autorisation de mandatement des subventions et des dépenses d'investissement avant le vote du BP 2019, des admissions en non-valeur et créances éteintes, des décisions modificatives au budget principal, aux budgets annexes du camping de La Bagasse et des abattoirs, les tarifs des services, services affermés et la redevance d'abattage 2019, la subvention exceptionnelle au comité de vigilance de l'hôpital de Brioude, le dépôt d'une candidature label commerce de proximité, le dépôt d'un dossier Leader pour l'audit des abattoirs et pour des actions en faveur du commerce, l'approbation des statuts du Sictom Issoire Brioude, le voeu du conseil communautaire en faveur de la préservation de l'engagement des sapeurs pompiers volontaires
</article>
<article titre="Électricité Du charbon à la biomasse en Guadeloupe" date="2018 11 29" journal="L'Usine Nouvelle, no. Hors série">
Après la mise en service de la centrale bagasse / biomasse Galion 2 en Martinique en septembre, l'opérateur Albioma a annoncé la conversion à la biomasse de sa centrale bagasse / charbon de Guadeloupe. La conversion permettra de fonctionner sans charbon hors des périodes de campagne sucrière. Elle élèvera la part des énergies renouvelable dans le mix énergétique de la Guadeloupe à environ 35 % (contre 20,5 % en 2017). Une fois convertie à la biomasse, la centrale réduira ses émissions de plus de 265 000 tonnes équivalent CO2 par an, soit une baisse de l'ordre de 87 % par rapport à son fonctionnement actuel au charbon. La centrale, d'une puissance installée de 34 MW, produit environ 260 GWh par an, soit 15 % environ de la consommation du territoire. §
</article>
<article titre="EDF EN signe un premier PPA au Brésil" date="2018 11 28" journal="GreenUnivers (site web)">
2018-11-28T14:50:45+00:00    Le Brésil, toujours un bon filon pour les énergéticiens français. EDF EN a signé un contrat de vente d'électricité avec le fabricant brésilien de résines thermoplastiques Braskem. Ce contrat de 20 ans concerne un projet éolien de 33 MW, qui doit être construit en 2019 et mis en service en 2021. Il s'agit d'une extension du parc de Folha Larga (114 MW), remporté dans le cadre de l'appel d'offres d'avril 2018. Pour rappel, EDF EN avait réussi un joli coup avec un prix record de 16,46 €/MWh, un montant très bas. Il a acquis par la suite les parcs de Folha Larga et Ventos de Bahia (183 MW). Avec l'extension, le portefeuille de projets éoliens, en construction ou en exploitation, d'EDF EN au Brésil sera porté à 1005 MW. Le groupe y possède aussi un portefeuille solaire de 398,5 MW. EDF EN n'est pas le seul Français à avoir un portefeuille EnR au Brésil. Voltalia a remporté 60 MW lors de l'appel d'offres éolien d'avril dernier. La production d'Albioma, qui possède quatre centrales de cogénération à partir de bagasse au Brésil, a atteint 180 GWh au T3 2018. Schneider Electric possède aussi des activités EnR dans le pays, et Engie est le premier producteur d'électricité privé.  Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="Télécharger .pdf (3 MB) - Credit Agricole Green Bond Investor Presentation_November 2018.pdf" date="2018 11 22" journal="Rapports - Banques et institutions financières - Crédit Agricole">
53 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... l 1 Green Bond – November 2018  Crédit Agricole Green Bond Investor Presentation  l 2 Green Bond – November 2018  Note:  The Crédit Agricole  Group scope of  consolidation  comprises:  the Regional Banks, the...    Pour voir le rapport :  https://www.credit-agricole.com/content/download/171790/3904640/version/2/file/Credit%20Agricole%20Green%20Bond%20Investor%20Presentation_November%202018.pdf
</article>
<article titre="Des substituts au plastique pollueur" date="2018 11 16" journal="Le Particulier (site web)">
La rédaction   De plus en plus de sites proposent de la vaisselle à usage unique biodégradable, et même compostable, anticipant ainsi l’interdiction de la vente des gobelets, verres et assiettes jetables en plastique à compter du 1er janvier 2020 (décret n° 2016-1170 du 30.8.16). Par exemple, biofutura.com, cotenature-dr.fr, laboutiquedujetable.fr, lsbio.fr et naturesparty.fr vendent des ustensiles en bois, bambou, pulpe de canne à sucre (bagasse), feuille de palmier, PLA (amidon de maïs), CPLA (bioplastique résistant à la chaleur), carton ou papier. Si vous préférez vous orienter vers des produits réutilisables, vous trouverez, sur boutiquezerodechet.com, stop-bpa.fr, gaspajoe.fr, des pailles en bambou, verre ou Inox, à nettoyer avec un goupillon. Ou encore, sur abeillons.fr, des carrés de toile en coton enduite de cire d’abeille lavables, en remplacement du film alimentaire. Enfin, certaines plateformes se sont spécialisées dans la commercialisation de vaisselle comestible : des verrines à base d’eau et de fécule de pomme de terre sur doeat.com, des mugs en biscuit enrobé de chocolat sur lespetitesfrancaises.fr.
</article>
<article titre="À bord du Jolly rouge pour un voyage en terre caribéenne" date="2018 11 10" journal="Midi Libre  MONTPELLIER">
Table. Un restaurant spécialisé dans le boucanage à la bagasse de canne à sucre. S'il a perdu ses célèbres tresses pour un crâne rasé et une barbe fournie, Garry Bocaly, l'ancien champion de France de football avec Montpellier en 2012, arbore toujours son large sourire communicatif. Et pourtant, ce jeune trentenaire en a connu des pépins physiques. Lui, l'ex-international espoir avec les Gourcuff, Ben Arfa et autres Nasri, qui a été déclaré inapte au football professionnel après une péritonite qui a failli lui coûté la vie à l'âge de 27 ans.   Le footballeur Garry Bocaly à la manoeuvre Alors, pour repartir du bon pied, il a franchi le pas en ouvrant un bar avec son pote, Martial Lawin, un barman réputé qui a pas mal tourné dans le secteur. Les deux compères ont d'abord ouvert un speakeasy (bar "clandestin") sous la boîte de nuit Le Huit, rue de l'Aiguillerie, pour tester le concept de leur Rhum runner, durant six mois, avant d'ouvrir un établissement ayant pignon sur rue, quartier Saint-Roch (12 rue Jules-Latreilhes). Un haut lieu pour les consommateurs de bons cocktails au rhum, à base, notamment, de canne à sucre bio, très parfumée et très florale.  « Et comme nous voulions aussi ouvrir un restaurant pour mettre en avant la richesse de la cuisine caribéenne, on s'est installé juste en face, au n°9 de la rue Jules-Latreilhe, avec notre enseigne Jolly rouge. Comme ça, on propose à nos clients apéro, repas et digestif créole sur une seule et même rue »  , confie avec un sourire "ultrabright" Martial Lawin, à l'origine du concept. Et pour ne rien gâcher, le binôme réutilise la bagasse de canne à sucre pour fumer les plats typiques caribéens au sein de leur four à braises. Coquelets, poissons, saucisses, ribs marinés... sont ainsi cuits à basse température durant de longues heures avant d'être fumés durant trois quarts d'heure. Généralement accompagnés de sauce curry ou cacahuètes et noix de cajou. Et pour ceux qui préfèrent rester sur de la cuisine française : entrecôtes, côtes de boeuf et autres côtelettes d'agneau sont proposées au barbecue par la cheffe Nidia Nandor, qui est parallèlement présidente de l'association des étudiants antillais et guyanais de Montpellier (Karakwela).  « L'objectif est de faire voyager les gens tant par nos cocktails que par la cuisine caribéenne »  , renchérit Garry Bocaly, qu'il n'est pas rare de voir lors des services du soir. Notez d'ailleurs que le Jolly rouge, ouvert du mardi au dimanche, propose des soirées à thème BBQ, le jeudi avec vente de viande au poids, et guinguette créole le dimanche de 17 h à 22 h avec buffet et DJ. À essayer les yeux fermés et le sourire aux lèvres, façon Garry Bocaly. LAURENT VERMOREL lvermorel@midilibre.com Jolly rouge (09 83 94 45 37) et Rhum runner (04 99 66 45 48). Nidia Nandor, Garry Bocaly et Martial Lawin à la manoeuvre. RDH
</article>
<article titre="Les bioplastiques Bio-on prennent pied au Tatarstan" date="2018 11 7" journal="Emballages magazine (site web)">
Le groupe de chimie verte italien a vendu une licence au groupe pétrochimique Taif pour lui permettre de produire des PHA à partir de 2020.-   Basé à Bologne (Italie), Bio-on est une entreprise de chimie verte dont l'activité consiste à développer des procédés pour fabriquer des bioplastiques à partir de ressources renouvelables, par fermentation. La société est spécialisée dans les polyhydroxyalcanoates (PHAs), une famille de plastiques issus de déchets agricoles, notamment de la bagasse de canne à sucre et de la betterave. La société vient de remporter un contrat de 17,6 millions d'euros avec le groupe russe Taif visant à permettre à ce dernier de produire du PHA au Tatarstan, l'un des états de la Fédération de Russie, où il est implanté. Sur ce montant, 5,6 millions d'euros sont destinés à l'exploitation de la licence pour utiliser la technologie, le complément- soit 12 millions d'euros- servant à la conception et à la fourniture des équipements. Taif rajoutera 90 millions d'euros à l'enveloppe, son objectif étant de construire un site d'une capacité de 10 000 tonnes par an, qu'il sera possible de porter à 20 000 tonnes annuelles. Remplacer les plastiques d'origine fossile   L'usine, dont le lieu d'implantation n'a pas été précisé, devrait entrer en fonctionnement en 2020. Parmi les principaux groupes russes actifs dans la pétrochimie, Taif veut utiliser les PHAs pour remplacer les plastiques d'origine fossile dans les domaines où ces matières sont traditionnellement utilisées ainsi que dans des secteurs à plus forte valeur ajoutée. La signature du contrat a eu lieu au Kremlin, à Moscou (Fédération de Russie), en présence du président de la Fédération russe Vladimir Putin et du président du Conseil des ministres italien Giuseppe Conte qui effectuait une visite officielle. Les PHAs sont crédités d'un fort potentiel pour des applications très diverses, y compris dans l'emballage et le conditionnement. Ils sont censés offrir les mêmes caractéristiques thermiques et mécaniques que les plastiques d'origine fossile tout en étant biodégradables et issus à 100% de ressources végétales alimentaires et non alimentaires. Une première usine utilisant les procédés Bio-on est déjà à l'oeuvre en Italie, à Castel San Pietro près de Bologne. Inaugurée en juin, elle va produire quelque 1000 tonnes par an de micro-granules pour la cosmétique.
</article>
<article titre="Nouveaux produits et nouvelles finitions pour Spectra" date="2018 10 29" journal="Emballages magazine (site web)">
Le fabricant d'emballages anglais a fourni un packaging biosourcé à la chaîne Firmdale Hotels. -   Fier d'une collaboration de longue date, Spectra a travaillé avec la chaîne Firmadale Hotels pour fournir des emballages durables dans ces 10 hôtels répartis entre Londres et New York. Pour cela, le fabricant britannique d'emballage en plastique a fourni des bouteilles en polyéthylène téréphtalate (PET) à base de canne à sucre. La ligne de produits pour la douche Rikrak de l'entreprise hôtelière affiche ainsi un impact environnemental réduit. Produite à partir d'éthanol de bagasse    « Nos biopolymères à base de canne à sucre, disponible en PET et en polyéthylène haute densité (PEhd), représentent un choix parfait pour les clients qui souhaite conserver le résultat final quand ils passent à un emballage durable, assure Dan Smith, le représentant des ventes de Spectra. Ce matériau véritable renouvelable apaise les craintes puisqu'il maintient les mêmes caractéristiques chimiques que le PET et le Pehd conventionnel. C'est-à-dire qu'il n'y a pas de différence dans les performances ou l'apparence du produit fini. »   La nouvelle gamme de Rikrak est produite à partir de l'éthanol récupéré dans la bagasse - le résidu des tiges de canne à sucre dont on a extrait le jus. Ces bouteilles reprennent le style du standard Messina de Spectra et sont disponibles en format de 50, 100 et 300 ml.   Un choix dans les matériaux   En plus de cette nouvelle offre, Spectra a lancé un nouveau format de sa gamme Eclipse. Avec ses épaules élégantes, les bouteilles remportent un franc succès auprès des clients. Désormais, il sera possible d'obtenir un emballage de 100 ml en PET. Jusqu'alors, cette contenance n'était réalisable qu'en PEhd. « En offrant des options comme le choix du matériau, de la capacité ou du col, nous proposons des alternatives lorsque la customisation n'est plus une option », insiste le directeur des ventes de l'entreprise, Jonathan Powell.   Dans un marché où la personnalisation devient un enjeu crucial, Spectra a récemment gonflé sa gamme de décoration. Avec sa finition Silk (soie), le fabricant d'emballage offre « une belle brillance douce ne présentant pratiquement aucune ligne d'écoulement visible par rapport aux couleurs nacrées conventionnelles » . Disponible pour les bouteilles en PET et les bouchons en polypropylène (PP) moulé par injection, cette gamme de finition comprend un large éventail d'effets colorants pour des rendus fluorescents, givrés, métallisés, nacrés ou encore mousseux. Elle s'adresse aux marques souhaitant des produits à l'apparence haut de gamme.   Spectra est un fabricant d'emballages en plastique pour les domaines de la beauté, de la cosmétique et du soin. Il propose des flacons et bouteilles en PET et en PEhd ainsi que des bouchons. La société assure être capable de fournir des matériaux biosourcés ou issus de plastique recyclé. Elle propose également une large gamme de personnalisation.
</article>
<article titre="[TÉLÉGRAMMES INAUGURATION Vous êtes fan...]" date="2018 10 25" journal="Midi Libre  LUNEL">
TÉLÉGRAMMES INAUGURATION Vous êtes fan de cuisine caribéenne ? Alors, n'hésitez pas à vous rendre ces jeudi, vendredi et samedi au Jolly rouge, le nouvel établissement de l'ancien joueur du MHSC Gary Bocaly et de son ami Martial Lawin, implanté au 9 rue Jules-Latreilhe, pour une inauguration on ne peut plus festive. Au menu, dorade en feuilles de bananier, coquelet boucané fumé à la bagasse de canne à sucre avec salade papaye-mangue, achards, rougail, sans oublier les crêpes au caramel lait de coco... Hummm ! GOURDIN Ce jeudi à 18 h 30, Sauramps propose une rencontre avec Henri Gourdin, dans l'auditorium du musée Fabre (boulevard Bonne-Nouvelle). Jean Hugo a créé, en soixante-dix ans de carrière, plus d'un millier de dessins et de peintures, plusieurs centaines de décors et costumes de théâtre, trente vitraux, trois cents gravures et gouaches illustrant une centaine de livres rares. Dans Jean Hugo, un pays selon mon goût (éditions de Paris-Max Chaleil), Henri Gourdin reconstitue la vie de l'artiste. Entrée libre.
</article>
<article titre="Albioma (ex-Sechilienne-Sidec) : une belle endormie" date="2018 10 15" journal="Boursier (site web)">
Perception.  Les idées reçues ont souvent la vie dure en bourse. Ainsi, Albioma est encore considéré par de nombreux investisseurs comme un dossier à risque. Les conflits sociaux qui émaillent assez régulièrement l'actualité du groupe et son positionnement géographique sur des zones exposées aux ouragans ne sont pas étrangers à cette perception dégradée. Encore que, dans le cas considéré, l'irrationnel l'emporte largement sur le rationnel ! Les contretemps subis par l'énergéticien ces dernières années n'ont eu en réalité qu'un impact mesuré sur le compte de résultat du groupe. Quant à l'autre épée de Damoclès qui pesait jusqu'ici sur l'avenir d'Albioma, à savoir les incertitudes sur la mise en service de Galion 2 en Martinique, elle est levée depuis une quinzaine de jours avec le début de son exploitation industrielle. D'une capacité de 40 MW, cette cette centrale de cogénération utilise comme combustible de la bagasse de canne à sucre et d'autres formes locales de biomasse végétale. Son entrée en fonctionnement va permettre de tripler la production d'électricité renouvelable de l'île. En outre, grâce à un contrat qui s'étale sur 30 ans, Albioma va pouvoir rentabiliser cet investissement en deux décennies, sachant que la contribution de Galion 2 à l'Ebitda du groupe est escomptée autour de 20 millions d'euros par an. Valorisation.  L'autre fait marquant de l'actualité du groupe est la prochaine mise en route de la centrale TAC de Saint-Pierre à La Réunion, un autre projet structurant du pipeline. L'entrée en service de cette unité fonctionnant au bioéthanol doit intervenir dans les toutes prochaines semaines avec des retombées difficiles à évaluer du fait de son mode de production intermittent. L'objet de cette centrale est de permettre à EDF de gérer les fluctuations de la demande, en période de pointe de consommation notamment. Représentant un investissement de 60 ME, ce site pourrait générer quelque 5 ME d'Ebitda en année pleine dans des conditions d'exploitation normales. Avec ces bonnes nouvelles et les nombreux projets encore dans les tuyaux, Albioma offre de fortes perspectives de croissance bénéficiaire pour les trois prochaines années. De fait, le free cash-flow devrait repasser dans le vert cette année, avant de franchir probablement le cap des 100 ME en 2020. Face à cela, la capitalisation boursière du dossier, inférieure à 550 ME, apparaît de bas étiage. D'autant que le groupe se mue progressivement en une valeur verte avec l'objectif d'atteindre une production à 80% renouvelable en 2023, contre 36% dix ans plus tôt. A 10,7 fois les attentes de bénéfice net pour 2019, Albioma est clairement revenu en zone d'achat. On peut mettre en portefeuille.Aymeric Val
</article>
<article titre="Le virage du foodtruck" date="2018 10 13" journal="Sud Ouest  Bassin d'Arcachon">
BERNADETTE DUBOURG   Ce mercredi après-midi, dans la cuisine de sa maison de Salles, Stéphane Perez pèle des pommes de terre (agatha), tout en surveillant des poivrons rouges au four. Dehors, dans la cour, le foodtruck au logo Food &amp; Saison (prononcer foudessaisons, pour le jeu de mots et l'esprit du concept) est ouvert, l'intérieur parfaitement nettoyé, pour s'aérer. Le mercredi, Stéphane Perez et sa compagne, Florence Philippe, gérante de la société, préparent le service du lendemain, et même leur participation, ce lundi, au premier défi des truckers, organisé dans le cadre du 10e salon professionnel CHR Expo et métiers de bouche à La Teste-de-Buch (1).   Produits frais et de saison   Avant de peler des pommes de terre et cuisiner des poivrons, Stéphane Pérez était dans l'événementiel pour les entreprises. Florence, dans la location longue durée de véhicules. Arrivés en Gironde en 2007, et installés à Salles en 2011, ils ont changé de vie en 2013 en lançant leur Foodtruck : " L'un des premiers de Gironde et le premier sur le Bassin ".   " La cuisine a toujours été dans ma vie, par mes origines familiales. J'ai grandi entre Marseille et Lyon, la capitale de la gastronomie. Il y a quatre ans et demi, on a tenté l'aventure ", sourit Stéphane.   Le couple a fait équiper leur camion par un aménageur de Villegouges : " Ce n'est pas un camion snack. Nous avons une cuisine à part entière, avec une cellule de refroidissement professionnelle, qui permet notamment de garder le boeuf haché à 2 degrés ", des appareils de cuisson dont une plancha " pour cuire vite, c'est de la cuisine de rue ", une friteuse " même si je ne propose pas toujours des frites avec les burgers ", des feux au gaz, et une hotte.   Stéphane et Florence cuisinent des " produits de saison, frais, de qualité, et en circuit court ".   Ils ont choisi de se positionner sur le burger, avec un " buns ", un pain spécialement créé par la boulangerie Roch, de Salles : " Moins brioché, il se rapproche plus du pain ". À l'intérieur, de la viande de boeuf, " label rouge de Chalosse ", mais aussi du canard, parfois du veau, du poulet mariné, l'été, et même du poisson un jour par semaine. Mais aussi un steack de légumes pour un burger végétarien : " 10 à 15 % de nos ventes ". Prix moyen du panier, 8 à 10 euros.   Un pain créé pour le burger   Le burger est servi dans des pochons en papier : " On passe de plus en plus à des emballages écoresponsables, avec du papier, du carton, du bagasse, résidu de canne à sucre, recyclable ". La tendance est aussi aux emballages consignés : " On le fait déjà avec nos crèmes caramel, ça permet également de fidéliser les clients ".   Après La Teste, le couple stationné désormais son foodtruck, le midi, à Bordeaux, au Haillan et Cestas. Plus des soirées d'entreprises, des fêtes, et même des mariages. Lundi, ils reviennent à La Teste, avec trois autres foodtruckers (Euskaltruck de Cestas, Wayo Wayo de Bordeaux, et Buteco de Bordeaux), pour un battle de cuisine. Frédéric Shueller, patron de Truck de Chef, et co-organisateur de ce défi, veut " montrer comment dans un petit espace, avec des ingrédients simples, on peut faire autre chose que des frites ".   (1) 10e salon CHR Expo et métiers de bouche, du 14 au 16 octobre, parc des expositions de La Teste-de-Buch,
</article>
<article titre="l'usine durable L'énergie sucrée de Tereos" date="2018 10 11" journal="L'Usine Nouvelle, no. 3580">
Comment faire tourner une usine de canne à sucre sans dépenser un centime en énergie ? Au Brésil, la coopérative sucrière française Tereos récupère la bagasse, ce résidu fibreux issu du broyage de la canne à sucre, pour alimenter ses sucreries. Brûlée dans une chaudière, la bagasse donne de la vapeur et de l'électricité qui permet à Tereos d'être autosuffisant en énergie toute l'année. Cette installation de cogénération lui permet également de revendre une partie de l'électricité produite au réseau brésilien d'électricité. « Nous signons des contrats de sept à huit ans avec les fournisseurs d'énergie du pays, ce qui garantit les prix sur la durée », explique Pierre Santoul, le directeur de Tereos Brasil. Dans le pays, Tereos dispose déjà de 21 chaudières. Une solution également mise en oeuvre par Tereos sur l'île de La Réunion où la coopérative exploite deux sucreries. Et pour faire face aux mois creux qui suivent la récolte de la canne à sucre, Tereos Brasil dispose d'un espace où peuvent être entreposés jusqu'à 150 000 tonnes de bagasse. Face au succès de cette production, Tereos a décidé d'augmenter les infrastructures. Sur son site historique de Cruz Alta, dans l'État de São Paulo au Brésil, une nouvelle chaudière à haute pression, « la première de ce type dans l'industrie sucrière brésilienne » et une nouvelle unité de cogénération sont en cours d'installation. Un investissement de 35 millions d'euros, avec, en ligne de mire, un accroissement de rentabilité. « La nouvelle chaudière devrait émettre 275 tonnes de vapeur par an, soit de quoi fournir l'énergie pour une ville de 400 000 habitants », précise Pierre Santoul. Des performances qui permettront à l'entreprise de fermer quatre anciennes chaudières basse pression.
</article>
<article titre="Les cleantech en Bourse [Dossier]" date="2018 10 4" journal="GreenUnivers (site web)">
2018-10-04T22:49:48+00:00    Avec le lancement aujourd'hui de son offre publique, Neoen franchit une nouvelle étape vers la Bourse. Sauf surprise, le développeur et exploitant d'énergies renouvelables devrait bientôt rejoindre les rangs des entreprises cotées des cleantech. Des rangs à vrai dire plutôt clairsemés puisque peu d'entreprises de la transition énergétique et écologique ont choisi la Bourse ces dernières années, malgré les efforts d'Euronext pour les attirer.   Maigre bataillon  Dans les énergies renouvelables, Albioma, champion de la biomasse à partir de bagasse, et Voltalia, développeur et exploitant d'énergies renouvelables (éolien, solaire..) comme Neoen, sont les deux ETI cotées. On trouve aussi quelques sociétés du stockage (McPhy Energy, Electro Power Systems), de la chimie verte (Carbios, Fermentalg), de la mobilité (Navya), de l'efficacité énergétique (Enertime, Lucibel) ou encore du traitement de l'eau (Bio-UV). Au global, un secteur morcelé avec des entreprises basées sur des business models très différents, ce qui rend les comparaisons impossibles au grand dam des analystes et à la différence des biotech, secteur vedette de la Bourse de Paris. Comment expliquer la faiblesse du bataillon vert ? Le secteur pâtit encore de quelques mauvaises expériences comme celles de Theolia dans l'éolien et Solabios dans le solaire. Des déconvenues qui ont échaudé plus d'un investisseur. L'absence d'une vraie réussite comme il en existe dans la biotech ou la medtech pèse aussi. Fin 2013/début 2014, les cleantech ont pourtant connu un bel engouement boursier : plusieurs start-up, comme Carbios, McPhy et Fermentalg, ont réussi des IPO largement sursouscrites et leurs cours se sont envolés. Las ! L'enthousiasme a vite été douché et les cours sont retombés à des niveaux décevants. "Les sociétés n'ont pas "délivré", constate Laurent Wilk, d'Invest Securities, l'un des rares analystes cleantech de la Place de Paris.   L'échec de Solairedirect a refroidi  A ce manque de résultats est venu s'ajouter en 2015 l'échec de Solairedirect qui a dû renoncer à son projet d'introduction en Bourse à la dernière minute, faute d'investisseurs. "A l'époque, plusieurs autres sociétés se préparaient à aller en Bourse : l'élan a été coupé net", poursuit Laurent Wilk. Résultat : depuis, les IPO cleantech se comptent presque sur les doigts d'une main, et celles qui se sont lancées n'ont pas forcément brillé à l'image de Navya qui a récolté moins qu'espéré en juillet dernier.   Le secteur a besoin de faire preuve de pédagogie pour éviter de provoquer des déceptions. "Le segment des producteurs d'EnR n'a rien à voir avec la biotech : il n'y a pas le côté " ticket de loto", ce sont des valeurs de fond de portefeuille", illustre l'analyste.   Neoen sera-t-elle la success story espérée ?   Neoen réussira-t-elle à redorer le blason des cleantech en Bourse ? "C'est peut-être l'étincelle que tout le monde attend", espère Laurent Wilk. Le momentum semble favorable car beaucoup d'institutionnels - notamment les fonds ISR - cherchent à verdir leur portefeuille. Reste cependant à voir l'impact de marchés financiers actuellement sous pression. Verdict le 16 octobre, jour prévu pour la fixation du prix de l'action Neoen.   Retrouvez ci-dessous notre sélection d'articles sur les cleantech en Bourse :  Neoen valorisée entre 1,4 et 1,5 Md€ après augmentation de capital   Neoen : à quoi seront sensibles les investisseurs ?   Global Bioenergies relève des fonds et chute en Bourse   Navya entre bien en Bourse mais récolte moins que prévu   Bio-UV lève 10 M€ lors de son introduction en Bourse   Lucibel repart à l'offensive et lève 4,4 M€ en Bourse   Voltalia lève 170 M€ en Bourse   Voltalia privilégie la Bourse pour lever des fonds et peaufine un modèle intégré   Introduction en Bourse post-Brexit : Enertime sauve les meubles   Cleantech en Bourse, la désillusion ?   Solairedirect ne fait pas le plein d'investisseurs et reporte son IPO   Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="Albioma : mise en service industriel de la centrale 100%  bagasse/biomasse Galion 2 en Martinique" date="2018 10 1" journal="Boursier (site web)">
Albioma  annonce la mise en service industriel, le 26 septembre 2018, de sa centrale 100 % bagasse/biomasse Galion 2 en Martinique. Les essais de la centrale ont débuté en janvier 2018, l'installation a pu fournir à la sucrerie du Galion la vapeur dont celle-ci avait besoin pendant la campagne sucrière, les tests de conformité au référentiel EDF se sont déroulés pendant l'été. La validation des résultats de cette dernière étape permet le démarrage du contrat conclu avec EDF pour une durée de 30 ans.   Sous l'effet de l'allongement du calendrier de mise en service, l'investissement du projet s'élève à 210 millions d'euros.   Galion 2 devient la première centrale 100% bagasse/biomasse en Outre-mer français Dédiée à la production d'énergie renouvelable, cette installation, d'une puissance installée de 40 MW, fournira toute l'année de l'électricité pour le réseau électrique de la Martinique à partir de la combustion de bagasse, résidu fibreux de la canne à sucre, d'autres formes locales de biomasse d'origine végétale et de résidus de bois d'oeuvre, en provenance d'exploitations forestières gérées durablement.   Galion 2 fournira aussi de la vapeur pour alimenter la sucrerie du Galion, partenaire historique d'Albioma, dans le cadre d'un échange vertueux avec celle-ci. À l'instar de toutes les installations thermiques du Groupe implantées en Outre-mer, à l'île Maurice et au Brésil, Galion 2 allie technologie de pointe et innovation en mettant en oeuvre le principe d'économie circulaire : la centrale apporte à la sucrerie du site une solution qui améliore son approvisionnement en énergie en optimisant la valorisation du résidu de la canne à sucre, la bagasse.   Plus largement, Galion 2 permet la création de plus de 40 emplois directs et contribue à pérenniser le tissu agricole, économique, industriel et social de la Martinique.   La centrale Galion 2 multipliera par trois la production d'électricité renouvelable sur l'île (passant de 7% à 22%) et contribuera à la décarbonation du mix énergétique martiniquais, dans le respect des normes environnementales les plus strictes.   Dans le contexte actuel de transition énergétique, les solutions proposées par Albioma de production d'une énergie stable et renouvelable, à partir de biomasse, permettent de garantir la stabilité des réseaux électriques et d'augmenter ainsi la part d'énergies intermittentes comme le solaire, notamment dans des zones isolées où le réseau est fragile...
</article>
<article titre="Albioma retrouve les 20 euros" date="2018 10 1" journal="Boursier (site web)">
Albioma  remonte sur les 20 euros ce lundi, en hausse de 1,5% après l'annonce la mise en service industriel, le 26 septembre 2018, de sa centrale 100 % bagasse/biomasse Galion 2 en Martinique. Les essais de la centrale ont débuté en janvier 2018, l'installation a pu fournir à la sucrerie du Galion la vapeur dont celle-ci avait besoin pendant la campagne sucrière, les tests de conformité au référentiel EDF se sont déroulés pendant l'été. La validation des résultats de cette dernière étape permet le démarrage du contrat conclu avec EDF pour une durée de 30 ans. Sous l'effet de l'allongement du calendrier de mise en service, l'investissement du projet s'élève à 210 millions d'euros. "Malgré un nouveau surcoût (l'investissement initialement prévu était de 170 ME), la mise en service de cette unité est une bonne nouvelle, qui clôt de nombreuses années difficiles sur le sujet" commente Portzamparc qui souligne que la contribution de Galion 2 en EBITDA devrait être de l'ordre de 20 ME. "Cette mise en service tardive nous conduit à anticiper un EBITDA 2018 de 159 ME, en bas de fourchette de la guidance (158-166 ME)" conclut l'analyste qui vise un cours de 23,7 euros en restant acheteur.   Galion 2 devient la première centrale 100% bagasse/biomasse en Outre-mer français Dédiée à la production d'énergie renouvelable, cette installation, d'une puissance installée de 40 MW, fournira toute l'année de l'électricité pour le réseau électrique de la Martinique à partir de la combustion de bagasse, résidu fibreux de la canne à sucre, d'autres formes locales de biomasse d'origine végétale et de résidus de bois d'oeuvre, en provenance d'exploitations forestières gérées durablement.   Galion 2 fournira aussi de la vapeur pour alimenter la sucrerie du Galion, partenaire historique d'Albioma, dans le cadre d'un échange vertueux avec celle-ci. À l'instar de toutes les installations thermiques du Groupe implantées en Outre-mer, à l'île Maurice et au Brésil, Galion 2 allie technologie de pointe et innovation en mettant en oeuvre le principe d'économie circulaire : la centrale apporte à la sucrerie du site une solution qui améliore son approvisionnement en énergie en optimisant la valorisation du résidu de la canne à sucre, la bagasse.   Plus largement, Galion 2 permet la création de plus de 40 emplois directs et contribue à pérenniser le tissu agricole, économique, industriel et social de la Martinique.   La centrale Galion 2 multipliera par trois la production d'électricité renouvelable sur l'île (passant de 7% à 22%) et contribuera à la décarbonation du mix énergétique martiniquais, dans le respect des normes environnementales les plus strictes.   Dans le contexte actuel de transition énergétique, les solutions proposées par Albioma de production d'une énergie stable et renouvelable, à partir de biomasse, permettent de garantir la stabilité des réseaux électriques et d'augmenter ainsi la part d'énergies intermittentes comme le solaire, notamment dans des zones isolées où le réseau est fragile...
</article>
<article titre="ALBIOMA : mise en service de la centrale Galion 2 en Martinique" date="2018 10 1" journal="Option Finance (site web)">
AOF - 01 octobre 2018    (AOF) - Albioma a annoncé la mise en service industriel, le 26 septembre 2018, de sa centrale 100 % bagasse/biomasse Galion 2 en Martinique. Les essais de la centrale ont débuté en janvier 2018, l'installation a pu fournir à la sucrerie du Galion la vapeur dont celle-ci avait besoin pendant la campagne sucrière, les tests de conformité au référentiel EDF se sont déroulés pendant l'été. La validation des résultats de cette dernière étape permet le démarrage du contrat conclu avec EDF pour une durée de 30 ans.   Sous l'effet de l'allongement du calendrier de mise en service, l'investissement du projet s'élève à 210 millions d'euros.   Cet article est paru dans Option Finance (site web)
</article>
<article titre="Préouverture Paris : le CAC40 va tenter de franchir les 5.500 points..." date="2018 10 1" journal="Boursier (site web)">
LA TENDANCE Déjà dans les cours, l'accord enfin trouvé cette nuit entre les Etats-Unis et le Canada, ne devrait pas constituer un véritable catalyseur à la hausse ce matin pour le CAC40. L'accord prévoit un accès au marché laitier pour les Etats-Unis, alors que le Canada sera exempté de taxe sur l'automobile. Il était moins une pour trouver un accord... A quelques minutes seulement de l'expiration de l'ultimatum fixé par Washington, les Etats-Unis, le Canada et le Mexique sont parvenus à un compromis pour remplacer l'Aléna, l'Accord de libre-échange nord-américain. Les indicateurs de préouverture annoncent une ouverture en très légère hausse pour le CAC40, proche des 5.500 points. Les nuages qui s'accumulent sur la croissance chinoise, vont toutefois peser sur la tendance. Les indices des directeurs d'achats (PMI) manufacturiers chinois sont ressortis en recul en septembre, à 50,0 pour le PMI Caixin-Markit, son plus bas   niveau depuis mai 2017, et à 50,8 pour le PMI officiel.   La guerre commerciale semble bel et bien peser sur la croissance... Dans ce contexte, les chiffres définitifs des PMI manufacturiers européens devraient animer la matinée sur les marchés. Les investisseurs surveilleront par ailleurs les débats de l'Eurogroupe à Bruxelles, à l'affût de nouvelles informations sur le budget italien.    WALL STREET Les actions américaines ont achevé la semaine sur une note presque stable, vendredi, en l'absence d'éléments déterminants en cette dernière séance boursière du 3ème trimestre. Les titres de  Tesla  et de  Facebook  ont pesé sur la cote, tandis que le pétrole a poursuivi son ascension, certains analystes voyant le baril de Brent atteindre les 100$.  Tesla  a plongé de 13,9% après les accusations de fraude de la SEC contre son patron Elon Musk, tandis que  Facebook  a flanché de 2,6% après la révélation d'un cyber-piratage qui a touché au moins 50 millions de ses 2,2 milliards d'utilisateurs dans le monde.   A la clôture, l'indice  Dow Jones  a progressé de 0,07% à 26.458 points, tandis que l'indice large  S&amp;P 500  a fini stable à à 2.914 pts, et que le  Nasdaq  composite, riche en valeurs technologiques et biotechnologiques, a avancé de 0,06%, pour finir à 8.046 pts.   ECO ET DEVISES France :    - PMI manufacturier français final. (09h50)   Etats-Unis :    - Indice PMI manufacturier américain. (15h45)    - Indice ISM manufacturier. (16h00)    - Dépenses de construction. (16h00)   Europe :    - PMI manufacturier espagnol. (09h15)    - PMI manufacturier italien. (09h45)    - PMI manufacturier allemand final. (09h55)    - Indice PMI manufacturier européen final. (10h00)    - Taux de chômage italien. (10h00)    - Indice PMI manufacturier final britannique. (10h30)    - Taux de chômage en Europe. (11h00)   La parité euro / dollar atteint 1,1587$ ce matin. Le baril de Brent se négocie 83,19$. L'once d'or se traite 1.187$.   VALEURS A SUIVRE Kaufman &amp; Broad  annonce ses résultats pour les neuf premiers mois de l'exercice 2018. Le chiffre d'affaires s'établit à 1.096 ME, en progression de 15,7% par rapport à la même période de 2017. Le résultat opérationnel courant s'établit à 98,3 ME à comparer à 75,4 ME sur les neuf premiers mois de 2017. Le taux de marge opérationnelle courante s'élève à 9% contre 8% à la même période en 2017. L'EBIT ajusté du groupe s'élève à 106,3 ME sur les neuf premiers mois 2018 à comparer à 80,6 ME à la même période en 2017. La marge d'EBIT ajusté s'élève à 9,7% contre 8,5% à la même période en 2017. Le résultat net - part du groupe ressort à 51,8 ME vs 33,2 ME sur les neuf premiers mois de 2017.   Bonduelle  : La rentabilité opérationnelle courante au titre de l'exercice 2017-2018 s'établit à 123,6 millions d`euros, soit une croissance de +14,2% en données publiées. Cette rentabilité permet d'afficher une marge opérationnelle courante de 4,5%, en progression de 60 bp à périmètre comparable et étayée par des investissements marketing en hausse de 6%. La charge nette des éléments non récurrents s'élève à 4,6 millions d'euros et comprend pour l'essentiel des frais liés à des franchises d`assurance sur des événements climatiques aux Etats Unis et de réorganisation. Le résultat net ressort à 72,3 millions d'euros, en progression de +20,9% et représente, à l'instar de l`exercice précédent, 2,6% du chiffre d`affaires.   Casino  a signé une promesse de vente avec un investisseur institutionnel de premier plan portant sur les murs de 55 magasins Monoprix ayant une répartition homogène avec celle du parc de magasins avec 19 actifs en Ile-de-France. Après déduction des droits d'enregistrement, la transaction s'élève à 565 ME pour un loyer annuel de 27 ME. Les produits de cession seront encaissés le 27 décembre 2018 au plus tard...En incluant la cession de 15% de  Mercialys  via un TRS, les opérations réalisées au titre du plan de désendettement s'élèvent à 778 ME.   Sanofi  a finalisé la cession de son activité Génériques européenne à Advent International (Advent) le 30 septembre. L'opération a été menée à son terme en avance sur le calendrier prévu pour un montant de 1,9 milliard d'euros (valeur de l'entreprise). Cette cession s'inscrit dans le cadre de la stratégie de simplification et de réorganisation déployée par Sanofi.   Société de la Tour Eiffel  -  Affine  : Les Conseils d'administration respectifs ont approuvé le projet d'une opération de fusion. Ce rapprochement créerait une foncière cotée sur Euronext Paris, bénéficiant du statut SIIC et dont le patrimoine atteindrait plus de 1,7 milliard d'euros. La fusion envisagée prendrait la forme d'une fusion-absorption d'Affine par STE sur la base d'une parité d'échange de 1 action nouvelle STE pour 3 actions Affine. La parité proposée sera soumise aux commissaires à la fusion.   Recylex  : enregistre une perte nette consolidée part du Groupe de -1,8 ME sur le semestre (+6,7 ME au 1er semestre 2017).   Airbus  aurait choisi le successeur de Tom Enders, le grand patron du géant aéronautique. Et ce n'est pas une surprise. Citant les "traditionnelles" sources proches du dossier, Reuters indique que le groupe se prépare à nommer Guillaume Faury, l'actuel patron de sa puissante division d'avions commerciaux, au poste de président exécutif d'Airbus.   CHIFFRES D'AFFAIRES / RESULTATS PUBLIES  :   Medasys ,  Stentys ,  Electro Power Systems ,  Tipiak ,  Hubwoo ,  DMS Group ,  Prologue ,  Crosswood ,  O2i ,  ADL Partner , Roctool ,  Metabolc Explorer ,  Safe Orthopaedics ,  Dietswell ,  Adthink .   ANALYSTES Goldman Sachs remonte son avis de 'vente' à 'neutre' sur  Scor  avec un cours cible réhaussé de 30 à 38 Euros.   Kepler Cheuvreux passe à 'conserver' sur  Kaufman &amp; Broad  avec un cours cible de 40 euros.   GreenSome Finance passe à l'achat sur  Safe Orthopaedics  en ciblant un cours de 0,40 euro.   EN BREF  :   Tarkett  finalise l'acquisition de Lexmark.   Albioma  met en service la centrale 100% bagasse/biomasse Galion 2 en Martinique.   Faurecia  a finalisé l'acquisition de Parrot Faurecia Automotive.   Innate Pharma  publie des résultats pour IPH4102 dans les lyphomes T cutanés.
</article>
<article titre="Les valeurs à suivre aujourd'hui à la Bourse de Paris - Lundi 1er octobre 2018" date="2018 10 1" journal="Option Finance (site web)">
AOF - 01 octobre 2018    (AOF) - AB SCIENCE   AB Science a accusé au premier semestre 2018 une perte nette 11,121 millions d'euros contre 13,511 millions au 30 juin 2017. Le résultat opérationnel en revanche correspond à une perte de 14,192 millions, contre une perte de 13,709 millions au 30 juin 2017. Les produits d'exploitation, exclusivement constitués du chiffre d'affaires lié à l'exploitation d'un médicament en médecine vétérinaire, se sont élevés au 30 juin 2018 à 872 000 euros, contre 842 000 euros un an plus tôt. ALBIOMA   Albioma a annoncé la mise en service industriel, le 26 septembre 2018, de sa centrale 100 % bagasse/biomasse Galion 2 en Martinique. Les essais de la centrale ont débuté en janvier 2018, l'installation a pu fournir à la sucrerie du Galion la vapeur dont celle-ci avait besoin pendant la campagne sucrière, les tests de conformité au référentiel EDF se sont déroulés pendant l'été. La validation des résultats de cette dernière étape permet le démarrage du contrat conclu avec EDF pour une durée de 30 ans.    INNATE PHARMA   Innate Pharma présente aujourd'hui de nouvelles données de l'essai clinique de phase 1 évaluant IPH4102 chez des patients présentant un lymphome T cutané (LTC) en rechute ou réfractaire. IPH4102 a montré une activité clinique encourageante, avec un taux de réponse élevé et une survie sans progression durable. Les patients atteints de Syndrome de Sézary traités dans l'escalade de dose montrent une médiane de survie sans progression proche d'un an.    TARKETT   Tarkett a finalisé l'acquisition de Lexmark Carpet Mills. Paul Cleary, directeur général de Lexmark rejoint Tarkett en qualité de président du segment hôtellerie en Amérique du Nord. L'acquisition sera immédiatement relutive pour la marge d'Ebitda du groupe, jusqu'à 50 points de base après synergies en année pleine. Cette opération sera financée au moyen d'une ligne de crédit existante. Sur une base pro forma, Tarkett prévoit de maintenir un ratio d'endettement net inférieur à 2,5 fois l'Ebitda fin 2018.   FAURECIA   Faurecia a finalisé l'acquisition à 100% de Parrot Faurecia Automotive. Cette opération fait suite à l'accord annoncé le 5 juillet dernier entre Parrot et Faurecia pour la prise de contrôle anticipée à 100% de Parrot Faurecia Automotive par Faurecia. La transaction est réalisée sur la base d'une valeur d'entreprise de 100 millions d'euros identique à celle retenue lors de l'entrée au capital par Faurecia en 2017.   CASINO   Le 28 septembre 2018, Casino a signé une promesse de vente avec un investisseur institutionnel de premier plan portant sur les murs de 55 magasins Monoprix ayant une répartition homogène avec celle du parc de magasins avec 19 actifs en Ile-de-France. Après déduction des droits d'enregistrement, la transaction s'élève à 565 millions d'euros pour un loyer annuel de 27 millions. Les produits de cession seront encaissés le 27 décembre 2018 au plus tard.   SANOFI   Sanofi a finalisé la cession de son activité Génériques européenne à Advent International (Advent) le 30 septembre. L'opération a été menée à son terme en avance sur le calendrier prévu pour un montant de 1,9 milliard d'euros (valeur de l'entreprise). Sanofi avait annoncé le début de négociations exclusives avec Advent en avril dernier et leur conclusion le 28 juin. Cette cession s'inscrit dans le cadre de la stratégie de simplification et de réorganisation déployée par Sanofi.   ADTHINK   Adthink, le spécialiste de la publicité, affiche un résultat net de -1,52 million d'euros au premier semestre 2018, à comparer avec +4,49 millions un an plus tôt. L'Ebitda ressort à -0,53 million contre 1,21 au premier semestre 2017. Le chiffre d'affaires est en recul à 7,19 contre 11,23 millions d'euros. Les résultats en baisse de ce premier semestre 2018 ont été marqués par la finalisation du plan de cession des activités d'édition, ainsi que par le recentrage sur une offre de publicité digitale à la performance. Une étape stratégique qui a modifié le périmètre d'activité de la société.    KAUFMAN &amp; BROAD   Kaufman &amp; Broad a publié ses résultats pour les neuf premiers mois de l'exercice 2018 (clos au 31 août). Ainsi, le promoteur immobilier a dévoilé un résultat net - part du groupe- de 51,8 millions d'euros, contre 33,2 millions d'euros un an plus tôt. De son côté, le résultat opérationnel courant s'établit à 98,3 millions d'euros, à comparer à 75,4 millions d'euros sur les neuf premiers mois de 2017. De ce fait, le taux de marge opérationnelle courante s'élève à 9% contre 8% à la même période en 2017.    ROCTOOL   Roctool vient de dévoiler ses résultats du premier semestre 2018. Ainsi, le spécialiste des technologies de chauffage et de refroidissement des moules pour les plastiques et les composites, a publié un Ebitda de -1,1 million d'euros (contre une perte de 0,3 million d'euros au premier semestre 2017). Le chiffre d'affaires s'établit à 3,5 millions d'euros, en progression de 10% par rapport au 30 juin 2017.    ADTHINK    Adthink, le spécialiste de la publicité, affiche un résultat net de -1,52 million d'euros au premier semestre 2018, à comparer avec +4,49 millions un an plus tôt. L'Ebitda ressort à -0,53 million contre 1,21 au premier semestre 2017. Le chiffre d'affaires est en recul à 7,19 contre 11,23 millions d'euros. Les résultats en baisse de ce premier semestre 2018 ont été marqués par la finalisation du plan de cession des activités d'édition, ainsi que par le recentrage sur une offre de publicité digitale à la performance. Une étape stratégique qui a modifié le périmètre d'activité de la société.    RECYLEX   Recylex a accusé au premier semestre 2018 une perte nette de 1,8 million d'euros, contre un bénéfice de 6,7 millions un an plus tôt. Le spécialiste du recyclage des métaux lourds a été pénalisé par le ralentissement de la production du segment Plomb. Le chiffre d'affaires a reculé de 11% à 194,6 millions. Au chapitre des perspectives, le groupe a rappelé que les cours des métaux en euros avaient enregistré une évolution favorable en début d'année 2018, avant de subir une légère correction au deuxième trimestre.   BONDUELLE   Le spécialiste de la transformation industrielle et conserve des légumes a publié des résultats annuels solides.  Cet article est paru dans Option Finance (site web)
</article>
<article titre="ALBIOMA : mise en service de la centrale Galion 2 en Martinique" date="2018 10 1" journal="Agence Option Finance (AOF)">
(AOF) - Albioma a annoncé la mise en service industriel, le 26 septembre 2018, de sa centrale 100 % bagasse/biomasse Galion 2 en Martinique. Les essais de la centrale ont débuté en janvier 2018, l'installation a pu fournir à la sucrerie du Galion la vapeur dont celle-ci avait besoin pendant la campagne sucrière, les tests de conformité au référentiel EDF se sont déroulés pendant l'été. La validation des résultats de cette dernière étape permet le démarrage du contrat conclu avec EDF pour une durée de 30 ans.   Sous l'effet de l'allongement du calendrier de mise en service, l'investissement du projet s'élève à 210 millions d'euros. AOF - EN SAVOIR PLUS  Services aux collectivités    Sous l'effet des fusions et acquisitions, le secteur mondial des utilities est en pleine reconfiguration. Selon une étude d'AT Kearney, 1040 opérations de ce type ont été annoncées dans le monde en 2016, pour un total de 329 milliards de dollars, montant record en valeur. AT Kearney estime que le marché bénéficie d'abord des cessions d'actifs opérées par les grands énergéticiens européens.   Comme le souligne le cabinet de conseil et d'audit PwC, l'année 2016 a été un exercice record en matière de fusions-acquisitions dans l'énergie : près de 300 milliards de dollars, en hausse de 47% par rapport à 2015. Et effectivement l'Europe a joué un rôle essentiel dans cette accélération car les transactions sur ce continent ont bondi de 25%, à 49 milliards de dollars. En France, 45 opérations de fusions-acquisitions ont été recensées en 2016, pour une valeur totale de 5,4 milliards de dollars. La prise de participation minoritaire de CNP Assurances et de la Caisse des dépôts (CDC) dans Réseau de transport d'électricité (RTE), la filiale d'EDF, pour 4,6 milliards de dollars, a surpassé toutes les autres.
</article>
<article titre="Pollution. Des bactéries à l’assaut du chlordécone" date="2018 09 28" journal="Ouest-France (site web)">
Et si des chercheurs guadeloupéens étaient sur la piste d’une solution pour éliminer le chlordécone ? La lueur d’espoir vient de l’université d’Antilles-Guyane où l’équipe de Sarra Gaspard, professeur en chimie au sein du laboratoire Covachim M2E enchaîne les thèses sur la question. Partis du traitement de l’eau   Les scientifiques se sont d’abord intéressés à la question du traitement de l’eau.   « Notre laboratoire fabrique, c’est une de ses spécialités depuis avant les années 2000, des charbons actifs à partir de biomasse locale, de la bagasse, de la noix de coco, des racines de vétiver, etc. , énumère Sarra Gaspard.   On part de ce que l’on trouve localement. »  Les pièges des charbons actifs   Le procédé des charbons actifs ?  « Des pièges. Leur structure interne, c’est comme une éponge avec des trous. »   De cette manière,   « ils peuvent fixer les molécules ».   Avec des capacités surprenantes.   « Il faut savoir qu’un gramme de charbon actif peut avoir une surface développée qui correspond à un terrain de football. »  Des bactéries pour attaquer le chlordécone   L’équipe de chercheurs a voulu poursuivre ses travaux reposant sur une méthode totalement naturelle. Elle s’est interrogée sur la manière de traiter les déchets obtenus avec les charbons actifs remplis de chlordécone. C’est ainsi qu’elle a trouvé une technique biologique. Avec  « des bactéries qui, mises au contact du polluant, vont s’acclimater et, finalement, pouvoir le dégrader ».  « Résultats prometteurs »    De là, à imaginer que l’on n’arrivera un jour à éliminer le chlordécone des sols des Antilles… La marche est encore grande mais pas insurmontable.   « On est en train de travailler là-dessus,  annonce sereinement la chimiste guadeloupéenne.   On a des résultats prometteurs en tous les cas. On a trouvé dans des sols anciennement pollués des bactéries qui, mises dans des conditions particulières, en l’absence d’oxygène, sont capables de dégrader la chlordécone. »  Besoin de moyens   Principale difficulté ?   « Savoir comment passer du laboratoire aux conditions réelles, savoir comment faire pour que ça marche. »   Pour l’instant, les chercheurs travaillent sur des milieux de culture, privés d’oxygène. Avant, peut-être, d’aller plus loin.   « Ça existe, les procédés anaérobies, ça n’est pas farfelu. Le truc, c’est qu’il faut voir comment le mettre en œuvre dans le sol. Ça demande des moyens. Il faudrait faire des tests en conditions réelles. »  Envie d’y croire   Pour cela, il faudra trouver des financements. Et, sans doute, des partenaires industriels spécialisés dans la dépollution. L’enjeu est de taille. Le petit million d’habitants des Antilles a envie d’y croire.  Cet article est paru dans Ouest-France (site web)
</article>
<article titre="Le pari brésilien du sucrier Tereos Au pays de l'agrobusiness, le sucre est roi. Le Brésil en est devenu le premier producteur mondial, et l'Etat de Sao Paulo représente 60 % de la production nationale. C'est ici que se développe la coopérative française" date="2018 09 27" journal="Le Monde">
Aperte de vue, la canne à sucre trace ses lignes vert vif épousant les reliefs. Une couleur qui tranche avec le rouge sang de la terre. Cette  " terra roxa ",  comme la dénomment les Brésiliens, si prisée de l'agrobusiness. Parfois, la silhouette d'un arbre à l'élégance surannée se détache sur l'horizon, témoignage d'un temps où la forêt régnait. Vous êtes au coeur de l'Etat de Sao Paulo, près de la ville de Sao Jose do Rio Preto.   Zones résidentielles barricadées, grands centres commerciaux, buildings... Apparemment, dans cette agglomération, l'argent de l'agrobusiness coule à flots. Alimenté aujourd'hui par deux sources principales : le suc de la canne et le latex de l'hévéa. Pourtant, ces cultures n'ont pas toujours été l'alpha et l'oméga de la richesse de cette région. Par le passé, les plantations de café régnaient en maître, avec force esclaves à l'origine. Elles ont migré depuis vers le nord, dans l'Etat de Minas Gerais. Remplacées un temps par la culture d'oranges, qui, elle, s'est déplacée vers le sud. La canne a pris le relais, l'Etat de Sao Paulo représentant désormais 60 % de la production nationale. Sachant que le Brésil est devenu le premier producteur mondial de sucre, avec 20 % de part de marché, et premier exportateur mondial, écoulant la moitié des 60 millions de tonnes échangées dans les flux internationaux. Avec 27 millions de tonnes, il est aussi le deuxième producteur mondial d'éthanol. Ces résultats illustrent la dynamique de l'industrie agricole brésilienne, au même titre que les positions de leader prises sur les marchés du soja, du boeuf, de la volaille, du café ou de l'orange. Au point que ses exportations agricoles ont pesé96 milliards de dollars (80 milliards d'euros) en 2017, soit 44 % desexportations totales, dans un pays où l'industrie agricole représente 22 % du PIB. Une stratégie de rouleau compresseur qui remonte aux années 1990.   La libéralisation du marché du sucre a servi de catalyseur. Et n'a pas manqué d'exciter les appétits. C'est ainsi que la coopérative française Tereos, qui pèse 5 milliards d'euros,est venue s'inscrire dans le paysage brésilien au tournant des années 2000. Un parcours pionnier mené sous la houlette d'Alexis Duval, nommé président du directoire du groupe en 2012, succédant ainsi à son père.   L'aventure brésilienne n'est pas de tout repos. Tereos a dû valser avec plusieurs partenaires. Il a fait ses premiers pas avec le groupe brésilien Cosan. Mais à la suite de l'acquisition de Beghin Say, en 2002, il trouve dans la corbeille de mariée la société brésilienne Guarani, qui sera l'ossature de son développement. Face à l'essor du marché de l'éthanol, l'autre débouché de la canne, il se rapproche de groupe pétrolier brésilien Petrobras, qui deviendra actionnaire de sa filiale en 2010. Cette alliance s'achèvera sept ans plus tard quand Petrobras, secoué par les affaires de corruption, choisira de se recentrer sur ses activités historiques. Aujourd'hui, Tereos fait cavalier seul. La bannière de la coopérative picarde, forte de ses 12 000 adhérents français, flotte sur les sept usines de broyage de canne achetées au fil du temps, toutes situées dans l'Etat de Sao Paulo. Cantonnant désormais Guarani au statut de marque commerciale.   " Une industrie capitalistique "En parallèle, le paysage concurrentiel a fortement évolué. Comme le raconte Jacyr Costa Filho, directeur de la division Brésil de Tereos, les grands groupes de négoce ont, les premiers, été attirés par l'aubaine sucrière. A l'image de Louis Dreyfus Company (LDC), Bunge ou ADM. Puis, l'essor de l'éthanol a entraîné dans son sillage les groupes pétroliers, comme BP, Shell et Petrobras. Shell s'est allié avec Cosan pour former Raizen, le plus gros opérateur de la canne au Brésil. Plus récemment, le géant chinois Cofco est entré dans le jeu.Cette ruée vers l'or vert de la canne n'est pas toujours couronnée de succès. La sortie de Petrobras, après avoir investi près de 700 millions dans l'aventure, en est une illustration. Après deux exercices déficitaires, LDC a dû injecter, cet été, 1 milliard de dollars dans son activité brésilienne, plombée par la dette. Il souhaiterait vendre. De même, l'américain Bunge cherche un repreneur pour ses actifs sucriers au Brésil, dans le rouge. En mai, il a annoncé son intention de mettre en Bourse ses usines, mais a dû renoncer à ce projet cet été. Fin août, il a finalement cédé des activités de trading au singapourien Wilmar.   Il est vrai que ces activités sucrières sont gourmandes en carburant financier.  " C'est une industrie capitalistique " , rappelle M. Duval. Il suffit pour s'en convaincre de visiter l'usine de Tereos à Cruz Alta. Une immense installation de brique et de métal où la canne fraîchement coupée des champs est broyée et transformée en sucres et en éthanol. Ce site historique est en travaux. Une nouvelle chaudière et une unité de cogénération sont en cours d'installation. Un investissement de 35 millions d'euros. Avec, en ligne de mire, un accroissement de rentabilité, quand le sous-produit de la canne, la bagasse, fournit l'énergie nécessaire à l'usine mais produit aussi une électricité vendue à des tiers.   A Cruz Alta, toujours, dans un bâtiment administratif, une " tour de contrôle " vient d'être inaugurée. Cette salle de contrôle informatisée permet de visualiser tout à la fois l'état des récoltes dans les champs et les cycles de production des usines. Avec la même obsession : améliorer les process et rentabiliser les lourds investissements. Un exercice délicat quand les pratiques ont connu une véritable révolution copernicienne.   " Il y a quinze ans, tout était manuel, il y avait 15 000 coupeurs de canne " , se souvient M. Duval. Aujourd'hui, la mécanisation règne. Et des drones surveillent l'état des cultures. Sachant que Tereos exploite lui-même la moitié des 300 000 hectares nécessaires à l'approvisionnement de ses usines. Et que le prix de la canne est fixé par l'université de Sao Paulo selon des indicateurs de marché. L'objectif est d'accroître la productivité, pour atteindre 100 tonnes à l'hectare, contre 83 aujourd'hui, alors que, paradoxalement, la mécanisation a fait baisser les rendements. A l'autre bout de la chaîne, Tereos vient d'annoncer un investissement de 50 millions d'euros avec l'opérateur logistique VLI dans le port de Santos pour l'export du sucre.   Pour M. Duval, le pari brésilien de Tereos est stratégique et rentable. Avec ses 10 000 salariés, cette filiale bénéficiaire, a affiché, malgré un real faible, un chiffre d'affaires de 750 millions d'euros et un résultat opérationnel de 200 millions en 2017. Cette année, alors que le prix du sucre est au plus bas, Tereos joue sur la flexibilité de sa production et mise sur l'éthanol, favorisé par la hausse du cours du baril. Il est vrai que, au pays de l'agrobusiness, 90 % des véhicules peuvent rouler avec de l'agrocarburant...   Laurence Girard
</article>
<article titre="Pourquoi Tereos fait pousser des usines au Brésil" date="2018 09 18" journal="L'Union (France)  L'Union Ardennes">
En 1932, des planteurs de betterave du nord de l'Aisne cassaient leur tirelire pour investir dans une distillerie coopérative. De rachat en fusion, la petite coop' devenue géante est le deuxième sucrier mondial. Tereos compte 25000 salariés et affiche 5 milliards d'euros de chiffre d'affaires. Pour parvenir à ce rang, le groupe coopératif est parti produire de l'amidon de blé ou de maïs en Asie ou du sucre de canne et de l'éthanol au Brésil. Dans ce pays, Tereos est implanté dans l'État de Sao Paulo, à 500 kilomètres de la côte. Alors qu'en France les arracheuses de betterave viennent de commencer leur travail, au Brésil la récolte de canne, commencée mi-avril, s'achèvera dans 15 jours.  sept usines, un parc de 1000 machines  Après avoir commencé au Brésil avec 62000 hectares en 2002, Tereos transforme aujourd'hui 300 000 hectares et en exploite 150 000 en propre. Par comparaison, les surfaces betteravières du groupe en France sont de 215000 hectares... Pour travailler la canne, Tereos s'est doté d'un parc de 1000 machines dont 400 tracteurs, 100 moissonneuses et 100 camions. Ces derniers alimentent pas moins de 7 usines dont l'action est coordonnée par un centre de contrôle, le «C3», installé sur le site de Cruz Alta. «Nous avons créé un concept de cluster qui n'existe pas ailleurs au Brésil», explique Pierre Santoul, directeur de Tereos Sucre et Energie au Brésil. Alors qu'en France, les usines fonctionnent au gaz qui représente 30% des coûts de production, celles de Tereos au Brésil produisent leur propre énergie. Elle est tirée des déchets de la canne à sucre après broyage, la bagasse.    Investissement portuaire  En plus de rendre les usines autonomes, la cogénération génère des surplus.Sur la campagne 2017/2018, Tereos aura vendu 1001 gigawatheures, ce qui est suffisant pour alimenter une ville de 1,5 million d'habitants. Les usines brésiliennes du groupe coopératif français produisent non seulement du sucre (1,8 million de tonnes en 2017/2018) mais aussi de l'éthanol (646000 m3 en 2017/2018). Il est possible industriellement de faire varier de 10% la part de l'une ou l'autre production pour profiter des opportunités de marché. Quand le sucre baisse, on ouvre davantage le robinet de l'éthanol et inversement. Dans un pays qui représente la moitié des exportations mondiales, la chaîne logistique est également primordiale. La production de sucre des plaines de l'État Sao Paulo, qui représente 60% du total national, converge vers le port Santos, le plus grand d'Amérique latine. Y affluent chaque année plus de 100 millions de tonnes de matières premières agricoles. Rançon du succès, trains comme camions bouchonnent à l'entrée en attendant d'être déchargés. Pour ne pas être ralenti Tereos a choisi de nouer un partenariat avec VLI, un opérateur ferroviaire et portuaire, qui a un terminal, à Tipalm, légèrement en amont du port. Un nouvel entrepôt va être construit pour Tereos à Tipalm et un autre dans les terres, à Guara, près des usines du groupe, les deux étant reliés par voie ferrée. Par ailleurs le sucrier français a signé avec VLI un contrat de 30 ans pour le transport d'un million de tonnes de sucre par an, marque d'une vision de long terme. On le voit, Tereos est sur les rails au Brésil. En 18 ans de présence, le groupe est devenu le 3e sucrier brésilien dans un pays où la production est morcelée (on compte 380 usines sucrières). Cette implantation est l'un des maillons d'une stratégie d'internationalisation et de diversification engagée par le groupe pour répondre à la baisse de consommation de sucre en Europe, à sa croissance en Asie comme à la globalisation des marchés. Cette option offensive et productiviste est-elle pertinente? Ou trop coûteuse et risquée? Ces questions sont justement au centre de la crise de gouvernance qui ébranle le groupe depuis plusieurs mois. Le directoire de Tereos, qui représente les managers salariés du groupe, est bien sûr convaincu d'être sur la bonne voie alors qu'une partie des représentants élus des coopérateurs veut semer le doute sur cette politique. Reportage au Brésil Julien Bouillé
</article>
<article titre="Tereos toujours plus ambitieux au Brésil Le sucrier français y investit pour doper sa productivité sur un marché mondial de plus en plus complexe." date="2018 09 17" journal="Le Figaro, no. 23046">
AGROALIMENTAIRE  En pilotage automatique, l'imposante machine agricole effeuille les cannes à sucre, avant de débiter leurs tiges. Bienvenue à Olimpia, au coeur du grenier à sucre du Brésil : la récolte en cours illustre l'avancée technologique à l'oeuvre dans la filière locale qui produit la moitié des 60 millions de tonnes de sucre échangées chaque année sur les marchés internationaux. On est loin de la coupe à la main, comme au début des années 2000, quand Tereos a débarqué au Brésil. La coopérative française avait racheté Guarani et sa sucrerie de Cruz Alta, près d'Olimpia.   Pionnier des géants internationaux du sucre à s'être implanté au Brésil, Tereos est plus que jamais convaincu du bien-fondé de sa stratégie. Comme tous les acteurs du secteur, le groupe détenu par 12 000 producteurs de betterave du nord-est de la France est pris dans la tourmente créée par la fin des quotas européens (supprimés en octobre 2017), la baisse du cours du real brésilien et les vicissitudes du climat, à l'origine de la hausse de la production en Inde et en Thaïlande. Autant d'éléments qui ont contribué à prolonger le bas de cycle des cours mondiaux, compliquant la situation des acteurs européens. Numéro trois au Brésil, Tereos y produit autant de sucre qu'en France.   « On n'est pas crédible sur le marché mondial sans être présent au Brésil,  assure Alexis Duval, président du directoire de Tereos.  Il faut avoir une offre globale et une présence industrielle pour accompagner la croissance de la demande mondiale. »  À l'heure où des acteurs internationaux, tels les traders Lou is Dreyfus et Bunge, ont mis en vente leurs actifs dans l'industrie sucrière au Brésil, certains s'interrogent sur la pertinence de la stratégie de Tereos, seul sucrier européen présent en Amérique du Sud.   « L'allemand Südzucker, leader mondial, répète depuis dix ans qu'il veut s'implanter au Brésil, sans pouvoir atteindre son objectif,  rappelle Alexis Duval, à l'origine de l'aventure brésilienne de la coopérative.  Et pour cause : les actifs de valeurs sont devenus très chers, et beaucoup d'autres n'ont pas d'intérêt, car le marché est devenu beaucoup plus complexe. »  Depuis son implantation, le groupe, longtemps épaulé par Petrobras, a densifié sa production. Il a construit ou acquis des sucreries pour bâtir un cluster dans la région de Sao José do Rio Preto, à 400 kilomètres de Sao Paulo. Il y a installé des distilleries pour produire de l'éthanol ainsi que des chaufferies pour produire de l'énergie à base de bagasse (résidu de la canne à sucre) et alimenter le réseau local. Objectif : multiplier les sources de revenus pour minimiser les risques et gagner en productivité. Tereos Açùcar e Energia traite désormais de la canne issue de 300 000 hectares, pour moitié cultivés par le groupe. Des drones survolent les plantations pour détecter les zones sous-performantes.   « Nous pouvons augmenter la productivité de nos terres et les faire passer de 83 à 100 tonnes de sucre par hectare d'ici cinq ans »  , assure Pierre Reboul, le patron de la filiale, qui compte sept sucreries.   Contribution aux profits  « Nous avons investi au bon moment et sommes bien mieux positionnés que d'autres au Brésil »  , assure Alexis Duval. La direction de Tereos révèle que sa filiale brésilienne a rapporté au groupe la moitié de son excédent brut d'exploitation l'an passé, soit 594 millions d'euros d'Ebitda, et que le résultat net réalisé au Brésil est de 60 millions par an depuis trois ans. Autant d'arguments sonnants et trébuchants destinés à rassurer les sceptiques sur la stratégie de Tereos hors d'Europe, dont les résultats sont critiqués par d'ex-membres du conseil de surveillance ( lire ci-dessous ).    « Depuis 2012, le résultat net cumulé de nos activités internationales s'est élevé à 147 millions d'euros, dont 118 ont été reversés aux coopérateurs sous forme de dividendes »  , assure Alexis Duval, qui a bien l'intension de persévérer dans son aventure brésilienne. Tereos vient ainsi d'investir 50 millions d'euros dans le terminal en cours de construction pour le géant local de la logistique VLI (filiale du minier brésilien Vale) dans le port de Santos, le plus grand d'Amérique du Sud. Là aussi, il s'agit de contrôler la filière et de gagner en productivité, grâce à une optimisation du temps de déchargement des trains.    « Ce qui compte pour nos clients internationaux, ce n'est plus le prix à la sortie de nos sucreries, mais celui à l'arrivée à leur usine,  explique Alexis Duval.  Or, la valeur ajoutée se trouve désormais dans la logistique. »   La complexité du marché mondial du sucre rend les investissements aussi cruciaux que les économies.
</article>
<article titre="Le Brésil, pays incontournable" date="2018 09 14" journal="Les Echos, no. 22781">
Les récolteuses de canne à sucre tournent à plein régime. La terre est sèche, l'herbe rare et jaune dans l'Etat de São Paulo, qui se prévaut de la meilleure terre à canne au Brésil. On y produit 60 % du sucre du pays. Les planteurs n'ont pas connu pareille sécheresse depuis cinq ans. Les rendements à l'hectare en seront indubitablement affectés, mais pas au-delà de 10 %, selon les prévisionnistes. Pas question d'irriguer pour autant. Cela coûterait bien trop cher. Et l'année dernière, la récolte a été exceptionnelle. Les stocks mondiaux sont élevés et les excédents pèsent sur les cours. Les Brésiliens s'attendent à des prix bas encore pendant deux ans. Une situation dans laquelle le pays doit faire jouer à plein ses atouts de compétitivité. Les coûts de production y sont nettement plus bas en raison de la main-d'oeuvre, de la durée de la campagne qui permet de rentabiliser les outils industriels et grâce à une énergie financièrement indolore issue de la combustion des déchets de la canne, la bagasse.   Le Brésil est de très loin le premier producteur de sucre devant l'Inde, avec 30 millions de tonnes de sucre en 2017, et en tant que tel fait et défait le marché mondial selon les volumes qu'il commercialise. Il est aussi le deuxième producteur mondial d'éthanol, avec 27 millions de tonnes l'an passé. Un carburant qui a retrouvé tout son intérêt avec la hausse des prix du pétrole et la baisse des prix du sucre. Il se vend deux fois moins cher à la pompe actuellement que l'essence, à environ 50 centimes d'euros, contre 1 euro. Le groupe sucrier français Tereos a des capacités de stockage, qui lui permettent d'attendre le moment où ses concurrents ont tout vendu. Près de 90 % des voitures au Brésil sont dotées de réservoirs « flex fuel », qui donnent la possibilité de choisir à la pompe la quantité d'éthanol en fonction du prix. Les moteurs peuvent tourner à 100 % à l'éthanol ou à 0 %.   Opportunités de croissance La consommation de sucre au Brésil tend à s'éroder, comme en Europe. Ce n'est donc pas là que sont les enjeux mais bien à l'export. Une tonne sur deux commercialisées dans le monde vient du Brésil.  « Un sucrier qui a fait le choix de l'internationalisation ne peut pas ne pas être au Brésil »  , dit Alexis Duval, le président du directoire. Tous les grands négociants y sont. Bunge, Cargill, Dreyfus..., dont certains qui perdent de l'argent n'ont pas caché leur souhait de vendre.  « Le sucre ne se résume pas au négoce. C'est un métier agricole. Un savoir-faire agronomique »  , explique Pierre Santoul, directeur de Tereos au Brésil. Autant d'opportunités de croissance externe à venir pour le groupe français.   Tereos a considérablement développé son activité depuis qu'il a acquis Guarani en 2002, quintuplant les volumes de canne broyée à 20,2 millions de tonnes. Il exploite 300.000 hectares de canne. Il est le troisième sucrier du pays avec sept sites de production de sucre, d'éthanol et d'énergie et un site de production d'amidon et d'édulcorants. Le chiffre d'affaires s'est établi à 3,7 milliards de reals (760 millions d'euros) et l'Ebitda à 952 millions (195 millions d'euros) sur l'exercice 2017-2018.   M.-J. C. (à São Paulo)
</article>
<article titre="Tables de passage des classifications des produits françaises (CPF) | Insee - Table de passage CPF rev.1 vers CPF rev.2 - table_CPA02-CPA08.pdf" date="2018 09 14" journal="Rapports - Marketing - INSEE">
139 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... Correspondence table CPA 2002 - CPA 2008  CPA 2002 CPA 2008 01.11.11 01.11.11Durum wheat Durum wheat  01.11.11 01.64.10Durum wheat Seed processing services for propagation...    Pour voir le rapport :  https://www.insee.fr/fr/statistiques/fichier/2579545/table_CPA02-CPA08.pdf
</article>
<article titre="Quand Yumi passe au 20 heures..." date="2018 09 13" journal="Emballages magazine (site web)">
Pourquoi une bouteille en PLA fait-elle l'objet d'un malus de 100% ?-   Les participants à l'édition 2018 de Stratégies Emballages organisé par Emballages Magazine le 28 juin dernier ont pu entendre Raphaël de Taisne expliquer pourquoi Yumi avait opté pour une bouteille biodégradable issue de la bagasse, un résidu de la canne à sucre. Le problème ? Fabriquée par Lyspackaging, cette bouteille en acide polylactique (PLA) fait l'objet d'un malus de 100% appliqué par Citeo. La société agréée pour la valorisation des emballages et papiers ménagers applique en l'occurence le barème qui concerne les perturbateurs de tri.   Fondateurs de cette jeune entreprise qui fabrique des jus de légumes bio pascalisés, Raphaël de Taisne et Louis de Bentzmann ne sont pas du tout d'accord avec cette mesure et défendent leur cause avec conviction comme en témoigne une tribune publiée dans Le Monde le 13 juillet. Le message ? "Nous attendons des actions concrètes pour valoriser les biodéchets et, a minima, une réécriture de la grille de malus (voire de bonus ?) appliquée par le gouvernement actuel." En quelque deux minutes, L'oeil du 20 heures de France 2 est revenu sur ce sujet quelque peu pointu le 11 septembre. Puis, dans l'enquête "Plastique, la grande intox" de Cash investigation diffusée juste après le journal télévisé, les plastiques biosourcés ne sont pas présentés comme une bonne solution... Regardez le sujet de France 2 consacré à Yumi : Pourquoi les bouteilles biodégradables sont-elles deux fois plus taxées que les autres bouteilles en plastique ?
</article>
<article titre="Albioma (ex-Sechilienne-Sidec) : les résultats évoluent de manière favorable" date="2018 08 8" journal="Boursier (site web)">
Porteurs.  Avec une base fidèle de près de 20.000 actionnaires individuels, Albioma est très suivi par les petits porteurs. Suite à la sortie partielle d'Altamir, la flottant s'est accru et deux nouveaux investisseurs se sont invités au capital avec la BPI et le producteur de spiritueux La Martiniquaise. Le modèle historique de la société repose sur la valorisation de la bagasse, un résidu de la canne à sucre, qui permet de fournir de la vapeur à la sucrerie et de l'électricité au réseau. En dehors de la campagne sucrière, le combustible complémentaire était jusqu'à présent du charbon mais il est remplacé progressivement par de la biomasse. Très bien implanté en Guadeloupe, à La Réunion et à l'Ile Maurice, Albioma commence à déployer son modèle dans d'autres pays. Le Brésil représente ainsi 40% du Milliard d'Euros d'investissements programmés par la société sur la période 2013-2023... Asie.  Associant Albioma et des partenaires locaux, le projet Vale do Paraná qui nécessite une enveloppe de 100 Millions de Dollars devrait être mis en service en 2021. A terme, Albioma n'exclue pas de s'installer dans d'autres pays d'Amérique Latine mais aussi en Asie. En dehors de la bagasse, le producteur d'énergie renouvelable devient progressivement un acteur qui compte dans le solaire. La société déploie ainsi des parcs photovoltaïques en Outre-mer tout en équipant les toitures de logements sociaux à la Réunion avec des panneaux solaires. Alors que le résultat net récurrent d'Albioma s'est élevé à 32,1 Millions d'Euros en 2017 contre 28,8 Millions d'Euros en 2016, il est prévu cette année entre 37 et 42 Millions d'Euros. L'ex-Sechilienne semble ainsi avoir résolu ses éternels soucis techniques et ses mouvements de grève à répétition... Dette.  Le financement des projets en cours de construction conduit à une augmentation du levier bancaire. La situation est néanmoins sous contrôle alors que la dette est à taux fixe et qu'elle est sans recours... Avec une action sur les 19 Euros, le résultat prévu pour cette année se paye 15 fois. Au vu de la moyenne sectorielle, la valorisation d'Albioma apparaît ainsi plutôt raisonnable d'autant plus que la société constitue une cible intéressante. Avec le désengagement progressif d'Altamir du capital, Albioma n'a plus d'actionnaire de référence. Les ambitions affichées par plusieurs poids lourds de la cote de se renforcer dans les énergies renouvelables pourraient les conduire à se pencher sur le dossier Albioma. On ne manquera pas de conserver cette ligne... On garde au chaud la position en Albioma.Christophe Voisin
</article>
<article titre="Albioma: résultats en hausse au 1S" date="2018 07 25" journal="Le Figaro.fr">
Le producteur français d'électricité Albioma, surtout présent Outremer, a vu son bénéfice net et son chiffre d'affaires progresser au premier semestre, ses centrales thermiques compensant une moindre production solaire. Sur les six premiers mois de l'année, la société a engrangé un bénéfice net de 20,6 millions d'euros, en hausse de 28%, pour un chiffre d'affaires de 202,3 millions d'euros en progrès de 3%. Si une partie de ses installations thermiques étaient à l'arrêt pour les mettre aux normes en matière de traitement des fumées, les compensations financières prévues ont permis de compenser les pertes de production, a expliqué Albioma dans un communiqué. Les centrales électriques antillaises ont aussi connu une performance "excellente", a ajouté l'entreprise.   La société a notamment mis partiellement en service en Martinique la centrale alimentée en biomasse et bagasse (résidus du broyage de canne à sucre) Galion 2, d'une capacité de 40 mégawatts (MW), et elle prévoit une mise en service complète d'ici la fin septembre. Elle permettra de fournir 15% de la consommation d'électricité de l'île. A l'inverse, la production de ses centrales solaires à La Réunion a été pénalisée par de fortes pluies.   Au Brésil, Albioma fait état d'un "très bon début de campagne sucrière", avec un volume de cannes broyées en hausse de 16% qui a permis d'augmenter la production d'électricité, dans un contexte favorable de prix du courant, en hausse sur le marché en mai et juin.   Le groupe a ainsi confirmé ses objectifs pour l'année 2018, avec un excédent brut d'exploitation (Ebitda) entre 158 et 166 millions d'euros et un bénéfice net de 37 à 42 millions d'euros.
</article>
<article titre="Belle soirée avec le Winston Band" date="2018 07 19" journal="L'Yonne républicaine  Nord ; Sud">
Belle soirée à Turny avec les Winston Band . Dans le cadre du cabaret itinérant du Festival en Othe, le groupe québecois Winston Band donnait, samedi à la salle des fêtes de Turny, leur dernier spectacle en France avant leur retour au Québec. Une chaude soirée qui a enthousiasmé les spectateurs.   Avec décontraction et dynamisme, ils ont enchaîné des airs traditionnels québécois et caribéens, interprétés de façon très folk rock. Se sont succédé des chansons traditionnelles comme La maladie d'amour ou des créations telles que Manger des grillons ou Coloniser Mars Ils ont fait bouger les spectateurs et leur ont fait chanter la recette de la bagasse « pomme, poire, eau, sucre, levure ». Leur plaisir d'être sur scène et de jouer ensemble était évident et ils ont su communiquer au public leur bonne humeur et leur joie d'être là. Une belle soirée qui a permis d'apprécier cette musique venue d'outre-Atlantique, écho des chants de nos compatriotes français émigrés il y a bien longtemps aux Amériques.  Cet article a également été publié dans les éditions suivantes : Sud, page 14
</article>
<article titre="United Caps investit tous azimuts" date="2018 07 6" journal="Emballages magazine (site web)">
Extension de son centre de R&amp;D en France, nouvelle usine au Royaume-Uni, innovations anticontrefaçon..., le bouchonnier luxembourgeois affirme ses ambitions.-   United Caps met le turbo. Le bouchonnier luxembourgeois a profité de l'inauguration de l'extension de son centre de recherche et développement (R&amp;D) de Messia (Jura), en France, pour dévoiler ses dernières « initiatives de croissance » . « Parce que, si nous ne sommes pas le plus gros fabricant, nous voulons être le meilleur en termes de développement et de valeur ajoutée offerte à nos clients », explique Benoît Henckes, le Pdg du groupe. Réactivité   À Messia, United Caps a porté la surface du centre à 600 m². Cela permet d'accueillir 30% d'ingénieurs et de techniciens supplémentaires - 25 personnes désormais - ainsi qu'une imprimante 3D pour la réalisation des prototypes. En outre, la réorganisation du service essai apporte plus de réactivité. L'entreprise enregistre de ce fait une réduction moyenne de 10 à 20% de ses délais de développement.   Étiquette IML   À l'occasion de cette inauguration, le plasturgiste présentait d'ailleurs le bouchon mis au point pour le café soluble Nescafé Gold. Nestlé souhaitant un aspect de métal brossé, il fallait envelopper la jupe de la coiffe d'un matériau 100% métallique. « Une opération jusqu'alors jamais réalisée », souligne Sébastien Rime, le directeur R&amp;D du groupe. Finalement, United Caps et Nestlé ont opté pour une solution étiquetée dans le moule (IML). Le groupe luxembourgeois met en avant ce projet - qui a pris deux ans du début des discussions au lancement commercial - pour rappeler que la moitié de ses produits sont conçus sur mesure. Au total, plus de 6 millions d'euros ont été injectés à Messia dans cette extension et sa première production.   Anticontrefaçon   D'autres innovations voient le jour grâce à cet outil. United Caps propose par exemple des solutions en polyéthylène à base de bagasse fourni par le brésilien Braskem. La lutte contre les contrefaçons constitue aussi un axe de travail important. Le groupe présente, ici, la technologie QR+, fruit d'une collaboration avec le belge Verstraete IML, qui combine un QR code et une empreinte digitale sur une étiquette IML ainsi que la gravure holographique, en partenariat avec une start-up française.   Royaume-Uni   Enfin, Benoît Henckes a annoncé l'implantation du groupe au Royaume-Uni à travers la construction d'une usine à Dinnington, près de Sheffield, pour un investissement de 20 millions d'euros. L'installation initiale est prévue sur 5000 m², avec la possibilité de s'étendre à 20 000 m² si nécessaire. Elle devrait entrer en service fin 2019 et être destinée aux bouchons et capsules pour boissons et produits laitiers dans un premier temps. À la clé, une augmentation prévue du chiffre d'affaires de l'ordre de 15%. Ce chantier est engagé alors qu'United Caps va inaugurer cet automne une nouvelle unité en Malaisie et qu'il a repris en avril 2018 l'espagnol Embalatap. « 2018 est une année exceptionnelle pour United Caps, conclut le Pdg. Nous nous positionnons pour accompagner encore mieux nos clients et pour accroître le nombre de produits que nous pouvons commercialiser chaque année. »
</article>
<article titre="Bombay traque le plastique, les contrevenants risquent de la prison Une brigade de 225 contrôleurs a saisi en quelques jours une tonne de sacs, pailles ou bouteilles jetables et délivré des centaines d'amendes" date="2018 07 4" journal="Le Monde">
Santosh n'en est pas encore revenu. Vendredi 29 juin, il a vu des agents de la municipalité de Bombay débarquer devant son minuscule étal et vérifier qu'il ne donnait plus de sacs en plastique aux clients qui viennent lui acheter ses bijoux de pacotille. Le marchand de rue, qui vit avec sa famille sur le trottoir le plus passant du quartier historique de Colaba, n'utilise plus de plastique depuis qu'en mars, les autorités ont annoncé la fin prochaine de ce matériau lorsqu'il n'est pas réutilisable. Mais à sa grande surprise, les inspecteurs ont fouillé dans ses affaires et trouvé à redire aux sacs en tissu qu'il utilise désormais. " Ils ont dit que mes nouveaux sacs n'étaient pas conformes et ils m'ont donné une amende de 5 000 roupies  62,8 euros  ",  raconte Santosh.  " J'ai dépensé 300 roupies  3,8 euros  pour me procurer une centaine de sacs en tissu qui ont finalement été saisis. Avec l'amende, ma perte est énorme et du coup, je suis obligé d'emballer mes marchandises dans du papier journal. " L'interdiction, qui porte sur les sacs de courses, les emballages de nourriture, les pailles, la vaisselle et les petites bouteilles en plastique, est officiellement en vigueur depuis le 23 juin. Elle devait démarrer à la fin de l'hiver mais, à la suite d'un recours des fédérations de la filière plastique, la justice avait suspendu le dispositif pour trois mois, afin de donner aux professionnels le temps de s'organiser et d'écouler leurs derniers stocks.   " Effort général "Si vingt-cinq des vingt-neuf Etats de l'Union indienne bannissent partiellement ou totalement le plastique non réutilisable, le Maharashtra (115 millions d'habitants) et sa capitale, Bombay, sont les premiers à appliquer des sanctions. D'où, sans doute, le zèle des 225 contrôleurs déployés sur le terrain. Les fabricants, les commerçants et tous les utilisateurs finaux sont désormais passibles d'une amende de 5 000 roupies, laquelle passe à 10 000 roupies (125,60 euros) en cas de récidive, et à 25 000 roupies (314 euros) et trois mois d'emprisonnement en cas de deuxième récidive.   Des dizaines de grandes enseignes, dont McDonald's et Starbucks, ont dû payer après avoir été contrôlées. Il faut dire que le premier ministre, Narendra Modi, parle du plastique comme d'une  " menace pour l'humanité "  et a chargé son gouvernement de faire disparaître le matériau honni d'ici à 2022.   Sur la longue artère de Colaba, la quasi-totalité des détaillants ont pris leurs dispositions et se disent  " heureux "  de cette petite révolution.  " Depuis que les médias ont montré à quel point la mer est polluée, on est tous contents de participer à l'effort général " , affirme Deepak, un pharmacien passé depuis un an aux sacs en papier pour envelopper ses médicaments. Pour Mahmood, qui écoule ses dernières mangues en ce début de mousson, l'affaire est coûteuse.  " Avant, j'achetais 30 roupies  38 centimes d'euro l es 50 sacs en plastique. Maintenant, je dois débourser 80 roupies  1 euro  pour 35 sacs en papier,  explique-t-il.  Mais ça ne fait rien, c'est pour la bonne cause ". En face, Vikram l'épicier a trouvé un fournisseur de sacs en bagasse, un matériau issu de la cellulose de la canne à sucre.  " Pour l'instant, on peut encore donner aux clients les sacs en plastique de plus de 50 microns d'épaisseur qui ont des poignées, car ils peuvent être réutilisés. Pareil pour les bouteilles d'eau de plus de 500 ml, elles sont toujours autorisées " , précise-t-il.   Certains continuent néanmoins à faire de la résistance, arguant de la soudaineté avec laquelle l'interdiction a été appliquée. L'association des fabricants de sacs en plastique estime ainsi que  " le secteur va perdre 2,2 milliards de dollars " et que " 300 000 emplois se trouvent menacés " . En une semaine, plusieurs dizaines de milliers de boutiques ont été contrôlées, 1,5 million de roupies (18 840 euros) d'amendes ramassées et plus d'une tonne et demie de plastique saisie.   Député de Colaba un quartier de Bombay sous les couleurs du Parti du peuple indien (BJP, nationaliste hindou), le parti de M. Modi, Raj Purohit, trouve que tout va trop vite. Il réclame une période de transition qui courrait  " jusqu'en décembre 2019 " , le temps de mettre en place des alternatives pérennes, comme le métal, le bois et le jute, et surtout une filière de recyclage du plastique digne de ce nom.   Dimanche 1er juillet, le gouvernement du Maharashtra a envoyé un message d'apaisement, en accordant un nouveau répit de trois mois aux sacs en plastique inférieur à 50 microns, mais uniquement chez les petits détaillants. Raison invoquée : la mousson et ses pluies diluviennes, qui rendent l'usage du papier impossible jusqu'en septembre. De quoi mettre en colère les écologistes, qui craignent que le plastique réapparaisse tôt ou tard.  " Au Maharashtra, 8 000 tonnes de sacs en polyéthylène sont utilisées chaque année, c'est une bonne raison pour montrer l'exemple ",  estime la blogueuse Vandana Vasudevan, qui rappelle qu'une simple paille en plastique employée pour boire l'eau d'une noix de coco met deux cents ans à se décomposer.   Guillaume Delacroix
</article>
<article titre="Festival de la chanson de Tadoussac: le retour à l'église" date="2018 06 30" journal="Le Devoir (site web)">
Il faisait exceptionnellement très chaud pour le coup d'envoi de cette 35e édition du Festival de la chanson de Tadoussac. Et ce rendez-vous avec l'exception aura aussi été l'occasion de marquer de belle façon l'unicité de l'événement, qui présente cette année une programmation bigarrée à souhait, voire déroutante.   Même si le festival de la Côte-Nord tarde toujours, à certains égards, à prendre ses aises dans le décor naturel des lieux, il a su s'approprier au fil des ans des sites emblématiques du village, au-delà de la beauté du fjord du Saguenay et du Saint-Laurent tout près. Il l'a d'ailleurs prouvé de belle façon vendredi en offrant, en formule « show caché », une courte prestation de Jean-Pierre Ferland dans l'église anglicane de Tadoussac. Véritable joyau patrimonial, la petite église de la communauté anglophone de Tadoussac, construite en 1866, se prêtait tout à fait à ce tour de chant intimiste, offert à une centaine de personnes.   Après Le petit roi et La musique, le monument Ferland a lui-même vanté la beauté des lieux. L'artiste, qui a accepté d'offrir deux spectacles vendredi, a aussi rappelé au Devoir son amour du métier, pour expliquer son besoin de monter sur scène, même à 84 ans.   « C'est un métier qu'on fait pour être aimé. On ne peut pas dire aux gens, qui nous ont aimés pour une bonne partie de notre vie, qu'on ne les aime plus et qu'on s'en va. Ça ne se fait pas. Et c'est plate à mort être tout seul, quand on est habitué de chanter, d'écrire et de groover. Je ne suis pas rendu au moment de rester chez moi, avec ma pension. Et je n'ai pas peur de mourir, tant que je peux me dire que j'ai réussi », a-t-il fait valoir, avant son « show caché ».   Marjo énergique    Photo: Jay Kearney Marjo à l'église du village    Pour le rendez-vous avec les valeurs sûres parmi les valeurs sûres, il fallait aussi être à l'église du village vendredi soir, pour le spectacle de Marjo. Prestation énergique de bout en bout, pour un public qu'elle a su charmer en quelques secondes d'Illégale.   Même accueil enthousiaste la veille pour Les Trois Accords, qui ont ouvert le festival cette année. En conférence de presse avant leur spectacle, ils saluaient d'ailleurs le tour de force de maintenir, en région, un tel festival année après année.   Les Trois Accords, qui ont eux-mêmes lancé le Festival de la poutine de Drummondville il y a de cela 11 ans, estiment aussi que la multiplication des rendez-vous musicaux au Québec témoigne d'une scène musicale en santé. « C'est une bonne nouvelle qu'il y ait de plus en plus de festivals au Québec. C'est difficile de lancer un événement. Il faut travailler très fort. Donc, c'est intéressant qu'il y ait un public pour la musique rock francophone », a fait valoir le batteur du groupe, Charles Dubreuil.   Opinion partagée par l'ancien directeur du Festival de la chanson de Tadoussac, Charles Breton, qui a reçu jeudi la Médaille de l'Assemblée nationale pour son implication de plus de 20 ans dans le festival.   « C'est très difficile de maintenir un festival année après année. C'est une énorme pression. Il faut avoir les moyens financiers, qui ne sont jamais assez grands, choisir les bons artistes, espérer la bonne météo, etc. Et tout cela dans un petit village. Mais c'est très emballant d'aider la culture à rayonner, notamment les jeunes artistes qui sont émergents, qui ne sont pas du tout connus aujourd'hui, mais qui vont gagner des prix comme celui de Révélation de l'année à l'ADISQ dans quelques années », a souligné M. Breton, devenu l'automne dernier le maire de Tadoussac.   Nuitée électro à venir   Au rayon des artistes inconnus ou méconnus, le festival offre en effet, comme à chaque année, une généreuse récolte de belles découvertes. Difficile de résister au groove invitant du Winston Band, qui distribuait au passage de l'alcool artisanal des Îles de la Madeleine, la bagasse, et des grillons rôtis.   La fête se poursuivait encore au milieu de la nuit, tandis que Valaire complétait son premier spectacle de la fin de semaine, avant de promettre une autre nuitée électro samedi, dans la formule Qualité Motel.   Sous un soleil de plomb, on a aussi succombé à Delgrès, trio de blues de la Guadeloupe qui rappelle à la fois John Lee Hooker et les Black Keys. Et ça se poursuit, maintenant sous la pluie, jusqu'à dimanche soir, avec notamment Klô Pelgag, Keith Kouna, Les Hay Babies et Tire le coyote.  Cet article est paru dans Le Devoir (site web)
</article>
<article titre="Cinq lauréats pour Stratégies Emballages" date="2018 06 29" journal="Emballages magazine.com">
La sixième édition du Trophée Emballage du manager de l'année a eu lieu le 28 juin.-   Ils sont cinq à rejoindre cette année le club des stratèges de l'emballage et du conditionnement. La sixième édition de Stratégies Emballages, le Trophée Emballage du Manager de l'année, a eu lieu le 28 juin 2018, à Paris. Quelque 200 professionnels avaient répondu présent à l'invitation d'Emballages Magazine qui organisait l'événement avec le concours d'ABB, d'All4Pack, de DS Smith, de B&amp;R et de l'Union nationale des fabricants d'étiquettes adhésives (UNFEA). ?? "Stratégies Emballages 2018"   Revivez la soirée @EmballagesMag #StratégiesEmballages et la remise des Trophées Emballages du manager de l'année 2018 à travers le #Livetweet de @monnier_phttps://t.co/VUBSW9TRhi   - Pierre Monnier (@monnier_p) 28 juin 2018   Laurent Dulau   Directeur général de Sturia, Laurent Dulau a montré comment le caviar, un produit premium sinon de luxe, intègre la responsabilité sociétale des entreprises (RSE). Troisième acteur mondial avec une production de 17 tonnes, Sturia souhaite développer ses ventes dans la grande distribution en limitant le suremballage. L'entreprise collabore actuellement avec Packitoo, une start-up paloise qui a fait du sourcing 4.0 son modèle économique, pour mettre au point un emballage disruptif. Est également en projet une usine de conditionnement conçue pour tirer le meilleur de l'industrie 4.0. Laurent Dulau s'est vu remettre son prix par Cyril Héroult, directeur du segment agroalimentaire d'ABB.   Gilles Bonnabeau   Dans le domaine cruciale de la lutte contre la contrefaçon, Gilles Bonnabeau, directeur de Cypheme Europe, a montré qu'un système de protection moderne pouvait s'appuyer sur les réseaux sociaux pour certifier aux consommateurs que le produit acheté est authentique. Hugo Garcia-Cotte et Pierre Guinaudeau ont mis au point une technologie qui repose sur l'intelligence artificielle exploitable à partir de l'appareil photo d'un téléphone. Le développement est né d'une expérience personnelle, un membre de la famille des fondateurs a été victime d'un médicament contrefait. Gilles Bonnabeau s'est vu remettre son prix par la rédaction d'Emballages Magazine.   Gilles Bonnabeau, DG de @cypheme, manager de l'année 2018 #StratégiesEmballages dans la catégorie #Traçabilité reçoit son trophée avec la rédaction de @EmballagesMag. pic.twitter.com/9IfHhydqZY   - Pierre Monnier (@monnier_p) 28 juin 2018   Marc Gaudillère   Directeur packaging adjoint de Sanofi Pasteur, Marc Gaudillère, diplômé de l'École supérieure européenne de packaging (Esepac), a le pack dans la peau. Son parcours au sein de Kodak Industrie, de Bel, de Rexam Healthcare, puis de Sanofi Pasteur en témoigne amplement. Marc Gaudillère fait de la peau son emballage idéal : cet organe protège et conserve tout en s'adaptant à son environnement. Un emballage qui interagit avec les cinq sens, tel est le rêve de Marc Gaudillère qui s'est vu remettre son prix par Thibault Laumonier, directeur général de DS Smith France.   Raphaël de Taisne   Cofondateur de Yumi, Raphaël de Taisne a mis la barre très haut en termes de packaging. L'entreprise commercialise des jus de fruits et de légumes pascalisés, conditionnés en bouteilles biodégradables de section carrée. Mais voilà! Considérée comme un perturbateur de tri, la bouteille à base de bagasse fait l'objet d'un malus selon le barème de Citeo, la société agréée pour la valorisation des emballages ménagers et des papiers. Le dossier est suivi par Brune Poirson, secrétaire d'État auprès de Nicolas Hulot, ministre de la Transition écologique et solidaire. Raphaël de Taisne s'est vu remettre son prix par Stéphane Potier, en charge du marketing de B&amp;R France.   Le cofondateur de @yumidrinks, Raphaël De Taisne : « Appliquer un malus à une bouteille qui se passe de #plastique va à l'encontre de l'histoire. » Ovation dans la salle de #StratégiesEmballages. Il reçoit son trophée de manager de l'année 2018 dans la catégorie #boissons pic.twitter.com/bHn8pK4o2q   - Pierre Monnier (@monnier_p) 28 juin 2018   Florence Bonnet-Touré   La transmission, le partage d'expérience entre les nouveaux diplômées et les experts aguerris : tel est le positionnement historique de Stratégies Emballages. Nouvelle présidente de Destination plasturgie, Florence Bonnet-Touré, déléguée générale adjointe de la Fédération de la plasturgie et des composites, a montré comment une filière professionnelle prenait en main la formation des jeunes. Alors à quand la création de Destination Packaging ? Florence Bonnet-Touré s'est vu remettre son prix par Aurélie Jouve, directrice de la communication d'All4Pack, qui se tient du 26 au 29 novembre à Paris.   27 membres   En 2013, étaient lauréats de la première édition Henri Demontrond (Somfy), Vincent Ferry (Danone), Bruno Garnier (Carrefour), Corinne Mercadié (Casino) et Arnaud Rolland (Coca-Cola Entreprise). En 2014, les stratèges de l'emballage de la deuxième cuvée étaient Michel Bourguignon (Nestlé), Françoise Lochet (Laboratoires Expanscience), Christophe Morin (consultant) et Françoise Poulat (Danone Nutricia Research). En 2015, Steven Tordjeman (So Shape), Ève Deregnaucourt (Bacardi-Martini), Véronique Bregeon (Merial), Fabrice Ballin (Cristalco) et Annette Freidinger-Legay (Côté emballage) étaient à leur tour distingués. Anne-Laure Bulliffon (Alpes Packaging), Johnny Grippon (Laiterie Les Fayes), Benoît Renauld (Werner &amp; Mertz France/Benelux), Alexandra Lagisz (Intermarché) et Michel Fontaine (CNE) ont rejoint les quatorze premiers lauréats en 2016, suivis de Julien Barreau (Lush), Christèle Chancrin (E³ Conseil) et Jean-François Molle (Gestes propres) en 2017. Voici maintenant Laurent Dulau (Sturia), Gilles Bonnabeau, (Cypheme), Marc Gaudillère (Sanofi Pasteur), Raphaël de Taisne (Yumi) et Florence Bonnet-Touré (Destination plasturgie). Le club des stratèges de l'emballage et du conditionnement compte actuellement 27 membres.   C'est la fin de cette sixième édition de #StratégiesEmballages. L'ensemble des lauréats se retrouvent pour une photo de clôture de la soirée. pic.twitter.com/IRuopfSEZr   - Pierre Monnier (@monnier_p) 28 juin 2018
</article>
<article titre="Yumi défend sa bouteille biodégradable" date="2018 06 26" journal="Emballages magazine.com">
La start-up veut promouvoir une alternative aux plastiques traditionnels.-   "On viendra à bout du plastique!" : l'engagement est total chez Yumi. Dans une tribune publiée au mois de juin, la start-up qui commercialise des jus de fruits et légumes pascalisés, rappelle qu'elle ne digère pas que ses bouteilles biodégradables et biosourcées fassent l'objet d'un malus. Société agréée pour la valorisation des papiers et des emballages ménagers, Citeo applique le barème selon lequel un emballage qui perturbe un flux existant - le polyéthylène téréphtalate (PET) et le polyéthylène haute densité (PEhd) en l'occurrence - fait l'objet d'une surtaxe de 100%. En octobre 2017, Yumi a opté pour une bouteille en bagasse, un résidu de la canne à sucre, fournie par Lyspackaging. Très technique, cette bouteille est suffisamment souple pour résister aux hautes pressions d'un stérilisateur à froid : 6000 bars pendant 3 minutes. Fin de vie   Pour défendre leur dossier, Raphaël de Taisne et Louis de Bentzmann, les fondateurs de l'entreprise, ont été reçus par Brune Poirson, secrétaire d'État auprès de Nicolas Hulot, ministre de la Transition écologique et solidaire, le 15 mai. Citeo a mis en route une étude pour analyser la fin de vie de ce matériau. Nicolas Moufflet, le patron de Lyspackaging, ne ménage pas ses efforts pour défendre son emballage... Alors que le gouvernement déploie sa feuille de route sur l'économie circulaire dans laquelle le plastique occupe une place de choix, Yumi souhaite promouvoir une alternative aux plastiques traditionnels. L'heure est au status quo, mais la confiance est de mise.   Stratégies Emballages   Raphael de Taisne aura l'occasion de s'exprimer sur ces sujets le 28 juin lors de la sixième édition de Stratégies Emballages, le Trophée Emballage du manager de l'année, organisé par Emballages Magazine.
</article>
<article titre="Luxe Pack Monaco prépare son édition 2018" date="2018 06 18" journal="Emballages magazine.com">
La trente et unième édition du salon de l'emballage de luxe se déroulera au Forum Grimaldi du 1er au 3 octobre. -   Le Forum Grimaldi accueillera une nouvelle fois le salon Luxe Pack Monaco. Du 1er au 3 octobre 2018, le rendez-vous de l'emballage de luxe doit réunir quelque 470 entreprises réparties sur les six niveaux du bâtiment monégasque. L'événement, organisé par Infopro Digital - éditeur d'Emballages Magazine -, fera une nouvelle fois la part belle au développement durable, lors d'un après-midi dédié le 2 octobre. Il s'agira alors de présenter les nouveautés écoconçues des exposants, mais aussi de débattre et de témoigner lors de conférences. La remise des Luxe Pack in green, les trophées récompensant une solution et une stratégie durables, clôturera cette journée. Un décryptage de la génération Z   Fort de sa légitimité acquise dans le secteur de la parfumerie, le salon inaugurera une nouvelle entité pour sa trente et unième édition : le Luxe Fragrance Forum. Le mercredi 3 octobre, une session réunira des experts mondiaux du domaine pour partager des chiffres de marché, des tendances, ou encore des designs en vogue. Cette nouvelle extension de Luxe Pack s'ajoute à Luxe Pack Digital Village - C2L, consacré aux solutions numériques dans le packaging de luxe, et Luxe Formulation, pour les questions de formules et de full service dans le secteur de la beauté.   Le Trends Observer cherchera à décrypter la génération Z. Béatrice Mariotti, la vice-présidente de Carré noir, Régine Charvet-Pello, la présidente de Mat &amp; Sens-Certesens, Stéphane Truchi, le président de l'Ifop, et Sabine Durand, la directrice de la rédaction de Formes de Luxe - également édité par Infopro Digital - présenteront les tendances qui définissent cette nouvelle génération d'acheteurs. Cette étude sera accompagnée de témoignages d'experts et de start-up.   Un succès américain en mai   Les 16 et 17 mai, Luxe Pack a tenu sa seizième édition new-yorkaise. Pour l'occasion, 3 586 professionnels de l'industrie de l'emballage de luxe sont venus à la rencontre des 240 entreprises représentées. Les trophées Luxe Pack in green ont récompensé Golden Arrow, pour son packaging en fibre moulé fabriqué à partir de bambou et de bagasse qui permet de se passer de plastique, et James Cropper, pour sa solution CupCycling dont le but est de produire des fibres pour le domaine du luxe à partir de gobelets pour café.
</article>
<article titre="Produits bio, gestion des déchets : les restaurateurs verdissent leurs démarches" date="2018 06 18" journal="Les Echos (site web)">
Au Sirha (salon de la restauration et de l'hôtellerie) l'an dernier, le parcours « bio » comptait 183 exposants, et celui du « développement durable » 179. Ce sont peu ou prou les pionniers de la première édition du Sirha Green organisé à Lyon jusqu'au 19 juin. «  Green ne signifie pas seulement bio, ce sont aussi les circuits courts et l'offre végétarienne. Et de nouvelles façons de cuisiner, de servir et... de ne pas jeter  », souligne Marie-Odile Fondeur, directrice de l'évènement.  «  La restauration en est aux balbutiements de l'économie circulaire   », estime François Blouin, de Food Service Vision. La prise de conscience a commencé dans l'assiette. «  La moitié des restaurateurs indépendants dit acheter localement 50 % de leurs produits  », dit-il. D'ailleurs, la priorité des clients va aux circuits courts plutôt qu'au bio : 85 % contre 15 %. Dans la restauration hors domicile, l'achat en valeur de produits bio ne dépasse pas 3 % pour la restauration collective, et 1 % en restauration commerciale, selon la dernière étude de l'Agence Bio. «  Une fois sur deux, l'offre bio se limite au vi n  », regrette Florent Guhl, son directeur.   Le marché alimentaire sous label AB représentait 411 millions en 2016 dans la restauration, en croissance de 6,8 % (contre 6,7 milliards à domicile, en croissance de 21 % !). Mais sous la pression des consommateurs qui mangent plus bio à la maison, la démarche est engagée. Et les 350 restaurants français réellement militants (plus de 50 % d'achats bio), à l'instar des exemplaires Bioburger ou Le Bréguet dans le 11e arrondissement parisien, «  sont appelés à devenir la norme d'ici à quinze ans  », estime Florent Guhl.   Interdiction de la vaisselle plastique jetable Cependant, les déchets sont encore plus problématiques que les approvisionnements. Selon Food Service Vision, en 2016, 38 % de ce qui rentrait dans un restaurant finissaient à la poubelle. Les lois antigaspillage tentent d'enrayer ce gâchis. Et les filières de collecte des biodéchets et de méthanisation se mettent doucement en place. L'association Lyon Bio Ressources testera à partir de juin 2019, avec GRDF, une tournée auprès de 90 restaurateurs, hôpitaux et commerces de bouche du 4e arrondissement.   Dans la restauration à emporter ou traiteur, c'est la vaisselle jetable en plastique qui impacte négativement l'environnement. Sous le coup d'une interdiction à partir de 2020, sauf pour les contenants (bols, barquettes, ou gobelets fermés d'un couvercle), elle est progressivement remplacée par des couverts, assiettes et verres en matière recyclable ou compostable.   Des lave-vaisselle à pompe à chaleur De nouvelles pratiques gagnent lentement les arrière-cuisines. «  La plus forte consommation énergétique vient de la ventilation et des frigos  », explique Bruno Dettwiller, président du Cinov Restauconcepteurs, au sein de la fédération des métiers du conseil. Il prône des solutions pas forcément coûteuses comme le variateur de vitesse sur l'extracteur de fumée et le compresseur des chambres froides. Et des récupérateurs de chaleur à la plonge, qui permettent en plus de faire baisser la température et l'humidité ambiantes.    «  On ne révolutionnera pas en un jour un secteur qui pèse 1,2 million d'emplois  », souligne Marie-Odile Fondeur, dont l'ambition avec le Sirha Green est de «  cocréer ce marché  » encore quasiment inexistant, en profitant des réseaux internationaux et du poids du Sirha dans le milieu. La philosophie du développement durable est aussi une affaire de détails : ils se nichent jusque dans les produits d'entretien écologiques (marque GreenCare, en flacons PET recyclés) et le tablier du chef (voir Lafont).     1 MonBento remet la consigne au goût du jour 2 Easylis pèse les déchets 3 Duni fait le tri dans la vaisselle jetable    Léa Delpont
</article>
<article titre="Edition principale Le bonheur est dans l'obscénité" date="2018 06 6" journal="Libération (site web)">
Image extraite du livre Martin Van Maele, de Luc Binet, éditions Humus 2017. (c) Van Maele. Daniel Lacotte a un combat. Il veut sauver le patrimoine linguistique, c'est-à-dire préserver de l'oubli les mots jugés vulgaires. Car ces mots-là sont porteur de joie: "asphyxier le pierrot", "avoir la gueule comme une écumoire", "mener les poules pisser"... A la mémoire du poète Jacques Prévert et de Philippe Soupault, co-fondateur du surréalisme, qui encouragent ses premiers pas, Daniel Lacotte mène depuis les années 1970 un combat pour le verbe haut. Il a publié 43 livres allant des Bizarreries de la langue française au Dico des mots pour briller en société. Son dernier livre en date - Le Bouquin des mots savoureux cocasses et polissons (publié chez Robert Laffont) - résume en 992 flamboyantes pages une vie dédiée à la défense du français. On pourrait trouver étrange qu'un homme se consacre entièrement à la quête de mots vulgaires et de locutions populaires. Mais dès les premiers lignes de l'introduction, Daniel Lacotte s'explique ou, plutôt, tape du poing : « dix mille langues se parlaient dans le monde au XVe siècle. Il n'en reste guère que cinq mille aujourd'hui. Le processus s'accélère : elles ne seront plus que cinq cents dans un siècle ! » La mondialisation s'accélère et même les langues qui résistent à la « déferlante destructive » de l'extinction des peuples se font prendre au piège de la simplification universaliste qui frappe déjà l'anglais, transformé en «globish».   Gare au Globish ! « Le plus incommensurable gâchis qui se profile à l'horizon consisterait à imposer au monde une langue planétaire supposée universelle. Un système de communication banalisé, nivelé, lisse. [...] Sans émotion ni mystères. » Daniel Lacotte tire la sonnette d'alarme. Il n'est pas anodin, dit-il, qu'une langue soit amputée de ses termes les plus fleuris, ni de ses tournures les plus complexes. La volonté qu'ont certains de «nettoyer» le français, afin qu'il soit plus international, va de pair -dit-il- avec cette double dynamique qui consiste à introduire sans cesse plus de termes anglais dans les dictionnaires et à en retirer sans cesse plus de vieux mots sous prétexte qu'ils sont périmés. « Ici ou là, d'aucuns s'échinent à désherber notre langue française. Comme s'il fallait, année après année, éradiquer des dictionnaires un hypothétique chiendent prétendument toxique, alors que tous ces mots assassinés ne demandent qu'à prospérer afin de proposer leurs services pour enrichir de maintes nuances un texte atone. »   Ce sont des mots qu'on assassine « Ainsi, tous ces gardiens autoproclamés du langage se sentent-ils investis d'une mission factice, que personne ne leur a confiée, pour désosser notre lexicographie, pour lui ôter progressivement sa chair, son sang et sa tonicité. Certes, le français reste une langue vivante. Aussi doit-elle s'enrichir de néologismes et de termes puisés aux meilleures sources d'autres cultures. Mais cet élargissement ne peut s'accomplir que progressivement. Lentement. Aucune règle ne peut justifier ces incessants chambardements sémantiques calendaires dictés uniquement par les lois du marketing et du commerce. Par surcroît, la langue française ne doit, en aucun cas, se dissoudre dans un plus petit dénominateur commun soumis aux excès de la facilité syntaxique, aux diktats de la pauvreté d'expression. »   Le cercle des mots disparus Méfiez-vous du « parler global », s'insurge Daniel Lacotte. C'est un parler « amorphe », utilitariste, fonctionnel, qui transforme l'utilisateur en « paralytique du verbe », en « impotent du rythme », en « invalide des subtilités, sensibilités, extravagances, charmes, élégances et finesses que doivent nourrir l'expression orale et écrite. Sans jamais oublier que le langage a aussi pour mission fondamentale de contribuer à affermir le ciment social d'une nation. » Animé par l'idée que le travail de la mémoire passe par la sauvegarde des mots, Daniel Lacotte s'emporte : « pour éviter à la langue française de subir l'humiliation d'un irréversible déclin, il faut impérativement redonner à chacun le goût et la saveur des mots », dit-il. Dont acte : son anthologie des mots savoureux s'offre comme une resource à la disposition de tous ceux et celles qui voudraient faire vivre la langue en puisant dans ses réserves. « Aliboron, attrape-minon, bagasse, bambochade, bousingot, dameret, évaltonner, gamache, marlotte, pasquiner, patafioler, requimpette, etc. La langue française en contient plusieurs centaines ! On leur a préféré un à-peu-près badigeonné d'une modernité provisoire en ignorant leur originelle vigueur rafraîchissante propre à ensoleiller le langage. »   « N'oublions jamais qu'une langue castrée rend le peuple impuissant »   Exhumant des trésors d'expressions gourmandes ou dérangeantes, Daniel Lacotte classe d'abord chaque mot disparu par analogie avant d'en faire l'objet d'un texte ciselé qu'il met ensuite en scène dans les deux derniers chapitres de son ouvrage : «Tranches de vie» et «Quand les mots ont des maux». On se régale de feuilleter ce pavé, tout rempli d'énigmes et de saillies : mais que signifie «avoir les abattis canailles» (que Daniel Lacotte compare à l'expression paternaliste «avoir une bonne sexualité de paysan») ? Pourquoi dit-on «Abbaye de Clunis» pour signifier «les fesses» ? «S'adoniser» quand un homme prend soin de son apparence ? Un «amoureux de Carême» pour un petit copain trop timide ? Chaque page redonne envie de s'amuser. L'ouvrage est traversé par l'énergie de cette langue verte, revigorante, dont Daniel Lacotte affirme qu'elle porte la joie. Car les propos « crus et drus », dit-il, valent mieux que le « bavardage émasculé » de ceux ou celles qui « veulent un langage lisse », poli, « propice à la masturbation intellectuelle ». Avant de conclure : « N'oublions jamais qu'une langue castrée devient muette et qu'elle rend le peuple impuissant. »   .   A LIRE: Bouquin des mots savoureux cocasses et polissons, de Daniel Lacotte, Robert Laffont, coll. Bouquins, 2017.  Cet article est paru dans Libération (site web)
</article>
<article titre="Innovation Les solutions biosourcées ont des freins à lever" date="2018 06 1" journal="Process Alimentaire, no. N° 1359">
Analyse  L'emballage est aujourd'hui le secteur le plus consommateur de bioplastiques dans le monde. «Il existe deux grands types de bioplastiques d'origine végétale, les premiers sont non biodégradables, comme le bio PET et le bio PE. Les seconds sont biodégradables et compostables, comme le PLA et le PHA», résume Thomas Lefèvre, p-dg de Natureplast. Créée en 2007, cette société française a pour vocation d'aider les plasturgistes à utiliser les bioplastiques. Elle exerce trois métiers : l'expertise, le négoce et la R &amp; D à travers sa société fille Biopolynov qui élabore des formulations sur mesure. L'intérêt des bioplastiques est double : marketing et commercial d'une part, environnemental de l'autre. La plupart du temps, bio PET, bio PE ou PLA sont utilisés seuls, mais on voit également émerger des «compounds» mélangeant plusieurs types de bioplastiques. C'est notamment le cas de BASF avec ses gammes Ecoflex et Ecovio ou du Français Vegeplast qui propose plusieurs typologies de son Vegemat, matériau fabriqué à partir d'amidon et de fibres, de façon à multiplier les applications : passage au four jusqu'à 130 °C, réchauffage au micro-ondes, congélation, barrière à l'oxygène, etc. En élargissant son spectre d'usages avec sa nouvelle barquette Vegepack Light (lire avril 2018 p. 93), la PME envisage d'atteindre 10 000 tonnes d'ici deux à trois ans. L'enjeu du coût De quoi lever un des nombreux freins au développement des bioplastiques : le prix. A usage comparable, ils peuvent coûter jusqu'à cinq fois plus cher que des plastiques pétrosourcés. Pour cette raison, le PHA (polyhydroxyalcanoate), dont les propriétés thermiques et barrières sont pourtant intéressantes, les biopolyesters et le PBS (polybutylène succinate développé dans le cadre du projet européen Succipack) sont encore peu utilisés par l'industrie agroalimentaire. «Il faut tenir compte de l'application précise pour comparer les coûts, assure Koen Bastiaens, Business Leader chez NatureWorks, qui propose une large gamme de PLA Ingeo. Ainsi, pour les bouteilles, il est plus pertinent d'opter pour du bio PET, alors que pour les bouchons, le PLA est la solution optimale». Beaucoup n'offrent pas non plus de propriétés physiques équivalentes.   L'atout écologique, argument numéro un des bioplastiques, est également critiqué par certains. En cause : la concurrence avec la filière alimentaire, toutefois à relativiser (seules 0,00002 % des surfaces de canne à sucre, de blé ou de maïs sont consacrées à la production de bioplastiques) et l'origine souvent lointaine des bioplastiques (Asie ou Amérique, très peu en Europe). Par ailleurs, si le bio PET et le bio PE s'intègrent sans souci aux filières de recyclage classiques PET et PE, c'est plus compliqué pour le PLA et le PHA. Alors même que les capteurs des centres de tri reconnaissent bien le PLA, aucune filière de recyclage n'est mise en place par manque de volume. Et les filières de compostage sont très en retard en France. Ce qui freine le développement du matériau et pénalise son éco-contribution (lire interview de Yumi ci-contre).   B &amp; G Products, racheté par Sirane le mois dernier, a peut-être résolu l'équation pour les emballages souples avec son Eartpouch en papier. Enduit d'un matériau sans plastique, ce sachet stand-up préformé est recyclable et compostable et dispose de propriétés barrières à l'humidité et à la graisse. Il peut contenir des produits secs comme humides. Pour les emballages rigides, la pulpe de cellulose rencontre aussi un vif succès dans les emballages jetables ou les barquettes pour le snacking.   Utiliser les déchets, une piste d'avenir ? Pour contrer les sceptiques, les bioplastiques de demain pourraient être issus de la poubelle verte (déchets végétaux), des eaux usées ou encore des algues. De nombreux projets de recherche vont dans ce sens. Les coproduits s'avèrent prometteurs dans le secteur agroalimentaire et les pistes sont nombreuses : coques, pulpes, noyaux d'olives, pépins, coquillages, déchets de cacao, etc. «Il serait envisageable d'en incorporer 10 à 30 % dans tous types de plastiques ou bioplastiques, mais des contraintes réglementaires sont à prévoir, sur la traçabilité et l'aptitude au contact alimentaire», explique Thomas Lefèvre. Chez Europlastiques, le projet R &amp; D BluecoPha (lire septembre 2015 p.89) vise à produire localement des emballages en PHA synthétisé par une bactérie à partir de substrats de l'industrie agroalimentaire. Tandis que Lactips utilise déjà l'excédent de caséines de lait de vache non comestibles (dates dépassées, résidus de nettoyages) pour fabriquer des emballages de fromage. La boucle est bouclée.
</article>
<article titre="FCL Item code" date="2018 06 1" journal="Rapports - Agroalimentaire - ONUAA">
NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... FCL Item code FCL Title Group Scientific Name Common Name Definition 0015 Wheat 1 - CEREALS AND CEREAL PRODUCTS Triticum spp.: Triticum aestivum Triticum durum Triticum spelta  common durum spelt  Common and...    Pour voir le rapport :  http://www.fao.org/fileadmin/user_upload/commodity/excel/export_items_1527869558.xls
</article>
<article titre="Ouganda : l'AFD débloque 37 millions d'euros pour le réseau électrique" date="2018 05 23" journal="La Tribune Afrique (site web)">
L'Ouganda vient de décrocher une enveloppe de 37,1 millions d'euros de l'Agence Française de Développement (AFD) après la signature par les deux parties d'une convention de financement pour la modernisation d'une ligne de transmission électrique stratégique.   Objectif export Ce financement servira à la modernisation d'une ligne de transport de 400 Kv couvrant une distance de 135 km entre les villes de Masaka et Mbarara. A terme, cette ligne de transport devrait permettre l'évacuation de l'énergie de différentes sources et faciliter l'exportation d'électricité vers les pays voisins notamment le Rwanda et la République démocratique du Congo. Ce projet est par ailleurs cofinancé par la KfW (Allemagne), qui suit des études de faisabilités financées par le Fonds fiduciaires des infrastructures de l'Union européenne, il y'a trois ans. L'Ouganda continue de faire face à de nombreuse difficultés dans son réseau de distribution électrique. Pour y remédier, de nombreux projets de production électriques, tels que les centrales hydroélectriques de Karuma et Isimba devraient entrer en service au cours des neufs prochains mois.   Parallèlement, la livraison de ces centrales fait craindre aux autorités que le réseau de transport ne soit pas en mesure d'évacuer toute l'énergie produite. Le gouvernement a ainsi mis au point des politiques de compensation en faveur des gros consommateurs capables de mettre en place des réseaux de distribution d'électricité. D'ailleurs, le projet d'interconnexion régionale du Programme d'action subsidiaire des lacs équatoriaux du Nil devrait contribuer à la mise en place de réseaux de transport.   La Proparco soutient les sucriers La Proparco a de son côté, débloqué 40 millions de dollars en faveur de Sugar Corporation of Uganda, la troisième plus grande usine de sucre en Ouganda. Cette enveloppe sera allouée au financement d'une nouvelle centrale de cogénération de 26 MW. Selon l'accord financier, Proparco a accordé un prêt de 20 millions de dollars et a facilité le déblocage du reste de l'enveloppe de la part de la banque de développement néerlandaise FMO.   Ce projet devra selon la FMO permettre à l'entreprise de produire de l'électricité verte à un prix compétitif pour répondre à ses besoins en énergie et vendre le surplus au réseau national. L'électricité sera générée en brûlant de la bagasse, un résidu de canne à sucre. La production d'énergie via la bagasse est devenue populaire, attirant l'attention des industriels utilisant la canne à sucre comme matière première.  Cet article est paru dans La Tribune Afrique (site web)
</article>
<article titre="A Maurice, la canne à sucre dans tous ses états" date="2018 05 22" journal="Le Monde.fr">
Au moment où Maurice fête les 50 ans de son indépendance, une page économique semble définitivement tournée : la place longtemps tenue par l'exportation de sucre dans l'économie de l'île. Certes, les plaines et plateaux restent dominés (90 % des surfaces cultivées) par les champs de canne qui, depuis plus de trois siècles, ont marqué l'histoire humaine et économique de Maurice. Mais en 2018, des mutations profondes de la filière sont arrivées à maturité : en moins de deux décennies, la traditionnelle exploitation sucrière a cédé la place à une industrie cannière diversifiée. Dans le nord-ouest de l'île, l'Aventure du sucre expose les principaux axes de cette mutation. Voilà quinze ans, cette ancienne sucrerie de la commune de Pamplemousses a été transformée en musée interactif dédié à l'histoire de l'or blanc mauricien. De l'introduction de la canne à sucre de Java par les Hollandais, au XVIIe siècle, à l'économie esclavagiste développée par les Français au XVIIIe siècle, suivie de l'« engagisme » des travailleurs indiens, un système proche du servage expérimenté par les Britanniques dès 1835, les grandes étapes de la monoculture sucrière mauricienne sont méthodiquement retracées.   Jusqu'aux évolutions les plus récentes : fluctuations brutales et effondrement des cours du sucre, dans les années 1990 et 2000; démantèlement progressif, depuis 2009, du « Protocole sucre » signé en 1975 entre l'Union européenne et 19 pays ACP (Afrique, Caraïbes, Pacifique) et qui accordait à Maurice à un accès préférentiel au marché européen via des quantités et des prix garantis; et, enfin, annonce officielle, en 2017, de la fin des quotas sucriers européens, livrant désormais aux seules lois du marché le cours hasardeux de l'or blanc.   Quatre centrales thermiques   « Avant même ces deux dernières étapes, les grandes familles mauriciennes du sucre et les autorités du pays avaient déjà amorcé la transformation de la filière,  explique Arvind Nilmadhub, directeur du cabinet de conseil Afribrains.  La baisse de la rentabilité a conduit à une concentration et une diversification qui ont finalement donné naissance à l'industrie cannière moderne. » La première raffinerie de sucre est inaugurée en 2009, alors que le nombre de sucreries ne cesse de chuter : 19 au début des années 1990, 11 en 2003 et seulement 4 aujourd'hui, contre 124 sites en 1891. Ce changement s'est accompagné de programmes sociaux pilotés par l'Etat pour favoriser l'accès à la propriété des petits cultivateurs sur des lopins de terre cédés par les grandes familles sucrières.   Mais la plus forte mutation de la filière provient de la place conquise par la production d'électricité à partir de bagasse, un résidu fibreux issu de la première étape de broyage des cannes.  « Selon les variétés, une tonne de canne fournit entre 250 et 350 kg de bagasse, pour une valeur énergétique de 420 kWh par tonne »,  explique un ancien ingénieur de la filière.   Grâce aux quatre centrales thermiques à présent installées à proximité des sucreries (et alimentées au charbon hors saison de la canne), la capacité de production d'électricité de l'île a fortement progressé. Selon le dernier rapport annuel du Central Electricity Board, les énergies renouvelables contribuaient en 2015 à 22 % de l'électricité produite, plus des deux tiers provenant de la combustion de bagasse.   Renaissance des distilleries   Autre diversification : la production de bioéthanol.  « En 2016, l'Etat a amendé un texte de loi afin de rendre obligatoire le mélange de bioéthanol à l'essence destinée aux carburants automobiles »,  explique Arvind Nilmadhub.   Pour l'heure, seule la compagnie Omnicane (1 500 salariés), en association avec le groupe belge Alcogroup, s'est lancée dans cette nouvelle aventure industrielle, avec une production annuelle de plus de 20 millions de litres de bioéthanol. Situé dans le sud-est de l'île, ce projet répond au souhait du groupe de compenser ses contre-performances sur le seul marché du sucre, avec des pertes de 26,4 millions de roupies en 2016 (soit 670 000 euros à l'époque).   Enfin, la production de rhum, traditionnelle sur l'île, permet aussi au sucre de retomber sur ses cannes.  « Nos distilleries sont anciennes, mais leur production est repartie, de façon artisanale ou industrielle, avec la chute des cours du sucre »,  souligne Eric Guimbeau, propriétaire du groupe Saint-Aubin. Sur son domaine dans le sud de Maurice, où la production de rhum bat son plein depuis l'acquisition, notamment, d'un alambic de marque Hradecky Pacova en 2001, la canne est toujours cultivée mais le broyage n'est plus manuel : l'énergie entraînant l'énorme moulin est produite par... une chaudière à bagasse.   Olivier Piot   Cet article est paru dans Le Monde.fr
</article>
<article titre="À la découverte des champignons hallucinogènes du Mexique" date="2018 05 20" journal="L'Obs (site web)">
Par Arnaud Exbalin, Université Paris Nanterre - Université Paris Lumières. Cet article a initialement été publié sur le site The Conversation. Mexican magic mushrooms ! Aujourd'hui, tout le monde peut commander sur des sites spécialisés des spores de champignons hallucinogènes mexicains à cultiver à domicile. Les kits comprennent bac, terreau, spores et notice d'utilisation. Ces dernières recommandent toutefois de ne pas combiner leur ingestion avec de l'alcool ou d'autres drogues, de les consommer dans un environnement calme et, avant toute chose, d'être en bonne santé physique et mentale.   Les champignons mexicains ne sont en rien des drogues récréatives : leurs effets sont si puissants qu'ils peuvent faire perdre la raison à certaines personnes mal préparées.   Avant d'être qualifiés de « magiques » et placés au rang de produits mondialisés, ces champignons étaient considérés comme des entités sacrées par les peuples méso-américains : teonanacatl en nahuatl, littéralement « chair de dieu ».   À partir du XVIe siècle, avec la conquête du Mexique et sa colonisation par les Espagnols, les moines évangélisateurs en firent des agents démoniaques. Pendant près de 400 ans, les usages sacrés des champignons hallucinogènes furent ainsi combattus au point de disparaître du Mexique.   C'est en tout cas ce que l'on croyait avant qu'ils ne soient redécouverts dans les années 1950.   À la saison des pluies Aujourd'hui, la consommation des champignons hallucinogènes subsiste encore dans certains villages des hauts plateaux centraux du Mexique. Dans la région montagneuse de la Sierra Mazatèque (située au nord de l'État d'Oaxaca), il existe trois espèces d'hallucinogènes qui croissent à la saison des pluies, entre juin et septembre.   Le psilocybe mexicana Heim est un agaric à la robe foncée, de petite taille, qui pousse dans les champs de maïs et les pâturages. Il est appelé angelito (« petit ange ») en castillan et 'nti ni se (« oiseau ») dans la langue mazatèque. Le stropharia cubensis Earle croît à proximité des déjections des bovins. Il n'existait donc pas à l'époque préhispanique. Moins estimé, il est appelé « petit champignon de Saint Isidore le laboureur ». Le psilocybe caerulescens Murrill est quant à lui le plus puissant et le plus convoité. Il pousse dans les éboulis ou dans la bagasse des moulins à sucre. Il est appelé « éboulement » (derrumbe en castillan ou Ki so en mazatèque) autant pour le lieu où on le cueille que pour la puissance des visions qu'il procure.   Toutes ces espèces sont qualifiées de medicina par les habitants de la Sierra Mazatèque en raison de leurs vertus curatives (contre la goutte ou les fièvres).   Pour se soigner, le malade peut procéder de deux façons. Soit il décide de consulter un guérisseur qui, en voyageant grâce aux propriétés des champignons, sera capable d'identifier les causes du mal et prédire la guérison ou bien augurer la mort. Soit le patient les consomme lui-même. Il doit alors suivre les règles d'usage : tout d'abord, un régime alimentaire strict qui prohibe la consommation de viande, de haricots et toute relation sexuelle trois jours avant et après la prise. Puis, des marches dans la montagne environnante sont nécessaires pour que le voyageur s'acclimate au pays et se déprenne de ses préoccupations quotidiennes.   Planche d'illustration tirée de Roger Heim, Nouvelles investigations sur les champignons hallucinogènes, avec la collaboration de R.Cailleux, R.Gordon Wasson, P.Thévenard, édition du Muséum d'histoire naturelle (1966). DR, Author provided   Expérience indicible Le jour du rituel, il est indispensable de se laver à l'eau claire, de s'apprêter soigneusement et d'attendre la nuit avant de consommer les champignons à jeun. Le guérisseur ou la guérisseuse procède alors à une purification (limpia) à base d'herbes aromatiques et de fumée de copal.   Il demande à la personne quel est son mal (physique ou psychologique) et vérifie que les règles prescrites ont bien été respectées. Il passe alors un oeuf de poule ou de dindon sur les membres du corps, tête, bras, jambes, tout en psalmodiant des prières où le castillan et le mazatèque s'entremêlent. Une fois l'oeuf cassé, il peut en lire le contenu pour évaluer l'état psychique de son patient. Il est donc possible qu'il refuse l'accès au voyage si la personne n'est pas disposée à le faire.   Les champignons vont toujours par paires ; on dit qu'ils sont « mariés » (casados). Ils sont ainsi soigneusement placés deux par deux dans une feuille d'épi de maïs qui constitue un « voyage ». Selon son état, la personne peut consommer jusqu'à trois voyages, ce qui correspond à une vingtaine de champignons.   Documentaire de 1979 sur la guérisseuse mexicaine María Sabina (North Wind/YouTube).   La prise a toujours lieu à la tombée de la nuit, en général dans une pièce qui fait office de chapelle privative avec autel et images saintes. Les psilocybes sont consommés crus, de préférence frais. Ils doivent être longuement mastiqués et ingérés un à un, d'abord le chapeau puis le stipe. Leur goût âcre peut provoquer des vomissements. Au bout de quelques minutes, l'effet de la psilocybine, le principe actif du champignon, ne tarde pas à se manifester...   Il est difficile de décrire l'expérience extatique du champignon. Elle varie selon la personnalité, les portions consommées, l'humeur. Au cours du voyage même, les effets oscillent sans cesse : ils arrivent par vagues, plus ou moins espacées, qui s'étalent sur une plage de cinq à six heures. Les colorations (voile vert, formes géométriques fluorescentes), les objets qui se déforment et s'animent, la distorsion du temps et l'amplification des sons font passer de crises d'hilarité à des phases de profonde tristesse, voire d'angoisse, surtout lorsque surgit la première vague.   Des images d'une splendeur éblouissante cohabitent avec des visions terrifiantes. Enfance, mort, êtres aimés se télescopent. L'esprit est comme détaché de sa lourde enveloppe charnelle. Il pérégrine. Mais tout au long du voyage, la personne demeure consciente et assiste, comme médusée, à ses propres hallucinations. Au petit matin, les effets finissent par s'estomper, même si certaines visions reviennent dans les semaines qui suivent, l'espace d'un instant, comme un flash.   De la science au psychédélisme Au milieu des années 1950, trois savants explorent en voiture et à cheval la Sierra Mazatèque en quête des champignons disparus.   Le premier, Robert Gordon Wasson, est un banquier nord-américain, passionné de mycologie. Il est l'un des premiers à redécouvrir, en 1953, les usages rituels des psilocybes chez les Mazatèques. Dans la bourgade de Huautla de Jímenez (État d'Oaxana), située à une dizaine d'heures de Mexico, il rencontre la chamane María Sabina qui l'initie au « voyage ». Le second, Robert Heim, vient directement de Paris. Mycologue de renom, alors professeur au Museum national d'histoire naturelle, il est la caution scientifique de la mission. Le troisième, Guy Stresser-Péan, fin connaisseur des langues et des cultures indigènes, est leur guide.   Ensemble, ils lancent une vaste enquête pluridisciplinaire croisant les approches botaniques, linguistiques, ethnographiques.   De gauche à droite : Robert Heim, Guy Stresser-Péan et Robert Gordon Wasson avec la guérisseuse mazatèque María Sabina. DR, Author provided   À l'autre bout du monde, dans les laboratoires de l'Université de Bâle en Suisse, le chimiste Albert Hoffmann - père du LSD en 1943 - reçoit des échantillons d'agaric et parvient à en synthétiser la molécule, la psilocine. À Paris, l'éminent psychiatre Jean Delay, directeur de l'Institut de psychologie de l'Université de Paris, teste des médicaments à base de psilocine sur ses patients atteints de troubles mentaux. Cette enquête débouche sur un film documentaire réalisé par Pierre Thévenard en 1963 (jamais diffusé en salle) ainsi que sur plusieurs publications dont Les Champignons hallucinogènes du Mexique, oeuvre monumentale éditée en 1958 par le Muséum national d'histoire naturelle.   Les conclusions de ces études échappent rapidement au cadre académique auquel elles étaient destinées. Des routards avides de nouvelles expériences s'en emparent. Fuyant le matérialisme consumériste des villes, ils se lancent sur les chemins de la Sierra Madre à la recherche des agarics magiques et de María Sabina. Le tour du Mexique devient un voyage initiatique pour les adeptes de la vogue psychédélique. John Lennon, Jim Harrison, Aldous Huxley, Walt Disney, Bob Dylan, Mick Jagger suivent le mouvement.   Dans le miel Si le chaman a la capacité de guérir et de prédire, l'historien a, quant à lui, le pouvoir de voyager dans le temps. Faisons un saut de plusieurs siècles en arrière pour nous situer aux lendemains des Grandes découvertes.   On y découvre que les Européens se sont intéressés de manière précoce aux pharmacopées indigènes, s'appropriant les plantes américaines, comme l'a récemment montré Samir Boumedienne dans un passionnant ouvrage publié en 2016, La colonisation du savoir. Une histoire des plantes médicinales du Nouveau Monde.   Mais contrairement au tabac, au gaïac ou au cacao, les champignons hallucinogènes ne font l'objet d'aucune convoitise de la part des Espagnols, bien au contraire.   Bernardino de Sahagún. Wikipedia   Mexico, quelques années après la conquête par les Espagnols (1521), dans le monastère de Santa Cruz Tlatelolco. Le religieux Bernardino de Sahagún, l'un des premiers franciscains à s'installer dans l'ancienne capitale aztèque, s'entoure d'une équipe de traducteurs, de peintres et de médecins indigènes. Pendant de longues années, il s'évertue à collecter l'ensemble des savoirs et des moeurs des Mexica qu'il fait coucher sur les pages d'un épais manuscrit, L'Histoire générale des choses de la Nouvelle-Espagne. Dans les livres X et XI, il consacre plusieurs passages aux champignons malos, associés au malin, car « ils enivrent comme le vin et provoquent la luxure ». Il y décrit certains rites toujours vivaces qu'il compare à des orgies alors pratiquées de jour comme de nuit, en extérieur et ouvertes à l'homme ordinaire.   Un autre franciscain, Toribio de Benavente, dit Motolinía, grand défenseur des Indiens, nous livre d'autres informations dans son Histoire des Indiens de la Nouvelle-Espagne, publié en castillan en 1558. Il explique d'abord que certains champignons étaient consommés dans du miel : était-ce un moyen de les conserver ? Peut-on en déduire que les champignons étaient transportés et commercialisés ?   Il évoque ensuite les visions infernales des Indiens qui les ingéraient : couleuvres menaçantes, vers grouillants s'échappant des cuisses, tendances suicidaires. Les guérisseurs indigènes ne parvenaient-ils alors vraiment pas à canaliser leur extase ? On peut se permettre d'en douter. Les agarics si vénérés des indigènes, Motolinía en fait une claire manifestation du démon.   Interdits, pourchassés par les hommes d'Église au même titre que nombre de coutumes indigènes, les rites liés aux champignons sacrés, sans pour autant complètement disparaître, finissent par être occultés pour ne subsister que dans l'obscurité des nuits, au plus profond des villages isolés de l'altiplano.   Sans doute, leur nom, « chair de dieu », leurs potentialités divinatoires, leur mode de consommation étaient-ils trop proches de l'eucharistie. La christianisation des masses indigènes n'empêcha nullement la survivance du culte des champignons sacrés et encore aujourd'hui, les psilosybes mexicains sont appelés localement les santos niños.   Le maintien de leur consommation dénote une acculturation incomplète, de surface, voire une acculturation à rebours, les éléments du monde indigène digérant les repères occidentaux : au cours du long temps colonial (début XVIe-XIXe siècle), combien de métis, de sang-mêlés et d'Espagnols ont-ils été tentés par le voyage psychotrope ?   Ce que nous apprend l'archéologie Les seules traces de l'importance fondamentale jouée par les champignons hallucinogènes dans les rituels, la pharmacopée et plus globalement dans les visions du monde des peuples méso-américains, nous sont connues grâce à l'archéologie.   À proximité des prodigieuses pyramides de Teotihuacan, dans les somptueuses demeures de cette mégapole, les archéologues ont mis au jour des fresques datant de la période classique (300-600 après J.-C.).   La fresque de Tepantitla, reproduite dans la salle Teotihuacan du Musée national d'anthropologie de Mexico, est une représentation du paradis de Tlaloc, dieu de la pluie, celui qui pourvoit la terre en eaux et fait germer la végétation. Cet Eden, appelé le Tlalocan, est peuplé de nombreux petits personnages et habité d'une nature luxuriante.   La fresque de Tepantitla. DR, Author provided   Sur la fresque, de gauche à droite, on voit un homme avec une toge blanche sur le dos en train de cueillir des fleurs (à usage ornemental ?) sur une plante de grande taille non identifiée. Puis, un plan de maïs, céréale civilisationnelle par excellence des peuples méso-américains. Un peu plus loin, au centre de l'image, un magnifique plan de Datura stramonium aux effets hautement hallucinogènes qui semblent s'exercer sur l'homme assis à son pied ; c'est l'herbe du diable si chère à Carlos Castaneda. Juste à côté, un cacaoyer.   Enfin, en bas à droite de l'image, au bord du lac qui se déverse dans la rivière de Tlaloc, on distingue trois agarics, associés à des bivalves (coquillages). Au-dessus, un homme pleure. Des flots sortent de ses poumons. Il vient de périr noyé. Il est entouré de papillons qui représentent les âmes des défunts. Quatre volutes s'échappent de sa bouche. Il déclame une longue mélopée.   À l'origine du sentiment religieux ? D'autres traces font remonter l'usage des champignons à des époques plus anciennes. C'est le cas des « champignons de pierre ». Plus de deux cents exemplaires ont été retrouvés au Chiapas et au Guatemala en pays Quiché et Mam.   Certains datent de la période pré-classique ancienne, soit il y a plus de trois mille ans. Ces sculptures d'une trentaine de centimètres de hauteur avaient d'abord donné lieu à des interprétations phalliques avant d'être identifiées à des champignons. Nous ignorons leur usage, peut-être servaient-ils de pilon de mortier (molcajete). Pour broyer du maïs, des champignons ? Pour sacraliser la nourriture ? Toujours est-il qu'ils sont le plus souvent associés à des animaux : jaguars, crapauds, ici un lapin et un coati (photo). S'agit-il de figures individuelles qui renverraient au nahualli (soit des doubles animaux des humains) ?   Champignons de pierre. DR, Author provided   Ces différents éléments ont poussé les ethnobotanistes, à commencer par Robert Gordon Wasson, à proposer des lectures audacieuses des pratiques chamaniques dans les sociétés anciennes.   En rapprochant les cultes méso-américains du teonanacatl de la consommation d'amanites tue-mouches par les chamans sibériens, on pouvait raisonner à une échelle asiatico-américaine, faire remonter la connaissance des champignons psychotropes aux premiers peuplements du continent américain et émettre l'hypothèse que le sentiment religieux aurait pu prendre sa source dans les effets réellement hallucinatoires des psilocybes.   Arnaud Exbalin, Maître de conférence, histoire, Labex Tepsis - Mondes Américains (EHESS), Université Paris Nanterre - Université Paris Lumières   La version originale de cet article a été publiée sur The Conversation.  Cet article est paru dans L'Obs (site web)
</article>
<article titre="Distillerie Depaz : bien plus qu'un rhum, une histoire d'hommes" date="2018 05 14" journal="marie france (site web)">
Le rhum Depaz tire sa singularité de la localisation de sa plantation sur les flancs de la montagne Pelée, mais également de son histoire intimement liée à celle de la Martinique.    90 partages 1/52/53/54/55/5    À rhum d'exception, histoire d'exception. Le 8 mai 1902, la nuée ardente provoquée par l'éruption de la montagne Pelée détruit en quelques minutes la totalité de la ville de Saint-Pierre. Victor Depaz suit des études à Bordeaux lorsqu'il apprend que toute sa famille, travaillant dans la culture cannière, a péri. Devenu orphelin, il décide d'émigrer vers le Canada, mais son bateau fait escale dans son île natale, La Martinique, qu'il ne quittera plus. Il achète les terres recouvertes de cendres de l'habitation La Montagne, domaine créé en 1 651 par le premier gouverneur de l'île, pour y fonder sa distillerie. Deux de ses fils, Victor et Raoul, reprennent le flambeau dans les années 50, puis la distillerie Depaz est rachetée par La Martiniquaise, second groupe de spiritueux français (Dillon, Porto Cruz, Label 5 et Poliakov), il y a une vingtaine d'années.  Des rhums de caractère    Terres volcaniques, fertiles, où la pluie et le soleil abondent : la plantation Depaz est située sur le flanc de la montagne Pelée, et cela donne des rhums d'exception. Ceux-ci se déclinent en rhum blanc, rhum Plantation (vieux de 3 ans), Réserve spéciale (7 ans) et Grande réserve (8 à 10 ans) : des recettes inchangées depuis les origines. Pionnière du rhum vieux, la marque propose également quelques produits de prestige à l'image du Millésime 2002, du Finition porto (maturé en fût de porto) ou encore du Single Cask (chaque bouteille provenant d'un fût unique). Des produits à retrouver chez les cavistes mais pas en grande surface. Tournée vers la production raisonnée, la distillerie Depaz utilise l'eau de la montagne, dispose de panneaux solaires et se fournit en énergie grâce à la bagasse (résidus de canne à sucre).    Une distillerie authentiquement martiniquaise    Site ouvert et vivant, on peut facilement passer une journée à flâner sur le domaine Depaz. Au programme : le musée, la distillerie et l'impressionnant Château Depaz, bâtiment unique en Martinique construit en 1917 et 1922, seule la visite de ce dernier étant payante. Le tout entrecoupé d'un repas au Moulin à Cannes ou d'un goûter à La case à Louisette, pour finir par la boutique qui propose la dégustation. Le lieu accueille également des séminaires, des mariages et des concerts, notamment au moment du carnaval : difficile de ne pas tomber sous le charme.   Distillerie Depaz   Plantation de la Montagne pelée   97 250 Saint Pierre   Tel : 05 96 78 13 14   Fax : 05 96 78 30 28   http://depaz.fr/  Cet article est paru dans marie france (site web)
</article>
<article titre="QUESTIONNAIRE COMMUN SUR LE SECTEUR FORESTIER" date="2018 05 9" journal="Rapports - Agroalimentaire - ONUAA">
11 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... DÉFINITIONS  TERMES GÉNÉRAUX C Conifère  Bois provenant d’arbres classés en botanique sous le nom de "gymnospermes", par exemple: Abies spp, Araucaria spp, Cedrus spp, Chamaecyparis spp, Cupressus spp, Larix spp...    Pour voir le rapport :  http://www.fao.org/forestry/7801-04edfcb768e777551b122207dfdfda03d.pdf
</article>
<article titre="Export items" date="2018 05 7" journal="Rapports - Agroalimentaire - ONUAA">
NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... FCL Item code FCL Title Group Scientific Name Common Name Definition 0015 Wheat 1 - CEREALS AND CEREAL PRODUCTS Triticum spp.: Triticum aestivum Triticum durum Triticum spelta  common durum spelt  Common and...    Pour voir le rapport :  http://www.fao.org/fileadmin/user_upload/commodity/excel/export_items_1525666431.xls
</article>
<article titre="En Guadeloupe, la canne à sucre cherche un nouvel élan" date="2018 04 27" journal="Challenges (site web)">
La machette du coupeur s'abat, la canne à sucre se couche et rejoint le tas à ramasser. Sucre, rhum et énergie: en Guadeloupe, la canne sert à tout. Mais avec la chute du marché du sucre, la filière cherche un nouvel élan.   La récolte, à la main, bat son plein dans ce champ appartenant à la distillerie de rhum Bologne, comme dans près d'un tiers des champs de cannes en Guadeloupe. Entre Guadeloupe continentale et Marie-Galante, on compte 5.500 planteurs de cannes à sucre sur 14.000 hectares: 90% de la production est livrée aux sucreries, le reste à huit distilleries de rhum agricole.   Touchée par le cyclone Maria en septembre 2017, la production de canne à sucre guadeloupéenne a moins souffert que celle de la banane. Les producteurs s'attendent cependant cette année à une baisse de 7%, à 724.000 tonnes.   "Les bonnes conditions climatiques font de bons taux de sucre, par contre le tonnage n'est pas au rendez-vous", constate Franck Buffon, président de la Sicagra, l'une des quatre sociétés d'intérêt collectif agricole rassemblant les cultivateurs de canne en Guadeloupe.    La bonne nouvelle pour les planteurs, c'est qu'au bout de 15 ans de stabilité, "les prix ont été renégociés cette année avec les industriels" jusqu'à 2022: 4,52 euros de plus, amenant la tonne à 63 euros en incluant toutes les aides, explique Thierry Orfèvre, du GIE Cannes.   - Soutenir les prix -   Avec la fin des quotas sucriers européens, l'Etat a décidé de subventionner davantage la filière en augmentant de 38 millions d'euros l'aide aux industriels sucriers des DOM. En retour, ceux-ci devaient soutenir le prix du sucre.   Cette aide "va couvrir une partie de la fin des quotas mais pas la totalité", nuance Sylvain Icart, directeur de la sucrerie Gardel.    "Pour l'avenir, il faut essayer d'équilibrer les volumes vers le sucre de consommation directe et trouver de nouveaux débouchés", en jouant par exemple sur l'appellation "sucre de Guadeloupe", assure-t-il.   Dans une forte odeur de mélasse, les cannes apportées par camions dans l'usine du Moule sont broyées pour en extraire le jus qui, après évaporation et cristallisation, permet de produire le sucre.   Sur une production moyenne de 50.000 tonnes, l'usine sort 30.000 à 40.000 tonnes de sucre en vrac, à plus faible valorisation. Le reste consiste en sucres spéciaux, des sucres roux qui n'entrent pas en concurrence avec le sucre de betterave.   "Nous avons dès cette année décidé de changer l'équilibrage entre le vrac et le sucre de consommation", raconte le directeur, arrivé en janvier à la sucrerie.   Par ailleurs, Gardel utilise les déchets de cannes (la "bagasse") dans une centrale de cogénération électricité et vapeur, qui "génère 10% de la production électrique de la Guadeloupe", assure M. Icart.   - La consommation progresse -   La bagasse fait aussi tourner les machines de la distillerie Bologne.    Pour élaborer le rhum, la procédure est au début la même que pour le sucre. Mais le jus est ensuite gardé de 24 à 48 heures dans des cuves de fermentation où les levures donnent leur arôme à cette mixture brune. Elle passera ensuite dans un alambic pour aboutir à un alcool clair et titré à 75 degrés.    Le rhum est le seul alcool dont la consommation progresse aujourd'hui en France, en particulier les ambrés qui représentent le tiers des ventes, selon la fédération française des spiritueux.    "Nous payons la canne plus cher que ne le fait la sucrerie, et on règle en 15 jours", note François Monroux, directeur général des Rhums Bologne.    Face à la concurrence des rhums industriels, issus de la mélasse et non de la canne comme les rhums agricoles, l'enjeu consiste à monter en gamme.   Depuis les variétés de cannes noires utilisées jusqu'à une nouvelle ligne d'embouteillage gérant les formats spéciaux, en passant par un travail sur les levures, "nous sommes sans arrêt à la recherche du +premium+", souligne Harry Callard, président de l'entreprise familiale.   Bologne, qui produit du rhum blanc depuis 1887, s'est surtout lancé depuis une dizaine d'années dans les rhums vieillis en fûts de chêne pendant au moins trois ans, pour obtenir l'arôme et la couleur brune plébiscités par les consommateurs.   Cet article est paru dans Challenges (site web)
</article>
<article titre="Albioma : chiffre d'affaires en hausse de 6%" date="2018 04 25" journal="Boursier (site web)">
L'excellente disponibilité des installations en France, la rémunération complémentaire liée aux équipements de traitement des fumées et la bonne performance du Brésil se traduisent par une hausse de 6% du chiffre d'affaires du premier trimestre 2018 (+5% hors effet prix des combustibles), qui atteint 104,2 millions d'euros sur la période contre 98,8 millions d'euros au premier trimestre 2017. Frédéric Moyne, Directeur Général, a déclaré : "Après un bel exercice 2017, nos centrales continuent d'afficher de solides performances sur ce premier trimestre. Notre usine Galion 2 alimente désormais en vapeur la sucrerie adjacente, ce qui a permis le démarrage de la campagne sucrière dans les délais, et devrait injecter ses premiers kWh sur le réseau électrique martiniquais dans les prochains jours, avec l'objectif d'une mise en service à la fin du premier semestre 2018. À La Réunion, nous poursuivons les travaux IED au Gol et à Bois-Rouge et prévoyons d'initier la phase de test de la turbine à combustion de Saint-Pierre."   Brésil : maintenance annuelle des deux centrales pendant l'inter-campagne sucrière.   Albioma Rio Pardo Termoelétrica et Albioma Codora Energia ont réalisé leur maintenance annuelle dans de très bonnes conditions pendant les trois premiers mois de l'exercice. Albioma Codora Energia a par ailleurs continué à produire en janvier, grâce à l'utilisation de stock de bagasse excédentaire de 2017. L'activité enregistre un chiffre d'affaires de 1,5 million d'euros (contre 1,0 million d'euros au premier trimestre 2017).   Île Maurice : maintenance annuelle programmée et arrêt fortuit sur une centrale.   La disponibilité des installations mauriciennes est de 83% (à comparer à 99,4% au premier trimestre 2017), du fait de l'arrêt annuel de la centrale OTEO La Baraque et d'un arrêt fortuit sur cette même centrale. La production est en baisse de 20% à 267 GWh (336 GWh au premier trimestre 2017).   Confirmation des objectifs Le Groupe confirme ses objectifs 2018 d'EBITDA de 158 à 166 millions d'euros et de résultat net part du Groupe de 37 à 42 millions d'euros.   Prochain rendez-vous : Assemblée Générale annuelle, le 30 mai 2018 à 15h00.
</article>
<article titre="Jour et nuit" date="2018 04 24" journal="La Montagne  Vichy">
Cusset Il détenait un pistolet à impulsion électrique   Intervenant pour mettre fin à une bagarre avenue de l'Europe, à Cusset, dans la nuit de dimanche à lundi, les policiers ont contrôlé un homme de 22 ans qui détenait un pistolet à impulsion électrique. L'arme a été confisquée. L'homme devra s'expliquer ultérieurement.   Vichy Vol à l'étalage   Un mineur de 14 ans a tenté de voler un article d'une valeur de 90 ?, samedi matin, dans un magasin des Quatre-Chemins. Il en avait préalablement retiré l'antivol, mais a toutefois été intercepté aux caisses. Le magasin a été dédommagé, le jeune homme sera, de son côté, prochainement convoqué au tribunal. Intempéries Accidents matériels en série   Les fortes pluies tombées sur le bassin vichyssois, hier matin, ont causé une série d'accidents matériels. Un premier a eu lieu à Varennes-sur-Tèche , vers 8 heures. Les gendarmes de Lapalisse sont intervenus au lieu-dit « Bagasse », après qu'un conducteur a perdu le contrôle de sa voiture, finissant contre un mur d'habitation. Le dépistage d'alcool était négatif, l'homme a été transporté à l'hôpital de Moulins par les pompiers de Lapalisse et Jaligny, pour passer des examens de contrôle.   Vers 9 heures, à Montaiguet-en-Forez au lieu-dit « Tonkin », un poids lourd a, de son côté, terminé dans un fossé après que son conducteur en a également perdu le contrôle. Le dépistage d'alcool était aussi négatif. Aucun transport médical n'a été nécessaire.   Vers 9 h 40, à Saint-Pourçain-sur-Sioule , sur la D46 en direction de Montmarault, deux véhicules roulant en sens opposés se sont percutés sur la chaussée mouillée. L'un des deux conducteurs a été transporté au centre hospitalier de Vichy pour passer des examens de contrôle.   Enfin, sur l'A 71, peu après 9 heures et à hauteur de la commune de Vicq , la voiture d'une automobiliste a terminé sa course sur le bas-côté de la route. Cette femme de 58 ans, qui a perdu le contrôle sous la pluie battante, a été prise en charge par les pompiers de Gannat, Ébreuil et Vichy, puis transportée à l'hôpital de Vichy pour contrôles.   Abrest Conducteur ivre   Un automobiliste âgé de 37 ans originaire du Puy-en-Velay a été contrôlé, dimanche, vers 22 heures, alors qu'il se faisait remarquer par une conduite zigzaguante. Cet homme affichait une alcoolémie de 1,28 g/l de sang. Une procédure à son encontre a été ouverte.   Coutansouze Le véhicule termine sur le toit   Dimanche après-midi, peu après 17 heures, un homme de 68 ans a perdu le contrôle de son véhicule à l'intersection de la RD 987 et de la RD 118. Son véhicule a terminé sur le toit sur le bas-côté de la route. Le sexagénaire a été transporté par les pompiers de Bellenaves à l'hôpital de Vichy.   Voussac Effraction   Des traces d'effraction ont été relevées sur la fenêtre d'une habitation. Aucun vol n'a été constaté, la maison n'a visiblement pas été fouillée.
</article>
<article titre="Jour et nuit" date="2018 04 24" journal="La Montagne  Moulins">
Moulins Deux jeunes interpellés après l'incendie d'un container   Deux adolescents de 15 et 16 ans ont été interpellés par les policiers dans la nuit de vendredi à samedi. Ils sont suspectés d'avoir mis le feu à un container, avenue Etienne-Sorrel, dans le quartier des Champins. Ils ont été placés en garde à vue au commissariat et auraient reconnu être les auteurs de l'incendie. Ils comparaîtront en février 2019 devant le juge des enfants.   Intempéries Accidents matériels   Les fortes pluies tombées sur le bassin vichyssois hier matin ont causé une série d'accidents matériels. A Varennes-sur-Tèche vers 8 heures, les gendarmes de Lapalisse sont intervenus au lieu-dit « Bagasse » après qu'un conducteur a perdu le contrôle de sa voiture, finissant contre un mur d'habitation. Il a été transporté à l'hôpital de Moulins pour des examens de contrôle. Vers 9 heures, à Montaiguet-en-Forez , un camion dont le chauffeur a perdu le contrôle, a terminé dans un fossé. Vers 9 h 40, à Saint-Pourçain-sur-Sioule , sur la D46, en direction de Montmarault, deux véhicules se sont percutés sur la chaussée mouillée. Allier Alcoolémies et vitesses   Plusieurs conducteurs ont été contrôlés en excès de vitesse le week-end dernier. A Sazeret , sur la RCEA, quatre personnes ont été prises, samedi, à 150, 153, 151 et 154 km/h au lieu de 110. A Target , sur l'A71, dimanche, trois automobilistes ont été contrôlés à la même vitesse de 172 km/h au lieu de 130. Vendredi, à Saint-Prix , sur la RN7, deux conducteurs ont été flashés à 152 km/h et sur la RD46, à Saint-Marcel-en-Murat , un automobiliste roulait à 131 au lieu de 90. Tous ont fait l'objet d'une suspension de permis de conduire.   Les gendarmes ont également contrôlé des conducteurs en alcoolémie. A Commentry , samedi, deux personnes roulaient avec des taux de 1,52 g et 0,86 g. A Vallon-en-Sully , un, Bourbonnais avait un taux de 0,94 g. Vendredi soir, à Meillers , sur la RD46, un homme accusait un taux d'alcoolémie de 1 g. A Lusigny , un Bourbonnais contrôlé samedi après-midi, avait un taux de 1,32 g et le même jour, à Saint-Bonnet-de-Tronçais , sur la RD39, un conducteur du Cher roulait avec un taux de 0,84 g.   Stupéfiant Positifs au cannabis   Deux Bourbonnais et un Puydomois ont été dépistés positifs au cannabis le week-end dernier, à Montluçon , Montmarault et Néris-les-Bains . Agés de 17 ans (contrôlé au guidon d'un cyclomoteur), 23 et 18 ans, ils seront convoqués au tribunal si la prise de sang confirme le résultat du dépistage salivaire. Le jeune homme de 18 ans posssédait également 0,54 g d'herbe sur lui.
</article>
<article titre="Allier : la pluie provoque une série d'accidents matériels dans le bassin de Vichy" date="2018 04 23" journal="La Montagne (site web)">
Les fortes pluies tombées sur le bassin vichyssois, ce lundi matin, ont causé une série d'accidents matériels.    Un premier a eu lieu à Varennes-sur-Tèche, vers 8 heures. Les gendarmes de Lapalisse sont intervenus au lieu-dit « Bagasse », après qu'un conducteur a perdu le contrôle de sa voiture, finissant contre un mur d'habitation. Le dépistage d'alcool était négatif, l'homme a été transporté à l'hôpital de Moulins par les pompiers de Lapalisse et Jaligny, pour passer des examens de contrôle.  Vers 9 heures, à Montaiguet-en-Forez au lieu-dit « Tonkin », un poids lourd a, de son côté, terminé dans un fossé après que son conducteur en a également perdu le contrôle. Le dépistage d'alcool était aussi négatif. Aucun transport médical n'a été nécessaire.   Une femme blessée dans une collision en chaîne à Vassel (Puy-de-Dôme)    Vers 9?h?40, à Saint-Pourçain-sur-Sioule, sur la D46 en direction de Montmarault, deux véhicules roulant en sens opposés se sont percutés sur la chaussée mouillée. L'un des deux conducteurs a été transporté au centre hospitalier de Vichy pour passer des examens de contrôle.    Enfin, sur l'A 71, peu après 9 heures et à hauteur de la commune de Vicq, la voiture d'une automobiliste a terminé sa course sur le bas-côté de la route. Cette femme de 58 ans, qui a perdu le contrôle sous la pluie battante, a été prise en charge par les pompiers de Gannat, Ébreuil et Vichy, puis transportée à l'hôpital de Vichy pour contrôles.   Pierre Géraudie  Cet article est paru dans La Montagne (site web)
</article>
<article titre="Monoprix flashe sur la créatrice paloise" date="2018 04 18" journal="Sud Ouest  Béarn et Soule">
ODILE FAURE   Il fallait faire vite, hier, sur le corner disposé à l'entrée du magasin Monoprix de Pau pour pouvoir repartir avec un objet signé Nadia Gallardo. Les petites merveilles en céramique, en bois d'acacias, en pierre et en lin à prix mini se sont vendues comme des petits pains. Une soixantaine de personnes est, en effet, venue saluer la sortie de la collection " Sur le chemin " de la créatrice paloise, retenue par l'enseigne nationale. Monoprix la diffuse jusqu'à épuisement des stocks (et il en reste à Pau !) dans une centaine de magasins en France, dont une vingtaine à Paris mais aussi dans toutes les grandes villes, ainsi qu'à Pau donc et à Saint-Jean-de-Luz. " Notre but est de dénicher des talents et de démocratiser le beau ", explique Céline Chevreux, directrice du magasin de la place Clemenceau. C'est la première fois qu'elle met en avant une créatrice du cru et elle en est ravie.   Une centaine de magasins   Nadia Gallardo est connue dans le monde de la mode et de la création. Elle s'est illustrée en janvier au magasin Merci de Paris avec ses bocaux Le Parfait transformés en imparfaits qu'elle a fait fondre dans la verrerie d'origine de Puy-Guillaume (63). Cette fois-ci, elle entre dans la maison de tout le monde grâce à Monoprix, avec sa vaisselle en céramique blanche, belle et épurée, où la matière est visible, inspirée de la bagasse des cannes à sucre. " Elle va à l'essentiel ", confie une cliente et amie. Nadia Gallardo indique avoir puisé son inspiration dans le potager de son grand-père, " souvenir de terre, de petits cailloux, d'herbes. "   Née à Coarraze de grands-parents espagnols, Nadia Gallardo a fait ses études au lycée Barthou, à Pau, puis aux Beaux-Arts de Toulouse et Paris. Elle a ensuite passé plus de dix ans dans la capitale auprès de créateurs pour revenir ensuite au pays avec son mari et son premier enfant en 1998. En Béarn, elle a travaillé pour les tissages Moutet, les bérets Beatex, les étains Goardère d'Orthez.   Aujourd'hui, elle poursuit sa quête dans son atelier du quartier Saint-Dominique et n'a pas hésité à frapper à la porte de Monoprix. " La directrice artistique m'a reçue à Paris; j'étais venue avec des objets, des dessins et elle m'a programmée. J'ai ensuite fourni des fiches techniques pour la fabrication en série des objets. J'ai eu une très grande liberté de création. J'ai dû aussi faire des choix quand la réalisation n'était pas possible. Aujourd'hui, c'est très chouette de savoir que ce sont des objets que les personnes vont pouvoir trouver à petits prix. " Nadia Gallardo a construit sa collection de vaisselle, de linge de maison, de vannerie et de papeterie à partir de matières nobles et simples. Résultat, une gamme modeste mais qui a du chien. Tout à l'image de sa génitrice.
</article>
<article titre="tour de ville Un restaurant végétarien et végétalien" date="2018 04 14" journal="La Dépêche du Midi  Hautes-Pyrénées">
Depuis début avril, Planète Veggie, restaurant végétarien et végétalien, a ouvert ses portes au 533, rue des Résistants à Lannemezan. Florent Ayroulet et Lætitia Ritzzo, sa compagne, ont acheté en 2017 cette grande demeure dont une partie est privative et l'autre réservée au restaurant. À l'intérieur, 13 personnes peuvent se restaurer et 10 autres en terrasse. Les propriétaires travaillaient tous les deux au magasin Biocoop à Lannemezan. Lui y est toujours tandis que Lætitia, elle, s'est lancée dans la restauration. « Depuis toute petite, j'ai une passion pour la bonne cuisine. Venant d'une famille sicilienne et d'origine pied-noir, j'ai une bonne pratique de ce qui est bon ! Je cuisine avec plaisir. C'est ma façon aussi de pouvoir m'exprimer aux travers des plats », explique-t-elle. Du fait maison Elle veut désormais créer, faire découvrir sa cuisine. De par leur conviction écologique et éthique, le couple rajoute : « On peut bien se nourrir, sans viande, du moment que tout est bien étudié, bien préparé, bien cuisiné ». Tout est fait maison de A à Z, les producteurs sont locaux, labellisés Nature et Progrès ou AB. Sur place, Lætitia Ritzzo propose également à la vente d'autres produits faits maison (comme le confit de coing aux épices).   On peut retrouver des plats cuisinés sous emballage écolo et compostable (bagasse) en vente au magasin Biocoop de Lannemezan. Les menus à la carte, au restaurant, sont à partir de 4 €, et le plat du jour à 12,50 €. La carte du jour est changée chaque semaine, pour mieux faire voyager les clients. Restaurant ouvert du mardi au vendredi, de 10 h 30 à 14 h 30 et de 17 h 30 à 20 heures; le samedi, de 10 h 30 à 14 h 30 et de 17 h 30 à 23 heures; le dimanche, de 10 h 30 à 14 h 30.   Planète Veggie, restauration sur place ou à emporter, tél. 05.62.49.23.31/www.planete-veggie.bio
</article>
<article titre="Pino Café, pionnier des sacs compostables à Montréal-Nord" date="2018 04 10" journal="Guide de Montréal-Nord, no. Vol. 61 n° 13">
Écologie. La Coop Éconord veut inciter les commerçants de l'arrondissement à proposer des sacs compostables à la place de ceux en plastique. Le Pino Café est la première enseigne à s'engager dans cette démarche. « J'aimerais que Montréal-Nord soit le premier arrondissement libéré des sacs en plastique. Ce serait un beau message », ambitionne Lucy Pino, propriétaire du Pino Café sur la rue de Charleroi. Sensibilisée aux enjeux environnementaux, cette commerçante était à la recherche depuis un peu plus d'un an d'une alternative écoresponsable aux contenants en plastique qu'elle utilisait dans son établissement. La solution est finalement venue de la Coop de solidarité Éconord qui, grâce à un partenariat avec une entreprise québécoise, fournit depuis peu des sacs et ustensiles compostables fabriqués dans la province. « Les gens peuvent même utiliser ces sacs dans le bac brun. Ils disparaissent complètement, aucune particule ne demeure », assure Julie Demers, la directrice d'Éconord.   Ces sacs sont fabriqués à base de bagasse, un résidu végétal de la transformation de la canne à sucre et ont donc un impact minime sur l'environnement. L'arrondissement de Montréal-Nord soutient d'ailleurs Éconord dans sa démarche en achetant les ustensiles compostables vendus par l'organisme pour ses événements.   Promouvoir l'initiative Depuis le 1er janvier, un nouveau règlement de la Ville de Montréal interdit les sacs en plastique d'une épaisseur de 50 microns et moins. L'administration mise sur le fait que les citoyens sont plus enclins à réutiliser des sacs plus épais. Les commerçants ont néanmoins jusqu'au 1er juillet pour écouler leur stock de sacs fins et la Coop Éconord veut profiter de cette échéance pour promouvoir sa solution compostable dans l'arrondissement.   « Les sacs en plastique plus épais seront encore autorisés, mais ça reste du plastique. On va donc lancer une campagne auprès de 480 commerces de Montréal-Nord pour expliquer ce nouveau règlement et éduquer nos commerçants pour trouver des solutions et comprendre leurs besoins », indique Julie Demers.   Si la responsable d'Éconord et la propriétaire du Pino Café garantissent que les sacs compostables sont suffisamment résistants, la question du prix peut encore constituer un frein. Ils sont vendus 135 $ la boîte de 500, tandis qu'une entreprise d'emballage propose des sacs en plastique blancs conformes à la nouvelle réglementation à 55 $ pour la même quantité. Lucy Pino plaide toutefois pour une responsabilisation écologique.   « C'est un investissement, mais la conscience de bon citoyen doit prendre le dessus sur le coût. On n'a pas augmenté nos prix par rapport à ça et ce n'est pas une promotion hypocrite, c'est parce que je suis consciente de mon empreinte », soutient la commerçante. Cette dernière entend d'ailleurs participer à la campagne d'Éconord afin d'encourager les autres cafés et magasins de l'arrondissement à suivre son initiative.
</article>
<article titre="Energies renouvelables : la prime «  bagasse » revalorisée" date="2018 04 8" journal="Collectivités locales (site web)">
Conformément à l'engagement du Premier ministre lors de son déplacement à la Réunion au mois de juin, le montant de la « prime bagasse » a été revalorisé. Cette prime est versée pour toutes les centrales existantes produisant de l'électricité à partir du résidu de canne à sucre en Outre-Mer. L'arrêté du 18 octobre 2015 fixe le montant de cette prime à 14,5 €/tonne de canne à sucre, contre 13 € auparavant. Le montant de la prime bagasse était jusqu'alors indexé sur le cours du charbon et le cours du CO2, ainsi que sur le taux de fibre présent dans la canne, avec de plus un coefficient de dégressivité de la prime nécessitant une augmentation du taux de fibre de 3 % par an pour maintenir la rémunération. Le nouvel arrêté supprime l'indexation de la prime sur le cours du charbon et du CO2, qui dans son principe ne faisait que maintenir la dépendance des systèmes électriques insulaires à la fluctuation du cours des énergies fossiles. Le coefficient de dégressivité est neutralisé jusqu'à 2017 inclus.   Cet article est paru dans Collectivités locales (site web)
</article>
<article titre="Capacité des EnR dans le monde : 167 GW ajoutés en 2017 [Étude Irena]" date="2018 04 6" journal="GreenUnivers (site web)">
2018-05-29T16:29:37+00:00    Quelque 167 GW de capacité EnR ont été ajoutés en 2017, soit une augmentation de 8,3%. La puissance totale atteint 2 179 GW, selon le rapport 2017 de l'Agence internationale des énergies renouvelables (Irena). Le solaire PV prend la tête avec une croissance de 32%, suivi par l'éolien qui a augmenté de 10%. Les prix ont baissé de 73% pour le PV et de près d'un quart pour l'éolien terrestre. L' off grid  a connu une augmentation sans précédent : +10% - environ 146 millions de personnes ont désormais accès à de l'électricité renouvelable hors-réseau. L'hydroélectricité a connu la plus faible croissance de la décennie. L'Asie en tête  L'Asie a largement participé à la croissance des EnR, avec 106 GW (+ 58%). La Chine représente près de la moitié de la capacité mondiale installée en 2017, avec entre autres 53 GW de solaire et 15 GW éoliens.   L'Europe, elle, a participé à hauteur de 24 GW, suivie par l'Amérique du Nord avec 16 GW. En Amérique du Sud, le Brésil se démarque avec l'installation de 1 GW solaire : c'est 10 fois plus par rapport à 2016.   La France possède une capacité totale de 46,6 GW en 2017, dont 13,1 GW d'éolien terrestre, 8,1 GW de solaire, 1,3 GW de bioénergies et 0,3 GW de biogaz. Des chiffres qui diffèrent légèrement du Panorama du SER, mais qui se rapprochent de ceux du Commissariat général au développement durable.   Plus de 200 pays et territoires ont été évalués par l'Irena selon plusieurs catégories : EnR globales, solaire photovoltaïque et thermique, éolien terrestre et offshore, bioénergies, déchets renouvelables, hydroélectricité, énergie géothermique, biogaz, off-grid, biocarburants et bagasse.    Consultez "Les statistiques de capacité renouvelables 2018" de l'Irena   (ici en PDF) :  Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="Yumi cherche la  bagasse!" date="2018 04 3" journal="Emballages magazine.com">
Les bouteilles végétales de Lyspackaging font l'objet d'un malus de 100%.-   Le débat est bien connu des experts de l'emballage et du recyclage et Yumi y prend sa part dans LSA le 30 mars. Créé en 2014 par Raphaël de Taisne et Louis de Bentzman, Yumi commercialise des jus de légumes issus de l'agriculture biologique. Ces jus sont traités par hautes pressions, un procédé de pasteurisation à froid également connu sous le nom de pascalisation. Mais la jeune entreprise a également choisi de les conditionner dans les bouteilles végétales fabriquées par Lyspackaging implanté à Chaniers (Charente-Maritime) plutôt qu'en polyéthylène téréphtalate (PET) voire en PET recyclé (rPET).   Veganbottle Issue de résidus de bagasses de canne à sucre, la Veganbottle est présenté comme compostable par Yumi et Lyspackaging mais considérée, en l'absence d'une filière de traitement constituée, comme un perturbateur de tri par Citeo, l'organisme agréé pour le traitement des emballages ménagers et des papiers. Yumi se voit donc infliger un malus de 100% au titre du Point vert comme le barème le prévoit explicitement. Raphaël de Taisne n'apprécie pas : "Cette annonce m'a mis en colère car elle met à mal tous nos efforts - et ils sont lourds à supporter pour une jeune pousse comme la nôtre - pour agir de façon responsable pour l'avenir de notre planète. Pire, notre initiative, loin d'être encouragée, est récompensée par un doublement de la taxe de recyclage des emballages plastique".
</article>
<article titre="Conseil Albioma Le coup de coeur" date="2018 03 10" journal="Investir-Le Journal des finances, no. 2305">
L'année 2017 s'est bien terminée pour le producteur d'électricité, implanté dans les territoires d'outre-mer, malgré le report du démarrage de la centrale de Galion 2 en Martinique. On l'attendait pour la fin de l'exercice mais, pour des raisons techniques, cette unité de 40 mégawatts « devrait être couplée au réseau ce mois-ci et mise en service au deuxième trimestre 2018 », A précisé Julien Gauthier, directeur financier, rappelant que la campagne sucrière débute dès avril. Cette centrale thermique, qui a coûté 205 millions d'euros, brûlera de la bagasse, le résidu de la canne à sucre, durant la récolte, puis des granulés de bois le reste de l'année, et non pas du charbon comme les anciennes unités. Ces dernières doivent d'ailleurs être converties au bois. Même sans la contribution de Galion 2, Albioma a dégagé des résultats 2017 meilleurs que prévu. « Nos centrales thermiques ont connu une très bonne disponibilité malgré des arrêts par tranches importants pour travaux »,  a commenté le directeur financier.   Pour la première fois, le chiffre d'affaires a dépassé 400 millions d'euros (à 403 millions), en progression de 10 % mais de 4 % si l'on neutralise la répercussion dans les tarifs de l'augmentation des prix du combustible.   A 138,3 millions, l'Ebitda (excédent brut d'exploitation), en amélioration de 5 %, est ressorti dans le haut de la fourchette prévue de 130 à 138 millions. Fort de produits fiscaux non récurrents, le bénéfice net s'est accru de 13 %, à 37,4 millions, au-dessus de l'objectif de 30 à 35 millions. Les actionnaires se verront proposer un dividende de 0,60 € (+ 5,3 %) avec une option de paiement à 50 % en actions.   La dette à un pic Cette disposition permet d'économiser du cash alors que la dette (de 613 millions fin 2017, ou 4,4 fois l'Ebitda) sera à son pic cette année et en 2019, reflétant les investissements consentis par le groupe. Ces investissements de développement ont atteint 147 millions l'an dernier, « au-delà du rythme habituel d'environ 100 millions »,  a indiqué Julien Gauthier. Outre Galion 2, une autre centrale, une turbine à combustible (du bioéthanol), doit entrer en service au deuxième trimestre à La Réunion. Par ailleurs, le Brésil absorbe 40 % des investissements pour deux centrales thermiques, là aussi couplées à des sucreries et déjà remises aux standards du groupe depuis leur acquisition, et pour deux autres unités en cours de modernisation.   Le prochain démarrage des deux nouvelles centrales va entraîner un bond des résultats cette année. L'Ebitda est visé entre 158 et 166 millions d'euros (+ 17 %, en milieu de fourchette) et le bénéfice net entre 37 et 42 millions, contre un résultat récurrent de 32 millions en 2017.   Notre conseil Acheter Le rebond attendu des résultats cette année est confirmé après une très bonne année 2017. En outre, le groupe se dit peu sensible à une remontée des taux, sa dette étant couverte ou à taux fixe à 91 %.   Par Cécile Le Coz
</article>
<article titre="Albioma fait le pari de la  bagasse dans les îles" date="2018 03 8" journal="Les Echos, no. 22651">
D'ici à la fin du mois de mars, Albioma espère connecter au réseau électrique de Martinique sa nouvelle centrale biomasse « Galion 2 » . Une unité de 40 mégawatts, détenue à 80 % par l'ex-Séchilienne-Sidec et à 20 % par Cofepp, le groupe sucrier plus connu sous le nom de ses rhums - notamment La Martiniquaise. Le spécialiste de la biomasse outre-mer mise de fait sur la bagasse - le résidu fibreux de la canne à sucre - pour lui fournir,  « à terme »  , 40 % du combustible nécessaire pour produire de l'électricité (et de la vapeur pour le groupe sucrier). Albioma, qui importera d'ici là l'essentiel des pellets des Etats-Unis, a conclu avec EDF un contrat d'achat de l'électricité garanti pendant trente ans. Le prix d'achat est confidentiel, mais il se compare avec le prix de l'électricité issue de combustibles fossiles, indique-t-on dans l'entreprise.   Remplacer les combustibles fossiles par de la biomasse : la stratégie d'Albioma a été confirmée par son directeur général, Frédéric Moyne, mercredi, à l'occasion de la publication des résultats annuels. L'an dernier, l'entreprise a enregistré un chiffre d'affaires de 403 millions d'euros, en hausse de 10 %. Le résultat net a, quant à lui, progressé de 13 %, à 37,4 millions d'euros.   Développement au Brésil Cette année, Albioma table sur un résultat brut d'exploitation (Ebitda) entre 158 et 166 millions d'euros (contre 138 millions d'euros en 2017), et sur un résultat net part du groupe entre 37 et 42 millions d'euros. Albioma compte ainsi poursuivre son développement au Brésil (numéro un mondial de la canne à sucre), voire à d'autres géographies (Colombie, Thaïlande...). Mais pas en métropole, redoutant un effet de ciseaux sur le coût du combustible et se jugeant trop peu familier de la gestion des réseaux de chaleur.   Alors qu'une phase de consolidation est en cours dans les énergies vertes, Albioma, dont les deux tiers du capital sont en Bourse, ne ressemble guère aux acteurs du solaire et de l'éolien.  « Albioma est très difficile à vendre, notamment à cause du statut des industries électriques et gazières dans les îles »  , estime notamment un banquier - un préavis de grève est, de fait, en cours à la Réunion.  « Quand notre mix sera à 80 % renouvelable (contre 53 % l'an dernier, NDLR) et qu'on se sera un peu plus développé à l'international, l'entreprise sera une cible plus attractive »  , explique-t-on au sein de l'entreprise.   V. L. B.
</article>
<article titre="Albioma ou comment gérer au mieux la transition dans les DOM" date="2018 03 7" journal="GreenUnivers (site web)">
2018-03-07T18:36:57+00:00    L'énergie de la bagasse se porte bien, sur le marché de production d'électricité le plus cher de France, celui des DOM - 290 ?/MWh en moyenne, subventionné à hauteur de 1,7 Md? dans le cadre de la péréquation tarifaire. Spécialiste de la valorisation énergétique de ce sous-produit de la canne à sucre et acteur majeur de la production d'électricité à La Réunion, à Maurice et en Guadeloupe, Albioma affiche en effet de bons résultats pour l'exercice 2017.   Le chiffre d'affaires progresse de 10%, à 403 M? et la rentabilité de cette société cotée en Bourse (flottant de 60%) augmente de 13%, à 37,4 M?. Une performance solide car elle est basée sur le rendement qu'obtient Albioma de sa matière première, en l'occurrence la production de 120 kWh par tonne de bagasse. Par comparaison, dans les deux centrales qu'il a reprises ces dernières années au Brésil, le rendement était de 40 kWh. Albioma est en train de le doubler. La reconversion impose de stopper les machines  Le parc de centrales améliore en outre son taux de disponibilité, de 89,6% - il a d'ailleurs continué à produire pendant les cyclones dans les Antilles. La question pour Albioma est maintenant de ne pas réduire ce taux pendant la phase d'abandon du charbon pour les granulés de bois importés et certifiés. Rappelons que le charbon alimente les centrales lorsque la campagne sucrière est terminée - c'est-à-dire quand même la moitié de l'année... « Nous visons un mix électrique à 80% renouvelable en 2023, annonce le directeur général Frédéric Moyne, mais cela nous impose de gérer au mieux les arrêts de plusieurs mois nécessaires au changement. Le vrai sujet pour Albioma est celui de la reconversion ». Une reconversion qui consiste aussi à tenter de mobiliser d'autres déchets locaux, des résidus végétaux ou des combustibles solides issus de la collecte des déchets.   Le Brésil, prudemment Le modèle reste en tous cas assez robuste pour qu'Albioma ait investi 265 M? dans deux nouvelles centrales, à la Martinique (ce qui va lui donner une part de marché de 20% dans l'électricité) et à La Réunion. La première sera alimentée en bagasse/pellets, la seconde majoritairement au bioéthanol ; les deux entreront en service cette année. Son expertise dans la bagasse électrique incite aussi le développeur à pousser un peu plus ses pions au Brésil, avec deux récentes acquisitions à booster. Le pays pourrait être une sorte d'eldorado pour le développeur français, car 700 millions de tonnes de résidus de canne à sucre y sont à brûler chaque année - contre 2,5 million de tonnes à La Réunion... Mais manifestement Albioma compte investir avec mesure sur un marché certes sorti de récession mais à risque : cours du réal, marché spot, taux d'intérêt... Aux yeux d'Albioma, ce n'est pas parce que la ressource est massive qu'il existe un marché : « Nous nous intéressons à tous les gisements de résidus végétaux, par exemple la cosse de riz, en Amérique du Sud et en Asie. Mais il faut une approche industrielle : nous avons notamment besoin d'une disponibilité maximale et d'une saisonnalité pertinente », explique Frédéric Moyne qui regarde en ce moment du côté de la Colombie et de la Thaïlande. En revanche, la méthanisation est sortie des ambitions d'Albioma, même si la branche n'est plus en déficit. Le groupe, alors appelé    Séchilienne Sidec,    avait pourtant racheté 60% de Méthaneo en 2012, l'un des pionniers du biogaz à l'époque avec un portefeuille de 22 projets pour 25 MW. Aujourd'hui, Albioma n'exclut pas de se séparer des trois centrales en service dans l'Hexagone. Selon un porte-parole du groupe, « la production de biométhane ne dégage aucune rentabilité à l'heure actuelle et les processus français restent difficiles à maîtriser, donc à industrialiser. »    Le stockage en point d'interrogation (Crédit : Albioma)Albioma ne fait pas que sortir progressivement du charbon. Il est aussi revenu vers les photons et se présente même comme le premier producteur du photovoltaïque outre-mer, avec 95 GWh en 2017 et 17% du marché. « Le développement de projets solaires avec stockage est l'un de nos trois axes stratégiques », annonce Frédéric Moyne. Il déclare 16 MW de stockage installé et a gagné 10 MW lors des compétitions de 2016 et 2017. Mais ne veut pas annoncer d'objectifs à moyen terme. « Le gestionnaire de réseau comme le régulateur préconisent un stockage centralisé et non pas réparti », explique-t-il.   De fait, dans ses propositions relatives aux zones non interconnectées de janvier dernier, la CRE recommande « de ne pas lancer de nouvel appel d'offres PV + stockage mais de privilégier un soutien distinct du photovoltaïque et du stockage centralisé », pour mieux correspondre à l'évolution des besoins du réseau et à moindre coût.   La question est majeure, car le réseau électrique des îles a de plus en plus besoin de stabilité, en raison de l'accueil des EnR non commandables au-delà du seuil conventionnel de déconnexion. Reste à décider comment procéder, dans un paysage qui n'a pas besoin de capacité nouvelle : la population n'augmente plus, au contraire, et la consommation d'énergie globale va maintenant diminuer dans la plupart des îles.  Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="Albioma tient le cap de son plan d'investissements" date="2018 03 7" journal="L'AGEFI Quotidien">
A mi-parcours de son plan 2013-2023, visant à investir 1 milliard d'euros dans de nouvelles centrales de production d'électricité, biomasse ou solaire, Albioma a déjà sécurisé 774 millions d'euros d'investissements fin 2017. Ces projets développés dans les départements d'outre-mer et au Brésil visent à porter la part de la production d'électricité à partir d'énergies renouvelables à 80% en 2023, contre 53% fin 2017. Les 47% sont essentiellement composés de charbon. Albioma n'exclut pas de se développer sur de nouveaux marchés, comme la Thailande ou la Colombie, où les ressources en résidus de canne à sucre (bagasse) brûlés par ses centrales sont abondants. En 2017, Albioma a réalisé un chiffre d'affaires de 403,2 millions d'euros, en hausse de 10%. Son Ebitda a atteint 138,3 millions d'euros (+5%) et le résultat net part du groupe a progressé de 13% à 37,4 millions d'euros. Le groupe propose de verser un dividende de 0,60 euro par action, soit un rendement de 3% par rapport au cours de Bourse de ce matin (19,76 euros, +1,96%). Pour 2018, Albioma vise un Ebitda compris entre 158 à 166 millions d'euros et un résultat net part du groupe de 37 à 42 millions d'euros.
</article>
<article titre="Albioma (ex-Sechilienne-Sidec) : les clignotants passent au vert" date="2018 02 15" journal="Boursier (site web)">
Turbulences.  Albioma n'a pas échappé à la consolidation des marchés sur la période récente. Au-delà des dégagements de prudence avant les comptes annuels du 7 mars prochain, le flux vendeur constaté sur le titre est à mettre en perspective avec les problèmes sociaux rencontrés par le groupe, sans impact financier jusqu'ici, et surtout par la remontée des taux... On rappellera à toutes fins utiles que le business model du producteur d'énergie renouvelable est par nature gourmand en capitaux. Des investissements conséquents sont nécessaires pour construire de nouvelles centrales à bagasses et solaires, voire pour rénover des installations. Cette caractéristique engendre une sensibilité au taux, ne serait-ce que par le jeu des refinancements, avec des effets que le marché cherche toujours à anticiper... Quoi qu'il en soit, l'endettement net d'Albioma s'élevait à 580 millions d'euros au 30 juin dernier, soit un gearing de 132% contre 125% six mois plus tôt. Cette situation n'a rien de préoccupante au regard de la montée en puissance attendue des nouveaux projets en construction et du niveau de l'Ebitda, lequel devrait se situer entre 130 et 138 ME en 2017 selon les derniers objectifs communiqués par le management. Les financements d'Albioma sont en outre à taux fixe ou sécurisés par des couvertures à hauteur de 90%, avec une maturité et un coût adaptés à l'activité. Pipeline.  Ces objections étant formulées, il convient d'analyser sereinement les perspectives moyen-long terme d'Albioma. Or celles-ci apparaissent attractives grâce au pipeline de projet. Fait positif, le dispositif industriel du groupe va s'élargir au premier trimestre 2018 de deux unités thermiques supplémentaires. La plus importante est Galion 2. Il s'agit de la première centrale 100% bagasse/biomasse dans les DOM. Si son coût final devrait dépasser les estimations initiales pour atteindre au moins 205 ME en incluant les derniers aléas du chantier, l'unité va apporter une capacité supplémentaire de 40 MW et une visibilité sur les revenus sécurisée par 30 ans de contrat. Mais Albioma ne s'endort pas sur ses lauriers. La firme dispose notamment de plusieurs projets en cours au Brésil qui sont autant de leviers de croissance pour les années à venir. Le budget d'investissements alloué à ce pays est de 400 ME sur une période de 10 ans (2013-2023), ce qui représente 40% de l'enveloppe totale du groupe. Face à cette densité de projets, la valorisation d'Albioma apparaît bien modeste. Le dossier se traite moins de 14 fois le bénéfice net estimé sur l'année en cours avec un PER qui retombe à 11,5 fois le profit escompté en 2019, sur la base d'hypothèses conservatrices. L'amélioration de la visibilité sur les perspectives, déjà soutenue, milite pourtant pour des multiples plus élevés... Avis positif.Aymeric Val
</article>
<article titre="ZNI : l'autonomie en 2030 suppose une envolée des capacités" date="2018 02 8" journal="GreenUnivers (site web)">
2018-02-09T18:17:39+00:00    Ils sont souvent un angle mort des débats sur la Programmation pluriannuelle de l'énergie (PPE). Pourtant, la loi leur fixe un objectif assez ambitieux : l'autonomie énergétique en 2030. Les territoires ultra-marins font l'objet d'une publication du Syndicat des énergies renouvelables (SER) à l'occasion de son colloque annuel organisé ce 8 février. Le syndicat publie des scénarios pour satisfaire en 2030 les besoins de l'Outre-mer français - ainsi que la Corse - avec un mix composé à 100% d'énergies renouvelables.   Une capacité photovoltaïque à décupler  L'exercice a été décliné territoire par territoire : Corse, Guadeloupe, Guyane, Martinique, Mayotte et La Réunion (voir les deux encadrés ci-dessous). Globalement, le message est clair : "le rythme de développement actuel des énergies renouvelables n'y est pas du tout à la hauteur des enjeux", a pointé Jean-Louis Bal, président du SER, en ouverture du colloque. Alors que le syndicat se montre plutôt confiant en France métropolitaine (à part pour la chaleur), le constat est très différent pour l'Outre-mer et la Corse. "Les capacités à installer pour atteindre l'autonomie énergétique doivent être multipliées, selon les scénarios, par 4 à 7 par rapport au parc de 2015, et jusqu'à 10 pour le parc photovoltaïque. Or, les différentes PPE prévoient jusqu'à 2023 un doublement du parc de 2015, sans garantie que les moyens pour y parvenir soient adaptés", analyse le SER. Dans ses scénarios, le syndicat reconnait que "plusieurs hypothèses peuvent faire l'objet de débats", notamment sur l'évolution de la consommation d'énergie. Le SER ne précise pas le montant des investissements à consentir, plus que conséquents étant donné les volumes. Mais il entend ouvrir la discussion sur ce sujet "structurant pour les prochaines décennies". Il a donc établi des scénarios pour chaque territoire sur la base des mix énergétiques de 2015, des objectifs fixés par les PPE actuelles pour 2023 et avec une cible 100% EnR en 2030. En Corse , le photovoltaïque passerait de 102 MW en 2015 à 146 puis 880 MW en 2023 et 2030. L'hydraulique, de 194 MW à 206 puis 286 MW. L'éolien, de 18 à 36 puis 100 MW. La biomasse et les biodéchets représentent un potentiel à développer pour atteindre 7 MW en 2023 et 187 MW en 2030. En Guadeloupe , le photovoltaïque est aussi la cible n°1 avec 1 GW en 2030, contre 67 MW en 2015 et 134 MW en 2023. L'éolien serait la deuxième source avec 296 MW en 2030, soit 3 fois plus qu'en 2023 et 10 fois plus qu'en 2015. Derrière, figureraient en 2030 la géothermie (74 MW), la biomasse et les cultures énergétiques (71 MW), le biogaz (36 MW), la bagasse (30 MW) et l'hydraulique (18 MW). En Guyane , le mix en 2030 serait de 221 MW de photovoltaïque (5,7 fois plus qu'en 2015), 164 MW d'hydraulique (1,4 fois plus) et 52 MW de biomasse (26 fois plus). Il faudrait aussi développer 30 MW d'éolien et 25 MW de valorisation des déchets. Réviser les PPE  Les chiffres parlent d'eux mêmes : il va falloir sérieusement accélérer le développement des EnR si ces territoires veulent atteindre l'autonomie énergétique. Pour cela, le SER propose de réviser les PPE pour 2023 et 2028 avec "de nouveaux appels d'offres par territoire, planifiés sur plusieurs années et plus importants en volume, pour les filières en fort développement et d'ores et déjà compétitives".   En complément, le syndicat demande "une stabilité des conditions pour les filières bénéficiant d'un tarif" et "une meilleure compatibilité entre les documents d'urbanisme, POS et PLU, les schémas de mise en valeur de la mer, les schémas d'aménagement régionaux et les zones de développement potentielles des énergies renouvelables".   Il appelle aussi à développer de façon "accélérée" les solutions pour les réseaux intelligents et la gestion de la variabilité afin d'augmenter le taux de pénétration des EnR "tout en maintenant un haut degré de sûreté du réseau". En Martinique , la capacité photovoltaïque serait multipliée d'ici 2030 par 17,4 pour atteindre 1,15 GW. Loin devant les autres filières : 203 MW d'éolien, 68 MW pour les déchets, 40 MW de géothermie, 29 MW à la fois pour le bioéthanol et les énergies marines (éolien offshore inclus), 20 MW pour l'hydraulique et 15 MW pour le biogaz. A Mayotte , il faudrait développer en 15 ans 420 MW de capacité photovoltaïque (9 MW installés en 2015), 50 MW d'éolien terrestre, 36 MW d'énergies marines, 32 MW de biomasse et 6 MW de méthanisation. A La Réunion , le mix en 2030 serait composé de 1033 MW de photovoltaïque (173 MW en 2015), 250 MW d'éolien (15 MW en 2015), 150 MW d'hydraulique (136 MW en 2015) et 84 MW de valorisation des déchets. A compléter par 58 MW à base de bagasse, déjà existants. Il faudrait développer une cinquantaine de MW à la fois pour la cogénération biomasse, les énergies marines et la géothermie. Le SER évoque aussi 32 MW de méthanisation, 22 MW de biogaz et 21 MW de gazéification (sans plus de précisions sur ces trois dernières composantes).  Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="Maison Richard renoue avec les bouteilles consignées" date="2018 01 22" journal="Emballages magazine.com">
Le fournisseur de la restauration lance la gamme de vins bio Le Titi avec le bouchon Helix.-   Maison Richard, le distributeur de vins, d'eaux minérales et d'alcools auprès des cafés, hôtels et restaurants (CHR), étoffe son offre de vins issus de l'agriculture biologique avec la gamme Le Titi. Mais, à cette occasion, il pousse davantage sa démarche écoresponsable. Tout d'abord sur le plan des produits. Le circuit court est privilégié : les vins- un côtes-de-provence, un fronton, un côtes-du-rhône villages Plan de Dieu et un blaye-côtes-de-bordeaux- arrivent du Var, de Haute-Garonne, du Vaucluse et de Gironde en citerne et sont mis en bouteille à Gennevilliers (Hauts-de-Seine), sur une ligne installée il y a six mois. Sur le plan de l'emballage ensuite. Le distributeur a choisi la solution de bouchage à vis Helix d'Amorim et O-I, qui utilise du liège issu de suberaies gérées durablement. Fournies par l'Imprimerie Étienne, les étiquettes, collées sans solvant, sont à base de bagasse et les encres sont végétales. Enfin, et surtout, les bouteilles sont consignées. Après utilisation, elles sont collectées par les chauffeurs livreurs de la société pour être lavées, et les étiquettes grattées, dans l'unité de Gennevilliers. « Jeter les bouteilles est un vrai souci pour les restaurateurs, indique Céline Richard, l'une des dirigeantes de la société. C'est pourquoi nous avons voulu remettre le verre consigné au goût du jour. Même si les premiers retours- 200 bouteilles récupérées sur les 3000 vendues- montrent qu'il nous faut réapprendre la consigne aux clients, quand bien même ils y sont habitués pour les eaux minérales. »
</article>
<article titre="Des bouteilles 100 % vegan avec vos déchets" date="2018 01 19" journal="Le Maine Libre  Angers ; Cholet ; Deux-Sevres ; Nord Anjou ; Saumur">
100 % végétales et intégralement fabriquées à partir de déchets : ces bouteilles « vegan » uniques en leur genre sont nées de l'imagination de Nicolas Mouflet, qui a installé son usine à Chaniers (Charente-Maritime).   Il aura fallu un an et demi de recherches pour   « lever le frein technologique »   qui empêchait la bagasse, résidu fibreux issu de la production de sucre à partir de la canne, de devenir la matière première de sa   « Veganbottle »   biodégradable. Mais Nicolas Mouflet, 41 ans, est finalement parvenu à transformer cette fibre en granulés qui composent la   « préforme »  , sorte d'éprouvette qui est ensuite chauffée et soufflée pour devenir une bouteille transparente. D'autres ont déjà développé le concept du bioplastique, à partir du maïs par exemple, reconnaît le patron de la société Lyspackaging. Seulement   « le maïs est un aliment, en plus, il consomme beaucoup d'eau. Moi je ne voulais utiliser que des déchets de production »  , précise l'industriel.   Teinte hétérogène   Problème marketing de taille :   « une bouteille transparente 100 % végétale ne se différencie pas d'une autre à base de pétrole. Pour qu'un bioplastique se distingue, il faut qu'il soit visible d'un seul coup d' oe il, pour le consommateur comme pour le producteur »  , relève le chef d'entreprise.   Pour se démarquer visuellement, Lyspackaging incorpore donc dans ses bouteilles une foule de résidus naturels issus d'autres productions que la canne à sucre : noyaux d'olive, coquilles de moules, mais aussi huîtres, bois, pépins de raisin, blé, cuir, etc. Réduits en poudre, ces additifs donnent aux flacons végétaux une teinte hétérogène.
</article>
<article titre="1 - CEREALS AND CEREAL PRODUCTS" date="2018 01 19" journal="Rapports - Agroalimentaire - ONUAA">
NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... FCL Item code FCL Title Group Scientific Name Common Name Definition 0015 Wheat 1 - CEREALS AND CEREAL PRODUCTS Triticum spp.: Triticum aestivum Triticum durum Triticum spelta  common durum spelt  Common and...    Pour voir le rapport :  http://www.fao.org/fileadmin/user_upload/commodity/excel/export_items_1516380491.xls
</article>
<article titre="BONNES IDÉES BOUTEILLE BIO" date="2018 01 17" journal="L'Express, no. 3472">
Jusqu'où ira la mode vegan? L'entreprise Lyspackaging a créé la Veganbottle, une bouteille compostable, 100 % végétale et 100%biodégradable à base de bagasse, un résidu fibreux de canne à sucre. 2 millions d'exemplaires ont déjà été fabriqués.
</article>
<article titre="Chaniers : il fabrique des bouteilles en plastique végétal" date="2017 12 30" journal="L'Hebdo de Charente-Maritime (site web)">
Nicolas Moufflet » Ces bouteilles 100% végétales sont également une alternative au verre sans ses inconvénients »   Nicolas Moufflet a mis au point un procédé unique au monde à base de déchets végétaux. Rencontre. Le patron de l'entreprise Lyspackaging ne serait-il pas l'ingénieux inventeur de la réduction radicale des plastiques dans l'environnement??   Toujours est-il que les 2 millions de bouteilles commercialisées et produites à Chaniers, sont conçues sans aucune goutte de pétrole. « Et malgré cela elles sont plus taxées que les emballages pétrochimiques » commente un brin rageur l'industriel chagnolais.   En cause, une norme établit par un consortium qui introduit entre autres la notion de durée?: fixée à 6 mois maximum, les bouteilles 100 % végétales prennent elles un mois de plus pour disparaître. Et des freins comme celui-là le quadragénaire doit en affronter plus d'un. « Je ne peux pas dire que mes bouteilles sont biodégradables et compostables. Mais je peux dire qu'elles sont fabriquées avec des matériaux biodégradables et compostables » ironise encore Nicolas Moufflet.   Alors justement ces bouteilles végétales, de quoi sont-elles issues?? Tout simplement de la bagasse, un déchet de la canne à sucre. La recette est en partie tenue secrète par l'industriel. « Je transforme la matière en granulé additionné d'un autre produit dont je ne vous dirai rien » s'amuse Nicolas Moufflet. Au final, ces granulés donneront une préforme?: un flacon en forme d'éprouvette munie d'un système de bouchage.   100 % végétal    Passée dans une machine de son atelier, la préforme devient en bout de chaîne une bouteille. D'aspect transparent, le flacon peut aussi être en couleur. » Avec des déchets de cacao, ou de noyaux d'olive ou encore de coquilles d'huîtres, elles sont colorées et pigmentées » détaille l'industriel. Pas satisfaisant selon lui, pour obtenir des débouchés par exemple sur le marché de la cosmétique. C'est pourquoi il développe une autre formule à partir de plantes. Là encore le plus grand secret entoure le processus de fabrication.   Et des idées comme celles-ci Nicolas Moufflet en a plein la tête. À tel point qu'il conseille les fabricants d'emballages de toute la planète. » L'alternative offerte par l'utilisation des déchets végétaux est intéressante. Elle peut offrir aux consommateurs par exemple des pots de chocolat faits à partir de déchets du cacao. Une bouteille de vin blanc à partir de coquilles d'huîtres » énumère le jeune chimiste vert.   « Faire quelque chose pour Saintes »    Pour autant, il réfute l'image d'écolo. « Non je ne suis pas écologiste. Mon savoir-faire apporte un truc en plus. Trouver des choses pour éviter de retrouver plus de 6 milliards de tonnes de plastique dans la nature comme c'est le cas aujourd'hui, ça me plaît » confesse Nicolas Moufflet.   La prochaine étape de l'industriel est tout aussi honorable. Il souhaite voir émerger à Saintes un pôle de fabricants de solutions bio sourcée, en lieu et place des ex-bâtiments de la caisse locale du Crédit Agricole délocalisée à Lagord. « Il faut faire quelque chose pour Saintes » affirme l'industriel. Et si l'avenir de la cité saintaise venait des start-up vertes??  Cet article est paru dans L'Hebdo de Charente-Maritime (site web)
</article>
<article titre="Le carton de la bouteille végétale" date="2017 12 26" journal="Sud Ouest  Saintes ~ Royan ~ Jonzac">
Végétal, compostable et biodégradable, recyclable à l'infini... la bouteille de Nicolas Moufflet, fondateur de Lyspackaging, jeune entreprise née le 12 janvier 2015 dans les murs de la pépinière d'entreprise de Saint-Georges-des-Coteaux, a, cette année, suscité un grand intérêt dans la presse nationale. " Il y a une attente ", interprète Nicolas Moufflet.   Le patron a mis au point un plastique composé uniquement de déchets de canne à sucre (la bagasse). C'est ce flacon, toutes formes confondues, qu'il commercialise actuellement et qui provoque un vrai engouement. La production est de deux millions d'unités. Installé depuis septembre à Chaniers, derrière la zone des Nattiers, Nicolas Moufflet espère raisonnablement atteindre les 8 millions en 2018. Dès lors, la petite entreprise aura totalement inversé la part du plastique PET et celle du plastique végétal, qui pèsera 80 % de la production.   Pour le moment, tous ses clients sont en France. Un développement à l'international devrait voir le jour en 2018 mais sous forme de franchise. Nicolas Moufflet a pour projet bien avancé d'installer une unité de production dans les DOM-TOM français, aux Antilles. Une manière de préserver la recette de ses granulés végétaux, tout en restant cohérent avec sa philosophie : il exportera les granulés qui seront transformés sur place. Il serait absurde de faire voyager des flacons végétaux, plus volumineux. Lyspackaging vise aussi l'Afrique du sud, la Californie et le Vietnam.   Le patron programme également d'investir 500 000 euros dans une machine à injection pour les flacons cosmétiques notamment. Une troisième souffleuse, dont il tient la technologie secrète, devrait également arriver dans les nouveaux entrepôts.   Ce fort développement devrait s'accompagner d'embauches. Lyspackaging emploie actuellement sept personnes, 10 quand elle recourt aux intérimaires.   S. J.
</article>
<article titre="Bulletin Amérique Latine : Eurofins, Albioma, Egis, Voltalia..." date="2017 12 22" journal="CFNEWS (site web)">
CFNEWS vous propose l'actualité des deals, des investisseurs et conseils en Amérique Latine et "cross-border" et les nominations - en accès libre !     Les deals Fusions-Acquisitions &amp; Private Equity   Produits Pharmaceutiques &amp; matériel médical : Eurofins - Pasteur (Brésil)    Le numéro 1 mondial des tests alimentaires et environnementaux Eurofins Scientific (voir sa fiche et deal-list dans notre référentiel) fort de plus de 3 Md€ de chiffre d'affaires en 2017, s'offre le groupe brésilien Pasteur Group, acteur majeur des diagnostics cliniques dans la région de Santos. Basé à São Paulo et avec un effectif de 190 personnes, Pasteur réalise un chiffre d'affaires annuel de 25 millions de reals (6,4 M€). Cette transaction, dont les termes financiers n'ont pas été divulgués, permet à Eurofins d'entrer sur le marché des diagnostics cliniques au Brésil. Voir la fiche de l'opération M&amp;A Eurofins - Pasteur.     Energie &amp; Utilities : Albioma (Brésil)    Le producteur d'énergie coté Albioma (voir sa fiche et deal-list) , générant 367,8 M€ de chiffre d'affaires l'an dernier, acquiert 60 % d'une centrale de cogénération au Brésil auprès du groupe Jalles Machado. Localisée à Goianésia dans l'État du Goiás cette centrale utilise la bagasse, résidu de la canne à sucre, pour produire de l'électricité et la chaleur. La capacité de broyage annuelle atteint 2,8 millions de tonnes de canne. L'accord prévoit notamment la rénovation des chaudières existantes ainsi que l'installation d'une nouvelle turbine de 25 mégawatts (MW), portant la capacité totale du site à 65 MW. Grâce à l'amélioration des rendements de la cogénération et à la réduction de la consommation de vapeur de la sucrerie, l'énergie exportée sur le réseau de distribution devrait ainsi atteindre 145 gigwattheures (GWh) par an à compter de la campagne 2019, soit près de deux fois plus qu'actuellement. Cet investissement, le quatrième au Brésil pour le groupe français, sera financé à 60 % par un financement à long-terme en monnaie locale.     Immobilier &amp; construction, bâtiment : Egis - Ocacsa (Mexique)    Le groupe d'ingénierie Egis (voir sa fiche et deal-list) , réalisant 1,02 Md€ de chiffre d'affaires en 2016, prend une participation majoritaire dans la société mexicaine Operación y Conservación de Autopistas Concesionadas (Ocacsa), spécialisée dans l'exploitation d'infrastructures routières équipées de péages. Fondé en 1990 par Emilio Gutiérrez, Ocacsa exploite notamment le contournement de villes moyennes au Mexique pour le compte de concessionnaires privés: Tecpan, Celaya, Ciudad Obregón et bientôt Hermosillo. "Cette prise de participation majoritaire conforte le développement d'Egis en Amérique Centrale et Latine, véritable région d'investissement pour le groupe", a commenté Nicolas Jachiet, Pd-g d'Egis. Ce dernier devient ainsi le premier exploitant autoroutier indépendant du Mexique avec plus de 360 collaborateurs et 200 kilomètres en exploitation. Voir la fiche de l'opération M&amp;A Egis - Ocacsa.     Energie &amp; Utilities : Voltalia (Brésil)    Voltalia (voir sa fiche et deal-list) , la PME de la famille Mulliez dans les énergies vertes, a remporté 155 MW de projets éoliens dans le cadre des enchères organisées les 18 et 20 décembre par le régulateur brésilien ANEEL. Les projets, situés dans le cluster de Sera Branca dans l'État du Rio Grande do Norte développé par Voltalia, sont Vila Paraíba II (32 MW) et Vila Paraíba III (32 MW) ainsi que Vila Acre II (27,3 MW), Vila Paraíba I (32 MW) et Vila Paraíba IV (32 MW). Les contrats remportés, d'une durée de 20 ans, assurent au groupe des revenus sécurisés cumulés de plus de 400 M€. "Je me réjouis de ce nouveau succès. Après deux ans sans appel d'offres éolien au Brésil, les nouveaux projets remportés cette semaine viennent enrichir notre portefeuille de projets à construire. Nous avons désormais une bonne visibilité sur une capacité totale de plus de 900 MW, dont 508 actuellement en exploitation", a déclaré Sébastien Clerc, directeur général de Voltalia.     Energie &amp; Utilities : Engie (Brésil)    Le géant de l'énergie Engie (voir sa fiche et deal-list) , fort de 66,6 Md€ de chiffre d'affaires en 2016, a remporté une concession pour la construction, l'assemblage, l'exploitation et la maintenance de lignes de transmission électrique dans l'État du Parana au Brésil. La durée de la concession sera de 30 ans, nécessitera un investissement de 500 M€ et permettra la création de plus de 4 000 emplois directs. Le lot, d'une longueur de 1 146 kilomètres, intègre huit nouvelles lignes de transmission, des tronçons supplémentaires et l'installation de cinq sous-stations électriques. Avec une capacité installée de plus de 11 GW, le groupe français est le premier producteur indépendant d'électricité au Brésil. 90 % de ses capacités installées dans le pays proviennent de sources d'énergie propre, renouvelable et à faible émission de gaz à effet de serre.     Une information à nous soumettre pour ce Bulletin Amérique Latine? Écrivez-nous à : dalila.weygang@cfnews.net   Retrouver les autres Chroniques de CFNEWS :     Tous les précédents bulletins Amérique Latine Tous les bulletins Asie Aussi sur Facebook , Twitter , Google+ et LinkedIn  Cet article est paru dans CFNEWS (site web)
</article>
<article titre="Des centrales solaires pas comme les autres" date="2017 12 21" journal="Sciences et Avenir">
À La Réunion, des panneaux photovoltaïques abritent des potagers ou des bassins de pisciculture. Une solution originale développée pour éviter que les énergies renouvelables ne prennent des terres à l'agriculture.   Casquette vissée sur la tête, tee-shirt vert et short gris, Georges et Doumy portent la tenue de travail réglementaire de la prison du Port sur l'île de La Réunion. Les mains enfoncées dans la terre noire, ils s'affairent sur les rangées de mâche qui s'étirent sous les serres conçues pour résister aux cyclones. Au-dessus d'eux, le toit en dents de scie est à moitié occulté par des panneaux solaires. Car cette serre produit aussi de l'électricité. Unique au monde, ce lieu ? nommé « Bardzour » (l'aube en créole) ? est à la fois ferme, prison et centrale solaire. Les gardiens en patrouille, les miradors et les murs de grillage hérissé de barbelés rappellent bien un univers carcéral. Les terrains appartiennent en effet à l'administration pénitentiaire mais une partie, située en bordure de l'enceinte, a été convertie en centrale solaire. Elle abrite une installation de dernière génération couplant 9 mégawatts (MW) de panneaux solaires à 9MW de stockage sur batteries lithium-ion. Au total, cela représente 45 000 m 2 de panneaux photovoltaïques dont 6000 m2 ont été installés sur les toits des serres anticycloniques. Depuis la création de la centrale fin 2014, ces serres ont permis de former une vingtaine de détenus au métier d'ouvrier agricole. « Ce sont des volontaires ayant obtenu une autorisation du juge d'application des peines », précise Lionel Grand, le directeur adjoint du centre de détention de la ville du Port. Bardzour est l'illustration du concept d'Agriénergie développé par la société française Akuo Energy. « Notre idée est de faire que la production d'énergie solaire, qui occupe des surfaces importantes au sol, ne prenne pas des terres à l'agriculture, explique Steve Arcelin, l'un des dirigeants de l'entreprise. Bardzour a donc été implantée sur un no man's land qui entourait la prison. En plus des panneaux classiques, nous avons construit les serres photovoltaïques où sont formés et employés les détenus pour faire du maraîchage, de l'horticulture et une pépinière. » En 2015, ce modèle a été reproduit pour la centrale des Cèdres sur la commune d'Étang-Salé dans l'est de l'île, avec là encore une puissance de 9MW de panneaux installés et 9MW de batteries lithium-ion. L'idée est la même mais la configuration différente, la centrale étant divisée en deuxparties. La première regroupe 1,5 MW de panneaux solaires installés sur des toitures, de grandes ombrières qui recouvrent les 12 bassins d'un pisciculteur élevant des tilapias. « L'ombre des panneaux au-desan.   sus des bassins permet de réduire les amplitudes thermiques de l'eau qui variaient de 18 à 35 °C entre l'hiver et l'été contre 26 à 28 °C désormais, se réjouit Max Dyckerhoff, le dirigeant d'Aqua Austral la société aquacole sur laquelle les panneaux sont installés. Ces températures, optimales pour le tilapia, nous ont permis d'augmenter le rendement de 20 %. En plus, les ombrières nous protègent des prédateurs, notamment des butors, sortes de petits hérons qui prélevaient 30 à 40 000 poissons par » Le pisciculteur a aussi profité de ces installations pour lancer un élevage plutôt atypique dans la région: l'esturgeon. Son idée: pro duire d'ici à deux ou trois ans du caviar made in La Réunion. « Grâce aux ombrières et à un groupe froid, nous pouvons amener l'eau à 22 °C, une température idéale pour cette espèce », précise Max Dyckerhoff. À un kilomètre de ces bassins, raccordée par un câble enterré, se trouve la deuxième partie de la centrale. Elle héberge 7,5 MW ¦ de panneaux et les 9 MW de batteries installées dans des containers climatisés. Cette fois, sous les rangées de panneaux, espacées de trois mètres, c'est une production agricole presque classique qui a été lancée : du maraîchage et un élevage de poules, mais aussi des moutons qui entretiennent l'espace. « Ce terrain a longtemps été exploité pour la culture de l'ananas, très exigeante pour les sols, puisant beaucoup de nutriments, explique Julien Mathiot, directeur des opérations d'Akuo à La Réunion. Nous voulons désormais faire de la permaculture, en essayant de créer sur le site un environnement varié comme un écosystème où chaque culture apporte aux autres. » L'enjeu est donc de trouver les variétés adaptées à cette agriculture sous panneaux, dont une partie de la surface se trouve effectivement à l'ombre. « Ce pourrait être le cas d'espèces comme la vanille qui n'aiment pas trop le soleil », poursuit l'expert.   Si l'idée de combiner production électrique et production agricole semble judicieuse, les obstacles restent nombreux, notamment en matière de compétitivité économique. Difficile de juger selon Jean-Michel Bordage, directeur régional de l'Ademe (Agence de l'environnement et de la maîtrise de l'énergie) La Réunion-Mayotte, qui affirme ne pas disposer des données nécessaires. « L'intérêt du modèle est de tenir compte de la pression qui pèse sur le foncier et la production agricole dans une île comme La Réunion. Mais Akuo ne communique pas assez sur son bilan économique. Par ailleurs, la tendance est désormais de privilégier des toitures déjà existantes plutôt que de créer des installations au sol. » Relier production électrique et production agricole Le contexte économique n'est effectivement plus très favorable à ce modèle de centrale qui nécessite de créer des infrastructures adéquates : « Ainsi, lors de la création de Bardzour, le prix de rachat de l'électricité était de 40 centimes du kilowattheure (kWh), ce qui a permis de financer la centrale. Aujourd'hui dans les derniers appels d'offres, le prix du kWh est de 12 centimes ce qui, financièrement, ne permet plus de trouver un terrain et d'y installer des serres anticycloniques très coûteuses », regrette Julien Mathiot. Pour l'ingénieur, les appels d'offres ne devraient passe focaliser que sur la production électrique mais inclure le financement d'une production agricole : « Ce serait un moyen de sécuriser l'agriculture. Les derniers cyclones qui ont frappé l'île ont dévasté toutes les cultures. Nos serres, elles, sont restées intactes. » ¦ De notre envoyé spécial, textes et photos, OlivierHertel OlivierHertel
</article>
<article titre="Portzamparc passe à l'achat sur Albioma" date="2017 12 20" journal="Boursier (site web)">
Dans le peloton de tête du SRD,  Albioma  grimpe ce mercredi matin de 3,3% à 20,32 euros. Le producteur d'énergie a annoncé hier après bourse avoir réalisé un quatrième investissement au Brésil, sous la forme de l'acquisition de 60% de l'unité de cogénération bagasse adossée à la sucrerie-distillerie Jalles Machado, localisée à Goianésia (Etat du Goiás), dont la capacité de broyage annuelle atteint 2,8 millions de tonnes de canne. L'accord prévoit notamment la rénovation des chaudières existantes ainsi que l'installation d'une nouvelle turbine de 25 MW, portant la capacité totale du site à 65 MW.   Mais le flux acheteur sur le titre est également à relier avec un changement d'opinion de broker. Dans une note publiée ce jour, Portzamparc est passé de "conserver" à "acheter" sur Albioma, dont l'objectif est laissé à 22,90 euros. Le titre fait par ailleurs l'entrée ddans la liste de convictions de l'intermédiaire financier. Au-delà du nouveau développement réalisé au Brésil, Portzamparc justifie sa décision par le reflux récent du titre et par les nombreux catalyseurs identifiés, à savoir :des résultats 2017 de bonne qualité, la mise en service de Galio et celle de la TAC...
</article>
<article titre="Un nouvel investissement au Brésil apprécié par les investisseurs" date="2017 12 20" journal="Trading Sat (site web)">
Albioma a signé un accord portant sur l'acquisition de 60 % de l'unité de cogénération bagasse adossée à la sucrerie-distillerie éponyme, localisée à Goianésia au Brésil. L'accord prévoit notamment la rénovation des chaudières existantes ainsi que l'installation d'une nouvelle turbine de 25 MW, portant la capacité totale du site à 65 MW.   Grâce à l'amélioration des rendements de la cogénération et à la réduction de la consommation de vapeur de la sucrerie, l'énergie exportée sur le réseau de distribution devrait ainsi atteindre 145 GWh par an à compter de la campagne 2019, soit près de deux fois plus qu'actuellement. Plus de 80 % des ventes d'énergie sont d'ores et déjà sécurisés sur le marché réglementé : la cogénération est en effet déjà titulaire de contrats à long terme et vient de vendre 75 GWh par an à compter de 2021 au prix de 258 BRL/MWh indexé sur l'inflation à la faveur du dernier appel d'offres du 18 décembre.   Pour Oddo, "cette nouvelle opération s'inscrit totalement dans la stratégie du groupe qui vise un investissement global de 400 millions d'euros au Brésil, soit 40% de l'enveloppe globale sur la période 2013-2023". Le courtier explique également que "les prix contractualisés sont globalement dans la moyenne des contrats précédents (environ 75% de l'énergie contractualisée à 241 BRL par MWh pour la période 2017-2019". L''analyste maintient tout de même sa recommandation "Neutre" ainsi que son objectif de cours de 19,5 euros sur la valeur, estimant que "la valorisation actuelle est en ligne avec les références historiques".   En Bourse, le titre Albioma prend 2,2% à un peu plus de 20 euros.
</article>
<article titre="ALBIOMA soutenu par une acquisition au Brésil" date="2017 12 20" journal="Agence Option Finance (AOF)">
(AOF) - Albioma progresse de 2,29% à 20,13 euros après l'annonce d'une acquisition au Brésil. Le spécialiste des énergies renouvelables a acheté 60% d'une unité de cogénération bagasse adossée à la sucrerie-distillerie de Goianésia, le solde étant conservé par le bréslien Jalles Machado. L'accord prévoit la rénovation des chaudières existantes ainsi que l'installation d'une nouvelle turbine de 25 MW, portant la capacité totale du site à 65 MW.    La production exportée devrait atteindre 145 GWh par an à compter de la campagne 2019, soit le double de 2017, grâce à des gains de productivité et à la hausse des volumes. Plus de 80 % des ventes d'énergie sont sécurisées à des tarifs attractifs. Lors de la dernière enchère du 18 décembre, l'unité a remporté 75 GWh annuel à compter de 2021 au prix de 258 reals brésliens/MWh indexé sur l'inflation.    Albioma va financer cet investissement à 60% par de la dette locale.    Le groupe n'a pas donné d'indications chiffrées sur cette opération. Selon Portzamparc, le prix d'acquisition se se situerait entre 30 et 35 millions d'euros. Albiomia devrait ensuite investir 25 millions d'euros pour installer la nouvelle turbine. Le site générerait 6,5 millions d'euros d'Ebitda à partir de 2019.    Par ailleurs, le broker a fait le point avec le management sur deux sujets d'actualité. En premier lieu, le contexte social semble un peu moins tendu même si le risque de mouvement social demeure.   Par ailleurs, concernant les annonces de possible fin des centrales au charbon en 2022, le Dom pourraient être exemptés de par leurs spécificités, Dans le cas contraire, Albioma engagera des discussions pour transformer ces unités à la combustion bois, sur la base de négociations proches de celles concernant les investissements directs à l'étranger (IED).   Au final, le bureau d'études rappelle que ce nouveau projet était attendu et vient plus de 18 mois après le dernier. Il présente des caractéristiques proches des investissements précédents (taille, taux de sécurisation des tarifs, montée en productivité...).    A la suite de cette annonce, et eu égard à légère consolidation du titre et à la détente sociale, l'intermédiaire a relevé sa recommandation sur le titre de Conserver à Acheter et l'a rentré de nouveau dans sa liste Convictions. Il a également revu à la hausse son objectif de cours de 21,6 à 22,9 euros.   Bryan Garnier a également salué cet investissement qui devrait apporter selon lui 5 à 6 millions d'Ebitda au groupe à partir de 2019. En conséquence, le broker a relevé son objectif de cours sur le titre de 21 à 22 euros tout en réitérant sa recommandation d'Achat.   AOF - EN SAVOIR PLUS  Services aux collectivités    Sous l'effet des fusions et acquisitions, le secteur mondial des utilities est en pleine reconfiguration. Selon une étude d'AT Kearney, 1040 opérations de ce type ont été annoncées dans le monde en 2016, pour un total de 329 milliards de dollars, montant record en valeur. AT Kearney estime que le marché bénéficie d'abord des cessions d'actifs opérées par les grands énergéticiens européens.   Comme le souligne le cabinet de conseil et d'audit PwC, l'année 2016 a été un exercice record en matière de fusions-acquisitions dans l'énergie : près de 300 milliards de dollars, en hausse de 47% par rapport à 2015. Et effectivement l'Europe a joué un rôle essentiel dans cette accélération car les transactions sur ce continent ont bondi de 25%, à 49 milliards de dollars. En France, 45 opérations de fusions-acquisitions ont été recensées en 2016, pour une valeur totale de 5,4 milliards de dollars. La prise de participation minoritaire de CNP Assurances et de la Caisse des dépôts (CDC) dans Réseau de transport d'électricité (RTE), la filiale d'EDF, pour 4,6 milliards de dollars, a surpassé toutes les autres.
</article>
<article titre="Les valeurs à suivre aujourd'hui à la Bourse de Paris - Mercredi 20 décembre 2017" date="2017 12 20" journal="Agence Option Finance (AOF)">
(AOF) - ALBIOMA   Albioma a signé le 18 décembre dernier, avec le groupe Jalles Machado, un accord portant sur l'acquisition de 60 % de l'unité de cogénération bagasse adossée à la sucrerie-distillerie éponyme, localisée à Goianésia (État du Goiás), dont la capacité de broyage annuelle atteint 2,8 millions de tonnes de canne.   ASSYSTEM  L'AMF et Euronext ont publié un avis par lequel ils ont annoncé l'apport de 10.507.041 actions à l'offre publique de rachat par la société Assystem de ses propres actions, laquelle était ouverte du 24 novembre 2017 au 14 décembre 2017 (inclus) et proposait aux actionnaires de la société de leur racheter leurs actions Assystem dans la limite de 6 000 000 d'actions au prix unitaire de 37,50 euros.   CREDIT AGRICOLE   Crédit Agricole S.A est contrainte par la Banque centrale européenne (BCE) de supprimer de ses statuts la clause de majoration du dividende d'ici septembre 2018. Ce procédé avait été mis en place en 2011 afin de récompenser la fidélité de ses actionnaires mais les autorités bancaires européennes ont considéré que le dividende majoré constitue une "distribution préférentielle" non conforme au règlement Capital Requirements Regulation (CRR).   DBT   DBT, spécialiste des bornes de recharge rapide pour véhicules électriques, a fait savoir que la société Holding HFZ, contrôlé par le dirigeant de la société Hervé Borgoltz, a franchi le seuil de 50% à la baisse des droits de vote de DBT et en détient désormais 49,2%. A la date du présent communiqué, la société Holding HFZ détient 1 826 100 actions soit 32,67% du capital social de DBT composé de 5 589 913 actions. Les actions disposant du droit de vote double représentent un total de droits de vote de 3 652 200 sur 7 416 013 droits de vote double au total.   ELIS   Philippe Audouin, membre du conseil de surveillance d'Elis depuis le 4 octobre 2007 en tant que représentant d'Eurazeo, a informé Elis qu'il démissionnait de ses fonctions au sein du conseil de surveillance avec effet au 14 décembre 2017. Cette démission fait suite au communiqué d'Eurazeo du 5 octobre 2017 relatif à la cession d'une partie de ses actions Elis, ramenant ainsi la participation d'Eurazeo en dessous de 15%.   FIGEAC AERO   Figeac Aéro a enregistré une croissance de 20,8% de son bénéfice net part du groupe au premier semestre (clos fin septembre), à 15,72 millions d'euros et une progression de 8,5% de son Ebitda corrigé à 35,79 millions, selon des données provisoires. Ce dernier a représenté 21% (-1,6 point) d'un chiffre d'affaires de 170,43 millions d'euros (+16,5%) au premier semestre.   GECINA   Six foncières européennes (Alstria, Coima Res, Colonial, Gecina, Great Portland Estates et NSI) ont annoncé la création d'un think tank européen rassemblant leurs réflexions et bonnes pratiques sur l'innovation et la RSE. "Ce forum d'échanges et d'initiatives est complémentaire des initiatives prises par chacune des foncières. Il peut être le cadre de projets de recherche ou de toute autre initiative visant à accroître la capacité d'innovation des foncières et à mettre en oeuvre les meilleurs pratiques en RSE", expliquent-elles dans un communiqué.   GENOMIC VISION   Genomic Vision, société spécialisée dans le développement de tests de diagnostic pour la détection précoce des cancers et des maladies héréditaires et d'applications pour les laboratoires de recherche, annonce avoir renégocié, à la faveur de ses perspectives de développement, les modalités de son financement OCABSA, mis en place le 29 mars 2017 avec Bracknor Fund.   ILIAD   Iliad annonce à l'instant l'acquisition, pour environ 320 millions d'euros, d'une participation minoritaire de 31,6% dans eir, l'opérateur historique irlandais, aux côtés de NJJ (holding privée de Xavier Niel). Iliad et NJJ investissent dans les mêmes conditions. Les actionnaires existants, Anchorage Capital Group et Davidson Kempner Capital Management, restent investis dans eir au travers d'une participation de 35,5% et avec un crédit sans recours.   LACROIX   Au cours de son exercice 2016/2017, clos le 30 septembre 2017, Lacroix a publié un résultat net de 8 millions d'euros contre une perte nette de 8,8 millions d'euros l'année dernière. Le résultat opérationnel a augmenté de 53,8% à 12,4 millions d'euros. Le chiffre d'affaires a crû de 3,0% à 441,4 millions d'euros.   LA FRANÇAISE DE L'ENERGIE   La Française de l'Energie, leader de l'écologie industrielle, franchit une étape stratégique dans le développement de son activité en Lorraine puisque les premiers résultats du test de production actuellement en cours sur le site de Lachambre, confirment le potentiel d'une production de gaz profitable.   LATECOERE   Latécoère a contracté un financement de 55 millions d'euros auprès de la Banque Européenne d'Investissement (BEI). Ce financement viendra soutenir les investissements programmés par l'équipementier aéronautique dans le cadre du plan de Transformation 2020 qui vise à conquérir de nouveaux marchés grâce à une plus grande place donnée à l'innovation et à améliorer la compétitivité industrielle avec des process modernisés et automatisés, dans un objectif d'excellence.   NEOLIFE   Neolife a ajusté à la hausse son objectif de croissance qui devrait s'établir en hausse à + 55% (vs + 50% annoncés précédemment). Ce réajustement fait suite à une accélération des ventes au mois de décembre, supérieures aux prévisions, avec notamment une forte activité sur la zone Europe du nord, a indiqué le fabricant d'éco-matériaux.   OL GROUP   Le dimanche 17 janvier 2017, le président Jean-Michel Aulas a officialisé, la prolongation jusqu'en 2020 du partenariat avec le constructeur automobile coréen Hyundai Motor. La signature officielle s'est tenue en salle du conseil du Groupama Stadium. Le Président Jean-Michel Aulas était accompagné de son homologue coréen de Hyundai Motor, Kyoobok Lee ainsi que du Directeur Général France, Lionel French Keogh.   REXEL   Rexel, expert mondial de la distribution professionnelle multicanale de produits et services pour le monde de l'énergie, annonce aujourd'hui la cession de ses activités en Asie du Sud-Est à American Industrial Acquisition Corporation Group (AIAC), une société de capital investissement spécialisée dans le secteur industriel. Cette transaction, qui représente environ 17% de l'ensemble du plan de cession présenté en février 2017, devrait entraîner une perte avant impôt estimée à environ 60 millions d'euros.   SPINEGUARD   SpineGuard a franchi la barre des 60 000 chirurgies réalisées avec sa famille d'instruments de perçage PediGuard et des vis équipées de la technologie DSG pour le placement sécurisé des vis pédiculaires. La gamme de produits PediGuard comprend le PediGuard Straight, le PediGuard Curved, le PediGuard Cannulated et le PediGuard Threaded qui répondent à des besoins cliniques spécifiques et aux préférences opératoires des chirurgiens.   THERADIAG   Theradiag a annoncé la cessation de son activité de recherche, Prestizia, filiale à 100% de Theradiag. Prestizia travaille sur des projets de recherche s'apparentant à de la recherche fondamentale dont l'objectif est de développer de nouveaux marqueurs de diagnostic grâce à la plateforme microARN, dans le domaine du cancer (rectum et ovarien) et du VIH/SIDA.   VOLTALIA   Voltalia, acteur international des énergies renouvelables, annonce aujourd'hui la mise en service de la centrale solaire de Castellet II (3,8 MWc), dans le département du Var, en France métropolitaine. Sélectionnée dans le cadre de l'appel d'offres solaire national CRE III au mois de décembre 2015, la centrale bénéficie d'un contrat de vente d'électricité de long-terme de 20 ans signé avec EDF.
</article>
<article titre="Energie: Albioma acquiert 60% d'une centrale de cogénération au Brésil" date="2017 12 19" journal="AFP Infos Economiques">
Paris, 19 déc 2017 (AFP) - - Le producteur d'électricité français Albioma a annoncé mardi l'acquisition au groupe sucrier Jalles Marchado de 60% du capital d'une centrale de cogénération dans l'Etat de Goias (centre du Brésil).   Cette centrale utilise la bagasse, résidu du broyage de la canne à sucre, pour produire de l'électricité et de la chaleur. L'accord prévoit la rénovation des chaudières existantes et l'installation d'une nouvelle turbine de 25 mégawatts (MW) qui portera la capacité de la centrale à 65 MW, a détaillé Albioma dans un communiqué.   A terme, elle pourra produire deux fois plus d'énergie qu'actuellement et bénéficie de contrats de vente de long terme pour 80% de sa production.   Albioma n'a dévoilé aucun élément financier sur l'opération, soumise à des conditions qui devraient être levées "mi-2018 au plus tôt", et indique simplement qu'elle sera financée "à 60% par un financement à long-terme en monnaie locale".   Il y a deux ans, Albioma avait déjà pris une participation de plus de 60% dans une autre centrale de cogénération de Jalles Marchado dans la même région.   Cette opération "est une véritable reconnaissance du savoir-faire de nos équipes au Brésil", s'est félicité Frédéric Moyne, directeur général d'Albioma, cité dans le communiqué.   Actuellement Albioma, surtout présent dans les territoires outre-mer français, exploite deux centrales biomasse au Brésil.   La société a réalisé un chiffre d'affaires de 367,8 millions d'euros l'an dernier pour un bénéfice net de 33 millions d'euros.   mhc/sw/la/eb
</article>
<article titre="Albioma: nouvel investissement au Brésil" date="2017 12 19" journal="Votre argent (site web)">
Albioma a rapporté ce mardi soir avoir signé hier avec le groupe Jalles Machado un accord portant sur l'acquisition de 60% de l'unité de cogénération bagasse adossée à la sucrerie-distillerie éponyme, localisée à Goianésia (État du Goiás), dont la capacité de broyage annuelle atteint 2,8 millions de tonnes de canne. L'accord prévoit notamment la rénovation des chaudières existantes ainsi que l'installation d'une nouvelle turbine de 25 mégawatts (MW), portant la capacité totale du site à 65 MW.   Grâce à l'amélioration des rendements de la cogénération et à la réduction de la consommation de vapeur de la sucrerie, l'énergie exportée sur le réseau de distribution devrait ainsi atteindre 145 gigwattheures (GWh) par an à compter de la campagne 2019, soit près de deux fois plus qu'actuellement.   La réalisation de la transaction demeure soumise à un certain nombre de conditions suspensives qui devraient être levées mi-2018 au plus tôt.   Copyright (c) 2017 CercleFinance.com. Tous droits réservés.
</article>
<article titre="Albioma : signature d'un 4e investissement au Brésil" date="2017 12 19" journal="Boursier (site web)">
Albioma  a signé le 18 décembre, avec le groupe Jalles Machado, un accord portant sur l'acquisition de 60% de l'unité de cogénération bagasse adossée à la sucrerie-distillerie éponyme, localisée à Goianésia (Etat du Goiás), dont la capacité de broyage annuelle atteint 2,8 millions de tonnes de canne.   L'accord prévoit notamment la rénovation des chaudières existantes ainsi que l'installation d'une nouvelle turbine de 25 MW, portant la capacité totale du site à 65 MW. Grâce à l'amélioration des rendements de la cogénération et à la réduction de la consommation de vapeur de la sucrerie, l'énergie exportée sur le réseau de distribution devrait ainsi atteindre 145 GWh par an à compter de la campagne 2019, soit près de deux fois plus qu'actuellement.   Plus de 80% des ventes d'énergie sont d'ores et déjà sécurisés sur le marché règlementé : la cogénération est en effet déjà titulaire de contrats à long terme et vient de vendre 75 GWh par an à compter de 2021 au prix de 258 BRL/MWh indexé sur l'inflation à la faveur du dernier appel d'offres du 18 décembre (" leilão de energia nova A-4 ").   L'investissement sera financé à environ 60% par un financement à long-terme en monnaie locale. La réalisation de la transaction demeure soumise à un certain nombre de conditions suspensives qui devraient être levées mi-2018 au plus tôt.   Frédéric Moyne, Directeur Général d'Albioma, déclare : "Nous sommes en ligne avec nos objectifs de croissance au Brésil, où le Groupe constitue progressivement un portefeuille équilibré d'actifs de qualité grâce à la pertinence de son positionnement"
</article>
<article titre="ALBIOMA : signature d'un quatrième investissement au Brésil" date="2017 12 19" journal="Option Finance (site web)">
AOF - 19 décembre 2017   (AOF) - Albioma a signé le 18 décembre dernier, avec le groupe Jalles Machado, un accord portant sur l'acquisition de 60 % de l'unité de cogénération bagasse adossée à la sucrerie-distillerie éponyme, localisée à Goianésia (État du Goiás), dont la capacité de broyage annuelle atteint 2,8 millions de tonnes de canne.   L'accord prévoit notamment la rénovation des chaudières existantes ainsi que l'installation d'une nouvelle turbine de 25 MW, portant la capacité totale du site à 65 MW. Grâce à l'amélioration des rendements de la cogénération et à la réduction de la consommation de vapeur de la sucrerie, l'énergie exportée sur le réseau de distribution devrait ainsi atteindre 145 GWh par an à compter de la campagne 2019, soit près de deux fois plus qu'actuellement.   Plus de 80 % des ventes d'énergie sont d'ores et déjà sécurisés sur le marché règlementé : la cogénération est en effet déjà titulaire de contrats à long terme et vient de vendre 75 GWh par an à compter de 2021 au prix de 258 BRL/MWh indexé sur l'inflation à la faveur du dernier appel d'offres du 18 décembre (" leilão de energia nova A-4 ").   L'investissement sera financé à environ 60 % par un financement à long-terme en monnaie locale. La réalisation de la transaction demeure soumise à un certain nombre de conditions suspensives qui devraient être levées mi-2018 au plus tôt.  Cet article est paru dans Option Finance (site web)
</article>
<article titre="Les valeurs à suivre mercredi 20 décembre à la Bourse de Paris" date="2017 12 19" journal="Option Finance (site web)">
AOF - 19 décembre 2017   (AOF) - ALBIOMA   Albioma a signé le 18 décembre dernier, avec le groupe Jalles Machado, un accord portant sur l'acquisition de 60 % de l'unité de cogénération bagasse adossée à la sucrerie-distillerie éponyme, localisée à Goianésia (État du Goiás), dont la capacité de broyage annuelle atteint 2,8 millions de tonnes de canne. ASSYSTEM    L'AMF et Euronext ont publié un avis par lequel ils ont annoncé l'apport de 10.507.041 actions à l'offre publique de rachat par la société Assystem de ses propres actions, laquelle était ouverte du 24 novembre 2017 au 14 décembre 2017 (inclus) et proposait aux actionnaires de la société de leur racheter leurs actions Assystem dans la limite de 6 000 000 d'actions au prix unitaire de 37,50 euros.   ELIS   Philippe Audouin, membre du conseil de surveillance d'Elis depuis le 4 octobre 2007 en tant que représentant d'Eurazeo, a informé Elis qu'il démissionnait de ses fonctions au sein du conseil de surveillance avec effet au 14 décembre 2017. Cette démission fait suite au communiqué d'Eurazeo du 5 octobre 2017 relatif à la cession d'une partie de ses actions Elis, ramenant ainsi la participation d'Eurazeo en dessous de 15%.   LACROIX   Au cours de son exercice 2016/2017, clos le 30 septembre 2017, Lacroix a publié un résultat net de 8 millions d'euros contre une perte nette de 8,8 millions d'euros l'année dernière. Le résultat opérationnel a augmenté de 53,8% à 12,4 millions d'euros. Le chiffre d'affaires a crû de 3,0% à 441,4 millions d'euros.   LA FRANÇAISE DE L'ENERGIE   La Française de l'Energie, leader de l'écologie industrielle, franchit une étape stratégique dans le développement de son activité en Lorraine puisque les premiers résultats du test de production actuellement en cours sur le site de Lachambre, confirment le potentiel d'une production de gaz profitable.   NEOLIFE   Neolife a ajusté à la hausse son objectif de croissance qui devrait s'établir en hausse à + 55% (vs + 50% annoncés précédemment). Ce réajustement fait suite à une accélération des ventes au mois de décembre, supérieures aux prévisions, avec notamment une forte activité sur la zone Europe du nord, a indiqué le fabricant d'éco-matériaux.   OL GROUP   Le dimanche 17 janvier 2017, le président Jean-Michel Aulas a officialisé, la prolongation jusqu'en 2020 du partenariat avec le constructeur automobile coréen Hyundai Motor. La signature officielle s'est tenue en salle du conseil du Groupama Stadium. Le Président Jean-Michel Aulas était accompagné de son homologue coréen de Hyundai Motor, Kyoobok Lee ainsi que du Directeur Général France, Lionel French Keogh.   SPINEGUARD   SpineGuard a franchi la barre des 60 000 chirurgies réalisées avec sa famille d'instruments de perçage PediGuard et des vis équipées de la technologie DSG pour le placement sécurisé des vis pédiculaires. La gamme de produits PediGuard comprend le PediGuard Straight, le PediGuard Curved, le PediGuard Cannulated et le PediGuard Threaded qui répondent à des besoins cliniques spécifiques et aux préférences opératoires des chirurgiens.   THERADIAG   Theradiag a annoncé la cessation de son activité de recherche, Prestizia, filiale à 100% de Theradiag. Prestizia travaille sur des projets de recherche s'apparentant à de la recherche fondamentale dont l'objectif est de développer de nouveaux marqueurs de diagnostic grâce à la plateforme microARN, dans le domaine du cancer (rectum et ovarien) et du VIH/SIDA.  Cet article est paru dans Option Finance (site web)
</article>
<article titre="ALBIOMA : signature d'un quatrième investissement au Brésil" date="2017 12 19" journal="Agence Option Finance (AOF)">
(AOF) - Albioma a signé le 18 décembre dernier, avec le groupe Jalles Machado, un accord portant sur l'acquisition de 60 % de l'unité de cogénération bagasse adossée à la sucrerie-distillerie éponyme, localisée à Goianésia (État du Goiás), dont la capacité de broyage annuelle atteint 2,8 millions de tonnes de canne.   L'accord prévoit notamment la rénovation des chaudières existantes ainsi que l'installation d'une nouvelle turbine de 25 MW, portant la capacité totale du site à 65 MW. Grâce à l'amélioration des rendements de la cogénération et à la réduction de la consommation de vapeur de la sucrerie, l'énergie exportée sur le réseau de distribution devrait ainsi atteindre 145 GWh par an à compter de la campagne 2019, soit près de deux fois plus qu'actuellement.   Plus de 80 % des ventes d'énergie sont d'ores et déjà sécurisés sur le marché règlementé : la cogénération est en effet déjà titulaire de contrats à long terme et vient de vendre 75 GWh par an à compter de 2021 au prix de 258 BRL/MWh indexé sur l'inflation à la faveur du dernier appel d'offres du 18 décembre (" leilão de energia nova A-4 ").   L'investissement sera financé à environ 60 % par un financement à long-terme en monnaie locale. La réalisation de la transaction demeure soumise à un certain nombre de conditions suspensives qui devraient être levées mi-2018 au plus tôt.   AOF - EN SAVOIR PLUS  Services aux collectivités    Sous l'effet des fusions et acquisitions, le secteur mondial des utilities est en pleine reconfiguration. Selon une étude d'AT Kearney, 1040 opérations de ce type ont été annoncées dans le monde en 2016, pour un total de 329 milliards de dollars, montant record en valeur. AT Kearney estime que le marché bénéficie d'abord des cessions d'actifs opérées par les grands énergéticiens européens.   Comme le souligne le cabinet de conseil et d'audit PwC, l'année 2016 a été un exercice record en matière de fusions-acquisitions dans l'énergie : près de 300 milliards de dollars, en hausse de 47% par rapport à 2015. Et effectivement l'Europe a joué un rôle essentiel dans cette accélération car les transactions sur ce continent ont bondi de 25%, à 49 milliards de dollars. En France, 45 opérations de fusions-acquisitions ont été recensées en 2016, pour une valeur totale de 5,4 milliards de dollars. La prise de participation minoritaire de CNP Assurances et de la Caisse des dépôts (CDC) dans Réseau de transport d'électricité (RTE), la filiale d'EDF, pour 4,6 milliards de dollars, a surpassé toutes les autres.
</article>
<article titre="Vite vu" date="2017 12 15" journal="GreenUnivers (site web)">
2017-12-20T14:08:54+00:00    Mercredi 20 décembre Lors de son audition le 14 décembre par l'Agence française pour la biodiversité, le consortium Engie-EDPR-CDC lauréat du projet éolien offshore de Dieppe-Le-Tréport a proposé entre autres de  rehausser les éoliennes  , pour limiter les éventuels chocs avec les oiseaux,  suspendre la pose des pieux de fondations pendant quatre mois  , limiter  le largage de métaux lourds  dû à la corrosion de ces pieux et faire passer le budget du groupe d'intérêt scientifique chargé du contrôle de ces mesures de 0,65 M€ à 8 M€, selon des explications données par Engie à l'AFP. L'avis conforme de l'AFB serait prévu le 1er février. Albioma   boucle un 4e investissement au Brésil : il va acquérir 60% d'une unité de cogénération bagasse adossée à la sucrerie-distillerie de Jalles Machado, à Goianésia (État du Goiás). L'accord prévoit la rénovation des chaudières existantes et l'installation d'une nouvelle turbine de 25 MW, portant la capacité totale à 65 MW.   Le groupe allemand  BayWa're  se renforce dans l'éolien en Italie : il y acquiert Energy Systems Services (ESS), un spécialiste de l'O&amp;M.   Akuo Energy  lance la construction d'une deuxième centrale hydroélectrique au fil de l'eau, Aqua Bella (2 MW) en Savoie. Elle est implantée sur l'Arc, un affluent de l'Isère. Les fonds propres ont été apportés par Johes, Vatel Capital et    Akuo. Une tranche de 250 000 euros est ouverte au financement participatif via AkuoCoop.    Mardi 19 décembre Voltalia  a remporté  64 MW éoliens au Brésil  . Acquis dans le cadre des dernières enchères nationales, les deux projets de 32 MW tourneront au nord-est du pays à partir de 2021.   Candidat à l'appel d'offres éolien au large de Dunkerque,  Vattenfall  annonce s'être associé à deux acteurs « de tout premier plan ». L'énergéticien suédois veut aussi concourir pour le premier appel d'offres éolien offshore non-subventionné au monde, au large des Pays-Bas.   L'agence  indienne de l'énergie solaire   a l'intention de lancer un appel d'offres pour la technologie du solaire flottant de pas moins de 10 GW, pendant les trois ans qui viennent, pour des projets qui devront être développés, construits et exploités. Des PPAs avec les énergéticiens locaux du pays seront proposés aux gagnants. Statkraft  vend sa participation de 30% dans le parc éolien offshore de Dudgeon (Royaume-Uni) pour 555 M£ (630 M€) à un consortium mené par le conglomérat public chinois  China Resources, présent dans l'énergie via CR Power.   Cet accord marque la sortie complète de l'énergéticien norvégien de l'éolien offshore.   Lundi 18 décembre La  Corée du Sud  a présenté un nouveau plan ambitieux pour réduire sa dépendance au nucléaire et au charbon. Elle vise désormais 20% d'électricité d'origine renouvelable en 2030, soit 58,5 GW de capacité installée contre 11,3 GW actuellement.   Dans le cadre du programme des Investissements d'Avenir, le gouvernement lance l'appel à projets « Concours d'innovation » pour  accompagner et financer les start-up et PME  innovantes. L'Ademe va piloter quatre thématiques : transport et mobilité durable ; agriculture innovante ; qualité des écosystèmes terrestres, aquatiques et marins ; EnR, stockage et systèmes énergétiques.   Avant sa généralisation en 2018,  le chèque énergie  a été expérimenté dans quatre départements. Le bilan montre une très faible utilisation pour des travaux de rénovation thermique. Il préconise de mieux informer les bénéficiaires sur ce volet et de faire le lien avec d'autres dispositifs, comme les primes CEE "coup de pouce".   Une étude est en cours au  ministère de l'Économie et des Finances  sur les coûts de renforcement du réseau liés au développement de la voiture électrique.   Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="Veganbottle, un bouteille 100 % végétale créée à partir de déchets" date="2017 12 2" journal="Metro (Bruxelles) (site web)">
100% végétales et intégralement fabriquées à partir de déchets: ces bouteilles « vegan » uniques en leur genre sont nées de l’imagination de Nicolas Mouflet dans son usine de Chaniers (Charente-Maritime).   Il aura fallu un an et demi de recherches pour « lever le frein technologique » qui empêchait la bagasse, résidu fibreux issu de la production de sucre à partir de la canne, de devenir la matière première de sa « Veganbottle » biodégradable. Mais Nicolas Mouflet, 41 ans, est finalement parvenu à transformer cette fibre en granulés qui composent la « préforme », sorte d’éprouvette qui est ensuite chauffée et soufflée pour devenir une bouteille transparente, comme l’industrie le fait depuis longtemps pour ces contenants plastiques en matériaux issus du pétrole.   D’autres ont déjà développé le concept du bioplastique, à partir du maïs par exemple, reconnaît le patron de la société Lyspackaging. Seulement « le maïs est un aliment, en plus, il consomme beaucoup d’eau. Moi je ne voulais utiliser que des déchets de production », précise l’industriel, qui fait venir par bateau la bagasse d’Afrique ou d’Inde dans son usine.   L’idée de cette bouteille écologique lui trottait dans la tête depuis 2001. A l’époque, il travaillait pour un producteur de bouteilles ayant de grands groupes pour clients. Chargé du dessin industriel, M. Mouflet a ainsi créé « plus de 4.000 bouteilles » pour de célèbres marques d’eau, de sodas, de jus de fruits ou encore de cosmétiques.   Engagé en 2010 par une entreprise spécialisé dans l’événementiel, le futur chef d’entreprise conditionnera ensuite l’eau pour des marques de luxe, grands groupes automobiles ou hôtels chics, jusqu’à la faillite de cette société en 2015.   Nicolas Mouflet décide de consacrer sa période de chômage à la création de Lyspackaging et au démarchage de clients potentiels. Prudent, il axe d’abord son activité sur des bouteilles en plastique PET (polyéthylène téréphtalate) classique, au design travaillé et notamment orientées vers les producteurs de spiritueux. Car les bouteilles en plastique, plus légères que le verre, permettent d’exporter 25% de liqueurs en plus.   Ces bouteilles assurent d’ailleurs encore aujourd’hui 80% de ses 600.000 euros de chiffre d’affaires annuel. Mais le jeune patron compte bien « inverser ce ratio dès l’année prochaine » avec sa Veganbottle. Problème marketing de taille: « une bouteille transparente 100% végétale ne se différencie pas d’une autre à base de pétrole. Pour qu’un bioplastique se distingue, il faut qu’il soit visible d’un seul coup d’oeil, pour le consommateur comme pour le producteur », relève le chef d’entreprise.  Cet article est paru dans Metro (Bruxelles) (site web)
</article>
<article titre="Écoconception Les jus Yumi adoptent la VeganBottle" date="2017 12 1" journal="Process Alimentaire, no. N° 1353">
Start-up de la Food Tech, Yumi conditionne désormais ses jus dans des bouteilles 100 % végétales et compostables. Pour remplacer le plastique, ce sont les résidus de canne à sucre qui ont été choisis : 40 % d'émissions de carbone en moins, zéro hydrocarbure. Le flacon utilisé est en fait une déclinaison de la VeganBottle fabriquée par Lyspackaging à Saintes (17). Elle est le fruit de technologies brevetées dans le domaine de la transformation d'extraction du sucre issu de la bagasse. Conçue par l'ingénieur Nicolas Moufflet, à la tête de l'entreprise, elle peut combiner ce biopolymère avec d'autres coproduits variés comme les noyaux d'olives broyés, les pépins de raisin, le cacao, les coquilles de crustacés, etc. Plastiques issus de noyaux d'olives   «  Dans la mesure où il est difficile de distinguer visuellement en rayon une bouteille issue de pétrole d'une bouteille transparente 100 % végétale, nous imaginons attirer l'oeil avec des bouteilles colorées  », explique-t-on chez Lyspackaging. Celles qui sont issues de noyaux d'olive, par exemple, prennent une teinte marron qui devient dorée une fois remplies. «  On peut aussi imaginer des bouteilles à base carotte pour des jus de carotte ou à base de pomme pour des jus de pomme  », ajoute l'entreprise. Qui fabrique également des bouchons et étiquettes biodégradables.   La VeganBottle peut disposer de propriétés barrières selon les applications. Elle a déjà été vendue à plus d'un million d'exemplaires en France et en Belgique pour différentes applications : herbes, jus de fruits frais, fontaines à eau, bouteilles vides chez Biocoop (qui interdit désormais les bouteilles en plastique issues de ressources fossiles), bouteilles personnalisées pour des start-up mais aussi pour des produits cosmétiques. Car Lyspackaging forme aussi des pots. Le département design de l'entreprise assure une prestation d'agence pour créer des préformes puis des bouteilles ou pots aux formes originales, soufflées sur place. «  Son comportement sur les machines d'injection-soufflage puis de remplissage est identique à celui d'une bouteille en PET classique  », garantit Lyspackaging. Mais elle coûte encore 30 % plus cher à l'achat.   En fin de vie, la VeganBottle peut être compostée industriellement suivant la norme EN13432. Mais pas mise au compost chez les particuliers, ni jetée dans la nature. Car elle n'est pas « home compost ». En l'absence de filières de compostage industrielles bien identifiées, ce type de bouteilles n'offre donc pas encore tout son potentiel en fin de vie. Elles peuvent même poser des difficultés aux filières de recyclage des plastiques. Néanmoins, la VeganBottle gagne sur le tableau des ressources utilisées. C'est d'ailleurs l'enjeu qui a motivé Yumi.
</article>
<article titre="En Charente-Maritime, des bouteilles 100 % végétales sont fabriquées à partir de déchets" date="2017 11 30" journal="Le Journal du Centre  Nièvre">
Écologie. 100 % végétales et intégralement fabriquées à partir de déchets : ces bouteilles « vegan » uniques en leur genre ( photo AFP ) sont nées de l'imagination de Nicolas Mouflet, auteur de milliers de contenants pour de grandes marques de boissons, et produites depuis cet été dans son usine de Chaniers (Charente-Maritime). Il aura fallu un an et demi de recherches pour « lever le frein technologique » qui empêchait la bagasse, résidu fibreux issu de la production de sucre à partir de la canne, de devenir la matière première de sa « Veganbottle » biodégradable. Mais Nicolas Mouflet, 41 ans, est finalement parvenu à transformer cette fibre en granulés qui composent la « préforme », sorte d'éprouvette qui est ensuite chauffée et soufflée pour devenir une bouteille transparente. D'autres ont déjà développé le concept du bioplastique, à partir du maïs par exemple, reconnaît le patron de la société Lyspackaging. Seulement « le maïs est un aliment, en plus, il consomme beaucoup d'eau. Moi je ne voulais utiliser que des déchets de production », précise l'industriel.
</article>
<article titre="Lyspackaging déploie la Veganbottle" date="2017 11 29" journal="Emballages magazine.com">
Le conditionneur promeut une alternative au PET que les recycleurs n'apprécient pas du tout.-   À Chaniers (Charente-Maritime), Lyspackaging, conditionneur à façon fondé en janvier 2015, déploie actuellement la production de bouteilles entièrement issues de ressources végétales provenant de déchets. En l'occurrence, la bagasse de la canne à sucre. Nicolas Moufflet, le fondateur de l'entreprise, positionne la Veganbottle comme "une sérieuse alternative au verre", mais aussi au polyéthylène téréphtalate (PET) et aux végétaux comme le maïs qui consomme beaucoup d'eau. Spécialisé dans les jus de légumes frais biologiques, Yumi a opté pour ce conditionnement. Afin de rendre ces bouteilles biosourcées identifiables, Lyspackaging incorpore des traces de noyaux d'olives ou des coquilles de moules ainsi que des particules d'huîtres, de roseaux ou encore de pépins de raisin. Lyspackaging a également mis au point une bouteille d'eau biosourcée et compostable baptisée Naturo. De la conception du contenant au remplissage en passant par l'injection de la préforme et l'étiquetage, l'entreprise livre une prestation de conditionnement complète avec une capacité de plusieurs millions de cols. L'essentiel des volumes est toujours réalisé en PET, le métier d'origine de Nicolas Mouflet. Recyclage   Les développements de Lyspackaging ne laissent pas les plasturgistes indifférents. Échaudés par l'affaire du PET opaque dans le domaine du lait, les recycleurs de plastique ne souhaitent pas vraiment voir une bouteille végétale perturber le recyclage du PET alors que la feuille de route de l'économie circulaire à paraître en mars 2018 intègre un objectif de recyclage de 100% des plastiques en 2025. Les recycleurs ajoutent que la Veganbottle ne peut pas se prévaloir d'une filière de recyclage.
</article>
<article titre="Lyspackaging.Des bouteilles 100 % végétales" date="2017 11 29" journal="Le Télégramme (Bretagne)  LORIENT">
Il aura fallu un an et demi de recherches pour « lever le frein technologique » qui empêchait la bagasse, résidu fibreux issu de la production de sucre à partir de la canne, de devenir la matière première de la « Veganbottle » biodégradable mise au point par Nicolas Mouflet. Il est finalement parvenu à transformer cette fibre qui finira en une bouteille transparente, comme l'industrie le fait depuis longtemps pour ces contenants plastiques en matériaux issus du pétrole. D'autres ont déjà développé le concept du bioplastique, à partir du maïs par exemple, reconnaît le patron de la société Lyspackaging. Seulement « le maïs est un aliment, en plus, il consomme beaucoup d'eau. Moi je ne voulais utiliser que des déchets de production », précise l'industriel, qui fait venir par bateau la bagasse d'Afrique ou d'Inde dans son usine. Teintée pour être différenciée du plastique   Problème marketing de taille : « Une bouteille transparente 100 % végétale ne se différencie pas d'une autre à base de pétrole. Pour qu'un bioplastique se distingue, il faut qu'il soit visible d'un seul coup d'oeil, pour le consommateur comme pour le producteur », relève le chef d'entreprise. Pour se démarquer visuellement, Lyspackaging incorpore donc dans ses bouteilles une foule de résidus naturels. Réduits en poudre, ces additifs donnent aux flacons végétaux une teinte hétérogène et une surface qui peut paraître lisse ou en « peau d'orange », selon la demande.   1
</article>
<article titre="Charente-Maritime: des bouteilles 100% végétales à partir de déchets" date="2017 11 28" journal="DH (site web)">
Publié le mardi 28 novembre 2017 à 11h44 à Chaniers (France) (AFP)   100% végétales et intégralement fabriquées à partir de déchets: ces bouteilles "vegan" uniques en leur genre sont nées de l'imagination de Nicolas Mouflet, auteur de milliers de contenants pour de grandes marques de boissons, et produites depuis cet été dans son usine de Chaniers (Charente-Maritime).   Il aura fallu un an et demi de recherches pour "lever le frein technologique" qui empêchait la bagasse, résidu fibreux issu de la production de sucre à partir de la canne, de devenir la matière première de sa "Veganbottle" biodégradable. Mais Nicolas Mouflet, 41 ans, est finalement parvenu à transformer cette fibre en granulés qui composent la "préforme", sorte d'éprouvette qui est ensuite chauffée et soufflée pour devenir une bouteille transparente, comme l'industrie le fait depuis longtemps pour ces contenants plastiques en matériaux issus du pétrole.   D'autres ont déjà développé le concept du bioplastique, à partir du maïs par exemple, reconnaît le patron de la société Lyspackaging. Seulement "le maïs est un aliment, en plus, il consomme beaucoup d'eau. Moi je ne voulais utiliser que des déchets de production", précise l'industriel, qui fait venir par bateau la bagasse d'Afrique ou d'Inde dans son usine.   L'idée de cette bouteille écologique lui trottait dans la tête depuis 2001. A l'époque, il travaillait pour un producteur de bouteilles ayant de grands groupes pour clients. Chargé du dessin industriel, M. Mouflet a ainsi créé "plus de 4.000 bouteilles" pour de célèbres marques d'eau, de sodas, de jus de fruits ou encore de cosmétiques.   Engagé en 2010 par une entreprise spécialisé dans l'événementiel, le futur chef d'entreprise conditionnera ensuite l'eau pour des marques de luxe, grands groupes automobiles ou hôtels chics, jusqu'à la faillite de cette société en 2015.   - Noyaux d'olive et coquilles de moules -   Nicolas Mouflet décide de consacrer sa période de chômage à la création de Lyspackaging et au démarchage de clients potentiels. Prudent, il axe d'abord son activité sur des bouteilles en plastique PET (polyéthylène téréphtalate) classique, au design travaillé et notamment orientées vers les producteurs de spiritueux. Car les bouteilles en plastique, plus légères que le verre, permettent d'exporter 25% de liqueurs en plus.   Ces bouteilles assurent d'ailleurs encore aujourd'hui 80% de ses 600.000 euros de chiffre d'affaires annuel. Mais le jeune patron compte bien "inverser ce ratio dès l'année prochaine" avec sa Veganbottle.   Problème marketing de taille: "une bouteille transparente 100% végétale ne se différencie pas d'une autre à base de pétrole. Pour qu'un bioplastique se distingue, il faut qu'il soit visible d'un seul coup d??il, pour le consommateur comme pour le producteur", relève le chef d'entreprise.   Pour se démarquer visuellement, Lyspackaging incorpore donc dans ses bouteille une foule de résidus naturels issus d'autres productions que la canne à sucre: noyaux d'olive pour obtenir du beige, coquilles de moules pour du noir, mais aussi huîtres, roseaux, cacao, café, bois, pépins de raisin, blé, cuir, etc.   Réduits en poudre, ces additifs donnent aux flacons végétaux une teinte hétérogène et une surface qui peut paraître lisse ou en "peau d'orange", selon la demande.   D'ici quelques mois, la start-up créée en 2015 et employant sept personnes, proposera aussi des colorants naturels à partir de plantes pour les clients soucieux d'avoir une bouteille à la couleur uniforme.   © 2017 AFP. Tous droits de reproduction et de représentation réservés. Toutes les informations reproduites dans cette rubrique (dépêches, photos, logos) sont protégées par des droits de propriété intellectuelle détenus par l'AFP. Par conséquent, aucune de ces informations ne peut être reproduite, modifiée, rediffusée, traduite, exploitée commercialement ou réutilisée de quelque manière que ce soit sans l'accord préalable écrit de l'AFP.  Cet article est paru dans DH (site web)
</article>
<article titre="Charente-Maritime: des bouteilles 100% végétales à partir de déchets" date="2017 11 28" journal="La République des Pyrénées (site web)">
100% végétales et intégralement fabriquées à partir de déchets: ces bouteilles "vegan" uniques en leur genre sont nées de l'imagination de Nicolas Mouflet, auteur de milliers de contenants pour de grandes marques de boissons, et produites depuis cet été dans son usine de Chaniers (Charente-Maritime).   Il aura fallu un an et demi de recherches pour "lever le frein technologique" qui empêchait la bagasse, résidu fibreux issu de la production de sucre à partir de la canne, de devenir la matière première de sa "Veganbottle" biodégradable. Mais Nicolas Mouflet, 41 ans, est finalement parvenu à transformer cette fibre en granulés qui composent la "préforme", sorte d'éprouvette qui est ensuite chauffée et soufflée pour devenir une bouteille transparente, comme l'industrie le fait depuis longtemps pour ces contenants plastiques en matériaux issus du pétrole.   D'autres ont déjà développé le concept du bioplastique, à partir du maïs par exemple, reconnaît le patron de la société Lyspackaging. Seulement "le maïs est un aliment, en plus, il consomme beaucoup d'eau. Moi je ne voulais utiliser que des déchets de production", précise l'industriel, qui fait venir par bateau la bagasse d'Afrique ou d'Inde dans son usine .   L'idée de cette bouteille écologique lui trottait dans la tête depuis 2001. A l'époque, il travaillait pour un producteur de bouteilles ayant de grands groupes pour clients. Chargé du dessin industriel, M. Mouflet a ainsi créé "plus de 4.000 bouteilles" pour de célèbres marques d'eau, de sodas, de jus de fruits ou encore de cosmétiques.   Engagé en 2010 par une entreprise spécialisé dans l'événementiel, le futur chef d'entreprise conditionnera ensuite l'eau pour des marques de luxe, grands groupes automobiles ou hôtels chics, jusqu'à la faillite de cette société en 2015.   - Noyaux d'olive et coquilles de moules -   Nicolas Mouflet décide de consacrer sa période de chômage à la création de Lyspackaging et au démarchage de clients potentiels. Prudent, il axe d'abord son activité sur des bouteilles en plastique PET (polyéthylène téréphtalate) classique, au design travaillé et notamment orientées vers les producteurs de spiritueux. Car les bouteilles en plastique, plus légères que le verre, permettent d'exporter 25% de liqueurs en plus.    Ces bouteilles assurent d'ailleurs encore aujourd'hui 80% de ses 600.000 euros de chiffre d'affaires annuel. Mais le jeune patron compte bien "inverser ce ratio dès l'année prochaine" avec sa Veganbottle.   Problème marketing de taille: "une bouteille transparente 100% végétale ne se différencie pas d'une autre à base de pétrole. Pour qu'un bioplastique se distingue, il faut qu'il soit visible d'un seul coup d'oeil, pour le consommateur comme pour le producteur", relève le chef d'entreprise.    Pour se démarquer visuellement, Lyspackaging incorpore donc dans ses bouteille une foule de résidus naturels issus d'autres productions que la canne à sucre: noyaux d'olive pour obtenir du beige, coquilles de moules pour du noir, mais aussi huîtres, roseaux, cacao, café, bois, pépins de raisin, blé, cuir, etc.    Réduits en poudre, ces additifs donnent aux flacons végétaux une teinte hétérogène et une surface qui peut paraître lisse ou en "peau d'orange", selon la demande.   D'ici quelques mois, la start-up créée en 2015 et employant sept personnes, proposera aussi des colorants naturels à partir de plantes pour les clients soucieux d'avoir une bouteille à la couleur uniforme.   Source : AFP    France-Monde France Monde Société  Cet article est paru dans La République des Pyrénées (site web)
</article>
<article titre="Les EnR comme moteur des territoires d'outre-mer" date="2017 11 23" journal="Les Echos - Le Cercle (site web)">
Les EnR, une source d'indépendance des territoires insulaires Organisée au Palais Brongniart à Paris le samedi 18 novembre 2017, la sixième édition de la Journée pour le développement de l'outre-mer (JOMD) a été, une nouvelle fois, l'occasion de confronter les points de vue sur la situation, les perspectives économiques et sociales dans les outre-mer, et d'approfondir ainsi la compréhension des enjeux et des besoins de ces territoires isolés du continent. Si plusieurs thématiques étaient au programme de cette manifestation comme l'entrepreneuriat, l'emploi, l'insertion professionnelle, la sécurité ou la santé, le développement durable et la protection de la biodiversité ont occupé une large part des débats et interventions. Les risques du changement climatique dans les outre-mer et la nécessité de décarboner rapidement leur mix énergétique ont mis au goût du jour le développement des renouvelables, de l'efficacité énergétique, de la mobilité propre comme autant de stratégies de développement durable. Les territoires ultramarins sont engagés dans cette réflexion, qui prend corps dans leur Programmation pluriannuelle de l'énergie (PPE), introduit par la loi de Transition énergétique pour une croissance verte, et prévoient de compter 50 % d'énergies renouvelables dans leur mix énergétique d'ici 2020 et d'atteindre l'autonomie énergétique à l'horizon 2030. Des objectifs ambitieux donc qui nécessitent une vraie politique incitative afin de mettre tous les moyens à disposition des collectivités pour exploiter leurs forts potentiels renouvelables et d'encourager les investissements privés.   Pérenniser les financements et encourager les investissements Caractérisés de manière générale par de forts taux d'ensoleillement, des côtes très venteuses, de larges ressources hydroélectriques et de grandes quantités de biomasses valorisables (via l'exploitation de la bagasse notamment), les Départements et Régions d'outre-mer (DROM) ont toujours constitué des territoires propices au développement des énergies vertes. La diversité des conditions naturelles permet d'y expérimenter tous les modes de production dans des conditions réelles, et fait de ces territoires de véritables vitrines de notre mix énergétique futur.   Ce statut implique toutefois un financement relativement conséquent qu'il revient à l'État d'organiser, soit par des subventions directes attribuées aux collectivités ou aux porteurs de projets, soit par des avantages fiscaux incitatifs. Fonds obtenus dans le cadre des Territoires à énergie positive (Tepos), pérennisation de la contribution au service public de l'électricité (CSPE) dans son objectif de promotion des énergies renouvelables, maintien des tarifs d'achat spécifiques à l'outre-mer, et mesures d'incitations fiscales appropriées dans le but d'encourager les investissements dans les renouvelables ou les économies d'énergie de la part des particuliers comme des professionnels, les dispositifs de soutien sont multiples et devront être reconduits si ces zones veulent réaliser les objectifs annoncés. En tant que lieu de rencontres et d'échanges, la sixième JOMD rassemblait dans ce but nombreux décideurs économiques, sociaux et politiques venus de l'ensemble des outre-mer afin d'évoquer cette question du financement. Invitée à s_exprimer en ouverture, la ministre des outre-mer, Annick Girardin, a réaffirmé le soutien du gouvernement en la matière :  "l'ambition de ce gouvernement, c'est d'accompagner avec une revue de toutes les aides, d'accompagner mieux et plus, les entreprises dans leur projet et leurs initiatives" , a-t-elle affirmé.   Valoriser les innovations vertes dans les DROM Il faut dire que les besoins sont nombreux dans les DROM et que ces filières d'activités représentent un fort potentiel d'industrialisation, d'emplois locaux et d'innovation. Créer davantage de synergies entre les territoires, organiser un meilleur pilotage de la chaîne de production/distribution dans le développement des renouvelables, concentrer les efforts de la recherche et du développement sur le stockage de l'énergie, favoriser la mise en place de démonstrateurs, ou adapter les réseaux énergétiques de distribution vers les "réseaux intelligents", les chantiers identifiés sont à la fois innovants et ambitieux, et nécessitent toutes les bonnes volontés.   Plusieurs projets sont déjà en cours d'expérimentation sous l'impulsion du groupe EDF qui a déjà investi plus de 5 milliards d'euros lors de la dernière décennie pour financer près de 90 innovations. On peut citer entre autres, un dispositif de micro-réseau solaire avec stockage par hydrogène dans le cirque de Mafate à La Réunion, un projet photovoltaïque Toucan 2 qui viendra compléter la centrale solaire avec stockage sur batteries de Toucan, en exploitation depuis 2015 en Guyane, ou un système de recharge intelligente destiné à alimenter des véhicules électriques en énergie renouvelable en Corse. En Martinique enfin, le projet Green Technologies vise à déployer quant à lui un réseau de recharge collaboratif 100 % solaire de véhicules électriques dans les territoires insulaires, et s'est vu décerné en 2016 le prix Innovation outre-mer de BPI France dans la catégorie "Green Business et industrie". La prochaine édition de ce concours, organisé le 14 décembre prochain, sera par ailleurs l'occasion d'apprécier les nouvelles initiatives prises en faveur du développement durable dans les territoires ultra-marins et d'offrir aux lauréats les soutiens financiers nécessaires à leur développement.
</article>
<article titre="Emballage Lyspackaging sucre les plastiques" date="2017 11 23" journal="L'Usine Nouvelle, no. 3539">
La canne à sucre est pleine de ressources, y compris lorsqu'il s'agit de produire des emballages pour l'agroalimentaire. Alternative aux plastiques pétrosourcés, les bioplastiques ouvrent de nouvelles perspectives. Produits à partir de matière végétale comme l'amidon de maïs, de betterave ou de pomme de terre, ils nécessitent moins d'énergie que les polymères issus d'hydrocarbures. Lyspackaging, fabricant de bouteilles et flacons en polytéréphtalate d'éthylène (PET), développe, lui, des emballages agrosourcés à partir d'acide polylactique (PLA) provenant de la bagasse de canne à sucre. Certifiés contact alimentaire par le laboratoire Pourquery, ces conditionnements s'adressent d'abord aux industriels de l'agroalimentaire, mais l'entreprise lorgne également du côté du marché des cosmétiques. Une fois approvisionné en granulés, Lyspackaging formule la matière afin de produire par injection des pots, des flacons ainsi que des bouteilles transparentes ou blanches. Dans certains cas, les emballages peuvent être agrémentés de déchets comme des noyaux, des coquilles, des épluchures, des pépins... Un aspect qui permet aux emballages de se distinguer des plastiques couramment utilisés. Une valeur ajoutée, aussi, en termes d'image pour les clients. Certains, présents dans la filière biologique, ont déjà témoigné leur intérêt pour renforcer la cohérence entre le contenu et leur contenant, explique son président, Nicolas Mouflet. Ainsi, l'entreprise a déjà broyé des noyaux d'olive pour donner une teinte et un grain particuliers à une bouteille d'huile d'olive. Elle a aussi ajouté des pelures de carottes broyées pour colorer d'un bel orangé l'emballage d'un jus de carotte. Elle a même expérimenté les brisures de coquilles d'huîtres. Ressemblant à une pierre blanche, la matière devait apporter un rendu original à une bouteille de vin blanc destinée à accompagner un plateau de fruits de mer (mais aussi à être consommée rapidement). Dans tous ces cas, moins de 3 % de matières sont intégrées, révèle Nicolas Mouflet, qui reconnaît que cela ajoute quelques grammes au poids final du produit.   Commercialisée depuis janvier, la VeganBottle, qui est aussi le nom de la marque, a été récompensée par un prix Emergence de Bpifrance et a profité du soutien de l'incubateur régional de l'innovation pour mettre au point cette recette. La matière posséderait, selon le dirigeant, des propriétés mécaniques et des barrières supérieures aux autres formulations à base de PLA. Dépourvu de toute trace de pétrole, l'emballage est aussi biodégradable, compostable industriellement ou dépolymérisable. Située à Saintes (17), l'entreprise Lyspackaging a été créée en 2015. Elle compte dix collaborateurs et a réalisé un chiffre d'affaires de 350 000 euros en 2016.
</article>
<article titre="Paroles d'Experts Lyspackaging, Le Packaging Innovant et Responsable" date="2017 11 22" journal="La Tribune (site web)">
Le packaging est un élément-clé dans l'acte d'achat. Aujourd'hui, les attentes des clients se focalisent sur sa praticité, sur le respect de l'environnement ainsi que sur la cohérence entre le contenu et le contenant. «  La tendance est à l'emballage  écologique  et surtout au packaging connecté : le digital permet d'améliorer la traçabilité de produits, la sécurité alimentaire et la sécurité du produit. Dans le cadre d'un marketing ludique, l'interactivité s'invite également sur les emballages  » explique Nicolas Moufflet, Directeur général de Lyspackaging.   Depuis 2016, Lyspackaging conçoit des bouteilles entièrement végétales, fabriquées sans une seule goutte de pétrole. Leur composition ? De la bagasse de canne à sucre, un déchet non alimentaire et non OGM qui est ensuite transformé en petites billes bio-plastiques. «  Seule 1 bouteille en plastique pétrochimique sur 2 est recyclée, ce qui engendre une grande pollution et l'urgence d'adopter des solutions sans pétrole. Le bio-plastique est quant à lui biodégradable et la plupart du temps compostable.  ». La start-up française produit des petites et grandes séries de flacons, pots et bouteilles pour les secteurs cosmétiques, agro-alimentaires et non-alimentaires. Pour mettre en valeur ses travaux de recherche Lyspackaging a créé la marque VEGANBOTTLE®, une bouteille optimisée afin de réduire l'impact sur l'environnement tout au long de son cycle de vie. Fabriquée avec des matériaux respectant la norme EN13432 de biodégradabilité et de compostabilité, VEGANBOTTLE® est certifié en contact alimentaire. «  Le compostage est une solution, qui contrairement au recyclage, permet d'éliminer complètement les déchets. Il est dommage de constater que les principales filières de recyclage en France n'encouragent pas ce type de démarche, qui s'inscrit pourtant dans une dynamique d'économie circulaire et de réduction de la pollution  » précise Nicolas Moufflet.   Lyspackaging produit actuellement 10 millions de flacons par an dans son usine située en Charente-Maritime. Sa prochaine étape est de lever des fonds afin de poursuivre ses programmes d'innovation. La start-up prévoit aussi d'aborder de nouveaux marchés en créant des unités de production dans les DOM-TOM ainsi qu'à l'international. Son ambition ? Répondre à la problématique de pollution des plastiques pétrochimiques tout en proposant un packaging innovant et éco-responsable.   CONTACT LYSPACKAGING8 chemin NATTIER17610 CHANIERSTel : 0033 9 52 92 95 68projet@lyspackaging.comwww.lyspackaging.comwww.veganbottle.com  Cet article est paru dans La Tribune (site web)
</article>
<article titre="ENQUÊTE ANNUELLE 2017 SUR LES CONSOMMATIONS D’ÉNERGIE DANS L’INDUSTRIE" date="2017 11 17" journal="Rapports - Marketing - INSEE">
8 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... ENQUÊTE ANNUELLE 2017 SUR LES CONSOMMATIONS D’ÉNERGIE DANS L’INDUSTRIE  Numéro Siret : I__I__I__I I__I__I__I I__I__I__I Pour tout renseignement concernant cette enquête...    Pour voir le rapport :  https://www.insee.fr/fr/metadonnees/source/fichier/EACEI2017_questionnaire.pdf
</article>
<article titre="Colombie: la symbiose industrielle au service de la croissance verte" date="2017 10 27" journal="LesAffaires.com">
ÉCONOMIE CIRCULAIRE - Pour produire des biens et des services, les entreprises utilisent de l'eau, des carburants et des métaux, responsables de déchets et d'émissions. En Colombie, l'agriculture et le pâturage du bétail ne représentent que 6 % du produit national brut (PNB), mais consomment 40 % de l'eau du pays, générant 48 % de gaz à effet de serre.   Par Jooyoung Park, École de management de l'Université des Andes (UASM), article publié dans le journal économique colombien Portafolio. Après 50 ans de guerre, l'accord de paix conclut en 2016 par la Colombie avec les forces de la guérilla, a ouvert la voie de l'aide internationale au soutien du pays dans les domaines de la construction d'infrastructures rurales et l'accompagnement de la croissance verte. Les entreprises sont incitées à réduire les coûts et à causer moins de préjudices à l'environnement en optimisant l'utilisation des ressources, notamment au moyen de la maîtrise des fuites, du renouvellement des équipements et de la revalorisation des déchets. L'échange des déchets participe à cette optimisation. En effet, les déchets d'une entreprise peuvent servir de matière première à une autre. Les producteurs de pâtes et de papier utilisent la bagasse, un résidu issu du broyage de la canne à sucre, comme substitut au bois. Le partage des infrastructures et des services entre les entreprises ? telles que les stations d'épuration des eaux usées ?, contribue également à une meilleure efficacité.   Le terme de « symbiose industrielle » (SI) désigne la coopération entre les entreprises en vue d'optimiser les ressources. Si la concurrence a favorisé l'innovation et la croissance du marché, la coopération constitue le sésame pour promouvoir une meilleure utilisation des ressources et le développement durable.   L'expérience de SI la plus connue au monde a été lancée il y a 40 ans à Kalundborg, au Danemark : une douzaine d'usines a mis au point trente projets de mutualisation des ressources. Aujourd'hui, la SI figure dans les politiques environnementales des pays industrialisés. En Corée du Sud, entre 2005 et 2014, 159 projets de SI ont été menés par 592 entreprises, générant des bénéfices combinés de 1,3 milliard de dollars (1 milliard d'euros). En Chine, plus de 108 parcs industriels labellisés « parcs éco-industriels » assurent la promotion de l'économie circulaire.   En Colombie, la SI constitue l'un des axes du programme « Réseau d'entreprises durables » (en espagnol, RedES) encadré par l'UASM. En 2017, un projet pilote réunissant 13 entreprises a été lancé depuis la capitale, Bogotá, avec le soutien de l'Autorité environnementale du département de Cundinamarca (CAR). Ces sociétés s'étaient auparavant inscrites au programme de « Production plus propre » du RedES, qui réunit aujourd'hui quelque 330 entreprises. Dix grandes entreprises (plus de 200 collaborateurs selon les standards colombiens), deux petites (moins de 50), et une entreprise de taille moyenne suivent le projet de SI. Toutes relativement proches géographiquement les unes des autres, elles représentent des secteurs allant des matériaux de construction et de la construction à l'emballage, en passant par les boissons non alcoolisées, la transformation alimentaire, la chimie, les cosmétiques, l'aviculture et le traitement des déchets.   À ce jour, neuf prototypes de projets de SI sont en phase de mise au point. Dans l'un d'eux, un transformateur alimentaire livre gratuitement, chaque mois, 62 tonnes de déchets de bois et de plastique à une entreprise de matériaux de construction, qui fabrique des cagettes plastiques qu'elle lui retourne ensuite. D'une durée de vie cinq fois supérieure, elles remplacent les anciennes cagettes en bois. Dans un autre projet, une chaîne de restauration fournit près de 400 tonnes de résidus de café à un aviculteur qui les utilise pour absorber les odeurs et nourrir le compost. Dans un troisième projet, plusieurs entreprises mutualisent un service de traitement des déchets pour collecter et éliminer les boues et les déchets dangereux. Elles réduisent ainsi le coût du traitement des déchets et optimisent les transports, ce qui entraîne une baisse des émissions de carbone. Au total, on estime que ces neuf projets généreront près de 475 000 dollars (401 805 euros) de retombées économiques positives grâce aux économies réalisées et à l'augmentation du chiffre d'affaires ; cette réduction de l'impact environnemental équivaut à la revalorisation de 1 446 tonnes de déchets et à une baisse des émissions de gaz à effet de serre de 1 018 tonnes.   Pour générer des retombées économiques positives dans le cadre d'une SI, les entreprises doivent investir du temps et des ressources. Le juste équilibre entre les déchets entrants et sortants exige des informations relatives aux propriétés des matériaux échangés, et des technologies compatibles. Des infrastructures sont parfois requises - par exemple, un pipeline, pour conduire la vapeur issue des pertes de chaleur d'une usine à celle du partenaire. D'un point de vue organisationnel, la SI implique une confiance réciproque. Les relations des 330 entreprises engagées dans le programme RedES sont fondées sur la confiance, chaque société concevant et réalisant son projet de production propre de manière individuelle avec l'aide des consultants de l'UASM.   Une seconde initiative de SI démarrera prochainement, avec davantage d'entreprises. L'ensemble des entreprises du programme RedES bénéficie d'un programme d'accompagnement au niveau de la mise en oeuvre de la SI et des projets de production propre. De plus, l'autorité départementale reconnaît les entreprises qui réduisent leur impact environnemental, réduisant ainsi les coûts de l'application des précédents règlements. Au fil du développement du programme à travers la Colombie, des projets de SI supplémentaires seront lancés pour encourager la croissance verte.   CLIQUEZ ICI POUR CONSULTER L'ENSEMBLE DU DOSSIER: ÉCONOMIE CIRCULAIRE: DE L'OR DANS LES DÉCHETS  Cet article est paru dans LesAffaires.com
</article>
<article titre="Albioma: l'activité continue de progresser au 3T, grâce au Brésil" date="2017 10 25" journal="AFP Infos Economiques">
Paris, 25 oct 2017 (AFP) - - Le producteur d'électricité Albioma a vu son chiffre d'affaires progresser au troisième trimestre, toujours porté par la bonne disponibilité de ses centrales et son activité au Brésil.   Entre juillet et septembre, le chiffre d'affaires s'affiche à 107,2 millions d'euros, en hausse de 18%. Il est en croissance de 14% à 304,1 millions d'euros sur les neuf premiers mois de l'année. Les revenus sont portés par "l'excellente disponibilité" de ses installations, notamment les centrales biomasse en France, qui représentent l'essentiel de l'activité du groupe, et par un avenant au contrat d'achat d'électricité d'une centrale aux Caraïbes, explique Albioma dans un communiqué.   La société a confirmé la mise en service au premier trimestre 2018 de sa centrale bagasse/biomasse Galion 2 en Martinique.   Au Brésil, l'activité est en "nette amélioration" au troisième trimestre, avec un retour à des niveaux "normaux" d'approvisionnement en bagasse (résidus de canne à sucre) après un début de campagne sucrière tardif qui avait pénalisé la production.   Elle a aussi bénéficié du taux de change entre l'euro et le real.   Albioma confirme donc ses objectifs pour 2017, avec un excédent brut d'exploitation (Ebitda) attendu entre 130 et 138 millions d'euros et un bénéfice net compris entre 30 à 35 millions d'euros, contre 33 millions d'euros l'an dernier.   mhc/soe/ggy
</article>
<article titre="Des bouteilles sans une goutte de pétrole Nicolas Moufflet a créé des bouteilles en plastique végétal 100 %, biodégradables et recyclables à l'infini grâce à un procédé unique au monde... qui intéresse les grands groupes industriels." date="2017 10 10" journal="Aujourd'hui en France  Edition Principale">
Par Fabien Paillot   Une nouvelle adresse et un site de production cinq fois plus grand que le précédent : le pari audacieux de Nicolas Moufflet se révèle payant. Cet ingénieur de 41 ans, installé à Saintes (Charente-Maritime), a mis au point un plastique végétal 100 % biodégradable, compostable et recyclable à l'infini.   A la tête de Lyspackaging, son entreprise fondée en 2015, il commercialise depuis des bouteilles et flacons « sans une seule goutte de pétrole », contrairement à l'immense majorité des plastiques vendus à travers la planète. Le procédé, breveté et secret, reste pour l'heure unique au monde. Aucun industriel n'a encore investi ce créneau. Plus de 22 000 tonnes de plastique issu de l'industrie pétrochimique finissent pourtant chaque jour dans les océans. Pour réaliser cette prouesse, Nicolas Moufflet a choisi la bagasse, le déchet de la canne à sucre, aux propriétés étonnantes. « J'ai développé spécifiquement des granulés de bagasse optimisés pour recevoir des aliments », détaille-t-il. Transformée en plastique végétal, la bagasse garde la faculté d'être recyclée ou de se composter facilement. « Une alternative crédible au pétrole », fait valoir cet entrepreneur auréolé de plusieurs prix et du soutien de Bpifrance, la Banque publique d'investissement.   Le Palais de Tokyo lui a commandé 100 000 bouteilles Les commandes, elles, commencent à affluer. Lyspackaging espérait atteindre une production de 700 000 bouteilles végétales cette année. Elle devrait aisément franchir la barre des 2 millions.   Le Palais de Tokyo, centre d'art contemporain situé à Paris, lui a récemment commandé 100 000 bouteilles. Et de grands groupes du luxe ou de l'agroalimentaire s'y intéressent de près.   Nicolas Moufflet continue ses recherches et rêve d'une filière capable de mieux valoriser les déchets verts. A ses plastiques végétaux issus de la bagasse, l'ingénieur peut désormais adjoindre des noyaux d'olive broyés, des roseaux ou des pépins de raisin. Coquilles de crustacés, déchets de céréales ou de légumes : « Nous allons attaquer une bonne dizaine de nouveaux coproduits ! Leur quantité varie de 1 à 20 %, cela dépend du rendu que l'on veut obtenir », détaille-t-il.   Car Nicolas Moufflet mise sur « la cohérence entre le contenu et le contenant » et cite en exemple les engrais développés pour l'agriculture biologique encore vendus dans des flacons... en plastique. Lui rêve de « créer de nouveaux codes pour le consommateur » : demain, l'huile d'olive se vendra dans une bouteille en noyaux d'olive. Economiquement, cette voie a des atouts : « Les déchets verts s'achètent aujourd'hui une misère. » Ces bouteilles végétales sont encore 30 % plus chères à l'achat. Mais peu importe, sourit Nicolas Moufflet : « Nous avons démontré que cela est possible. »
</article>
<article titre="Les valeurs à suivre aujourd'hui à la Bourse de Paris - Jeudi 21 septembre 2017" date="2017 09 21" journal="Option Finance (site web)">
AOF - 21 septembre 2017   (AOF) - 1000MERCIS   1000mercis a enregistré une croissance de 8% de son bénéfice net au premier semestre, à 3,4 millions d'euros, et de 2% de son bénéfice d'exploitation, à 5,1 millions. La marge d'exploitation du spécialiste du data marketing est passée de 18% à 17% de son chiffre d'affaires. Ce dernier a atteint 29,1 millions d'euros au premier semestre, en progression de 4%. ABIVAX   Abivax a accusé au premier semestre 2017 une perte nette de 5,5 millions d'euros, contre une perte nette de 8,3 millions au premier semestre 2016. La biotech spécialisée dans le traitement contre le sida a bénéficié du contrôle rigoureux des dépenses ainsi que de l'arrêt du développement d'ABX203 depuis le second semestre 2016. Les dépenses de R&amp;D s'élèvent à 5,7 millions, principalement issues du développement d'ABX464 (50% des dépenses) et aux investissements réalisés dans la plateforme antivirale (30%).   ACCES INDUSTRIES   Les fonds Parquest Capital, associés à CM-CIC Investissement, l'IRDI et Serge Ansaldo ont acquis, auprès notamment de Butler Capital, un bloc de contrôle représentant 91,68 % du capital et 95,03 % des droits de vote de la société Accès Industrie, au prix de 6,78 euros par action. Conformément à la réglementation applicable, les acquéreurs déposeront prochainement un projet d'offre publique d'achat simplifiée sur le solde des actions Accès Industrie à un prix identique de 6,78 euros par action, sous réserve de la décision de conformité de l'Autorité des marchés financiers.   ALBIOMA   Après le passage d'Irma, un nouvel ouragan, Maria, a durement frappé les Antilles entre le 18 et le 20 septembre 2017. Bien que la Martinique et la Guadeloupe se soient trouvées au coeur de ce phénomène climatique d'une intensité exceptionnelle, les installations d'Albioma n'ont subi que très peu de dommages, sans conséquence sensible sur leur fonctionnement. L'ouragan n'a pas eu d'impact sur le chantier de la centrale bagasse/biomasse Galion 2, assure le groupe.   AURES TECHNOLOGIES   Aures Technologies, un fabricant de matériels de gestion et d'encaissement, a publié un résultat net semestriel de 3,077 millions d'euros en hausse de 10% sur un an. Le résultat opérationnel a lui augmenté de 12,7% à 5,109 millions d'euros. Enfin le chiffre d'affaires s'est établi à 42,489 millions d'euros, soit une hausse de 11,7%.    Aurès a signé hier une lettre d'intention avec la société française CJS, spécialisée dans la réalisation de présentoirs sur mesure pour les magasins, afin de prendre, d'ici le 2 janvier 2018, une participation à hauteur de 15 % dans le capital de cette société, ceci afin de développer l'activité bornes du groupe. Cette opération serait concomitante à la création d'une structure commune (sous la forme d'une SAS) qui serait détenue majoritairement par Aurès et dédiée à l'ingénierie et à l'industrialisation des bornes.   COHERIS   Coheris a réalisé au premier semestre 2017 un résultat net de 0,35 million d'euros, en repli de 18,6% en raison d'un attentisme fort des clients et d'un effet de base défavorable. Le résultat opérationnel courant a reculé de 45% à 0,39 million tandis que le chiffre d'affaires a baissé de 8,4% à 7,07 millions. L'éditeur français de solutions CRM et analytiques a indiqué que ce premier semestre s'inscrivait dans la continuité du second semestre 2016.    CREDIT AGRICOLE   Suite au communiqué de presse du 12 septembre 2017, Crédit Agricole S.A. a annoncé la réalisation de la cession par sa filiale Crédit Agricole Corporate &amp; Investment Bank S.A. d'une participation de 16,2% dans Banque Saudi Fransi à Kingdom Holding Company. Cette transaction a obtenu toutes les autorisations réglementaires. Les termes et conditions de la transaction restent inchangés par rapport à ceux communiqués le 12 septembre 2017.    ECOSLOPS   Ecoslops, entreprise technologique innovante produisant du carburant et du bitume léger à partir de résidus d'hydrocarbures issus du transport maritime, a déposé le dossier de demande d'autorisation d'exploitation (DDAE) auprès des autorités françaises compétentes concernant la future unité de régénération de résidus maritimes de La Mède en partenariat avec le groupe Total.   EDF   Le Conseil d'administration d'EDF, réuni hier sous la présidence de Jean-Bernard Lévy, Président-Directeur Général, a décidé de coopter Maurice Gourdault-Montagne, Secrétaire général du Ministère de l'Europe et des Affaires étrangères, en qualité d'administrateur, en remplacement de Christian Masset, pour la durée du mandat restant à courir de ce dernier, soit jusqu'à l'Assemblée générale ordinaire qui statuera sur les comptes de l'exercice clos le 31 décembre 2018.    EUROPCAR   Europcar, le leader de la location de véhicules en Europe et l'un des acteurs majeurs de la mobilité, annonce la finalisation de l'acquisition du groupe Buchbinder, l'une des plus importantes sociétés de locations de véhicules en Allemagne, suite à l'accord des autorités de la concurrence.   INFOTEL   Infotel a enregistré au premier semestre une hausse de 6% de résultat net part du groupe à 7,1 millions d'euros. Le résultat opérationnel courant s'est établi à 11,2 millions d'euros (+ 8,6 %), soit une marge opérationnelle courante stable à 10,8 %. Le chiffre d'affaires spécialiste des systèmes de gestion des grands comptes, du mobile aux bases de données de très gros volume (Big Data) a atteint 103,3 millions d'euros, en croissance organique de 8,3 % sur un marché global des services informatiques en progression de 3 %.   JCDECAUX   JCDecaux, qui sera présent au Tribunal de grande instance de Nanterre le 3 octobre dans le dossier Vélib', a assuré à AOF que "le transfert automatique des contrats de travail des salariés est une évidence et devrait constituer pour Smovengo une formidable opportunité. Les salariés dédiés à l'activité Vélib' veulent poursuivre sereinement l'exercice de leur métier au bénéfice des utilisateurs. Et c'est bien légitime!". Une procédure a été lancée devant le TGI au nom du Comité d'entreprise, du CHSCT et des syndicats CGT et Sud de Cyclocity, filiale de JCDecaux employant les salariés Vélib'.   LE BELIER   Le Bélier a réalisé au premier semestre 2017 un résultat net en baisse de 45% à 7,1 millions d'euros. Le résultat opérationnel des activités est en légère hausse à 20 millions. Il fait ressortir un ratio sur chiffre d'affaires de 11,4%, contre 12,5% un an plus tôt. L'Ebitda atteint 28 millions, et dépasse son record historique sur un semestre ; son ratio sur chiffre d'affaires est de 16%.   MAUNA KEA TECHNOLOGIES   Mauna Kea Technologies a creusé ses pertes au premier semestre 2017. La medtech française e accusé sur les six premiers mois de l'année une perte nette de 5,787 millions d'euros contre une perte de 4,919 millions d'euros un an auparavant. Le résultat opérationnel a également fait état d'une perte de 5,54 millions d'euros contre 4,977 millions d'euros il y a un an. Le chiffre d'affaires, déjà publié, a reculé de 26% à 3,285 millions d'euros.   MBWS    Marie Brizard Wine &amp; Spirits a réalisé au premier semestre 2017 un résultat net part du groupe qui s'élève à 2,2 millions d'euros contre une perte de 13,5 millions au premier semestre 2016. La progression est tirée essentiellement par une plus-value liée à la vente du siège historique de Marie Brizard à Bordeaux, et par la baisse des charges financières suite au remboursement de l'intégralité de la dette gelée en 2016.   MGI COUTIER   L'équipementier automobile dévoilera ses résultats du premier semestre.   NICOX   Nicox, société internationale spécialisée en ophtalmologie, annonce aujourd'hui que sa filiale Nicox Ophthalmics a conclu un accord de licence exclusif avec Eyevance Pharmaceuticals LLC pour la commercialisation de Zerviate (solution ophtalmique de cétirizine) 0,24% aux Etats-Unis.   PLASTIC OMNIUM   Plastic Omnium a annoncé hier soir son projet de cession de sa division Environnement pour se concentrer exclusivement sur le développement mondial de ses activités automobiles. Activité historique dans le groupe Plastic Omnium, la division Environnement réalise un chiffre d'affaires de l'ordre de 330 millions d'euros, soit 4% du chiffre d'affaires de Plastic Omnium. Rentable et génératrice de trésorerie, elle est le leader européen de la conteneurisation des déchets. Elle emploie 1 800 personnes, opérant dans 12 pays.   SES-IMAGOTAG   SES-imagotag, spécialiste des solutions digitales pour le commerce physique et leader mondial des systèmes d'étiquetage électronique, a annoncé la signature d'un contrat avec les chaînes Elkjøp et Elgiganten, filiales de DixonsCarphoneNordic, pour l'installation de ses étiquettes électroniques dans leurs magasins.   TIKEHAU CAPITAL   La société de gestion détaillera ses résultats semestriels.   TOTAL   Le conseil d'administration de Total, réuni le 20 septembre 2017, a décidé la mise en distribution d'un premier acompte au titre de l'exercice 2017, d'un montant de 0,62 euro par action, conformément à la décision du conseil du 26 avril 2017, identique au solde proposé au titre de l'exercice 2016 et en hausse de 1,6% par rapport à celui du premier acompte au titre de l'exercice 2016   VALLOUREC   Vallourec, leader mondial des solutions tubulaires premium, a signé un contrat avec la compagnie pétrolière Badr El Din Petroleum Company (BAPETCO), joint-venture entre Shell et l'Egyptian General Petroleum Corporation. Cet accord porte sur la fourniture de solutions tubulaires (OCTG) pour équiper 60 à 70 puits de gaz d'un projet onshore dans le désert occidental de l'Egypte.  Cet article est paru dans Option Finance (site web)
</article>
<article titre="ALBIOMA a résisté à Maria" date="2017 09 21" journal="Agence Option Finance (AOF)">
(AOF) - Après le passage d'Irma, un nouvel ouragan, Maria, a durement frappé les Antilles entre le 18 et le 20 septembre 2017. Bien que la Martinique et la Guadeloupe se soient trouvées au coeur de ce phénomène climatique d'une intensité exceptionnelle, les installations d'Albioma n'ont subi que très peu de dommages, sans conséquence sensible sur leur fonctionnement. L'ouragan n'a pas eu d'impact sur le chantier de la centrale bagasse/biomasse Galion 2, assure le groupe.   Dans les conditions de sécurité les plus strictes, les équipes locales ont pu assurer une permanence durant la période de confinement des populations et ont permis aux trois centrales thermiques (Albioma Le Moule et Albioma Caraïbes en Guadeloupe et Albioma Galion en Martinique) de fonctionner sans interruption, contribuant significativement à la stabilité du réseau électrique des deux îles. Les centrales photovoltaïques du groupe en Guadeloupe et Martinique (14 MWc) n'ont connu que des dégâts mineurs. Certaines d'entre elles ont néanmoins interrompu leur production faute de connexion au réseau électrique, et quelques installations demeurent inaccessibles compte tenu de l'état des accès routiers. Le groupe poursuit l'inspection détaillée de ses centrales et assurera leur remise en service au fur et à mesure du rétablissement du réseau dans les jours qui viennent.   AOF - EN SAVOIR PLUS  Services aux collectivités    Sous l'effet des fusions et acquisitions, le secteur mondial des utilities est en pleine reconfiguration. Selon une étude d'AT Kearney, 1040 opérations de ce type ont été annoncées dans le monde en 2016, pour un total de 329 milliards de dollars, montant record en valeur. AT Kearney estime que le marché bénéficie d'abord des cessions d'actifs opérées par les grands énergéticiens européens.   Comme le souligne le cabinet de conseil et d'audit PwC, l'année 2016 a été un exercice record en matière de fusions-acquisitions dans l'énergie : près de 300 milliards de dollars, en hausse de 47% par rapport à 2015. Et effectivement l'Europe a joué un rôle essentiel dans cette accélération car les transactions sur ce continent ont bondi de 25%, à 49 milliards de dollars. En France, 45 opérations de fusions-acquisitions ont été recensées en 2016, pour une valeur totale de 5,4 milliards de dollars. La prise de participation minoritaire de CNP Assurances et de la Caisse des dépôts (CDC) dans Réseau de transport d'électricité (RTE), la filiale d'EDF, pour 4,6 milliards de dollars, a surpassé toutes les autres.
</article>
<article titre="Albioma: les installations ont résisté à Irma." date="2017 09 20" journal="Cercle Finance (site web)">
Albioma a publié un communiqué ce mercredi soir après le passage dévastateur de l'ouragan Irma, lequel a précédé celui d'un autre ouragan, Maria, qui a durement frappé les Antilles entre le 18 et le 20 septembre. Le groupe a tenu à faire savoir que ses installations n'ont subi que 'très peu de dommages, sans conséquence sensible sur leur fonctionnement'.   Il n'y ainsi pas eu d'impact sur le chantier de la centrale bagasse/biomasse Galion 2 et 'dans les conditions de sécurité les plus strictes, les équipes locales ont pu assurer une permanence durant la période de confinement des populations et ont permis aux 3 centrales thermiques (Albioma Le Moule et Albioma Caraïbes en Guadeloupe et Albioma Galion en Martinique) de fonctionner sans interruption, contribuant significativement à la stabilité du réseau électrique des 2 îles'. De même, les centrales photovoltaïques du groupe en Guadeloupe et Martinique (14 mégawatts crête (MWc) ) n'ont connu que des dégâts mineurs. Certaines d'entre elles ont néanmoins interrompu leur production faute de connexion au réseau électrique et quelques installations demeurent encore inaccessibles compte tenu de l'état des accès routiers.    Albioma poursuit l'inspection détaillée de ses centrales et assurera leur remise en service au fur et à mesure du rétablissement du réseau dans les jours qui viennent.   Copyright (c) 2017 CercleFinance.com. Tous droits réservés.
</article>
<article titre="Albioma : les installations peu touchées par les ouragans" date="2017 09 20" journal="Boursier (site web)">
Après le passage d'Irma et l'ouragan Maria, qui a rudement frappé les Antilles entre le 18 et le 20 septembre 2017,  Albioma  fait le point sur ses les installations. Heureusement, bien que la Martinique et la Guadeloupe se soient trouvées au coeur de ce phénomène climatique d'une intensité exceptionnelle, les installations du groupe n'ont subi que très peu de dommages. Ils sont sans conséquence sensible sur leur fonctionnement. "L'ouragan n'a pas eu d'impact sur le chantier de la centrale bagasse/biomasse Galion 2", explique Albioma.   Dans les conditions de sécurité les plus strictes, les équipes locales ont pu assurer une permanence durant la période de confinement des populations et ont permis aux trois centrales thermiques (Albioma Le Moule et Albioma Caraïbes en Guadeloupe et Albioma Galion en Martinique) de fonctionner sans interruption, contribuant significativement à la stabilité du réseau électrique des deux îles. Les centrales photovoltaïques du Groupe en Guadeloupe et Martinique (14 MWc) n'ont connu que des dégâts mineurs. Certaines d'entre elles ont néanmoins interrompu leur production faute de connexion au réseau électrique, et quelques installations demeurent inaccessibles compte tenu de l'état des accès routiers.    Le Groupe poursuit l'inspection détaillée de ses centrales et assurera leur remise en service au fur et à mesure du rétablissement du réseau dans les prochains jours.
</article>
<article titre="Maurice : Omnicane produira de l'électricité à partir du biogaz issu de la canne à sucre" date="2017 08 12" journal="La Tribune Afrique (site web)">
La part d'énergie renouvelable dans le mix énergétique de Maurice sera bientôt augmentée à partir d'un nouveau projet. Connu pour ses initiatives novatrices, le groupe Omnicane, spécialisé dans la canne à sucre et l'électricité, vient de lancer un projet de production d'électricité à partir de biogaz issu de la canne à sucre et de biométhanisation de la vinasse.   Pour l'entreprise sucrière, ce nouveau projet vient augmenter la part des énergies renouvelables dans la politique de mix énergétique mise sur pied par le gouvernement mauricien. Lancé ce jeudi 10 août à la distillerie de La Baraque au sud de l'île, le projet présente un double enjeu : fournir davantage d'électricité au pays et mettre à la disposition des agriculteurs des engrais de bonne qualité, tout en contribuant à la protection de l'environnement. La production de l'électricité à partir de la canne à sucre permet de réduire l'impact des installations industrielles d'Omnicane en valorisant les produits dérivés de ses activités. Il s'agit d'une initiative en cohérence avec la politique mauricienne de valorisation des produits dérivés de la canne à sucre ainsi que de promotion du développement durable, indique-t-on chez Omnicane.   En effet, en broyant la canne à sucre pour en extraire le sucre, le reste, la bagasse et la vinasse forment des déchets organiques qui désormais feront l'objet d'une valorisation énergétique. Habituellement, la distillerie de Omnicane fermente la mélasse produite par la raffinerie de sucre et distille le liquide fermenté pour le transformer en bioéthanol. Celui-ci est alors utilisé dans l'agro-industrie, dans le secteur pharmaceutique ou encore comme carburant. Quant à la vinasse issue de la mélasse, elle était jusqu'alors uniquement concentrée à travers des évaporateurs pour produire un bio fertilisant.   43% de la production énergétique de l'île Le projet de production de l'énergie électrique à travers les déchets carboniques de la canne à sucre est une innovation. Côté processus, Omnicane produit, à partir de la vinasse, un engrais de meilleure qualité et aussi de l'électricité à partir du biogaz émis lors de la méthanisation. Un processus biologique de dégradation de la vinasse en l'absence d'oxygène.   Omnicane n'est pas à ses débuts. Producteur du tiers de la canne à sucre de Maurice, le groupe produit aussi 43% de l'électricité de l'île et emploie près de 2 000 personnes. Avec ce nouveau projet de production d'électricité, Omnicane décide désormais de l'avenir de l'industrie cannière et de la valorisation de ses dérivés.  Cet article est paru dans La Tribune Afrique (site web)
</article>
<article titre="Afrique en marche" date="2017 08 11" journal="Xinhua - Agence de presse">
BEIJING, 11 août (Xinhua) -- Voici notre rubrique "Afrique en marche" :    - - -    La Guinée-Bissau et l'Indonésie signent un accord de coopération dans le secteur de la défense    BISSAU -- Les ministres de la Défense de la Guinée-Bissau, Eduardo Costa Sanha, et de l'Indonésie, Ryamizard Ryacudu, ont signé une lettre d'intention pour établir des lignes de coopération dans le secteur de la défense, a annoncé, ce jeudi, dans un communiqué, le ministère bissau-guinéen de la Défense. L'accord a été signé à l'occasion de la visite à Bissau du ministre indonésien de la Défense, a précisé le communiqué. Ryamizard Ryacudu a affirmé que Jakarta veut contribuer à transformer les forces armées bissau-guinéennes en "une puissante armée en Afrique". De son côté, le ministre de la Défense de la Guinée-Bissau a indiqué qu'une charte a été signée et qu'elle sera bientôt transformée en "accords concrets dans le domaine militaire", y compris le soutien à la formation et à l'équipement pour le contrôle des espaces maritimes et aériens bissau-guinéens.  - - -    Guinée : atelier sous-régional pour l'intégration numérique dans l'espace CEDEAO    CONAKRY -- Les représentants de plusieurs pays de la sous-région ouest-africaine sont réunis à Conakry pour réfléchir sur les possibilités de mise en place d'une intégration numérique dans l'espace CEDEAO (Communauté économique des pays de l'Afrique de l'Ouest). Les délégués venus du Burkina Fasso, de la Côte D'Ivoire, du Sénégal, du Mali, du Togo, de la Serra Léone et de la Guinée doivent se concerter pendant deux jours sur l'ensemble des paramètres pouvant aboutir à la création d'un marché intégré des télécommunications. Entre la Guinée et le Sénégal, cette volonté des autorités politiques a atteint des résultats encourageants et probants, car on enregistre de nos jours plus de 60% de réduction des coûts téléphoniques à l'émission et une réduction de 100% à la réception entre les deux pays, a dit le directeur de l'autorité guinéenne de régulation des postes et télécoms Atigou Chérif.    - - -    Maurice : lancement d'un projet de production d'électricité à partir de biogaz issu de la canne à sucre    PORT-LOUIS -- Un projet de bio-méthanisation de la vinasse et de production d'électricité à partir de biogaz de l'entreprise sucrière mauricienne Omnicane a été présenté à la distillerie de La Baraque (sud de l'île), a indiqué jeudi dans un communiqué d'Omnicane. En effet, ce nouveau projet d'Omnicane qui augmente la part d'énergie renouvelable dans le mix énergétique du pays, permet également de réduire l'impact environnemental de l'industrie cannière à travers une valorisation des produits dérivés. Concrètement, une fois la canne broyée pour en extraire le sucre, les déchets organiques, la bagasse et la vinasse, font l'objet d'une valorisation énergétique. La distillerie d'Omnicane fermente actuellement la mélasse produite par la raffinerie de sucre et distille le liquide fermenté pour le transformer en bioéthanol, notamment utilisé dans l'agro-industrie, le secteur pharmaceutique ou comme carburant. La vinasse, issue de la mélasse, est à ce jour seulement concentrée à travers des évaporateurs pour produire un bio fertilisant.    - - -    Le Soudan et la Jordanie signent 10 accords de coopération    KHARTOUM -- Jeudi, le Soudan et la Jordanie ont signé 10 accords de coopération dans divers domaines. Les accords ont été conclus jeudi à Khartoum lors de la septième session du Comité mixte Soudan-Jordanie. La réunion a été présidée par le premier vice-président et Premier ministre du Soudan, Bakri Hassan Saleh, et le Premier ministre jordanien Hani Al-Mulki. S'exprimant lors d'une conférence de presse conjointe à l'issue de la réunion, M. Saleh s'est engagé à trouver les mécanismes appropriés pour mettre en oeuvre les accords signés par les deux pays. Les économies des deux pays sont complémentaires, mais malgré ces potentialités, la coopération entre le Soudan et la Jordanie reste insuffisante, a indiqué M. Saleh. Al-Mulki a, pour sa part, réitéré l'engagement de son pays à mettre en oeuvre les accords signés. "A cette époque, nous devons utiliser les privilèges relatifs des économies de nos deux pays afin qu'il y ait une intégration et une mise en oeuvre claires des accords précédents", a déclaré Al-Mulki.
</article>
<article titre="Maurice : lancement d'un projet de production d'électricité à partir de biogaz issu de la canne à sucre" date="2017 08 10" journal="Xinhua - Agence de presse">
PORT-LOUIS, 10 août (Xinhua) -- Un projet de bio-méthanisation de la vinasse et de production d'électricité à partir de biogaz de l'entreprise sucrière mauricienne Omnicane a été présenté à la distillerie de La Baraque (sud de l'île), a indiqué jeudi dans un communiqué d'Omnicane.    En effet, ce nouveau projet d'Omnicane qui augmente la part d'énergie renouvelable dans le mix énergétique du pays, permet également de réduire l'impact environnemental de l'industrie cannière à travers une valorisation des produits dérivés.   Concrètement, une fois la canne broyée pour en extraire le sucre, les déchets organiques, la bagasse et la vinasse, font l'objet d'une valorisation énergétique. La distillerie d'Omnicane fermente actuellement la mélasse produite par la raffinerie de sucre et distille le liquide fermenté pour le transformer en bioéthanol, notamment utilisé dans l'agro-industrie, le secteur pharmaceutique ou comme carburant. La vinasse, issue de la mélasse, est à ce jour seulement concentrée à travers des évaporateurs pour produire un bio fertilisant.    L'innovation du projet d'Omnicane consiste en la production, grâce à la vinasse, d'un engrais de meilleure qualité, mais aussi d'électricité à partir du gaz émis par le processus de méthanisation.    Ce projet est en cohérence avec la politique mauricienne de valorisation des produits dérivés de la canne à sucre ainsi que de promotion du développement durable, selon la société. Fin
</article>
<article titre="Du plastique écologique" date="2017 06 14" journal="Sud Ouest  Charente Maritime">
Nicolas Moufflet se présente comme un " professionnel de la bouteille en plastique ". À la tête de Lyspackaging, une toute jeune entreprise implantée à Saintes depuis janvier 2015, il conçoit et produit des flacons, des carafes, des flasques et autres bouteilles à la demande. Une production sur-mesure fabriquée à base de... pétrole. Pour changer la donne, le Saintais a décidé d'innover. Nicolas Moufflet a mis au point un plastique composé uniquement de déchets de canne à sucre !   Transformés en granulés, ces déchets de canne - appelés bagasse - permettent de produire un plastique 100 % bio-dégradable. La recette est tenue secrète. Mais Nicolas Moufflet l'assure, ces granulés ne contiennent " aucune molécule pétrochimique. À l'intérieur, ce n'est que du végétal, entièrement compostable et dégradable. Nous sommes les seuls à le faire sur la planète ", assure le patron de Lyspackaging. Aux noyaux d'olives   Recyclables à l'infini - ce qui n'est pas le cas des plastiques traditionnels-, ces bouteilles en canne à sucre coûtent 30 % plus cher que celles issues du pétrole. Mais ce n'est qu'un début. Nicolas Moufflet travaille désormais sur l'ajout d'autres produits naturels à ces plastiques. Il propose ainsi des modèles contenant 30 % de noyaux d'olives. " Nous avons passé en revue tout un tas de déchets : orge, blé, carotte, pépins de raisin... " Cela permet de " valoriser les déchets " tout en colorant naturellement les plastiques. Le succès de Lyspackaging, lui, ne s'est pas fait attendre. " Nous avons triplé nos heures de production. Nous avons les commandes ", souffle le quadragénaire qui devrait doubler cette année son chiffre d'affaires. Pour l'heure, Lyspackaging continue à produire des bouteilles en plastique à base de pétrole. Une position qui permet à l'entreprise de financer ses innovations et son développement. Mais Nicolas Moufflet compte bien basculer très rapidement sur une production respectueuse de l'environnement. " Nous avons démontré que tout était possible. Il n'y a plus qu'à le faire ! "   F. P.
</article>
<article titre="La Réunion: affrontements sur le prix de la canne" date="2017 06 9" journal="Le Figaro.fr">
Planteurs et industriels de La Réunion s'affrontent depuis plus de deux semaines sur le prix de la tonne de canne, ce qui a conduit à des opérations de blocage dans l'île et poussé l'Etat à jouer les médiateurs dans ce conflit. La canne est actuellement payée 39,09 euros la tonne.Depuis le 24 mai, les trois syndicats (la FDSEA, les JA et la CGPER) réclament 6 euros de revalorisation, mais le groupe Tereos, qui regroupe les industriels de la production du sucre sur l'île, n'a accordé qu'une hausse de 49 centimes. Les différents rencontres houleuses entre les deux parties ont conduit à plusieurs actions coups de poing. Les planteurs ont bloqué l'accès aux deux usines sucrières, situées dans le sud et l'est du département. Des opérations escargot ainsi que des barrages filtrants de la circulation ont aussi été menés. Une sonde servant à mesurer la richesse de la canne a été incendiée sur une des usines. Le centre technique interprofessionnel de la canne et du sucre de La Réunion a porté plainte et une enquête a été ouverte. Depuis jeudi, l'État joue le rôle de médiateur. Les trois syndicats se sont unis pour formuler une proposition de sortie de crise, remise au secrétaire général de la préfecture, qui l'a transmise aux industriels. Cette entente entre les organisations syndicales est "historique", a souligné Frédéric Vienne, président de la FDSEA. "Le coût total de nos demandes s'élève à 14 millions d'euros, Tereos pourra se positionner sur cette somme", a énoncé le syndicaliste après la réunion. Cette somme comprend la revalorisation du prix de la canne et le reversement total de la recette bagasse (un sous-produit de la canne servant de combustible) aux planteurs. Selon Frédéric Vienne, pour le moment, les industriels n'ont mis que "800.000 euros sur la table".Une trêve des blocages et barrages a été décidée par les syndicalistes : jeudi, une vingtaine de camions transportant du charbon a pu livrer les centrales thermiques, productrices d'électricité.Le groupe sucrier Tereos estime que les montants demandés par les agriculteurs sont "totalement disproportionnés au regard de la réalité économique de la filière". Dans un communiqué, les industriels ont indiqué jeudi que "dès la semaine prochaine, l'évolution possible du prix de la canne en fonction du prix du sucre blanc fera partie des échanges". La campagne sucrière doit démarrer fin juin, mais vendredi, aucune date du début de coupe des cannes n'avait encore été donnée. À La Réunion, la filière canne-sucre est le premier secteur exportateur, selon la dernière étude de l'Institut d'émission des départements d'outre-mer.
</article>
<article titre="LES PETITES & GRANDES INNOVATIONS QUI ALNNONCENT 'AVENIR Du distributeur automatique de voitures au robot-maçon qui pose 1 000 briques à l'heure" date="2017 06 9" journal="La Tribune Hebdomadaire, no. N° 214">
1 Contre l'infertilité, des ovaires imprimés en 3D CHICAGO - États-Unis Médecine.  Et si les femmes stériles pouvaient enfanter quand même ? Des chercheurs de l'Université de Northwestern viennent d'imprimer en 3D des ovaires qui ont permis à des souris de donner naissance à des bébés en parfaite santé. Les scientifiques ont utilisé une encre gélatineuse pour imprimer un ovaire artificiel, dans lequel ont été placés des follicules ovariens. Cet ovaire synthétique a ensuite été implanté chez des souris n'ayant pas d'ovaires. Avec succès. Reste désormais le plus difficile : reproduire la performance, mais pour des femmes. Cette avancée majeure pourrait donner un espoir aux 80 millions de couples souffrant d'infertilité.   2 Le nid-de-poule qui tweete pour alerter les pouvoirs publics PANAMA CITY - Panama Activisme 2.0.  Twitter, le média du temps réel, est aussi un moyen de pression pour les citoyens. La preuve au Panama, où une émission d'actualité et l'agence P4 Ogilvy ont lancé le compte « El hueco Twitero ». Grâce à des capteurs placés sur les trous de la chaussée, le passage de chaque voiture sur un nid-de-poule déclenche la publication d'un tweet. L'objectif : protester contre le piteux état du réseau routier du pays et encourager les pouvoirs publics à réagir. L'opération a porté ses fruits : le ministre des Transports a dû s'expliquer devant les téléspectateurs.   3 L'ordinateur qui aide la police à prévoir le risque de récidive DURHAM - Royaume-Uni Intelligence artificielle.  À la rentrée prochaine, les policiers de Durham auront un nouveau collègue : Hart, une intelligence artificielle d'aide à la décision. Présenté comme un « outil d'évaluation des risques de préjudice », Hart va donner son avis sur la remise en liberté, ou non, d'un suspect arrêté, en moulinant un ensemble de données récoltées de 2008 à 2012 et traitées par un algorithme. La machine classera les suspects en trois catégories : risque faible, moyen ou important de récidive. Testé en 2013, Hart a correctement évalué ces risques : 98 % des individus classés dans la catégorie « risque faible » n'ont pas violé la loi dans l'attente de leur procès, tandis que 88 % de ceux classés en « risque élevé » l'ont fait. Les policiers seront libres de suivre ou d'ignorer les recommandations de Hart.   4 Bientôt des pistes d'atterrissage circulaires ? NOORDWIJK - Pays-Bas Aéronautique.  Depuis deux ans, le centre technique de l'Agence spatiale européenne étudie une nouvelle organisation des aéroports, autour d'une immense piste d'atterrissage circulaire. Le principe ? Faciliter le travail des pilotes, qui pourraient ainsi toujours décoller et atterrir sans être gênés par le vent. D'une circonférence de plus de 9 kilomètres et d'une largeur de 140 mètres, cette piste permettrait de réaliser plusieurs décollages et atterrissages en simultané. L'aéroport serait positionné au centre de la piste, ce qui permettrait une optimisation des mouvements sur la plateforme, un gain de temps et de carburant. Mais si l'Agence spatiale européenne a sérieusement étudié ce concept, sa mise en place lors de la construction de nouveaux aéroports reste très hypothétique, notamment en raison des contraintes liées au contrôle aérien.   5 Grovio, l'assistant qui arrose les plantes KIEV - Ukraine Domotique.  Entre les oublis liés au travail, aux vacances et au stress de la vie quotidienne, les plantes ne sont pas toujours entretenues de manière optimale. Dans la maison connectée et intelligente de demain, ce ne sera plus un problème. Grovio est un objet sans fil design, une petite tour sombre et minimaliste, discrète mais efficace. Des tuyaux la relient à trois de vos plantes. Il suffit de la remplir d'eau, d'indiquer les caractéristiques de vos végétaux, et le système les arrosera pour vous au bon moment, tout en vous rappelant de remettre de l'eau de temps en temps pour alimenter son réservoir. La startup Techware, qui a inventé le produit, vient de lancer une campagne de financement participatif sur Kickstarter, et compte débuter la commercialisation en décembre.   6 Quand des déchets de graines de café font tourner les usines NAIROBI - Kenya Environnement.  Jusqu'à présent, les coques impropres à la consommation qui enveloppent les graines de café étaient jetées. Au Kenya, l'entreprise Lean Energy Solutions a décidé de les récupérer pour créer des rondins écologiques vendus aux usines environnantes. La société récupère les coques puis les fait sécher au soleil. Elles sont ensuite mélangées à d'autres déchets végétaux récupérés par l'entreprise : sciure de bois, résidus de charbon, bagasse de canne à sucre. Le mélange produit des rondins à fort pouvoir calorifique, qui sont livrés aux usines de ses clients, dont le géant de l'agroalimentaire Unilever, ou encore Coca-Cola et Pepsi. Chaque jour, l'entreprise transforme 80 tonnes de débris en 40 tonnes de rondins. Elle emploie 400 personnes dans le pays, dont une majorité de femmes.   7 25 % de policiers-robots dans les rues d'ici à 2030 DUBAI - Émirats arabes unis Sécurité.  Ils mesurent presque 1,70 mètre et sont équipés d'une tablette tactile sur le torse. Ces robots-policiers, conçus par l'entreprise espagnole PAL Robotics, devraient représenter 25 % des effectifs de police de Dubaï en 2030. Tel est, en tout cas, l'objectif de la ville. Ces robots n'auront pas pour mission de résoudre des crimes ni de surveiller la circulation, mais de faciliter les démarches des citoyens. Disséminés dans la ville, ils permettront de signaler un incident, de payer une amende ou encore de porter plainte, pour permettre aux agents humains de se concentrer sur d'autres tâches.   8 Des voitures de luxe dans un distributeur automatique SINGAPOUR Commerce.  Acheter une voiture comme une barre chocolatée, c'est désormais possible. À Singapour, pour remédier au manque de place dans la ville, une enseigne de vente de voitures d'occasion a trouvé la parade : l'empilement vertical. Les voitures sont entreposées dans quatre colonnes vitrées de quinze étages, que le consommateur peut voir depuis la rue. Dans le magasin, le client peut demander une Ferrari, une Maserati ou une Lamborghini, la commander en appuyant sur un bouton, et regarder un film présentant les caractéristiques du véhicule en attendant que celui-ci descende par ascenseur. Grâce à ce procédé, la société assure que ses ventes ont augmenté de 30 % depuis décembre dernier.   9 Record : la Chine a fait danser 1 000 drones lumineux CANTON - Chine Informatique.  La performance est entrée dans le  Guiness book des records.  Pour le Nouvel An chinois, en février dernier, la Chine a remplacé les traditionnels lancers de ballons en papier dans le ciel par 1 000 drones lumineux. Lâchés en même temps, les engins ont suivi une chorégraphie définie à l'avance et contrôlée par un ordinateur central. Sans accident. Le précédent record datait du dernier Superbowl américain, quand la société Intel avait fait voler 5 00 drones lumineux pendant la performance de Lady Gaga.   10 Hadrian X, le premier robot-maçon poseur de briques PERTH - Australie Robotique.  La révolution de la maçonnerie porte un nom : Hadrian X. Créé par l'entreprise FastBrick, ce bras robotique long de 30 mètres, mobile puisque monté sur camion, peut poser automatiquement 1 000 briques par heure, à un rythme quasi ininterrompu, 24 heures sur 24. Son guidage laser lui indique où poser les briques et comment les empiler, le rendant très autonome. Chaque brique est enduite de glu ou de mortier colle. Le robot ménage également les espaces nécessaires pour que des ouvriers humains ajoutent par la suite les portes et les fenêtres. Malgré son coût - 2 millions de dollars l'unité -, le gain de productivité apparaît énorme pour les entreprises qui en feront l'acquisition à partir de 2019, date de la commercialisation à grande échelle.   @SylvRolland   PLUS D'INNOVATIONS SUR LATRIBUNE.FR
</article>
<article titre="Conseil Albioma" date="2017 06 3" journal="Investir-Le Journal des finances, no. 2265">
Le producteur d'électricité, installé outre-mer, a fait part d'une bonne nouvelle lors de son assemblée générale : il a signé lundi un avenant avec EDF fixant à 22 millions d'euros la rémunération des investissements nécessités par la mise en conformité du système d'extraction des fumées de sa centrale Albioma Caraïbes, en Guadeloupe.   Par ailleurs, conformément à la loi sur la transition énergétique, qui vise à augmenter de 50 % la capacité installée des énergies renouvelables d'ici à 2023, Albioma souhaite améliorer la part de celles-ci dans son mix énergétique, à un niveau de 80 % (contre 50 % actuellement). « Objectif très ambitieux, mais nous souhaitons nous y tenir »,  a déclaré Frédéric Moyne, le directeur général d'Albioma, à ses actionnaires. Le groupe compte aussi sur de nouveaux projets dans le solaire et sur la mise en service, prévue en fin d'année, de sa centrale thermique Galion 2 en Martinique, unité 100 % biomasse, pour atteindre ses objectifs. Cette unité doit contribuer à hauteur de 20 millions d'euros à l'Ebitda du groupe.   Serein au brésil Albioma, qui produit de l'énergie à partir de la bagasse, résidu fibreux de la canne à sucre, a poursuivi ses acquisitions au Brésil, premier producteur mondial de sucre de canne, en y signant un troisième projet, dans l'Etat de São Paulo. Le groupe a vu le montant de son bénéfice d'exploitation bondir de plus de 50 % dans ce pays, qui devrait concentrer 40 % des investissements sur la période 2013-2023. Concernant les tensions politiques récentes, « Albioma n'est pas impacté au quotidien puisque la plupart des contrats sont signés »,  a indiqué Frédéric Moyne.   Notre conseil Acheter Le groupe a confirmé ses objectifs pour 2017 et envisage de multiplier par deux son bénéfice net à l'horizon 2023 grâce à la mise en service de nouvelles centrales.   Par Adrien Roure
</article>
<article titre="Révision du système de paiement de la canne à sucre en Guadeloupe" date="2017 06 2" journal="Rapports - Agroalimentaire - MAAF">
51 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... établi par  Thierry Berlizot Ingénieur général des ponts, des eaux et des forêts  avril 2017 SOMMAIRE RÉSUMÉ........................................................................................................................5 LISTE CHRONOLOGIQUE DES RECOMMANDATIONS......................................................................6 1. LA FORMULE DE FIXATION DU PRIX DE...    Pour voir le rapport :  http://agriculture.gouv.fr/sites/minagri/files/cgaaer_16058_2017_rapport.pdf
</article>
<article titre="Révision du système de paiement de la canne à sucre en Guadeloupe" date="2017 06 2" journal="Rapports - Agroalimentaire - MAAF">
51 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... Rapport n° 16058  Révision du système de paiement de la canne à sucre en Guadeloupe  établi par  Thierry Berlizot Ingénieur général des ponts, des eaux et des forêts  avril 2017 SOMMAIRE...    Pour voir le rapport :  http://agriculture.gouv.fr/telecharger/86600?token=5e970caac4cf00c975ebcd584bf0a465
</article>
<article titre="Le béton écolo à l'essai" date="2017 05 30" journal="L'Union (France)  Chalons">
Rendez-vous au carrefour de Mazagran. Hubert Harpillon, agriculteur à Saint-Etienne-à-Arnes, «le premier village des Ardennes en venant de Sommepy», roule vers ses champs de miscanthus. Rien à voir avec les roseaux de Chine qui décorent les jardins. Celui-là, c'est un costaud: la variété giganteus, celle dont se régalent les éléphants, d'où son surnom d'herbe à éléphant. Elle a fait son apparition dans la moitié sud des Ardennes et la moitié nord de l'Aisne sous l'impulsion de la coopérative Luzéal (lire par ailleurs).    Livraison de mauvais rhizomes  Pas de chance pour notre producteur: il a dû replanter une deuxième fois sa parcelle de 4,30hectares «à cause d'une livraison de mauvais rhizomes en 2009. Ils arrivent d'Angleterre et de Bretagne et sont mis au frigo. Les premiers que j'ai reçus ont eu des difficultés à germer». La seconde tentative en 2010 est concluante. «À l'époque, le contrat d'implantation avec Luzéal donnait un droit d'agrandissement. En 2012, je suis arrivé à 5,72 hectares.» Il a choisi une terre qui n'offre pas assez de portance pour les engins agricoles. «Il y a trente ans, c'était une prairie. Quand j'ai arrêté l'élevage, j'ai retourné la pâture pour faire du blé. Les rendements allaient bien mais je plantais le tracteur dans certains endroits inondés, même en juillet avec la moissonneuse.» Plus aucun problème avec le roseau de Chine. «Et ce qui me semblait intéressant, c'est que cette culture est une faible consommatrice en intrants. Les feuilles poussent et quand elles tombent en séchant, en janvier février sous l'effet du gel, elles redonnent tous les minéraux à la terre. C'est la plante du fainéant!» Ce n'est pas une source importante de revenus... La plus faible marge de toute mon exploitation Hubert Harpillon, producteur Hormis nettoyer les bords de ses champs, il n'intervient pas. Luzéal s'occupe de faucher les tiges séchées sur pied. Sans rien faire, alors qu'il a commencé «avec un reliquat azoté de 40 unités», la terre s'est enrichie: 80 kg/ha. Pas de travail, une plante qui nourrit la terre, que demander de plus? «Ce n'est pas une source importante de revenus... et depuis trois ans, avec le printemps froid et sec, la plante se stabilise à 90 cm de haut.» Loin des 4,80 mètres atteints précédemment. C'est autant de biomasse en moins, car seule «la bagasse» est récoltée. Alors quand la tige ne s'élève pas... Costaud mais grand buveur, le miscanthus giganteus requiert 650 mm d'eau par an et de la luminosité. De 3 à 6tonnes par hectare lors de la première récolte en 2011, avec un maximum à 13,5 tonnes de moyenne, contre 15 à 17 tonnes attendues. Combien est rémunérée la tonne? «55 à 60 centimes. C'est la plus faible marge de mon exploitation.»  Trop faible rentabilité  Hubert Harpillon a vite fait les calculs: «En termes de marge, je suis légèrement positif. Je couvre le prêt mais cela ne donne pas de quoi vivre pour une exploitation.» À moins... «de trouver une rentabilité plus importante». La luzerne, également récoltée par Luzéal, s'avère encore «légèrement plus intéressante, avec 12 à 13tonnes à l'hectare, rémunérée de 65 à 80 centimes la tonne.» L'équation est simple pour Hubert Harpillon: «J'ai besoin de vivre. Ce sont des surfaces productives. Si je n'ai pas de rentabilité...» D'aucuns ont déjà fait marche arrière: «Ils ont tout broyé en retournant leurs champs.» En est-il arrivé à ce point? «J'ai besoin que la culture du miscanthus me procure un revenu décent.» Surtout, l'investissement est trop conséquent: 3000 € l'hectare. «Et à l'époque, nous avions eu des aides!» Celui qui a fait figure de pionnier est-il prêt à planter davantage si la rémunération est plus conséquente? Hubert Harpillon réfléchit, un temps, et sa réponse tombe: «Je ne pense pas.» Les porteurs du projet de filière dans l'économie verte doivent trouver au plus vite une solution, sous peine de voir «l'amont» ne jamais se développer, voire régresser.Nancy Gouin
</article>
<article titre="Deuxième semaine Une France qui se réinvente (1/4) L'outre-mer, laboratoire des énergies renouvelables Cap aujourd'hui sur l'outre-mer, où le vent, le soleil et la houle sont autant d'atouts pour développer les énergies propres de demain." date="2017 05 22" journal="La Croix, no. 40804">
Des parcs solaires qui voient le jour au milieu des palmiers, des installations marines pionnières testées au creux des océans ou bien encore des montagnes de bagasse - résidus de canne à sucre - venant alimenter des centrales thermiques... En outre-mer, le tournant des énergies renouvelables (EnR) est pris. Et le constat dressé dans un rapport de 2011 par le Conseil économique, social et environnemental (CESE) est plus que jamais d'actualité: « L'outre-mer doit être, désormais, clairement considéré comme un véritable laboratoire. »Alors que l'urgence climatique impose une transition volontariste vers des énergies propres, la France peut s'appuyer sur ces territoires répartis sur trois continents - l'Afrique, l'Amérique et l'Océanie - et qui lui permettent de posséder la deuxième plus grande surface maritime du monde.Dans le domaine de l'énergie, l'outre-mer offre deux atouts majeurs. Tout d'abord, les ressources, « variées et abondantes », rappelle ainsi Jérôme Billerey, président de la commission des régions ultramarines au Syndicat des énergies renouvelables. « Entre le soleil, le vent, la biomasse, les mers, les fleuves et le potentiel géothermique, on couvre toute la filière. » Soit tout le spectre des énergies, des « plus "traditionnelles" - hydroélectricité, biomasse - aux plus futuristes comme celles issues de la mer dont il conviendrait d'accélérer le développement industriel », complète le CESE.Ensuite, un environnement très spécifique. Parce qu'ils sont isolés, non interconnectés avec d'autres régions pouvant assurer leur approvisionnement, les territoires d'outre-mer doivent miser sur une production locale, sauf à rester dépendants d'une énergie importée et coûteuse.Cet impératif est un puissant moteur d'innovation et de développement, notamment dans des îles qui n'ont guère le choix. D'autant que la loi de transition énergétique de 2015 fixe des objectifs ambitieux: les EnR devront représenter 50 % de la production énergétique de ces territoires dès 2020, pour une autonomie complète en 2030. À l'heure actuelle, les renouvelables représentent 27 % de la production électrique des Drom (départements et régions d'outre-mer) - avec de fortes disparités d'un territoire à l'autre: 62 % en Guyane (en raison d'une forte production hydraulique), 36 % à La Réunion mais seulement 7 % en Martinique.« En outre-mer, le gros potentiel se trouve aujourd'hui dans le solaire photovoltaïque et thermique, la biomasse et la géothermie », souligne Raphaël Gerson, chef de service adjoint à l'Ademe, l'Agence de l'environnement et de la maîtrise de l'énergie, prenant pour exemple la centrale géothermique de Bouillante, en Guadeloupe. Cette dernière alimente 23 000 habitants de l'archipel en électricité. De son côté, la Martinique s'est dotée de parcs solaires, comme ceux du Lorrain et de Morne-Etoile (Saint-Pierre), qui, à eux deux, fournissent du courant à plus de 5 000 personnes.Pour l'avenir, les énergies marines renouvelables suscitent elles aussi de grands espoirs, d'autant que la France, outre l'étendue de son domaine maritime, « combine trois grands atouts pour innover: sa filière scientifique, ses fleurons industriels comme DCNS ou STX, et ses petites start-up très performantes », fait remarquer Raphaël Gerson, à l'Ademe. Là encore, le potentiel est vaste: énergie houlomotrice (des vagues), hydrolienne (des courants), éolien flottant mais aussi énergie thermique des mers, qui consiste à exploiter la différence de température entre l'eau de surface et celle des profondeurs. « Cette source d'énergie est prometteuse en outre-mer, où les îles sont proches de tombants, rappelle Jérôme Billerey, du syndicat SER. Dans certaines zones, à un kilomètre des côtes, le fond descend à 1 000 mètres. » En Martinique, le démonstrateur « Nemo » préfigure ainsi une centrale de production thermique des mers, pouvant alimenter 35 000 foyers.« Il faut encore traduire ce potentiel en une filière pérenne et compétitive », tempère Raphaël Gerson. Tout d'abord, en surmontant les obstacles techniques, nombreux dans un milieu, la mer, où les installations sont soumises à rude épreuve (corrosion, tempêtes, etc.); ensuite, en parvenant à des baisses de coûts substantielles. Pour Jérôme Billerey, la compétitivité économique reste le grand défi. « L'énergie thermique marine restera chère un bon moment, prévoit ce dernier, également directeur général de l'entreprise Quadran. Ce n'est pas cette filière qui permettra d'atteindre les objectifs de la loi de transition. »Selon lui, le calendrier reste de toute façon bien trop serré, et il dénonce tout à la fois un manque de volonté politique, des freins administratifs et fiscaux et la limite des 30 % d'énergies renouvelables imposée pour préserver la qualité de service - dans ces petits territoires, les réseaux supportent mal l'intermittence.Sans se prononcer sur la pertinence de cette limite, l'Ademe y voit plutôt une occasion supplémentaire d'être à l'avant-garde. Cette fois en misant sur le développement de solutions de stockage et sur l'autoconsommation, qui permet de contourner le problème des réseaux. « Dans ces domaines aussi, indique ainsi Raphaël Gerson, les territoires ultramarins peuvent être à la pointe de la transition énergétique. » Et, à terme, exporter les technologies acquises dans ce laboratoire des renouvelables.
</article>
<article titre="Gilbert Dupont vise toujours 46 euros" date="2017 05 15" journal="Trading Sat (site web)">
Global Bioenergies grimpait de plus de 3% lundi après-midi (+3,2% à 21,3 euros à 16 heures), alors que le courtier Gilbert Dupont vient de réitérer sa recommandation "Acheter" et son objectif de cours de 46 euros sur la société de chimie verte.    Celle-ci a inauguré jeudi dernier son démonstrateur de Leuna en Allemagne, conçu pour la fermentation directe des hydrocarbures gazeux, qui permettra de produire de l'isobutène de haute pureté à partir de diverses ressources (sucre industriel dérivé de la betterave et de la canne à sucre, sirop de glucose dérivé des céréales, sucres de deuxième génération extraits de paille de blé, de bagasse, de copeaux de bois...). A cette occasion, Global Bioenergies "a confirmé avoir obtenu des performances sur son démonstrateur supérieures à celles obtenues sur le pilote de Pomacle avec notamment l'atteinte d'un rendement allant jusqu'à 90% de la cible commerciale", rapporte Gilbert Dupont.
</article>
<article titre="Programmation pluriannuelle de l'Energie pour la Guadeloupe" date="2017 05 12" journal="Rapports - Environnement - MEDDTL">
48 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... 1. Avis du CSE  2. Avis du CNTE  3. Avis du CETE  4. Avis du public  5. Avis de l'AE  Délibération n° 2017-02 : avis du CNTE sur le projet...    Pour voir le rapport :  http://www.developpement-durable.gouv.fr/sites/default/files/Avis%20du%20CNTE%2C%20du%20CSE%2C%20du%20CETE%2C%20du%20public%20et%20de%20l%27Autorit%C3%A9%20environnementale%20sur%20la%20PPE%20de%20la%20Guadeloupe.pdf
</article>
<article titre="Comment Bpifrance finance la transition énergétique" date="2017 05 3" journal="La Tribune (site web)">
Il y a quelques jours, Bpifrance annonçait une prise de participation dans la société Albioma (ex. Séchilienne Sidec). Spécialiste des centrales à co-génération (électricité et chaleur) implantées dans les Dom-Tom, l'entreprise française est en train d'opérer sa propre transition énergétique. En effet, alors qu'historiquement elle remplace la bagasse (co-produit de la canne à sucre) par du charbon en dehors des périodes de production, elle s'emploie depuis plusieurs années déjà à substituer progressivement à ce dernier d'autres déchets végétaux.   En juin 2015, le fonds d'investissement Apax Partners a distribué à ses actionnaires 95% des parts qu'il détenait dans Albioma (soit environ 30% du capital). Le semaine dernière, Bpifrance a annoncé avoir acquis 5% du capital. Albioma en ligne avec le plan de verdissement de ses centrales « Nous connaissons Albioma depuis longtemps, c'est une société qui, sous l'égide de Jacques Petry, le président de son Conseil d'administration, est de plus en plus décarbonée » , détaille Emilie Brunet, directrice d'investissement « mid and large caps » chez Bpifrance. Avec 80% du charbon pour 20% de biomasse, l'entreprise a pour objectif d'inverser le ratio avant de tendre vers le 100% renouvelables. Résidus de bois, autres déchets verts et même déchets d'autres natures peuvent remplacer le charbon, à condition de modifier les centrales.  « La société est en ligne avec le plan de rénovation de ses centrales visant notamment à les rendre plus « vertes » » , observe Emilie Brunet.   « Il faut souligner qu'Albioma autofinance ce plan de rénovation, ajoute-t-elle. Nous ne leur apportons pas de capital nouveau pour les aider à le financer. Nous avons constitué notre position en procédant à des achats sur le marché sur plusieurs mois, dans l'optique de devenir un nouvel actionnaire de référence du groupe.» Cette prise de participation s'inscrit dans le rôle dévolu à Bpifrance dans le financement de la transition énergétique.   Priorité à la transition énergétique, à l'économie sociale et au numérique « La transition énergétique et écologique (TEE) est un secteur économique dynamique qui a besoin de financements long terme, rappelle Pascal Lagarde, directeur exécutif au sein de la direction de l'international, de la stratégie, des études et du développement. Avec l'économie sociale et solidaire et le numérique, elle figure dans la loi de création de Bpifrance, assortie d'objectifs dès le premier plan stratégique établi en 2013. » A ce titre, Bpifrance accompagne aussi bien les entreprises de la transition énergétique et écologique, dont les acteurs des énergies renouvelables, celles qui développent de nouvelles solutions technologiques, et celles qui opèrent leur propre transition, notamment en limitant leur consommation d'énergie et d'autres fluides via une remise à niveau industrielle.   1,4 milliard d'euros pour la transition énergétique en 2016 En 2016, Bpifrance a ainsi accompagné 1.400 entreprises pour un montant global de 1,4 milliard d'euros, soit 14% de plus qu'en 2015. La moitié de ce montant consiste en co-financements longue durée de projets d'énergies renouvelables, pour lesquels la banque a développé un axe d'investissement dédié.   Depuis 2013, Bpifrance a investi pour 158 millions d'euros de fonds propres au capital de développeurs indépendants à la tête de 1,4 gigawatts (GW). Elle est ainsi présente au capital de Quadran, Neoen ou encore Eren. Elle a également engagé 2,8 millions d'euros pour co-financer des projets d'énergies renouvelables, pour une puissance installée entre 3 et 4 GW.   Co-investissement aux côté de corporate Concernant l'innovation dans d'autres secteurs de la TEE (mobilité, stockage d'énergie, efficacité énergétique, chimie verte, etc.) Bpifrance a investi 70 millions d'euros en capital innovation dans des cleantech et 250 millions d'euros dans des projets industriels innovants.   « Sur l'innovation, notre fonds écotechnologie a notamment investi dans le fabriquant de LED nouvelle génération Aledia aux côtés d'Ikea et de Valeo, cite Pascal Lagarde, notre fonds SPI (Société de projets industriels), a aussi accompagné DCNS dans la filialisation de ses activités dans les énergies marines. » Les énergies marines sont l'un des secteurs émergents les plus dynamique sur lesquels Bpifrance est active.   Enfin, 510 millions de prêts sans garantie ont été octroyés à de entreprises engagées dans une évolution de leurs process industriels visant une amélioration de leur performance énergétique et environnementale.  Cet article est paru dans La Tribune (site web)
</article>
<article titre="Bpifrance acquiert une part de 5% dans Albioma" date="2017 04 27" journal="L'AGEFI Quotidien">
Bprifrance a acquis une participation de 5% au capital d'Albioma «afin d'accompagner le groupe dans sa stratégie de transition énergétique de ses installations». Le conseil d'administration d'Albioma proposera à l'AG la nomination de Bpifrance Investissement comme administrateur. Très présent en Outre-Mer, Albioma compte convertir ses centrales actuelles utilisant la bagasse (résidus de canne à sucre) ou le charbon en infrastructures 100% biomasse qui valorisent, aux côtés de la bagasse, des déchets verts ou des résidus de l'industrie du bois.
</article>
<article titre="Albioma : un courtier ajuste son objectif" date="2017 04 27" journal="Boursier (site web)">
Invest Securities réitère son conseil "neutre" sur  Albioma  tout en ajustant son objectif de 16,1 à 16,6 euros. Le broker note que le premier trimestre s'est avéré solide (+3% hors effet prix combustible qui n'impacte pas les marges), bien que contrasté. La biomasse thermique en France est portée par la bonne performance des installations qui se traduit par un taux de disponibilité en hausse. En revanche, le CA solaire est stable et le CA Brésil est légèrement impacté (-0,3 ME) par un manque de stocks de bagasse, en attendant les prochaines récoltes. Cette publication conforte le groupe dans ses guidances annuelles.  Le titre progresse de 1% à 17,7 euros en fin de matinée à Paris.
</article>
<article titre="Albioma en croissance au 1T, Bpifrance entre au capital" date="2017 04 26" journal="AFP Infos Economiques">
Paris, 26 avr 2017 (AFP) - - Bpifrance a annoncé mercredi être entré au capital du producteur d'électricité Albioma, qui a publié le même jour un chiffre d'affaires en progression au premier trimestre grâce au prix des matières premières.   La banque publique d'investissement a pris une participation de 5% au capital de l'entreprise, surtout présente en outre-mer, afin de l'"accompagner (...) dans sa stratégie de transition énergétique de ses installations", indique-t-elle dans un communiqué. L'entrée de Bpifrance au conseil d'administration sera également proposée à la prochaine assemblée générale, prévue le 31 mai.   Actuellement, Albioma produit de l'électricité à partir de centrales thermiques utilisant la bagasse (co-produit de la canne à sucre) mais aussi du charbon une partie de l'année.   La société souhaite convertir ses installations pour qu'elles produisent du courant à 100% à partir de biomasse (bagasse, résidus de bois, déchets verts, etc.), explique Bpifrance.   Au premier trimestre, Albioma a vu son chiffre d'affaires progresser de 13% à 98,8 millions d'euros, tiré par le faible prix des matières premières.   Les revenus de son activité biomasse thermique en France, qui représentent l'essentiel de son activité, ont crû de 16% à 85,9 millions d'euros, dont "une contribution de 9 millions d'euros liée à l'évolution du prix des combustibles", explique la société dans un communiqué.   A cela s'ajoute un bon fonctionnement des centrales, dont le taux de disponibilité atteint plus de 91% sur la période, grâce à la "bonne gestion" des arrêts de maintenance, selon Albioma.   Toujours en France, l'activité solaire atteint 10,4 millions d'euros (+1%), tandis qu'au Brésil, deux centrales en maintenance n'ont pas fonctionné au premier trimestre, et le chiffre d'affaires dans ce pays a logiquement reculé de 23% à 1 million d'euros.   Albioma a par ailleurs confirmé ses objectifs financiers pour 2017. Elle vise un excédent brut d'exploitation (ebitda) compris entre 130 et 138 millions d'euros et un bénéfice net entre 30 et 35 millions d'euros.   La société poursuit également la construction de sa centrale bagasse/biomasse Galion 2 à La Martinique, dont l'autorisation d'exploiter, qui avait été annulée dans un premier temps par la justice, a été finalement validée en appel mi-avril.   Cette centrale, qui doit entrer en service à la fin de l'année, fournira 15% de la consommation électrique totale de l'île.   mhc/ef/ggy
</article>
<article titre="Bpifrance prend 5% du capital d'Albioma" date="2017 04 26" journal="L'AGEFI Quotidien">
Bprifrance a acquis une participation de 5% au capital d'Albioma « afin d'accompagner le groupe dans sa stratégie de transition énergétique de ses installations ». Le conseil d'administration d'Albioma proposera à l'AG la nomination de Bpifrance Investissement comme administrateur. Très présent en Outre-Mer, Albioma compte convertir ses centrales actuelles utilisant la bagasse (résidus de canne à sucre) ou le charbon en infrastructures 100% biomasse qui valorisent, aux côtés de la bagasse, des déchets verts ou des résidus de l'industrie du bois.
</article>
<article titre="Un nouveau champion de la chimie verte dans l'Oise" date="2017 04 26" journal="Courrier picard  Toutes éditions">
COMPIÈGNE Demain, la chimie sera-t-elle entièrement végétale? L'aventure se poursuit sur les berges de l'Oise près de Compiègne. CIMV (Compagnie industrielle de la matière végétale) vient d'annoncer un investissement de 2,5M€ sur fonds propres pour son nouveau pilote installé sur le site Pivert, créée en 2012 dans le cadre des Programmes d'investissements d'avenir. CIMV bénéficiera notamment d'un accès à la plateforme technologique Biogis Center, une extension de Pivert consacrée à la mise au point de brevets industriels. Créée en 1998, CIMV développe un process de séparation de la matière végétale qui s'applique à tous les résidus non alimentaires. Comme la paille de riz ou de blé, les tiges de maïs, la bagasse de la canne à sucre, ainsi que les restes de l'industrie forestière. «Les premières applications s'inscrivent en parfaite ligne avec les enjeux liés à la transition énergétique», explique CIMV; alors que les premières productions de bioéthanol (notamment à Compiègne) étaient très critiquées du fait de leur usage de la totalité de la plante, notamment la partie alimentaire. L'enjeu est de produire des biocarburants de deuxième génération et des biopolymères et des biomatériaux pour l'industrie. Donc des produits biosourcés, en remplacement des matières dérivées du pétrole. Les premières expériences s'étaient déroulées dans un premier pilote près de Toulouse. Pour CIMV, il s'agit désormais de passer au stade de pré-industrialisation, avec une capacité portée à une tonne par jour par rapport à la production précédente. La compagnie va dépêcher sur place trois chercheurs.
</article>
<article titre="Bpifrance contribue à stabiliser l'actionnariat d'Albioma" date="2017 04 26" journal="GreenUnivers (site web)">
2017-04-30T22:36:35+00:00    Bpifrance vient de franchir le seuil de 5% du capital d'Albioma. Ce développeur-producteur particulièrement actif outre-mer est coté sur Euronext avec un flottant de plus de 60% désormais, depuis le départ partiel d'Apax Partners l'année dernière. Apax détenait alors 30% du capital. Bpifrance a effectué cette opération par achats successifs d'actions, depuis plusieurs mois ; il n' y a pas d'augmentation de capital.   Retrouver de la stabilité  Avec ce nouvel arrivant, Albioma souhaite contribuer à restabiliser son actionnariat et être accompagné par des partenaires de long terme. D'où la présence d'un autre "poids lourd", la Caisse des dépôts, via son fonds valeurs moyennes (5% du capital). Mais aussi d'autres actionnaires fidèles : la Cofepp, un gros producteur d'alcool, la financière de l'Echiquier et le fonds Altamir, qui est d'ailleurs proche d'Apax Partners. Auxquels s'ajoute désormais Bpifrance, donc. Bpifrance veut peser plus fort sur la transition énergétique  Déjà présente au capital de Quadran, Neoen et Eren, entre autres, la banque publique d'investissement « complète ainsi son axe transition énergétique et investit dans un producteur incontournable en territoire ultramarin, justement engagé dans une évolution majeure, celle du charbon vers la biomasse », explique Emilie Brunet, directrice d'investissement dans l'équipe  mid &amp; large caps  de Bpifrance. L'investisseur veut être actionnaire de référence - comme le souhaite Albioma - et va d'ailleurs entrer au conseil d'administration. Il n'exclut pas de grimper encore au capital si Albioma a besoin de capitaux supplémentaires. Mais ce n'est pas à l'ordre du jour.   Feu vert pour Galion 2   Hasard du calendrier, cette arrivée se produit au moment où Albioma décroche une confirmation d'autorisation d'exploiter pour l'un de ses projets phares outremer. En l'occurrence la centrale bagasse et biomasse Galion 2 de 40 MW en Martinique, un investissement de 170 M€. Dans son arrêt du 13 avril dernier, la Cour administrative d'appel de Bordeaux a infirmé le jugement du tribunal administratif de Fort-de-France du 4 octobre 2016 qui annulait l'arrêté d'exploitation. La cour a rejeté les reproches de l'Association pour la sauvegarde du patrimoine martiniquais à l'égard du projet.« Galion 2 sera la première centrale 100% biomasse en outre mer », indique un porte-parole d'Albioma. Le projet représente à ce titre un jalon important dans la transition énergétique ultramarine en général et dans l'évolution du modèle de production d'Albioma, en particulier. Actuellement en construction, elle devrait entrer en service à la fin de l'année.   Albioma a en revanche moins de visibilité sur un autre projet, sur l'île de Marie-Galante, beaucoup moins avancé que Galion 2. Adossé à une sucrerie, il est en développement depuis quatre ans mais se trouve désormais concurrencé par d'autres solutions, notamment photovoltaïques, suite à de nouvelles demandes de la communauté de communes concernée. A 17,40 euros, le cours de l'action d'Albioma progressait aujourd'hui de 2,75%, pour s'établir à 17,53 €.   Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="L'Oréal se tourne de plus en plus vers le naturel" date="2017 04 25" journal="Les Echos, no. 22432">
Pour L'Oréal, utiliser des ingrédients « bio » est devenu une priorité depuis une dizaine d'années.  « C'est un processus continu »,  explique Laurent Gilbert, directeur de l'innovation durable du groupe. «  Nous sommes déjà passés de 35 à 40 % de matières premières d'origine renouvelable en 2005, à 54 % aujourd'hui. Et nous espérons monter à 65 % d'ici dix ans.  » Pour le géant français, il s'agit de répondre à une demande croissante de ses clients, alors que les associations de consommateurs alertent sur la présence de perturbateurs endocriniens dans les cosmétiques. «  Il y a une attente des consommateurs, sur l'origine des ingrédients, mais aussi des filières : ils sont de plus en plus attachés au caractère durable des produits comme des processus de production.  » Le groupe attend donc des efforts en ce sens de la part de ses fournisseurs.  « Il peut être complexe de mettre en place des filières répondant à ces critères de durabilité, en maintenant la performance des ingrédients mais sans en renchérir le coût - un critère indispensable »,  poursuit-il. «  Les chimistes ont pris conscience de ces enjeux, et on voit naître des filières industrielles inimaginables il y a encore quelques années.  » Le groupe a, par exemple, adopté récemment le pentylène glycol du fabricant français Minasolve, qui a réussi à le produire à partir de bagasse (résidu de la canne à sucre) : or il s'agit d'un solvant extrêmement courant, utilisé comme humectant et comme conservateur dans les cosmétiques.   L'Oréal a aussi conclu un partenariat avec la start-up française Global Bioenergies, qui produit à partir de sucre de l'isobutène, une molécule jusque-là issue du pétrole, et dont les dérivés sont largement utilisés dans la cosmétique.   A. F.
</article>
<article titre="Décret no 2017-570 du 19 avril 2017 relatif à la programmation pluriannuelle de l’énergie de la Guadeloupe" date="2017 04 25" journal="Rapports - Environnement - MEDDTL">
3 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... Décrets, arrêtés, circulaires  TEXTES GÉNÉRAUX  MINISTÈRE DE L'ENVIRONNEMENT, DE L'ÉNERGIE ET DE LA MER, EN CHARGE DES RELATIONS INTERNATIONALES SUR LE CLIMAT  NOR : DEVR1710009D  Publics concernés : Etat et ses...    Pour voir le rapport :  http://www.developpement-durable.gouv.fr/sites/default/files/PPE%20Guadeloupe%20-%20Decret.pdf
</article>
<article titre="Albioma : le marché salue l'autorisation d'exploitation de Galion 2" date="2017 04 18" journal="Boursier (site web)">
Albioma  grappille 11,2% à 16,90 euros ce mardi dans les premiers échanges à la Bourse de Paris, dans d'importants volumes de transactions. Les investisseurs réagissent à la décision de la Cour Administrative d'Appel de Bordeaux qui, dans son arrêt du 13 avril 2017, a décidé d'infirmer le jugement du Tribunal Administratif de Fort-de-France en date du 4 octobre 2016 qui annulait l'arrêté d'exploitation de la centrale bagasse/biomasse Galion 2. La voie est par conséquent libre pour le démarrage de l'exploitation.   Portzamparc salue cette nouvelle qui améliore la visibilité sur l'activité. Le broker, qui en profite pour rehausser de 19,50 à 20,40 euros son objectif de cours sur la valeur, rappelle l'importance de Galion 2 pour le groupe et l'île de la Martinique : "la centrale Galion 2 sera une unité de base, la première centrale de cogénération 100% biomasse de Martinique et fournira 15% de la consommation électrique totale de l'île, portant la part d'énergie renouvelable en Martinique de 7 à 22%". Dans son hypothèse (prudente), Portzamparc prévoit une mise en service commerciale le 1er décembre 2017, contre début 2018 dans son scénario précédent. De quoi rester acheteur du titre...
</article>
<article titre="Les valeurs à suivre aujourd'hui à la Bourse de Paris - Mardi 18 avril 2017" date="2017 04 18" journal="Option Finance (site web)">
AOF - 18 avril 2017   (AOF) - ADOCIA   Adocia a réalisé au premier trimestre 2017 un chiffre d'affaires de 19,417 millions d'euros contre 6,799 millions un an plus tôt. Il provient essentiellement de l'accord de collaboration et de licence signé en décembre 2014 avec Eli Lilly relatif au développement d'une insuline ultra-rapide BioChaperone Lispro. En janvier 2017, Adocia a annoncé la décision d'Eli Lilly de mettre fin à la collaboration sur BioChaperone Lispro. ALBIOMA   Albioma annonce que la Cour Administrative d'Appel de Bordeaux, dans son arrêt du 13 avril 2017, a décidé d'infirmer le jugement du Tribunal Administratif de Fort-de-France en date du 4 octobre 2017 qui annulait l'arrêté d'exploitation de la centrale bagasse/biomasse Galion 2. Tous les griefs soulevés par l'Association pour la Sauvegarde du Patrimoine Martiniquais à l'encontre du projet ont été rejetés. L'exploitation du projet est en conséquence de nouveau pleinement autorisée.   CASINO   Casino a vu ses ventes croitre de 11,6% au premier trimestre 2017 à 9,32 milliards d'euros contre un consensus de 9,27 milliards. En comparable, elles ont progressé de 2,5%. En France, principal marché du groupe, la croissance en comparable s'est élevée à +0,2% et, en Amérique latine, à 4,6%.   EUROTUNNEL   Europorte France, spécialisée dans le fret ferroviaire d'Eurotunnel, et ses quatre organisations syndicales CGT, CFDT, SUD et CFTC, viennent de signer à l'unanimité un accord sur le temps de travail dans le cadre de la réforme ferroviaire. Cet accord inscrit dans la durée les engagements de l'entreprise en faveur d'une politique sociale favorable à la qualité de vie sans nuire à la productivité. Il offre une meilleur visibilité des plannings de travail tant pour les salariés que pour les clients.   HORIZONTAL SOFTWARE   Horizontal Software, éditeurs de logiciels de ressources humaines en mode SaaS et SaaS-Apps, a annoncé la signature de trois nouveaux contrats. Il a signé deux nouveaux contrats sur le module Gestion des Temps et Activités des collaborateurs auprès de deux clients de premier plan. Le Ministère de l'Agriculture a ainsi signé un contrat portant sur 2 500 agents de la fonction publique. Ce contrat est d'une durée de 3 ans. Il est renouvelable par période de 12 mois pendant 10 ans.   ICADE   Icade Santé (filiale d'Icade à 56,5%) a acquis deux établissements de santé les 12 et 13 avril 2017. Suite à la promesse signée fin 2016, le groupe immobilier a racheté la polyclinique MCO de l'Ormeau à Tarbes (65) auprès du groupe Médipôle Partenaires, pour une valeur de 43 millions d'euros (droits inclus). La polyclinique est composée de deux sites : Ormeau Centre (10 437 mètres carrés) et Ormeau Pyrénées (10 609 mètres carrés).   LATECOERE   Latecoère annonce plusieurs nominations au sein de son Comité Exécutif. Thierry Mootz devient directeur des opérations (COO) du groupe et Emmanuel Remy prend la direction de la branche Aérostructures. Enfin, Sébastien Rouge devient Directeur Administratif et Financier de Latécoère. Il était jusqu'alors Directeur Général de la JV GE/Alstom Nuclear Systems et CFO de Steam Power Systems, branche de GE Power. Il a auparavant occupé le poste de SVP Finance chez Alstom Power après avoir été Directeur Financier dans l'industrie en France, en Allemagne et aux Etats-Unis.   RIBER   Riber a annoncé la commande d'une machine de recherche MBE. L'université de Californie à Santa Barbara, USA, a commandé une machine de recherche MBE, modèle C21T, pour des travaux de recherche sur les chalcogénides. Cette classe de matériaux aura des applications pour la détection et l'émission de lumière dans le spectre infrarouge, et servira de banc d'essai pour une variété de phénomènes exotiques récemment découverts.   SPIE   Spie, le leader européen indépendant des services multi- techniques dans les domaines de l'énergie et des communications, a signé le 13 avril 2017 un accord pour l'acquisition de Lück Verwaltungs GmbH (Groupe Lück), un spécialiste allemand des services multi-techniques pour le secteur tertiaire. L'acquisition du Groupe Lück va permettre à Spie d'étendre son offre de services mécaniques et électriques en Allemagne, tout en densifiant davantage son réseau et en renforçant sa position d'acteur de premier plan des services multi-techniques dans ce pays.    VALNEVA   Valneva a signé un nouvel accord commercial avec la société de biotechnologie Danoise Bavarian Nordic pour la production de vaccins issus du virus de la variole à l'aide de la technologie EB66 de Valneva. Dans le cadre cet accord, Bavarian Nordic pourra utiliser la lignée cellulaire EB66 pour développer et commercialiser différents vaccins issus du virus de la variole. Les candidats vaccins actuels de Bavarian Nordic sont produits sur fibroblastes embryonnaires de poulet (CEF) et la société va explorer la possibilité de les transférer sur la technologie EB66 de Valneva.   VETOQUINOL   Le chiffre d'affaires consolidé du premier trimestre de Vetoquinol s'est élevé à 86,1 millions d'euros, en progression de 0,4%. Il a bénéficié d'un impact de change favorable de 1,8%, essentiellement lié aux devises hors Europe. Il a cependant reculé de 1,4% à périmètre et change constants. Le niveau d'activité des produits de référence s'est établi à 38,8 millions d'euros, en hausse de 2,2% à données publiées. Ils ont représenté 45% des ventes, contre 44,2% pour la même période de 2016.   VEXIM   Vexim lance Masterflow Plus en Allemagne, un nouveau produit destiné au traitement des fractures ostéoporotiques à basse énergie. Avec ce nouveau produit, développé spécialement pour le marché allemand, le groupe a l'ambition d'augmenter à nouveau sa croissance et sa pénétration sur ce pays.   Cet article est paru dans Option Finance (site web)
</article>
<article titre="A suivre aujourd'hui... ALBIOMA" date="2017 04 18" journal="Agence Option Finance (AOF)">
(AOF) - Albioma annonce que la Cour Administrative d'Appel de Bordeaux, dans son arrêt du 13 avril 2017, a décidé d'infirmer le jugement du Tribunal Administratif de Fort-de-France en date du 4 octobre 2017 qui annulait l'arrêté d'exploitation de la centrale bagasse/biomasse Galion 2. Tous les griefs soulevés par l'Association pour la Sauvegarde du Patrimoine Martiniquais à l'encontre du projet ont été rejetés. L'exploitation du projet est en conséquence de nouveau pleinement autorisée.    Depuis début 2015, les travaux de construction de la centrale se sont poursuivis sans discontinuer, et comme annoncé, la mise en service de l'installation est prévue pour le quatrième trimestre 2017.  L'installation Galion 2 (40 MW) renforcera le réseau électrique martiniquais et contribuera à répondre aux besoins énergétiques de l'île.La centrale Galion 2 (40 MW), première centrale de cogénération 100 % biomasse de Martinique, fournira 15 % de la consommation électrique totale de l'île.   Reposant sur un échange vertueux avec la sucrerie du Galion, le projet vise à assurer l'avenir de la sucrerie en la rendant plus performante.   La centrale Galion 2 fera progresser la part d'énergie renouvelable en Martinique de 7 % à 22 % et, ainsi, favoriser la transition énergétique de l'île dans le respect des normes environnementales les plus strictes.   La centrale Galion 2, financée par des investissements privés, participe au développement de l'emploi en Martinique. Aujourd'hui, 350 personnes travaillent à la construction du site. Le fonctionnement de la centrale va permettre de créer 34 emplois directs (dont 32 emplois martiniquais, conformément aux engagements du groupe de promotion de l'emploi local) et de générer plus de 200 emplois indirects.   AOF - EN SAVOIR PLUS  Services aux collectivités    Les grands énergéticiens européens affrontent un environnement difficile, marqué par l'effondrement des prix sur les marchés de gros et une situation de surcapacité. Les utilities doivent également faire face à l'apparition de concurrents venus d'autres secteurs d'activité, tels que ceux issus du "smart building " (gestion technique du bâtiment associée à un ensemble de services). En France, EDF affiche des difficultés financières, d'énormes besoins d'investissements pour entretenir son parc nucléaire et des tensions sur l'approvisionnement. En Allemagne, E.ON et RWE enregistrent de mauvais résultats financiers. Les fusions et acquisitions se multiplient dans le secteur de l'énergie. Au second trimestre 2016, leur valeur a progressé de 42% par rapport à 2015. Les sociétés opérant dans les énergies renouvelables restent très attractives. Les acquisitions, ciblées et de taille moyenne, de renforcement dans des activités spécifiques se poursuivent également. On note également un intérêt croissant des acheteurs asiatiques pour les actifs européens du secteur.
</article>
<article titre="Morning meeting AOF France / Europe - Ouverture étale en vue" date="2017 04 18" journal="Agence Option Finance (AOF)">
(AOF) - Les marchés actions européens sont attendus proches de l'équilibre après ce long week de Pâques. Wall Street et la Bourse de Tokyo ont terminé dans le vert mais la prudence semble continuer de prévaloir dans un contexte d'aversion pour le risque. Les investisseurs observent avec inquiétude le regain de tensions entre les Etats-Unis et la Corée du Nord et s'interrogent sur l'élection présidentielle en France. Par ailleurs, de nombreuses multinationales vont dévoiler leurs chiffre d'affaires ou leurs résultats cette semaine. Ce matin, Casino a confirmé ses objectifs de ROC 2017.   L'analyse technique du CAC 40   Du point de vue de l'analyse graphique, le bureau d'études DayByDay observe la formation d'une seconde bougie noire dans des volumes plutôt importants à 3.51 milliards d'euros. Les cours sont ainsi revenus sur le bas de l'intervalle de consolidation à 5054.91 points. Ce soutien est crucial pour conserver une lecture haussière; en effet, sa rupture en clôture validerait une figure de retournement en double sommet, mettant ainsi fin à la tendance haussière initiée au mois de février. Une phase de correction plus ample serait alors à privilégier en direction de 4958.88 points. Les valeurs à suivre aujourd'hui   ADOCIA   Adocia a réalisé au premier trimestre 2017 un chiffre d'affaires de 19,417 millions d'euros contre 6,799 millions un an plus tôt. Il provient essentiellement de l'accord de collaboration et de licence signé en décembre 2014 avec Eli Lilly relatif au développement d'une insuline ultra-rapide BioChaperone Lispro. En janvier 2017, Adocia a annoncé la décision d'Eli Lilly de mettre fin à la collaboration sur BioChaperone Lispro.   ALBIOMA   Albioma annonce que la Cour Administrative d'Appel de Bordeaux, dans son arrêt du 13 avril 2017, a décidé d'infirmer le jugement du Tribunal Administratif de Fort-de-France en date du 4 octobre 2017 qui annulait l'arrêté d'exploitation de la centrale bagasse/biomasse Galion 2. Tous les griefs soulevés par l'Association pour la Sauvegarde du Patrimoine Martiniquais à l'encontre du projet ont été rejetés. L'exploitation du projet est en conséquence de nouveau pleinement autorisée.   ICADE   Icade Santé (filiale d'Icade à 56,5%) a acquis deux établissements de santé les 12 et 13 avril 2017. Suite à la promesse signée fin 2016, le groupe immobilier a racheté la polyclinique MCO de l'Ormeau à Tarbes (65) auprès du groupe Médipôle Partenaires, pour une valeur de 43 millions d'euros (droits inclus). La polyclinique est composée de deux sites : Ormeau Centre (10 437 mètres carrés) et Ormeau Pyrénées (10 609 mètres carrés).   VETOQUINOL   Le chiffre d'affaires consolidé du premier trimestre de Vetoquinol s'est élevé à 86,1 millions d'euros, en progression de 0,4%. Il a bénéficié d'un impact de change favorable de 1,8%, essentiellement lié aux devises hors Europe. Il a cependant reculé de 1,4% à périmètre et change constants. Le niveau d'activité des produits de référence s'est établi à 38,8 millions d'euros, en hausse de 2,2% à données publiées. Ils ont représenté 45% des ventes, contre 44,2% pour la même période de 2016.   Les chiffres macroéconomiques   Les investisseurs prendront connaissance à 14h30 des permis de construire et mises en chantier en mars et à 15h15 de l'indice de la production industrielle de mars.   Vers 8h30, l'euro cote 1,0647 dollar, en hausse de 0,27%.   Jeudi dernier à Paris   Les marchés européens ont enchainé leur quatrième séance consécutive de baisse avant un long week-end de quatre jours. Le CAC 40 a ainsi perdu 0,59% à 5 071,1 points, au plus bas depuis fin mars, et l'EuroStoxx 50 a reculé de 0,58% à 3 448,26 points. En quatre séances, l'indice parisien a cédé 0,98% et son homologue européen 1,18%. Toutes choses égales par ailleurs, la semaine a été l'une des pires depuis le début de l'année pour les investisseurs européens : au 20 janvier, sur cinq jours, le CAC 40 avait reculé de 1,46% et l'EuroStoxx de 0,75%. L'aversion au risque a dominé cette semaine.   Hier à Wall Street   Les marchés actions américains sont repartis à la hausse hier, mettant fin à une série de trois replis consécutifs. Les investisseurs ont mis de côté les risques liés aux tensions géopolitiques pour se concentrer sur les actualités en provenance des entreprises. A cet égard, les premiers résultats du premier trimestre dévoilés la semaine dernière JPMorgan Chase et Citigroup sont jugés encourageants pour la suite. Au chapitre des valeurs, Exxon a pâti du repli de 1% des cours du brut. Le Dow Jones a gagné lundi 0,9% à 20 636,92 points. Le Nasdaq Composite a progressé de 0,89% à 5 856,79 points.
</article>
<article titre="BREF-Albioma-Validité confirmée en appel de l'autorisation d'exp BREF-Albioma-Validité confirmée en appel de l'autorisation d'exploiter Galion 2" date="2017 04 13" journal="Reuters économique">
13 avril (Reuters) - Albioma annonce jeudi dans un communiqué :   * La cour administrative d'appel de Bordeaux confirme la validité de l'autorisation d'exploiter de la centrale bagasse/biomasse Galion 2 (Martinique)   * Tous les griefs soulevés par l’Association pour la sauvegarde du patrimoine martiniquais à l’encontre du projet ont été rejetés * Comme annoncé, la mise en service de l’installation est prévue pour le quatrième trimestre 2017.   * Le communiqué : http://bit.ly/2oqXhSg Pour plus de détails, cliquez sur (Rédaction de Paris)
</article>
<article titre="ALBIOMA-LA COUR ADMINISTRATIVE D'APPEL DE BORDEAUX CONFIRME LA V ALBIOMA-LA COUR ADMINISTRATIVE D'APPEL DE BORDEAUX CONFIRME LA VALIDITÉ DE L'AUTORISATION D'EXPLOITER DE LA CENTRALE  BAGASSE/BIOMASSE GALION 2 (MARTINIQUE)" date="2017 04 13" journal="Reuters général">
ALBIOMA-LA COUR ADMINISTRATIVE D'APPEL DE BORDEAUX CONFIRME LA VALIDITÉ DE L'AUTORISATION D'EXPLOITER DE LA CENTRALE BAGASSE/BIOMASSE GALION 2 (MARTINIQUE)
</article>
<article titre="Albioma: décision administrative favorable." date="2017 04 13" journal="Votre argent (site web)">
La Cour Administrative d'Appel de Bordeaux (Gironde) a dans un arrêt rendu ce jeudi décidé d'infirmer le jugement du Tribunal administratif de Fort-de-France (Martinique) en date du 4 octobre 2016 qui annulait l'arrêté d'exploitation de la centrale bagasse/biomasse Galion 2, a rapporté Albioma après séance.  Tous les griefs soulevés par l'Association pour la Sauvegarde du Patrimoine Martiniquais à l'encontre du projet ont été rejetés. En conséquence, l'exploitation du projet est de nouveau pleinement autorisée.    Depuis début 2015, les travaux de construction de la centrale, d'une capacité de production de 40 mégawatts (MW), se sont poursuivis sans discontinuer et, comme annoncé, la mise en service de l'installation est prévue pour le quatrième trimestre 2017, a précisé la société.    Copyright (c) 2017 CercleFinance.com. Tous droits réservés.
</article>
<article titre="ALBIOMA : décision juridique favorable pour la centrale Galion 2" date="2017 04 13" journal="Option Finance (site web)">
AOF - 13 avril 2017   (AOF) - Albioma annonce que la Cour Administrative d'Appel de Bordeaux, dans son arrêt du 13 avril 2017, a décidé d'infirmer le jugement du Tribunal Administratif de Fort-de-France en date du 4 octobre 2017 qui annulait l'arrêté d'exploitation de la centrale bagasse/biomasse Galion 2. Tous les griefs soulevés par l'Association pour la Sauvegarde du Patrimoine Martiniquais à l'encontre du projet ont été rejetés. L'exploitation du projet est en conséquence de nouveau pleinement autorisée.    Depuis début 2015, les travaux de construction de la centrale se sont poursuivis sans discontinuer, et comme annoncé, la mise en service de l'installation est prévue pour le quatrième trimestre 2017.  L'installation Galion 2 (40 MW) renforcera le réseau électrique martiniquais et contribuera à répondre aux besoins énergétiques de l'île.La centrale Galion 2 (40 MW), première centrale de cogénération 100 % biomasse de Martinique, fournira 15 % de la consommation électrique totale de l'île.   Reposant sur un échange vertueux avec la sucrerie du Galion, le projet vise à assurer l'avenir de la sucrerie en la rendant plus performante.   La centrale Galion 2 fera progresser la part d'énergie renouvelable en Martinique de 7 % à 22 % et, ainsi, favoriser la transition énergétique de l'île dans le respect des normes environnementales les plus strictes.   La centrale Galion 2, financée par des investissements privés, participe au développement de l'emploi en Martinique. Aujourd'hui, 350 personnes travaillent à la construction du site. Le fonctionnement de la centrale va permettre de créer 34 emplois directs (dont 32 emplois martiniquais, conformément aux engagements du groupe de promotion de l'emploi local) et de générer plus de 200 emplois indirects.  Cet article est paru dans Option Finance (site web)
</article>
<article titre="Les valeurs à suivre mardi 18 avril 2017 à la Bourse de Paris -" date="2017 04 13" journal="Option Finance (site web)">
AOF - 13 avril 2017   (AOF) - ALBIOMA   Albioma annonce que la Cour Administrative d'Appel de Bordeaux, dans son arrêt du 13 avril 2017, a décidé d'infirmer le jugement du Tribunal Administratif de Fort-de-France en date du 4 octobre 2017 qui annulait l'arrêté d'exploitation de la centrale bagasse/biomasse Galion 2. Tous les griefs soulevés par l'Association pour la Sauvegarde du Patrimoine Martiniquais à l'encontre du projet ont été rejetés. L'exploitation du projet est en conséquence de nouveau pleinement autorisée.  HORIZONTAL SOFTWARE   Horizontal Software, éditeurs de logiciels de ressources humaines en mode SaaS et SaaS-Apps, a annoncé la signature de trois nouveaux contrats. Il a signé deux nouveaux contrats sur le module Gestion des Temps et Activités des collaborateurs auprès de deux clients de premier plan. Le Ministère de l'Agriculture a ainsi signé un contrat portant sur 2 500 agents de la fonction publique. Ce contrat est d'une durée de 3 ans. Il est renouvelable par période de 12 mois pendant 10 ans.    ICADE   Icade Santé (filiale d'Icade à 56,5%) a acquis deux établissements de santé les 12 et 13 avril 2017. Suite à la promesse signée fin 2016, le groupe immobilier a racheté la polyclinique MCO de l'Ormeau à Tarbes (65) auprès du groupe Médipôle Partenaires, pour une valeur de 43 millions d'euros (droits inclus). La polyclinique est composée de deux sites : Ormeau Centre (10 437 mètres carrés) et Ormeau Pyrénées (10 609 mètres carrés).    LATECOERE   Latecoère annonce plusieurs nominations au sein de son Comité Exécutif. Thierry Mootz devient directeur des opérations (COO) du groupe et Emmanuel Remy prend la direction de la branche Aérostructures. Enfin, Sébastien Rouge devient Directeur Administratif et Financier de Latécoère. Il était jusqu'alors Directeur Général de la JV GE/Alstom Nuclear Systems et CFO de Steam Power Systems, branche de GE Power. Il a auparavant occupé le poste de SVP Finance chez Alstom Power après avoir été Directeur Financier dans l'industrie en France, en Allemagne et aux Etats-Unis.   VETOQUINOL   Le chiffre d'affaires consolidé du premier trimestre de Vetoquinol s'est élevé à 86,1 millions d'euros, en progression de 0,4%. Il a bénéficié d'un impact de change favorable de 1,8%, essentiellement lié aux devises hors Europe. Il a cependant reculé de 1,4% à périmètre et change constants. Le niveau d'activité des produits de référence s'est établi à 38,8 millions d'euros, en hausse de 2,2% à données publiées. Ils ont représenté 45% des ventes, contre 44,2% pour la même période de 2016.  Cet article est paru dans Option Finance (site web)
</article>
<article titre="Albioma : la validité de de l'exploitation de la centrale Galion 2 est confirmée" date="2017 04 13" journal="Boursier (site web)">
D'après un communiqué du groupe  Albioma , la Cour Administrative d'Appel de Bordeaux, dans son arrêt du 13 avril 2017, a décidé d'infirmer le jugement du Tribunal Administratif de Fort-de-France en date du 4 octobre 2017 qui annulait l'arrêté d'exploitation de la centrale bagasse/biomasse Galion 2.    Tous les griefs soulevés par l'Association pour la Sauvegarde du Patrimoine Martiniquais à l'encontre du projet ont été rejetés. L'exploitation du projet est en conséquence de nouveau pleinement autorisée.  Depuis début 2015, les travaux de construction de la centrale se sont poursuivis sans discontinuer, et comme annoncé, la mise en service de l'installation est prévue pour le quatrième trimestre 2017.    La centrale Galion 2 (40 MW), première centrale de cogénération 100 % biomasse de Martinique, fournira 15% de la consommation électrique totale de l'île.
</article>
<article titre="Les valeurs à suivre mardi 18 avril 2017 à la Bourse de Paris -" date="2017 04 13" journal="Agence Option Finance (AOF)">
(AOF) - ALBIOMA   Albioma annonce que la Cour Administrative d'Appel de Bordeaux, dans son arrêt du 13 avril 2017, a décidé d'infirmer le jugement du Tribunal Administratif de Fort-de-France en date du 4 octobre 2017 qui annulait l'arrêté d'exploitation de la centrale bagasse/biomasse Galion 2. Tous les griefs soulevés par l'Association pour la Sauvegarde du Patrimoine Martiniquais à l'encontre du projet ont été rejetés. L'exploitation du projet est en conséquence de nouveau pleinement autorisée.    HORIZONTAL SOFTWARE Horizontal Software, éditeurs de logiciels de ressources humaines en mode SaaS et SaaS-Apps, a annoncé la signature de trois nouveaux contrats. Il a signé deux nouveaux contrats sur le module Gestion des Temps et Activités des collaborateurs auprès de deux clients de premier plan. Le Ministère de l'Agriculture a ainsi signé un contrat portant sur 2 500 agents de la fonction publique. Ce contrat est d'une durée de 3 ans. Il est renouvelable par période de 12 mois pendant 10 ans.    ICADE   Icade Santé (filiale d'Icade à 56,5%) a acquis deux établissements de santé les 12 et 13 avril 2017. Suite à la promesse signée fin 2016, le groupe immobilier a racheté la polyclinique MCO de l'Ormeau à Tarbes (65) auprès du groupe Médipôle Partenaires, pour une valeur de 43 millions d'euros (droits inclus). La polyclinique est composée de deux sites : Ormeau Centre (10 437 mètres carrés) et Ormeau Pyrénées (10 609 mètres carrés).    LATECOERE   Latecoère annonce plusieurs nominations au sein de son Comité Exécutif. Thierry Mootz devient directeur des opérations (COO) du groupe et Emmanuel Remy prend la direction de la branche Aérostructures. Enfin, Sébastien Rouge devient Directeur Administratif et Financier de Latécoère. Il était jusqu'alors Directeur Général de la JV GE/Alstom Nuclear Systems et CFO de Steam Power Systems, branche de GE Power. Il a auparavant occupé le poste de SVP Finance chez Alstom Power après avoir été Directeur Financier dans l'industrie en France, en Allemagne et aux Etats-Unis.   VETOQUINOL   Le chiffre d'affaires consolidé du premier trimestre de Vetoquinol s'est élevé à 86,1 millions d'euros, en progression de 0,4%. Il a bénéficié d'un impact de change favorable de 1,8%, essentiellement lié aux devises hors Europe. Il a cependant reculé de 1,4% à périmètre et change constants. Le niveau d'activité des produits de référence s'est établi à 38,8 millions d'euros, en hausse de 2,2% à données publiées. Ils ont représenté 45% des ventes, contre 44,2% pour la même période de 2016.
</article>
<article titre="Vite vu" date="2017 04 9" journal="GreenUnivers (site web)">
2017-04-16T23:48:42+00:00    Lundi 10 avril Suite à un recours d'associations anti-éolien, la cour administrative d'appel de Bordeaux a annulé, début avril, le  Schéma régional éolien établi en 2012 en Poitou-Charentes  . Motif : l'absence d'évaluation environnementale préalable. Un nouveau schéma sera élaboré. La région, intégrée désormais dans la « Nouvelle Aquitaine », était à l'époque présidée par Ségolène Royal. Le juge administratif a déjà procédé à des annulations similaires dans d'autres régions.   Selon le quotidien indien  The Economic Times , le producteur local d'EnR  ReNew Power  prépare une IPO de 600 M$ (567 M€) d'ici 12 mois. Goldman Sachs, principal actionnaire, et la Banque de développement asiatique pourraient revendre une partie de leurs actions. A l'automne, l'entreprise évoquait une opération d'un montant inférieur, autour de 150 M$ (142 M€), selon le média économique indien  Mint . Mardi 11 avril Un an après un premier tour de table de 1,2 M€,  Entomo Farm  repart à la recherche de fonds pour financer l'aménagement d'un site industriel de 4 000m2 à Libourne (33) dédié à l'élevage et à la transformation d'insectes en farine pour l'alimentation des poissons d'élevage. La start-up a lancé une campagne de financement sur la plateforme d'investissement participatif Sowefund.com.   La première station-service produisant de  l'hydrogène vert sur site  a été mise en service à Sarreguemines, en Lorraine. La station est équipée d'un électrolyseur McPhy, qui produit de l'hydrogène sur site à partir d'eau et d'électricité verte (couverte par des garanties d'origine). Ce projet, baptisé FaHyence, consiste à créer une station multi-énergies permettant aux usagers de recharger les véhicules électriques à l'électricité ou à l'hydrogène produit sur place. Une station de remplissage GNC (gaz naturel comprimé) est également à l'étude. FaHyence s'inscrit dans le cadre du projet européen H2ME qui vise à déployer 29 stations-services à hydrogène et plus de 300 véhicules d'ici 2020.   Le groupement JC Decaux-SNCF-RATP a annoncé avoir déposé un recours en référé contre la décision d'attribution du marché  Vélib'  au montpelliérain Smoove par la commission d'appel d'offres du Syndicat Autolib' Vélib' Métropole.Mercredi 12 avril    Christina Figueres , l'ex-secrétaire exécutive de la Convention-cadre des Nations Unies sur le Changement Climatique, va faire son entrée au conseil d'administration d'Acciona , le développeur de parcs renouvelables espagnol présidé par José Manuel Entrecanales. Cette nomination au CA sera proposée lors de l'Assemblée générale du 11 mai 2017.   Une idée au service du climat ? L'appel à candidatures pour le programme de pré-incubation Greenhouse 2017 , lancé par Climate-KIC France , est ouvert jusqu'au 15 mai. D'une durée de 6 mois maximum, il permet d'être accompagné par un coach de l'incubateur Agoranov, d'accéder au réseau européen Climate-KIC et d'obtenir jusqu'à 5 000 €.   KfW IPEX-Bank et la Caisse d'Epargne CEPAC viennent d'accorder à Neoen deux prêts d'un montant total de 50M€ pour le financement de deux projets éoliens d'une puissance de 11 et 14 MW. Ces projets sont les premiers d'un portefeuille de 18 projets solaires et éoliens situés en France, représentant environ 170 MW, pour un montant total de 270 M€ de dette.   La jeune pousse francilienne Canibal, qui fabrique des collecteurs intelligents de déchets, a réalisé un prototype de prothèse de main, imprimée en 3D. Elle a travaillé en partenariat avec e-Nable, une communauté de bénévoles qui développe des technologies d'assistance à un prix abordable. A la clé : des prothèses dix fois moins chères à fabriquer et la possibilité, pour Canibal, de trouver un nouveau débouché pour les gobelets collectés - celui des filaments pour imprimantes 3D.Jeudi 13 avril La Banque Européenne d'Investissement a conclu un financement equity-linked en trois tranches pour un montant maximal de 30 M€ avec Electro Power Systems , société d'origine italienne spécialisée dans les systèmes de stockage à base d'hydrogène. Le financement de la BEI est garanti dans le cadre du Fonds européen pour les investissements stratégiques (FEIS).   La cour administrative d'appel de Bordeaux, dans son arrêt du 13 avril 2017, a décidé d'infirmer le jugement du tribunal administratif de Fort-de-France qui annulait l'arrêté d'exploitation de la centrale bagasse/biomasse Galion 2 (40 MW), d'Albioma . Tous les griefs soulevés par l'Association pour la Sauvegarde du Patrimoine Martiniquais à l'encontre du projet ont été rejetés. La mise en service de l'installation est désormais prévue pour le quatrième trimestre 2017.Vendredi 14 avril    Le décret relatif à la programmation pluriannuelle de l'énergie (PPE) de La Réunion est paru au Journal officiel. Il couvre deux périodes, 2016-2018 et 2019-2023. La part totale d'électricité produite à partir d'EnR ou d'énergie de récupération est fixée à 25% pour 2018 et 53% pour 2023. La PPE fixe par ailleurs un objectif de déploiement de 150 bornes de recharge de véhicules électriques et hybrides d'ici 2018 et de 225 bornes d'ici 2023.D'après nos confrères de Contexte, la France se bat pour éviter une amende en cas de non-respect de son objectif EnR , fixé à 23% pour 2020. Dans sa position sur la nouvelle gouvernance de l'Union de l'énergie, l'Hexagone s'oppose en effet à un "versement compensatoire" à l'UE. (Crédit : Energy Observer)Mis à l'eau ce 14 avril, Energy Observer part pour un tour du monde de six ans, uniquement alimenté en énergie par l'eau et le soleil. Le navire à propulsion électrique sera alimenté par une pile à combustible conçue par les ingénieurs du Commissariat à l'énergie atomique (CEA). Le carburant - l'hydrogène - sera fabriqué par électrolyse de l'eau de mer, grâce à l'électricité générée par 130 mètres carrés de panneaux photovoltaïques (Ines) répartis sur l'ensemble de la coque, couplés à deux éoliennes à axe vertical (Icam Nantes). Une aile de traction développée par la start-up Beyond the sea complète le dispositif.   Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="Les énergies marines renouvelables : une filière d'avenir en pleine expansion" date="2017 03 27" journal="Rapports - Énergie - FEE">
67 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... www.merenergies.fr  LES ÉNERGIES DE LA MER : UNE RÉALITÉ INDUSTRIELLE,  UNE DYNAMIQUE COLLECTIVE  RAPPORT #1 Mars 2017 2RAPPORT #1 / MARS 2017 / OBSERVATOIRE DES ÉNERGIES DE LA MER  SOMMAIRE  1. ÉDITO...    Pour voir le rapport :  http://fee.asso.fr/wp-content/uploads/2017/03/OEM-Rapport.pdf
</article>
<article titre="Lyspackaging Invente la bouteille compostable N L'entreprise saintaise Lyspackaging fabrique des flacons pour la filière cognac n Elle lance la marque Veganbottle n Des bouteilles en bioplastique à la recette secrète." date="2017 03 21" journal="Charente Libre  Toutes">
Agnès MARRONCLE   Il veut jeter les bouteilles en polytéréphtalate d'éthylène (PET)! Non pas dans la nature où elles rejoindraient le septième continent d'ordures flottantes, mais aux oubliettes de l'histoire industrielle. Pour Nicolas Moufflet, l'avenir c'est le bioplastique et notamment le sien, créé sous la marque Veganbottle même si aujourd'hui son entreprise vit grâce au PET.   Le dirigeant de Lyspackaging, entreprise créée à Saintes en 2015, a beaucoup fréquenté la filière cognac.  «Au départ, je suis designer. J'ai travaillé pendant quinze ans sur la conception et numérisation de modèles de flacons à Cognac mais aussi en Champagne. Puis j'ai monté cette entreprise de création et production de bouteilles, en particulier pour la filière vin et spiritueux que je connais bien.» La matière de base majoritaire actuellement chez Lyspackaging, c'est donc le polytéréphtalate d'éthylène, moins lourd que le verre, moins cher aussi et apprécié pour les exportations d'eau-de-vie.  «J'ai gagné des marchés auprès de maisons de cognac en proposant des flacons en PET plus attrayants que ce qui se fait.»  Ainsi, la TPE installée dans la zone des Coteaux à Saintes est-elle passée en deux ans de trois personnes au démarrage à neuf aujourd'hui, dont deux intérimaires que Nicolas Moufflet compte bien intégrer à la faveur du développement de Veganbottle.   Sous cette nouvelle marque officiellement lancée depuis janvier, il commercialise des bouteilles et flacons en bioplastique fabriqués à partir de bagasse de canne à sucre. Un prix «Émergence» de BPI France et le soutien de l'incubateur régional de l'innovation lui ont permis de mettre au point une recette propre.  «Je la garde secrète, mais je ne l'ai pas brevetée. Je me méfie de cette procédure. Rien n'est divulgué pendant dix-huit mois, mais ensuite le brevet est publié et il n'est pas difficile de contourner la protection en modifiant juste un petit élément.» Pas un gramme de pétrole   Ce qu'il faut savoir sur la matière exploitée par Veganbottle, c'est qu'elle est entièrement biosourcée et ne contient donc pas un gramme de pétrole. Les bouteilles sont biodégradables, compostables à condition qu'être hachées pour hâter la dégradation, bref des produits vraiment recyclables,  «contrairement à ce qu'on dit sur le PET»! «Seulement 25% des emballages fabriqués dans cette matière parviennent aux filières de revalorisation. De surcroît, le PET n'est réellement recyclable que deux ou trois fois.» La matière de Veganbottle est bien plus écologiquement correcte et c'est sur cette tendance que compte naviguer Nicolas Moufflet. Ses premiers clients sont dans le commerce du bio, épices et herbes de Provence, cosmétiques naturels,  «là où il y a une cohérence entre le contenu et le contenant» .   Pour que le client, in fine, parvienne bien à différencier un flacon Veganbottle d'un contenant en PET, l'entrepreneur saintais a eu l'idée d'intégrer une coloration elle aussi naturelle et qui plus est issue de déchets végétaux.  «Par exemple, nous broyons des noyaux d'olives et les intégrons au bioplastique, ce qui donne une teinte et un grain particulier à la bouteille d'huile d'olive.»  Idem avec des pelures de carottes broyées qui offrent un bel oranger, parfait pour contenir du jus de carottes. Les essais réalisés avec les brisures de coquilles d'huîtres donnent une matière à l'allure de pierre blanche.  «Cela serait pas mal pour un vin blanc accompagnant les huîtres!» 30% plus cher qu'une bouteille plastique   Côté marketing, les idées ne manquent pas, mais il faut aussi convaincre sur le prix les fabricants de liquides divers, car une bouteille en bioplastique revient pour l'instant 30% plus cher que sa soeurette en PET.  «C'est le pourcentage aujourd'hui, mais le prix du pétrole va grimper tandis que celui du bioplastique baissera à mesure qu'on en produira. Un jour, ces courbes se croiseront.»  Sans compter les raisons environnementales qui pourraient amener à devoir préférer des bouteilles issues du végétal à celles de la pétrochimie.  «Regardez, c'est bien arrivé pour les sac plastique» , fait valoir le jeune patron saintais.
</article>
<article titre="[L'innovation VeganBottle® est en fait une...]" date="2017 03 15" journal="Sud Ouest  Charente">
L'innovation VeganBottle® est en fait une nouvelle matière, issue de bagasse de canne à sucre
</article>
<article titre="Lyspackaging met du tout végétal dans l'emballage" date="2017 03 15" journal="Sud Ouest  Charente">
SÉVERINE JOUBERT   L'idée d'une bouteille 100 % végétale, Nicolas Moufflet, le fondateur de Lyspackaging, l'avait depuis longtemps. Le voilà désormais en mesure de proposer un contenant entièrement " végétal, compostable et biodégradable ".   Et ses propos ne relèvent pas du fantasme puisque le chef d'entreprise de 41 ans a déjà commercialisé un million de flacons végétaux depuis le lancement, en septembre 2016. Ses clients ? Des producteurs d'épices bio, de jus de fruits, le secteur de la cosmétique. " Créer un code visuel "   La petite révolution se joue dans la zone d'activité du parc des Coteaux, à l'endroit même où est née l'entreprise le 12 janvier 2015. Mais aujourd'hui, Lyspackaging est à l'étroit et elle cherche à déménager. " On a commencé à trois. On est maintenant 7 plus 2 intérimaires. "   Pour démarrer son activité, Nicolas Moufflet a eu recours au PET (polytéréphtalate d'éthylène), considéré comme plus écologique car 100 % recyclable, plus léger que le verre et donc moins énergivore dans le transport, idem dans sa fabrication. Le patron propose un catalogue ou réalise sur mesure des bouteilles, flacons, carafes et flasks. La production annuelle est de 5 millions d'unités en PET.   Son objectif est clair : amener la totalité de ses clients vers la bouteille végétale et en conquérir de nouveaux, évidemment. " Le 100 % végétal c'est faisable ", assène-t-il.   L'innovation VeganBottle® est en fait une nouvelle matière, issue de bagasse de canne à sucre. Pour y parvenir, Nicolas Moufflet a travaillé avec un laboratoire caennais, Biopolynov. Son projet a reçu le soutien du BPI Émergence et du Fonds européen de développement économique régional (Feder). Nicolas Moufflet montre les granulés blancs et le plastique transparent végétal. Bluffant.   Mais parce qu'il ne veut pas que cette matière 100 % compostable se fonde au milieu du verre ou du plastique pétrole, il a apporté une touche marketing au produit en lui incorporant des paillettes végétales, issues de coquilles d'huîtres, de moules, de pépins de raisin, d'épluchures de fruits, de noyaux d'olive, etc. " On est au moment où il faut créer un code visuel ", plaide-t-il.   Preuve que les possibles sont immenses, le jeune patron déploie des échantillons de plastique de toutes les nuances. Lesquelles teintes peuvent se retrouver en poudre dans la paroi du flacon ou carrément constituer le bouchon. " Il n'est plus question d'emballages à base d'énergies fossiles mais bien d'un écoemballage s'inscrivant dans un schéma d'économie circulaire. "   30 % plus cher   Dans l'absolu, la bouteille végétale peut donc être jetée dans la poubelle à compost. Pour accélérer sa dégradation, Nicolas Moufflet suggère une pré-étape de broyage. L'usage n'en est pas là mais le patron de Lyspackaging est confiant sur l'avenir puisqu'il inscrit la bouteille verte dans la nouvelle politique des sacs de caisse compostables. " Une très bonne chose. " Le discours est rodé, convaincu et militant. Ainsi n'hésite-t-il pas à dire que si le geste de tri est salvateur, il ne faudrait pas oublier que " le plastique pétrole n'est pas recyclable à l'infini. "   Et le coût ? " 30 % de plus que le PET ", assume-t-il. Un investissement qui en vaut la chandelle et qui semble susciter l'intérêt des entreprises puisque de nombreux essais sont réalisés. Le flaconnage est par exemple en test pour la nouvelle ligne de cosmétique de Flora Coquerel et un grand groupe de vins et spiritueux fait les yeux doux au bouchon 100 % recyclable. Nicolas Moufflet concède manquer de recul sur l'évolution dans le temps de cette nouvelle matière au contact de certains produits.   En deux années d'existence, Lyspackaging a en tout cas fait montre de son audace. Nicolas Moufflet la projette payante, avec espère-t-il, un chiffre d'affaires en 2017 de 700 000 euros contre 350 000 euros en 2016.   Lyspackaging, 19 rue Champagne-Saint-Georges, 17810 Saint-Georges-des-Coteaux. Tél. 09 52 92 95 68.
</article>
<article titre="Albioma dépasse ses objectifs grâce à la bonne performance de ses centrales" date="2017 03 1" journal="AFP Infos Economiques">
Paris, 1 mars 2017 (AFP) - - Le producteur français d'électricité Albioma, surtout présent outre-mer, a vu ses résultats progresser en 2016, dépassant même ses objectifs, grâce la bonne performance de ses centrales et aux renégociations tarifaires avec EDF à La Réunion.   L'an dernier, le bénéfice net a atteint 33 millions d'euros en hausse de 9%, alors que le groupe tablait sur un résultat compris entre 25 et 30 millions d'euros. Le chiffre d'affaires s'affiche à 367,8 millions d'euros, en croissance de 5% et l'excédent brut d'exploitation a progressé (+10%) à 131,4 millions d'euros, là encore au dessus de la fourchette visée entre 122 et 130 millions.   Le groupe a notamment pu compter sur une production stable de ces centrales thermiques de l'outre-mer français.   Albioma a également obtenu auprès d'EDF une hausse tarifaire pour sa centrale hybride bagasse/charbon de Bois-Rouge à La Réunion, dans le cadre de nouveaux avenants au contrat de vente de l'électricité produite.   La production de ses centrales solaires est restée stable du fait de conditions d'ensoleillement "moins favorables".   Enfin au Brésil, malgré une saison sucrière écourtée, entrainant une baisse des volumes de canne à sucre disponible pour faire tourner ses centrales biomasse, Albioma a pu compter sur l'amélioration du rendement de ses installations.   Cette année, Albioma prévoit de mettre en service de nouvelles centrales, comme celle de Galion 2 à Fort-de-France. Ce projet est toutefois suspendu à une procédure administrative, après l'annulation de l'autorisation d'exploiter décidée par la justice. Le groupe a fait appel.   Le groupe a également remporté l'an dernier trois projets de centrales solaires avec stockage à la Réunion et en Guadeloupe qui doivent entrer en service en 2018/2019.   Pour 2017, Albioma vise un Ebitda compris entre 130 et 138 millions d'euros et un bénéfice net entre 25 et 30 millions, tablant ainsi sur la poursuite de son développement.   mhc/ef/mml
</article>
<article titre="Le producteur EnR Albioma arbore de bons résultats 2016" date="2017 03 1" journal="GreenUnivers (site web)">
2017-03-02T11:49:18+00:00    Le producteur français d'énergie renouvelable Albioma a présenté ce matin de bons résultats 2016 avec un chiffre d'affaires de 367,8 M€ (+5%) et un résultat net part du groupe de 33 M€ (+9%). Si les résultats sont meilleurs que prévu, 2017 ne devrait pas être une année mirobolante. "Le véritable pallier supplémentaire sera l'année 2018", avertit Frédéric Moyne, directeur général. Pour 2017, l'objectif est un résultat net part du groupe de 30 à 35 M€. Suite à ces annonces, l'entreprise - dont 65,6% du capital est en Bourse - a vu son action prendre +2,52% ce mercredi sur Euronext Paris. Passer à 80% de renouvelables en 2023  Son objectif est de passer de 50% d'énergie renouvelable en 2016 à 80% en 2023. Pour ce faire, Albioma cite trois pistes : convertir ses unités bagasse-charbon en centrales 100% biomasse, construire dans l'outre-mer français des centrales 100% biomasse et du solaire photovoltaïque, et enfin, acquérir et développer des centrales 100% bagasse au Brésil.Quel avenir pour le projet Galion 2 (40 MW) ? Malgré le recours juridique en cours contre cette centrale martiniquaise 100% bagasse/biomasse dont la construction a débuté en 2015, le groupe s'est dit aujourd'hui très confiant. Selon lui, les juges d'appel n'empêcheront pas l'entrée en service de ce site, attaqué pour des questions de pollution des eaux. Toutefois, la mise en service est repoussé d'un semestre, à la fin 2017.Une transition qui ne se fera pas grâce à la méthanisation : Albioma laisse en sommeil cette branche. Sans surprise, le groupe va se contenter de conserver ses trois centrales en exploitation (Tiper, Cap'ter et Sain'ter Méthanisation), héritées du rachat de Methaneo en 2012, à l'époque ou Albioma s'appelait encore Séchilienne Sidec. Le retour à l'équilibre économique de cette activité est escompté pour 2017. D'une puissance totale de 3 MW, ces unités se situent dans les départements d'outre-mer et leur production est stable à 19 GWh en 2016 (contre 20 GWh en 2015). Pour rappel, la durée des contrats d'achat passe de 15 à 20 ans suite à un arrêté ministériel de décembre 2016, publié cette semaine au journal officiel.   Garder le rythme au Brésil et conquérir d'autres pays  La variable d'ajustement sera donc plutôt au niveau de la biomasse brésilienne. Sur une capacité installée totale de 753 MW, le groupe détient 108 MW au Brésil. S'ajoutent 455 MW dans l'océan indien (La Réunion, l'île Maurice et Mayotte), 174 MW entre la Guadeloupe, la Martinique et la Guyane et 16 MW en Europe. Au Brésil, le rendement des centrales biomasse/charbon de Rio Pardo et Codora a augmenté de 23%. Désormais, une tonne de canne à sucre permet de produire 76 KWh. Prochaine étape : l'extension de la centrale de cogénération (16 MW) de son partenaire, la distillerie Vale do Parana. Sur les 48 MW qui seront atteints, 30 MW seront exportés sur le réseau électrique à compter de 2021.Albioma, spécialiste de la valorisation de la bagasse. La bagasse est un résidu solide et fibreux de la canne à sucre après passage au moulin. Il faut en moyenne 1 tonne de canne à sucre pour produire 300 kg de bagasse. Selon les pays, la campagne sucrière dure 5 à 9 mois, ce qui fait de la bagasse un combustible complémentaire et non pas disponible toute l'année. Albioma couple cette ressource à du charbon, combustible fossile qu'elle veut remplacer par de la biomasse durable. Parmi les pistes : la vinasse, coproduit de la canne à sucre.La production totale de l'activité biomasse thermique au Brésil se maintient à 238 GWh en 2016, contre 241 GWh en 2015. Le producteur réitère son objectif de lancer dans le pays un projet tous les 12 à 18 mois. Outre le Brésil, Albioma espère à terme exporter sa solution bagasse dans d'autres pays d'Amérique latine, telle que la Colombie, ou encore en Asie, mais ni en Chine ni en Inde.   Prêt pour le prochain AO solaire dans les ZNI  En juin 2016, le groupe avait raflé la mise en remportant 5,9 MW de solaire avec stockage - soit trois centrales - dans le cadre de l'appel d'offres ZNI (pour Zones non interconnectées). L'appel d'offres en cours, qui se clôture en juin 2017, suscite également toute son attention. "Nous serons bien sûr candidats. Nous positionnerons les projets non lauréats du précédent appel d'offres, qui représentent au total 14 MW, mais notre offre totale dépassera largement cette puissance", indique Pascal Langeron, directeur général adjoint d'Albioma. Le groupe se positionnera notamment en Guyane, à Mayotte et, contrairement à l'édition précédente, en Martinique. En revanche, il ne déposera aucun dossier dans l'AO spécifique à la Corse.   Monitoring dans le solaire  Le solaire est, avec la biomasse, l'autre axe stratégique du groupe. Ce dernier estime détenir aujourd'hui 17% de parts de marché dans les DOM dans ce secteur. C'est moins que dans ses autres activités : à La Réunion, 48% de l'électricité produite l'est par Albioma, selon les chiffres du groupe. Ce pourcentage est de 42% à Maurice et de 31% aux Antilles. Par ailleurs, en 2016, la hausse de 10% de l'EBITDA est due aux centrales thermiques du groupe (+14%) et se fait malgré la baisse de l'EBITDA de l'activité solaire (-3,7%).   Pour augmenter sa valeur ajoutée dans le photovoltaïque, le groupe mise sur des solutions de  monitoring  de pointe. "Nous travaillons notamment avec la start-up Reuniwatt, spécialiste de la prévision de la production photovoltaïque, rappelle le directeur général adjoint. Son progiciel est basé sur l'accumulation de données sur La Réunion : des données temps réel et des données des dix dernières années". Pour l'appel d'offres de 2012, les prédictions étaient fiables à 80%. Le taux est aujourd'hui de 95%, ce qui a besoin d'être encore amélioré : Albioma est payé par EDF pour sa disponibilité et supporte des pénalités en cas de défaut de livraison. D'où l'utilité du stockage dans les ZNI, technologie sur laquelle le groupe focalise une grande partie de son énergie.Actionnariat. Au 9 février 2017, l'actionnariat se répartit de la manière suivante : 65,6% de flottant, 13,4% pour Altamir et affiliés, 7% pour la CDC, 5,9% pour la Compagnie financière européenne de prise de participation (Cofepp), 5,1% pour la Financière de l'Echiquier. Le solde (3%) se divise entre les salariés (1,4%), de l'autodétention (1,2%) et les administrateurs, hors Altamir (0,4%). Pour rappel, la part de flottant a augmenté en 2015 quand Apax Partners est sorti d'Albioma après 10 ans passés à son capital.  Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="Albioma (ex-Sechilienne-Sidec) : montée en puissance des capacités" date="2017 03 1" journal="Boursier (site web)">
Périmètre.  Sous l'effet d'une performance satisfaisante du côté de ses centrales thermiques, Albioma affiche des revenus annuels en hausse de 5%. Aucun mouvement social n'est venu entacher la période alors que les installations ont fonctionné correctement. De son côté, l'Ebitda progresse de 10% à 131 ME contre 120 Millions d'Euros un an plus tôt. Un certain nombre de développements sont en cours qui devraient permettre d'augmenter graduellement le périmètre de l'entreprise. Le métier historique d'Albioma consiste à produire de l'électricité à partir de la bagasse qui est issue du broyage de la canne à sucre. En dehors des périodes sucrières, le combustible de ces centrales est le charbon mais il est remplacé progressivement par des résidus du bois. De nouvelles centrales sont en construction, l'une à la Martinique et l'autre à la Réunion. Solaire.  Albioma s'est aussi installé au Brésil avec deux centrales aujourd'hui en activité. La feuille de route dans le pays est ambitieuse car il s'agit d'en mettre en service une nouvelle centrale tous les 12-18 mois. Dans ce cadre, des financements appropriés ont été négociés dans des conditions intéressantes. Depuis 2013, Albioma s'est aussi diversifié dans la biométhanisation. Ce procédé permet de produire du biogaz et des fertilisants à partir de déchets agricoles. La société intervient également dans l'énergie solaire avec un parc situé en outremer qui représente actuellement 70 ME. De nouvelles capacités vont être déployées à horizon 2018-2019. Avec l'augmentation progressive des capacités, les différents indicateurs financiers sont appelés à augmenter lors des prochains exercices. 2017.  Albioma fait état d'un Ebitda 2017 prévu entre 130 ME et 138 Millions d'Euros, soit une légère progression sur les chiffres affichés en 2016. Avec un modèle prévisible et structurellement générateur de cash-flow, la valeur est intéressante. Elle distribue aussi un coupon à prendre en compte dans la mesure où le rendement est de 3,4%. Même si la progression de cours est de 40% sur une année glissante, le titre Albioma ne semble pas excessivement cher compte tenu de ses qualités. Nous sommes donc incités à le maintenir en fond de portefeuille d'autant plus que le flottant est très élevé. Avec des taux d'intérêt toujours attractifs, une opération sur le capital d'Albioma est loin d'être une hypothèse saugrenue... La ligne Albioma sera conservée.Christophe Voisin
</article>
<article titre="Spécial CFIA 2017. L'emballage écolo et 4.0" date="2017 02 23" journal="Process Alimentaire (site web)">
Le recyclabilité des matériaux devient une exigence qui va impliquer de trouver des alternatives économiquement viables aux multi-couches barrières.   L'éco-conception et le digital seront les tendances les plus marquantes des halls dédiés à l'emballage et au conditionnement sur le CFIA 2017, qui se tiendra du 7 au 9 mars au Parc Expo de Rennes Aéroport. Illustration à travers quelques nouveautés.  En 2022, tous les foyers français seront autorisés à mettre l'ensemble de leurs emballages plastiques dans les sacs jaunes. La profession se prépare depuis déjà un moment et les offres d'emballages mono-matériaux qui simplifieront les flux de recyclage (et réduiront également les éco-contributions des entreprises) se multiplient.Sur le marché très tendance des poches souples, par exemple, l'enjeu est de pouvoir sortir des films complexes trop difficiles à recycler. Ceisa Packaging et Malengé proposent chacun à leur façon des débuts de solutions en la matière sur leurs stands respectifs.    Côté emballages rigides, Knauf Industries expose ses barquettes en Kapseal mono-matériau à base de polypropylène destinées au marché de la viande sous atmosphère protectrice. CGL Pack dévoile sa gamme standard So'Pack de barquettes en mono-PET (contenant même du rPET) Clearseal pour toutes les applications. Du côté des films rigides, SP Group réduit l'épaisseur globale des emballages mono-matériaux avec son APET Efficient, tandis que Klöckner Pentaplast montre des applications de son APET ++, dont certaines sont associées aux films d'operculage de Linpac scellables sur de l'APET.    Les matériaux rivalisent de vertus écologiques   Toujours dans le plastique, les nouveautés « vertes » viennent également des barquettes et autres emballages souples biodégradables et compostables. A l'image, par exemple, des nouvelles barquettes préformées de Nutripack ou des films Nativia de Taghleef Industries. Sur le salon, les emballages en bois de Lelu et Lacroix ou en carton vont également faire valoir leurs atouts environnementaux. Les boîtes Can Packaging passent ainsi aux formes carrées pour se distinguer, les barquettes et boîtes Packalim sont complétées par le Pulp'Pack, un emballage en pulpe de canne à sucre (aussi appelé bagasse), biodégradable et compostable. Les barquettes en carton de Guillin passent au four micro-ondes et permettent aux cuisines collectives de personnaliser leur offre. Tandis que RAP emballe des sandwichs dans du papier/carton et que DS Smith remplace les barquettes de biscuits en plastique par des compartiments en carton.   L'Internet des Objets   Côté production des emballages, le digital est à l'honneur à travers l'Internet des Objets : il représente les échanges d'informations et de données provenant des machines vers le réseau Internet. Cela se concrétise par des capteurs qui remontent des informations dans l'objectif d'anticiper les pannes, de mesurer l'efficacité des lignes, de réduire les temps d'arrêt, voire d'abaisser les consommations énergétiques. Parmi les nombreux exemples figure le tout nouveau CogniPro Link de Sealed Air. OET, de son côté, a développé un îlot robotisé de palettisation de sacs qui est équipé d'un préhenseur connecté à l'aide de capteurs sans fils (LoRa). BA Systèmes propose aussi un logiciel de supervision qui permet de suivre l'activité de sa flotte de chariots AGV (Automated Guided Vehicle) et la performance des flux de production en temps réel ou en analyse a posteriori. Le marquage/codage n'échappe pas à la tendance. Exemple avec le cloud de Domino qui fournit des diagnostics, une surveillance à distance des imprimantes ainsi qu'un système de reporting client.   Visualisation 3D des emballages   Les outils 4.0 se mettent aussi au service de la conception d'emballages, dans laquelle s'imbrique la chaîne graphique. Visiativ, co-organisateur du Congrès « Entreprises du Futur » à Lyon en janvier dernier, illustre cette tendance avec le logiciel Perfect Package de Dassault Systèmes qui permet de centraliser les contraintes de toute nature (techniques, réglementaires,..) et d'intégrer tous les acteurs de la chaîne packaging grâce au langage commun que constitue la visualisation 3D, depuis le design jusqu'au test volume en rayon. De quoi gagner en temps et en efficacité de mise sur le marché.   Cet article est paru dans Process Alimentaire (site web)
</article>
<article titre="VALLESPIR ET CONFLENT LA RÉSERVE NATURELLE DE PRATS-DE-MOLLO EN HIVER" date="2017 02 16" journal="L'Indépendant  CATALAN_IN">
La réserve naturelle de Prats-de-Mollo La-Preste, c'est un espace merveilleux pour les amoureux des grands espaces. Entre 1600 et 2300 mètres d'altitude, de belles balades hivernales et originales offrent de magnifiques panoramas sur un massif peu fréquenté en hiver. Le tout sous l'oeil vigilant des vautours, des aigles et des gypaètes. â?º La boucle de Cagallops (2,5 km). Du refuge, partir vers le sud et suivre le sentier d'interprétation. Descendre dans le vallon puis remonter doucement dans la hêtraie. Ignorer la piste de gauche descendante et atteindre le Roca Rodona. Bifurquer à droite pour rejoindre la clairière du col de Baix, avant de revenir à droite au refuge. La cabane d'En Miquel (9 km). Du refuge, redescendre la route d'accès jusqu'à l'épingle à cheveu (1). Chausser les raquettes et suivre la route vers la gauche. Dépasser le refuge des Forquets et les panneaux marquant l'entrée de la Réserve naturelle (2), avancer dans la hêtraie puis sortir à découvert. Passer sous un petit éperon rocheux suivi d'une pinède et sortir à nouveau à découvert. Au panneau "La Togne" (3), deux options : les moins sportifs suivent la piste via le Col des Basses (4), jusqu'à revenir à l'aplomb de "La Togne" (5, 1850 m). Les plus sportifs quittent la piste pour suivre à gauche le thalweg formant la partie haute du ravin de la Bagasse. L'itinéraire est jalonné de piquets à la peinture jaune, certains invisibles sous la neige. Monter rectiligne au début puis obliquer vers l'est jusqu'à rejoindre la piste vers les bosses méridionales. Couper la piste (5) et aller vers le mamelon le plus au sud afin de rejoindre la cabane d'En Miquel (6), un orri régulièrement enseveli par une grosse congère. Superbe panorama sur le Haut-Vallespir et les Garrotxes espagnoles. Pour le retour, revenir par le même chemin ou les beaux champs de neige du vallon de la Bagasse. Les Esquerdes de Rotja (16,5 km). Depuis la cabane d'En Miquel, aller vers le nord en suivant les courbes de niveaux des petits mamelons jusqu'à atteindre le col de Bise (7, 1921 m). Ignorer la piste encombrée de congères et grimper le long de la ligne de crête. Rejoindre la piste dans le virage en épingle (8, 2005 m) et poursuivre l'ascension jusqu'à une série de chaos granitiques. Redescendre au col des Molles (9), longer l'enclos par la droite et monter de biais vers le nord-ouest en coupant plusieurs fois la piste jusqu'à atteindre la Collada des Roques Blanches (10, 2252 m). Passage au col de Bise enneigé et sous le soleil.
</article>
<article titre="[Sublimes panoramas sur les sommets...]" date="2017 02 16" journal="L'Indépendant  CATALAN_IN">
Sublimes panoramas sur les sommets de la Cerdagne, du Capcir, du Haut-Conflent et du Haut-Vallespir. Aller vers l'ouest, sans passer par les petites buttes sommitales ni suivre l'itinéraire estival du Tour des Réserves Naturelles situé 50 m plus bas : se maintenir au mieux sur la même courbe de niveau vers 2250 mètres d'altitude jusqu'à se rapprocher du versant nord du sommet du Ras del Garbé, côté 2316 m (11). Sa pente devient déversante avec une pente de 80% (ou 40°). Ce secteur, le seul partiellement boisé de jeunes pins à crochets, peut poser problème en cas de neige dure, ou pire, verglacée. La glissade ou la chute sont absolument interdites. Si les conditions de traversée de ce passage long de 200 m ne sont pas satisfaisantes, privilégier de passer par le sommet (+ 40 m), en suivant la clôture. La descente sur le versant occidental ne pose pas de problème. Parvenir à un petit col rocheux et suivre le versant méridional, entre la clôture et les buttes cotées 2268 m et 2259 m, jusqu'à rejoindre la Porteilla des Avets (12, 2229 m). Passer versant nord et avancer de 400 m maximum en longeant les buttes rocheuses au plus près. Découvrir dans la dernière d'entre-elles la statue de la Vierge noire des Esquerdes (13). Revenir à la Porteilla des Avets et franchir la clôture au sud. Se diriger vers le sud-est, en restant le plus haut possible parmi les jeunes pins à crochets et en cherchant quelques éphémères piquets peints en jaune. Ne pas descendre dans la combe mais la contourner par la gauche, traverser un petit bois puis un champ de neige vers la droite, à l'aplomb de gros rochers granitiques rosâtres (14). Les contourner dans la forêt et atteindre le Pla des Eugues (15). Délaisser le balisage estival partant à droite dans la forêt, car l'orientation y est rendue difficile en absence de balisage hivernal conséquent. Traverser le Pla des Eugues dans la longueur vers l'est et emprunter l'ancienne piste de ski, encore bien marquée. A mi-pente, ignorer la grande clairière du col de Baix à droite et rejoindre le refuge de Las Conques. Texte et photos : Christophe Levillain Sur la boucle de la hêtraie de Cagallops, après une grosse chute de neige. Dans le passage délicat du versant du Ras del Garbé (ici en bonnes conditions). Dans la montée du vallon de la Bagasse.
</article>
<article titre="Sucrerie de Marie-Galante: subvention versée dès qu'un accord est trouvé (Bareigts)" date="2017 02 14" journal="AFP Infos Economiques">
Paris, 14 fév 2017 (AFP) - - L'Etat est prêt à verser la subvention de 1,6 million d'euros qu'attend la sucrerie de Marie-Galante, en Guadeloupe, a dit mardi la ministre des Outre-mer Ericka Bareigts, dès qu'un "accord sera trouvé" autour de deux projets de centrales thermiques en concurrence.    La ministre répondait, lors des questions au gouvernement, au sénateur de Guadeloupe Jacques Cornano (app. PS), inquiet des incertitudes autour de la prochaine campagne sucrière, qui doit débuter à la mi-février. "L'Etat s'est engagé à verser une subvention de 1,6 million d'euros, subordonnée à un accord avec un opérateur dont la technologie n'est plus en phase avec la Cop 21", a-t-il déploré, réclamant le "déblocage de cette subvention afin d'éviter une tragique explosion sociale" dans l'île.   Les conventions qui conditionnent le versement d'aides publiques à la filière sucre prévoient pour la sucrerie de Marie-Galante la construction d'une centrale thermique pour moderniser son fonctionnement et externaliser la production d'électricité, mais cette construction n'est toujours pas lancée, car deux projets s'opposent.   "Compte tenu des contraintes particulières de l'île, la sucrerie doit faire face à des défis économiques et environnementaux particulièrement importants et il est nécessaire pour garantir sa survie de coupler son activité à une centrale thermique", a expliqué la ministre.   Selon elle, seul "le projet d'Albioma", producteur d'électricité par valorisation énergétique de la bagasse (résidu de la canne à sucre) avec la société Sucreries et Rhumeries de Marie-Galante (SRMG), est "suffisamment avancé pour remplacer rapidement la chaudière de la sucrerie", a déclaré la ministre.    "Mais cette mise en oeuvre n'obère en rien le développement de l'initiative +Marie-Galante Ile du tourisme durable+", portée par la communauté de commune de Marie-Galante (CCMG) et la Compagnie Nationale du Rhône (CNR), a-t-elle ajouté.   "Je suis convaincue que les deux projets ne sont pas concurrents mais compatibles", a insisté la ministre, estimant que "le projet de la centrale thermique d'Albioma peut s'intégrer dans le projet plus global du CNR, comprenant aussi des installation éoliennes et du photovoltaïque".   "Pour que le projet d'Albioma réponde davantage aux attentes des élus locaux, je souhaite que l'entreprise puisse prendre des engagements sur les subtitutions partielles de la biomasse", a demandé Mme Bareigts, assurant que "l'Etat est prêt à verser les 1,6 million d'euros dès qu'un accord sera trouvé sur le terrain".   caz/frd/pb
</article>
<article titre="Discours de François Fillon à Saint-Pierre de La Réunion" date="2017 02 12" journal="NewsPress (français)">
Seul le prononcé fait foi.   Mes chers amis,   Et, parce qu'ici ce mot a un retentissement tout particulier, mes chers compatriotes, Merci ! Merci d'être venus si nombreux !   Je retrouve toujours la Réunion et les Réunionnais avec un immense bonheur.   J'aime cette île qui est fière de son histoire, de sa culture, de ses valeurs, fière de sa diversité, qui est fière d'être elle-même et qui est fière d'être la France.   Nous sommes 66 millions de Français dans un monde de sept milliards d'habitants. Personne ne nous fera de cadeaux.   Notre seule marge de manoeuvre réside dans notre capacité à nous rassembler, à travailler plus, à travailler tous.   Notre avenir est dans notre inflexible volonté d'être souverain, écouté et respecté de par le monde.   Etre debout ou à genoux : voilà la question. Ma réponse est claire : debout toujours, à genoux, jamais !   Je veux, avec vous, avec la Réunion, faire de la France une puissance gagnante.   La Réunion peut être fière. Elle a donné tellement de grandes figures à notre nation. Des héros et des combattants, comme Juliette Dodu, comme Roland Garros, comme Jean Joly, qui se battirent pour l'honneur de la France.   Des poètes et des intellectuels, de Lacaussade à Leconte de Lisle.   Des hommes politiques comme Raymond Barre à qui je pense parce qu'il disait la vérité.   Des sportifs qui font gagner la France, de Jackson Richardson et Daniel Narcisse à Dimitri Payet.   A la Réunion on sait ce que signifie résister, créer, gagner !   Oui je suis heureux de vous revoir, de revoir tous vos visages.   Didier Robert, Michel Fontaine, Nassimah Dindar, Jean-Paul Virapoullé et tant d'autres visages familiers.   J'ai toujours dans mon coeur ce voyage de 2009, lorsque j'étais Premier ministre et que nous avons ensemble inauguré la route des Tamarins, ce chantier ambitieux et spectaculaire, qui fait notre fierté.   Si j'avais l'honneur de présider notre pays, je serais très heureux de venir inaugurer la nouvelle route du littoral, cher Didier Robert, cet ouvrage d'art novateur et indispensable à la vie des Réunionnais, que nous avons lancé ensemble.   Didier, je garde aussi dans mon coeur ce meeting que nous avions organisé pour les élections Régionales. C'était déjà ici, à Saint-Pierre, grâce à Michel Fontaine. On s'était battu ensemble, pour la victoire !   Mes amis, c'était il n'y a pas si longtemps, et pourtant comme cela paraît loin !   Long et insipide fut le quinquennat de François Hollande, qui est aussi, il faut le rappeler, celui de son conseiller, Emmanuel Macron.   Cinq ans de perdus pour la France.   Cinq ans de perdus pour les Outre-mer !   Pourtant, de belles promesses vous avaient été faites.   Lui président, tout devait aller mieux. Le chômage allait reculer. Les prix allaient baisser. La sécurité serait assurée. Les enfants auraient un avenir radieux. Et tout le monde pourrait se loger décemment.   Que reste-t-il de ces promesses, après cinq années d'exercice du pouvoir par Hollande?   Le chômage, la pauvreté sont toujours beaucoup trop élevés. Rien de fort n'a été fait pour les faire reculer.   L'insécurité atteint des niveaux jamais connus partout Outre-mer, et notamment ici, à la Réunion.   L'Etat et la justice sont en difficulté. En grande difficulté. Il y a trop de délinquance et pas assez de sanctions.   L'échec scolaire est un fléau.   La construction de logements a pris un retard dramatique.   Et puis il y a l'immigration clandestine.   Il est temps de serrer les vis et de rappeler que la France est une nation, pas une passoire.   Sur l'île voisine de Mayotte, l'immigration clandestine est une menace pour la cohésion sociale.   Je veux le dire à nos amis mahorais qui sont parmi nous ou qui nous écoutent : ce que vous vivez au quotidien n'est plus acceptable.   La pression migratoire à Mayotte est insupportable.   Il faut réagir.   Je veux qu'à Mayotte et en Guyane, l'acquisition de la nationalité française soit subordonnée à la régularité du séjour des parents.   Un enfant né à Mayotte ou en Guyane de parents en situation irrégulière ne pourra plus devenir automatiquement Français à sa majorité.   Mes chers amis, le temps est venu d'agir pour l'ensemble des Outre-mer, d'agir maintenant, d'agir pour vous.   Et il faut le faire immédiatement, pas dans dix ou vingt ans comme le proposent les socialistes avec leur loi pour l'égalité réelle.   Avec eux, l'égalité, c'est toujours pour demain !   Vous voulez des actes rapides, et vous avez raison !   La première mesure que je prendrai pour les Outre-mer, si les Français me font confiance en mai prochain, c'est le retour de l'Etat dans le financement de la continuité territoriale.   Je considère en effet que le retrait de l'Etat de ce dispositif, voulu et organisé par le gouvernement socialiste, est une faute politique lourde. Didier Robert a eu raison de maintenir coûte que coûte ce dispositif pour tous les réunionnais   Où est l'égalité lorsque les Français d'Outre-mer sont empêchés de se rendre dans l'hexagone à cause du renchérissement du prix des billets d'avion provoqué par cette décision injuste ?   Où est l'égalité lorsque l'on prive des familles d'ici du bonheur simple de retrouver une fille ou un fils parti faire ses études en métropole ?   Avec le retour de l'Etat dans le financement de la continuité territoriale, je plaiderai aussi pour que l'Europe accompagne cette politique.   Mes chers compatriotes, vous êtes trop nombreux à connaître, dans votre famille, parmi vos amis, le drame du chômage.   25% de la population active au chômage, parfois plus de 50% des jeunes, c'est inacceptable.   Je refuse d'y voir une sorte de fatalité contre laquelle on aurait déjà tout essayé.   Je veux faire ce que nous n'osons pas faire depuis trop longtemps : faire confiance à l'énergie française.   Nous allons nous détacher de cette chape de plomb idéologique qui a piétiné le goût du travail, piétiné la récompense du mérite, piétiné ce qu'il y a de plus lumineux en nous : la liberté.   La liberté, c'est le carburant des sociétés vivantes.   Avec tous les Français qui veulent réussir, je fonce pour une France du plein emploi, une France productive.   C'est ma priorité absolue !   Vous vous êtes engagés à La Réunion dans un nouveau modèle de développement qui doit conduire les Outre-mer à mieux s'affirmer dans leur bassin géographique, ici l'Océan Indien, là-bas l'Océan Atlantique ou Pacifique.   Je soutiens cette orientation.   Je baisserai massivement les charges qui pèsent sur le coût du travail et sur la feuille de paye des Français afin que chacun voit son pouvoir d'achat augmenter.   Je sortirai des 35 h, sans démagogie, sans diktat, puisque je donnerai aux entreprises la liberté de négocier avec leurs salariés.   Je généraliserai l'apprentissage, refonderai la formation professionnelle, créerai le statut de travailleur indépendant, développerai le statut d'auto-entrepreneur, je miserai à fond sur les technologies.   Je vais libérer l'activité de tous les secteurs qui ont été bloqués par le matraquage réglementaire et fiscal : l'agriculture, l'industrie, les services, la construction.   Je veux agir pour les jeunes qui se cassent les dents sur la rigidité du code du travail. Agir pour les ouvriers qui préfèrent travailler un peu plus que de ne plus travailler du tout.   Et cette restauration de notre production permettra de distribuer, notamment aux plus modestes, les richesses créées au lieu d'égaliser la misère comme aujourd'hui.   Je veux agir sur tous les fronts, secteur public comme secteur privé, en tenant compte des spécificités des Outre-mer, que je connais bien.   L'insularité et l'éloignement, rendent les choses plus difficiles ici qu'ailleurs.   C'est pourquoi, tant que la situation de l'emploi ne sera pas meilleure, tant que le niveau du chômage sera aussi critique, les Outre-mer ne seront pas concernés par le plan de réduction des effectifs de la fonction publique que je mettrai en oeuvre en métropole.   Cela vaut aussi pour les emplois aidés, cher Michel Fontaine, dont le volume sera stabilisé le temps de réamorcer la machine économique.   Mais il faut surtout donner les moyens aux entreprises d'Outre-mer de créer plus d'emplois !   Pour cette raison, je maintiendrai l'ensemble des exonérations fiscales et sociales à leur niveau actuel, pendant une période de 10 ans.   Et cela vaudra aussi pour la défiscalisation, qui sera donc prolongée en conséquence. Les entreprises et les acteurs économiques auront ainsi un environnement durablement sécurisé.   Mais je ne veux pas m'en tenir là.   Je propose que l'Etat consacre deux milliards d'euros par an en faveur de l'emploi Outre-mer.   En plus des baisses de charges qui seront engagées dans le cadre de mon programme national, un dispositif spécifique d'incitation à l'embauche dans les Outre-mer sera créé.   Ce dispositif sera ciblé sur les entreprises de moins de 10 salariés, et celles qui sont les plus exposées à la concurrence.   C'est un effort considérable, mais il est à la hauteur de ce qu'exige la situation.   En définitive, ce que je propose ici, c'est un nouveau cadre d'intervention : celui de « l'entreprise franche ».   Je soutiendrai aussi les activités agricoles, qui sont encore, et de loin, les premiers employeurs privés Outre-mer.   Je connais les inquiétudes de la filière canne, et notamment celle de ses milliers de petits planteurs, face à la perspective de la fin des quotas sucriers en Europe, le 1er octobre prochain.   Je veux les rassurer : avec moi ils auront un puissant soutien de la filière à la tête de l'Etat, comme je l'ai d'ailleurs démontré avec la revalorisation de la prime bagasse lorsque j'étais Premier ministre !   Je veux donc leur dire que je mettrai en place l'aide complémentaire de 38 millions d'euros pour compenser la fin des quotas.   Je mesure l'importance de la complémentarité entre la filière canne et les autres secteurs agricoles qui sont eux aussi fortement pourvoyeurs d'emplois.   Il faudra accompagner leur développement, en renforçant leur enveloppe budgétaire, pour la porter à 80 millions d'euros par an.   Il faut briser la fatalité du chômage.   Il faut donner aux entreprises les moyens de créer des emplois pour tous les Réunionnais.   Il faut aussi que l'Etat assume enfin clairement ses missions de solidarité et d'éducation dans les Outre-mer.   Je ferai de la santé dans les Outre-mer une véritable priorité de l'action de l'Etat en modernisant les hôpitaux, en soutenant l'installation de nouveaux médecins et en développant la télémédecine parce qu'il faut s'emparer de tout ce que le progrès nous apporte.   Plus de solidarité, c'est aussi améliorer la prise en charge des personnes âgées dépendantes et des personnes handicapées, Je sais, chère Nassimah Dindar, que c'est un de tes combats à la tête du Département de la Réunion.   Dans la tradition de notre famille politique, c'est aussi le mien.   On m'a fait le procès de vouloir mettre à mal notre assurance maladie et notre système de soins. C'est faux !   Ce que je dénonce, c'est la langue de bois de ceux qui se fichent des déserts médicaux, qui se fichent des pathologies non remboursées, qui se fichent des urgences débordées, qui se fichent de savoir que les personnels médicaux sont à bout, et puis qui se fichent de savoir que la médecine libérale ne supporte plus d'être maltraitée.   Les fossoyeurs de la sécurité sociale sont ceux qui s'opposent à toute réforme.   Je veux une santé de qualité pour tous, une santé capable de répondre au vieillissement de la population et au prix croissant des nouveaux médicaments.   J'ai créé le CHU en 2009. Je l'ai fait car je crois au rayonnement de l'excellence française en santé dans l'Océan Indien.   Je veux rendre hommage à l'ensemble de ses personnels et à ceux des hôpitaux de la Réunion qui font un travail remarquable dans des conditions souvent difficiles.   Le gouvernement a délaissé le CHU, n'a pas soutenu ses projets et n'a pas pris en compte la réalité de ses missions.   Je suis heureux qu'aujourd'hui le CHU fasse l'objet d'un consensus politique local et je ferai tout pour que cet état d'esprit se maintienne. Je veillerai à ce que le CHU ait les moyens nécessaires à l'exercice de ses missions.   Je veux aussi qu'on puisse vieillir dignement dans notre pays.   Mon gouvernement avait beaucoup fait pour les petites retraites pour les seniors de métropole, les « gramounes » de la Réunion.   Il n'est pas acceptable qu'après 30 ou 40 années de labeur, un agriculteur, un artisan, ne perçoive pas une retraite décente.   Je poursuivrai donc l'effort avec une revalorisation de plus de 300 euros pour les pensions de moins de 1000 euros et d'au moins 600 euros pour les petites pensions de réversion.   Je veux enfin miser sur les grands projets environnementaux et énergétiques.   Je me donne un objectif : zéro charbon et zéro fuel en 2025.   Ça veut dire une meilleure qualité de vie, ça veut dire aussi des techniques de pointe, de la compétitivité pour la France, de l'activité économique nouvelle, ça veut dire de l'emploi.   Ça veut dire aussi de l'attractivité, du tourisme. Ça veut dire la valorisation d'un patrimoine naturel exceptionnel.   Pour l'écologie, la protection de l'environnement, le développement durable, les énergies renouvelables, la Réunion est en pointe et cela fait longtemps. Je me souviens de l'avoir dit et de l'avoir vu de mes yeux dès 2009.   Pour moi l'écologie ce n'est pas la décroissance, c'est plus de progrès, plus de technologies innovantes, et à la clef de nouveaux emplois. Et la Réunion nous montre l'exemple.   La Réunion donne l'exemple en se préparant à devenir la véritable porte maritime de l'Océan indien avec le Port Ouest aujourd'hui et demain avec ce grand projet de Port Est, pour lequel tu oeuvres cher Jean-Paul Virapoullé avec la force de conviction qu'on te connait.   Mes chers amis, l'Etat doit se concentrer sur ses missions essentielles. Je veux que l'Etat protège tous les citoyens contre les violences et l'insécurité.   Je veux rétablir l'autorité. Il faut de l'ordre.   Il faut mettre les mots sur les actes.   Un délinquant n'est pas un mineur qui se cherche. C'est un délinquant!   Un criminel n'est pas une victime de la société. C'est un criminel !   La sécurité doit être partout et pour tous, avec des forces de l'ordre modernisées. Dans les Outre-mer, je veux augmenter dans les Outre-mer les moyens des forces de sécurité et de la justice. Et il faudra pour cela construire des prisons.   La gauche a refusé de le faire, ce fut une décision très grave, et nous allons devoir relancer un programme ambitieux de construction de prisons en métropole comme en Outre-mer.   Mes amis,   La voix des Outre-mer doit enfin être entendue, à Paris comme à Bruxelles.   Je défendrai les Outre-mer devant les instances européennes.   Quand il y a des politiques européennes qui ne correspondent pas aux Outre-mer, la France doit le dire, elle doit se faire entendre et taper du poing sur la table, si nécessaire.   L'Europe n'est pas notre ennemie ! Mais nous ne devons pas la subir. Nous devons l'orienter et mieux encore, la conduire.   Et pour la conduire il faut que nous soyons forts, il faut que nous soyons respectés. Il faut que notre économie fonctionne. Il faut que notre Etat se désendette.   Il faut que la France rayonne en montrant qu'elle a retrouvé la flamme de la création : création d'emplois, de richesses, création technologique, création culturelle.   Mon programme c'est faire que la France soit la première puissance en Europe dans 10 ans.   Je veux que la France fasse la course en tête.   Je veux que la France soit grande, et pas de grandeur sans les Outre mer !   Pour la grandeur, nous allons faire confiance aux talents, aux énergies de tous les Français.   Pour la grandeur, nous allons faire de la France le pays de l'audace et pas celui de la bureaucratie.   Le pays qui investit dans l'avenir et pas celui qui est étranglé par ses dettes. Le pays du dialogue social et pas celui des cadenas syndicaux.   Le pays des initiatives et pas celui des circulaires.   On me décrit, mes amis, comme le représentant d'une France traditionnelle.   J'ai des racines et des valeurs. J'en suis fier.   Comme vous êtes fiers de l'histoire qui vous porte, qui fait partie de notre histoire, de l'histoire de la France.   Vous êtes ici européens, africains, indiens, chinois. Vous êtes tout cela à la fois, français de l'Océan indien et vous êtes une part infiniment précieuse de la France dans son étonnante diversité qui s'est construite au sein de notre République.   Oui, soyez fiers de vos ancêtres comme je le suis aussi des miens.   Perpétuez leur mémoire assumons ensemble le passé, avec ses heures les plus indignes, celle de l'esclavagisme, avec aussi ses heures les plus lumineuses qui nous fraternisent et nous poussent à aller de l'avant, ensemble, sous le drapeau tricolore. Une seule chose compte pour moi, sans considération d'origine, d'opinion, de religion : le désir de servir la France et de rendre à la France ce que nous avons reçu d'elle.   Alors je ne cesserai pas de vous parler de la France, de cette France que tous peuvent aimer.   Une France entière et fraternelle, universelle et particulière, mais toujours à reprendre, à inventer, à sauver.   Mes amis, depuis deux semaines, je suis attaqué, 24h/24.   Mais je reçois aussi d'innombrables soutiens. J'en reçois énormément.   Je me battrai jusqu'au bout car mon programme est le seul capable de relancer la France.   Ma bataille est celle des projets ! Ma bataille est celle des idées.   Elle l'a toujours été.   Je porte la responsabilité d'une ambition politique qui est celle des millions de Français qui refusent le déclin de leur pays.   Je me bats contre la gauche.   Celle de M. Hamon qui rêve l'impossible parce qu'elle est incapable de construire le possible, qui rêve de subventions universelles parce qu'elle est incapable de créer de la croissance.   Celle de M. Macron qui est le radeau de sauvetage de tous les naufragés du hollandisme.   Je me bats contre l'extrême-droite qui mettrait la France en faillite au bout de six mois.   Mes amis, maintenant je vais relever avec vous le défi du redressement national.   Ce défi, je le gagnerai si toutes celles et tous ceux qui en ont assez de la torpeur et des mensonges se lèvent à mes côtés.   Je donne rendez-vous aux réunionnais, à toutes celles et tous ceux qui ont la tête sur les épaules.   J'appelle à la Réunion tous ceux qui ont des valeurs solides.   J'appelle tous ceux qui ont dans le coeur la fierté d'être français et la fierté d'être réunionnais.   J'ai besoin de vous tous !   Ensemble nous allons libérer les forces de notre pays !   Ensemble nous allons le replacer au premier rang !   Ensemble nous allons traverser les tempêtes, franchir tous les ponts qui mènent à la victoire.   Vive la Réunion ! Vive les Outre-mer ! Vive la République et vive la France !   Source : Fillon2017.fr
</article>
<article titre="Des bioplastiques issus de résidus de bois" date="2017 02 10" journal="Emballages magazine.com">
AkzoNobel, Avantium, Chemport Europe, RWE et Staatsbosbeheer veulent créer une usine à Delfzijl au nord des Pays-Bas.-   Utiliser des résidus végétaux tels que des branches d'arbre, des copeaux de bois et des sous-produits de l'agriculture comme la bagasse pour produire des plastiques et des carburants : l'idée n'est pas nouvelle. Elle évite de recourir aux ressources alimentaires qui, comme le blé ou le maïs, sont actuellement utilisées pour produire ces commodités, ce qui pose un vrai problème éthique. AkzoNobel, Avantium, Chemport Europe, RWE et Staatsbosbeheer apportent leur pierre à l'édifice en annonçant leur intention de créer une usine qui permettrait de fabriquer, à partir de biomasse non-alimentaire, et plus spécifiquement de bois, des sucres et de la lignine. Ces deux produits intermédiaires permettraient ensuite d'obtenir du glucose pur afin de fabriquer, par fermentation ou catalyse, des bioplastiques à l'instar de l'acide polylactique (PLA), du polyéthylène furanoate (PEF), du polybutylène adipate-co-téréphthalate (PBAT) ou du polyhydroxyalkanoate (PHA). Toutes ces matières sont déjà ou pourraient être employées pour fabriquer des bouteilles, des barquettes ou des films d'emballage. L'usine, qui sera basée à Delfzijl au nord des Pays-Bas, utilisera le procédé de fabrication de glucose pur Zambezi, développé par Avantium. Les autres partenaires interviendront, chacun à un stade différent du process : Staatsbosbeheer coordonnera la fourniture de matière première à partir des résidus des forêts situées à proximité de l'usine; AkzoNobel gérera la construction du site et des équipements; RWE sera chargé de la transformation des résidus de bio-lignine en énergie; Chemport Europe jouera le rôle de facilitateur du projet auprès des autorités locales et des pouvoirs publics. Aucun calendrier n'a été communiqué.
</article>
<article titre="A Marie-Galante, la campagne sucrière suspendue au conflit autour d'une centrale thermique" date="2017 02 2" journal="AFP Infos Economiques">
Pointe-à-Pitre, 2 fév 2017 (AFP) - - Le doute plane sur la date d'ouverture de la campagne sucrière à Marie-Galante, prévue aux alentours de la mi-février dans le reste de Guadeloupe, en raison d'un conflit autour d'un projet de centrale thermique adossée à l'usine sucrière de l'île.   "Nous voyons la récolte qui prend du retard, alors il faut prendre une décision", s'impatientait mardi Ferdi Creantor, président de la coopérative agricole de Marie-Galante, interrogé par l'AFP. Les conventions qui conditionnent le versement d'aides publiques à la filière sucre mentionnent pour cette sucrerie la construction d'une centrale thermique permettant de moderniser son fonctionnement et d'externaliser la production d'électricité, mais cette construction n'est toujours pas lancée, car deux projets s'opposent.   Le premier, estimé à 80 millions d'euros, est porté depuis 2006 par Albioma, producteur d'électricité par valorisation énergétique de la bagasse (résidu de la canne à sucre) et la société Sucreries et Rhumeries de Marie-Galante (SRMG).    L'idée est de produire la vapeur dont a besoin pour fonctionner l'usine sucrière, mais aussi de rentabiliser cette centrale de 12 MW par la vente d'électricité à la "Guadeloupe continentale".   L'autre, d'un coût maximum de 30 millions d'euros, connu depuis le 6 décembre 2016, est à l'initiative de Maryse Etzol, la présidente PS de la communauté de commune de Marie-Galante (CCMG) et de la Compagnie Nationale du Rhône (CNR). Il s'agit pour eux de porter un projet de 4 MW "dimensionné à l'échelle de Marie-Galante" seulement, qui ne répondrait qu'aux besoins de l'usine sucrière.    Le groupe Albioma a obtenu le permis de construire et d'exploitation et n'attend plus que le feu vert pour démarrer la construction de la centrale et une mise en service en 2020.   - Les planteurs inquiets - Le projet de la communauté de communes s'inscrit quant à lui dans le cadre du programme "Marie-Galante, île du Tourisme durable", retenu en 2016 par les ministères de l'Environnement et du Logement et qui doit rendre l'île à 100% autonome en énergie d'ici à 2020, en développant l'éolien et les panneaux solaires.    "Trop coûteux, trop polluant", accuse Maryse Etzol en évoquant le projet Albioma qui doit notamment utiliser des granulés bois importés des forêts d'Amérique du Nord, dont le groupe assure de son côté qu'ils ont des "critères environnementaux très stricts".   Les "deux projets sont techniquement parfaitement compatibles", assure de son côté Louis Decrop, le directeur du développement d'Albioma, mais il met en avant son projet "abouti" contre un autre encore au stade du "concept", dit-il à l'AFP.   Mais chez les planteurs, on s'inquiète.   La filière, sous perfusion d'aides publiques (90 millions d'euros d'aides de l'Etat pour l'ensemble des DOM en 2016), voit le versement de l'argent conditionné à l'existence du projet porté par Albioma, selon une convention passée entre l'Etat et les acteurs de la filière.   Sans cet argent, le début de la campagne sucrière est compromis, craint Michel Claverie, directeur général de la Sucreries et Rhumeries de Marie-Galante, propriétaire de l'usine sucrière. Pour le responsable, conscient de l'obsolescence des équipements de son usine et très dubitatif sur le projet CCMG, "une campagne, ce sont des besoins de financement à hauteur de 1,5 million d'euros par mois pour l'usine".   Un argument que Maryse Etzol repousse : "Les aides sont votées, elles seront versées", assure-t-elle. Le préfet a en effet débloqué 600.000 euros de subventions aux planteurs, dit-elle.   Mais ces derniers voient la récolte prendre du retard, d'autant que les autorisations administratives sur le nouveau projet pourraient prendre du temps, ajoute Ferdi Creantor, président de la coopérative agricole de Marie-Galante.   Maryse Etzol multiplie les rendez-vous, pour faire approuver son projet et réécrire une convention. Et ce malgré une "tentative d'intimidation" de la préfecture, dit la présidente qui croit savoir que l'Etat a tranché en faveur du projet Albioma.   Contactée, la préfecture n'a pas souhaité répondre.   asa/ff/chr/ktr
</article>
<article titre="Cassonade, vergeoise, demerara, etc." date="2017 01 21" journal="Le Quotidien (tablette)">
Sucre de betterave ou sucre de canne ?    Qu'il soit extrait de la tige de la canne à sucre ou de la racine de la betterave, le sucre blanc ordinaire est constitué de saccharose (ou sucrose) pur. Les deux sucres sont équivalents en termes de pouvoir sucrant et de goût. Le sucre de canne   La récolte se fait de façon manuelle ou mécanique et c'est dans le bas des tiges que se concentre l'essentiel du sucre. Après la récolte, les tiges sont déchiquetées puis broyées afin d'obtenir un jus et un résidu fibreux nommé « bagasse ». Ce sera ensuite par divers traitements et étapes que le sucre final sera empaqueté pour ses diverses utilisations.   Le sucre de betterave   Il est extrait des racines de la betterave à sucre (de couleur blanche). Elles sont alors coupées en minces morceaux, de la forme d'une nouille, appelés « cossettes ». Bien que le processus de production soit différent, le résultat final est le même.   Sucre brun ou le sucre blanc ?    Très souvent, les sucres bruns proviennent du sucre granulé blanc...auxquels on ajoute tout simplement de la mélasse. Certains sucres bruns sont également produits en faisant bouillir un sirop spécial dans un appareil de cuisson sous vide.   Et la mélasse, elle provient d'où ?   La mélasse est un sous-produit du raffinage du sucre de canne ou de betterave. Seules les mélasses de canne à sucre sont destinées à la consommation humaine. La mélasse la plus répandue est celle « de fantaisie » dont la saveur est plus douce que celle de la mélasse verte.   Les mélasses de betterave sont employées pour la production de levures et la fabrication d'acide citrique. Elles sont parfois aussi ajoutées aux moulées des animaux d'élevage.   Et les autres sucres ?    Demerara, muscovado, barbados, turbinado...    Le demerara : le véritable sucre demerara est fait à partir de sirop de canne à sucre qui contient encore de la mélasse. La mélasse se trouve donc à l'intérieur des cristaux et pas uniquement en surface, comme c'est le cas des cassonades. Ses cristaux sont très humides, brun doré et légèrement collants. Les sucres « de type » demerara sont fabriqués par l'ajout de sirop de raffineur (sorte de mélasse) et de caramel à du sucre raffiné à gros cristaux. On l'utilise comme la cassonade foncée, mais ses cristaux étant plus gros que cette dernière, ils prennent plus de temps à se dissoudre.   La vergeoise est un sucre roux toujours issu de la betterave sucrière. Elle doit son nom aux anciennes formes à sucre « les vergeoises », de grands cerceaux de coudrier (noisetiers) qui étaient employés autrefois pour mouler les pains de sucre. Il existe cependant une confusion entre la vergeoise et la cassonade en Europe. En effet, ce sont tous les deux des sucres roux, mais pour eux, la cassonade doit absolument provenir de la canne à sucre et la vergeoise, de la betterave à sucre. Son parfum de caramel et sa couleur typique sont obtenus grâce à plusieurs cuissons successives du sirop de base. Recuit une fois, on obtient la vergeoise blonde et après une deuxième cuisson, on obtient la vergeoise brune, plus foncée et à l'arôme plus particulier.   Le muscovado aussi appelé sucre de la Barbade : il est semblable au demerara, par contre, il est beaucoup plus sec par rapport à son mode de fabrication qui consiste à faire cristalliser des sirops foncés. La production du véritable demerara se fait au début du processus de raffinage, lorsque les pigments et les saveurs des plantes ne sont pas complètement retirés. Par rapport à la cassonade normale, ses cristaux sont légèrement plus gros et collants. Sa couleur varie du brun pâle à brun foncé et le goût de mélasse est assez prononcé.   Le sucre barbados se distingue par sa texture plus fine et sa saveur prononcée de mélasse et de caramel.   Le sucre turbinado aussi appelé sucre brut de plantation est un sucre de canne semi-raffiné. Il est fabriqué à partir d'un sucre brut qui est lavé pour ne laisser qu'une fine couche de mélasse à sa surface. Ses cristaux sont plus ou moins colorés et il est souvent vendu en vrac ou en morceaux. On l'utilise davantage pour sucrer les boissons chaudes et pour saupoudrer sur les pâtisseries que pour cuisiner. Attention aux imposteurs ! Les sucres dits « de style » ou « de type » turbinés ou les « sucre au naturel » ou « sugar in the raw ») sont simplement de gros cristaux de sucre recouverts d'une fine couche de mélasse.   La cassonade comme vous la connaissez probablement tous est un sucre granulé enrobé de sirop de raffineur (sorte de mélasse). La quantité et le type de sirop déterminent la couleur (pâle à foncée) et l'intensité du goût. Le mot cassonade vient de « casson », ancien mot qui était utilisé pour désigner du sucre brut ou granuleux et cassant.   Le sucanat (pour SUgar CAne NATural) est obtenu par la déshydratation sous vide de jus de canne à sucre. Le produit fini contient environ 15 % de mélasse, ce qui lui confère une saveur assez prononcée. La texture du sucanat n'est pas cristalline, mais granuleuse, comme celle du café instantané. Le sucanat s'utilise comme la cassonade foncée.   Conservation du sucre   Normalement, s'il est bien entreposé, soit dans un environnement sec, le sucre blanc aura une durée de conservation illimitée. Il n'y a pas assez d'humidité pour soutenir une croissance microbienne à l'intérieur de celui-ci.
</article>
<article titre="GLOBAL BIOENERGIES La cleantech fait son entrée dans les bouteilles à gaz des Français" date="2017 01 17" journal="Investir (site web)">
C'est le cas de le dire, ça gaze pour Global Bionenergies (GBE). La cleantech, qui savoure déjà la remontée des cours du pétrole (ce qui rend sa technologie plus compétitive) et a déjà noué des partenariats avec de grands industriels dans différents secteurs (la coopérative agro-industrielle Cristal Union, propriétaire des marques Daddy Sucre, le chimiste Arkema, Audi ou encore L'Oréal) annonçait hier soir la signature d'un accord commercial avec Butagaz. L'un des leaders du marché du gaz en France va incorporer, dès cette année, du bio-isobutène produit par le chimiste vert à ses bouteilles de butane et de propane. Global Bioenergies a mis au point une technologie qui permet, à partir de bactéries modifiées et nourries au sucre, de créer par fermentation de l'isobutène, un hydrocarbure obtenu jusqu'alors par craquage du pétrole. La cleantech est donc capable de produire en mode « bio » des carburants, de la peinture ou des cosmétiques à partir de glucose végétal (sucre de canne, betterave...), d'amidon ou de déchets agricoles (paille de blé, tiges de maïs, bagasse de canne à sucre...).   Ces lots de bio-IBN fournis par Global Bioenergies seront issus du démonstrateur de Leuna en Allemagne, d'une capacité de 100 tonnes, et « seront livrés à Butagaz en contrepartie d'une contribution financière (montant non dévoilé) aux efforts de développement industriel de GBE. En parallèle, les deux sociétés mèneront une étude technico économique portant sur l'incorporation à grande échelle de bio-IBN dans les filières butane et propane », écrivent les analystes de Gilbert Dupont, qui réitèrent leur conseil d'achat en Bourse pour viser 46 euros.   En cas de satisfaction, Butagaz, qui a pour objectif de réduire son empreinte carbone, pourrait acheter des milliers de tonnes de bio-IBN lorsque l'usine IBN-One d'une capacité de 50.000 tonnes sera opérationnelle et pourrait également participer au financement de la construction de cette dernière.   Ce mardi, l'action Global Bioenergies a gagné jusqu'à plus de 5% dans un marché globalement baissier, avec une poussée à 28,10 euros.
</article>
<article titre="ENQUÊTE ANNUELLE 2016 SUR LES CONSOMMATIONS D’ÉNERGIE DANS L’INDUSTRIE SPECIMEN" date="2016 12 20" journal="Rapports - Marketing - INSEE">
8 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ...  INSEE Service de Statistiques Nationales d’Entreprises 5 rue Claude Bloch BP 95137 14024 CAEN CEDEX  Questionnaire à retourner avant le XXXXXXX 2017  ÉVÈNEMENTS MARQUANTS DE LA PÉRIODE (VOIR NOTICE)  Indiquer...    Pour voir le rapport :  http://www.insee.fr/fr/metadonnees/source/fichier/DOC_EACEI_quest_2016.pdf
</article>
<article titre="L'action bien orientée avec le feu vert pour le démonstrateur de Leuna" date="2016 12 6" journal="Trading Sat (site web)">
L'action Global Bioenergies grimpait de plus de 3% mercredi à mi-séance (+3,3% à 22,83 euros à 12h55) après que la société de chimie biologique a annoncé avoir reçu de TÜV, la principale agence nationale de certification en Allemagne, l'autorisation officielle de commencer les opérations sur le site du Démonstrateur Industriel de Leuna, situé en Allemagne.    Ce démonstrateur industriel, d'une capacité nominale de 100 tonnes/an, doit permettre de produire de l'isobutène de haute pureté à partir de diverses ressources (sucre industriel dérivé de la betterave et de la canne à sucre, sirop de glucose dérivé des céréales, sucres de deuxième génération extraits de paille de blé, de bagasse, de copeaux de bois...) Cette mise en service "devrait ainsi générer la signature de contrats et/ou de licences dans les 18 prochains mois", commentent les analystes de Gilbert Dupont.    Le broker, qui maintient son opinion "Acheter" et son objectif de cours de 46 euros sur Global Bioenergies, note que "le pilote de Pomacle (situé en France, d'une capacité de 10 tonnes/an) est victime de son succès", les industriels intéressés pour tester le bio-isobutène étant mis sur file d'attente. Ainsi, "le démonstrateur de Leuna devrait permettre de pallier cette insuffisance de production", fait remarquer le courtier.
</article>
<article titre="Magie du papier" date="2016 12 1" journal="Challenges">
C'est comme un printemps en hiver, des feuilles parées d'or et de brillants poussent partout dans les vitrines et invitent aux cadeaux d'écriture. La cellulose prend des couleurs aussi bien chez les vendeurs de bougies qu'au rayon habillement. Le papier est de retour, et pas seulement pour emballer les présents. Sur les traces de la renaissance à succès de Moleskine, les marques de papeterie se multiplient et proposent carnets, boîtes à crayons ou blocs de listes à prix modérés. La France est particulièrement en pointe avec de jeunes entrepreneurs privilégiant le local et le développement durable. Il s'agit d'écrire vert et beau. La Petite Papeterie française édite ainsi de belles collections de cahiers graphiques, cartes postales rêveuses et objets au cordeau pour le bureau ; Monsieur Papier décline son univers doux et joyeux en mettant en avant les nouvelles matières comme les fibres de bambou, de coton et de bagasse, ou à base d'amidon cru de pomme de terre ; de son côté, Papier Tigre diffuse son graphisme débridé, ses rangements simples et ses calendriers intelligents - comme celui des primeurs pour manger en fonction des saisons.   Par Soizic Briand En France, le rayon papeterie des boutiques Muji cartonne et représente près du quart de leur chiffre d'affaires. En hommage au papier, le distributeur design vient même d'ouvrir une petite librairie dans son magasin des Halles. Dans les étagères, une sélection internationale de livres animés ou à système, popup en anglais, pour les petits et les grands. Parce que le papier est magique, surtout quand il jaillit en 3D des pages d'un livre.   «Avec de quoi écrire, il est possible de tout faire» CYNTHIA FLEURY, PHILOSOPHE, AUTEUR DES IRREMPLAÇABLES, GALLIMARD. « J'ai mon carnet Clairefontaine au format livre de poche à petits carreaux pratiquement 24 heures sur 24. Je l'ai adopté à l'âge de 15 ans, il est à la fois mastoc - du solide que je transporte partout et qui me permet de voyager léger - et suffisamment épais avec ses 192 pages pour tenir un mois de notes, de schémas ou tableaux. J'y dépose toutes mes idées, sans les classer et je les relis régulièrement à chaque travail d'écriture. Je suis nulle en cadeaux, mais je pense que tout le monde a besoin de carnets, de papeterie. C'est un présent matriciel : avec de quoi écrire, il est possible de tout faire! » Ingénieux Agenda Merci (Mark's) Petit, mince, grand (18,5 × 14 cm) pour ne pas obliger aux pattes de mouches, l'agenda de Merci par Mark's, première marque japonaise de papeterie fantaisie, associe rigueur et possibilités multiples. Sa couverture en PVC cache une POCHETTEzippée pour y glisser tous papiers, cartes ou photos. Ses quinze mois, d'octobre 2016 à décembre 2017, permettent de négocier le tournant de l'année. Deux marque-pages facilitent l'accès aux calendriers annuels et mensuels, et à l'agenda et ses 16 heures par jour.   Prix : 19,90 euros En vente à Merci, 111, boulevard Beaumarchais, 75003 Paris.   Matussiere Victor Gracieux Pochettes Sézane (Delfonics) De la rencontre d'un Français et d'un Japonais naît des pochettes avec un air anglais Leader de la papeterie japonaise arrivé en France en 2012, Delfonics a fait appel à Sézane, la griffe Web des mordus de mode. Résultats : des carnets, des trousses, des portecartes à petits carreaux, grands carreaux ou à motif « parapluies ». Du Swinging London pour conquérir l'étranger? Des projets de Morgane Sézalory, qui a fondé la marque en 2013 Prix : à partir de 11 euros.   En vente à Delfonics, 99, rue de Rivoli, 75001 Paris. www.sezane.com Soyeux Carnet de soie (Hermès) Un « carnet de soie » aux 144 pages de papier blanc vélin aussi doux et soyeux que la couverture taillée dans un fameux carré Hermès. Il se décline en quatre couleurs, et les motifs varient suivant l'emplacement de la coupe réalisée dans le foulard et permettant de couvrir un format de 15 sur 10,5 centimètres. Avec le marque-pages en spaghetti de soie, ce carnet s'affiche comme le plus bariolé d'une collection d'instruments d'écriture comprenant stylo plume et stylo bille, carnet de notes de différentes tailles, et enveloppes frappées du sceau d'Hermès et doublées de rouge.   Prix : 80 euros.   En vente chez Hermès, 17, Rue de Sèvres, 75006 Paris. France.hermes.com Curieux Maquette Archipaper (Cinqpoints) Une maquette d'architecture à monter, plier, coller côté noir ou côté blanc, au choix, pour s'initier aux subtilités des proportions ou simplement profiter d'une sculpture faite maison. Archipaper trouve sa place sur un bureau, une étagère ou dans une chambre d'enfant. Un modèle imaginé par Cinqpoints, la marque créée en 2013 par Laurence Calafat, architecte qui édite aussi bien des jeux en papier ou en bois que des objets pour le bureau et de la papeterie célébrant l'univers de l'architecture contemporaine. Une signature graphique tout ce qu'il y a de plus chic.   Prix : 15 euros.   En vente à Enograph, 89, rue de Turenne, 75003 Paris. www.cinqpoints.com Joyeux Carnets Diptyque (Papier Tigre) Ecailles, losanges, oiseaux, vagues Pour illustrer les couvertures d'une demi-douzaine de carnets, Papier Tigre, né en 2012, joue avec l'ovale si reconnaissable qui sert de logo à son aîné, le parfumeur Diptyque, fondé en 1961. Les deux marques se ressemblent, créées chacune par trois fondateurs, passionnés par la création, le papier et le tissu. Une façon de mettre Papier Tigre au parfum? L'éditeur de papeterie propose déjà du papier parfumé. Mais avec Kerzon.   Prix : 15 euros.   En vente dans les boutiques Diptyque et sur le site de Papier Tigre. www.papiertigre.fr/www.diptyqueparis.fr
</article>
<article titre=""Etape clé" franchie avec la construction du démonstrateur de Leuna" date="2016 11 15" journal="Trading Sat (site web)">
Gilbert Dupont a confirmé mardi sa recommandation "Acheter" sur Global Bioenergies (avec un objectif de cours ajusté à 46 euros, contre 50,7 euros auparavant) après que la société de génie chimique a annoncé que la construction de son démonstrateur industriel, commencée mi-2015, était maintenant terminée.   Le démonstrateur industriel de Global Bioenergies est une unité unique au monde, conçue spécifiquement pour la fermentation directe des hydrocarbures gazeux. Il est installé en Allemagne sur le site du Fraunhofer CBP, au sein de la raffinerie de Leuna. D'une capacité nominale de 100 tonnes/an, il permettra de produire de l'isobutène de haute pureté à partir de diverses ressources (sucre industriel dérivé de la betterave et de la canne à sucre, sirop de glucose dérivé des céréales, sucres de deuxième génération extraits de paille de blé, de bagasse, de copeaux de bois.).   L'isobutène est une molécule plateforme. Dans le secteur des carburants, il est actuellement utilisé pour produire de l'isooctane, de l'ETBE et du MTBE, des composants importants dans la formulation des essences. Il peut également être converti en isododécane, entrant dans la composition du kérosène.    L'isobutène est aussi utilisé pour la fabrication d'une vaste gamme de matériaux et produits chimiques tels que caoutchoucs, plastiques, verre organique, peintures et cosmétiques. La mise à l'échelle du procédé de Global Bioenergies jusqu'à son exploitation commerciale permettra de réduire considérablement l'empreinte carbone dans ces domaines industriels clé.   La construction du démonstrateur "constitue une étape clé dans le développement de Global Bioenergies", souligne Gilbert Dupont. "Elle permettra de valider la faisabilité de l'industrialisation du procédé et de délivrer des lots pour tests à grande échelle qui pourrait aboutir sur des contrats d'écoulement de la production ou d'octroi de licences", ajoute-t-il.    brightcove.createExperiences();
</article>
<article titre="Global Bioenergies:fin de la construction d'un démonstrateur" date="2016 11 14" journal="Votre argent (site web)">
Global Bioenergies a fait savoir ce lundi soir via un communiqué que la construction de son démonstrateur industriel, commencée mi-2015, est à présent terminée.  Le démonstrateur industriel de Global Bioenergies est une unité unique au monde, conçue spécifiquement pour la fermentation directe des hydrocarbures gazeux. Il est installé sur le site du Fraunhofer CBP, au sein de la raffinerie de Leuna (Allemagne), et a été financé par une subvention de 5,7 millions d'euros octroyée par le ministère fédéral allemand de la Recherche (BMBF), ainsi que par un prêt de 4,4 millions obtenu auprès d'un consortium de banques françaises (Société Générale, BNP-Paribas, CM-CIC et BPI).   Cette structure d'une capacité nominale de 100 tonnes par an permettra de produire de l'isobutène de haute pureté à partir de diverses ressources (sucre industriel dérivé de la betterave et de la canne à sucre, sirop de glucose dérivé des céréales, sucres de deuxième génération extraits de paille de blé, de bagasse, de copeaux de bois etc.).Copyright (c) 2016 CercleFinance.com. Tous droits réservés.
</article>
<article titre="Global Bioenergies : construction terminée pour le démonstrateur industriel de Leuna" date="2016 11 14" journal="Boursier (site web)">
Global Bioenergies  annonce que la construction de son démonstrateur industriel, commencée mi-2015, est maintenant terminée. Le démonstrateur industriel de Global Bioenergies est "une unité unique au monde, conçue spécifiquement pour la fermentation directe des hydrocarbures gazeux". Il est installé sur le site du Fraunhofer CBP, au sein de la raffinerie de Leuna, et a été financé par une subvention de 5,7 ME octroyée par le ministère fédéral allemand de la recherche (BMBF), et par un prêt de 4,4 ME obtenu auprès d'un consortium de banques françaises (Société Générale, BNP-Paribas, CM-CIC et BPI). Ce démonstrateur industriel, d'une capacité nominale de 100 tonnes/an, permettra de produire de l'isobutène de haute pureté à partir de diverses ressources (sucre industriel dérivé de la betterave et de la canne à sucre, sirop de glucose dérivé des céréales, sucres de deuxième génération extraits de paille de blé, de bagasse, de copeaux de bois).   L'isobutène est une molécule plateforme. Dans le secteur des carburants, il est actuellement utilisé pour produire de l'isooctane, de l'ETBE et du MTBE, des composants importants dans la formulation des essences. Il peut également être converti en isododécane, entrant dans la composition du kérosène. L'isobutène est aussi utilisé pour la fabrication d'une vaste gamme de matériaux et produits chimiques tels que caoutchoucs, plastiques, verre organique, peintures et cosmétiques. La mise à l'échelle du procédé de Global Bioenergies jusqu'à son exploitation commerciale permettra de réduire fortement l'empreinte carbone dans ces domaines industriels clé, selon Global Bioenergies.
</article>
<article titre="Des coques de grains de café pour faire tourner des usines kényanes" date="2016 11 12" journal="Le Monde.fr">
Imaginons une scène du quotidien kényan. Un homme est attablé dans l'un des nombreux cafés bondés et bruyants du centre de Nairobi. Il porte un pull rouge seyant tout en sirotant son soda glacé. Il ne se doute pas un instant qu'en mettant son pull ce matin et en buvant son soda, il participe à une petite révolution verte pour les industries: le rondin écologique. Pour comprendre son origine, reculons de quelques mètres, derrière le comptoir où un barista s'agite devant sa machine à expresso. Les grains de café qu'il moud proviennent des hauts plateaux du centre du pays, dans la région du mont Kenya. Là, dans les cultures de caféiers, des femmes font la récolte. Puis un procédé mécanique débarrasse les grains de café de leur enveloppe. Ces coques, ou exocarpes, impropres à la consommation, sont habituellement écartées et jetées. Mais, depuis 2006, Lean Energy Solutions les récupère pour les valoriser.   Mélange de déchets végétaux   L'entreprise kényane a fourni aux ouvrières des plantations de grandes bâches en plastique noir et, contre rémunération, leur demande d'y verser les coques afin qu'elles sèchent au soleil. Elles seront ensuite transportées par camion dans l'une des usines de rondins de la société. Dans celle de Kiambu, située près de la capitale kényane, il n'y a même pas besoin de transport. L'usine se trouve en face d'un producteur de café. Reliant en hauteur les deux bâtiments, un convoyeur d'une cinquantaine de mètres achemine les coques, rachetées à moindre coût, jusque dans une grande salle où elles forment des monticules de plusieurs mètres.   Puis les coques sont mélangées à d'autres déchets végétaux récupérés par l'entreprise: sciure de bois, bagasse de canne à sucre et résidus de charbon. Encore humide, le mélange traverse un immense séchoir au bout duquel un compresseur l'agglomère et le conditionne en rondins au fort pouvoir calorifique. Chaque jour, cette usine transforme 80 tonnes de débris en 40 tonnes de rondins que quatre camions porteront à ses clients.   Lean Energy a passé des contrats avec douze sociétés basées au Kenya et deux en Tanzanie qui ont toutes installé une chaudière industrielle dédiée. Parmi les clients les plus importants, on compte les géants de l'agroalimentaire Unilever, Coca-Cola, Pepsi ainsi que des sociétés de textile comme Spinners &amp;amp;Spinners. C'est dans les bâtiments de cette dernière que nous nous rendons pour observer comment de coques de grains de café font tourner l'usine.   Deux opérateurs chargent une immense chaudière en fonte de pelletées de rondins afin de maintenir la pression entre 7,5 et 8,5bars jour et nuit. L'eau froide bout rapidement et produit une vapeur à 250°C qui se faufile à travers les conduits de l'entreprise de textile jusqu'au département des teintures où, par imprégnation et vaporisage, les fibres textiles qui serviront à tricoter le pull de notre buveur de soda prennent leur jolie teinture rouge carmin.   D'ailleurs, qu'en est-il de son soda? Il faut, pour le savoir, faire 350km en direction de l'est du Kenya, dans l'usine Coca-Cola de Kisumu. Là, une chaudière semblable a été installée à l'extérieur du bâtiment mais la vapeur produite y est utilisée pour nettoyer les bouteilles de verre vides et transformer le sirop concentré en boisson gazeuse.   Un quart d'économies par rapport au mazout   «Auparavant, toutes ces usines utilisaient des chaudières au mazout ou au diesel,  explique Dinesh Tembhekar, directeur de Lean Energy.  Pour les convaincre, nous leur avons proposé d'installer à nos frais notre chaudière à rondins écologiques à côté de leur usine afin de produire une vapeur qu'on leur vendait 25% moins cher que leurs dépenses en mazout. Ainsi, ils étaient doublement gagnants.» Cette stratégie verte a permis à Lean Energy de signer des contrats de huit ans avec ces compagnies et de bénéficier d'une ligne de crédit de l'Agence française de développement (AFD, partenaire du  Monde Afrique ).  «Produire une tonne de ces rondins revient à créer douze jours de main-d'oeuvre pour un Kényan et à réduire d'une tonne les émissions de C02,  poursuit-il.  Les industries locales n'ont plus besoin d'importer du diesel ou du mazout des pays du Golfe. Tout est fait sur place et reste sur place.» Modèle de succès kényan dans les énergies renouvelables, Lean Energy emploie 400 personnes à travers le pays, dont une majorité de femmes en travail temporaire.  «Chaque année, nous grandissons de 40% et doublons notre personnel» , lance M. Tembhekar. Une croissance insolente qui lui a permis d'installer son entreprise dans de nouveaux locaux et d'établir des partenariats avec le groupe français Schneider Electric. Désormais, elle s'est lancée aussi sur le marché de l'énergie solaire en proposant des éclairages publics photovoltaïques et, à l'instar de D. Light ou de M-Kopa, des kits solaires domestiques pour les foyers kényans sans électricité.   Prochaine étape?  «Le marché rwandais puis, peut-être, ougandais» , confie-t-il à discrétion. Musique d'avenir ambitieuse.  «Le continent est vaste, regorge de solutions énergétiques vertes et de ressources inépuisables.»  Il tapote sur le bureau puis se redresse sur sa chaise.  «Vous voulez un café?» A l'occasion de la COP22 qui se déroule à Marrakech du 7 au 18novembre,  Le Monde Afrique  a réalisé la  série Traversée d'une Afrique bientôt électrique  en allant voir, du Kenya au Maroc, en passant par le Burkina, la Côte d'Ivoire, le Cameroun ou le Sénégal, l'effort d'électrification du continent.  Cet article est paru dans Le Monde.fr
</article>
<article titre="Albioma: l'activité progresse de 1% au 3T, tirée par le Brésil" date="2016 10 26" journal="AFP Infos Economiques">
Paris, 26 oct 2016 (AFP) - - Le producteur français d'électricité Albioma, surtout présent outre-mer, a publié mercredi un chiffre d'affaires en légère progression au troisième trimestre, tiré par son activité au Brésil.   Sur la période, le chiffre d'affaires atteint 90,5 millions d'euros, en hausse de 1%, et sur les neuf premiers mois de l'année l'activité a progressé de 5% à 267,6 millions d'euros. Globalement Albioma a produit 2.689 gigawattheures d'électricité entre janvier et septembre, contre 2.478 GWh sur la même période de l'an dernier.   Cette croissance s'explique essentiellement par le développement d'Albioma au Brésil depuis un an, avec notamment l'acquisition le 4 août 2015 de la centrale biomasse de Codora Energia, alimentée par des résidus de canne à sucre (bagasse).   Le chiffre d'affaires réalisé dans ce pays a progressé de 51% sur les neuf premiers mois de 2016.   Mais Albioma s'attend en revanche a une activité "en forte baisse" a Codora Energia au quatrième trimestre, du fait d'une campagne sucrière plus courte que prévu qui va réduire les volumes de matière première disponible.   En France, le chiffre d'affaires de l'activité des centrales biomasse a progressé de 4% sur les neuf premiers mois de l'année à 218,2 millions d'euros, tandis que l'activité solaire est en repli de 1% à 31,7 millions d'euros, du fait d'un ensoleillement moindre.   La disponibilité des centrales biomasses thermiques a progressé, du fait de la grève et un incident technique sur le site du Moule en Guadeloupe qui avait pénalisé leur activité en 2015.   Albioma a confirmé ses objectifs pour 2016 à savoir un excédent brut d'exploitation (Ebitda) compris entre 122 et 130 millions d'euros et un bénéfice net entre 25 et 30 millions d'euros.   Le groupe poursuit actuellement plusieurs développements de projets, comme la construction de la centrale thermique Galion 2 en Martinique, et trois centrales photovoltaïques avec stockage à La Réunion et en Guadeloupe.   mhc/smr/sg
</article>
<article titre="Incertitude pour l'exploitation de Galion 2, grand projet biomasse d'Albioma" date="2016 10 7" journal="GreenUnivers (site web)">
2016-10-07T17:48:52+00:00    Mauvaise nouvelle pour Albioma : le tribunal administratif de Fort-de-France (Martinique) a annulé l'arrêté d'exploitation de sa future centrale de cogénération 100% biomasse Galion 2 dans un jugement rendu le 4 octobre. Le groupe a annoncé ce matin son intention de faire appel devant la cour administrative d'appel de Bordeaux.   Mise en service prévue au 2e trimestre 2017  Galion 2 est l'un des grands projets du producteur d'énergie. Cette centrale (40 MW) doit être la première 100% biomasse d'Outre-Mer. En gestation depuis dix ans et en cours de construction avec 180 personnes mobilisées sur le chantier, elle repose sur un échange avec la sucrerie voisine du Galion (elle utilisera la bagasse produite par cette dernière et lui fournira de la vapeur) et doit couvrir à terme 15% de la consommation électrique totale de La Martinique et contribuer à verdir le mix énergétique de l'île. Elle représente un investissement de 185 M€ et sa mise en service est prévue pour le 2e trimestre 2017. Mais ce projet est farouchement combattu depuis plusieurs années par l'association écologiste Assaupamar (   Association pour la sauvegarde du patrimoine martiniquais   ) qui estime que la centrale représente un danger pour l'environnement et la santé. Outre des actions sur place, elle a demandé en justice l'annulation de l'arrêté préfectoral d'exploitation obtenu en 2014 en mettant en avant de nombreux griefs (problème de particules fines, impacts sur la faune et la flore...). Et elle vient d'obtenir gain de cause. "Mais le tribunal n'a retenu qu'un point technique : une insuffisante démonstration de la compatibilité du projet avec le schéma directeur d'aménagement et de gestion des eaux (SDAGE) de La Martinique, aucun des autres griefs n'a été retenu et le permis de construire n'est pas concerné", précise un porte-parole d'Albioma à GreenUnivers.   Un sursis dans l'exécution ?  Alors que la construction peut donc se poursuivre, l'enjeu pour l'entreprise est de pouvoir mettre la centrale en service au 2e trimestre 2017, comme prévu. Pour cela, elle espère bénéficier d'un sursis dans l'exécution du jugement en attendant la décision de la cour d'appel de Bordeaux.   L'annonce de cette procédure a rejailli sur le cours d'Albioma sur Euronext Paris, qui perdait 3,4% en milieu d'après-midi. Le jugement sur Galion 2 intervient alors que l'entreprise a annoncé en juillet de bons résultats pour le premier semestre 2016 avec un chiffre d'affaires de 177,1 M€, en hausse de 8% par rapport à la même période de 2015, un Ebitda de 65,4 M€ (+ 30%) et un résultat net de 14,5 M€ (+ 53%).   Des projets remportés dans le solaire + stockage  Albioma est spécialisée dans la production d'énergie à partir de biomasse, et notamment de bagasse, et se développe également dans le photovoltaïque. La société a ainsi remporté trois projets PV + stockage pour un total de 5,9 MW lors du dernier appel d'offres CRE-ZNI (deux à La Réunion et un en Guadeloupe). Elle exploite aussi trois centrales biogaz mais ne mène plus de nouveaux projets dans ce secteur actuellement. L'entreprise, présente en France métropolitaine et Outre-Mer et au Brésil, vise un objectif de 80% d'énergie d'origine renouvelable en 2023.  Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="Sucre: naissance d'un des plus gros exportateurs du monde" date="2016 10 4" journal="AFP - Journal Internet AFP (français)">
Paris (AFP) - Le groupe de négoce de matières premières agricoles de Singapour, Wilmar, a annoncé mardi la formation d'une société commune à parité avec le premier groupe sucrier mondial, le brésilien Raizen.   La nouvelle entité, baptisée Raw (Raizen And Wilmar Sugar), sera un des plus gros exportateurs de sucre du monde, avec 4,5 millions de tonnes traitées au total par an, dont 3 millions de tonnes de sucre de qualité (VHP pour very high polarization) produits par Raizen au Brésil, a indiqué Wilmar dans un communiqué. Elle sera dirigée par Jean-Luc Bohbot, qui dirige les activités sucre de Wilmar. Raw sera basée à Singapour et traitera essentiellement du sucre du Brésil, avec une filiale basée à Sao Paulo.   Raizen produit environ 2 milliards de litres d'éthanol par an, 4,5 millions de tonnes de sucre et a la capacité de produire 940 MW d'énergie électrique à partir de la bagasse de canne à sucre (résidu).   Le groupe, qui emploie plus de 30.000 salariés dans le monde, dispose aussi d'un réseau de 5.800 stations service sous la marque Shell.   Wilmar est un géant mondial de l'agro-business, basé à Singapour, qui emploie 92.000 salariés dans le monde, surtout en Asie. Ses spécialités vont des plantations de palmiers à huile au raffinage de sucre et d'huile en passant par les biocarburants, les engrais, la farine et le riz.
</article>
<article titre="Les assises de l'Ain l'avaient condamné à 15 ans" date="2016 10 4" journal="Le Progrès - Lyon  Oyonnax - Léman - Bugey - Bas-Bugey, Bourg - Bresse - Val-de-Saône-Nord, Dombes - Côtière de L'Ain">
« Un Janus à deux visages. Le premier vous présente comme un homme lisse, le second comme un animal. » Dès 2003, la dangerosité de la personnalité de Fabrice A. avait été soulignée par le procureur de la cour d'assises de l'Ain qui avait également parlé de « prédateur ». Le tribunal l'avait d'ailleurs condamné à 15 ans de prison pour viol. Il avait tendu un véritable piège qui dit bien le modus operandi particulièrement manipulateur de celui qui est une nouvelle fois à la barre aujourd'hui. Alors qu'une de ses collègues à la pizzeria où il travaillait cherchait un appartement, Fabrice A. lui avait fait croire qu'il avait une amie dans une agence immobilière. Il lui avait donné rendez-vous en lui demandant de venir avec deux mois de loyer. Sauf qu'en lieu et place d'appartement, il l'avait conduite au bois de la Bagasse, où il l'avait attachée et violée sous la menace d'un couteau. Situé à Ferney-Voltaire, ce bois avait été exploré par les gendarmes de la compagnie de Gex le jeudi 12 septembre 2013 après la disparition d'Adeline. L'individu avait déjà commis au même endroit un premier viol qui lui avait valu 5 ans de prison en 2001.  Fabrice A. avait donc déjà purgé 12 de ses 20 ans de prison (les deux peines avaient été cumulées) quand il a assassiné Adeline. S'il était en détention en Suisse, c'est qu'il avait demandé son transfert. Une fois son crime commis à Genève, Fabrice A. avait été arrêté en Pologne trois jours plus tard. Il était parti retrouver une ancienne petite amie qui l'avait quitté quelques années plus tôt et dont il voulait se venger...
</article>
<article titre="Packaging, la recherche du compromis" date="2016 10 1" journal="Néorestauration, no. 0545">
« En restauration, tout ne passe pas que par le produit, sa mise en scène compte aussi : on n'a jamais acheté le caviar dans des sacs en plastique ! »,  sourit Christian Duret, P-DG de Solia, spécialiste du packaging. La société qu'il dirige -26 M E de chiffre d'affaires en 2015, dont 80 % sur les circuits de la restauration -, est aux premières loges pour le constater. Du snacking aux traiteurs, en passant par les plateaux-repas ou la restauration collective..., les marchés qu'elle sert sont demandeurs de nouvelles solutions d'emballage. Une tendance de fond : en France, la restauration s'emporte de plus en plus, supplantant même, selon Gira Conseil, l'activité générée par le service à table depuis 2012 ; elle se livre aussi, en plateaux-repas dédiés aux déjeuners de travail, le soir à domicile...  Beaux, ludiques et gourmands Un casse-tête pour les opérateurs et leurs fournisseurs. Répondre aux attentes du mangeur nomade n'est pas sans poser des questions techniques. On emballe du froid, mais aussi du chaud, des jambon-beurre comme des plats en sauce ou des soupes, et sur tous les segments, du sandwich du coin au menu élaboré par un chef étoilé. L'étanchéité, la facilité de transport, la stabilité, la gestion des phénomènes de condensation sont autant de réponses à apporter.   « Notre offre s'articule autour du déjeuner de travail,  explique Marie Félix, directrice marketing de Room Saveurs.  Travailler en mangeant n'est pas réjouissant. Nos plateaux doivent être beaux, ludiques et gourmands. Le soin apporté aux emballages est essentiel. »  Un élément à part entière de l'expérience culinaire, où la propreté d'une découpe sur un contenant en fibres de sucre de canne, la transparence d'un plastique, le dessin d'un fond de barquette, peuvent mettre en appétit et assurer une praticité de dégustation. Ou pas.   Car ce sont aussi l'enseigne et son image qui sont en jeu. Avec, pour définir ces relais de marque que sont les emballages, deux critères de taille : leur prix, qui peut, sur les plateaux-repas, représenter jusqu'à 30 % des coûts de production ; leur impact environnemental, les évolutions réglementaires imposant aux opérateurs de se mettre en recherche d'alternatives au tout-plastique...   En matière de packaging, quelles sont les réponses apportées par la restauration ?
</article>
<article titre="Total s'associe à Bill Gates pour investir dans Renmatix" date="2016 09 19" journal="GreenUnivers (site web)">
2016-09-19T18:58:32+00:00    Déjà soutenue par plusieurs investisseurs de renom, la société américaine spécialisée dans les biocarburants et la chimie verte vient de boucler un nouveau tour de table de 14 M$ (12,5 M€) mené par Bill Gates, aux côtés du français Total, déjà présent au capital de l'entreprise depuis 2015. La répartition entre les deux bailleurs de fonds n'a pas été dévoilée. C'est la branche raffinage/chimie du groupe pétrolier qui a piloté cette opération.   Fondée en 2003, Renmatix est spécialisée dans la production de sucres cellulosiques à bas coût. Elle a développé un process breveté, baptisé Plantrose, qui utilise de l'eau supercritique pour réduire les coûts de conversion de la biomasse (bois, paille, bagasse...) en sucres cellulosiques, intermédiaires clés pour la production de biocarburants ou de produits chimiques biosourcés de deuxième génération. La société est implantée en Pennsylvanie, en Géorgie et à New York. Accord de licence  Total complète son investissement en signant avec Renmatix un accord de licence pour une capacité de production de sucres cellulosiques d'un million de tonnes par an, les installations correspondantes devant être construites par le français. Ces sucres cellulosiques seront utilisés pour la production de molécules d'intérêt par Total.« L'ambition de Total est de devenir la major de l'énergie responsable. Nous voulons que nos activités à faible empreinte carbone soient un catalyseur de développement et représentent 20% de notre portefeuille dans les 20 années à venir. C'est pour répondre à ces objectifs que nous avons établi et étendu notre collaboration avec Renmatix, » a expliqué Patrick Pouyanné, PDG de Total, dans un communiqué.   Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="énergie Des centrales à  bagasse" date="2016 08 20" journal="Investir-Le Journal des finances, no. 2224">
Biomasse : Albioma se dit au coeur de la transition énergétique. Sa production électrique est à 50 % d'origine renouvelable aujourd'hui, contre 36 % en 2013 et 80 % visés en 2023. Le solaire représente seulement 10 % de ses capacités totales (753 MW), la spécificité du groupe tenant à ses centrales thermiques. Installé dans des territoires où la canne à sucre est abondante, il utilise la bagasse, les résidus fibreux de la plante, qu'il brûle dans ses chaudières. Comme le montre l'illustration ci-contre, les sites d'Albioma sont adossés à un sucrier, auquel elles fournissent vapeur et électricité pour faire fonctionner son usine. Le surplus d'énergie est transféré sur le réseau électrique et vendu à EDF pour ce qui est des régions d'outre-mer, à des prix garantis (et plutôt élevés) par des contrats de longue durée. Ils peuvent faire l'objet d'avenants, de compensations au profit d'Albioma à l'occasion d'investissements importants, par exemple pour réduire les sources de pollution d'une centrale comme récemment à Bois-Rouge à La Réunion. En dehors des périodes de récolte de la canne, ces centrales de cogénération recourent au charbon, nettement moins écologique et accepté des populations ! Aussi, la centrale de Galion 2 en chantier à La Martinique utilisera-t-elle de la biomasse toute l'année, en important des déchets de bois des Etats-Unis. Progressivement, le parc thermique d'Albioma va abandonner le charbon, « ce qui va nécessiter des investissements liés notamment aux capacités de stockage et à la manipulation de la biomasse, mais sans impact sur la productivité des usines », promet Frédéric Moyne.
</article>
<article titre="Albioma (ex-Sechilienne-Sidec) : des résultats qui turbinent" date="2016 08 11" journal="Boursier (site web)">
Inflexion.  Les mauvaises nouvelles se tarissent sur Albioma. Après avoir dû composer ces deux dernières années avec des incidents techniques, des travaux de rénovation, des grèves et des turbulences monétaires (dépréciation du Real brésilien), le producteur d'électricité à base de biomasse voit tous ses clignotants passer progressivement au vert. Au premier semestre 2016, le chiffre d'affaires du groupe s'est inscrit en progression de 8% à 177,1 ME, tandis que l'EBITDA se hissait à 65,4 ME, en hausse de 30%. Ces performances, supérieures aux attentes, traduisent le retour à la normale pour les centrales thermiques en France après les déboires de l'an dernier. L'amélioration des indicateurs économiques et financiers reflète également une bonne résilience de l'activité solaire et un bon niveau de production au Brésil, principal relais de croissance du groupe. Conforté par cette bonne entame d'année, Albioma est à l'aise avec ses prévisions pour 2016. L'exercice devrait se solder par un Ebitda compris entre 122 et 130 ME, pour un résultat net dans une fourchette 25-30 ME. Pipeline.  Comme nous l'avons maintes fois souligné lors de nos derniers conseils sur la valeur, Albioma constitue un dossier à part dans l'univers de l'énergie. L'entité bénéficie d'une excellente visibilité sur ses revenus et résultats assurée par des contrats de long terme la liant à ses clients. La période 2016-2018 devrait d'ailleurs se caractériser par une accélération de l'expansion grâce à la mise en service de nouvelles unités. On pense notamment à Galion 2, une centrale à bagasse de 40 MW qui débutera sa production dans un an. On pense également à la turbine à combustion de Saint-Pierre à La Réunion (41 MW) qui entrera en activité fin 2017. Ce pipeline, riche de nombreux projets, permet de garder un biais optimiste sur Albioma. Le titre apparaît en effet modérément valorisé eu égard à ses qualités fondamentales. Le PER 2016 du groupe énergétique est inférieur à 16, tandis que le rendement de l'action ressort à 3,7%. De quoi offrir un potentiel de revalorisation non négligeable malgré le récent rebond. On visera plus haut sur ce dossier de qualité.Aymeric Val
</article>
<article titre="Albioma: le bénéfice net progresse de 53% à 14,5 millions d'euros au 1S" date="2016 07 26" journal="AFP Infos Economiques">
Paris, 26 juil 2016 (AFP) - - Le producteur français d'électricité Albioma, surtout présent outre-mer, a vu son bénéfice net fortement progresser au premier semestre, grâce à la bonne performance de ses centrales thermiques qui avaient subi des incidents l'an dernier.   Sur les six premiers mois de l'année, le bénéfice net s'élève à 14,5 millions d'euros, en hausse de 53% par rapport au premier semestre de 2015, a détaillé le groupe dans un communiqué. Le chiffre d'affaires atteint lui 177,1 millions d'euros, en progression de 8%, tandis que l'excédent brut d'exploitation (Ebitda) s'affiche à 65,4 millions d'euros, en hausse de 30%.   Le semestre est marqué par "les bonnes performances de l'ensemble de notre parc installé, à la fois en France et au Brésil", s'est félicité Frédéric Moyne, directeur général d'Albioma, cité dans le communiqué, qui confirme les objectifs du groupe pour 2016.   Albioma vise un Ebitda compris entre 122 et 130 millions d'euros et un bénéfice net de 25 à 30 millions d'euros.   Au premier semestre, la production d'électricité de ses installations thermiques en France a atteint 1.047 gigawattheures, contre 951 GWh l'an dernier.   Le groupe a pu compter sur la totale disponibilité de ses centrales après les incidents de l'an dernier dans la centrale thermique du Moule (ALM) en Guadeloupe, et dans celle de Le Gol à La Réunion, auxquels s'était ajouté un conflit social en Guadeloupe.   La production solaire en France a atteint 47 GWh, quasi stable, malgré un ensoleillement moins favorable que l'an dernier.   Sur cette activité, trois projets d'Albioma incluant du stockage (deux à La Réunion et un en Guadeloupe) ont été sélectionnés en juin dans le cadre d'un appel d'offres national, pour une capacité totale de 5,9 MW.   Au Brésil, les deux centrales actuellement exploitées par Albioma ont aussi vu la production augmenter, mais les revenus ont été pénalisés par la baisse du prix de vente moyen de l'électricité.   L'Ebitda est toutefois repassé dans le vert à 1,4 millions d'euro contre une perte de 0,2 million l'an dernier.   Dans ce pays, le groupe a annoncé fin mai le lancement d'une nouvelle centrale de cogénération à partir de bagasse (résidus de canne à sucre) dans l'Etat de Sao Paulo, dont le groupe détiendra 40%.   L'Ebitda de l'activité à l'Ile Maurice progresse également, à 1,3 million d'euro, contre 1,2 million l'an dernier.   mhc/fpo/csg
</article>
<article titre="Albioma : redressement des résultats" date="2016 07 26" journal="Boursier (site web)">
Albioma  a vu son chiffre d'affaires progresser de 8% au premier semestre 2016 à 177,1 millions d'euros (+14% hors effet prix des matières premières). L'EBITDA est en forte croissance de 30% par rapport au premier semestre 2015 et atteint 65,4 millions d'euros. Le bénéfice net part s'établit à 14,5 millions d'euros, en croissance de plus de 50%.   La construction de la centrale bagasse/biomasse Galion 2 (40 MW, investissement de 185 millions d'euros) se poursuit en Martinique. L'achèvement des ouvrages de génie civil pour la chaudière a permis d'initier le montage des équipements. La mise en service de l'installation est prévue à l'été 2017.  Début juillet 2016, Albioma Saint-Pierre, détenue à 51% par Albioma, a levé à des conditions favorables une dette bancaire de 45 millions d'euros, d'une maturité de 22 ans, assurant le financement de la construction de la turbine à combustion de Saint-Pierre, à La Réunion (41 MW, investissement de 60 millions d'euros). Les travaux de construction de l'installation ont débuté et sa mise en service est prévue pour la fin 2017.    Les travaux de mise en conformité IED de la centrale du Gol (investissement de 80 millions d'euros) sont engagés. Un financement de 135 millions d'euros a été levé au cours du semestre en vue de leur financement et du refinancement de la dette existante de la centrale dans de très bonnes conditions de marché.   Au Brésil, la centrale Albioma Rio Pardo Termoelétrica a exporté 56 GWh sur le réseau électrique brésilien, à comparer à 36 GWh au premier semestre 2015.   Albioma confirme ses objectifs 2016 d'EBITDA de 122 à 130 millions d'euros et de bénéfice net de 25 à 30 millions d'euros.
</article>
<article titre="[Focus Pays] Brésil : un marché compliqué mais porteur" date="2016 07 10" journal="GreenUnivers (site web)">
2016-07-18T16:11:03+00:00    GreenUnivers poursuit sa série sur les marchés les plus porteurs à l'international avec l'aide des correspondants locaux de Business France. Après le Mexique, la rédaction s'intéresse au Brésil, un géant en train de vivre un véritable paradoxe économique. Annoncé comme le grand marché aux perspectives durables, le premier Etat d'Amérique du Sud traverse une double crise politique et économique. L'affaire Petrobras, le feuilleton sur la destitution de la présidente Dilma Rousseff, son éviction et enfin la formation d'un nouveau gouvernement ont profondément affecté le climat des affaires. Selon les dernières prévisions, le Brésil sera en 2016, pour la deuxième année consécutive, en récession (- 3,5 à 4%). De quoi sérieusement s'interroger sur l'opportunité d'investir dans le pays. Pour autant les perspectives et quelques belles réussites françaises incitent à l'optimisme. Le marché brésilien s'appuie sur des fondamentaux solides de croissance. Un marché organisé  Les énergies renouvelables (EnR) ne sont pas inconnues au Brésil. L'hydroélectricité répond depuis des années à la majorité des besoins électriques du pays. Selon un rapport publié par le ministère brésilien de l'Energie en juin dernier, la part des EnR dans le mix énergétique a progressé de 39,4% en 2014 à 41,2% en 2015. Naturellement, l'éthanol/bagasse et l'hydroélectricité représentent à eux seuls près de 70% de la production d'énergie verte, l'éolien 1,5%.   Mais la crise énergétique de 2001 a fait prendre conscience des risques liés à un mix trop peu diversifié. Les énergies alternatives ont depuis fait l'objet d'un intérêt croissant. Pour les intégrer dans la matrice énergétique, le modèle est encore très dirigiste. Le ministère fixe les capacités additionnelles nécessaires et attribue les contrats aux meilleurs offrants par le biais d'un système d'enchères publiques annuelles.« Le plan énergétique décennal élaboré par le ministère de l'Energie est révisé tous les ans ce qui donne une bonne visibilité aux investisseurs. Ainsi, si on se réfère à sa dernière actualisation (décembre 2015) consacrée à l'électricité, quelque 73 GW de capacités devront être installés d'ici les dix prochaines années, soit plus de la moitié du parc existant », explique Paul Vandebeuque, chargé de développement Energie à Business France Brésil. Elles seront majoritairement issus des EnR : 24 GW d'éolien, 20 GW de biomasse et 7 GW de solaire (grandes centrales au sol uniquement), le solde étant consacré à des centrales à gaz.   A noter enfin l'engagement pris par le pays dans le cadre de la Cop21 de réduire ses émissions de gaz à effet de serre de 43% d'ici à 2030 (par rapport à 2005). Brazilia s'est également engagé à atteindre 45% d'EnR dans la consommation nationale d'énergie à l'horizon 2030, ce qui ne devrait pas constituer un grand défi.   Un système d'enchères bien rodé  Le système d'enchères mis en place par le ministère de l'Energie est ouvert à toute entreprise disposant d'une implantation dans le pays et des autorisations nécessaires (lire encadré ci-dessous). Les enchères sont non-discriminantes ce qui veut dire que toutes les technologies de production d'énergie sont admises. Les autorisations nécessaires Les projets de production d'énergie sont soumis à des autorisations diverses principalement du ressort de l'Agence nationale de l'énergie électrique (Aneel) et de l'Institut Brésilien de l'Environnement (Ibama), autorité environnementale fédérale sous tutelle du ministère de l'Environnement. Parmi les exigences, les porteurs de projets supérieurs à 10 MW doivent réaliser une étude d'impact environnemental (EIA) et un rapport d'impact environnemental (Rima), pouvant être simplifié selon les cas.   A partir de ces études, il existe 3 types d'autorisations à obtenir de l'Ibama : Licença Prévia (LP) : autorisation préalable Licença de Instalação (LI) : autorisation d'implantation Licença de Operação (LO) : autorisation de mise en serviceEn ce qui concerne les autorisations électriques : Simple enregistrement pour des installations de faible puissance (< 3 MW) ; Autorisations de concession pour les installations « octroyées » par décret du ministère ou résolution de l'Aneel ; Autorisations de raccordement au réseau de distribution auprès du distributeur local et au réseau de transport auprès de l'ONS.Le ministère brésilien de l'Energie a organisé, en 2009, une enchère spécifique à l'éolien terrestre attribuant quelque 1 800 MW répartis sur 71 parcs. Ces rounds spécifiques ont existé entre 2009 et 2013 pour l'éolien. Une réussite puisque depuis 2012, il est tout à fait compétitif et peut désormais concurrencer d'autres technologies matures lors des enchères générales. Plus récemment, en 2015, plus de 2,5 GW ont été attribués sur les deux appels d'offres EnR. « Le ministère va réitérer l'expérience d'appels d'offres spécifiques pour l'énergie solaire même si l'objectif à terme est qu'elle soit en concurrence avec les autres énergies », confirme Paul Vandebeuque.   Boom en vue pour la production solaire décentralisée  Le marché des petites installations diffuses devrait également exploser ces prochaines années grâce à l'adoption, en mai dernier, d'une nouvelle réglementation particulièrement favorable. Elle permet à chaque particulier de déduire de sa facture d'électricité la production générée par un ou des équipements EnR tel qu'un panneau solaire ou une mini-éolienne. Désormais, le principe s'étend à des groupes d'individus, ce qui permet la constitution de petites coopératives locales plus à même de financer ce genre de projets pouvant aller jusqu'à 5 MW. De plus, l'Agence nationale de l'énergie électrique (Aneel) a étendu le système même si les sites de production et consommation sont distincts (exemple type : maison de campagne/appartement en ville), à condition que le distributeur soit identique.   Rodrigo Sauaia, président exécutif de l'Association brésilienne d'énergie solaire photovoltaïque (Absolar), estime que « la révision de ces normes va permettre au Brésil de se positionner comme un référent international pour la promotion de l'énergie décentralisée, notamment solaire ». En effet, la courbe est exponentielle : entre 2014 et 2016, les connexions d'installations EnrR sont passées de 424 à 1 930, dont 90% photovoltaïques. Selon l'Aneel, les connexions devraient augmenter de 800% en 2016...   Financements coûteux  La principale problématique des investisseurs réside dans le financement des projets. Si les autorités n'ont pas imposé de  "local content"  (contrairement au secteur pétrolier), le système est fait de telle manière qu'il est néanmoins indispensable de justifier d'un contenu local pour bénéficier de financements publics de la Banque nationale de développement économique et social (BNDES), qui sont nettement meilleur marché et qui peuvent couvrir jusqu'à 70% du coût total du projet. Les taux pratiqué au Brésil n'ont en effet aucune commune mesure avec ceux du Vieux Continent : début juin, pour la septième fois consécutive, le Comité de politique monétaire (Copom) a maintenu le taux directeur de la banque centrale brésilienne à 14,25%...« Dans ces conditions, la recherche de financements de long terme à coûts attractifs est devenu la marotte des investisseurs », indique Paul Vandebeuque. A l'exception notable d'Enel Green Power, qui a remporté en 2014 des projets éoliens et solaires cumulant 344 MW (dont 254 MW PV), et qui semble s'en dispenser. En effet, il n'existe pour l'instant aucun fabricant local de panneaux photovoltaïques capable de répondre à une telle demande. De fait, le groupe italien n'a pas pu bénéficier des financements de la BNDES et a dû trouver d'autres sources de financements, probablement en devises étrangères. Une stratégie risquée en cas de variation importante des taux de change.   A noter enfin un certain nombre de barrières douanières qui renchérissent le coût d'importation des équipements énergétiques.   Des acteurs déjà en place  Les acteurs français sont déjà présents et plutôt bien placés à l'image d'Engie, premier producteur d'énergie indépendant (sa filiale Solairedirect a remporté 200 MW en décembre dernier). A noter également les succès de Voltalia (présent depuis 2006), EDF EN (suite au rachat du portefeuille de projets de 800 MW de Sowitec l'année dernière), Albioma (3 projets signés), Vinci Energies, ou encore Schneider Electric.   Côté concurrence, l'Américain SunEdison (en faillite) a remporté en 2015 quatre projets (Juazeiro Solar I à IV) pour un total de 138 MW, et est entré au capital du Brésilien Renova, deuxième producteur d'EnR du pays. Autres acteurs étrangers à prendre en considération : les entreprises chinoises. Lors d'une conférence sur le secteur en mai dernier, Charles Tang, président de la Chambre de Commerce sino-brésilienne, a indiqué que des compagnies électriques et des fonds d'investissement chinois pourraient prendre des participations dans des projets existants cumulant 1,35 GW (750 MW éoliens et 600 PV). Les fabricants de modules et autres composants démarchent déjà les grands groupes industriels brésiliens (Renova, Electrobras, Odebrecht, etc).   Report du prochain appel d'offres  La crise politique brésilienne s'est traduite par la destitution de la Présidente Dilma Rousseff. Le ministre intérimaire de l'Energie, Fernando Coelho Filho, a annoncé début juillet le report de l'appel d'offres prévu pour août. Une décision justifiée par le nécessaire recalibrage de l'équation offre/demande. « L'offre d'énergie en 2016 (contractualisée par les distributeurs il y a plusieurs années par le biais d'appels d'offres) est en excédent par rapport à la demande (qui a chuté avec la crise) et cela créé des doutes par rapport au modèle des appels d'offres qui font reposer les risques sur les distributeurs », indique Paul Vandebeuque .  Si le nouveau gouvernement est confirmé en août, il est probable qu'il engage une réforme profonde du système visant à équilibrer les risques. En les faisant porter également sur les producteurs ? Une option plus que crédible.   Accepter de perdre de l'argent à court terme  Pour autant, le ministre de centre-droit s'est montré pro-marché et a tenu à rassurer les investisseurs qui préparaient l'appel d'offres du mois prochain. Il sera, comme c'est souvent le cas au Brésil, couplé avec celui d'octobre. « Les appels d'offres restent indispensables pour soutenir les filière EnR naissantes ou en développement et le ministre souhaite donc les maintenir », explique Paul Vandebeuque. Malgré ce climat d'incertitudes, l'ensemble des acteurs interrogés par la rédaction confirme leur intérêt pour le Brésil, même si les prochaines années vont être compliquées.  «  Il faut s'attendre à souffrir, voire à perdre de l'argent à court terme, mais c'est le prix à payer pour continuer à exister sur ce marché formidable », confiait récemment, sous couvert d'anonymat, un haut dirigeant d'un groupe espagnol implanté dans le pays.    Les événements à venir  EVENEMENTS PROCHAINE EDITION THÈME CENTRAL COBEE - Congresso Brasileiro de Eficiência EnergéticaAoût 2016Efficacité énergétiqueBRAZIL WIND POWERSeptembre 2016EolienINTERSOLAR SOUTH AMERICAAoût 2016SolaireLATIN AMERICAN UTILITY WEEKSeptembre 2016UtilitiesENERSOLAR +Avril 2017SolairePOLLUTEC BRASILAvril 2017EnvironnementPOWERGEN BRASILAvril 2017ElectricitéCONNECTED SMART CITIESJuin 2017Ville intelligente    Contact Business France Paul Vandebeuque  , chargé de développement - Energie - Business France BrésilMail : paul.vandebeuque@businessfrance.fr   Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="AFOC –Objectif Afrique N° 38 – 24 JUIN 2016" date="2016 06 24" journal="Rapports - Gouvernance - Objectif Afrique">
26 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... SOMMAIRE   ACTUALITE REGIONALE   CONJONCTURE   ENVIRONNEMENT DES AFFAIRES   PROJETS ET FINANCEMENTS   SECTEURS D’ACTIVITES   AGRICULTURE ET AGROALIMENTAIRE   ENERGIE ET MATIERES PREMIERES   INDUSTRIE   SERVICES   ENTREPRISES   CARNET ET AGENDA  CAMEROUN : VIVENDI OUVRE  UNE SALLE...    Pour voir le rapport :  http://www.diplomatie.gouv.fr/IMG/pdf/objectif_afrique_n38_24_juin_2016_cle875d75.pdf
</article>
<article titre="Eolien + stockage : gros plan sur la centrale pionnière de Quadran en Guadeloupe" date="2016 06 23" journal="GreenUnivers (site web)">
2016-06-23T19:24:21+00:00    C'est une première en France et une rareté partout ailleurs : une centrale éolienne de 2,5 MW dotée de capacités de stockage de 460 kWh/600 kW sera inaugurée la semaine prochaine sur l'île de Marie-Galante (Guadeloupe). Issue d'un repowering à 9 M€ réalisé par le développeur Quadran qui l'a construite en 1997, l'installation de Petite-Place compte 9 turbines Vergnet rabattables de 275 kW - du fait des difficultés de transport par barge et du gros risque cyclonique - couplées à des batteries Saft, le tout conclu par un convertisseur Nidec ASI de 1 700 kVA.   Sans stockage, pas de croissance des EnR outre-mer  Avec les batteries et la conversion d'électricité, le lissage de production représente un surcoût de 30%. Son intérêt ? « Il offre une meilleure tenue à la fréquence et à la tension et surtout intervient en soutien de la prévision, ce qui correspond à une exigence majeure du gestionnaire de réseau », explique Jérôme Billerey, directeur général de Quadran. EDF SEI (Systèmes énergétiques insulaires) autorise certes les énergies renouvelables ultramarines à dépasser la fameuse limite de 30% de l'électricité injectée - une nécessité puisque les EnR doivent en théorie composer 50% de la consommation finale d'énergie de ces territoires dès 2020... - mais avec des règles considérées comme draconiennes par le développeur biterrois. Lequel il est vrai bénéficie d'un tarif  ad hoc  de 235 euros/MWh, issu de l'arrêté tarifaire du 8 mars 2013 conçu sur mesure par la Direction générale énergie et climat.Repère : EnR et réseau : le projet Venteea entre dans sa dernière phase - Juin 2015 Pour 15%, t'as plus rien  Un exemple des conditions à respecter ? EDF SEI n'achète pas le courant de la centrale de Petite-Place lorsque la production varie de 15%, à la hausse ou à la baisse, par rapport à la prévision adressée la veille. Et la fiabilité de ladite prévision ne garantit pas du tout de rester dans cette plage. Le stockage devient alors un  must.  Comme l'a précisé Denis Lefebvre, directeur prospective et réseaux de Quadran, lors de la 7e édition de la conférence Energie et météo ce 23 juin à Paris, « les batteries permettent de compenser les inévitables écarts et de rester plus longtemps à l'intérieur du bandeau de prévision. Avec un choix à faire sur la puissance : les grosses batteries coûtent cher, les petites garantissent de fortes pénalités ».   Un business plan à 10% de pénalités  Pourtant, même avec les batteries, les incertitudes du projet ne fondent pas totalement : les pannes sur les auxiliaires, les difficultés de communication, les déficiences mêmes du réseau de transport peuvent saboter la qualité de la prévision et mettre en péril l'équilibre du projet, tout au moins dans les premiers mois de rodage - EDF SEI accepte d'ailleurs une tolérance sur les engagements pendant les deux premières années.   Pour ce parc entré en service en avril, Quadran a en tous cas retenu une hypothèse financière qualifiée de conservatrice, basée sur un volume de pénalités de 10% du chiffre d'affaires total.   Les banques ne se précipitent pas   Cette prudence n'a pas suffi à convaincre les banquiers habituels des sociétés de projets de Quadran, qui a dû se tourner vers de nouveaux intervenants. L'AFD (Agence française de développement) a pour sa part prêté 3,7 M€ sur 15 ans.   Dommage pour les absents, c'était le moment d'apprendre ! L'éolien et le solaire avec stockage deviennent en effet la règle dans les ZNI et non l'exception. Ce que montre le succès du récent appel d'offres solaire dédié aux ZNI, de même que les nouveaux projets éoliens avec lissage de production, en  repowering  ou pas, que préparent Quadran (deux fois 20 MW en Guadeloupe), EDF EN ou Valorem, entre autres.   Cette percée correspond à une tendance lourde, qui n'est pas seulement celle impulsée par la loi sur la transition énergétique, mais aussi une réalité économique déjà structurante rappelée par Jérôme Billerey : « Dans le seul éolien avec stockage, notre offre à 235 euros se situe au niveau du prix du mix énergétique dans les Antilles, fait de charbon, de fuel et de bagasse. Elle est très inférieure au prix de l'électricité issue du seul diesel, qui atteint 330 euros par mégawatt. Dans le solaire plus batteries, nous sommes déjà bien en-deçà des 200 euros... ».   Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="De la  bagasse pour le lait" date="2016 06 22" journal="Emballages magazine.com">
RPC Promens a développé la bouteille Modul avec Scanfill.-   Le britannique RPC annonce le lancement d'une bouteille de lait blanche d'un litre, baptisée Modul, entièrement fabriquée en polyéthylène haute densité (PEhd) issu de la canne à sucre. Le biopolymère est fourni par le brésilien Braskem. Le groupe d'emballage plastique souligne une autre caractéristique essentielle : une charge minérale spécifique- à hauteur de 45%- est mélangée au PEhd par le suédois Scanfill (groupe Polykemi), ce qui permet de réduire la quantité de matériau nécessaire- sans impacter la résistance- et d'améliorer le profil environnemental du flacon. « Une première sur le marché européen », assure RPC. La Modul a été produite par extrusion-soufflage par la division RPC Promens Consumer en Scandinavie. Elle est proposée avec plusieurs bouchons et comporte une étiquette entourante apposée par RPC Promens. Le leader suédois des produits laitiers Skånemejerier l'a sélectionnée pour sa gamme de lait non homogénéisé. « Avec ce nouveau matériau, nous pouvons offrir à nos clients une solution renouvelable et durable », indique Jan Weier, le directeur des ventes senior de RPC Promens. RPC a réalisé en 2015-2016 (exercice décalé) un chiffre d'affaires de 1,64 milliard de livres (2,1 milliards d'euros), compte 112 usines et emploie 18 300 salariés.
</article>
<article titre="Quotas sucriers outre-mer: Marine Le Pen interpelle le gouvernement" date="2016 06 15" journal="AFP Infos Françaises">
Paris, 15 juin 2016 (AFP) - - Marine le Pen a interpellé la ministre des Outre-mer George Pau-Langevin sur la fin programmée par l'Europe des quotas sucriers, l'accusant d'être "spectatrice devant le désastre qui s'annonce", faisant réagir mercredi l'ancien ministre des Outre-mer Victorin Lurel, qui a dénoncé sa "méconnaissance" du dossier.   Dans une lettre ouverte adressée lundi à la ministre, la présidente du Front national explique que la suppression des quotas sucriers est une "mesure terriblement inquiétante pour nos compatriotes ultramarins", qui "verront disparaître la digue fragile qui protégeait encore la filière de la canne à sucre à la Réunion, en Guadeloupe et en Martinique".  En place depuis 1968, les quotas de production de sucre et de prix garantis seront levés le 1er octobre 2017, dernière étape de la libéralisation de la Politique agricole commune (PAC).   "Allez-vous encore longtemps, Madame la Ministre, rester spectatrice devant le désastre économique et social qui s'annonce et qui n'est pas sans rappeler la situation vécue par nos producteurs de lait, confrontés eux-aussi à une mesure de suppression des quotas ?", interroge Mme Le Pen.   La présidente du FN estime qu'il faut "cesser de subir les diktats ultralibéraux de l'Union européenne" et "préparer le rétablissement de quotas sucriers encadrés par une Politique Agricole Française souveraine".   L'entourage de la ministre a précisé à l'AFP qu'une lettre de réponse avait été envoyée à Mme Le Pen, pour lui rappeler que le gouvernement avait engagé des discussions avec la Commission européenne pour le versement d'une aide annuelle complémentaire au secteur de la canne à partir d'octobre 2017, et qu'elle avait demandé que les sucres spéciaux (sucres roux non destinés au raffinage), qui constituent un marché de niche stratégique pour les outre-mer, soient exclus des accord de libre-échange.   Pour le secrétaire national du PS chargé des Outre-mer et ancien ministre des Outre-mer Victorin Lurel, Mme Le Pen "fait une nouvelle fois preuve de sa méconnaissance des dossiers communautaires" et semble ignorer le sens de l'action du gouvernement engagée depuis 2012 en matière de modernisation et de soutien des filières agricoles traditionnelles des Outre-mer".   "C'est bien la gauche qui a décidé d'une aide supplémentaire de 38 millions d'euros par an à la filière canne, qui a veillé à exclure les sucres spéciaux des négociations d'accords commerciaux communautaires et qui a revalorisé substantiellement la prime bagasse (résidus de canne à sucre utilisé dans les centrales électriques pour produire de l'énergie, ndlr) aux producteurs", a-t-il fait valoir.   caz/dch/ct
</article>
<article titre="GLOBAL BIOENERGIES L'Oréal pourrait bien inclure son hydrocarbure bio dans ses crèmes" date="2016 06 7" journal="Investir (site web)">
Gilbert Dupont croit à fond en la technologie du chimiste vert Global Bioenergies (GBE), capable de produire des biocarburants à partir de glucose végétal (sucre de canne, betterave...), d'amidon ou de déchets agricoles (paille de blé, tiges de maïs, bagasse de canne à sucre...). Dans une note publiée ce matin, au lendemain de l'annonce par la « cleantech » d'un nouveau partenariat d'envergure - dans les cosmétiques cette fois, avec L'Oréal -, le cabinet d'analyses estime que le cours de l'action a encore le potentiel de doubler en dépit d'un bond de déjà 20% ce mardi. « L'Oréal prend désormais part au projet IBN-One et testera des lots d'isobutène produits par Global Bioenergies, salue-t-il, confirmant son objectif de cours de 50,7 euros. [...] Ce partenariat confirme une fois de plus le fort intérêt des industriels pour le bio-isobutène, renforce clairement la crédibilité de GBE auprès des acteurs du secteur et pourrait aboutir à un contrat d'approvisionnement dans le cas où le produit apporterait satisfaction. »   Global Bioenergies a mis au point une technologie qui permet à partir de bactéries modifiées et nourries au sucre de créer par fermentation de l'isobutène, un hydrocarbure - utilisé dans la fabrication de carburants, de pneus, de bouteilles en plastique et aussi des cosmétiques - obtenu jusqu'alors par craquage du pétrole.    L'usine opérationnelle en 2019   Voilà huit ans que la société travaille à son procédé. Elle a d'abord apporté la preuve de concept en faisant la démonstration dans un prototype de petite taille que de l'isobutène pouvait être produit à partir de ressources renouvelables. Une fois la phase de recherche achevée en 2011, le procédé a ensuite été développé en laboratoire puis vint à la mi-2013 le début de la phase d'industrialisation qui s'est déroulée en deux étapes : la construction d'un pilote industriel d'une capacité de 10 tonnes à Pomacle-Bazancourt, près de Reims, puis celle d'un démonstrateur d'une capacité de 100 tonnes à Leuna en Allemagne. La prochaine étape pour la « cleantech » est la construction d'une usine d'une capacité de 50.000 tonnes qui devrait être opérationnelle en 2019. Elle a créé pour ce faire une joint-venture à 50%, IBN-One, avec la coopérative agro-industrielle Cristal Union, propriétaire des marques Daddy Sucre et Erstein, et qui représente environ 40% de la production de betteraves en France. Les deux entités se connaissent très bien puisque la bioraffinerie de Pomacle-Bazancourt dans laquelle est installé le pilote industriel de l'entreprise verte appartient à ARD, filiale de développement industriel à environ 20% de Cristal Union.   En même temps que GBE annonçait hier soir que L'Oréal allait tester ses lots de composés dérivés d'isobutène biologique, elle faisait également état de l'octroi d'un financement de 9 millions d'euros par le programme Investissements d'Avenir de l'Agence de l'environnement et de la maîtrise de l'énergie (Ademe). Composé d'avances remboursables, ce financement sera progressivement libéré sur une durée de 44 mois en fonction de l'avancement du projet IBN-One. 5,7 millions, dont une tranche initiale 0,86 million, seront attribués à Global Bioenergies afin de lui permettre de terminer le développement industriel du procédé (comme améliorer la qualité du carburant produit). Les 3,3 millions d'euros restants seront destinés à IBN-One et permettront notamment de financer les travaux d'ingénierie de l'usine et la validation commerciale de dérivés destinés, outre aux cosmétiques, aux carburants (essence, kérosène) et aux matériaux.    « L'obtention de ce financement marque une étape importante sur la route qui nous emmène vers la construction et le démarrage de l'usine IBN-One, se réjouit Barnard Chaud, PDG d'IBN-One. Nous venons de clore la phase d'ingénierie conceptuelle de l'usine, et basculons maintenant dans les phases aval d'ingénierie. »   A ce jour, Global Bioenergies a noué des partenariats industriels avec Arkema, Audi, Cristal Union et le polonais Synthos. Par ailleurs, Arlanxeo, une filiale de Saudi Aramco et Lanxess, a récemment reçu des lots d'isobutène bio-sourcés pour des essais.   Aux côtés de Gilbert Dupont, à qui l'on pourrait reprocher d'avoir une opinion biaisée puisque le cabinet assure la liquidité du titre en Bourse et a été en chargé du placement de l'augmentation du capital, quatre autres analystes tiennent une opinion positive sur l'action du chimiste vert. D'après la compilation faite par l'agence Bloomberg, l'objectif de cours médian est de 48,57 euros... Soit là aussi plus du double du cours actuel, qui a tout de même bien souffert de la chute des cours du pétrole rendant la technologie de Global Bioenergies moins compétitive.
</article>
<article titre="CSR : un nouveau cadre réglementaire pour faire émerger la filière Déchets" date="2016 05 31" journal="La Gazette des communes (site web)">
Le Décret n° 2016-630 du 19 mai 2016 modifiant la nomenclature des installations classées pour la protection de l'environnement (NOR: DEVP1510181D) s'inscrit bien entendu dans les mesures d'application de la Loi n° 2015-992 du 17 août 2015 relative à la transition énergétique pour la croissance verte, et cela à plusieurs titres. En effet, valoriser en énergie ces déchets non recyclables, c'est tout d'abord un objectif de la politique nationale prévention et de gestion des déchets et du développement de l'économie circulaire mais c'est aussi contribuer à l'objectif affiché par cette loi, la moitié des déchets mis en centre de stockage devant en être détournés en 2025. Des installations complémentaires au recyclage et réemploi Or, actuellement, les combustibles solides de récupération (CSR) sont surtout utilisés en cimenteries. Alors que l'on estime le tonnage à 150 000 tonnes environ, l'objectif du ministère est de passer à 2,5 millions de tonnes par an. Mais ce sujet avait fait débat lors de l'examen de la loi sur la transition énergétique car le risque est grand que, dans ces conditions, les incinérateurs deviennent des « aspirateurs à déchets » au détriment d'une politique de recyclage et de réemploi. La loi a donc proposé un dispositif qui a satisfait les professionnels et même les associations. Elle a confirmé avec l'article L.541-1 du code de l'environnement (CE) la nécessité d'assurer uniquement la valorisation énergétique des déchets qui ne peuvent pas (actuellement) être recyclés ou qui sont issus d'une collecte séparée ou d'un tri (les « refus »).   Il reste que la notion de « actuellement non recyclables » est toujours aussi relative. A ce sujet, il y a toutefois un point positif : ce même article du CE prévoit que « l'Agence de l'environnement et de la maîtrise de l'énergie remette tous les trois ans un rapport au gouvernement sur la composition des combustibles solides de récupération et sur les pistes de substitution et d'évolution des techniques de tri et de recyclage ». Une autre contrainte est inscrite par la loi dans cet article : chaque installation doit être « dimensionnée au regard d'un besoin local et conçue de manière à être facilement adaptable pour brûler de la biomasse ou ou, à terme, d'autres combustibles afin de ne pas être dépendantes d'une alimentation en déchets. »   Répondre à une demande locale de production de chaleur Ces dispositions sont d'ailleurs reprises dans le décret du 19 mai dernier mais simplement dans sa notice et non dans ses articles. Inutile de répéter ce qui est déjà dans la loi. La notice indique que « ces installations devront répondre à une demande locale pour justifier de la capacité de l'installation ».   La notice précise par ailleurs que « l'objectif principal est de produire de la chaleur avec ou sans cogénération » (une précision non reprise dans le corps du texte réglementaire) avec par ailleurs une dérogation possible pour les les départements d'outre-mer. « Au vu de la spécificité de leur réseau électrique, local par nature, et de la faiblesse du besoin de chaleur, la valorisation énergétique réalisée à partir de combustibles solides de récupération peut être pratiquée dans des installations produisant exclusivement de l'électricité » précise le texte.   Des installations de production d'énergie Dans ce cadre et au regard de ces objectifs, le Décret n° 2016-630 crée tout d'abord une nouvelle rubrique 2971 dans le régime installation classée pour la protection de l'environnement (ICPE). Ces installations de production d'énergie sont en effet soumises à un régime d'autorisation mais elles étaient préalablement mentionnées à la rubrique 2771, c'est-à-dire les incinérateurs, ce qui ne permettait pas de tenir compte des spécificités relatives à leur finalité de production d'énergie. A noter que cette rubrique 2917 est relative aux ICPE qui ont pour objets la préparation des CSR et la production d'énergie à partir de ces derniers. Elles entrent donc dans la catégorie des « 29XX », c'est-à-dire celle des installations thermiques produisant de l'énergie (comme les chaufferies gaz et biomasse).   Cette simplification a pour but d'inciter les industriels et surtout les exploitants des Réseaux de chaleur à se détourner des énergies fossiles et à s'orienter vers des ressources locales. Evidemment, il faudra que le ou les arrêtés fixant les prescriptions générales de fonctionnement des ICPE soumises à autorisation différencient ces nouvelles installations des incinérateurs « classiques ». Certaines précautions resteront toutefois évidentes : par exemple, l'éjection des cendres produites en plus grande quantité que lorsqu'on brûle du bois ou de la bagasse, le traitement des oxydes d'azote, la désulfurisation ... Des filtres seront à prévoir mais ils devraient être moins complexes que ceux des incinérateurs.   Une définition des CSR en attente de précisions Ceci implique que les exploitants devront être attentifs à la qualité des CSR utilisés. C'est d'ailleurs déjà le cas pour les cimentiers qui imposent aux collectivités d'exclure les déchets contenant du PVC des CSR fournis, ceci afin d'éviter la production de chlore. Ceci renvoie donc à la définition des CSR (inscrit dans le décret) et à leurs caractéristiques (précisées dans un futur arrêté mis en consultation en octobre 2015).   Concernant la définition, le Décret du 19 mai dernier confirme bien que les CSR restent « un déchet non dangereux solide » et que, comme cela a déjà été précisé, ils sont des « refus de tri » puisqu'ils doivent être issus d'un tri « de manière à en extraire la fraction valorisable sous forme de matière dans les conditions technico-économiques du moment ». Leurs spécificités est bien entendu d'être utilisés comme combustible dans une installation relevant de la rubrique 2017 ICPE. Le décret précise par ailleurs que ces déchets sont toujours des CSR Lorsqu'ils « sont associés des combustibles autorisés au B de la rubrique 2910 » : des combustibles sortis du statut de déchets ou le biogaz produit par méthanisation, donc.   Mais pour être plus précis, un arrêté sera publié. Jusqu'à présent, on pouvait déjà s'appuyer sur la norme de l'Afnor EN-15359 qui propose trois critères: un critère économique (le pouvoir calorifique inférieur ou PCI), un critère technique (la teneur en chlore) et un critère environnemental (la teneur en mercure). Le syndicat Valordec (branche spécialisée de FEDEREC) a aussi initié une nouvelle nomenclature pour les CSR dans une étude diffusée début mars 2016. Elle définit quatre critères chimiques pour répondre à des exigences réglementaires : le pouvoir calorifique inférieur (PCI), la concentration d'azote, le taux de souffre et la quantité totale de composants halogénés (brome, fluor, chlore). Pour chaque valeur, quatre seuils ont été établis pour permettre aux fabricants de CSR De se fixer des objectifs précis. Par ailleurs, trois critères physiques apparaissent essentiels pour l'optimisation de l'installation de combustion : le taux de cendre, la densité du CSR et sa granulométrie.   Des soutiens au financement Concernant l'arrêté en attente, le débat a eu lieu car certains critères restaient similaires à ceux des incinérateurs : le PCI, le taux d'humidité, le taux d'indésirables ... Mais le taux du chlore ou du brome n'est pas finalisé. Les industriels estiment qu'il faut être plus souple si l'on veut développer cette filière tout en l'encadrant.   En effet, la Fnade (la fédération nationale des activités de la dépollution et de l'environnement, qui soutient cette évolution réglementaire) indiquait en 2015 que les débouchés autres que les cimenteries demanderaient des dispositifs de soutien. Dans l'industrie, la Fédération estime le surinvestissement et l'aide nécessaires pour concevoir une chaufferie brûlant des CSR représentent 35 à 45 % du montant des travaux. Pour produire la chaleur d'un réseau de chauffage urbain, le chiffre monte d'après elle, à 50 voire 60 %.
</article>
<article titre="Albioma développe un 3ème projet au Brésil" date="2016 05 30" journal="AFP Infos Economiques">
Paris, 30 mai 2016 (AFP) - - Le producteur français d'électricité Albioma a annoncé lancé lundi un troisième projet au Brésil avec le développement d'une nouvelle centrale de cogénération dans l'État de Sao Paulo.   Cet investissement est estimé à 100 millions de réals (24,9 millions d'euros), précise la société française dans un communiqué. "Ce projet s'inscrit parfaitement dans la stratégie de développement d'Albioma au Brésil", a souligné Jacques Pétry, PDG d'Albioma, cité dans le communiqué.   Albioma détiendra 40% d'une société qui sera propriétaire exploitante de l'usine de cogénération de Vale do Paraná, filiale de du groupe Pantaleon, le principal producteur de sucre d'Amérique centrale.   Cette centrale d'une puissance de 48 mégawatts (MW), dont la mise en service est prévue en 2021, utilisera comme source d'énergie la bagasse, le résidu ligneux résultant du broyage de la canne à sucre.   Albioma affirme que le projet est rentable dans la mesure où 80% des ventes de la production d'électricité attendue sont "sécurisées à long terme à un tarif élevé".   L'an dernier, Albioma, surtout présent outre-mer, a acquis 65% du capital de la société Codora Energia, propriétaire d'une centrale de cogénération de 48 mégawatts (MW) dans l'Etat de Goias (centre du Brésil).   En 2014, elle avait acquis pour 43 millions d'euros Rio Pardo Termoeletrica, une centrale de cogénération située dans la région de Sao Paulo qui fonctionne avec de la bagasse.   pan/fpo/php
</article>
<article titre="ALBIOMA signe un troisième projet au Brésil" date="2016 05 30" journal="Option Finance (site web)">
AOF - 30 mai 2016   (AOF) - Albioma a annoncé la signature, le 20 mai, d'un accord de coopération avec Vale do Paraná en vue de développer une nouvelle cogénération dans l'État de São Paulo au Brésil. Vale do Paraná, une distillerie dont la capacité annuelle de broyage est de 2 millions de tonnes de cannes, est une filiale du groupe Pantaleon, principal producteur d'Amérique Centrale (capacité de broyage de 13 millions de tonnes de canne à sucre) basé au Guatemala, et de Manuelita, un acteur diversifié de l'agro-industrie basé en Colombie.   Ce dernier a une capacité de broyage de 4,5 millions de tonnes de canne à sucre en Colombie et au Pérou. Le producteur d'énergie indépendant spécialisé dans la bagasse, coproduit fibreux de la canne à sucre et Vale do Paraná créeront ensemble une société de projet qui sera propriétaire exploitante de l'usine de cogénération de Vale do Paraná. Cette entité construira une extension portant la puissance de l'installation de 16 à 48 MW, dont 30 MW seront exportés sur le réseau électrique.   Albioma détiendra 40 % de la société de projet et assurera le rôle de partenaire technique pendant les phases de construction et d'exploitation en apportant son savoir-faire de spécialiste mondial de la cogénération à haute efficacité énergétique de la bagasse.   Vale do Paraná a participé avec succès au dernier appel d'offres (leilão A-5), sécurisant la vente sur le marché règlementé de 80 % de la production d'électricité attendue. 120 GWh/an ont ainsi été vendus sur 25 ans à compter du 1er janvier 2021 au prix historiquement élevé de 245,2 réals/MWh (base 2016) indexé sur l'inflation, garantissant la rentabilité du projet.   L'investissement, estimé à 100 millions de réals (base 2016), est éligible à un financement à long terme de la BNDES (Banque nationale de développement du Brésil). Sa réalisation reste soumise à la levée de conditions suspensives (dont l'obtention des financements et de toutes les autorisations nécessaires à la construction et au raccordement de la cogénération).  Cet article est paru dans Option Finance (site web)
</article>
<article titre="ALBIOMA se renforce au Brésil" date="2016 05 30" journal="Option Finance (site web)">
AOF - 30 mai 2016   (AOF) - Albioma progresse de 1,05% à 14,45 euros après la signature d'un troisième projet au Brésil jugé porteur par des analystes. Le producteur français d'énergie indépendant spécialisé dans la bagasse, coproduit fibreux de la canne à sucre, a conclu un accord de coopération avec Vale do Paraná en vue de développer une nouvelle cogénération dans l'État de São Paulo.    Vale do Paraná, une distillerie dont la capacité annuelle de broyage est de 2 millions de tonnes de cannes, est une filiale du groupe Pantaleon, principal producteur d'Amérique Centrale (capacité de broyage de 13 millions de tonnes de canne à sucre) basé au Guatemala, et de Manuelita, un acteur diversifié de l'agro-industrie basé en Colombie. Ce dernier a une capacité de broyage de 4,5 millions de tonnes de canne à sucre en Colombie et au Pérou.   Albioma et Vale do Paraná créeront ensemble une société de projet qui sera propriétaire exploitante de l'usine de cogénération de Vale do Paraná. Cette entité construira une extension portant la puissance de l'installation de 16 à 48 MW, dont 30 MW seront exportés sur le réseau électrique.   Albioma détiendra 40 % de la société de projet et assurera le rôle de partenaire technique pendant les phases de construction et d'exploitation en apportant son savoir-faire de spécialiste mondial de la cogénération à haute efficacité énergétique de la bagasse.   Vale do Paraná a participé avec succès au dernier appel d'offres, sécurisant la vente sur le marché règlementé de 80 % de la production d'électricité attendue. 120 GWh/an ont ainsi été vendus sur 25 ans à compter du 1er janvier 2021 au prix historiquement élevé de 245,2 réals/MWh (base 2016) indexé sur l'inflation, garantissant la rentabilité du projet.   L'investissement, estimé à 100 millions de réals (base 2016), est éligible à un financement à long terme de la BNDES (Banque nationale de développement du Brésil).   Portzamparc a salué cette opération qui a toujours comme critères la qualité de la contrepartie, la sécurisation des tarifs et le financement local à long terme dans de bonnes conditions. Le broker a donc confirmé sa recommandation Acheter et son objectif de cours de 18,3 euros sur la valeur.   Un avis partagé par CM-CIC Securtities qui a maintenu sa recommandation d'Achat et son objectif de cours de 23 euros. Le bureau d'études observe qu'Albioma constitue un portefeuille de projets brésilien qui permet d'augmenter la valeur de la croissance du groupe qui, sur le niveau de cours de bourse actuel, n'est selon lui pas valorisée.  Cet article est paru dans Option Finance (site web)
</article>
<article titre="Nouveau projet brésilien pour Albioma" date="2016 05 30" journal="GreenUnivers (site web)">
2016-05-30T18:29:23+00:00    Et de trois ! Le producteur indépendant d'énergie, spécialisé dans la valorisation de la bagasse, va développer un troisième projet de cogénération au Brésil, un peu plus de deux ans après avoir réussi une première implantation dans ce pays. Il a signé un accord de coopération avec la distillerie Vale do Paraná pour monter une unité de cogénération dans l'État de São Paulo. Cette distillerie dont la capacité annuelle de broyage est de 2 millions de tonnes de cannes, est une filiale du groupe Pantaleon, gros producteur d'Amérique centrale (capacité de broyage de 13 millions de tonnes de canne à sucre) basé au Guatemala et de Manuelita, un acteur diversifié de l'agro-industrie basé en Colombie (capacité de broyage de 4,5 millions de tonnes de canne à sucre en Colombie et au Pérou), précise Albioma dans un communiqué. Albioma et Vale do Paraná créeront ensemble une société de projet qui sera propriétaire exploitante de l'usine de cogénération de Vale do Paraná. Cette entité construira une extension portant la puissance de l'installation de 16 à 48 MW, dont 30 MW seront exportés sur le réseau électrique. Le groupe français détiendra 40% de la société de projet et assurera le rôle de partenaire technique pendant les phases de construction et d'exploitation. L'investissement est estimé à 100 millions de réals (25 M€).   Ce projet a été retenu lors du dernier appel d'offres brésilien (leilão A-5), sécurisant la vente sur le marché règlementé de 80% de la production d'électricité attendue. 120 GWh/an ont ainsi été vendus sur 25 ans à compter du 1er janvier 2021 au prix historiquement élevé de 245,2 réals/MWh (base 2016) indexé sur l'inflation, garantissant la rentabilité du projet, selon le communiqué.    Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="La  bagasse, un bois guyanais qui pourrait concurrencer le teck..." date="2016 05 2" journal="NewsPress (français)">
La bagasse est un bois de Guyane durable, relativement dense et très stable en service, c'est-à-dire peu sensible aux variations de l'humidité de l'air. Cette stabilité lui est conférée par des métabolites mises en place lors de la transformation de l'aubier* en bois de coeur. C'est ce que viennent de démontrer des chercheurs du CNRS et du Cirad qui ont publié ces résultats dans PloS One.   Un produit 100 % naturel, issu des forêts de Guyane, durable et se comportant bien en plantation : la bagasse (Bagassa guianensis Aubl.) pourrait devenir un sérieux concurrent local au teck qui a pourtant, lui, a fait l'objet de presque 200 ans de sélection... Les propriétés du bois de bagasse sont en effet remarquables à plusieurs titres. « C'est un bois d'une grande stabilité » , souligne Jacques Beauchêne, responsable du laboratoire de Sciences du bois du Cirad à Kourou (Pariacabo). « Cette stabilité est due à des métabolites mises en place par l'arbre dans son bois de coeur pour le protéger des insectes et des champignons. Chez la bagasse, nous avons constaté que plus il y avait de métabolites, plus le bois de coeur était stable. Ces métabolites présents chez la plupart des espèces ont souvent un rôle modérateur sur le retrait du bois au séchage, mais ce n'est pas toujours ainsi, chez l'angélique par exemple, la première essence commerciale de Guyane, la mise en place des métabolites augmente le retrait » , explique Jacques Beauchêne. Le retrait est le phénomène de variation des dimensions du bois avec le taux d'humidité.   Ces résultats publiés fin mars dans PloS One ont été obtenus dans le cadre du travail de thèse de Julie Bossu, doctorante au laboratoire de Sciences du bois.   La bagasse était déjà connue pour sa croissance rapide, sa densité moyenne (bois ni trop lourd ni trop léger) et sa forte durabilité (bonne résistance dans le temps aux agressions extérieures).   Sa stabilité en fait aujourd'hui une espèce locale prometteuse pour de futures plantations en Guyane. La mise en place de plantations durables de bois d'oeuvre, en Guyane, pourrait à terme soutenir, voire remplacer en grande partie l'exploitation de la forêt naturelle.   *L'aubier est la partie de bois du tronc, physiologiquement active. Le bois de coeur est le bois exploité commercialement.
</article>
<article titre="Développements Barquettes et pots : les matériaux en effervescence" date="2016 05 1" journal="Process Alimentaire, no. 1336">
De façon visible ou non, les emballages rigides opèrent une mue sur la plupart des marchés. Ici le polystyrène laisse place à du PET, là le carton remplace le plastique. À l'inverse, le plastique vient imiter le kraft, le bois, la céramique et concurrencer le verre au rayon des conserves. Pour des raisons écologiques, la cellulose moulée fait aussi son apparition, notamment en CHR. Tandis que le métal et le verre se défendent en innovant. Aux rayons frais et épicerie, le champ des possibles s'élargit. Mais le principal arbitre reste le coût total (prix et coûts de production et d'éco-contribution), devant l'attractivité et les vertus écologiques. 1 Le carton s'associe au plastique et à l'alu La qualité et la surface d'impression qu'offre le carton séduit de plus en plus de marchés. A l'image des barquettes et boîtes Pack Alim déployées, depuis leur création en 2004, sur les segments des salades (taboulé Leclerc Bio Village, carottes râpées Biocoop par exemple), du snacking (boîtes à cheeseburger, kebab et panini Charal), du poisson et de la viande (avec notamment une récente version «Flat'Pack» pour les produits sous skin). Nature &amp; Cie vient d'adopter le concept Novabox de Pack Alim pour ses nouvelles box de pâtes au rayon frais. Avec ses barquettes en carton imprimées et pelliplaquées à fort impact visuel, Virgin BioPack multiplie aussi les contacts. «Nous sommes déjà présents avec notre Hybric Tray chez Biocoop et La Vie Claire à travers certains produits Léa Nature, illustre Christophe Tranchard chez Virgin BioPack. Ainsi que chez Rewe en Allemagne sur des produits bio à marque de distributeur.»   Scolarest (groupe Compass), spécialiste de la restauration collective en établissements scolaires, expérimente pour sa part depuis début 2016, la fourniture de préparations chaudes en barquettes en carton Food K (Guillin-Rescaset) dans 30 écoles de Rhône-Alpes. «Ces barquettes présentent des avantages certains par rapport aux barquettes en polypropylène utilisées auparavant. En termes d'impact écologique et de maîtrise des accidents du travail dans la manipulation de produits chauds, notamment», justifie Francis Faghel, directeur de la cuisine centrale Scolarest de Pusignan. Seul bémol : ces barquettes génèrent un surcoût de huit centimes par couvert. «Dans la conjoncture actuelle, les communes ne sont pas prêtes à intégrer ce surcoût dans leur budget», déplore le directeur, qui n'est pas certain que l'expérimentation dure. Il appelle ses confrères et concurrents à tenter l'expérience pour faire baisser le coût unitaire des barquettes en carton.   Au rayon épicerie (sucrée et salée), les boîtes composites en carton s'implantent aussi sur des marchés de plus en plus variés. Sonoco, par exemple, vient de développer pour l'Allemand Hans Freitag des boîtes à biscuits signées Fit for Fun, composées d'une paroi multicouche (carton et liner en aluminium). Comparativement à un sachet, les biscuits sont mieux protégés et se conservent davantage après ouverture grâce à un couvercle pression en plastique.   2 Le 100 % papier-carton se thermoforme Le carton tente aussi de concurrencer le plastique sur les produits secs ou peu humides. BillerudKorsnäs, par exemple, a développé de nouvelles formes de pots thermo-soufflés en papier étirable Fibre Form suivant le concept établi en 2013 avec le fabricant italien de machines Curti (lire avril 2015 p. 79 et 80). Leur société conjointe, FreeForm Packaging, a déjà commercialisé deux machines FFS (Form-Fill-Seal) en Europe dédiées à ce concept. L'une d'elles est installée chez le spécialiste du co-packing Vetipak aux Pays-Bas. «Pour la première fois au monde, notre système permet aux industriels de thermoformer eux-mêmes des pots en papier/carton, explique Benoist de Moucheron, Business Development Manager Europe du Sud, de FreeForm Packaging. Il offre l'opportunité de réaliser plusieurs formes différentes car les moules ne sont pas très coûteux et les changements de formats très rapides.» La joint-venture vise les produits secs mais aussi les produits pâteux peu humides, voire les plats préparés.   3 La cellulose moulée attire le CHR L'intérêt du carton réside notamment dans sa recyclabilité et sa surface imprimable. Cet impact visuel est un peu moins évident sur les barquettes en cellulose moulées, qui se développent pourtant, mais davantage en collectivité pour l'instant. Là où l'attractivité visuelle est moins essentielle. Cellulopack, par exemple, vient de lancer en janvier 2016 ses premières barquettes scellables en cellulose moulée 100 % compostables (y compris le liner assurant la barrière aux gaz et à l'humidité). «Cette première gamme de barquettes alimentaires gastro norm baptisée RestoKompost se destine à la restauration collective, indique Olivier Mas, dirigeant et fondateur de Cellulopack en 2013. Nous partons d'une pâte à papier d'origine française que l'on sèche, presse puis moule sur nos machines. Les barquettes formées supportent la remise en température au four.» Les premières barquettes ont été référencées par des enseignes nationales de la restauration collective privée ainsi que par des cuisines centrales municipales. D'autres formats peuvent être réalisés, pour la grande distribution, par exemple. Il suffit de créer les moules. «Nous sommes en discussion avec certains industriels», confie Olivier Mas.   Une filière de cellulose locale Ecofeutre fabrique également depuis peu une barquette scellable en cellulose moulée destinée au marché alimentaire. Elle est issue de papiers recyclés et dispose d'un pelliplacage de film plastique. Pack Alim se lance aussi sur ce créneau avec de la pulpe de canne à sucre (bagasse), biodégradable et compostable. Ses Pulp'Pack aptes au contact alimentaire lancés le mois dernier peuvent être pelliplaqués (pour les propriétés barrières) et supportent un réchauffage au four traditionnel (200 °C) et au micro-ondes. Virgin BioPack est également en mesure de proposer des barquettes en cellulose moulée (dénommées Hybric Bio) issues de pâte à papier vierge venue du Limousin (comme Cellulopack). Et travaille actuellement sur la création d'une filière locale en Rhône-Alpes à partir de cellulose issue de fibre de Miscanthus, une plante vivace souvent présente dans les marais dont la croissance ne nécessite aucun intrant. Reste que ces développements doivent faire leur preuve en matière de coûts, priorité des industriels. «Sur ce point, nous arrivons à nous montrer compétitifs face aux barquettes en plastique injecté, par exemple, explique Olivier Mas chez Cellulopack. On reste un peu plus cher mais on arrive à le justifier par des avantages en matière de gestion des déchets.»   4 Le plastique se renforce avec du carton À l'inverse, le plastique a trouvé la parade face au carton. Il l'imite ou s'y associe. Dans cet esprit, Greiner Packaging a développé pour les producteurs de pommes de terre Parmentine, la Parmentine's Cup en polypropylène recouverte d'un carton lancée en janvier 2016. Elle permet de cuire les pommes de terre à la vapeur directement dans le pot et de les consommer de façon nomade grâce à la fourchette et à la sauce disposées dans le couvercle en APET. Le carton permet alors d'éviter de se brûler. PlastoBreizh imite aussi la céramique grâce à un effet de surface, tout comme Faerchplast qui a créé l'an passé des barquettes «façon terre cuite» pour la choucroute Stoeffler. Le plastique se permet même de copier le bois, comme Plastobreiz avec ses boîtes Bento à la japonaise, ou de remplacer le verre sur le marché de la conserve.   5 Le PET remplace le polystyrène Au rayon frais, les pots de yaourts eux cherchent petit à petit à se défaire de leur dépendance au polystyrène pour s'orienter vers de nouvelles pistes, moins coûteuses du point de vue de l'éco-contribution : le polypropylène chargé, le PLA, le PET, le PET expansé, etc. (Lire l'article détaillé sur le sujet «Les yaourts changent de pots» en juin 2015 p.76 à 81). Klöckner Pentaplast, par exemple, vient de lancer son clikPET, qui permet de gagner 10 jours de durée de vie et d'améliorer la résistance aux chocs. «Certains industriels souhaitent transformer leurs pots de polystyrène mais peu ont franchi le pas d'investir dans une nouvelle machine ou d'adapter l'existant pour partir vers l'inconnu. Car le PS est totalement maîtrisé depuis des années. Contrairement aux alternatives qui montrent toutes, pour l'instant, leur limite», explique un spécialiste du secteur.   La chasse au polystyrène s'opère aussi sur le marché de la viande et du poisson, où les barquettes en plastique font la course au mono-matériau (en APET ou en polypropylène). Le leader des barquettes en PSE, Sirap (qui a racheté son concurrent Vitembal en 2015) se prépare d'ailleurs à ce virage en développant des barquettes operculables et des boîtes en PET.
</article>
<article titre="ALBIOMA : activité en hausse de 12% au premier trimestre" date="2016 04 27" journal="Agence Option Finance (AOF)">
(AOF) - Albioma a réalisé au premier trimestre 2016 un chiffre d'affaires de 87,2 millions d'euros, en hausse de 12 % grâce à la forte progression de la production d'énergie. Cette dernière a grimpé de 70% à 887 GWh avec une disponibilité totale des installations de 91,4% contre 88,6% un an plus tôt. Le producteur d'énergie indépendant spécialisé dans la bagasse, coproduit fibreux de la canne à sucre, bénéficie d'une base de comparaison favorable. Un an plus tôt, le groupe essentiellement présent dans l'Outre-Mer et à l'Île Maurice, avait connu une grève sur son site de la Moule, en Guadeloupe. Albioma a confirmé ses objectifs d'Ebitda de 122 à 130 millions d'euros et de résultat net part du groupe de 25 à 30 millions d'euros pour l'exercice 2016.   Le PDG, Jacques Pétry, a déclaré : " Alors que l'exercice 2015 avait été marqué par des difficultés opérationnelles en France, je me réjouis de constater un retour à une très bonne performance de l'ensemble de nos installations en France, à l'Île Maurice et au Brésil. La forte efficacité énergétique et la disponibilité élevée de nos usines sont le socle du développement international d'Albioma. "   AOF - EN SAVOIR PLUS  Services aux collectivités    Dans les services à l'environnement, l'économie circulaire représente un enjeu de taille. Elle consiste à faire des déchets des uns les ressources des autres, dans un contexte où les matières premières seront de plus en plus rares et plus chères. L'économie circulaire permet de réduire les rejets de gaz à effet de serre des décharges. Avec un volume de déchets qui s'accroit, l'objectif est de capter une partie du méthane dégagé pour le commercialiser en gaz ou pour fournir de l'électricité. Cela consiste aussi à utiliser les boues d'une station d'épuration comme engrais ou les eaux usées partiellement traitées pour irriguer les champs. Les deux rivaux, Veolia Environnement et Suez, empruntent tous deux cette voie, impliquant de larges défis en termes de recherche et d'innovation.
</article>
<article titre="L’ENVIRONNEMENT EN FRANCE Les grandes tendances" date="2016 04 14" journal="Rapports - Environnement - MEDDTL">
27 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... Ministère de l’Environnement, de l’Énergie et de la Mer  www.developpement-durable.gouv.fr Sommaire  À l’approche de la conférence Paris- Climat et à l’issue de la...    Pour voir le rapport :  http://www.developpement-durable.gouv.fr/IMG/pdf/14028-1_REE2014_A5_Web_DEF_Planches.pdf
</article>
<article titre="Firplast. Sur la vague du packaging "bio"" date="2016 04 1" journal="Le Journal des entreprises">
A ttention, vaisselle à usage unique : coupelle en bois à base de peuplier, récipients en fibre de canne à sucre, couverts à base d'acide polyactique (amidon de maïs)... Chez Firplast (CA 2015 : 26 M€), les plaisirs de la table passent désormais par la case " biodégradable ". Alignés dans une partie du showroom de l'entreprise (qui dispose d'un catalogue de quelque 5.000 références), les produits " écolo " de cette maison fondée en 1983 par Guy Merran occupent une place de plus en plus importante. « Nous doublons nos ventes chaque année sur ces gammes », précise Thierry Merran, fils du fondateur et actuel directeur commercial de la PME. La raison de cette envolée du " bio " ? « Un cadre réglementaire qui privilégie les matières recyclées. A partir de cet été par exemple, les sacs plastiques jetables ne seront plus autorisés; ils devront être recyclables ». Idem pour la vaisselle jetable qui sera interdite en 2020. Créée il y a 6 ans, la gamme biodégradable de Firplast (baptisée " vision verte ") - même si toujours minoritaire dans le CA global - représente pour 2016 l'un des principaux axes de développement de l'entreprise. Plus de la moitié des 50 nouveaux modèles dessinés chaque année par les équipes dédiées entrent ainsi dans cette catégorie " biodégradable et compostable ". Et Thierry Merran de citer notamment les modèles à base de bagasse, résidu de canne à sucre, matière particulièrement appréciée une fois transformée; les produits pouvant être micro-ondables, étanches et résistants aux graisses. Barquettes innovantes   « Ces gammes " développement durable " deviennent incontournables sur un secteur de plus en plus concurrentiel », insiste-t-il. Et ce, sur tous les marchés que " sert " Firplast : restauration rapide (snacks, traiteurs, boulangeries) mais aussi les collectivités; ces cuisines géantes qui chaque jour fabriquent des milliers de repas envoyés sur des " satellites " (écoles, hôpitaux, etc.). « Pour les collectivités, nous venons de mettre au point une barquette spéciale biodégradable, qui peut être thermoscellée à base de carton et d'un couché de cellulose (et non à base de polypropylène comme habituellement proposé, Ndlr) ». Produit unique sur le marché, cette barquette " verte " - permettant de contenir un produit pendant 3 ou 4 jours sans l'altérer - pourrait ainsi prochainement inonder les cantines scolaires et professionnelles. « Nous sommes au début de la commercialisation mais les perspectives sont encourageantes », assure Thierry Merran. Dans la foulée, les équipes de Firplast et de Sagem Networks (l'agence de design adossée à l'entreprise) ont lancé une nouvelle gamme 100 % kraft en brun " brut ". Autre axe de développement pour cette PME qui dispose de 3 hectares en propre d'entrepôts : le packaging haut de gamme pour le traiteur et l'événementiel. Secteur qui connaît également une croissance de 100 % chaque année.   Innovation et personnalisation   « Pour sortir du lot, il faut à la fois diversifier nos collections, apporter de l'innovation et du design mais aussi du conseil et de la personnalisation à nos clients », insiste le directeur commercial de cette PME qui - pour des raisons de contraintes réglementaires, dixit - tient à ne pas dépasser le cap de 49 salariés. « Nous avons pour cela créé des plateformes à Bordeaux, Montpellier, et Marseille qui sont des sociétés et des centres de profits à part entière », précise-t-il. Côté production, que l'entreprise externalise auprès d'injecteurs et et thermoformeurs, Firplast opte pour une stratégie " éclatée " entre la Chine (10 % environ de sa production globale), la France, l'Espagne et le reste de l'Europe.   Firplast   (Saint Priest) Pd-g : Guy Merran CA 2015 : 26 M€ 49 salariés (une centaine à l'échelle du groupe) www.firplast.com   Normal
</article>
<article titre="Albioma (ex-Sechilienne-Sidec) : retournement de tendance ?" date="2016 03 9" journal="Boursier (site web)">
Sursaut.  Malmené ces derniers mois sur le marché parisien, Albioma connaît depuis la publication de ses comptes annuels une séquence de rebond appuyé. Ce sursaut, qui reste malgré tout modeste par rapport au terrain perdu précédemment, vient récompenser une gestion solide et des fondamentaux de qualité, à peine dégradés par le contexte difficile dans l'industrie énergétique. Le portefeuille d'actifs apparaît bien diversifié avec des implantations situées sur les départements et territoires d'outre-mer, la France métropolitaine, l'île Maurice ou encore le Brésil. Le producteur d'électricité axe par ailleurs sa stratégie sur des contrats de long terme et l'exploitation de sources renouvelables comme la biomasse, en particulier la bagasse de canne à sucre, confortant la visibilité sur les résultats. En 2015, Albioma a d'ailleurs franchi, pour la première fois de son histoire, le cap des 50% de production d'énergie en provenance de sources renouvelables, avec l'objectif d'atteindre 80% d'ici quelques années. La puissance totale des installations d'Albioma représentait 753 MegaWatts l'année dernière, à rapporter à une capitalisation boursière d'environ 400 ME et à un endettement net de 502 ME. Il convient de noter que le gearing, qui s'élevait à 116% au 31 décembre dernier, n'est pas un obstacle au développement du groupe grâce aux cash-flows sécurisés sur chacune des installations. Le financement des nouveaux projets, assez nombreux, ne pose aucun problème. Dynamique.  Alors qu'Albioma était attendu au tournant par les opérateurs sur son point annuel, les comptes 2015 n'ont guère déçus. Bien qu'en baisse de 7%, l'Ebitda est ressorti supérieur aux attentes à 119,9 ME. Idem pour le bénéfice net qui atteint 30,2 ME sur le millésime écoulé, contre 28,1 ME attendus par le consensus des analystes. On notera que l'érosion des résultats est uniquement liée à la baisse des matières premières et à celui des prix de l'électricité au Brésil. Concernant ce pays, au coeur des préoccupations des investisseurs depuis un an, il faut noter que la déflation tarifaire induit en contrepartie des conditions attractives pour les nouveaux projets dans le pipeline, ce qui est favorable pour les perspectives de long terme d'Albioma. Le groupe, qui se montre prudent sur l'évolution de ses indicateurs de performance en 2016, vient d'ailleurs de réaffirmer ses objectifs pour 2013-2023, à savoir un doublement de son bénéfice net sur la période. En se projetant à plus brève échéance, il y a fort à parier que les investisseurs vont commencer à miser sur l'accélération de la croissance à partir de 2017, année qui verra les mises en service successives de la centrale 100% biomasse de Galion 2 à la Martinique, de la turbine à combustion de Saint-Pierre à la Réunion et probablement de deux nouvelles usines au Brésil. De quoi garder un prisme positif sur ce dossier, dont l'attrait pour le moyen terme reste intact. Avis positif maintenu.Aymeric Val
</article>
<article titre="Albioma : un nouveau DG, mais une stratégie identique" date="2016 03 3" journal="Les Echos, no. 22142">
C'est un nouveau directeur général qui mettra en oeuvre la stratégie définie par Jacques Pétry à son arrivée à la tête d'Albioma début 2012. Dès le 1er juin prochain et pour un mandat de quatre ans, le PDG de l'ex-Sechilienne-Sidec va devenir président de la société. La direction générale en sera confiée à Frédéric Moyne : entré dans le groupe en 2000, ce diplômé de HEC de quarante ans est aujourd'hui directeur de la filiale brésilienne. «  A soixante ans, j'ai souhaité préparer ma succession en douceur  », explique Jacques Pétry. Albioma, qui s'est rebaptisé en mai 2013, a alors défini un plan d'investissement de 1 milliard d'euros sur 10 ans pour se réorienter vers la biomasse. Spécialiste de la production d'électricité à partir de bagasse (résidu de la canne à sucre), elle s'est fixé pour objectif de porter la part des renouvelables dans son mix électrique de 36 % en 2013 à 80 % en 2023. Et ce, en convertissant ses centrales bagasse/charbon, en investissant dans de nouvelles centrales 100 % renouvelables, ou encore grâce à l'acquisition d'usines au Brésil. «  Ce pays, premier producteur de canne à sucre au monde, valorise mal sa bagasse. Grâce à notre savoir-faire, nous pouvons améliorer le rendement des unités rachetées  », explique Frédéric Moyne. Ayant déjà racheté deux usines dans le pays, Albioma compte poursuivre sur son rythme, d'une acquisition tous les 12 à 18 mois.   « En ligne avec nos objectifs » Ni les résultats décevants enregistrés cette année, ni la baisse du cours de bourse (-15,6 % en trois ans), ne font dévier les dirigeants de cette stratégie. «  Nous sommes en ligne avec nos grands objectifs : nous avons pour la première fois dépassé 50 % d'électricité renouvelable en 2015  », insiste Jacques Pétry. Les résultats 2015 ont été pénalisés par une grève et des incidents techniques sur des centrales en Outre-mer, ainsi que par la chute du prix de l'électricité au Brésil : le résultat net part du groupe a baissé de 20 % à 30,2 millions d'euros, pour un chiffre d'affaires de 350 millions. La société compte notamment sur la mise en route de sa centrale Galion 2, à la Martinique, et sur la signature de contrats long terme avec des clients comme EDF, pour monter une marche en terme de rentabilité en 2018. Elle maintient son objectif d'un doublement de son résultat net à l'horizon 2023, par rapport aux 32 millions réalisés en 2012.   Anne Feitz
</article>
<article titre="La chimie du végétal Des produits chimiques issus de la biomasse" date="2016 03 1" journal="Industrie et Technologies, no. 0985">
L a chimie du végétal vise à obtenir des produits chimiques en partant de la biomasse végétale plutôt que du pétrole. Prometteuse dans la double optique d'une économie des ressources fossiles et de la réduction des émissions de gaz à effet de serre, elle est déjà très utilisée dans la détergence, la cosmétique ou encore la peinture. On estime que 8 % des matières premières de l'industrie chimique française sont d'origine renouvelable. Trois approches complémentaires sont possibles en chimie du végétal. Les produits biosourcés peuvent reproduire à l'identique des molécules d'origine pétrochimique. Ils peuvent être originaux, mais présenter des fonctionnalités ou des usages similaires à ces dernières. Il s'agit alors d'une innovation de substitution. Enfin, il est possible d'élaborer des molécules dotées de nouvelles fonctionnalités permettant de valoriser les caractéristiques propres des matières premières végétales. L'innovation est alors dite « de rupture ».   Pour autant, la chimie du végétal ne remplacera jamais toute la chimie du pétrole, plus compétitive. Cependant, elle peut constituer une alternative intéressante pour des marchés de niche dans lesquels les propriétés des molécules biosourcées et leur moindre toxicité constituent des atouts. Les investissements en recherche et développement dans le domaine de la chimie du végétal se sont accélérés depuis ces dix dernières années. Ils devraient encore s'accroître avec le soutien de grands programmes européens tels que BIC JU Biobased Industrie Consortium Joint Undertaking, qui consacre à l'horizon 2020 près d'un milliard d'euros au développement de la recherche et de l'innovation, et surtout à la montée en puissance des procédés jusqu'à la première usine commerciale. Peu à peu, une filière chimie du végétal se structure, accélérée par les évolutions des cadres réglementaires, la mise en place d'une fiscalité plus avantageuse (bonus vert), ou encore l'accès privilégié aux marchés publics pour les produits biosourcés.   1. TRAITEMENTS. À chaque végétal, son procédé Les ressources qui peuvent trouver des débouchés au sein de la filière chimie du végétal sont de natures très hétérogènes : ressources agricoles, ressources forestières, déchets, coproduits industriels, ressources aquatiques... Parmi les ressources agricoles, on peut citer les plantes amidonnières, (blé, maïs, pommes de terre) et sucrières (betteraves) pour la chimie des sucres basée sur le glucose, les plantes oléagineuses comme le colza, pour l'oléo-chimie, et les plantes protéagineuses comme le pois, pour les peptides actifs et acides aminés. Quelle que soit son origine, cette biomasse offre une large gamme de structures moléculaires. Il s'agit d'en valoriser la complexité et les propriétés intrinsèques afin de répondre aux attentes de chacun des marchés spécifiques et de substituer des produits pétrochimiques très techniques qui ont fait la démonstration de leur performance depuis des années.   La transformation de la biomasse fait appel à différentes technologies de prétraitement et de transformation dans lesquels les procédés physicochimiques et thermiques classiques (voie catalytique) se distinguent des procédés de biotechnologies industrielles (voie enzymatique). La catalyse et les procédés de purification doivent être adaptés à la spécificité des matières premières végétales  (Fig. 1) . Celles-ci se distinguent par la variabilité de la composition, la présence d'inhibiteurs potentiels et d'impuretés et par leurs nombreux atomes d'oxygène, contrairement aux hydrocarbures fossiles utilisés habituellement. Autant de facteurs qui peuvent limiter l'efficacité des catalyseurs chimiques et des fermentations. Une voie de recherche majeure est le développement de procédés hybrides couplant les procédés chimiques et biotechnologiques. Pour les applications matériaux, une autre approche innovante consiste à concevoir des procédés permettant de valoriser directement les associations complexes de molécules dans le végétal, plutôt que de séparer ses molécules pour ensuite les recombiner. Le végétal est en effet déjà un matériel « composite » en soit. L'intégration des procédés au sein des bioraffineries est un enjeu fort. La montée en puissance des procédés, le passage du laboratoire au stade industriel via des étapes de démonstrateur et de pilotes industriels sont nécessaires pour valider les paramètres de viabilité économiques et techniques des procédés. Cette mise en situation quasi réelle permet de tester les marchés et de préparer l'intégration dans des bioraffineries.   Quelle que soit son origine, la biomasse est composée de grandes familles de molécules : les glucides ou carbohydrates, les lipides, les protéines, la lignine, les hémicelluloses et autres dérivés métabolites secondaires, comme les terpénoïdes, phénols, tanins... La composition de la biomasse lignocellulosique varie beaucoup selon son origine et son humidité. On pourra cependant retenir des valeurs voisines de 75 % de carbohydrates Cn(H2O)m divisés en cellulose (C6H10O5) et en hemicellulose (sucres en C6) et de 25 % de lignine (C40H44O6).   2. VALORISATION. Transformer les briques de base Les industriels transforment et fonctionnalisent depuis des années les sucres en C6  (Fig. 2) . À partir de ceux-ci, ils développent des amidons modifiés, des amyloses, des amylopectines et des dérivés du glucose, tels que le sorbitol ou le mannitol, et enfin des celluloses de spécialités : éthers de cellulose, nitro-cellulose, acétates de cellulose, cellulose microcristalline, et les dérivés issus du raffinage de la pâte de bois de résineux. Bien qu'elle soit pratiquée depuis longtemps, la valorisation des glucides fait aussi l'objet de travaux de R&amp;D. Ils visent à mettre au point des intermédiaires chimiques à partir des glucides de base, comme l'isosorbide dérivé du sorbitol, ou encore le FDCA (furane dicarboxylique acide), une alternative aux phtaliques pour le développement de résines. D'autres travaux de R&amp;D se consacrent à la réalisation de biomolécules spécifiques. En particulier à la synthèse d'agro-tensioactifs spécifiques par le greffage sélectif d'acides gras et dérivés sur les fonctions hydroxyles de chaînes de sucres. Ils ont aussi pour but de mettre au point la fonctionnalisation de nouvelles « cyclodextrines », molécules cages pour le transport et la livraison ciblée de molécules actives, pour des applications cosmétiques ou pharmacologiques. Enfin, l'approche de la biocatalyse en synthèse est à l'étude.   Les hémicelluloses sont les deuxièmes constituants en poids de la biomasse. Les pentoses (C5) peuvent représenter jusqu'à 35 % de la biomasse mais en constituent plus généralement 15 à 25 %. Le son de blé, la bagasse, les pulpes de betteraves issues d'amidonneries, de sucreries, et d'éthanoleries et aussi certains produits forestiers, sont des coproduits riches en hémicelluloses. Le fractionnement, la purification et la valorisation des hémicelluloses et des sucres en C5 constitue un axe majeur dans le développement de la bioraffinerie. Les débouchés des sucres en C5 sont divers : furfural et dérivés comme le FDCA, xylitol. Des biomolécules à base de sucres en C5, surfactant doux (alkylpolypentosides) sont déjà commercialisés. Des travaux portent sur la synthèse chimio-enzymatique de molécules à bases de pentoses. Les verrous associés sont les rendements d'extraction et les puretés des fractions de C5, ou encore le développement de catalyseurs adaptés.   La lignine constitue la principale source potentielle d'aromatiques pour la chimie. Néanmoins, elle présente une structure complexe et variable, selon la source de la biomasse et le procédé d'extraction, ce qui la rend difficile à valoriser. Sur les 70 millions de tonnes de lignines produites par an et dans le monde, essentiellement issues de procédés papetiers, 95 % sont dédiées à une valorisation énergétique. Le reste est utilisé dans des secteurs à basse valeur ajoutée : additif dans le ciment, dans l'asphalte, ou liant dans les aliments pour animaux. Les industriels prennent peu à peu conscience de leur potentiel pour des applications de plus haute valeur ajoutée  (Fig. 3) .   Transformer la structure complexe et variable des lignines en dérivés aromatiques est un challenge à long terme. Celui-ci empruntera deux voies. La première s'attaque à la dépolymérisation agressive et non sélective de la lignine conduisant au benzène, toluène et xylène et leurs dérivés. La seconde passe par une dépolymérisation sélective. L'enjeu est d'obtenir des lignines à façon... Plusieurs dérivés phénoliques peuvent résulter de la dégradation de la lignine. Les plus importants sont la vanilline et les acides vanilliques et syringiques. Les deux derniers peuvent servir à la fabrication de polyesters, en particulier l'acide vanillique qui peut conduire à des polyesters totalement aromatiques souples et solubles. Les recherches portent sur la modification des voies de biosynthèse des lignines par biotechnologie, l'ajustement des paramètres technologiques de délignification et la transformation chimique ou biotechnologique des lignines. Elles devront s'appuyer sur une meilleure connaissance de base des lignines et prendre en compte leur diversité biologique.   Les huiles végétales sont issues de la trituration des graines des plantes oléagineuses (colza ou tournesol) mais aussi de plantes importées (ricin, palme, palmiste, soja, coco). Elles peuvent être extraites et raffinées pour des usages en chimie. La fonctionnalisation chimique ou enzymatique des acides gras issus des huiles peut conduire à de multiples dérivés tels que les esters, amides, époxy, carbonate, hydroxyl, standolies, thiolène et savons. Ces dérivés trouvent déjà des débouchés dans des marchés aussi variés que les biocarburants dits « EMHV » (Esters méthyliques d'huiles végétales, ou « diesters »), la formulation destinée aux solvants, biolubrifants, agro-tensioactifs, ou encore bioplastifiants et les résines biosourcées comme les polyamides à base d'huile de ricin ou les résines alkyde pour des applications dans la peinture.    La substitution des huiles de palme utilisées en agroalimentaire mais aussi en chimie est également un défi. Elle suppose l'obtention de coupes particulières d'acide gras à chaînes courtes. Les voies levuriennes ou algales ouvrent la porte aux acides gras longues chaînes et aux acides gras particuliers aux nombreuses perspectives. Les travaux en R&amp;D portent aujourd'hui notamment sur la synthèse de monomères pour le développement de matériaux lipidiques originaux. Les voies de métathèse et de synthèse chimio-enzymatiques des acides gras conduisent à de nouveaux synthons, qui peuvent servir de base à la fabrication de nouveaux matériaux comme les élastomères.   Les voies de valorisation du glycérol sont aussi importantes, en acide acrylique, épichlorydrine, polyglycerol. Certaines ont déjà atteint le stade de la commercialisation. Le carbonate de glycérol constitue plus spécifiquement une molécule plateforme d'intérêt.   3. SUBSTITUTION. Remplacer les molécules existantes Le règlement Reach, entré en vigueur en 2007 au sein de l'Union européenne, a conduit au cours des dernières années à l'identification, parmi un premier portefeuille de plus de 30 000 substances chimiques produites ou importées à plus d'une tonne, d'un certain nombre de molécules jugées très préoccupantes (SVHC : substance of very high concern), à l'instar des CMR (cancérigène, mutagène, reprotoxique), PBT (persistant, bioaccumulable, toxique), VPVB (très persistant, très bioaccumulable), ou ayant un niveau de préoccupation équivalent (les perturbateurs endocriniens, par exemple). Le classement de substances dans une liste candidate à l'interdiction a déclenché des travaux de recherche sur leur substitution. L'alternative végétale aux substances s'est avérée une option évidente pour remplacer les phtalates, parabènes, bisphenol A, isocyanate, silicones... Il s'avère cependant difficile de substituer des molécules dont les propriétés fonctionnelles très techniques ont été acquises au cours de décennies de recherche et développement.   Les phtalates sont les premières catégories de substances jugées préoccupantes qui ont été identifiées et surtout révélées dans la presse. Ils sont utilisés comme plastifiants dans les matériaux PVC pour leur donner leur souplesse, leur extensibilité et leur élasticité. Des solutions de substitution pétrochimiques ou végétales (esters d'isosorbide ou d'acides succiniques, et autres esters d'acides gras issus de colza, de ricin...) ont vu ou revu le jour mais se sont vite confrontées à de nombreuses difficultés : au départ souvent trop chères, ces alternatives devaient être introduites à des quantités plus élevées que les phtalates pour obtenir des flexibilités identiques. Ils étaient beaucoup moins généralistes que le phtalate dit « DEHP », d'influence parfois néfaste et d'une toxicité encore non évaluée... Une première génération de plastifiants végétaux s'est toutefois aujourd'hui imposée, notamment sur des marchés sensibles, comme les revêtements de sols en milieu hospitalier. D'autres substances bénéficient de recherches pour les remplacer, à l'instar du bisphénol A, finalement jugé non dangereux par l'EFSA en 2015, des parabènes, des biocides naturels ou encore des retardateurs de flamme.   Au-delà de l'impact de Reach sur le secteur industriel, le durcissement général de la réglementation sur l'environnement, l'étiquetage ou les installations industrielles favorise le développement de la filière végétale. C'est le cas dans le domaine des solvants, qui sont utilisés dans de très nombreuses applications industrielles - peintures, détergents, revêtements, produits phytosanitaires, pharmacie, imprimerie, etc. - et représentent 20 % des COV émis dans l'atmosphère. Le développement d'agro-solvants passe par le développement de nouveaux synthons et des formulations plus complexes, associant plusieurs agrosolvants pour répondre aux besoins sectoriels. Le projet ANR Inbiosynsolv sur « La formulation inverse de solvants issus de ressources renouvelables », piloté par le laboratoire Génie chimique de Toulouse, a axé la recherche des molécules avant leur synthèse  (Fig. 4) . Les propriétés désirées sont fixées dès le départ. Partant d'un building block, le glycérol, de sept co-réactants et d'une soixantaine de réactions chimiques potentielles de base, le logiciel construit systématiquement des molécules candidates, et retient celles qui satisfont les propriétés cibles. L'utilisation des huiles végétales comme lubrifiants de hautes performances est aussi un exemple caractéristique de la volonté de s'inscrire dans une industrie plus respectueuse de l'environnement. Ces lubrifiants trouvent toute leur utilité dans les applications où la récupération des lubrifiants est quasi-impossible ou bien sujette à des pertes accidentelles dans le sol ou les eaux. Il s'agit, par exemple, des huiles perdues de décoffrage ou de tronçonneuse, des huiles pour moteur deux temps, des fluides hydrauliques ou des huiles pour le travail des métaux... Ils sont facilement biodégradables par les micro-organismes, ne sont nocifs ni pour l'homme ni pour les organismes environnants et ne sont pas inflammables. En outre, ils permettent de limiter à 50 % les émissions de gaz à effet de serre (équivalent CO2) et de réduire la consommation d'énergie de 65 %.   L'appauvrissement des ressources pétrolières conduit aussi les acteurs à se tourner vers la chimie du végétal, la seule source alternative disponible. À chaque choc pétrolier, les travaux de recherche et développement sur la chimie du végétal se sont intensifiés. Actuellement, le changement d'alimentation des crackers pétrochimiques en Amérique de Nord en gaz de schiste, préféré aux naphta, provoque des tensions sur les prix des molécules à chaînes courtes. Le prix du butadiène (C4), l'un des intermédiaires chimiques indispensables à la production de caoutchouc synthétique (55 % du marché du caoutchouc) a ainsi plus que triplé en moins de dix ans, alors que la demande en caoutchouc ne cesse de croître. En 2013, Michelin en collaboration avec l'IFP énergies nouvelles (Ifpen) Axens a lancé le projet AMI Chimie du végétal Biobutterfly qui vise à développer et commercialiser une technologie de production de butadiène à partir d'intermédiaires alcools biosourcés, pour remplacer le procédé de Lebedev utilisé depuis la seconde guerre mondiale avec des alcools pétrosourcés.   4. INNOVATION. Valoriser les caractéristiques originales du végétal La chimie du végétal peut enfin conduire à des molécules avec de nouvelles fonctionnalités ou des matériaux avec de nouvelles performances. C'est le cas par exemple des tubes de polyamides 11  (Fig. 5)  pour le transport du gaz, qui ont reporté le prix pierre Potier 2013. La société Arkema a réussi à réaliser des tubes en polyamide 11 issu de l'huile de ricin supprimant tout risque de corrosion, pour remplacer les tubes en polyéthylène. Les qualités de ces tubes sont telles que le gaz peut être transporté à une pression deux fois plus élevée qu'avec les autres matériaux, les fuites de méthane étant fortement diminuées en raison de la faible perméabilité du polyamide 11.    Les différents intérêts des industriels pour la chimie verte
</article>
<article titre="La chimie du végétal POUR ALLER PLUS LOIN" date="2016 03 1" journal="Industrie et Technologies, no. 0985">
Ressources. La vitrine des bioproduits   L'agrobiobase est une base de données   internationale sur les produits biosourcés hébergée et tenue à jour par le pôle de compétitivité IAR (Industrie et Agroressources). Elle sert à la mise en contact entre fournisseurs et industriels spécialisés dans le cadre d'une recherche de solutions techniques innovantes. Le site Internet réunit plus de 200 produits d'origine végétale, et génère près de 30 000 consultations annuelles. Les fiches techniques incluent une description du produit, ses avantages environnementaux, son origine végétale ainsi que des données techniques et les coordonnées du fournisseur. Les dossiers thématiques apportent des compléments d'information sur les bioproduits, les marchés applicatifs et les familles de produits : données de marché, gains d'utilisation, applications, caractéristiques techniques ...  Rendez-vous sur www.agrobiobase.com. Bibliographie. La chimie verte   L'ouvrage de Paul Colonna « La chimie verte » est une référence en la matière.  Loin de se limiter aux biocarburants, cet ouvrage passe en revue les applications de la chimie verte dans les multiples filières industrielles concernées. Paul Colonna, expert reconnu de la chimie verte, est délégué scientifique développement durable à l'Inra et responsable de l'institut Carnot 3BCAR. Il est entré au Collège de France en 2012.    Rencontres. Une conférence sur la filière biomasse   C'est la plus grande rencontre  de scientifiques, industriels, responsables politiques ou étudiants cherchant des alternatives durables par la chimie et l'ingénierie verte. Lors de la Green chemistry and engineering conference, plus de 25 conférences techniques se tiennent pendant plus de trois jours, réalisées par des experts dans leur domaine. Une large place y est accordée aux innovations émergentes et aux stratégies correspondantes. L'édition 2016, la vingtième du genre, se tiendra du 14 au 16 juin à Portland, dans l'Oregon.
</article>
<article titre="Université de Montréal - L'économie circulaire, l'économie qui n'en finit pas" date="2016 02 27" journal="Le Devoir">
Pour encourager et soutenir concrètement le développement durable et l'économie circulaire, l'Université de Montréal a créé, avec HEC et Polytechnique Montréal, l'Institut de l'environnement, du développement durable et de l'économie circulaire (EDDEC). C'était il y a deux ans. Les résultats commencent à se faire sentir.   Michelin loue ses pneus à de grandes entreprises de transport en Europe en leur offrant un service d'entretien serré afin d'augmenter la durée de vie des pneus, de diminuer la consommation d'essence, puis de les reprendre par la suite pour les remettre à neuf. On utilise un rebut comme matière première d'un nouveau produit. On est en plein dans l'économie circulaire, un concept dont on parle encore assez peu au Québec. Pour mieux faire connaître l'économie circulaire aux Québécois, l'EDDEC sortira d'ailleurs un livre cet automne sur le sujet. " Nous souhaitons que monsieur et madame Tout-le-Monde comprennent mieux le concept de l'économie circulaire ", explique Stéphane Béranger, coordonnateur au développement durable à l'Université de Montréal.   On est plutôt maintenant dans l'économie linéaire : on prend une matière première, du coton par exemple, on fabrique de la fibre, puis un jeans et on le porte jusqu'à ce qu'on le jette.   " Avec l'économie circulaire, le jeans en fin de vie devient autre chose, puis autre chose : on le réutilise indéfiniment ", ajoute M. Béranger.   L'économie circulaire peut aussi passer par le développement de l'industrie de la réparation pour diminuer grandement la mise au rebut.   " Actuellement, disons que notre machine à laver casse, on n'a plus l'habitude de regarder si on peut la réparer parce que ça coûte souvent presque aussi cher que d'en acheter une neuve ", constate Stéphane Béranger.   "Speed dating" de chercheurs   Le développement durable et l'économie circulaire sont des concepts très larges, très inclusifs, et faire travailler ensemble les chercheurs de différentes disciplines, de trois établissements universitaires différents, s'avérait un défi.   " Les chercheurs ne sont pas forcément au courant de ce que font ceux d'autres disciplines ", remarque Stéphane Béranger.   Il fallait créer des maillages entre différents laboratoires et amener les chercheurs à trouver des thématiques transversales pour travailler sur des projets communs.   " Nous avons organisé des soirées de speed dating, où les chercheurs avaient trois minutes pour présenter leur travail ", explique Stéphane Béranger.   Des projets réunissant des étudiants, des chercheurs et le milieu ont pris forme à l'institut EDDEC et ont commencé à donner des résultats.   Par exemple, l'Université de Montréal souhaite créer un corridor écologique entre le campus de la montagne et le site Outremont. La rue Darlington a été choisie pour réaliser différentes initiatives vertes. Originellement, elle suivait une rivière.   " Les étudiants aimaient l'idée du cours d'eau, de la rivière perdue ", raconte Stéphane Béranger.   Le boisé du campus de la montagne abrite une riche biodiversité avec de nombreux insectes et oiseaux. L'UdeM souhaite étendre cette biodiversité dans le quartier.   " Une façon de permettre aux espèces d'aller d'un point à l'autre, c'est de créer un corridor avec de gros pots dans lesquels on plante des fleurs et on réalise de l'agriculture urbaine, explique Stéphane Béranger. On a parlé du projet aux résidents et aux commerçants et ils ont instantanément voulu mettre la main à la pâte ! Ils sont venus planter des légumes et se sont proposés pour l'entretien et l'arrosage. "   La Ville de Montréal a donné des pots pour la réalisation du projet.   " Un maillage s'est créé, se réjouit M. Béranger. C'est vraiment devenu un projet de cocréation. C'est parti de l'université, puis ça a été porté par la communauté. L'université n'est pas seulement en haut de la montagne, derrière ses murs, elle va vers les gens. "   En plus de la question de la biodiversité, de nombreux enjeux seront travaillés dans la rue Darlington : gestion de l'eau pluviale pour favoriser la marche et le vélo, atténuation des îlots de chaleur, communication efficace avec la communauté universitaire et le milieu, etc.   Les professeurs de différentes disciplines au baccalauréat et à la maîtrise se sont même fait suggérer de donner des travaux dirigés à leurs étudiants sur une de ces thématiques.   " Différentes équipes ont été formées et nous aurons les rapports bientôt, explique M. Béranger. On essaye de profiter de la transversalité. On veut apporter un peu de rêve, aussi. "   Prêcher par l'exemple   L'Université de Montréal prône une économie circulaire, le développement durable et une biodiversité riche, mais jusqu'à tout récemment, on aurait pu lui reprocher de ne pas prêcher par l'exemple. En ce qui a trait notamment à son offre alimentaire sur le campus.   Par exemple, il s'y distribue chaque année 460 000 gobelets à café. Ils étaient en carton avec une couche de plastique, ce qui fait qu'ils n'étaient pas compostables et difficilement recyclables. C'était la même chose pour les assiettes et les différents contenants utilisés dans les cafétérias. On retrouvait aussi de la styromousse.   " On a trouvé des assiettes et des contenants compostables en bagasse, un résidu de la transformation de canne à sucre, indique Stéphane Béranger. Ces assiettes sont en plein dans la logique de l'économie circulaire. On utilise les déchets d'une industrie comme matière première pour créer un nouveau produit qui retourne ensuite à la terre en étant composté. "   La beauté de la chose ? Ces nouvelles assiettes et nouveaux contenants écologiques sont même moins chers que les anciens.   On a aussi commencé à servir des poissons issus de la pêche durable dans les cafétérias.   Stéphane Béranger réalise que les mentalités évoluent énormément. Maintenant, des gens dans l'université vont vers lui pour tenter de trouver des solutions plus vertes. Par exemple, pour l'offre alimentaire, c'est le nouveau cuisinier en poste qui est venu frapper à sa porte.   Après avoir multiplié les efforts dans le domaine des services étudiants, Stéphane Béranger compte s'attaquer maintenant aux immeubles, aux services informatiques, aux approvisionnements et aux finances.
</article>
<article titre="L'économie circulaire, l'économie qui n'en finit pas" date="2016 02 27" journal="Le Devoir (site web)">
Pour encourager et soutenir concrètement le développement durable et l'économie circulaire, l'Université de Montréal a créé, avec HEC et Polytechnique Montréal, l'Institut de l'environnement, du développement durable et de l'économie circulaire (EDDEC). C'était il y a deux ans. Les résultats commencent à se faire sentir.   Michelin loue ses pneus à de grandes entreprises de transport en Europe en leur offrant un service d'entretien serré afin d'augmenter la durée de vie des pneus, de diminuer la consommation d'essence, puis de les reprendre par la suite pour les remettre à neuf. On utilise un rebut comme matière première d'un nouveau produit. On est en plein dans l'économie circulaire, un concept dont on parle encore assez peu au Québec. Pour mieux faire connaître l'économie circulaire aux Québécois, l'EDDEC sortira d'ailleurs un livre cet automne sur le sujet. « Nous souhaitons que monsieur et madame Tout-le-Monde comprennent mieux le concept de l'économie circulaire » , explique Stéphane Béranger, coordonnateur au développement durable à l'Université de Montréal.   On est plutôt maintenant dans l'économie linéaire : on prend une matière première, du coton par exemple, on fabrique de la fibre, puis un jeans et on le porte jusqu'à ce qu'on le jette.   « Avec l'économie circulaire, le jeans en fin de vie devient autre chose, puis autre chose : on le réutilise indéfiniment » , ajoute M. Béranger.   L'économie circulaire peut aussi passer par le développement de l'industrie de la réparation pour diminuer grandement la mise au rebut.   « Actuellement, disons que notre machine à laver casse, on n'a plus l'habitude de regarder si on peut la réparer parce que ça coûte souvent presque aussi cher que d'en acheter une neuve » , constate Stéphane Béranger.   «Speed dating» de chercheurs   Le développement durable et l'économie circulaire sont des concepts très larges, très inclusifs, et faire travailler ensemble les chercheurs de différentes disciplines, de trois établissements universitaires différents, s'avérait un défi.   « Les chercheurs ne sont pas forcément au courant de ce que font ceux d'autres disciplines » , remarque Stéphane Béranger.   Il fallait créer des maillages entre différents laboratoires et amener les chercheurs à trouver des thématiques transversales pour travailler sur des projets communs.   « Nous avons organisé des soirées de speed dating, où les chercheurs avaient trois minutes pour présenter leur travail » , explique Stéphane Béranger.   Des projets réunissant des étudiants, des chercheurs et le milieu ont pris forme à l'institut EDDEC et ont commencé à donner des résultats.   Par exemple, l'Université de Montréal souhaite créer un corridor écologique entre le campus de la montagne et le site Outremont. La rue Darlington a été choisie pour réaliser différentes initiatives vertes. Originellement, elle suivait une rivière.   « Les étudiants aimaient l'idée du cours d'eau, de la rivière perdue » , raconte Stéphane Béranger.   Le boisé du campus de la montagne abrite une riche biodiversité avec de nombreux insectes et oiseaux. L'UdeM souhaite étendre cette biodiversité dans le quartier.   « Une façon de permettre aux espèces d'aller d'un point à l'autre, c'est de créer un corridor avec de gros pots dans lesquels on plante des fleurs et on réalise de l'agriculture urbaine, explique Stéphane Béranger. On a parlé du projet aux résidents et aux commerçants et ils ont instantanément voulu mettre la main à la pâte ! Ils sont venus planter des légumes et se sont proposés pour l'entretien et l'arrosage. »   La Ville de Montréal a donné des pots pour la réalisation du projet.   « Un maillage s'est créé, se réjouit M. Béranger. C'est vraiment devenu un projet de cocréation. C'est parti de l'université, puis ça a été porté par la communauté. L'université n'est pas seulement en haut de la montagne, derrière ses murs, elle va vers les gens. »   En plus de la question de la biodiversité, de nombreux enjeux seront travaillés dans la rue Darlington : gestion de l'eau pluviale pour favoriser la marche et le vélo, atténuation des îlots de chaleur, communication efficace avec la communauté universitaire et le milieu, etc.   Les professeurs de différentes disciplines au baccalauréat et à la maîtrise se sont même fait suggérer de donner des travaux dirigés à leurs étudiants sur une de ces thématiques.   « Différentes équipes ont été formées et nous aurons les rapports bientôt, explique M. Béranger. On essaye de profiter de la transversalité. On veut apporter un peu de rêve, aussi. »   Prêcher par l'exemple   L'Université de Montréal prône une économie circulaire, le développement durable et une biodiversité riche, mais jusqu'à tout récemment, on aurait pu lui reprocher de ne pas prêcher par l'exemple. En ce qui a trait notamment à son offre alimentaire sur le campus.   Par exemple, il s'y distribue chaque année 460 000 gobelets à café. Ils étaient en carton avec une couche de plastique, ce qui fait qu'ils n'étaient pas compostables et difficilement recyclables. C'était la même chose pour les assiettes et les différents contenants utilisés dans les cafétérias. On retrouvait aussi de la styromousse.   « On a trouvé des assiettes et des contenants compostables en bagasse, un résidu de la transformation de canne à sucre, indique Stéphane Béranger. Ces assiettes sont en plein dans la logique de l'économie circulaire. On utilise les déchets d'une industrie comme matière première pour créer un nouveau produit qui retourne ensuite à la terre en étant composté. »   La beauté de la chose ? Ces nouvelles assiettes et nouveaux contenants écologiques sont même moins chers que les anciens.   On a aussi commencé à servir des poissons issus de la pêche durable dans les cafétérias.   Stéphane Béranger réalise que les mentalités évoluent énormément. Maintenant, des gens dans l'université vont vers lui pour tenter de trouver des solutions plus vertes. Par exemple, pour l'offre alimentaire, c'est le nouveau cuisinier en poste qui est venu frapper à sa porte.   Après avoir multiplié les efforts dans le domaine des services étudiants, Stéphane Béranger compte s'attaquer maintenant aux immeubles, aux services informatiques, aux approvisionnements et aux finances.  Cet article est paru dans Le Devoir (site web)
</article>
<article titre="Conférences pédagogiques sur le climat en régions" date="2016 02 19" journal="Rapports - Agroalimentaire - MAAF">
8 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... COP 21  février 2016  A l’occasion de la COP 21, Stéphane Le Foll, ministre de l’agriculture, de l’agroalimentaire et de la forêt a mandaté le CGAAER pour animer...    Pour voir le rapport :  http://agriculture.gouv.fr/telecharger/78050?token=534dc7ecba62b40ee0d03f3fc3c56b23
</article>
<article titre="Vrac: le hub ne se justifie pas" date="2016 02 5" journal="Le Journal de la marine marchande, no. 5017">
Après l’Europe du Nord, l’Asie et la mer Noire, l’océan Indien occupe la quatrième place des marchés du vrac sec, a expliqué Gilles Cossy, directeur de l’affrètement de Setaf-Saget. Quelque 175 Mt de marchandises en vrac transitent dans la région tous les ans. Ce sont principalement des trafics d’agrégats, de calcaire, de charbon, minerai de fer, céréales et engrais qui sont réalisés dans la région. Même si les installations portuaires sont en développement, la taille maximale des navires pouvant entrer dans les ports se limite aux Supramax. «  Nous avons anticipé ce mouvement en investissant dans ce type de navire. Nous avons commandé des unités avec une faible consommation en fioul et des émissions de So x  et No x  réduites  », continue le directeur d’affrètement de Setaf-Saget. Des navires spécifiques qui disposent de panneaux de cale larges pour une cadence pouvant atteindre jusqu’à 20 000 t par jour en 24h/24. Coopérative ouvrière Pour sa part, la COR (Coopérative ouvrière réunionnaise), manutentionnaire de vracs et de diverses à Port Réunion, s’est spécialisée dans ces trafics de vrac. «  Nous avons organisé l’entreprise en coopérative ouvrière pour que les dockers prennent leur destin en main  », a expliqué Michel Séraphine, un des fondateurs de la COR. Aujourd’hui spécialisée sur les trafics de vrac, la COR s’est diversifiée vers les marchandises conventionnelles. «  Notre mode opératoire diffère de ceux des autres sociétés de manutention de la place. Nous avons une gestion collégiale dans une économie sociale et solidaire. Les bénéfices sont réinvestis.  » Une gestion particulière qui a eu pour effet de solidariser l’ensemble des équipes des ouvriers dockers «  et d’éviter les mouvements sociaux  », précise Michel Séraphine. Au global, la COR traite environ 1,5 Mt de marchandises par an, surtout le clinker, le charbon, les céréales, le bois et les colis lourds. «  Nous n’intervenons pas sur les conteneurs à l’exception de ceux destinés à l’armée française basée à La Réunion.  »   Le démarrage des travaux de la Nouvelle route du Littoral, entre Saint-Denis et Saint Paul, nécessite un approvisionnement important de matériaux. La majorité de ces trafics, et notamment les enrochements en provenance de Madagascar, sont traités par la COR.   Un marché destiné à croître En ayant pris place sur ces trafics à La Réunion, la COR a posé le pied sur un marché qui devrait croître dans les prochaines années. Pour Gilles Cossy, de Setaf-Saget, la croissance démographique sera source de nouveaux potentiels. Les céréales pour l’alimentation humaine et animale vont augmenter. Avec le retour de la croissance, les matériaux de construction devraient aussi connaître dans les prochaines années un nouvel essor. C’est sur le charbon et les minerais de fer que des baisses sont attendues. «  Avec les décisions prises lors de la COP 21, les trafics de charbon destinés aux centrales électriques sont voués à s’amoindrir  », envisage Gilles Cossy. Néanmoins, si le charbon n’est plus la source de chaleur de ces centrales, il pourrait bien être remplacé par la biomasse. À La Réunion, la bagasse pourrait être utilisée comme combustible des centrales. Un bienfait pour l’écologie, un trafic en moins pour la COR. Une position non encore adoptée.   Face au développement du hub sur la partie conteneurs, tant Gilles Cossy que Michel Séraphine n’imaginent pas décliner ce concept sur les trafics de vracs. «  L’éclatement des trafics de vrac existe relativement peu dans le monde,  explique Gilles Cossy.  Les manutentionnaires de Dunkerque ont déployé ce système vers l’Allemagne, mais dans la région, faire un hub de vracs secs ne serait pas envisageable dans un avenir proche.  » Et Michel Séraphine de confirmer cette position avec néanmoins une nuance. «  Nous pourrions imaginer faire de nos silos un hub pour redistribuer dans la région des céréales. Ce n’est pas nous qui le commandons. Nous le proposons et nous sommes ouverts à toute activité nouvelle, mais cela doit venir des opérateurs.  »
</article>
<article titre="Grâce au stockage, le réseau réunionnais va tolérer 32% d'électricité solaire en 2016" date="2016 02 3" journal="GreenUnivers (site web)">
2016-02-03T14:41:36+00:00    En tournée à La Réunion les 1er et 2 février, Jean-Bernard Lévy, PDG d'EDF, en a profité pour annoncer que l'île allait franchir pour la première fois en 2016 le seuil de 30% d'injection instantanée d'électricité intermittente, issue de la production photovoltaïque pour l'essentiel. C'est aussi une première sur le territoire français : les gestionnaires de réseau - Systèmes électriques insulaires (SEI) à La Réunion - limitent toujours à 30% la quantité d'électricité intermittente acceptée, en application de l'arrêté du 23 avril 2008.   Halte au gâchis  Cette exception réunionnaise doit tout à la présence depuis 2010 d'une batterie de stockage de 1 MW (fabricant NGK, sodium soufre) installée à Saint-André et pleinement opérationnelle aujourd'hui. Elle est associée à un système prédictif de la météo baptisé Pegase; il sécurise l'utilisation du réseau en anticipant de quelques dizaines de minutes sur la production photovoltaïque. Le coût de la batterie serait de 3,5 M euros, avec une subvention de l'Ademe de 1,2 M euros selon le site EcCO2. Ce nouveau seuil de 32% est tout sauf neutre pour la production d'énergies renouvelables (EnR) sur l'île. « A chaque fois que le seuil de 30% risquait d'être dépassé, SEI devait débrancher les centrales ; 2015 a ainsi compté 67 jours de déconnexion, indique un porte-parole d'EDF. La batterie et Pegase permettent de diviser par trois le nombre de jours de déconnexion et par cinq l'énergie perdue ».   Stockage par hydrogène dans le Parc national  Le plafond devrait être porté à 35% dans les années qui viennent, sans doute autour de 2018, pour répondre aux objectifs gouvernementaux de développement des EnR ultramarines. Pour y contribuer, EDF prévoit d'installer 5 MW de stockage sur batterie supplémentaires. Les choix technologiques devraient être effectués dans les prochains mois.   Les nouveautés du stockage ne s'arrêtent pas là à La Réunion. L'opérateur public d'électricité installe au coeur de l'île, dans le Cirque de Mafate, là où le réseau électrique et même routier est inexistant, une station autonome en énergie conçue et fabriquée par la jeune pousse Powidian. Destinée à alimenter 300 familles en électricité 24/7, cette station stocke le courant photovoltaïque excédentaire en fabriquant de l'hydrogène. Le dispositif remplace les actuels groupes électrogènes au diesel. Ce premier micro-réseau 100% solaire allié au stockage est expérimental mais « EDF a l'intention d'en fournir plusieurs autres à La Réunion » annonce le porte-parole.   50% d'EnR dans quatre ans  EDF n'est pas seul à développer le stockage sur l'Île de l'océan indien. Le développeur-producteur Akuo a passé un contrat avec le fabricant de batteries Saft pour équiper une centrale solaire de 9 MW d'une capacité de stockage de 9 MWh, dans le cadre du projet Bardzour.   L'électricité réunionnaise est composée à 36% d'énergies renouvelables, contre 33% en 2014, dont 17,1% pour l'hydroélectricité, 9,3% pour la biomasse à base de bagasse et 9,1% pour les énergies intermittentes. Le parc photovoltaïque y a augmenté de 44% entre 2011 et 2015 et représente 8,5% du mix électrique.   La loi sur la transition énergétique et le projet de programmation pluriannuelle de l'énergie (PPE Réunion) prévoient de porter la part des énergies renouvelables à 50% d'ici à 2020, puis à 100% en 2030 sur l'île.   Un appel d'offres a été lancé par le ministère de l'Ecologie (MEDDE) en mai 2015 et clos en novembre dernier pour des installations solaires entre 100 kW et 50 MW avec stockage dans les DOM et en Corse.   Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="Chiffres clés de l'énergie - édition 2015" date="2016 01 26" journal="Rapports - Environnement - MEDDTL">
60 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... février 2016 Chiffres clés de l’énergie 1www.statistiques.developpement-durable.gouv.fr  COMMISSARIAT GÉNÉRAL AU DÉVELOPPEMENT DURABLE  Février 2016  Chiffres clés de l’énergie Édition 2015  Service de l’observation...    Pour voir le rapport :  http://www.developpement-durable.gouv.fr/IMG/pdf/reperes-chiffres-cles-energie-2015.pdf
</article>
<article titre="Situation des outre-mer confrontés aux déréglements climatiques" date="2016 01 18" journal="La Gazette des communes, no. 2300">
Sommaire Avant-propos 04   Chapitre I La gestion de la ressource en eau et l'assainissement 05   I. La fragilité des eaux ultramarines stimule des modes de production innovants 05  A. Connaître les ressources en eau et comprendre leur évolution sous l'effet du changement climatique 05    B. Accroître la production sans compromettre la ressource 06   C. Adapter les politiques territoriales de l'eau 07   II. Réalisation et projets08    A. La création d'un réseau internationel pour la gestion de l'eau sur le plateau des Guyanes (Guyane) 08   B. Le réservoir d'Ouaboudé (Nouvelle-Calédonie) 08   C. L'assainissement des eaux usées par filtration sur roseaux (Martinique) 08   D. La station d'eau potable de Matiti (Guyane) 09   E. Osmosun : le couplage de l'énergie solaire du dessalement (Polynésie Française) 09   F. Le couplage de l'incinération des déchets et du dessalement (Saint-Barthélémy) 09   Chapitre II La définition d'un modèle agricole robuste et résilient 10   I. Vers une agriculture climato-intelligente conjuguant logiques d'adaptation et d'atténuation 10    A. Agriculture et élevage, acteurs et victimes du changement climatique 10    B. De la recherche à la transformation des pratiques et des habitudes 11   C. L'outre-mer, interface entre le Nord et le Sud 12   II. Réalisation et projets 13    A. Les jardins créoles : l'exemple de Marie-Galante 13    B. La réintroduction d'anciennes variétés de cultures (indigo et pois) en Guadeloupe 14   C. La protection des abeilles sauvages : le projet Terre de pollinisateurs 14   D. Le couplage du photovoltaïque et de l'agriculture : le projet Bardzour 15   E. Les centres de ressources biologiques Inra-Cirad 15   Chapitre III La préservation et la mise en valeur des biodiversités ultramarines 16   I. La biodiversité ultramarine face au changement climatiques : des écosystèmes sous pression à protéger et à valoriser 16    A. La biodiversité ultramarine : une richesse sans équivalent 16    B. Une pression sur les écosystèmes lourdement accentuée par le changement climatique 16   C. Un enjeu de préservation et protection 16   D. Une nouvelle vision dynamique de la biodiversité : la valorisation 17   II. Réalisation et projets 17    A. Programme de conservation des espèces marines emblématiques en Polynésie Française 17    B. Recherche étiologique des maladies associées aux mortalités massives des poissons de la Côte Ouest de La Réunion dans le contexte du changement climatique 17   C. Développement de la filière des oléagineux sur la base expérimentale du concept Galbas en Guadeloupe 18    D. Projet d'Institut caribéen de la biodiversité insulaire à Saint-Martin 18   E. Restauration des écosystèmes marins en Guadeloupe et à Saint-Barthélémy 18   Chapitre IV La promotion d'une grande diversité d'énergies renouvelables 19   I. Les outre-mer, à l'avant-garde de la transition énergétique 19    A. Des difficultées liées aux problématiques spécifiques des outre-mer 19    B. De fortes potentialités dans le domaine des énergies renouvelables, soutenues par la loi sur la transition énergétique 19   II. Réalisations et projets 21    A. Valorisation de l'énergie fatale issue des fumées d'échappement des moteurs de production thermiques par un cycle ORC (Organic Rankine Cycle) à Mayotte 21    B. SWAC (Sea water air conditioning) du Centre hospitalier de la Polynésie Française 22   C. Valorisation de la biomasse et de la bagasse en Guadeloupe 22   D. Le projet d'énergie thermique de la mer en Martinique 23   Chapitre V La prévention des risques et la protection du littoral 23   I. La prévention des risques dans les outre-mer : une question de survie 23    A. Les outre-mer en première ligne face aux nouveaux risques climatiques 23    B. La nécessité d'une nouvelle approche préventive intégrée 25   II. Réalisations et projets 26    A. Cartographie du risque de submersion marine consécutive au passage d'un cyclone à Mayotte 26    B. Création de l'Observatoire de la dynamique côtière et colloque international sur les écosystèmes côtiers en Guyane 27   C. Wallis-et-Futuna : le projet LITTOFORT 27   Chapitre VI La sensibilisation et l'éducation du public 28   I. La connaissance et l'information, prémisses incontournables à l'action face aux impacts du changement climatique 28    A. Connaître pour mieux éduquer 28    B. Sensibiliser pour mieux protéger 30   II. Réalisations et projets 31    A. Feuille de route « Erosion » à Mayotte 31    B. Campagne de communication sur la maîtrise de la demande en énergie et mise en place d'un Espace Info énergie en Polynésie Française 31   C. Réseau d'aires marines en Polynésie Française 32   Avant-propos Mesdames, Messieurs,    Présente, grâce à ses outre-mer, sur tous les océans et sous toutes les latitudes, la France dispose d'une situation originale et privilégiée tant pour observer des évolutions climatiques que pour concevoir et expérimenter, avant d'en promouvoir la transposition, des solutions contribuant à atténuer la virulence et l'ampleur des dérèglements et permettant aux territoires de s'adapter.    Avec, dans l'arc Antillais, deux départements insulaires, la Guadeloupe et la Martinique, et deux collectivités, Saint-Barthélemy et Saint-Martin, un département continental amazonien d'une superficie équivalente à celle du Portugal, la Guyane, et une autre collectivité d'outre-mer en position septentrionale, Saint-Pierre-et-Miquelon, notre pays figure en de nombreux points de l'Atlantique nord. Les départements de La Réunion et de Mayotte dans l'océan Indien, les collectivités de la Polynésie française et de Wallis-et-Futuna ainsi que la Nouvelle-Calédonie dans le Pacifique sud viennent compléter le tour du monde des terres françaises habitées. Mais il convient d'ajouter à cet ensemble nombre d'îles et d'îlots inhabités, qui élargissent encore l'étendue et la diversité du territoire maritime de la France tout en constituant autant de positions stratégiques et de postes d'observation scientifique : nous pouvons citer l'îlot de Clipperton dans le Pacifique nord, administrativement rattaché à la Polynésie française, mais également les îles constituant le territoire d'outre-mer des Terres australes et antarctiques françaises (T.A.A.F.) comprenant les Îles Eparses dans l'océan Indien (1) et les îles subantarctiques d'Amsterdam et Saint-Paul, Crozet et Kerguelen.    A tous ces territoires disséminés autour de la planète, il convient d'ajouter la Terre Adélie, qui ne constitue pas à proprement parler une possession française puisque le traité de Washington de 1959 a établi un gel des prétentions territoriales tout en affirmant la liberté de la recherche scientifique sur l'ensemble du continent Antarctique. Le protocole de Madrid en 1991 a renforcé sa protection en faisant de ce continent « une réserve naturelle consacrée à la paix et à la science ». Ainsi, dans les bases scientifiques de Dumont d'Urville, sur l'île des Pétrels, et de Concordia, sur le plateau continental, de nombreuses études sont menées dans le cadre de collaborations internationales, notamment en matière de géophysique et de climatologie.    Aux avant-postes de la vulnérabilité climatique, les outre-mer sont également à l'avant-garde en matière d'observation et d'évaluation des impacts, mais aussi en termes de définition de stratégies d'adaptation et de conception de projets innovants. Ils bénéficient de l'expertise de centres de recherche de premier plan comme le Cirad, l'Inra, l'Irstea et le BRGM pour modéliser précisément les conséquences du changement climatique et ainsi élaborer par anticipation des plans d'action efficaces.    Il faut dire que la menace s'exerce déjà très concrètement sur ces territoires avec notamment une fréquence accrue des épisodes météorologiques extrêmes, l'élévation de la température et du niveau des océans, la raréfaction de la ressource en eau douce et la prolifération d'espèces invasives. En effet, des secteurs-clefs tels que l'agriculture et l'élevage, l'exploitation forestière, la pêche et l'aquaculture ainsi que le tourisme subissent de plein fouet les effets du dérèglement climatique. L'enjeu est donc tout à la fois d'organiser la préservation et la valorisation de patrimoines naturels exceptionnels et d'anticiper les adaptations nécessaires au maintien de conditions de vie satisfaisantes pour des populations qui doivent intégrer le paramètre environnemental dans tous les actes de la vie quotidienne.    Ce stress climatique tangible conduit à une prise de conscience collective et à une mobilisation des collectivités territoriales et des acteurs locaux, entreprises et société civile. Des stratégies territoriales, qui se déclinent en mesures d'atténuation et en dispositifs d'adaptation, prennent corps et inspirent déjà des dynamiques internationales dans leur environnement régional. Certaines réalisations sont parfois érigées en exemple, ce qui témoigne du rôle de pont entre les pays du Nord et du Sud que jouent les outre-mer. Ce fut encore le cas récemment au mois de juillet 2015 dans le Pacifique où la gestion vertueuse de l'eau et de l'assainissement depuis un quart de siècle à Bora Bora a pu être montrée en exemple aux dirigeants de sept États insulaires de la zone.    En outre, la perspective de la Conférence des parties de fin 2015 à Paris a activement stimulé la réflexion opérationnelle : de la Conférence internationale climat-énergie qui a abouti à la Déclaration des îles sur le changement climatique à La Réunion le 25 juin 2014 avec la création d'un Hub R20 Énergie Océan Indien, véritable plate-forme régionale pour le développement des énergies renouvelables dans la zone océan Indien, à l'appel de Fort-de-France du 9 mai 2015 qui a réuni plus d'une trentaine de chefs d'État de la Caraïbe, en passant par la Déclaration de Lifou « Paris 2015 : Sauvez l'Océanie ! » du 30 avril 2015 et le message de la Guadeloupe du 25 octobre 2014 consécutif à la Conférence internationale sur la biodiversité et les changements climatiques tenue sur ce même territoire, il apparaît que dans tous les bassins océaniques les territoires français sont fer de lance de la réflexion et de la promotion d'un nouveau modèle de développement.    L'appel à la définition d'un agenda des solutions afin de contenir l'évolution climatique dans une trajectoire maîtrisée et de mettre en oeuvre les adaptations requises a suscité par ailleurs l'adoption de nombreux documents locaux de planification fondateurs de stratégies territoriales, en particulier dans le domaine énergétique. De contrainte, le changement climatique pourrait se convertir en opportunités de tracer de nouvelles voies de développement plus frugal et plus durable grâce aux multiples possibilités offertes par l'économie circulaire.    Dans la perspective de la Conférence de Paris (COP 21) et conformément à la logique opérationnelle retenue, le groupe de travail commun constitué par votre Commission de l'aménagement du territoire et du développement durable et votre Délégation à l'outre-mer a opté, en apportant sa contribution à l'ensemble des travaux menés par le Sénat, pour une démarche pragmatique et concrète montrant la grande diversité et le foisonnement des initiatives locales. Il propose de mettre en valeur un ensemble de réalisations vertueuses et parfois tout à fait innovantes dont il a eu connaissance, notamment en effectuant un déplacement en Guadeloupe et à Saint-Barthélemy. Les projets présentés illustrent une sélection de six thématiques majeures pour les outre-mer face au défi climatique : la gestion de la ressource en eau et l'assainissement ; la définition d'un modèle agricole robuste et résilient ; la préservation et la mise en valeur des biodiversités ultramarines ; la promotion d'une grande diversité d'énergies renouvelables ; la prévention des risques et la protection du littoral ; la sensibilisation et l'éducation du public.    Chapitre I La gestion de la ressource en eau et l'assainissement I. LA FRAGILITÉ DES EAUX ULTRAMARINES STIMULE DES MODES DE PRODUCTION INNOVANTS A. Connaître les ressources en eau et comprendre leur évolution sous l'effet du changement climatique Le changement climatique, qui conjugue ses effets avec l'accroissement démographique, a tendance à exercer une pression sur la quantité et la qualité de l'eau douce disponible, qui ne représente que 2,6 % de l'eau sur la Terre. A la raréfaction globale de l'eau s'ajoutent en effet les risques d'accroissement de la salinité ou de modifications de la dureté, ce qui pèse fortement sur le rendement des réseaux. La surexploitation des nappes phréatiques, qui représentent 90 % des ressources mondiales d'eau douce, est un problème majeur dans la mesure où, d'un côté, elles sont directement utilisables et de l'autre, elles se rechargent lentement (2).    Les îles sont particulièrement vulnérables. De nombreuses zones comme le Pacifique Sud et la Caraïbe connaissent déjà des phénomènes d'intrusion d'eau de mer dans les aquifères côtiers où sont effectués les prélèvements (3). L'équilibre osmotique entre une lentille d'eau douce et la masse d'eau salée sur laquelle elle est posée est tel que le pompage d'un mètre d'eau douce fait remonter de 40 mètres l'eau salée (4). Un « surpompage » peut donc conduire à la disparition de l'eau douce par salinisation.    Les effets du changement climatique sur les précipitations sont modélisés avec plus d'incertitude que l'élévation des températures. Il se dessine cependant un accord pour prévoir des phénomènes catastrophiques plus fréquents et des sécheresses plus marquées. L'intensification et l'allongement des saisons sèches impliquent un abaissement significatif des niveaux d'étiage de la ressource ce qui exercera une très forte pression sur l'alimentation en eau potable, mais aussi sur l'environnement via une moindre dilution des produits phytosanitaires et une densification des matières en suspension. C'est la conclusion en particulier d'études de l'Inra sur deux bassins versants étudiés en Guadeloupe et Martinique afin de comprendre les transferts de polluants (5).    C'est pourquoi connaître précisément l'état de la ressource et la configuration du système hydrologique de chaque territoire est essentiel pour anticiper son évolution tendancielle et dessiner des solutions opérationnelles capables d'assurer la continuité du service en restant économiquement viables. Le travail sur l'accès à la ressource en eau et sa préservation demande une approche très fine et locale. Les diagnostics méritent souvent d'être dressés à une échelle infraterritoriale pour tenir compte du fait que les enjeux diffèrent non seulement entre les outre-mer (par exemple, entre la Martinique et la Guyane ou entre Saint-Martin et Saint-Pierre-et-Miquelon) mais aussi entre les parties d'un même territoire ultramarin (par exemple entre Basse-Terre et Grande-Terre, les Saintes, Marie-Galante et la Désirade en Guadeloupe, entre les Hauts et le rivage de La Réunion ou entre les îles hautes et les atolls en Polynésie).    Les outre-mer bénéficient déjà d'une batterie d'études de vulnérabilité. Ainsi, en Martinique, où la production d'eau passe à plus de 90 % par des prélèvements sur les rivières, la direction de l'environnement, de l'aménagement et du logement (DEAL) et le Bureau de recherches géologiques et minières (BRGM) ont développé conjointement entre 2011 et 2013 des modélisations quantitatives, uniques dans toute la Caraïbe, sur la base des projections climatiques de Météo France. On peut notamment en tirer comme leçons que la moitié Nord de la Martinique, où sont situés la totalité des captages d'adduction en eau potable, sera significativement impactée par le changement climatique, les tensions croissantes sur la ressource aboutissant à une rupture après 2080. En revanche, la recharge des aquifères pendant la saison des pluies devrait être plus intense, si bien que la ressource en eau souterraine serait moins touchée et qu'une augmentation du potentiel exploitable dans le centre de la Martinique est possible (6). Ces projections tendancielles appellent à la vigilance mais dessinent aussi des pistes de réorganisation possible de la production d'eau.    A La Réunion, les eaux souterraines représentent 50 % de la ressource en eau potable et les forages se trouvent principalement sur le littoral, ce qui les rend sensibles aux intrusions d'eau de mer dans les nappes phréatiques et donc à toute élévation supplémentaire du niveau de la mer. Pour mieux comprendre l'avancée du coin salé, 50 piézomètres ont été installés le long du littoral par l'office de l'eau, qui met ensuite les résultats des mesures à la disposition des industriels et des collectivités territoriales gestionnaires. Parallèlement, l'Université de La Réunion, le BRGM et l'Office de l'eau mènent des recherches sur les aquifères d'altitude pour évaluer les possibilités de mobilisation de cette ressource nouvelle (7).    B. Accroître la production sans compromettre la ressource Toute une gamme de techniques déjà développées à l'échelle industrielle permet de soutenir un accroissement de la production d'eau potable pour faire face à la dynamique démographique. En fonction de la géographie physique, géologique et hydrologique du territoire, des prévisions d'évolution de la ressource et de la consommation et des coûts estimés, il peut être fait recours à de nouveaux forages, à des galeries drainantes, à des déplacements d'ouvrages en amont pour prévenir les biseaux salés ou à des unités de dessalement d'eau de mer.    La question de la continuité du service et du maintien d'une qualité constante de l'eau se pose avec une acuité particulière en Polynésie où, sur 45 communes réparties sur 118 îles, seulement cinq parviennent à distribuer de l'eau potable avec 100 % de conformité (8). Dans l'archipel des Tuamotu, le problème est aggravé par l'éparpillement extrême de la population et par l'absence de source d'énergie locale, ce qui rend très délicat à la fois la production et le financement de la production et de l'adduction d'eau. Les solutions adoptées en Polynésie sont très diversifiées pour tenir compte de ces situations incomparables d'une île à l'autre, selon qu'il s'agit d'un atoll ou d'une île haute.    À Tahiti, sur la commune de Pirae, l'eau potable a été amenée en six mois grâce à une interconnexion avec Papeete et la mise en place de galeries drainantes. Le coeur de la technique est d'aller chercher l'eau dans les nappes phréatiques en corrigeant les variations de turbidité dues aux épisodes climatiques grâce à un système de filtration tirant parti de l'environnement lui-même (gravier, pierres). A Papeete même, deux types de ressources en eau sont conjuguées : des galeries drainantes et des forages horizontaux, particulièrement atypiques mais adaptés à la conformation géologique du territoire.    Lorsqu'il n'y a pas de source d'eau douce directement utilisable, il faut recourir au dessalement par osmose inverse, ce qui est très coûteux en énergie. Aux Tuamotu et aux Gambier, par exemple, il faudra recourir à des groupes électrogènes et donc à la consommation de matières fossiles pour alimenter l'osmoseur. En outre, l'osmose inverse génère des volumes de saumures équivalents à la moitié des volumes d'eau de mer prélevés (9), qui doivent être rejetés dans l'environnement avec précaution.    De fait, le dessalement de l'eau de mer représente une technique très à part et très spécialisée, qui requiert des compétences pointues pour :    - optimiser les coûts de production et maintenir dans le temps des rendements de production élevés afin de garantir un équilibre économique ;    - prévenir la corrosion des matériaux qui pourrait détériorer une usine en quelques semaines ;    - procéder à la reminéralisation, puisque l'eau obtenue par dessalement doit être reminéralisée pour être conforme aux normes sanitaires en vigueur (10).    Tout un travail de recherche et développement est en cours pour limiter l'impact de la production d'eau par dessalement sur les émissions de GES et sur le milieu naturel ; de nouvelles procédures avec une moindre empreinte carbone sont déjà en phase de test grandeur nature. Elles reposent notamment sur un couplage avec l'énergie photovoltaïque, une réduction de taille des stations ou la récupération de chaleur de traitement des déchets par incinération. En outre, de nouvelles voies de potabilisation de l'eau osmosée par électro-chloration solaire et de traitement des saumures sont en cours de développement (11).    En parallèle, pour tous les grands ouvrages d'alimentation en eau, une réflexion sur les économies d'énergie dépensée pour le pompage est menée. La préoccupation environnementale rejoint ici le souci de rentabilité économique puisque le coût de l'énergie est beaucoup plus élevé dans les outre-mer que dans l'Hexagone, notamment dans les collectivités du Pacifique.    En attendant la mise au point de nouvelles techniques, c'est l'optimisation du rendement des pompes qui est privilégiée en agissant sur différents facteurs comme le niveau de remplissage des pompes ou le moment de leur activation et, en général, tout ce qui peut améliorer le positionnement sur la courbe de fonctionnement de l'ouvrage.    A Papeete, depuis 1992, le volume d'eau prélevé est passé de 17 m3 à 7 m3. Cette optimisation résulte d'une forte amélioration du rendement du réseau et d'un changement dans les habitudes de consommation. Toutefois, malgré une baisse significative, un habitant de Polynésie continue d'avoir une très forte consommation d'eau (450 m3 par an par foyer contre 100 m3 environ en Martinique) (12). Les collectivités territoriales sont sensibles à la nécessité de faire un effort supplémentaire de pédagogie et d'éducation du public. Le prix de l'eau reste un sujet sensible dans l'ensemble des outre-mer. Les habitants considèrent à juste titre l'eau comme une ressource vitale et sa mise à disposition comme un service public. Toutefois, il est essentiel de rappeler que ce n'est pas une ressource inépuisable et que son adduction et sa potabilisation nécessitent des investissements et un entretien coûteux.    L'abaissement du coût de la production de l'eau est envisageable dans certains cas par transposition sur des ouvrages plus petits. C'est la solution retenue à Nuku Hiva aux Marquises. Les unités de traitement classiques étaient trop chères pour la collectivité territoriale et les habitants devaient parfois parcourir 20 km pour disposer d'eau potable. Pour répondre à ce besoin, un procédé spécifique a été mis au point pour gérer de petites quantités d'eau avec de ministations d'épuration construites pour traiter l'eau non potable du réseau. Trois fontaines publiques ont été installées pour distribuer gratuitement quelques m3 par jour d'eau potable, les coûts étant assumés par les usagers du service d'eau non potable.    C. Adapter les politiques territoriales de l'eau 1. L'expérience marie-galantaise en Guadeloupe Le sous-sol de Marie-Galante possède une structure sédimentaire comprenant des calcaires aquifères. Les nappes d'eau souterraines y sont bien répertoriées et les écoulements bien modélisés, ce qui permet depuis longtemps leur captage. Actuellement, neuf forages ou puits sur Marie-Galante peuvent être utilisés. Une fissure tectonique sépare néanmoins l'île en deux systèmes hydrauliques étanches, les Bas à la pointe Nord et les Hauts où sont situés les centres des communes de Grand-Bourg, Saint-Louis et Capesterre.    Grâce à une modélisation du BRGM, il est établi que 20 500 m3 d'eau par jour sont échangés avec la mer avec une fuite particulièrement marquée vers l'Ouest et le Sud.    Une baisse des prélèvements par pompage a été constatée et a conduit l'exploitant à mener un diagnostic précis avant de procéder aux adaptations nécessaires pour garantir le maintien de la qualité du service. Plusieurs facteurs explicatifs ont été envisagés :    - la baisse de l'alimentation naturelle de la nappe due à une baisse de la pluviométrie et de l'infiltration efficace ;    - la surexploitation des ouvrages mis à contribution en continu jour et nuit ;    - le vieillissement du système de pompage par pertes de rendement ou de fuites (pompe vieille/colonne percée) ;    - la diminution intrinsèque de la capacité des forages par colmatage des crépines et corrosion des tubages (13).    Un accroissement de la ressource disponible peut paradoxalement aller de pair avec une réduction de la production brute si le réseau n'est pas correctement entretenu. Si l'on tient compte ensuite de l'attrition potentielle occasionnée par le changement climatique, on mesure combien il est fondamental de prévenir les déperditions en procédant à des états des lieux, à un suivi et à la réparation des systèmes de forage et d'adduction. Pour réaliser des économies d'eau, il peut également se révéler intéressant de réduire la pression de pompage et d'installer des vannes intelligentes pour moduler le pompage selon le rythme diurne/nocturne de la consommation. Ce type d'action présente l'intérêt d'abaisser le coût de l'eau et de réduire la consommation d'énergie pour un même rendement, ce qui conjugue rentabilité économique et efficacité environnementale (14).    Cependant, sur sept forages actifs, le renouvellement des pompes à Marie-Galante, n'a pas permis de retrouver les débits initiaux sur deux d'entre eux et n'a conduit à une augmentation nette de la production que sur un seul. C'est pourquoi une salve d'inspections a été lancée en utilisant des caméras numériques à têtes orientables. Elle a révélé que plusieurs forages étaient en très mauvais état (entartrage massif, corrosion, détritus accumulés). Un programme de travaux a été défini sur la base de ce diagnostic. Dans le cas de trois sources, les forages existants ont été rebouchés et remplacés par de nouveaux forages bis. Les quatre autres forages ont été réhabilités (récupération des éléments tombés, acidification, injection d'air comprimé, brossage). Les résultats de cette campagne menée en 2014 sont probants puisque le débit global sur Marie-Galante après travaux a augmenté de 37 m3/h, soit 32 % de débit supplémentaire (15).    Des compléments de ressource ont également été explorés. Il est envisageable en cas de besoin de procéder à deux nouveaux forages sur des sites encore inexploités et de reprendre un ancien forage abandonné depuis 30 ans après une phase de surexploitation ayant conduit à sa salinisation.    2. Le modèle de Bora Bora en Polynésie française Bora Bora ne dispose que de peu de ressources en eau, mais elle doit alimenter non seulement ses 9 600 habitants, mais aussi chaque année 100 000 touristes venus du monde entier. Outre l'approvisionnement en eau potable, il faut également parvenir à assainir les eaux usagées et à gérer les déchets pour protéger l'environnement et en particulier le lagon, sans quoi la manne touristique se tarirait rapidement. La gestion de l'eau est donc un élément clef d'une stratégie de développement touristique durable assurant les ressources économiques nécessaires à Bora Bora pour donner à ses habitants des emplois stables.    Dix-neuf forages, complétés par deux usines de dessalement par osmose inverse, permettent de produire 3 000 m3 par jour. Le réseau de collecte des eaux usées est organisé autour de 70 postes de refoulement et de deux stations d'épuration : 80 % de l'eau épurée est rejetée dans l'océan, tandis que 20 % passe dans un réseau d'eau industrielle recyclée. Le système d'ultrafiltration membranaire produit 600 m3 par jour d'eau recyclée utilisée pour l'arrosage, la protection incendie et les usages industriels. Les boues résiduaires sont traitées par rhizocompostage pour être valorisées en compost dans une logique d'économie circulaire (16).    Bora Bora est considérée comme un exemple en matière de technologie et d'innovation. En juillet 2015, les dirigeants de Tonga, des Samoa et des Tuvalu s'y sont rendus afin d'étudier la possibilité de transposer ses installations (17).    3. La régie de Saint-Pierre (Saint-Pierre-et-Miquelon) Deux étangs situés sur les hauteurs de Saint-Pierre alimentent la commune. L'eau prélevée dans l'étang du Goéland et l'étang de la Vigie, fermés par des barrages, est acheminée à une station de traitement avant l'alimentation du réseau de distribution. La ressource d'un volume global stocké de 690 000 m3 est disponible toute l'année sauf pendant les tempêtes hivernales qui peuvent provoquer le gel des prises d'eau. Compte tenu de la répartition des précipitations sur l'année, il est nécessaire de remplir au maximum de leur capacité les deux réservoirs dès la fonte des glaces pour répondre aux besoins de la population jusqu'à l'automne, marqué par de fortes pluies.    Une grande campagne de recherche de fuites a permis de diminuer les prélèvements de 60 % entre 2008 et 2014 (18). Les campagnes de maintenance préventive au printemps et à l'automne doivent être prolongées par une vigilance accrue des particuliers, qui ont à s'assurer de la mise hors-gel de leur installation car les fuites hivernales demeurent un sujet de préoccupation. En effet, pour éviter le gel de la conduite, il est régulièrement fait recours par les particuliers à la pratique du coulage hivernal, c'est-à-dire qu'ils laissent leur robinet couler. Ce coulage se justifie quand la température extérieure baisse au-dessous de - 7°C en continu sur plusieurs jours, mais l'écoulement est souvent excessif par rapport à ce qui serait suffisant pour éviter le gel. La régie estime à environ 66 000 m3 le volume d'eau surconsommé (19).    La réduction des prélèvements permet de disposer de marges de manoeuvre supplémentaires en cas de sécheresse prolongée, en maîtrisant le volume d'eau retenu dans l'étang du Goéland. L'alimentation de la ville est par là-même davantage sécurisée, même dans le cas où le changement climatique perturberait le schéma annuel des précipitations.    II. RÉALISATIONS ET PROJETS A. La création d'un réseau international pour la gestion de l'eau sur le plateau des Guyanes (Guyane) Du 23 au 25 octobre 2013 s'est tenu à Cayenne un colloque international sur les modalités de coopération transfrontalière en matière de développement durable entre les pays du plateau des Guyanes (Brésil, Guyane, Suriname, Guyana, Venezuela, Colombie). Il avait notamment pour objectif d'identifier au niveau régional les problèmes de disponibilité (quantité, accessibilité) et de qualité (pollutions et assainissement) de la ressource en eau, les enjeux de préservation des milieux aquatiques, ainsi que les coopérations potentielles à mettre en oeuvre pour faciliter sa gestion.    Cet événement, qui a permis aux participants de partager leurs expertises et de mieux appréhender leurs cadres institutionnels, légaux et réglementaires respectifs pour la gestion de l'eau, a fait émerger un réseau, le Water Views and Information Network in the Guiana Shield (WAVINGS). Un portail Internet (www.guianashield.org/wavings/) est hébergé par le Guiana Shield Facility, qui est un organisme de financement géré par le Programme des Nations Unies pour le développement (PNUD) et destiné à soutenir des projets de conservation des écosystèmes et de développement durable dans les pays du plateau des Guyanes.    Sur la période 2015-2017, les axes de travail sont les suivants :    -croisement de l'expertise des services hydrologiques des différents pays du plateau des Guyanes ;    -étude de préfiguration de la création d'un service hydrologique régional à l'échelle du plateau des Guyanes ;    -création du service et intégration au système World Hydrological Cycle Observing System (WHYCOS), programme de l'Organisation météorologique internationale (OMI).    Sources : DEAL Guyane, Guiana Shield Facility.   B. Le réservoir d'Ouaboudé (Nouvelle-Calédonie) Le réservoir d'Ouaboudé est un ouvrage majeur situé à proximité de l'aéroport de Nouméa. Il a été conçu comme un ouvrage de secours en cas d'épisodes pluvieux forts qui demandent de gérer une variation très importante de la turbidité. Ce projet a été réalisé à la demande du Grand Nouméa. Il comprend deux unités d'une capacité de 4 000 m3 situées en surplomb, entre la station du Mont Té et la rivière Tontouta, où sont utilisés 8 à 10 puits de forage.    La fréquence d'utilisation du réservoir demeure variable par construction, puisqu'il s'agit d'un ouvrage de secours, mais elle doit croître, dans des proportions difficilement prévisibles, avec le changement climatique. Il a fallu tenir compte de ce caractère aléatoire au moment d'estimer a priori les risques et les coûts à la fois pour la construction et l'entretien. La concession a été récemment révisée pour permettre une clarification des rôles : la collectivité territoriale récupère l'ouvrage et porte l'investissement, la Calédonienne des eaux récupère l'exploitation.    De nouveaux outils d'aide à la décision faciliteront la construction des prochains ouvrages. Le programme hydrologique international (PHI) de l'Unesco permet depuis 2013 de décrire l'état actuel des aquifères de la Nouvelle-Calédonie et de fournir des projections sur vingt ans, dans le cadre d'une étude globale concernant 43 petits pays insulaires en développement. Une cartographie est prévue, de même que des schémas d'utilisation de la ressource en eau souterraine sur le long terme.    Sources : Calédonienne des eaux, Suez, Ministère des outre-mer.   C. L'assainissement des eaux usées par filtration sur roseaux (Martinique) Le projet ATTENTIVE (Assainissement des eaux usées adaptées au contexte tropical par traitements extensifs utilisant des végétaux) vise à développer des stations écologiques innovantes, adaptées au climat et au contexte économique et social, et à mettre en place les conditions de généralisation de ces filières en autonomie. Il utilise les recherches menées par l'équipe Epure de l'Institut national de recherche en sciences et technologie pour l'environnement et l'agriculture (Irstea) à Lyon qui travaille depuis de nombreuses années sur les petites collectivités en milieu rural et le développement des filières de traitement extensives. L'Irstea, dont 70 % des recherches menées concernent l'eau, en particulier la bio-indication (travaux à partir de diatomées et de macrophytes) et l'adaptation des systèmes d'épuration, est à l'origine du système français de filtres plantés de roseaux. Ce traitement optimise le cycle naturel d'épuration de l'eau. Il consiste à faire circuler de manière gravitaire les eaux brutes domestiques à travers des bassins successifs dans lesquels un mélange de minéraux plantés de végétaux permet le traitement des eaux usées. Par construction, ces dispositifs sont économes en énergie et permettent également de limiter les rejets azotés en milieu naturel.    Face aux problématiques d'assainissement des petites et moyennes collectivités des outre-mer, le projet ATTENTIVE doit permettre de valider des dispositifs adaptés au contexte tropical. Il permet, à partir de suivi à l'échelle pilote et en taille réelle, de diffuser des règles de dimensionnement avec les règles de gestion associées et de développer localement une expertise forte aussi bien en termes de construction que d'appui technique.    Trois installations (deux en Martinique et une en Guadeloupe) sont aujourd'hui en phase de test et fonctionnent de manière satisfaisante. Elles sont 2,5 fois plus compactes que les installations similaires dans l'Hexagone, ce qui est particulièrement important sur des îles exiguës. Les premières analyses relatives à leur efficacité spécifique au contexte tropical, déjà prometteuses, permettent d'être optimiste sur la généralisation de ces installations de traitement.    Ce projet est cofinancé à hauteur de 40 % par l'Office national de l'eau et des milieux aquatiques (Onema) dans le cadre de la solidarité interbassin. Il a été récompensé en 2014 par le grand prix national du génie écologique, porté par le ministère de l'écologie, du développement durable et de l'énergie.    Sources : DEAL Martinique, Onema.   D. La station d'eau potable de Matiti (Guyane) La Guyane connaît une augmentation très forte de la population, de l'ordre de + 3 % par an. Cette croissance démographique est notamment remarquable dans l'Ouest guyanais, la commune de Saint-Laurent du Maroni connaissant un rythme d'expansion de + 8 % par an. Cette situation entraîne une hausse rapide et intense des besoins en eau, qu'il faut parvenir à satisfaire malgré de fortes contraintes naturelles, au premier rang desquelles le biseau salé du Maroni.    Le changement climatique provoque l'élévation du niveau de la mer de 3,5 mm par an. Sur la surface très plane que constitue le rivage guyanais, son impact est très net. En outre, des saisons sèches plus marquées conduisent au tarissement relatif du débit des fleuves. Conjugué avec les projets de barrage sur le Maroni au Suriname pour le développement de l'agriculture, le changement climatique augmente très fortement le risque de salinisation des prises d'eau dans le bassin du Maroni.    Il a donc été nécessaire de prendre en compte le facteur climatique dans la construction de l'usine d'eau potable de Matiti. Pour sécuriser la ressource en évitant la salinisation, l'ouvrage a été remonté de 15 km en amont sur le Maroni. L'opération a duré six ans. Le financement des investissements est venu de subventions publiques (Etat : 12 millions euros, Feder : 7,5 millions et région : 2,3 millions), auxquelles s'ajoute un emprunt de 12,3 millions auprès de la Caisse des dépôts et consignations.    Sources : Société guyanaise des eaux, Suez.   E. Osmosun : le couplage de l'énergie solaire et du dessalement (Polynésie française) Suez s'appuie sur l'expertise de la société Mascara-NT pour développer actuellement un projet de production d'eau potable par osmose inverse avec désinfection au fil du soleil. La société Mascara-NT a, en effet, développé le couplage de l'osmose inverse et du photovoltaïque, avec une totale autonomie à l'égard des énergies fossiles. La production varie avec l'intensité solaire captée et la production est assurée sans stockage d'énergie. Cette approche technologique très innovante doit être désormais testée sur sites réels pour répondre efficacement aux attentes des collectivités isolées des zones côtières arides et insulaires.    Est d'ores et déjà prévue l'installation d'un démonstrateur d'une capacité de production de 80 m3 par jour à Bora Bora où la filiale polynésienne de Suez exploite déjà deux unités traditionnelles de dessalement par osmose inverse utilisant l'énergie électrique. Ce sera le premier site permettant de valider les performances sur le terrain, avant d'installer l'unité de production sur l'île voisine de Maupiti, en conditions réelles d'autonomie énergétique et d'exploitation.    Sources : Suez, Daniel Villessot (colloque Entreprises et dynamiques sectorielles du Pacifique, Sénat, 25 juin 2015).    F. Le couplage de l'incinération des déchets et du dessalement (Saint-Barthélemy) Les ressources en eau sur l'île de Saint-Barthélemy sont très limitées et la consommation humaine est satisfaite par le dessalement d'eau de mer. Les techniques utilisées permettent d'anticiper la croissance de la consommation d'eau. Si la consommation est aujourd'hui d'environ 730 000 m3 par an, elle n'était en effet que de 480 000 m3 il y a dix ans et de 300 000 m3 il y a quinze ans.    Après l'utilisation passée de la technique de compression mécanique, la production est désormais réalisée par deux procédés complémentaires et différents : l'osmose inverse et le dessalement thermique appelé Multi-Effect Distillation (MED).    L'unité MED est couplée à l'usine d'incinération d'ordures ménagères, qui lui assure sa source d'énergie principale, c'est-à-dire la production de vapeur. Cette technique, particulièrement innovante, est très économe en utilisation d'énergie fossile et permet simultanément une valorisation du traitement des déchets. Saint-Barthélemy a été la première île à utiliser ce procédé novateur et respectueux de l'environnement.    Sources : déplacement de la mission à Saint-Barthélemy en juillet 2015, Tiru, SIDEM Saint-Barthélemy Veolia Water.   Chapitre II La définition d'un modèle agricole robuste et résilient I. VERS UNE AGRICULTURE CLIMATO-INTELLIGENTE CONJUGUANT LOGIQUES D'ADAPTATION ET D'ATTÉNUATION A. Agriculture et élevage, acteurs et victimes du changement climatique L'agriculture et l'élevage constituent un secteur cardinal pour la problématique climatique. A l'aune des évolutions démographiques mondiales et du défi alimentaire corrélatif, l'activité agricole occupe une place déterminante dans la définition des politiques de réponse au changement climatique. En grande partie responsable de l'émission de gaz à effet de serre (GES) puisque l'exploitation des terres au sens large contribue à hauteur de 24 % des émissions globales (20), ce secteur offre toutefois à l'inverse des moyens de piéger le carbone. En outre, il est encore essentiel pour l'activité économique et l'emploi dans les outre-mer et subit directement les effets des transformations du climat. Cet impact porte autant sur les rendements et sur les volumes que sur les espèces et les variétés produites. La politique agricole doit donc conjuguer logique d'adaptation et exigence d'atténuation. Les actions à mener en ce domaine sont d'autant plus complexes qu'elles doivent se ramifier pour apporter simultanément des réponses aux questions connexes de la gestion des ressources en eau, de la protection de la biodiversité, de la frugalité énergétique et de l'éducation du public.    De multiples pressions liées, directement ou indirectement, aux transformations du climat affectent potentiellement l'agriculture. Il faudra compter avec des modifications de la pluviométrie à la fois en quantité, en fréquence et en distribution géographique et calendaire. Les risques, non seulement de sécheresse mais aussi d'épisodes de précipitations violentes, s'accroîtront. La salinisation des nappes phréatiques et des terrains littoraux devrait également s'étendre. La propagation induite de parasites, d'espèces invasives et de maladies vectorielles nécessiteront une adaptation des cultures, tant au niveau des variétés que des façons de produire à l'échelle de la parcelle et du territoire.    L'impact climatique se conjugue avec l'effet démographique pour mettre en tension et parfois remettre en cause les modes de production actuels. L'accroissement de la population mondiale appelle en effet un accroissement des rendements, sans épuiser les sols et les ressources en eau. C'est tout l'intérêt du concept de Climate Smart Agriculture, mis en avant par la Food and Agriculture Organization (FAO) des Nations Unies de proposer une approche de développement agricole intégrée qui vise à garantir la sécurité alimentaire dans un contexte de changement climatique, en liant l'augmentation durable de la productivité et des revenus agricoles, le renforcement de la résilience et de la capacité d'adaptation des systèmes agricoles et l'atténuation des émissions de GES (21).    Les enjeux sont intégrés et doivent être traités de façon globale, sans omettre en particulier la question de l'évolution des revenus des agriculteurs et des éleveurs. Les scénarios d'évolution du climat doivent aussi tenir compte des effets socioéconomiques du changement climatique : dans le scénario tendanciel par exemple, le principal effet du changement climatique serait de faire monter les prix des productions animales et végétales, de stimuler la déforestation pour trouver de nouvelles terres arables et, par ce biais, d'accélérer les émissions de GES.    La modélisation des effets prévisibles du changement climatique sur les rendements des productions agricoles essentielles des outre-mer est ardue. Cependant, tant pour la canne à sucre que pour la banane, les études disponibles pointent une diminution probable à moyen terme des rendements liée à un raccourcissement du cycle de culture, à une augmentation de la température et à des périodes de sécheresse plus marquées (22). L'impact serait plus important sur la canne, car la banane a une température optimale supérieure à celle de la canne. La productivité en sucre de la canne devrait se réduire à l'horizon 2030-2040 d'après ces simulations. En revanche, les bananeraies sont particulièrement vulnérables aux cyclones. Pour la culture de la banane se pose surtout la question de l'irrigation, bien mise en évidence par le modèle SIMBA développé par le Centre de coopération internationale en recherche agronomique et développement (Cirad) et l'Institut national de la recherche agronomique (Inra) sur le cas antillais.    A La Réunion, ont été développés, outre les modèles de croissance et de prévision pour la canne, d'autres modèles biotechniques sur l'effet du changement climatique sur les cultures d'ananas et de mangue. En Guyane, des travaux sont menés sur la croissance de la forêt naturelle guyanaise, maltraitée par une succession d'événements secs et pluvieux ; ils visent à anticiper le temps de régénération des espèces dans la forêt tropicale. Leurs résultats montrent une diminution des taux de croissance liée à l'augmentation de la température et une augmentation de la mortalité en liens avec des événements climatiques extrêmes (grande sécheresse ou pluies intenses). On peut s'attendre à une diminution forte de la productivité des peuplements forestiers. Pour les essences commerciales, comme l'angélique qui représente 60 % des volumes extraits, la crainte principale, est liée au stress hydrique dû à l'allongement et l'intensification de la saison sèche (23).    Il convient de ne pas sous-estimer le risque induit par le développement des espèces invasives. Ces dernières constituent un enjeu majeur pour les îles en raison de leur impact aussi bien sur la santé humaine que sur les cultures et l'environnement, soit directement, soit indirectement via le recours à des pesticides. A La Réunion, par exemple, l'augmentation de la température provoque très vite la pullulation des insectes et autres espèces invasives. Des études montrent qu'une augmentation d'un ou deux degrés peut faire varier les infections de cultures lors d'une épidémie de 10 % à 100 % (24). C'est une menace pour les agriculteurs, en particulier les cultivateurs de canne dans les Hauts, où la pression des insectes ravageurs s'intensifie. En particulier, la recrudescence du thrips noir de la canne à sucre paraît être un effet du changement climatique (25). Le même type de danger risque d'affecter la biodiversité du Parc national de La Réunion, classé au patrimoine mondial protégé par l'Unesco. Aux Antilles, il ne faut pas négliger les maladies fongiques -comme la cercosporiose du bananier -qui peuvent peser sur le rendement de la récolte mais aussi sur la conservation après récolte.    Par ailleurs, en raison de leur plus grande diversité génétique, les plantes adventices (« les mauvaises herbes ») devraient s'adapter plus rapidement et plus efficacement au changement climatique que les grandes cultures. La pression des plantes adventices devraient donc s'accroître sans qu'il soit envisageable d'intensifier la lutte chimique en raison de son impact environnemental, notamment sur les ressources en eau. C'est pourquoi des recherches ont actuellement menées, par exemple sur l'igname en Guadeloupe, pour développer des variétés de culture plus compétitives en termes de vigueur, d'efficacité d'interception du rayonnement et d'architecture du feuillage (26).    L'élevage risque également d'être durement touché par les perturbations climatiques qui peuvent créer des conditions plus favorables à la prolifération des vecteurs de certains pathogènes. Par exemple, le virus de la fièvre de la vallée du Rift est apparu à Mayotte, via du bétail probablement importé illégalement depuis les Comores. Toute l'Afrique de l'Est est touchée par cette épidémie qui a émergé à la suite d'un épisode d'El Niño et aux violentes pluies qui y ont été associées (27).    B. De la recherche à la transformation des pratiques et des habitudes L'enjeu pour l'agriculture des outre-mer est de faire du défi et de la contrainte climatiques une opportunité de réorienter le développement vers des modes de production durables consommant moins d'intrants et capables de porter le défi démographique mondial, en particulier en termes de production de protéines végétales.    C'est surtout la variabilité climatique qui pose problème aux agriculteurs, notamment dans les zones tropicales où sont situés, pour l'essentiel, les outre-mer français. Elle est d'ores et déjà perçue par les agriculteurs et les filières de transformation. Il faut par conséquent parvenir à tracer des pistes concrètes de réduction de la vulnérabilité des agriculteurs à cette variabilité accrue, notamment en construisant des réseaux participatifs, en élaborant des outils d'aide à la décision suffisamment souples pour être transposables d'un territoire à l'autre et en renforçant la coopération entre les chercheurs et les producteurs. Les organismes de recherche agronomique comme le Cirad et l'Inra sont bien implantés dans les outre-mer et contribuent déjà de concert à engager ce processus dans une logique de transition agroécologique.    Le Cirad a spécifiquement pour mandat de travailler sur la recherche agronomique dans les zones tropicales en développement et dans les départements d'outre-mer. Les outre-mer représentent d'ailleurs un quart des effectifs du centre, qui y est présent depuis les années 1960, soit environ 400 personnes travaillant à La Réunion, à Mayotte, en Guadeloupe, en Martinique et en Guyane, dont 100 postes permanents de chercheurs. Dans les Antilles et en Guyane, cinq grands thèmes de recherche sont étudiés : l'agroécologie ; les maladies émergentes et la santé animale ; la biodiversité et la génétique des plantes tropicales ; la forêt ; les pâturages et l'élevage. Pour la zone Réunion-Mayotte, la recherche porte sur trois grands thèmes : la biodiversité et les risques sanitaires ; la qualité des productions agricoles et alimentaires ; les services environnementaux de l'agriculture et le recyclage des déchets organiques. Les collaborations avec l'Inra sont très étroites et intenses.    L'Inra est le premier établissement en termes de publication en sciences agronomiques au niveau international. Il a fait de l'adaptation au changement climatique et de son atténuation une des priorités de son action. Dans les outre-mer, il s'appuie sur un centre de recherche Antilles-Guyane qui compte quatre implantations, trois en Guadeloupe (Petit-Bourg, Godet et Gardel) et une en Guyane à Kourou. Y exercent au total 191 agents titulaires et 25 contractuels. Les recherches portent sur l'agroécologie, l'élevage et la socio-économie des territoires en Guadeloupe et sur la biodiversité de la forêt et sa sensibilité au changement climatique en Guyane.    L'adaptation des cultures présente de multiples facettes. L'une des plus importantes consiste à trouver des espèces plus résistantes à la sécheresse. Un programme du Cirad en Guadeloupe vise à réaliser des croisements de diverses variétés et espèces de bananiers pour obtenir des plants plus résistants au stress hydrique et disposant d'un système racinaire plus adapté qui permet une meilleure capture hydrique. D'autres travaux portent sur l'amélioration de la résistance des agrumes à la sécheresse et à la salinité. Ils consistent à valoriser la mise en culture de porte-greffes tétraploïdes (28). En effet, depuis le début du XXe siècle, les agrumes sont cultivés sur des porte-greffes qui les rendent plus résistants aux maladies. Pour limiter encore l'impact des maladies tout en s'adaptant au changement climatique, des porte-greffes hybrides ont été créés pour additionner les caractères des deux plants parents et cumuler ainsi leurs propriétés de tolérance. Cette hybridation confère à la variété greffée sur le porte-greffe tétraploïde une meilleure tolérance à la sécheresse et au stress salin sans affecter la qualité des fruits produits (29). Elle permet aussi de renforcer les agrumes contre une maladie bactérienne mortelle que les changements climatiques vont certainement contribuer à développer, le huanglongbing (HLB) appelé encore greening qui ravage la zone Caraïbe, le Brésil et la Floride, sans qu'aucun traitement curatif efficace ne soit disponible (30).    Des moyens de lutte biologique ont été développés par le Cirad à La Réunion pour faire face à des espèces invasives potentiellement stimulées par le changement climatique, en bénéficiant du soutien du Pôle de protection des plantes de Saint-Pierre. On peut citer comme exemples de réussite la lutte contre le ver blanc de la canne à sucre grâce à un champignon ou contre les mouches blanches grâce à l'introduction de microguêpes en cultures maraîchères et fruitières. Deux sociétés privées, Betel Réunion et La coccinelle, travaillent en partenariat étroit avec le Cirad pour développer des gammes d'auxiliaires biologiques (31). Avant même la lutte, ce sont la surveillance et le contrôle en amont qui sont essentiels. A cet effet des bases de données et des outils de science participative sont développés pour permettre une détection précoce des espèces préoccupantes. On peut citer comme exemple les applications mobiles Pl@ntnet et Diagnoplant, adaptées avec l'Inra pour identifier des bioagresseurs sur une parcelle.    Une plus grande diversité des cultures garantit prima facie une plus grande résilience aux aléas climatiques. C'est pourquoi la protection ou la réintroduction de variétés anciennes et la diversification des productions pour contrebalancer la fragilité des grandes monocultures (banane, canne à sucre) constituent des axes majeurs de transformation de l'agriculture ultramarine. Le modèle de polyculture offert par le jardin créole est particulièrement pertinent (32). Dans le souci de renforcer l'agrobiodiversité, l'Inra travaille aux Antilles à soutenir la petite agriculture familiale dont 75 % des exploitations sont en polyculture associée à de l'élevage. Les perturbations climatiques poussent à changer de paradigme pour reconquérir les productions vivrières à destination du marché local (33) alors que les grandes cultures exportatrices sont plus exposées tant aux risques naturels qu'à une concurrence internationale exacerbée.    Le renforcement de la résilience environnementale et socioéconomique des systèmes de production et des territoires exige non seulement de travailler sur l'adaptabilité des espèces animales et végétales, mais aussi de développer des systèmes intégrés associant animaux et végétaux pour favoriser la substitution et l'interconversion des biomasses (perspective d'économie circulaire) ou encore de stimuler la production de services de régulation au niveau d'un territoire donné (séquestration du carbone, pollinisation, maintien de la qualité de l'eau, etc.) (34). Les modèles biophysiques et socioéconomiques méritent d'être couplés pour proposer des voies de développement innovantes dans lesquelles peuvent être combinés :    - une approche agroécologique des cultures réorientées vers le marché intérieur, dans une logique de réduction des importations, encore massives dans les outre-mer. Cela contribuera à réduire l'empreinte carbone en économisant sur les transports et en favorisant les circuits courts ;    - le recyclage par compostage de déchets organiques ;    - et les cultures énergétiques pour générer de la biomasse.    Ces pistes intéressantes pour conjuguer l'adaptation climatique et la transition énergétique constituent un exemple de perspective systémique pour penser l'adaptation de l'agriculture ultramarine au changement climatique. C'est la même démarche holistique qui inspire les recherches en cours d'application sur les plantes de services, c'est-à-dire sur les plantes qui apportent un service environnemental complémentaire dans la parcelle cultivée. L'introduction d'une nouvelle biodiversité végétale dans les systèmes de culture permet la régulation des bioagresseurs et des plantes adventices, la lutte contre l'érosion, l'amélioration de la fertilité des sols ou le recyclage d'éléments nutritifs par exemple (35).    En guise d'illustration on peut citer les pratiques d'enherbage des cultures d'arbres fruitiers ou la lutte contre des flétrissements bactériens des tomates avec des plantes assainissantes. Dans le cadre du plan Banane durable aux Antilles, des systèmes intégrant des plantes de service ont été conçus et sont en cours d'adoption par les producteurs. L'agriculture de conservation consiste à associer des plantes de couverture à une culture principale, de sorte que le sol soit couvert en permanence au fil des saisons. Cette démarche est associée à une diminution du labour. Les travaux sur la complémentarité entre végétaux offrent des résultats très intéressants pour limiter le recours à des intrants. Par exemple, aux Antilles est testée dans des bananeraies la culture de canavalia, qui fixe l'azote de l'air dans le sol sans en prélever trop, ce qui fournit au bananier une sorte d'engrais naturel. En outre, les racines du canavalia rejettent des substances nocives pour certains parasites du bananier.    Les modalités de transition agroécologique font d'ores et déjà l'objet de simulations à l'échelle des territoires concernés. D'après les simulations transmises par l'Inra, en 2040, une adaptation agroécologique de la Guadeloupe au changement climatique peut conduire à réduire de 10 % le total des émissions de GES produites dans l'archipel tous secteurs économiques confondus (36). Le programme Gaia-Trop financé par l'Agence nationale de la recherche (ANR) depuis 2013 porte sur la viabilité et la gouvernance adaptative des agrosystèmes des îles tropicales. Il a permis de concevoir des modèles mathématiques pour concevoir des chemins de transition agroécologique par aménagement de la biodiversité à l'échelle de l'exploitation agricole. Il constitue un outil d'aide à la décision à la disposition des responsables territoriaux et des exploitants agricoles des outre-mer (37).    C. L'outre-mer, interface entre le Nord et le Sud Les implantations du Cirad et de l'Inra ultramarines offrent à ces organismes la possibilité d'étudier des agroécosystèmes tropicaux dont on sait qu'ils seront particulièrement vulnérables au changement climatique. Les outre-mer disposent ainsi d'un potentiel important de recherche et d'expertise scientifique qui peut être mobilisé pour développer des solutions adaptées et transposables dans d'autres régions tropicales. Par exemple, l'étude de la forêt en Guyane, les prévisions d'évolution comme l'élaboration de stratégies d'adaptation de l'exploitation ne peuvent manquer de se répercuter sur ce qui se fait au Brésil. Des interactions positives pour les outre-mer sont possibles dans l'autre sens ; par exemple, en matière de résistance à la sécheresse, les travaux du Cirad les plus importants sont situés en Afrique (mil, sorgho, arachide, caféier) mais peuvent irriguer l'agriculture ultramarine, qui est elle-même historiquement le fruit de la confluence d'imports sud-américains, africains et européens.    D'une manière générale, tous les sujets agricoles méritent d'être traités au niveau régional en interaction avec les pays environnant les outre-mer. C'est particulièrement vrai pour les aspects sanitaires puisque le risque épidémique et de propagation des maladies et des agents infectieux nécessite une surveillance au-delà des frontières nationales pour inclure aussi des pays dont les systèmes de santé humaine et vétérinaire sont moins performants. Le réseau CaribVET a été ainsi créé en 2000 pour renforcer et harmoniser la surveillance et le contrôle des maladies animales transfrontalières. Ce réseau rassemble désormais tous les pays de la zone et se développe dans le cadre du Caricom, l'organisation régionale des États de la Caraïbe. Les objectifs sont d'améliorer la situation sanitaire et la productivité des élevages et de promouvoir le commerce dans la Caraïbe. Des programmes de recherches bénéficiant de financements européens y trouvent aussi un ancrage et une infrastructure (38). Le réseau SEGA -« One Health », né en 2006 après une épidémie de Chikungunya et basé à La Réunion est le seul en son genre à aborder conjointement la santé humaine et la santé animale. Il se développe dans le cadre de la Commission de l'Océan Indien (COI). Un de ses membres essentiels est Madagascar qui est soumis à de fortes contraintes et fait de ce réseau un acteur en première ligne du suivi des maladies émergentes liées au changement climatique (39).    Des volets agricoles sont également prévus dans des programmes de coopération régionale. C'est le cas par exemple à La Réunion du programme régional de protection des végétaux (ePRPV) adossé à l'Initiative régionale agroécologie et changement climatique (IRACC), coordonné par la COI et financé par le Fonds international de développement agricole. Une plateforme régionale est installée à La Réunion pour discuter avec tous les ministres de l'agriculture et de l'environnement des pays voisins, en particulier pour diffuser les bonnes pratiques de lutte contre les ravageurs et les maladies affectant les cultures maraîchères et fruitières dont l'importance est essentielle pour les économies de la zone. Des actions sont en cours dans ce cadre contre la mouche des fruits aux Comores et contre le varroa, responsable de l'effondrement des populations d'abeilles, à Maurice et à Madagascar.    Les outre-mer apparaissent comme les fers de lance d'une nouvelle révolution agroécologique dont bénéficieront les pays les moins avancés (PMA). Grâce à la conjonction unique sur ces territoires de la vulnérabilité qui pousse à l'action, de l'expertise qui permet de comprendre et d'orienter les décisions et de l'exemplarité de pratiques déjà testées et mises en place, les outre-mer ont vocation à jouer un rôle d'interface entre les pays du Nord et les pays du Sud. Cela constitue assurément un atout pour la France, qui mériterait d'être valorisé au service du succès de la COP 21 au Bourget.    II. RÉALISATIONS ET PROJETS A. Les jardins créoles : l'exemple de Marie-Galante La particularité du jardin créole est de présenter un synoptique de l'ensemble des grandes migrations qu'ont connues les Antilles. Les Siboney, puis les Arawak et les Caraïbes apportèrent des plantes d'Amérique du Sud comme la dictame et le roucou. Ensuite, les peuples européens commencèrent à arriver à la fin du XVe siècle, dont les Français qui s'installèrent définitivement au XVIIe siècle. Pendant la période de l'esclavage, les esclaves cultivaient des petites parcelles, où se trouvaient rassemblées une grande variété de cultures. C'était l'ancêtre du jardin créole qui se présente essentiellement comme un modèle de polyculture organisée.    Par exemple, dans le jardon créole de M. Richard Chelza visité par vos rapporteurs à Marie-Galante, se trouve au milieu un arbre-fruitier qui donne des pommes-cannelles. A proximité sont plantées des salades et tout autour des sources de protéines végétales dont une multitude de légumineuses rapportées d'Afrique : les pois de bois ou pois d'Angole, les pois cannes appelés aussi niébé ou cornille, les pois boucoussou, Sainte-Catherine, savons, etc. Ces pois sont d'un excellent rendement et adaptés aux conditions climatiques de la Caraïbe. Leur culture est associée à celle de la canne à sucre qui leur sert de tuteurs. L'intérêt des pois est qu'il s'agit de plantes qui fleurissent en période de jours courts dont la culture peut donc parfaitement associée au rythme de pousse de la canne à sucre, elle-même récoltée à l'hivernage au moment où la teneur en sucre est la plus élevée, d'où une excellente gestion coordonnée des productions traditionnelles sur l'année. La variété de canne cultivée est une variété ancienne qui a besoin de moins d'eau que celle que l'on retrouve dans les grandes plantations. Elle est cultivée sans engrais et sans pesticide avec le projet de produire du jus de canne biologique, voire du rhum biologique. Dans le jardin créole visité, se trouvent enfin des melons, des aubergines et de petits concombres à cornichons.    Le jardin créole est un jardin de subsistance d'où l'on peut tirer tous les aliments nécessaires à la vie humaine (glucides, lipides, protéines, vitamines et oligoéléments). Il se distingue par sa capacité à fournir une grande diversité de protéines d'origine végétale, dont le couplage traditionnel avec des céréales facilite l'assimilation par le corps humain, qui est essentielle pour son approvisionnement en acides aminés.    Il permet de moins dépendre de l'élevage pour la production de protéines, ce qui signifie à la fois produire plus de protéines par surfaces mobilisées mais aussi consommer moins d'intrants (nourriture des animaux). On retrouve avec le jardin créole un bon exemple de conservation de la biodiversité faunistique (abeilles, oiseaux, chauve-souris), de recours à la permaculture (création d'humus en permanence qui permet une économie d'eau) et de croissance en synergie des plantes les unes avec les autres (protection réciproque contre les agresseurs, favorisation réciproque de pollinisation).    Ce modèle de biodiversité agricole est particulièrement intéressant alors que l'agriculture mondiale doit relever conjointement le défi démographique et le défi climatique. Il va de pair avec une valorisation des productions locales et des circuits courts, avec des bénéfices en termes tant de stimulation économique que de réduction des émissions carbonées par reflux des importations et baisse subséquente des transports. C'est un paradoxe que 80 % de l'alimentation des Guadeloupéens soit importée alors qu'il existe une grande diversité de cultures qui peuvent s'épanouir sur son sol. Bien sûr, autosuffisance alimentaire et autarcie totales sont utopiques mais il demeure une marge de progression conséquente. Il convient donc de ne pas laisser perdre le savoir associé à la culture du jardin créole et d'en poursuivre la transmission.    Sources : DEAL de Guadeloupe -M. Henry Joseph -déplacement du groupe de travail à Marie-Galante (juillet 2015)    B. La réintroduction d'anciennes variétés de cultures (indigo et pois) en Guadeloupe Les plantations d'indigotiers étaient très nombreuses jusqu'au XIXe siècle à Marie-Galante ; mais elles connurent un fort déclin jusqu'à leur disparition à cause de la synthèse de l'indigotine par Baeyer en 1880 et l'essor de la production synthétique par BASF en 1900. Le même problème s'était déjà posé dans le Midi de la France avec la disparition de la production de garance après la synthèse et la commercialisation de rouge d'alizarine par BASF dès 1871.    Cependant, un projet de relance de la production d'indigo, piloté par le laboratoire Phytobôkaz à Gourbeyre, est en cours de réalisation en profitant de l'entrée en vigueur du règlement européen REECH qui touche les colorants de synthèse et demande un retour à des produits naturels obtenus par extraction. Si les résultats se révèlent probants, le projet pourra s'étendre avec la relance d'une production locale de coton en tirant partie du fait que l'origine première d'une variété importante de plants de coton (Gossypium Barbadense) se trouve dans les îles de la Caraïbe, notamment La Barbade et Marie-Galante qui partagent le même terroir.    Marie-Galante connaît un autre projet de réintroduction de cultures anciennes qui concerne certaines variétés de pois. Ce projet est cofinancé par le laboratoire Phytobôkaz et Iguavie (lnterprofession guadeloupéenne de la viande et de l'élevage) avec le partenariat de la mairie de Saint-Louis de Marie-Galante qui met à disposition du foncier agricole (d'abord 5 000 m2 avec une possibilité d'extension). Trois variétés résistantes aux maladies ont été retenues pour éviter d'avoir à recourir à des traitements et des pesticides. Deux pieds peuvent donner sans épisode de sécheresse 15 kg de pois l'année donc 1 000 pieds (pour chaque variété plantée) donnent 7,5 tonnes en un an. En outre, quatre rangées sont consacrées à une plantation expérimentale d'indigo, car l'indigo et les pois poussent sur des terroirs similaires.    Depuis 1995, les terrains concernés relèvent d'une exploitation cannière, avec un complément d'élevage. L'exploitation est périodiquement laissée en jachère pour ne pas utiliser de pesticides. Le projet pois-indigo intervient dans le cadre d'un processus de transformation de l'exploitation vers une polyculture organisée. La diversification se poursuit par une réduction de la surface de canne et par l'introduction du maracuja (grenadille ou fruit de la passion). Ce choix est motivé par le fait que la fécondation tant des fleurs de maracuja que des fleurs de pois est assurée par une espèce d'abeille sauvage de Guadeloupe, communément appelée les « von-vons ». En revanche, l'abeille domestique ne peut pas les féconder. On retrouve l'idée d'un projet global associant des cultures en synergie et tenant compte de la faune endémique de l'île, ce qui permet d'assurer ainsi une diversité floristique et faunistique. Dans un même mouvement, des cultures anciennes sont réintroduites et des espèces animales sauvages endémiques sont protégées.    Sources : DEAL de Guadeloupe - Phytobôkaz - déplacement du groupe de travail à Marie-Galante (juillet 2015)    C. La protection des abeilles sauvages : le projet Terre de pollinisateurs L'archipel guadeloupéen compte environ huit espèces d'abeilles sauvages qui sont solitaires et ne vivent pas en colonie, sauf une seule qui donne du miel, la mélipone, en voie de disparition. Depuis plusieurs années, se met en place un plan de sauvegarde, amplifié par un arrêté ministériel de 2015 de protection des abeilles sauvages, qui recoupe d'autres politiques de diversification et de résilience agricole. Avec l'aide du Cirad, depuis cinq ans, pour ne pas utiliser d'herbicides dans les bananeraies, est utilisé un couvert végétal de plantes de service ; or les plantes sélectionnées sont favorables aux abeilles sauvages, ce qui est bénéfique tant pour les producteurs de miel que pour les planteurs de bananes.    A partir de 2016, sur toute la Guadeloupe, sera mis en place le projet Archipel Guadeloupe -Terre de pollinisateurs. Il est porté par l'association des apiculteurs de Guadeloupe (Apigua) et vise la création d'un observatoire des pollinisateurs, la réintégration des pollinisateurs dans les jardins créoles, l'élaboration d'un modèle d'agroforesterie pour concentrer et stimuler la diversité et l'activité des pollinisateurs, la création d'un espace de communication, assorti d'un plan et d'actions de communication orientées vers le grand public.    L'idée centrale est d'obtenir, en multipliant les observateurs, des données collectées sur un grand nombre de sites et dans des environnements variés. Il devrait aussi contribuer à enrichir les connaissances sur le fonctionnement des réseaux d'interactions plantes-insectes. Au-delà du recueil de données qui intéresse les scientifiques, l'observatoire des pollinisateurs a une vocation pédagogique : sensibiliser le public à la diversité du vivant. À terme, le projet s'étendra vers la réalisation d'une banque de graines des espèces mellifères et la généralisation des haies végétales dans le paysage agricole. Il contribuera à diffuser et à valoriser les bonnes pratiques visant à préserver les pollinisateurs sauvages en différents contextes (agricole, forestier, urbain et naturel).    Une boucle de rétroaction (feedback) pédagogique est prévue pour assurer un retour d'information au public scolaire qui aura participé à ces actions. L'enjeu fondamental est de mettre en place une protection efficace des écosystèmes environnementaux ruraux et de valoriser leur biodiversité dans un contexte de développement durable et équitable.    Ce projet mobilise en collaboration avec l'Apigua plusieurs partenaires :    - l'Inra pour son expertise sur les systèmes agricoles ;    - le Parc national de Guadeloupe pour son expertise sur les espèces et pour son territoire ;    - le Lion's Club pour mobiliser le public et pour assurer la diffusion des informations ;    - la DEAL, le Conseil départemental, Phytobôkaz, la chambre d'agriculture et l'association Les dimanches de la création pour leurs rôles respectifs dans la protection de l'environnement ;    - le rectorat pour le volet éducatif.    Sources : DEAL de Guadeloupe - Phytobôkaz - déplacement du groupe de travail à Marie-Galante (juillet 2015)    D. Le couplage du photovoltaïque et de l'agriculture : le projet Bardzour Le 11 juin 2015 a été inaugurée à La Réunion la centrale photovoltaïque Bardzour, qui signifie en créole « aube naissante ». Porté par la société Akuo Energy, ce projet a été récompensé dans le cadre du concours My Positive Impact lancé par Nicolas Hulot. Il repose sur le concept d' « agrinergie », créé par le groupe Akuo Energy comme un mode de valorisation de l'économie locale grâce à l'établissement de synergies entre l'énergie solaire et l'agriculture. Il s'agit d'éviter ou de contrebalancer la perte de terrains cultivés qu'entraîne l'implantation au sol d'une centrale photovoltaïque. Akuo Energy propose, grâce à la vente de l'électricité produite, de financer des projets agricoles innovants (protection de variétés endémiques menacées, culture biologique, prise en compte de la vulnérabilité climatique,...).    Plusieurs particularités singularisent Bardzour. La production de 9 MWc à partir d'énergie solaire, soit un tiers de la consommation de la commune du Port où la centrale est implantée, est couplée avec un dispositif de stockage performant (batteries lithuim-ion) permettant la réinjection différée de 9 MWh dans le réseau. Les panneaux solaires ont été déposés sur une serre agricole de 6 000 m2 avec notamment des productions biologiques. Un arboretum est également installé aux pieds de la centrale et regroupe 48 espèces d'arbres endémiques, 18 espèces de plantes aromatiques et médicinales et 35 espèces fruitières). De même, 14 ruches ont été implantées pour produire du miel. Enfin, ce projet comporte un volet social. Situé au sein du centre de détention du Port, il offre aux détenus des possibilités de réinsertion via des formations au maraîchage et à l'apiculture. Cette vitrine contribue déjà au succès à l'international d'Akuo Energy, qui vient de signer un contrat avec l'électricien Pertamina pour la construction de 560 MW de capacités d'énergie renouvelable en Indonésie.    Sources : Akuo Energy - Les Échos, « Solaire : Akuo joue la carte de l'innovation », 11 juin 2015 - Fondation Nicolas Hulot.   E. Les centres de ressources biologiques Inra-Cirad Les outre-mer français constituent des lieux essentiels de conservation de ressources biologiques. L'Inra et le Cirad ont créé et gèrent conjointement un réseau de centres de ressources biologiques (CRB) permettant de travailler sur les ressources génétiques.    Depuis le sommet de Rio en 1992, les ressources biologiques sont définies comme des matériels d'origines diverses (végétale, animale, microbienne,...) contenant des unités fonctionnelles de l'hérédité présentant une valeur potentielle ou effective. Elles sont conservées avec un ensemble d'informations associées (morphologiques, biochimiques, taxonomiques, agronomiques, juridiques,...). Les régimes de conservation varient en termes de modalités (in situ, ex situ, dans une ferme) et de formes (plantes entières, vitroplants, de graines...).    Ces centres ont pour finalité de freiner l'érosion génétique issue de la diminution -historique depuis la préhistoire mais en accélération depuis le XVIIIe siècle - du nombre d'espèces cultivées et de la perte de diversité génétique au sein des variétés cultivées, seules quelques variétés performantes étant développées. Les ressources génétiques apparaissent désormais comme des enjeux stratégiques : il est essentiel de pouvoir conserver un maximum de gènes et d'allèles qui permettront de créer des variétés adaptées en fonction des contextes climatiques, énergétiques et sociaux (comportements des consommateurs) (40).    Le réseau des CRB Cirad-Inra comprend sept centres dont la tête de pont est située en Guadeloupe au sein du CRB Plantes tropicales créé en 2010. Outre son rôle de conservation, il met du matériel végétal à la disposition des utilisateurs qui en font la demande, soit toute personne, entreprise ou organisation qui souhaite valoriser les ressources du centre à des fins économiques, pédagogiques ou scientifiques. Ce réseau permet de mettre en relation les outre-mer entre eux (Guadeloupe, Martinique, Guyane, La Réunion), avec l'Hexagone (Montpellier et Corse) et avec l'étranger (Madère).    Le CRB Plantes tropicales a, par exemple, pris en charge l'introduction de 15 variétés de madères en provenance de Fidji dans le cadre du projet EVA-Transfert, une plateforme d'évaluation et de transfert d'innovation. L'objectif du programme est de « renforcer l'offre potentielle en tubercules alimentaires et en fruits à la Guadeloupe par la mise en place d'une plateforme d'innovations et d'un réseau participatif destinés à promouvoir la sélection, la multiplication et la diffusion de variétés performantes, bien adaptées aux conditions pédoclimatiques locales, et valorisables dans un cadre agricole diversifié et durable. » (41) Le même CRB fournit des plants de manguier en Afrique (Guinée, République démocratique du Congo) pour le développement de filières. D'autres travaux portent sur l'igname, l'ananas, la canne à sucre, les agrumes et les orchidées.    Sources : Inra-Cirad - déplacement du groupe de travail en Guadeloupe (juillet 2015).   Chapitre III La préservation et la mise en valeur des biodiversités ultramarines I. LA BIODIVERSITÉ ULTRAMARINE FACE AU CHANGEMENT CLIMATIQUE : DES ÉCOSYSTÈMES SOUS PRESSION À PROTÉGER ET À VALORISER A. La biodiversité ultramarine : une richesse sans équivalent La biodiversité ultramarine constitue 80 % du patrimoine naturel français. Selon les chiffres du plan d'action outre-mer annexé à la stratégie nationale pour la biodiversité, les collectivités françaises d'outre-mer appartiennent à huit grandes régions biogéographiques terrestres et marines situées dans les trois océans de la planète, en zones tropicale, équatoriale, australe et antarctique et possèdent, du fait de leur caractère majoritairement insulaire, un niveau d'endémisme élevé. Leur patrimoine biologique naturel est unique, tant par sa richesse que par sa diversité. Il représente ainsi 98 % des vertébrés et 96 % des plantes vasculaires présentes en France. Dans tous les groupes, ces collectivités abritent plus d'espèces que la métropole (100 fois plus de poissons d'eau douce, 60 fois plus d'oiseaux, 26 fois plus de plantes et 3,5 fois plus de mollusques endémiques). Avec 3 450 espèces végétales vasculaires et 380 animaux vertébrés uniques au monde, elles hébergent même, pour ces deux groupes, plus d'espèces endémiques que toute l'Europe continentale.    La richesse de cette biodiversité ne se concentre pas uniquement sur les espèces, mais se retrouve également dans les milieux et les habitats. Le cas des mangroves et des coraux l'illustre bien. La France dispose de 55 000 kilomètres carrés de récifs coralliens et de lagons, soit près de 10 % de la surface des récifs mondiaux, dans trois océans. Les récifs coralliens sont, avec les forêts tropicales, les écosystèmes les plus riches et les plus productifs de la planète. Leur biodiversité est exceptionnelle : un tiers des espèces marines connues vit dans ces récifs. C'est dans ce cadre que se développent les mangroves et les herbiers.    L'ensemble de ces écosystèmes garantissent aux communautés humaines de nombreuses ressources et de nombreux services. On estime par exemple qu'en Martinique ils produisent près de 245 millions d'euros par an.    B. Une pression sur les écosystèmes lourdement accentuée par le changement climatique La pression exercée sur eux est de ce fait très alarmante. Les récifs coralliens régressent continuellement. D'ici à 2050, 75 % d'entre eux pourraient disparaître. Le projet de loi relatif à la biodiversité, en cours d'examen, prévoit, dans ce cadre, un programme d'actions pour la protection des mangroves et des récifs coralliens.    Autre exemple, dans le Pacifique en Polynésie française, les atolls sont les systèmes les plus vulnérables face au changement climatique, qui amplifie considérablement les impacts anthropiques locaux.    La biodiversité ultramarine est, de manière globale, fortement menacée par le changement climatique.    En effet, si les écosystèmes naturels des outre-mer constituent les zones les plus riches en biodiversité, ils sont aussi les espaces les plus menacés de la planète. La Nouvelle-Calédonie ou la Guyane, par exemple, sont listés comme des « points chauds » de la biodiversité par l'Union internationale pour la conservation de la nature (UICN).    Pour prendre le seul exemple de la biodiversité marine, les pressions exercées par le changement climatique se manifestent par :    - des modifications des propriétés physiques et chimiques des océans et des écosystèmes marins, dont la rapidité jamais observée depuis des millions d'années, est particulièrement critique et remet en cause les capacités d'adaptation de bon nombre d'espèces ;    - des bouleversements de la distribution géographique et de l'abondance des espèces, voire des invasions biologiques par des espèces exotiques.    Au cours des tables rondes « Biodiversités des outre-mer et changement climatique », organisées le 11 juin 2015 par la commission de l'aménagement du territoire et du développement durable et la délégation à l'outre-mer du Sénat, l'ensemble des menaces pesant sur la biodiversité ultramarine ont été exposées par les différents intervenants (42). Philippe Goulletquer, directeur scientifique adjoint chargé de la coordination biodiversité marine et côtière à l'Ifremer, a par exemple cité le cas des effets du changement climatique sur l'abondance et la répartition d'une espèce de thon (Listao) à l'échelle mondiale : « L'application des scénarios du Giec sur l'évolution des conditions environnementales et d'alimentation des thons, et par voie de conséquence, la répartition géographique de leurs populations, démontre les modifications induites sur les activités de pêche futures - en l'occurrence, une très forte réduction -, sur l'économie et les conditions d'alimentation des populations en zone tropicale ».    C. Un enjeu de préservation et de protection Face à cette situation alarmante, le premier défi pour les outre-mer est la préservation de ce patrimoine naturel exceptionnel, tant à travers la mobilisation d'outils territoriaux adaptés aux spécificités locales que par le développement de la connaissance et de la recherche innovante. Les populations sont, sur ce sujet, les premiers acteurs de protection de leur cadre de vie et de leur richesse. De nombreuses collectivités l'ont d'ores et déjà compris.    A titre d'exemple, la direction de l'environnement du Gouvernement de la Polynésie française mène un programme ambitieux de lutte contre les espèces exotiques envahissantes, qui a pour objectif d'informer et de former les populations pour mieux lutter contre ces espèces menaçant la biodiversité locale.    Dans le Pacifique, alors que la salinisation et la contamination des nappes phréatiques menacent les atolls, une des réponses consiste en la mise en place et dans le développement d'aires marines et littorales protégées et de conservatoires des ressources génétiques afin de soutenir la résilience de ces écosystèmes et de garantir leur développement économique.    Dans le cadre des travaux du Grand Port maritime de la Guadeloupe nouvelle génération, est prévue la transplantation d'herbiers de phanérogames marines ainsi que de coraux, après que l'avis de l'Autorité environnementale d'octobre 2013 a mis en évidence une atteinte directe sur ces milieux marins présentant un grand intérêt écologique (voir encadré ci-dessous).    D. Une nouvelle vision dynamique de la biodiversité : la valorisation Au-delà de la protection de ce patrimoine, la priorité doit être aujourd'hui également donnée à sa valorisation, qui consacre une nouvelle approche de la biodiversité, non plus statique, mais dynamique. La biodiversité peut en effet être valorisée économiquement, comme le développement du biomimétisme en témoigne. Par l'imitation de propriétés remarquables du vivant appliquées aux activités humaines, le biomimétisme permet des innovations dans de nombreux secteurs, comme l'industrie ou l'agriculture. Les structures osseuses des ailes d'oiseaux ont ainsi inspiré la diminution de poids et l'augmentation de la résistance des ailes de l'Airbus A 380. La structure en picot hydrophobe de la feuille de nénuphar est à l'origine du concept de pare-brise anti-pluie. L'exploitation des oléagineux pour le développement d'une filière cosmétique centrée autour de l'huile de galba constitue un autre exemple de valorisation économique de la biodiversité.    Le protocole de Nagoya, adopté en 2010, traduit cette volonté nouvelle de concilier préservation et valorisation de notre patrimoine naturel. Ce protocole, par la mise en place de procédures d'accès aux ressources génétiques et de partage des avantages, permet à la fois de garantir un accès responsable et maîtrisé aux ressources naturelles ultramarines mais aussi de sécuriser l'utilisation de la biodiversité dans le cadre de projets à visée commerciale. Il permet enfin, notamment en Guyane, de préserver les droits des communautés d'habitants autochtones comme les Bushinengue.    II. RÉALISATIONS ET PROJETS A. Programme de conservation des espèces marines emblématiques en Polynésie française La faune et la flore terrestre de Polynésie française présentent parmi les plus forts taux d'endémisme au monde : près de 60 % pour les végétaux avec 900 espèces végétales indigènes dont 570 endémiques et 100 % d'endémisme pour certains genres de mollusques et d'insectes.    Le milieu marin de Polynésie française renferme une des plus grandes diversités géomorphologiques récifales avec par exemple plus de 800 espèces de crustacés, 170 espèces de coraux, ou encore 425 espèces d'algues indigènes. La Polynésie comporte près de 20 % des atolls dans le monde et certains des principaux stocks de pêche encore préservés.    Cette richesse est cependant menacée. Les espèces insulaires ont en effet une aire de répartition réduite et des populations de faible effectif qui les rendent plus vulnérables au risque d'extinction. Ainsi, plus de 140 plantes sont considérées comme rares, vulnérables ou menacées, selon les listes rouges de l'Union internationale pour la conservation de la nature (UICN). De manière globale, la Polynésie est l'un des 25 points chauds de la biodiversité dans le monde, toujours selon l'UICN.    Dans ce contexte, depuis 2002, la direction de l'environnement du Gouvernement de la Polynésie française a lancé un programme de conservation des espèces marines emblématiques. L'objectif est de préserver les populations de baleines, de dauphins, de tortues marines, de requins entre autres, afin de répondre aux aspirations des polynésiens et protéger leur patrimoine naturel et culturel. L'objectif est également d'adapter localement le plan d'actions du Programme régional océanien de l'environnement (2013-2017) aux espèces marines.    Au total, une zone de 5,5 millions de kilomètres carrés a été déclarée sanctuaire des baleines et autres mammifères marins depuis 2002. Sur cinq espèces de tortues, trois (tortue verte, tortue imbriquée, tortue caret) font l'objet d'études de migration inter-îles et régionales, d'études génétiques et de répartition de populations dans les archipels par la collecte d'échantillons à des fins génétiques.    La mise en oeuvre de ce programme a permis de mesurer une diversité biologique élevée avec plus d'une vingtaine de baleines et de dauphins recensés, une vingtaine d'espèces de requins protégés.    Le programme a également permis diverses campagnes de communication et de sensibilisation, en particulier sur l'activité de d'observation des baleines, qui connaît depuis dix ans un intérêt croissant.    Des ateliers de formation ont été mis en oeuvre (baguage, suivi de sites de pontes, prélèvement d'échantillons d'espèces marines protégées pour analyse génétique, réglementation, approche des mammifères marins).    Dans les années à venir, le programme permettra la poursuite des suivis d'espèces et des inventaires pour améliorer la connaissance de la biodiversité polynésienne ainsi que la poursuite des formations, des contrôles et de la surveillance de l'eau.    Source : Gouvernement de la Polynésie française.   B. Recherche étiologique des maladies associées aux mortalités massives des poissons de la Côte Ouest de La Réunion dans le contexte du changement climatique Le programme REMPOR, « Recherche étiologique des maladies associées aux mortalités massives des poissons de la côte Ouest de La Réunion », s'inscrit dans le cadre de l'observation de phénomènes récurrents de mortalité de poissons à La Réunion ayant des impacts économiques (pêche, image de La Réunion au national et donc impact sur le tourisme) et sur la biodiversité, notamment du récif corallien. Ces phénomènes sont liés à des températures de l'eau élevées et seront probablement amenés à se répéter dans les années à venir du fait du changement climatique.    Ce programme s'inscrit dans le cadre des objectifs du Livre bleu et doit permettre de développer les compétences nécessaires à un « Pôle Mer ». Il s'intègre également dans la démarche de gestion intégrée mer-littoral visant à prendre en compte le continuum Terre-Mer. Il vise en effet à mieux comprendre l'origine épidémiologique de ces maladies dans ce continuum : milieu marin, milieu aquatique d'eau douce.    Durant ces 20 dernières années, l'émergence et la prolifération des maladies ont affecté la diversité et l'abondance de nombreuses espèces récifales. Aujourd'hui elles sont reconnues comme une menace importante, capables d'altérer la structure et le fonctionnement des écosystèmes coralliens. La recherche des causes a été amorcée lors du dernier épisode de 2014 démontrant la présence d'une bactérie. L'objectif de ce programme est donc de mieux comprendre les causes et les conséquences de ces maladies afin de mieux prévenir leur apparition et de mieux comprendre l'impact du changement climatique sur cet écosystème.    Les résultats de cette étude pourront être intégrés dans d'autres programmes de recherche, et/ou de suivi-monitoring (locaux, nationaux et internationaux) traitant de l'état de santé des récifs coralliens tels que le programme d'évaluation des risques pesticides pour les récifs coralliens de La Réunion (ERICOR) ou encore dans le cadre de la Directive Cadre sur l'Eau (DCE).    Source : DEAL de La Réunion.   C. Développement de la filière des oléagineux sur la base expérimentale du concept Galbas en Guadeloupe Ce projet, lancé en 2007 à Gourbeyre en Guadeloupe, est porté par le docteur Henry Joseph, également co-fondateur du laboratoire Phytobôkaz, spécialisé dans la fabrication locale de produits dérivés de plantes médicinales ultramarines.    Ce projet vise à développer la filière des oléagineux sur la base expérimentale de l'huile de galba.    Le développement de la production de graines de galba (Calophyllum calaba), en Guadeloupe, est un marché de niche. La composition chimique exceptionnelle de cette huile et ses propriétés biologiques particulières dans le domaine des antioxydants et de la réparation cellulaire constituent un exemple intéressant d'une valorisation économique réussie de la biodiversité.    Le projet, implanté à Gourbeyre, a permis de concevoir un ensemble agro-écologique durable et de recréer un écosystème interdépendant sur une plantation alternée de galbas, calebassiers, cocotiers et avocatiers. Il a stimulé la création d'une filière des oléagineux de Guadeloupe reposant sur des circuits courts. Ce projet a également eu des impacts positifs sur la biodiversité animale, d'une part, avec une augmentation constatée des populations de chauve-souris et d'abeilles, sur la biodiversité végétale d'autre part, grâce à la reforestation visible sur la parcelle.    Source : DEAL -Guadeloupe.   D. Projet d'Institut caribéen de la biodiversité insulaire à Saint-Martin Le projet d'institut caribéen de la biodiversité insulaire (ICBI) est porté par la Réserve nationale naturelle de Saint-Martin. L'objectif affiché est de fournir un outil au service du développement du territoire, de l'écologie, de la protection de la biodiversité, de la recherche universitaire et scientifique, de l'écotourisme et du développement économique.    L'ICBI en est pour l'heure et jusqu'à octobre 2016, au stade des études opérationnelles, la phase de travaux devant se dérouler tout au long de l'année 2017. Il doit :    - devenir un lieu privilégié de mise en valeur du patrimoine naturel et de la biodiversité de Saint-Martin ;    - développer des outils d'accueil et de transfert de connaissance pour expliquer, sensibiliser, éduquer tous les publics à l'environnement et à l'éco-citoyenneté ;    - apporter et soutenir de nouvelles synergies axées sur la conservation de la biodiversité, la recherche, la formation professionnelle, la création de filières économiques durables, la promotion touristique, en s'affirmant comme un pôle de rayonnement et d'échanges scientifiques régionaux et internationaux ;    - créer de nouveaux emplois liés aux métiers de la mer, de la pêche, de la protection et de la valorisation des espaces naturels ;    - contribuer à la préservation du cadre de vie exceptionnel de Saint-Martin, condition essentielle pour le maintien de l'attractivité du territoire.    Ce projet permettra de faire émerger un véritable pôle de recherche centré sur la biodiversité du territoire et surtout, plus largement, de l'espace Caraïbe, qui est un élément stratégique pour Saint-Martin.    Il est enfin central dans le cadre de la lutte contre le changement climatique et ses impacts, comme par exemple via l'introduction d'herbivores marins dans le cadre du développement d'un pôle de recherche aquacole.    Source : Préfecture de Saint-Martin.   E. Restauration des écosystèmes marins en Guadeloupe et à Saint-Barthélemy Le groupe de travail du Sénat sur « les outre-mer confrontés au changement climatique », s'est rendu, en juillet 2015, en Guadeloupe et à Saint-Barthélemy.    Dans ce cadre, vos rapporteurs ont pu assister à une présentation des mesures compensatoires mises en oeuvre dans le cadre des travaux d'aménagement liés au Grand port maritime de Guadeloupe, à savoir des programmes de transplantation d'herbiers, de coraux et de replantation de mangroves.    L'autorité environnementale, dans un avis datant d'octobre 2013, avait en effet recommandé de compléter les mesures d'évitement, de réduction et de compensation des impacts du projet sur l'environnement, et plus spécifiquement, les milieux marins.    Ces mesures de compensation concernent les herbiers et les coraux (transplantation des herbiers, immersion de récifs artificiels pour les coraux, sentier sous-marin,...), les mangroves (replantation, protection et nettoyage), une sensibilisation du public, la pêche (capture de « cultures post larvaires », dispositif de concentration des poissons, actuellement non autorisé, l'usage de récifs artificiels, la reconversion des pêcheurs.    Dans ce contexte, des mesures de restauration de ces trois écosystèmes en étroite relation (3 348,1 hectares de mangroves, 11 870 hectares d'herbiers et 33 890 hectares de mangroves) sont mises en oeuvre :    - une transplantation d'herbiers dans le cadre des travaux du Port Nouvelle génération : 3 300 m2 d'herbiers prélevés avec un objectif de 13 000 m2 d'herbiers à reconstituer (la 1re campagne de suivi a eu lieu en août 2015) ;    - une transplantation de récifs coralliens dans le même cadre : 4 300 colonies transplantées (les 1re et 2e campagnes de suivi ont eu lieu en avril et juin 2015 et la 3e en septembre 2015).    Par ailleurs, le groupe de travail sénatorial s'est vu présenter à Saint-Barthélemy deux projets de reconstitution de coraux partant d'initiatives locales. Le premier mise sur la constitution de pépinières et le bouturage de cornes de cerf et de cornes d'élan, deux espèces d'acropora dont le rythme de croissance est particulièrement rapide jusqu'à 10 cm par an. Cette technique a été perfectionnée en Floride.    Le second projet s'appuie sur une technique d'accrétion électrolytique qui permet grâce à des structures métalliques immergées et placées sous basse tension de former des structures calcaires qui serviront d'habitat artificiel aux coraux. Ce procédé s'est particulièrement développé à Bali.    Source : Déplacement du groupe de travail en Guadeloupe et à Saint-Barthélemy en juillet 2015, entretiens avec le Grand Port maritime de Guadeloupe et MM. Didier Laplace et Turenne Laplace.    Chapitre IV La promotion d'une grande diversité d'énergies renouvelables I. LES OUTRE-MER, À L'AVANT-GARDE DE LA TRANSITION ÉNERGÉTIQUE A. Des difficultés liées aux problématiques spécifiques des outre-mer Les outre-mer, malgré leur grande diversité géographique et culturelle, présentent tous, à l'exception de la Guyane, la particularité de l'insularité. Cette insularité constitue un réel défi pour l'approvisionnement des populations en énergie. L'enjeu de l'interconnexion des réseaux sur ces territoires, y compris pour la Guyane, est essentiel. En effet, tandis qu'en métropole près de 80 % de l'électricité consommée est d'origine nucléaire, dans les outre-mer, une très faible part de cette électricité est produite localement, la plus grande part étant produite à partir d'hydrocarbures importés.    Conséquence directe de cette spécificité, le coût de l'énergie est, selon les territoires, cinq à dix fois supérieur qu'en métropole. Le coût supporté par les consommateurs est en revanche identique grâce à la contribution au service public de l'électricité (CSPE), dispositif fiscal assurant un tarif identique à toute la population française.    Un grand nombre de territoires ultramarins se distinguent également par un usage différent de l'énergie, dans la mesure où c'est davantage le besoin de « froid », produit à partir d'électricité, qui prime le besoin de chaleur.    Tandis que les enjeux de lutte contre le changement climatique nécessitent de réaliser une véritable transition énergétique sur tous les territoires, les implications ne sont pas les mêmes pour les outre-mer, compte tenu des spécificités évoquées ci-dessus.    B. De fortes potentialités dans le domaine des énergies renouvelables, soutenues par la loi sur la transition énergétique La loi relative à la transition énergétique pour la croissance verte fixe toutefois des objectifs ambitieux pour la croissance verte. Les départements d'outre-mer doivent viser l'autonomie énergétique en 2030, avec, dès 2020, un objectif de 30 % d'énergies renouvelables à Mayotte et 50 % à La Réunion, en Martinique, en Guadeloupe et en Guyane, contre 23 % en métropole. La Polynésie française, par une loi du pays de 2013, s'est fixé « un objectif minimum de 50 % de production électrique issue de l'exploitation des énergies renouvelables à échéance 2020 ». Le schéma pour la transition énergétique de la Nouvelle-Calédonie arrêtée par une délibération du Congrès du 20 octobre 2015 fixe quant à lui pour objectif à l'horizon 2030 un doublement de la part du renouvelable pour atteindre une production équivalente à 100 % de la consommation électrique de la consommation publique sur la Grande Terre et l'autonomie électrique des Îles Loyauté. Compte tenu de la situation de forte dépendance énergétique aux hydrocarbures, atteindre ces objectifs nécessitera un effort important et la mise au point de solutions innovantes, utilisant toutes les ressources offertes par ces territoires. Ces derniers présentent tous en effet de fortes potentialités. Notons que les coûts élevés de production des centrales thermiques traditionnelles font d'ores et déjà dans les outre-mer de certaines sources d'énergie renouvelable comme l'éolien terrestre, la biomasse ou encore la géothermie et le photovoltaïque des sources compétitives.    A) L'énergie photovoltaïque L'énergie photovoltaïque convertit l'énergie lumineuse solaire en énergie électrique, notamment à La Réunion, où cette filière représente 20 % du mix énergétique.    En 2011, La Réunion a lancé le dispositif « Eco-solidaire », qui consiste en une aide destinée aux foyers réunionnais à faibles revenus et qui leur permet de diminuer d'au moins 30 % leur facture d'électricité. La collectivité met à disposition un fonds de garantie pour permettre aux ménages défavorisés, propriétaires de leur bien, de s'équiper en chauffe-eau solaire. La Région finance à hauteur de 80 % l'installation du chauffe-eau solaire réalisé par une entreprise bénéficiant de l'appellation « Qualisol », et EDF apporte une aide de 500 euros par équipement. En 2011, 635 familles ont pu bénéficier du dispositif et 1000 en 2012. A terme, ces opérations devraient permettre d'économiser 900 MWh et de diminuer l'émission de gaz à effet de serre de 737 tonnes eqCO-.    Citons pour la Guyane le projet associatif pour une petite électrification domestique sur les fleuves frontaliers. L'association de Cayenne, Kwala Faya, propose une solution transitoire réalisable à court-terme pour assurer un service électrique répondant aux attentes des habitants des sites isolés, fortement dépendants des énergies fossiles, et notamment des générateurs individuels. A la demande de certains acteurs impliqués dans des projets d'électrification (Programme régional pour la maîtrise de l'énergie (PRME), Parc amazonien de Guyane (PAG), Préfecture, EDF, commune de Camopi, commune de Maripasoula), Kwala Faya a proposé une nouvelle approche dont la particularité est d'intégrer un volet social ainsi qu'un volet économique. Deux actions ont été menées dans ce cadre en 2014, en partenariat avec les acteurs de l'électrification sur les communes de Camopi et de Maripasoula :    - des actions de formation en photovoltaïque à destination des habitants des sites isolés de l'Oyapock (Camopi, Trois-Sauts) et du Haut Maroni ;    - une mission de prospection à Trois-Sauts en vue de l'électrification du site à base d'installations solaires photovoltaïques.    Une telle démarche contribue à l'appropriation des systèmes proposés et à leur gestion autonome, propose des débouchés économiques et permet à tous les bénéficiaires de diminuer leur dépendance aux énergies fossiles, d'augmenter de fait leur capacité financière mais aussi de renouer avec l'élément naturel du « Kwalai », le soleil.    B) L'énergie éolienne L'énergie éolienne vise à produire de l'électricité à partir de l'énergie mécanique produite par le vent, notamment en Guadeloupe, qui compte actuellement 12 parcs produisant près du quart de la production électrique d'origine renouvelable de ce territoire.    C) Les énergies marines Les énergies marines recouvrent l'ensemble des technologies permettant de produire de l'électricité à partir de différentes forces ou ressources du milieu marin (hydrolien, houlomoteur, énergie marémotrice, etc.).    En Martinique par exemple, le projet de centrale de 10 MW reposant sur l'énergie thermique des mers (ou ETM) illustre ce développement (voir encadré ci-dessous).    D) La biomasse-énergie La biomasse-énergie est produite à partir de matières organiques, particulièrement importante en Guyane grâce à ses ressources forestières, ou encore dans les centrales thermiques utilisant de la bagasse (résidu de canne à sucre), comme par exemple en Guadeloupe à l'usine du Moule à Grande-Terre.    A La Réunion, le projet de valorisation énergétique des broyats de déchets verts préparés par l'entreprise ILEVA sur sa plateforme de broyage par la centrale thermique du Gol qui doit les utiliser comme combustible en substitution du charbon en est à sa deuxième phase d'essai. Le projet doit permettre, à terme, la valorisation énergétique des déchets verts produits au Sud de La Réunion et d'utiliser d'un combustible vert pour la production d'électricité de l'île, en complément de la bagasse déjà utilisée, visant à réduire la consommation de charbon importé.    E) L'énergie géothermique L'énergie géothermique récupère la chaleur contenue dans le sous-sol pour produire de l'électricité.    Le seul exemple actuellement en fonctionnement se trouve sur le site de Bouillante en Guadeloupe que le Bureau de recherches géologiques et minières (BRGM) exploite. Les possibilités de coopération régionale dans l'arc Caraïbe pourraient être mises à l'étude pour tirer également profit du potentiel géothermique de Montserrat et de la Dominique.    F) L'énergie hydraulique L'énergie hydraulique est produite grâce aux cours d'eau, avec par exemple le barrage de Petit-Saut et la centrale hydraulique de Saut Maman Valentin en Guyane, qui représentent une puissance installée de 119 MW, soit plus de la moitié de l'électricité utilisée en Guyane.    Un séminaire sur le bilan des connaissances pour le développement de la petite hydroélectricité en sites isolés a été organisé le 4 juin 2015 par l'Office de l'eau de Guyane afin de présenter un état des lieux des connaissances et des gisements hydroélectriques, du projet de guide opérationnel pour le montage de projets en petite hydroélectricité ainsi que leurs impacts environnementaux potentiels.    L'accélération de la mise en oeuvre des projets de transition énergétique est d'autant plus urgente dans ces territoires qu'ils sont les plus exposés aux conséquences du changement climatique, et notamment aux risques que comporte l'insécurité en matière d'approvisionnement énergétique. C'est pourquoi la loi sur la transition énergétique s'est fixé comme objectif de lever les obstacles à ce déploiement, en prévoyant un certain nombre de mesures spécifiques. Ainsi, en Guyane, où 40 000 citoyens ne sont pas desservis par le réseau d'électricité, la programmation pluriannuelle de l'énergie, nouvel outil créé par la loi, devra préciser les actions mises en oeuvre pour corriger cette situation, ainsi que les investissements prévus dans les installations de production d'électricité de proximité. En outre, le Gouvernement devra, d'ici fin 2015, présenter un rapport « indiquant quelles mesures spécifiques d'accompagnement il entend développer en faveur de la Nouvelle-Calédonie, de la Polynésie française et de Wallis-et-Futuna, afin de permettre à ces trois collectivités territoriales d'appliquer les principaux dispositifs de la présente loi » (43). Il devra notamment étudier l'élargissement de la CSPE à ces territoires. Enfin, l'alignement progressif des tarifs de l'électricité de Wallis-et-Futuna sur ceux de la métropole a été voté et permettra à plus de 4 000 foyers de voir leur facture d'électricité divisée par 5 en 5 ans.    Les territoires ultramarins ont très vite pris cette problématique à bras le corps en soutenant le développement des énergies renouvelables et en mobilisant des moyens sur le lancement de projets contribuant à diversifier les sources d'énergie, lutter contre le changement climatique, protéger la biodiversité et préserver les ressources.    Ainsi, en Guyane, le Conseil régional, les collectivités locales et les partenaires locaux de l'énergie et du développement durable, ont soutenu la création, en 2008, de l'association Guyane Énergie-Climat. Les objectifs principaux de cette association consistent en une amélioration de la connaissance de la situation énergétique et des émissions de gaz à effet de serre, une centralisation des données énergies-climat et une facilitation de leur mise à disposition auprès des acteurs régionaux. Guyane Energie-Climat a mis en place un « Observatoire de l'énergie » puis, à partir de 2014, un Observatoire du carbone et des gaz à effet de serre ». Elle a également entrepris de recenser les installations de productions d'énergie du territoire, d'élaborer des bilans de production et de consommation d'énergie et d'émissions de gaz à effet de serre, de mettre en place des indicateurs et de diffuser largement l'ensemble de ces données. Depuis janvier 2015, Guyane Energie-Climat développe PRIME, la Plate-forme régionale d'information sur la maîtrise de l'énergie.    En Polynésie, ce soutien au développement de filières d'énergies renouvelables adaptées aux spécificités du territoire s'est concrétisé à travers l'élaboration du plan climat-énergie (PCE) entre mai et septembre 2015, en partenariat technique et financier avec l'Ademe. La définition de ce document de cadrage transversal, centralisant les actions d'adaptation et d'atténuation au changement climatique doit viser en effet une réduction des émissions de gaz à effet de serre, une réduction de la dépendance énergétique du Pays aux énergies fossiles et un développement des énergies renouvelables, ainsi qu'une résilience du Pays face aux catastrophes liées au changement climatique.    Certaines collectivités, comme la Guadeloupe et la Martinique, ont fait le choix de recourir à une « habilitation » en matière d'énergie, comme l'article 73 alinéa 3 de la Constitution les y autorise. Cette disposition leur permet en effet d'édicter leurs propres règles dans un domaine particulier - ici l'énergie - et d'adapter les dispositions et outils élaborés au niveau national en fonction des spécificités et contraintes locales. La Guadeloupe est la première région française à avoir demandé et obtenu une habilitation telle que prévue à l'art. 73 de la Constitution.    Par la délibération n° 2009-269 du 27 mars 2009, le Conseil régional de Guadeloupe a ainsi demandé une habilitation dans le domaine de l'environnement et de l'énergie sur son territoire, habilitation qui lui a été donnée dans la Loi n° 2009-594 du 27 mai 2009 pour le développement économique des outre-mer.    Grâce à cet outil, la région a mis en oeuvre des mesures innovantes en matière d'efficacité énergétique adaptées à son territoire. Elle a notamment voté 16 délibérations toutes publiées au JORF.    Ces textes ne sont qu'une première étape puisque le conseil régional a décidé de s'engager de façon durable sur cette thématique. Par la délibération du 17 décembre 2010, la Région a demandé au Parlement une prolongation de son habilitation dans le domaine afin de poursuivre les actions engagées.    Celle-ci lui a été accordée le 28 juillet 2011 par la Loi n° 2011-884 du 27 juillet 2011 pour une durée de deux ans. Le travail d'élaboration des textes se poursuit.    Pour les prochaines phases de développement, la région Guadeloupe souhaite travailler en partenariat avec ses voisins caribéens, notamment la Martinique qui dispose depuis juillet 2011 de la même habilitation que la Guadeloupe.    Source : Conseil régional de Guadeloupe    La réussite de cette transition énergétique dans les territoires ultramarins grâce au développement des énergies renouvelables et des procédés innovants permettra une forte diminution de l'empreinte carbone française et donc, à terme, une diminution des impacts du changement climatique.    II. RÉALISATIONS ET PROJETS A. Valorisation de l'énergie fatale issue des fumées d'échappement des moteurs de production thermiques par un cycle ORC (Organic Rankine Cycle) à Mayotte Dans le cadre de la transition écologique, la recherche industrielle pour mettre au point des procédés innovants permettant de valoriser l'énergie fatale est au coeur de la question de l'efficacité énergétique. On appelle énergie fatale l'énergie produite au cours d'un processus industriel ou de traitement des déchets qui ne visait pas spécifiquement à la produire et que l'on cherche aujourd'hui à valoriser, comme les collectivités territoriales commencent à le faire via la mise en oeuvre de réseaux de chaleur.    L'Ademe a publié en novembre 2012 une contribution à l'élaboration de la vision énergétique pour la France 2030-2050, qui démontre que l'industrie française, consommatrice d'environ 24 % de l'énergie finale du pays, pourrait réduire sa consommation de 9,2 % à l'horizon 2030. En outre, l'Ademe évalue à 17 % la consommation de combustible dans l'industrie perdue en chaleur fatale à plus de 100° C. A Mayotte, Électricité de Mayotte (EDM) a lancé un projet, dans sa centrale de production de Longoni, de valorisation de l'énergie fatale issue des fumées d'échappement des moteurs de production thermiques grâce au procédé du cycle organique de Rankine ».    Les machines à cycle organique de Rankine permettent de transformer des sources de chaleur basse et moyenne température en électricité. Elles fonctionnent sur les mêmes principes thermodynamiques que les cycles vapeur. Elles emploient un fluide organique qui récupère la chaleur perdue en circulant en cycle fermé, puis passe dans une turbine qui la transforme en électricité.    Pour les sources de chaleur inférieures à 250° C, ce type de procédé présente de nombreux avantages, notamment de rendement et de coût. En effet, à la centrale de Longoni, le projet d'EDM devrait coûter 4 millions d'euros au total, pour la fourniture du procédé, son installation et son raccordement. Les bénéfices attendus sont nombreux : amélioration du rendement global de la centrale de production d'énergie, diminution des consommations de gazole, économies de CSPE (contribution au service public de l'électricité), amélioration du mix énergétique de Mayotte et promotion des énergies renouvelables.    Le projet est actuellement en phase d'étude. Les études de faisabilité fournisseur ont été réalisées. La consultation du public a été lancée. EDM espère une mise en service en septembre 2016.    Ce projet innovant est conçu comme un projet pilote dans la perspective d'être répliqué sur d'autres systèmes électriques, insulaires notamment.    Source : DEAL de Mayotte.   B. SWAC (Sea water air conditioning) du Centre hospitalier de Polynésie française Si l'insularité induit des contraintes lourdes, au premier rang desquelles la difficulté d'approvisionnement énergétique, la dimension maritime constitue cependant un atout et une chance car les océans recèlent de forts potentiels d'innovation, dont le projet SWAC est une parfaite illustration en permettant une climatisation faisant l'économie d'énergies fossiles.    Ce système de climatisation par l'eau froide, puisée en profondeur sous l'océan grâce à un tuyau de 900 mètres, déjà éprouvé dans le secteur hôtelier, doit alimenter en ventilation le centre hospitalier de Polynésie française. Ce projet ambitieux coûtera 3 milliards de XPF (soit plus de 25 millions d'euros), cofinancés par l'Agence française de développement (AFD), la Banque européenne d'investissement (BEI), le Fonds chaleur de l'Agence de l'environnement et de la maîtrise de l'énergie (Ademe) et le Pays.    Source : Creocéan.   Cette technique de climatisation via le SWAC est une particularité de la Polynésie française. La première centrale de climatisation SWAC est en activité à Bora Bora, à l'hôtel Intercontinental depuis 2006. Ce projet est d'autant plus pertinent que la consommation en énergie du centre hospitalier est importante, du fait de ses missions de service public de santé, des contraintes sanitaires et de l'obligation de continuité du service. En 2013, le centre a consommé plus de 26 000 MWH, soit 5 % de la demande totale d'électricité de l'île, dont près de la moitié était dédiée exclusivement à la production de froid.    La mise en place du SWAC a été accompagnée préalablement d'un diagnostic énergétique qui a permis de réduire les besoins en énergie. La fin des travaux est programmée pour 2017.    Ce dispositif innovant pourrait facilement être dupliqué dans le secteur touristique ainsi que dans les collectivités insulaires présentant les mêmes caractéristiques géomorphologiques.    Source : Gouvernement de Polynésie française.   C. Valorisation de la biomasse et de la bagasse en Guadeloupe La canne à sucre constitue la principale culture de la Guadeloupe tant en surface agricole qu'en production. La bagasse, résidu fibreux, issu de cette canne à sucre, est utilisée comme combustible.    La majeure partie du gisement est valorisée en Guadeloupe par la centrale thermique du Moule (CTM) pour la production de vapeur et d'électricité. En effet, en 1992, la société Albioma a mis en service sa première centrale de cogénération à combustion hybride bagasse/charbon sur le site de Bois-Rouge à La Réunion (62 MW). Cette centrale a permis d'apporter à la sucrerie voisine une solution d'approvisionnement énergétique durable, tout en valorisant son sous-produit d'exploitation, la bagasse. Depuis, plusieurs centrales utilisant cette techniques ont vu le jour.    A la centrale du Moule, à Marie-Galante, le projet est très ambitieux puisqu'il vise à abandonner progressivement le charbon au profit d'un approvisionnement à 100 % en biomasse d'ici à 2020. Par ailleurs, la mise en place d'une centrale 100 % biomasse est envisagée pour 2018, combinant une installation de combustion bagasse/granule de bois en fonction de la période (sucrière ou non) avec une turbine vapeur à Marie-Galante. La vapeur produite alimentera la sucrerie adjacente ainsi que le réseau EDF de l'île de Marie-Galante.    Le 18 octobre 2015, un arrêté interministériel redéfinissant les conditions du soutien applicable aux installations produisant de l'électricité à partir de biomasse issue de canne à sucre (ou bagasse) en outre-mer a été publié. Il revalorise le montant de la « prime bagasse », versée pour toutes les centrales existantes produisant de l'électricité à partir du résidu de canne à sucre.    Cet arrêté fixe le montant de cette prime à 14,5 euros la tonne de canne à sucre, contre 13 euros auparavant. Le montant de la prime bagasse était jusqu'alors indexé sur le cours du charbon et le cours du CO2, ainsi que sur le taux de fibre présent dans la canne, avec de plus un coefficient de dégressivité de la prime nécessitant une augmentation du taux de fibre de 3 % par an pour maintenir la rémunération. Le nouvel arrêté supprime l'indexation de la prime sur le cours du charbon et du CO2 qui, dans son principe, ne faisait que maintenir la dépendance des systèmes électriques insulaires à la fluctuation du cours des énergies fossiles. Le coefficient de dégressivité est neutralisé jusqu'à 2017 inclus. Par ailleurs il est divisé par 5, ramenant l'augmentation du taux de fibre nécessaire pour maintenir la rémunération à un niveau plus réaliste de 0,6 % par an.    La revalorisation permettra de développer les énergies renouvelables et de conforter la filière canne-sucre-rhum-bagasse dans les outre-mer, et particulièrement en Guadeloupe.    Source : DEAL de Guadeloupe.   D. Le projet d'énergie thermique de la mer en Martinique L'énergie thermique des mers (ETM) consiste en l'exploitation de la différence de température entre les eaux de surface et les eaux profondes des océans. Dans les outre-mer situés en zone intertropicale où le gradient de température est important, cette technique est particulièrement pertinente.    Contrairement aux autres énergies renouvelables, l'ETM fournit une énergie de base, stable, garantie, et permet une production d'électricité non intermittente, renouvelable et décarbonée, sans aucune incertitude sur la ressource.    En Martinique, le projet NEMO (New Energy for Martinique and Overseas), est une centrale flottante de production d'énergie thermique des mers. Elle produit un courant électrique d'une puissance de 16 MW à partir de la différence de température d'eau de mer pompée à 1 km de profondeur à 4° C et en surface à 28° C.    Ce projet pilote s'appuie sur le prototype à terre mis en oeuvre par DCNS depuis 2010 à La Réunion. Le projet NEMO est une plateforme flottante qui constitue une modalité offshore du développement de la technologie ETM. Cette modalité permettra de viser les sites isolés densément peuplés, nécessitant une production d'électricité dans des zones où le coût de l'énergie fossile est élevé et apportera donc une alternative intéressante aux centrales thermiques pour la production d'électricité de base dans les zones insulaires tropicales.    Ce projet est en outre soutenu par un partenariat 100 % français associant le développeur de projet Akuo Energy, l'industriel DCNS et le fournisseur de technologie ETM.    Le début des travaux est prévu en 2016 pour une mise en service à l'horizon 2018-2019. La réussite d'un tel projet pourrait déboucher sur le développement de plusieurs centrales industrielles de puissance supérieure (20 MW), pouvant être déployées dans la zone intertropicale, notamment dans l'arc antillais.    Le projet NEMO pose le premier jalon d'une filière industrielle française dont les perspectives commerciales internationales à l'export sont extrêmement prometteuses. Il s'agit d'une première mondiale à cette échelle sur la base d'un concept déjà éprouvé pour de petites productions. L'innovation technologique porte notamment sur l'adaptation des dispositifs nécessités par le pompage de grandes quantités d'eau à grande profondeur.    Source : DEAL de la Martinique.   Chapitre V La prévention des risques et la protection du littoral I. LA PRÉVENTION DES RISQUES DANS LES OUTRE-MER : UNE QUESTION DE SURVIE A. Les outre-mer en première ligne face aux nouveaux risques climatiques Les outre-mer français, quelle que soit leur situation géographique et océanique, sont de manière globale particulièrement vulnérables à l'augmentation du niveau des mers et à l'intensification des événements climatiques extrêmes. Le cinquième rapport du Groupe d'experts intergouvernemental sur l'évolution du climat (GIEC), paru en 2014, met en évidence de manière alarmante le développement de ces risques climatiques et leur multiplication à venir.    Ainsi, alors qu'en 2007 on évoquait une élévation du niveau de la mer de 18 à 59 cm d'ici 2100, les dernières études prévoient une hausse probable de 26 à 82 cm. Les deux causes les plus importantes de cette montée de la mer qui semble s'accélérer ces deux dernières décennies sont l'expansion thermique, c'est-à-dire la dilatation de l'eau liée au réchauffement, et la fonte accélérée des glaciers. Le niveau moyen mondial de la mer s'est déjà élevé de 19 cm au cours de la période 1901-2010.    Cette hausse du niveau des mers va notamment accroître les risques d'inondation dans les régions de basse altitude et les régions côtières. L'effet conjugué de l'aggravation des inondations côtières, de la salinisation et de l'érosion sous l'effet du relèvement du niveau des mers et des activités humaines pourrait à terme contaminer et réduire les superficies agricoles productives et, en conséquence, menacer la sécurité alimentaire des ménages et des communautés.    1) La multiplication d'événements climatiques extrêmes Les risques liés au changement climatique se manifestent aussi dans les événements climatiques extrêmes. La multiplication de ces événements, comme les vagues de chaleur, les tempêtes, les inondations et sécheresses, aura un impact croissant sur les outre-mer.    L'exemple du phénomène El Niño est symptomatique. El Niño est un phénomène climatique récurrent, qui se caractérise par des températures de l'eau anormalement élevées, dans l'Est de l'océan Pacifique Sud, qui peut avoir pour conséquence de dérouter des cyclones tropicaux de leurs routes habituelles, de déplacer les zones de précipitations et de sécheresse mais aussi, localement, de changer le niveau de la mer par le changement de la pression moyenne.    Les années El Niño se traduisent généralement par des sécheresses en Nouvelle-Calédonie et par un risque cyclonique accru en Polynésie française. Depuis les années 1980, dans le Pacifique tropical, le réseau TAO/TRITON, qui comprend 70 bouées de mesures, fournit des données océanographiques et météorologiques qui permettent de surveiller l'océan en temps réel et de mieux prévoir le phénomène El Niño. Pour autant, ce réseau de mesures est aujourd'hui mal entretenu. Au vu de l'accentuation du phénomène El Niño par le changement climatique, la communauté internationale a lancé le projet Tropical pacific observing system for 2020 (TPOS2020). Ce projet inclut la France par le biais de l'Institut de recherche pour le développement (IRD), mais également plusieurs partenaires de différents pays concernés par la problématique.    L'objectif est de mettre à jour le réseau TAO/TRITON en l'équipant des nouvelles techniques : altimétrie spatiale (mesures de hauteurs de mer par satellite), flotteurs Argos (flotteurs autonomes fournissant des profils de températures et de salinité de la surface à 2 000 mètres de profondeur), etc.    Ce type d'initiatives va devenir essentiel pour mieux comprendre la variabilité climatique, l'anticiper et ainsi mieux prévenir les risques naturels associés.    2) Une vulnérabilité plus grande aux risques littoraux Le plus fréquemment, plusieurs risques se combinent, le relief provoquant souvent un effet d'enchaînement des processus. Le passage d'un cyclone peut par exemple avoir pour effet d'inonder les plaines du fait de crues, de provoquer la submersion des zones basses et de créer un pic d'érosion côtière et des glissements de terrain.    La forte littoralisation de l'habitat et des activités économiques dans les outre-mer constitue un facteur de risque aggravant : l'aménagement des littoraux a à la fois pour effet de perturber leur fonctionnement et d'accroître leur sensibilité à l'érosion et à la submersion et de concentrer les habitations et les activités au même endroit, les rendant par là-même plus vulnérables en cas de catastrophe.    De manière générale, les outre-mer présentent une vulnérabilité plus grande aux risques littoraux. Ces risques comprennent l'érosion côtière, les submersions, les inondations, les mouvements de terrain, ou encore la salinisation des sols et des nappes phréatiques. Ils sont directement liés au changement climatique dans la mesure où ils résultent, dans la plupart des cas, du passage de tempêtes, à la fois cycloniques donc tropicales, et issues de la zone tempérée.    3) L'apparition de risques émergents Enfin, le changement climatique se manifeste dans les outre-mer par l'émergence de nouveaux risques, en particulier sanitaires. Le rapport de 2012 de l'Observatoire sur les effets du réchauffement climatique (ONERC) l'a directement mis en évidence.    Le changement climatique a une influence sur la santé des populations ultramarines. Il favorise de manière directe le développement des pathologies liées aux vagues de chaleur et aux épisodes caniculaires intenses, comme celles liées à la hausse du rayonnement solaire.    L'augmentation des températures et l'évolution du régime des précipitations, que celles-ci augmentent en volume ou se répartissent désormais différemment dans l'année, ont une influence sur la multiplication des insectes qui sont d'importants vecteurs de maladie dans la zone intertropicale. Là où ces vecteurs se multiplient, les maladies vectorielles se développent aussi : paludisme, dengue, chikungunya, fièvre du Nil occidental, filariose, etc.    Le changement climatique aura également des effets sanitaires plus indirects. L'évolution globale des ressources alimentaires, en quantité et en qualité, aura une influence sur l'évolution des maladies digestives et alimentaires, comme la ciguatera. Or, l'évolution de ces ressources est directement liée à l'évolution du climat. Une hausse des températures ou une diminution des précipitations se traduisent par une diminution de la quantité d'eau disponible, ainsi que par des effets néfastes sur les stocks en poissons notamment. Dans le cas des maladies vectorielles et digestives, l'influence du changement climatique sur la santé est indirecte : c'est en agissant sur le développement des vecteurs comme les insectes ou des ressources, comme l'eau et le poisson, qu'il peut engendrer une dégradation de la santé humaine.    Le changement climatique facilite donc le développement de ces maladies, sans en être la cause initiale, que ces maladies soient directement provoquées ou simplement aggravées par l'évolution du climat.    Face à ces évolutions prévisibles, l'enjeu de la connaissance et de la mesure des risques est essentiel. De nombreux territoires ultramarins l'ont compris, La Réunion en particulier.    Les maladies infectieuses sont une menace avérée dans la région du sud-ouest de l'océan Indien. La Réunion et les îles environnantes ont connu de lourds épisodes de maladies à transmission vectorielle. Le chikungunya y a émergé, avec une incidence très forte, la dengue est une menace permanente et l'arrivée du virus zika est à craindre. Les hausses de températures directement liées au changement climatique accentuent les invasions biologiques dans les territoires insulaires à endémisme élevé et fragile. Le risque d'émergence ou de réémergence de maladies infectieuses à transmission vectorielle s'en trouve augmenté d'autant.    Face à ces menaces, il est plus que nécessaire de développer des stratégies efficaces et innovantes, tout en essayant de réduire l'usage de produits biocides aux effets néfastes pour l'environnement. C'est dans cette optique que l'Institut de Recherche pour le Développement (IRD) de La Réunion mène des recherches pour développer de nouvelles solutions et réduire le risque sanitaire lié aux vecteurs et aux réservoirs d'animaux sauvages. L'IRD poursuit par ailleurs, sur le long terme, le relevé d'indicateurs pertinents, notamment sur l'évolution des populations vectorielles.    B. La nécessité d'une nouvelle approche préventive intégrée Face à la multiplication des risques auxquels les territoires ultramarins doivent d'ores et déjà faire face et auxquels ils seront de plus en plus exposés à l'avenir du fait du changement climatique, il est nécessaire de mettre davantage l'accent sur la prévention. Compte tenu des spécificités géographiques des outre-mer, afin que ces politiques de prévention soient efficaces, il convient d'envisager le changement climatique de manière globale et dynamique.    1) Prévention et gestion des crises Le premier aspect à prendre en considération dans la mise en place d'une politique adaptée aux évolutions climatiques à venir est celui de la prévention et de la gestion des crises.    La connaissance et la prévision des événements, ainsi que la mise en place de systèmes d'alerte performants vont devenir de plus en plus cruciales pour sauver des vies humaines et réduire considérablement le coût économique des catastrophes climatiques à venir. Au niveau international, la question de la diffusion efficace des alertes est une des problématiques rencontrées par les Etats insulaires : seuls cinq des quarante Etats insulaires, tous océans confondus, disposent d'un système performant. Or, l'alerte est à la base de l'adaptation. La France et plus particulièrement les territoires ultramarins ont ici un rôle à jouer.    La France, en coordination avec la Norvège, le bureau des Nations Unies pour la réduction des catastrophes (UNISDR), l'Organisation mondiale de la météorologie et la Banque mondiale, a lancé la coalition CREWS : Climate Risk Early Warning Systems. L'objectif est de lever des fonds pour augmenter de manière significative la capacité des systèmes d'alerte à générer et communiquer des alertes précoces et efficaces sur les risques d'événements météorologiques et climatiques dans les pays ne disposant pas de tels systèmes.    Cette initiative se traduit par une intensification de la coopération régionale afin de renforcer les services météorologiques locaux et de mettre à disposition l'information. Les services et établissements français présents en Guyane, en Martinique, en Guadeloupe, à La Réunion et en Nouvelle-Calédonie sont au coeur du dispositif.    D'ici à 2020, l'objectif est de mobiliser au total 100 millions de dollars afin de parvenir à une couverture globale des populations qui seront exposées aux événements climatiques extrêmes.    Au-delà de l'alerte, le défi à relever par les territoires ultramarins est celui de la gestion des crises. Là aussi, des dispositifs existent d'ores et déjà. Citons l'exemple de La Réunion.    L'océan Indien est actuellement la troisième région du monde la plus touchée par les catastrophes naturelles, avec tout ce que cela implique de conséquences en termes humains et économiques. Face à ce constat, il est apparu pertinent, pour les pays et territoires présents dans la région, de mettre en place des outils de gestion globale et concertée, afin de réduire les impacts du changement climatique.    La Commission de l'Océan Indien (COI) a lancé en 2011 un programme régional sur le risque naturel. Il implique les Comores, Madagascar, Maurice, les Seychelles et La Réunion. Il développe une approche intégrée de la prévention et la préparation jusqu'à la gestion de la crise et la réparation. Ce programme se traduit par des actions visant à améliorer la connaissance des risques et à renforcer les capacités de veille, d'alerte et de secours des différents acteurs : mise en place d'une plateforme d'échanges d'informations, d'un programme de formations, de concepts communs de gestion de crise. Le programme prévoit également de tester et de développer des solutions très concrètes de reconstruction et de réhabilitation rapides, comme des maisons d'urgence à faible coût ou encore la constitution de stocks.    2) Aménagement des littoraux et gestion des côtes En matière de prévention des risques, ainsi qu'il a été exposé précédemment, l'aménagement des littoraux constitue la plupart du temps un facteur aggravant des risques pesant sur les territoires ultramarins. L'élévation du niveau de la mer, l'augmentation des phénomènes météorologiques extrêmes, les hausses de températures de l'air et de l'eau et l'acidification des océans devraient avoir un impact majeur sur les formations coralliennes et les écosystèmes associés ainsi que sur les ouvrages et aménagements présents sur le littoral : ces ouvrages risquent d'être fortement dégradés par la multiplication des événements liés au changement climatique, et constituent un risque supplémentaire, notamment de pollution, pour les zones marines associées.    Sur la base de ce constat, l'IFRECOR (Initiative française pour les récifs coralliens) a décidé, dans un objectif de prévention, d'élaborer et de diffuser un guide méthodologique devant servir d'aide à la décision pour les décideurs et les porteurs de projets. Ce guide analyse, pour chaque type d'aménagement, les impacts envisageables et les effets probables sur les zones marines sensibles proches du littoral, en particulier les récifs coralliens. Le guide intègre également des recommandations techniques en matière d'aménagement du littoral, avec une typologie des ouvrages susceptibles d'être affectés en fonction des aléas climatiques pris en compte.    Ce type d'outils d'aide à la décision est précieux pour les territoires d'outre-mer, et sera largement diffusé à l'occasion de la 21e conférence des parties à la convention des Nations Unies sur le changement climatique à Paris en décembre 2015. Ce guide de l'IFRECOR pourrait par ailleurs utilement être diffusé à l'ensemble des outre-mer tropicaux possédant des récifs coralliens et des écosystèmes associés.    Au-delà du sujet de l'aménagement des littoraux, la question de gestion du trait de côte est centrale pour améliorer la prévention des risques dans les outre-mer. Le phénomène d'érosion côtière va connaître une accélération dans les années à venir. Pour anticiper l'évolution du littoral côtier, il est nécessaire de prévoir sur le long terme les conséquences des choix relatifs aux usages du littoral.    Le gouvernement a adopté en 2012 une stratégie nationale de gestion intégrée du trait de côte, sur la base d'une responsabilité partagée entre l'État et les collectivités territoriales. Sa mise en oeuvre se décline avec la mise en place d'un réseau d'observation de l'évolution du trait de côte, l'élaboration de stratégies locales concertées et la préparation, le cas échéant, de la relocalisation de certaines activités et de certains biens.    Un appel à projets a permis de désigner cinq sites expérimentaux dont celui de Petit-Bourg, en Guadeloupe. L'objectif est d'étudier, de manière concrète, les moyens de mieux occuper et de mieux aménager le littoral, de l'adapter aux risques naturels croissants tout en le préservant. L'expérimentation de Petit-Bourg doit permettre d'élaborer un guide méthodologique national à destination de toutes les collectivités littorales. Des propositions ont d'ores et déjà été émises, comme la création d'un système d'allocation d'aide financière, l'identification, ou si nécessaire la création, d'un opérateur dédié à la gestion des zones soumises aux risques littoraux ou encore la création d'un schéma de planification spécifique.    3) Des politiques d'adaptation face à l'émergence des nouveaux risques liés au changement climatique La prévention des risques implique, pour les collectivités, de mettre en place des dispositifs d'adaptation afin de réduire ces risques et surtout la vulnérabilité des territoires en cas de catastrophe.    En Guadeloupe, où les changements climatiques sont déjà largement perceptibles, le territoire est particulièrement vulnérable du fait de son caractère insulaire et de son exposition aux événements climatiques extrêmes. Les collectivités guadeloupéennes sont confrontées à l'élévation du niveau de la mer, qui pourrait atteindre 80 centimètres en 2100, et à l'évolution du trait de côte qui empiète petit à petit sur leur littoral. La température moyenne pourrait par ailleurs augmenter de 4° C d'ici à 2100, avec les conséquences que cela implique en termes de durée des périodes de sécheresse et de recrudescence des épisodes de très fortes précipitations durant la saison des pluies.    Face à ce constat, le Conseil régional de Guadeloupe, en partenariat avec l'Ademe, ont engagé une démarche d'adaptation des collectivités au changement climatique. Ils travaillent actuellement au développement d'un outil, à destination des collectivités territoriales, pour les accompagner dans la définition d'une stratégie d'adaptation après une phase de diagnostic de leur vulnérabilité aux risques. Depuis 2014, deux appels à projets ont été lancés afin de mettre en oeuvre ce nouvel outil d'aide à la décision. Cinq candidatures ont été retenues : la communauté d'agglomération Nord Grande-Terre et les communes de Petit-Bourg, Baie-Mahault, Pointe-à-Pitre et Sainte-Rose.    D'autres projets encourageant la mise en place de politiques d'adaptation sont mis en oeuvre dans les autres territoires.    A Saint-Pierre-et-Miquelon, la destruction en janvier 2009 de la route de Miquelon-Langlade par une forte tempête a nécessité d'importants travaux de confortement par enrochement et a conduit à la mise en place d'outils de mesure et de modélisation des phénomènes marins et de leur impact sur l'érosion littorale, notamment l'installation d'un houlographe, d'un marégraphe, de deux courantomètres ainsi qu'une antenne GNSS (Global Navigation Satellite System) pour mesurer les mouvements de la croûte terrestre. L'archipel étant très exposé aux risques d'érosion et de submersion, une coopération scientifique avec le Canada a été mise en place permettant le recours aux puissants outils de modélisation des changements climatiques développés par le Québec sur le Saint-Laurent. Par ailleurs, une collaboration avec le Centre d'études et d'expertise sur les risques, l'environnement, la mobilité et l'aménagement (CEREMA) a conduit en 2011 à la définition d'un protocole de suivi de l'évolution du trait de côte, les premiers levers topographiques par GPS des zones basses et vulnérables ayant été effectués en 2013. Une étude de vulnérabilité tendant à définir une stratégie d'adaptation a été confiée au BRGM et la cartographie des aléas devrait permettre d'ici début 2018 l'élaboration du plan de prévention des risques littoraux.    En Nouvelle-Calédonie et en Polynésie française, depuis 2014, une initiative vise à promouvoir une politique d'adaptation centrée sur les écosystèmes et sur une gestion intégrée des zones côtières.    Les écosystèmes insulaires sont fortement fragilisés et détériorés par les effets du changement climatique, et plus généralement par certaines pratiques anthropiques. Pour que ces écosystèmes continuent de fournir aux populations des services éco-systémiques dont elles dépendent pour leur activité ou leur survie, notamment la pêche, l'agriculture et l'approvisionnement en eau potable, il convient de les restaurer, de les maintenir et de les renforcer pour améliorer leur résilience face aux risques.    C'est l'objet de l'initiative RESCCUE (restauration des services écosystémiques et adaptation au changement climatique), portée par la Communauté du Pacifique Sud (CPS). Elle vise à soutenir les pays et territoires océaniens dans la mise en oeuvre de la gestion intégrée des zones côtières, avec des sites pilotes à Fidji, en Nouvelle-Calédonie, en Polynésie française et au Vanuatu. Le projet doit permettre de maintenir et améliorer la fonctionnalité des écosystèmes et de mettre en place des mécanismes économiques et financiers contribuant à la pérennité des services écosystémiques. A terme, cette initiative doit se traduire par un renforcement de la gestion intégrée des zones côtières, « de la crête au tombant », par le déploiement d'actions concrètes : accompagnement des pratiques agricoles et notamment perlicoles durables afin de lutter contre l'érosion, réhabilitation de la biodiversité terrestre, ou encore gestion durable des déchets. Ce projet est d'autant plus intéressant qu'il doit permettre la capitalisation et la diffusion des résultats dans le Pacifique.    La prévention des risques liés au changement climatique et la mise en place de politiques d'adaptation ambitieuses dans les outre-mer doivent constituer une priorité à court et moyen termes. Le coût de l'inaction, ou d'une action repoussée dans le temps, sera très important. Les années à venir d'ici à 2050 sont véritablement une période de responsabilité climatique pour les collectivités et les territoires.    II. RÉALISATIONS ET PROJETS A. Cartographie du risque de submersion marine consécutive au passage d'un cyclone à Mayotte Afin de mieux prévenir les risques liés aux événements climatiques extrêmes, comme les cyclones, la DEAL de Mayotte a demandé en 2012 une étude au BRGM en vue d'établir une cartographie du risque de submersion consécutif au passage d'un cyclone.    L'objectif est de définir une démarche méthodologique présentant l'avantage de combiner les différents paramètres générateurs des submersions marines et de définir les valeurs de référence correspondantes.    L'étude, qui doit être livrée en mars 2016, comprend deux phases distinctes :    - une première phase de définition des valeurs de référence de la surcote et des vagues cycloniques, par façade, dans le lagon de Mayotte, au moyen d'un couplage de modèles numériques (vent, niveau d'eau, vague), en collaboration avec Météo France ;    - une seconde phase ayant pour objectif d'utiliser ces valeurs de référence pour la cartographie des submersions marines d'origine cyclonique en ayant recours à une modélisation numérique de submersion permettant le calcul du niveau extrême atteint par la mer sur la côte, suite au déferlement des vagues et en prenant en compte la montée du niveau de la mer liée au changement climatique.    Le travail réalisé dans le cadre de la phase 1 a permis de sélectionner puis de modéliser les vingt cyclones historiques les plus impactants de ces trois dernières décennies sur le littoral mahorais. Trois cyclones de synthèse ont également fait l'objet d'une modélisation.    L'analyse de ces travaux fait apparaître les comportements très hétérogènes des différentes façades de l'île vis-à-vis des forçages maritimes. Les façades ouest et nord sont fortement exposées aux houles du large, qui pénètrent dans le lagon au travers des larges passes situées à l'ouest du récif barrière. Ce milieu relativement ouvert préserve, en revanche, ces façades littorales des effets de surcote marine. A contrario, les façades est et sud sont bien abritées des houles mais plus exposées aux effets des surcotes.    La phase 2 du projet, en cours, a pour objectif de transformer cette base de connaissance en une information cartographiée de la submersion marine des zones littorales mahoraires en vue de son intégration dans les plans de prévention des risques naturels.    Source : DEAL Mayotte.   B. Création de l'Observatoire de la dynamique côtière et colloque international sur les écosystèmes côtiers en Guyane Afin de prévenir les risques naturels d'érosion et de submersion du littoral, la DEAL de Cayenne a lancé, le 10 avril 2015, un observatoire de la dynamique côtière de Guyane.    Le constat est effectivement sans appel : le littoral guyanais fait partie des côtes les plus instables au monde. Il est soumis à des phénomènes d'érosion très importants sous l'influence du passage successif de bancs de vase de l'embouchure du fleuve Amazone vers l'océan Atlantique. La côte guyanaise est en situation d'évolution constante. Les fluctuations du trait de côte peuvent atteindre jusqu'à 6 kilomètres en 50 ans dans certains secteurs, avec des variations annuelles comprises entre 100 et 300 mètres. Ces contraintes vont certainement s'accentuer dans l'avenir du fait du changement climatique et de l'augmentation du niveau moyen des mers.    La Guyane connaît par ailleurs une tendance à la concentration des activités humaines sur le littoral. Les problématiques liées à l'érosion des côtes sont donc amenées à devenir de plus en plus centrales (planification et aménagement du territoire, mise en sécurité des biens et des personnes, fonctionnalité des structures portuaires, amélioration des connaissances liées à la recherche et à la protection des milieux, gestion et conservation des espaces naturels).    Dans ce contexte, l'objectif de l'observatoire de la dynamique côtière est de comprendre, connaître et anticiper les évolutions littorales et leurs conséquences, d'appuyer les politiques publiques en matière de gestion du littoral en tenant compte du changement climatique et de l'élévation du niveau de la mer, et enfin de constituer une plateforme commune de partage et d'expertise pour tous les acteurs impliqués.    Signe que la problématique de la prévention de l'érosion des littoraux est aujourd'hui centrale en Guyane et que tous les acteurs sont mobilisés sur cette question, le groupement de recherche GDR LIGA (littoral de Guyane sous influence amazonienne) organisera du 24 au 28 novembre 2015 un colloque sur les écosystèmes côtiers.    Ce colloque, qui s'adresse aux scientifiques spécialisés dans ce domaine mais aussi aux gestionnaires et aux usagers des espaces littoraux guyanais, doit permettre de faire le point autour de cinq grandes thématiques :    - eaux côtières et estuariennes ;    - vasières et mangroves ;    - réseaux trophiques, contaminants et espèces bio-indicatrices ;    - vulnérabilité du littoral guyanai ;    - méthodes, outils et modèles.    Source : DEAL Guyane.   C. Wallis-et-Futuna : le projet LITTOFORT Le projet LITTOFORT s'inscrit dans la problématique de lutte contre les changements climatiques et de développement durable en s'intéressant particulièrement à la zone littorale.    Ce choix a été fait en raison du phénomène de littoralisation croissant des populations et des activités économiques et sociales observé au cours des dernières années. Cette zone de contact et d'interaction entre la mer et la terre concentre alors une accumulation d'activités diverses et concurrentielles, de populations, de besoins mais aussi de risques. Il y existe à la fois une concurrence pour l'occupation du sol et une recherche nécessaire de préservation des espaces naturels qui concourent à l'attractivité touristique et à l'économie qu'ils représentent. C'est d'autant plus vrai pour les milieux insulaires dans le Pacifique où les populations dépendent directement des activités sur le littoral. La nécessité de conserver l'intégrité des côtes face aux agressions naturelles et anthropiques est donc une évidence pour ces territoires.    Les facteurs de dégradations possibles de la zone littorale sont nombreux. Le réchauffement climatique a pour conséquence l'élévation du niveau de la mer, qui provoque une érosion des côtes. Les effets du réchauffement se conjuguent par ailleurs avec des phénomènes climatiques extrêmes tels que les ouragans ou encore les cyclones, qui se produisent régulièrement dans les zones tropicales et subtropicale et qui fragilisent d'autant plus les côtes.    Dans ce contexte, l'objet du projet LITTOFORT concerne la protection et la réhabilitation du littoral, adaptées à la zone Pacifique. Il s'appuie sur une méthode originale, fondée sur la formation d'un dépôt de consolidation du littoral, par application d'un faible courant électrique à travers une armature métallique plongée dans l'eau de mer. Ce dépôt minéral, ou « dépôt calcomagnésien », réalisé à partir des éléments calcium et magnésium présents naturellement dans l'eau de mer, sert alors de mortier en englobant le sable, coquillages et autres débris à proximité de l'armature, réalisant ainsi un béton naturel. Ce système doit permettre de limiter l'érosion de berges fragilisées ou encore de renforcer des digues existantes.    L'application d'un tel procédé présente de nombreux intérêts. Tout en étant peu onéreux et simple à mettre en place, et donc utilisable par des structures aux moyens limités, il est respectueux de l'environnement, car il ne met en jeu que des éléments présents dans l'eau de mer.    Source : Ministère des affaires étrangères, Fonds de coopération économique, sociale et culturelle pour le Pacifique.   Chapitre VI La sensibilisation et l'éducation du public I. LA CONNAISSANCE ET L'INFORMATION, PRÉMISSES INCONTOURNABLES À L'ACTION FACE AUX IMPACTS DU CHANGEMENT CLIMATIQUE A. Connaître pour mieux éduquer « La biodiversité, c'est nous et tout ce qui vit. Nous sommes tributaires de tout le vivant. Nous sommes, au quotidien, dépendants des autres espèces pour notre alimentation, nos vêtements. Nous le sommes pour notre respiration, car l'oxygène est une production des plantes vertes. La biodiversité est notre assurance-vie. Il est donc très important de bien la connaître. Et le premier constat est que le niveau de connaissance de la biodiversité est insuffisant. Mais cette diversité du vivant ne se résume pas à des listes d'espèces : il s'agit aussi du fonctionnement des écosystèmes, car les espèces ne peuvent exister que dans un milieu qui leur est propice ; elles ne sont pas indépendantes les unes des autres, et ne le sont pas non plus les écosystèmes dans lesquels chacune agit. Un arbre est un monde souterrain avec toute une cour de serviteurs microscopiques autour de ses racines pour l'aider à capter les éléments minéraux du sol ; c'est un monde aérien avec ses mousses, ses lichens, ses oiseaux, ses insectes ; et il y a l'arbre lui-même, un géant issu d'une toute petite graine... à condition qu'il y ait un sol où elle puisse s'installer. »    Ces mots d'Hubert Reeves d'avril 2015 soulignent ce que doit être l'orientation première de toute politique en matière de biodiversité et de protection des espèces et des habitats : la connaissance. Ce n'est qu'en connaissant la diversité de la biodiversité que l'on pourra, au mieux, protéger les espèces et les espaces qui constituent notre richesse, et agir en atténuant les impacts du changement climatique sur les milieux.    Si les Français ont maintenant une meilleure connaissance de ce qu'est la biodiversité - en effet, d'après une étude du Centre de recherche pour l'étude et l'observation des conditions de vie (44), deux tiers d'entre eux déclarent ainsi savoir ce qu'est la biodiversité - cette dernière, en ce qu'elle est essentielle et le préalable à toute forme d'action, est encore insuffisante, en particulier dans les territoires ultramarins, qui sont les plus exposés aux impacts du changement climatique, et qui présentent le plus grand nombre de spécificités. A titre d'exemple, il est considéré à ce jour que la majorité des espèces marines comme leurs interactions sont encore méconnues : environ 250 000 espèces marines sont actuellement répertoriées avec un champ de valorisation potentielle largement sous-évalué.    La même étude fait d'ailleurs ressortir que nos concitoyens placent de plus en plus les questions de perte de biodiversité parmi les problèmes de dégradation de l'environnement les plus préoccupants. On sait aujourd'hui que la connaissance en matière de biodiversité est largement insuffisante.    La question de la connaissance de la biodiversité comme préalable à toute forme d'action en matière de protection de l'environnement a été soulignée lors des tables rondes organisées au Sénat le 11 juin 2015 « Biodiversités des outre-mer et changement climatique ». Pascale Joannot, déléguée à l'outre-mer du Muséum national d'histoire naturelle, rappelait en ce sens que « les questions de la biodiversité outre-mer doivent faire l'objet d'approches spécifiques et valorisantes pour les collectivités d'outre-mer », mobilisant une somme de connaissances importantes. Elle a également souligné que cette connaissance était, dans ces territoires, le résultat du travail des naturalistes et des écologues, mais aussi, avant eux, des populations locales qui ont écouté la nature, appris et transmis les savoirs locaux, et, enfin, du grand public et du monde associatif.    Cet apport des sciences collaboratives a d'ailleurs également été mis en avant dans un rapport remis à la ministre de l'écologie en janvier 2012 (45), Gilles Boeuf, Yves-Marie Allain et Michel Bouvier ont mis en lumière l'apport des sciences participatives dans la connaissance de la biodiversité.    A compter de la seconde moitié du XIXe siècle, en France, les " sociétés savantes" se développent et face à la diversité des mondes naturels, vivant et inerte, les niveaux des connaissances progressent à une vitesse exponentielle, et rapidement elles se spécialisent dans un domaine naturaliste plus ou moins spécifique : botanique, entomologie, ornithologie, géologie, minéralogie, paléontologie... Durant cette même période, dans les grandes villes françaises sont créés des Muséums régionaux d'Histoire naturelle, et se développent des sorties naturalistes dans toutes les provinces.    Les publications locales ou régionales, - botanique, entomologie, géologie... - sont révélatrices du travail d'inventaire effectué par ces réseaux, y compris sur des groupes assez délaissés actuellement comme les lichens, les mousses (bryophytes), les fougères (ptéridophytes) ou encore les "algues" marines... Les réseaux constitués sont bien identifiés avec des réunions à caractère scientifique assez fréquentes, hebdomadaires ou mensuelles. Ils font avancer la connaissance dans de très nombreuses régions qui ne possèdent pas encore d'Université. Sur le reste du territoire, le réseau, dense, des instituteurs, des ecclésiastiques, des médecins ruraux, des vétérinaires de campagne... va être également l'un des facteurs de diffusion des sciences, de la formation à l'observation naturaliste, de l'encadrement des collecteurs bénévoles, de la collecte de ces informations, des vérifications du bien-fondé des inventaires et de leur publication.    Malgré cette abondance de prospecteurs, une très grande partie des observations et données récoltées ont disparu. Souvent inscrits sur des carnets plus ou moins lisibles pour une tierce personne, ces relevés plus ou moins complets, avec un cadre méthodologique souvent personnel, non reproductible, pas toujours fiable, sont bien souvent très partiels et difficilement exploitables. Dans les dernières décennies du XXe siècle, pour des raisons diverses, les associations naturalistes "académiques" ont perdu de leur attrait, les sciences de l'observation ont quelque peu périclité, étant considérées comme peu novatrices, trop descriptives, voire sans intérêt par les commissions d'évaluation des "sciences de la vie" et par certains comités éditoriaux de grandes revues scientifiques.    A partir des années 1960, le travail de fond effectué par de nombreuses associations de défense de la nature a permis d'enrayer ce déclin, de renverser la tendance et de recréer des réseaux de naturalistes actifs, effectuant comptages, recensements, observations surtout chez les botanistes, ornithologues et entomologistes. Si les techniques de relevés des observations n'ont guère évolué sur le terrain, la banalisation des outils modernes de communication dont internet, va profondément modifier les possibilités de diffusion des données et leur mise à disposition pour tous. Devant cette facilité technique, avec des outils informatiques plus conviviaux, une rigueur méthodologique certaine va s'avérer nécessaire pour tous les acteurs, dont les "amateurs" (souvent "très éclairés"), avec le respect des protocoles d'observation et de récolte des données tant dans les objets observés que dans la manière de les traiter afin de pouvoir les exploiter et en tirer des conclusions les plus pertinentes et étayées. Pour illustrer le lien entre l'histoire des simples inventaires et une reprise actuelle, autorisant à proposer des hypothèses sur la dynamique des populations, rappelons l'exemple de la traversée des Pyrénées (sur mandat napoléonien) et l'herbier d'A. de Candolle, qui ont donné lieu à une répétition comparative 200 ans plus tard, à l'initiative d'une association (Terranoos) et à la parution d'un bel ouvrage.    Des articles récents (2010) dans des revues de vulgarisation comme Terre sauvage (France) ou la Revue durable (Suisse) communiquent sur les sciences participatives, notamment sur les principales actions pilotées par le Muséum national d'Histoire naturelle. Par ses circulaires, le ministère en charge de l'écologie, incite également les services déconcentrés à s'appuyer sur des démarches participatives pour la mise en oeuvre de certaines politiques dont celle de la stratégie nationale de création d'espaces protégés, voire de la stratégie nationale pour la biodiversité. En outre, deux colloques sur les réseaux collaboratifs et les sciences citoyennes ont été organisés à Montpellier en 2009, le premier "Réseaux collaboratifs et sciences citoyennes" sous l'égide de l'association Tela botanica, le second "Le réveil du dodo" lors des Journées francophones de la biologie de la conservation.    Ce phénomène de participation du public n'est pas propre à la France. A l'étranger des expériences, dont celles de la Grande-Bretagne, ou celles des "Citizen Sciences" nord-américaines créées en 1970, montrent la très forte implication du public, avec des taux de participation qui peuvent laisser pantois par rapport aux dispositifs français. Aux Etats-Unis, le Christmas Bird Count, créé en 1900 et piloté par la Société Audubon est régulièrement reconduit depuis 1900 Mathieu, 2011) : 10 000 "observateurs participatifs" ont comptabilisé 63 millions d'oiseaux dans le cadre d'un programme qui a souligné, sur une quarantaine d'années, une régression de 68 % de 20 espèces communes. Le programme français du suivi temporel des oiseaux communs (STOC) en est l'équivalent depuis 1989. »    C'est également dans cette perspective que le projet de loi pour la biodiversité, actuellement en cours d'examen devant le Sénat, consacre dans son article 2 la connaissance de la biodiversité en en faisant une action d'intérêt général, au II de l'Article L. 110-1 du code de l'environnement, au même titre que la protection, la restauration, la mise en valeur, la remise en état, la gestion des espaces, ressources et milieux naturels, de même que « la préservation de leur capacité à évoluer et la sauvegarde des services qu'ils fournissent ».    B. Sensibiliser pour mieux protéger L'approfondissement de la connaissance sur les effets du changement climatique outre-mer tant en matière de biodiversité, qu'en matière d'érosion des côtes, ou encore de conséquences sur l'agriculture, n'est pas le seul prérequis pour une politique d'adaptation efficace. Cette dernière passe aussi par une sensibilisation du public large et massive. C'est la condition pour que les réactions aux pressions exercées par le changement climatique et les actions d'adaptation qui en découlent soient correctement mises en oeuvre et partagées par l'ensemble des citoyens.    Cette diffusion de la connaissance n'est d'ailleurs pas qu'une condition de bonne mise en oeuvre des politiques publiques, elle est aussi un formidable outil de prévention des effets néfastes du changement climatique. L'appropriation de ce savoir encore en construction permet que les comportements de l'homme ne viennent plus aggraver les effets des dérèglements climatiques.    De nombreuses collectivités d'outre-mer se sont d'ores et déjà emparées de cet enjeu. La Guyane a ainsi compris les bénéfices à tirer d'une information et d'une sensibilisation du grand public. Le conseil régional a par exemple organisé le 6 juin 2015 un grand débat citoyen sur l'énergie et le climat. L'objectif était de faire émerger des pistes stratégiques et des engagements locaux en matière de limitation des émissions de gaz à effet de serre et d'adaptation au changement climatique. Les résultats de ce débat ont conduit à l'adoption d'orientations ambitieuses, preuve s'il en fallait, de l'intérêt d'associer le public et la population en général à la protection de son milieu de vie. Ses conclusions ont notamment porté sur :    - l'adaptation visant à construire une société résiliente possédant des capacités intrinsèques d'adaptation au changement climatique et à poursuivre l'acquisition de connaissances sur les impacts du changement climatique et les voies d'adaptation ;    - le transport et l'urbanisme, avec l'objectif de développer les circuits courts et l'économie circulaire à toutes les échelles et pour tous les usages et de mettre en place une gouvernance et une coordination des acteurs ;    - les énergies renouvelables et la maîtrise de l'énergie, dans le but de diversifier l'offre en énergies renouvelables en ouvrant les champs d'expérimentation vers les énergies marines, de développer les habitations et bâtiments à énergie positive et les éco-constructions et de mettre en place un parcours d'accompagnement à l'excellence énergétique ;    - la limitation de la déforestation en s'attachant à développer les outils de surveillance de l'environnement pour lutter plus efficacement contre les pratiques illégales de déforestation et de dégradation forestière et à favoriser les mesures permettant le stockage de carbone ;    - la coopération en développant les plates-formes d'échanges d'initiatives, de bonnes pratiques et de politiques publiques sur le plateau des Guyanes.    Dans le même esprit, le conseil régional du Guyane a également assuré en avril 2015, en lien avec l'Ademe et le BRGM, la publication d'une brochure contenant toutes les connaissances (46) relatives aux effets du changement climatique en Guyane sur les risques et les secteurs tels que la santé, l'eau, la pêche, l'agriculture, la forêt ou encore l'énergie. L'objectif d'une telle diffusion est de donner à la population et aux différents acteurs locaux les principales clés de compréhension des changements climatiques et de connaissance de la vulnérabilité du territoire afin de proposer des pistes d'actions d'atténuation et d'adaptation au changement climatique. Plus de 400 exemplaires ont ainsi été distribués sur le territoire.    Toujours en Guyane, le projet original d'éducation à l'environnement vers un développement durable (EEDD), lancé en 2013 à travers la mise en place d'un « carbet de transmission » en village Wayampi sur l'Oyapock, souligne l'importance de la transmission comme support à l'éducation au développement durable, puis comme préalable à l'action et à la lutte contre les impacts du changement climatique. Certaines familles amérindiennes sollicitent en effet un appui pour la mise en place d'ateliers de transmission afin que les adultes puissent apprendre aux jeunes et aux enfants les savoirs et savoir-faire nécessaires à la vie en site isolé. Dans ces sites, le lieu de rencontre et d'échange entre les villageois est le carbet communautaire (47). Dans ce contexte, à la demande du capitaine du village Yawapa, le projet de transmission des savoirs et savoir-faire relatifs à la construction a été mené dans un village de la commune de Camopi (Yawapa, Village Trois-Sauts). Ce projet a ainsi permis, à travers le chantier de construction du carbet, de transmettre aux jeunes et aux scolaires des savoirs sur les lieux de collecte de matières premières, des savoir-faire sur les techniques de collecte et de préparation de ces matériaux et enfin des savoir-faire sur les techniques de construction d'un carbet traditionnel.    On peut enfin citer en Guyane l'exemple du « Projet Carbone », qui a permis de mettre en place des outils pédagogiques d'animation sur la question du carbone, du changement climatique et du rôle de la forêt tropicale. Créé au sein de la première réserve naturelle régionale d'outre-mer, protégeant 2 500 hectares de milieux naturels, ce « sentier carbone » unique au monde grâce à sa forte dimension pédagogique,a pour but de permettre à chacun de visualiser in situ les enjeux liés au carbone forestier amazonien face aux changements climatiques. Tout au long du sentier, des totems délivrent des informations simples et claires qui permettent de mieux explorer et comprendre les phénomènes liés au cycle du carbone forestier. Un livret pédagogique réalisé par l'équipe de la réserve a également été mis à la disposition des scolaires.    Au-delà de la sensibilisation du public, l'appropriation des savoirs et des connaissances passe également par des actions de formation, initiale ou continue, professionnelle ou universitaire. La Réunion a ainsi par exemple mis en place en juin 2015 une journée de formation des agents de l'Etat aux pré-requis sur la question de l'énergie et du climat, conformément à la démarche de l'Etat exemplaire et dans la perspective de la COP 21 (48).    Sur le plan universitaire, le comité outre-mer de l'Alliance nationale de recherche pour l'environnement (AllEnvi) a piloté la réalisation du recensement pour l'année 2012 des activités de recherche des membres de l'AllEnvi dans les collectivités d'outre-mer : sur un total global déclaré de 1 275 équivalents temps plein (ETP), 90 % des activités scientifiques sur l'outre-mer sont réalisés par les agents sur place et seulement 10 % par des agents basés en métropole. En outre, pour les organismes de recherche, les deux thématiques les plus importantes sont l'agrobiologie et l'élevage, puis les ressources marines.    Des structures comme le Muséum national d'histoire naturelle participent également très fortement à la progression de la connaissance relative à la biodiversité et aux impacts du changement climatique dans les outre-mer. Ce dernier s'est ainsi engagé, depuis 2006, dans plusieurs projets d'inventaires massifs dans le cadre de l'initiative intitulée « La Planète revisitée » avec des partenaires publics et privés.    II. RÉALISATIONS ET PROJETS A. Feuille de route « Érosion » à Mayotte La situation géographique de Mayotte, sur un plancher océanique à l'entrée du canal du Mozambique, est celle d'un archipel dont les sols sont d'origine quasi exclusivement volcanique. L'altération et l'érosion, favorisées par un climat chaud et humide, contribuent à façonner la géomorphologie de l'île. Les sols ferralitiques argileux sont très friables et s'érodent facilement, ce qui a pu conduire à la formation de zones de sol nu, non propices aux cultures.    Ce phénomène entraîne un nombre important de risques pour la biodiversité : réduction de la fertilité des sols, dégradation du bilan hydrique, augmentation du ruissellement en amont et pics de crues avec risques d'inondations en aval pouvant entraîner l'envasement du lagon, etc.    Face à cela, les acteurs locaux se sont concertés pour rédiger en 2012 une feuille de route pour lutter contre l'érosion. Le projet, porté par la DEAL, la DAAF et le Parc naturel marin de Mayotte (PNMM) doit permettre de disposer sur les années 2014 à 2020 d'un calendrier de travail avec des actions prioritaires et des porteurs identifiés.    L'objectif est d'améliorer la connaissance des processus d'érosion des sols, grâce à des outils de mesure et à une ingénierie spécifique, afin de mettre en place des mesures qui permettront de limiter la production de sédiments et donc de protéger le lagon.    Cette feuille de route concertée a d'ores et déjà permis de nombreuses avancées dans la connaissance de la biodiversité et dans l'identification des phénomènes et des milieux :    - identification de la typologie des phénomènes érosifs ;    - identification des pratiques existantes ;    - identification de bassins versants représentatifs de l'ensemble des problématiques liées à l'érosion en milieu continental.    Il pourrait, à terme, déboucher sur la création d'un observatoire de l'érosion, qui serait à même de récolter toutes ces données essentielles, et de quantifier les phénomènes à l'oeuvre, centré sur deux bassins versants pilotes (Dzoumogné et Tsamboro). Une modélisation de la circulation des courants en 3D dans le lagon serait alors rendue possible et, en conséquence, l'évaluation des transferts sédimentaux littoraux et d'envasement lagonaire.    Source : DEAL - Mayotte.   B. Campagne de communication sur la maîtrise de la demande en énergie et mise en place d'un Espace Info énergie en Polynésie française La Polynésie française importe 87 % de son énergie (gaz et hydrocarbures), ce qui la place dans une situation de dépendance énergétique encore très forte. Sa consommation d'électricité est par ailleurs en constante progression, notamment dans les secteurs de l'industrie et de l'hôtellerie, mais aussi dans le résidentiel.    C'est pourquoi elle met en oeuvre une politique de promotion du développement des énergies renouvelables. Elle a ainsi engagé un programme pluriannuel d'installation de panneaux photovoltaïques sur certains de ses bâtiments administratifs. Mais elle a également mis l'accent depuis quelques années, sur une politique de maîtrise de la demande d'énergie.    Dans ce cadre, entre avril et septembre 2015, le Gouvernement de la Polynésie française a mené, en partenariat technique et financier avec l'Ademe, une grande campagne de communication sur ce sujet. Cette campagne a principalement consisté en la diffusion au grand public de messages radiophoniques et audiovisuels de sensibilisation sur des éco-gestes simples du quotidien, ayant pour but la maîtrise de l'énergie et la démarche éco-citoyenne. L'évaluation des impacts de cette campagne devraient permettre d'adapter les actions de sensibilisation sur ce sujet.    En outre, les actions menées dans le cadre de cette campagne servent également de support aux actions de l'Espace Info Énergie de Papeete, qui a ré-ouvert en mars 2015, et qui a pour mission d'informer gratuitement et de manière objective les citoyens sur la maîtrise de l'énergie, le recours aux énergies renouvelables et le changement climatique. Cet espace participe à plus d'une dizaine d'événements et parvient à sensibiliser de larges publics.    La transposition d'un tel dispositif à d'autres pays et territoires d'outre-mer du Pacifique est envisageable, comme cela est prévu en Nouvelle-Calédonie par exemple.    Source : Gouvernement de Polynésie française    C. Réseau d'Aires marines éducatives en Polynésie française Une aire marine éducative est une zone maritime littorale gérée par des élèves de cycle 3 d'une école à travers une démarche participative qui les implique autour d'un projet d'action citoyenne de protection du milieu marin.    Le concept d'aire marine éducative est né de l'imagination des enfants de l'école primaire de Tahuata (Marquises). Son lancement fait suite à la campagne océanographique Pakaihi i te moana, qui s'est déroulée aux Marquises en 2012. Suite à des échanges avec les scientifiques, les enfants avaient alors exprimé le souhait de devenir responsables de leur propre aire marine protégée dans la baie en face de leur école. Ce projet a été présenté lors du troisième congrès international des aires marines protégées en octobre 2013.    La Polynésie française, en collaboration avec ses partenaires, a ainsi décidé de structurer la démarche autour d'un label « Aire marine éducative » (Ame) récompensant les écoles souhaitant s'engager dans ce processus de gestion du milieu marin. L'objectif est de mettre les écoles au coeur des dispositifs écocitoyens. Trois axes structurent le label : connaître la mer, échanger avec les professionnels et gérer la zone littorale.    Les objectifs de ces six aires marines éducatives sont de plusieurs ordres :    - éducatif : faire connaître son patrimoine naturel et culturel maritime en valorisant une approche intergénérationnelle en langue française et polynésienne ;    - civique : participer à la gestion durable du domaine public qu'est la mer ;    - environnemental : sensibiliser à l'environnement dans une approche transversale mêlant l'écologie, les usages et la culture ;    - scientifique : rendre la science accessible à la population et aux jeunes générations ;    - professionnel : concourir à l'émergence de nouveaux métiers liés au patrimoine, notamment les référents des aires marines éducatives pour appuyer les enseignants ;    - touristique : valoriser le patrimoine maritime polynésien et les baies d'exception ;    - coopératif : insuffler des démarches similaires dans le Pacifique.    Le programme Pukatai bénéficie déjà à l'issue de la première année pilote d'un rayonnement à l'international grâce au Congrès mondial des Parcs de Sydney en novembre 2014 et au Congrès Oceania 21 en avril 2015 où des présentations ont été effectuées.    A l'échelle locale, la création des aires marines éducatives a entraîné des changements de comportement chez la population sans avoir eu besoin de cadre juridique formel. En effet, dans plusieurs baies AME, les adultes et plus particulièrement les pêcheurs respectent le périmètre de l'AME et les recommandations des enfants (zone de mouillage, taille minimale de capture, etc.). En outre, les élèves prennent leur rôle de « gestionnaire » de leur AME très à coeur et se rendent régulièrement dans la baie afin d'observer le milieu et les activités qui y sont réalisées.    Les états écologiques réalisés dans les AME confirment un très bon état de santé du milieu. Un travail de sensibilisation à la démarche scientifique du suivi de l'état écologique et une restitution des principaux résultats ont été réalisés dans les classes.    Source : Gouvernement de Polynésie française.
</article>
<article titre="Une commode qui invite au voyage" date="2016 01 16" journal="La Nouvelle République du Centre-Ouest  LOIR ET CHER">
Philippe Rouillac en profite pour nous faire voyager dans l'univers du meuble : « Quel enchantement n'éprouve-t-on pas à la prononciation de ces mots ? On rêve d'aventures... Au XVIIIesiècle, la clientèle titrée, qui fréquente la cour du roi se doit de posséder des meubles parisiens dans ses châteaux de province. La majorité, composée soit de la petite noblesse terrienne, soit de la bourgeoisie financière agissante (qui investit dans l'immobilier) se tourne, pour se meubler, vers les artisans locaux. Ceux-ci ont à charge de concilier les désirs de leurs clients. Ils doivent allier la tradition régionale du meuble en bois massif fourni sur place et les idées et formes nouvelles venues de Paris. Le mobilier ancien n'est donc pas que parisien puisque certaines provinces, aux caractéristiques marquées, jouent la singularité et l'originalité, tant pour leur forme que par les bois utilisés. Ainsi, l'armoire dite de mariage la plus riche est normande, le Val de Loire privilégiant les buffets bas et le Sud-Ouest des commodes très recherchées.  En bois de noyer Classique, notre commode présente en façade trois rangs de tiroirs sculptés en applique, le premier en comportant deux, séparés par un montant. Les côtés sont galbés et en retrait à la base, comme les pieds dits « pieds de biche » se terminant par des volutes. La traverse basse est mouvementée et moulurée. Le dessus est en bois, les fixations apparaissent nettement en façade et dans les angles avec des chevilles de bois assemblant les divers éléments constitutifs. Critère d'ancienneté, aucun clou ni trace de colle ne sont d'usage au XVIIIesiècle. Les bronzes sont classiques et de petite qualité : poignées dites mains de tirage en bronze non doré et entrées de serrure simulée pour celle de la rangée de tiroir supérieure. Cette forme dite « tombeau » reprend la forme des sarcophages antiques, aux formes généreuses. Notre meuble indique une influence bordelaise atténuée, comme pour les productions de Saintonge ou de la Guyenne. D'après cette simple photo d'amateur, la couleur et la patine du bois, cette commode semble être en noyer, bois naturel de ces provinces, et n'est malheureusement pas en acajou comme les belles et recherchées commodes bordelaises dites de port. Ce bois des îles importé des Antilles ou des Indes occidentales est l'apanage des grands ports tels Nantes et Bordeaux. D'où le nom meuble de port. La Rochelle, Saint-Malo, voire Morlaix tirent aussi leur épingle du jeu mais plus modestement. Bois exotiques de palissandre, bois de rose, bois bagasse, amourette, bois serpent, gaïac de Cayenne, bois citron, acajou de Cuba... Rêves et voyages. Bois fin, rouge, aux veines luisantes, l'acajou est travaillé en bois massif pour les armoires, buffets deux corps et surtout commodes, telle celle-ci. Notre commode en bois de noyer, bois moins précieux et donc moins recherché, peut être estimée en vente aux enchères  autour de 1.000 E. Ce qui serait un prix de départ et non d'arrivée pour la même... en acajou ! Mais l'invitation au voyage demeure...
</article>
<article titre="Aérien et maritime : Didier Robert veut connecter les îles et dynamiser le tourisme" date="2016 01 14" journal="NewsPress (français)">
Les Îles Vanille ont tenu une assemblée générale ce mardi 13 janvier à Port-Louis avec les ministres du tourisme et représentants des îles.   Au préalable, le Président des Îles Vanille en exercice, le Ministre du Tourisme de Maurice, Xavier-Luc Duval, a salué les chiffres du tourisme en augmentation dans les îles, qui attestent de la pertinence des actions et du travail réalisés depuis 2010, qui portent leurs fruits. C'est en effet une croissance de plus de 10% dont bénéficient les îles de l'océan Indien. Plus de 50 000 personnes ont voyagé grâce aux combinés des Îles Vanille. Et La Réunion dans cette dynamique a pu accueillir 40 000 touristes supplémentaires.   Dans le cadre de ce premier déplacement après les élections régionales de décembre, le Président Didier ROBERT était accompagné de deux nouveaux conseillers régionaux : Faouzia Vitry, déléguée à la Coopération Régionale océan Indien, et Stéphane Fouassin délégué au Tourisme.   - Actions 2016 des Îles Vanille : les grandes arrivées des croisiéristes dans les îles Le calendrier et les actions 2016 s'inscrivent dans une continuité avec un point focus concernant les croisières. Ce sujet phare engagé depuis plusieurs années par les acteurs des Îles Vanille et concrétisé en 2015 par la signature d'une convention avec la première compagnie de croisière européenne Costa marque en effet une nouvelle étape pour l'activité économique et le tourisme des îles.   Pour  Didier Robert,  cette convention avec la  Cie Costa  est une opportunité sans précédent pour le développement des croisières dans la zone océan Indien et à La Réunion. La filière correspond à un marché en pleine croissance. Costa mise sur cette région du monde en s'appuyant sur les Îles Vanille. Il faut donc réussir ce démarrage pour amplifier davantage encore les rotations et amplifier les activités (restauration, loisirs...).   Dès 2016, les rotations de bateaux de croisières vont augmenter de 80%, et en 2017 de 100 % et permettre de passer de 5 200 touristes à plus de  18 000 touristes . Les retombées économiques pour la Réunion sont incontestables.   - La Compagnie low-cost océan Indien régionale : la connexion des îles est la clé du développement du tourisme Le Président Didier Robert  plaide pour un  développement des accès aériens  entre les îles. La compagnie low-cost de l'Océan indien devra répondre à cette exigence et faciliter les déplacements des populations et des touristes entre les îles à un coût raisonnable. Des points d'avancée sont d'ores et déjà à mettre au crédit du travail des Îles Vanille.  L'Alliance Vanille  a été créée pour poser des actes concrets en regroupant l'ensemble des compagnies régionales de l'océan Indien. L'assemblée générale à acté l'organisation d'une rencontre avec les différentes compagnies régionales à La Réunion dans les prochaines semaines pour avancer sur les propositions.   - Un nouvel Accord cadre de Coopération signé en 2016 avec le Premier Ministre Didier Robert  s'est entretenu avec le  Premier Ministre - Sir Anerood Jugnauth  à son bureau de Port Louis en présence du  Vice-Président Jean-Paul Virapoulle  et de l'ambassadeur de France. Les deux hommes veulent réaffirmer leur engagement pour le développement d'une coopération davantage axée sur les échanges économiques. A ce titre, ils ont décidé de travailler à de nouveaux accords de coopération autour de projets précis et d'une feuille de route dans des domaines ciblés tels que  l'aérien, les énergies renouvelables (solaire, bagasse, énergie des mers), le numérique (l'île Maurice s'est positionnée sur l'acquisition d'un troisième câble), la stratégie de conquête africaine  ... l'objectif étant de faire des pôles de coopération et ainsi gagner du temps, multiplier les chances de réussite dans la zone.   Le Premier Ministre devrait se rendre à La Réunion sur invitation du  Président Didier Robert  dans le courant de cette année en déplacement officiel pour la signature des nouveaux accords.
</article>
<article titre="Stratus Packaging améliore les étiquettes de Daddy" date="2016 01 11" journal="Emballages magazine.com">
L'imprimeur a opté pour un film en polyéthylène biosourcé issu de la canne à sucre, comme l'emballage et comme... le produit.-   C'est ce que l'on pourrait appeler une stratégie « gagnant-gagnant » avec un vainqueur final, l'environnement. Elle implique, d'un côté, un client qui, comme CristalCo (Cristal Union), se soucie toujours plus de ses émissions de CO2 et, de l'autre, un fournisseur, en l'occurrence l'imprimeur d'étiquettes Stratus Packaging, qui cherche à lui proposer les meilleures solutions pour atteindre son but. Le sucrier utilise depuis 2014 une boîte en plastique végétal, en l'occurrence un polyéthylène haute densité (PEhd) issu de canne à sucre, produit par le brésilien Braskem. Il ne manquait plus que l'étiquette! Stratus Packaging s'en est chargé. Après avoir travaillé sur un film en polyéthylène (PE) de 100 microns contenant environ 80% de matière biosourcée (bagasse de canne à sucre), l'imprimeur a réussi à trouver une solution encore meilleure, un film en PE de 85 microns, blanc ou transparent, comprenant près de 83% de matière biosourcée pour la variété blanche et plus de 90% pour le PE transparent. Les étiquettes, autoadhésives, sont imprimées en flexo UV, comme n'importe quelle autre étiquette, sur le site de SEEC, près de Bourg-en-Bresse (Ain). Résultat pour Daddy : non seulement une boîte, mais aussi une étiquette d'origine renouvelable, qui plus est fabriquées à partir de sucre, comme le produit! D'après Cristal Union, son emballage en plastique biosourcé permettrait de réduire de près de 13% le poids du contenant, par rapport à la boîte précédente, soit 10 tonnes de plastique en moins par an et un équivalent de 400 tonnes de CO2 de moins rejetées dans l'atmosphère. L'entreprise indique que la fabrication d'une tonne de PE biosourcé capte 2,5 tonnes de CO2 alors que la fabrication d'une tonne de PE issu du pétrole émet 2 tonnes de CO2.   Stratus Packaging compte parmi les leaders français de l'étiquette en France. La société dirigée par Isidore Leiser réalise un chiffre d'affaires de 40 millions d'euros avec un effectif de 290 personnes réparties sur cinq sites. Elle imprime des étiquettes autoadhésives, des manchons thermorétractables, des étiquettes IML et des étiquettes-livrets.
</article>
<article titre="Green techs : des business qui bourgeonnent dans le monde entier 1 / 5" date="2016 01 8" journal="Chef d'Entreprise (site web)">
À l'occasion de la COP21, la rédaction fait le point sur les tendances écolo, formidables mines d'opportunités de business et d'innovations.   États-Unis : emballé, c'est mangé !   Pointés du doigt comme source de déchets, les emballages alimentaires pourraient bien vivre prochainement une sacrée révolution... en devenant eux-mêmes comestibles. C'est l'idée pas si folle de David Edwards, professeur d'ingénierie biomédicale à l'université d'Harvard. Féru de design culinaire, il compare son invention, baptisée WikiCells, à "un grain de raisin entouré de sa peau". En l'occurrence, une enveloppe tendre constituée de particules de chocolat, fruits, noix ou graines, et d'alginate issu des algues. Ou encore des coques faites de bagasse, résidu fibreux de la canne à sucre, pour des yaourts dont on mangerait le pot, en quelque sorte. Les acheteurs se sont pour l'heure montrés frileux quant à la capacité des consommateurs de changer leurs habitudes, pointant la difficulté de distribuer les WikiCells autrement que... dans un emballage.   Un concept plus facile à avaler au pied d'un food truck, par exemple. à l'image de Coolhaus, concepteur californien de ces "ice-cream sandwiches", qui a généralisé le papier comestible entourant ses glaces. Un emballage fabriqué à base de pommes de terre et élégamment imprimé... à l'encre comestible.     De l'autre côté de la Manche, trois chercheurs basés à Londres ont mis au point Ooho, d'étonnantes bulles comestibles pour remplacer les bouteilles d'eau en plastique. Quand la start-up parisienne Wisepack a, elle, créé des doses hydratantes et énergétiques naturelles et comestibles, une alternative aux bouteilles et gels d'effort qui devrait arriver sur les compétitions sportives en 2017.    Article suivant   Cet article est paru dans Chef d'Entreprise (site web)
</article>
<article titre="La loi sur la transition énergétique : déclaration d'intentions ou socle d'une transformation dirigée de l'économie française? Deuxième partie" date="2016 01 1" journal="Droit de l'Environnement, no. 241">
Le soutien au développement des "renouvelables"    Ce dispositif est profondément remanié, afin que la collectivité en maîtrise le coût.  Le régime de soutien au développement des moyens de production d'électricité à partir de sources d'énergie renouvelables reposait, jusqu'à la loi de transition énergétique sur deux mécanismes : l'obligation d'achat et les appels d'offres.  Le régime de l'obligation d'achat, régi par les articles L. 314-1 et sq du Code de l'énergie, oblige un acheteur désigné par la loi - EDF ou, jusqu'à présent, l'entreprise locale de distribution (ELD) territorialement compétente - à conclure un contrat avec tout producteur d'électricité d'origine renouvelable pour lui acheter la totalité de sa production. Il couvre notamment les éoliennes terrestres ou installées en mer, ainsi que les installations utilisant l'énergie géothermique, solaire thermique ou photovoltaïque. Les caractéristiques des installations et les conditions à remplir pour bénéficier de l'obligation d'achat sont déterminées par décret. Les conditions tarifaires d'achat sont précisées, pour chaque filière, par un arrêté des ministres de l'Éco-nomie et de l'Énergie, pris après avis de la Commission de régulation de l'énergie (CRE).    Par exception, lorsque les capacités de production ne répondent pas aux objectifs de la programmation pluriannuelle établie par l'État, l'autorité administrative peut recourir à des appels d'offres, aux termes des articles L. 311-10 et sq du Code de l'énergie. Différents appels d'offres ont été organisés, en métropole ou outre-mer, notamment pour les filières éolienne et photovoltaïque.    Ces mécanismes, fréquemment utilisés dans l'Union européenne, ont provoqué une hausse considérable du prix de l'électricité pour les consommateurs, tout en contribuant à déprécier les prix de gros, causant même des épisodes de prix négatifs sur les marchés européens, et soulevé par ailleurs de sérieuses interrogations au regard de la réglementation des aides d'État. La France s'est en particulier abstenue de toute forme de notification à ce titre, jusqu'à être condamnée par la Cour de justice1 .    La Commission européenne a donc lancé une consultation des acteurs de marché et finalement adopté le 28 juin 2014 de nouvelles « lignes directrices » concernant les aides d'État à la protection de l'environnement et à l'énergie pour la période 2014-20202 . Ces lignes directrices sont entrées en vigueur le 1er juillet 2014, en même temps que la réforme de la loi allemande sur l'électricité, qui en est très proche.    Aux termes de ce document de "droit souple", qui encadre le pouvoir de décision de la Commission au cas par cas, les nouvelles aides en faveur de l'énergie produite à partir de ressources renouvelables ne sont plus compatibles avec le marché intérieur, à partir du 1er janvier 2016, qu'à la condition que leurs bénéficiaires « vendent leur électricité directement sur le marché, et qu'ils soient soumis aux obligations du marché » dans les conditions cumulatives suivantes : l'aide est octroyée sous la forme d'une prime s'ajoutant au prix du marché; les producteurs sont soumis à des responsabilités en matière d'équilibrage; des mesures sont mises en place pour faire en sorte que les producteurs ne soient pas incités à produire de l'électricité à des prix négatifs. En outre, les lignes directrices prévoient qu'à compter du 1er janvier 2017, ces aides seront octroyées « à l'issue d'une procédure de mise en concurrence fondée sur des critères clairs, transparents et non discriminatoires ».    En conséquence, l'article 104 de la loi de transition énergétique a procédé à la refonte des mécanismes français de soutien, par des dispositions entrées en vigueur le 1er janvier 2016 et qui doivent être déclinées par deux décrets : l'un pour définir les installations éligibles, l'autre relatif au complément de rémunération.    Désormais, le soutien aux producteurs d'énergies renouve-lables repose sur trois dispositifs, dont les conditions de contrôle et les sanctions sont identiques.  1. Le mécanisme de l'obligation d'achat ne disparaît pas, mais son champ d'application est réduit  L'article L. 314-1 du Code de l'énergie renvoie désormais à un décret simple la fixation de la liste des installations qui continuent d'en bénéficier et de leurs caractéristiques. Le projet de décret « définissant les listes et les caractéristiques des installations mentionnées aux articles L. 314-1 et L. 314-18 du Code de l'énergie » mis en consultation par l'administration3 mentionne la filière hydroélectrique, les éoliennes terrestres, de petites installations photovoltaïques, des installations utilisant le biogaz, les éoliennes flottantes, les installations utilisant l'énergie houlomotrice, les hydroliennes, les petites cogénérations, les installations de récupération du gaz de mine et des centrales à bagasse. La CRE en a dressé le tableau ci-contre4 .    Le texte précise les conditions dans lesquelles une installation existante aidée pourra bénéficier une nouvelle fois d'un contrat d'achat. Cette possibilité est ouverte aux installations hydroélectriques sous réserve de la réalisation d'un programme d'investissement de rénovation et, dans les autres filières, à celles dont les coûts d'exploitation d'une installation de référence amortie et performante sont supérieurs aux recettes, y compris les aides financières et fiscales auxquelles elles sont éligibles. Dans les deux cas, le niveau de rémunération de ces installations devra être adapté afin de tenir compte de ces conditions économiques.    Précisons enfin que le projet de décret simple permet aux producteurs à partir de biomasse et aux petits producteurs (dont la puissance de l'installation est de 36 kVA au plus) qui ont déposé une demande complète de raccordement avant le 31 décembre 2015 de conserver le bénéfice des conditions d'achat définies dans le dernier arrêté tarifaire5 , à la condition de réaliser l'installation dans un délai, respectivement, de trois ans et dix-huit mois.    2. Le mécanisme du complément de rémunération est donc au cœur, désormais, de la politique de soutien à ces moyens de production.  Les nouveaux articles L. 314-18 à L. 314-27 du Code de l'énergie définissent le régime général du complément de rémunération, applicable aux installations de production d'électricité d'origine renouvelable situées sur le territoire métropolitain continental6 dont les caractéristiques sont définies par décret. Il s'agit d'une    prime versée en complément du prix de la vente de l'électricité sur le marché. Pour chaque filière, le complément de rémunération doit compenser, sur une base annuelle, l'écart entre, d'une part, la rémunération tirée par les producteurs de la valorisation de leurs installations sur les marchés de l'énergie et, d'autre part, un niveau de rémunération de référence, équivalent à un tarif d'achat. Il est proportionnel à la quantité d'énergie produite et est censé permettre aux producteurs de couvrir leurs coûts en assurant une rentabilité normale à leurs projets, alors que le tarif d'achat était parfois à l'origine d'effets d'aubaine, surtout avec la baisse des prix de gros.    Toutes les installations ne sont pas concernées. Le projet de décret simple diffusé par le gouvernement mentionne les installations de production situées en France métropolitaine continentale relevant des filières de l'éolien terrestre, de l'hydroélectricité, de l'incinération des déchets ménagers, de la valorisation du biogaz, de la géothermie et de la cogénération. La puissance unitaire maximale est d'1 MW pour l'hydraulique7 et la cogénération; pour le biogaz, le seuil minimal est fixé à 500 kW et le maximum à 12 MW. L'énergie géothermique, celle produite par la valorisation des déchets ménagers et l'éolien terrestre ne connaissent aucun seuil. Le dispositif devait prendre effet le 1er janvier 2016.    Certaines installations ne peuvent donc bénéficier ni de l'obligation d'achat, ni du complément de rémunération : éolien en mer, installations photovoltaïques de plus de 100 kW, méthanisation agricole, solaire thermodynamique ou encore géothermie dans les départements d'outre-mer. Inversement, les producteurs éoliens pourront arbitrer entre complément de prix et obligation d'achat, nécessairement en faveur du dispositif le plus coûteux pour la collectivité, ce qui a suscité de vives critiques de la CRE. Enfin, le projet de décret simple prévoit que sont éligibles au complément de rémunération, outre les installations nouvelles, les installations existantes qui bénéficient d'un contrat d'achat conclu dans le cadre de l'obligation d'achat ou d'un appel d'offres, pour la durée résiduelle du contrat d'achat initial8 . Dans ce cas, le contrat précédent est résilié, mais les conditions de rémunération fixées par le nouveau contrat demeurent celles de l'obligation d'achat et le tarif d'achat évolue comme prévu initialement.    Un projet de décret en Conseil d'État, « relatif au complément de rémunération mentionné à l'article L. 314-18 du Code de l'énergie et à l'obligation d'achat mentionnée à l'article L. 314-1 du Code de l'énergie », tend à modifier le décret n° 2001-410 du 10 mai 2001 relatif aux conditions d'achat de l'électricité produite par des producteurs bénéficiant de l'obligation d'achat9 . Le certificat ouvrant droit à l'obligation d'achat, délivré par le préfet, est supprimé. Désormais, un producteur souhaitant bénéficier de ce mécanisme adressera directement sa demande à EDF, seul acheteur obligé, qui l'instruira et lui retournera un contrat d'achat dans un délai de trois mois. Les modèles de contrat de complément de rémunération seront approuvés par le ministre en charge de l'énergie. L'entrée en vigueur de ce contrat sera subordonnée à la fourniture par le producteur à EDF d'une attestation de conformité à la réglementation et aux données techniques déclarées dans la demande de contrat, établie par un organisme agréé. Dans son avis sur le projet de décret, la CRE a regretté que les conditions de rémunération des producteurs sous obligation d'achat ne soient pas dégradées pendant les épisodes de prix négatifs.    Des conditions spécifiques seront arrêtées pour chaque filière par les ministres chargés de l'énergie et de l'économie, après avis de la CRE. Elles seront établies en tenant compte des coûts et des recettes d'installations performantes représentatives de chaque filière, dans le respect des dispositions de l'article L. 314-20 du Code de l'énergie qui prévoit qu'il ne « pourra conduire à ce que la rémunération totale des capitaux immobilisés, résultant du cumul de toutes les recettes de l'installation et des aides financières ou fiscales, excède une rémunération raisonnable des capitaux, compte tenu des risques inhérents à ces activités ». Elles seront revues périodiquement, afin de tenir compte notamment de l'évolution des coûts de production de chaque filière.    Les termes du contrat de complément de rémunération répondront à ces conditions. Le contrat sera administratif par détermination de la loi. Sa durée, fixée par les arrêtés interministériels et qui pourra donc varier selon les filières, sera de vingt ans au plus. La « prise d'effet » du contrat sera conditionnée à l'achèvement de l'installation et à la fourniture par le producteur à l'acheteur d'une attestation de conformité à la réglementation et aux données déclarées dans la demande de contrat (technologie, puissance installée...), établie par un organisme agréé. Cependant, jusqu'au 1er janvier 2018, une attestation sur l'honneur du producteur tiendra lieu d'attestation.    Le contrat de complément de rémunération pourra être suspendu ou résilié par EDF en cas de violation par le producteur de ses obligations réglementaires ou contractuelles. La résiliation du contrat par le producteur entraînera le remboursement de la somme actualisée des montants perçus depuis la date d'entrée en vigueur du contrat.    Les nouveaux articles L. 314-19 et L. 314-21 du Code de l'énergie consacrent le principe selon lequel une installation ne peut bénéficier qu'une seule fois d'un mécanisme de soutien. Le bénéfice du complément de rémunération n'est ainsi pas ouvert aux installations ayant bénéficié d'un ancien mécanisme de soutien; celui-ci ayant eu pour objet d'en permettre l'amortissement, les producteurs doivent désormais vendre purement et simplement sur le marché. Des dérogations sont toutefois posées, d'une part, en faveur des installations amorties dont les coûts de fonctionnement sont supérieurs aux recettes et, d'autre part, pour les installations hydrauliques ou de cogénération sur lesquelles sont réalisés des investissements de rénovation. La CRE a critiqué cette seconde dérogation, dont bénéficient depuis plusieurs années ces producteurs. Le projet de décret prévoit que les conditions de rémunération de ces installations hydrau-liques tiendront compte de leurs conditions économiques de fonctionnement, de façon à éviter de les favoriser par rapport à de nouveaux investisseurs.    En vertu de l'article L. 314-23 du Code de l'énergie, et sous réserve de la poursuite des contrats en cours, le ministre chargé de l'énergie pourra suspendre le dispositif du complément de rémunération s'il cesse de répondre aux objectifs de la programmation pluriannuelle de l'énergie (PPE) prévue aux nouveaux articles L. 141-1 et sq du même code.    Enfin, le projet de décret décrit le mécanisme d'acheteur de dernier recours prévu à l'article L. 314-26 nouveau du Code de l'énergie. Désigné par le ministre chargé de l'énergie à l'issue d'une mise en concurrence, l'acheteur de dernier recours conclut des contrats d'achat avec les producteurs bénéficiant du complément de rémunération qui ne parviennent pas à vendre leur production sur le marché, qui se substituent aux contrats de complément de prix; le tarif d'achat sera fixé par l'arrêté tarifaire de la filière correspondante, mais ne pourra excéder 80 % de la rémunération totale qui aurait été issue de la vente sur le marché majorée du complément de prix.    3. Le dispositif de l'appel d'offres est maintenu, puisqu'il est prévu par les nouvelles lignes directrices.  L'article 106 de la loi de transition énergétique modifie toutefois l'article L. 311-12 du Code de l'énergie relatif à la procédure d'appel d'offres et introduit la possibilité pour les lauréats de bénéficier alternativement d'un contrat d'achat ou d'un contrat de complément de rémunération, en fonction des prescriptions du cahier des charges.    Par ailleurs, selon le projet de décret en Conseil d'État, les producteurs exploitant des installations bénéficiant d'un contrat d'achat aux termes d'un appel d'offres pourront, comme dans le cas de l'obligation d'achat, résilier le contrat conclu au prix qu'ils ont initialement proposé pour passer au régime du complément de rémunération, faculté d'arbitrage qui a provoqué une critique de la CRE.    Le nouveau régime mérite deux dernières observations .  Les contrôles et les sanctions ont été durcis.  La CRE a souligné en effet que le contrôle de la conformité des installations aux dispositions réglementaires qui leur sont applicables constitue un enjeu majeur, s'agissant en particulier de la vérification des caractéristiques (technologie utilisée, puissance installée, etc.) qui conditionnent directement l'éligibilité des installations à ces avantages financiers, car les dispositifs actuels de contrôle des installations sous obligation d'achat sont inefficients, voire inexistants, alors même que de nombreuses fraudes ou possibilités de fraude ont été identifiées.    En conséquence, l'article 107 de la loi de transition énergétique met en place un mécanisme de contrôle des installations bénéficiant de l'obligation d'achat ou du complément de rémunération, ainsi que des sanctions. Les modalités de contrôle et les sanctions en cas de non-conformité avérée sont identiques dans les deux cas. Le législateur a complété l'article L. 311-14 du Code de l'énergie pour prévoir notamment que l'administration peut suspendre ou résilier le contrat d'achat passé entre le producteur et l'acheteur obligé. La résiliation du contrat peut s'accompagner de l'obligation de remboursement de tout ou partie des sommes perçues en application du contrat pendant la période de non-respect des dispositions réglementaires, ainsi que d'une sanction déterminée en fonction de la puissance de l'installation mais plafonnée à 100 000 € par MW.    Le projet de décret en Conseil d'État précise que le producteur doit s'engager à faire réaliser les contrôles demandés par l'autorité administrative sur les caractéristiques techniques, les performances et les coûts de son installation, puis à en tenir les résultats à la disposition du ministre. Les coûts doivent être communiqués chaque année à la CRE.    Dans son avis du 9 décembre 2015, la CRE s'est dite favorable aux dispositions d'organisation des contrôles prévues dans le projet de décret, « qui permettent de répondre à cet enjeu pour les nouvelles installations ».    Le financement  de l'ensemble de ces dispositifs de soutien doit être assuré par la contribution au service public de l'électricité (CSPE). Celle-ci vient d'être profondément réformée par la loi de finances rectificative pour 2015, qui l'a intégrée dès le 1er janvier 2016 à l'accise sur l'électricité, dénommée jusqu'ici « taxe intérieure sur les consommations finales d'électricité » (TICFE). Le champ d'application de cette taxe est étendu afin de prendre en compte toutes les consommations d'électricité (elle n'était jusqu'ici prélevée qu'au-delà d'une puissance souscrite de 250 kVA), mais également, à partir de 2017, celles de gaz et de carburants. Les tarifs de la TICFE, rebaptisée « contribution au service public de l'électricité », sont en outre augmentés, car elle intègre depuis 2014 la contribution climat-énergie (« taxe carbone ») que le législateur a décidé de rehausser progressivement jusqu'à 56 € en 2020, puis 100 € en 2050, aux termes du VIII de l'article 1er de la loi de transition énergétique.    Le montant de cette nouvelle CSPE est ainsi fixé à 22,50 € par mégawattheure (€/MWh) en 2016 et sera en principe maintenu à ce montant en 2017. Des réductions sont cependant accordées aux installations industrielles électro-intensives, et notamment à celles qui appartiennent à des secteurs considérés comme exposés à un risque significatif de « fuite de carbone » (c'est-à-dire de délocalisation hors d'Europe) en raison des coûts liés à la politique environnementale européenne répercutés sur les prix de l'électricité (les quotas d'émission de gaz à effet de serre), conformément aux règles de droit souple édictées par la Commission européenne sur cette problématique10 . La contribution des autres redevables sera majorée d'autant.    Cette budgétisation permettra le contrôle politique du Parlement et l'intervention des services des douanes dans les contrôles administratifs. Elle clarifiera surtout les choses : les charges financées jusqu'ici par la CSPE seront désormais réparties entre, d'une part, un compte d'affectation spéciale "Transition énergétique", alimenté par la fiscalité énergétique (la CSPE et la fraction de la taxe intérieure sur la consommation finale de gaz naturel [TICGN] correspondant au montant de l'actuelle contribution biométhane) et qui supportera les seules charges liées à la transition énergétique (soutien aux énergies renouvelables, à l'injection de bio-méthane dans les réseaux publics et aux effacements de consommation, remboursement à EDF du déficit de CSPE accumulé); et, d'autre part, un programme budgétaire "Service public de l'énergie" où seront imputées les autres dépenses (péréquation tarifaire au profit des consommateurs ultramarins, lutte contre la précarité énergétique, soutien à la cogénération, budget du Médiateur de l'énergie, intérêts de la dette envers EDF...). En raison de l'extension de l'assiette et de la hausse du tarif, il est prévu que les produits excéderont notablement les recettes : le solde sera versé au budget général.     Les délais de raccordement des installations de production d'électricité    Le développement des énergies d'origine renouvelable requiert une adaptation des réseaux électriques, car cette production d'électricité est décentralisée et intermittente (elle dépend de facteurs climatiques), à la différence de la production thermique (nucléaire ou à flamme), concentrée sur un nombre restreint de grandes centrales dont la puissance est maîtrisée en permanence à la fois par l'exploitant (pour vendre son énergie) et par l'opérateur du système électrique (pour en assurer l'équilibre). Pour mieux organiser le développement de ces outils de production qui n'émettent pas de gaz à effet de serre, tant les gestionnaires de réseaux publics que les producteurs ont besoin de sécurité juridique et de maîtrise des coûts.    C'est pourquoi la loi Grenelle II11 a créé au sein des schémas régionaux du climat, de l'air et de l'énergie les schémas régionaux de raccordement des énergies renouvelables (S3REnR), dont l'objectif est de mutualiser les coûts entre les divers gestionnaires de réseaux au sein d'une même région. Aux termes de l'article L. 321-7 du Code de l'énergie, ces schémas, qui sont élaborés par le gestionnaire du réseau public de transport en accord avec les gestionnaires de réseaux publics de distribution et après avis des autorités organisatrices de la distribution publique, définissent « les ouvrages à créer ou à renforcer pour atteindre les objectifs fixés par le schéma régional du climat, de l'air et de l'énergie », ainsi que « le périmètre de mutualisation des postes du réseau public de transport, des postes de transformation entre les réseaux publics de distribution et le réseau public de transport » et « les liaisons de raccordement de ces postes au réseau public de transport »; ils doivent également mentionner « les capacités d'accueil de production permettant d'atteindre les objectifs définis par les schémas régionaux du climat, de l'air et de l'énergie ».    Mais, simultanément, le coût moyen du raccordement, supporté par les producteurs, a doublé entre 2009 et 2013, tandis que le délai moyen de raccordement atteignait trente mois en 2014.    Se fondant sur le partage d'informations et la planification concertée que permettent les S3REnR, les sénateurs qui sont à l'origine de l'article 105 de la loi sur la transition énergétique ont donc décidé d'imposer aux gestionnaires de réseaux publics un délai maximum de raccordement. Aux termes de l'article L. 342-3 modifié du Code de l'énergie, ce délai est fixé à dix-huit mois à compter de l'acceptation de la convention de raccordement par le producteur (et même à deux mois pour les micro-producteurs, jusqu'à 3 kVA), sauf si le raccordement impose d'entreprendre des travaux d'extension ou de renforcement du réseau12 . Le gestionnaire du réseau de raccordement défaillant devra indemniser le producteur, selon un barème à fixer par décret en Conseil d'État.    Il faut souligner que ce nouveau délai intervient au terme d'une procédure longue et complexe, articulée autour de la logique de la file d'attente (« premier arrivé, premier servi »), afin de respecter le principe de non-discrimination. La CRE adresse depuis 2009 des orientations aux gestionnaires de réseaux, soit en approuvant la procédure de raccordement du gestionnaire du réseau de transport (RTE) 13, soit en adoptant des décisions de nature réglementaire pour celles des gestionnaires de réseaux de distribution14 . Ces orientations ont été complétées par des décisions de son Comité de règlement des différends et des sanctions (CoRDiS), saisi par des producteurs. Les procédures sont incluses dans la documentation technique de référence (DTR) de chaque gestionnaire.    La plupart des installations étant raccordées en distribution publique15 , le principal texte applicable est la délibération de la CRE du 25 avril 2013 portant décision sur les règles d'élaboration des procédures de traitement des demandes de raccordement aux réseaux publics de distribution d'électricité et le suivi de leur mise en œuvre. Aux termes de cette décision, les procédures doivent contenir des engagements envers les producteurs sur les délais de traitement des demandes de raccordement, ainsi que sur les coûts et délais de mise à disposition des ouvrages. De même, l'article 13 du cahier des charges type de concession du réseau public de transport d'électricité impose-t-il à RTE de fixer des délais de traitement des demandes et de s'engager sur les coûts et délai maximaux de réalisation de chaque raccordement.    Les demandes de raccordement d'installations de production de plus de 36 kVA à un réseau de distribution16 doivent faire    l'objet, par le gestionnaire de réseau compétent, d'une proposition technique et financière (PTF) dans un délai que la DTR ne peut fixer à plus de trois mois, extensible à six mois en cas d'afflux exceptionnel de demandes de raccordement17 . Le producteur pétitionnaire dispose de trois mois pour accepter la PTF, la DTR ne pouvant prévoir une prorogation qu'à la condition que celle-ci n'affecte aucune autre demande de raccordement. Le gestionnaire du réseau de distribution dispose ensuite d'un délai de neuf mois pour transmettre une convention de raccordement au demandeur, sous réserve néanmoins du bon aboutissement des procédures administratives (permis de construire...); le délai est de douze mois si des ouvrages doivent également être réalisés sous la maîtrise d'ouvrage de RTE. Le producteur, qui peut solliciter la suspension du traitement de sa demande si des contentieux pèsent sur les autorisations administratives relatives à ses installations, doit accepter et signer la convention de raccordement dans un délai que la DTR doit fixer entre trois et six mois - délai qui ne peut être prorogé que si aucune autre demande n'est affectée.    C'est donc à l'issue de ce processus que joue le nouveau délai de réalisation des travaux de raccordement.    Cette problématique de délai présente une sensibilité particulière lorsque l'installation de production est destinée à fonctionner sous le régime de l'obligation d'achat (articles L. 314-1 du Code de l'énergie et sq), en raison d'un historique conflictuel.    Le pouvoir réglementaire ayant, d'une part, entrepris d'abaisser par à-coups les tarifs auxquels l'énergie d'origine photovoltaïque est achetée pendant vingt ans aux producteurs et, d'autre part, lié le bénéfice de chacun des tarifs successifs à la date d'acceptation de la PTF, un important contentieux est né de la parution du décret n° 2010-1510 du 9 décembre 2010, qui a instauré avec effet rétroactif un moratoire sur l'obligation d'achat18 , en préalable à une nouvelle et substantielle réduction tarifaire19 . Le moratoire a frappé les producteurs qui n'avaient pas notifié l'acceptation de la PTF le 2 décembre 2010. Or, les DTR de RTE et d'ERDF (principal gestionnaire de réseaux de distribution) prévoyaient la transmission au producteur d'une PTF dans les trois mois de la réception de la demande complète de raccordement. Le gouvernement ayant annoncé ses intentions plusieurs mois à l'avance, ERDF a reçu un afflux de demandes et n'a souvent pas respecté ce délai.    Le CoRDiS a décidé qu'ERDF a ainsi « méconnu ses obligations contractuelles et réglementaires, tout comme sa propre documentation technique de référence »20 . Puis les producteurs ont saisi les juridictions commerciales aux fins de voir réparer le préjudice né de la perte du bénéfice de l'obligation d'achat lorsque leur modèle économique était remis en cause ou, tout au moins, l'écart entre l'ancien et le nouveau tarif. La Cour de cassation a confirmé21 l'arrêt par lequel la cour d'appel de Paris avait jugé qu'ERDF avait commis une faute en transmettant une PTF au-delà du délai de trois mois fixé par sa DTR, délai qui revêtait selon elle un caractère impératif22 . Sur le préjudice, les solutions retenues par les juridictions du fond ne sont pas encore stabilisées. On signalera par exemple que, par un arrêt du 11 février 2014, la cour d'appel de Versailles a indemnisé un producteur à hauteur de 80 % de la perte subie au motif qu'il pouvait se prévaloir de la perte d'une espérance légitime, après avoir rejeté les causes exonératoires avancées par ERDF (imprévisibilité, extériorité, force majeure résultant de l'afflux des demandes de raccordement et de l'entrée en vigueur du moratoire) 23 . Ce lourd contentieux a rendu plus évidente la nécessité de préciser les délais dans lesquels les gestionnaires de réseaux doivent traiter les dossiers et a également conduit le législateur à prévenir les contentieux en "forfaitisant" l'indemnisation.    La loi de transition énergétique a toutefois prévu une dérogation au délai de dix-huit mois, par une habilitation relativement vague, mais qui semble ne concerner que les grandes installations éloignées du réseau préexistant : sur demande motivée du gestionnaire de réseau, l'autorité administrative peut décider d'une prorogation, « en fonction de la taille des installations et de leur localisation par rapport au réseau ou lorsque le retard pris pour le raccordement est imputable à des causes indépendantes de la volonté du gestionnaire de réseau ».    Il faut enfin signaler que les cahiers des charges des appels d'offres lancés par le gouvernement24 , comme les nouveaux arrêtés fixant les tarifs d'achat, neutralisent les conséquences pour les producteurs du dépassement du délai de raccordement quant à la durée de leur contrat, lorsque ledit dépassement résulte du retard pris dans les travaux de raccordement.    Ainsi par exemple, en matière d'appel d'offres, le cahier des charges portant sur la réalisation et l'exploitation d'installations photovoltaïques sur bâtiments et ombrières de parking de puissance-crête comprise en 100 et 250 kW : « Le candidat s'engage à mettre en service l'installation dans un délai de vingt-deux (22) mois à compter de la notification de la décision des résultats par le ministre chargé de l'énergie. La date de notification est la date d'envoi des courriers de désignation, cachet de la poste faisant foi. Des dérogations à ce délai sont toutefois possibles dans le cas où les travaux de raccordement sont non achevés dans les vingt (20) mois à compter de la notification de la décision des résultats par le ministre chargé de l'énergie et sous réserve que l'exploitant puisse justifier qu'il a mis en œuvre toutes les démarches pour que ceux-ci soient réalisés dans les délais (dépôt d'une demande de raccordement notamment et respect des exigences du gestionnaire de réseau). Un délai supplémentaire de deux (2) mois à compter de la fin des travaux de raccordement est alors accordé »25 . De même l'arrêté du 4 mars 2011 fixant les conditions d'achat de l'électricité produite par les installations photovoltaïques : « Le contrat d'achat est conclu pour une durée de vingt ans à compter de la date de mise en service de l'installation. La date de mise en service de l'installation correspond à la date de mise en service de son raccordement au réseau public. Cette mise en service doit avoir lieu dans un délai de dix-huit mois à compter de la date de demande complète de raccordement au réseau public par le producteur. En cas de dépassement de ce délai, la durée du contrat d'achat est réduite du triple de la durée de dépassement. Le délai mentionné au premier alinéa est prolongé lorsque la mise en service de l'installation est retardée du fait des délais nécessaires à la réalisation des travaux de raccordement et à condition que l'installation ait été achevée dans le délai prévu au premier alinéa. La mise en service de l'installation doit, dans ce cas, intervenir au plus tard deux mois après la fin des travaux de raccordement. »26     La réforme de l'autorisation d'exploiter et le pilotage de la capacité nucléaire    L'article 187 de la loi sur la transition énergétique fait évoluer le dispositif de l'autorisation d'exploiter une installation de production de l'énergie au sens des articles L. 311-5 et sq du Code de l'énergie. L'autorisation, dont le régime est simplifié, devient un outil de pilotage du mix de production électrique dans le but d'atteindre les objectifs de la « programmation pluriannuelle de l'énergie ».    1. La simplification du régime de l'autorisation d'exploiter Principe de la liberté d'installation et restrictions éventuelles  Dans sa nouvelle rédaction, l'article L. 311-1 du Code de l'énergie reprend le principe posé par l'article 9 de la loi du 10 février 2000, selon lequel l'exploitation de toute nouvelle installation de production d'électricité est subordonnée à l'obtention d'une autorisation administrative.    Ce faisant, il accentue une ambiguïté sur les motifs de refus éventuel : alors que la loi ancienne disposait que, « dans le cadre de la programmation pluriannuelle des investissements, les nouvelles installations de production sont exploitées par toute personne (...) titulaire d'une autorisation d'exploiter », le texte impose désormais la compatibilité de l'autorisation et donc du projet avec la programmation pluriannuelle de l'énergie prévue à l'article L. 141-1 du code, mais également la prise en compte de l'équilibre offre-demande, de la sécurité d'approvisionnement, de la nature et de l'origine des sources d'énergie primaire, de l'efficacité énergétique, ainsi que de l'impact sur les objectifs de lutte contre l'aggravation de l'effet de serre. Si aucune demande n'a, à notre connaissance, été officiellement refusée depuis 2000, il est permis de se demander ce qu'il en sera à l'avenir, notamment pour les moyens de production d'électricité à partir d'énergies fossiles. Il est vrai que ces critères sont proches de ceux posés par l'article 7 de la Directive 2009/72/CE du 13 juillet 2009 concernant des règles communes pour le marché intérieur de l'électricité, qui viennent donc limiter le principe de la liberté d'installation27 . On relèvera à cet égard que l'article 7 (4) de cette directive dispose non seulement que « les procédures et critères d'autorisation sont rendus publics », mais encore que « les demandeurs sont informés des raisons d'un refus d'autorisation. Ces raisons doivent être objectives et non discriminatoires; elles doivent en outre être justifiées et dûment motivées ». Si la loi de transition énergétique est silencieuse sur ce point, ces obligations précises et inconditionnelles s'imposent au ministre; elles découlent d'ailleurs du principe d'égalité et des articles L. 211-1 et sq du Code des relations entre l'administration et le public.    Évolution de la définition de l'installation nouvelle  Jusqu'à présent, outre les installations nouvellement construites, étaient considérées comme nouvelles « les installations (...) qui augmentent la puissance installée (d'une installation existante) d'au moins 10 % ainsi que les installations dont la source d'énergie primaire change »28 . Le nouvel article L. 311-1 du Code de l'énergie élève à 20 % le seuil de la « nouvelle installation », qui nécessite donc le dépôt d'une demande d'autorisation d'exploiter.    Délivrance éventuelle d'une autorisation d'exploiter unique  L'article L. 311-5-1 nouveau du Code de l'énergie permet à l'administration de délivrer une autorisation d'exploiter unique pour toutes les installations d'un même site de production, si celles-ci sont proches ou connexes, utilisent la même source d'énergie primaire et ont le même exploitant, sans que cela ne constitue toutefois un droit pour le producteur.    Par exception, l'article L. 311-5-2 nouveau du même code consacre la notion d'« unité de production » sans la définir29 et fixe un seuil de puissance à 800 MW : l'autorité administrative devra délivrer une autorisation par unité de production dès lors que l'installation regroupera plusieurs unités dont la puissance unitaire dépasse ce seuil. Le législateur a en réalité visé ici les seules centrales nucléaires30 , car aucune des unités de production thermiques "à flamme" françaises n'atteint cette puissance aujourd'hui31 .    Transfert de l'autorisation d'exploiter  La loi sur la transition énergétique est venue modifier le régime du transfert de l'autorisation d'exploiter.    L'ancien article L. 311-5 du Code de l'énergie précisait que l'autorisation d'exploiter était « nominative et incessible », et qu'« en cas de changement d'exploitant », elle ne pouvait être « transférée au nouvel exploitant que par décision de l'autorité administrative ». Ce n'était que le rappel du principe général de l'incessibilité des autorisations administratives, confirmé par le Conseil constitutionnel dans sa décision sur la loi de transition énergétique32 . Ce principe interdit en effet la cession du titre par un simple contrat, mais non le transfert de l'autorisation de son titulaire à un tiers par une décision de l'administration.    Peut-être par souci de clarification, le législateur a écrit au nouvel article L. 311-5-4 du code que « l'autorisation est transférée au nouvel exploitant », ce qui pourrait laisser à penser que ce transfert est automatique en cas de changement d'exploitant; or, le cessionnaire doit bien entendu remplir les conditions fixées à l'article L. 311-5.    Outre que l'autorisation d'exploiter a été concentrée sur ses seuls aspects énergétiques, elle devient un outil entre les mains de l'administration au service de la politique énergétique et, plus précisément, de la transformation du mix de production électrique en faisant notamment peser des contraintes croissantes sur la production nucléaire.    2. Un outil de pilotage de l'évolution du mix de production électrique  Il s'agit en réalité des centrales de production thermiques, à flamme et nucléaires.    Restrictions au fonctionnement des installations de production émettant des gaz à effet de serre  Par exception à la très large habilitation de l'article L. 311-1 nouveau du Code de l'énergie, le législateur a pris lui-même l'initiative, à l'article L. 311-5-3 nouveau du même code, d'autoriser l'administration, lorsqu'une installation de production émet des gaz à effet de serre, à restreindre, dans l'autorisation d'exploiter, le nombre maximal d'heures de fonctionnement annuelles, afin d'imposer au producteur le respect des valeurs limites d'émissions fixées par voie réglementaire. Le lien avec la politique climatique est ici évident : cette limitation est en effet à mettre en relation avec les objectifs de réduction des émissions de gaz à effet de serre qui figurent à l'article 1er de la loi sur la transition énergétique et qui sont désormais codifiés à l'article L. 100-4 du Code de l'énergie (« réduire les émissions de gaz à effet de serre de 40 % entre 1990 et 2030 » et « diviser par quatre les émissions de gaz à effet de serre entre 1990 et 2050 »). Le producteur d'électricité à partir d'énergie fossile (gaz, mais surtout fioul et charbon) gagne en sécurité juridique à voir ces contraintes fixées ex ante, même si ces valeurs limites peuvent évoluer.    Plafonnement de la capacité de production nucléaire  Afin de rééquilibrer le mix électrique en faveur des énergies renouvelables (conformément au 4° de l'article L. 100-4 du Code de l'énergie) tout en tenant compte de l'absence d'émission de gaz à effet de serre par les centrales nucléaires (objectif figurant au 1° du même article L. 100-4), la loi de transition énergétique a instauré un plafonnement de la capacité de production d'électricité d'origine nucléaire à la puissance actuellement disponible du parc EDF (c'est-à-dire avant la mise en service du réacteur EPR de Flamanville 3), soit 63,2 GW. Cette contrainte rejoint l'objectif de réduction de la part d'énergie nucléaire dans la production d'électricité à 50 % à l'horizon 2025, inscrit désormais au 5° de l'article L. 100-4 du Code de l'énergie33 . Le nouvel article L. 311-5-5 du Code de l'énergie énonce ainsi que « l'autorisation mentionnée à l'article L. 311-1 ne peut être délivrée lorsqu'elle aurait pour effet de porter la capacité totale autorisée de production d'électricité d'origine nucléaire au-delà de 63,2 gigawatts ».    Concrètement, lorsqu'EDF mettra en service Flamanville 3, d'une puissance de 1600 MW, elle devra simultanément renoncer à une puissance nucléaire équivalente34 . Si l'entreprise est théoriquement libre de choisir, ce n'est un mystère pour personne que les pouvoirs publics privilégient actuellement l'arrêt des deux réacteurs de 900 MW du CNPE35 de Fessenheim. La fermeture anticipée de ce site a en effet été annoncée lors de la campagne électorale de 2012 et est régulièrement débattue depuis lors. EDF ayant fait connaître son refus de demander une autorisation préalable de mise à l'arrêt définitif et de démantèlement de ce CNPE, contraire à ses intérêts et à ceux de ses partenaires industriels36 , les deux seules solutions ouvertes au gouvernement, de lege lata, sont soit l'interdiction de la remise en fonctionnement après un arrêt d'une durée d'au moins deux ans (article L. 593-24 du Code de l'environnement), mais aucun arrêt de tranche d'une telle durée ne s'est jamais produit en France, soit la démonstration par les pouvoirs publics de l'existence de risques graves pour la sécurité, la santé ou la salubrité publique, justifiant la suspension du fonctionnement, puis la mise à l'arrêt par le ministre ou l'Autorité de sûreté nucléaire (articles L. 593-1 et L. 593-20 à L. 593-23 du Code de l'environnement), mais les deux réacteurs ont successivement franchi avec succès leurs visites décennales en 2001 et 201337 et le Conseil d'État a jugé que cette installation ne présentait pas de risques graves pour les intérêts mentionnés à l'article L. 593-1 du Code de l'environnement, notamment à la lumière des tests de résistance conduits après l'accident de Fukushima38 . Par conséquent, les pouvoirs publics ne pourront parvenir à leur fins qu'en recourant à la voie législative ou en usant de la présence majoritaire de l'État au capital d'EDF, dont le PDG est nommé et révoqué en Conseil des ministres. Dans l'un et l'autre cas, cependant, l'État devra indemniser soit l'entreprise, soit ses actionnaires privés et les partenaires étrangers du CNPE de Fessenheim, alors qu'EDF est en réelles difficultés financières et doit faire face à un très lourd programme d'investissements.    Sur cet article très politique, le Conseil constitutionnel a rejeté les moyens tirés par les saisissants de la violation du droit de propriété, du principe d'égalité et de l'obligation européenne de mise en place de conditions non discriminatoires entre producteurs d'électricité. Sur le premier, il a jugé que « les autorisations d'exploiter des installations de production d'électricité accordées par l'autorité administrative ne sauraient être assimilées à des biens objets pour leurs titulaires d'un droit de propriété; que, par suite, les dispositions contestées n'entraînent ni une privation de propriété au sens de l'article 17 de la Déclaration de 1789 ni une atteinte à ce droit en méconnaissance de l'article 2 de cette déclaration »39 . Il a ensuite constaté que ces dispositions affectaient le principe de sécurité juridique, mais il a admis cette atteinte pour deux raisons : d'une part, dès lors que l'article L. 311-5-5 n'a pas imposé l'abrogation immédiate d'une autorisation d'exploiter et qu'il a pour objet de promouvoir la diversification des sources d'énergie et la réduction de la part de l'électricité d'origine nucléaire, « l'atteinte portée aux effets qui peuvent légitimement être attendus de situations légalement acquises est justifiée par des motifs d'intérêt général suffisants et proportionnée aux buts poursuivis »; d'autre part, la loi n'a pas exclu l'indemnisation du préjudice subi par EDF40 . Quant au grief tiré de la violation de l'article 88-1 de la Constitution, il était inopérant à l'encontre de dispositions qui n'avaient pas pour objet de transposer une directive européenne41 .    Mise en cohérence du régime de l'autorisation d'exploiter une INB  En vertu des dispositions des articles L. 593-7 et L. 593-8 du Code de l'environnement, la création d'une installation nucléaire de base (INB) est soumise à une autorisation et cette autorisation est délivrée après enquête publique et avis de l'Autorité de sûreté nucléaire. Cette autorisation prend la forme d'un décret : le décret d'autorisation de création (DAC) 42, qui mentionne l'identité de l'exploitant, la nature de l'installation et sa capacité maximale, ainsi que la durée de l'autorisation, le délai de mise en service et les éléments essentiels que requiert la protection des intérêts mentionnés à l'article L. 593-1 du Code de l'environnement.    Dans le projet de loi de transition énergétique, les dispositions qui figurent désormais à l'article L. 311-5-6 du Code de l'énergie imposaient au futur exploitant d'une installation de production d'électricité d'origine nucléaire de déposer sa demande d'autorisation d'exploiter au plus tard dix-huit mois avant la date de mise en service de l'INB délivrée par l'Autorité de sûreté nucléaire, afin que l'autorité administrative ait plus de temps pour instruire le dossier. Par un amendement présenté par son auteur comme de « cohérence », l'Assemblée nationale a ajouté que la demande doit être déposée « en tout état de cause au plus tard dix-huit mois » avant l'expiration du délai de mise en service mentionné dans le DAC. Cela fait donc peser en réalité un aléa sur l'exploitant. Cette disposition a été contestée devant le Conseil constitutionnel au motif qu'elle introduisait une différence de traitement entre les installations de production d'électricité, contraire au principe d'égalité devant la loi. Ce moyen a été abruptement écarté, au motif que, « en prévoyant un tel délai avant la mise en service pour les demandes d'autorisation d'exploiter une installation nucléaire, qui diffère de celui prévu pour les demandes d'autorisation d'exploiter d'autres installations de production d'électricité, le législateur a traité différemment des situations différentes » et que « cette différence de traitement est en rapport avec l'objet de la loi qui l'établit »43 .    Obligation pour EDF d'élaborer un plan stratégique, sous le contrôle du gouvernement  L'article L. 311-5-7 nouveau du Code de l'énergie prévoit désormais que « tout exploitant produisant plus du tiers de la production nationale d'électricité » établit « un plan stratégique », qui présente « les actions qu'il s'engage à mettre en œuvre pour respecter les objectifs de sécurité d'approvisionnement et de diversification de la production d'électricité fixés dans la première période de la programmation pluriannuelle de l'énergie ». Il s'agit donc d'une obligation pesant spécifiquement sur EDF, destinée à en refaire l'outil principal, sinon de conception comme jadis, du moins de mise en œuvre de la politique nationale en matière de production d'électricité : « Ce plan propose, si besoin, les évolutions des installations de production d'électricité, en particulier d'origine nucléaire, nécessaires pour atteindre les objectifs de la première période de la programmation pluriannuelle de l'énergie). Il est élaboré dans l'objectif d'optimiser les conséquences économiques et financières de ces évolutions, ainsi que leurs impacts sur la sécurité d'approvisionnement et l'exploitation du réseau public de transport d'électricité ». C'est bien l'État qui, désormais, détermine la stratégie et EDF doit la « respecter ».    D'un côté, en effet, la programmation pluriannuelle de l'énergie (PPE) est fixée par décret; elle établit les priorités d'action des pouvoirs publics pour la gestion de l'ensemble des formes d'énergie sur le territoire métropolitain continental, afin d'atteindre les objectifs définis par la loi de transition énergétique44 . Elle couvre deux périodes quinquennales successives et est donc renouvelée tous les cinq ans.    De l'autre côté, le plan stratégique est soumis par EDF au ministre chargé de l'énergie dans les six mois de l'approbation de la PPE. Le ministre apprécie sa compatibilité avec la PPE; en cas de rejet, EDF doit élaborer un nouveau plan. De plus, EDF rend compte, chaque année, devant les commissions permanentes du Parlement chargées de l'énergie, du développement durable et des finances, de la mise en œuvre dudit plan stratégique. Enfin, le législateur confirme la présence d'un commissaire du gouvernement auprès d'EDF45 , qui est informé des décisions d'investissement et qui « peut s'opposer à une décision dont la réalisation serait incompatible avec les objectifs du plan stratégique ou avec la programmation pluriannuelle de l'énergie en l'absence de plan stratégique compatible avec celle-ci »; si l'opposition est confirmée par le ministre chargé de l'énergie, la décision ne peut être appliquée sans révision du plan stratégique.    L'État acquiert ainsi la maîtrise de la stratégie et de la politique d'investissement d'EDF en France, ce que le législateur de 1946 avait soigneusement écarté en prônant la « nationalisation » et non l'« étatisation », et ce qui contredit la logique de normalisation dans laquelle EDF était engagée depuis la libéralisation des marchés de l'énergie, avec notamment sa transformation en société commerciale46 . Vingt ans après l'adoption de la première directive de libéralisation des marchés de l'électricité, l'État se dote ainsi des moyens d'un contrôle sans précédent sur le principal opérateur économique du secteur et en fait l'instrument d'une politique publique centrée sur le territoire national. Au même moment, les grands électriciens allemands se scindent en un groupe centré sur la maximisation de la rente administrée des renouvelables et un autre gérant le parc de centrales thermiques, appelant à terme des subventions au-delà même de la « réserve stratégique »; et le gouvernement britannique accorde à EDF Energy une aide d'État sur soixante ans avec le consentement de la Commission européenne, afin de remplacer par le nucléaire le gaz de la Mer du Nord désormais épuisé. Il est permis de se poser des questions sur la libéralisation de l'Europe de l'énergie.     La réforme du régime des concessions hydroélectriques    La France se singularise dans l'Union européenne par la part très prépondérante de l'opérateur historique EDF dans la production d'électricité. Ailleurs, les opérateurs historiques nationaux ont été scindés par la loi (Royaume-Uni) ou forcés de céder à des concurrents une part importante de leur parc de production (Italie), tandis que la concurrence se développait dans d'autres pays entre les anciens monopoles régionaux dont le contrôle est souvent passé du secteur public au secteur privé (Allemagne, Espagne). EDF détient non seulement la totalité du parc de production nucléaire, mais également 80 % du parc hydraulique concédé, c'est-à-dire celui constitué des unités de production d'une puissance supérieure à 4,5 MW, sachant que l'essentiel du potentiel hydrologique est désormais exploité. Ces concessions hydroélectriques présentent de nombreuses particularités, dont les trois suivantes : les investissements y sont presque partout amortis, de sorte qu'elles sont très rentables; ce sont des concessions d'État47 , c'est-à-dire des contrats administratifs traditionnellement passés de gré à gré avec le ministre chargé de l'énergie; enfin, accordées pour de longue durée (souvent 75 ans), elles viennent progressivement à renouvellement.    Dans ce contexte, il était inévitable que la Commission européenne, proprio motu ou sous la pression des concurrents d'EDF, attache une importance particulière à l'ouverture à la concurrence de ce potentiel de production.    En 2006, en votant l'article 7 de la Lema48 , le Parlement a supprimé le droit de préférence du concessionnaire sortant et le gouvernement a pris l'initiative de confier à plusieurs corps d'inspection une mission d'expertise sur le renouvellement des concessions. Les auteurs de ce rapport ont préconisé une procédure de mise en concurrence49 et les textes ont été adaptés en conséquence50 .    Un programme de renouvellement des concessions avec mise en concurrence a donc été annoncé par le gouvernement le 22 avril 2010, portant sur dix vallées et 20 % du parc. Pour maintenir la cohérence de l'exploitation au sein d'une même vallée (pour des raisons d'optimisation énergétique, économique, écologique et de sûreté d'exploitation) il a notamment été décidé de regrouper certains ouvrages au sein d'une même concession en résiliant certaines concessions, la soulte due au concessionnaire sortant étant mise à la charge du nouveau concessionnaire (méthode dite « des barycentres »51 ).    Ce projet a suscité de nombreuses et vives oppositions, notam-ment de la part du personnel des ouvrages et de certains des élus locaux concernés. Les gouvernements successifs ont tergiversé et le processus n'a jamais débuté, suscitant l'incompréhension des opérateurs candidats, dont certains ont quitté la France après avoir travaillé en vain sur diverses concessions. Le Parlement s'est saisi du dossier en 2013, à travers un rapport d'information sur l'hydroélectricité, présenté par les députés Marie-Noëlle Battistel et Éric Straumann : ceux-ci ont proposé quatre solutions pour ne pas mettre les concessions en concurrence52 .    La Commission européenne a perdu patience et, selon la presse, aurait finalement adressé à la France une mise en demeure le 22 octobre 2015, bien que le Parlement ait renoncé à la tentation et voté dans la loi de transition énergétique un dispositif particulier d'ouverture à la concurrence. C'est l'objet des ar-ticles 118 et 119, qui modifient le livre V du Code de l'énergie, issu de la loi du 16 octobre 1919 relative à l'utilisation de l'énergie hydraulique.    Les principaux apports de la loi du 17 août 2015 peuvent être résumés en trois points. On dira également un mot de quelques « mesures techniques complémentaires ».    1. Le regroupement des concessions hydroélectriques  Inspiré de la logique des barycentres, l'article 116 de la loi de transition énergétique crée au sein du Code de l'énergie les articles L. 521-16-1 à L. 521-16-3.    L'article L. 521-16-1 permet au gouvernement de regrouper en une seule concession, par décret en Conseil d'État, « plusieurs concessions hydrauliques formant une chaîne d'aménagements hydrauliquement liés » et actuellement confiés à un même concessionnaire, « afin d'optimiser l'exploitation de cette chaîne au regard des objectifs mentionnés aux articles L. 100-1, L. 100-2 et L. 100-4 du présent code, ou des objectifs et exigences mention-nés à l'article L. 211-1 du Code de l'environnement ». La date d'échéance commune doit être calculée à partir des dates d'échéance prévues par les cahiers des charges des contrats regroupés et les modalités de calcul garantir au concessionnaire actuel le maintien de l'équilibre économique, apprécié sur l'ensemble des concessions regroupées. Un décret en Conseil d'État doit préciser les critères utilisés pour ce calcul et les conditions et modalités du regroupement.    Cette méthode a été préférée à celle du rachat des concessions les plus longues, qui aurait permis une mise en concurrence rapide, car le rachat est prévu aux cahiers des charges et constitue en tout état de cause un principe général du droit des contrats administratifs. Gouvernement et Parlement ont estimé que la soulte demandée au nouveau concessionnaire au profit de l'ancien aurait été prohibitive et aurait affecté l'égalité dans la concurrence.    L'article L. 521-16-2 adapte la méthode des barycentres aux cas dans lesquels des concessionnaires distincts sont titulaires de concessions hydrauliques formant une chaîne d'aménagements hydrauliquement liés. Par décret en Conseil d'État, le gouvernement peut également fixer une date d'échéance commune à tous les contrats dans le but de regrouper ces concessions lors de leur renouvellement, afin d'optimiser l'exploitation de cette chaîne au regard des mêmes objectifs d'intérêt général, tout en garantissant le maintien de l'équilibre économique, apprécié concessionnaire par concessionnaire sur l'ensemble de leurs concessions concernées. Pour résoudre la difficulté née de la diversité des opérateurs, les auteurs du décret en Conseil d'État pourront décider que les opérateurs dont les concessions auront été prolongées verseront une indemnité à ceux dont la durée des concessions aura été réduite. Un décret en Conseil d'État à portée réglementaire doit notamment préciser les critères utilisés pour le calcul de la date d'échéance et de l'indemnité, ainsi que les conditions et modalités du regroupement.    Enfin, l'article L. 521-16-3 permet au gouvernement de proroger les concessions arrivant normalement à renouvellement « lorsque la réalisation de travaux nécessaires à l'atteinte des objectifs mentionnés aux articles L. 100-1, L. 100-2 et L. 100-4 et non prévus au contrat initial l'exige », y compris en excédant la durée maximale de soixante-quinze ans fixée par les lois antérieures. On ne peut exclure que la Commission européenne s'intéresse particulièrement à cette disposition.    2. Les sociétés d'économie mixte hydroélectriques (Sem-H)  L'article 118 de la loi de transition énergétique crée, aux articles L. 521-18 à L. 521-20 du Code de l'énergie, une nouvelle catégorie de sociétés d'économie mixte, dans le but d'atténuer les conséquences politiques de la mise en concurrence des concessions hydrauliques. Ses auteurs se sont inspirés de la logique des « partenariats public-privé institutionnalisés » (PPPI) de la Commission européenne53 , consacrés par la Cour européenne de justice54 et qui ont précédemment donné lieu à la création de la société d'économie mixte à opération unique55 . Mais, tandis que cette dernière est une Sem locale, la Sem H est constituée par l'État et à son initiative.    Le gouvernement peut désormais créer, pour chaque concession, une « société d'économie mixte hydroélectrique » avec au moins un opérateur économique, qualifié d'« actionnaire opérateur », et, le cas échéant, avec des collectivités locales riveraines des cours d'eau dont la force hydraulique est exploitée, des entreprises ou des organismes dont le capital est exclusivement détenu par des personnes morales de droit public (entreprises et organismes qualifiés de « partenaires publics »). Une SEM H est une société anonyme « à opération unique », pouvant ne compter que deux actionnaires, constituée exclusivement en vue de la conclusion et de l'exécution d'une concession dont l'objet est l'aménagement et l'exploitation d'une installation de production d'électricité ou de plusieurs ouvrages constituant une chaîne d'aménagements hydrauliquement liés. L'État est maître de la composition de l'actionnariat de chaque SEM H.    Néanmoins, l'État, les collectivités territoriales et les partenaires publics doivent détenir ensemble entre 34 % et 66 % du capital de la société et entre 34 % et 66 % des droits de vote dans les organes délibérants56 . Réciproquement, la part du capital et des droits de vote détenue par l'actionnaire opérateur doit donc être d'au moins 34 % et au plus de 66 %. Les statuts de la société et le pacte d'actionnaires doivent permettre à l'État de rester actionnaire pendant toute la durée de la concession. La Sem H est dissoute de plein droit au terme de la concession ou à la suite de sa résiliation.    Conformément à la logique des PPPI, la répartition du capital, les règles de gouvernance et les modalités de contrôle de l'État, des collectivités territoriales et des partenaires publics sur la société, les règles de dévolution de l'actif et du passif de la société lors de sa dissolution, les caractéristiques principales du contrat de concession conclu entre l'État et la Sem H et du cahier des charges, les contrats de sous-traitance éventuellement conclus, ainsi que l'estimation provisoire de la quote-part des investissements initiaux à la charge de l'État, des collectivités territoriales et des partenaires publics doivent être fixés préalablement à la sélection de l'actionnaire opérateur et portés à la connaissance des candidats éventuels, de sorte que la mise en concurrence soit objective, transparente et non discriminatoire.    Enfin, la sélection de l'actionnaire opérateur et l'attribution de la concession à la Sem H interviennent au terme d'une procédure unique d'appel public à la concurrence, qui respecte les mêmes règles et critères d'attribution que la procédure de renouvellement des concessions prévue à l'article L. 521-16 du Code de l'énergie et qui est conduite par l'État, selon des modalités encadrées par la loi et définies par décret en Conseil d'État.    Les redevances de concessions hydroélectriques  La rentabilité des concessions hydroélectriques est réputée élevée, même si le faible niveau actuel des prix de gros de l'électricité affecte naturellement le résultat de leur exploitation. Le partage de la rente hydraulique est donc, avec les arbitrages entre les divers usages de l'eau et l'optimisation du potentiel hydroélectrique (national et non émetteur de gaz à effet de serre), l'un des trois enjeux traditionnels de cette législation. Conjugué au caractère territorial, donc national de la ressource hydraulique, c'est l'un des arguments des opposants à l'ouverture des concessions à la concurrence d'opérateurs étrangers. Mais cette même dimension tellurique alimente une autre des critiques adressées de manière récurrente au régime de l'hydroélectricité : celle des collectivités locales et des populations voisines des cours d'eau exploités, qui subissent les contraintes portant sur les autres usages de l'eau, alors que l'État perçoit seul les redevances versées par les concessionnaires.    Sur ces questions délicates, la loi de transition énergétique a cherché à établir de nouveaux équilibres, en récrivant l'article L. 523-2 du Code de l'énergie.    Le concessionnaire versera désormais à l'État une redevance proportionnelle aux recettes des concessions renouvelées ou prolongées, sous réserve d'un taux plafond déterminé pour chaque concession et connu des candidats opérateurs. En cas de regroupement de concessions, la redevance, dont le taux sera fixé pour chaque concession, sera prise en compte dans la fixation de la nouvelle date d'échéance garantissant le maintien de l'équilibre économique. On comprend que l'État cherche à capter une fraction accrue de la rente hydraulique, au-delà d'un seuil de rentabilité suffisant pour attirer au moins un candidat par concession. Il conviendra toutefois d'éviter de constituer ainsi une barrière à l'entrée.    Le cas échéant, le taux de la redevance tiendra compte des volumes et des prix de vente de l'électricité que le concessionnaire s'engage à céder soit à ses filiales, soit à des industriels membres de sociétés de capitaux qui ont pour objet la conclusion de contrats d'approvisionnement à long terme auprès de producteurs d'électricité d'une durée minimale de quinze ans57 , soit à des consommateurs industriels électro-intensifs58 investissant dans la concession et partageant les risques d'exploitation par des contrats d'une durée supérieure à dix ans ou allant jusqu'au terme de la concession. Ces dispositions répondent à l'argument, également plus politique qu'économique, selon lequel la ressource hydraulique, parce que nationale, devrait être consacrée en priorité à lutter contre les délocalisations industrielles. Il est vrai qu'il rejoint une réalité historique : celle des industriels qui, avant la loi de nationalisation du 8 avril 1946, avaient développé leurs propres moyens de production d'électricité en construisant des ouvrages hydrauliques (compagnies ferroviaires, aluminiers, chloriers...). Dans le contexte actuel, ce sera sans doute également un sujet de discussion avec la Commission européenne.    Enfin, il est désormais prévu qu'un douzième de la redevance sera affecté aux communes (ou groupements de communes) sur le territoire desquelles coulent les cours d'eau exploités. La répartition entre les communes sera proportionnelle à la puissance hydraulique devenue indisponible dans les limites de chaque commune du fait de l'ouvrage hydroélectrique. Avec la participation au capital des SEM H, l'amélioration de l'information des collectivités territoriales et des riverains sur l'exécution de la concession et leur participation à la gestion des usages de l'eau, cohérentes avec la participation croissante des élus locaux et des populations à la politique énergétique, c'est l'amorce d'un partage de la rente hydraulique entre l'État et les collectivités locales.    4. Mesures techniques complémentaires propres à l'hydroélectricité  Sur le fondement de l'article 38 de la Constitution, l'article 119 de la loi de transition énergétique habilite le gouvernement à prendre par ordonnance, dans les six mois, diverses mesures de clarification du régime juridique de l'hydroélectricité.    Il s'agit notamment de réformer le régime des sanctions applicables aux concessions hydrauliques (en particulier pendant la phase délicate de « fin de concession ») 59, d'instituer des sanctions à l'encontre des auteurs d'actes portant atteinte à l'intégrité, à l'utilisation ou à la conservation du domaine hydroélectrique concédé (pour lutter en particulier contre les décharges sauvages autour des lacs ou des cours d'eau) ou encore de permettre l'institution des servitudes nécessaires à l'exploitation d'une concession hydroélectrique (il faut actuellement passer par une déclaration d'utilité publique à l'occasion de travaux).    Mais on soulignera plus particulièrement l'invitation ainsi faite au gouvernement de « préciser les conditions dans lesquelles sont exploitées les installations hydrauliques concédées avant le 16 juil-let 1980 et d'une puissance comprise entre 500 et 4 500 kilowatts pendant la période temporaire qui va de l'expiration de la concession jusqu'à l'institution d'une nouvelle concession ou à la délivrance d'une autorisation, dans le cas où l'ouvrage relève de ce régime, ainsi que, dans ce dernier cas, l'articulation entre la procédure d'autorisation et la procédure de gestion des biens faisant retour à l'État en fin de concession ». L'enjeu est la clarification de la situation des concessions "autorisables", c'est-à-dire des petites installations initialement concédées, mais dont le renouvellement n'est plus possible sous forme concessive depuis l'intervention de la loi n° 80-531 du 15 juillet 1980 qui a relevé de 500 kW à 4,5 MW le seuil séparant les ouvrages devant faire l'objet d'une autorisation de ceux qui relèvent de la concession60 . C'est une bonne chose, car : d'une part, les conséquences de ce changement de seuil sur les concessions en cours n'ont jamais été tirées par le législateur, suscitant de nombreuses questions61; et, d'autre part, l'administration a laissé perdurer trente-cinq ans cette situation d'incertitude juridique, souvent bien au-delà du terme contractuel des concessions.
</article>
<article titre="Chiffres clés des énergies renouvelables - Édition 2015" date="2015 12 14" journal="Rapports - Environnement - MEDDTL">
64 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ...  Chiffres clés des énergies renouvelables décembre 2015 www.statistiques.developpement-durable.gouv.fr  COMMISSARIAT GÉNÉRAL AU DÉVELOPPEMENT DURABLE  Décembre 2015  Chiffres clés des énergies renouvelables  Édition 2015  Service de l’observation...    Pour voir le rapport :  http://www.developpement-durable.gouv.fr/IMG/pdf/Rep_-_Chiffres_cles_energies_renouvelables.pdf
</article>
<article titre="En Afrique, les « gogos » du marché carbone crient à l'aide" date="2015 12 11" journal="Le Monde.fr">
Michel Ngapanoun, directeur général d'Hysacam une société spécialisée dans la collecte des déchets au Cameroun, crie à l'arnaque. Il croyait faire un investissement rentable tout en contribuant à la lutte contre le changement climatique. Il se retrouve avec des centaines de milliers de crédits carbone et pas à un acheteur à l'horizon.  « Il faut sauver le MDP, il faut sauver Hysacam » , exhorte le chef d'entreprise en s'interrogeant sur le sort de ses 5 000 salariés si personne ne lui vient en aide. Hysacam traite les ordures de seize villes au Cameroun. MDP, pour Mécanisme de développement propre, est l'un de ces sigles barbares dont regorge la Convention des Nations unies sur les changements climatiques (COP) et derrière lequel se cache l'un des principaux instruments imaginés pour réduire les émissions de gaz à effet de serre. Il permet à une entreprise d'un pays industrialisé de financer un projet sobre en carbone dans un pays en développement afin de récupérer des crédits carbone qui lui permettront de respecter à moindre coût ses objectifs d'émissions. Cela peut être la construction d'une éolienne, d'un petit barrage hydroélectrique, une unité de méthanisation sur une décharge d'ordures... En principe, tout le monde est gagnant.   Sauf que l'Afrique a raté le train. Quand elle a voulu prendre sa part des MDP, la Chine et les autres pays émergents avaient raflé toute la demande et, en Europe, seule zone à avoir mis en place un marché carbone susceptible d'absorber ces crédits MDP, les prix s'étaient effondrés.   « L'ONU n'a pas tenu ses engagements »   Muni de plaquettes d'information en papier glacé qu'il distribue à qui veut, le Camerounais désabusé raconte son histoire.  « Quand le protocole de Kyoto est entré en vigueur en 2005, une campagne de sensibilisation a été menée au Cameroun, des consultants se réclamant des Nations unies sont venus nous vanter les bienfaits des MDP, et j'ai décidé de me lancer seul,  explique-t-il.  On me garantissait alors un prix de 11 euros la tonne et Orbeo, une société de courtage en crédits carbone, s'était engagée à me les racheter. Je me suis complètement fait avoir. » Orbeo était alors une filiale de la Société générale à qui l'entrepreneur a emprunté une grande partie des 7 millions d'euros pour équiper d'unités de méthanisation ses deux sites de traitement des déchets de Yaoundé et de Douala. Lorsque le marché du carbone s'est effondré, Orbeo - qui depuis a mis la clé sous la porte - s'est retiré. Mais les deux unités de méthanisation continuent de produire 400 crédits carbone par jour, que, dans le meilleur des cas, M. Ngapanoun parvient à placer sur des marchés de compensation volontaire à 3 euros la tonne.  « Grâce à mon projet, 300 000 tonnes de gaz à effet de serre ont été séquestrées au lieu d'être relâchées dans l'atmosphère. Mais l'ONU n'a pas tenu ses engagements »,  dénonce-t-il, presque encore surpris de constater que ce marché du carbone, qu'il croyait garanti par le protocole de Kyoto, se soit dérobé sous ses pieds.   Beaucoup d'autres entrepreneurs se sont fait piéger.  « Le Kenya a développé de grands projets dans la géothermie et l'éolien en intégrant une partie de leur financement sur des MDP. Aujourd'hui, nous avons les crédits, mais nous nous demandons où est passé le marché »,  témoigne Maurice Otieno, de l'Autorité nationale de l'environnement. Même écho en Zambie, dont le gouvernement a voulu stimuler la cogénération d'électricité à partir de la bagasse récupérée après la transformation de la canne à sucre.  « Nous importons 100 % de notre énergie d'Afrique du Sud, c'était un moyen de diversifier nos sources d'approvisionnement et d'introduire de nouvelles technologies dans le pays »,  raconte Khetiswe Khumalo du ministère de l'environnement et du tourisme.   « Les investisseurs quittent le marché et les projets ne peuvent plus être menés à terme,  analyse Adriann Tas, directeur de Carbon Africa, une société de conseil et de trading en finance carbone installée à Nairobi.  Les Africains sont montés dans le train trop tard. La compréhension des règles du MDP, l'élaboration des projets a pris un temps fou. Maintenant, il n'y a plus de demande. »  Les chiffres des Nations unies parlent d'eux-mêmes : sur les 7 681 projets enregistrés à travers le monde, moins de 3 % sont développés en Afrique.   Les promesses de la monnaie carbone   « Ce mécanisme était trop compliqué pour l'Afrique. Les Nations unies auraient dû très vite simplifier les règles et exclure les pays qui en avaient beaucoup profité pour laisser de la place aux pays les moins avancés »,  défend Gareth Philips, de la division changement climatique de la Banque africaine de développement (BAD).   Le MDP, est-il mort ?  « Ce mécanisme n'a pas d'avenir dans le cadre du futur accord sur le climat. Chaque pays devra faire des efforts pour réduire ses émissions sans recourir à des méthodes de compensation dans un pays voisin »,  tranche l'expert de la BAD.   La place des mécanismes de marché et de la compensation alimente en tout cas l'un des plus vifs débats de la COP21. Alors que la conférence joue les prolongations, cela reste un des points qui n'a pas été tranché. Quelle que soit la réponse, il est peu probable qu'elle apporte une solution rapide à Michel Ngapanoun et à tous ceux qui, en Afrique, ont cru un peu trop vite aux promesses de la monnaie carbone.
</article>
<article titre="COP21 : deux aventuriers témoignent sur France 5" date="2015 12 9" journal="TV Mag (site web)">
Entre la question de l'eau et les Frères La Tullaye, c'est une véritable passion. Après quatre expéditions en aventuriers, pour mieux comprendre ce sujet et sensibiliser à ses enjeux, on pourrait se dire qu'ils ont fait le tour de la question. Et bien non! Après leur tour du monde Hydrotour en 2003 (qui a donné lieu à un livre) Expédition Yantsé (en Chine) Expédition Kachgar (la Route de la Soie jusqu'en Asie centrale) et Expédition Grand Rift (d'Israël à la Tanzanie), ils n'ont pas pu s'empêcher de récidiver. Entre temps, Loïc est devenu père de deux enfants, mais le virus de l'aventure est tenace. Alors l'an dernier, ils n'ont pu s'empêcher de se lancer dans un nouveau périple, direction l'Amérique du Sud cette fois. D'avril 2014 à mars 2015, toujours au volant de leurs side-cars - le moyen de locomotion pour tous leurs leurs périples, à l'exception de leur tour du monde en 2CV - Loïc et Geoffroy de La Tullaye ont parcouru 14 000 kilomètres en cinq mois en sillonnant quatre pays: Brésil, Pérou, Bolivie et Chili, de São Paulo à la Terre de Feu.    Paysages époustouflants    Ils sont revenus avec dans leurs bagages une série documentaire en trois épisodes de 52 minutes que France 5, qui leur a ouvert son antenne depuis des années, diffuse, pour la deuxième fois, trois mercredis d'affilée à partir d'aujourd'hui.   Sur fond de paysages époustouflants, des contreforts amazoniens aux champs de glace patagoniens en passant par les glaciers de la cordillère des Andes, les deux baroudeurs vont à la rencontre des populations locales pour essayer de comprendre leur rapport à l'eau, à travers leurs modes de vie et leurs modes de production. Avec le style décontracté et naturel qui leur est propre, loin des documentaires traditionnels, et toujours une touche d'humour, même dans les situations les plus cocasses (comme une panne de scooter en pleine zone désertique), ils rendent leurs reportages vivants à travers une série de témoignages.   Une véritable désolation    A São Paulo, ce sont les habitants qui leur expliquent à quel point la surpopulation et les égouts à ciel ouvert dans la favela de Vila Bella rendent impropre à la consommation l'eau du lac Guarapiringa. Plus loin, le directeur d'un site de cogénération montre comment la bagasse de la canne à sucre alimente les chaudières de son usine. Après avoir passé une nuit sur un bateau de pêcheurs d'anchois et assisté à la fonte des glaciers à La Paz, la situation qu'ils découvrent au lac Poopo (Bolivie) - situé en amont du lac Titicaca dans la plaine de l'Altiplano - pollué au zinc et au cadmium par l'exploitation minière, est une véritable désolation.    Changement de comportement    De retour du Pérou, où ils ont participé au pèlerinage des glaces, du désert d'Atacama (Chili), le plus aride au monde et du salar d'Uyuni (Bolivie), la plus vaste étendue de sel au monde avec plus de 10 000 kilomètres carrés, les deux frères sont rentrés avec une même conviction: «en tant que consommateurs, nous sommes les seuls à pouvoir initier un changement de comportement».   Si France 5 a choisi de diffuser ce road-movie environnemental pendant la COP21, c'est parce qu'il apporte, à travers le prisme de la conquête de l'eau, un éclairage inédit sur l'impasse climatique actuelle. Par exemple, lorsqu'on y apprend que 80 % de la production énergétique du Brésil provient de l'hydroélectricité (les barrages). En fait, l'intuition des deux aventuriers est très simple: l'eau ne devrait-elle pas devenir un indicateur au même titre que le C02, pour guider l'homme dans sa quête d'équilibre entre développement humain, exploitation des ressources naturelles et respect de l'environnement? Une question à méditer.    Expédition Amérique du Sud, série de trois documentaires de 52 minutes, production BIGLO Productions et Belle Bleue. Diffusion sur France 5 en deux vagues: les samedis 28 novembre, 5 décembre et 12 décembre à 17H10 et les mercredis 9, 16 et 23 décembre à 15h40. Voir aussi : COP21: les chaînes à l'heure du climat    La Glace et le Ciel: Luc Jacquet rend hommage à Claude Lorius    La page Facebook des Frères La Tullaye    Vidéo de leur intervention au TEDx Panthéon Sorbonne
</article>
<article titre="CONFIDENTIEL llllllllllllllllllllllllllllllllllllllllllllllllllllll" date="2015 12 1" journal="Environnement Magazine, no. 1743">
l SANTÉ  L'Anses déconseille la généralisation des retardateurs de flamme,  jugés inefficaces et toxiques, dans le mobilier rembourré. l CIRCUITS COURTS  Après un recensement des marchés de restauration des ministères,  celui de l'Agriculture promet pour début 2016 une feuille de route sur l'approvisionnement local. l MOBILITÉ  La Commission européenne propose jusqu'au 15 janvier  une consultation sur l'étiquetage des voitures. l JUSTICE  Le tribunal de commerce de Bobigny a condamné Engie  pour la révocation injustifiée de Jean-Michel Germa de la présidence de La Compagnie du vent. l RECYCLAGE  L'association 2ACR est chargée de l'animation  du réseau collaboratif national pour développer le recyclage des plastiques. l MOBILITÉ  Primagaz commercialisera en 2016 en France  du biopropane, principalement produit à partir d'huiles et de déchets végétaux. l FRET  Un plan national d'action sur le fret fluvial  est annoncé pour juin 2016. l VHU  La France a atteint avec un an d'avance  l'objectif de 85 % de réutilisation et de recyclage des VHU, mais rate celui de 95 % de réutilisation et valorisation, avec 89,3 %. l ENR  La prime Bagasse passe dans les zones non interconnectées d'outre-mer  de 13 à 14,50 euros la tonne de canne à sucre. l TRANSITION ÉNERGÉTIQUE  L'Ademe et GRDF renouvellent leur coopération  pour trois ans.
</article>
<article titre="Quelle ambition pour la filière betterave-sucre française dans la perspective de la fin des quotas européens ?" date="2015 11 30" journal="Rapports - Agroalimentaire - MAAF">
106 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... 1  Rapport n° 15016  établi par  Serge Lhermitte Délégué ministériel aux entreprises agroalimentaires  Thierry Berlizot Ingénieur général des ponts, des eaux et des forêts   Septembre 2015  2  3  REMERCIEMENTS  Nous ayant...    Pour voir le rapport :  http://agriculture.gouv.fr/sites/minagri/files/rapport_mission_sucre.pdf
</article>
<article titre="Un consortium pour un PLA plus éthique" date="2015 11 20" journal="Emballages magazine.com">
Corbion Purac a développé un bioplastique fabriqué à partir de déchets agricoles.-   La majorité des bioplastiques sont fabriqués à partir de ressources végétales destinées à l'alimentation humaine et animale telles que le maïs, le blé, la canne à sucre ou la betterave. Cela pose un problème de nature éthique que les grandes marques comme Coca-Cola, Danone ou PepsiCo appréhendent, ce qui les conduit à demander à leurs fournisseurs d'emballages de préférer- lorsque c'est possible- des bioplastiques dits de « seconde génération », issus de ressources non alimentaires. Déchets agricoles   Parmi les principaux fabricants mondiaux d'acide polylactique (PLA), un bioplastique servant à produire, entre autres, des films et des barquettes compostables, Corbion Purac ambitionne de remplacer les ressources végétales à parti desquelles il obtient ses produits par des déchets agricoles. Dans ce but, il crée un consortium visant à rassembler les utilisateurs de PLA afin de doper ses moyens en R&amp;D. Le chimiste néerlandais a déclaré avoir réussi à produire de l'acide lactique, le principal constituant du PLA, à partir de bagasse, fourrage de maïs, paille de blé et autres produits non comestibles comme les copeaux de bois. Il a besoin toutefois de davantage de moyens pour poursuivre la recherche et industrialiser le procédé. D'où son appel.   Corbion Purac est une division de Corbion. Ce dernier a réalisé, en 2014, un chiffre d'affaires de 770 millions d'euros avec 1893 personnes. La société se revendique leader sur le marché dans la production d'acide lactique et de lactides ainsi que dans les mélanges issus de ces dérivés contenant des enzymes, des émulsifiants, des minéraux et des vitamines. Ses produits sont utilisés, notamment, par l'industrie alimentaire et pharmaceutique.
</article>
<article titre="Le groupe français Albioma mise sur la biomasse" date="2015 11 19" journal="Les Echos, no. 22069">
La transition énergétique dans les DOM passera par Albioma. L'ex-Séchilienne-Sidec assure une part importante de la production d'électricité outre-mer avec ses centrales thermiques au charbon et à la bagasse (le résidu fibreux de la canne à sucre) : 30 % en moyenne (hors Mayotte) et même 53 % à La Réunion. «  La montée en puissance de la biomasse sera le principal contributeur à l'atteinte des objectifs dans les DOM  » , assure Michel Magnan, directeur régional d'EDF SEI. Le mix actuel du groupe, qui réalise 350 millions d'euros de chiffre d'affaires, n'est pour l'instant pas très vertueux : en 2014, 59 % de sa production électrique provenait du charbon (et jusqu'à 80 % à La Réunion). Mais le potentiel d'amélioration est élevé. Les usines d'Albioma comptent parmi les rares au monde à être capables de brûler indifféremment de la biomasse et du charbon. «  Nous avons développé ce savoir-faire il y a vingt ans pour valoriser la bagasse issue des cultures sucrières  » , explique Jacques Pétry, le PDG de la société. «  Il devrait nous permettre de faire tomber la proportion de charbon à 55 % en 2015 et à 20 % en 2023.  » A elle seule, cette évolution permettrait de faire passer la part des énergies renouvelables dans l'électricité des DOM de 28 % en 2013 à 47 % en 2023.   Albioma prévoit d'investir pour cela 350 millions d'euros sur dix ans, à la fois pour remplacer le charbon par de la biomasse dans ses centrales existantes et pour construire de nouvelles centrales 100 % biomasse. L'adaptation des équipements (stockage, etc.) ne sera pas neutre financièrement : le groupe a calculé que l'impact sur la facture énergétique de La Réunion serait compris entre 40 et 50 millions d'euros par an. Mais, à terme, il parie sur la compétitivité de ces centrales. A la Martinique, il mettra en service en 2017 une nouvelle usine 100 % biomasse, Galion 2, dont l'électricité sera vendue au prix de 190 euros le MWh. «  C'est moins cher que le fuel, et équivalent au coût de l'électricité d'une nouvelle centrale à charbon, compte tenu des nouvelles normes  » , assure Pascal Langeron, directeur général adjoint d'Albioma.   Recherche de biomasse Le pari n'a toutefois rien d'évident. Notamment parce qu'il faudra trouver les ressources à brûler. «  Le potentiel maximum de la bagasse est déjà atteint  » , souligne Pascal Langeron. Cela prendra du temps. Albioma prévoit de démarrer la transition, à la Martinique ou à La Réunion, avec des pellets de bois importés, une option pas totalement verte au regard de son bilan carbone global. «  Il faudra aussi développer des filières locales, à base de paille de canne à sucre, ou de déchets verts provenant de la collecte sélective et de l'exploitation forestières  » , avance Pascal Langeron. Les cultures énergétiques pourraient également être envisagées, à condition toutefois qu'elles n'entrent pas en conflit avec l'agriculture locale. Enfin, Albioma a aussi passé à La Réunion un accord avec Bioalgostral, une start-up qui produit du bioéthanol à partir de microalgues. Une ressource potentiellement immense, à condition de parvenir à en réduire les coûts.   A. F.
</article>
<article titre="La difficile marche de l'outre-mer vers les énergies renouvelables" date="2015 11 19" journal="Les Echos, no. 22069">
A La Réunion, le mois de novembre se trouve en plein coeur de la campagne sucrière. Les « cachalots » , ces camions conteneurs vert vif qui transportent la bagasse (le résidu fibreux de la canne à sucre), sillonnent les routes de l'île. Ils transportent leur précieux chargement jusqu'aux deux centrales thermiques d'Albioma, au Gol et à Bois-Rouge, où il sera brûlé et transformé en électricité. Ici, la bagasse représente plus de 10 % de la production d'électricité. Une ressource importante, mais qui, seule, ne permettra pas à La Réunion d'atteindre les objectifs fixés par la loi sur la transition énergétique adoptée cet été. Comme la Guadeloupe, la Martinique et la Guyane, ce département d'outre-mer devra produire plus de la moitié de ses électrons à partir d'énergies renouvelables en 2020. Pour atteindre l'objectif, tous les leviers devront être mobilisés.  « Nous sommes en train de travailler sur les trajectoires »  , explique Jean-Claude Futhazar, directeur général adjoint du développement durable à la région Réunion. Les programmations pluriannuelles de l'énergie (PPE) devront être adoptées au début de 2016.   Si, en moyenne, la part des énergies vertes dans ces DOM atteint aujourd'hui 28 %, l'effort à consentir ne sera pas le même selon les territoires. A la Martinique elle est de 6 % et en Guyane elle atteint 62 %, grâce notamment à l'hydroélectricité. Le taux est de 18 % en Guadeloupe et de 38 % à La Réunion, où la mise en service, fin 2013, d'une centrale au fuel par EDF fait encore grincer les dents.  « Les élus locaux y étaient très opposés »  , rappelle Jean-Claude Futhazar.   Dans ces territoires isolés, l'intégration des énergies renouvelables devrait être plus facile, car le coût de production de l'électricité y est bien plus élevé qu'en métropole, qui bénéficie du nucléaire amorti et de l'interconnexion aux réseaux. Selon le dernier rapport de la CRE (Commission de régulation de l'énergie), il était compris entre 200 et 250 euros le MWh, contre moins de 40 euros sur le marché de gros en métropole. Dans les îles, le solaire et l'éolien se comparent au charbon ou au fuel importés.   Certains gisements verts sont déjà bien exploités, comme l'hydroélectricité ou la bagasse. Le remplacement du charbon par de la biomasse dans les centrales d'Albioma devrait contribuer à se rapprocher de l'objectif (lire ci-dessous). Le développement des énergies intermittentes est plus compliqué.  « Au-delà de 30 % de la capacité électrique installée, elles risquent de déstabiliser le réseau »  , assure Michel Magnan, directeur régional d'EDF SEI (Systèmes Energétiques Insulaires). Cette limite imposée est contestée par les producteurs de solaire et d'éolien. EDF pourrait la revoir sensiblement à la hausse, à 35 % en 2018 et entre 40 et 45 % en 2023.  « Nous testons un système de stockage centralisé, qui permettrait de prendre le relais en cas de baisse brutale de la production »  , explique Michel Magnan.   Les autres voies explorées La chute récente du coût des batteries pourrait aussi donner un coup de fouet à l'éolien ou au solaire avec stockage, comme en témoigne le succès de l'appel d'offres en cours (lire ci-contre). Enfin d'autres voies sont explorées, comme la géothermie ou l'énergie des mers, susceptibles de receler un fort potentiel dans ces territoires marins et volcaniques.  « L'efficacité énergétique aura aussi un grand rôle à jouer »  , insiste Michel Magnan, soulignant que les 130.000 chauffe-eau solaires installés à La Réunion  « ont sans doute permis d'éviter deux ou trois turbines à combustion »  .   L'impact financier de la transition énergétique dans les DOM n'est pas encore connu.  « Il sera évalué une fois les trajectoires définies »  , assure Jean-Claude Futhazar. Les acteurs du secteur ne seront alors pas au bout de leurs peines. Car ils devront ensuite définir la façon de parvenir à l'objectif fixé par la loi pour 2030 : l'autonomie énergétique. Un objectif qui exclut les importations, de biomasse par exemple, et qui, surtout, ne se limite plus à l'électricité. Compte tenu du poids des transports dans les îles, l'équation ne sera pas simple à résoudre.   Anne Feitz   38 euros le MWh De 200 à 250 euros le MWh
</article>
<article titre="Les entreprises cotées en bourse se préparent aussi à la COP21" date="2015 11 17" journal="Midi Libre  CATALAN_ML ; LOZERE ; RODEZ_ML ; MILLAU ; CARCA_ML ; NARBONNE_ML ; ALES ; GARD_RHOD ; NIMES ; BEZIERS ; SETE ; LODEVE ; LUNEL ; MONTPELLIER">
Point    La conférence sur le climat a lieu début décembre.   L'objectif de cette conférence est de trouver un accord pour limiter le réchauffement de la planète à deux degrés par rapport à l'ère pré-industrielle. Nombre d'entreprises cotées sont mobilisées. À commencer par les partenaires qui entendent profiter de cette vitrine mondiale pour exposer leur savoir-faire. Engie (ex-GDF Suez) a installé sur le site du Bourget une chaudière au biogaz, Suez Environnement aura la haute main sur les déchets et Renault véhiculera les participants dans ses Zoé électriques. Les financiers sont eux aussi très mobilisés par la transition énergétique. Ils souhaitent « décarboner » les portefeuilles boursiers, car un petit 40 % des sociétés de l'indice boursier MSCI World est responsable de 75 % des émissions de CO2 de l'indice. Orienter l'épargne vers l'énergie renouvelable Les investisseurs institutionnels, dont les assureurs-vie, qui ont un horizon d'investissement très long, jusqu'à 25 ans, financent une partie des infrastructures en énergie renouvelable. Les besoins d'investissement sont estimés à 170 milliards d'euros par an dans le monde, ce qui implique d'orienter l'épargne vers ce secteur. Suez et Veolia ne sont pas avares d'exemples sur leurs prestations en matière d'économie circulaire. Il est ainsi possible de réduire les rejets de gaz à effet de serre des décharges en captant le méthane pour le commercialiser en gaz ou fournir de l'électricité. Le recyclage du plastique est aussi un défi important. L'eau, surtout, est un enjeu majeur, car avec le réchauffement climatique une part importante de la population vivra en situation de stress hydrique. L'économie circulaire de l'eau est capitale pour les industries minières et pétrolières situées en zone désertique. Pour sa part, EDF entend doubler sa capacité installée dans le renouvelable d'ici 2030 et investira pour cela 2 milliards d'euros par an, avec notamment trois champs d'éoliennes au large des côtes françaises. Engie a acquis Solaire Direct (parcs solaires). Il existe aussi des électriciens « verts » cotés en Bourse, comme Albioma (production de courant à partir de bagasse, résidu de la canne à sucre) ou Voltalia (éolien au Brésil).   CÉCILE LE COZ
</article>
<article titre="Climat Des électriciens « verts » en Bourse" date="2015 11 14" journal="Investir-Le Journal des finances, no. 2184">
Alternatifs : L'indice MSCI World ne compte que 16 sociétés dédiées aux énergies renouvelables, mais la cote parisienne est riche en petites valeurs « vertes ». Parmi les producteurs d'électricité, Albioma est le plus important, avec un chiffre d'affaires de 354 millions d'euros en 2014, réalisé à plus de 85 % par des centrales thermiques situées outre-mer et au Brésil. L'entreprise est spécialiste de la production de courant à partir de bagasse, le résidu de la canne à sucre, pendant les campagnes sucrière et de charbon en dehors. Mais la centrale de Galion 2 en construction en Martinique remplacera le charbon par des déchets verts. Albioma exploite aussi des fermes solaires. Après avoir levé plus de 100 millions d'euros depuis 2014, Voltalia se développe rapidement dans l'éolien au Brésil, pays qui assure près de 60 % de ses ventes d'électricité. La société, présente aussi dans le solaire, la biomasse et l'hydroélectricité, en France et en Grèce, pose des jalons au Maroc.   Parmi les acteurs plus modestes, figurent, pour l'éolien, Futuren (auparavant Theolia) et Global EcoPower, dirigé par l'ex-équipe de Theolia. Velcan exploite une centrale hydraulique au Brésil et oeuvre à d'autres projets en Inde. Methanor prend des participations minoritaires dans des usines de méthanisation et des fermes solaires. On citera encore Vergnet, qui construit des éoliennes, et McPhy Energy, retenu pour l'équipement de stations-service à hydrogène.   Nos conseils Acheter Albioma (ABIO, viser 18 euros), Voltalia (VLTSA, 12,50 euros) et, à titre spéculatif, Futuren (FTRN, 0,70 euros), Méthanor (ALMET, 7,50 euros), McPhy Energy (ALPHY, 8 euros). Velcan (ALVEL) et Vergnet (ALVER).
</article>
<article titre="Cop 21 : une vitrine vertueuse" date="2015 11 14" journal="Le Populaire du Centre  Limoges ; Creuse ; Haute Vienne">
La conférence sur le climat se tiendra du 30 novembre au 11 décembre au Bourget sous l'égide des Nations Unies. L'objectif est de parvenir à un accord pour limiter le réchauffement de la planète à deux degrés par rapport à l'ère pré-industrielle. Nombre d'entreprises cotées sont mobilisées. À commencer par les partenaires qui entendent profiter de cette vitrine mondiale pour exposer leur savoir-faire.    « Décarboner » les portefeuilles    EDF s'est chargé de l'électricité sur le site, Engie (ex-GDF Suez) a installé une chaudière au biogaz, Suez Environnement aura la haute main sur les déchets et Renault véhiculera les participants dans ses Zoé électriques. Un consensus se dégage sur la nécessité de fixer un prix au carbone par des taxes ou l'instauration d'un marché, afin de pénaliser les gros émetteurs et de favoriser les plus vertueux.   Les financiers sont eux aussi très mobilisés par la transition énergétique. Ils souhaitent « décarboner » les portefeuilles, sachant qu'un petit 40 % des sociétés de l'indice boursier MSCI World est responsable de 75 % des émissions de CO2 de l'indice.   Les investisseurs institutionnels, dont les assureurs-vie, qui ont un horizon d'investissement très long, jusqu'à 25 ans, financent une partie des infrastructures en énergie renouvelable, par le biais de fonds dédiés ou d'obligations vertes. Les besoins d'investissement sont estimés à 170 milliards d'euros par an dans le monde, ce qui implique d'orienter l'épargne vers ce secteur.   Suez et Veolia ne sont pas avares d'exemples sur leurs prestations en matière d'économie circulaire. Il est ainsi possible de réduire les rejets de gaz à effet de serre des décharges en captant le méthane pour le commercialiser en gaz ou fournir de l'électricité.   Le recyclage du plastique est aussi un défi important. L'eau, surtout, est un enjeu majeur, car avec le réchauffement climatique une part importante de la population vivra en situation de stress hydrique.   L'économie circulaire de l'eau est capitale pour les industries minières et pétrolières souvent situées en zone désertique.   Pour sa part, EDF entend doubler sa capacité installée dans le renouvelable à l'horizon 2030 et investit pour cela 2 milliards d'euros par an. L'essentiel ira au solaire et à l'éolien, avec notamment trois champs d'éoliennes au large des côtes françaises.   Engie a renoncé à investir pour de nouvelles centrales au charbon, car le gaz émet deux fois moins de CO2. Le groupe a acquis Solaire Direct, qui dispose de près de 60 parcs solaires en exploitation ou en construction.   Il existe aussi des électriciens « verts » cotés en Bourse. C'est le cas d'Albioma, spécialiste de la production de courant à partir de bagasse, le résidu de la canne à sucre, ou de Voltalia, qui se développe dans l'éolien au Brésil.    Cécile Le Coz  Cet article a également été publié dans les éditions suivantes : Creuse, page 51; Haute Vienne, page 51
</article>
<article titre="Albioma: activité stable au 3T" date="2015 10 28" journal="AFP Infos Economiques">
Paris, 28 oct 2015 (AFP) - - Le producteur français d'électricité Albioma, surtout actif en outre-mer, a vu son activité rester stable au troisième trimestre avec un chiffre d'affaires de 89,8 millions d'euros.   Sur les neuf premiers mois de l'année, il atteint 253,7 millions d'euros, également stable par rapport à l'an dernier. "Le troisième trimestre a été marqué par une bonne disponibilité de l'ensemble de nos centrales thermiques et photovoltaïques en France, à l'Île Maurice et au Brésil, avec en particulier la prise en main réussie de notre deuxième installation, Codora Energia", s'est félicité Jacques Pétry, le PDG de la société spécialisée dans l'exploitation de la biomasse.   Albioma a finalisé en août l'acquisition de 65% de cette société, propriétaire d'une centrale de cogénération de 46 mégawatts au Brésil.    Sur les neuf premiers mois de l'année, les centrales biomasses françaises ont produit 1.467 gigawattheures (1.522 GWh l'an dernier), pénalisées par une grève sur un site en Guadeloupe et un incident technique sur une autre à La Réunion. Cette dernière a redémarré le 5 août.   La production des 14 centrales solaires d'Albioma a atteint 73 GWh, stable par rapport à l'an dernier.   Au Brésil, la production a augmenté (89,7 GWh contre 69,6 GWh), mais le chiffre d'affaires a été pénalisé par la faiblesse du real par rapport à l'euro.   La société a ainsi confirmé ses objectifs pour 2015, révisés à la baisse d'environ 10% en mai dernier après des incidents d'exploitation sur deux centrales électriques, en Guadeloupe et à La Réunion.   Le PDG d'Albioma indique aussi que "la récession économique au Brésil (offre) des opportunités d'acquisitions (...) de centrales bagasse de grande qualité".   mhc/sl/mcj
</article>
<article titre="Albioma : les revenus sur neuf mois sont presque stables" date="2015 10 28" journal="Boursier (site web)">
Albioma  n'a pas encore renoué avec la croissance au 3ème trimestre 2015, dont les revenus sont en légère baisse en glissement annuel à 89,8 millions d'euros. Sur neuf mois, le chiffre d'affaires ressort à 253,7 ME, en baisse de -1%. Oddo visait 85,6 ME de revenus sur le 3ème trimestre.   Le management se réjouit d'une "bonne disponibilité" de ses installations durant l'été, et des perspectives fixées par la loi de transition énergétique en août, qui fixe outre-mer un objectif de 50% d'énergie renouvelable en 2020. "Nos installations actuelles et nos projets, comme celui de Galion 2 en Martinique, qui valorisent la bagasse et d'autres formes de biomasse, participeront de façon significative à l'atteinte des objectifs fixés par la loi dans ces territoires", se félicite le président Jacques Pétry. Albioma s'en tient aux objectifs qui avaient été revus en baisse en mai dernier, après les incidents d'exploitation : les résultats seront inférieurs d'environ 10% aux objectifs initiaux de 126 à 130 ME d'EBITDA et 34 à 37 ME de bénéfice net.   Il faudra attendre la réaction des analystes pour en savoir plus ce matin. Albioma présente la particularité d'être un des dossiers français les plus appréciés des bureaux d'études actuellement, avec un indicateur synthétique Bloomberg de 4,60 points sur 5 possibles (il mesure le nombre de recommandations positives, neutres et négatives), soit 8 avis positifs et 2 neutres, pour un objectif moyen de 19,10 euros.
</article>
<article titre="ALBIOMA confirme ses objectifs 2015" date="2015 10 28" journal="Option Finance (site web)">
AOF - 28 octobre 2015   (AOF) - Le chiffre d'affaires du troisième trimestre 2015 d'Albioma a atteint 89,8 millions d'euros, à comparer à 90,2 millions d'euros au troisième trimestre de l'exercice précédent. Sur les neuf premiers mois de l'exercice, il s'élève à 253,7 millions d'euros, en repli de 1%. Le groupe a confirmé ses objectifs 2015 d'Ebitda et de résultat net part du groupe, tels que révisés le 28 mai, avec l'hypothèse de la signature, dès 2015, des avenants liés aux sous-produits de combustion.   Il estimait alors que ses objectifs 2015 d'Ebitda de 126 à 130 millions d'euros et de résultat net part du Groupe de 34 à 37 millions d'euros devaient être réduits d'environ 10 %. Jacques Pétry, Président-Directeur Général, a déclaré : « Le troisième trimestre a été marqué par une bonne disponibilité de l'ensemble de nos centrales thermiques et photovoltaïques en France, à l'Île Maurice et au Brésil, avec, en particulier, la prise en main réussie de notre deuxième installation, Codora Energia ».   Avant d'jouter : « Au Brésil, la récession économique et la faiblesse de la monnaie nous offrent des opportunités d'acquisitions, dans des conditions économiques favorables, de centrales bagasse de grande qualité que nous choisissons de façon très sélective ».  Cet article est paru dans Option Finance (site web)
</article>
<article titre="Des chemises qui en disent long sur l'identité des Premières Nations" date="2015 10 28" journal="Le Reflet (Delson, QC), no. n° 42">
DELSON - L'auteur et artiste laprairien Sylvain Rivard publie son troisième livre pour enfants consacré aux vêtements amérindiens. Intitulé  La chemise à rubans , l'ouvrage paru aux éditions Hannenorak retrace la signification des couleurs et divers ornements de celle-ci. «La chemise à rubans est un vêtement très identitaire des Premières Nations. Des Navajos du Sud des États-Unis jusqu'aux Cris, tout le monde a sa propre version. La chemise parle beaucoup: ses couleurs, la disposition des rubans, les motifs perlés, sa coupe,  etc.  Tout a une signification», explique M. Rivard. En guise d'exemple, la couleur noire peut symboliser l'Ouest, la nuit et l'automne. Selon les lieux géographiques, la chemise à rubans arbore des motifs à carreaux ou arc-en-ciel.   Langue Tout comme les deux premiers tomes de la collection, les dialogues sont en anglais et en français ainsi que dans une langue amérindienne.   «Ce que j'aime beaucoup avec mon dernier livre, c'est qu'il est rédigé en langue cri syllabique. Graphiquement, c'est une belle écriture à laquelle les gens n'ont pas beaucoup accès», précise M. Rivard.   L'abénaquis et l'inuktitut seront employés pour les deux prochains livres portant sur la couverture et le parka qui sortiront en 2016.   Collage Sylvain Rivard s'intéresse aux divers types de papier. Il a réalisé les collages qui servent d'images au livre, tout comme pour ses autres publications.   «Ce que j'entends le plus parler, ce sont mes illustrations. Je vais faire une exposition de mes images durant le Salon du livre des Premières Nations en novembre à Wendake», mentionne-t-il.   Il parcourt d'ailleurs le monde à la recherche de papier fin qu'il utilisera par la suite dans ses oeuvres.   «L'année passée, je suis allé au Népal et au Bhoutan situé à côté. Le Bhoutan est un petit royaume fermé aux tourismes. Il faut demander la permission au roi pour y entrer. C'est là qu'on confectionne un des papiers les plus fins au monde. Celui-ci est fait avec l'écorce d'un arbre nommé daphné. Il fait partie des papiers fins himalayens», raconte l'artiste qui a pu en fabriquer sur place.   Prochainement, il s'envolera vers Cuba pour rapporter un papier fait avec de la bagasse, le résidu fibreux provenant de la canne à sucre.   david.penven@tc.tc
</article>
<article titre="Des chemises qui en disent long sur l'identité des Premières Nations" date="2015 10 27" journal="Le Reflet (Delson, QC) (site web)">
L'auteur et artiste laprairien Sylvain Rivard publie son troisième livre pour enfants consacré aux vêtements amérindiens. Intitulé La chemise à rubans, l'ouvrage paru aux éditions Hannenorak retrace la signification des couleurs et divers ornements de celle-ci.    «La chemise à rubans est un vêtement très identitaire des Premières Nations. Des Navajos du Sud des États-Unis jusqu'aux Cris, tout le monde a sa propre version. La chemise parle beaucoup: ses couleurs, la disposition des rubans, les motifs perlés, sa coupe, etc. Tout a une signification», explique M. Rivard. En guise d'exemple, la couleur noire peut symboliser l'Ouest, la nuit et l'automne. Selon les lieux géographiques, la chemise à rubans arbore des motifs à carreaux ou arc-en-ciel.    «J'utilise la langue dont la culture autochtone est la plus près du vêtement.» Sylvain Rivard, auteur   Langue   Tout comme les deux premiers tomes de la collection, les dialogues sont en anglais et en français ainsi que dans une langue amérindienne.   «Ce que j'aime beaucoup avec mon dernier livre, c'est qu'il est rédigé en langue cri syllabique. Graphiquement, c'est une belle écriture à laquelle les gens n'ont pas beaucoup accès», précise M. Rivard.   L'abénaquis et l'inuktitut seront employés pour les deux prochains livres portant sur la couverture et le parka qui sortiront en 2016.   Collage   Sylvain Rivard s'intéresse aux divers types de papier. Il a réalisé les collages qui servent d'images au livre, tout comme pour ses autres publications.   «Ce que j'entends le plus parler, ce sont mes illustrations. Je vais faire une exposition de mes images durant le Salon du livre des Premières Nations en novembre à Wendake», mentionne-t-il.   Il parcourt d'ailleurs le monde à la recherche de papier fin qu'il utilisera par la suite dans ses oeuvres.   «L'année passée, je suis allé au Népal et au Bhoutan situé à côté. Le Bhoutan est un petit royaume fermé aux tourismes. Il faut demander la permission au roi pour y entrer. C'est là qu'on confectionne un des papiers les plus fins au monde. Celui-ci est fait avec l'écorce d'un arbre nommé daphné. Il fait partie des papiers fins himalayens», raconte l'artiste qui a pu en fabriquer sur place.   Prochainement, il s'envolera vers Cuba pour rapporter un papier fait avec de la bagasse, le résidu fibreux provenant de la canne à sucre.   Ouvrage sur la tuque   Sylvain Rivard a publié dans la même collection La tuque, en 2014. Pour des raisons de marketing, il n'a y pas eu d'annonce officielle. L'éditeur a décidé de présenter cet ouvrage, aux côtés de La chemise à rubans et La ceinture fléchée (premier livre de la collection), cette semaine à Québec pour lancer la série.   «Il a été rédigé en innu. Je raconte comment que la tuque est arrivée en Amérique avec les premiers moutons à bord des drakkars. Les Inuits tricotent beaucoup. Ils sont très près de Terre-Neuve qui est reconnue pour ses tricots épais que les pêcheurs portaient», souligne l'auteur.    Sylvain Rivard, aux origines abénakises, fait découvrir à travers ses ouvrages la culture des Premières Nations.    ©Photo gracieuseté - J. Fred Bédard  Cet article est paru dans Le Reflet (Delson, QC) (site web)
</article>
<article titre="ŠŠArrêté  Bagasse" date="2015 10 26" journal="Environnement Magazine hebdomadaire, no. 34">
Le ministère de l'Écologie annonce la signature de l'arrêté interministériel redéfinissant les conditions du soutien applicable aux installations produisant de l'électricité à partir de biomasse issue de canne à sucre, ou bagasse, en outre-mer.    ERDF a inauguré le Linky lab, son centre de tests de compteurs communicants. Une quarantaine d'ingénieurs et de techniciens y mettront à l'épreuve une centaine de compteurs Linky chaque jour. Leur but : tester la fiabilité et la robustesse du matériel. > erdf.fr
</article>
<article titre="Soutien aux Energies renouvelables / Prime «  bagasse » dans les Zones Non Interconnectées" date="2015 10 20" journal="NewsPress (français)">
Soutien aux énergies renouvelables Publication de l'arrêté revalorisant la prime « bagasse » dans les Zones Non Interconnectées   L'arrêté interministériel redéfinissant les conditions du soutien applicable aux installations produisant de l'électricité à partir de biomasse issue de canne à sucre (ou bagasse) en Outre-mer a été signé. La loi de transition énergétique pour la croissance verte fixe des objectifs ambitieux dans les départements d'Outre-mer. Pour atteindre ces objectifs, il est nécessaire d'accélérer le développement de toutes les sources d'énergie renouvelable, et en particulier de celles à caractère prévisible et non-intermittent comme la bagasse.   Conformément à l'engagement du Premier ministre lors de son déplacement à la Réunion au mois de juin, le montant de la « prime bagasse » a été revalorisé. Cette prime est versée pour toutes les centrales existantes produisant de l'électricité à partir du résidu de canne à sucre.   L'arrêté du 8 octobre publié au journal officiel du 18 octobre 2015 fixe le montant de cette prime à 14,5euros/tonne de canne à sucre, contre 13euros auparavant.   Le montant de la prime bagasse était jusqu'alors indexé sur le cours du charbon et le cours du CO2, ainsi que sur le taux de fibre présent dans la canne, avec de plus un coefficient de dégressivité de la prime nécessitant une augmentation du taux de fibre de 3% par an pour maintenir la rémunération.   Le nouvel arrêté supprime l'indexation de la prime sur le cours du charbon et du CO2, qui dans son principe ne faisait que maintenir la dépendance des systèmes électriques insulaires à la fluctuation du cours des énergies fossiles. Le coefficient de dégressivité est neutralisé jusqu'à 2017 inclus. Par ailleurs il est divisé par 5, ramenant l'augmentation du taux de fibre nécessaire pour maintenir la rémunération à un niveau plus réaliste de 0,6 % par an.   La revalorisation permettra de développer les énergies renouvelables et de conforter la filière canne-sucre-rhum-bagasse dans les outre-mer.   Cette revalorisation sera prise en compte dès la campagne sucrière 2015.
</article>
<article titre="Une prime plus sucrée pour l'électricité issue de la  bagasse" date="2015 10 20" journal="GreenUnivers (site web)">
2015-10-21T12:43:38+00:00    Le Premier ministre s'y était engagé lors d'un déplacement à La Réunion en juin dernier. La revalorisation de la "prime bagasse", du nom de ces [am4show have='g1;g2;g4' user_error='Please_Upgrade' guest_error='Please_Subscribe']résidus de la canne à sucre utilisés pour produire de l'électricité, est désormais actée, selon un arrêté publié au Journal Officiel du 18 octobre.   Le montant de la prime, versée pour toutes les centrales utilisant de la bagasse dans les départements d'outre-mer et à Mayotte, passe à 14,5 euros par tonne, contre 13 auparavant. Plus précisément, il s'agit désormais de 14,5 euros par tonne de canne "dont les résidus sont valorisés dans des conditions de rendement énergétique conformes ou supérieures à celles de l'installation en 2008 en moyenne annuelle", précise l'annexe 1 de l'arrêté.   Fin de l'indexation sur les énergies fossiles  Jusqu'à aujourd'hui, le montant de la prime était indexé sur les cours du charbon et du CO2. L'arrêté supprime cette indexation, "qui dans son principe, explique le ministère de l'Ecologie, ne faisait que maintenir la dépendance des systèmes électriques insulaires à la fluctuation du cours des énergies fossiles".   Auparavant, s'ajoutait à cette indexation un coefficient de dégressivité de la prime nécessitant une augmentation du taux de fibre de 3% par an pour maintenir la rémunération. Ce coefficient, neutralisé jusqu'à 2017 inclus, est aussi divisé par 5. Ainsi, "l'augmentation du taux de fibre nécessaire pour maintenir la rémunération [est abaissée] à un niveau plus réaliste de 0,6% par an", détaille le ministère.   La filière canne-sucre-rhum-bagasse confortée  L'objectif est de faciliter le développement de la bagasse dont le caractère prévisible et non-intermittent est mis en avant par le ministère, pour qui c'est l'ensemble de la filière canne-sucre-rhum-bagasse qui se trouve confortée.   Sur le plan juridique, cette revalorisation est issue de l'annexe 1 de l'arrêté du 8 octobre 2015, publié au Journal Officiel du 18 octobre, qui modifie l'arrêté du 20 novembre 2009 relatif aux conditions d'achat de l'électricité produite à partir de bagasse par des producteurs bénéficiant de l'obligation d'achat dans les départements d'outre-mer et à Mayotte.   Bonne nouvelle pour Albioma  Cette revalorisation devrait réjouir Albioma, qui, après s'être recentré sur la production d'EnR à partir de bagasse en outre-mer en 2012, ambitionne aujourd'hui de produire 50% de son énergie à partir de biomasse à l'horizon 2020, notamment grâce aux résidus de canne à sucre.   Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="[Le président de la République François HOLLANDE se rend...]" date="2015 10 20" journal="La Correspondance économique">
Le président de la République François HOLLANDE se rend aujourd'hui à La Courneuve, pour le lancement de l'Agence nationale de développement économique annoncée en février, a indiqué l'Elysée.   Le gouverneur de la Banque de France Christian NOYER a jugé hier non nécessaire de changer le calibrage du programme de rachat de dette de la Banque centrale européenne (BCE), ou "QE", malgré une inflation encore faible dans la zone euro. "Pour moi, c'est le bon calibrage compte tenu de ce qu'est l'économie. Ça a déjà porté des fruits. (...) Je trouve qu'aujourd'hui le pied est poussé loin sur l'accélérateur et la machine réagit bien.", a-t-il déclaré, sur BFM Business. Le président de la République François HOLLANDE se livre à un plaidoyer pour la ruralité dans un entretien au "Chasseur Français" à paraître demain. Il dit avoir "beaucoup de considération pour ceux qui défendent la nature", dont "les chasseurs font d'ailleurs partie". "Mais en même temps, je n'admets pas qu'au nom de la nature, on empêche un certain nombre d'installations", ajoute le chef de l'Etat en rappelant le drame de Sivens dans le Tarn.   Le montant des baisses de cotisations sociales pour les agriculteurs annoncées début septembre par le gouvernement dépassera 180 millions d'euros en 2015, dont 120 millions "en faveur des éleveurs en difficulté", a indiqué hier le ministre de l'Agriculture Stéphane LE FOLL.   Un plan d'action visant à améliorer la compétitivité du transport fluvial sera présenté en juin prochain, a annoncé hier le secrétaire d'Etat aux Transports Alain VIDALIES dans un communiqué, à l'issue de la conférence nationale sur le fret fluvial.   Plus de 20 000 internautes ont participé à la consultation publique sur le projet de loi sur le numérique, lancée par Manuel VALLS et la secrétaire d'Etat chargée du numérique Axelle LEMAIRE fin septembre et qui s'est achevée hier.   La grève des avocats contre le projet de réforme de l'aide juridictionnelle (AJ), qui consiste à ne pas désigner d'avocats commis d'office, était suivie hier par "161 des 164 barreaux de France", sachant que les trois barreaux restant ne font pas ou peu d'AJ, a indiqué le Conseil national des barreaux (CNB).   Le gouvernement a annoncé un train de mesures pour renforcer considérablement les contrôles et les fouilles dans les transports en commun, après l'attaque déjouée dans le Thalys en août. Une proposition de loi sur ces mesures sera bientôt déposée par le député PS Gilles SAVARY pour être examinée le 18 novembre en commission et le 23 en séance de l'Assemblée nationale. Le gouvernement espère son adoption au printemps 2016.   Les centrales électriques fonctionnant avec du résidu de canne à sucre (bagasse) et situées dans les zones non interconnectées, c'est-à-dire en Outre-mer, auront dorénavant droit à une prime plus élevée, a annoncé hier le ministère de l'Ecologie, du Développement durable et de l'Energie.
</article>
<article titre="Sécurité dans les transports - ANC - Empreinte écologique - Parcs naturels régionaux - Arafer - World Efficiency - Grand Paris..." date="2015 10 20" journal="La Gazette des communes (site web)">
Sécurité dans les Transports -  Faisant suite à l'attaque du Thalys du mois d'août dernier, le Comité national de sécurité dans les transports s'est réuni le 19 octobre. Plusieurs mesures liées à la lutte contre l'insécurité et à la fraude, détaillées par le ministre de l'Intérieur Bernard Cazeneuve et le secrétaire d'Etat aux Transports, Alain Vidalies, devraient ainsi rapidement voir le jour. Parmi elles, figurent la possibilité pour les agents de la SNCF et de la RATP de procéder à des fouilles de bagages et des palpations de sécurité, l'autorisation de contrôles en civil, ou encore l'abaissement du seuil de caractérisation du délit de fraude par habitude, qui passe de dix contraventions annuelles à cinq. Assainissement non collectif -  Les 12ème Assises nationales de l'assainissement non collectif (ANC), se sont déroulées les 14 et 15 octobre derniers à Troyes. La rencontre, qui réunissait les collectivités et leurs partenaires autour de cette question qui concerne 5 millions de foyers en France, est notamment revenue sur 20 ans d'ANC en France, ainsi que sur les diverses "nouveautés législatives et règlementaires parues cet été", indique un communiqué de présentation des Assises, du 6 octobre dernier. [Lire notre article sur le sujet]   Filière bois-Forêt -  Un rapport " Pour une meilleure valorisation économique de la filière bois-forêt en France ", a été présenté le 14 octobre par les députés Pascale Gott (PS) et Damien Abad (LR). Parmi les mesures préconisées, figurent la création d'un " délégué interministériel pour la forêt et le bois ", la mise en place d'un " Observatoire économique de la forêt et du bois ", ainsi que le lancement d'un " label " qualité France " pour développer l'esprit de patriotisme économique de la filière ", indique le document. [Lire le rapport]   Empreinte écologique -  Selon une étude de l'Institut d'aménagement et d'urbanisme d'Île-de-France (I.A.U. Île-de-France) du 13 octobre dernier, citée par le Journal de l'environnement, l'empreinte écologique des habitants d'île-de-France s'est réduite de 13,8% entre 2004 et 2014, et est passée de 5,58 hectares globaux par habitant à 4,81. Cette baisse notable serait notamment à imputer au développement des transports en commun et modes doux. [Consulter l'étude]   Energie -  Un arrêté du 8 octobre 2015, modifiant les " conditions d'achat de l'électricité produite à partir de biomasse issue de la canne à sucre " est paru au journal officiel le 18 octobre dernier. Visant à encourager le développement de la filière, le texte revalorise notamment le montant de la prime " bagasse ", octroyée aux producteurs d'électricité issue de résidus de cette plante. La prime passe ainsi de 13 euros la tonne de canne à sucre à 14,5 euros.   Climat -  Le 14 octobre dernier, les parcs naturels régionaux ont présenté leurs actions en faveur de l'environnement à quelques semaines de la Cop 21. Parmi elles figurait notamment la publication d'un livre blanc recensant les innovations mises en place dans ce sens dans 51 "territoires-laboratoires". [Lire notre article sur le sujet]   Loi Macron -  Le 15 octobre dernier, un décret encadrant l'ouverture des liaisons d'autocar longue distance de moins de 100 kilomètres sur l'Hexagone, a officialisé la naissance l'Autorité de régulation des activités ferroviaires et routières (Arafer), en remplacement de l'Araf, dont le champ de compétences se limitaient jusque-là au rail. [Lire notre article sur le sujet].   Et aussi... Semaine de l'innovation publique -  La Semaine de l'innovation publique s'est déroulée du 12 au 18 octobre 2015. A cette occasion, Clotilde Valter, secrétaire d'Etat chargée de la Réforme de l'État et de la Simplification, s'est exprimée dans nos colonnes pour détailler le volet numérique de son action. [Lire notre interview].   Grand Paris -  Le troisième comité interministériel sur le Grand Paris s'est réuni le 15 octobre dernier. Plusieurs mesures ont été annoncées dans la foulée par le gouvernement, telle que l'identification d'une quinzaine de territoires d'intérêt national, qui seront accompagnés par l'Etat dans le but d'accueillir des projets d'aménagement.[Le Moniteur]   World Efficiency -  World Efficiency, le salon dédié aux ressources et au climat, s'est déroulé du 13 au 15 octobre 2015 à Paris. Plusieurs innovations y ont été présentées, telle que Wattway, une route solaire développée par la société Colas. [Lire notre article sur le sujet]
</article>
<article titre="Sécurité dans les transports - ANC - Empreinte écologique - Parcs naturels régionaux - Arafer - World Efficiency - Grand Paris..." date="2015 10 20" journal="La Gazette des communes">
Sécurité dans les Transports -  Faisant suite à l'attaque du Thalys du mois d'août dernier, le Comité national de sécurité dans les transports s'est réuni le 19 octobre. Plusieurs mesures liées à la lutte contre l'insécurité et à la fraude, détaillées par le ministre de l'Intérieur Bernard Cazeneuve et le secrétaire d'Etat aux Transports, Alain Vidalies, devraient ainsi rapidement voir le jour. Parmi elles, figurent la possibilité pour les agents de la SNCF et de la RATP de procéder à des fouilles de bagages et des palpations de sécurité, l'autorisation de contrôles en civil, ou encore l'abaissement du seuil de caractérisation du délit de fraude par habitude, qui passe de dix contraventions annuelles à cinq. Assainissement non collectif -  Les 12ème Assises nationales de l'assainissement non collectif (ANC), se sont déroulées les 14 et 15 octobre derniers à Troyes. La rencontre, qui réunissait les collectivités et leurs partenaires autour de cette question qui concerne 5 millions de foyers en France, est notamment revenue sur 20 ans d'ANC en France, ainsi que sur les diverses "nouveautés législatives et règlementaires parues cet été", indique un communiqué de présentation des Assises, du 6 octobre dernier. [Lire notre article sur le sujet]   Filière bois-Forêt -  Un rapport " Pour une meilleure valorisation économique de la filière bois-forêt en France ", a été présenté le 14 octobre par les députés Pascale Gott (PS) et Damien Abad (LR). Parmi les mesures préconisées, figurent la création d'un " délégué interministériel pour la forêt et le bois ", la mise en place d'un " Observatoire économique de la forêt et du bois ", ainsi que le lancement d'un " label " qualité France " pour développer l'esprit de patriotisme économique de la filière ", indique le document. [Lire le rapport]   Empreinte écologique -  Selon une étude de l'Institut d'aménagement et d'urbanisme d'Île-de-France (I.A.U. Île-de-France) du 13 octobre dernier, citée par le Journal de l'environnement, l'empreinte écologique des habitants d'île-de-France s'est réduite de 13,8% entre 2004 et 2014, et est passée de 5,58 hectares globaux par habitant à 4,81. Cette baisse notable serait notamment à imputer au développement des transports en commun et modes doux. [Consulter l'étude]   Energie -  Un arrêté du 8 octobre 2015, modifiant les " conditions d'achat de l'électricité produite à partir de biomasse issue de la canne à sucre " est paru au journal officiel le 18 octobre dernier. Visant à encourager le développement de la filière, le texte revalorise notamment le montant de la prime " bagasse ", octroyée aux producteurs d'électricité issue de résidus de cette plante. La prime passe ainsi de 13 euros la tonne de canne à sucre à 14,5 euros.   Climat -  Le 14 octobre dernier, les parcs naturels régionaux ont présenté leurs actions en faveur de l'environnement à quelques semaines de la Cop 21. Parmi elles figurait notamment la publication d'un livre blanc recensant les innovations mises en place dans ce sens dans 51 "territoires-laboratoires". [Lire notre article sur le sujet]   Loi Macron -  Le 15 octobre dernier, un décret encadrant l'ouverture des liaisons d'autocar longue distance de moins de 100 kilomètres sur l'Hexagone, a officialisé la naissance l'Autorité de régulation des activités ferroviaires et routières (Arafer), en remplacement de l'Araf, dont le champ de compétences se limitaient jusque-là au rail. [Lire notre article sur le sujet].   Et aussi... Semaine de l'innovation publique -  La Semaine de l'innovation publique s'est déroulée du 12 au 18 octobre 2015. A cette occasion, Clotilde Valter, secrétaire d'Etat chargée de la Réforme de l'État et de la Simplification, s'est exprimée dans nos colonnes pour détailler le volet numérique de son action. [Lire notre interview].   Grand Paris -  Le troisième comité interministériel sur le Grand Paris s'est réuni le 15 octobre dernier. Plusieurs mesures ont été annoncées dans la foulée par le gouvernement, telle que l'identification d'une quinzaine de territoires d'intérêt national, qui seront accompagnés par l'Etat dans le but d'accueillir des projets d'aménagement.[Le Moniteur]   World Efficiency -  World Efficiency, le salon dédié aux ressources et au climat, s'est déroulé du 13 au 15 octobre 2015 à Paris. Plusieurs innovations y ont été présentées, telle que Wattway, une route solaire développée par la société Colas. [Lire notre article sur le sujet]
</article>
<article titre="Electricité/outre-mer: le gouvernement renforce le soutien public à la biomasse" date="2015 10 19" journal="AFP Infos Françaises">
Paris, 19 oct 2015 (AFP) - - Les centrales électriques fonctionnant avec du résidu de canne à sucre (bagasse) et situées dans les zones non interconnectées, c'est-à-dire en outre-mer, auront dorénavant droit à une prime plus élevée, a annoncé lundi le ministère de l'Ecologie et de l'Energie.   La prime versée à ces installations passe de 13 euros à 14,5 euros par tonne de canne à sucre, selon un arrêté publié dimanche au journal officiel, une mesure décidée pour favoriser le développement des énergies renouvelables en outre-mer. Le nouvel arrêté modifie d'autres éléments entrant en compte dans le calcul de la prime.   "Cette revalorisation sera prise en compte dès la campagne sucrière de 2015", précise le ministère dans un communiqué.   Le potentiel de production d'électricité à partir de biomasse est très important pour les territoires d'outre-mer, dans la mesure où cette énergie n'a pas le caractère intermittent d'autres énergies renouvelables comme le solaire ou l'éolien.   Comme les autres énergies renouvelables, la biomasse bénéficie également d'un tarif de rachat bonifié de l'électricité produite.   mhc/fpo/phc
</article>
<article titre="Soutien aux énergies renouvelables Publication de l’arrêté revalorisant la prime «  bagasse » dans les zones non interconnectées" date="2015 10 19" journal="Rapports - Environnement - MEDDTL">
2 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... MINISTÈRE DE L’ÉCOLOGIE, DU DÉVELOPPEMENT DURABLE  ET DE L’ÉNERGIE  MINISTÈRE DES OUTRE-MER  Communiqué de presse ____________________________________________________  Paris, le Lundi 19 octobre 2015  L’arrêté interministériel redéfinissant les conditions du...    Pour voir le rapport :  http://www.developpement-durable.gouv.fr/IMG/pdf/2015-10-19_Revalorisation_prime_bagasse.pdf
</article>
<article titre="Le climat change Agissons!" date="2015 09 18" journal="Rapports - Environnement - MEDDTL">
4 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... ÉNER GÉTI  QUETRA NSIT  ION  AGIS  SONS !  Le cl imat   chan ge  Actu Transition  énergétique La transition énergétique pour la croissance verte va nous permettre de renouveler notre modèle énergétique, de...    Pour voir le rapport :  http://www.developpement-durable.gouv.fr/IMG/pdf/15173_Lettre-Onerc_no24_DEF_Light.pdf
</article>
<article titre="Le climat change AGISSONS" date="2015 09 18" journal="Rapports - Environnement - MEDDTL">
4 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... La transition énergétique pour la croissance verte va nous permettre de renouveler notre modèle énergétique, de lutter contre le changement climatique et va également développer de nouveaux secteurs économiques. Elle s...    Pour voir le rapport :  http://www.developpement-durable.gouv.fr/IMG/pdf/ONERC_lettre_24.pdf
</article>
<article titre="Les enjeux du développement durable en Guadeloupe : Relever les défis sociaux, préserver les atouts environnementaux" date="2015 08 17" journal="Rapports - Marketing - INSEE">
4 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... No 7 Août 2015  La Guadeloupe présente des atouts à préserver, mais aussi des fragilités à prendre en compte afin de maintenir ou de rechercher les équilibres pour chacun des volets...    Pour voir le rapport :  http://www.insee.fr/fr/insee_regions/guadeloupe/themes/insee-analyses/ga_ina_07/ga_ina_07.pdf
</article>
<article titre="La Bourse de Paris-Les valeurs à suivre jeudi (actualisé)" date="2015 08 6" journal="Reuters économique">
(Actualisé avec Sopra Steria, Société Générale, CGG et Naturex) * Variation des futures sur indice CAC 40 <<0#FCE:> * Valeurs qui se traitent ex-dividende * Le point sur les marchés européens PARIS, 6 août (Reuters) - Les valeurs à suivre jeudi à la Bourse de Paris, attendue en baisse à l'ouverture au lendemain d'une hausse de 1,65% du CAC 40, qui a retrouvé son niveau de la fin avril. A 8h05, le contrat à terme sur l'indice CAC 40 perd 0,33% : * SOPRA STERIA a annoncé jeudi un relèvement de son objectif de taux de marge pour 2015 après avoir enregistré un premier semestre meilleur qu'attendu dans ses activités de conseil et d'intégration de systèmes en France. * SOITEC a annoncé mercredi l'arrêt des négociations avec Concensolar portant sur la cession de certains actifs solaires tout en assurant poursuivre son recentrage sur l'électronique. * VINCI a annoncé mercredi la signature d'un contrat de partenariat public-privé (PPP) d'un montant d'environ 1,9 milliard de dollars canadiens (1,3 milliard d'euros). * SOCIETE GENERALE - Barclays a relevé son objectif de cours de 42 euros à 47 euros, avec une recommandation maintenue à "pondération en ligne". * CGG - HSBC a abaissé son objectif de cours de 7,5 euros à 5,5 euros, avec une recommandation maintenue à "acheter". * NATUREX - Société Générale a relevé son objectif de cours de 62 euros à 72,5 euros, avec une recommandation maintenue à "conserver". * EDF et ENGIE figurent parmi les candidats possibles pour l'appel à projets pour quatre fermes pilotes d'éoliennes flottantes en France lancé mercredi par l'Agence de l'environnement et de la maîtrise de l'énergie (Ademe). * NUMERICABLE-SFR - La Ligue nationale de basket (LNB) a indiqué mercredi à Reuters qu'Altice, maison-mère de Numericable-SFR, s'était engagée à lui verser 50 millions d'euros pour les droits de retransmission des cinq prochaines saisons du championnat français sur Ma Chaîne Sport, confirmant une information de lequipe.fr. * MAISONS FRANCE CONFORT a confirmé mercredi attendre un retour d'une croissance de sa production dès l'exercice 2016 lui permettant de retrouver progressivement sa rentabilité normative, après avoir enregistré un recul de 5,3% de son activité au premier semestre, conséquence du recul des prises de commandes en 2014 et de délais administratifs "particulièrement longs". (Le communiqué: http://bit.ly/1T3ACSG) Société Générale a relevé son objectif de cours de 36 euros à 49 euros, avec une recommandation maintenue à "acheter". * CIS a indiqué mercredi envisager une progression de son activité en 2015 et confirmé son objectif de 500 millions d'euros de chiffre d'affaires à l'horizon 2019, après avoir enregistré une croissance de 4,1% de ses revenus au premier semestre (+1,8% au deuxième trimestre) et une hausse de son carnet de commandes, passé de 256 millions de dollars (235 millions d'euros) en 2014 à 311 millions en 2015. (Le communiqué: http://bit.ly/1MOQ7hA) * ALBIOMA a annoncé mercredi avoir finalisé l'acquisition de 65% du capital de la société Codora Energia, propriétaire d'une unité de cogénération bagasse d'une puissance de 48 mégawatts située dans l'Etat de Goiás au Brésil. Le groupe Jalles Machado, deuxième producteur mondial de sucre issu de l'agriculture biologique, conserve 35% des actions de la société. (Le communiqué: http://bit.ly/1W1LFA2) PUBLICATIONS: * DERICHEBOURG / CA du T3 (après Bourse) (Raphaël Bloch et Alexandre Boksenbaum-Granier, édité par Benjamin Mallet)
</article>
<article titre="La Bourse de Paris-Les valeurs à suivre jeudi (avant-Bourse)" date="2015 08 6" journal="Reuters économique">
* Variation des futures sur indice CAC 40 <<0#FCE:> * Valeurs qui se traitent ex-dividende * Le point sur les marchés européens PARIS, 6 août (Reuters) - Les valeurs à suivre jeudi à la Bourse de Paris : * VINCI a annoncé mercredi la signature d'un contrat de partenariat public-privé (PPP) d'un montant d'environ 1,9 milliard de dollars canadiens (1,3 milliard d'euros). * NUMERICABLE-SFR - La Ligue nationale de basket (LNB) a indiqué mercredi à Reuters qu'Altice, maison-mère de Numericable-SFR, s'était engagée à lui verser 50 millions d'euros pour les droits de retransmission des cinq prochaines saisons du championnat français sur Ma Chaîne Sport, confirmant une information de lequipe.fr. * SOITEC a annoncé mercredi l'arrêt des négociations avec Concensolar portant sur la cession de certains actifs solaires tout en assurant poursuivre son recentrage sur l'électronique. * EDF et ENGIE figurent parmi les candidats possibles pour l'appel à projets pour quatre fermes pilotes d'éoliennes flottantes en France lancé mercredi par l'Agence de l'environnement et de la maîtrise de l'énergie (Ademe). * MAISONS FRANCE CONFORT a confimé mercredi attendre un retour d'une croissance de sa production dès l'exercice 2016 lui permettant de retrouver progressivement sa rentabilité normative, après avoir enregistré un recul de 5,3% de son activité au premier semestre, conséquence du recul des prises de commandes en 2014 et de délais administratifs "particulièrement longs". (Le communiqué: http://bit.ly/1T3ACSG) * CIS a indiqué mercredi envisager une progression de son activité en 2015 et confirmé son objectif de 500 millions d'euros de chiffre d'affaires à l'horizon 2019, après avoir enregistré une croissance de 4,1% de ses revenus au premier semestre (+1,8% au deuxième trimestre) et une hausse de son carnet de commandes, passé de 256 millions de dollars (235 millions d'euros) en 2014 à 311 millions en 2015. (Le communiqué: http://bit.ly/1MOQ7hA) * ALBIOMA a annoncé mercredi avoir finalisé l'acquisition de 65% du capital de la société Codora Energia, propriétaire d'une unité de cogénération bagasse d'une puissance de 48 mégawatts située dans l'Etat de Goiás au Brésil. Le groupe Jalles Machado, deuxième producteur mondial de sucre issu de l'agriculture biologique, conserve 35 % des actions de la société. (Le communiqué: http://bit.ly/1W1LFA2) PUBLICATIONS: * SOPRA STERIA / résultats du S1 (avant-Bourse) * DERICHEBOURG / CA du T3 (après Bourse) (Alexandre Boksenbaum-Granier)
</article>
<article titre="Les valeurs à suivre aujourd'hui à la Bourse de Paris - Jeudi 06 août 2015" date="2015 08 6" journal="Option Finance (site web)">
AOF - 06 août 2015   (AOF) -    ALBIOMA    Albioma a finalisé, le 4 août 2015, l'acquisition de 65 % du capital de la société Codora Energia, propriétaire d'une unité de cogénération bagasse d'une puissance de 48 MW située dans l'État de Goiás au Brésil. Le groupe Jalles Machado, deuxième producteur mondial de sucre issu de l'agriculture biologique, conserve 35 % des actions de la société. DERICHEBOURG   Le groupe de services aux entreprises et aux collectivités publiera son chiffre d'affaires du troisième trimestre après la clôture de la Bourse.   DNXCORP   DNXCorp a enregistré une baisse de son chiffre d'affaires de 10,6% à 23,6 millions d'euros au premier semestre. Le groupe spécialisé dans le développement et la valorisation d'audience sur Internet a précisé que ses revenus avaient reculé de 8,8% à 11,7 millions au seul deuxième trimestre 2015 par rapport à la même période de 2014.    MAISONS FRANCE CONFORT   Maisons France Confort a publié un chiffre d'affaires semestriel en baisse de 5,3% (-6,8% à périmètre constant) à 250,1 millions d'euros. Cette baisse d'activité est consécutive au repli des prises de commandes en 2014 mais également à des délais administratifs particulièrement longs (permis de construire, actes notariés, traitement des dossiers par des banques saturées par les renégociations de prêts).    NATUREX   Société Générale a relevé son objectif de cours de 62 à 72,5 euros et maintenu sa recommandation Conserver sur Naturex. Le broker salue la publication d'un chiffre d'affaires du deuxième semestre supérieur à ses attentes et une forte croissance organique des ventes. L'absence d'impact de cette augmentation sur la marge et une valorisation qui a déjà intégré la plupart des perspectives favorables pour le titre incite le bureau d'études à maintenir ses positions sur Naturex.    SOITEC   Soitec a annoncé hier soir qu'en dépit de l'accord signé le 20 mai 2015 avec ConcenSolar, un partenaire commercial de Suncore Photovoltaic Technology Co Ltd, pour lui vendre une partie de ses activités Energie solaire (actifs technologiques et sites de production basés en Allemagne et aux États-Unis), la transaction ne sera pas finalisée.   SOPRA STERIA   Sopra Steria a présenté un résultat net part du groupe en hausse de 1,5% à 26,9 millions d'euros, après prise en compte d'une charge de 2,4 millions d'euros d'intérêts minoritaires, pour le compte de son premier semestre. Le résultat opérationnel courant de la société de conseil en transformation numérique a également augmenté, de 5,5% à 97,5 millions après prise en compte des charges liées aux stock-options et assimilées ainsi que des dotations aux amortissements sur incorporels affectés.    VICAT   Société Générale a abaissé son objectif de cours de 78 à 77 euros et maintenu sa recommandation d'Achat sur Vicat après avoir abaissé ses estimations de résultats pour l'exercice 2015. Après la publication des résultats semestriels du fabricant de ciment et béton, le broker a réduit de 3% son hypothèse de croissance organique en 2015, à +1% sur l'année, de 5% sa prévision d'Ebitda et de 130 points de base celle de marge. Société Générale précise tout de même que Vicat reste "un excellent véhicule pour jouer le redressement de la France".   VINCI   Vinci Concessions, via sa filiale à 37,5% Regina Bypass Partners et en partenariat avec Parsons Entreprises (25 %), le fonds Connor Clark &amp; Lunn GVest (25 %) et Gracorp Capital (12,5 %), a annoncé la signature d'un contrat de partenariat public-privé d'une durée de 30 ans portant sur la réalisation et l'exploitation de l'autoroute de contournement de Regina, capitale de la province canadienne de la Saskatchewan.  Cet article est paru dans Option Finance (site web)
</article>
<article titre="Morning meeting AOF France / Europe - Baisse attendue pour l'ouverture" date="2015 08 6" journal="Option Finance (site web)">
AOF - 06 août 2015   (AOF) - Après leur belle progression d'hier, les marchés européens devraient consolider leurs gains aujourd'hui ouvrant la séance en rouge. Alors que le secteur des médias pourrait souffrir après le repli de Walt Disney à Wall Street, les investisseurs seront en quête de nouveaux catalyseurs pour continuer à faire monter les cours. Aucune donnée macroéconomique n'est attendue en Europe et les résultats d'entreprises sont peu nombreux alors que la saison des publications semestrielles arrive à son terme. Les marchés surveilleront tout de même la décision de politique monétaire de la Banque d'Angleterre.   L'analyse technique du CAC 40    Les analystes de DayByDay constatent la formation d'une grande bougie blanche précédée d'un gap haussier accélérateur. L'Europe a repris sa marche en avant, distançant une nouvelle fois Wall Street. Ce découplage n'est pas sans rappeler celui du premier trimestre, où le fameux "alignement des planètes" associé au QE portait les espoirs de reprise en Europe. Le sommet de l'année du CAC se situe à 5283 : c'est la prochaine cible. Mais ce ralliement ne se fera pas en ligne droite : une consolidation intermédiaire vers 5150 est tout à fait plausible, du fait du positionnement des prix sur la borne haute du canal intraday. Les valeurs à suivre    ALBIOMA   Albioma a finalisé, le 4 août 2015, l'acquisition de 65 % du capital de la société Codora Energia, propriétaire d'une unité de cogénération bagasse d'une puissance de 48 MW située dans l'État de Goiás au Brésil. Le groupe Jalles Machado, deuxième producteur mondial de sucre issu de l'agriculture biologique, conserve 35 % des actions de la société.   MAISONS FRANCE CONFORT   Maisons France Confort a publié un chiffre d'affaires semestriel en baisse de 5,3% (-6,8% à périmètre constant) à 250,1 millions d'euros. Cette baisse d'activité est consécutive au repli des prises de commandes en 2014 mais également à des délais administratifs particulièrement longs (permis de construire, actes notariés, traitement des dossiers par des banques saturées par les renégociations de prêts).    SOITEC   Soitec a annoncé hier soir qu'en dépit de l'accord signé le 20 mai 2015 avec ConcenSolar, un partenaire commercial de Suncore Photovoltaic Technology Co Ltd, pour lui vendre une partie de ses activités Energie solaire (actifs technologiques et sites de production basés en Allemagne et aux États-Unis), la transaction ne sera pas finalisée.   SOPRA STERIA   Sopra Steria a présenté un résultat net part du groupe en hausse de 1,5% à 26,9 millions d'euros, après prise en compte d'une charge de 2,4 millions d'euros d'intérêts minoritaires, pour le compte de son premier semestre. Le résultat opérationnel courant de la société de conseil en transformation numérique a également augmenté, de 5,5% à 97,5 millions après prise en compte des charges liées aux stock-options et assimilées ainsi que des dotations aux amortissements sur incorporels affectés.    Les chiffres macroéconomiques    La seule statistique du jour est attendue à 14h30 aux Etats-Unis avec les inscriptions hebdomadaires au chômage.   Hier à Paris    Les marchés européens ont connu une séance favorable, bénéficiant de résultats meilleurs que prévu, dont ceux de Société Générale et Beiersdorf (crème Nivea). Les statistiques étaient également de bonne facture en zone euro, confirmant la reprise en cours. L'affaiblissement de l'euro, qui évolue en dessous de 1,09 dollar, a également soutenu les indices. Le CAC 40 a clôturé sur un gain de 1,65% à 5 196,73 points, à portée de fusil du plus haut 2015 touché à 5 283,71 points le 27 avril. L'EuroStoxx a gagné 1,59% à 3 676,75 points.   Hier à Wall Street    Les marchés américains ont rompu la belle unanimité qui présidait pendant la première partie de la séance, terminant en ordre dispersé alors que la hausse semblait acquise au cours des premières heures de cotation. Le fort repli de Walt Disney a notamment pesé sur le Dow Jones, alors que des indicateurs mitigés - un décevant ADP sur l'emploi dans le privé a laissé place à un ISM Services record - ravivent les incertitudes quant au calendrier de relèvement des taux qui sera adopté par la Fed. Le Dow Jones a donc perdu 0,06% à 17 540,47 points et le Nasdaq a gagné 0,67% à 5 139,94 points.  Cet article est paru dans Option Finance (site web)
</article>
<article titre="A suivre aujourd'hui... ALBIOMA" date="2015 08 6" journal="Option Finance (site web)">
AOF - 06 août 2015   (AOF) - Albioma a finalisé, le 4 août 2015, l'acquisition de 65 % du capital de la société Codora Energia, propriétaire d'une unité de cogénération bagasse d'une puissance de 48 MW située dans l'État de Goiás au Brésil. Le groupe Jalles Machado, deuxième producteur mondial de sucre issu de l'agriculture biologique, conserve 35 % des actions de la société.   Codora Energia bénéficie d'un contrat de vente d'électricité à long terme sécurisé jusqu'en 2026 pour l'exportation de 87 GWh/an au prix élevé de 205 reals/MWh, indexé sur l'inflation. En outre, en prévision de l'installation d'une nouvelle turbine de 20 MW en 2017, Codora Energia a sécurisé la vente à long terme de 54 GWh/an pendant 20 ans, à compter de 2020, au prix historiquement élevé de 278 reals/MWh, indexé sur l'inflation. Albioma supervise l'exploitation de la centrale depuis le début de la campagne sucrière. La finalisation de cette opération devrait se traduire par une contribution positive à l'EBITDA et au résultat net part du Groupe dès 2015.  Cet article est paru dans Option Finance (site web)
</article>
<article titre="Des indicateurs pour apprécier le Développement Durable en Guadeloupe" date="2015 08 6" journal="Rapports - Marketing - INSEE">
34 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... N°5 Août 2015  Signes conventionnels utilisés … Résultat non disponible /// Absence de résultat due à la nature des choses e Estimation p Résultat provisoire r Résultat révisé par rapport à l...    Pour voir le rapport :  http://www.insee.fr/fr/insee_regions/guadeloupe/themes/insee-dossier/ga_ind_05.pdf
</article>
<article titre="Albioma finalise l'acquisition de 65% d'une centrale de cogénération au Brésil" date="2015 08 5" journal="AFP Infos Economiques">
Paris, 5 août 2015 (AFP) - - Le producteur français d'électricité Albioma a annoncé mercredi avoir finalisé l'acquisition de 65% du capital de la société Codora Energia, propriétaire d'une centrale de cogénération de 48 mégawatts (MW) dans l'Etat de Goias (centre du Brésil).   Cette usine, auparavant propriété totale du groupe sucrier Jalles Marchado, utilise comme source d'énergie la bagasse, le résidu ligneux résultant du broyage de la canne à sucre. Cette acquisition s'inscrit dans la stratégie de développement au Brésil de la société française, jusqu'ici surtout présente dans les territoires d'outremer.   "La finalisation de cette opération devrait se traduire par une contribution positive à l'ebitda (excédent brut d'exploitation) et au résultat net part du groupe dès 2015", a précisé Albioma dans un communiqué.   La société a publié fin mai un avertissement sur ses objectifs financiers pour 2015 à cause de deux incidents d'exploitation sur deux de ses centrales, l'une en Guadeloupe et l'autre à La Réunion.   Albioma avait ainsi réduit "d'environ 10%" ses objectifs.   Au premier semestre, la société a publié un bénéfice net en baisse de 46% à 9,5 millions d'euros.   mhc/sl/jpr/LyS
</article>
<article titre="Albioma: finalisation d'une acquistion au Brésil." date="2015 08 5" journal="Cercle Finance (site web)">
Albioma a annoncé ce mercredi soir avoir finalisé hier l'acquisition de 65 % du capital de Codora Energia, la société propriétaire d'une unité de cogénération bagasse d'une puissance de 48 mégawatts (MW) située dans l'État de Goiás (Brésil).    Deuxième producteur mondial de sucre issu de l'agriculture biologique, me groupe Jalles Machado conserve 35 % des actions restantes de la société.  Codora Energia bénéficie d'un contrat de vente d'électricité à long terme sécurisé jusqu'en 2026 pour l'exportation de 87 gigawattheures (GWh) par an au prix élevé de 205 reals/MWh, indexé sur l'inflation.    En prévision de l'installation d'une nouvelle turbine de 20 MW en 2017, Codora Energia a par ailleurs sécurisé la vente à long terme de 54 GWh/an pendant 20 ans, à compter de 2020, au prix historiquement élevé de 278 reals/MWh, indexé sur l'inflation.    La finalisation de cette opération devrait se traduire par une contribution positive à l'Ebitda et au résultat net part du Groupe dès 2015.    Copyright (c) 2015 CercleFinance.com. Tous droits réservés.
</article>
<article titre="Les valeurs à suivre jeudi 06 août 2015 à la Bourse de Paris" date="2015 08 5" journal="Option Finance (site web)">
AOF - 05 août 2015   (AOF) - ALBIOMA   Albioma a finalisé, le 4 août 2015, l'acquisition de 65 % du capital de la société Codora Energia, propriétaire d'une unité de cogénération bagasse d'une puissance de 48 MW située dans l'État de Goiás au Brésil. Le groupe Jalles Machado, deuxième producteur mondial de sucre issu de l'agriculture biologique, conserve 35 % des actions de la société.   DERICHEBOURG Le groupe de services aux entreprises et aux collectivités publiera son chiffre d'affaires du troisième trimestre après la clôture de la Bourse.   DNXCORP   DNXCorp a enregistré une baisse de son chiffre d'affaires de 10,6% à 23,6 millions d'euros au premier semestre. Le groupe spécialisé dans le développement et la valorisation d'audience sur Internet a précisé que ses revenus avaient reculé de 8,8% à 11,7 millions au seul deuxième trimestre 2015 par rapport à la même période de 2014.    MAISONS FRANCE CONFORT   Maisons France Confort a publié un chiffre d'affaires semestriel en baisse de 5,3% (-6,8% à périmètre constant) à 250,1 millions d'euros. Cette baisse d'activité est consécutive au repli des prises de commandes en 2014 mais également à des délais administratifs particulièrement longs (permis de construire, actes notariés, traitement des dossiers par des banques saturées par les renégociations de prêts).    SOITEC   Soitec vient d'annoncer qu'en dépit de l'accord signé le 20 mai 2015 avec ConcenSolar, un partenaire commercial de Suncore Photovoltaic Technology Co Ltd, pour lui vendre une partie de ses activités Energie solaire (actifs technologiques et sites de production basés en Allemagne et aux États-Unis), la transaction ne sera pas finalisée.   SOPRA STERIA   La société de services informatiques communiquera ses résultats du premier semestre avant l'ouverture de la séance.  Cet article est paru dans Option Finance (site web)
</article>
<article titre="Paris climat 2015 A La Réunion, la promesse de l'autonomie Comme l'ensemble des territoires d'outre-mer, l'île doit acquérir, d'ici à 2030, son indépendance énergétique" date="2015 08 3" journal="Le Monde">
Saint-Denis de La Réunion - envoyé spécial - C'est une image saturée de contrastes, de lumière aussi. Nous sommes début juillet, l'hiver austral, sur la côte nord-ouest de La Réunion. Au bout d'une route pierreuse, des grilles, surmontées de miradors coiffés de tôles vertes, gardent les cinq cents détenus d'un centre pénitentiaire. Autour des baraquements, des dizaines de milliers de panneaux photovoltaïques couvrent le sol, reliés à des batteries de stockage qui alimentent 12 000 foyers en évitant le rejet annuel de 8 000 tonnes de CO2.   Il y a là, encore, des serres équipées de capteurs, des ruches, un arboretum. Les jardiniers sont des détenus qui préparent leur réinsertion en faisant pousser tomates, piments et fruits de la passion, ou en plantant des essences d'arbres indigènes, bois de senteur blanc ou bois de natte. Une quinzaine de condamnés ont déjà bénéficié de cette formation sur le tas et un apprenti apiculteur va bientôt s'installer à son compte pour produire son miel. Aube naissante , en créole  bardzour  (barre du jour) : la société Akuo Energy a bien choisi le nom de cette centrale solaire hors normes, mise en service fin 2014 et lauréate de la Fondation Nicolas Hulot, qui encourage les  « solutions pour le climat »  . Bien sûr, ce n'est qu'une lueur d'espoir sur le chemin d'un développement durable. Mais elle est exemplaire d'une  « démarche adaptée aux besoins du territoire, sur le plan économique, social et environnemental »  , assure Jean-François Moser, directeur général adjoint de l'entreprise.   A La Réunion, comme dans l'ensemble des départements et régions d'outre-mer, la transition énergétique est un impératif, en même temps qu'une gageure. C'est du ciel, depuis un hélicoptère, seul moyen d'accéder aux villages les plus reculés de cette île volcanique aux reliefs vertigineux, que l'on prend la mesure du défi, mais aussi des efforts déployés pour tirer parti de ses ressources naturelles.   Voici d'abord la zone d'activité du Port. C'est ici qu'arrivent, par bateaux, le charbon d'Afrique du Sud, le fioul, l'essence et le gaz de Suède, de Singapour ou des Emirats arabes unis. Ces combustibles fossiles assurent près de 90 % de l'approvisionnement énergétique d'une île livrée à elle-même, au milieu de l'océan Indien, sans aucune connexion avec un réseau électrique. C'est ici aussi qu'EDF a investi 500 millions d'euros dans une nouvelle centrale thermique au fioul, plus performante et moins polluante, pour  « sécuriser le développement des filières renouvelables »  , explique Michel Magnan, directeur régional de l'entreprise publique.   Trop vite, trop tôt Voici maintenant, dominé par le majestueux piton des Neiges, le cirque de Mafate. Un amphithéâtre de verdure et de roc entre ciel et terre, hérissé de promontoires, creusé de gorges et de ravines. Dans des hameaux perchés sur des nids d'aigle (les îlets) vivent toujours les descendants des esclaves marrons qui, aux temps sombres de la colonie, avaient trouvé refuge sur ces à-pics. Ceux-là, coupés du monde, ne doivent leur électricité qu'à de petites installations photovoltaïques, couplées à des batteries. Au sud, les cheminées fumantes de l'une des deux centrales thermiques d'Albioma, le premier producteur d'électricité de l'île. Pendant la campagne sucrière, de mi-juillet à mi-décembre, elles brûlent en priorité de la bagasse, le résidu fibreux de la canne à sucre - principale culture locale -, le charbon prenant ensuite le relais.   Sur l'autre versant, une fois franchies les arêtes où s'accrochent les nuages, des cascades jaillissent de la végétation épaisse. A flanc de paroi, EDF a édifié des barrages acrobatiques et creusé des galeries, où des usines hydroélectriques turbinent les eaux tumultueuses de la rivière des Marsouins. Voici encore une batterie régulant la production de fermes solaires et éoliennes, des moulins à vent plantés sur une crête, des centres commerciaux et des parkings aux toitures tapissées de panneaux solaires...   Avec tout cela, si les énergies renouvelables fournissent un tiers de l'électricité de l'île, elles couvrent à peine 14 % de ses besoins énergétiques. La loi pour la croissance verte que viennent d'adopter définitivement les députés fixe pourtant des objectifs très ambitieux aux régions ultramarines : 50 % d'énergies renouvelables en 2020, puis  « l'autonomie énergétique à l'horizon 2030 »  . Trop vite, trop tôt, pensent les élus réunionnais, peu séduits par le souhait de la métropole de faire de leur communauté un  « laboratoire de la transition énergétique »  .   « L'autonomie énergétique reste le but à atteindre. Mais il faut être réaliste, avancer par étapes , défend le président du conseil régional, Didier Robert (Les Républicains ). Plutôt que de servir de terrain d'expérimentation, nous préférons être exemplaires, en impliquant toute la population. »  Sur ce point au moins, la députée socialiste de Saint-Denis, Ericka Bareigts, est d'accord.  « Notre territoire est en souffrance, il n'a pas vocation à devenir un banc d'essai,  plaide-t-elle . Ce qu'il nous faut, c'est une vision d'avant-garde pour les vingt ou trente ans à venir, incluant l'énergie, l'habitat, les transports... C'est d'un projet de société qu'il s'agit. » En souffrance, La Réunion l'est assurément. Le chômage touche 30 % de la population, le double chez les jeunes. Près d'un habitant sur deux vit sous le seuil de pauvreté, un foyer sur trois est en précarité énergétique. A ces plaies s'ajoute le fardeau d'un parc de logements sociaux construit à la va-vite depuis les années 1970 - des cubes de béton mal isolés inspirés des banlieues métropolitaines -, qui n'empêche pas les poches d'habitat insalubre ou surpeuplé, tandis que plus de 20 000 familles sont en attente d'un toit.   Avec d'autres élus insulaires, Ericka Bareigts a bataillé pour que les territoires d'outre-mer élaborent eux-mêmes leur programmation pluriannuelle de l'énergie. Une façon de  « reprendre la main »  et de conquérir une autonomie politique, à défaut d'énergétique, sur ce dossier crucial pour leur avenir. Etabli par le conseil régional et la préfecture, ce document dessine une trajectoire conduisant, en 2023, à une électricité pour les deux tiers d'origine renouvelable. Deux fois plus qu'aujourd'hui. L'indépendance énergétique complète sera pour plus tard.   Route des cyclones Le chantier est déjà gigantesque. Isolé, de petite taille, fragile - les coupures sont encore fréquentes -, le système électrique réunionnais, alimenté par une multiplicité de producteurs, mais dont EDF a le monopole du transport et de la distribution, est deux à trois fois plus coûteux que celui de l'Hexagone. Il est donc tributaire de la péréquation tarifaire, autrement dit la solidarité nationale, qui rend tous les Français égaux devant leur facture. Il doit aussi faire face à une explosion démographique : la population de l'île, 840 000 personnes aujourd'hui, a doublé en un demi-siècle et devrait dépasser un million d'habitants en 2040, entraînant une hausse continue de la demande, même si son rythme ralentit.   « Notre chance est d'avoir un mix électrique diversifié »,  observe Philippe Beutin, délégué régional de l'Agence de l'environnement et de la maîtrise de l'énergie (Ademe). Mais l'essor des filières vertes est freiné par la géographie et le climat. La Réunion croise la route des cyclones tropicaux, qui obligent à arrimer les éoliennes par des haubans ou à installer des mâts rabattables. Les vagues soulevées sur leur passage - jusqu'à 15 mètres de haut - ainsi que la profondeur des fonds océaniques contrarient la pose de turbines en mer. En janvier 2014, le cyclone Bejisa a même balayé un prototype houlomoteur.   Variétés de canne sans sucre Le relief, mais aussi l'inscription au Patrimoine mondial des  « pitons, cirques et remparts »  limitent les grands projets hydroélectriques, autant que les forages géothermiques autour de l'éruptif piton de la Fournaise. Quant au photovoltaïque, son déploiement est surtout envisagé sur les bâtiments, afin de ne pas empiéter sur des terres agricoles déjà rares. Il y faudra de grosses capacités de stockage pour lisser les fluctuations du soleil et du vent, et permettre aux énergies intermittentes, aujourd'hui plafonnées à 30 % sur le réseau, de monter en régime.   En dépit de ces contraintes, La Réunion n'a pas le choix. Elle doit, d'abord, maîtriser sa consommation. Les travaux d'isolation, l'achat par les ménages de modules photovoltaïques ou de chauffe-eau solaires - l'île en est la région d'Europe la mieux équipée après Chypre - sont fortement subventionnés. Des PME se lancent sur ce marché, comme Dak Industries qui, avec ses chauffe-eau  « anticyclones et anticorrosion »  , marqués du label réunionnais  Nou la fé  (Nous l'avons fait), a déjà créé une quinzaine d'emplois.   Elle doit aussi « métisser » toutes ses ressources. La biomasse est prometteuse. De nouvelles variétés de canne, sans sucre mais riches en fibres, pourraient être cultivées, les déchets verts et les résidus forestiers valorisés, des unités de méthanisation mises en place. Fin 2016, Albioma prévoit de faire tourner, au sud de l'île, la première turbine à combustion alimentée au bioéthanol, un alcool issu de la mélasse de canne.   Autre piste, novatrice, la géothermie marine. Il s'agit de pomper, à plus de 1 000 mètres de profondeur, l'eau fraîche (5°C) des abysses, pour refroidir un réseau urbain d'eau douce et climatiser des bâtiments. A Saint-Denis et Sainte-Marie, le projet Seawater Air Conditioning, porté par Engie et soutenu par la région, l'Ademe et l'Europe, pour un coût de 150 millions d'euros, devrait desservir, d'ici deux ans, l'aéroport, le centre hospitalier, l'université, des immeubles de bureaux et des centres commerciaux.   « investissement démesuré » Reste un énorme point noir : les transports, qui représentent 70 % des besoins énergétiques. La précédente majorité, menée par le sénateur Paul Vergès (Parti communiste réunionnais), avait un projet de « tram-train » pour desservir la côte ouest, congestionnée et menacée d'éboulements rocheux. La droite lui a préféré une « nouvelle route du littoral », avec viaduc sur pilotis et digue - là même où viennent se reproduire les baleines à bosse -, chiffrée à 1,6 milliard d'euros.   « Une erreur historique, un investissement démesuré, qui favorise le tout-automobile »  , juge Jacques Lowinsky, premier adjoint (PS) de Saint-Denis. Début juin, à l'appel du collectif Touch pas nout roche, 4 000 personnes ont protesté contre l'exploitation de l'une des carrières d'où doivent être extraites les 18 millions de tonnes de galets nécessaires à cet ouvrage. De mémoire de créole, la plus grande manifestation environnementale jamais organisée ici. L'émancipation énergétique de La Réunion sera encore une longue marche.
</article>
<article titre="A La Réunion, la promesse de l'autonomie" date="2015 08 3" journal="Le Monde.fr">
C'est une image saturée de contrastes, de lumière aussi. Nous sommes début juillet, l'hiver austral, sur la côte nord-ouest de La Réunion. Au bout d'une route pierreuse, des grilles, surmontées de miradors coiffés de tôles vertes, gardent les cinq cents détenus d'un centre pénitentiaire. Autour des baraquements, des dizaines de milliers de panneaux photovoltaïques couvrent le sol, reliés à des batteries de stockage qui alimentent 12 000 foyers en évitant le rejet annuel de 8 000 tonnes de CO2.   Il y a là, encore, des serres équipées de capteurs, des ruches, un arboretum. Les jardiniers sont des détenus qui préparent leur réinsertion en faisant pousser tomates, piments et fruits de la passion, ou en plantant des essences d'arbres indigènes, bois de senteur blanc ou bois de natte. Une quinzaine de condamnés ont déjà bénéficié de cette formation sur le tas et un apprenti apiculteur va bientôt s'installer à son compte pour produire son miel. Aube naissante , en créole  bardzour  (barre du jour) : la société Akuo Energy a bien choisi le nom de cette centrale solaire hors normes, mise en service fin 2014 et lauréate de la Fondation Nicolas Hulot, qui encourage les  « solutions pour le climat » . Bien sûr, ce n'est qu'une lueur d'espoir sur le chemin d'un développement durable. Mais elle est exemplaire d'une  « démarche adaptée aux besoins du territoire, sur le plan économique, social et environnemental » , assure Jean-François Moser, directeur général adjoint de l'entreprise.   Pitons et ravines   A La Réunion, comme dans l'ensemble des départements et régions d'outre-mer, la transition énergétique est un impératif, en même temps qu'une gageure. C'est du ciel, depuis un hélicoptère, seul moyen d'accéder aux villages les plus reculés de cette île volcanique aux reliefs vertigineux, que l'on prend la mesure du défi, mais aussi des efforts déployés pour tirer parti de ses ressources naturelles.   Voici d'abord la zone d'activité du Port. C'est ici qu'arrivent, par bateaux, le charbon d'Afrique du Sud, le fioul, l'essence et le gaz de Suède, de Singapour ou des Emirats arabes unis. Ces combustibles fossiles assurent près de 90 % de l'approvisionnement énergétique d'une île livrée à elle-même, au milieu de l'océan Indien, sans aucune connexion avec un réseau électrique. C'est ici aussi qu'EDF a investi 500 millions d'euros dans une nouvelle centrale thermique au fioul, plus performante et moins polluante, pour  « sécuriser le développement des filières renouvelables » , explique Michel Magnan, directeur régional de l'entreprise publique.   Voici maintenant, dominé par le majestueux piton des Neiges, le cirque de Mafate. Un amphithéâtre de verdure et de roc entre ciel et terre, hérissé de promontoires, creusé de gorges et de ravines. Dans des hameaux perchés sur des nids d'aigle (les îlets) vivent toujours les descendants des esclaves marrons qui, aux temps sombres de la colonie, avaient trouvé refuge sur ces à-pics. Ceux-là, coupés du monde, ne doivent leur électricité qu'à de petites installations photovoltaïques, couplées à des batteries. Au sud, les cheminées fumantes de l'une des deux centrales thermiques d'Albioma, le premier producteur d'électricité de l'île. Pendant la campagne sucrière, de mi-juillet à mi-décembre, elles brûlent en priorité de la bagasse, le résidu fibreux de la canne à sucre - principale culture locale -, le charbon prenant ensuite le relais.   Trop vite, trop tôt   Sur l'autre versant, une fois franchies les arêtes où s'accrochent les nuages, des cascades jaillissent de la végétation épaisse. A flanc de paroi, EDF a édifié des barrages acrobatiques et creusé des galeries, où des usines hydroélectriques turbinent les eaux tumultueuses de la rivière des Marsouins. Voici encore une batterie régulant la production de fermes solaires et éoliennes, des moulins à vent plantés sur une crête, des centres commerciaux et des parkings aux toitures tapissées de panneaux solaires...   Avec tout cela, si les énergies renouvelables fournissent un tiers de l'électricité de l'île, elles couvrent à peine 14 % de ses besoins énergétiques. La loi pour la croissance verte que viennent d'adopter définitivement les députés fixe pourtant des objectifs très ambitieux aux régions ultramarines : 50 % d'énergies renouvelables en 2020, puis  « l'autonomie énergétique à l'horizon 2030 » . Trop vite, trop tôt, pensent les élus réunionnais, peu séduits par le souhait de la métropole de faire de leur communauté un  « laboratoire de la transition énergétique » .   « L'autonomie énergétique reste le but à atteindre. Mais il faut être réaliste, avancer par étapes , défend le président du conseil régional, Didier Robert (Les Républicains ). Plutôt que de servir de terrain d'expérimentation, nous préférons être exemplaires, en impliquant toute la population. »  Sur ce point au moins, la députée socialiste de Saint-Denis, Ericka Bareigts, est d'accord.   « Notre territoire est en souffrance, il n'a pas vocation à devenir un banc d'essai, plaide-t-elle. Ce qu'il nous faut, c'est une vision d'avant-garde pour les vingt ou trente ans à venir, incluant l'énergie, l'habitat, les transports... C'est d'un projet de société qu'il s'agit. »Chantier gigantesque   En souffrance, La Réunion l'est assurément. Le chômage touche 30 % de la population, le double chez les jeunes. Près d'un habitant sur deux vit sous le seuil de pauvreté, un foyer sur trois est en précarité énergétique. A ces plaies s'ajoute le fardeau d'un parc de logements sociaux construit à la va-vite depuis les années 1970 - des cubes de béton mal isolés inspirés des banlieues métropolitaines -, qui n'empêche pas les poches d'habitat insalubre ou surpeuplé, tandis que plus de 20 000 familles sont en attente d'un toit.   Avec d'autres élus insulaires, Ericka Bareigts a bataillé pour que les territoires d'outre-mer élaborent eux-mêmes leur programmation pluriannuelle de l'énergie. Une façon de  « reprendre la main »  et de conquérir une autonomie politique, à défaut d'énergétique, sur ce dossier crucial pour leur avenir. Etabli par le conseil régional et la préfecture, ce document dessine une trajectoire conduisant, en 2023, à une électricité pour les deux tiers d'origine renouvelable. Deux fois plus qu'aujourd'hui. L'indépendance énergétique complète sera pour plus tard.   Le chantier est déjà gigantesque. Isolé, de petite taille, fragile - les coupures sont encore fréquentes -, le système électrique réunionnais, alimenté par une multiplicité de producteurs, mais dont EDF a le monopole du transport et de la distribution, est deux à trois fois plus coûteux que celui de l'Hexagone. Il est donc tributaire de la péréquation tarifaire, autrement dit la solidarité nationale, qui rend tous les Français égaux devant leur facture. Il doit aussi faire face à une explosion démographique : la population de l'île, 840 000 personnes aujourd'hui, a doublé en un demi-siècle et devrait dépasser un million d'habitants en 2040, entraînant une hausse continue de la demande, même si son rythme ralentit.   Route des cyclones   « Notre chance est d'avoir un mix électrique diversifié »,  observe Philippe Beutin, délégué régional de l'Agence de l'environnement et de la maîtrise de l'énergie (Ademe). Mais l'essor des filières vertes est freiné par la géographie et le climat. La Réunion croise la route des cyclones tropicaux, qui obligent à arrimer les éoliennes par des haubans ou à installer des mâts rabattables. Les vagues soulevées sur leur passage - jusqu'à 15 mètres de haut - ainsi que la profondeur des fonds océaniques contrarient la pose de turbines en mer. En janvier 2014, le cyclone Bejisa a même balayé un prototype houlomoteur.   Le relief, mais aussi l'inscription au Patrimoine mondial des  « pitons, cirques et remparts »  limitent les grands projets hydroélectriques, autant que les forages géothermiques autour de l'éruptif piton de la Fournaise. Quant au photovoltaïque, son déploiement est surtout envisagé sur les bâtiments, afin de ne pas empiéter sur des terres agricoles déjà rares. Il y faudra de grosses capacités de stockage pour lisser les fluctuations du soleil et du vent, et permettre aux énergies intermittentes, aujourd'hui plafonnées à 30 % sur le réseau, de monter en régime.   En dépit de ces contraintes, La Réunion n'a pas le choix. Elle doit, d'abord, maîtriser sa consommation. Les travaux d'isolation, l'achat par les ménages de modules photovoltaïques ou de chauffe-eau solaires - l'île en est la région d'Europe la mieux équipée après Chypre - sont fortement subventionnés. Des PME se lancent sur ce marché, comme Dak Industries qui, avec ses chauffe-eau  « anticyclones et anticorrosion » , marqués du label réunionnais  Nou la fé  (Nous l'avons fait), a déjà créé une quinzaine d'emplois.   « Investissement démesuré »   Elle doit aussi « métisser » toutes ses ressources. La biomasse est prometteuse. De nouvelles variétés de canne, sans sucre mais riches en fibres, pourraient être cultivées, les déchets verts et les résidus forestiers valorisés, des unités de méthanisation mises en place. Fin 2016, Albioma prévoit de faire tourner, au sud de l'île, la première turbine à combustion alimentée au bioéthanol, un alcool issu de la mélasse de canne.   Autre piste, novatrice, la géothermie marine. Il s'agit de pomper, à plus de 1 000 mètres de profondeur, l'eau fraîche (5 °C) des abysses, pour refroidir un réseau urbain d'eau douce et climatiser des bâtiments. A Saint-Denis et Sainte-Marie, le projet Seawater Air Conditioning, porté par Engie et soutenu par la région, l'Ademe et l'Europe, pour un coût de 150 millions d'euros, devrait desservir, d'ici deux ans, l'aéroport, le centre hospitalier, l'université, des immeubles de bureaux et des centres commerciaux.   Reste un énorme point noir : les transports, qui représentent 70 % des besoins énergétiques. La précédente majorité, menée par le sénateur Paul Vergès (Parti communiste réunionnais), avait un projet de « tram-train » pour desservir la côte ouest, congestionnée et menacée d'éboulements rocheux. La droite lui a préféré une « nouvelle route du littoral », avec viaduc sur pilotis et digue - là même où viennent se reproduire les baleines à bosse -, chiffrée à 1,6 milliard d'euros.   « Une erreur historique, un investissement démesuré, qui favorise le tout-automobile » , juge Jacques Lowinsky, premier adjoint (PS) de Saint-Denis. Début juin, à l'appel du collectif Touch pas nout roche, 4 000 personnes ont protesté contre l'exploitation de l'une des carrières d'où doivent être extraites les 18 millions de tonnes de galets nécessaires à cet ouvrage. De mémoire de créole, la plus grande manifestation environnementale jamais organisée ici. L'émancipation énergétique de La Réunion sera encore une longue marche.
</article>
<article titre="Valorisation de la Biomasse en Guadeloupe" date="2015 07 16" journal="Rapports - Agroalimentaire - MAAF">
63 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... Rapport n° 14063  établi par  Jean-Yves Grosclaude Ingénieur général des ponts, des eaux et des forêts  Juin 2015 PRÉAMBULE  Les Etats Généraux de l’Outre Mer ont mis l’accent...    Pour voir le rapport :  http://agriculture.gouv.fr/sites/minagri/files/cgaaer_14063_2015_rapport.pdf
</article>
<article titre="L'essence du Brésil" date="2015 07 9" journal="Challenges">
L'essence du Brésil L'Or de Quipapa, d'Hubert Tézenas, Métailié, 208 pages, 9 euros.   Recife, dans l'Etat de Pernambouc, au nord-est du Brésil. Les chemises collent aux torses, l'air sent la bagasse (déchets de canne à sucre), et les pistolets claquent. Alberico Cruz, agent immobilier sans envergure, se retrouve accusé de meurtre. De son plongeon dans des geôles sordides aux exactions des propriétaires terriens, Hubert Tézenas réussit un coup de maître avec ce premier roman situé dans les années 1980. Entre le je du fils de famille qui s'octroie tous les droits et le il de celui qui est le jouet des événements, les voix rythment un polar haletant et social. L'auteur, amoureux du Brésil, en révèle l'essence.
</article>
<article titre="Les CM1-CM2 font revivre l'oeuvre de Marcel Pagnol" date="2015 07 4" journal="L'Indépendant  CATALAN_IN">
ORTAFFA Sous la direction de leur professeur des écoles Christophe Pellissier, les élèves ont préparé avec soins, depuis novembre 2014, un spectacle avec comme thème « Autour de l'oeuvre de Marcel Pagnol », avec huit extraits de Marius, Fanny et César :   Un bon mélange, Landolfi, La partie de cartes, Oh bagasse ! Le Pitalugue, Un ami... ye, La Partie de pétanque, Le Chapeau e  t, un chant collectif, Oui, j'ai gardé l'accent !   Chaque extrait interprété avec brio par ces jeunes acteurs, avec des costumes de scène adaptés, ont agréablement surpris par leurs prestations leurs camarades de l'école élémentaire, mais aussi leurs parents, qui ont marqué leur admiration en battant des mains, en demandant des rappels. Félicitations à Christophe Pellissier, metteur en scène.
</article>
<article titre="CIMV mobilise 55 millions pour son projet de bioraffinerie" date="2015 07 2" journal="Les Echos, no. 21971">
La production de carburant et de plastique avec des denrées agricoles est peu rentable et diminue, de surcroît, les ressources alimentaires. Au contraire, la bioraffinerie de deuxième génération récupère des déchets de plantes (pailles de céréales, bagasse de canne à sucre...) et de forêts. Pour prouver sa rentabilité, un consortium de sept entreprises et centres de R&amp;D européens : Toulouse White Biotechnology, Rolkem, Taurus Energy, Dyadic... piloté par CIMV ouvrira, en 2017, un démonstrateur préindustriel sur la zone des Portes du Tarn, près de Toulouse, qui créera 15 emplois. Baptisé « 2G Biotic » , ce projet nécessitera 55 millions d'euros sur trois ans, dont 20 millions de l'Union européenne et 15 millions de CIR. Ce démonstrateur produira 700 tonnes d'éthanol et 750 tonnes de lignine par an, en traitant 24 tonnes de biomasse par jour. C'est à partir d'une découverte de Michel Delmas, professeur de chimie à l'Ensiacet à Toulouse et cofondateur de CIMV, que la société a mis au point son procédé de séparation douce de la cellulose, des hémicelluloses et de la lignine, en trempant les végétaux dans des acides acétique et formique, sans solvant chimique. Après avoir enlevé la cellulose, la raffinerie sépare en douceur la lignine par précipitation en ajoutant de l'eau. Ainsi, elle ne se dégrade pas et reste pure. Or la lignine peut remplacer les résines phénoliques dérivées du pétrole pour produire colles, plastiques, pneus, époxy... «  Cela va changer la face de l'industrie chimique  » , affirme Michel Delmas. Plus rentable, ce procédé produit 360 litres d'éthanol par tonne de biomasse au lieu de 240 litres d'ordinaire. CIMV, qui a déposé 10 familles de brevets et investi 40 millions en R&amp;D, veut construire ensuite une bioraffinerie. Laurent Marcaillou
</article>
<article titre="Lettre de décision relative à CENDRES DE  BAGASSE ALBIOMA LE GOL" date="2015 06 25" journal="Rapports - Agroalimentaire - MAAF">
5 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... MINISTERE DE L’ AGRICULTURE,  DE L’ AGROALIMENTAIRE ET DE LA FORET  MFSC CENDRES DE BAGASSE ALBIOMA LE GOL page 1/5 251, rue de Vaugirard 75732 PARIS CEDEX 15 Tél.: 01...    Pour voir le rapport :  http://agriculture.gouv.fr/sites/minagri/files/dec_mfsc_cendres_de_bagasse_albioma_le_gol.pdf
</article>
<article titre="Lettre de décision relative à CENDRES DE  BAGASSE ALBIOMA BOIS-ROUGE" date="2015 06 25" journal="Rapports - Agroalimentaire - MAAF">
5 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... MINISTERE DE L’ AGRICULTURE,  DE L’ AGROALIMENTAIRE ET DE LA FORET  MFSC CENDRES DE BAGASSE ALBIOMA BOIS-ROUGE page 1/5 251, rue de Vaugirard 75732 PARIS CEDEX 15 Tél.: 01...    Pour voir le rapport :  http://agriculture.gouv.fr/sites/minagri/files/dec_mfsc_cendres_de_bagasse_albioma_bois-rouge.pdf
</article>
<article titre="Lettre de décision relative à CENDRES DE  BAGASSE ALBIOMA LE MOULE" date="2015 06 25" journal="Rapports - Agroalimentaire - MAAF">
5 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... MINISTERE DE L’ AGRICULTURE,  DE L’ AGROALIMENTAIRE ET DE LA FORET  MFSC CENDRES DE BAGASSE ALBIOMA LE MOULE page 1/5 251, rue de Vaugirard 75732 PARIS CEDEX 15 Tél.: 01...    Pour voir le rapport :  http://agriculture.gouv.fr/sites/minagri/files/dec_mfsc_cendres_de_bagasse_albioma_le_moule.pdf
</article>
<article titre="CONFÉRENCE DES NATIONS UNIES SUR LES CHANGEMENTS CLIMATIQUES Les territoires en route pour la COP 21" date="2015 06 21" journal="Rapports - Banques et institutions financières - Caisse des Dépôts et consignation">
42 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... en partenariat avec :  n Les territoires en route pour la COP 21  Les notions indispensables sur les impacts du changement climatique,  les politiques climatiques et les outils économiques 2 3  Avertissement...    Pour voir le rapport :  http://www.cdcclimat.com/spip.php?action=telecharger&amp;arg=3035
</article>
<article titre="CONFÉRENCE DES NATIONS UNIES SUR LES CHANGEMENTS CLIMATIQUES Livret des territoires en route pour la COP 21 Juin 2015" date="2015 06 21" journal="Rapports - Banques et institutions financières - Caisse des Dépôts et consignation">
42 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... en partenariat avec :  n Les territoires en route pour la COP 21  Les notions indispensables sur les impacts du changement climatique,  les politiques climatiques et les outils économiques 2 3  Avertissement...    Pour voir le rapport :  http://www.cdcclimat.com/spip.php?action=telecharger&amp;arg=3036
</article>
<article titre="Conseil Albioma" date="2015 06 20" journal="Investir-Le Journal des finances, no. 2163">
Biomasse : Albioma est le spécialiste mondial de la valorisation par cogénération à haute efficacité énergétique de la bagasse, résidu de la canne à sucre, seconde ressource biomasse de la planète.   Le groupe détient une capacité installée d'environ 800 mégawatts (MW) de production d'électricité d'origine renouvelable, équivalente à la puissance d'une tranche de centrale nucléaire. Albioma est née à La Réunion en 1992 et s'est développée ensuite dans les pays de canne à sucre, Guadeloupe, Maurice, Martinique et aujourd'hui Brésil, avec une politique d'investissements de 100 millions d'euros par an en moyenne. Chaque investissement est porté par une société de projet autonome dont le financement bancaire à long terme est sans recours à l'actionnaire.   Projets à la Martinique et à Marie-Galante En complément de sa propre discipline d'investissement, Albioma utilise ainsi le passage de ses projets au crible de banques pour en confirmer la solidité.   La rentabilité moyenne de chaque projet attendue par Albioma est d'environ 15 %, auxquels il faut ajouter une éventuelle prime de risque pays, notamment au Brésil.   Outre-mer, Albioma va construire de nouvelles centrales 100 % biomasses, assurant ainsi la transition énergétique de ces territoires isolés en augmentant progressivement la part des énergies renouvelables. Le projet Galion 2 à la Martinique (40 MW, 170 millions d'euros) et celui de Marie-Galante (13 MW, 80 millions d'euros) mobiliseront, avec des rendements très élevés, le pouvoir énergétique de la bagasse pour fournir aux usines sucrières toute l'énergie dont elles ont besoin et exporter le solde d'énergie vers le réseau électrique.   En dehors des périodes de coupe de la canne à sucre, les usines brûleront de la biomasse issue de filières locales ainsi que de la biomasse importée.   Un milliard investi entre 2013 et 2023 Au Brésil, Albioma se développe rapidement, avec un modèle moins consommateur de ressources financières, en achetant des usines de cogénération de bagasse existantes. La performance énergétique de ces centrales sera plus que doublée, grâce à l'expertise unique du groupe développée depuis plus de vingt ans.   De 2013 à 2023, Albioma va réaliser un programme ambitieux de 1 milliard d'euros d'investissements autofinancés par les cash-flows générés par les activités existantes et de la dette bancaire, sans recours à l'actionnaire, doublant ainsi la taille de son patrimoine industriel tout en maintenant un niveau élevé de rentabilité.   Il convient, pour conclure, d'évoquer l'investissement humain dans le recrutement et la formation des collaborateurs de l'entreprise. C'est l'investissement le plus réjouissant et le plus rémunérateur, sans lequel l'investissement dans les usines n'est rien.   Les 400 employés d'Albioma garantissent la qualité du service aux clients, sucriers et électriciens, grâce à leur expertise unique qui assure le développement durable de l'entreprise au service de la transition énergétique de la planète.   * PDG d'Albioma. Philippe Berterottière*   Jacques   Pétry*
</article>
<article titre="CIMV compte investir plus de 55 millions d'euros dans son démonstrateur industriel" date="2015 06 19" journal="L'Usine Nouvelle.com">
La Compagnie industrielle de la matière végétale (CIMV) vient de confirmer son ambition d'implanter un démonstrateur industriel de son procédé de bioraffinage à Saint-Sulpice (Tarn).   Un premier pilote industriel a été implanté dès 2007 à Pomacle, dans la Marne. L'objectif de CIMV (Compagnie industrielle de la matière végétale) est maintenant de changer d'échelle. Après avoir envisagé de passer directement au stade industriel, la société a finalement retenu l'option d'une étape intermédiaire, celle du démonstrateur industriel. L'objectif est d'engager l'ultime étape de R&amp;D pour valider le concept de bioraffinerie autour duquel travaille CIMV depuis une quinzaine d'année. La société, créée en 1998, propose une nouvelle technologie de séparation de la matière végétale, avec à la clef une filière complète de valorisation de la cellulose, de la lignine et des sucres. Le procédé s'applique à tout type de résidu non alimentaire (paille de riz, de blé, tiges de maïs, bagasse de canne à sucre ou résidus de l'industrie forestière). "Il suffit juste d'adapter le pré traitement des matières premières et notamment les broyeurs en entrée de ligne", précise Michel Delmas, co-fondateur et directeur scientifique de CIMV.   Projet européen Biocore   Les travaux de R&amp;D de la société sont protégés par une dizaine de familles de brevets et un premier projet européen (Biocore), conduit de 2010 à 2014 et coordonné par l'Inra (Institut national de recherche agronomique) vient de confirmer le potentiel de cette voie technologique pour la conversion des résidus agricoles et forestiers en biocarburants de deuxième génération et en produits chimiques pour la production de biopolymères et de biomatériaux. Biocore préconise par ailleurs dans ses conclusions, de construire une usine de démonstration avant de passer au stade industriel.   Programme européen 2G Biopic   C'est maintenant dans le cadre d'un nouveau programme européen, baptisé 2G Biopic et lauréat de l'appel à projets "Demonstrating adanced biofuel technologies" de l'Union européenne, que s'inscrit le nouvel investissement de CIMV, avec à la clef un soutien financier de l'Europe, au titre du programme Horizon 2020 recherche et innovation, de 20 millions d'euros sur un budget de 35 millions d'euros. Porté par un consortium de sept partenaires industriels issus de quatre pays européens, dont les industriels Dyatic (Pays-Bas), Taurus Energy (Suède) et Rolkem (France), et piloté par CIMV, 2G Biopic associe également le Critt Bio-Industries à Toulouse et bénéficie d'un partenariat avec la plateforme toulousaine TWB (Toulouse White Biotechnology).   Une enveloppe de plus de 55 millions d'euros   CIMV vient de retenir 2,5 hectares sur le nouveau parc d'activités des Portes du Tarn, à Saint-Sulpice. Outre le démonstrateur industriel, qui permettra de traiter jusqu'à 24 tonnes de biomasse par jour et de produire annuellement 700 tonnes de bioéthanol et 750 tonnes de biolignine, CIMV compte installer son siège et ses équipes. La société emploie actuellement une vingtaine de personnes. "Globalement, notre projet devrait dépasser les 55 millions d'euros", souligne Thierry Scholastique, le directeur général de CIMV.   La société, qui a déjà mobilisé une quarantaine de millions d'euros depuis sa création, s'apprête à boucler dans les prochains mois un nouveau tour de table de l'ordre de 20 millions d'euros, pour conduire cette nouvelle étape de R&amp;D. Son calendrier est serré : le chantier devrait démarrer dès le début de l'année prochaine, pour une mise en production du démonstrateur en début d'année 2017. Et il faudra alors préparer l'étape suivante, celle de l'unité industrielle.   Marina Angel
</article>
<article titre="Énergies renouvelables : Albioma dans une phase incertaine" date="2015 06 15" journal="E-news Environnement Magazine - Énergie (site web)">
Contrôlé depuis 2005 par le fonds d'investissement britannique Apax Partners et son associé Altamir, Albioma n'aura plus d'actionnaire de référence dans les prochains jours.   Pour l'exploitant de centrales électriques à biomasse (bagasse-charbon) et photovoltaïques, une période d'incertitudes s'ouvre. Apax a décidé de distribuer la participation de 30 % qu'il détenait au capital de l'ex-Séchilienne Sidec à ses propres investisseurs. Une façon pour eux de récupérer l'argent qu'ils avaient confié à Apax en 2000, lors de la constitution du véhicule Apax France VI. Ce fonds arrivait à maturité cette année et il devait donc être fermé. N'ayant pas réussi à vendre sa part à un industriel ou à un autre fonds, Apax a décidé de rembourser ses investisseurs directement en actions Albioma. Ces derniers vont ainsi devenir les premiers actionnaires du producteur d'électricité, avec environ 30 % de son capital.  Mais il est probable qu'ils ne le restent pas longtemps, leur objet social n'étant pas nécessairement d'être directement actionnaire d'une société d'électricité. D'autant qu'environ la moitié d'entre eux se trouvent aux États-Unis. La direction d'Albioma s'est voulue rassurante en expliquant que le plus gros investisseur d'Apax représente à lui seul à peine 2 % de son capital. Pour autant, un industriel ou un financier pourrait profiter de l'éparpillement du capital pour acheter une partie des actions de la société et en devenir le nouvel actionnaire fort. En 2005, Apax avait emporté le rachat de Séchilienne-Sidec, mis en vente à l'époque par Air liquide, au nez et à la barbe de l'espagnol Endesa.    Malgré ce changement brutal d'actionnaires, la direction d'Albioma a confirmé vouloir maintenir la stratégie mise en oeuvre depuis 2012, avec la volonté de mener à bien le plan d'investissement d'un milliard d'euros sur dix ans. Près de la moitié a déjà été lancée avec la volonté de faire du Brésil le deuxième marché du groupe après les départements français d'outre-mer.  Cet article est paru dans E-news Environnement Magazine - Énergie (site web)
</article>
<article titre="La Réunion: malgré la polémique, Valls a parlé emploi, agriculture et énergie" date="2015 06 11" journal="AFP Infos Françaises">
Saint-Denis de la Réunion, 11 juin 2015 (AFP) - - La première journée de Manuel Valls dans l'océan Indien s'est ouverte et close sur la polémique de son voyage à Berlin, mais dans l'intervalle, le Premier ministre a balayé de nombreux sujets économiques et sociaux de la Réunion, île frappée par un chômage de masse.   C'est à Saint-Joseph, à l'extrême-sud et fief du député maire socialiste Patrick Lebreton, que M. Valls a reçu son plus bel accueil et réservé ses plus importantes annonces.  Dans la rue principale pavoisée, quelques centaines de citoyens se massaient, enthousiastes pour ceux qui sont militants socialistes, juste curieux de "voir le Premier ministre" pour les autres.    C'est dans cette mairie créole, toit de tôle rouge et fenêtres aux volets de bois blanc et bleu, que le Premier ministre a signé une charte en faveur de l'emploi local avec neuf entreprises privées (Air Austral, Brasserie Bourbon, GTOI filiale de Colas, etc.) et les collectivités.   Une occasion pour M. Valls de redire toute sa "confiance" aux entrepreneurs, et rappeler les mesures dévoilées mardi pour les PME et TPE, qui forment l'essentiel du tissu économique de La Réunion.   Il a annoncé que le gouvernement prendrait une ordonnance, sans doute d'ici la fin de l'année, pour inscrire dans les appels d'offres de marchés publics une clause favorisant les entreprises employant des Réunionnais.   Et si l'emploi privé est essentiel pour contrer un "chômage insupportable" de 30% sur l'île, dont 60 % chez les jeunes, Manuel Valls est venu annoncer la part de l'Etat dans l'effort: 4.400 emplois aidés supplémentaires pour La Réunion, sur les 100.000 prévus nationalement. Cela portera à plus de 40.000 le nombre total de personnes bénéficiant en 2015 d'un emploi aidé dans ce département, le plus peuplé d'outre-mer avec 849.000 habitants.   - 'Made in La Réunion' - Autre sujet important, les négociations tendues sur la nouvelle convention 2015-2021 liant les planteurs de canne à sucre avec les industriels et l'Etat ont abouti jeudi soir, permettant une signature dument médiatisée en préfecture à Saint-Denis.   La prime bagasse (résidu fibreux de la canne utilisé dans les centrales électriques à la biomasse) a été revalorisée de 11 à 14,50 euros la tonne et les sucriers ont obtenu des engagements de l'Etat pour les défendre contre la fin des quotas et la libéralisation sur le marché européen.   Ce secteur emploie directement 12.000 personnes, 18.300 en comptant les empois indirects soit 9,9% de la population active de La Réunion.   Manuel Valls, qui n'a cessé de dire que les "outre-mer ne sont pas un fardeau mais une chance pour la France", a aussi eu à coeur de visiter plusieurs réussites réunionnaises.   Le matin, au Port (nord-ouest), c'était l'inauguration sur les terrains de la prison d'une centrale de production et de stockage d'électricité par des panneaux photovoltaïque placé sur les toits de serres de maraîchage, où des détenus se réinsèrent par le travail.   Le Premier ministre y a dégusté des fruits de la passion, comme il a déjeuné d'un rougaille saucisse (plat typique) lors du déjeuner avec des jeunes volontaires du régiment du service militaire adapté (SMA). Cet encadrement militaire de jeunes déscolarisés pour leur redonner des repères et leur apprendre un métier s'enorgueillit d'un taux d'insertion de plus de 70%. Un système qui va essaimer dans l'Hexagone vu son efficacité.   Dans le complexe avicole Crête d'Or à Etang Salé (ouest), Manuel Valls aura inauguré l'abattoir flambant neuf, un investissement de 73 millions d'euros dont 28 millions d'aides d'Etat via la défiscalisation. Cet exemple de filière locale, qui compte dans ses débouchés la restauration scolaire, c'est le "made in La Réunion" salué le premier ministre.   sla/mat/sd
</article>
<article titre="Manuel Valls, une journée de contrition à la Réunion Le Premier ministre a profité de son déplacement dans l'île pour faire son mea culpa." date="2015 06 11" journal="Libération (site web)">
Manuel Valls a entamé jeudi une visite de trois jours dans l'Océan indien. À la Réunion, il a reconnu que son escapade sportive à Berlin en compagnie de ses enfants était une erreur et a promis de rembourser. Récit d'une journée de contrition, menée au pas de charge.   9 heures, préfecture de Saint-Denis   Autant en finir tout de suite ! Au lieu de se rendre après la descente de son avion au monument aux morts de Saint-Denis, comme prévu initialement, Manuel Valls a convié les journalistes dans les salons boisés de la préfecture du chef lieu de la Réunion. Là, le visage crispé, les traits fatigués, le Premier ministre a tenté d'éteindre la polémique née dimanche, suite à son déplacement à Berlin, en Falcon avec ses enfants, pour assister à la finale de la Ligue des champions de football. Le supporter du Barça a d'abord réaffirmé qu'il s'agissait bel et bien d'un voyage officiel, à l'invitation de Michel Platini, président de l'UEFA. «Mon rôle est de représenter la France lors de ces grands événements sportifs, c'est essentiel pour le rayonnement de notre pays, pour son attractivité et donc pour l'emploi.» Mais la présence de ses enfants ? «L'avion était déjà affrété, leur présence n'a pas coûté un euro de plus au contribuable.» Pour autant, Manuel Valls s'est dit «sensible à la réaction des Français, à ce que pensent nos compatriotes». Et de reconnaître : «Je me dois d'incarner un comportement parfaitement rigoureux, il ne peut y avoir de place pour le moindre doute.» Aussi a-t-il décidé de prendre en charge le voyage de ses deux enfants, dont le coût est estimé à 2500 euros. L'aveu d'une erreur chez un «homme fait de sang et de chair», comme il l'a dit en fin de journée sur les chaines de télévisions locales. «Si c'était à refaire, je ne le referais pas», a-t-il conclu, espérant en avoir fini avec cette mauvaise passe et «cette erreur de communication». Il était enfin temps de rendre hommage aux morts, rue de la Victoire, devant un public clairsemé qui réclamait son petit bain de foule. Le Premier ministre est déjà parti et seulement 300 manifestants protestent devant les grilles de la préfecture contre «la politique d'austérité» du gouvernement et sa réforme du collège. Najat Vallaud-Belkacem, la ministre de l'Éducation, qui arrive vendredi sur l'île, va pouvoir poser la première pierre d'une école sereinement...   10h30, centrale photovoltaïque du Port   Le Premier ministre passe par la case prison. Le centre de détention du Port, au nord-ouest de l'île, a mis de la tôle sur ses taulards : les cellules où dorment 450 condamnés ont été couvertes de 27 000 cellules photovoltaïques. Une première mondiale, dans la mesure où l'électricité ainsi produite est stockée dans d'énormes batteries en lithium-plomb. Mais l'originalité tient surtout à la démarche sociale et environnementale d'Akuo Energy, la société à l'origine du projet. Sept serres immenses, également tapissées de panneaux solaires, abritent des cultures de légumes, cuisinés à la cantine de la prison. Les détenus eux-mêmes se chargent des plantations, dans cette prison inédite où 300 d'entre eux vivent en semi-liberté dans une enceinte en plein air et arborée. Le Premier ministre y a d'ailleurs planté une espèce endémique, un Bois de senteur blanc.   Pierre, 25 ans, condamné à quatre ans d'enfermement, tatoué d'une vilaine toile d'araignée, est fier de ses tomates qu'il présente dans une brouette. Manuel Valls préfère goûter un fruit de la passion et une cuillère de miel, également produit dans la prison, avant de féliciter l'outre-mer français, «à l'avant-garde de la transition énergétique». Et de citer un autre projet innovant : le Swac, qui consiste à puiser l'eau froide de la mer à plus de 1000 m de profondeur pour refroidir les circuits de climatisation de Saint-Denis et Sainte-Marie, permettant une économie de 75% de la facture d'électricité.   11 heures, les requins de Saint-Gilles les Bains   Mille internautes ont répondu à l'invitation de l'Effet Péi, une marque locale de tee-shirts, qui a convié Manuel Valls à se baigner dans l'océan. Objectif : prouver qu'il n'y a pas des requins partout autour de l'île. Mais l'ancien ministre de l'Intérieur ne s'est pas mouillé pour une enseigne dont le nom évoque le bien-être provoqué par le zamal (le hachisch en créole)...   12h30, Saint-Pierre, régiment du service militaire adapté   Plus décontracté qu'au petit matin, Manuel Vall «casse la blague» avec George Pau Langevin, la ministre de l'Outre-Mer qui l'accompagne, lors de la visite de la caserne du RSMA de la capitale du Sud. «T'es déjà venue ici, toi, tu as ta chambrée, hein ?» Chapeau de paille pour se protéger du soleil encore fort de l'hiver austral, sa collègue du gouvernement sourit : «C'est bien qu'il vienne sur le terrain se rendre compte des difficultés; après j'ai plus de pouvoir pour lui arracher un arbitrage.» En l'occurrence, le régiment de service militaire adapté se porte bien: il a doublé ses capacité et accueille aujourd'hui 1300 stagiaires. «Pas pour en faire des guerriers», comme l'explique le colonel Winckel, mais pour leur donner un projet de vie : 40% des volontaires sont illettrés et exclus du système scolaire. Après un an de formation dans ce cadre militaire, 70% d'entre eux dénichent une solution d'insertion. Le système fait ses preuves depuis cinquante ans et va être étendu en métropole, où sept centres doivent créés d'ici l'an prochain.   14h30, mairie de Saint-Joseph   Après une salade chouchou et un rougail saucisses, les plats «nationaux» de la Réunion, dégustés en compagnie de l'ancien champion de handball réunionnais, Patrick Cazal, Manuel Valls prend la direction du Sud Sauvage. À Saint-Joseph, bourgade plantée au milieu des champs de canne, il y rencontre son «ami» Patrick Lebreton. Ce dernier est le pendant du frondeur socialiste Christian Paul. Même physique, moustache en moins, mais un frondeur à la droite du PS. Le député-maire est l'un des rares socialistes à avoir voté contre le mariage pour tous, et s'oppose au premier secrétaire de la fédération locale, le maire tout-puissant de Saint-Denis, Gilbert Annette. Aucun des deux ne bénéficiera d'un coup de pouce du Premier ministre en cette journée plutôt oecuménique, pour savoir lequel conduira la liste aux prochaines régionales... Le scrutin s'annonce difficile : si aux présidentielles de 2012, François Hollande l'a emporté à la Réunion au second tour contre Nicolas Sarkozy avec 71,5% des voix, si cinq des sept députés de l'île sont socialistes, la gauche a depuis perdu le conseil général; et la droite tient désormais seize des vingt-quatre communes.   Mais la séquence du jour est plutôt tournée vers l'économie. Manuel Valls proclame à nouveau son amour pour les patrons, invités à signer une charte sur la préférence au recrutement local. «Il faut en finir avec l'opposition stérile chefs d'entreprise/salariés, assène le Premier ministre devant une foule acquise, qui secoue des petits drapeaux tricolores. Bien sûr, il faut des droits et du dialogue social, mais il faut faire confiance aux entrepreneurs qui créent de la richesse.» Pas d'annonce sur le sujet, mais 10 000 emplois aidés supplémentaires pour le département, afin de réduire «le niveau insupportable» du chômage, qui dépasse ici les 30%. Le discours passe bien auprès des groupies, indifférents au «coup de l'avion à Berlin». «Faut bien qu'il en profite un peu, de toute façon, cet argent ne nous serait pas revenu», soupirent Marie-Thérèse et Roland, mariés depuis 41 ans et bénéficiaires du RSA : les deux sexagénaires se plaignent du prix des denrées dans les supermarchés réunionnais mais gardent leur foi, «depuis Mitterrand», au gouvernement.   16h30, abattoir d'Etang-Salé   Coup de froid et ambiance nettement plus glaciale à l'Etang-Salé, où la délégation visite un abattoir flambant neuf. Quatre degrés à l'intérieur des bâtiments. Charlotte bleue sur la tête, blouse et bottes blanches, Manuel Valls déambule entre les crochets et les tapis roulants, rendant hommage à «l'un des plus beaux outils de production de France». Chaque semaine, 160 000 poulets y sont abattus. L'usine du groupe Urcoopa a bénéficié de l'aide fiscale de l'État et de subventions européennes...comme l'ensemble des grands projets portés dans le département.   18h30, préfecture de Saint-Denis   Manuel Valls termine la journée par «une bonne nouvelle» en arrachant la signature de la convention canne, sur laquelle butaient depuis plusieurs jours les planteurs de canne à sucre, les usiniers et l'État. La bagasse, le résidu après extraction du jus, brûlé pour fabriquer de l'électricité, sera désormais achetée 14,50 euros la tonne aux planteurs, au lieu de 11.   20 heures, dîner républicain   Dîner avec les élus locaux, à la préfecture, avant une nouvelle journée au pas de charge vendredi et une visite, samedi, à Mayotte. Il y sera question des immigrés clandestins et des boat-people qui meurent noyés au large des côtes françaises...
</article>
<article titre="Énergies renouvelables : Biomasse : Albioma entre dans une zone de turbulences" date="2015 06 10" journal="Environnement Magazine (site web)">
Contrôlé depuis 2005 par le fonds d'investissement britannique Apax Partners et son associé Altamir, l'exploitant de centrales électriques à biomasse (bagasse-charbon) ou photovoltaïques Albioma n'aura plus d'actionnaire de référence dans les prochains jours.   Apax a décidé de distribuer la participation de 30 % qu'il détenait au capital de l'ex-Séchilienne Sidec à ses propres investisseurs. Une façon pour eux de récupérer l'argent qu'ils avaient confié à Apax en 2000, lors de la constitution du véhicule Apax France VI. Ce fonds arrivait à maturité cette année et il devait donc être fermé. N'ayant pas réussi à vendre à un industriel ou à un autre fonds sa part, Apax a décidé de rembourser ses investisseurs directement en actions Albioma. Ces derniers vont ainsi devenir les premiers actionnaires du producteur d'électricité, avec environ 30 % de son capital. Mais il est probable qu'ils ne le restent pas longtemps, leur objet social n'étant pas nécessairement d'être directement actionnaire d'une société d'électricité. D'autant qu'environ la moitié d'entre eux se trouvent aux États-Unis. La direction d'Albioma s'est voulue rassurante en expliquant que le plus gros investisseur d'Apax représente à lui seul à peine 2 % de son capital. Pour autant, un industriel ou un financier pourrait profiter de l'éparpillement du capital pour acheter une partie des actions de la société et en devenir le nouvel actionnaire fort. En 2005, Apax avait emporté le rachat de Séchilienne-Sidec, mis en vente à l'époque par Air Liquide, au nez et à la barbe de l'espagnol Endesa. Malgré ce changement brutal d'actionnaires, la direction d'Albioma a confirmé vouloir maintenir la stratégie mise en oeuvre depuis 2012, avec la volonté de mener à bien le plan d'investissement d'un milliard d'euros sur dix ans. Près de la moitié a déjà été lancé avec la volonté de faire du Brésil le deuxième marché du groupe après les départements français d'outre-mer.  Cet article est paru dans Environnement Magazine (site web)
</article>
<article titre="Biomasse : le fonds Apax sort du capital d'Albioma" date="2015 06 8" journal="Les Echos, no. 21953">
L'opération présage-t-elle d'un nouveau tour de table pour Albioma ? Vendredi, Apax a annoncé le transfert à ses investisseurs de ses 30 % de participation dans l'ex-Séchilienne-Sidec, dont il avait pris le contrôle en 2005 auprès d'Air Liquide pour 160 millions d'euros. L'opération a pour conséquence d'augmenter d'autant le flottant du producteur d'énergie coté en Bourse. Ces investisseurs sont à 44 % nord-américains et à 56 % européens. En tant que fonds de fonds ou fonds de pension, ils n'ont pas forcément vocation à détenir ces titres en direct. Ils pourraient donc les revendre.  « Nous ne connaissons pas le profil ni les intentions des nouveaux investisseurs avec un risque de retour de papier »  , commente une analyse d'Oddo. Néanmoins, relève la note,  « l'investisseur le plus important au sein de la partie reclassée représente moins de 2 % du capital du groupe »  . L'impact sur le titre pourrait donc être limité. Altamir, un véhicule indirectement lié à la galaxie Apax, va conserver 12 % d'Albioma. Ces investisseurs n'ont pas eu leur mot à dire dans la reprise des titres, conformément au règlement du fonds. En cédant ses 30 %, Apax peut enfin clore cet ancien véhicule, lancé en 2000. La société d'investissement dirigée par Eddie Misrahi portait cette participation depuis dix ans et, en toute logique, aurait dû céder le contrôle d'Albioma plus tôt, compte tenu des cycles d'investissement de cinq ans dans le non-coté. Selon des sources, Apax avait tenté l'an dernier de trouver des investisseurs sans que le processus aboutisse. De même en 2009-2010, mais le projet avait été mis à mal par la crise.   Jusqu'à l'arrivée de Jacques Pétry, en 2012, l'ex-Séchilienne-Sidec, dont l'actionnaire de référence Hélios est passé par une procédure de sauvegarde, n'était pas au meilleur de sa forme. Depuis, le redressement opéré pourrait relancer l'intérêt de nouveaux investisseurs. Le bénéfice net du groupe aux objectifs clarifiés est passé de 25,1 millions d'euros à 38 millions d'euros, le cours de Bourse a grimpé de 70 % et Jacques Pétry s'est lancé dans un plan ambitieux.   Nouvelle usine Essentiellement présent dans la transformation d'énergie à partir de la canne à sucre (bagasse) et valorisé 510 millions d'euros en Bourse, Albioma a lancé un plan d'investissement de 1 milliard d'euros sur dix ans, dont la moitié est déjà assurée. Le groupe a opéré deux acquisitions au Brésil. Outre-mer, une nouvelle usine de cogénération, qui va nécessiter 170 millions d'euros d'investissements, est en cours de lancement.  « Nous voulons doubler le patrimoine industriel d'Albioma et assurer une croissance annuelle de 7,5 % du résultat »  , indique Jacques Pétry. L'offre d'énergie à partir de biomasse dispose d'un vrai potentiel.  « Actuellement, seuls quelques pourcents des 500 millions de tonnes de bagasse produits par an sont valorisés »  , poursuit le dirigeant, qui n'exclut pas de lancer une augmentation de capital pour assurer, au besoin, le financement de ses nouveaux projets.   Pour l'heure, cependant, Albioma a dû faire un avertissement sur ses résultats après des incidents techniques sur des centrales. Pour 2015, le bénéfice est attendu en baisse de 10,7 %. Après l'annonce du transfert de capital, le titre a chuté de 6,36 %, à 17,23 euros.   Anne Drif
</article>
<article titre="Apax Partners cède sa participation dans Albioma" date="2015 06 5" journal="Le Figaro.fr">
Le fonds d'investissement Apax Partners va céder à ses actionnaires 95% de sa participation dans le producteur français d'électricité Albioma, ce qui équivaut à environ 30% de son capital. Dans un communiqué, Albioma (ex-Séchilienne-Sidec) précise que cette distribution, qui aura pour conséquence d'augmenter significativement le flottant de la société, interviendra dans la deuxième quinzaine du mois de juin.   A l'issue de cette opération, le fonds d'investissement Altamir détiendra une participation directe et indirecte de 12% d'Albioma, dont l'activité est essentiellement dans l'Outre-mer.   Le conseil d'administration d'Albioma a par ailleurs réaffirmé la poursuite de la stratégie approuvée en mars 2012 par son assemblée générale, portant d'une part sur l'optimisation de la performance des centrales thermiques existantes, et d'autre part le développement de nouveaux projets dans l'Outre-mer, sur l'Île Maurice, au Brésil et dans d'autres pays.   "Depuis trois ans, nos équipes ont travaillé avec succès pour signer de nouveaux contrats dans les domaines bagasse/biomasse thermique et solaire", a déclaré dans un communiqué le PDG d'Albioma, Jacques Pétry.   La bagasse est le résidu ligneux produit lors du broyage de la canne à sucre.   "Nous sommes heureux de voir notre programme d'investissements de croissance d'un milliard d'euros de 2013 à 2023 sécurisé à plus de 50%. Il devrait nous permettre de doubler notre patrimoine industriel sur la période tout en maintenant un niveau élevé de rentabilité", a-t-il ajouté.
</article>
<article titre="Le fonds Apax sort du capital d'Albioma" date="2015 06 5" journal="Les Echos (site web)">
L'opération présage-t-elle d'un nouveau tour de table pour Albioma ? Vendredi, Apax a annoncé le transfert à ses investisseurs, de ses 30 % de participation dans l'ex-Séchilienne Sidec dont il avait pris le contrôle en 2005 auprès d'Air Liquide pour 160 millions d'euros. L'opération a pour conséquence d'augmenter d'autant le flottant du producteur d'énergie coté en bourse. Ces investisseurs sont à 44 % nord-américains et à 56 % européens. En tant que fonds de fonds ou fonds de pension, ils n'ont pas forcément vocation à détenir ces titres en direct. Ils pourraient donc les revendre.   « Nous ne connaissons pas le profil, ni les intentions des nouveaux investisseurs avec un risque de retour de papier »  , commente une analyse d'Oddo. Néanmoins relève la note,   « l'investisseur le plus important au sein de la partie reclassée représente moins de 2 % du capital du groupe »  . L'impact sur le titre pourrait donc être limité. Altamir, un véhicule indirectement lié à la galaxie Apax, va conserver 12 % d'Albioma. Ces investisseurs n'ont pas eu leur mot à dire dans la reprise de ces titres. C'est le règlement du fonds dans lesquels ils étaient investis, qui les y contraint. En cédant ses 30 %, Apax peut enfin clore cet ancien véhicule lancé en 2000. La société d'investissement dirigée par Eddie Misrahi portait cette participation depuis 10 ans et en toute logique aurait dû céder le contrôle d'Albioma plus tôt compte tenu des cycles d'investissement usuels de 5 ans dans le non-coté. Selon des sources, Apax avait tenté l'an dernier de trouver des investisseurs sans que le processus aboutisse. De même en 2009/2010, mais le projet avait été mis à mal par la crise financière.   Augmentation de capital ? De fait aussi, jusqu'à l'arrivée de Jacques Pétry en 2012, l'ex-Séchilienne-Sidec, dont l'actionnaire de référence Hélios est passé par une procédure de sauvegarde, n'était pas au meilleur de sa forme. Depuis, le redressement opéré pourrait relancer l'intérêt de nouveaux investisseurs. Le bénéfice net du groupe aux objectifs clarifiés est passé de 25,1 millions d'euros à 38 millions d'euros, le cours de bourse a grimpé de 70 % et Jacques Pétry s'est lancé dans un plan ambitieux. Essentiellement présent dans la transformation d'énergie à partir de la canne à sucre (bagasse) et valorisé à hauteur de 510 millions d'euros en Bourse, Albioma a lancé un plan d'investissement de 1 milliard d'euros sur dix ans, dont la moitié est déjà assuré. Le groupe a opéré deux acquisitions au Brésil. En Outre-Merune nouvelle usine de co-génération qui va nécessiter 170 millions d'euros d'investissements est en cours de lancement.   « Nous voulons doubler le patrimoine industriel d'Albioma et assurer une croissance annuelle de 7,5 % du résultat »  , indique Jacques Pétry. L'offre d'énergie à partir de biomasse dispose d'un vrai potentiel de croissance « Actuellement seuls quelques pourcents des 500 millions de tonnes de bagasse produits par an sont valorisés » , poursuit le dirigeant. Ce dernier n'exclut pas de lancer une augmentation de capital pour assurer au besoin le financement de ses nouveaux projets compte tenu du rythme de développement du marché.    Pour l'heure cependant, Albioma a dû faire un avertissement sur ses résultats suite à des incidents techniques sur des centrales. Pour 2015, le bénéfice de l'entreprise est attendu en baisse de 10,7 %. Suite à l'annonce du transfert de capital, le titre a chuté de 6,36 % à 17,23 euros.
</article>
<article titre="Albioma: Apax Partners distribue ses actions." date="2015 06 5" journal="Cercle Finance (site web)">
Apax Partners a informé Albioma de son intention de distribuer à ses investisseurs plus de 95 % de sa participation directe et indirecte dans Albioma, soit environ 30 % du capital. Cette opération aura pour conséquence d'augmenter significativement le flottant de la Société. Cette distribution devrait intervenir dans la deuxième quinzaine du mois de juin 2015.   A l'issue de cette distribution, Altamir détiendra une participation directe et indirecte de près de 12 % du capital. Réuni le 2 juin 2015, le Conseil d'Administration a pris note de l'intention d'Apax Partners.   ' Depuis trois ans, nos équipes ont travaillé avec succès pour signer de nouveaux contrats dans les domaines bagasse/biomasse thermique et solaire. Nous sommes heureux de voir notre programme d'investissements de croissance de 1 milliard d'euros de 2013 à 2023 sécurisé à plus de 50 %. Il devrait nous permettre de doubler notre patrimoine industriel sur la période tout en maintenant un niveau élevé de rentabilité. ' a déclaré Jacques Pétry, Président-Directeur Général.   Copyright (c) 2015 CercleFinance.com. Tous droits réservés.
</article>
<article titre="Futurol, un bioéthanol bon marché et réellement écolo" date="2015 06 2" journal="Challenges (site web)">
Imaginez un carburant tiré des plantes, parfaitement neutre sur le plan du carbone. Cet alcool serait produit par des distilleries disséminées dans nos campagnes et ferait appel non pas au fruit de la plante, mais à sa tige et à ses feuilles. Histoire de ne pas empiéter sur les besoins de la filière alimentaire mais plutôt de la compléter. Telles sont les promesses du bioéthanol cellulosique, autrement appelé bioéthanol de seconde génération. Ils sont nombreux partout dans le monde à travailler depuis une dizaine d'années à la mise au point d'un procédé de fabrication compétitif, tant sur le plan économique que sur celui du rendement énergétique. Or, demain, mercredi 3 juin 2015, ce sont les onze partenaires français du projet Futurol (industriels, financiers et chercheurs) qui dévoileront le fruit de sept années de recherche. Ils entameront la phase d'industrialisation de leur procédé de fabrication de bioéthanol cellulosique vanté comme le plus rapide et le plus économique au monde. Une petite révolution.   Le bioéthanol Futurol en huit pointsQuel intérêt pour moi, automobiliste et citoyen ?Qui consomme le bioéthanol français ?Le bioéthanol cellulosique ne concurrence pas l'alimentationUn bioéthanol moins cher que l'essence1.000 à 2.000 euros investis par tonne de bioéthanol produiteAutonomie énergétique : Futurol produit l'énergie qu'elle consommeBioéthanol cellulosique, un carburant non subventionnéDes sucres pour le bioéthanol comme pour l'industrie de la chimie   Le bioéthanol, quel intérêt pour moi ?   C'était il y a une petite dizaine d'années. Le bioéthanol revenait sur le devant de la scène, après un premier faux départ au début des années 1990. A cette époque-là, les agriculteurs français s'étaient unis pour faire la promotion de cet alcool carburant qui faisait - et fait encore - les beaux jours de leurs homologues au Brésil. Là-bas, la canne à sucre procure des sucres qui, une fois fermentés, donnent un alcool que brûlent des moteurs à essence à peine modifiés. L'intérêt est double : le Brésil réduit sa dépendance énergétique et diminue son emprunte carbone.   Car à la différence du CO2 fossile émis par la combustion d'hydrocarbures non renouvelables (supercarburant, gazole, gaz naturel, gaz de pétrole liquéfié GPL), le CO2 issu de la combustion du bioéthanol est réputé renouvelable. En ce sens que ses molécules de carbone flottaient dans l'atmosphère avant que les feuilles des plantes ne s'en saisissent durant le processus de photosynthèse. Un prêté pour un rendu, en somme.   Point de canne à sucre sur le territoire de la Métropole bien sûr, mais de la betterave en quantité, du maïs et du blé fourrager. Le pétrole vert de la France ! Selon la Collective du Bioéthanol, près de 3 % des terres cultivées en betteraves et en céréales en France sont destinées à cet alcool carburant. Un hectare de betteraves permet de produire le bioéthanol nécessaire pour rouler durant près de 100.000 km. Mais c'est autant de sucre qui ne va pas à l'industrie agroalimentaire ou chimique.   Retour au sommaire.   Qui consomme le bioéthanol français ?   Pour l'heure, la production française suffit aux besoins de nos raffineries de pétrole qui sont tenues légalement d'ajouter du bioéthanol dans les carburants. A hauteur de 5 % maximum dans les supercarburants SP95 et SP98, et de 10 % maximum dans le SP95-E10. La France exporte même une part de sa production de bioéthanol.   La quasi-totalité des voitures en circulation de moins de quinze ans d'âge peuvent carburer au SP95-E10 (liste complète des modèles compatibles sur bioethanolcarburant.com). Voilà qui explique que ce carburant vendu 3,5 centimes de moins au litre en moyenne que le SP 95 représentait 32 % des essences vendues en 2014, soit 10 % de plus qu'en 2013.   Pour ce qui est du supercarburant E85 (entre 65 % et 85 % de bioéthanol incorporé) en revanche, la consommation était en baisse de 9 % en 2014. Pour la bonne raison qu'il faut une adaptation du moteur plus poussée et plus coûteuse, de l'ordre de 600 euros, départ usine. En 2006, le Gouvernement soucieux de tenir auprès de Bruxelles ses engagements de réduction de ses émissions de CO2 offrait son soutien à la filière bioéthanol en annonçant que la flotte de l'État compterait sous peu 15 % de véhicules dits flexfuel, adaptés au E85. Mieux, la France se donnait comme objectif un taux d'incorporation de 10 % de bioéthanol à l'horizon 2010.   Pompes à carburant : bioéthanol E85, SP95-E10 Image © PIERRE ANDRIEU / AFP   Malheureusement, ce seuil n'a jamais été atteint. Pétroliers et constructeurs se rejettent la faute, les uns disant qu'il ne sert à rien d'installer des pompes E85 s'il n'y a pas d'auto flexfuel pour le consommer. Et inversement.   Ces dernières années toutefois, il semble que la balle soit passée dans le camp des constructeurs. Ils ne proposent plus de voitures polycarburants sur le marché français, alors que le nombre de stations délivrant du bioéthanol E85 va croissant. En mai 2015, il franchissait le seuil des 600 pompes, contre 560 à fin 2014. Seuls nos voisins nordiques font mieux en proportion.   Retour au sommaire.   Le bioéthanol cellulosique ne concurrence pas l'alimentation   La Collective du Bioéthanol estime que rouler au supercarburant E85 plutôt qu'au SP95 conventionnel autorise une réduction de 40 % des émissions nettes de gaz à effet de serre. Pour un kilométrage annuel moyen de 8.600 km (soit 700 litres de super), cette économie représenterait le retrait de la circulation de plus d'un million de moteurs à essence. C'est 2 millions de tonnes de CO2 fossile en moins dans l'atmosphère.   A ce tableau idyllique, les détracteurs du bioéthanol opposent un scénario d'horreur. Ils décrivent un monde où les besoins croissants en mobilité des pays en voie de développement conduiraient à une déforestation sauvage à mesure que les cultures vivrières seraient remplacées par celles de l'agrocarburant. Rouler plutôt que manger, quelle folie !   Là est l'atout numéro un du bioéthanol cellulosique, que partage avec ses rivales la technologie Futurol : les sucres sont extraits de la tige et des feuilles, non pas des fruits ou des graines. Il n'y a plus de concurrence avec la filière alimentaire. Mais là où la technologie Futurol montre son avantage, c'est dans son extraordinaire capacité à trouver de nouveaux gisements de sucres dans la biomasse d'origines les plus diverses.   "Le procédé fonctionne avec toutes sortes de plantes et s'adapte aux types de cultures, variables d'une région à l'autre et d'une saison à l'autre", explique Anne Wagner, Présidente du Projet Futurol. "Le procédé fonctionne avec tous les déchets issus des exploitations agricoles ou forestières : de la paille, de la sciure de bois, des déchets végétaux du jardin, de la bagasse, du fourrage, etc." Cette grande flexibilité permet de garantir l'approvisionnement en matière première - la cellulose et la lignine - tout au long de l'année et dans toutes les régions.    La technologie Futurol doit cette flexibilité aux caractéristiques inégalées des enzymes et des levures mis au point et protégés par près de vingt-six brevets. Les enzymes sont chargés d'extraire les sucres de la cellulose, et les levures de transformer en alcool ces sucres (fermentation). "Là se situe le point de rupture de la technologie Futurol", renchérit Frédéric Martel, Directeur général du Projet Futurol. "On trouve 40 % environ de glucose dans la biomasse, contre 25 % de xylose (le sucre du bois). De ces deux types de sucres, le premier est le plus facile à extraire. Pour le second, notre partenaire Lesaffre, expert des levures et des enzymes, a mis au point une souche capable de digérer le xylose. De quoi augmenter le rendement en sucres de la biomasse."   Les Marines américains harcelés par un champignonLa découverte du champignon capable de digérer le sucre du bois, le xylose remonte aux années 1940, lorsque l'intendance des Marines déployés dans le Pacifique Sud avait constaté que la toile des tentes se déchirait plus vite qu'ailleurs. Les scientifiques identifièrent un champignon - Trichoderma reesei - qui prospérait sous la chaleur et l'humidité locales, et se régalaient du fil de coton. C'est cette souche qui a été sélectionnée comme base de travail par les chercheurs du Projet Futurol.   Retour au sommaire.    Un bioéthanol moins cher que l'essence   Ces levures brevetées Futurol agiraient plus rapidement que n'importe quelles autres employées par les procédés concurrents. "Voilà un autre atout inégalé de la technologie Futurol : ce gain de temps se traduit mécaniquement par un gain de rendement massique et de productivité. Ce qui fait qu'à taille identique, une éthanolerie Futurol sera moins chère qu'une autre produisant du bioéthanol cellulosique selon un autre procédé", renchérit A. Wagner.   Selon la Présidente du Projet Futurol, le prix de revient du bioéthanol varierait selon la taille de l'installation et le type de biomasse entre 0,50 euro et 0,70 euro le litre. A cela s'ajouteraient les coûts de distribution classique des carburants (5 à 6 % en moyenne) et les 45 % de taxes que prélève l'État français. Il y a là de quoi compenser la surconsommation induite par l'alcool (de l'ordre de 20 % avec le E85) et rivaliser avec les essences et le gazole dont les prix subissent de surcroît environ 7 % de surcoût dû au raffinage. "Notre objectif est de produire un bioéthanol à un coût compétitif avec l'essence conventionnelle, sans aucune subvention publique", martèle A. Wagner.   Retour au sommaire.   1.000 à 2.000 euros investis par tonne de bioéthanol produite   La grande souplesse de la technologie Futurol permet d'envisager des unités de production de taille très variable. "L'investissement va de 30 millions d'euros environ pour une unité venant se greffer sur une installation existante à une centaine de millions d'euros pour la création d'une grosse unité de production", estime F. Martel. "Ce qui nous donne environ 1.000 à 2.000 euros investis par tonne de bioéthanol produit."   L'usine Futurol produit l'énergie qu'elle consomme   Sur une tonne de bois traitée selon le procédé Futurol, on obtiendrait 45 % de bioéthanol et 45 % d'énergie ligneuse, c'est-à-dire issue de la lignine, l'autre principal composant du bois avec la cellulose. Très énergétique, cette lignine alimente les chaudières qui entraînent les turbines qui produisent du courant électrique. Une énergie produite sur place et consommée sur place par l'usine. "Selon la nature de la biomasse et le taux de lignine, il arrive même que l'usine produise plus d'énergie qu'elle en consomme", certifie F. Martel. Ce courant est réinjecté dans le réseau général.   Les 10 % restant correspondent à des résidus riches en nutriments. "Ces résidus peuvent être employés sous forme d'épandage pour enrichir ses sols ou bien dans un processus de méthanisation, pour la production de gaz", précise F. Martel.    Une éthanolerie faisant appel à la technologie Futurol s'avère par ailleurs capable, une fois le processus amorcé, de produire ses propres enzymes et ses propres levures. Par conséquent, l'exploitant ne dépend d'aucun fournisseur. Attrait supplémentaire, la production de bioéthanol permet d'augmenter le rendement de l'exploitation en faisant pousser des herbes pour Futurol sur des surfaces jusque-là considérées comme perdues.   Retour au sommaire.   Bioéthanol cellulosique, un carburant non subventionné   Certes, le Projet Futurol a bénéficié du soutien de l'État et d'un financement à hauteur de 30 millions d'euros (sur un coût total de 76 millions d'euros sur huit ans). La première unité de production à l'échelle industrielle qui sera établie à Bucy, dans le Nord de la France profitera encore partiellement de cette manne. Au-delà, l'engagement de l'État cesse. "Cette usine de l'ordre de 70.000 tonnes traitées représente un investissement de 12 millions d'euros qui devra être amorti par la production de bioéthanol", certifie A. Wagner. "Elle nous servira de vitrine pour convaincre les clients potentiels. A partir de 2016, la société Axens, filiale à 100 % d'IFPEN pourra rentrer dans la phase de commercialisation de la technologie Futurol."   Retour au sommaire.   Des sucres pour le bioéthanol comme pour l'industrie de la chimie   Les sirops de sucres issus de la première phase du procédé Futurol ne sont pas tous destinés à être fermentés. Ils peuvent être employés par l'industrie pour la production par exemple de solvants. Les débouchés sont innombrables car Futurol est capable de fournir des cocktails d'enzymes adaptés à tous les besoins. "A ce titre, la technologie Futurol va dans le sens des recommandations formulés par Emmanuel Macron qui voudrait voir doublée la quantité de chimie biosourcée en France", se félicite A. Wagner. Un moyen de plus de garantir la compétitivité financière de cette filière 100 % française.   Retour au sommaire.
</article>
<article titre="Emballages rigides Les barquettes s'activent sur tous les fronts" date="2015 06 1" journal="Process Alimentaire, no. 1326">
1 Pour une meilleure conservation Dans les plats cuisinés, deux technologies récentes proposent un allongement des DLC de produits frais accompagné d'une préservation des qualités nutritionnelles des ingrédients mis en oeuvre. La technologie Micvac permet ainsi de cuire et de pasteuriser en même temps les aliments dans la barquette grâce à un process breveté de cuisson dans un tunnel micro-ondes. Les barquettes disposent d'une valve sur le film supérieur, qui se ferme après cuisson, et se met à siffler lorsque le plat est ensuite uniformément réchauffé par le consommateur au micro-ondes. Avantages de cette solution proposée en barquettes de différentes tailles et couleurs : le plat peut être tenu à la verticale en rayon, les aliments se conservent au frais pendant 30 à 60 jours. Plusieurs lignes ont déjà été installées dans les pays scandinaves et au Japon, dont dernièrement chez Konan Foods, un fabricant de plats cuisinés. Dans le même esprit, le leader norvégien des plats préparés a opté en 2014 pour le concept Convitray. Grâce à une fermeture étanche, les aliments sont directement pasteurisés et cuits à la vapeur dans la barquette. Ce traitement élève à quatre semaines la durée de conservation d'un plat frais. Cette barquette en aluminium et PET, operculée par un film pelable haute barrière antibuée, est aussi réchauffable au four à 240 °C. Les emballages skin sous-vide séduisent aussi de plus en plus de marchés : la viande, le fromage, la charcuterie, les produits de la mer. En témoigne la chaîne de magasins Grand Frais qui conditionne depuis 2014 tous ses pavés et filets de poissons en barquettes skin sous-vide. A l'image de certaines références Delpierre. «Les consommateurs étaient au départ réticents à adopter la solution «skin» sous-vide, à cause d'une esthétique qu'ils jugeaient inférieure aux emballages traditionnels sous atmosphère modifiée, remarque Alan Davey, directeur de l'Innovation chez Linpac. Le style Form-fill-seal de l'emballage et son dos plat et instable ne favorisaient pas la présentation en rayon.» D'où l'idée de faire du skin sous-vide en operculage sur des barquettes préformées. Avec son concept Rfresh Evolve, Linpac propose ainsi des barquettes en rPET rigides contenant jusqu'à 95 % de PET recyclé, nettoyé et purifié. «Ce n'est que le début de l'histoire. Nous envisageons déjà de proposer une gamme de formes, couleurs et designs attractifs complètement nouveaux», affirme-t-il.   Pour les mêmes raisons d'esthétisme et de conservation, Sealpac propose sa technologie Trayskin X Plus sur l'ensemble de sa gamme d'operculeuses pour petites, moyennes, ou grandes capacités. «Avec l'effet double peau du Trayskin X, les reliefs et couleurs des produits sont mis en avant.» Cette technologie peut être couplée à diverses solutions techniques telles que l'EasyPeelpoint (ouverture facile), le Tenderpac (captation des exsudats), ou l'operculage supérieur à plat permettant l'étiquetage ou l'ajout d'une recette ou dosette d'épices. Sur ce principe, Sealed Air décline aussi son Darfresh on Tray sur de nombreux marchés, avec possibilité d'operculer des produits de plus en plus débordants (lire décembre 2014 p. 73).   2 Pour de nouveaux moments de consommation La montée en puissance du marché des salades-repas a vu naître récemment un grand nombre d'innovations emballages. Packalim, par exemple, présente une barquette aux compartiments 100 % personnalisables (dimensions, formes, etc.). Ils peuvent être placés à l'horizontale ou à la verticale, ce qui va permettre de proposer des volumes adaptés à chaque produit. Virgin Biopack lance, pour sa part, cinq références standard pour les usages snacking et tapas. Chaque barquette Hybric Tray (en carton recouverte d'un film intérieur en plastique) présente un compartimentage adapté à l'utilisation finale : duo de salades, compartiment service, sauce, etc.   Plastobreiz vient juste de lancer son concept Visibility (lire mai 2015 p. 81.). Et revisite également sa gamme Easy2Snack pour s'inscrire dans un contexte demandeur de solutions repas complètes et modulables. L'ensemble comprend un insert thermoformé avec séparateur qui se positionne dans la barquette operculable réalisée en polyéthylène téréphtalate amorphe (APET). Le tout peut recevoir un couvercle ou être utilisé avec un habillage carton. L'ensemble est proposé avec les couverts injectés réalisés par Europlastiques et commercialisés avec ou sans packs serviette. Plus Pack décline, de son côté, son concept Safe Snack sur sa gamme d'emballages nomades. Spécificité : la mini-fourchette-cuiller est placée au fond de la barquette dans un compartiment individuel propre et hygiénique à l'abri de la nourriture. Elle est protégée par une étiquette transparente. Cette évolution réduit la manipulation d'emballages par le conditionneur et accroît la visibilité du produit.   La diversification des concepts oblige les fabricants d'operculeuses à gagner en flexibilité. Comme l'illustre la gamme de Multivac, par exemple. «Depuis le modèle semi-automatique T200 jusqu'aux modèles en ligne des familles T700 et T800, les plages de température et de pression permettent de traiter sans restriction tous les matériaux plastique communément utilisés», indique-t-on chez Multivac. Les matériaux tels que le carton, l'aluminium ou le verre peuvent aussi être traités sur une même machine. À condition que la planéité des zones de soudure soit assurée pour garantir l'étanchéité en cas de conditionnement sous atmosphère modifiée. Preuve que techniquement, l'innovation en matière de barquettes a peu de limites.   Parfois, elles nécessitent toutefois des operculages spécifiques. Comme en témoigne le dernier concept de barquettes refermables sans couvercle Clipseal 2 Clips de CGL Pack (lire avril 2015 p, 93) développé avec Mondini mais adaptable à d'autres machines.
</article>
<article titre="Opinions Electricité dans les îles: une exemple de dérive de la péréquation" date="2015 05 28" journal="La Tribune (France), no. 5716">
La visite du président Hollande aux Antilles est l'occasion d'évoquer l'une des composantes de la Contribution au Service Public de l'Electricité, à savoir la compensation des surcoûts de production dans les zones non interconnectées au réseau électrique métropolitain continental (Corse, départements d'outre-mer, Saint-Martin, Saint-Barthélemy, Saint-Pierre et Miquelon, îles bretonnes de Molène, d'Ouessant, de Sein, archipel des Glénan et îles Chausey). Particularités insulaires Si l'on excepte les lignes reliant la Corse à l'Italie, les systèmes électriques insulaires français ne peuvent compter que sur leurs installations locales, essentiellement alimentées par des importations de combustibles fossiles. Par exemple, en Guadeloupe la production électrique était issue en 2012 pour 53% du pétrole, 32% du charbon et 15% d'énergies renouvelables. En 2009, le charbon n'occupait que 13% du total. L'augmentation de la consommation de charbon, mauvais choix en termes d'émissions de gaz à effet de serre, s'explique à la fois par la recherche de débouchés pour le charbon nord-américain chassé par les gaz de schiste moins onéreux, et par la mise en service en 2010 d'une deuxième centrale thermique au charbon. Les systèmes insulaires ont aussi des ressources renouvelables importantes : pour la Guadeloupe, les 15% de production d'électricité d'origine renouvelable en 2012 venaient surtout du photovoltaïque (5,3%), de la bagasse (3,6%), de la géothermie et de l'éolien (3% pour chacun).Sans accès à l'électricité produite par le parc nucléaire et les grandes unités hydroélectriques, voire à des centrales thermiques de grande taille, le coût de l'énergie électrique est beaucoup plus élevé dans les zones non interconnectées (ZNI) qu'en France continentale.    Prix nodaux Les principes économiques appliqués au secteur électrique (ou à n'importe quel réseau) sont clairs : il n'y a pas de différence de nature entre ces zones non interconnectées (ZNI) et, par exemple, la Bretagne ou PACA. Il s'agit de zones mal connectées à l'ensemble du parc de production national, la mauvaise connexion des ZNI n'étant qu'un cas extrême puisqu'il n'y a pas de connexion du tout. La solution optimale consiste à y appliquer le principe des prix nodaux : en chaque noeud la valeur de l'électricité résulte de l'équilibre entre offre (ordre de préséance des injections) et demande (ordre de préséance des prélèvements), les lignes d'interconnexion apportant, selon les cas, un flux supplémentaire d'offre (importations) ou de demande (exportations). En l'absence d'apport extérieur, les prix insulaires ne devraient donc refléter que l'offre et la demande insulaires. Dans la plupart des ZNI la demande d'électricité est plus faible que la demande moyenne en France continentale. D'abord parce qu'il n'y a pas de chauffage électrique, ensuite parce qu'il y a très peu d'industrie, grosse consommatrice d'énergie. Mais le facteur demande ne joue quasiment aucun rôle dans la détermination de la valeur de l'électricité en ZNI. En effet, la technologie marginale, celle dont le coût d'exploitation donne la valeur du produit, est systématiquement la même : le thermique à flamme (fioul ou charbon). Le prix de l'électricité en ZNI devrait donc être le coût marginal du thermique à flamme, bien plus élevé que le prix en France métropolitaine.   Contrainte de péréquation C'est là qu'intervient la politique. Sous l'impulsion de groupes d'intérêts locaux et d'idéologues de la chose publique s'est progressivement propagée l'idée selon laquelle les services publics doivent être facturés à un prix qui soit indépendant des contraintes géographiques : c'est le principe de péréquation géographique tarifaire. Ce principe de solidarité nationale semble être un brin de l'ADN de la République française. Comme l'écrit F.-M. Poupeau, « la péréquation géographique tarifaire fait partie de ces institutions que l'ordre politique pose comme des objets 'naturels', voire immuables ». En réalité, ajoute-t-il, «des spécificités de nature géographique ou technique peuvent justifier un traitement tarifaire modulé entre usagers, faisant de la péréquation une exception plutôt qu'une règle absolue dans le mode de gestion des grands réseaux de service public ».   Aussi bizarre que cela puisse paraitre, il aura fallu attendre la loi du 10 février 2000 transposant la première directive européenne sur la libéralisation du secteur électrique pour voir apparaître pour la première fois la péréquation géographique tarifaire dans le corpus législatif français sur l'énergie. Et, puisque les ZNI font partie du territoire de la République, la loi impose que l'électricité y soit vendue au même tarif que dans sa partie continentale.   Un prix inférieur aux coûts de production L'application de la contrainte de péréquation donne un résultat simple : puisque le tarif de vente de l'électricité est une moyenne des coûts de production nationaux et puisque les coûts en ZNI sont plus élevés que les coûts en France continentale, le tarif est inférieur au coût en ZNI et supérieur à celui de la France continentale. Pour couvrir l'écart entre les recettes et les coûts des producteurs des ZNI provoqué par cette obligation de service public, il faut leur verser une compensation. La compensation se fait par la Contribution au Service Public de l'Electricité (CSPE). La CSPE regroupe les charges imposées aux fournisseurs historiques (EDF et entreprises locales de distribution) que ceux-ci sont autorisés à répercuter dans la facture des consommateurs. Jusqu'en 2010, c'est la péréquation géographique tarifaire qui générait le plus de charges au sein de la CSPE. A partir de 2011, le poste 'énergies renouvelables' a pris la première place (60% en 2013). Pour l'année 2015, la couverture de la CSPE prévue par la Commission de Régulation de l'Energie nécessite la levée de 6,3 milliards d'euros, dont 1,5 milliard au titre de la péréquation tarifaire dans les ZNI.   Solidarité et redistribution La nécessité de redistribuer les richesses au sein de la collectivité nationale est indiscutable. C'est la méthode qui pose problème. Les prix servent d'indicateur de rareté aux consommateurs et d'indicateur d'utilité aux producteurs. Un prix inférieur au coût de production est donc une incitation à surinvestir en équipements de consommation (notamment d'air conditionné) et à surconsommer. Symétriquement, les producteurs sont incités à sous-investir et à sous produire, ce qu'il faut corriger par des interventions réglementaires répétées. A l'inconvénient de distordre les signaux de prix, la péréquation ajoute de l'injustice. En effet, même si les ZNI sont plus pauvres en moyenne que la France métropolitaine, il y a aussi des riches dans les premières et des pauvres dans la seconde. Une partie du transfert réalisé est donc régressive puisque la CSPE est payée par tous.C'est par des transferts de revenu ciblés que devrait se matérialiser la solidarité nationale, et non par des manipulations de prix. Il est vrai que cette affirmation doit être corrigée dans le cas de biens tutélaires pour lesquels le prix doit refléter non seulement le coût de production mais aussi les externalités positives générées. Mais dans le cas présent, pourquoi cette externalité serait-elle différente selon les zones géographiques?La manipulation des tarifs publics est simplement une manifestation de la faiblesse des gouvernants qui n'osent pas utiliser leurs pouvoirs régaliens en matière fiscale et préfèrent se défausser sur les entreprises en disséminant des mesures de transfert sous le couvert d'obligations de service public, probablement plus indolores mais assurément coûteuses pour la collectivité.Il ne faut pas espérer que la compensation des ZNI cessera avec la disparition des tarifs. En effet, les ZNI sont considérées par les pouvoirs publics français comme inaptes à voir leur secteur électrique s'ouvrir à la concurrence. Ces systèmes, au titre de « petits réseaux isolés », bénéficient d'un régime dérogatoire autorisé par les directives européennes de 2003 et 2009. Il y aura donc encore pour longtemps des tarifs pour l'électricité dans les ZNI, et pendant longtemps il faudra compenser leur insuffisance à couvrir les coûts de production.   Plus d'informations sur  le blog de TSE (lien : http://debate.tse-fr.eu/?language=fr)
</article>
<article titre="Albioma (ex-Sechilienne-Sidec) : nouveaux déboires" date="2015 05 28" journal="Boursier (site web)">
Résultats.  Encore des soucis chez Albioma ce dont les actionnaires se seraient bien passés après les déboires des années passées. Un nouvel incident technique affecte une centrale thermique située en Guadeloupe tandis qu'une autre unité située à La Réunion fait l'objet de travaux de remise à niveau. L'indisponibilité d'une partie du parc conduit la société à réviser ses ambitions. Elle revoit en baisse d'une dizaine de pourcents son objectif annuel en matière d'Ebitda et de résultat net... Ce dernier indicateur devrait ainsi se situer entre 30 et 34 Millions d'Euros. Le reflux sera donc assez sensible sur l'année dans la mesure où le résultat net était de 38 Millions d'Euros en 2014. On rappellera qu'un conflit social a eu lieu courant premier trimestre 2015 dans les unités Albioma situées dans les Caraïbes avec un coût estimé à 3 Millions d'Euros pour l'entreprise. Brésil.  Les résultats de la société sont régulièrement entachés de déboires de la sorte. A caractère non récurent, ils ont tendance à le devenir tant ils se répètent. Ce phénomène est dommage car Albioma offre des caractéristiques attractives. Le parc de centrales thermiques augmente au fil du temps avec les unités à l'Ile Maurice et les projets brésiliens. La société a initié également un développement dans la bio méthanisation. Cela consiste à valoriser les déchets agricoles avec un marché potentiel qui semble très large. Malheureusement, le cadre tarifaire n'est pas clair et les défis techniques ne sont pas complètement résolus. Le gros chantier du moment consiste plutôt à réussir la montée en puissance prévue au Brésil. La bagasse fournie par la canne à sucre est utilisée pour produire de l'électricité. Investissements.  Maintenant que la première unité brésilienne fonctionne bien, Albioma entend investir 400 Millions d'Euros dans le pays sur les dix prochaines années. Tout cela doit conduire à doubler la taille de l'entreprise d'ici 2023. Les actionnaires ne sont pas oubliés alors que la moitié du résultat net doit leur revenir sous forme de dividende. On aimerait que la visibilité offerte par les contrats signés avec les électriciens sur le long terme ne soit pas gommée par des perturbations diverses et variées...Nous estimons néanmoins que l'on peut maintenir la ligne en portefeuille même si le début de reprise des cours observé récemment est brisé net par le profit warning du jour.... Au-delà de la mauvaise nouvelle du jour, on pourra conserver la ligne Albioma.Christophe Voisin
</article>
<article titre="ALBIOMA révise à la baisse ses objectifs 2015" date="2015 05 28" journal="Option Finance (site web)">
AOF - 28 mai 2015   (AOF) - Albioma a révisé à la baisse ses objectifs financiers 2015 après un incident d'exploitation à la Réunion et un autre en Guadeloupe. Le groupe a réduit d'environ 10% sa prévision d'EBITDA comprise entre 126 à 130 millions d'euros et de résultat net part du Groupe compris entre 34 à 37 millions d'euros. Albioma a par ailleurs signé avec EDF un avenant au contrat d'achat d'électricité en vue de la mise en conformité de la tranche B de la centrale Albioma Le Gol de La Réunion avec les futures normes environnementales issues de la directive européenne IED (Industrial Emissions Directive). Albioma a annoncé en avril avoir signé la documentation définitive en vue de l'acquisition au Brésil de 65% du capital de Codora Energia, une société propriétaire d'une unité de cogénération bagasse de 48 MW située dans l'État de Goiás. La finalisation de l'acquisition est soumise à des conditions suspensives devant être levées au second semestre de l'exercice 2015.   Dans le sillage de cette opération, Codora Energia a sécurisé au Brésil la vente à long terme de 54 GWh par an à un prix historiquement élevé.  Cet article est paru dans Option Finance (site web)
</article>
<article titre="Portzamparc maintient sa confiance malgré un 1er semestre difficile" date="2015 05 28" journal="Trading Sat (site web)">
Portzamparc a confirmé jeudi sa recommandation "Acheter" et son objectif de cours 21,7 euros sur Albioma, dont l'action chute de 5% après un avertissement du producteur d'énergie indépendant (-5,05% à 18,25 euros à 15h20).    Le groupe a dû revoir à la baisse ses objectifs 2015 en raison de deux incidents d'exploitation. Le premier a touché, fin avril, la tranche 1 de la centrale thermique Albioma Le Moule (ALM) en Guadeloupe. D'autre part, le 25 mai 2015, la chaudière de la tranche B de la centrale thermique Albioma Le Gol à La Réunion a également connu un incident.  En conséquence, Albioma estime que ses objectifs 2015 d'EBITDA de 126 à 130 millions d'euros et de résultat net part du groupe de 34 à 37 millions d'euros doivent être réduits d'environ 10 %.   Portzamparc évoque de fait un "premier semestre difficile pour Albioma qui multiplie des éléments ponctuels négatifs (mouvements sociaux, incidents techniques...).   Pour autant, le broker souligne "les avancés significatives réalisées au Brésil".   Albioma a en effet annoncé en avril avoir signé la documentation définitive en vue de l'acquisition au Brésil de 65 % du capital de Codora Energia, une société propriétaire d'une unité de cogénération bagasse de 48 MW située dans l'état de Goiás.   Ainsi, bien que "peu de catalyseurs [soient] à attendre à court terme", "le profil du groupe et la montée en puissance sur les différents projets" conduisent Portzamparc a maintenir sa recommandation positive.
</article>
<article titre="Préouverture Paris : accord ou pas ?" date="2015 05 28" journal="Boursier (site web)">
LA TENDANCE Les marchés financiers continuent de vivre au gré des nouvelles en provenance de Grèce... Même Wall Street a emboîté le pas mercredi soir à la tendance européenne, où l'espoir d'un accord sur la Grèce a soutenu les marchés actions. Le Nasdaq a même inscrit un nouveau record en clôture, sur fond de fusions-acquisitions dans les semi-conducteurs, tandis que le dollar a stoppé son ascension face à l'euro...   WALL STREET Mercredi à la clôture, l'indice le Dow Jones a gagné 0,67% à 18.163 points, tandis que l'indice large S&amp;P 500 a pris 0,92% à 2.123 points. Le Nasdaq, riche en valeurs technologiques, a bondi de 1,47% à 5.106 points, un nouveau record historique. En l'absence de statistiques américaines de premier plan, les marchés ont voulu croire en un accord sur la dette grecque, alors que dans l'après-midi, des responsables grecs ont annoncé la rédaction d'une ébauche d'accord avec les créanciers de la Grèce. Mais rapidement, des hauts fonctionnaires européens ont démenti ces propos, estimant que les négociations étaient encore loin d'avoir abouti... Athènes a dénoncé de son côté des divergences entre ses créanciers, qui retarderaient la conclusion de l'accord sur le programme de réformes de la Grèce.   Ce dernier, s'il était enfin trouvé, ouvrirait la voie au déblocage du solde du programme d'aide à la Grèce (7,2 MdsE), ce qui éloignerait le risque de défaut et de sortie de la zone euro.   ECO ET DEVISES Sur le marché des changes, l'euro est remonté au-dessus de 1,09$ à 1,0910$ (+0,3% depuis 24 heures). A l'inverse, le cours du pétrole a poursuivi son reflux, après avoir plongé de 3% mardi. Le baril de brut WTI a cédé 0,9% à 57,51$ tandis que le Brent a lâché 2,6% à 62,06$.    Enfin, la volatilité, mesurée par l'indice VIX, a reculé de 5,6% à 13,27 après avoir fait la veille son plus gros bond depuis janvier dernier.   Mardi, les indices américains avaient perdu plus de 1%, affichant leur plus fort recul depuis trois semaines, dans l'anticipation d'une hausse des taux directeurs aux Etats-Unis.    Sur le plan macro-économique, les marchés examineront ce jeudi les demandes hebdomadaires de chômage et les promesses de vente de maisons en avril, avant la publication, vendredi, de la 2ème estimation du PIB du 1er trimestre. Celui-ci avait à peine progressé en première lecture (+0,2% en rythme annuel) et pourrait avoir chuté de 0,8% en 2ème lecture, selon le consensus des économistes. Le premier trimestre a été plombé par la hausse du dollar, la baisse des prix du pétrole et une météo hivernale exceptionnellement froide et tempétueuse, mais les économistes tablent sur un rebond à partir du 2ème trimestre.   VALEURS A SUIVRE *  JCDecaux  : Le numéro un mondial de la communication extérieure annonce le dépôt, dans le cadre de son programme de rachat d`actions, d`une offre publique d`achat simplifiée (OPAS) portant sur 12.500.000 actions, représentant 5,57% du capital (au 30 avril 2015), au prix de 40 euros par action, soit un montant total de 500 millions d`euros.   Comme annoncé, l`offre est proposée aux fins d`optimiser la structure financière de JCDecaux. Elle sera majoritairement financée sur les ressources propres de la société et pour partie par tirage sur une ligne de crédit existante.   *  Orange  : Stéphane Richard (Président Directeur Général d'Orange) a décidé de nommer Fabienne Dulac (48 ans) comme Directrice exécutive en charge d'Orange France à partir du 22 août 2015. Elle rejoindra à ce titre le Comité exécutif du Groupe. "Je suis très heureux d'annoncer l'arrivée de Fabienne Dulac à la tête d'Orange France, et au sein du comité exécutif du Groupe. Fabienne, qui connaît parfaitement Orange France, apportera ses qualités humaines et managériales, son expertise, sa grande connaissance du terrain et son dynamisme. Ce choix illustre la richesse des talents dont dispose le Groupe", déclare Stéphane Richard.   Fabienne Dulac succède ainsi à Delphine Ernotte Cuni, appelée à la tête de France Télévisions.   *  Faiveley  : Au cours de l'exercice 2014/15, Faiveley Transport a réalisé un chiffre d'affaires de 1 048,4 millions d'euros, en progression de 9,5 % par rapport à l'exercice 2013/14. Le résultat opérationnel Groupe (incluant la quote-part de résultat net dans les co-entreprises) est en progression de 8,7%, à 95,3 millions d'euros (9,1% du chiffre d'affaires) au 31 mars 2015, contre 87,6 millions d'euros (9,2% du chiffre d'affaires) pour l'exercice 2013/14 retraité.   Le résultat net part du Groupe atteint 55,7 millions d'euros, en progression de 11,1% par rapport à l'exercice 2013/14. Le bénéfice net par action s'élève à 3,88 euros en 2014/15, en progression de 10,9% (3,50 euros au 31 mars 2014 retraité).   *  Pierre et Vacances  : Le chiffre d'affaires touristique du Groupe Pierre et Vacances au 1er semestre de l'exercice 2014/2015 s'élève à 476,2 millions d'euros, en progression de 2,4% par rapport au 1er semestre 2013/2014, malgré une réduction nette du parc exploité (incidence globale de -2% sur le chiffre d'affaires de location). Le résultat opérationnel courant des activités touristiques ressort à -90 millions d'euros, à comparer à -100,7 millions d'euros sur la même période de l'exercice précédent. Il bénéficie notamment de la croissance de l'activité, ainsi que de la réduction de la charge de loyers lors des renouvellements de baux (incidence de 8 millions d'euros sur le semestre, en ligne avec l'objectif annuel de 12 millions d'euros). Ces gains permettent de compenser largement l'évolution des charges liée à l'inflation (estimée à environ 5 millions d'euros).   Le résultat opérationnel des activités immobilières s'élève à 10,7 millions d'euros, en ligne avec la croissance du chiffre d'affaires. Le résultat net est de -87,2 millions d'euros (hors prise en compte de la variation de juste valeur du droit d'attribution d'action de l'ORNANE), en forte progression (+20,7%) par rapport au 1er semestre 2013/2014.   *  Laurent Perrier  : La société a publié les comptes de son exercice clos le 31 mars 2015 marqués par une croissance du chiffre d'affaires de 5,2% (3,3% à change constant) à 231,9 ME. Hors impact de change favorable de 1,8%, le chiffre d'affaires du groupe Laurent-Perrier a crû de 3,3% grâce à la combinaison d'un effet prix/mix de 2,6%, positif pour la quatrième année consécutive, et du redressement des volumes de vente (+ 0,7% à comparer à - 2,1% pour l'exercice 2013-2014).   Les volumes de la marque Laurent-Perrier ont ainsi bénéficié du retour à la croissance en France et de la poursuite de son développement international avec, notamment, des taux de progression à deux chiffres en Asie et Océanie. Au total, la part des exportations dans le chiffre d'affaires a progressé de 0,7 point pour atteindre 76,9%.   Les marques Salon, dont le nouveau millésime 2002 a été lancé cette année avec succès, et Delamotte, ont également enregistré de très belles performances.   La marge opérationnelle s'établit à 18,2%, contre 18,4% l'an dernier. Dans un contexte de taux bas et donc d'amélioration sensible du résultat financier, le résultat net s'inscrit pour sa part en hausse de 5,9% à 22,9 millions d'euros.   *  Soitec  : Le groupe publie ce jour ses résultats consolidés audités pour l'exercice 2014-2015. Comme annoncé le 20 avril dernier, le chiffre d'affaires total consolidé s'élève à 222,9 millions d'euros au titre de l'exercice 2014-2015 (contre 247,1 millions d'euros pour l'exercice 2013-2014). La société précise qu'elle s'est engagée à se recentrer sur son coeur de métier, les matériaux semi-conducteurs.   Le résultat opérationnel courant ressort en perte de 125,9 millions d'euros (contre une perte opérationnelle courante de 137,3 millions d'euros pour l'exercice 2013-2014). Pour le second semestre de l'exercice 2014-2015, le Groupe a enregistré un chiffre d'affaires consolidé de 116,9 millions d'euros (contre 106,0 millions d'euros au premier semestre 2015) et une perte opérationnelle courante de 51,5 millions d'euros (contre une perte opérationnelle courante de 74,4 millions d'euros au premier semestre 2015).   Soitec a enregistré une charge opérationnelle nette non courante de 151,5 millions d'euros pour l'exercice 2014-2015 (contre 82,4 millions d'euros comptabilisés sur l'exercice 2013-2014), imputable en majeure partie aux charges de restructuration de la division Énergie solaire (138,9 millions d'euros liés aux dépréciations d'actifs et provisions pour contrats en cours, licenciements et démantèlement de centrales). Après frais financiers et impôts, le résultat net ressort en perte (part du Groupe) de 259,2 millions d'euros pour l'exercice 2014-2015 (contre une perte nette de 236,7 millions d'euros pour l'exercice 2013-2014).   La trésorerie disponible ressort à 22,9 millions d'euros à fin mars 2015, tandis que la dette brute s'élève à 173,0 millions d'euros, et la dette nette à 150,1 millions d'euros.   *  Albioma  : Compte tenu des incidents techniques et des mesures mises en oeuvre, le groupe estime que ses objectifs 2015 d'Ebitda de 126 à 130 millions d'euros et de résultat net part du Groupe de 34 à 37 millions d'euros doivent être réduits d'environ 10%.   Par ailleurs, Albioma a signé un avenant au contrat d'achat d'électricité conclu avec EDF en vue de la mise en conformité de la tranche B de la centrale d'Alioma Le Gol avec les futures normes environnementales issues de la directive européenne IED (Industrial Emissions Directive). L'investissement de 26 millions d'euros sera rémunéré au taux habituel des contrats du Groupe dans les départements d'Outre-mer. Sa mise en service interviendra en septembre 2016. Le programme de modernisation de ses centrales se chiffre à 200 ME d'investissement d'ici 2019.   Enfin, Albioma rappelle son annonce d'avril visant à l'acquisition au Brésil de 65% du capital de Codora Energia, une société propriétaire d'une unité de cogénération bagasse de 48 MW située dans l'Etat de Goiás. La finalisation de l'acquisition est soumise à des conditions suspensives devant être levées au second semestre de l'exercice 2015. Les modalités de l'accord avec le vendeur prévoient l'installation d'une nouvelle turbine de 20 MW qui sera mise en service en 2017. Dans ce contexte, Codora Energia a sécurisé la vente sur le marché règlementé de 54 GWh par an pendant 20 ans à compter de 2020, dans le cadre d'un appel d'offres (leilão de energia nova A-5), au prix historiquement élevé de 278 reals/MWh indexé sur l'inflation.   *  Manutan  : Le spécialiste de la distribution multicanal à destination des entreprises et collectivités, a dégagé un résultat net de 10,1 Millions d'Euros (ME) sur les six premiers mois de l'exercice fiscal 2014/2015, en recul de 7,7% par rapport à la période comparable de l'année précédente. Le résultat opérationnel courant s'établit à 17 ME, en repli de 12,3% en rythme annuel, pénalisé par la diminution du taux de marge commerciale et des coûts liés au déploiement du nouveau système d'information sur la France.   Le chiffre d'affaires atteint 303,2 ME au premier semestre, en baisse de 0,8% en rythme annuel. Dans le détail, Manutan a renoué avec la croissance et enregistre sur le second trimestre (janvier à mars 2015) une hausse de 1,8% de son chiffre d'affaires (+0,7% hors effet de change) après un premier trimestre (octobre à décembre 2014) en décroissance de 3,1% (-3,6% hors effet de change).   La situation financière du distributeur demeure très solide avec une trésorerie disponible de 81 millions d'euros et un endettement financier ne représentant pas plus de 8% du total du bilan.   *  Cegedim  : La société a enregistré au premier trimestre 2015 un chiffre d'affaires de 121,3 ME, en croissance de 7% en données publiées et de 4,9% en données organiques par rapport à la même période de 2014. L'EBIT des activités poursuivies avant éléments exceptionnels s'élève à 8,2 ME en progression de 76,3%. Ainsi, la marge d'EBIT des activités poursuivies avant éléments exceptionnels ressort à 6,8% au premier trimestre 2015, contre 4,1% un an plus tôt.   Consécutivement à la publication des résultats du premier trimestre 2015, Cegedim revoit à la hausse son anticipation de croissance pour 2015 de l'EBIT courant des activités poursuivies de 5% à 10%, après avoir revu en hausse le 28 avril dernier son anticipation de croissance du chiffre d'affaires 2015...   *  Naturex  : La société informe de la réorganisation de son activité industrielle pharmaceutique. Cette mesure s'inscrit dans le cadre du plan d'actions 2015 dont l'objectif principal est d'améliorer la rentabilité.   La société a décidé de concentrer l'ensemble des activités pharmaceutiques sein d'une seule et unique unité de production à Reyssouze en France. Cette réorganisation industrielle contribuera à améliorer la performance de l'activité pharmaceutique grâce à la centralisation de l'ensemble des expertises et une meilleure maîtrise des flux de production.
</article>
<article titre="Albioma : révise ses objectifs financiers 2015" date="2015 05 27" journal="Boursier (site web)">
Albioma révise ses objectifs financiers 2015 après deux incidents d'exploitation.    Incidents en Guadeloupe et à La Réunion Un incident d'exploitation important a touché, fin avril, la tranche 1 de la centrale thermique Albioma Le Moule (ALM) en Guadeloupe. Lors d'un arrêt programmé pour maintenance, le dysfonctionnement d'un équipement de mise en sécurité a provoqué la montée en survitesse du groupe turbo-alternateur. À l'issue de l'inspection des équipements endommagés, l'indisponibilité de l'installation est estimée à 50 jours. D'ailleurs, suite à l'incident d'Albioma Le Moule, de nature exceptionnelle et inédite, le Groupe a décidé de renforcer les procédures de sécurité, de modifier certains équipements et d'augmenter le stock de pièces stratégiques.    D'autre part, le 25 mai 2015, la chaudière de la tranche B de la centrale thermique Albioma Le Gol à La Réunion a connu un incident. Afin de réduire les perturbations sur le réseau de distribution d'électricité, il a été décidé avec EDF de redémarrer de manière anticipée les deux autres tranches, qui faisaient l'objet d'une maintenance programmée. Les travaux de réparation d'urgence sont en cours et le remplacement préventif de l'échangeur indisponible sera réalisé lors de l'arrêt programmé fin juillet 2015.  Réduction de voilure de 10% Compte tenu de ces incidents techniques et des mesures mises en oeuvre, le Groupe estime que ses objectifs 2015 d'Ebitda de 126 à 130 millions d'euros et de résultat net part du Groupe de 34 à 37 millions d'euros doivent être réduits d'environ 10%.    200 ME d'investissements d'ici 2019 Par ailleurs, Albioma a signé un avenant au contrat d'achat d'électricité conclu avec  EDF  en vue de la mise en conformité de la tranche B de la centrale d'Alioma Le Gol avec les futures normes environnementales issues de la directive européenne IED (Industrial Emissions Directive). L'investissement de 26 millions d'euros sera rémunéré au taux habituel des contrats du Groupe dans les départements d'Outre-mer. Sa mise en service interviendra en septembre 2016. Le programme de modernisation de ses centrales se chiffre à 200 ME d'investissement d'ici 2019.   Sécurisation de la vente de 54 Gwh/an à prix élevé Enfin, Albioma rappelle son annonce d'avril visant à l'acquisition au Brésil de 65% du capital de Codora Energia, une société propriétaire d'une unité de cogénération bagasse de 48 MW située dans l'Etat de Goiás. La finalisation de l'acquisition est soumise à des conditions suspensives devant être levées au second semestre de l'exercice 2015. Les modalités de l'accord avec le vendeur prévoient l'installation d'une nouvelle turbine de 20 MW qui sera mise en service en 2017. Dans ce contexte, Codora Energia a sécurisé la vente sur le marché règlementé de 54 GWh par an pendant 20 ans à compter de 2020, dans le cadre d'un appel d'offres (leilão de energia nova A-5), au prix historiquement élevé de 278 reals/MWh indexé sur l'inflation.
</article>
<article titre="[1 re course 1. Chamara de Guez (3), J.-M. Bazire, G. 1,80 P. 1,50 ; 2. Cara...]" date="2015 05 13" journal="Aujourd'hui en France">
1 re course  1. Chamara de Guez (3),  J.-M. Bazire, G. 1,80 P. 1,50 ;  2. Cara d'Occagnes (7),  M. Gauvin, P. 3,30 ;  3. Coquine du Vivier (2),  Y.-A. Briand, P. 2,40 ;  4. Candiska Mauzun (6),  C. Martens. Cp. gg. 17,60. Cp. pl. (3-7) : 8,60 (3-2) 4,60 (7-2) 15,20. Trio (3-7-2) : 49,90.    2 e course  1. Cash Booster (7),  P. Callier, G. 3,10 P. 1,60 ;  2. Cazirius de Guez (10),  J.-M. Bazire, P. 1,50 ;  3. Cilento (11),  J. Boillereau, P. 6,70 ;  4. Call Me Love (5),  G. Raffestin. Cp. gag. 3,70. Cp. pl. (7-10) : 2,70 (7-11) 18,90 (10-11) 17,20. Trio (7-10-11) : 62,60.  3 e course  1. Just Magic (1),  C. Martens, G. 3,60 P. 1,60 ;  2. Spirou d'Hautefois (4),  J. Boillereau, P. 1,90 ;  3. Urbain de Guez (9),  J.-M. Bazire, P. 1,40 ;  4. Utiwit (7),  M. Cormy.  5. Une Sérénade (12),  F.-J. Peltier. Coup. gag. 8,90. Coup. pl. (1-4) : 4,70 (1-9) 2,90 (4-9) 4,30. Trio (1-4-9) : 8,70.    4 e course  1. Arto d'Urzy (1),  L. Grosbot, G. 7,10 P. 2,80 ;  2. After Visais (12),  S. Busset, P. 2,90 ;  3. Ascot Beji (3),  J.-T. Uttewiller, P. 6,10 ;  4. Aude Gaëlle (6),  N. Paleau. Coup. gag. 29,90. Coup. pl. (1-12) : 11,40 (1-3) 23,90 (12-3) 45,80. Trio (1-12-3) : 440,70. Non partant 13 Angonito.    5 e course  1. Atilano (7),  P.-Y. Verva, G. 7,20 P. 2,70 ;  2. Vespa de Bourgogne (12),  P. Callier, P. 3,10 ;  3. Avenir (2),  L. Lamazière, P. 3,40 ;  4. Véloce d'Occagnes (9),  C. Martens.  5. Verso des Oliviers (5),  J. Asselie. Cp. gg. 41,10. Cp. pl. (7-12) : 12,30 (7-2) 11,10 (12-2) 15,60. Trio (7-12-2) : 136,80. NP 6 Vérone du Rabutin.  PICK 5 (7-12-2-9-5) : 706. 6 e course  1. Barrio Josselyn (8),  J.-M. Bazire, G. 1,80 P. 1,40 ;  2. Bright Charm (5),  T. Collaud, P. 2,40 ;  3. Bémol d'Occagnes (3),  C. Martens, P. 1,90 ;  4. Bleven (2),  P. Callier. Coup. gag. 16,60. Coup. pl. (8-5) : 5,50 (8-3) 3,90 (5-3) 9,80. Trio (8-5-3) : 30,10.    7 e course  1. Ambassadeur d'Am (7),  C. Martens, G. 2 P. 1,40 ;  2. Augusto (13),  J.-M. Bazire, P. 1,60 ;  3. Aramis Gédé (1),  J. Uroz, P. 2,30 ;  4. Aladin du Noyer (5),  A. Bonnefoy. Coup. gag. 3,90. Coup. pl. (7-13) : 2,50 (7-1) 5,30 (13-1) 7,10. Trio (7-13-1) : 16,10. NP 9 Apache Elde.    8 e course  1. Baie des Anges (7),  Y.-A. Briand, G. 1,60 P. 1,20 ;  2. Berenice's Fella (5),  Y. Lacombe, P. 1,50 ;  3. Biche du Colombier (10),  J. Filograsso, P. 2 ;  4. Balata (2),  B. Ruet. Coup. gag. 5,10. Coup. pl. (7-5) : 2,60 (7-10) 4,80 (5-10) 7,40. Trio (7-5-10) : 25,70. Non partant 3 Bagasse Darche, 4 Bacherra du Moulin, 9 Bona du Gabaron.
</article>
<article titre="Courses" date="2015 05 13" journal="Le Parisien">
1 re course  1. Chamara de Guez (3),  J.-M. Bazire, G. 1,80 P. 1,50 ;  2. Cara d'Occagnes (7),  M. Gauvin, P. 3,30 ;  3. Coquine du Vivier (2),  Y.-A. Briand, P. 2,40 ;  4. Candiska Mauzun (6),  C. Martens. Cp. gg. 17,60. Cp. pl. (3-7) : 8,60 (3-2) 4,60 (7-2) 15,20. Trio (3-7-2) : 49,90.    2 e course  1. Cash Booster (7),  P. Callier, G. 3,10 P. 1,60 ;  2. Cazirius de Guez (10),  J.-M. Bazire, P. 1,50 ;  3. Cilento (11),  J. Boillereau, P. 6,70 ;  4. Call Me Love (5),  G. Raffestin. Cp. gag. 3,70. Cp. pl. (7-10) : 2,70 (7-11) 18,90 (10-11) 17,20. Trio (7-10-11) : 62,60.  3 e course  1. Just Magic (1),  C. Martens, G. 3,60 P. 1,60 ;  2. Spirou d'Hautefois (4),  J. Boillereau, P. 1,90 ;  3. Urbain de Guez (9),  J.-M. Bazire, P. 1,40 ;  4. Utiwit (7),  M. Cormy.  5. Une Sérénade (12),  F.-J. Peltier. Coup. gag. 8,90. Coup. pl. (1-4) : 4,70 (1-9) 2,90 (4-9) 4,30. Trio (1-4-9) : 8,70.    4 e course  1. Arto d'Urzy (1),  L. Grosbot, G. 7,10 P. 2,80 ;  2. After Visais (12),  S. Busset, P. 2,90 ;  3. Ascot Beji (3),  J.-T. Uttewiller, P. 6,10 ;  4. Aude Gaëlle (6),  N. Paleau. Coup. gag. 29,90. Coup. pl. (1-12) : 11,40 (1-3) 23,90 (12-3) 45,80. Trio (1-12-3) : 440,70. Non partant 13 Angonito.    5 e course  1. Atilano (7),  P.-Y. Verva, G. 7,20 P. 2,70 ;  2. Vespa de Bourgogne (12),  P. Callier, P. 3,10 ;  3. Avenir (2),  L. Lamazière, P. 3,40 ;  4. Véloce d'Occagnes (9),  C. Martens.  5. Verso des Oliviers (5),  J. Asselie. Cp. gg. 41,10. Cp. pl. (7-12) : 12,30 (7-2) 11,10 (12-2) 15,60. Trio (7-12-2) : 136,80. NP 6 Vérone du Rabutin.  PICK 5 (7-12-2-9-5) : 706. 6 e course  1. Barrio Josselyn (8),  J.-M. Bazire, G. 1,80 P. 1,40 ;  2. Bright Charm (5),  T. Collaud, P. 2,40 ;  3. Bémol d'Occagnes (3),  C. Martens, P. 1,90 ;  4. Bleven (2),  P. Callier. Coup. gag. 16,60. Coup. pl. (8-5) : 5,50 (8-3) 3,90 (5-3) 9,80. Trio (8-5-3) : 30,10.    7 e course  1. Ambassadeur d'Am (7),  C. Martens, G. 2 P. 1,40 ;  2. Augusto (13),  J.-M. Bazire, P. 1,60 ;  3. Aramis Gédé (1),  J. Uroz, P. 2,30 ;  4. Aladin du Noyer (5),  A. Bonnefoy. Coup. gag. 3,90. Coup. pl. (7-13) : 2,50 (7-1) 5,30 (13-1) 7,10. Trio (7-13-1) : 16,10. NP 9 Apache Elde.    8 e course  1. Baie des Anges (7),  Y.-A. Briand, G. 1,60 P. 1,20 ;  2. Berenice's Fella (5),  Y. Lacombe, P. 1,50 ;  3. Biche du Colombier (10),  J. Filograsso, P. 2 ;  4. Balata (2),  B. Ruet. Coup. gag. 5,10. Coup. pl. (7-5) : 2,60 (7-10) 4,80 (5-10) 7,40. Trio (7-5-10) : 25,70. Non partant 3 Bagasse Darche, 4 Bacherra du Moulin, 9 Bona du Gabaron.
</article>
<article titre="La nouvelle jeunesse des biocarburants" date="2015 05 12" journal="Les Echos, no. 21936">
Et si cette fois était la bonne ? Le regain d'intérêt pour les biocarburants observé durant la première moitié des années 2000, quand le prix du baril de pétrole est reparti à la hausse, semblait plus ou moins avoir fait long feu. Après avoir fait l'objet de plans de développement nationaux (comme en 2005 en France) et de réglementations censées les favoriser (à l'image de la directive européenne RED, datant de 2009), les biocarburants ont vu leur intérêt de plus en plus remis en cause, que ce soit à cause de leur manque de compétitivité économique, de leur bénéfice environnemental moins évident qu'on ne l'avait cru ou - et c'est sur ce point peut-être que les débats ont été les plus vifs - de la concurrence déloyale qu'ils font à la production alimentaire, les terres arables mobilisées pour faire rouler les voitures ne l'étant pas pour nourrir la planète. Ces controverses ont terni l'image des biocarburants - rebaptisés « agrocarburants » par leurs détracteurs écologistes -, et ceux-ci ont progressivement disparu des écrans radar médiatiques. Mais, dans les laboratoires publics ou privés, les recherches ont continué, et les bioéthanols de première génération, qui entraient en concurrence frontale avec la production alimentaire, ont laissé place à une deuxième génération ne prêtant pas le flanc à cette critique. Parallèlement, d'autres filières que celle de l'éthanol, à l'image de celle développée par la société française Global Bioénergies créée en 2008, ont émergé : la start-up d'Evry vient d'annoncer la livraison à Arkema de son premier lot d'isobutène biosourcé, produit à partir du glucose issu d'amidon de blé. Les prémices d'un vrai décollage après ce qui a tout l'air d'avoir été un faux départ ?   Un champignon star A Rueil-Malmaison, dans les locaux de l'IFP Energies Nouvelles (Ifpen), étuves, boîtes de Petri et enceintes de confinement sont là pour nous rappeler que nous sommes dans un laboratoire de microbiologie. Le champignon qui y est examiné sous toutes les coutures a une curieuse histoire : les hommes ont pris connaissance de son existence pendant la Seconde Guerre mondiale, lorsque les biologistes de l'armée américaine se sont intéressés au micro-organisme qui dégradait le coton (et donc la cellulose) des toiles de tente ou de parachute des boys casernés dans les îles Salomon. Depuis, Trichoderma reesei, c'est son nom, n'a pas quitté les labos de biologie. Il en est même devenu l'une des stars, car l'industrie n'a pas tardé à voir quelle utilité elle pourrait tirer de la formidable capacité de ce champignon à transformer la cellulose en glucose, le plus courant des sucres simples.   Et c'est bien sur lui, Trichoderma reesei, que reposent aujourd'hui tous les espoirs des nombreux acteurs français du projet Futurol, qui associe partenaires de recherche (Ifpen, Inra, ARD, Lesaffre) et industriels (Tereos, Total, ONF...).  « Contrairement aux biocarburants de première génération qui sont produits à partir des cultures vivrières, les biocarburants de deuxième génération sont issus de la biomasse lignocellulosique »  , explique Frédéric Monot, chef du département des biotechnologies à l'Ifpen. Cette lignocellulose - un composé de cellulose, d'hémicellulose et de lignine présent dans la paroi des cellules des végétaux - peut provenir de multiples sources : résidus agricoles (rafles, feuilles, enveloppes des épis...), résidus forestiers, mais aussi plantes dédiées, tel le miscanthus. Encore faut-il mettre au point un procédé rentable pour extraire de la lignocellulose le glucose et les autres sucres simples, avant de les faire fermenter pour les transformer en éthanol. C'est tout le but du projet Futurol, lancé en 2008 et aujourd'hui à la veille d'aboutir.   L'hydrolyse, étape qui permet d'extraire les sucres, est effectuée par les enzymes sécrétées par Trichoderma reesei. Non pas la souche originale, trouvée par les scientifiques de l'US Army dans les îles Salomon, mais une souche génétiquement reprogrammée afin de produire le maximum d'enzymes avec un minimum d'énergie.  « Le champignon peut être vu comme une petite usine à fabriquer des enzymes , explique Antoine Margeot, chercheur au département des biotechnologies de l'Ifpen.  Sur les 9.000 gènes que compte son génome, nous en avons identifié 600 qui sont impliqués de près ou de loin dans le processus de sécrétion d'enzymes. Sur ces 600, nous en avons isolé une vingtaine qui peuvent potentiellement nous permettre d'optimiser le fonctionnement de l'usine. » Les recherches sur Trichoderma reesei vont donc bon train et, après la construction d'un pilote industriel sur le site de Pomacle, près de Reims, les acteurs du projet Futurol espèrent que la société française Axens, un « spin off » de l'Ifpen, pourra commencer de commercialiser le procédé auprès des fabricants de bioéthanol dès l'an prochain.   Le « mur du mélange » Mais, à moins d'être équipés d'un module spécial (appelé « Flex fuel » ) comme au Brésil, où la bagasse de canne à sucre est reine, les moteurs de voiture ne supportent qu'une petite dose d'éthanol mélangée à l'essence classique : de 10 à 15 %, les avis divergent. Ce « blend wall » (mur du mélange) est le principal obstacle qui se dresse sur la route menant à un déploiement massif des bioéthanols. D'où le parti pris par Marc Delcourt et Philippe Marlière, les deux fondateurs de Global Bioénergies, qui a récemment conclu un partenariat avec le constructeur Audi et inaugurera l'an prochain un premier démonstrateur en Allemagne.   Au coeur de l'activité de Global Bioénergies : l'isobutène, un composé chimique entrant dans la composition des caoutchoucs, plastiques, lubrifiants mais aussi des carburants. Il suffit en effet d'associer deux molécules d'isobutène pour obtenir de l'isooctane, c'est-à-dire de la (vraie) essence - et non un substitut concerné par le « blend wall » comme l'éthanol.  « En associant les molécules d'isobutène non plus deux par deux mais trois par trois, on obtient de l'isododécane, un carburant pouvant alimenter les moteurs d'avion »  , ajoute le PDG Marc Delcourt, déjà à la recherche d'un partenaire industriel dans l'aérien pour tester son biokérosène.   Jusqu'ici, l'isobutène ne s'obtenait qu'à partir du pétrole. Mais la biologie de synthèse permet aujourd'hui de se passer de l'or noir et de lui substituer comme matière première les mêmes ressources végétales (sucre, céréales, déchets agricoles et forestiers) utilisées pour produire les bioéthanols de première et deuxième génération. C'est avec une autre star des labos que Trichoderma reesei, la bactérie Escherichia coli, que Global Bioénergies a développé son procédé. Comme le champignon utilisé dans le cadre du projet Futurol, cette bactérie a été génétiquement reprogrammée afin de la forcer à transformer le glucose en isobutène, ce qu'elle ne fait pas naturellement. La livraison d'un premier lot à Arkema prouve que le procédé est au point. Reste, pour que les biocarburants inondent enfin le marché, à modifier les esprits. Ce que le génie génétique ne saurait faire...   Yann Verdo
</article>
<article titre="Albioma s'étend au Brésil et vise de nouveaux marchés (entretien)" date="2015 05 6" journal="GreenUnivers (site web)">
2015-05-07T11:26:40+00:00    Interview de Jacques Pétry, PDG d'Albioma - Recentrage sur la biomasse à base de canne à sucre dans les DOM, développement rapide au Brésil, investissement dans la méthanisation et le solaire couplé au stockage : depuis trois ans, Albioma a changé de visage sous la houlette de Jacques Pétry, ancien PDG de Suez Environnement appelé à la rescousse par Apax, premier actionnaire d'Albioma, après une période de turbulences. Le groupe (354 M euros de CA en 2014, 701 MW de capacité installée) commence à récolter les fruits de sa stratégie, sauf dans le biogaz. Il regarde aujourd'hui de nouvelles opportunités d'investissement, explique son PDG à GreenUnivers. GreenUnivers : Vous avez pris les commandes d'Albioma il y a un peu plus de trois ans en définissant une nouvelle stratégie recentrée sur la biomasse. Où en est son déploiement ? Jacques Pétry :  Depuis trois ans, nous déroulons la stratégie présentée à nos actionnaires en mars 2012. Notre priorité a effectivement consisté à développer la biomasse à partir de bagasse, activité dans laquelle Albioma dispose d'un savoir-faire unique construit à partir de plus de 20 ans d'expérience. En France, nous nous concentrons sur les Dom où nous allons progressivement évoluer vers des centrales 100% biomasse : les déchets verts et les résidus de l'industrie du bois se substituent au charbon, en complément de la canne à sucre. Ce sera le cas de la centrale du Galion 2 (40 MW) à La Martinique et de Saint-Pierre (une turbine à combustion de 40 MW utilisant du bio-ethanol) à La Réunion pour lesquelles les contrats d'achat avec EDF ont été signés, et de Marie-Galante (13 MW) en Guadeloupe.   Parallèlement aux Dom, Albioma devait s'internationaliser pour grandir. Nous sommes partis à la conquête du Brésil, premier producteur de canne à sucre dans le monde, où nous avons acheté un premier projet en 2014, Rio Pardo, avec l'objectif de sortir un nouveau projet tous les douze à dix-huit mois. Nous maintenons le rythme puisque nous venons d'annoncer un deuxième investissement dans une centrale de cogénération bagasse de 48 MW dans l'état de Goiás. Et nous discutons actuellement avec d'autres sucriers pour nous renforcer.   GU : Quels avantages avez-vous trouvés au Brésil ?  Le contexte global est d'abord un élément positif avec une demande en énergie qui augmente de 5% par an en moyenne. Et du coup, les prix progressent : nous sommes aujourd'hui à 180-200 reals /MWh contre 130-150 il y a un an et demi. De plus, nous bénéficions de contrats d'achat de long terme qui nous assurent une belle visibilité : les deux tiers des kWh que nous produisons sont vendus dans ce cadre. Le potentiel est gigantesque, la bagasse, qui produit déjà plus de 7% de l'électricité produite au Brésil, pourrait représenter jusqu'à 20% de l'électricité du pays. Nous avons une stratégie de partenariat : nous investissons dans des centrales dont nous assurons l'exploitation et que nous modernisons. Nous cherchons à en avoir le contrôle mais nous faisons une place importante aux sucriers pour aligner nos intérêts. A Rio Pardo par exemple, le propriétaire de la sucrerie peut racheter jusqu'à 40% de l'usine de cogénération. Nous avons prévu d'investir 400 millions d'euros au Brésil sur la période 2014-2023 et nous récoltons déjà les fruits de cette stratégie : notre EBITDA dans le pays a atteint 12 millions d'euros en 2014.   GU : Qui sont vos concurrents ?  Les premiers, ce sont les sucriers eux-mêmes. Il y a 248 centrales à bagasse exploitées au Brésil, dont deux par nous. Les rendements sont notre premier argument pour les convaincre de nous laisser l'exploitation : nos centrales produisent en moyenne 120 kWh/tonne de bagasse, contre 40 kWh/tonne pour les sucriers locaux. Nous avons augmenté la production exportée de Rio Pardo de 30% par tonne de cannes en 2014 par rapport à 2013.   GU : Une présence dans un seul pays pourrait vous fragiliser, prévoyez-vous d'investir dans d'autres zones ?  Nous ne voulons pas nous disperser mais nous ne comptons pas non plus mettre tous nos oeufs dans le même panier. Nous regardons d'autres marchés où la culture de la canne à sucre est importante comme l'Inde, la Thaïlande ou la Chine. Et s'il y a des opportunités dans d'autres pays d'Amérique latine, nous les étudierons également. Après le bois, la bagasse est la deuxième ressource dans le monde pour la biomasse : il y a 500 millions de tonnes valorisables, seulement 3 à 4% qui sont valorisés. Il faut juste bien choisir son terrain de jeu.   GU : Quelles sont vos critères pour vous implanter dans un pays ?  La possibilité de monter des financements de long terme en monnaie locale, l'absence de de risque de change et une stabilité juridique qui fait que les contrats sont respectés. Nous cherchons aussi des pays où il est possible de signer des contrats d'achat de l'électricité sur le long terme comme au Brésil.   GU : Vous n'investissez que dans du brownfield ?  En termes d'allocation de nos fonds, le  brownfield  offre une meilleure opportunité. Notre investissement dans Rio Pardo, c'est 40% du prix d'une centrale neuve. Mais nous regardons aussi le  greenfield,  notamment pour des extensions de centrales.   GU : Vous êtes sortis de l'éolien en 2013 en vendant vos actifs à EDF EN, mais vous restez dans le solaire. Vous venez d'ailleurs de vous renforcer à La Réunion via une acquisition de 14 toitures PV (3 MW). Que représente cette activité ?  La biomasse concentre 90% de nos investissements, le solaire est une activité complémentaire très rentable qui représente 22% de l'EBITDA du groupe. Nous ne voulons en faire que dans les zones ensoleillées où elle est rentable. Nous avons développé du solaire couplé avec du stockage à La Réunion sur une centrale de 1 MW, en service depuis 2014. Cela nous a permis de parfaire notre apprentissage. La technologie est de plus en plus performante et maîtrisée par le groupe. Nous avons un deuxième projet en Guyane (2 MW, mise en service prévue en 2017) et nous attendons le lancement du prochain appel d'offres solaire dans les Dom avec stockage qui nous intéresse beaucoup. Il serait aussi logique de proposer du solaire au Brésil, à condition de disposer d'un tarif d'achat attractif, ce qui n'est pas le cas aujourd'hui.   GU : Pourriez-vous investir dans la technologie du stockage ?  Non. Nous achetons des technologies, nous n'avons pas vocation à en développer nous-mêmes. Nous avons travaillé avec le groupe LG à La Réunion, mais nous n'avons pas d'accord d'exclusivité. Nous avons néanmoins développé un savoir-faire unique en matière de prévision de production à partir de données météorologiques et de gestion de l'énergie (Energy Management System).   GU : Vous avez mis un pied dans le biogaz en rachetant Methaneo en 2012. Mais vous avez dû procéder à une dépréciation d'actifs de 4,6 M euros en 2014 sur cette activité. Que comptez-vous en faire ?  Nous avons investi dans Methaneo qui avait un beau portefeuille de 20 projets. Nous en avons mis en service trois en 2014 en France, où le marché est effectivement difficile. Il faut d'abord une grande maîtrise industrielle : nous sommes en train d'y parvenir. Mais il y a aussi un problème économique : l'équilibre n'est pas atteignable en l'état. Il faudra un coup de pouce sur les tarifs pour l'atteindre. Nous avons donc mis notre développement en stand by. Mais nous restons intéressés par cette technologie que nous voulons maîtriser car elle représente un complément d'activité potentiellement important. Au Brésil, par exemple, les sous-produits susceptibles d'être méthanisés, comme la vinasse, résidu liquide de distillation, constituent un gros tonnage.   GU : Vous avez annoncé un milliard d'euros d'investissement entre 2013 et 2023, dont 30 à 40% de fonds propres, avec un doublement attendu du résultat net. Aurez-vous besoin de lever des fonds pour mener à bien ce programme ?  Le cash flow dégagé par nos centrales nous permet d'autofinancer nos projets. Notre développement doit être auto-portant. Notre patrimoine industriel est aujourd'hui de l'ordre de 1 milliard d'euros, il est appelé à doubler en dix ans. Nous avons un rythme moyen de croissance de 7,5% par an que nous comptons maintenir.Albioma en chiffres en 2014 CA : 354 M euros (363 M euros en 2013)EBITDA : 129 M euros (135,8 M euros en 2013)RN part du groupe : 38 M euros (37 M euros en 2013)Production : 3 432 GWhCapitalisation boursière : 557 M eurosActionnariat : Financière Hélios et affiliés du groupe Apax Partners (42,5%), Financière de l'Echiquier, flottant.  Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="Albioma: activité en repli au premier trimestre" date="2015 04 29" journal="AFP Infos Economiques">
Paris, 29 avr 2015 (AFP) - - Le producteur français d'électricité Albioma, surtout actif en outre-mer, a fait part mercredi d'un chiffre d'affaires en repli de 3% au premier trimestre, affecté par une grève en Guadeloupe et la baisse du prix du charbon.   Le chiffre d'affaires s'est élevé à 78,1 millions d'euros contre 80,2 millions au cours des trois premiers mois de 2014, détaille la société dans un communiqué. La production totale d'électricité atteint 521 gigawattheures (GWh) sur le trimestre contre 539 GWh lors de la même période de l'exercice précédent.   "Le premier trimestre est marqué par une performance opérationnelle en léger retrait à cause de la grève sur le site du Moule en Guadeloupe. L'ensemble des autres installations a présenté de bonnes performances", a estimé Jacques Pétry, PDG d'Albioma, cité dans le communiqué.   La grève, qui a affecté le site de Guadeloupe entre le 21 janvier et le 5 mars, a amputé le chiffre d'affaires de 3,1 millions d'euros.   La baisse du prix du charbon, combustible qui entre en concurrence avec la biomasse pour la production électrique, a grevé l'activité à hauteur d'1 million d'euros, précise la société.   Globalement, le chiffre d'affaires de la branche Biomasse thermique a atteint 66,9 millions d'euros, en baisse de 5%.   A l'inverse, les activités dans le solaire ont connu un bon premier trimestre, avec un chiffre d'affaires en hausse de 3% à 9,9 millions d'euros.   Par ailleurs, Albioma a annoncé avoir signé un contrat de financement (qui porte sur 120 millions d'euros sur un investissement total de 170 millions) et a débuté la construction du projet Galion 2 de centrale biomasse de 40 mégawatts en Martinique, la plus grande centrale 100% biomasse de l'outre-mer français.    La société a aussi sécurisé la vente de 50% de l'électricité produite par la centrale de Rio Pardo Termoelectrica sur 20 ans à un tarif "historiquement élevé", selon Jacques Pétry. Le groupe avait acquis l'an dernier cette centrale de cogénération (production simultanée d'électricité et de chaleur) située au Brésil, et alimentée en bagasse (résidu de canne à sucre) pour un montant de 43 millions d'euros.   Dans ce contexte, Albioma confirme ses objectifs pour l'ensemble de l'année, d'un bénéfice d'exploitation (Ebitda) compris entre 126 et 130 millions d'euros et d'un bénéfice net part du groupe entre 34 et 37 millions.   mhc/fpo/mml
</article>
<article titre="Albioma : les objectifs 2015 sont confirmés" date="2015 04 29" journal="Boursier (site web)">
La grève sur le site Guadeloupéen d' Albioma  a coûté sa croissance du 1er trimestre 2015 à la société. Les revenus se tassent de -3% sur la période, à 78,1 millions d'euros. La production d'électricité sur ce début d'année a représenté 521 GWh contre 539 GWh un an avant.   Le groupe évalue l'impact de la grève du site de production du Moule en Guadeloupe, du 21 janvier au 5 mars 2015, à 3,1 ME de revenus. Il précise avoir signé le contrat de financement du projet Galion 2 en Martinique (usine bagasse/biomasse), qui nécessite 170 ME. Un contrat de prêt de 120 ME a été signé dans des conditions très favorables. Le solde sera financé en fonds propres par Albioma et son partenaire COFEPP. La centrale de 40 MW devrait être opérationnelle au 1er semestre 2017. La direction a confirmé viser cette année 126 à 130 ME d'EBITDA et 34 à 37 ME de bénéfice net.
</article>
<article titre="AVIS de l'Agence nationale de sécurité sanitaire de l'alimentation, de l'environnement et du travail relatif à la demande d'homologation du produit CENDRES DE  BAGASSE ALBIOMA LE MOULE, à base de cendres de  bagasse de canne à sucre, de la société ALBIOMA LE MOULE (Guadeloupe)" date="2015 04 27" journal="Rapports - Agroalimentaire - MAAF">
9 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... AVIS de l'Agence nationale de sécurité sanitaire de l'alimentation, de l'environnement et du travail relatif à la demande d'homologation du produit CENDRES DE BAGASSE ALBIOMA LE MOULE...    Pour voir le rapport :  http://agriculture.gouv.fr/sites/minagri/files/avis_mfsc_cendres_de_bagasse_albioma_le_moule.pdf
</article>
<article titre="AVIS de l'Agence nationale de sécurité sanitaire de l'alimentation, de l'environnement et du travail relatif à la demande d'homologation du produit CENDRES DE  BAGASSE ALBIOMA LE GOL, à base de cendres de  bagasse de canne à sucre, de la société ALBIOMA LE GOL (Ile de La Réunion)" date="2015 04 27" journal="Rapports - Agroalimentaire - MAAF">
10 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... AVIS de l'Agence nationale de sécurité sanitaire de l'alimentation, de l'environnement et du travail relatif à la demande d'homologation du produit CENDRES DE BAGASSE ALBIOMA LE GOL...    Pour voir le rapport :  http://agriculture.gouv.fr/sites/minagri/files/avis_mfsc_cendres_de_bagasse_albioma_le_gol.pdf
</article>
<article titre="Le préoccupant exil de la jeunesse" date="2015 04 22" journal="L'Express, no. 3329">
Damien Tinas, 23 ans, a quitté la Martinique il y a quatre ans. Désormais titulaire d'un master 2 des structures, option aéronautique, il n'a pas l'intention d'y revenir dans l'immédiat. « Pour moi, ce n'est pas intéressant en termes de débouchés. Dans l'Hexagone, j'ai davan tage d'opportunités de développement, avec des postes au niveau natio nal ou dans des sociétés internationales. » Comme lui, 36 % des jeunes Martiniquais de 18 à 34 ans ont quitté leur île aujourd'hui, selon l'Insee. « Et 60% de ceux qui partent poursuivre leurs études ne reviennent pas » , assure le site Internet du conseil régional. D'ici à 2030, les moins de 20 ans ne représenteront plus que 22 % de la population totale.   Des pistes existent, mais les freins sont nombreux Catherine Bertho-Lavenir, rectrice de l'académie, s'efforce d'inciter les étudiants à rester à la Martinique après leur bac. « En cohérence avec les priorités de la région, nous ouvrons des places de BTS dans des secteurs comme le tourisme, le bâtiment ou les services à la personne. Nous développons aussi l'offre de classes préparatoires, notamment dans le domaine paramédical. Et nous travaillons à proposer des DUT avec l'université des Antilles, ainsi que davantage de licences. » L'an passé, 200 élèves de plus qu'en 2013 auraient ainsi, d'après le rectorat, poursuivi leurs études supérieures à la Martinique.   Daniel Robin, président de la commission éducation et formation professionnelle à la région, mesure la nécessité de conserver les forces vives du pays. « De très beaux projets en cours doivent remettre la Martinique au coeur du choix de ces jeunes à fort potentiel : les nouvelles technologies avec l'arrivée du très haut débit d'ici à 2022 ou encore le développement durable. L'un des meilleurs exemples, c'est la centrale bagasse-biomasse du Galion 2, à La Trinité, qui va créer de nombreux emplois. [NDLR: 39 directs et 200 indirects, d'après la région]. »   « Pour faire revenir nos jeunes et rendre notre territoire attractif, nous disposons de pistes très intéressantes » , ajoute Philippe Jock, président du Medef de l'île. Il cite, pêle-mêle, les services à la personne, l'agro-transformation - « le pôle agroalimentaire régional Martinique [Parm] fonctionne très bien » - ou encore l'hôtellerie, dont « le personnel est aujourd'hui âgé, pour moitié, de plus de 50 ans » .   Mais les freins demeurent nombreux sur une île marquée par son enclavement à la fois physique et conjoncturel.   Avant l'arrivée du très haut débit, promis pour 2022, la fracture numérique est bien réelle. « On doit aussi réfléchir en termes de logistique et de transport dans le cadre de notre environnement caribéen. Il faut savoir si nous voulons, par exemple, exporter notre savoir-faire dans les énergies renouvelables » , soutient le patron des patrons martiniquais.   Une majorité de ceux qui reviennent ne sont pas diplômés Plus présents dans les écoles, comme lors des semaines « école et entrepreneuriat » , les acteurs économiques semblent avoir enclenché une timide prise de conscience depuis les grandes grèves de 2009. « Beaucoup ont compris la nécessité de confier des postes de cadres à des jeunes originaires du territoire plutôt qu'à des métropolitains, à diplômes équivalents. Cela n'a pas toujours été le cas » , reconnaît Philippe Jock. « Mais nos entreprises ne pourront être compétitives sans des mesures compensatoires, au plus haut niveau de l'Etat » , revendique, pour sa part, Pierre Marie-Joseph, de l'Association martiniquaise pour la promotion de l'industrie (Ampi).   Quoi qu'il en soit, cela n'est pas encore suffisant pour faire rester la jeunesse dans son île. Sachant aussi que la majorité de ceux qui reviennent sont des non-diplômés qui n'ont pas trouvé de travail ailleurs. Et qu'ils font leur come-back dans un département où le chômage des 18-25 ans atteignait 68,2% en 2013...
</article>
<article titre="Conseil général de l’environnement et du développement durable N° 009548-02 Conseil général de l’économie" date="2015 04 22" journal="Rapports - Défense - Ministère Intérieur France">
19 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... N° 2015/28/CGE/CIInspection générale de l’administrationN° 14-079/14-058/03 Inspection générale des financesN°2014_M_047_05 RAPPORT  Note additionnelle au rapport de la mission MAP...    Pour voir le rapport :  http://www.interieur.gouv.fr/fr/content/download/84721/655788/file/14079-14058-03-map-dÃ©chet-outre-mer.pdf
</article>
<article titre="Conseil général de l’environnement et du développement durableN° 009548-02 Conseil général de l’économie" date="2015 04 22" journal="Rapports - Défense - Ministère Intérieur France">
19 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... N° 2015/28/CGE/CIInspection générale de l’administrationN° 14-079/14-058/03 Inspection générale des financesN°2014_M_047_05 RAPPORT  Note additionnelle au rapport de la mission MAP...    Pour voir le rapport :  http://www.interieur.gouv.fr/content/download/84720/655781/file/14079-14058-03 - MAP DÃ©chet - Note outre-mer.pdf
</article>
<article titre="Coup de coeur de la semaine Albioma" date="2015 04 18" journal="Investir-Le Journal des finances, no. 2154">
Deux acquisitions annoncées en deux jours : voilà de quoi ranimer l'action Albioma, qui s'est octroyé 8 % sur la semaine. Le producteur d'électricité a d'abord fait part mardi soir du rachat de 14 centrales solaires à La Réunion, d'une puissance totale de 3 mégawatts (MW). Ces installations sont situées sur des toitures, comme la centrale déjà mise en service en 2014 par Albioma sur un hypermarché à Saint-Pierre. « Cette acquisition est intéressante car les parcs sont à côté de nos installations », nous a confié Jacques Pétry, le PDG d'Albioma.   L'autre opération, d'une tout autre envergure, était plus attendue. Il s'agit de l'acquisition d'une centrale de cogénération au Brésil, la deuxième après une première opération réalisée en mars 2014 à Rio Pardo, dans le sud du pays. Cette deuxième centrale est reprise, pour 65 % de son capital, à un groupe sucrier brésilien qui exploite une sucrerie voisine. Selon le modèle des autres centrales thermiques exploitées par Albioma, elle brûle de la bagasse, les résidus de la canne à sucre. D'une puissance de 98 MW, elle est située dans l'Etat de Goias, celui de la capitale, Brasilia. « C'est un avantage d'être loin de Rio Pardo pour diversifier les risques en termes de climat et de récolte », selon Jacques Pétry. A la différence de la première, cette unité bénéficie déjà d'un contrat de vente d'électricité à prix sécurisé jusqu'en 2026, ce qui limite également les risques. Albioma respecte son programme d'acquisitions au Brésil tous les douze à dix-huit mois, grâce à une enveloppe d'investissements de 400 millions entre 2013 et 2023. Le prix de cette deuxième centrale serait proche de celui consenti pour Rio Pardo, soit 43 millions d'euros. Les objectifs donnés par Albioma pour 2015, dont un bénéfice net de 34 à 37 millions, intègrent cette croissance externe.   Notre conseil Acheter Outre ses ambitions au Brésil, le groupe a deux projets bien avancés de centrales aux Antilles pour 2017-2018. Viser 23 euros (ABIO).   Cécile Le Coz
</article>
<article titre="BOURSE-Albioma portée par une nouvelle acquisition au Brésil" date="2015 04 17" journal="Reuters économique">
PARIS, 17 avril (Reuters) - L'action Albioma s'inscrit en vive hausse vendredi matin à la Bourse de Paris au lendemain de l'annonce par le producteur d'énergie indépendant d'une nouvelle acquisition au Brésil.   A 10h45, le titre avance de 2,98% à 19,7 euros pendant que l'indice CAC Mid &amp; Small cède 0,2% au même instant. Albioma a annoncé jeudi la signature de la documentation définitive en vue de l'acquisition au Brésil de 65% du capital de Codora Energia, propriétaire d'une usine de cogénération bagasse (48 MW) située dans l'Etat de Goiás.   L'acquisition sera financée pour moitié en dette locale et le reste en fonds propres et sa finalisation est soumise à des conditions suspensives devant être levées au second semestre de l'exercice 2015.   "Cette opération s'inscrit dans la lignée du discours du management et semble similaire à celle réalisée l'an passé", constate dans une note Louis Capital Markets-Midcap Partners.   Cette nouvelle opération au Brésil intervient après l'acquisition annoncée en mars 2014 de la centrale Rio Pardo Termoelétrica, une unité de cogénération - fonctionnant à partir de bagasse (résidu de canne à sucre) et de charbon - située dans l'Etat de Sao Paulo.   "L'opération devrait donc permettre de combler la baisse de l'Ebitda brésilien attendue pour 2015 tout en offrant un beau potentiel de croissance sur la zone pour 2016. Le groupe ne précise pas le prix de cette opération mais indique que son impact sur le compte de résultat était déjà compris dans la guidance indiquée pour 2015", ajoute le broker.   Le groupe, qui a également annoncé ce mardi l'acquisition de 14 centrales photovoltaïques à La Réunion, vise pour cette année un Ebitda (résultat avant intérêt, impôts, dépréciations et amortissements) compris entre 126 et 130 millions d'euros et un résultat net part du groupe de 34 à 37 millions d'euros.   Le communiqué:   http://bit.ly/1DclZUA   (Alexandre Boksenbaum-Granier, édité par Jean-Michel Bélot)
</article>
<article titre="Albioma boucle un deuxième rachat au Brésil" date="2015 04 17" journal="GreenUnivers (site web)">
2015-04-17T15:01:16+00:00    Le spécialiste français de la biomasse, qui s'est engagé à réaliser une acquisition tous les 12 à 18 mois au Brésil, tient pour l'instant sa promesse : il rachète une centrale de cogénération bagasse (résidu fibreux de la canne à sucre) dans l'État de Goiás, un peu plus d'un an après sa première acquisition dans l'Etat de Sao Paulo.Appel d'offresLe groupe a remporté un appel d'offres lancé par Jalles Machado, deuxième producteur mondial de sucre biologique, pour l'exploitation d'une centrale de cogénération bagasse de 48 MW, adossée à l'une de ses distilleries traitant 1,6 million de tonnes de cannes à sucre par an. Les deux partenaires se sont entendus sur le rachat à 65% de la société de projet Codora Energia par Albioma, Jalles Machado conservant les 35% restants. Le prix de la transaction n'est pas dévoilé.Contrat d'achat et marché spotMise en service en 2011, la centrale exporte actuellement 98 GWh sur le réseau électrique, dont 87 GWh vendus dans le cadre d'un contrat de vente d'électricité à long terme sécurisé jusqu'en 2026 au tarif de 205 reals/MWh (62,73 euros/MWh). Le reste est vendu sur le marché spot.Augmenter la production de 75%« L'augmentation graduelle du volume de canne à sucre traité par la distillerie - à 2,1 millions de tonnes par an - et, à moyen terme, l'ajout d'une troisième turbine d'une puissance de 20 MW devraient permettre de porter la quantité d'électricité exportée à plus de 170 Gwh », indique Albioma dans un communiqué. Là aussi, de nouveaux appels d'offres publics pourraient permettre d'obtenir des contrats d'achats de long terme pour l'électricité produite. L'acquisition sera financée à 50% en dette locale et à 50% sur fonds propres. Sa finalisation est soumise à des conditions suspensives devant être levées au second semestre de l'exercice 2015.   Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="ALBIOMA Le Brésil lui va si bien" date="2015 04 17" journal="Investir.fr">
Albioma la tient sa nouvelle acquisition au Brésil ! Comme il l'avait montré à l'occasion de la publication des résultats annuels début mars, le producteur d'électricité a pour volonté de poursuivre son développement dans ce pays, devenu son premier marché cible à l'international. C'est désormais chose faite avec le rachat partiel, à 65% du capital, de la centrale de cogénération Codora Energia, dans l'état de Goiás. Elle fonctionne à la bagasse, les résidus de la canne à sucre.   Cette usine, entrée en service en 2011, est adossée à une distillerie traitant aujourd'hui 1,6 million de tonnes de canne à sucre par an, détenue par le groupe sucrier Jalles Machado. C'est d'ailleurs ce groupe, deuxième producteur mondial de sucre issu de l'agriculture biologique, qui gardera les 35% restants de Codora Energia. Le financement de l'opération se fera à 50% en dette locale et à 50% en fonds propres, selon des conditions suspensives devant être levées au second semestre de 2015. Même si le prix d'achat n'a pas été communiqué, les analystes de Gilbert Dupont estiment qu'il ressort autour des 43 millions d'euros, comme l'acquisition précédente. Alors que la centrale exporte actuellement 98 GWh et bénéficie d'un contrat de vente d'électricité sécurisé jusqu'en 2026, elle devrait voir sa capacité augmenter avec l'arrivée d'Albioma et de son savoir-faire industriel. Elle pourra également jouir de l'augmentation graduelle du volume de canne à sucre et, à moyen terme, de l'ajout d'une troisième turbine d'une puissance de 20 MW, afin de porter la quantité d'électricité exportée à plus de 170 GWh à l'horizon 2018.   Le PDG d'Albioma, Jacques Pétry, s'est félicité de cette nouvelle acquisition brésilienne, après celle de Rio Pardo en avril 2014. Il a d'autre part précisé que « le projet Codora devrait à son tour contribuer positivement aux résultats du Groupe, dès sa première année d'exploitation par Albioma. »   Le titre gagne ce vendredi près de 3%, à 19,65 euros, après avoir touché un plus haut annuel plus tôt ce matin. Il avait déjà grimpé de plus de 3,50% mercredi à la suite de l'annonce d'une autre acquisition, plus modeste, de 14 centrales photovoltaïques à La Réunion.
</article>
<article titre="ALBIOMA en hausse après une acquisition au Brésil" date="2015 04 17" journal="Option Finance (site web)">
AOF - 17 avril 2015   (AOF) - Albioma (-0,26% à 19,08 euros) limite ses pertes après l'acquisition d'une centrale au Brésil. Le producteur d'énergie indépendant a racheté 65 % du capital de Codora Energia, propriétaire d'une usine de cogénération bagasse (48 MW) située dans l'État de Goiás. L'acquisition sera financée à 50% en dette locale et à 50% en fonds propres. Mise en service en 2011, la centrale est adossée à une distillerie traitant aujourd'hui 1,6 million de tonnes de canne à sucre par an et détenue par le groupe sucrier Jalles Machado. Ce dernier conservera les 35% restants.   Deuxième producteur mondial de sucre issu de l'agriculture biologique, ce groupe a traité 4,3 millions de tonnes de canne à sucre au cours de la dernière campagne. La centrale exporte actuellement 98 GWh sur le réseau électrique et bénéficie d'un contrat de vente d'électricité à long terme sécurisé jusqu'en 2026 pour l'exportation de 87 GWh/an au tarif attractif de 205 reais/MWh, indexé sur l'inflation.   L'amélioration de l'efficacité énergétique issue du savoir-faire industriel d'Albioma, l'augmentation graduelle du volume de canne à sucre traité par la distillerie à 2,1 millions de tonnes par an et, à moyen terme, l'ajout d'une troisième turbine d'une puissance de 20 MW et la mobilisation de la paille de canne à sucre, devraient permettre de porter la quantité d'électricité exportée à plus de 170 GWh.   Dans une note publiée ce matin, CM-CIC Securtities estime que ce projet permettra de donner une valeur plus importante à la croissance du groupe, avec des résultats qui progresseront significativement en 2016 et 2017. Pour le courtier, cette acquisition valide la stratégie de développement d'Albioma ainsi que son savoir-faire industriel. Dans ce cadre, l'analyste a confirmé sa recommandation d'Achat et relevé son objectif de cours de 21 à 23 euros.   (P-J.L)  Cet article est paru dans Option Finance (site web)
</article>
<article titre="Albioma: acquisition d'une centrale au Brésil." date="2015 04 17" journal="Votre argent (site web)">
Albioma a annoncé hier après Bourse la signature de la documentation définitive en vue de l'acquisition au Brésil de 65 % du capital de Codora Energia, propriétaire d'une usine de cogénération bagasse de 48 mégawatts (MW) située dans l'État de Goiás. Mise en service en 2011, cette structure est adossée à une distillerie traitant aujourd'hui 1,6 million de tonnes de canne à sucre par an et détenue par le groupe sucrier Jalles Machado. Deuxième producteur mondial de sucre issu de l'agriculture biologique, ce groupe a traité 4,3 millions de tonnes de canne à sucre au cours de la dernière campagne.  La centrale exporte actuellement 98 gigawatts heure (GWh) sur le réseau électrique et bénéficie d'un contrat de vente d'électricité à long terme sécurisé jusqu'en 2026 pour l'exportation de 87 GWh/an au tarif attractif de 205 reals/MWh, indexé sur l'inflation. A terme, la quantité d'électricité exportée devrait être portée à plus de 170 GWh.   L'acquisition de 65% du capital de Codora Energia sera financée pour moitié en dette locale et pour moitié en fonds propres. Sa finalisation est soumise à des conditions suspensives qui doivent être levées au second semestre de l'exercice en cours.   Copyright (c) 2015 CercleFinance.com. Tous droits réservés.
</article>
<article titre="Albioma: Portzamparc salue l'acquisition au Brésil." date="2015 04 17" journal="Cercle Finance (site web)">
Portzamparc reste à l'achat sur le titre Albioma après l'annonce de l'acquisition au Brésil de 65 % du capital de Codora Energia, propriétaire d'une usine de cogénération bagasse de 48 mégawatts (MW) située dans l'État de Goiás. L'objectif de cours est fixé à 22,70 E.   ' Les conditions sont réunies pour que cette nouvelle acquisition soit un succès. Les résultats du groupe n'attendront pas la mise en service de CCG2 en 2017 pour poursuivre leur croissance ' précise Portzamparc. ' Cette deuxième opération présente selon nous des risques limités. Albioma connaît désormais le marché brésilien et le profil et la taille de Codora Energia est comparable à celui de Rio Pardo première opération couronnée de succès d'Albioma au Brésil ' explique le bureau d'études.   Portzamparc explique également que ' près de 90% de la production actuelle bénéficient d'un contrat de vente d'électricité à long terme (2026) et 50% de l'acquisition sont financés en dette locale ce qui limite l'exposition devises '.   ' Nous estimons le prix à environ 50ME pour 100% et un EBITDA d'environ 7ME en 2018. L'opération devrait être validée au 2ème semestre 2015 et avoir un effet relutif, selon nous, de 7% en 2016 (intégration au 4ème trimestre 2015e) '.   Copyright (c) 2015 CercleFinance.com. Tous droits réservés.   Les informations et analyses diffusées par Cercle Finance ne constituent qu'une aide à la décision pour les investisseurs. La responsabilité de Cercle Finance ne peut être retenue directement ou indirectement suite à l'utilisation des informations et analyses par les lecteurs. Il est recommandé à toute personne non avertie de consulter un conseiller professionnel avant tout investissement. Ces informations indicatives ne constituent en aucune manière une incitation à vendre ou une sollicitation à acheter.
</article>
<article titre="Albioma : place au soleil" date="2015 04 17" journal="Boursier (site web)">
Albioma  grimpe de 3% à 19,70 euros ce vendredi, après avoir annoncé la signature de la documentation définitive en vue de l'acquisition au Brésil de 65% du capital de Codora Energia, propriétaire d'une usine de cogénération bagasse (48 MW) située dans l'État de Goiás. Mise en service en 2011, la centrale est adossée à une distillerie traitant aujourd'hui 1,6 million de tonnes de canne à sucre par an et détenue par le groupe sucrier Jalles Machado.  La centrale exporte actuellement 98 GWh sur le réseau électrique et bénéficie d'un contrat de vente d'électricité à long terme sécurisé jusqu'en 2026 pour l'exportation de 87 GWh/an au tarif de 205 reals/MWh, indexé sur l'inflation.    "Suramortissement" Portzamparc reste à "Acheter" sur le dossier avec un objectif de cours de 21,9 euros. "La mesure fiscale de "suramortissement" pourrait profiter au groupe mais son éligibilité n'est pas certaine" explique l'analyste...   Albioma pense que l'amélioration de l'efficacité énergétique issue de son savoir-faire industriel, l'augmentation graduelle du volume de canne à sucre traité par la distillerie à 2,1 millions de tonnes par an et, à moyen terme, l'ajout d'une troisième turbine d'une puissance de 20 MW et la mobilisation de la paille de canne à sucre, devraient permettre de porter la quantité d'électricité exportée à plus de 170 GWh.    Albioma détiendra 65% du capital de Codora Energia, le groupe Jalles Machado conservant les 35 % restants. L'acquisition sera financée à 50% en dette locale et à 50% en fonds propres. Sa finalisation est soumise à des conditions suspensives devant être levées au second semestre de l'exercice 2015...
</article>
<article titre="Les valeurs à suivre vendredi 17 avril 2015 à la Bourse de Paris" date="2015 04 17" journal="Option Finance (site web)">
AOF - 17 avril 2015   (AOF) - ACCOR   Accor a réalisé au premier trimestre 2015 un chiffre d'affaires de 1,225 milliard d'euros, en hausse de 5,6% à périmètre et change constants et de 7,9% en données publiées. La branche HotelInvest, propriétaire et investisseur hôtelier, a vu son chiffre d'affaires progresser de 4,8% à périmètre et change constants à 1,055 milliard. La branche a connu de bonnes progressions sur l'ensemble des zones, notamment NCEE (+6,4%) et MMEA (+7,3%), et dans une moindre mesure dans la zone Amériques (+3,5%), qui ralentit par rapport au dernier trimestre 2014.   ALBIOMA Albioma a annoncé la signature de la documentation définitive en vue de l'acquisition au Brésil de 65 % du capital de Codora Energia, propriétaire d'une usine de cogénération bagasse (48 MW) située dans l'État de Goiás. L'acquisition sera financée à 50 % en dette locale et à 50 % en fonds propres. Mise en service en 2011, la centrale est adossée à une distillerie traitant aujourd'hui 1,6 million de tonnes de canne à sucre par an et détenue par le groupe sucrier Jalles Machado. Ce dernier conservera les 35 % restants.   GROUPE GORGE   Le Groupe ECA, filiale du Groupe Gorgé, a obtenu deux contrats à l'export d'un montant total supérieur à 10 millions d'euros concernant les drones navals de surface (USV - Unmanned Surface Vehicle). D'une part, un premier client vient d'exercer son option pour deux USV de type Inspector Mk2 supplémentaires après en avoir commandé deux début 2014.   LAFARGE   Lafarge et Holcim ont annoncé le détail des actifs qu'ils proposent de désinvestir aux Etats-Unis dans le cadre de leur projet de fusion pour créer LafargeHolcim. Les actifs proposés incluent la cimenterie de Davenport, détenue par le groupe français dans l'Iowa et d'une capacité de 1,1 MT, ainsi que 7 terminaux le long du Mississippi. Ces actifs seront cédés à Summit Materials pour un montant total de 450 millions de dollars (420 millions d'euros) auquel s'ajoute le terminal cimentier de Summit à Bettendorf, dans l'Iowa.   MERCIALYS   Mercialys a réalisé de bonnes performances opérationnelles au 1er trimestre 2015, qui se sont traduites par une progression de 2,5% des loyers facturés à périmètre constant, dont 2,6% hors impact d'une indexation légèrement négative. Les revenus locatifs s'élèvent à 39,6 millions d'euros au 31 mars 2015, en progression de 6,7%. A périmètre constant, les loyers facturés progressent de 2,5%. Les centres commerciaux de Mercialys ont continué à surperformer le secteur en France, à la fois en termes de fréquentation et d'évolution du chiffre d'affaires des commerçants, a précisé le groupe.   NEXTRADIOTV   NextRadioTV a réalisé au premier trimestre un chiffre d'affaires de 48,2 millions d'euros, en progression de 13%. Le groupe de médias a bénéficié de la bonne performance de division TV, dont le chiffre d'affaires a bondi de 31% à 28,6 millions d'euros. « Au 2ème trimestre 2015, la division TV devrait capitaliser sur les bonnes audiences de BFMTV et RMC Découverte pour continuer à gagner des parts de marché publicitaires et afficher, dans une moindre mesure qu'au 1er trimestre, une croissance soutenue de ses recettes », a ajouté NextRadioTV.   PIERRE &amp; VACANCES   Pierre &amp; Vacances a fait état, après la clôture des marchés, d'un chiffre d'affaires trimestriel en progression de 15,8% à 659,7 millions d'euros. Dans le détail, le revenu des activités touristiques s'élève à 476,2 millions d'euros, en progression de +2,4% par rapport à l'année dernière à pareille époque tandis que le chiffre d'affaires du "développement immobilier" s'élève à 174,5 millions d'euros, en croissance de 79,9%.   PLASTIC OMNIUM   A l'occasion du Salon Automobile de Shanghai, Plastic Omnium a détaillé ses dernières avancées sur le marché chinois. Le groupe lancera, en 2015-2016, 4 nouvelles unités de production, pour porter son tissu industriel chinois à 25 usines. L'équipementier entend renforcer ses parts de marché qui s'élèveront en 2018 à 25% dans les pare-chocs (2014 : 18%) et à 15% dans les systèmes à carburant (2014 : 8%). Le chiffre d'affaires doublera, avec 1 milliard d'euros attendus en 2018, contre 0,5 milliard en 2014.   RENAULT   Le Conseil d'Administration de Renault, qui s'est réuni jeudi 16 avril 2015, a demandé le maintien de l'équilibre actuel entre ses deux principaux actionnaires, l'Etat et Nissan. Le constructeur automobile estime que des droits de votes supplémentaires pour l'Etat risqueraient de le compromettre. Le Conseil d'Administration demande à ce que cet équilibre « soit maintenu lors de la prochaine assemblée générale ou restauré après celle-ci ». Il ainsi demandé à Carlos Ghosn de veiller à la pérennité et à l'équilibre de l'Alliance.   STENTYS   Stentys a fait état, après la clôture des marchés, d'un chiffre d'affaires en forte progression, de 28% à 1,09 million d'euros pour le compte de son premier trimestre. Cette accélération de la croissance résulte du démarrage des ventes du stent auto-apposant actif à libération de Sirolimus, Stentys SES, qui a obtenu le marquage CE fin octobre 2014 et dont le déploiement géographique se met progressivement en place. STENTYS SES représente 68% des ventes sur la période alors que le produit qu'il remplace, le stent actif à élution de paclitaxel, ne représente plus que 16% des ventes.  Cet article est paru dans Option Finance (site web)
</article>
<article titre="Albioma: Portzamparc salue l'acquisition au Brésil." date="2015 04 17" journal="Votre argent (site web)">
Portzamparc reste à l'achat sur le titre Albioma après l'annonce de l'acquisition au Brésil de 65 % du capital de Codora Energia, propriétaire d'une usine de cogénération bagasse de 48 mégawatts (MW) située dans l'État de Goiás. L'objectif de cours est fixé à 22,70 E. ' Les conditions sont réunies pour que cette nouvelle acquisition soit un succès. Les résultats du groupe n'attendront pas la mise en service de CCG2 en 2017 pour poursuivre leur croissance ' précise Portzamparc.   ' Cette deuxième opération présente selon nous des risques limités. Albioma connaît désormais le marché brésilien et le profil et la taille de Codora Energia est comparable à celui de Rio Pardo première opération couronnée de succès d'Albioma au Brésil ' explique le bureau d'études.   Portzamparc explique également que ' près de 90% de la production actuelle bénéficient d'un contrat de vente d'électricité à long terme (2026) et 50% de l'acquisition sont financés en dette locale ce qui limite l'exposition devises '.   ' Nous estimons le prix à environ 50ME pour 100% et un EBITDA d'environ 7ME en 2018. L'opération devrait être validée au 2ème semestre 2015 et avoir un effet relutif, selon nous, de 7% en 2016 (intégration au 4ème trimestre 2015e) '.   Copyright (c) 2015 CercleFinance.com. Tous droits réservés.Les informations et analyses diffusées par Cercle Finance ne constituent qu'une aide à la décision pour les investisseurs. La responsabilité de Cercle Finance ne peut être retenue directement ou indirectement suite à l'utilisation des informations et analyses par les lecteurs. Il est recommandé à toute personne non avertie de consulter un conseiller professionnel avant tout investissement. Ces informations indicatives ne constituent en aucune manière une incitation à vendre ou une sollicitation à acheter.
</article>
<article titre="Albioma acquiert 65% d'une centrale de cogénération au Brésil" date="2015 04 16" journal="AFP Infos Economiques">
Paris, 16 avr 2015 (AFP) - - Albioma a annoncé jeudi l'acquisition au groupe sucrier Jalles Marchado de 65% du capital de la société Codora Energia, propriétaire d'une centrale de cogénération de 48 mégawatts (MW) dans l'Etat de Goias (centre du Brésil).   Cette usine utilise comme source d'énergie la bagasse, le résidu ligneux résultant du broyage de la canne à sucre. La centrale produit actuellement 98 gigawattsheure (GWh) d'électricité par an et Albioma compte atteindre 170 GWh en augmentant le volume de canne à sucre traité et en ajoutant une troisième turbine d'une puissance de 20 MW, a précisé la société dans un communiqué.   Elle bénéficie d'un contrat de vente d'électricité sécurisé jusqu'en 2026 pour vendre 87 GWh par an à un tarif bonifié.   La centrale est adossée à une distillerie traitant 1,6 million de tonnes de canne à sucre par an, détenue par Jalles Machado, deuxième producteur mondial de sucre issue de l'agriculture biologique. Albioma prévoit de porter sa capacité à 2,1 millions de tonnes par an.   Albioma précise qu'elle financera cette acquisition à 50% en dette locale et à 50% en fonds propres.    La société, surtout présente dans les territoires d'outremer, avait acheté en 2014 une première centrale électrique au Brésil à Rio Pardo, déjà alimentée par des résidus de canne à sucre.    Mardi, elle avait annoncé l'acquisition de 14 centrales photovoltaïques en toiture à La Réunion d'une puissance de 3 MW.   mhc/fga/nas
</article>
<article titre="Les valeurs à suivre demain à la Bourse de Paris - Jeudi 16 Avril 2015" date="2015 04 16" journal="Option Finance (site web)">
AOF - 16 avril 2015   (AOF) -    ACCOR   Accor a réalisé au premier trimestre 2015 un chiffre d'affaires de 1,225 milliard d'euros, en hausse de 5,6% à périmètre et change constants et de 7,9% en données publiées. La branche HotelInvest, propriétaire et investisseur hôtelier, a vu son chiffre d'affaires progresser de 4,8% à périmètre et change constants à 1,055 milliard. La branche a connu de bonnes progressions sur l'ensemble des zones, notamment NCEE (+6,4%) et MMEA (+7,3%), et dans une moindre mesure dans la zone Amériques (+3,5%), qui ralentit par rapport au dernier trimestre 2014. ALBIOMA   Albioma a annoncé la signature de la documentation définitive en vue de l'acquisition au Brésil de 65 % du capital de Codora Energia, propriétaire d'une usine de cogénération bagasse (48 MW) située dans l'État de Goiás. L'acquisition sera financée à 50 % en dette locale et à 50 % en fonds propres. Mise en service en 2011, la centrale est adossée à une distillerie traitant aujourd'hui 1,6 million de tonnes de canne à sucre par an et détenue par le groupe sucrier Jalles Machado. Ce dernier conservera les 35 % restants.   MERCIALYS   Mercialys a réalisé de bonnes performances opérationnelles au 1er trimestre 2015, qui se sont traduites par une progression de 2,5% des loyers facturés à périmètre constant, dont 2,6% hors impact d'une indexation légèrement négative. Les revenus locatifs s'élèvent à 39,6 millions d'euros au 31 mars 2015, en progression de 6,7%. A périmètre constant, les loyers facturés progressent de 2,5%. Les centres commerciaux de Mercialys ont continué à surperformer le secteur en France, à la fois en termes de fréquentation et d'évolution du chiffre d'affaires des commerçants, a précisé le groupe.   PIERRE &amp; VACANCES   Pierre &amp; Vacances a fait état, après la clôture des marchés, d'un chiffre d'affaires trimestriel en progression de 15,8% à 659,7 millions d'euros. Dans le détail, le revenu des activités touristiques s'élève à 476,2 millions d'euros, en progression de +2,4% par rapport à l'année dernière à pareille époque tandis que le chiffre d'affaires du "développement immobilier" s'élève à 174,5 millions d'euros, en croissance de 79,9%.   STENTYS   Stentys a fait état, après la clôture des marchés, d'un chiffre d'affaires en forte progression, de 28% à 1,09 million d'euros pour le compte de son premier trimestre. Cette accélération de la croissance résulte du démarrage des ventes du stent auto-apposant actif à libération de Sirolimus, Stentys SES, qui a obtenu le marquage CE fin octobre 2014 et dont le déploiement géographique se met progressivement en place. STENTYS SES représente 68% des ventes sur la période alors que le produit qu'il remplace, le stent actif à élution de paclitaxel, ne représente plus que 16% des ventes.  Cet article est paru dans Option Finance (site web)
</article>
<article titre="Albioma : signature de l'acquisition de Codora Energia au Brésil" date="2015 04 16" journal="Boursier (site web)">
Albioma  a annoncé la signature de la documentation définitive en vue de l'acquisition au Brésil de 65% du capital de Codora Energia, propriétaire d'une usine de cogénération bagasse (48 MW) située dans l'État de Goiás. Mise en service en 2011, la centrale est adossée à une distillerie traitant aujourd'hui 1,6 million de tonnes de canne à sucre par an et détenue par le groupe sucrier Jalles Machado.    La centrale exporte actuellement 98 GWh sur le réseau électrique et bénéficie d'un contrat de vente d'électricité à long terme sécurisé jusqu'en 2026 pour l'exportation de 87 GWh/an au tarif de 205 reals/MWh, indexé sur l'inflation.  Albioma pense que l'amélioration de l'efficacité énergétique issue de son savoir-faire industriel, l'augmentation graduelle du volume de canne à sucre traité par la distillerie à 2,1 millions de tonnes par an et, à moyen terme, l'ajout d'une troisième turbine d'une puissance de 20 MW et la mobilisation de la paille de canne à sucre, devraient permettre de porter la quantité d'électricité exportée à plus de 170 GWh.    Albioma détiendra 65% du capital de Codora Energia, le groupe Jalles Machado conservant les 35 % restants. L'acquisition sera financée à 50% en dette locale et à 50% en fonds propres. Sa finalisation est soumise à des conditions suspensives devant être levées au second semestre de l'exercice 2015.
</article>
<article titre="Albioma : est éligible au PEA-PME" date="2015 04 1" journal="Boursier (site web)">
Albioma  confirme son éligibilité au PEA-PME pour l'exercice 2015. Les actions de la société peuvent donc continuer d'être logées dans ce support de placement dédié au financement des petites et moyennes entreprises (PME) et des entreprises de taille intermédiaire (ETI).   Albioma est un producteur d'énergie indépendant, leader mondial de la valorisation à haute efficacité énergétique de la biomasse en partenariat avec les agro-industriels. Depuis plus de 20 ans, Albioma exploite des centrales thermiques valorisant la bagasse, coproduit fibreux de la canne à sucre remplacé par le charbon hors campagne sucrière. Son expertise unique lui a permis de s'imposer comme une partenaire incontournable de l'industrie du sucre et de l'éthanol dans l'Outre-Mer et à l'Ile Maurice. Albioma développe désormais des centrales 100% biomasse qui valoriseront, aux côtés de la bagasse, des déchets vers ou des résidus de l'industrie du bois. En complément, le Groupe exploite un parc photovoltaïque performant et des unités de méthanisation agricole. Présent en France métropolitaine, dans l'Outre-Mer français et à l'IÎle Maurice, le Groupe est implanté depuis 2014 au Brésil, premier producteur mondial de canne à sucre. L'action Albioma se tient sur un cours de 17,8 euros. Elle gagne près de 8% depuis le 1er janvier 2015.
</article>
<article titre="Tarn. La CIMV veut s'installer sur la Zac des Portes du Tarn" date="2015 04 1" journal="Le Journal des entreprises">
La Compagnie Industrielle de la Matière Végétale (CIMV) devrait bientôt présenter un dossier au comité d'agrément de la Zac des Portes du Tarn pour y installer, en 2016, son unité de démonstration industrielle. La société francilienne, dont les équipes de R&amp;D se répartissent entre la Marne (où se situe le pilote de démonstration) et l'INP Toulouse, travaille sur une technologie innovante de bioraffinerie, permettant de valoriser les parties non-alimentaires des végétaux (paille de céréales, bagasse de canne à sucre, tige de maïs, etc.). Elle emploie 15 personnes.
</article>
<article titre="[1 re course 1. Best Perle (10), R. Derieux, G. 9,60 P. 2,40 ; 2. Broardo...]" date="2015 03 14" journal="Aujourd'hui en France">
1 re course  1. Best Perle (10),  R. Derieux, G. 9,60 P. 2,40 ;  2. Broardo (12),  C. Martens, P. 1,70 ;  3. Bently de Raconis (7),  N. Ensch, P. 2,80 ;  4. Brooklyn Fligny (9),  Y. Lacombe. Coup. gag. 14,70. Coup. pl. (10-12) : 5,30 (10-7) 12,50 (12-7) 8,00. Trio (10-12-7) : 48,80.    2 e course  1. Sirius de Buquet (13),  J. Guelpa, G. 5,30 P. 2,20 ;  2. Too Much Blue (8),  J.-M. Bazire, P. 2,30 ;  3. Tel Kiss (5),  R. Derieux, P. 12,70 ;  4. Pablo Di Jesolo (10),  R. Andreghetti.  5. Uttingeois (2),  R. Métayer. Coup. gag. 14,40. Coup. pl. (13-8) : 6,10 (13-5) 41,40 (8-5) 40,60. Trio (13-8-5) : 186,40.  PICK 5 (13-8-5-10-2) : 319,30. 3 e course  1. Chapeau (5),  C. Martens, G. 2,10 P. 1,50 ;  2. Potenza Om (3),  P. Gubellini, P. 3,20 ;  3. Turbo Jet (10),  N. Ensch, P. 4,20 ;  4. Midsummer King (1),  J.-C. Sorel. Coup. gag. 11,70. Coup. pl. (5-3) : 4,30 (5-10) 9,60 (3-10) 26,60. Trio (5-3-10) : 120,80.    4 e course  1. Azelie de Neuvy (3),  J. Guelpa, G. 3,60 P. 1,80 ;  2. Véloce d'Occagnes (15),  C. Martens, P. 1,30 ;  3. Vent d'Échal (9),  Y. Lacombe, P. 3,30 ;  4. Une Etoile Bleue (14),  A. Hureau. Coup. gag. 4,60. Coup. pl. (3-15) : 3,30 (3-9) 13,20 (15-9) 7,90. Trio (3-15-9) : 52,70.    5 e course  1. Cristal Wood (10),  J.Pier. Dubois, G. 2,60 P. 1,40 ;  2. Cash For Ever (4),  C. Martens, P. 1,60 ;  3. Cash du Ponthieu (3),  A.-A. Chavatte, P. 2,10 ;  4. Colibri du Vivier (9),  Y.-A. Briand. Coup. gag. 5,60. Coup. pl. (10-4) : 2,70 (10-3) 5,20 (4-3) 4,70. Trio (10-4-3) : 17,80. Tous Couru 3'45"51 (1'16"4).    6 e course  1. Brindille Darche (5),  N. Ensch, G. 5,70 P. 2,20 ;  2. Bagasse Darche (10),  J. Guelpa, P. 1,60 ;  3. Brescia Turgot (7),  Y.-A. Briand, P. 2,70 ;  4. Barbadine Passion (14),  C.-A. Mary. Coup. gag. 28,50. Coup. pl. (5-10) : 11,80 (5-7) 6,10 (10-7) 10,80. Trio (5-10-7) : 65,20.
</article>
<article titre="Courses" date="2015 03 14" journal="Le Parisien">
1 re course  1. Best Perle (10),  R. Derieux, G. 9,60 P. 2,40 ;  2. Broardo (12),  C. Martens, P. 1,70 ;  3. Bently de Raconis (7),  N. Ensch, P. 2,80 ;  4. Brooklyn Fligny (9),  Y. Lacombe. Coup. gag. 14,70. Coup. pl. (10-12) : 5,30 (10-7) 12,50 (12-7) 8. Trio (10-12-7) : 48,80.    2 e course  1. Sirius de Buquet (13),  J. Guelpa, G. 5,30 P. 2,20 ;  2. Too Much Blue (8),  J.-M. Bazire, P. 2,30 ;  3. Tel Kiss (5),  R. Derieux, P. 12,70 ;  4. Pablo Di Jesolo (10),  R. Andreghetti.  5. Uttingeois (2),  R. Métayer. Coup. gag. 14,40. Coup. pl. (13-8) : 6,10 (13-5) 41,40 (8-5) 40,60. Trio (13-8-5) : 186,40.  PICK 5 (13-8-5-10-2) : 319,30. 3 e course  1. Chapeau (5),  C. Martens, G. 2,10 P. 1,50 ;  2. Potenza Om (3),  P. Gubellini, P. 3,20 ;  3. Turbo Jet (10),  N. Ensch, P. 4,20 ;  4. Midsummer King (1),  J.-C. Sorel. Coup. gag. 11,70. Coup. pl. (5-3) : 4,30 (5-10) 9,60 (3-10) 26,60. Trio (5-3-10) : 120,80.    4 e course  1. Azelie de Neuvy (3),  J. Guelpa, G. 3,60 P. 1,80 ;  2. Véloce d'Occagnes (15),  C. Martens, P. 1,30 ;  3. Vent d'Échal (9),  Y. Lacombe, P. 3,30 ;  4. Une Etoile Bleue (14),  A. Hureau. Coup. gag. 4,60. Coup. pl. (3-15) : 3,30 (3-9) 13,20 (15-9) 7,90. Trio (3-15-9) : 52,70.    5 e course  1. Cristal Wood (10),  J.Pier. Dubois, G. 2,60 P. 1,40 ;  2. Cash For Ever (4),  C. Martens, P. 1,60 ;  3. Cash du Ponthieu (3),  A.-A. Chavatte, P. 2,10 ;  4. Colibri du Vivier (9),  Y.-A. Briand. Coup. gag. 5,60. Coup. pl. (10-4) : 2,70 (10-3) 5,20 (4-3) 4,70. Trio (10-4-3) : 17,80. Tous Couru 3'45"51 (1'16"4).    6 e course  1. Brindille Darche (5),  N. Ensch, G. 5,70 P. 2,20 ;  2. Bagasse Darche (10),  J. Guelpa, P. 1,60 ;  3. Brescia Turgot (7),  Y.-A. Briand, P. 2,70 ;  4. Barbadine Passion (14),  C.-A. Mary. Coup. gag. 28,50. Coup. pl. (5-10) : 11,80 (5-7) 6,10 (10-7) 10,80. Trio (5-10-7) : 65,20.    7 e course  1. Tonnerre d'Avril (4),  K. Fresneau, G. 2,90 P. 1,20 ;  2. Sacco de Blary (12),  V. Boudier-Cormy, P. 1,20 ;  3. Sire le Roi (8),  A. Unterreiner, P. 3,90 ;  4. Union de Malac (3),  S. Briand. Coup. gag. 2,10. Coup. pl. (4-12) : 1,40 (4-8) 10,60 (12-8) 10,20. Trio (4-12-8) : 21,40.
</article>
<article titre="HÉROS ÉROTIQUES" date="2015 03 11" journal="L'Express, no. 3323">
C'est un livre paillard et polisson. Provocateur et désopilant. Ah, si l'histoire de France nous était contée de la sorte, tout le monde deviendrait historien! Depuis neuf cents ans, on rabâche le destin d'Héloïse et Abélard de la plus chaste façon. On conserve des amants malheureux l'image pieuse qui fut figée dans le marbre de leur tombeau, aujourd'hui au Père-Lachaise. Et les lettres qu'ils s'échangèrent, au soir de leur vie, si magnifiques soient-elles, témoignent d'une passion contrariée devenue symbole de la pureté. Enfin... du côté d'Abélard. Parce qu'Héloïse, elle, évoque entre les lignes une autre forme d'amour. Dans son couvent, elle confie qu'au cours de la messe « des images obscènes assaillent [s]a pauvre âme » . Des images obscènes? Il n'en fallait pas davantage à Jean Teulé pour se glisser dans les blancs de l'Histoire. Eh, oui! Héloïse et Abélard se sont livrés sans retenue et avec obscénité aux jeux des amants pris dans les rets de la folie érotique. Jean Teulé n'a plus qu'à laisser libre cours à sa plume naturellement déjantée pour nous offrir le roman le plus hilarant de l'année. Pierre Abélard est une rockstar et le philosophe le plus célèbre de la chrétienté, en cette année 1113, lorsqu'il devient le professeur particulier de la très jeune et très belle Héloïse. La mignonne n'est pas une oie blanche. Coup de foudre. Et l'amour, entre eux, est loin d'être courtois. « Il polissonne la bagasse, bélute la donzelle » , laquelle ne peut passer une journée sans se faire « heurtebiller » par son amant, à qui elle fait en retour tout ce qu'il lui fait. Teulé, digne héritier du grand Frédéric Dard, s'en donne à coeur joie. La langue (française) s'évade, s'amuse, joue et jouit. On connaît la suite : l'oncle d'Héloïse fait châtrer Abélard, qui entrera dans les ordres et imposera à Héloïse un destin similaire, puis ne lui donnera plus le moindre signe de vie pendant deux ou trois décennies, avant d'écrire les plus belles lettres d'amour de l'Histoire. Entre-temps, on aura découvert le rôle capital des frères et soeurs portiers dans les monastères, bien rigolé et même médité sur cette liaison dangereuse. Hénaurme et rabelaisien, Teulé fait la nique au moralement correct. Jubilatoire!   Héloïse, ouille!,par Jean Teulé.Julliard, 350p., 20 euros.
</article>
<article titre="San Antonio au couvent des Oiseaux" date="2015 03 5" journal="Le Figaro, no. 21950">
DANS LA BIBLIOGRAPHIE de Jean Teulé, on relève les noms de Rimbaud, Verlaine et Villon. L'auteur a certainement partie liée avec la poésie.   Et si son nouveau roman se propose de raconter la belle histoire d'Héloïse et Abélard, c'est que durant sa fréquentation de maître François il a sans nul doute été ému par « la très sage Héloïse / pour qui châtré fut et puis moine / Pierre Esbaillart à Saint-Denis » ... En des pages admirables, le philosophe Étienne Gilson a placé leur amour sur un sommet. Héloïse et Abélard, ou l'humanité écartelée entre l'âme et le corps, la chair et l'esprit, la Terre et le Ciel. « Si ta présence m'est dérobée, que la tendresse de tes mots dont tu es si riche me rende du moins la douceur de ton image » : leur correspondance est un authentique chef-d'oeuvre de la littérature. On n'en dira pas autant du roman de Teulé. La Renaissance, les clercs, la bonne chère, les choses du sexe, Rabelais et tout le saint frusquin, il a vite décidé qu'à cette époque on s'en mettait jusque-là.   Pour faire bref, la passion dévorante de l'écolière de Notre-Dame et de son précepteur devient sous sa plume du sous-Frédéric Dard : San Antonio lâché dans le couvent des Oiseaux. On sait que, par amour pour Héloïse, Abélard renonça à une carrière à laquelle sa science et son brio le destinaient dans l'Église. Traduit par Teulé, cela donne : « Putain ! Le trône de saint Pierre s'éloigne. » Teulé appelle un chat un chat et Abélard un cochon. Obscénité, scatologie, la vulgarité est à toutes les pages, parfois dissimulée sous un vernis vieux françois : « Il la heurtebille à la sauvage (...). Il polissonne la bagasse, bélute la donzelle. » La ruse ne change rien : le grossier Jacquouille est devenu gribouille, et c'est grande pitié au royaume des lettres.   Pour qualifier les interminables scènes d'érotisme de pacotille que nous inflige Teulé, la formule « Cinquante nuances de Grey » vient à l'esprit, à ce détail près que jamais la nuance n'apparaît.   L'auteur épouse tous les clichés du beau conte des amants en butte aux vexations du monde entier. Abélard y laisse sa virilité. Si ce n'était pas suffisant, il doit faire face à l'hostilité des autorités intellectuelles. Sans compter que tous ces savants, théologiens et autres évêques sont forcément incultes, hypocrites et goinfres. Bernard de Clairvaux, le grand saint Bernard, est exécuté d'un mot : « hyène en soutane » . On ne chipotera pas sur le fait que le mot soutane date du XVIe siècle : Teulé - c'est son genre - joue avec les anachronismes, il meuble son histoire en ancien et en moderne. Abélard, qui fut tout de même tenu pour le Sénèque du moment, devient le « mec » d'Héloïse, elle qui ne l'appelait que « mon unique » .   Le titre du livre aurait dû nous mettre la puce à l'oreille : Héloïse, ouille ! L'onomatopée, c'est le degré zéro de l'écriture. L'auteur y a souvent recours pour décrire la vie de ses personnages : burps, hic, prout. De la poésie, disions-nous.   Laissons là le malheureux et revenons à Héloïse et Abélard dans le texte. Magnifique figure de la passion et du combat intérieur, la jeune femme écrivait à celui qui était devenu son mari : « Plaise au Ciel que ce que tu as souffert un moment dans ta chair, je le souffre moi aussi. » Le châtiment est arrivé, Teulé s'est emparé de leur histoire. Pour les amoureux aujourd'hui réunis au Père Lachaise, elle est là, la damnation éternelle.   Héloïse, ouille ! De Jean Teulé, Julliard, 336 p., 20 eur.
</article>
<article titre="Critique du livre «Héloïse, ouille !», par Etienne de Montety (Le Figaro)" date="2015 03 5" journal="Evene (site web)">
Héloïse et Abélard, ou l'humanité écartelée entre l'âme et le corps, la chair et l'esprit, la Terre et le Ciel. Leur correspondance est un authentique chef-d’œuvre de la littérature. On n'en dira pas autant du roman de Teulé. Pour faire bref, la passion dévorante de l'écolière de Notre-Dame et de son précepteur devient sous sa plume du sous-Frédéric Dard : San Antonio lâché dans le couvent des Oiseaux. Le renoncement d’Abélard à une brillante carrière dans l’Eglise par amour, cela donne : « Putain ! Le trône de saint Pierre s'éloigne. » Teulé appelle un chat un chat et Abélard un cochon. Obscénité, scatologie, la vulgarité est à toutes les pages, parfois dissimulée sous un vernis vieux françois : « Il la heurtebille à la sauvage (...). Il polissonne la bagasse, bélute la donzelle. » Sans compter que tous les savants, théologiens et autres évêques sont forcément incultes, hypocrites et goinfres. Bernard de Clairvaux, le grand saint Bernard, est exécuté d'un mot : « hyène en soutane ». On ne chipotera pas sur le fait que le mot soutane date du XVIe siècle : Teulé - c'est son genre - joue avec les anachronismes, il meuble son histoire en ancien et en moderne. Abélard, qui fut tout de même tenu pour le Sénèque du moment, devient le « mec » d'Héloïse, elle qui ne l'appelait que « mon unique ». Le titre du livre aurait dû nous mettre la puce à l'oreille : Héloïse, ouille ! L'onomatopée, c'est le degré zéro de l'écriture. L'auteur y a souvent recours pour décrire la vie de ses personnages : burps, hic, prout. De la poésie, disions-nous.
</article>
<article titre="Global Bioenergies : de l'isobutène produit avec des déchets végétaux" date="2015 03 3" journal="Boursier (site web)">
Global Bioenergies  a produit de l'isobutène de seconde génération afin de diversifier les matières premières utilisables dans son procédé Isobutène et pour se tourner vers les ressources les moins coûteuses.   Dans la phase de développement de son procédé de production d`isobutène bio-sourcé, Global Bioenergies a d`abord utilisé des matières premières de première génération, telles que le glucose dérivé du blé. Cependant, le procédé a été conçu pour être polyvalent : sous réserve d`ajustements techniques, il était considéré qu`il serait adapté à l`utilisation de ressources non-alimentaires, - définies au sens large comme des ressources de "seconde génération" -, telles que la paille de blé, les tiges de maïs, la bagasse de canne à sucre ou encore les copeaux de bois. De nombreuses sociétés travaillent à développer des technologies d`extraction des sucres fermentescibles emprisonnés dans les déchets végétaux. Certaines de ces technologies ont aujourd`hui atteint la maturité industrielle : cinq usines ont débuté leur production au cours des 24 derniers mois. Cette activité a, in fine, le potentiel d`alimenter les procédés de fermentation en sucres à bas coûts dérivés de ressources abondantes.   Global Bioenergies a récemment mis en place des collaborations avec neuf sociétés de trois continents différents, développant les technologies les plus prometteuses de conversion de ressources diverses (paille, bagasse, bois.) en sucres fermentescibles. Des tests préliminaires ont permis de produire avec succès de l`isobutène de seconde génération à l`échelle du laboratoire. Ainsi, le procédé a fonctionné avec des performances comparables à celles observées en utilisant du glucose dérivé du blé.
</article>
<article titre="Global Bioenergies: nouvelle avancée dans l'isobutène." date="2015 03 3" journal="Cercle Finance (site web)">
Global Bioenergies a annoncé mardi avoir réussi à produire de l'isobutène de 'seconde génération' à partir de déchets végétaux.   Le spécialiste de la conversion de ressources renouvelables en hydrocarbures explique avoir d'abord utilisé des matières premières de première génération, telles que le glucose dérivé du blé. L'entreprise, qui vise un procédé 'polyvalent', a estimé qu'il serait adapté à l'utilisation de ressources non-alimentaires, telles que la paille de blé, les tiges de maïs, la bagasse de canne à sucre ou encore les copeaux de bois.   Certaines de ces technologies ont aujourd'hui atteint la maturité industrielle, fait valoir Global Bioenergies, qui souligne que cinq usines ont débuté leur production au cours des 24 derniers mois.   L'isobutène est l'une des plus importantes briques élémentaires de la pétrochimie qui peut être convertie en carburants, plastiques.   Copyright (c) 2015 CercleFinance.com. Tous droits réservés.
</article>
<article titre="Flambe après le franchissement d'une nouvelle étape technologique" date="2015 03 3" journal="Trading Sat (site web)">
Global Bioenergies flambait de plus de 8% mardi (+8,6% à 35,4 euros vers 11h30) après avoir annoncé le franchissement d'un nouveau cap technologique dans le cadre du développement de son bioprocédé isobutène.   La société, qui a mis au point un procédé de conversion de ressources renouvelables en hydrocarbures par fermentation, a en effet réussi à produire de l'isobutène de "seconde génération", c'est-à-dire à partir de déchets végétaux (paille de blé, tiges de maïs, bagasse de canne à sucre...). La société a rappelé à cette occasion la nécessité de "diversifier les matières premières utilisables dans son procédé Isobutène" et de "se tourner vers les ressources les moins coûteuses".   Oddo Securities a confirmé mardi sa recommandation d' "Achat" et son objectif de cours de 44 euros sur Global Bioenergies après cette annonce.    Selon le broker, "le prochain défi pour la société pourrait être la conversion de déchets ménagers ou d'effluents industriels comme matière première pour les bioprocédés qu'elle a développés". En cas de succès, Global Bioenergies "pourrait s'affranchir des risques liés au coût des matières premières"; souligne-t-il.
</article>
<article titre="GLOBAL BIOENERGIES De la paille de blé pour produire des carburants" date="2015 03 3" journal="Investir.fr">
Global Bioenergies, chimiste vert spécialisé dans le développement de procédés de conversion de ressources renouvelables en hydrocarbures par fermentation a franchi un nouveau cap technologique. La société basée à Evry (Essone), qui produit habituellement son isobutène dit de « deuxième génération », c'est-à-dire « bio », à partir de glucose végétal (sucre de canne, betterave...), est parvenue cette fois à produire ce gaz avec des déchets agricoles (paille de blé, tiges de maïs, bagasse de canne à sucre...). Jusque-là, cet hydrocarbure, utilisé notamment dans la fabrication de carburants, de pneus ou de bouteilles en plastique, était obtenu par craquage du pétrole.   Le cabinet d'analyses Oddo Securities renouvelle son conseil d'achat sur la valeur pour viser 44 euros. L'action progresse actuellement de 8,62%, à 35,39 euros (+29% depuis le début de l'année). « Les matières premières comme le sucre, utilisé actuellement par la société, constituent une part importante du coût de revient des biocarburants produits par la société. Une hausse sensible et durable du prix d'achat de ces matières premières pourrait remettre en cause la rentabilité du bioprocédé concerné. En réussissant à convertir des déchets agricoles, la société réussit donc à limiter ce risque en étendant les performances de ses procédés à l'utilisation de matières premières disponibles en plus grande quantité et surtout à moindre coût », explique-t-il dans une note. Les avancées technologiques s'enchaînent pour Global Bioenergies. Il y a un mois, le chimiste vert annonçait le franchissement de la première étape-clé du projet BioMA+ qui porte sur la définition d'une filière de conversion de végétaux en acide méthacrylique, un composant essentiel des peintures acryliques, des vernis ou des ciments. Il est financé par le programme Investissements d'Avenir de l'Etat qui a accordé une enveloppe de 5,2 millions d'euros à un consortium réunissant Arkema et le CNRS autour de la société.
</article>
<article titre="Première production d'isobutène à partir de déchets végétaux" date="2015 03 3" journal="Trading Sat (site web)">
Global Bioenergies a annoncé mardi matin être parvenu à produire de l'isobutène de "seconde génération", c'est-à-dire à partir de déchets végétaux. La société; qui a mis au point un procédé de conversion de ressources renouvelables en hydrocarbures par fermentation, rappelle en effet la nécessité de "diversifier les matières premières utilisables dans son procédé Isobutène" et de "se tourner vers les ressources les moins coûteuses".   Dans la phase de développement de son procédé de production d'isobutène bio-sourcé, Global Bioenergies a d'abord utilisé des matières premières de première génération, telles que le glucose dérivé du blé. Mais certaines des technologies d'extraction des sucres fermentescibles emprisonnés dans les déchets végétaux ont désormais atteint la maturité industrielle.  Global Bioenergies a ainsi récemment mis en place des collaborations avec neuf sociétés de trois continents différents, développant les technologies les plus prometteuses de conversion de ressources diverses (paille, bagasse, bois.) en sucres fermentescibles.   Des tests préliminaires ont permis de produire avec succès de l'isobutène de seconde génération à l'échelle du laboratoire. Ainsi, le procédé a fonctionné avec des performances comparables à celles observées en utilisant du glucose dérivé du blé.   Frédéric Pâques, Directeur des Opérations chez Global Bioenergies commente : "Nous avons désormais démontré expérimentalement que notre procédé de production d'isobutène est compatible avec plusieurs ressources de seconde génération. L'utilisation de solutions de sucre brut est en général difficile, dans les procédés de fermentation classiques menant à un composé liquide, car l'accumulation des impuretés dans le milieu de culture complexifie les opérations de purification du produit. Notre procédé, basé sur la production d'un produit en phase gazeuse, répond à cette problématique et nous permettra d'utiliser les matières premières les moins chères."   Thomas Buhl, Directeur du Business Development chez Global Bioenergies conclut : "L'accès aux matières premières de seconde génération renforce la perspective que notre procédé Isobutène soit massivement utilisé pour la fabrication de carburants, tels que l'essence et le kérosène."
</article>
<article titre="L’emploi salarié marchand au 3e trimestre 2014" date="2015 03 2" journal="Rapports - Marketing - INSEE">
2 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... L’emploi salarié marchand au 3e trimestre 2014 La croissance de l’emploi s’accentue  u 3 e trimestre 2014, l’emploi salarié marchand poursuit sa progression à La Réunion (+ 1...    Pour voir le rapport :  http://www.insee.fr/fr/insee_regions/reunion/themes/insee-flash/re_inf_22/re_inf_22.pdf
</article>
<article titre="Conseil Albioma" date="2015 02 21" journal="Investir-Le Journal des finances, no. 2146">
Albioma, l'ex-Séchilienne-Sidec, est la société la plus importante par la taille, la plus profitable (de loin !) et la plus connue de nos lecteurs parmi notre panel de producteurs d'électricité verte, hors EDF et GDF Suez. Détenue à 42,5 % par le fonds d'investissement Apax, Albioma devrait annoncer le 4 mars un chiffre d'affaires de près de 350 millions d'euros, une marge d'Ebitda (excédent brut d'exploitation) d'environ 35 % et un bénéfice net de 33 à 35 millions pour 2014, contre 37 millions en 2013. Le groupe produit l'essentiel de son électricité outremer : à La Réunion et à Maurice, en Martinique, en Guadeloupe, en Guyane et, depuis l'année passée, au Brésil. Le parc d'Albioma représentait en 2013 une puissance installée de 700 MW, dont 627 MW pour les centrales thermiques situées à proximité de sucreries. Durant la récolte de canne à sucre, ces unités brûlent la bagasse, les résidus fibreux de la canne, pour produire leur électricité. Hors saison, elles recourent au charbon. Parallèlement, Albioma exploite des fermes solaires (pour 70 MW), notamment sur le toit d'un hypermarché de La Réunion, avec stockage de l'électricité, ainsi que deux unités de biométhanisation, entrées en service en 2013 dans les Deux-Sèvres pour un coût de 20 millions d'euros. Faute de pouvoir investir tous azimuts, le groupe a vendu ses parcs éoliens début 2013. Il s'est fixé un programme d'investissements de 1 milliard d'euros pour la décennie 2013-2023, dont 200 à 300 millions dans la méthanisation agricole, pour y atteindre une puissance installée de 40 à 50 MW.   Le Brésil pourrait recevoir 200 à 400 millions d'investissement. Ce pays est la cible privilégiée d'Albioma, par ses sucreries et ses prix de vente attractifs de l'électricité. De plus, les récoltes sucrières s'y effectuent toute l'année. Une première centrale a été acquise il y a près d'un an pour 43 millions, auxquels devaient s'ajouter 30 millions pour en accroître les capacités et la productivité. Une acquisition devrait ensuite être annoncée tous les douze à dix-huit mois.   Le groupe, par ailleurs, compte dépenser 180 millions pour une deuxième centrale (Galion 2) en Martinique, avec une mise en service en 2016. Elle devrait utiliser des déchets verts et non du charbon en complément de la bagasse, tout comme une petite unité prévue pour 2017-2018 et 80 millions d'euros à Marie-Galante.   Notre conseil Acheter La société est celle qui offre la rentabilité la plus élevée parmi les producteurs d'électricité cotés, un dividende assuré (ce qui n'est pas le cas des autres valeurs moyennes de ce dossier) et des perspectives prometteuses au Brésil. Objectif : 22 euros.
</article>
<article titre="La technique FAO-Thiaroye de transformation (FTT-Thiaroye) - Guide de Réalisation et d'utilisation" date="2015 02 17" journal="Rapports - Agroalimentaire - FAOUN">
80 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... Photo de couverture : FAO/Lars Helgeby Organisation des Nations Unies pour l’alimentation et l’agriculture Rome, 2014  Guide de réalisation et d’utilisation  La technique FAO-Thiaroye de transformation (FTT...    Pour voir le rapport :  http://www.fao.org/3/a-i4174f.pdf
</article>
<article titre="PROJET DE RECOMMANDATION POUR LA DEUXIÈME LECTURE relative à la position du Conseil en première lecture en vue de l'adoption de la directive du Parlement européen et du Conseil modifiant la directive 98/70/CE concernant la qualité de l'essence et des carburants diesel et modifiant la directive 2009/28/CE relative à la promotion de l'utilisation de l'énergie produite à partir de sources renouvelables" date="2015 02 16" journal="Rapports - Défense - Europe Securité défense">
71 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... PR1044004FR.doc PE544.412v01-00  FR Unie dans la diversité FR  PARLEMENT EUROPÉEN 2014 - 2019  Commission de l'environnement, de la santé publique et de la sécurité alimentaire  2012/0288(COD...    Pour voir le rapport :  http://www.europarl.europa.eu/meetdocs/2014_2019/documents/envi/pr/1044/1044004/1044004fr.pdf
</article>
<article titre="Soyons spiritueux La Bielle harmonie du rhum «brut de fût»" date="2015 02 14" journal="Libération">
En allant, lundi à Paris, au premier Salon de la gastronomie des outre-mer, on ne s'attend pas à convoquer le souvenir de Charb, assassiné le 7 janvier. Pourtant, on retrouve les traits épais de ses trognes impayables sur le stand du rhum Bielle, l'une des trois distilleries de Marie-Galante. Sur cette île, à une trentaine de kilomètres de la Guadeloupe, Charb avait dessiné une interprétation toute personnelle de la fabrication du rhum à domicile, avec culture de la canne à sucre dans la caisse du chat et distillation dans une cocotte-minute. Aujourd'hui, il reste son autodérision gouleyante dans ce salon festif qui embaume les acras de morue, le colombo, la muscade et la vanille. A 7 000 km du périph, on a débuté la récolte de canne à sucre, qui s'étale jusqu'en juin. Chez Bielle, on apprécie les variétés indigènes, comme les cannes rouge et grise qui aiment le sol calcaire de Marie-Galante. Leur teneur en sucre définit le prix d'achat des cannes auprès des planteurs. Bielle privilégie la récolte à la main - garantissant davantage de pureté que la coupe industrielle qui peut agréger de la terre et des morceaux de roche. Les cannes sont ensuite broyées et écrasées pour obtenir le vesou, un jus qui fermentera près de quarante-huit heures avant d'être distillé dans une chaudière à vapeur utilisant la bagasse, le résidu broyé des cannes.   Le rhum blanc agricole issu de cette cuite titre 75 degrés d'alcool, et il est ramené à 59 - une spécificité de Marie-Galante - grâce à l'ajout de  «l'eau du ciel»,  dit joliment Jacques Larrent. Ex-responsable des chais de vieillissement des cognacs Martell, il prodigue ses conseils pour l'élaboration des rhums vieux de chez Bielle :  «Je connaissais la maison pour leur avoir vendu des tonneaux d'occasion.»  Car le rhum vieux s'élabore dans des fûts de chêne américain, qui furent remplis de bourbon, ou français, où a séjourné du cognac ou du vin blanc doux. Le rhum vieux va s'affiner au minimum trois ans dans les tonneaux :  «On ne cherche pas à donner un goût de bois au rhum, mais un équilibre entre les différentes origines des fûts,  explique Larrent.  Sur le chêne français vont se développer des arômes de vanille et de miel tandis que sur les fûts ayant contenu du bourbon, on aura des notes de fruits secs et d'amandes grillées.»  On apprécie ces dernières flaveurs dans le rhum Bielle «brut de fût» (sans filtrage, ni ajout) 2007 (1), un millésime très équilibré, complet, avec une interminable longueur en bouche et des notes de fruits cuits au nez.   (1) Prix salon, 68 €. Points de vente sur www.rhumbielle.com
</article>
<article titre="Le moulin de Val d'or, comme au XVIIIe siècle" date="2015 02 14" journal="Le Journal de Montréal  final">
SAINTE-ANNE (Martinique) - SAINTE-ANNE (Martinique) | Vers la fin du XVIIIe siècle, lorsqu'elle fut édifiée, la version originale du moulin à canne à sucre de l'habitation Val d'or représentait un grand progrès technologique. En rebâtissant à l'identique cet élément du patrimoine, la communauté de communes du sud de la Martinique a apporté une contribution exceptionnelle à l'histoire sucrière de l'île.   Haut de 18 mètres, le moulin à étage, par sa prestance, détonne dans le paysage. Sur le côté, une rampe maçonnée permet de faire monter à l'étage les mulets que l'on attelle pour actionner le moulin. Ils tournent ainsi inlassablement, à l'abri du soleil et aussi d'éventuelles intempéries, à même la construction circulaire d'une largeur de 6 mètres. Quatre ouvertures donnent accès au coeur du moulin, un mécanisme constitué de trois rolles verticaux servant à écraser la canne à sucre. Une fois passée à deux reprises entre ces rolles, de la canne il ne reste que la bagasse. Cette matière sèche servait autrefois de combustible pour chauffer la batterie de cinq chaudières vers laquelle était acheminé le vesou, comme on appelle le jus de canne à sucre frais.    Selon Vincent Huyghes Belrose, historien attaché au Parc naturel régional de la Martinique, dans toute l'île, où l'on comptait à cette époque plusieurs centaines d'habitations sucrières, on n'a recensé que trois moulins à étage du même type que celui de Val d'or. Ce domaine se composait de 290 ha et, selon les périodes, on y a recensé de 70 à 100 esclaves.    À présent, les groupes des écoles viennent sur place suivre une leçon d'histoire pratique. Certains jours de la semaine, l'équipe d'animation formée par le Parc naturel régional de la Martinique accueille les visiteurs pour une démonstration du travail du moulin, dont les vedettes sont l'âne Zéphyr et le mulet Tinou.    Cet organisme a en effet réintroduit le travail avec ces deux races animales en créant au sein de la presqu'île de la Caravelle un atelier de formation d'âniers-muletiers. En plus d'apprendre à nourrir et à soigner ces bêtes, les candidats doivent ensuite faire travailler ces animaux de bâts.    DÉMONSTRATION    Au moulin de Val d'or, l'animation commence dans un carré de canne à sucre où un employé montre la façon de couper la canne, de débarrasser celle-ci de ses feuilles, puis de lier les tiges en fagots. L'âne bâté transporte ensuite la canne jusqu'au moulin où les animateurs se chargent de l'introduire entre les rolles dont le mécanisme est activé par la force du mulet attelé à l'étage.    Les anecdotes, puisées dans l'histoire par les soins de Vincent Belrose, et racontées par les animateurs, donnent une idée des valeurs qui avaient cours à l'époque du système esclavagiste. Une machette était toujours prête à proximité des rolles du moulin de façon à ce que le surveillant puisse trancher le bras de l'employé qui, par mégarde, se serait fait prendre par le mécanisme. D'ailleurs, plutôt que des hommes, c'était des femmes que l'on chargeait d'alimenter le moulin en canne. Si jamais il fallait leur trancher le bras, elles pouvaient en effet toujours servir sur le domaine comme reproductrices...    * * *    REPÈRES    . Le moulin de Val d'or est situé en bordure de la route menant du Marin à Sainte-Anne, dans le sud de la Martinique.    . Il s'agit du seul moulin à canne à sucre à étage actuellement fonctionnel dans toute la Caraïbe.    . L'animation du moulin est assurée par les soins du Parc naturel régional de la Martinique. Info : www.pnr-martinique.com    . Une bonne adresse : La Maison rousse, maison d'hôtes, à Fonds- Saint-Denis, en pleine nature audessus de Saint-Pierre. www.maisonrousse.com    . Air Transat, durant la saison hivernale, ainsi que Air Canada desservent la Martinique au départ de Montréal.    . Infos : www.lamartinique.ca
</article>
<article titre="Promouvoir l'Agroforesterie dans les Politiques Publiques. Guide pour les Décideurs" date="2015 02 9" journal="Rapports - Agroalimentaire - FAOUN">
50 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... © FAO 2015  I3182F/1/01.14  Document de travail sur l ’agroforesterie no.1  Promouvoir l’agroforesterie dans  les politiques publiques Guide pour les décideurs  L’agroforesterie est un système de...    Pour voir le rapport :  http://www.fao.org/3/a-i3182f.pdf
</article>
<article titre="SPECIAL CFIA 2015 Des applications pour tous les marchés" date="2015 02 1" journal="Process Alimentaire, no. 1322">
Si vous recherchez des solutions en matière de barquettes de plats cuisinés ou de viande (et poisson) rendez-vous en premier lieu Hall 2 et Hall 4. Parmi les nouveautés ou les solutions exposées en avant-première dans ce numéro figurent encore cette année un très grand nombre de barquettes soit éco-conçues, soit élaborées en plastique mono-matériau en vue d'intégrer les nouvelles consignes de tri des plastiques qui seront opérationnelles partout en France d'ici 2022. Barquettes en carton, en PLA ou en bagasse En premier lieu, le carton a la cote. Packartis, par exemple, propose une barquette thermoformée en complexe carton étanche pour un conditionnement sous atmosphère modifiée. Un peu plus loin dans le Hall 4, Smurfit Kappa dévoile une nouvelle version de sa barquette Evo Tray, constituée à plus de 70 % de carton ondulé. Son film intérieur est désormais plus facile à séparer de la barquette en vue du recyclage. Tandis que Pack Alim fait découvrir ses solutions en carton associées à un film plastique très fin. La nouveauté réside dans une barquette plate destinée, cette fois, à la charcuterie, au poisson ou au fromage en tranches par exemple.   La tendance biodégradable est aussi visible sur le salon. Agro Concept Embal lance ainsi des barquettes en pulpe de canne à sucre (bagasse) micro-ondables et résistantes à la chaleur et au froid. Tandis que Form'Plast, expose, entre autres, une barquette en acide polylactique (PLA) à base d'amidon de maïs intégrant une couche barrière 100 % biodégradable. Elle se destine au conditionnement sous atmosphère modifiée de viandes, poissons, plats cuisinés, etc.   Barquettes plastique mono-matériau Sur les matières plastiques, de nombreux efforts sont menés pour proposer des barquettes en mono-matériau et des films de scellage compatibles. Plusieurs solutions l'illustrent dans le Hall 4 : Europlastiques expose ainsi sa barquette Pellican injectée en mono-polypropylène, alternative aux barquettes thermoformées multicouches avec EVOH. Elle est pasteurisable et imprimable en in-moulding-labelling. Hall 2, Silver Plastics expose une innovation, l'AirPET Eco. Il s'agit de barquettes en mousse PET expansée qui disposent de propriétés barrière sans ajout de film intérieur. Elles résistent à la chaleur (jusqu'à 200 °C) et, surtout, s'intègrent dans le flux de PET des bouteilles plastiques. Knauf Industries présente, de son côté, son mono-matériau à base de polypropylène utilisable en extrusion comme en thermoformage pour le conditionnement de viande en barquettes sous atmosphère modifiée. Celles-ci pourront être recyclées dans le cadre du futur flux PP de l'extension des consignes de tri.   Films pour emballages mono-matériau Les fabricants de films s'adaptent aussi à ces évolutions en développant des solutions d'operculage pour barquettes mono-matériau. Klöckner Pentaplast dispose ainsi de films scellables sur du mono APET ou du mono-polypropylène. Dans le Hall 10 des équipements, Linpac Packaging expose également un film rétractable pour l'operculage de barquettes en PP, en polystyrène expansé (PSE) avec soudant en PET ou en rPET (avec soudant en PE). Atout supplémentaire : ce film est imprimable en surface.   Face à cette multiplicité de profils de barquettes, les fabricants de thermoformeuses et d'operculeuses trouvent des solutions pour faciliter la vie et réduire les coûts des fournisseurs de plats cuisinés.   Operculeuses et thermoformeuses polyvalentes Parmi les avancées majeures opérées sur les machines exposées figurent ainsi la flexibilité et la facilité de changements de formats. Voire, aussi, la polyvalence des matériaux à sceller sur une même machine. Le Hall 2 en témoigne notamment avec l'operculeuse GV4 Selvex d'Ixapack qui se décline en quatre tailles d'outillages et opercule des barquettes de différents matériaux avec trois soudures possibles. Sodima expose aussi une operculeuse Tecnovac polyvalente qui permet le conditionnement sous atmosphère modifiée de tous types de barquettes et de films. Elle peut même travailler de concert deux barquettes différentes. Un peu plus loin, Ulma Packaging expose sa TSA 680 fonctionnant à cadences élevées sur une large gamme d'outillages. Dans le Hall 4, cette fois, l'operculeuse O2 2 500 de Mecapack démontre aussi sa versatilité car elle peut embarquer jusqu'à quatre outillages différents. Avec un changement d'outil de soudure se faisant en deux minutes seulement. Sa thermoformeuse FS 910, elle, dispose d'un pack Ergonomie qui permet notamment de réduire les temps de changement de formats. Dans le Hall 5, enfin, Guelt illustre aussi l'accélération de la vitesse de changement de formats avec son OPP2000, entre autres, qui passe d'une taille de barquette à l'autre en moins d'une minute. Et ce, sans outillage.   Ouverture et refermeture faciles Toutes ces avancées servent également l'univers des produits laitiers, autre segment de marché particulièrement couvert par les exposants du Cfia. De nombreuses solutions concernent aussi les produits secs conditionnés en sachets et les produits de snacking salé et sucré. En la matière, par exemple, les barquettes et shakers à salades font florès. ANL Plastics avec son système Peelpaq ou encore CGL Pack avec son Clipseal Carton proposent dans le Hall 4 des solutions refermables sans couvercle pour les salades traiteurs. À quelques encablures, Dynaplast expose son Tusipack muni d'une fourchette clipsée sur le couvercle. Il comprend un pot et une coupelle réchauffable séparément pour des repas sur le pouce. Côté produits secs, de nombreuses entreprises proposent des solutions d'ensachage et d'étuyage standard et sur mesure. Avec des systèmes d'ouverture et de refermeture toujours plus pratiques comme en témoigne, par exemple, Etik Ouest et son Up'nPack qui fonctionne avec un système adhésif. Brodart Packaging propose, quant à lui, un sachet soudé quatre côtés pour produits pulvérulents, pâteux ou liquides refermable soit par zip, soit par un bouchon. Dans le domaine des sachets, l'une des autres grandes innovations est à retrouver Hall 4 sur le stand de Leygapack. Sa poche souple avec poignée devrait, en effet, susciter l'intérêt, notamment pour des activités BtoB. Cette alternative aux emballages rigides fait d'ailleurs partie des trois nominés aux Trophées de l'Innovation du Cfia 2015.   kermenier@editionsduboisbaudry.fr
</article>
<article titre="SPECIAL CFIA 2015 Packaging" date="2015 02 1" journal="Process Alimentaire, no. 1322">
Environnement Des boîtes en carton étanches et recyclables La gamme Greencan repose sur un concept d'emballage carton, écologique, composé entre 92 et 98 % de carton recyclé. 2 à 8 % de films haute barrière préservent les propriétés des produits. Can Packaging a, depuis deux ans, étendu sa gamme de façon à répondre à tous types de produits alimentaires (biscuits, snacks, farine, cacao, beurre, fromage, crème fraîche, crème glacée, fromage râpé, sel, etc.) : boîte carton étanche gerbable avec couvercle clipsable, boîte salière avec grille doseuse, boîte light en carton ultralégère, etc. Pour faire face à la demande, Can Packaging augmente sa capacité sur ses deux sites de production à Habsheim (68) et Ancenis (44). Can Packaging Hall 4 - Stand C18   Recyclabilité Film pour scellage sur mono-matériaux Spécialiste des films rigides de thermoformage de structure monocouche et multicouche, Klöckner Pentaplast développe actuellement des solutions de scellage sur monomatériaux. Ceci pour répondre à une demande croissante du marché pour des barquettes en mono-APET ou mono-polypropylène (en vue d'une meilleure recyclabilité). «Nous répondons aussi à d'autres exigences environnementales ou d'ordre réglementaire en proposant : des films expansés permettant une réduction du poids de l'emballage, des films à base de matériaux recyclés post-consommation ou postindustriel ou encore des solutions à base de ressources renouvelables», indique-t-on chez Klöckner Pentaplast   Klöckner Pentaplast Hall 4 - Stand D26   Anti-gaspillage Un sac rétractable absorbeur d'odeurs de confinement Le sac rétractable sous-vide Cryovac Freshness Plus FP3055 de Sealed Air apporte une fonctionnalité novatrice, celle d'absorber les odeurs de confinement liées à la viande fraîche. Ce sac fait disparaître les odeurs naturelles déplaisantes pouvant laisser croire au consommateur que le produit n'est pas frais alors qu'il l'est. Cela réduit juste les odeurs inutiles mais ne masque pas les odeurs liées au rancissement de la viande. Il permet aussi d'augmenter la durée de conservation des produits (sur les plans microbiologique et sensoriel) grâce au conditionnement sous vide et malgré une épaisseur du film réduite de 6 %. «Il prolonge les possibilités de vente de 8-9 à 10-12 jours pour les volailles entières et les découpes et sécurise la fraîcheur au terme de la durée de conservation pour la viande fraîche (boeuf, agneau et porc avec ou sans os)», indique Sealed Air. Le sachet est aussi muni d'une ouverture facile Cryovac Grip &amp; Tear. Objectifs poursuivis par le Cryovac Freshness Plus : réduire les invendus, augmenter la satisfaction consommateurs et contribuer à la réduction du gaspillage alimentaire.   Sealed Air Hall 4 - Stand B47/C46   Praticité Un présentoir à salami pour consommation fractionnée La société RPC Superfos a développé avec le norvégien Grilstad un emballage en plastique injecté pour le salami tranché. Celui-ci présente des propriétés barrière à l'oxygène et offre surtout un plus en matière de présentation : le pot se transforme en assiette de présentation une fois le couvercle enlevé. Autre atout : la consommation peut être fractionnée grâce au couvercle twist-off qui se revisse (en un clic) hermétiquement.   RPC Superfos Hall 4 - Stand A31/B36   Éco-conception Des barquettes barrière en PET expansé L'AirPET ECO de Silver Plastics est un emballage en mousse PET expansée qui répond à un grand nombre d'attentes du marché en matière d'éco-conception et de recyclabilité. Par sa nature expansée, son poids est 30 % moins élevé qu'une barquette en PET équivalente. Il peut, en outre, utiliser 100 % de rPET (PET recyclé). Il dispose de propriétés barrière (pour allonger la durée de vie des produits) sans ajout de film barrière supplémentaire. Sa recyclabilité est possible dans le circuit actuel de tri des déchets grâce à son caractère mono-matériau : rentre dans le flux des bouteilles plastique PET. Dans les versions airPET To Go, Touch et MAP, ce matériau résiste à la chaleur (de - 20 °C jusqu'à 200 °C pour le airPET touch) tout en conservant «un toucher frais». Cela signifie que le consommateur ne se brûle pas. Les applications sont nombreuses : viande, poisson, snacking froid ou chaud, plats préparés. Seul petit bémol : ce matériau, disponible sous forme de barquettes ou de pots, est opaque.   Silver Plastics Hall 2 - Stand A6/B7   Conservation Des films rétractables à propriétés multiples Bolloré présente sa gamme de films rétractables «Bolfresh» dédiés à l'emballage des viandes, volailles, fromages, fruits et légumes, produits traiteur, produits de la mer, etc. «Des films conçus pour apporter la protection, la conservation et la mise en valeur des produits frais et surgelés», indique le fabricant. La gamme comprend des solutions barrière, antibuée, étiro-rétractable, perforée ainsi que des films barrière Oxbtec pour le conditionnement flowpack et Lidtec pour l'operculage barrière. Ils se distinguent par leur brillance, leur transparence et leur taux de rétraction élevé. Autres propriétés : résistance à la soudure et à la perforation, étanchéité permettant d'éviter d'éventuels risques de contamination croisée, possibilité de mise sous film à des cadences élevées.   Bolloré Hall 4 - Stand A30   Praticité Un sachet multicouche rigide et stable Brodart Packaging propose un sachet soudé quatre côtés adapté aux produits pulvérulents, pâteux et/ou liquides. Son fond automatique permet un remplissage facile et une grande stabilité associée à une rigidité liée à la structure multicouche. La matière peut différer entre la face et les côtés (film métallisé, transparent, blanc, etc.). Il peut être équipé d'une ouverture facile, d'un zip ou d'un bouchon.   Brodart Packaging   Hall 4 - Stand B43   Sécurité Une mesure de fuite précise Aneolia expose les systèmes de détection de fuite de la marque Abiss. Le modèle Oxylos, par exemple, est un semi-automate permettant d'effectuer de façon sécurisée sur un même emballage, soit une analyse de gaz (O2 seul ou O2 et CO2), soit une mesure de fuites et de résistance des soudures, soit une combinaison de ces différents tests. Et ce, via la méthode de compensation en pression qui consiste à percer l'opercule via un septum et à appliquer une pression à l'ensemble des soudures. Avec ce système semi-automatique, l'opérateur ne manipule plus d'aiguille. Les performances de mesure de fuite atteignent un niveau de précision jusqu'à 0,1 ml/min à faible surpression (à partir de 20 mbar), soit l'équivalent de 5 µm, en quelques secondes. Aneolia profite du Cfia 2015 pour présenter sur l'Oxylos et l'Exos (son équivalent manuel) les premiers résultats des travaux menés pour augmenter l'automatisation d'un test complet (mesure de gaz + fuites + résistance des soudures) sur un même emballage.   Aneolia Hall 3 - Stand C29   Personnalisation Des opercules en impression numérique La société CB Emballage, distributeur de barquettes pour l'agroalimentaire, dévoile ses opercules en aluminium et en matières plastique imprimables en numérique. Une vraie innovation dans ce secteur qui offre tous les avantages du numérique : pas de frais de clichés d'impression, tirages économiques pour de petites séries (10 000 pièces minimum), une haute résolution, une flexibilité et des délais courts. De quoi ouvrir la voie de la personnalisation des opercules. C'est dans l'air du temps...   CB Emballage Hall 3 - Stand F17   Praticité Une barquette refermable sans couvercle 2015 Nominé   Clipseal Carton breveté par CGL Pack est un système qui permet de refermer une barquette sans recourir à un opercule repositionnable ni à un couvercle. La barquette fermée se présente scellée avec un opercule cartonné qui, une fois retiré, peut être replacé à la main par le consommateur. L'opercule est légèrement enfoncé dans la barquette et «câlé» par un contour intégré à la barquette. Les aliments sont ainsi protégés et peuvent être préservés lorsqu'ils ne sont pas consommés en une fois. Ce concept packaging peut se décliner dans différents formats et matériaux selon les utilisations requises.   CGL Pack Hall 4 - Stand A13/B14   Nomadisme Pot avec coupelle réchauffable Dynaplast présente son modèle Tusipack, operculé et muni d'une fourchette clipsée sur le couvercle. Composé d'un pot, d'une coupelle réchauffable au micro-ondes et d'un couvercle (simple ou couvercle permettant d'intégrer le scellage), ce concept-snacking est adapté aux préparations à emporter. Les pots sont empilables. Il existe en un seul diamètre mais en huit hauteurs, en polypropylène pour le chaud et polyéthylène pour le froid. Pour la communication entreprise, Dynaplast propose une étiquette positionnée sur le couvercle avec deux cavaliers qui redescendent sur les flancs du pot afin de conserver la transparence et la visibilité du contenu (pâtes et sauce).   Dynaplast Hall 4 - Stand C42   Service Boîtes pâtissières à charnière elliptique   2015 Nominé   La nouvelle boîte pâtissière de Gault et Frémont est élaborée en carton de haute rigidité. Son ouverture et sa fermeture sont facilitées par sa charnière brevetée en forme elliptique. Autres astuces : une encoche de fermeture située à l'avant permet de maintenir la boîte fermée pendant le transport tandis qu'un rebord abaissé sur le devant permet d'insérer plus facilement les pâtisseries. Le tout augmente la maniabilité et la tenue des boîtes pâtissières destinées aux boulangers et pâtissiers artisanaux.   Gault et Frémont Hall 4 - Stand A22   Éco-conception Une barquette en carton avec film «encliqueté» Smurfit Kappa fait évoluer son concept EvoTray, une barquette constituée à plus de 70 % de carton ondulé complétée par un film barrière intérieur. Originalité de la solution : ce film n'est pas collé mais fixé par encliquetage dans les encoches spécifiques de la structure en carton ondulé avec micro ou nano-canelures. La géométrie de la barquette a également été revue avec des coins arrondis pour permettre un meilleur mariage avec les outils de scellage habituellement utilisés pour l'operculage des barquettes plastique. A l'éco-conception s'ajoute une volonté d'apporter un atout marketing avec la possibilité d'imprimer la barquette en offset. «L'Evotray s'insère sans difficulté dans les lignes d'operculage existantes comprenant du dépilage et du convoyage», explique Smurfit Kappa.   Smurfit Kappa Hall 4 - Stand C15/D14   Praticité Sachet souple avec bouchon verseur 2015 Nominé   Le Leygapack PacXpert de Leygatech est une poche souple de 1 à 20 litres avec poignée et bouchon vissé permettant un stockage minime à vide (car il est stocké à plat) et une réduction des coûts de matière et de déchets. Il ne nécessite pas de suremballage et s'empile sur plusieurs couches grâce à la stabilité de sa base en quatre coins soudés. Il permet, en outre, de gagner jusqu'à 40 % de volume sur les palettes. L'écoulement des produits, même visqueux, est facilité jusqu'à la dernière goutte : de quoi emballer des produits liquides, pâteux et des poudres. Cette alternative aux emballages rigides occupe aussi peu de place dans les poubelles lorsqu'il est vide. Le PacXpert est remplissable à chaud et assure une conservation longue durée. Il est conçu en film coextrudé PE/PA (barrière aux arômes et à l'oxygène), existe en version transparente ou opaque et peut aussi être imprimé.   Leygatech Hall 4 - Stand B22   Service Des sachets cuisson minces en PET Sealed Air dévoile son sachet cuisson Cryovac Sealappeal OSF AW. Elaboré en film PET fin (le plus fin du marché selon Sealed Air), il garantit une durée de conservation identique à celle des autres solutions sous atmosphère protectrice. Son atout principal : permettre une cuisson directe au four traditionnel de volailles entières, rôtis, poissons pré-assaisonnés (tous les assaisonnements sont possibles) en réduisant les temps de cuisson. Le maintien des jus de cuisson à l'intérieur de l'emballage évite les projections et garantit la jutosité du produit final. Pour les industriels, le PET mince permet de réduire les arrêts machine pour changement de bobines grâce à des rouleaux plus longs. Avantage marketing, ce film est imprimable jusqu'à huit couleurs.   Sealed Air Hall 4 - Stand B47/C46   Polyvalence Opercule compatible avec plusieurs supports mono-matériaux Linpac Packaging lance un film rétractable haute barrière nouvelle génération, le Lintop PP HB F27. Transparent et disposant de propriété antibuée, il s'adapte à l'operculage de plusieurs supports mono-matériaux : barquettes en polypropylène, en polystyrène expansé (avec soudant en polyéthylène), barquettes en RPET (soudant PE). Conçu pour le conditionnement sous atmosphère modifiée d'une large gamme de produits alimentaires frais, ce film se destine aux operculeuses à découpe extérieure («outside cut»). Son taux moyen de rétraction donne un aspect net et tendu aux emballages. Il est disponible en bobines de grande longueur pour augmenter la productivité des lignes. Le scellage «lock seal» étanche, même sur des bords souillés, réduit les pertes et allonge la durée de conservation des produits. Le film est imprimable en surface, offrant des opportunités de communication et réduisant le besoin d'étiquettes supplémentaires.   Linpac Packaging Hall 10 - Stand C4   Éco-conception Des barquettes thermoformées en carton Packartis propose une barquette thermoformée en complexe carton étanche pour le conditionnement sous atmosphère modifiée. Une grande partie du plastique de cette barquette ThermoCart a été remplacée par une matière renouvelable, à savoir du carton provenant de la sylviculture certifiée PEFC. «En faisant des modifications minimales aux outils de thermoformage, on peut utiliser les mêmes machines d'emballage que pour les complexes APET/PE ou PVC/PE», assure-t-on chez Packartis. Les produits tranchés (saucisses, jambon cru, viande des Grisons, fromage, saumon fumé) sont «posés» de manière identique dans la barquette thermoformée, injectée de gaz et soudée avec un film supérieur étanche et antibuée.   Packartis Hall 4 - Stand A32   Biodégradabilité Barquette en PLA pour applications sous atmosphère modifiée Form'Plast expose, entre autres, une barquette en acide polylactique (PLA) à base d'amidon de maïs ou de sucre. 100 % biodégradable, elle se destine au conditionnement d'aliments (viande, poissons, plats cuisinés, fruits, etc.) sous atmosphère modifiée grâce à l'intégration d'une couche barrière également 100 % biodégradable. Le tout conserve une parfaite transparence et s'approche des caractéristiques mécaniques du PET grâce à une version de PLA modifiée avec des charges végétales. Laquelle permet de pallier le côté cassant du PLA.   Form'Plast Hall 2 - Stand D2   Recyclabilité Une barquette injectée en mono-polypropylène Europlastiques présente sa barquette Pellican injectée transparente en polypropylène. Elle apporte une solution mono-matériau en polypropylène (PP) alternative à des barquettes thermoformées multicouche avec EVOH. Ce qui est donc bien meilleur pour la recyclabilité. Autre atout : la possibilité de l'imprimer en In Moulding Labelling (IML). Elle cumule donc transparence et attractivité marketing via une impression de qualité, sans avoir à ajouter d'étiquette. Elle est aussi pasteurisable : intéressant pour de nombreuses applications. Pour être barrière à l'oxygène, cette barquette impose d'ajouter un film barrière Pellican qui est séparable après utilisation pour faciliter le recyclage de la barquette.   Europlastiques Hall 4 - Stand B35/C34   Recyclabilité Barquette operculable en mono-matériau Knauf Industries présente une nouvelle matière, le Kapseal, pour le conditionnement de la viande en barquettes individuelles sous atmosphère modifiée. Utilisable en extrusion comme en thermoformage, ce mono-matériau à base de polypropylène (PP) est 100 % recyclable. «Ce nouveau matériau donne des résultats comparables à ceux obtenus par les multicouches PET/PS, tant au niveau de l'évolution de l'atmosphère modifiée qu'au niveau de la flore microbienne. Avec l'avantage d'un bilan carbone bien meilleur», argumente-t-on chez Knauf Industries. Kapseal assure aussi une bonne qualité de scellage avec des opercules adaptés aux barquettes PP économiquement compétitifs. Ces barquettes déclinables en plusieurs couleurs (jaune, noir, bleu, blanc, etc.) pourront être recyclées dans le cadre du flux PP et PE de l'extension des consignes de tri.   Knauf Industrie   Hall 4 - Stand A23/B26   Écoconception Barquette plate en carton Après la boîte burger micro-ondable, la box avec fourchette intégrée, Pack Alim lance la Flat'Pack. Cette barquette plate a été conçue pour répondre aux besoins des rayons charcuterie et produits de la mer. La Flat'Pack apporte une surface de communication supplémentaire au dos de la barquette, grâce à la base en carton, 100 % imprimable (recto et verso). Cet emballage reprend le concept Pack Alim, à savoir : une base en carton pour la solidité et la surface de communication, associée à un film plastique très fin. Le film plastique est défini en fonction des besoins du produit (mise sous-vide, sous atmosphère protectrice, réchauffage au micro-ondes, perméabilité plus ou moins forte, rayon frais/surgelé, etc.). La forme et les dimensions de l'emballage sont également personnalisables.   Pack Alim Hall 2 - Stand E8   Conservation Un adhésif faisant office de couvercle Souvent les films ou les sachets filent à l'ouverture et ne permettent plus de conserver le produit dans son emballage. Ou alors au risque qu'il sèche. Pour y remédier, Etik Ouest propose l'Up'n Pack, un dispositif adhésif intégré au film d'emballage. Il définit une zone d'ouverture (les formes et dimensions sont adaptables) pour éviter les déchirures intempestives. L'ouverture se libère en tirant sur un onglet. La fermeture se fait simplement en replaçant le système à l'emplacement d'origine.   Etik Ouest Hall 4 - Stand B20   Alimentarité Boîte à pizza en carton ondulé alimentaire Emin Leydier présente une boîte à pizza Show Pizz' en carton ondulé fabriquée selon le procédé breveté Elytra basé sur la méthode HACCP. Celui-ci assure l'inertie de l'emballage et la non-modification des caractéristiques organoleptiques (goût/odeur) du produit alimentaire contenu. Ce processus est certifié EN 15 593, BRC IoP et FEFCO ESBO et permet de délivrer une déclaration d'aptitude au contact alimentaire direct (certificat d'alimentarité). Caractéristiques de cette boîte : une forte résistance liée à la microcannelure, une aptitude au contact alimentaire de tous les composés de l'emballage (colle, encre, vernis, etc.), une totale recyclabilité, un complexe papiers adapté aux produits humides et gras, une impression haute définition pour une meilleure différenciation.   Emin Leydier Hall 4 - Stand C19/D18   Attractivité Un pot en forme de demi-lune La société RPC Superfos a récemment été primée aux Worldstar Awards avec son seau en demi-lune en plastique mono-matériau, le HalfMoon. Par sa forme, ce conditionnement adapté aux produits alimentaires, aux huiles, au pet-food, etc., est facile à manipuler et à transporter (proche du corps, directement sous l'épaule). La taille standard du premier produit lancé dans cette nouvelle gamme est 396 mm x 240 mm pour un volume de 16,8 litres. Un 5,6 litres et d'autres contenances sont en développement selon RPC Superfos. Le fabricant est d'ailleurs prêt à étudier tout nouveau volume. Les décors sont réalisés en haute définition grâce au procédé d'In Moulding labelling (IML), de quoi garantir un bel impact en rayons. Le HalfMoon (demi-lune) est doté d'une anse plastique intégrée sur la collerette. Il est facile à empiler, dépiler et palettiser.   RPC Superfos Hall 4 - Stand A31/B36   Naturalité Le procédé Micvac débarque en France Technologie brevetée combinant cuisson micro-ondes et emballage spécifique, le procédé Micvac permet de préparer des aliments (entrées, plats, soupes) au rayon réfrigéré avec une DLC (date limite de conservation) de 30 à 60 jours sans conservateur. Son objectif : préserver la texture, la fraîcheur, les couleurs et les saveurs des aliments. Ce procédé économe en énergie, permet de limiter les pertes alimentaires et énergétiques. Pour le consommateur, la barquette, à chauffer au four à micro-ondes, est dotée sur son film de scellage d'une valve qui siffle lorsque la température est atteinte. Pour l'industriel, tous les ingrédients se mettent en barquette sous forme crue et cuisent en moins de 10 min, en continu. Les propriétés organoleptiques et nutritionnelles seraient mieux préservés que dans les procédés de cuisson classiques. Un lancement produit est annoncé en France en 2015.   Micvac Hall 4 - Stand D9
</article>
<article titre="LE TIC SIATIC" date="2015 02 1" journal="Zeste Cuisinons simple et bon, no. 18">
Pour s'attabler : Bol à soupe « collectionMaïko » , poterie artisanale en grès, coloris anthracite ou ivoire, Ø 12,5 cm x H 9,5 cm, (30 env., Les Guimards); Assiette plate « collection Les Ombrelles » , design de Sarah Lavoine, existe en petit ou grand modèle, et en 4 coloris (55 ou 68 La Compagnie de l'Orient et de la Chine, 107 Rivoli); Lot de 6 bols ronds « Wasara » by Shinikiro Ogata, en bagasse, résidu fibreux issu de la canne à sucre, biodégradable, à usage unique, rigide, convient pour le chaud et le froid, 500 ml, Ø 12,9 cm x H 6cm (9,90 les 6, Merci);Baguettes en bambou (9,90 les 5 paires, Ambiance &amp; Styles). Pour siroter : Bol avec bec verseur, réalisé à la main par Émilie Pedron, Ø 12 cm x H 7 cm (32 , Neëst); Bols en raku réalisés par Camille Campignion, noirs et blancs, Ø 11 cm x H 9,5 cm (44 /pièce chez Talents Étoile); Plateau « Quattro muri e due case » , en bambou, design de Michele de Lucchi, 46,6 cm x 36,6 cm (80 , Alessi); Bloc de 72 sets de table « Petits papiers » , en papier imprimé avec des encres végétales, façon papier reliure ou papier cadeau japonais, 25 x 45 cm, (44 le bloc de 72 sets détachables, Caravane).   Pour mitonner : Cuit-vapeur multifonction, paniers en acier inox satiné 2 x 5,5 L et 1 bol de 2 L, 4 programmes de cuisson allant de 40 °C pour les yaourts à 100 °C pour les viandes et légumes, 1 900 W. (249,90 Magimix chez Ambiance &amp; Styles); Cuiseur à riz et céréales, cuisson précise et automatique, permet de cuire à la vapeur et de mijoter, cuve antiadhérente, fonction maintien au chaud, capacité 6 ou 12 personnes, garanti 3 ans (69,90 ou 89,90 Cuisinart chez Culinarion); Wok en fonte, 2 anses, fourni avec couvercle en verre, spatule, baguettes et demi-grille, compatible tous feux et induction, garanti à vie, Ø 30 cm (99 Staub chez Culinarion).
</article>
<article titre="L'AVANCÉE Les entreprises ont les solutions Si les gouvernements peinent à trouver un accord, les entreprises sont de plus en plus nombreuses à orienter leur stratégie vers un monde décarboné. De l'énergie aux bâtiments en passant par les transports, l'industrie et l'agriculture, des solutions s'imposent tandis que les innovations se poursuivent, comme nous le décrivons ci-dessous et dans les pages suivantes. Plusieurs événements, programmés en amont de la COP21, permettront aussi de les découvrir : après le Business and Climate Summit des 20 et 21 mai à l'Unesco et le salon World Efficiency du 13 au 15 octobre Porte de Versailles, Solutions COP21 s'exposera au Grand Palais, à partir du 4 décembre. LES ENTREPRISES ONT LES SOLUTIONS CLIMATO-INTELLIGENCE L'agriculture en première ligne" date="2015 01 23" journal="La Tribune Hebdomadaire, no. No 116">
Des champs inondés qui ne peuvent plus être cultivés, des récoltes dont les rendements et les qualités nutritives diminuent : l'agriculture paiera le prix fort du réchauffement climatique. Elle est même parmi les premières activités humaines à être affectées. La situation est ironique, puisque, selon les dernières données du Giec, la production agricole, mais aussi la surexploitation de la forêt et d'autres usages des ressources terrestres, sont responsables d'un quart des émissions. Autant dire que le potentiel de réduction est grand, sans parler d'un autre enjeu planétaire : nourrir quelque 10 milliards d'humains en 2050... Les initiatives se multiplient, d'abord pour changer les pratiques, ne serait-ce que pour mieux utiliser la terre et préserver une autre richesse, l'eau, et ensuite, pour tirer parti de certains effets de l'activité agricole, avec, entre autres, la production de biomasse. Ainsi, les images satellite servent à gérer au plus près les besoins, tandis que les engrais épandus sont (parfois) naturels. De même, l'arrosage est dosé, voire envoyé en goutte à goutte aux plantes. Enfin, certaines pratiques néfastes, comme le labourage, source de perturbation du sol et de libération de carbone dans l'atmosphère, sont abandonnées... Bref, émerge aujourd'hui ce qu'il est convenu d'appeler l'agriculture « climato-intelligente ».   Quant aux innovations liées à l'agriculture ou la forêt, elles sont légion. Ainsi, des unités de production d'électricité et de chaleur à partir de biomasse sortent de terre un peu partout. De plus, on torréfie désormais la biomasse, pour la transformer en « biochar », sorte de charbon de bois qui, comme la capture du carbone, permet de soustraire du dioxyde de carbone de l'atmosphère. Enfin, si le bois, mais aussi la paille ou le fourrage sont les plus utilisés en Europe, c'est la bagasse de canne à sucre qui sert à la biomasse au Brésil et la paille de riz en Asie, le tout pouvant aussi alimenter un gazogène (la biomasse étant alors convertie en gaz naturel de synthèse). Autre casse-tête de la production agricole, la libération par les vaches de méthane - gaz qui se classe juste derrière le dioxyde de carbone en effet de serre. S'il faudrait évidemment consommer moins de viande, les vaches laitières peuvent, elles, être nourries de façon à ce qu'elles produisent moins de méthane. Des expériences, en cours au Canada, notamment, font état de progrès si des graines de tournesol remplacent la nourriture habituelle ou si de la levure y est ajoutée, afin d'améliorer le processus de digestion des animaux...
</article>
<article titre="Position du Conseil en première lecture en vue de l'adoption de la DIRECTIVE DU PARLEMENT EUROPÉEN ET DU CONSEIL modifiant la directive 98/70/CE concernant la qualité de l'essence et des carburants diesel et modifiant la directive 2009/28/CE relative à la promotion de l'utilisation de l'énergie produite à partir de sources renouvelables - Adoptée par le Conseil le 9 décembre 2014" date="2015 01 13" journal="Rapports - Défense - Europe Securité défense">
74 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... 10710/2/14 REV 2 EB/vvs  DGE 2 FR  Conseil de  l'Union européenne  Bruxelles, le 10 décembre 2014  (OR. en)  10710/2/14  REV 2  ENER 273  ENV 569...    Pour voir le rapport :  http://www.europarl.europa.eu/meetdocs/2014_2019/documents/cls/cons_cons(2014)10710(rev2)_/cons_cons(2014)10710(rev2)_fr.pdf
</article>
<article titre="S'éclairer à la canne à sucre" date="2014 12 22" journal="L'Est Républicain  VOM ; Franche-Comté ; Lorraine">
Besançon. « À terme, on pourrait très bien imaginer des cellules transparentes qui, intégrées aux vitres des bâtiments, généreraient de l'électricité à partir des infrarouges et permettraient ainsi d'alimenter certains appareils », explique Jérôme Husson. « Ceci dit, on n'en est pas encore là... »   Certes. Mais la dernière trouvaille de l'enseignant-chercheur et de son équipe de l'institut Utinam va dans cette direction. L'eurêka qui vient de valoir au Dr Jérôme Husson, au Dr Jérémy Dehaudt et au Pr Laurent Guyard d'être lauréats du Prix des Techniques innovantes pour l'environnement au salon Pollutec 2014 concerne en effet de nouveaux sensibilisateurs pour cellules photovoltaïques.   Mais aussi à la rafle de maïs, à la sciure, à la paille... Si 80 % des panneaux solaires sont équipés de cellules à base de silicium, d'autres voies sont développées, dont les cellules à colorant, ou cellules Grätzel (du nom de leur inventeur Allemand travaillant en Suisse).   Une technologie émergente qui permet en effet d'optimiser la captation de l'énergie solaire en cas de temps couvert mais aussi de réaliser des panneaux translucides, voire souples.   Reste que pour fonctionner, les molécules en question doivent être « habillées » d'un colorant. Or, jusqu'à présent, la fabrication de l'un des colorants les plus performants, le Black-Dye, s'avérait très polluante. Avec 336 g de déchets par gramme produit.   Après s'être penchés sur le sujet et avoir agité méninges et cornues, les chimistes bisontins ont trouvé comment passer sous la barre des 100 g de déchets.   Qui plus est sans utiliser de sels de chrome 6 (jusqu'alors incontournables bien que cancérigènes), mais en développant une solution à base de furfural, obtenu à partir d'éléments végétaux renouvelables tels que la bagasse de canne à sucre, les rafles de maïs, la sciure, la paille... « Le furfural est une vieille chimie qui servait notamment à réaliser des moules en fonderie », indique Jérôme Husson.   Fruit de six années de recherche, le « colorant bio » mis au point par les chimistes de l'institut Utinam ne servira pas seulement aux panneaux photovoltaïques dotés de cellules solaires à colorant (qui intéressent déjà bigrement les architectes et dont on peut voir un exemple de 300m2 d'installation à Lausanne).   Molécules anticancéreuses Si le prix du salon Pollutec 2014 a offert une belle vitrine aux chimistes de l'institut Utinam (qui ont déjà été contactés par une société allemande intéressée par leurs travaux), ils envisagent déjà d'autres prolongements à leur découverte.   Ainsi explorent-ils actuellement la catalysation du dioxyde de carbone en vue de sa valorisation ; l'utilisation de leur procédé comme fixateur de métaux afin de dépolluer les eaux usées ; ou encore, en lien avec des collègues de Dijon, la mise au point de cellules anticancéreuses.   Pierre LAURENT
</article>
<article titre="Albioma mise 170 M euros sur la plus grande centrale biomasse d'Outre-mer" date="2014 12 16" journal="GreenUnivers (site web)">
2014-12-19T19:18:04+00:00    Albioma poursuit son recentrage vers la biomasse, après avoir bouclé sa première acquisition au Brésil en début d'année, à savoir une centrale de co-génération de 60 MW adossée à une exploitation sucrière. Le producteur indépendant d'énergie, ex-Séchilienne Sedec, a signé début décembre son avenant tarifaire avec EDF pour une centrale biomasse de 40 MW située à côté de la sucrerie du Galion en Martinique. Il s'agit de la plus grande centrale biomasse de l'Outre-mer français.   Un investissement de 170 millions d'euros  Elle utilisera la bagasse (le résidu fibreux de la canne à sucre) et fournira de la vapeur, qui devrait représenter à terme près de 40% de l'énergie consommée par la sucrerie du Galion. Des granulés de bois en provenance d'Amérique du Nord et du Brésil compléteront l'apport en biomasse. Albioma prévoit également de développer de nouvelles filières biomasses à partir d'autres déchets de la canne à sucre. L'investissement prévu est à hauteur de 170 millions d'euros, avec une dette projet à long terme d'environ 120 millions d'euros. Le projet est porté par Albioma Galion, filiale à 80% du producteur d'énergie et à 20% de la compagnie financière européenne de prises de participation (Cofepp), le partenaire historique d'Albioma aux Antilles.   Démarrage en 2017  L'installation devrait démarrer au premier semestre 2017, avec un contrat de 30 ans à compter de la mise en service industrielle. Acteur historique dans la production de l'électricité Outre-mer, Albioma a progressivement délaissée le charbon au profit de la biomasse-énergie, la méthanisation et le solaire et a définitivement adopté ces axes de développement en 2013. Le modèle de l'entreprise, qui a fait ses preuves à l'île Maurice, repose sur un partenariat avec les sucriers pour valoriser la bagasse. En 2014, Albioma revendique une puissance installée cumulée de 700 MW, essentiellement composée de centrales thermique 100% biomasse.   Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="Albioma: accord pour la construction d'une centrale biomasse" date="2014 12 15" journal="Votre argent (site web)">
Albioma a annoncé ce lundi matin la signature du contrat bagasse/biomasse avec EDF pour la construction de la centrale Galion 2 en Martinique, structure 100% biomasse la plus importante de l'outre-mer français. L'avenant tarifaire a été signé le 5 décembre 2014 avec l'électricien français après validation par la Commission de Régulation de l'Energie (CRE).   D'une puissance de 40 mégawatts (MW), l'installation, implantée à côté de la sucrerie du Galion, à laquelle elle fournira de la vapeur, utilisera la bagasse produite par cette dernière. Elle permettra de valoriser efficacement les ressources en biomasse locale et contribuera à la réduction de la dépendance de la Martinique aux énergies primaires importées, produisant 15% des besoins électriques de l'île sous forme d'énergie renouvelable de base.    Le groupe chimique développera également de nouvelles filières qui permettront de mobiliser localement d'autres formes de biomasse : la paille de canne à sucre, la partie non valorisée de la bagasse de distilleries, les taillis sous futaie forestière, la fraction non-fermentescible des déchets verts et les plantes énergétiques cultivées sans conflit d'usages.    À terme, la biomasse locale devrait représenter près de 40 % des approvisionnements de la centrale.   Le complément de biomasse sera importé sous forme de granulés de bois d`Amérique du Nord et du Brésil. Le démarrage de l'installation est désormais prévu au premier semestre 2017 et le contrat a cours sur une durée de trente ans à compter de la mise en service industrielle.   De l'ordre de 170 millions d`euros, l'investissement sera porté par Albioma Galion, filiale à 80 % d'Albioma et à 20 % de COFEPP, son partenaire historique aux Antilles. Il sera financé par une dette projet à long terme d'environ 120 millions d'euros.   Copyright (c) 2014 CercleFinance.com. Tous droits réservés.
</article>
<article titre="ALBIOMA : feu vert pour Galion 2, plus grande centrale 100% biomasse de l'Outre-mer français" date="2014 12 15" journal="Option Finance (site web)">
AOF - 15 décembre 2014   (AOF) - L'avenant tarifaire bagasse/biomasse pour la centrale Galion 2 en Martinique a été signé le 5 décembre 2014 avec EDF, après validation par la Commission de Régulation de l'Énergie. D'une puissance de 40 MW, l'installation, implantée à côté de la sucrerie du Galion à laquelle elle fournira de la vapeur, utilisera la bagasse produite par cette dernière.    Albioma développera également de nouvelles filières qui permettront de mobiliser localement d'autres formes de biomasse : la paille de canne à sucre, la partie non-valorisée de la bagasse de distilleries, les taillis sous futaie forestière, la fraction non-fermentescible des déchets verts et les plantes énergétiques cultivées sans conflit d'usages. À terme, la biomasse locale devrait représenter près de 40 % des approvisionnements de la centrale. Le complément de biomasse sera importé sous forme de granulés de bois d'Amérique du Nord et du Brésil. Le démarrage de l'installation est désormais prévu au premier semestre 2017. Le contrat a une durée de 30 ans à compter de la mise en service industrielle.   Cette centrale 100 % biomasse permettra de valoriser efficacement les ressources en biomasse locale et sera à l'origine de la création de nombreux emplois dans les filières d'approvisionnement à développer. Elle contribuera à la réduction de la dépendance de la Martinique aux énergies fossiles importées : l'installation produira 15 % des besoins en électricité de l'île sous forme d'énergie renouvelable de base. Ce projet majeur s'inscrit ainsi dans les orientations définies dans le Schéma Régional Climat Air Énergie de la Région Martinique et devient un des piliers de la mise en oeuvre de la loi sur la transition énergétique.   L'investissement, de l'ordre de 170 millions d'euros, sera porté par Albioma Galion, filiale à 80 % d'Albioma et à 20 % de COFEPP, son partenaire historique aux Antilles. Il sera financé par une dette projet à long terme d'environ 120 millions d'euros.   Jacques Pétry, Président-Directeur Général d'Albioma, déclare : « Cet accord avec notre partenaire EDF est une étape historique de la mise en oeuvre de la stratégie biomasse d'Albioma. Avec détermination, nous avons développé ce projet innovant grâce au soutien d'EDF, des pouvoirs publics et de la Région Martinique. En complément de la bagasse, ressource énergétique stratégique de l'Outre-Mer français, les autres formes de biomasse locale et importée permettront de produire de l'électricité de base, 100 % renouvelable, sans empreinte carbone significative et à un prix compétitif. »  Cet article est paru dans Option Finance (site web)
</article>
<article titre="Albioma : feu vert pour la centrale biomasse Galion 2" date="2014 12 15" journal="Boursier (site web)">
Albioma  indique ce matin que l'avenant tarifaire bagasse/biomasse pour la centrale Galion 2 en Martinique a été signé le 5 décembre avec  EDF , après validation par la Commission de Régulation de l'Énergie. D'une puissance de 40 MW, l'installation, implantée à côté de la sucrerie du Galion à laquelle elle fournira de la vapeur, utilisera la bagasse produite par cette dernière. Albioma développera également de nouvelles filières qui permettront de mobiliser localement d'autres formes de biomasse : la paille de canne à sucre, la partie non-valorisée de la bagasse de distilleries, les taillis sous futaie forestière, la fraction non-fermentescible des déchets verts et les plantes énergétiques cultivées sans conflit d'usages. À terme, la biomasse locale devrait représenter près de 40 % des approvisionnements de la centrale.   Le complément de biomasse sera importé sous forme de granulés de bois d'Amérique du Nord et du Brésil. Le démarrage de l'installation est désormais prévu au premier semestre 2017. Le contrat a une durée de 30 ans à compter de la mise en service industrielle.   L'investissement, de l'ordre de 170 millions d'euros, sera porté par Albioma Galion, filiale à 80 % d'Albioma et à 20 % de COFEPP, son partenaire historique aux Antilles. Il sera financé par une dette projet à long terme d'environ 120 millions d'euros.
</article>
<article titre="Les valeurs à suivre aujourd'hui à la Bourse de Paris - Lundi 15 décembre 2014" date="2014 12 15" journal="Option Finance (site web)">
AOF - 15 décembre 2014   (AOF) -    AEROPORTS DE PARIS   L'opérateur des aéroports de région parisienne publiera avant la séance son trafic de novembre. ALBIOMA   L'avenant tarifaire bagasse/biomasse pour la centrale Galion 2 en Martinique a été signé le 5 décembre 2014 avec EDF, après validation par la Commission de Régulation de l'Énergie. D'une puissance de 40 MW, l'installation, implantée à côté de la sucrerie du Galion à laquelle elle fournira de la vapeur, utilisera la bagasse produite par cette dernière.    AIRBUS GROUP   Selon une source proche du dossier, Airbus a reprogrammé a livraison de son premier A350 à Qatar Airways. La compagnie avait annoncé la suspension "sine die" mercredi de la livraison de l'appareil initialement prévue le 13 décembre. Le responsable du programme long-courrier avait ensuite indiqué qu'Airbus devait régler quelques "problèmes très mineurs". Autre élément permettant d'aller dans le sens d'une livraison avant la fin de l'année, le maintien du premier vol commercial entre Doha et Francfort le 15 janvier, annoncé par le directeur général de la compagnie, réputé pour son exigence.   AREVA   Le commissariat à l'énergie atomique et aux énergies alternatives va céder une nouvelle tranche de sa participation dans Areva à l'Etat français selon un arrêté ministériel publié vendredi au Journal officiel. "Le CEA est autorisé à céder à l'Etat 27.412.875 actions Areva, représentant 7,15% du capital de la société Areva, pour un prix de cession égal à 334 300 010,63 euros", soit l'équivalent de 12,19 euros l'action, indique l'arrêté des ministres des Finances et de l'Economie. La date de l'opération n'a pas été précisée.   CGG    CGG a dit prendre acte de la décision de Technip de ne pas déposer une offre publique d'achat sur son capital. A la suite de l'offre non sollicitée de Technip en date du 10 novembre, CGG assure être resté ouvert au dialogue. La société de services pétroliers dit avoir étudié toutes les propositions de Technip au regard des intérêts de ses actionnaires, de ses clients et de ses salariés. Le conseil d'administration de CGG a considéré qu'aucune des alternatives proposées ne permettait de créer de la valeur pour l'entreprise et l'ensemble de ses parties prenantes.   FIMALAC   Après approbation de leur Conseil d'administration, Fimalac et Hearst ont conclu un protocole d'accord qui conduira à la cession par le premier de la cession de 30 % de Fitch Group au second. Le prix provisoire de cette transaction a été fixé à 1,965 milliard de dollars pour 30% des actions, ce qui correspond à une valeur nette globale de 6,55 milliards de dollars pour l'agence de notation. Ce montant représente près de 7 fois le chiffre d'affaires 2013 (981,7 millions de dollars) et environ 32 fois le bénéfice net 2013 (205,5 millions de dollars).   PEUGEOT   Selon le quotidien économique Les Echos, Quatre sites de PSA sont mis en concurrence pour l'attribution d'un volume de production de 200 000 moteurs essence de dernière génération ,"EB Turbo Puretech", ce qui pourrait conduire le constructeur automobile à produire pour la première fois hors de France. Selon les informations du journal, outre la Française de Mécanique à Douvrin (Nord), qui produit déjà ce moteur, on trouve parmi les candidats l'usine de Trémery en Moselle, mais aussi le site espagnol de Vigo, ainsi que le site slovaque de Trnava.   ONXEO   Onxeo a annoncé vendredi dans un communiqué le succès de son augmentation de capital qui lui a permis de lever 40,7 millions d'euros, après exercice intégral de la clause d'extension. Lancée le 17 novembre, l'opération, qui s'est achevée le 3 décembre, a été sursouscrite avec une demande plus de deux fois supérieure à l'offre, a indiqué Onxeo. "Cette augmentation de capital va nous permettre d'accélérer et d'élargir nos programmes de développement, en continuité avec l'étape charnière réalisée cette année suite à la fusion avec Topotarget" a ajouté le groupe.   PLASTIVALOIRE   Plastivaloire a signé le 12 décembre 2014 un protocole d'accord pour l'acquisition de 100% du capital du plasturgiste allemand Karl Hess. Cette implantation en Allemagne est parfaitement en ligne avec le plan stratégique du groupe de plasturgie français qui vise à devenir l'un des leaders européens du secteur. Le groupe renforcera ainsi sa position sur l'ensemble des marchés européens majeurs, et accédera plus facilement aux grands donneurs d'ordre allemands.    TECHNIP   Technip a conclu un accord avec Air Liquide Global E&amp;C Solutions Germany, portant sur l'acquisition de l'ensemble de ses activités liées aux technologies Zimmer destinées à la production de polymères. Basé à Francfort, en Allemagne, cela comprend les technologies de traitement des polyesters et des polyamides, les installations de recherche et de développement, ainsi qu'une équipe composée d'une quarantaine d'ingénieurs, de chercheurs et de responsables de projet. La transaction, dont le montant n'a pas été communiqué, devrait être conclue à la fin de cette année.   TIVOLY   Le 12 décembre 2014, dans le cadre d'une opération d'acquisition, hors marché, la société Holding Tivoly a acquis, auprès de la société 2RBI, qui accompagnait la société Tivoly depuis plus de vingt ans, les 15% du capital social (83 100 actions) qu'elle détenait dans la société Tivoly. Cette opération a été réalisée sur la base d'une valorisation unitaire de l'action du spécialiste des outils de coupe et de produits et services associés à 39,71 euros, supérieure au cours de l'action qui s'élève à 30 euros à la date de la cession.  Cet article est paru dans Option Finance (site web)
</article>
<article titre="L’ENVIRONNEMENT EN FRANCE Les grandes tendances" date="2014 12 3" journal="Rapports - Environnement - MEDDTL">
27 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... L’ENVIRONNEMENT EN FRANCE Les grandes tendances  Ministère de l’Écologie, du Développement durable et de l’Énergie  www.developpement-durable.gouv.fr Sommaire  À l’approche de la conférence Paris...    Pour voir le rapport :  http://www.developpement-durable.gouv.fr/IMG/pdf/L_environnement_en_France_-_Edition_2014_-_Synthese.pdf
</article>
<article titre="Open Agrifood met à l'honneur les bonnes pratiques sociales" date="2014 12 1" journal="Process Alimentaire, no. 1320">
Open Agrifood, qui s'est tenu à Orléans fin octobre, est née d'une discussion entre Xavier Beulin, président de Sofiprotéol et de la FNSEA, et Emmanuel Vasseneix, p-dg de LSDH et président d'Unijus. Leur volonté : que collectivement l'agroalimentaire français se donne les moyens de la reconquête. «En s'appuyant sur ce que pensent les filières, nous avons voulu développer une vision prospective pour réveiller l'agroalimentaire», explique Emmanuel Vasseneix. Pour ce faire, le cabinet Greenflex-Becitizen a mené une étude baptisée Plan Agrifood 2030. Trois actions ont été choisies lors du forum pour être mises en oeuvre (Lire www.processalimentaire.com).   Le modèle de la Laiterie du Berger Open Agrifood Orléans a aussi permis de nombreux échanges sur les bonnes pratiques. À l'instar de l'implication de Danone dans la laiterie du Berger au Sénégal. «C'est en reliant ces projets à l'efficacité business, que l'on apporte une définition moderne du double projet social et économique imaginé par mon père», explique Franck Riboud, le président du groupe français.   Pour le porter, Danone a créé Danone Communities. Le modèle financier s'appuie sur une Sicav créée en 2007. 10 % de l'encours, qui flirte avec les 100 millions d'euros, sont réservés au financement de projets de «social-business».   Parmi les projets soutenus, la Laiterie du Berger fait figure de modèle. Le fondateur Bagoré Bathily est parti d'un constat : plus de 90 % du marché local sont importés sous forme de poudre de lait. «C'est le paradoxe. Comme nos éleveurs ne sont pas assez productifs, on va sourcer sur le marché international. Au moment où notre économie se développe, les marchés sont déjà très ouverts et ne nous profitent pas». Son projet : proposer des produits à base de lait collecté localement et permettre d'améliorer les conditions de vie des éleveurs.   En 2006, ce vétérinaire, qui a travaillé en Mayenne, lance la Laiterie du Berger, à l'aide du fonds privé I &amp; P. Huit ans plus tard, l'entreprise a multiplié par dix son activité. Elle compte 150 employés et collecte 1,1 million de litres de lait à 800 éleveurs, dans un rayon de 50 km autour de l'usine située à Richard Toll dans le nord du pays. Les éleveurs ont une production moyenne de 9 kg/j, rémunérée sur un prix fixe (37 c /kg) pendant sept ans. Leur revenu a triplé. «La laiterie s'est développée grâce à la détermination des équipes sur place, mais aussi à l'aide des nouveaux modes de financement comme Danone Communities, qui nous a soutenu à partir de 2009, et la Fondation Grameen Crédit Agricole et PhiTrust entrés au capital», souligne-t-il.   La clef dans cette zone sahélienne est de sécuriser l'alimentation animale. La laiterie vend aux éleveurs, à des prix très compétitifs, 600 tonnes d'aliments pour bétail issus des productions locales (paille et son de riz, bagasse et mélasse de canne à sucre). Les éleveurs ont aussi besoin de formation et d'infrastructures. Bagoré Bathily s'appuie sur des partenaires comme le GRET, qui contribue à la construction de réseau d'adduction d'eau et à la mise en place de formations.   L'apport de Danone a été déterminant Second business laitier produits frais du pays, avec quatre millions d'euros de chiffre d'affaires, en croissance rapide, la Laiterie du Berger surfe sur l'urbanisation croissante au Sénégal, fournissant Dakar et les principales villes. Elle livre les 12 000 petits points de vente en produits frais toutes les semaines, au moyen de systèmes de petits camions et de pousse-pousse. En 2010, la gamme a été simplifiée autour des yaourts sous la marque Dolima (ressers-moi, en wolof).   Pour tous ces aspects, la laiterie a bénéficié du savoir-faire des «Danoners». De jeunes cadres du groupe se succèdent sur place pour contribuer à la formulation produit, au marketing, à la production et à la commercialisation. «Pour valoriser le lait local, il faut une marque forte portée par un système commercial efficace. L'apport de Danone en la matière a été déterminant», conclut Bagaré Bathily.
</article>
<article titre="Ferney-Voltaire déniche de l'eau dans ses bois" date="2014 11 26" journal="Tribune de Genève">
Située des deux côtés de la frontière, la nappe phréatique de Montfleury est testée pour voir si son eau est potable   La foreuse a remplacé le bâton de sourcier mais le but reste le même: trouver de l'eau! Dans le Pays de Gex, cette denrée est rare et donc recherchée. D'où le sourire des élus et de l'hydrogéologue de la communauté de communes (CCPG) en présentant leur dernière trouvaille: la nappe phréatique de Montfleury, transfrontalière, pourrait permettre une exploitation en eau potable.  10 à 15% des besoins   «Située en partie sur le sol ferneysien, cette nappe est stratégique: elle pourrait couvrir 10 à 15% des besoins de la population du Pays de Gex», s'enthousiasme Daniel Raphoz, maire de Ferney et vice-président de la CCPG, en charge du service Eau et Assainissement. A l'avenir, Ferney ainsi qu'une partie de Prévessin et Ornex pourraient être approvisionnés par ce sillon fluvio-glaciaire.    L'heure est actuellement aux tests. Dans les bois de la Bagasse, à deux pas de l'aéroport, les ouvriers creusent et pompent. «On procède à un forage de reconnaissance afin de vérifier la potabilité de l'eau», souligne Guillaume Marsac, hydrogéologue à la CCPG, précisant que le coût de cette opération est de 140 000 euros (168 400 fr. ). «On est descendu à 100 mètres de profondeur. On a trouvé de l'eau à 50 mètres. On doit maintenant déterminer la largeur, la profondeur de la nappe ainsi que le volume d'eau qui transite et donc le volume prélevable. »   Plus d'eau au robinet   Des connaissances précises qui doivent permettre de ne pas reproduire les erreurs du passé. Car, en la matière, le Pays de Gex a frôlé la catastrophe. «Il y a quelques années, on pensait que les nappes étaient inépuisables», rappelle l'hydrogéologue. Du coup, le Pays de Gex a puisé sans compter dans la source de Pré Bataillard, à Gex. Depuis 2001, ladite nappe a subi une baisse de près de 25 mètres, alors qu'elle en faisait 30. «Il a bien failli ne plus y avoir d'eau au robinet!» précise Guillaume Marsac.    A l'époque, le Pays de Gex a trouvé la solution en se tournant vers ses voisins. «Nous achetons environ 1 million de m3par an aux Vaudois, soit 20% de notre approvisionnement», commente Daniel Raphoz. Venue du Léman, cette eau alimente Divonne mais aussi une partie de Gex et Saint-Genis.    En parallèle, la CCPG a lancé une vaste campagne de recherches. «Notre indépendance en matière d'eau potable ainsi que la garantie d'un prix stable de l'eau pour l'usager passent par ce type de sondage», lance l'élu ferneysien. Dans le cadre de ces études, portant notamment sur les calcaires profonds, la CCPG s'est rapprochée de l'Etat de Genève pour mieux appréhender le contexte géologique local. «Je dialogue beaucoup avec le géologue cantonal Gabriel de Los Cobos», ajoute Guillaume Marsac. Logique puisque l'eau n'a pas de frontière. A l'image de la nappe du Genevois, qui, du coup, fait l'objet d'une convention franco-genevoise pour son exploitation.    La nappe de Montfleury n'échappe pas à cette réalité géographique. Elle est d'ailleurs connue des SIG qui l'ont autrefois exploitée. Comme en témoigne une cahute située en bout de piste de l'aéroport, à deux pas du forage en cours côté français. «Si on choisit de l'exploiter, il faudra une nouvelle convention franco-suisse», conclut Daniel Raphoz.
</article>
<article titre="Maurice : grève générale illimitée dans l'industrie sucrière" date="2014 11 19" journal="Xinhua - Agence de presse">
Maurice : grève générale illimitée dans l'industrie sucrière    PORT-LOUIS, 19 novembre (Xinhua) -- Une grève générale illimitée a débuté dans l'industrie sucrière à Maurice mercredi.     Les artisans et laboureurs de ce secteur important de l' économie mauricienne ont répondu à un mot d'ordre de leurs syndicats après l'échec des négociations portant sur une révision des salaires de 30% et de nouvelles conditions de travail.    De sources syndicales, on affirme que le mot d'ordre est respecté à 100% et qu'aucune usine ne fonctionnait durant toute la journée.   Des centaines de laboureurs et d'artisans étaient mobilisés devant l'usine d'Alteo dans l'est du pays, a pu constater Xinhua.     Cette action pourrait s'intensifier si l'association des usiniers ne cède pas sur les demandes des ouvriers, indique Ashok Subron, porte-parole du Joint Negociating Panel (JNP) regroupant des syndicats de l'industrie sucrière.    "Les patrons doivent devenir plus raisonnables et accepter de signer un accord avec les artisans et laboureurs. S'il n'y a pas un accord d'ici la semaine prochaine, les travailleurs descendront à Port-Louis devant le siège de la MSPA (Association des producteur du sucre de Maurice, ndlr) et l'Hôtel du gouvernement", a-t-il déclaré à la presse mercredi.    Maurice compte quatre usines sucrières et usines de production d'énergie électrique appartenant à des groupes sucriers et opérant en partie avec de la bagasse, un sous-produit de l'usinage de la canne à sucre.    Le JNP réclame une hausse salariale de 30% pour les artisans et laboureurs après avoir longtemps réclamé 40% tandis que la MSPA proposant elle 11% étalés sur quatre ans.     Toutes les tentatives de négociation menées par le ministère du Travail et des Relations industrielles, sont restées vaines jusqu' à présent. Fin
</article>
<article titre="Bioraffineries : tout est bon dans le végétal" date="2014 11 1" journal="Industrie et Technologies, no. 0970">
L e premier plastique biosourcé (une boule de billard, en 1870), l'était déjà à partir de cellulose ! Un composant végétal qui reste pourtant sous-utilisé, comme une très large partie des ressources végétales terrestres. Issue du bois, de la bagasse, de la paille ou du miscanthus et constituant presque 70 % de la biomasse disponible sur Terre, cette matière première à base de lignocellulose peut être valorisée en énergie, en combustibles, ou en produits chimiques. Cette biomasse dite de deuxième génération (2G) pourrait se substituer à la biomasse alimentaire (de première génération ou 1G). Dès le début des années 2000, Américains comme Européens s'intéressent à la lignocellulose pour remplacer la biomasse alimentaire en vue de produire de l'éthanol utilisé comme biocarburant. Mais ce n'est que dix ans plus tard que ces premières bioraffineries de deuxième génération à échelle industrielle - plus de 300 000 tonnes produites par an - démarrent enfin leur production aux États-Unis et au Brésil. Fortement soutenus aux États-Unis à travers une politique volontariste qui prévoit que soient construites près de 400 unités d'ici 2020, les projets de bioraffineries sont pénalisés en Europe par une législation instable. Une situation qu'illustre parfaitement le choix de la start-up française Biométhodes d'installer son unité-pilote en Virginie (États-Unis), alors que son siège et une partie de son département R&amp;D sont restés à Évry, en région parisienne.   Tout comme une raffinerie conventionnelle sépare les molécules du pétrole, c'est à la bioraffinerie que revient de séparer les constituants du végétal puis de les traiter adéquatement selon l'usage auquel elle est destinée. Les bioraffineries 2G s'appuient principalement sur l'exploitation de la cellulose et de l'hémicellulose, des chaînes de sucres qui sont découpées en sucres élémentaires de type C6 et C5, soit par catalyse enzymatique en faisant intervenir des enzymes produits par des micro-organismes, soit par catalyse thermochimique. Suite à cette étape d'hydrolyse, les sucres sont fermentés par des micro-organismes qui les transforment en éthanol.   Obtenir de l'hémicellulose et de la cellulose de bonne qualité Mais contrairement à la biomasse alimentaire, directement disponible pour le procédé d'hydrolyse, la lignocellulose souffre d'un défaut de taille : sa structure elle-même, qu'il faut casser pour la fragmenter dans ses trois constituants (la cellulose, l'hémicellulose et la lignine), une étape dite de « prétraitement » coûteuse et dont la qualité est essentielle pour la suite de la production.   Le procédé kraft, utilisé par la majorité des papetiers, ne date pourtant pas d'hier. Il libère les fibres cellulosiques du bois dans une opération de cuisson par dissolution de la lignine et d'une partie des hémicelluloses dans la solution chimique. « Seule la cellulose est alors récupérée pour faire de la pâte à papier, explique Nina Quélénis, responsable intelligence économique à l'IAR (pôle de compétitivité Industries et agro-ressources), tandis que la « liqueur noire » constituée des résidus dissous de lignine et d'hémicellulose est utilisée pour produire de la chaleur. Mais à partir du début des années 2000, les papetiers vont chercher à exploiter les autres composants comme co-produits. »   Reste que les applications pour les carburants ou pour la chimie exigent des celluloses et hémicelluloses de très bonne qualité. Celles-ci pourront alors être repolymérisées après avoir été réduites en sucres élémentaires, ou même être travaillées telles quelles par des procédés chimiques classiques, pour profiter au maximum des fonctionnalités apportées par leurs groupements d'atomes originaux, sans passer par les étapes d'hydrolyse et de fermentation.   De même, la lignine fait l'objet de nombreuses recherches. Il s'agit du principal composé aromatique présent dans la nature qui pourrait se substituer à ceux qui sont issus de la pétrochimie. D'où le besoin de procédés de prétraitements adaptés à l'aval de la production.   Selon le procédé de prétraitement, la cellulose est plus ou moins bien séparée de la lignine, mais aussi décristallisée. Si celle-ci demeure trop figée, elle nécessite ensuite des catalyses enzymatiques complexes et donc coûteuses. En outre, la plupart des micro-organismes utilisés lors de la fermentation des sucres sont très sensibles aux conditions de prétraitement, qui peuvent produire des inhibiteurs de fermentation qui viennent les contaminer.   Dela vapeur d'eau sous haute pression comme trétraitement Parmi les différents procédés, le plus utilisé est celui dit « d'explosion à la vapeur », utilisé et adapté par les premières bioraffineries de deuxième génération aux États-Unis et au Brésil, mais aussi par le consortium Procéthol qui porte le projet Futurol en France, auquel participe notamment l'IFP-Énergies nouvelles (IFP-EN). « Nous faisons passer la biomasse lignocellulosique dans la vapeur d'eau sous haute pression à une température d'environ 120 °C, en présence d'acide. Puis en détendant violemment le gaz, on produit une explosion qui rend accessibles la lignine, la cellulose, et l'hémicellulose. Celles-ci forment alors un mélange à partir duquel sont séparés les éléments par un autre procédé physico-chimique », explique Pierre Porrot, responsable du programme biocarburants de l'IFP-EN. « Nous avons étudié tous les types de prétraitements existants. Avec le prétraitement à la vapeur, les conditions dans lesquelles arrivent les constituants pour les étapes suivantes ne sont peut-être pas optimales, mais après on peut jouer sur la dose et la nature des enzymes. Nous y arrivons car nous travaillons sur le procédé global, et c'est nous qui produisons les enzymes dont nous avons besoin. » Autrement dit, on peut compenser un procédé de traitement de moins bonne qualité avec des enzymes plus efficaces, et des micro-organismes plus résistants aux inhibiteurs. Toutefois, cela ne permet pas de récupérer une lignine de qualité et le procédé Futurol prévoit qu'elle soit brûlée pour produire de la chaleur. Dans une autre logique, la jeune start-up française Biométhodes a préféré adopter un type de prétraitement à l'acide, pour produire dès le départ des constituants de qualité optimale (voir encadré).   Diversifier les sources pour une production continue Outre la nécessité pour les procédés - en particulier le prétraitement - d'être optimisés pour produire des molécules de bonne qualité, ceux-ci doivent pouvoir s'adapter aux différentes sources lignocellulosiques. Se reposer sur une seule source peut en effet s'avérer dangereux pour des raisons logistiques comme économiques. La paille de maïs, principalement utilisée aux États-Unis, a une récolte annuelle, au mieux biannuelle et supporte mal le stockage, de même que la bagasse de canne à sucre utilisée au Brésil. Diversifier les sources permet de faire tourner en continu la bioraffinerie, mais aussi de vendre un procédé qui puisse s'adapter aux ressources locales. Enfin, selon les ressources utilisées, les trois constituants de la lignocellulose ont des qualités spécifiques et des taux de répartition différents. « Nous voulons développer le procédé le plus flexible possible en matières premières. Le plus difficile est de travailler avec des matières premières mélangées, qui ont des types de lignocellulose différents », commente Jean-Pierre Burzynski, directeur du centre de résultats procédés de l'IFP-EN. « Chaque matière première a en effet ses poisons propres, qui peuvent désactiver les enzymes sélectionnées pour une autre matière première. » D'où le choix de Procéthol de cultiver chez eux la souche productrice d'enzymes pour concevoir au mieux le cocktail enzymatique qui leur convient. À l'instar de Procéthol, Biométhodes souhaite pouvoir utiliser tout type de biomasse même si son usine-pilote en Virginie (États-Unis), d'une capacité de 50 000 à 150 000 tonnes par an, n'utilise pour l'instant que du bois, disponible toute l'année. Celui-ci, avec les résidus de bois et la paille, devrait rester à terme les matières premières les plus utilisées, auxquelles on pourra ajouter les cultures dédiées aux fortes teneurs en lignocellulose, comme les taillis ou les miscanthus.   Pour Procéthol comme pour Biométhodes, la prochaine étape sera l'industrialisation des procédés. « Dans les biotechnologies en particulier, les micro-organismes ont du mal à passer de la R&amp;D à l'industrie, en raison d'une part du décloisonnement et de l'extrapolation nécessaire au passage d'une unité pilote à l'usine, d'autre part car les procédés catalytiques fonctionnent mieux dans des petits volumes. Dans des grands volumes, les températures ne sont pas forcément totalement homogènes », ajoute Jean-Pierre Burzynski. Un passage d'autant plus difficile que la qualité des sucres est exigeante, d'où la nécessité de commencer par la production de biocarburants demandant une qualité moindre. « Avec les États-Unis et le Brésil, le marché va s'ouvrir. À moyen terme, le procédé Futurol pourrait permettre la construction d'unités à près de 400 000 tonnes par an de bioéthanol, dont près de 10 % pourront être utilisées pour la chimie », conclut Jean-Pierre Burzynski. Encore un peu de recherche donc, et beaucoup de développement, avant que les plastiques biosourcés à partir de lignocellulose n'envahissent nos étalages.
</article>
<article titre="Bioplastiques : les nouvelles voies pour se passer de pétrole" date="2014 11 1" journal="Industrie et Technologies, no. 0970">
E n 2012, les plastiques produits à partir des ressources végétales, dits biosourcés, ne représentaient que 0, 4 % de la part de plastiques produits dans le monde mais devraient voir augmenter leurs capacités de production de près de 400 % d'ici 2017, selon le groupement d'industriels European Bioplastics. D'une part parce que les plasturgistes vont être confrontés tôt ou tard à une pénurie d'intrants, la production de pétrole n'augmentant pas aussi vite que celle des plastiques (tirée par la Chine et les autres pays en développement). D'autre part, les industriels veulent réduire leur dépendance au pétrole et améliorer leur image de marque. Et pour cela, rien de tel que la responsabilité environnementale. « Si la chimie biosourcée existe depuis plus d'un siècle, elle suscite un regain d'intérêt depuis le début des années 2000, explique François Monnet, directeur de la plateforme de recherche et innovation "Chimie du renouvelable" de Solvay. Mais la mode du renouvelable, si elle n'est pas tirée par la compétitivité, risque de passer. Deux évolutions peuvent permettre le basculement partiel de l'approvisionnement sur les intrants biosourcés : d'une part, la valorisation des déchets carbonés et d'autre part la possibilité d'utiliser les molécules qui apportent des spécificités naturelles. »    Des molécules prometteuses issues de la biomasse Les molécules biosourcées peuvent en effet se substituer aux mêmes molécules pétrosourcées, mais aussi ouvrir la voie à de nouveaux plastiques. Depuis un an, cinq bioraffineries de taille industrielle qui exploitent la biomasse lignocellulosique pour faire du bioéthanol destiné aux biocarburants ont démarré leur production aux États-Unis et au Brésil. Demain, en s'appuyant sur les mêmes procédés de fractionnement de la ressource végétale, elles pourraient produire en masse des produits pour la chimie et la plasturgie.   Concernant la première catégorie de molécules, substituables à l'identique aux pétrosourcées (parmi lesquelles l'éthylène, le propylène ou le paraxylène), la montée de leur production à l'échelle industrielle est tirée par les industriels désireux de présenter des plastiques d'origine totalement végétale. Depuis décembre 2011, Coca-Cola a annoncé des partenariats avec les entreprises Gevo et Virent pour produire de l'éthylène et du paraxylène, qui, associés ensemble, permettent la production de PET 100 % biosourcé. Il a depuis été suivi par Danone, Ford Motor, Heinz, Nike et d'autres. Strictement semblable à l'éthylène pétrosourcé, le bioéthylène qui mène au PET n'exige ni des plasturgistes, ni des recycleurs qu'ils adaptent leurs procédés.   Outre ces molécules déjà obtenues à partir des ressources pétrolières, d'autres sont particulières à la voie du végétal. En 2004, puis à nouveau en 2010, le département américain de l'énergie (DOE) a listé douze molécules particulièrement prometteuses qui pouvaient être produites à partir de sucres issus de la biomasse via des conversions biologiques ou chimiques. La plus connue est l'acide lactique, qui donne l'acide polylactique (PLA), un plastique déjà sur le marché. D'autres plastiques proches du PLA, mais aux meilleures propriétés physico-chimiques, font l'objet de recherches. C'est le cas du polyhydroxyalcanoate (PHA) sur lequel travaille PHApack. Ce consortium étudie la production de PHA à partir de déchets de l'industrie agroalimentaire. Ces derniers sont digérés et transformés par des bactéries marines, qui synthétisent sur cette base le PHA qu'il ne reste plus qu'à extraire.   Une autre molécule listée par le DOE, l'acide furane dicarboxylique (FDCA), suscite de forts enjeux. Grâce à une technologie de conversion catalytique de différents sucres en furanes, la société hollandaise Avantium fabrique du FDCA qui, polymérisé avec de l'éthylène glycol, produit du polyéthylène-furanoate (PEF), un polyester aux propriétés très similaires au PET, et pour lequel Coca-Cola a signé un partenariat avec Avantium.   En cours d'industrialisation, le PBS (polybutylène succinate) produit à partir de l'acide succinique et du butanediol biosourcés suscite aussi beaucoup d'espoirs en raison de ses bonnes qualités de résistance tout en restant biodégradable, contrairement au PET. La construction en Thaïlande d'une première usine pilotée par PTTMCC Biochem, d'une capacité de 20 000 tonnes de PBS par an, a débuté en septembre. De son côté, Bioamber devrait démarrer en Ontario (Canada) une première usine d'acide succinique en 2015, puis une autre pour le butanediol en 2017, signe qu'une véritable industrie pourrait naître autour du PBS.   Valoriser les sources de carbone lignocellulosiques Jusqu'à la fin des années 2000, la majorité des plastiques biosourcés sont issus de la biomasse alimentaire - dite de première génération (1G) - transformée en huile, en amidon ou en sucre selon ses caractéristiques. Aujourd'hui, c'est à l'ensemble des ressources carbonées que s'élargit le spectre des matières premières utilisables, parmi lesquelles les ressources lignocellulosiques, dites de deuxième génération (2G), voire les déchets ménagers, ou encore les microalgues (parfois qualifiées de 3G). Si l'exploitation à échelle industrielle de ces dernières devait encore attendre une dizaine d'années, l'utilisation des ressources lignocellulosiques, elle, est en pleine évolution, et profite de la volonté de se passer des ressources alimentaires pour produire des biocarburants. « Aux États-Unis, 50 % du maïs est utilisé pour produire des biocarburants. De plus, le cours du sucre a plus de volatilité que le pétrole, et expose aux fluctuations des ressources alimentaires », explique Romain Fouache, responsable du développement de la start-up française Biométhodes qui commercialise une technologie innovante d'exploitation de la lignocellulose. Celle-ci, qui représente près de 70 % de la biomasse présente sur Terre, a longtemps été utilisée seulement par l'industrie papetière pour transformer la cellulose en pâte à papier. Présente en grande partie dans le bois mais aussi constitutive de la paille ou de la bagasse issue de la canne à sucre, la lignocellulose - une fois fragmentée en cellulose, hémicellulose et lignine - fournit des polymères biosourcés d'intérêt. « Il y a des attentes fortes sur les polymères que l'on pourrait obtenir à partir de la lignocellulose, commente Alba Departe, ingénieure au service bioressources de l'Agence de l'environnement et de la maîtrise de l'énergie (Ademe). Soit pour développer de nouvelles molécules, grâce à la fonctionnalisation de ces polymères soit pour revenir aux sucres à partir desquels peuvent être produites les mêmes molécules tout en diminuant la pression sur les ressources agricoles. »   Ainsi, si la cellulose et l'hémicellulose peuvent tout à fait être transformées en sucres puis en éthanol destiné au biocarburant selon les procédés bien connus des premières générations, la tendance est de chercher à valoriser leurs fonctionnalités originales. « À partir de la cellulose, on obtient des molécules plus longues, aux terminaisons plus fonctionnelles et plus réactives (double C, doubles O ou de l'azote). À partir de ces molécules, on peut obtenir des PLA plus résistants à l'eau, au soleil ou à l'air », explique Nina Quelenis, responsable intelligence économique à l'IAR. « Ils ne nécessitent pas de la part des plasturgistes qui travaillent sur le PLA conventionnel de changer leur procédé. Cela peut même leur faire gagner des étapes car les molécules sont déjà fonctionnalisées. »   Mais c'est sur la lignine, qui protège la cellulose et l'hémicellulose, que reposent le plus d'enjeux. Elle constitue l'un des rares groupes de polymères biosynthétisés composés de motifs aromatiques. Les industriels prennent conscience de leur potentiel pour substituer les phénols et produits aromatiques issus de la pétrochimie, parmi lesquels par exemple le bisphénol A, considéré comme dangereux pour la santé.
</article>
<article titre="Enzymes, levures et bactéries à l'assaut de la biomasse" date="2014 11 1" journal="Industrie et Technologies, no. 0970">
S 'affranchir des hydrocarbures pour produire des polymères à partir de bois, de bagasse de canne ou de déchets verts : c'est désormais la tâche d'enzymes et de micro-organismes introduits dans les raffineries de seconde génération. Après une étape préalable - impliquant un procédé physico-chimique qui permet de déstructurer la lignocellulose en différents composants (cellulose, hémicellulose et lignine) - ces facilitateurs d'hydrolyse et de fermentation entrent en scène. Par exemple, des cellulases. Actives sur les celluloses, ces enzymes vont hydrolyser la biomasse, c'est-à-dire transformer l'hémicellulose et la cellulose en sucres utilisables par les micro-organismes fermentaires pour fabriquer les intermédiaires tels l'éthanol, l'acide lactique ou encore l'acide succinique. Lesquels entrent dans la composition finale, entre autres, des plastiques biosourcés. « Au niveau des enzymes, il s'agissait d'améliorer l'efficacité des cellulases dont on dispose pour l'hydrolyse initiale de la biomasse et la mise à disposition de sucres fermentescibles », explique Michael O'Donohue, directeur de cherche à l'Institut national de la recherche agronomique (Inra) et au Laboratoire d'ingénierie des systèmes biologiques et des procédés (LISBP).   Des enzymes obtenues par manipulation génétique Du côté des micro-organismes, « des souches de levure ont été développées pour pouvoir utiliser davantage de sucres, comme les pentoses [sucres simples - C5 - en chaînes dans l'hémicellulose, ndlr]. C'est notamment le domaine de la société française Lesaffre et de la société danoise DSM », poursuit Michael O'Donohue. La manipulation génétique des souches a permis d'introduire les enzymes manquantes à partir d'autres micro-organismes capables de consommer des pentoses. Sur ces derniers, on extrait le matériel génétique afin de conférer à la souche initiale cette même capacité.   D'autres micro-organismes encore réalisent naturellement la fermentation de glucides végétaux pour assurer leur activité cellulaire. En d'autres termes, le polymère récupéré est... un sous-produit de leur digestion. Le laboratoire d'ingénierie des matériaux de Bretagne (Limat B), en partenariat avec l'université de Rennes 1 et l'École nationale supérieure de chimie de Rennes (ENSCR), a étudié les bactéries marines vivant au large des côtes bretonnes dont le métabolisme permet de produire des polyhydroxyalcanoates ou PHA de pureté satisfaisante à partir de substrats issus des déchets d'industries agroalimentaires locales. Résultat : « On est capable aujourd'hui de produire sur des réacteurs de 50 litres. La phase suivante est de pouvoir encore augmenter cette échelle de production et passer sur des réacteurs jusqu'à 2,5 m3 », souligne Stéphane Bruzaud, directeur du programme PHApack qui s'est terminé en septembre 2014.   La synthèse du PHA, polyester biodégradable, se réalise à l'intérieur du cytoplasme, lors du processus de fermentation. En jouant sur la quantité de substrat fourni, on va produire un stress métabolique et la bactérie, en réponse à ce stress, produit des réserves énergétiques, c'est-à-dire des PHA. Les grains de plastique sont ensuite extraits de la bactérie dans un réacteur où on la fait réagir avec un chloroforme qui dissout sa paroi cellulaire.   Ces procédés, multiples selon la molécule d'intérêt recherchée, génèrent une même certitude : les micro-organismes seront les opérateurs les plus nombreux de la plasturgie du futur.
</article>
<article titre="À la Réunion, la transition énergétique profite aux PME" date="2014 10 27" journal="Journal de l'environnement.com">
Les dirigeants de la région de l'île de la Réunion préparent de longue date la transition énergétique. Sous l'égide du précédent président du conseil régional, Paul Vergès, l'ancienne île Bourbon a conduit nombre d'expérimentations (énergies solaire, éolienne, marine, géothermique) en vue d'atteindre l'autonomie énergétique, fixée à 2030, par le Grenelle de l'environnement.   L'industrialisation des plus rentables ne devrait pas tarder. Ce lundi 27 octobre, le Syndicat intercommunal d'exploitation des eaux océaniques (Sideo) a officiellement passé commande d'un système de climatisation utilisant l'eau de mer profonde. Climatisation à l'eau de mer   Déjà utilisé en Polynésie française et à Hawaï, ce dispositif pompe l'eau des grands fonds pour rafraîchir, via un échangeur thermique, l'eau d'un circuit de climatisation de bâtiments. Ce système Swac a été conçu par la société Climabyss (une filiale de GDF Suez) qui l'exploitera. Il devra assurer la climatisation d'immeubles de bureaux, d'une université, de logements et de l'aérogare de l'aéroport Roland Garros. Sa puissance est estimée à 40 mégawatts.   D'un coût estimé à 150 millions d'euros (dont 90 millions apportés par l'état), cette centrale à froid renouvelable réduira des trois quarts la demande d'électricité nécessaire à la climatisation des bâtiments.   Small business act   Autre originalité: l'état a fortement incité l'exploitant à signer un « small business act » [1]. En vertu de cette charte, également signée ce lundi, Climabyss s'engage à contractualiser avec des PME réunionnaises, notamment pour enfouir les 23 kilomètres de canalisations nécessaires. Les premiers coups de pioche doivent être donnés avant l'été 2015.   Un autre système Swac, mais de moindre puissance (8 MW), doit prochainement entrer en service dans le centre hospitalo-universitaire de Saint-Pierre (sud).   Au total, 8 projets fondés sur l'exploitation des énergies marines (houle, énergie thermique, climatisation, osmose) ont été mis à l'étude dans le cadre de la stratégie régionale, visant à atteindre l'autonomie énergétique en 2030.   A elle seule, l'énergie thermique des mers pourrait satisfaire, à cette échéance, 20% des besoins électriques de base et remplacer les centrales à bagasse-charbon.   [1] Le « Small Business Act » est une loi du Congrès des Etats-Unis voté le 30 juillet 1953, modifiée à de nombreuses reprises, et visant à favoriser les petites et moyennes entreprises dans le tissu économique du pays.
</article>
<article titre="[La Réunion, tirant profit de sa situation insulaire...]" date="2014 10 27" journal="La Gazette des communes (site web)">
La Réunion, tirant profit de sa situation insulaire, avance résolument vers son objectif d'autonomie énergétique à l'horizon 2030 avec un projet de réseau urbain de climatisation à l'eau de mer, unique au monde par son ampleur.   Le réseau de climatisation marine, fondé sur le procédé SWAC (« Sea Water Air Conditioning » ou climatisation à l'eau naturellement froide), est déjà exploité en Polynésie française ou à Hawaï, mais à petite échelle. Dans le département français de l'océan Indien, le projet ambitionne de climatiser une soixantaine de bâtiments publics et privés (aéroport, université, centres commerciaux.) des zones urbaines de Saint-Denis et Sainte-Marie (nord).   Le système pompera l'eau d'origine polaire des profondeurs de la mer (5° à 1.100 mètres) pour refroidir, via un échangeur thermique, un circuit d'eau douce alimentant les locaux à climatiser. 75% d'électricité sera ainsi économisée.   Piloté par le Syndicat intercommunal d'exploitation des eaux océaniques (Sideo), ce projet présente un coût de 150 millions d'euros dont 90 millions financés par l'Etat. Les travaux commenceront mi-2015 pour une mise en service mi-2017.   Entre 300 à 400 emplois seront créés sur le chantier, notamment pour creuser les 23 km de tranchées nécessaires pour enfouir les canalisations. Il faudra également 7 km de canalisations afin de transporter l'eau de mer jusqu'à la côte et 1km pour la rejeter dans l'océan, après usage.   La réalisation à la Réunion de ce vaste réseau urbain de climatisation à l'eau de mer a fait l'objet, lundi à Saint-Denis, d'un accord pour ouvrir ce marché de 150 millions d'euros aux petites entreprises de l'île.   Huit projets liés à l'énergie marine - Dans une île qui compte plus de 135.000 chômeurs sur une population de 826.400 personnes, les partenaires institutionnels et industriels du projet (Etat, Région, GDF Suez, ADEME) ont signé avec des acteurs économiques un « Small business act » destiné à ouvrir le marché, directement ou en sous-traitance, à de petites entreprises locales.   « Quel beau symbole, quel bel exemple de ce qu'on peut faire ensemble! » s'est réjoui Jean-Lou Blachier, médiateur des marchés publics, en déplacement sur l'île. « C'est une commande essentielle pour le secteur du BTP », a souligné le préfet de la Réunion Dominique Sorain.   La baisse des prix pour les clients du réseau n'interviendra toutefois qu'à moyen terme, à compter de 2022, selon Eric Bassac, du pôle international de GDF Suez.   Un second projet SWAC, de moindre importance, a été lancé à Saint-Pierre (sud) en vue de climatiser le Centre hospitalier universitaire sud Réunion. Il vise à économiser sur 5 ans, 3 millions d'euros, soit 50% de sa consommation électrique, selon ses promoteurs.   Au total, huit projets fondés sur l'exploitation de l'énergie marine (houle, énergie thermique, climatisation, osmose) ont été mis à l'étude dans le cadre de la stratégie initiée par la Région et le Grenelle de l'environnement, visant à atteindre l'autonomie énergétique de l'île en 2030.   L'ETM (Energie thermique des mers) pourrait assurer à cette date 20% des besoins électriques de base et remplacer l'équivalent de l'importation actuelle de charbon.   Deux tiers de la production électrique de la Réunion sont aujourd'hui d'origine fossile (charbon et fuel). Environ 20% proviennent des centrales hydrauliques et 10% à 15% de la bagasse (résidus ligneux de la canne), de l'éolien et du solaire.   ii-sla/glr/fm   GDF SUEZ  Cet article est paru dans La Gazette des communes (site web)
</article>
<article titre="La Réunion se lance dans un projet de climatisation à l'eau de mer" date="2014 10 27" journal="Le Moniteur (site web)">
La réalisation à la Réunion d'un vaste réseau urbain de climatisation à l'eau de mer, unique au monde par son ampleur, a fait l'objet, lundi 27 octobre à Saint-Denis, d'un accord pour ouvrir ce marché de 150 millions d'euros aux petites entreprises de l'île. Huit projets liés à l'énergie marine sont actuellement en développement, ils pourraient assurer 20% des besoins électriques de base d'ici à 2030.    La Réunion, tirant profit de sa situation insulaire, avance résolument vers son objectif d'autonomie énergétique à l'horizon 2030 avec un projet de réseau urbain de climatisation à l'eau de mer, unique au monde par son ampleur. Le réseau de climatisation marine, fondé sur le procédé SWAC (« Sea Water Air Conditionning » ou climatisation à l'eau naturellement froide), est déjà exploité en Polynésie ou à Hawaï, mais à petite échelle.   Dans le département français de l'océan indien, le projet ambitionne de climatiser une soixantaine de bâtiments publics et privés (aéroport, université, centres commerciaux.) des zones urbaines de Saint-Denis et Sainte-Marie (nord).   75% d'électricité économisée    Le système pompera l'eau d'origine polaire des profondeurs de la mer (5° à 1.100 mètres) pour refroidir, via un échangeur thermique, un circuit d'eau douce alimentant les locaux à climatiser. 75% d'électricité sera ainsi économisée.   Piloté par le Syndicat intercommunal d'exploitation des eaux océaniques (Sideo), ce projet présente un coût de 150 millions d'euros dont 90 millions financés par l'Etat. Les travaux commenceront mi-2015 pour une mise en service mi-2017.   Entre 300 à 400 emplois seront crées sur le chantier, notamment pour creuser les 23 kilomètres de tranchées nécessaires pour enfouir les canalisations. Sept kilomètres de canalisations seront nécessaires pour transporter l'eau de mer jusqu'à la côte et 1 km pour la rejeter dans l'océan, après usage.   La réalisation à la Réunion de ce vaste réseau urbain de climatisation à l'eau de mer a fait l'objet, lundi à Saint-Denis, d'un accord pour ouvrir ce marché de 150 millions d'euros aux petites entreprises de l'île.   Marché ouvert aux petites entreprises locales    Dans une île qui compte plus de 135 000 chômeurs sur une population de 826 400 personnes, les partenaires institutionnels et industriels du projet (Etat, Région, GDF Suez, ADEME) ont signé avec des acteurs économiques un « Small business act » destiné à ouvrir le marché, directement ou en sous-traitance, à des petites entreprises locales.   « Quel beau symbole, quel bel exemple de ce qu'on peut faire ensemble! » s'est réjoui Jean-Lou Blachier, médiateur des marchés publics, en déplacement sur l'île. « C'est une commande essentielle pour le secteur du BTP », a souligné le préfet de la Réunion Dominique Sorain.   La baisse des prix pour les clients du réseau n'interviendra toutefois qu'à moyen terme, à compter de 2022, selon Eric Bassac, du pôle international de GDF Suez.   Huit projets liés à l'énergie marine    Un second projet SWAC, de moindre importance, a été lancé à Saint-Pierre (sud) en vue de climatiser le Centre hospitalier universitaire sud Réunion. Il vise à économiser sur 5 ans, 3 millions d'euros, soit 50% de sa consommation électrique, selon ses promoteurs.   Au total, huit projets fondés sur l'exploitation de l'énergie marine (houle, énergie thermique, climatisation, osmose) ont été mis à l'étude dans le cadre de la stratégie initiée par la Région et le Grenelle de l'environnement, visant à atteindre l'autonomie énergétique de l'île en 2030.   L'ETM (Energie thermique des mers) pourrait assurer à cette date 20% des besoins électriques de base et remplacer l'équivalent de l'importation actuelle de charbon.   Deux tiers de la production électrique de la Réunion sont aujourd'hui d'origine fossile (charbon et fuel). Environ 20% proviennent des centrales hydrauliques et 10% à 15% de la bagasse (résidus ligneux de la canne), de l'éolien et du solaire.  Cet article est paru dans Le Moniteur (site web)
</article>
<article titre="Tetra Pak lance la première brique 100 % d'origine végétale" date="2014 10 17" journal="Process Alimentaire (site web)">
Le Tetra Rex renouvelable se compose de plusieurs couches de matériaux issus de la bagasse.   Tetra Pak lancera début 2015 sur le marché une brique en carton entièrement fabriquée à partir de matériaux renouvelables d'origine végétale. Ce nouvel emballage baptisé Tetra Rex végétal est composé de film polyéthylène basse densité (PEBD) d'origine végétale et d'un bouchon en polyéthylène haute densité (PEHD) d'origine végétale, tous deux provenant de canne à sucre. S'y ajoute évidemment le carton, certifié par le Forest Stewardship Council (FSC). Développé en partenariat avec Braskem, l'un des leaders mondiaux de production de biopolymères, ce nouveau Tetra Rex pourra remplacer l'emballage Tetra Rex 1L standard avec bouchon TwistCap OSO 34. Sans aucun investissement supplémentaire pour les industriels ni modification à apporter à leurs machines de remplissage.   « Aux côtés de nos fournisseurs, clients et autres parties prenantes, nous guidons l'industrie vers l'adoption d'emballages 100% renouvelables, a déclaré Charles Brand, vice-président Marketing et Product Management chez Tetra Pak. Nous sommes convaincus que le fait d'augmenter la part de matériaux renouvelables de nos emballages n'est pas seulement bénéfique pour l'environnement, mais offre également à nos clients un avantage concurrentiel en termes de profil environnemental général de leurs produits. »   Cet article est paru dans Process Alimentaire (site web)
</article>
<article titre="Le Brésil, pays stratégique pour le producteur de sucre français Tereos" date="2014 10 17" journal="AFP Infos Economiques">
Rio de Janeiro, 17 oct 2014 (AFP) - - Tereos, premier producteur français de sucre et cinquième mondial (marque Béghin-Say), tire un quart de son chiffre d'affaires du Brésil, où il vient d'investir 100 millions d'euros dans sa huitième usine, une fabrique d'amidon de maïs.   Il s'agit d'un investissement stratégique pour le groupe, explique Jacyr Costa, son directeur pour le Brésil.   QUESTION: Le Brésil est le premier producteur mondial de sucre et importait du maïs il y a encore 15 ans. Pourquoi y investir dans l'amidon ? REPONSE: Le Brésil est passé de pays insignifiant sur le marché du maïs à la position de deuxième exportateur mondial. Avec une telle offre, l'approvisionnement en matière première est garanti et nous savons qu'elle sera compétitive.   Nous croyons à l'amidon car il alimente une immense gamme de produits dérivés, presque sur-mesure pour chaque client.    Les caractéristiques de l'amidon sont différentes s'il est destiné à une gomme à mâcher ou à un bonbon, par exemple. L'amidon sert pour l'industrie alimentaire et de la bière mais aussi pour le papier et la cellulose, des secteurs importants de l'industrie brésilienne.   La demande au Brésil a tellement augmenté que nous avons les moyens de nous concentrer sur le marché intérieur. La consommation par habitant y est de 10 kilogrammes par an contre 21 kg par an en Europe. Dans le secteur de la canne à sucre, nous investissons dans la réduction des coûts; pour l'amidon, dans l'expansion du marché.   QUESTION: Les cours du sucre sont bas depuis deux ans et les quotas en Europe vont disparaître en 2017. Investir dans l'amidon permet de se prémunir contre la volatilité de ce marché ?   REPONSE: La diversification, tant géographique qu'en termes de matières premières, vise à améliorer la régularité des résultats. Elle est à l'oeuvre depuis les années 1990, lorsque le groupe s'alimentait auprès de coopératives de betteraves et s'est ouvert à la canne à sucre, aux céréales, au manioc, à la pomme de terre... Tereos a aussi investi dans des pays émergents à fort potentiel de croissance comme la Chine ou l'Indonésie, mais aussi le Mozambique et la Tanzanie.   QUESTION: Tereos est le troisième producteur de sucre au Brésil. Comment se présente le secteur actuellement dans le pays ?   REPONSE: Lorsque nous sommes arrivés au Brésil, à travers Cosan, en 2000, le pays représentait environ 30% de la production mondiale de sucre. Aujourd'hui c'est plutôt 50%.   Le secteur connaît un moment compliqué à cause du prix de l'éthanol (Ce carburant à base de canne à sucre est en "concurrence" avec l'essence, dont le prix est maintenu à la baisse par des subventions, NDLR). Mais nous avons un potentiel et nous y croyons. Nous bénéficions également d'une synergie avec la France.   Au Brésil, l'énergie a toujours été peu chère, mais le prix a beaucoup augmenté dernièrement. Les usines françaises ont le savoir-faire pour limiter ce poste de dépenses. Nous avons importé les techniques de co-génération : nous brûlons la bagasse (déchets ligneux) de la canne à sucre pour la transformer en énergie. Cela diminue nos frais et joue fortement sur nos résultats car l'énergie excédentaire est vendue.   (Propos recueillis par Hélène SEINGIER)   hse/emi/fga/ggy
</article>
<article titre="Repères Chiffres clés des énergies renouvalables édition 2014" date="2014 10 13" journal="Rapports - Environnement - MEDDTL">
64 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... octobre 2014 Chiffres clés des énergies renouvelables 1www.statistiques.developpement-durable.gouv.fr  COMMISSARIAT GÉNÉRAL AU DÉVELOPPEMENT DURABLE  octobre 2014  Chiffres clés des énergies renouvelables  Édition 2014  Service de l’observation...    Pour voir le rapport :  http://www.developpement-durable.gouv.fr/IMG/pdf/Rep_-_Chifffres_cles_energies_renouvelables_2014.pdf
</article>
<article titre="vie des entreprises" date="2014 10 1" journal="Environnement Magazine et Energie, no. 19">
Malgré de nombreux déboires, dont une suspension de cotation en Bourse liée à une trésorerie fragile, Europlasma a annoncé la réception préliminaire de sa centrale CHO Power de valorisation énergétique de déchets à Morcenx (40).    Le brésilien Copel a pris une participation de 49 % au capital des centrales éoliennes Sao Miguel do Gostoso de Voltalia, dont la production doit commencer au second trimestre 2015.  La plateforme d'autopartage TravelerCar a levé 750 000 euros auprès de Network Finances pour développer sa présence en Europe.    Voltalia a levé un peu plus de 100 millions d'euros sur le marché réglementé d'Euronext à Paris pour financer les constructions en cours de ses centrales en France et au Brésil.    Le spécialiste de la mobilité électrique Wattmobile a bouclé une levée de fonds de plus de 1,5 million d'euros auprès de FullCharger, son actionnaire majoritaire depuis avril, et de nouveaux investisseurs (SNCF, EADS, Crédit coopératif, Total).    Soitec a lancé une nouvelle augmentation de capital, la troisième en trois ans.    D'un montant d'environ 83 millions d'euros, elle doit renforcer la trésorerie et assurer le fonds de roulement du groupe. La filiale française de Centrosolar passe sous le contrôle de Solarwatt. Le groupe allemand a également repris les actifs néerlandais de son compatriote, qui avait été placé en liquidation judiciaire.    Pour financer son plan de développement dans la biomasse, l'exploitant de centrales bagasse-charbon Albioma a émis 80 millions d'euros d'obligations auprès d'investisseurs institutionnels.    Remporter 20 % de parts de marché dans l'éolien en mer en Europe d'ici à 2020 : c'est l'objectif que se fixent le français Areva et l'espagnol Gamesa pour leur nouvelle société commune dans l'offshore.    La Semmaris, gestionnaire du marché francilien de Rungis, profite du bonus écologique : sept Renault Zoé et quatre Kangoo zéro émission frigorifique se greffent à la vingtaine de voitures thermiques de l'entreprise.    Qinergy a lancé une offre pour suivre en temps réel la consommation électrique de son foyer depuis un ordinateur, un téléphone portable ou une tablette numérique.
</article>
<article titre="Quel avenir pour la production de canne à sucre dans les DOM après la fin des quotas ?" date="2014 10 1" journal="La Revue Parlementaire, no. 964">
Le système des quotas, régi par le règlement du 20 février 2006, assure jusqu'à présent le maintien de la production sucrière dans l'Union européenne ainsi que le contrôle du prix du sucre blanc fixé à 404,4 euros la tonne, alors que son prix mondial s'élève à 296,2 euros la tonne. Cependant, dans le cadre de la renégociation de la PAC, Bruxelles a fixé à 2017 la fin des quotas, sonnant le glas des prix de soutien et de l'achat en cas de mévente. Si les betteraviers s'organisent d'ores et déjà en prévoyant une hausse de leur production grâce notamment à la recherche génomique de la betterave, les industriels sucriers des DOM vont certainement connaître des difficultés. Ils font déjà face à des coûts de production et de transports plus élevés qu'en métropole ainsi qu'à l'étroitesse des marchés qui limite fortement la possibilité pour eux de réaliser des économies d'échelle. Mais à cela s'ajoutera probablement une chute des cours européens qui devront s'aligner sur les cours mondiaux, extrêmement volatiles, suite à la fin des quotas. Or, la production de canne à sucre fait partie intégrante de l'économie des DOM que sont la Martinique, la Guadeloupe et La Réunion : non seulement utilisée pour faire du sucre blanc et du sucre roux, elle sert aussi à la production de rhum, très liée à l'identité de ces îles, ainsi qu'à l'alimentation des centrales thermiques qui représentent 10 à 12% de la consommation d'électricité en Guadeloupe et à La Réunion. Ce rapport d'information présenté au nom de la Délégation aux Outre-Mer fait plusieurs propositions afin d'aider ce secteur industriel qui ne manquera pas d'accuser un retard certain par rapport à la production betteravière française à partir de 2017, d'autant plus  «qu'il existe  [déjà]  un différentiel de compétitivité entre le sucre de canne et le sucre de betterave» . De nouveaux débouchés devront ainsi être trouvés et  «les recherches en ce sens (...) fortement encouragées» . Les rapporteurs demandent aussi  «de recourir à une aide gouvernementale»  qui passerait de 90 millions à 128 millions d'euros annuels, de préserver le foncier agricole et de créer un observatoire  «chargé du suivi de l'évolution des prix sur le marché du sucre» , ce qui pourrait mener à une demande de subvention auprès de l'UE. Ils recommandent également d'inciter EDF à augmenter le prix d'achat de la bagasse (résidu de la canne à sucre, utilisé dans les centrales thermiques) qui s'élève en 2011 à 11,05 euros la tonne. Or, une hausse à 14 euros entraînerait un gain de 3 millions d'euros supplémentaires pour l'industrie sucrière. En outre, le rapport suggère d'entamer des démarches pour obtenir, au niveau européen, le sigle IGP (Indication géographique protégée) sur les productions des DOM,  «plus-value indéniable» , et notamment sur le sucre roux dont la production nationale s'élève à 2,8 millions de tonnes. En ce sens, une bataille devra être engagée pour le maintien des tarifs douaniers sur ce type de sucre  «lorsqu'il est produit par des pays extra-européens» .
</article>
<article titre="La Martinique veut devenir un laboratoire de la transition énergétique Energie" date="2014 09 9" journal="La Gazette des communes">
Solaire, géothermie, biomasse, énergies marines, ... les ressources énergétiques alternatives ne manquent pas en Martinique et pourtant, ce territoire n'utilise quasiment que des énergies fossiles, se retrouvant ainsi extrêmement dépendant des fluctuations du prix du pétrole et émetteur d'une grande quantité de gaz à effet de serre. Un virage a été pris en 2010 avec la décision par le conseil régional de la Martinique de faire du développement des énergies renouvelables une priorité. Et pour engager une véritable transition énergétique, la ministre de l'Ecologie, Ségolène Royal, la ministre des Outre-mer, George Pau-Langevin, et le président de la région Martinique, Serge Letchamy ont signé le 1er septembre 2014 une déclaration d'intention visant à faire ce cette région un laboratoire pour l'Outre-mer, voire un modèle pour toutes les régions, selon les propos de la ministre. Ce qui, vu la faiblesse de la production locale d'énergies renouvelables, représente tout de même un "défi", mais aussi une "chance" pour développer des emplois locaux, comme l'a souligné Ségolène Royal. Relancer le photovoltaïque -  Plus encore qu'en métropole, les territoires ultramarins recèlent " un formidable potentiel de déploiement des énergies renouvelables ", a rappelé la ministre. Elle a notamment annoncé le lancement à venir d'un nouvel appel d'offres dans les Outre-mer concernant le solaire photovoltaïque pour les puissances supérieures à 100 kilowatt crête. Il prévoit notamment une bonification du prix de rachat si l'électricité produite est autoconsommée dans le bâtiment.   Plusieurs projets pilotes exemplaires et implantés en Martinique ont été présentés lors de la visite de la ministre, comme le projet Nemo. Il s'agit d'une centrale flottante d'énergie thermique (cf. photo ci dessus) qui utilise la différence de température entre les eaux de surface (chaudes) et les eaux profondes (froides). Elle est implantée à Bellefontaine et développée avec le soutien de la région, l'Etat, l'Europe et deux entreprises (Akuo Energy et le groupe DCNS).   D'autres initiatives concernent le recours à la géothermie pour alimenter des bâtiments publics (tels que le CHU de La Meynard) ou l'utilisation de la biomasse, notamment la bagasse. Un projet porté par le producteur d'énergie Albioma vise ainsi à valoriser par cogénération le biogaz tiré de cette biomasse locale.   Recharge solaire de véhicules électriques -  En matière de mobilité, les premiers véhicules électriques vont être importés sur le territoire à la fin de l'année 2014. Et pour éviter que ces nouveaux usages fassent augmenter la consommation électrique et nécessitent d'élever la capacité de production, un projet visant à coupler les bornes de recharge des véhicules à la production d'électricité par panneaux photovoltaïques a été proposé par le groupe Bolloré.   Enfin, il est également prévu le déploiement de l'éclairage solaire dans toutes les collectivités de la Martinique.
</article>
<article titre="La Martinique veut devenir un laboratoire de la transition énergétique Energie" date="2014 09 4" journal="La Gazette des communes">
Solaire, géothermie, biomasse, énergies marines, ... les ressources énergétiques alternatives ne manquent pas en Martinique et pourtant, ce territoire n'utilise quasiment que des énergies fossiles, se retrouvant ainsi extrêmement dépendant des fluctuations du prix du pétrole et émetteur d'une grande quantité de gaz à effet de serre. Un virage a été pris en 2010 avec la décision par le conseil régional de la Martinique de faire du développement des énergies renouvelables une priorité. Et pour engager une véritable transition énergétique, la ministre de l'Ecologie, Ségolène Royal, la ministre des Outre-mer, George Pau-Langevin, et le président de la région Martinique, Serge Letchamy ont signé le 1er septembre 2014 une déclaration d'intention visant à faire ce cette région un laboratoire pour l'Outre-mer, voire un modèle pour toutes les régions, selon les propos de la ministre. Ce qui, vu la faiblesse de la production locale d'énergies renouvelables, représente tout de même un "défi", mais aussi une "chance" pour développer des emplois locaux, comme l'a souligné Ségolène Royal. Relancer le photovoltaïque -  Plus encore qu'en métropole, les territoires ultramarins recèlent " un formidable potentiel de déploiement des énergies renouvelables ", a rappelé la ministre. Elle a notamment annoncé le lancement à venir d'un nouvel appel d'offres dans les Outre-mer concernant le solaire photovoltaïque pour les puissances supérieures à 100 kilowatt crête. Il prévoit notamment une bonification du prix de rachat si l'électricité produite est autoconsommée dans le bâtiment.   Plusieurs projets pilotes exemplaires et implantés en Martinique ont été présentés lors de la visite de la ministre, comme le projet Nemo. Il s'agit d'une centrale flottante d'énergie thermique (cf. photo ci dessus) qui utilise la différence de température entre les eaux de surface (chaudes) et les eaux profondes (froides). Elle est implantée à Bellefontaine et développée avec le soutien de la région, l'Etat, l'Europe et deux entreprises (Akuo Energy et le groupe DCNS).   D'autres initiatives concernent le recours à la géothermie pour alimenter des bâtiments publics (tels que le CHU de La Meynard) ou l'utilisation de la biomasse, notamment la bagasse. Un projet porté par le producteur d'énergie Albioma vise ainsi à valoriser par cogénération le biogaz tiré de cette biomasse locale.   Recharge solaire de véhicules électriques -  En matière de mobilité, les premiers véhicules électriques vont être importés sur le territoire à la fin de l'année 2014. Et pour éviter que ces nouveaux usages fassent augmenter la consommation électrique et nécessitent d'élever la capacité de production, un projet visant à coupler les bornes de recharge des véhicules à la production d'électricité par panneaux photovoltaïques a été proposé par le groupe Bolloré.   Enfin, il est également prévu le déploiement de l'éclairage solaire dans toutes les collectivités de la Martinique.
</article>
<article titre="SUCRÉ OU SALÉ:FAITES VOTRE CHOIX" date="2014 09 4" journal="Le Matin">
1De quoi le saccharose (sucre blanc ou de table) est-il constitué?   a)   De 2 molécules de glucose   b) De 2 molécules de fructose   c)   D'une molécule de glucose et d'une molécule de fructose   2Combien de kilocalories apporte un gramme de sucre blanc?   a)   4 kcal   b)   8 kcal   c)   10 kcal   3Combien pèse un morceau de sucre standard (de calibre 4)?   a)   Environ 6 g   b)   Environ 8 g   c)   Environ 10 g   4Comment appelle- t-on un collectionneur de morceaux de sucre emballé?   a)   Un glucophile   b)   Un glycophile   c)   Un glacophile   5En moyenne, jusqu'à combien de kilos de sucre fournit une tonne de betteraves?   a)   Environ 110 kg   b)   Environ 160 kg   c)   Environ 220 kg   6Quel est l'autre expression pour «sucres complexes»?   a)   Sucres raffinés   b)   Sucres rapides   c)   Sucres lents   7Comment appelle- t-on le liquide de la canne à sucre quand on l'écrase?   a)   La bagasse   b)   La mélasse   c)   Le vesou   8De quel pays le sucre a-t-il été la monnaie officielle de 1884 à 2000?   a)   L'Equateur   b)   Le Venezuela   c)   La Bolivie   9A quoi fait référence l'appellation de sel ignigène?   a)   Au sel de saumure   b)   Au sel cristallisé   c)   Au sel non raffiné   10Qu'est-ce que la fleur de sel?   a)   Un sel dont la forme des cristaux ressemble à une fleur   b)   Une couche de cristaux de sel qui se forme à la surface des marais salants   c)   La couche supérieure d'une mine de sel   11Qu'est-ce que le sel gemme?   a)   Une pierre fine   b)   Un sel en cours de minéralisation   c)   Un sel fossile   12La gabelle fut un impôt sur le sel institué par saint Louis. Dans quel but?   a)   Financer les croisades   b)   Servir de monnaie d'échange   c)   Rémunérer les fermiers de la Ferme générale   13Quel personnage de la Bible fut transformé en statue de sel pour avoir regardé en arrière la destruction de Sodome et Gomorrhe?   a)   Jisca, la nièce d'Abraham   b)   Sarah, la femme d'Abraham   c)   Ildith, la femme de Loth   14Quel est la composition du sel?   a)   40% de sodium et 60% de chlorure   b)   50% de sodium et 50% de chlorure   c)   60% de sodium et 40% de chlorure   15Quel est le plus grand lac salé?   a)   La mer d'Aral   b)   La mer Caspienne   c)   Le Grand Lac Salé
</article>
<article titre="Albioma : forte hausse" date="2014 09 2" journal="Boursier (site web)">
Albioma  avance de 2,3% à 21 euros en fin de séance, soutenu par une note d'analyste. Selon nos informations, Bryan Garnier a initié le suivi du dossier avec une opinion positive et une fair value de 24 euros. Le spécialiste de la génération d'électricité à travers la biomasse, le solaire et la biométhanisation entre dans une importante phase d'investissement au Brésil notamment où il a pour ambition de croître sur le marché de la canne à sucre et de la bagasse. Ce plan d'investissement devrait permettre au groupe de générer 8,6% de croissance annuelle au niveau BPA de 2014 à 2024, estime le broker. De plus, le courtier juge la valorisation attractive puisque le titre s'échange actuellement avec une décote de 15-20% par rapport aux multiples historiques en dépit d'une forte augmentation de la dette nette sur 2014-15 due à la montée en puissance des investissements.
</article>
<article titre="LA TRANSITION ÉNERGÉTIQUE DE LA MARTINIQUE ET DES DOM CONTRIBUTION ALBIOMA" date="2014 09 1" journal="Rapports - Environnement - MEDDTL">
23 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... LA TRANSITION ÉNERGÉTIQUE DE LA MARTINIQUE ET DES DOM  CONTRIBUTION ALBIOMA  Martinique Ile Durable – 31 août 2014 2 PRÉSENTATION GÉNÉRALE DU GROUPE ALBIOMA LA CENTRALE BIOMASSE DU GALION EN MARTINIQUE LA...    Pour voir le rapport :  http://www.developpement-durable.gouv.fr/IMG/pdf/Albioma.pdf
</article>
<article titre="L’objectif de la visite de travail est d’engager la Martinique dans la transition énergétique dans le cadre du projet de territoire à énergie positive pour valoriser pleinement les outre-mer et déclencher des créations d’emplois" date="2014 08 29" journal="Rapports - Environnement - MEDDTL">
3 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... Invitation   Paris, le 29 août 2014  Madame Ségolène Royal Ministre de l’Écologie, du Développement durable et de  l’Énergie  se rendra en Martinique du samedi 30 août au lundi 1er...    Pour voir le rapport :  http://www.developpement-durable.gouv.fr/IMG/pdf/Invit_Martinique_VDEF5.pdf
</article>
<article titre="Albioma : associe ses collaborateurs à la création de valeur actionnariale" date="2014 07 31" journal="Boursier (site web)">
Le 14 mars 2012, l'Assemblée générale mixte des actionnaires d' Albioma  a validé la nouvelle stratégie de développement dans la valorisation à haute efficacité énergétique de la bagasse et plus généralement de la biomasse. Afin d'aligner les intérêts de l'ensemble du personnel sur ceux de l'entreprise, elle a autorisé un plan d'attribution gratuite d'actions portant sur un nombre maximal de 810.000 actions, représentant 2,7% du capital. "Il est important que les femmes et les hommes qui mettent leur talent et leur énergie au service l'Albioma soient associés à la création de valeur actionnariale du Groupe. Nous nous réjouissons de cette première attribution qui illustre le succès de la stratégie mise en oeuvre et la pleine mobilisation de nos équipes", déclare Jacques Pétry.   L'acquisition définitive des actions, qui a lieu par tranches d'un tiers, suppose que la moyenne mobile 6 mois du cours de bourse atteigne, au cours de la période du 26 juillet 2014 au 26 janvier 2015, les seuils de 18,5 euros, 22,5 euros et 26,5 euros. La nouvelle stratégie porte aujourd'hui ses fruits avec, en particulier, les nouveaux projets annoncés récemment dans l'Outre-Mer Français et au Brésil. Le cours de bourse a progressé de plus de 80% depuis le 26 juillet 2012 et le premier seuil de 18,5 euros a été atteint le 26 juillet dernier, entraînant l'acquisition définitive d'un maximum d'environ 270.000 actions au cours de la période du 26 juillet 2014 au 13 janvier 2016, dont 197.969 actions créées le 30 juillet 2014.
</article>
<article titre="ILS FERONT LE MONDE - Marc Delcourt Roi du pétrole vert Ce biologiste reconverti en entrepreneur mise sur les plantes pour remplacer les ressources fossiles -" date="2014 07 29" journal="Le Monde">
Depuis peu, il s'est laissé pousser un collier de barbe, où le sel commence à manger le poivre. Manière de masquer son allure d'étudiant attardé -  « je faisais un peu jeune pour lever des fonds »  - et d'inspirer confiance aux investisseurs. Ceux-ci n'ont pourtant pas attendu cet artifice pileux pour lui faire crédit. Global Bioenergies, la start-up qu'il a fondée en 2008 et qui est cotée en Bourse, est la seule en Europe, et l'une des rares au monde, à exploiter un filon neuf : la production d'hydrocarbures biologiques. Il s'agit de remplacer le pétrole d'origine fossile par des ressources végétales renouvelables. De transmuter l'or noir en or vert. Il y va, est-il convaincu, de  « l'avenir de notre civilisation » . Sous ses airs poupins de doux rêveur, Marc Delcourt est en réalité, à 43 ans, un redoutable businessman, doublé d'un visionnaire. Un mélange de culture scientifique et de goût de l'entreprise, comme on en voit encore rarement dans l'Hexagone. Sans doute ce métissage puise-t-il dans ses racines familiales, auprès de parents enseignants-chercheurs, physique des hautes énergies côté père, chimie des agrégats côté mère. Agréger les énergies, au propre comme au figuré : tout son parcours tient dans cette formule physico-chimique. De Rabat, où il est né, au Genopole d'Evry (Essonne), où est implantée sa société, son itinéraire n'a pourtant rien de linéaire. Il a 1 an quand ses parents se rapatrient dans les Yvelines, où il mène une vie  « modeste » ,  avec des résultats scolaires tout aussi  « moyens » .  Il n'entame pas moins des études de médecine et se révèle une « bête à concours » . Au point de se présenter en candidat libre à l'Ecole normale supérieure, où il est reçu en section de biologie, avant de soutenir une thèse en immunologie.  « J'allais , dit-il,  là où le vent me portait. » Le vent du large le mènera jusqu'à Montréal. C'est là qu'il contracte le virus de l'industrie.  « Il y avait,  raconte-t-il,  une effervescence entrepreneuriale inconnue en France. Autour de moi, je voyais des tas de sociétés se créer à partir de labos, portées par des jeunes pleins d'enthousiasme. » Depuis le Québec, il rédige le business plan de sa première entreprise, Biométhodes, une plate-forme d'ingénierie des protéines qui voit le jour en 1997 à Evry et qui emploie toujours une quarantaine de personnes.   Ce ne sera qu'un tour de rodage. Il repart de zéro, l'expérience en plus. Voilà six ans, il se lance, avec son complice Philippe Marlière, pionnier de la biologie de synthèse, dans l'aventure de Global Bioenergies. Objectif :  « Produire des hydrocarbures autrement. »  Autrement, c'est-à-dire en s'affranchissant des ressources fossiles en voie d'épuisement et fortement émettrices de CO2.   La jeune société vise une production « pur sucre » . Ses matières premières sont les déchets de l'industrie sucrière (mélasse de betterave et bagasse de canne), l'amidon du blé et du maïs, ou encore la cellulose des résidus agricoles et forestiers. Tout l'art consiste à les faire digérer par des bactéries au génome reprogrammé, pour les convertir en une famille de gaz tirés du pétrole : isobutène, propylène, butadiène... Avec eux, on peut ensuite faire du plastique, du caoutchouc, du verre organique, du vernis, de la mousse, mais aussi du carburant. A la clé, un marché chiffré, au niveau mondial, en centaines de milliards d'euros.   Marc Delcourt, qui s'affiche  « volontiers écolo » ,  pense à la planète.  « En termes de gaz à effet de serre, le gain est compris entre 0 % et 70 %, selon les végétaux utilisés » ,  assure-t-il. Pour trouver les terres nécessaires, il faut  « mettre fin au gaspillage alimentaire et terminer la réforme agraire » .  Mais il pense aussi à l'addiction de nos sociétés au pétrole et au choc que sera l'inévitable cure de désintoxication.   « Si on ne fait rien aujourd'hui, le jour où on s'apercevra qu'il ne reste plus que dix ans ou vingt ans de réserves de pétrole, ce sera trop tard et on retournera à l'âge de pierre,  prévient-il.  Nous vivons dans un village mondial et, pour la première fois, nous sommes confrontés à un risque de régression générale. »  Bien sûr, il ne prétend pas que ses plantes sucrières sont la panacée. Mais, il en est persuadé, «  les bioénergies sont un morceau de la solution »  , qui passe aussi par  « une nouvelle frugalité »  .   Son vrai ressort est de se vivre en  « guerrier de la biotechnologie »  , décrypte toutefois Philippe Marlière. Il le dépeint en  « sorte de héros, façon Superdupont » ,  car, dit-il,  « il faut aujourd'hui de l'héroïsme pour croire, seul ou presque, au génie français » ,  tant  « vouloir innover et réussir en France sonne comme un oxymore »  .  « Il pense qu'il a un rôle à jouer,  ajoute son partenaire.  Son truc, c'est de lancer des choses, de créer des entreprises, d'inciter d'autres à en créer. » Marc Delcourt le confirme : s'il fait un jour fortune, ce qui pourrait bien lui arriver, il en fera de l'argent utile, fonds d'amorçage ou capital-risque, pour  « faire marcher le système de l'entrepreneuriat »  . Il a déjà transféré ses actions à une holding personnelle dont la vocation est de réaliser des investissements.   L'affaire est bien engagée. Global Bioenergies, qui a levé 37 millions d'euros depuis sa création, emploie aujourd'hui une cinquantaine de personnes. Elle a signé des accords de partenariat avec des géants industriels, comme le chimiste français Arkema ou le constructeur automobile allemand Audi. Des discussions sont en cours avec une centaine d'entreprises intéressées par sa technologie, jusqu'au Brésil et aux Etats-Unis.   En octobre sera mis en service, près de Reims (Marne), un premier pilote industriel, d'une capacité de production annuelle de 10 tonnes d'isobutène. Un deuxième pilote, dix fois plus gros, est prévu sur le site de la raffinerie de Leuna, en ex-RDA. Les futures unités de production, elles, seront encore cent fois plus grandes. Marc Delcourt se voit déjà en roi du pétrole vert.
</article>
<article titre="ILS FERONT LE MONDE - Marc Delcourt Roi du pétrole vert Ce biologiste reconverti en entrepreneur mise sur les plantes pour remplacer les ressources fossiles -" date="2014 07 29" journal="Le Monde">
Depuis peu, il s'est laissé pousser un collier de barbe, où le sel commence à manger le poivre. Manière de masquer son allure d'étudiant attardé -  « je faisais un peu jeune pour lever des fonds »  - et d'inspirer confiance aux investisseurs. Ceux-ci n'ont pourtant pas attendu cet artifice pileux pour lui faire crédit. Global Bioenergies, la start-up qu'il a fondée en 2008 et qui est cotée en Bourse, est la seule en Europe, et l'une des rares au monde, à exploiter un filon neuf : la production d'hydrocarbures biologiques. Il s'agit de remplacer le pétrole d'origine fossile par des ressources végétales renouvelables. De transmuter l'or noir en or vert. Il y va, est-il convaincu, de  « l'avenir de notre civilisation » . Sous ses airs poupins de doux rêveur, Marc Delcourt est en réalité, à 43 ans, un redoutable businessman, doublé d'un visionnaire. Un mélange de culture scientifique et de goût de l'entreprise, comme on en voit encore rarement dans l'Hexagone. Sans doute ce métissage puise-t-il dans ses racines familiales, auprès de parents enseignants-chercheurs, physique des hautes énergies côté père, chimie des agrégats côté mère. Agréger les énergies, au propre comme au figuré : tout son parcours tient dans cette formule physico-chimique.   De Rabat, où il est né, au Genopole d'Evry (Essonne), où est implantée sa société, son itinéraire n'a pourtant rien de linéaire. Il a 1 an quand ses parents se rapatrient dans les Yvelines, où il mène une vie  « modeste » ,  avec des résultats scolaires tout aussi  « moyens » .  Il n'entame pas moins des études de médecine et se révèle une « bête à concours » . Au point de se présenter en candidat libre à l'Ecole normale supérieure, où il est reçu en section de biologie, avant de soutenir une thèse en immunologie.  « J'allais , dit-il,  là où le vent me portait. » Le vent du large le mènera jusqu'à Montréal. C'est là qu'il contracte le virus de l'industrie.  « Il y avait,  raconte-t-il,  une effervescence entrepreneuriale inconnue en France. Autour de moi, je voyais des tas de sociétés se créer à partir de labos, portées par des jeunes pleins d'enthousiasme. » Depuis le Québec, il rédige le business plan de sa première entreprise, Biométhodes, une plate-forme d'ingénierie des protéines qui voit le jour en 1997 à Evry et qui emploie toujours une quarantaine de personnes.   Ce ne sera qu'un tour de rodage. Il repart de zéro, l'expérience en plus. Voilà six ans, il se lance, avec son complice Philippe Marlière, pionnier de la biologie de synthèse, dans l'aventure de Global Bioenergies. Objectif :  « Produire des hydrocarbures autrement. »  Autrement, c'est-à-dire en s'affranchissant des ressources fossiles en voie d'épuisement et fortement émettrices de CO2.   La jeune société vise une production « pur sucre » . Ses matières premières sont les déchets de l'industrie sucrière (mélasse de betterave et bagasse de canne), l'amidon du blé et du maïs, ou encore la cellulose des résidus agricoles et forestiers. Tout l'art consiste à les faire digérer par des bactéries au génome reprogrammé, pour les convertir en une famille de gaz tirés du pétrole : isobutène, propylène, butadiène... Avec eux, on peut ensuite faire du plastique, du caoutchouc, du verre organique, du vernis, de la mousse, mais aussi du carburant. A la clé, un marché chiffré, au niveau mondial, en centaines de milliards d'euros.   Marc Delcourt, qui s'affiche  « volontiers écolo » ,  pense à la planète.  « En termes de gaz à effet de serre, le gain est compris entre 0 % et 70 %, selon les végétaux utilisés » ,  assure-t-il. Pour trouver les terres nécessaires, il faut  « mettre fin au gaspillage alimentaire et terminer la réforme agraire » .  Mais il pense aussi à l'addiction de nos sociétés au pétrole et au choc que sera l'inévitable cure de désintoxication.   « Si on ne fait rien aujourd'hui, le jour où on s'apercevra qu'il ne reste plus que dix ans ou vingt ans de réserves de pétrole, ce sera trop tard et on retournera à l'âge de pierre,  prévient-il.  Nous vivons dans un village mondial et, pour la première fois, nous sommes confrontés à un risque de régression générale. »  Bien sûr, il ne prétend pas que ses plantes sucrières sont la panacée. Mais, il en est persuadé, «  les bioénergies sont un morceau de la solution »  , qui passe aussi par  « une nouvelle frugalité »  .   Son vrai ressort est de se vivre en  « guerrier de la biotechnologie »  , décrypte toutefois Philippe Marlière. Il le dépeint en  « sorte de héros, façon Superdupont » ,  car, dit-il,  « il faut aujourd'hui de l'héroïsme pour croire, seul ou presque, au génie français » ,  tant  « vouloir innover et réussir en France sonne comme un oxymore »  .  « Il pense qu'il a un rôle à jouer,  ajoute son partenaire.  Son truc, c'est de lancer des choses, de créer des entreprises, d'inciter d'autres à en créer. » Marc Delcourt le confirme : s'il fait un jour fortune, ce qui pourrait bien lui arriver, il en fera de l'argent utile, fonds d'amorçage ou capital-risque, pour  « faire marcher le système de l'entrepreneuriat »  . Il a déjà transféré ses actions à une holding personnelle dont la vocation est de réaliser des investissements.   L'affaire est bien engagée. Global Bioenergies, qui a levé 37 millions d'euros depuis sa création, emploie aujourd'hui une cinquantaine de personnes. Elle a signé des accords de partenariat avec des géants industriels, comme le chimiste français Arkema ou le constructeur automobile allemand Audi. Des discussions sont en cours avec une centaine d'entreprises intéressées par sa technologie, jusqu'au Brésil et aux Etats-Unis.   En octobre sera mis en service, près de Reims (Marne), un premier pilote industriel, d'une capacité de production annuelle de 10 tonnes d'isobutène. Un deuxième pilote, dix fois plus gros, est prévu sur le site de la raffinerie de Leuna, en ex-RDA. Les futures unités de production, elles, seront encore cent fois plus grandes. Marc Delcourt se voit déjà en roi du pétrole vert.
</article>
<article titre="Comment le cinquième sucrier mondial a fait du Brésil son nouvel eldorado" date="2014 07 28" journal="Le Figaro, no. 21764">
Dans cette immense usine flambant neuve, située à Palmital, au sud-ouest de São Paulo, les camions se succèdent pour déverser leur chargement de maïs. Quatre cents tonnes par jour, aussitôt nettoyées puis ramollies dans deux cuves de macération avant d'être broyées. Un précieux ingrédient, l'amidon, en est extrait.   Cette amidonnerie de maïs du groupe Tereos, sa 8e usine au Brésil, a été inaugurée mercredi dernier en grande pompe : au rythme de  La Marseillaise , après la bénédiction d'un évêque et en présence, notamment, de l'ancien président Lula. La coopérative française, qui fut le premier groupe sucrier occidental à s'installer au Brésil il y a quatorze ans, a investi 50 millions d'euros dans cette usine. Une cinquantaine de produits destinés au secteur des biens de grande consommation y seront fabriqués, de l'amidon pour l'industrie du papier au glucomalt pour la brasserie, en passant par du sirop de glucose pour la confiserie.  « Avec l'augmentation des classes moyennes, la demande de nos clients industriels pour des produits de plus en plus sophistiqués a explosé au Brésil , explique Alexis Duval, président du directoire de Tereos.  Nous devons être capables de les accompagner en Europe comme au Brésil avec un portefeuille de produits adaptés à leurs besoins. »   Le groupe est convaincu que, malgré le ralentissement de l'économie (les prévisions de croissance pour 2014 ont été revues à la baisse à + 2,5 %), le marché de l'amidon devrait croître au rythme annuel de 5 % d'ici à 2018 (contre + 9 % depuis 2010).   Petrobras actionnaire de la branche internationale Après avoir investi le secteur de la canne à sucre, Tereos s'est lancé dans la fabrication d'amidons et de produits sucrants à partir du manioc (2011) puis du maïs (2014). Il réalise aujourd'hui au Brésil un quart de son chiffre d'affaires et un tiers de son résultat d'exploitation.  « Lorsque nous avons débarqué en 2000, le Brésil faisait peur dans le monde sucrier , se souvient Alexis Duval, qui joua les « conquistadores » dans le pays.  Nous savions qu'il deviendrait incontournable. Nous n'imaginions pas que ça irait aussi vite. Cela nous a permis de franchir une étape à l'international. »  Au début des années 2000, le pays sort tout juste d'une grave crise financière. Le marché du sucre est aux mains d'une multitude de sociétés familiales et reste très manuel. Avec la libéralisation du secteur de la canne et le développement de l'éthanol, notamment dans les voitures flex-fuel, le Brésil s'impose comme le premier producteur et le premier exportateur mondial de sucre.   L'intuition de Philippe Duval, père d'Alexis et patron de Tereos jusqu'en 2012, a été la bonne. Tereos, qui a fait ses armes en créant une coentreprise avec le n° 1 brésilien Cosan, met la main sur Guarani (ex-Béghin Say) en 2002. Alors aux commandes du Brésil, Alexis Duval multiplie les acquisitions en allant convaincre les familles, une à une, de céder leurs usines. La croissance est telle que Tereos décide de financer son développement en introduisant à la Bourse de São Paulo son activité internationale. Dans l'éthanol, il se rapproche du géant Petrobras, qui investit 725 millions d'euros pour devenir actionnaire de Guarani (jusqu'à 45,7 %).   Si Tereos, n° 3 du sucre au Brésil, a acquis la taille critique dans le pays (20 millions de tonnes de canne transformées par an), il n'exclut pas de participer à la consolidation du secteur. Mais il doit s'atteler à améliorer sa compétitivité, entamée par l'augmentation du coût de la main-d'oeuvre et de la logistique. Un programme quinquennal d'investissement de 500 millions de dollars a été lancé en 2010 pour généraliser la mécanisation et doter les exploitations des outils les plus sophistiqués comme des GPS pour piloter automatiquement ses machines.   Pour valoriser les déchets de la canne (la bagasse), Tereos s'est également lancé l'an passé dans la cogénération. L'électricité non utilisée dans la production est vendue au réseau : si cette activité ne représente que 5 % du chiffre d'affaires de Guarani, elle pèse 20 % de son résultat d'exploitation. D'ici à 2016, il devrait quadrupler sa capacité de production.   Reste que Tereos est aujourd'hui pris entre deux eaux au Brésil. Le pays a entamé sa quatrième année d'excédents, avec des prix du sucre toujours orientés à la baisse. Même chose pour ceux de l'éthanol, indexés - de fait - sur les prix de l'essence, qui ont été plafonnés par le gouvernement pour des raisons politiques. Si Tereos reste confiant sur la croissance à long terme du marché mondial du sucre (+ 2,5 % par an), il navigue à vue. D'autant que la sécheresse qui frappe cette année le pays pourrait entraîner un retournement du marché dès l'an prochain. K. L. (à Palmital, État de São Paulo, Brésil)
</article>
<article titre="Marc Delcourt, roi du pétrole vert" date="2014 07 28" journal="Le Monde.fr">
Ce biologiste reconverti en entrepreneur mise sur les plantes pour remplacer les ressources fossiles   Depuis peu, il s'est laissé pousser un collier de barbe, où le sel commence à manger le poivre. Manière de masquer son allure d'étudiant attardé -  « je faisais un peu jeune pour lever des fonds »  - et d'inspirer confiance aux investisseurs. Ceux-ci n'ont pourtant pas attendu cet artifice pileux pour lui faire crédit. Global Bioenergies, la start-up qu'il a fondée en 2008 et qui est cotée en Bourse, est la seule en Europe, et l'une des rares au monde, à exploiter un filon neuf : la production d'hydrocarbures biologiques. Il s'agit de remplacer le pétrole d'origine fossile par des ressources végétales renouvelables. De transmuter l'or noir en or vert. Il y va, est-il convaincu, de  « l'avenir de notre civilisati on ». Sous ses airs poupins de doux rêveur, Marc Delcourt est en réalité, à 43 ans, un redoutable businessman, doublé d'un visionnaire. Un mélange de culture scientifique et de goût de l'entreprise, comme on en voit encore rarement dans l'Hexagone. Sans doute ce métissage puise-t-il dans ses racines familiales, auprès de parents enseignants-chercheurs, physique des hautes énergies côté père, chimie des agrégats côté mère. Agréger les énergies, au propre comme au figuré : tout son parcours tient dans cette formule physico-chimique. De Rabat, où il est né, au Genopole d'Evry (Essonne), où est implantée sa société, son itinéraire n'a pourtant rien de linéaire. Il a 1 an quand ses parents se rapatrient dans les Yvelines, où il mène une vie  « modeste »,  avec des résultats scolaires tout aussi  « moyens ».  Il n'entame pas moins des études de médecine et se révèle une « bête à concours ». Au point de se présenter en candidat libre à l'Ecole normale supérieure, où il est reçu en section de biologie, avant de soutenir une thèse en immunologie.  « J'allais , dit-il,  là où le vent me portait. » Il y avait, raconte-t-il, une effervescence entrepreneuriale inconnue en France »   Le vent du large le mènera jusqu'à Montréal. C'est là qu'il contracte le virus de l'industrie.  « Il y avait,  raconte-t-il,  une effervescence entrepreneu riale inconnue en France. Autour de moi, je voyais des tas de sociétés se créer à partir de labos, portées par des jeunes pleins d'enthousiasm e. » Depuis le Québec, il rédige le business plan de sa première entreprise, Biométhodes, une plate-forme d'ingénierie des protéines qui voit le jour en 1997 à Evry et qui emploie toujours une quarantaine de personnes.   Ce ne sera qu'un tour de rodage. Il repart de zéro, l'expérience en plus. Voilà six ans, il se lance, avec son complice Philippe Marlière, pionnier de la biologie de synthèse, dans l'aventure de Global Bioenergies. Objectif :  « Produire des hydrocarbures autrement. »  Autrement, c'est-à-dire en s'affranchissant des ressources fossiles en voie d'épuisement et fortement émettrices de CO2.   La jeune société vise une production « pur sucre ». Ses matières premières sont les déchets de l'industrie sucrière (mélasse de betterave et bagasse de canne), l'amidon du blé et du maïs, ou encore la cellulose des résidus agricoles et forestiers. Tout l'art consiste à les faire digérer par des bactéries au génome reprogrammé, pour les convertir en une famille de gaz tirés du pétrole : isobutène, propylène, butadiène... Avec eux, on peut ensuite faire du plastique, du caoutchouc, du verre organique, du vernis, de la mousse, mais aussi du carburant. A la clé, un marché chiffré, au niveau mondial, en centaines de milliards d'euros.   Marc Delcourt, qui s'affiche  « volontiers écol o »,  pense à la planète.  « En termes de gaz à effet de serre, le gain est compris entre 0 % et 70 %, selon les végétaux utilis és »,  assure-t-il. Pour trouver les terres nécessaires, il faut  « mettre fin au gaspillage alimentaire et terminer la réforme agrair e ».  Mais il pense aussi à l'addiction de nos sociétés au pétrole et au choc que sera l'inévitable cure de désintoxication.   « Si on ne fait rien aujourd'hui, le jour où on s'apercevra qu'il ne reste plus que dix ans ou vingt ans de réserves de pétrole, ce sera trop tard et on retournera à l'âge de pierre,  prévient-il.  Nous vivons dans un village mondial et, pour la première fois, nous sommes confrontés à un risque de régression général e. »  Bien sûr, il ne prétend pas que ses plantes sucrières sont la panacée. Mais, il en est persuadé, «  les bioénergies sont un morceau de la solution » , qui passe aussi par  « une nouvelle frugalité » .   Vouloir innover et réussir en France sonne comme un oxymore »   Son vrai ressort est de se vivre en  « guerrier de la biotechnologie » , décrypte toutefois Philippe Marlière. Il le dépeint en  « sorte de héros, façon Superdupont »,  car, dit-il,  « il faut aujourd'hui de l'héroïsme pour croire, seul ou presque, au génie français »,  tant  « vouloir innover et réussir en France sonne comme un oxymore » .  « Il pense qu'il a un rôle à jouer,  ajoute son partenaire.  Son truc, c'est de lancer des choses, de créer des entreprises, d'inciter d'autres à en créer. » Marc Delcourt le confirme : s'il fait un jour fortune, ce qui pourrait bien lui arriver, il en fera de l'argent utile, fonds d'amorçage ou capital-risque, pour  « faire marcher le système de l'entrepreneuriat » . Il a déjà transféré ses actions à une holding personnelle dont la vocation est de réaliser des investissements.   L'affaire est bien engagée. Global Bioenergies, qui a levé 37 millions d'euros depuis sa création, emploie aujourd'hui une cinquantaine de personnes. Elle a signé des accords de partenariat avec des géants industriels, comme le chimiste français Arkema ou le constructeur automobile allemand Audi. Des discussions sont en cours avec une centaine d'entreprises intéressées par sa technologie, jusqu'au Brésil et aux Etats-Unis.   En octobre sera mis en service, près de Reims (Marne), un premier pilote industriel, d'une capacité de production annuelle de 10 tonnes d'isobutène. Un deuxième pilote, dix fois plus gros, est prévu sur le site de la raffinerie de Leuna, en ex-RDA. Les futures unités de production, elles, seront encore cent fois plus grandes. Marc Delcourt se voit déjà en roi du pétrole vert.   Voir aussi : le monde de demain selon Marc Delcourt Demain : Ils feront le monde (17/43) Juli Zeh, écrivaine
</article>
<article titre="Albioma : l'avenir est dans la canne à sucre" date="2014 07 23" journal="Boursier (site web)">
Le producteur d'électricité  Albioma  a précisé que le projet d'avenant bagasse/biomasse au contrat d'achat initialement bagasse/charbon de la centrale Galion 2 en Martinique (38 MW) a été transmis par  EDF  à la Commission de Régulation de l'Énergie. La validation du contrat par la commission est attendue au second semestre 2014 et la mise en service de cette centrale électrique de nouvelle génération reste prévue au cours du second semestre 2016.    La centrale Galion 2 utilisera ainsi de la biomasse en complément de la bagasse et ne conservera le charbon que par sécurité. Ce projet devrait mobiliser 180 millions d'euros d'investissement pour une détention cible de 80%. La bagasse est un résidu fibreux de la canne à sucre obtenu après l'extraction du jus pour la production de rhum ou de sucre. Jusqu'ici, les centrales Albioma en Outre-Mer utilisent de la bagasse pendant la campagne sucrière qui dure de 5 à 7 mois mais le reste du temps, ce combustible végétal est remplacé par du charbon.   Sur ce même modèle plus respectueux de l'environnement, Albioma a gagné un appel à projet pour la construction et l'exploitation d'une centrale à biomasse de 40 MW à La Réunion. Ce projet représente un investissement de 45 millions d'euros et sera détenu à 51% par Albioma et à 49% par ses partenaires sucriers. Sa mise en service est prévue au second semestre 2016. Un contrat d'achat long terme d'électricité d'une durée de 25 ans sera signé avec  EDF .    Cette unité sera la première centrale de pointe française fonctionnant essentiellement avec de la biomasse, utilisant du biocarburant de deuxième génération (éthanol de distillerie de mélasse), fourni par les partenaires sucriers du groupe à La Réunion et à l'île Maurice. La validation du contrat d'achat d'électricité par la Commission de Régulation de l'Energie est également attendue au cours du deuxième semestre 2014.
</article>
<article titre="LEAD 1-Bourse-Albioma grimpe, l'activité au Brésil saluée" date="2014 07 23" journal="Reuters économique">
* L'action grimpe de 8% dans des volumes élevés   * CA -9% au S1, résultat net part du groupe +9% hors exceptionnels   * Un nouveau projet au Brésil tous les 12 à 18 mois. (Actualisé avec commentaires et précisions)   PARIS, 23 juillet (Reuters) - L'action Albioma s'inscrit en vive hausse mercredi matin à la Bourse de Paris dans des volumes déjà très étoffés, le marché saluant l'activité au premier semestre du producteur indépendant d'électricité français au Brésil. A 9h32, le titre gagne 8,27% à 18,19 euros dans des volumes représentant 2,3 fois leur moyenne quotidienne des trois derniers mois sur Euronext, quand l'indice CAC Mid &amp; Small s'octroie 0,29% dans des volumes équivalant à 10% de leur moyenne journalière habituelle.   Albioma a confirmé ses objectifs pour les exercices 2014 et 2016, malgré une baisse de 9% du chiffre d'affaires au premier semestre à 166,7 millions d'euros et un recul de 27% de son résultat net part du groupe à 20,7 millions d'euros, imputable, notamment, à deux incidents techniques à La Réunion et à la Guadeloupe.   Hors éléments exceptionnels, le bénéfice semestriel ressort en progression de 9% grâce à une amélioration du résultat financier et de la charge d'impôt ainsi qu'aux résultats de la centrale brésilienne.   "Les bons résultats du Brésil et la sécurisation des tarifs d'ici à 2016 sont des bonnes nouvelles", juge dans un mémo Portzamparc.   L'activité au Brésil s'est traduite par un chiffre d'affaires de 5,4 millions d'euros après 2 mois et 6 jours d'exploitation.   Albioma a précisé avoir vendu à prix fixe environ 60% de sa production électrique au Brésil attendue entre le second semestre 2014 et la fin 2016, profitant des tarifs élevés liés à la sécheresse et au niveau bas des réservoirs hydroélectriques, et a confirmé son intention de développer un nouveau projet dans le pays tous les 12 à 18 mois.   "Albioma publie un chiffre d'affaires au premier semestre (...) supérieur à notre attente grâce à une contribution plus forte que prévu de la centrale brésilienne", souligne dans une note Gilbert Dupont qui visait un chiffre d'affaires de un million d'euros pour le Brésil.   La Société de Bourse a ajouté le titre à sa liste de valeurs moyennes favorites et relevé son conseil d'"accumuler" à "acheter".   En début d'année, l'ex-Séchilienne Sidec avait annoncé le rachat pour 137 millions de reals brésiliens (43 millions d'euros) de la centrale Rio Pardo Termoelétrica, une unité de cogénération - fonctionnant à partir de bagasse (résidu de canne à sucre) et de charbon - située dans l'Etat de Sao Paulo.   Le communiqué:   http://bit.ly/UpTVOi (Alexandre Boksenbaum-Granier, édité par Gilles Guillaume)
</article>
<article titre="Les 10 indicateurs clés de l'environnement - Édition 2014" date="2014 07 17" journal="Rapports - Environnement - MEDDTL">
2 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... COMMISSARIAT GÉNÉRAL AU DÉVELOPPEMENT DURABLE  www.statistiques.developpement-durable.gouv.fr  10 indicateurs clés de l'environnement  Édition 2014 Juillet 2014  Service de l'observation et des statistiques  Repères  10 indicateurs...    Pour voir le rapport :  http://www.developpement-durable.gouv.fr/IMG/pdf/Rep_-_Indicateurs_cles_de_l_environnement.pdf
</article>
<article titre="" La France doit se projeter sur les mers ! "" date="2014 07 9" journal="La Tribune.fr">
La Tribune - Pour beaucoup d'experts, vous êtes le premier homme politique à vous être penché sur la situation des DOM TOM lors de votre passage au ministère du Plan et de l'Aménagement du territoire, de 1981 à 1983, puis à Matignon, de 1988 à 1991. Comment justifiez-vous votre action ? Michel Rocard - En tant que ministre du Plan, j'avais pour mission d'élaborer la programmation de l'avenir économique tricolore ce qui, soit dit au passage, était assez savoureux, François Mitterrand étant un homme du court terme, un homme de « coups ». Les DOM-TOM étaient évidemment concernés par cette mission. Connaissiez-vous la France outre-mer ? Je n'y avais jamais mis les pieds ! Je n'en connaissais que le passé colonial et des cartes postales de plages paradisiaques. Quelle était la réalité ? Les DOM TOM étaient dans une situation déplorable. L'industrie était à peine embryonnaire et l'agriculture déclinait. A l'exception de la Nouvelle-Calédonie, qui affichait une balance commerciale excédentaire, tous ces territoires étaient dans l'impasse économique. Ils importaient entre 90% et 95% de ce qu'ils consommaient. Même les jus de fruits venaient d'ailleurs. Pourtant, la plupart de ces îles possèdent et possédaient des sources d'eau et des fruits abondantes ! Résultat, les excès de population entraînaient d'importants phénomènes migratoires vers la métropole. Ces territoires étaient au bord de l'implosion sociale. Je voyais arriver la catastrophe. Comment en était-on arrivé là ? Trois éléments peuvent être avancés. Comme les importateurs avaient la mainmise sur la plupart des activités économiques, ils avaient tout intérêt à étouffer les initiatives entrepreneuriales. En outre, l'essentiel des ressources fiscales de ces territoires étant assurées par l'octroi de mer, la taxe pesant sur les marchandises transitant par les ports, les Chambres de commerce et d'industrie locales n'étaient en aucune manière à inciter à stimuler l'économie. A ces blocages structurels s'ajoutaient des obstacles institutionnels. Lesquels ? En dressant l'inventaire de la situation économique et sociale des DOM -TOM, j'ai découvert le principe de l'exclusive. Ce principe, qui datait de Louis XIV et de Colbert, empêchait les opérateurs publics mais aussi privés d'avoir des relations administratives et commerciales avec les pays étrangers voisins, et donc avec les ambassadeurs français des pays voisins. Tout devait passer par Paris. C'était aberrant. A cause de ce principe hérité de l'économie de traite, nous avons perdu plusieurs contrats commerciaux qui auraient pu favoriser le développement économique de ces territoires. Quelles mesures avez-vous prises ? C'est en tant que Premier ministre que j'ai mis fin au principe de l'exclusive. Pour ce faire, j'ai créé les secrétariats de coordination, des structures obligatoires au sein desquelles se rencontraient les préfets, les Hauts-commissaires et les ambassadeurs des pays frontaliers. Grâce à ce réseau, la France pouvait commencer à tisser des liens diplomatiques et commerciaux via ses territoires ultra-marins. Les DOM TOM ont-ils été concernés par le processus de décentralisation initié dès l'arrivée de François Mitterrand au pouvoir ? Ils ne pouvaient en aucun cas en être exclus. Dans ce contexte, la loi de 1982 a créé des contrats de plan avec des clauses contractuelles de droit privé. Toutes les régions l'ont signé. Cette loi prévoyait que les Régions avaient l'autorisation de négocier, par exemple, des contrats de développement avec les régions frontalières. La Guadeloupe, la Martinique et La Réunion ont saisi cette opportunité. L'idée sous-jacente était d'autoriser enfin les Régions à négocier avec les puissances étrangères, ce qui était mal acceptée par tous les partis Jacobins, farouchement attachés à la domination de Paris sur le reste des territoires français. Cette politique de désenclavement des territoires ultra-marins a-t-elle portée ses fruits ? Il faut bien admettre que les choses avancent assez doucement. La diffusion d'une culture politique et économique prend du temps. Certaines initiatives sont néanmoins porteuses d'espoir. Grâce au développement de centrales brûlant de la bagasse, la paille issue du traitement de la canne à sucre, l'Ile de la Réunion tente de parvenir à l'indépendance énergétique. Ce savoir-faire ne pourrait-il pas être exporté ? Les travaux et les projets menés par le Cluster maritime français, qui se penche notamment sur les services portuaires du futur, sur la connaissance et l'exploitation des grands fonds marins, sont aussi très prometteurs. Il faut bien que la classe politique et les forces vives de ce pays prennent enfin conscience que ce n'est pas en copiant l'Allemagne que la France restera une grande puissance économique mais en se projetant sur les mers. Ce doit être une priorité. Pourtant, les gouvernements successifs de droite comme de gauche s'acharnent à vouloir dupliquer le modèle allemand, malgré les difficultés industrielles chroniques de la France incitent. Font-ils fausse route ? En partie, je le crains. Les principaux atouts de l'industrie allemande reposent sur la force de leurs filières sidérurgiques et chimiques qui datent de la fin du XIXème siècle et qui structurent une grande partie de l'économie outre-Rhin. Il est très difficile de pouvoir lutter dans ces domaines précis. Mais la France a bien d'autres atouts à faire valoir. C'est bien évidemment le cas dans certains secteurs industriels comme l'aéronautique et le spatial mais aussi et surtout dans le domaine maritime. Il y a tant à faire, à construire, que ce soit dans le domaine énergétique, le développement durable ou l'alimentaire. Quel rôle peut jouer le ministère des DOM TOM ? C'est une question très importante. Sur le plan institutionnel, les DOM-TOM payent beaucoup les conséquences du fait que leurs affaires soient gérées par un ministère des DOM-TOM qui est trop petit pour être efficace et pour assurer le développement social et économique de ces territoires. Ce ministère ne doit plus être un ministère de gestion mais un ministère de mission avec des objectifs de développement bien précis. Vous êtes ambassadeur de France chargé des négociations internationales relatives aux pôles Arctique et Antarctique. Quels sont les intérêts de la France dans ces zones ? Je vais être clair. En Arctique et en Antarctique, la France mène exclusivement une politique d'intérêt général. En Arctique, en permettant l'exploitation de sous-sols marins jusqu'ici inaccessibles, la fonte des glaces entraîne des problèmes qui ne se posaient pas avant. Savez-vous que l'Arctique regorge de matières premières, que dans son sol se trouverait 30% des réserves mondiales de gaz naturel et de 14% des réserves de pétrole ? Cette richesse encore inexploitée attire bien des convoitises. Dans ce contexte, il faut trouver des solutions collectives pour résoudre les problèmes d'intérêts nationaux puisque, à terme, une fois les revendications territoriales acceptées, il ne restera que 10% d'eaux libres. C'est le sens de ma mission. En Antarctique, outre des travaux de recherche dans les deux bases installées en Terre Adélie et à Concordia, la France mène avec les marines américaine et australienne une mission de préservation de l'environnement naturel dans le cadre du traité de l'Antarctique.    Lire aussi :  > > Pourquoi la France doit enfin se doter d'une politique maritime
</article>
<article titre="Résumé et Vision 2024 Le secteur électrique" date="2014 07 3" journal="Rapports - Banques et institutions financières - Banque Mondiale">
18 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... 237  14. Le secteur électrique  I. Résumé et Vision 2024  La vision pour Madagascar à l’horizon 2024 serait l’accroissement significatif du taux d’accès au service de l’électricité...    Pour voir le rapport :  http://www-wds.worldbank.org/external/default/WDSContentServer/WDSP/IB/2014/07/07/000350881_20140707114006/additional/047856072_201407225003215.pdf
</article>
<article titre="Décret 542-2014, 18 juin 2014 Loi sur la qualité de l’environnement (chapitre Q-2)" date="2014 06 25" journal="Rapports - Environnement - MDDEFP">
28 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... Partie 2 GAZETTE OFFICIELLE DU QUÉBEC, 2 juillet 2014, 146e année, no 27 2235  Règlements et autres actes  Gouvernement du Québec  Décret 542-2014, 18 juin 2014 Loi sur la qualité...    Pour voir le rapport :  http://www.mddefp.gouv.qc.ca/matieres/regime-compensation/D542_2014.pdf
</article>
<article titre="AREVA : livre une centrale biomasse au thaïlandais U-Thong Bio Power" date="2014 06 23" journal="Boursier (site web)">
AREVA  annonce avoir livré avec succès une centrale biomasse à son client thaïlandais U-Thong Bio Power Co., Ltd. Construite par AREVA et son partenaire Ensys, l'installation est implantée dans la province thaïlandaise de Suphanburi.   "D'une puissance de 9,9 MW, cette centrale est alimentée par de la balle de riz et de la bagasse", explique le Groupe. "Elle fournira de l'électricité à environ 8.000 foyers". Selon les termes du contrat, AREVA a assuré la conception de la centrale, des prestations d'ingénierie, la fourniture des principaux équipements ainsi que la supervision des essais et de la mise en exploitation. Ensys a fourni les équipements auxiliaires et assuré des prestations de construction et de montage.   AREVA a déjà livré deux centrales biomasse similaires dans le pays et construit actuellement deux autres unités en Thaïlande et aux Philippines.
</article>
<article titre="Marie-Galante (Guadeloupe) - 12 000 hab. Une île à énergie positive au large de la Guadeloupe" date="2014 06 16" journal="La Gazette des communes, no. 2226">
Il est toujours plus facile d'engager la transition énergétique dans les îles. En effet, ces territoires isolés ne bénéficient pas de l'« effet réseau » : ils ne peuvent compter que sur eux-mêmes pour assurer en permanence l'offre et la demande d'électricité. C'est encore plus vrai quand, comme Marie-Galante, on se trouve en situation de « double insularité » : l'île, de 158 km2, est une dépendance de la Guadeloupe, à 30 km en bateau. Certes, un câble électrique, qui exclut toute idée d'autarcie, les relie et règle le problème de l'intermittence des sources locales d'électricité renouvelable.    Graal décarboné Mais l'important gisement d'énergies vertes de Marie-Galante, bien supérieur aux besoins des quelque 12 000 habitants, laisse penser qu'un jour, l'îlot pourrait être parfaitement autonome et se servir du câble sous-marin seulement pour exporter du courant 100 % décarboné vers la Guadeloupe « continentale ». Ce qui aiderait le DOM à atteindre le graal : « l'autonomie électrique en 2030, sans hydrocarbures importés », annonce Céline Jules-Soret, chef du service « énergie » au conseil régional. C'est dans ce but qu'en 2012, le pôle de compétitivité Synergîle a monté le projet Modèle applicatif d'île autonome (Maia). « Eolien, solaire, effluents de distilleries... il s'agit de déverser toutes les énergies renouvelables de Marie-Galante dans un même câble électrique, après autoconsommation pour les besoins locaux et stockage des surplus dans des batteries », résume François Monroux, responsable de la zone Antilles chez Akuo Energy, qui exploite sur place une centrale solaire au sol de 2 mégawatts, stockant ainsi une partie de sa production quotidienne. Non loin de là, la société Quadran a démantelé un ancien parc éolien de 1,5 MW pour en ériger un de 2,5 MW, équipé, lui aussi, de batteries.    Maia est devenu un véritable projet de territoire, impliquant les collectivités (les trois communes de l'île réunies dans la communauté de communes de Marie-Galante, CCMG), lorsque s'y est greffée une vieille idée : relancer la sucrerie-rhumerie de l'île en l'adossant à une centrale thermique, qui, en sus d'électricité, lui fournirait de la chaleur... avec ses propres déchets : la bagasse (lire l'encadré). L'investisseur, la société Albioma, l'avait imaginée au charbon, mais a dû la convertir à la biomasse, sous la pression des élus locaux, qui attendent l'équipement pour 2018. Le président de la communauté de communes de Marie-Galante, Harry Selbonne, parle de « victoire politique dans un combat sans merci », qui s'est jouée dans le bureau du ministre de l'Outre-mer.   Préfiguration Dans cette aventure, le stockage constitue le paramètre clé, plus que la ressource : lisser une production intermittente, c'est-à-dire variable selon la météo, s'avère fondamental pour imprimer une transition énergétique en profondeur. En effet, si le gestionnaire du réseau électrique a l'assurance de recevoir une quantité précise de kilowattheures tous les jours, il n'a plus de raison de poser des limites à l'insertion des énergies renouvelables dans le réseau (30 % en Guadeloupe) ; ni de conserver des centrales au fioul ou au charbon, polluantes et onéreuses, chargées de démarrer à la moindre panne de vent ou de soleil. Ainsi conçu, Maia préfigure donc les « smart grids », ces réseaux électriques intelligents.
</article>
<article titre="Plus de crédit pour Albioma" date="2014 06 16" journal="Environnement Magazine hebdomadaire, no. 21">
Albioma finance son plan de développement dans la biomasse. L'exploitant de centrales bagasse-charbon a émis 80 millions d'euros d'obligations auprès d'investisseurs institutionnels. Le groupe a également renouvelé ses lignes de financements bancaires pour un montant de 40 millions d'euros.
</article>
<article titre="Gaffe à l'orthographe !" date="2014 06 12" journal="Pèlerin, no. 6863">
1 La lecture tu pratiqueras   Si elle ne suffit pas pour nos deux champions, c'est l'une des recettes imparables pour entretenir son orthographe. « Je suis un lecteur assidu depuis l'âge de 4 ans », raconte Antoine, par ailleurs enfant précoce. Le secret : dévorer beaucoup et de tout. « Je lis des BD, des romans (le dernier en date,  Inconnu à cette adresse,  de Kressmann Taylor) des pièces de théâtre, des livres et revues scientifiques, comme  Science &amp; Vie Junior.  Tous les jours, je feuillette quelques pages du  Guinness Book World Records 2014.  Il m'arrive parfois de carrément lire le dictionnaire, même en classe, où je m'ennuie beaucoup. Les profs comprennent et me laissent faire. Je joue au Scrabble sur ma tablette. » Rien de tel pour enrichir son vocabulaire. « Mais attention, ça ne m'empêche pas d'aimer les jeux vidéo ou de regarder la télévision quand j'ai le temps », précise Antoine. Thibaut, 18 ans, passionné de théâtre, qui achève sa première année de médecine, ajoute : « La lecture seule ne suffit pas. Le réflexe à acquérir, c'est de pratiquer la lecture active. Dès que l'on rencontre un mot dont le sens ne nous est pas familier, aller vérifier sa définition, ne pas se contenter de deviner ce qu'il veut dire en fonction du contexte. C'est parfois contraignant mais l'effort est payant », confie-t-il. Évelyne Vernisse, membre du projet Voltaire  (lire l'encadré),  formatrice et coach en communication écrite en entreprise, confirme : « Cinq minutes de lecture active par jour suffisent, sur n'importe quel support : roman, essai, magazine. Interrogez-vous, décortiquez les phrases : que signifie ce mot ? Pourquoi l'auteur a-t-il utilisé cet accord de l'adjectif ? Car l'orthographe est autant lexicale (la manière dont s'écrivent les mots) que grammaticale (la manière dont les mots s'accordent et se relient entre eux). » Thibaut prévient tout de même en riant : « Ce n'est pas infaillible. L'an dernier, en finale, j'ai chuté sur le mot "concomitamment". Et je n'ai su écrire l'expression "dès potron-minet", que je n'avais jamais vue nulle part avant, que par pure chance. Idem pour le terme "bagasse", le résidu fibreux de la canne à sucre. Maintenant, c'est bon, je suis incollable sur ces deux mots. » 2 Les règles de base tu reverras   Antoine : « Pour réviser en m'amusant, je fais régulièrement des tests de grammaire, d'orthographe et de conjugaison sur Internet. Je m'entraîne avec les applications du projet Voltaire. « Il y a pléthore de sites de ce genre, ludiques et pédagogiques, indique Évelyne Vernisse. Mais j'en recommande particulièrement deux, l'un québécois, celui du Centre collégial de développement de matériel didactique, à Sherbrooke (www.ccdmd.qc.ca) et l'autre, luxembourgeois : w3.restena.lu/amifra. Côté livre, je suis une grande utilisatrice de  Bescherelle, l'essentiel.  » « La pratique du latin et du grec au collège m'a beaucoup aidé en me permettant de décortiquer un mot grâce à son étymologie. Savoir, par exemple, qu'enthousiasme signifie "Dieu ( theo  en grec) en soi" permet d'éviter l'oubli du h après le t. On parle de langues mortes, pourtant, elles me semblent bien vivantes », plaide Thibaut.   3 À la rigueur tu t'accrocheras   « Je fais très attention à l'orthographe quand j'écris des mails, des textos, ou quand je chatte (discussion en ligne). J'écris rarement en abrégé, même à mes amis. Ma mère, qui est correctrice professionnelle, m'a toujours conseillé d'éviter le langage SMS autant que possible, afin de ne pas régresser », raconte Antoine. « J'évite de céder à la facilité et de me reposer sur les correcteurs automatiques sur mon ordinateur ou mon mobile, poursuit Thibaut. Les machines ne sont pas infaillibles. En cas de doute, je vérifie toujours. » Et cela, même si une étude publiée en mars soulignait l'absence de corrélation entre l'usage de l'écriture des SMS par des jeunes de 12 ans et le niveau en orthographe. La lettre « c » pour « c'est », « 2m1 », pour « demain », « g t » pour « j'étais ». Selon les chercheurs, l'écrit traditionnel et l'écriture SMS appartiennent à deux registres qui ne s'influencent pas.   « Chez les adultes, je ne suis pas certaine que le niveau d'orthographe baisse, note Évelyne Vernisse. Mais aujourd'hui, on échange beaucoup par écrit : via les SMS, les mails, les réseaux sociaux. Résultat : les défaillances orthographiques deviennent plus visibles qu'avant. En entreprise, cela finit par devenir un vrai problème. » « Envoyer une lettre de motivation bourrée de fautes peut freiner la recherche d'emploi », souligne Thibaut, qui souhaite devenir kinésithérapeute. Pire, d'après une étude de TextMaster, une start-up proposant des outils de correction, 90 % des mails professionnels contiennent au moins une faute. Et sur un site de commerce en ligne, une seule coquille pourrait diviser les ventes par deux. « De façon plus personnelle, à travers la façon de s'exprimer à l'écrit, on fait aussi passer une certaine image de soi », continue Évelyne Vernisse. « Écrire avec des termes précis garantit un minimum de rigueur et une bonne articulation de la pensée. Et puis c'est une marque de respect envers son interlocuteur », ajoute Thibaut. Bien lui écrire, comme bien lui parler, c'est lui montrer qu'il compte. Alice Le Dréau. Illustration : Jérôme Sié   Le certificat Voltaire   Il est, pour l'orthographe, l'équivalent du TOEFL (Test of English as a Foreign Language) pour l'anglais : un justificatif de niveau et un document de référence de plus en plus pris en compte par les entreprises, comme un « + », sur le CV. Lancé par une société lyonnaise, Wooz, noté sur 1000 (« Si vous atteignez 700, vous avez un bon niveau », précise Évelyne Vernisse, membre du conseil d'experts travaillant sur le projet), l'examen se déroule en salles, dans des centres certifiés, ce qui lui confère une valeur académique. Il est constitué pour environ 65 % de règles grammaticales, 15 % de règles sémantiques, 15 % de règles lexicales, et pour 5 % de règles syntaxiques. À quelques exceptions près, il s'attache aux difficultés pas ou mal gérées par les correcteurs orthographiques.
</article>
<article titre="Mettez des algues dans votre moteur" date="2014 06 3" journal="Bilan (blogues)">
En reportage au Brésil, on réalise comment cinq ans de crise financière, et de pétrole cher, peuvent transformer une économie. En 2008, il n'était question que d'éthanol. Mais cette essence renouvelable extraite de la canne à sucre est désormais en crise.   Plus encore que les découvertes de pétrole offshore par Petrobras, le maintien des prix de l'éthanol par le gouvernement pour contenir l'inflation a provoqué les premières faillites de distilleries. A l'usine de Sao Joao, au nord de Sao Paulo, la bagasse - la paille de canne - brûle sans discontinuer depuis 1944 pour chauffer les alambics. Les cheminées crachent toujours autant de CO2. L'autre grand déchet du processus, la vinasse, produite à raison de 12?? litres pour un litre d'éthanol, est devenue un polluant encombrant. Le miracle de l'éthanol brésilien, qui avec seulement 2% de la surface agricole produit 50% de l'essence du pays, n'est pas victime, comme le maïs américain, d'une concurrence avec l'alimentaire. Simplement, son inefficience l'a conduit dans une impasse.   Est-ce pour autant la fin de l'histoire? Certes, les biofuels pâtissent de l'effondrement global des investissements dans les technologies environnementales: moins 11% l'an dernier après un recul de 10% en 2012. Mais des leçons ont été tirées. Elles ont influencé la recherche. Les démarches sont moins productivistes et sont davantage intégrées à une série de processus industriels pour les rendre plus efficaces ou se servir des déchets pour les valoriser. Divers projets suisses au Brésil témoignent de cette approche inspirée par l'écologie industrielle.   A l'Université fédérale de San Carlos, dans l'Etat de Sao Paulo, Algae, une start-up du groupe Ecogeo     Normal    0    21    false    false    false    FR-CH    X-NONE    X-NONE   que vient de racheter le zurichois Ernst Basler?+?Partner, a sélectionné parmi une quarantaine de micro-algues des espèces cultivables en se nourrissant de vinasse. Le projet est de les valoriser d'abord en tant que complément alimentaire pour l'aquaculture puis comme source de biocarburant. Fondateur d'EcoGeo, Ernesto Moeri considère que «le Brésil pourra ainsi se passer complètement d'énergies fossiles dans vingt ans».   Un climat favorable   Dans sa ferme expérimentale de l'Etat de Paraíba dans le nord-est du pays, Pierre Landolt, le président de la Fondation Sandoz, partage le même optimisme.    Dans des réservoirs de 10 000 litres, les biologistes de sa Fazenda Tamandua cultivent une microalgue, la chlorelle, d'où pourraient être extraits 12? litres de diesel par cuve et par semaine.   Certes, le projet se bat avec une évaporation importante dans cette région semi-aride. Mais la possibilité de se servir du CO2 émis par les usines d'éthanol comme nutriment ou d'utiliser les restes de matière sèche comme aliments pour le bétail intéressent les industriels.    Fort de ce constat, Gilles Maag, un doctorant de l'Institut de technologie énergétique de l'ETH Zurich, implante actuellement sa start-up Sunbiotec     Normal    0    21    false    false    false    FR-CH    X-NONE    X-NONE   au Brésil. Il propose un processus susceptible de transformer toute la canne, bagasse incluse, en carburant. Pour cela, il s'est attaqué à l'un des problèmes inhérents à la transformation en hydrocarbures de n'importe quelle source de carbone: la synthèse Fischer-Tropsch.   Utilisé par l'Allemagne nazie pour produire du diesel à partir de charbon, ce procédé butait sur la nécessité de chauffer à très haute température des déchets agricoles pour craquer les chaînes d'hydrogène et de carbone avant de les recomposer en hydrocarbures. Le recours à des concentrateurs solaires prévu par Sunbiotec contourne astucieusement cet obstacle.   Il y a une dernière raison au come-back des biocarburants de seconde génération au Brésil. Outre l'abondance de soleil et de déchets agricoles, les entrepreneurs environnementaux suisses et européens y trouvent un climat moins détestable autour des cleantechs que sur le Vieux-Continent. Et pas la concurrence des pétroles et des gaz de schiste comme aux Etats-Unis.   Publié le 3 Juin 2014 à 17:30  Cet article est paru dans Bilan (blogues)
</article>
<article titre="Pour un soutien actif au sucre des DOM après la fin des quotas (rapport)" date="2014 05 7" journal="AFP Infos Economiques">
Paris, 7 mai 2014 (AFP) - - Le marché du sucre dans les départements d'Outre-Mer, essentiel à l'économie locale, aura besoin d'un soutien actif de l'Etat, notamment financier, pour encaisser la suppression des quotas programmée par l'Europe fin 2017, insiste un rapport parlementaire.   Ce "Rapport d'information" présenté mercredi au nom de la Délégation aux Outre-Mer préconise de porter la subvention de l'Etat à la filière sucre de 90 millions à 128 millions d'euros, de solliciter EDF pour augmenter le prix d'achat de la bagasse, de protéger les terres agricoles soumises à une forte pression foncière dans les îles et de créer un Observatoire chargé de surveiller le marché mondial, particulièrement volatile. EDF achète actuellement (en 2011) 11,05 euros la tonne de bagasse, amas végétal (tiges de canne à sucre,..) : en la portant à 14 euros, le gain serait de 3 millions d'euros supplémentaire pour l'industrie estiment les rapporteurs. Les résidus de canne pour alimenter les centrales thermiques représentent 10 à 12% de la consommation d'électricité en Guadeloupe et à la Réunion, rappellent-ils.   Le rapport suggère encore d'engager des démarches pour obtenir une IGP, Indication géographique protégée, au niveau européen pour le sucre roux des DOM - comme la Réunion l'a déjà fait - et de batailler pour maintenir des tarifs douaniers sur le sucre roux importé de pays tiers (hors UE) pour protéger la production nationale qui s'élève à 2,8 millions de tonnes.   La fin des quotas, décidée par Bruxelles sur le principe en 2006 et reportée l'an dernier de deux ans à 2017 à la demande de la France va avoir pour effet "infiniment probable" une chute des cours européens qui vont s'aligner sur les cours mondiaux, prévient le rapport. Car le système garantissait à la fois l'achat en cas de mévente et des prix de soutien donc un revenu aux industriels.   Si les industriels les plus compétitifs (ceux de la betterave) peuvent compter profiter de la fin des quotas d'exportation, ceux des DOM sont handicapés par les coûts de production, de transports, et par l'étroitesse des marchés, énumèrent les auteurs.   Malgré ces handicaps, la production de canne occupe 57% de la surface agricole utile (SAU) à la Réunion et 45% en Guadeloupe (16% en Martinique). Les aides publiques à l'hectare s'élèvent à 3.760 euros dans les DOM contre 300 en moyenne pour les grandes cultures en métropole. Mais la surface moyenne des exploitations y plafonne à 5 ha contre 119 en métropole.   En outre, 7% de la récolte de canne est affecté aux distilleries pour la production de rhum, une "production de prestige" très liée également à l'identité des DOM.   ach/ih/ide
</article>
<article titre="Pour un soutien actif au sucre des DOM après la fin des quotas (rapport)" date="2014 05 7" journal="AFP Infos Françaises">
Paris, 7 mai 2014 (AFP) - - Le marché du sucre dans les départements d'Outre-Mer, essentiel à l'économie locale, aura besoin d'un soutien actif de l'Etat, notamment financier, pour encaisser la suppression des quotas programmée par l'Europe fin 2017, insiste un rapport parlementaire.   Ce "Rapport d'information" présenté mercredi au nom de la Délégation aux Outre-Mer préconise de porter la subvention de l'Etat à la filière sucre de 90 millions à 128 millions d'euros, de solliciter EDF pour augmenter le prix d'achat de la bagasse (résidu des tiges de canne à sucre), de protéger les terres agricoles soumises à une forte pression foncière dans les îles et de créer un Observatoire chargé de surveiller le marché mondial, particulièrement volatile. EDF achète actuellement (en 2011) 11,05 euros la tonne de bagasse: en la portant à 14 euros, le gain serait de 3 millions d'euros supplémentaire pour l'industrie estiment les rapporteurs. Les résidus de canne pour alimenter les centrales thermiques représentent 10 à 12% de la consommation d'électricité en Guadeloupe et à la Réunion, rappellent-ils.   Le rapport suggère encore d'engager des démarches pour obtenir une IGP, Indication géographique protégée, au niveau européen pour le sucre roux des DOM - comme la Réunion l'a déjà fait - et de batailler pour maintenir des tarifs douaniers sur le sucre roux importé de pays tiers (hors UE) pour protéger la production nationale qui s'élève à 2,8 millions de tonnes.   La fin des quotas, décidée par Bruxelles sur le principe en 2006 et reportée l'an dernier de deux ans à 2017 à la demande de la France va avoir pour effet "infiniment probable" une chute des cours européens qui vont s'aligner sur les cours mondiaux, prévient le rapport. Car le système garantissait à la fois l'achat en cas de mévente et des prix de soutien donc un revenu aux industriels.   Si les industriels les plus compétitifs (ceux de la betterave) peuvent compter profiter de la fin des quotas d'exportation, ceux des DOM sont handicapés par les coûts de production, de transports, et par l'étroitesse des marchés, énumèrent les auteurs.   Malgré ces handicaps, la production de canne occupe 57% de la surface agricole utile (SAU) à la Réunion et 45% en Guadeloupe (16% en Martinique). Les aides publiques à l'hectare s'élèvent à 3.760 euros dans les DOM contre 300 en moyenne pour les grandes cultures en métropole. Mais la surface moyenne des exploitations y plafonne à 5 ha contre 119 en métropole.   En outre, 7% de la récolte de canne est affecté aux distilleries pour la production de rhum, une "production de prestige" très liée également à l'identité des DOM.   ach/ih/jag
</article>
<article titre="Traitement des eaux usées : des réponses spécifiques à l'Outre-mer ASSAINISSEMENT" date="2014 04 9" journal="La Gazette des communes">
Ce n'est un secret pour personne : l'Outremer accuse un retard important en termes d'infrastructures d'assainissement. Au mieux, le raccordement au réseau collectif concerne environ 40 % de la population pour la Réunion, la Martinique et la Guadeloupe. À Mayotte, département d'Outre-mer (Dom) depuis 2011, " au moins 80 % des foyers mahorais ne sont pas raccordés " selon Michel Charpentier, président des Naturalistes de Mayotte. En Nouvelle-Calédonie, seulement 13 % le sont. De nombreuses zones sont donc en assainissement non collectif (ANC), plus ou moins performants ; les eaux usées domestiques finissent souvent en canalisation directe dans la cour. La mise en oeuvre des services d'ANC (Spanc) débute cependant. Toutes les communes de la Réunion en sont pourvues. En Guadeloupe, sept Spanc sont créés et trois sont opérationnels. En Martinique, les zonages sont délimités. " La réhabilitation de l'existant est difficile car les revenus des propriétaires sont souvent insuffisants pour mettre en place un système efficace " constate Jean-Max Coranson-Beaudu, directeur de la régie de Fort-de-France (Martinique), Odyssi. Assainissement : un retard important à rattraper   Dans les Dom , la législation européenne s'applique au même titre que dans les départements métropolitains. Conformément à la DCE, ces départements ultramarins disposent tous de Sdage. Les objectifs et échéances de la Deru sont les mêmes. Cependant, les équipements sont souvent non conformes, défectueux ou inexistants. Ainsi, sur les 74 stations d'épuration (Step) prioritaires définies dans le plan d'action 2012- 2018 du ministère de l'Écologie, dix sont situées dans les Dom (Mayotte non compris).   Depuis quelques années, de gros investissements sont menés pour rattraper ce retard. Entre 2008 et 2011, l'Onema a engagé   50,7 millions d'euros de subventions en assainissement en Outre-mer. Aujourd'hui, la Réunion est la plus avancée des Dom dans la mise aux normes. Les programmes opérationnels européens 2007-2013 ont permis de mobiliser près de 170 millions d'euros de subventions. " À l'horizon 2014, la totalité des agglomérations devraient être mises en conformité avec la Deru, à l'exception de la Step de Saint-Louis " déclare Denys Le Petit, de la Deal Réunion. " Quand tous les travaux en cours seront achevés, ce département aura le parc le plus jeune et le plus performant de France " . En Guadeloupe, depuis 2010, environ 23 millions d'euros de travaux ont été réalisés . À la Réunion et aux Antilles, les chantiers avancent sur les Step urbaines, mais il reste beaucoup de travail sur celles inférieures à 2 000 équivalents habitant et sur les réseaux de collecte. En Guyane, depuis septembre, les propriétaires ont l'obligation de se raccorder au réseau d'assainissement collectif dès que celui-ci est mis en place. À Mayotte, il n'existe que deux Step ! Cinq sont en projet. Ailleurs, cela démarre doucement. En Polynésie française, quatre programmes d'assainissement collectif public sont en cours sur huit communes pour quarante-huit existantes . Mais comme en Nouvelle-Calédonie, les financements ne sont pas bouclés.   Des spécificités techniques liées au climat...   Les spécificités dans le traitement des eaux usées en Outre-mer sont nombreuses. Tout d'abord, la température des eaux usées est supérieure à la métropole et atteint 24 à 26 °C. Ceci entraîne des effets positifs et négatifs. Positifs tout d'abord, car le rendement de la dénitrification sur les boues activées est proportionnel à la température. " On peut donc prévoir un bassin plus petit pour cette étape, mais a contrario cela nécessite d'amener plus d'oxygène " déclare Philippe Drouet, directeur de l'ingénierie pour la société Stereau. De même, la température influe positivement sur le débit de la filtration membranaire. Négatifs ensuite, car l'élévation de la température des effluents est un des facteurs aggravants pour la production d'hydrogène sulfuré (H2S). Outre sa dangerosité, ce gaz entraîne une corrosion des matériaux (acier, béton), accéléré également par la proximité avec la mer, la présence de sel dans les réseaux et l'humidité. " Un ouvrage conçu pour cinquante ans ne dure en fait que vingt-cinq à trente ans ici " constate Jean-Max Coranson-Beaudu. " Pour la partie amont des Step, nous devons utiliser un béton spécifique avec de la résine, qui a un coût plus élevé. Depuis deux ans, nous le stipulons dans le cahier des charges. "   En contrepartie, grâce au soleil, presque toutes les nouvelles Step sont équipées de panneaux photovoltaïques, l'énergie produite étant vendue à EDF. Les serres de séchage solaire des boues sont également appropriées et se développent ; deux installations ont été créées en 2011 et 2012 à la Réunion et toutes les nouvelles l'intègrent, selon la Deal.   ...et à l'éloignement   Tous les matériaux, les pièces et parfois les compétences pour la construction des Step doivent être importés. Cela implique des délais d'acheminement à prévoir dans l'organisation du chantier. Sans parler des décalages horaires. Tout cela entraîne un coût de revient plus élevé, ...   ... de l'ordre de 25 % , en comptant les frais de transport et les taxes douanières. En Martinique, cela se répercute sur le prix de l'eau, qui est dans la fourchette haute de celui de la métropole. L'éloignement engendre également des difficultés d'approvisionnement en réactifs (chaux, chlorure ferrique) : " nous les recevons en conteneur ou en big bag, et c'est à nous de les gérer " explique Philippe Drouet. " Il est nécessaire d'anticiper les pannes, d'avoir du matériel en double et du stock de pièces. En outre, l'eau de javel perd de son efficacité de façon importante au cours du transport. Pour y remédier, la Step du Grand Prado va la fabriquer in situ par électrolyse de sels " complète Éric Le Corre d'Egis Eau, maître d'oeuvre de ce chantier exemplaire qui sera achevé cette année.    Un environnement plus fragile à protéger   La majorité des Dom se trouve dans un environnement fragile (lagon, mangrove, cours d'eau à faible débit, zone maritime protégée). La Martinique, par exemple, est entourée de récifs coralliens, très sensibles non seulement à la pollution (matières en suspension, pesticides, nitrates, etc.), mais aussi aux variations de température. Ces départements dépendent souvent d'activités maritimes liées à la qualité du milieu aquatique : pêche, aquaculture, tourisme, nautisme, etc. " Nous devons donc être très performants sur la qualité des rejets des stations " souligne Jean-Max Coranson-Beaudu. De fait, depuis quelques années, des technologies plus complexes avec des procédés membranaires se développent à la Réunion et aux Antilles. " En Nouvelle-Calédonie, où le lagon est exceptionnel, les trois stations en construction font appel aux bioréacteurs à membrane (BRM). Outre la qualité des rejets, ils nécessitent moins de réactifs " déclare Pierre-Yves Pouliquen, président de Synteau, syndicat national des entreprises du traitement de l'eau. Les traitements tertiaires incluent également parfois la désinfection UV pour la réutilisation des eaux usées (Reuse) pour l'arrosage des espaces verts. La station de l'Étang Salé (Réunion), par exemple, a mis en oeuvre cette technologie.   Des matériaux résistants aux séismes et aux cyclones   La Réunion, comme les Antilles, sont situées sur des zones sismiques et cycloniques qui obligent à l'emploi de matériaux plus résistants (ferraillage, contreventement, etc.), dits " tropicalisés ". Les microséismes impactent les canalisations : déboîtement, entrée d'eaux claires parasites (60 % en Guadeloupe). Le renouvellement des réseaux doit donc être plus fréquent. La roche basaltique qui constitue le sol est également plus difficile à fracturer pour enterrer les canalisations et demande d'avoir recours à un génie civil spécifique et plus coûteux.   Enfin, la géographie impose souvent la construction de multiples petites unités d'assainissement, à cause du découpage des côtes, du relief et du peu d'espace disponible.   Autre particularité, les réseaux sont plus jeunes qu'en métropole (environ trente ans de moyenne d'âge) et sont conçus en séparatif. Ainsi, la Martinique est à 98 % en réseau séparatif, ce qui fait que les eaux pluviales ne transitent pas par les stations d'épuration. Crucial en cas de fortes pluies, en période cyclonique par exemple.
</article>
<article titre="Expansion Albioma transpose au Brésil le modèle qui a fait son succès aux Antilles" date="2014 04 5" journal="Investir-Le Journal des finances, no. 2100">
Albioma a confirmé, mardi, sa première acquisition au Brésil, à Rio Pardo dans l'Etat de São Paulo, annoncée début mars. L'ex-Séchilienne Sidec a acheté pour 43 millions d'euros une centrale thermique qui produit de l'électricité (revendue sur le réseau) et de la vapeur pour la sucrerie voisine. Elle brûle uniquement de la bagasse, le résidu fibreux de la canne à sucre que l'on récolte presque toute l'année au Brésil. D'autres acquisitions devraient suivre, au rythme d'une opération tous les douze à dix-huit mois. Cette centrale de Rio Pardo « va contribuer tout de suite aux résultats d'Albioma », a assuré son président, Jacques Pétry. Toutefois, 30 millions vont y être investis pour augmenter ses capacités et sa productivité. Les centrales brésiliennes ne produisent que 40 kWh par tonne de canne, contre 120 kWh pour les unités d'Albioma ! Le groupe occupe un positionnement original en matière de production d'électricité, encore renforcé par la stratégie mise en oeuvre par Jacques Pétry depuis son arrivée, fin 2011. Les centrales thermiques d'Albioma, qui assurent 88 % de son chiffre d'affaires, sont situées dans des régions sucrières (Antilles, île de La Réunion) où elles fonctionnent à la bagasse durant la récolte de canne et au charbon le reste du temps. Le groupe exploite également des fermes solaires dans des zones très ensoleillées : Martinique, La Réunion, Guyane et sud de l'Europe.   Albioma bénéficie outre-mer de prix garantis élevés pour la revente de son électricité à EDF, ce qui lui assure une rentabilité enviable. Favorisée aussi par un très bon taux de disponibilité des centrales thermiques (à 92,3 %), la marge d'Ebitda (excédent brut d'exploitation) du groupe a atteint 36,7 % l'année passée, contre 22,2 % chez EDF.   Face à la performance du modèle, Jacques Pétry a souhaité l'exporter, en érigeant en priorité voilà un an le Brésil, grand pays sucrier. Albioma compte y investir 400 millions en dix ans sur le milliard d'euros que l'entreprise prévoit de consacrer à son développement entre 2013 et 2023, en se finançant pour 30 à 40 % en fonds propres avec d'éventuels partenaires.   Autre caractéristique de ce plan, il privilégie la biomasse, comme l'illustre la nouvelle dénomination de la société. La centrale de Galion 2 en Martinique (mise en service prévue en 2016) n'utilisera pas de charbon, comme envisagé initialement, mais des déchets verts en complément de la bagasse. Ce renoncement au charbon a permis de relancer le projet d'une petite centrale à Marie-Galante pour 2017-2018.   Débuts dans la méthanisation En métropole, le groupe a racheté en 2012 Methaneo, qui construit des unités de méthanisation (du biogaz) en zone agricole. Deux ont démarré en 2013 dans les Deux-Sèvres, trois ou quatre autres sont prévues en 2014. Les défis techniques intéressent également Jacques Pétry, comme la centrale solaire avec stockage d'électricité en construction sur le toit d'un hypermarché de La Réunion. Les fermes éoliennes, « où nous ne faisions pas la différence techniquement », rappelle le président d'Albioma, ont été cédées en 2013.   Compte tenu de tous ces projets, les résultats devraient décoller en 2016 après la progression plus modérée prévue en 2014 et 2015. L'Ebitda devrait passer de 123-126 millions cette année à 160-163 millions dans deux ans.   Notre conseil Acheter Pour la rentabilité élevée et la relance de la croissance. Le ratio cours/bénéfice est repassé au-dessus de celui du Cac 40, à 16,5 fois 2014, mais il reste loin des sommets de 2006-2007 à 20-27 fois. Viser 25 .   Cécile Le Coz
</article>
<article titre="Bioénergie L’énergie au naturel" date="2014 04 3" journal="Rapports - Énergie - Areva">
8 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... Bioénergie L’énergie au naturel Plus de 2,5 GW dans le monde : la plus grande base installée de centrales biomasse AREVA développe des solutions de production d’électricité avec moins...    Pour voir le rapport :  http://www.areva.com/mediatheque/liblocal/docs/activites/energ-renouvelables/pdf-enr-bioenergies-07-04-2014/Brochure-Bioenergy_FR-04-2014.pdf
</article>
<article titre="Projets majeurs d’infrastructure publique pour lesquels les organismes publics demeurent responsables et en conservent la maîtrise" date="2014 04 2" journal="Rapports - Environnement - MDDEFP">
30 pages   NB : cette extraction brute des 30 premiers mots ne forme peut-être pas un paragraphe bien construit  ... 1318 GAZETTE OFFICIELLE DU QUÉBEC, 9 avril 2014, 146e année, no 15 Partie 2  ANNEXE  Projets majeurs d’infrastructure publique pour lesquels les organismes publics demeurent responsables et en conservent la...    Pour voir le rapport :  http://www.mddefp.gouv.qc.ca/matieres/regime-compensation/D283-2014.pdf
</article>
<article titre="Albioma : l'expansion brésilienne a officiellement débuté" date="2014 04 1" journal="Boursier (site web)">
Albioma  a finalisé l'acquisition de la totalité des actions de l'unité de cogénération Rio Pardo Termoelétrica, au Brésil. Cet investissementavait été annoncé lors de la présentation des résultats annuels 2013 du groupe le 5 mars dernier.   Cette unité de cogénération bagasse, d'une puissance de 60 MW, a été mise en service en 2009. Albioma pense être en mesure d'en améliorer "fortement" l'efficacité, et aura l'opportunité de construire une extension de 15 MW. L'opération devrait avoir un effet positif à la fois sur l'EBITDA et le bénéfice net cette année. Il s'agit aussi de la première étape du développement du groupe dans le pays.
</article>
<article titre="Produits de la mer Comment réduire les teneurs en benzopyrènes" date="2014 04 1" journal="Process Alimentaire, no. 1313">
Longtemps considéré comme une technique de conservation, le fumage des produits de la mer est aujourd'hui davantage pratiqué pour son action aromatisante et, dans une moindre mesure, colorante. L'arôme typique de la fumée est dû aux phénols mais d'autres composés, nocifs, sont formés durant le processus. Tout l'enjeu réside dans le développement des caractéristiques organoleptiques en limitant la production de composés indésirables, goudrons et hydrocarbures polycycliques aromatiques (HAP) en tête. Ces derniers tombent sous le coup d'une réglementation qui se durcit. Dès le premier septembre, les teneurs maximales autorisées en benzopyrène, benzanthracène, benzofluoranthène et chrysène seront réduites de 60 %. Pour limiter leur développement, l'application d'une température de pyrolyse inférieure à 450 °C est un premier levier. Il est possible de faire varier efficacement la composition globale d'une fumée et de favoriser sélectivement la formation des composés phénoliques volatils plutôt que celle des HAP. Le Codex Alimentarius contient les Codes d'usages pour la réduction de la contamination des aliments par les HAP issus des processus de fumage et de séchage direct (CAC/RCP 68-2009). Ce document précise les paramètres clés à prendre en compte, notamment le combustible et la ventilation (voir page 58). Autant de pistes pour les industriels et les équipementiers. Jean-Fabrice Boré de JV La Française précise par exemple que la ventilation des équipements Schröter, que la société distribue en France, évolue pour ajouter aux flux d'air verticaux la possibilité de passer en horizontal. Reich, distribué par Technocarne, a développé une cellule à flux transversal, Airmaster UKQ Airjet, dans le but de garantir une répartition plus homogène de la fumée sur le poisson.   La ventilation et le combustible en 1re ligne Le combustible le plus utilisé reste le bois, mais d'autres sources comme la bagasse (de canne à sucre), la rafle de maïs et la coque de noix de coco peuvent être mises en oeuvre. Il est important de les examiner au regard d'une contamination potentielle des aliments. Par la présence de corps étrangers dans un premier temps. «Le générateur externe évite le croisement sciure/produit», explique Gilles Velly, responsable commercial agro chez Capic. Dans un second temps, la contamination des HAP est plus ou moins importante suivant l'essence de bois utilisée. Elle augmente avec la teneur en lignine. Pour cette raison, les résineux sont à proscrire. La dureté du bois, lorsqu'il est utilisé sous forme de bûche, aurait également un rôle. C'est en tout cas l'une des conclusions d'une thèse soutenue en 2011 à l'université de Nantes (1).   Après la formation, c'est le dépôt de ces molécules nocives sur le poisson qui est à éviter. L'humidité du produit et du fumoir, la circulation et température de l'air, la durée d'exposition, la densité de la fumée et son hygrométrie sont autant de paramètres qui influencent le phénomène. Le «lavage» de la fumée devient la règle. Les hydrocarbures se situant plus dans la phase particulaire que dans la phase gazeuse de la fumée, on obtient de bons résultats en installant des écrans déflecteurs à la sortie du générateur de fumée, munis d'un dispositif pour décanter le goudron.   La société Arcos met en avant sa technologie de fumage climatisé en circuit fermé. La fumée ne se dépose pas directement sur le produit, mais est déshumidifiée par condensation. Elle passe par une batterie froide et une gaine de ventilation, ce qui permet le piégeage des particules les plus lourdes, dont les goudrons. L'air sec obtenu est réchauffé pour permettre le maintien de la température de la cellule.   Les cellules à circulation horizontale Jet Smoke H195-2 850 Fishsmoker de Kerres, distribuées en France par Hoegger Alpina bénéficient quant à elles d'une régénération automatique de la fumée, qui circule en système fermé.   Fumer moins chaud avec la friction de bûche Certains processus sont de nature à générer moins de HAP. Nombreux sont les fournisseurs qui valorisent les générateurs de fumée à bûches, plutôt qu'à sciures ou à copeaux. Parmi les facteurs qui influencent l'enrichissement d'une fumée en HAP lors de sa combustion, on compte notamment la température de combustion et la présence d'oxygène lors de la combustion. La teneur en HAP augmente lorsque la température de combustion augmente et/ou lorsque le taux d'oxygène diminue.   Dans les systèmes à friction de bûche, une roue crantée frotte le bois. Le processus de fumage est une succession de séquences friction/arrêt pour éviter la formation de flamme. Il n'y a pas de foyer, ni de résistance chauffante dans l'installation. Le pilotage proportionnel (régulation de la pression d'appui) breveté par Maurer-Atmos, distribué en France par Fésia, a un rôle de stabilisation de la densité de fumée. «Dans la technologie à friction, on maîtrise la concentration de fumée, sans risque d'incendie», conclut Alain Philippi, technico-commercial Fésia. Il est possible de combiner deux générateurs sur une cellule, comme une cellule à friction et un générateur à copeaux. Cette combinaison pallie la moindre intensité du goût fumé due aux procédés de fumage à froid. Une cartouche à combustion de copeaux, brevetée par Maurer-Atmos, peut être intégrée et/ou installée ultérieurement sur les cellules à friction. Par la combustion de copeaux de bois différents (hêtre, genévrier etc.) elle permet d'élargir les nuances de goût.   La flexibilité aromatique «Lorsque nous conseillons de passer aux bûches plutôt qu'à la sciure ce n'est pas tant pour le taux de benzopyrène que pour la flexibilité aromatique», ajoute Pascal Blanc, gérant de la société Arcos. En effet, il n'existe actuellement sur le marché que des sciures de chêne et de hêtre alors qu'il est possible de fumer avec du châtaignier, du pommier et ainsi obtenir une large palette aromatique. Il est également envisageable de fendre une bûche en deux, d'insérer des aromates avant de cheviller l'ensemble. Les aromates seront ainsi libérés au cours de la friction.   Une autre approche de réduction des teneurs en HAP consiste à mettre en oeuvre des systèmes de revaporisation de fumée liquide. Produite à partir de fumée fractionnée et purifiée, elle est pauvre en hydrocarbures. «Dans ce cas, la maîtrise de l'apport en HAP est rendu extrêmement simple.Un bilan matière permet de démontrer que l'apport en HAP par le fumage à la fumée liquide est très inférieur aux exigences légales en Europe», explique François Deumier, responsable du département technologie alimentaire chez Lutetia. De la même manière qu'un générateur à copeaux, il est envisageable d'intégrer à une installation de fumage utilisant la friction un petit générateur à fumée liquide.   «L'évolution actuelle de la législation est clairement à l'avantage des produits dérivés de fumée naturelle tels que la gamme Tradismoke. Un nombre croissant de transformateurs passeront à cette technique d'aromatisation afin de garantir à leurs produits une qualité sanitaire irréprochable», prédit Raphaël Levesque, responsable R &amp; D chez l'aromaticien Nactis. Les fumées Tradismoke de Nactis ont reçu l'agrément européen CE 1321/2013 en décembre 2013. La gamme se décline en forme liquide et poudre pour des applications d'atomisation mais également trempage, injection ou incorporation en tant qu'ingrédient aromatisant. La réglementation reste floue quant à leur utilisation dans les produits de la mer.   Le Codex Alimentarius reste néanmoins prudent, rappelant «que tout conseil donné dans le but de minimiser les HAP ne doit pas entraîner l'augmentation des risques pour la santé humaine suite à la hausse des autres contaminants ou la réduction de la sécurité microbiologique». Un équilibre de plus à trouver, en sachant qu'il faut relativiser puisque les principaux contributeurs à l'ingestion des HAP ne seraient pas les fumés mais les produits céréaliers, les graisses et huiles végétales.   A savoir L'appellation «fumée liquide» est une dénomination descriptive des dérivés liquides issus de fumée de bois.   La déclaration «arôme de fumée» concerne les produits dérivés de fumée de bois alors que la déclaration «arôme fumée» (équivalente à «arôme») est associée à des bouquets de molécules nature identiques c'est-à-dire non naturelles.
</article>
<article titre="Ouverture Paris : au rebond, Alstom se distingue" date="2014 04 1" journal="Boursier (site web)">
LA TENDANCE Après un premier trimestre 2014 positif pour le CAC40 (+2,2%), le second démarre également sur de bonnes bases avec un indice en hausse de 0,30% à 4.405 points dans les premiers échanges. La place parisienne est favorisée par un effet de rattrapage puisque les principales places européennes ont baissé hier, alors que Wall Street a terminé en vive hausse.  Alstom  et  EDF  se distinguent notamment en tête de CAC40.  Les marchés s'intéresseront aujourd'hui à une batterie d'indicateurs macroéconomiques, principalement ceux des directeurs d'achats en Europe et aux Etats-Unis, qui constituent de bons baromètres. L'Europe regarde la France après la nomination d'un nouveau premier ministre, et s'interroge sur les conséquences d'une inflation de plus en plus faible, comme l'a démontré hier l'indice préliminaire de l'évolution des prix pour mars.    WALL STREET Wall Street a amplifié ses gains ce lundi après le discours rassurant de Janet Yellen. La présidente de la Fed a réaffirmé le soutien "pendant un certain temps" de la Banque fédérale américaine à l'économie nationale. Par ailleurs, l'indice d'activité manufacturière régionale de la Fed de Dallas pour le mois de mars 2014 dépasse le consensus. Ces nouvelles ont permis au marché américain de résister à l'indice d'activité manufacturière régionale PMI de Chicago pour le mois de mars 2014 qui est, lui, ressorti inférieur aux attentes. Le Dow Jones prend, 0,82% à 16.458 points, le S&amp;P500 s'offre 0,79% à 1.872 points, et le Nasdaq s'adjuge 1,04% à 4.199 points.   ECO ET DEVISES En Europe, les ventes de détail allemandes de février (8h00) seront suivies du PIB français du quatrième trimestre 2013 dans son estimation finale (en Europe, les indices PMI manufacturiers des grandes économies sont attendus en matinée, dont à 9h50 pour la France. (consensus 51,9), à 9h55 pour l'Allemagne et à 10h00 pour la zone euro. Les chiffres de l'emploi allemand de mars seront eux aussi connus ce matin (09h55). A 11h00, place au taux de chômage européen de février (consensus 12%).    Aux Etats-Unis, l'indice manufacturier Markit PMI de mars (15h45 : consensus 56), l'indice ISM manufacturier de mars (16h00 : consensus 54) et les dépenses de construction de février (16h00 : consensus +0,0%) sont au programme.   La parité euro / dollar atteint 1,3776 ce matin. Le baril de Brent se négocie 107,69$ et le WTI 101,37$ pour l'échéance de mai. L'once d'or se traite 1.285$ +0,11%).    VALEURS EN HAUSSE Mauna Kea  (+6,2% à 13,1 euros). Le groupe, leader dans le domaine de la biopsie optique, a annoncé l'installation du Cellvizio dans trois hôpitaux brésiliens de premier plan : ICESP (Centre de Lutte Anti-Cancer de l'Université de Sao Paulo), INCA (Institut National du Cancer de Rio de Janeiro) et FUGAST (Institut de Gastro-entérologie du Rio Grande do Sul). Par ailleurs, l'autorité de santé brésilienne (ANVISA) a octroyé son autorisation pour la commercialisation des minisondes AQ-Flex 19, pour l'observation en temps réel et au niveau microscopique des kystes pancréatiques, pour l'UroFlex, première sonde dédiée à la réalisation de biopsies optiques en urologie et pour le logiciel de biopsie optique EVA, l'assistant virtuel pour l'endomicroscopie.   *  Teleperformance  (+5,2% à 44,3 euros), dopé par une note de Gilbert Dupont. Le broker a rehaussé de "accumuler" à "acheter" sa recommandation sur le dossier tout en ajustant son cours cible à 50,9 euros. Le courtier profite ainsi de la contreperformance actuelle du cours (-5% depuis le début de l'année vs +9% pour l'indice CAC M&amp;S) pour adopter une opinion plus agressive sur le titre. Les catalyseurs identifiés par le broker sont une poursuite d'une croissance organique soutenue, l'annonce du gain d'un contrat conséquent, et une éventuelle acquisition significative.   *  Alstom  (+5,2% à 20,86 euros) à l'ouverture parisienne, dopé par l'annonce de la vente, pour 730 millions d'euros, de son activité de fabrication d'équipements auxiliaires vapeur au fonds allemand Triton. Hier, le titre s'était déjà bien comporté avec la signature d'un gros contrat en Irak dans les turbines à gaz. Oddo estime que le "spectre latent d'une augmentation de capital s'éloigne", lui qui pesait lourdement dernièrement.   * Air France KLM  (+0,8% à 11 euros), soutenu par une note d'analyste. Liberum est passé ce matin de "conserver" à "acheter" sur le titre de la compagnie tout en revalorisant le dossier de 7,10 à 17 euros !   *  Essilor International  (+0,2% à 73,4 euros). La société a finalisé l'acquisition des 51% du capital détenus par PPG dans Transitions Optical, leader mondial de la fourniture de verres photochromiques pour les producteurs de l'industrie optique, et de 100% du capital d'Intercast, fabricant de verres solaires de haute qualité. L'opération, annoncée le 29 juillet 2013, a obtenu sans réserve l'agrément des autorités de la concurrence en Allemagne, en Australie, au Brésil, en Espagne, aux Etats-Unis, en Nouvelle-Zélande, au Portugal et au Royaume-Uni. Fondée en 1990, Transitions Optical est basée à Pinellas Park en Floride (Etats-Unis). Son chiffre d'affaires s'élève à 844 millions de dollars en 2013, dont 279 millions de dollars avec des producteurs de verres autres qu'Essilor. Le montant de la transaction s'élève à 1,73 milliard de dollars (sous réserve des ajustements usuels post-clôture) auxquels s'ajoutent 125 millions de dollars de paiement différé sur cinq ans.    VALEURS EN BAISSE *  Naturex  (-3% à 62,3 euros), après la publication de résultats 2013 mitigés. La correction est accentuée par l'abaissement de la recommandation de Natixis de "acheter" à "neutre", pour un objectif inchangé à 65 euros. L'action apparaît désormais bien valorisée au courtier, qui pense en outre que la performance du premier trimestre a sans doute été pénalisée par une base de comparaison élevée.   *  Capgemini  (-1,4% à 54,2 euros), pénalisé par une note de BofA ML. La banque américaine a dégradé le titre de la SSII de "neutre" à "sousperformance" tout en abaissant son cours cible à 48 euros.   *  M6  (-1% à 15,7 euros). Le groupe a finalisé la vente de Mistergooddeal à Darty. Les deux entreprises étaient entrées en discussions en décembre dernier. Le montant de l'opération, 2 millions d'euros, sera perçu en deux versements en juin et décembre 2014. Au 31 décembre 2013, la position de trésorerie de Mistergooddeal était positive de 5,2 millions d'euros et ses capitaux propres atteignaient 10,4 millions d'euros. L'impact de la transaction a été pris intégralement dans les comptes 2013 de M6.   *  Albioma  (-0,7% à 19,4 euros). L'entreprise a finalisé l'acquisition de la totalité des actions de l'unité de cogénération Rio Pardo Termoelétrica, au Brésil. Cet investissementavait été annoncé lors de la présentation des résultats annuels 2013 du groupe le 5 mars dernier. Cette unité de cogénération bagasse, d'une puissance de 60 MW, a été mise en service en 2009. Albioma pense être en mesure d'en améliorer "fortement" l'efficacité, et aura l'opportunité de construire une extension de 15 MW. L'opération devrait avoir un effet positif à la fois sur l'EBITDA et le bénéfice net cette année. Il s'agit aussi de la première étape du développement du groupe dans le pays.
</article>
<article titre="Préouverture Paris : le nouveau trimestre devrait démarrer en hausse" date="2014 04 1" journal="Boursier (site web)">
LA TENDANCE Après un premier trimestre 2014 positif pour le CAC40 (+2,2%), le second devrait démarrer sur de bonnes bases avec des indicateurs avancés qui pointent vers le haut ce matin. Le mouvement devrait être favorisé par un effet de rattrapage puisque les principales places européennes ont baissé hier, alors que Wall Street a terminé en vive hausse.    Les marchés s'intéresseront aujourd'hui à une batterie d'indicateurs macroéconomiques, principalement ceux des directeurs d'achats en Europe et aux Etats-Unis, qui constituent de bons baromètres. L'Europe regarde la France après la nomination d'un nouveau premier ministre, et s'interroge sur les conséquences d'une inflation de plus en plus faible, comme l'a démontré hier l'indice préliminaire de l'évolution des prix pour mars. Côté entreprises, le 31 mars a été propice à la conclusion de plusieurs opérations, notamment pour  Alstom ,  Essilor ,  M6  et  Albioma  à Paris.  WALL STREET Wall Street a amplifié ses gains ce lundi après le discours rassurant de Janet Yellen. La présidente de la Fed a réaffirmé le soutien "pendant un certain temps" de la Banque fédérale américaine à l'économie nationale. Par ailleurs, l'indice d'activité manufacturière régionale de la Fed de Dallas pour le mois de mars 2014 dépasse le consensus. Ces nouvelles ont permis au marché américain de résister à l'indice d'activité manufacturière régionale PMI de Chicago pour le mois de mars 2014 qui est, lui, ressorti inférieur aux attentes. Le Dow Jones prend, 0,82% à 16.458 points, le S&amp;P500 s'offre 0,79% à 1.872 points, et le Nasdaq s'adjuge 1,04% à 4.199 points.   ECO ET DEVISES En Europe, les ventes de détail allemandes de février (8h00) seront suivies du PIB français du quatrième trimestre 2013 dans son estimation finale (en Europe, les indices PMI manufacturiers des grandes économies sont attendus en matinée, dont à 9h50 pour la France. (consensus 51,9), à 9h55 pour l'Allemagne et à 10h00 pour la zone euro. Les chiffres de l'emploi allemand de mars seront eux aussi connus ce matin (09h55). A 11h00, place au taux de chômage européen de février (consensus 12%).    Aux Etats-Unis, l'indice manufacturier Markit PMI de mars (15h45 : consensus 56), l'indice ISM manufacturier de mars (16h00 : consensus 54) et les dépenses de construction de février (16h00 : consensus +0,0%) sont au programme.   La parité euro / dollar atteint 1,3776 ce matin. Le baril de Brent se négocie 107,69$ et le WTI 101,37$ pour l'échéance de mai. L'once d'or se traite 1.285$ +0,11%).    VALEURS A SUIVRE *  Alstom  : le groupe a vendu son activité de fabrication d'équipements auxiliaires vapeur au fonds allemand Triton, pour une valeur d'entreprise d'environ 730 millions d'euros (la valeur d'entreprise intègre la dette). L'opération s'inscrit dans le programme de cession d'actifs non-stratégiques annoncé en novembre dernier par l'industriel. Elle sera finalisée avant la fin du premier semestre de l'année fiscale 2014/2015, qui démarre ce 1er avril.    *  Essilor International  : la société a finalisé l'acquisition des 51% du capital détenus par PPG dans Transitions Optical, leader mondial de la fourniture de verres photochromiques pour les producteurs de l'industrie optique, et de 100% du capital d'Intercast, fabricant de verres solaires de haute qualité. L'opération, annoncée le 29 juillet 2013, a obtenu sans réserve l'agrément des autorités de la concurrence en Allemagne, en Australie, au Brésil, en Espagne, aux Etats-Unis, en Nouvelle-Zélande, au Portugal et au Royaume-Uni. Fondée en 1990, Transitions Optical est basée à Pinellas Park en Floride (Etats-Unis). Son chiffre d'affaires s'élève à 844 millions de dollars en 2013, dont 279 millions de dollars avec des producteurs de verres autres qu'Essilor. Le montant de la transaction s'élève à 1,73 milliard de dollars (sous réserve des ajustements usuels post-clôture) auxquels s'ajoutent 125 millions de dollars de paiement différé sur cinq ans.    *  M6  : le groupe a finalisé la vente de Mistergooddeal à Darty. Les deux entreprises étaient entrées en discussions en décembre dernier. Le montant de l'opération, 2 millions d'euros, sera perçu en deux versements en juin et décembre 2014. Au 31 décembre 2013, la position de trésorerie de Mistergooddeal était positive de 5,2 millions d'euros et ses capitaux propres atteignaient 10,4 millions d'euros. L'impact de la transaction a été pris intégralement dans les comptes 2013 de M6.   *  Albioma  : l'entreprise a finalisé l'acquisition de la totalité des actions de l'unité de cogénération Rio Pardo Termoelétrica, au Brésil. Cet investissementavait été annoncé lors de la présentation des résultats annuels 2013 du groupe le 5 mars dernier. Cette unité de cogénération bagasse, d'une puissance de 60 MW, a été mise en service en 2009. Albioma pense être en mesure d'en améliorer "fortement" l'efficacité, et aura l'opportunité de construire une extension de 15 MW. L'opération devrait avoir un effet positif à la fois sur l'EBITDA et le bénéfice net cette année. Il s'agit aussi de la première étape du développement du groupe dans le pays.   *  Dexia  : l'établissement communique ce matin sur l'impact des travaux préparatoires au mécanisme de supervision unique par la Banque centrale européenne sur ses états financiers. Il est négatif, mais "sans conséquence en comptes de résultats ou solvabilité", explique l'établissement. La banque explique que dans le cadre de l'examen de la qualité de ses actifs et d'une revue de ses méthodes de valorisation comptable en cours, mené par la Banque de Belgique, "il a été jugé opportun de prendre en compte, dès fin 2013, certains ajustements dans les comptes annuels", ayant trait à la valorisation de titres de collectivités locales et de "covered bonds" espagnols, illiquides et classés en actifs disponibles à la vente. Il en résulte que les gains ou pertes latents ou différés du groupe passent à -6,47 milliards d'euros, soit une baisse de -447 millions d'euros par rapport aux comptes publiés le 20 février 2014.    *  Hubwoo ,  Medicrea ,  Coheris ,  Acteos ,  Naturex ,  La Foncière Verte ,  Genoway ,  DMS  ont publié leurs comptes.  Atari  procède au remboursement anticipé de 23,5 millions d'euros d'OCEANE.  Esker  retenu par Eurofeu pour personnaliser son service client.  Société Foncière Lyonnaise  se réorganise en 3 pôles opérationnels.  Graines Voltz  acquiert son partenaire marocain. Concerto European Developer ( Affine ) construira la plateforme logistique de Petit Bateau.  BioAlliance  Pharma signe un accord de production de Validive et signe également avec Penn Pharma. Les cotations d' Europlasma  et  AgroGénération  sont suspendues.  Cellectis  a réalisé son placement privé de 20,5 millions d'euros.  Implanet  recrute un directeur des ventes pour les Etats-Unis. Le système AltaCVD d'Altatech ( Soitec ) s'exporte aux Etats-Unis.  Immobilière Dassault  cède un immeuble parisien.  Mauna Kea  installe les 3 premiers systèmes Cellvizio et obtient des autorisations clés au Brésil.  Toupargel  rachète Eismann France. Feu vert à l'opération entre  Vranken  et Castel.  Derichebourg  signe un nouvel accord de financement avec neuf banques.  Salvepar  participe à hauteur de 20 millions de dollars au rachat de MultiPlan. Jean-Noël Reynaud nommé directeur général de  Belvédère .
</article>
<article titre="Albioma: finalise l'achat d'un premier actif au Brésil." date="2014 04 1" journal="Cercle Finance (site web)">
Albioma (ex-Séchilienne Sidec) a finalisé l'acquisition de l'unité de production électrique à cogénération de Rio Pardo Termoelétrica, située dans la région brésilienne de Sao Paulo. L'opération s'annonce relutive dès 2014.   Le producteur d'électricité ultramarin précise que cette centrale à cogénération utilisant la bagasse, un résidu de canne à sucre, affiche une puissance de 60 MW et a été mise en service en 2009. Il compte en améliorer l'efficacité. ?Le groupe aura également l'opportunité de construire une extension de 15 MW?, indique-t-il.   Essentiellement présent dans les départements d'outre-Mer (Martinique, Guadeloupe, Guyane, La Réunion, Mayotte...), Albioma développe ainsi sa présence hors du territoire français, pour l'heure constituée d'unités sur l'île Maurice. Il n'était pas implanté au Brésil auparavant.    ?Cette opération représente la première étape du développement d'Albioma au Brésil, priorité du groupe à l'international, et devrait avoir une contribution positive tant au niveau de l'EBITDA (excédent brut d'exploitation, ndlr) que du résultat net part du groupe dès l'exercice 2014?, indique Albioma.   Copyright (c) 2014 CercleFinance.com. Tous droits réservés.
</article>
<article titre="Albioma met un pied au Brésil" date="2014 03 17" journal="Environnement Magazine hebdomadaire, no. 9">
Albioma passe aux actes. Un an après avoir changé de nom, et surtout un an après avoir adopté une stratégie plus offensive, l'ancienne Séchilienne-Sidec vient de boucler sa première acquisition au Brésil, sa cible numéro un. Le producteur d'électricité à partir de biomasse va racheter pour 137 millions de reals brésiliens (environ 43 millions d'euros) la centrale Rio Pardo Termoelétrica, une unité de cogénération fonctionnant à partir de bagasse (résidu de la canne à sucre) et de charbon. Elle est située dans l'État de Sao Paulo. Mise en service en 2009, la centrale affiche une puissance installée de 60 MW, adossée à une sucrerie qui traite 2,1 millions de tonnes de canne à sucre par an. Ses capacités pourraient être augmentées de 15 MW, à condition de réaliser un investissement supplémentaire de 30 millions d'euros. Cette acquisition sera financée pour moitié en dette et pour moitié en fonds propres.    L'opération « confirme la crédibilité de l'expansion internationale d'Albioma », estiment les analystes financiers du courtier Portzamparc. En mars 2013, Albioma s'était donné trois ans pour investir au Brésil dans des projets de centrales brûlant de la bagasse, dont le pays est le premier producteur mondial. Depuis, la première opération se faisait attendre. Elle devrait en annoncer d'autres. Le groupe a indiqué mener des « discussions actives avec plusieurs sucriers brésiliens pour la reprise d'usines existantes, ou pour la construction de nouvelles unités de cogénération avec l'objectif de réaliser un nouveau projet tous les 12 à 18 mois ».  Albioma réalise encore 90 % de son chiffre d'affaires dans les départements d'outre-mer et souhaite diversifier ses positions géographiques. 400 millions d'euros devraient être consacrés pendant dix ans au développement au Brésil. À terme, le plus grand pays d'Amérique latine pourrait représenter un tiers de son chiffre d'affaires. Au total, en ajoutant les projets dans d'autres régions, dont les DOM, Albioma prévoit de mobiliser un milliard d'euros sur cette période.
</article>
<article titre="Après 46 ans d'indépendance, Maurice cherche à réinventer son succès économique (ANALYSE)" date="2014 03 13" journal="Xinhua - Agence de presse">
Après 46 ans d'indépendance, Maurice cherche à réinventer son succès économique (ANALYSE)    PORT-LOUIS, 13 mars (Xinhua) -- L'île Maurice a fêté ses 46 ans d'indépendance mercredi sous le signe d'un succès économique qui en fait en modèle en Afrique. Pourtant, cette réussite commence à s'essouffler, nécessitant de grandes mutations dictées souvent par des facteurs exogènes.     Le Premier ministre Navin Ramgoolam n'a pas manqué d'en faire mention lors de son discours à la Nation mercredi à cette occasion.   Il y a 46 ans quand l'île du sud-ouest de l'océan Indien s' affranchissait du joug colonial britannique, peu de spécialistes auraient prédit un avenir aussi brillant.     Ainsi l'économiste britannique James Meade, prix Nobel en 1977, estimait en 1961 que "les perspectives de développement paisible ne sont pas bonnes" à cause d'une forte densité de population, de la dépendance à une seule industrie (la canne à sucre) et des conflits ethniques au sein de la population. Un avis que partagea un autre prix Nobel, l'écrivain V.S.Naipaul.     La première décennie post-indépendance allait effectivement leur donner raison. La monoculture de la canne couplée à la première crise pétrolière du début des années 70 et à un régime politique miné par la corruption et le favoritisme allait mettre le pays à genoux. A tel point que le Fonds Monétaire International (FMI) et la Banque Mondiale viennent à la rescousse avec des pilules très amères.    Toutefois, quelques tentatives de diversification ont été lancées, notamment dans le tourisme et l'industrie textile. Mais il faudrait l'arrivée d'un nouveau gouvernement avec à la tête l' avocat Aneerood Jugnauth pour véritablement mettre le pays sur les rails d'un développement économique qui allait le faire sortir de l'état de sous-développement.    De 1982 à 1995, le pays connaît une prospérité sans précédent grâce à la multiplication des hôtels, des usines de textile, de l' émergence du secteur offshore et d'une rationalisation de la production sucrière. La stabilité politique, une main d'oeuvre bon marché et un niveau d'éducation élevé de la population, aidèrent à attirer les investisseurs étrangers.    Mais le gouvernement Jugnauth, gagné par l'usure, perd le pouvoir aux élections de 1995. Le premier gouvernement de Navin Ramgoolam ne sait pas garder le même cap. Les inégalités sociales sont exacerbées, débouchant sur les terribles émeutes de 1999. D' autres nuages arrivent avec l'annonce de la fin prochaine des accords de Lomé concernant l'accès du sucre de pays ACP (Afrique Caraïbes Pacifique) sur le marché européen.    Le retour de l'ancien Premier ministre Jugnauth au pouvoir en 2000 s'accompagne de mesures courageuses avec une réorientation du secteur de la canne à sucre en une industrie cannière. Elle comprend, aujourd'hui, plusieurs créneaux, notamment la production d'énergie à partir de la bagasse et la filière éthanol, en sus des autres dérivés exploités comme la mélasse et la vinasse.    Le tourisme connaît un nouveau souffle avec un parc hôtelier en constante hausse et l'émergence de l'île Maurice comme une destination haut de gamme, avec en parallèle le développement du secteur des services et de loisirs, le tout couronné par un secteur immobilier destiné à une clientèle étrangère : les fameux IRS et RES.     D'autre part les services financiers aidés par l'avancée technologique dans les télécommunications permirent l'expansion du système bancaire et des assurances.     Dans le même souffle, l'offshore et le port franc se consolide, alors que l'ICT-BPO est en passe de devenir le troisième pilier de l'économie.     Maurice a négocié avec succès divers accords économiques pour assurer la survie de différents secteurs. Le pays a pu limiter les dégâts causés par les pertes successives de l'accès préférentiel de son sucre et de ses produits textiles aux marchés occidentaux. Une diplomatie efficace a également contribué au maintien des marchés.    Le changement de gouvernement en 2005 avec le retour de Navin Ramgoolam n'allait cette fois pas changer le cours des choses.     Fort de sa première expérience maladroite, il s'entoure de Rama Sithanen, ancien ministre des Finances de Sir Aneerod Jugnauth, devenu entretemps président de la République, pour continuer le progrès enclenché.     Les réformes de Rama Sithanen ont permis notamment au pays de montrer une résilience exemplaire face à la crise financière de 2008.     Il a aussi consolidé les secteurs financiers et des TIC ( Technologies de l'information et de la communication) et ouvert des chantiers pour le développement du secteur de la pêche en une industrie professionnelle connue sous l'appellation "seafood hub".    Mais aujourd'hui, les recettes des succès précédents commencent à s'essouffler. Les protections commerciales disparaissent peu à peu, tandis que les avantages de la main-d'oeuvre à bon marché se sont longtemps épuisés.     Le modèle de croissance mauricien cherche maintenant de nouveaux vecteurs de compétitivité, soit une bonne dose de productivité et d'innovation.     C'est ainsi que les récents discours budgétaires et celui du Premier ministre fait mercredi montrent la volonté du gouvernement reconduit en 2010 de réussir cette transformation à travers d'une économie de la connaissance, "knowledge-based economy".     Les bases ont déjà été jetées avant même l'indépendance avec la création de l'Université de Maurice en 1965-1970, de l' introduction de l'éducation gratuite et l'institution de plusieurs collèges techniques dès les années 1970.     L'implantation de nouvelles universités, locales et étrangères à Maurice suit désormais l'objectif d'attirer jusqu'à 100.000 étudiants étrangers sur l'île d'ici à l'an 2020. Le gouvernement veut également au moins un diplômé universitaire pour chaque famille mauricienne.     D'autre part, le gouvernement veut devenir un "hub aéronautique " dans la région et l'agrandissement de l'aéroport avec l'aide de la Chine entre dans ce cadre.     Les autorités veulent mettre sur pied un centre de formation technique pour l'entretien des avions, de même qu'un centre régional d'entretien des appareils. Les facilités du port franc seront étendues à l'aéroport.     Ce projet-cadre parfaitement avec l'évolution du secteur manufacturier vers la fabrication de haute technologie et l' émergence du secteur de l'ingénierie légère.     Depuis janvier 2013, un statut "port franc" est accordé à toute entreprise manufacturière qui exporte principalement vers la région.     Du côté de la mer, outre le développement du "Seafood hub", il est question d'un "Marine Services Hub" qui entraînera dans son sillage la libéralisation des services d'entreposage.     Il est estimé que la demande dans la région atteindra près de 8, 6 millions de tonnes, alors que Maurice est jusqu'ici concerné par seulement 300.000 tonnes. Un développement qui influera positivement sur d'autres activités comme l'entretien des navires.     Dans le même souffle, un "parc de logistique" sera aménagé à la zone économique de Jin Fei au nord de Port-Louis, sur 65 hectares.     Par ailleurs, Maurice se réclame désormais comme un Etat-océan avec la revendication d'une zone maritime exclusive d'environ 2,3 millions de km2, y compris les 400.000 kms d'espace marine cogérée avec les Seychelles.     Outre la pêche, l'exploration pétrolière et minière est aussi à l'ordre du jour. L'utilisation d'eau de mer profonde est propice à plusieurs usages, de la climatisation à l'aquaculture, en passant par la biotechnologie et l'industrie pharmaceutique. La culture des algues fait aussi partie des projets, tout comme la culture de perles.     Enfin, Maurice compte bien renforcer ses liens économiques avec le continent africain. Ainsi Maurice a créé un fonds spécial, le Mauritius-Africa Fund, pour permettre à l'État d'être un partenaire en affaires avec des éventuels entrepreneurs locaux lorgnant les rives africaines. Le Mauritius-Africa Fund jouera aussi le rôle d'un consultant, en offrant ses services payants aux pays de la région dans les domaines où nous avons acquis une certaine expertise. Fin
</article>
<article titre="Coup de coeur de la semaine Albioma" date="2014 03 8" journal="Investir-Le Journal des finances, no. 2096">
Promis depuis un an, le premier investissement d'Albioma au Brésil a été annoncé mercredi, éclipsant les résultats annuels pourtant très solides, grâce à l'excellente disponibilité des installations et à des compléments de prix perçus d'EDF. L'ex-Séchilienne Sidec va reprendre, d'ici à avril, pour une valeur d'entreprise de 43 millions d'euros, une centrale thermique adossée à une sucrerie, selon le modèle déjà exploité par le groupe dans les DOM-TOM. Cette unité de 60 mégawatts, située dans l'Etat de São Paulo, produit de l'électricité et de la vapeur en brûlant de la bagasse, le résidu fibreux de la canne à sucre. La campagne sucrière durant presque toute l'année au Brésil, cette centrale n'a pas besoin de recourir au charbon en complément, ce qui en fait une unité plus profitable que ses consoeurs antillaises.   Albioma, toutefois, a prévu d'investir 30 millions dans des extensions de capacité et dans l'amélioration de la productivité : cette centrale produit seulement 40 kilowattheures par tonne de canne, contre le triple aux Antilles. Mais « nous achetons une usine qui va contribuer tout de suite aux résultats d'Albioma », s'est félicité le PDG Jacques Pétry. Ce dernier ne compte pas en rester là : « Notre ambition est de signer un projet au Brésil tous les douze à dix-huit mois compte tenu de la qualité des discussions en cours. » Cet investissement est le premier d'une enveloppe prévue à 400 millions entre 2013 et 2023 au Brésil, sur le milliard que le groupe compte dépenser sur cette décennie en nouveaux outils industriels, afin de nourrir sa croissance. Tous reposent sur des énergies renouvelables. Ce plan couvre aussi la centrale bagasse-biomasse de Galion 2, en Martinique, qui devrait entrer en service en 2016, puis celle de Marie-Galante en 2017-2018. Deux fermes solaires, avec stockage de l'électricité, seront opérationnelles en 2014 et 2015 à La Réunion et en Guyane ainsi que, cette année en métropole, quatre unités de biométhanisation après les deux entrées en service en Vendée en 2013.   Notre conseil Acheter Ce premier investissement très attendu au Brésil va contribuer dès 2014 aux bénéfices du groupe et compenser le retard de six à douze mois pris par Galion 2. L'action a décollé en conséquence de 11 % mercredi mais elle n'est qu'au tiers de son record de 2007. Viser 25 (ABIO).   Cécile Le Coz
</article>
<article titre="Albioma réalise sa première acquisition au Brésil" date="2014 03 6" journal="Les Echos, no. 21641">
Un an après avoir résolument mis le cap sur le Brésil, Albioma, anciennement Séchilienne-Sidec, a annoncé hier, à l'occasion de la présentation de ses résultats, sa première acquisition dans le pays. Le producteur français d'énergie, qui s'est spécialisé dans l'électricité à partir de bagasse (résidu de la canne à sucre), va racheter l'usine de Rio Pardo Termoeléctrica, dans l'Etat de São Paulo, pour 43 millions d'euros.   Albioma compte utiliser son savoir-faire pour doubler le rendement de l'unité, grâce à des investissements, mais aussi en améliorant la combustion, en diminuant l'humidité de la bagasse ou encore les vibrations de la turbine. «  L'impact sera positif sur nos résultats dès cette année  », indique Jacques Petry, PDG de la société. Ambitieux programme d'investissements Cette première opération, qui porte sur une capacité totale de 60 mégawatts, devrait être suivie par beaucoup d'autres. Avec 90 % de son chiffre d'affaires dans les départements d'outre-mer (La Réunion, Guadeloupe, île Maurice), Albioma a lancé l'an dernier un ambitieux programme d'investissements, de 1 milliard d'euros sur dix ans, dont 400 millions au Brésil. «  Le gisement y est considérable. Avec une signature espérée dans le pays tous les douze ou dix-huit mois, nous espérons que le Brésil représentera 30 % de notre activité dans dix ans  », indique Jacques Petry, assurant être déjà en discussion avec plusieurs autres sucriers brésiliens.   Albioma poursuivra par ailleurs son expansion dans les DOM, avec notamment deux gros projets dans les Antilles, en Martinique et à Marie-Galante : deux unités innovantes, produisant de l'électricité à partir de bagasse, mais aussi de biomasse (granulés de bois...), qui doivent entrer en production à partir de 2016 et représentent des investissements respectifs de 180 et 80 millions d'euros. «  Tous nos nouveaux investissements sont à 100 % dans l'énergie renouvelable  », insiste Jacques Petry. Alors que le charbon représente encore les deux tiers du combustible utilisé dans ses centrales, Albioma espère ramener ce taux entre 20 % et 40 % dans dix ans.   La société a confirmé l'ensemble de ses objectifs à moyen terme : porter l'Ebitda entre 160 et 163 millions en 2016, contre 133,6 millions en 2013, et le résultat net part du groupe (hors éléments exceptionnels) entre 40 et 42 millions (32,3 millions).   L'an dernier, l'activité et la rentabilité ont progressé «  grâce à la bonne disponibilité de nos centrales  », explique Jacques Petry.  Par ailleurs, nous avons pu renégocier avec EDF les tarifs de rachat de l'électricité, suite à des investissements de mise aux normes . » Des annonces saluées par la Bourse : le titre a bondi de 11,38 % hier, à 20,55 euros.   Anne Feitz
</article>
<article titre="Albioma boucle sa première acquisition au Brésil" date="2014 03 5" journal="GreenUnivers (site web)">
2014-03-05T20:16:13+00:00   Il aura donc fallu un an à Albioma pour concrétiser sa nouvelle stratégie, axée sur la biomasse et l'international, avec une priorité au Brésil. L'ex. Séchilienne Sidec a profité de la présentation de ses (bons) résultats ce matin pour communiquer sur sa première acquisition, une centrale de co-génération de 60 MW adossée à une exploitation sucrière dans l'Etat de Sao Paulo.   Pour un montant de 137 millions de reals brésiliens (43 M), financés à 50 % par de la dette locale et 50 % en fonds propres, le producteur français indépendant d'énergies renouvelables (notamment par valorisation de biomasse) s'empare du site de production de chaleur et d'électricité en cogénération Rio Pardo Termoelétrica. Le closing est prévu dans les prochaines semaines. Mise en service en 2009, cette unité valorise aujourd'hui les résidus des 2,1 millions de tonnes de canne à sucre traitées par l'usine sucrière. En améliorant le rendement de cette unité grâce à son expérience de la bagasse dans les DOM, et à des performances trois fois plus élevées dans ses propres centrales de cogénération, et en misant sur l'extension de l'usine de 2,1 à 3 millions de tonnes d'ici à 2018, Albioma compte doubler le volume d'électricité exportée sur le réseau à 160 GWh/an. Et malgré des prévisions relativement prudentes quant aux prix de vente (sur un marché de l'électricité où la demande est globalement en forte hausse et où la production hydraulique a été particulièrement médiocre cette année), elle en attend une marge nettement supérieure à celle générée par ses unités dans les DOM.Une acquisition au Brésil tous les 12 à 18 mois Cette externalisation de la valorisation énergétique de la bagasse, qui comprend l'exploitation de la centrale par Albioma, constitue une première au Brésil où l'approvisionnement en énergie est stratégique pour les usines sucrières. « Les exploitants ont besoin de liquidités pour investir, et la valorisation de la biomasse n'étant pas leur métier, notre expertise technique peut leur être précieuse », observe néanmoins Jacques Pétry, le PDG d'Albioma. Le modèle Albioma, qui a fait ses preuves à l'Ile Maurice, repose sur un partenariat avec les sucriers. Dans cette opération au Brésil, les précédents propriétaires de la centrale bénéficient de conditions préférentielles pour acquérir des parts de la nouvelle entité.   Dans ces circonstances, Albioma, qui a déjà entamé des négociations dans ce sens, annonce son intention de poursuivre ses acquisitions au Brésil, au rythme d'une tous les 12 ou 18 mois, pour une enveloppe globale de 400 M d'ici à 2023. « En démontrant le sérieux de nos ambitions, ce premier deal va certainement attirer l'attention d'autres sucriers», veut croire Jacques Pétry. Albioma entend sélectionner ses partenaires parmi les quelque 400 installations sucrières du pays. Priorité est donnée aux projets brownfield, c'est-à-dire la reprise d'équipements existants, qui ont un impact immédiat sur l'activité du groupe. D'ailleurs, l'acquisition de Rio Pardo Termoelétrica est prise en compte dans les prévisions 2014, d'autant que des équipes sont déjà présentes sur place pour procéder à des premières améliorations avant le début de la campagne sucrière en avril. Mais Albioma n'exclut pas des projets greenfield, que la banque brésilienne de développement accepte plus volontiers de financer depuis quelques temps.Un milliard d'euros investis d'ici à 2023Ces projets ne représentent qu'une partie de l'enveloppe d'1 milliard d'euros d'investissements d'ici à 2023 qui a été à nouveau confirmée . Ce montant doit servir à financer des projets 100% biomasse en Martinique et à Marie Galante, la modernisation d'unités existantes, des projets solaires incluant du stockage et la montée en puissance de l'activité biométhanisation, portée par l'équipe Methaneo, dans laquelle Albioma détient 60 % depuis mai 2012.   Le parc solaire de 70 MW dans les DOM représente 24 % de l'Ebitda du groupe, grâce à des tarifs de rachat très avantageux et un fort ensoleillement.   Outre cette acquisition, Albioma a publié un chiffre d'affaires de 364,3 M (+2% hors effet matières premières) pour un Ebitda en hausse de 11 % à 133,6 M et un résultat net part du groupe (RNPG) qui bondit de 33 % à 42,6 M. La trésorerie s'élève à 106 M.   Ces résultats, salués par un bond du cours de Bourse de 6,1% à 19,58 euros à l'ouverture des marchés ce matin puis de 11 % à plus de 20,50 dans la journée, sont notamment dus à la révision des contrats de vente d'électricité à EDF, la bonne disponibilité des centrales et un fort taux d'appel de la centrale de pointe Galion en Martinique.    Cet article est paru dans GreenUnivers (site web)
</article>
<article titre="Albioma annonce sa première acquisition de centrale au Brésil" date="2014 03 5" journal="Reuters économique">
PARIS, 5 mars (Reuters) - Le producteur indépendant d'électricité français Albioma a annoncé mardi sa première acquisition de centrale au Brésil et des résultats 2013 en forte progression, grâce notamment à des hausses de tarifs.   Spécialisé dans la biomasse, l'ex-Séchilienne Sidec a précisé dans un communiqué qu'il rachèterait pour 137 millions de reals brésiliens (43 millions d'euros) la centrale Rio Pardo Termoelétrica, une unité de cogénération - fonctionnant à partir de bagasse (résidu de canne à sucre) et de charbon - située dans l'Etat de Sao Paulo. Mise en service en 2009, cette unité affiche une puissance installée de 60 mégawatts (MW) comparable à celle des autres centrales du groupe. Elle est adossée à une sucrerie qui traite aujourd'hui 2,1 millions de tonnes de canne à sucre par an, a indiqué Albioma.   Le groupe a ajouté que cette acquisition, qui reste soumise à la levée de certaines conditions, serait financée à 50% en dette locale et 50 % en fonds propres et devrait avoir un impact positif sur sa profitabilité dès 2014.   Albioma avait annoncé en mars 2013 qu'il se donnait trois ans pour investir au Brésil dans des projets de centrales brûlant de la bagasse, un résidu de la canne à sucre dont le pays est le premier producteur mondial.   Le groupe a précisé mercredi qu'il était "en discussion active avec plusieurs sucriers brésiliens pour la reprise d'usines existantes ou la construction de nouvelles unités de cogénération avec l'objectif de réaliser un nouveau projet tous les 12 à 18 mois".   Albioma a également annoncé qu'il visait pour 2014 un résultat brut d'exploitation (Ebitda) de 123 à 126 millions d'euros, ainsi qu'un résultat net part du groupe de 33 à 35 millions, et a réitéré ses objectifs pour 2016.   Le groupe, qui a notamment obtenu d'EDF une revalorisation des tarifs de rachat de l'électricité produite par ses centrales à La Réunion, a enregistré au titre de 2013 un résultat net part du groupe de 42,6 millions d'euros (+33%), un Ebitda de 133,6 millions (+11%) et un chiffre d'affaires de 364,3 millions (+2% hors effet matières premières).   Il propose un dividende de 0,60 euro par action, contre 0,59 euro au titre de 2012, avec une option pour le paiement de 50% de ce dividende en actions nouvelles.   Selon le consensus Thomson Reuters I/B/E/S, les analystes attendaient en moyenne un résultat net de 42,1 millions d'euros, un Ebitda de 127 millions, un chiffre d'affaires de 376 millions et un dividende de 0,62 euro.   L'action Albioma a clôturé mardi sur un cours de 18,45 euros, enregistrant une hausse de 9,5% depuis le début de l'année, après +17,2% en 2013, pour une capitalisation proche de 538 millions.   * voir aussi: Le communiqué: http://link.reuters.com/zad47v INTERVIEW-Albioma parle aux sucriers pour investir au Brésil (Benjamin Mallet, édité par Jean-Michel Bélot)
</article>
<article titre="ALBIOMA annonce son premier investissement au Brésil" date="2014 03 5" journal="Agence Option Finance (AOF)">
(AOF) - Albioma a annoncé l'acquisition au Brésil, pour 137 millions de reals brésiliens (43 millions d'euros), de Rio Pardo Termoelétrica, une unité de cogénération située dans l'État de São Paulo. Cette unité, mise en service en 2009, possède des équipements de qualité et affiche une puissance installée de 60 MW comparable à celle des autres centrales du groupe spécialisé dans l'énergie tirée de la biomasse. Elle est adossée à une sucrerie qui traite aujourd'hui 2,1 millions de tonnes de canne à sucre par an.    Grâce à son expertise, Albioma prévoit d'améliorer fortement l'efficacité énergétique de l'outil existant qui devrait, à terme, exporter 160 GWh d'électricité vers le réseau chaque année. La sucrerie envisage d'augmenter à terme sa capacité de traitement pour broyer jusqu'à 3 millions de tonnes de canne à sucre par an. Albioma aura alors l'opportunité de construire une extension de 15 MW de l'usine de cogénération pour valoriser la bagasse supplémentaire. La centrale, implantée sur une zone très favorable à la culture de la canne à sucre, fonctionne toute l'année avec de la bagasse récoltée sur 9 mois. Grâce à cette acquisition, Albioma réalise la première opération d'externalisation de la production d'énergie (y compris de l'exploitation) d'une usine sucrière au Brésil et confirme son statut de partenaire privilégié des acteurs de l'industrie du sucre.   Cette opération, financée à 50% en dette locale et 50 % en fonds propres, devrait avoir un impact positif sur la profitabilité d'Albioma dès l'année 2014 ; elle reste soumise à la levée de certaines conditions suspensives.   Par ailleurs, Albioma est en discussion active avec plusieurs sucriers brésiliens pour la reprise d'usines existantes ou la construction de nouvelles unités de cogénération avec l'objectif de réaliser un nouveau projet tous les 12 à 18 mois.
</article>
<article titre="Flambe après des résultats 2013 supérieurs aux attentes" date="2014 03 5" journal="Trading Sat (site web)">
Albioma (ex Séchilienne-Sidec) flambe dans les premiers échanges mercredi matin (+7,3% à 19,8 euros vers 9h25) après la publication de ses comptes 2013. Le producteur d'énergie indépendant spécialisé dans la valorisation de la biomasse, a dévoilé un chiffre d'affaires de 364,3 millions d'euros, en hausse de 2%, pour un EBITDA (résultat brut d'exploitation) en progression de 11%, à 133,6 milions d'euros, et un résultat net de 42,6 millions d'euros, qui a bondi de 33%.   Le groupe annonce par ailleurs l'acquisition au Brésil, pour 137 millions de reals brésiliens (43 millions d'euros), de Rio Pardo Termoelétrica, une unité de cogénération située dans l'État de São Paulo. "La centrale, implantée sur une zone très favorable à la culture de la canne à sucre, fonctionne toute l'année avec de la bagasse récoltée sur 9 mois", indique Albioma. A la suite de cette publication marquée par un EBITDA supérieur à ses attentes et une baisse plus prononcée qu'attendu de la dette nette, le courtier Gilbert Dupont a confirmé ce matin sa recommandation "Acheter" et son objectif de cours de 21,1 euros sur Albioma.   Fin 2013, Albioma affichait une dette nette consolidée de 414 millions d'euros, en forte baisse par rapport à fin 2012 (528 millions d'euros).
</article>
<article titre="Albioma : en marge de ses comptes, le groupe annonce une grosse opération au Brésil" date="2014 03 5" journal="Boursier (site web)">
Pour  Albioma , l'exercice 2013 a été marqué par une amélioration de la rentabilité. Le groupe, dont les revenus ont baissé de -3% à 364,3 millions d'euros (mais +2% sur une base comparable), affiche un EBITDA de 133,6 millions d'euros, en hausse de 11%, et un bénéfice net amélioré de 33% à 42,6 millions d'euros. La dette consolidée a reculé de 528 à 414 millions d'euros. Le groupe disposait fin 2013 d'une trésorerie de 110 millions d'euros à l'appui de ses projets.   Cette année, Albioma vise 12 à 126 millions d'euros d'EBITA et 33 à 35 millions d'euros de bénéfice net, avant 160 à 163 millions d'euros et 40 à 42 millions d'euros respectivement en 2016. Le conseil d'administration soumettra à l'approbation de l'assemblée générale le versement d'un dividende de 0,60 euro par action, avec option pour le paiement de 50% de ce dividende en actions nouvelles. L'autre nouvelle du jour, c'est une grosse opération au Brésil : le rachat, pour l'équivalent de 43 millions d'euros, de Rio Pardo Termoelétrica, une unité de cogénération située dans l'Etat de São Paulo, implantée sur une zone "très favorable à la culture de la canne à sucre". La centrale, adossée à une usine de sucre, fonctionne toute l'année avec de la bagasse récoltée sur neuf mois, pour une puissance de 60 MW. Elle est en activité depuis 2009. "L'expertise unique d'Albioma devrait permettre d'améliorer fortement l'efficacité énergétique de l'outil existant", indique l'entreprise, qui ajoute être "en discussion active" avec plusieurs sucriers brésiliens pour la reprise d'usines existantes ou la construction de nouvelles unités de cogénération avec l'objectif de réaliser un nouveau projet tous les 12 à 18 mois.
</article>
<article titre="Albioma: accroit d'un tiers son résultat net part du groupe." date="2014 03 5" journal="Cercle Finance (site web)">
Albioma affiche un résultat net part du groupe de 42,6 millions d'euros, en hausse de 33%, en particulier grâce à la plus-value de cession de l'activité éolien en février (5,6 millions).   L'EBITDA progresse de 11% à 133,6 millions d'euros, grâce à la performance opérationnelle des activités et à l'impact positif des accords tarifaires conclus avec EDF au cours du premier semestre. Hors effet matière première, le chiffre d'affaires augmente de 2% à 364,3 millions. Conformément à la politique annoncée en 2012, le conseil d'administration proposera un dividende de 0,60 euro par action, avec option pour le paiement de 50% de ce dividende en actions nouvelles.   Le producteur d'électricité alternatif confirme ses objectifs de croissance de l'EBITDA (160-163 millions d'euros) et du résultat net part du groupe (40-42 millions) pour 2016.   Par ailleurs, Albioma annonce l'acquisition au Brésil, pour 137 millions de reals (43 millions d'euros), de Rio Pardo Termoelétrica, une unité de cogénération fonctionnant à la bagasse et située dans l'Etat de São Paulo. Elle affiche une puissance installée de 60 MW.    Copyright (c) 2014 CercleFinance.com. Tous droits réservés.
</article>
<article titre="Albioma: bond de 33% du bénéfice net 2013, s'implante au Brésil" date="2014 03 5" journal="AFP Infos Economiques">
PARIS, 05 mars 2014 (AFP) - - Le producteur français d'électricité Albioma (ex-Séchilienne Sidec), surtout actif en Outre-mer, a vu son bénéfice net bondir de 33% en 2013 et vient de procéder à sa première acquisition au Brésil, a-t-il annoncé mercredi dans un communiqué.   "Après un an de travail intensif, nous sommes très heureux de réaliser notre premier investissement au Brésil, priorité du groupe à l'international", a commenté Jacques Pétry, PDG d'Albioma, cité dans le communiqué diffusé à l'occasion de la publication des résultats annuels de la société. Elle vient d'acquérir pour 43 millions d'euros Rio Pardo Termoeletrica, une centrale de cogénération située dans la région de Sao Paulo qui fonctionne avec de la bagasse (résidu de canne à sucre).   Cette opération, financée à parité en dette locale et en fonds propres, devrait avoir un impact positif sur la rentabilité d'Albioma "dès l'année 2014". La finalisation est encore soumise à la levée de certaines conditions.   Cette centrale mise en service en 2009 fonctionne en partenariat avec une sucrerie, et devrait permettre d'injecter à terme dans le réseau 160 GWh d'électricité par an.    La société a aussi indiqué mener des "discussions actives" avec des sucriers brésiliens pour reprendre des usines existantes ou en construire de nouvelles afin de "réaliser un nouveau projet tous les 12 à 18 mois".   En 2013, Albioma a enregistré une hausse de 2% de son chiffre d'affaires à 364,3 millions d'euros (hors effet des matières premières). Son résultat brut d'exploitation (ebitda) et son résultat net part du groupe (RNPG) ont progressé respectivement de 11% à 133,6 millions et de 33% à 42,6 millions.   Le bénéfice net a notamment profité d'une plus-value de cession dans l'éolien début 2013 (5,6 millions). La dette nette s'est fortement résorbée de 114 millions pour s'établir à 414 millions d'euros, et la trésorerie consolidée atteignait 110 millions fin 2013.   "En s'appuyant sur ses excellents résultats et sa solidité financière, le groupe va pouvoir financer sa croissance robuste et rentable", a relevé M. Pétry.   Le groupe a confirmé ses objectifs: investir un milliard d'euros entre 2013 et 2023, et réaliser en 2016 un ebitda de 160 à 163 millions et un RNPG de 40 à 42 millions hors exceptionnels (32,3 millions en 2013).   Pour tenir compte de nouvelles normes environnementales pour les producteurs d'életricité, le groupe prévoit d'investir 200 millions d'euros d'ici 2020 dans la modernisation d'infrastructures existantes en Outre-mer.   Albioma prévoit de concrétiser "trois à quatre nouveaux projets en 2014", avec un investissement total de 20 millions d'euros.   elm/cb/mml
</article>
<article titre="ZOOM Albioma capitalise sur ses résultats et son acquisition au Brésil" date="2014 03 5" journal="AFP Infos Economiques">
PARIS, 05 mars 2014 (AFP) - - L'action d'Albioma bondissait à la Bourse de Paris mercredi matin, grâce à la progression de 33% du bénéfice net 2013 du producteur d'électricité et à sa première acquisition au Brésil.   A 10H46 (09H46 GMT), le titre prenait 5,85% à 19,53 euros, dans un marché en baisse de 0,19%.   Albioma (ex-Séchilienne Sidec), surtout actif en Outre-mer, a enregistré une hausse de 2% de son chiffre d'affaires (hors effet des matières premières) en 2013. Son résultat brut d'exploitation (ebitda) et son résultat net part du groupe (RNPG) ont progressé respectivement de 11% et de 33%. Les analystes du courtier Gilbert Dupont aplaudissent ainsi un "Editda 2013 supérieur à notre attente", et une "baisse plus prononcée qu'attendu de la dette".   Albioma a également annoncé avoir acquis Rio Pardo Termoeletrica, une centrale de cogénération située dans la région de Sao Paulo qui fonctionne avec de la bagasse (résidu de canne à sucre), pour 43 millions d'euros.   Cette opération, financée à parité en dette locale et en fonds propres, devrait avoir un impact positif sur la rentabilité d'Albioma "dès l'année 2014". La finalisation est encore soumise à la levée de certaines conditions.   Là encore, l'équipe de Gilbert Dupont souligne "des points positifs : un prix d'acquisition a priori raisonnable (si le prix indiqué porte sur 100% de la centrale), une unité récente (mise en service en 2009), la similitude de cette centrale avec les unités du groupe ce qui devrait en faciliter l'exploitation et l'optimisation".   "Le management confirme par ailleurs vouloir poursuivre son développement sur le marché brésilien sur un rythme d'un projet tous les 12/18 mois", ajoute le courtier.   Albioma a confirmé ses objectifs: investir un milliard d'euros entre 2013 et 2023, et réaliser en 2016 un ebitda de 160 à 163 millions et un RNPG de 40 à 42 millions hors exceptionnels.   rfo/jbo/pre
</article>
</corpus>"""

In [27]:
t = s.split('\n')
ss = t[305: 307]